In [ ]:
#@title Codes for JAFTA internally. { display-mode: "form" }


<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/thanhgis/PeatLands/blob/main/runme.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/thanhgis/PeatLands/blob/main/runme.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9oG4_w-dASOpiq1Rj6F0G6verxuzPk3Lzh3h-2Ujwwg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g48ONpJ3JlX6tNynCi1LUg0aMe6gPYD-uAMrbvZJko3XXT9-M5PZcM

Successfully saved authorization token.


## Add Librares

Convert neccessary libraries: 

In [3]:
import numpy as np
import datetime as dt
import math
import time
import os

# Library 1: 
class GEE_extent(object):
	def __init__(self, minlon, minlat, maxlon, maxlat, workdir, sampling=20):
		"""Return a new GEE extent object"""
		ee.Reset()
		ee.Initialize()

		# construct roi
		roi = ee.Geometry.Polygon([[minlon, minlat], [minlon, maxlat], [maxlon, maxlat], [maxlon, minlat], [minlon, minlat]])

		self.roi = roi
		self.MINLON = minlon
		self.MINLAT = minlat
		self.sampling = sampling
		self.workdir = workdir

		# Placeholders
		self.S1_SIG0_VV_db = None
		self.S1_G0VOL_VV_db = None
		self.S1_G0SURF_VV_db = None
		self.S1_ANGLE = None
		self.S1_LIA = None
		self.K1VV = None
		self.K2VV = None
		self.K3VV = None
		self.K4VV = None
		self.K1G0VV_V = None
		self.K2G0VV_V = None
		self.K3G0VV_V = None
		self.K4G0VV_V = None
		self.K1G0VV_S = None
		self.K2G0VV_S = None
		self.K3G0VV_S = None
		self.K4G0VV_S = None
		self.S1MEAN_VV = None
		self.S1STD_VV = None
		self.S1G0VOLMEAN_VV = None
		self.S1G0VOLSTD_VV = None
		self.S1G0SURFMEAN_VV = None
		self.S1G0SURFSTD_VV = None
		self.S1_DATE = None
		self.S1_SIG0_VH_db = None
		self.S1_G0VOL_VH_db = None
		self.S1_G0SURF_VH_db = None
		self.K1VH = None
		self.K2VH = None
		self.K3VH = None
		self.K4VH = None
		self.K1G0VH_V = None
		self.K2G0VH_V = None
		self.K3G0VH_V = None
		self.K4G0VH_V = None
		self.K1G0VH_S = None
		self.K2G0VH_S = None
		self.K3G0VH_S = None
		self.K4G0VH_S = None
		self.S1MEAN_VH = None
		self.S1STD_VH = None
		self.S1G0VOLMEAN_VH = None
		self.S1G0VOLSTD_VH = None
		self.S1G0SURFMEAN_VH = None
		self.S1G0SURFSTD_VH = None
		self.ESTIMATED_SM = None
		self.GLDAS_IMG = None
		self.GLDAS_MEAN = None
		self.LAND_COVER = None
		self.TERRAIN = None
		self.L8_IMG = None
		self.L8_MEAN = None
		self.L8_MASK = None
		self.TREE_COVER = None
		self.LAST_GLDAS = dt.datetime.today()  # temporary
		self.LC_ID = None
		self.FOREST_TYPE = None
		self.BARE_COVER = None
		self.CROPS_COVER = None
		self.GRASS_COVER = None
		self.MOSS_COVER = None
		self.SHRUB_COVER = None
		self.TREE_COVER = None
		self.URBAN_COVER = None
		self.WATERP_COVER = None
		self.WATERS_COVER = None
		self.SAND = None
		self.CLAY = None
		self.BULK = None
		self.EVI_IMG = None
		self.EVI_MEAN = None
		self.OVERWRITE = None

	def _multitemporalDespeckle(self, images, radius, units, opt_timeWindow=None):
		"""Function for multi-temporal despeckling"""

		def mapMeanSpace(i):
			reducer = ee.Reducer.mean()
			kernel = ee.Kernel.square(radius, units)
			mean = i.reduceNeighborhood(reducer, kernel).rename(bandNamesMean)
			ratio = i.divide(mean).rename(bandNamesRatio)
			return (i.addBands(mean).addBands(ratio))

		if opt_timeWindow == None:
			timeWindow = dict(before=-3, after=3, units='month')
		else:
			timeWindow = opt_timeWindow

		bandNames = ee.Image(images.first()).bandNames()
		bandNamesMean = bandNames.map(lambda b: ee.String(b).cat('_mean'))
		bandNamesRatio = bandNames.map(lambda b: ee.String(b).cat('_ratio'))

		# compute spatial average for all images
		meanSpace = images.map(mapMeanSpace)

		# computes a multi-temporal despeckle function for a single image

		def multitemporalDespeckleSingle(image):
			t = image.date()
			fro = t.advance(ee.Number(timeWindow['before']), timeWindow['units'])
			to = t.advance(ee.Number(timeWindow['after']), timeWindow['units'])
			meanSpace2 = ee.ImageCollection(meanSpace).select(bandNamesRatio).filterDate(fro, to).filter(ee.Filter.eq('relativeOrbitNumber_start', image.get('relativeOrbitNumber_start')))
			b = image.select(bandNamesMean)
			return (b.multiply(meanSpace2.sum()).divide(meanSpace2.count()).rename(bandNames)).set('system:time_start', image.get('system:time_start'))
		return meanSpace.map(multitemporalDespeckleSingle)

	def _slope_correction(self, collection, elevation, model, buffer=0):
		"""This function applies the slope correction on a collection of Sentinel-1 data

		   :param collection: ee.Collection of Sentinel-1
		   :param elevation: ee.Image of DEM
		   :param model: model to be applied (volume/surface)
		   :param buffer: buffer in meters for layover/shadow amsk

			:returns: ee.Image
		"""

		def _volumetric_model_SCF(theta_iRad, alpha_rRad):
			'''Code for calculation of volumetric model SCF

			:param theta_iRad: ee.Image of incidence angle in radians
			:param alpha_rRad: ee.Image of slope steepness in range

			:returns: ee.Image
			'''
			# create a 90 degree image in radians
			ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
			# model
			nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
			denominator = (ninetyRad.subtract(theta_iRad)).tan()
			return nominator.divide(denominator)

		def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
			'''Code for calculation of direct model SCF
			:param theta_iRad: ee.Image of incidence angle in radians
			:param alpha_rRad: ee.Image of slope steepness in range
			:param alpha_azRad: ee.Image of slope steepness in azimuth
			:returns: ee.Image
			'''
			# create a 90 degree image in radians
			ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
			# model
			nominator = (ninetyRad.subtract(theta_iRad)).cos()
			denominator = (alpha_azRad.cos().multiply((ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))
			return nominator.divide(denominator)

		def _erode(image, distance):
			'''Buffer function for raster
			:param image: ee.Image that shoudl be buffered
			:param distance: distance of buffer in meters
			:returns: ee.Image
			'''
			d = (image.Not().unmask(1).fastDistanceTransform(30).sqrt().multiply(ee.Image.pixelArea().sqrt()))
			return image.updateMask(d.gt(distance))

		def _masking(alpha_rRad, theta_iRad, buffer):
			'''Masking of layover and shadow
			:param alpha_rRad: ee.Image of slope steepness in range
			:param theta_iRad: ee.Image of incidence angle in radians
			:param buffer: buffer in meters

			:returns: ee.Image
			'''
			# layover, where slope > radar viewing angle
			layover = alpha_rRad.lt(theta_iRad).rename('layover')
			# shadow
			ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)
			shadow = alpha_rRad.gt(ee.Image.constant(-1).multiply(ninetyRad.subtract(theta_iRad))).rename('shadow')

			# add buffer to layover and shadow
			if buffer > 0:
				layover = _erode(layover, buffer)
				shadow = _erode(shadow, buffer)

				# combine layover and shadow
			no_data_mask = layover.And(shadow).rename('no_data_mask')

			return layover.addBands(shadow).addBands(no_data_mask)

		def _correct(image):
			'''This function applies the slope correction and adds layover and shadow masks

			'''

			# get the image geometry and projection
			geom = image.geometry()
			proj = image.select(1).projection()

			# calculate the look direction
			heading = (ee.Terrain.aspect(image.select('angle'))
					   .reduceRegion(ee.Reducer.mean(), geom, 1000, tileScale=4)
					   .get('aspect'))

			# Sigma0 to Power of input image
			sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))

			# the numbering follows the article chapters
			# 2.1.1 Radar geometry
			theta_iRad = image.select('angle').multiply(np.pi / 180)
			phi_iRad = ee.Image.constant(heading).multiply(np.pi / 180)

			# 2.1.2 Terrain geometry
			alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(np.pi / 180).setDefaultProjection(proj).clip(geom)
			phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(np.pi / 180).setDefaultProjection(proj).clip(geom)

			# we get the height, for export
			height = elevation.setDefaultProjection(proj).clip(geom)

			# 2.1.3 Model geometry
			# reduce to 3 angle
			phi_rRad = phi_iRad.subtract(phi_sRad)

			# slope steepness in range (eq. 2)
			alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

			# slope steepness in azimuth (eq 3)
			alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

			# local incidence angle (eq. 4)
			theta_liaRad = (alpha_azRad.cos().multiply((theta_iRad.subtract(alpha_rRad)).cos())).acos()
			theta_liaDeg = theta_liaRad.multiply(180 / np.pi)

			# 2.2
			# Gamma_nought
			gamma0 = sigma0Pow.divide(theta_iRad.cos())
			gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(['VV', 'VH'], ['VV_gamma0', 'VH_gamma0'])
			ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(gamma0dB.select('VH_gamma0')).rename('ratio_gamma0'))

			if model == 'volume':
				scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)

			if model == 'surface':
				scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)

			# apply model for Gamm0_f
			gamma0_flat = gamma0.divide(scf)
			gamma0_flatDB = (ee.Image.constant(10).multiply(gamma0_flat.log10()).select(['VV', 'VH'], ['VV_gamma0flat', 'VH_gamma0flat']))
			masks = _masking(alpha_rRad, theta_iRad, buffer)

			# calculate the ratio for RGB vis
			ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))
			if model == 'surface':
				gamma0_flatDB = gamma0_flatDB.rename(['VV_gamma0surf', 'VH_gamma0surf'])

			if model == 'volume':
				gamma0_flatDB = gamma0_flatDB.rename(['VV_gamma0vol', 'VH_gamma0vol'])

			return (image.rename(['VV_sigma0', 'VH_sigma0', 'incAngle'])
					.addBands(gamma0dB)
					.addBands(ratio_gamma)
					.addBands(gamma0_flatDB)
					.addBands(ratio_flat)
					.addBands(alpha_rRad.rename('alpha_rRad'))
					.addBands(alpha_azRad.rename('alpha_azRad'))
					.addBands(phi_sRad.rename('aspect'))
					.addBands(alpha_sRad.rename('slope'))
					.addBands(theta_iRad.rename('theta_iRad'))
					.addBands(theta_liaRad.rename('theta_liaRad'))
					.addBands(masks)
					.addBands(height.rename('elevation'))
					)

			# run and return correction

		return collection.map(_correct, opt_dropNulls=True)

	def init_SM_retrieval(self, year, month, day, hour=12, minute=0, track=None, overwrite=False):
		# initiate all datasets for the retrieval of soil moisture
		self.get_copernicus_lc()
		self.get_S1_tmp_collection(year, month, day, hour, minute, mask_copernicuslc=True, trackflt=None, ascending=False)
		self.match_evi()
		self.match_l8()
		self.get_l8()
		self.get_modis_evi()
		self.get_S1(tempfilter=False,mask_snow_frozen_GLDAS=False)
		self.get_sand_content()
		self.get_bulk_density()
		self.OVERWRITE = overwrite

	def get_S1(self, tempfilter=False, tempfilter_radius=7, mask_snow_frozen_GLDAS=False):

		def tolin(image):
			tmp = ee.Image(image)
			out = ee.Image(10).pow(tmp.select('VV_gamma0vol').divide(10))
			out = out.select(['constant'], ['VV_gamma0vol'])
			return out.set('system:time_start', tmp.get('system:time_start'))

		def todb(image):
			tmp = ee.Image(image)
			return ee.Image(10).multiply(tmp.log10()).set('system:time_start', tmp.get('system:time_start'))

		def create_gldas_snow_frozen_mask(image):
			s1 = ee.Image(image.get('primary'))
			gldas = ee.Image(image.get('secondary'))
			mask = ee.Image(gldas.expression("(b('SWE_inst') < 3) && (b('SoilTMP0_10cm_inst') > 275) ? 1 : 0"))
			return s1.updateMask(mask)

		def mask_evi(image):
			s1 = ee.Image(image.get('primary'))
			evi = ee.Image(image.get('secondary'))
			return s1.updateMask(evi.select(0).mask())

		gee_s1_filtered = self.S1_reference_stack
		if mask_snow_frozen_GLDAS:
			gldas_filt = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')
			innjoin = ee.Join.inner()
			joined_s1_gldas = innjoin.apply(gee_s1_filtered, self.GLDAS_STACK, gldas_filt)
			gee_s1_filtered = ee.ImageCollection(joined_s1_gldas.map(create_gldas_snow_frozen_mask))

		# mask evi
		evi_filt = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')
		innjoin2 = ee.Join.inner()
		joined_s1_evi = innjoin2.apply(gee_s1_filtered, self.EVI_STACK, evi_filt)
		gee_s1_filtered = ee.ImageCollection(joined_s1_evi.map(mask_evi, opt_dropNulls=True))

		# filter
		def getddist(image):
			return image.set(
				'dateDist', ee.Number(image.get('system:time_start')).subtract(
					ee.Date(doi.strftime('%Y-%m-%dT%H:%M:%S')).millis()).abs()
			)

		# select pixels with the smalles time gap to doi and mosaic spatially
		doi = self.S1_DATE
		s1_selected = ee.Image(gee_s1_filtered.map(getddist).sort('dateDist').first())
		s1_g0vol = s1_selected.select(['VV_gamma0vol', 'VH_gamma0vol'])

		if tempfilter == True:
			# despeckle
			radius = tempfilter_radius
			units = 'pixels'
			gee_s1_linear = gee_s1_filtered.map(tolin)
			gee_s1_dspckld_vv = self._multitemporalDespeckle(gee_s1_linear.select('VV'), radius, units, {'before': -12, 'after': 12, 'units': 'month'})
			gee_s1_dspckld_vv = gee_s1_dspckld_vv.map(todb)
			gee_s1_fltrd_vv = gee_s1_dspckld_vv.filterDate(date_selected.strftime('%Y-%m-%d'), (date_selected + dt.timedelta(days=1)).strftime('%Y-%m-%d'))
			s1_sig0_vv = gee_s1_fltrd_vv.mosaic()
			s1_sig0 = s1_sig0_vv.select(['constant'], ['VV'])

		# extract information
		s1_g0vol_vv = s1_g0vol.select('VV_gamma0vol')
		# calculate statistical moments
		gee_s1_filtered = gee_s1_filtered.filterDate(str(doi.year) + '-01-01', str(doi.year) + '-12-31').select('VV_gamma0vol')
		gee_s1_lin = gee_s1_filtered.map(tolin)

		# check if median was alread computed
		tmpcoords = self.roi.getInfo()['coordinates']
		mean_asset_path = 's1med_' + str(abs(tmpcoords[0][0][0])) + \
						  '_' + str(abs(tmpcoords[0][0][1])) + '_' + \
						  str(abs(tmpcoords[0][2][0])) + \
						  '_' + str(abs(tmpcoords[0][2][1])) + \
						  '_' + str(self.sampling) + '_' + str(self.TRACK_NR) + '_' + str(doi.year)
		mean_asset_path = mean_asset_path.replace('.', '')
		mean_gvv_v = ee.Image('users/thanhgeevn/' + mean_asset_path)
		try:
			mean_gvv_v.getInfo()
			print('S1 median exists')
		except:
			# compute median
			mean_gvv_v = ee.Image(gee_s1_lin.select('VV_gamma0vol').reduce(ee.Reducer.median(), parallelScale=16))

			# export asset
			self.GEE_2_asset(raster=mean_gvv_v, name=mean_asset_path, timeout=False)
			mean_gvv_v = ee.Image('users/thanhgeevn/' + mean_asset_path)

		# export
		# self.S1_SIG0_VV_db = s1_sig0_vv
		self.S1_G0VOL_VV_db = s1_g0vol_vv
		self.S1G0VOLMEAN_VV = ee.Image(10).multiply(mean_gvv_v.log10()).copyProperties(mean_gvv_v)

	def estimate_SM_GBR_1step(self):
		import sys

		g0_v_vv = self.S1_G0VOL_VV_db
		dg0_v_vv = g0_v_vv.subtract(self.S1G0VOLMEAN_VV)
		crops = self.CROPS_COVER
		grass = self.GRASS_COVER
		moss = self.MOSS_COVER
		l8b4 = self.L8_IMG.select('B4')
		l8b4med = self.L8_MEAN.select('B4_median')
		l8b5 = self.L8_IMG.select('B5')
		l8b5med = self.L8_MEAN.select('B5_median')
		l8b11 = self.L8_IMG.select('B11')
		l8b11med = self.L8_MEAN.select('B11_median')
		ndvi = self.EVI_IMG
		ndvi_med = self.EVI_MEAN
		bulk = self.BULK
		sand = self.SAND

		input_image1 = ee.Image([dg0_v_vv.toFloat(),
								 crops.toFloat(),
								 grass.toFloat(),
								 moss.toFloat(),
								 l8b4.toFloat(),
								 l8b4med.toFloat(),
								 l8b5.toFloat(),
								 l8b5med.toFloat(),
								 l8b11.toFloat(),
								 l8b11med.toFloat(),
								 ndvi.toFloat(),
								 ndvi_med.toFloat(),
								 bulk.toFloat(),
								 sand.toFloat()])

		input_image1 = input_image1.rename(['dg0_v_vv', 'crops', 'grass', 'moss', 'l8b4', 'l8b4_med', 'l8b5',
											'l8b5_med', 'l8b11', 'l8b11_med', 'ndvi', 'ndvi_med', 'bulk', 'sand'])

		ipt_img_mask1 = input_image1.mask().reduce(ee.Reducer.allNonZero())

		combined_mask = ipt_img_mask1

		input_image1 = input_image1.updateMask(ee.Image(combined_mask))

		sys.setrecursionlimit(5000)
		estimated_smc = tree(input_image1)
		estimated_smc = estimated_smc.updateMask(combined_mask)

		# mask negative values
		estimated_smc = estimated_smc.updateMask(estimated_smc.gt(0))

		## scaling
		estimated_smc = estimated_smc.multiply(10).round().int8()

		self.ESTIMATED_SM = estimated_smc.rename(['ESTIMATED_SM']).set({'system:time_start':
																	   ee.Date(self.S1_DATE.strftime('%Y-%m-%dT%H:%M:%S')).millis(),
																	   's1tracknr': int(self.TRACK_NR)})
		self.ESTIMATED_MEAN_SM = None

	def get_S1_dates(self, tracknr=None, dualpol=True, ascending=True, start='2014-01-01', stop='2021-01-01'):
		# load S1 data
		gee_s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

		# ASCENDING acquisitions
		gee_s1_filtered = gee_s1_collection.filter(ee.Filter.eq('instrumentMode', 'IW')) \
			.filterBounds(self.roi) \
			.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
			.filterDate(start, opt_end=stop)

		if ascending == True:
			# Consider only image from ascending orbits
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

		if dualpol == True:
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

		if tracknr is not None:
			if (type(tracknr)) == list:
				gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.inList('relativeOrbitNumber_start', tracknr))
			else:
				gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.eq('relativeOrbitNumber_start', tracknr))

		# create a list of availalbel dates
		tmp = gee_s1_filtered.getInfo()
		tmp_ids = [x['properties']['system:index'] for x in tmp['features']]

		dates = np.array(
			[dt.datetime(int(x[17:21]), int(x[21:23]), int(x[23:25]), hour=int(x[26:28]), minute=int(x[28:30])) for x in
			 tmp_ids])
		orbits = np.array([x['properties']['relativeOrbitNumber_start'] for x in tmp['features']])

		return dates, orbits

	def get_S1_tmp_collection(self, year, month, day, hour, minute, trackflt=None, dualpol=True, ascending=False, mask_copernicuslc=False):
		def mosaicByDate(imcol):
			def xf(d):
				d = ee.Date(d)
				im = imcol.filterDate(d, d.advance(1, "day")).mosaic()
				return im.set("system:time_start", d.millis(), "system:id", d.format("YYYY-MM-dd"))

			imlist = imcol.toList(imcol.size())
			unique_dates = imlist.map(lambda x: ee.Image(x).date().format("YYYY-MM-dd")).distinct()
			mosaic_imlist = unique_dates.map(xf)
			return ee.ImageCollection(mosaic_imlist)

		def mask_lc_copernicus(image):
			copernicus_collection = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V/Global')
			copernicus_image = ee.Image(copernicus_collection.toList(1000).get(0)).select('discrete_classification').setDefaultProjection(image.select(1).projection()).clip(image.geometry())

			valLClist = [20, 30, 40, 60, 125, 126, 121, 122, 123, 124]

			lcmask = copernicus_image.eq(valLClist[0]).bitwiseOr(copernicus_image.eq(valLClist[1])) \
				.bitwiseOr(copernicus_image.eq(valLClist[2])) \
				.bitwiseOr(copernicus_image.eq(valLClist[3])) \
				.bitwiseOr(copernicus_image.eq(valLClist[4])) \
				.bitwiseOr(copernicus_image.eq(valLClist[5])) \
				.bitwiseOr(copernicus_image.eq(valLClist[6])) \
				.bitwiseOr(copernicus_image.eq(valLClist[7])) \
				.bitwiseOr(copernicus_image.eq(valLClist[8])) \
				.bitwiseOr(copernicus_image.eq(valLClist[9]))
			maskimg = ee.Image.cat([lcmask,
									lcmask,
									image.select('angle').mask()])

			#tmp = ee.Image(image)
			#tmp = tmp.updateMask(lcmask)

			return image.updateMask(maskimg)

		gee_s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

		# Filter the image collection
		gee_s1_filtered = gee_s1_collection.filter(ee.Filter.eq('instrumentMode', 'IW')) \
			.filterBounds(self.roi).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \

		if ascending:
			# Consider only image from ascending orbits
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

		if dualpol:
			# Consider only dual-pol scenes
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

		if trackflt is not None:
			# Specify track
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.eq('relativeOrbitNumber_start', trackflt))

		if mask_copernicuslc:
			gee_s1_filtered = gee_s1_filtered.map(mask_lc_copernicus)


		# filter
		def getddist(image):
			return image.set(
				'dateDist', ee.Number(image.get('system:time_start')).subtract(
				ee.Date(doi).millis()).abs())

		# select pixels with the smalles time gap to doi and mosaic spatially
		doi = dt.datetime(year=year, month=month, day=day, hour=hour, minute=minute)
		print (doi)
		s1_selected = ee.Image(gee_s1_filtered.map(getddist).sort('dateDist').first())
		
		self.S1_DATE = dt.datetime.strptime(s1_selected.date().format('yyyy-MM-dd HH:mm:ss').getInfo(), '%Y-%m-%d %H:%M:%S')
		print ('Loi o day: ', self.S1_DATE)

		# get the track number
		#s1_sig0_info = s1_selected.getInfo()
		#track_nr = s1_sig0_info['properties']['relativeOrbitNumber_start']
		track_nr = s1_selected.get('relativeOrbitNumber_start').getInfo()
		self.TRACK_NR = track_nr
		self.ORBIT = s1_selected.get('orbitProperties_pass').getInfo()

		# only uses images of the same track
		gee_s1_filtered = gee_s1_filtered.filterMetadata('relativeOrbitNumber_start', 'equals', track_nr)

		# calculate g0
		# paths to dem
		dem = 'USGS/SRTMGL1_003'

		# list of models
		model = 'volume'
		gee_s1_fltd_vol = self._slope_correction(gee_s1_filtered,
												 ee.Image(dem),
												 model)

		gee_s1_filtered = gee_s1_fltd_vol

		# apply no data mask
		def mask_no_data(image):
			return image.updateMask(image.select('no_data_mask'))

		gee_s1_filtered = gee_s1_filtered.map(mask_no_data)

		# self.S1_reference_stack = mosaicByDate(gee_s1_filtered)
		self.S1_reference_stack = mosaicByDate(gee_s1_filtered)

	def get_available_S1_tracks(self, dualpol=True):
		# load S1 data
		gee_s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD')

		# ASCENDING acquisitions
		gee_s1_filtered = gee_s1_collection.filter(ee.Filter.eq('instrumentMode', 'IW')) \
			.filterBounds(self.roi) \
			.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
			.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
		#		   .filter(ee.Filter.eq('platform_number', 'A')) \

		if dualpol == True:
			gee_s1_filtered = gee_s1_filtered.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))

		# create a list of availalbel dates
		tmp = gee_s1_filtered.getInfo()
		tmp_tracks = [x['properties']['relativeOrbitNumber_start'] for x in tmp['features']]
		tracks = np.unique(tmp_tracks)

		return tracks

	def check_gldas_availability(self, year, month, day):
		gldas_test = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
			.select('SoilMoi0_10cm_inst')
		last_gldas = dt.datetime.strptime(gldas_test.aggregate_max('system:index').getInfo(),
										  "A%Y%m%d_%H%M")
		self.LAST_GLDAS = last_gldas
		doi = dt.date(year=year, month=month, day=day)
		return last_gldas.date() > doi

	def get_gldas(self, date=None):
		# get GLDAS, date can be passed as a string or copied from the extracted S1 scene
		if date is None:
			doi = ee.Date(self.S1_DATE.strftime(format='%Y-%m-%d'))

		# check if Sentinel-1 was retrieved and date is available
		if hasattr(self, 'S1_DATE'):
			gldas_mean = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
				.select('SoilMoi0_10cm_inst') \
				.filterDate('2014-10-01', dt.datetime.today().strftime('%Y-%m-%d')) \
				.filter(ee.Filter.calendarRange(self.S1_DATE.hour, self.S1_DATE.hour + 3, field='hour')) \
				.reduce(ee.Reducer.median())
		else:
			gldas_mean = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
				.select('SoilMoi0_10cm_inst') \
				.filterDate('2014-10-01', dt.datetime.today().strftime('%Y-%m-%d')) \
				.reduce(ee.Reducer.median())

		gldas_mean = ee.Image(gldas_mean).resample().clip(self.roi)

		gldas_img = gldas_mean

		try:
			self.GLDAS_IMG = gldas_img
			self.GLDAS_MEAN = gldas_mean
		except:
			return None

	def match_l8(self):

		def mask(image):
			# clouds
			def getQABits(image, start, end, newName):
				pattern = 0
				for i in range(start, end + 1):
					pattern = pattern + int(math.pow(2, i))

				return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

			def cloud_shadows(image):
				QA = image.select('pixel_qa')
				return getQABits(QA, 3, 3, 'Cloud_shadows').eq(0)

			def clouds(image):
				QA = image.select('pixel_qa')
				return getQABits(QA, 5, 5, 'Cloud').eq(0)

			# frozen soil / snow
			def frzn(image):
				doi = ee.Date(image.get('system:time_start'))
				snow = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
					.select('SWE_inst') \
					.filterDate(doi, doi.advance(3, 'hour'))

				snow_img = ee.Image(snow.first()).resample().clip(self.roi)

				snow_mask = snow_img.expression('(b(0) < 3) ? 1 : 0')

				fs = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
					.select('SoilTMP0_10cm_inst') \
					.filterDate(doi, doi.advance(3, 'hour'))

				fs_img = ee.Image(fs.first()).resample().clip(self.roi)

				fs_mask = fs_img.expression('(b(0) > 275) ? 1 : 0')

				return snow_mask.And(fs_mask)

			image = image.updateMask(cloud_shadows(image))
			image = image.updateMask(clouds(image))
			# image = image.updateMask(frzn(image))

			# # radiometric saturation
			# image = image.updateMask(image.select('radsat_qa').eq(2))
			return image.clip(self.roi)

		gee_l8_collection_all = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

		# apply landsat mask
		gee_l8_collection_all = gee_l8_collection_all.map(mask).select(['B4', 'B5', 'B11'])

		def date_mosaic(image):

			def addDate(image2):
				date_img = ee.Image(image2.date().difference(imdate, 'second')).abs().float().rename(
					['Ddate']).multiply(-1)
				return image2.addBands(date_img)

			imdate = image.date()

			gee_l8_date = gee_l8_collection_all.filterDate(imdate.advance(-40, 'day'),
														   imdate.advance(40, 'day'))
			gee_l8_date = gee_l8_date.map(addDate)
			gee_l8_date = gee_l8_date.qualityMosaic('Ddate').float()

			return gee_l8_date.set('system:time_start', imdate.millis())

		self.L8_STACK = self.S1_reference_stack.map(date_mosaic)
		# self.L8_STACK = gee_l8_collection_all

	def match_evi(self):

		def mask(image):
			# mask image
			immask = image.select('SummaryQA').eq(ee.Image(0))
			evimask = image.select('EVI').lte(5000)
			image = image.updateMask(immask).updateMask(evimask)

			return image.clip(self.roi)

		# load collection
		evi_collection = ee.ImageCollection('MODIS/006/MOD13Q1').map(mask).select('EVI')

		def date_mosaic(image):
			def addDate(image2):
				date_img = ee.Image(image2.date().difference(imdate, 'second')).abs().float().rename(
					['Ddate']).multiply(-1)
				return image2.addBands(date_img)

			imdate = image.date()

			gee_evi_date = evi_collection.filterDate(imdate.advance(-40, 'day'),
													 imdate.advance(40, 'day'))
			gee_evi_date = gee_evi_date.map(addDate)
			gee_evi_date = gee_evi_date.qualityMosaic('Ddate').float()

			return gee_evi_date.set('system:time_start', imdate.millis())

		self.EVI_STACK = self.S1_reference_stack.map(date_mosaic)

	def match_gldas(self):

		def mask(image):
			return image.clip(self.roi)

		# load collection
		gl_collection = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H") \
			.select(['SoilTMP0_10cm_inst', 'SWE_inst']) \
			.filterDate('2014-01-01', dt.datetime.today().strftime('%Y-%m-%d')).map(mask)

		def date_map(image):

			imdate = image.date()

			# filter
			def getddist(image2):
				return image2.set(
					'dateDist', ee.Number(image2.get('system:time_start')).subtract(
						imdate.millis()).abs()
				)

			# select the image with the smalles time gap
			gl_collection_date = gl_collection.filterDate(imdate.advance(-5, 'day'),
														  imdate.advance(5, 'day'))
			gee_gl_date = ee.Image(gl_collection_date.map(getddist).sort('dateDist').first())

			return gee_gl_date.set('system:time_start', imdate.millis())

		self.GLDAS_STACK = self.S1_reference_stack.map(date_map)

	def get_l8(self, date=None):
		
		def mask(image):
			# clouds
			def getQABits(image, start, end, newName):
				pattern = 0
				for i in range(start, end + 1):
					pattern = pattern + int(math.pow(2, i))
				return image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)

			def cloud_shadows(image):
				QA = image.select('pixel_qa')
				return getQABits(QA, 3, 3, 'Cloud_shadows').eq(0)

			def clouds(image):
				QA = image.select('pixel_qa')
				return getQABits(QA, 5, 5, 'Cloud').eq(0)

			# frozen soil / snow
			def frzn(image):
				doi = ee.Date(image.get('system:time_start'))
				snow = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H").select('SWE_inst').filterDate(doi, doi.advance(3, 'hour'))
				snow_img = ee.Image(snow.first()).resample().clip(self.roi)
				snow_mask = snow_img.expression('(b(0) < 3) ? 1 : 0')
				fs = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H").select('SoilTMP0_10cm_inst').filterDate(doi, doi.advance(3, 'hour'))
				fs_img = ee.Image(fs.first()).resample().clip(self.roi)
				fs_mask = fs_img.expression('(b(0) > 275) ? 1 : 0')
				return snow_mask.And(fs_mask)
			image = image.updateMask(cloud_shadows(image))
			image = image.updateMask(clouds(image))
			# image = image.updateMask(frzn(image))

			# # radiometric saturation
			# image = image.updateMask(image.select('radsat_qa').eq(2))
			return image.clip(self.roi)

		gee_l8_collection_all = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

		# apply landsat mask
		gee_l8_collection = gee_l8_collection_all.map(mask).select(['B4', 'B5', 'B11'])


		if date is None:
			doi = self.S1_DATE
		else:
			doi = date

		# check if median was alread computed
		tmpcoords = self.roi.getInfo()['coordinates']
		mean_asset_path = 'l8med_' + str(abs(tmpcoords[0][0][0])) + \
						  '_' + str(abs(tmpcoords[0][0][1])) + '_' + \
						  str(abs(tmpcoords[0][2][0])) + \
						  '_' + str(abs(tmpcoords[0][2][1])) + \
						  '_' + str(self.sampling) + '_' + str(doi.year)
		mean_asset_path = mean_asset_path.replace('.', '')
		gee_l8_mean = ee.Image('users/thanhgeevn/' + mean_asset_path)

		try:
			gee_l8_mean.getInfo()
			print('L8 median exists')
		except:
			# compute median
			gee_l8_mean = gee_l8_collection.filterDate(str(doi.year) + '-01-01', str(doi.year) + '-12-31') \
				.reduce(ee.Reducer.median(), parallelScale=16)

			#export asset
			self.GEE_2_asset(raster=gee_l8_mean, name=mean_asset_path, timeout=False)
			gee_l8_mean = ee.Image(mean_asset_path)

		def addDate(image2):
			date_img = ee.Image(image2.date().difference(doi.strftime('%Y-%m-%dT%H:%M:%S'), 'second')).abs().float().rename(
				['Ddate']).multiply(-1)
			return image2.addBands(date_img)

		# create mosaic for the doi
		gee_l8_date = gee_l8_collection_all.filterDate((doi - dt.timedelta(days=40)).strftime('%Y-%m-%d'),
													   (doi + dt.timedelta(days=40)).strftime('%Y-%m-%d'))
		gee_l8_date = gee_l8_date.map(addDate)
		gee_l8_date = gee_l8_date.qualityMosaic('Ddate').float()

		gee_l8_date.set('system:time_start', ee.Date(doi.strftime('%Y-%m-%dT%H:%M:%S')).millis())

		#outimg = gee_l8_mosaic.clip(self.roi)

		try:
			self.L8_IMG = gee_l8_date
			self.L8_MEAN = gee_l8_mean
			self.L8_DDATE = gee_l8_date.select('Ddate').clip(self.roi).multiply(-1)
			self.L8_MASK = self.L8_IMG.mask().reduce(ee.Reducer.allNonZero(), parallelScale=8)
		except:
			return None

	def get_terrain(self):
		# get SRTM data
		elev = ee.Image("CGIAR/SRTM90_V4").select('elevation').clip(self.roi).resample()
		aspe = ee.Terrain.aspect(ee.Image("CGIAR/SRTM90_V4")).select('aspect').clip(self.roi).resample()
		slop = ee.Terrain.slope(ee.Image("CGIAR/SRTM90_V4")).select('slope').clip(self.roi).resample()
		self.TERRAIN = (elev, aspe, slop)

	def get_modis_evi(self, date=None):

		def create_gldas_snow_frozen_mask(image):
			evi = ee.Image(image.get('primary'))
			gldas = ee.Image(image.get('secondary'))

			mask = ee.Image(gldas.expression("(b('SWE_inst') < 3) && (b('SoilTMP0_10cm_inst') > 275) ? 1 : 0"))

			return evi.updateMask(mask)

		def mask(image):
			# mask image
			immask = image.select('SummaryQA').eq(ee.Image(0))
			evimask = image.select('EVI').lte(5000)
			image = image.updateMask(immask).updateMask(evimask)

			return image.clip(self.roi)

		# load collection
		evi_collection = ee.ImageCollection('MODIS/006/MOD13Q1').map(mask).select('EVI')

		if date is None:
			doi = self.S1_DATE
		tmpcoords = self.roi.getInfo()['coordinates']
		mean_asset_path = 'evimed_' + str(abs(tmpcoords[0][0][0])) + \
						  '_' + str(abs(tmpcoords[0][0][1])) + '_' + \
						  str(abs(tmpcoords[0][2][0])) + \
						  '_' + str(abs(tmpcoords[0][2][1])) + \
						  '_' + str(self.sampling) + '_' + str(doi.year)
		mean_asset_path = mean_asset_path.replace('.', '')
		evi_mean = ee.Image('users/thanhgeevn/' + mean_asset_path)
		try:
			evi_mean.getInfo()
			print('EVI median exists')
		except:
			# compute avg
			evi_mean = evi_collection.filterDate(str(doi.year) + '-01-01', str(doi.year) + '-12-31') \
				.reduce(ee.Reducer.median(), parallelScale=16)

			# export asset
			self.GEE_2_asset(raster=evi_mean, name=mean_asset_path, timeout=False)
			evi_mean = ee.Image('users/thanhgeevn/' + mean_asset_path)

		# fiter
		# filter
		def addDate(image2):
			date_img = ee.Image(image2.date().difference(doi.strftime('%Y-%m-%dT%H:%M:%S'), 'second')).abs().float().rename(
				['Ddate']).multiply(-1)
			return image2.addBands(date_img)

		gee_evi_date = evi_collection.filterDate((doi - dt.timedelta(days=30)).strftime('%Y-%m-%d'),
												 (doi + dt.timedelta(days=30)).strftime('%Y-%m-%d'))
		gee_evi_date = gee_evi_date.map(addDate)
		gee_evi_date = gee_evi_date.qualityMosaic('Ddate').float()

		gee_evi_date.set('system:time_start',  ee.Date(doi.strftime('%Y-%m-%dT%H:%M:%S')).millis())

		try:
			self.EVI_IMG = gee_evi_date.select('EVI').clip(self.roi)
			self.EVI_MEAN = evi_mean.select('EVI_median')
		except:
			return None

	def get_bulk_density(self):
		bulkimg = ee.Image("OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02").select('b0')
		self.BULK = bulkimg.resample()

	def get_clay_content(self):
		clayimg = ee.Image("OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
		self.CLAY = clayimg.resample()

	def get_sand_content(self):
		sandimg = ee.Image("OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02").select('b0')
		self.SAND = sandimg.resample()

	def get_copernicus_lc(self):

		copernicus_collection = ee.ImageCollection('COPERNICUS/Landcover/100m/Proba-V/Global')
		copernicus_image = ee.Image(copernicus_collection.toList(1000).get(0))
		self.LC_ID = copernicus_image.select('discrete_classification')
		self.FOREST_TYPE = copernicus_image.select('forest_type')
		self.BARE_COVER = copernicus_image.select('bare-coverfraction').resample()
		self.CROPS_COVER = copernicus_image.select('crops-coverfraction').resample()
		self.GRASS_COVER = copernicus_image.select('grass-coverfraction').resample()
		self.MOSS_COVER = copernicus_image.select('moss-coverfraction').resample()
		self.SHRUB_COVER = copernicus_image.select('shrub-coverfraction').resample()
		self.TREE_COVER = copernicus_image.select('tree-coverfraction').resample()
		self.URBAN_COVER = copernicus_image.select('urban-coverfraction').resample()
		self.WATERP_COVER = copernicus_image.select('water-permanent-coverfraction').resample()
		self.WATERS_COVER = copernicus_image.select('water-seasonal-coverfraction').resample()

	def GEE_2_asset(self, outdir=None, raster='ESTIMATED_SM', name='SM', timeout=True):
		# Export GEE rasters as asset - specify raster as string

		if isinstance(raster, str):
			geds = self.__getattribute__(raster)
		else:
			geds = raster

		if outdir is None:
			outdir = self.workdir

		try:
			file_avail = ee.Image('users/thanhgeevn/' + outdir + '/' + name)
			file_avail.getInfo()
			if self.OVERWRITE:
				os.system('earthengine rm users/thanhgeevn/' + outdir + '/' + name)
				raise NameError(name + ' will be overwritten')
			else:
				print(name + 'already exists')
				return
		except:
			file_exp = ee.batch.Export.image.toAsset(image=geds, description='fileexp' + name,
													 assetId='users/thanhgeevn/'+ name,
													 region=self.roi.getInfo()['coordinates'],
													 scale=self.sampling,
													 maxPixels=1000000000000)

			file_exp.start()

			start = time.time()

			while file_exp.active():
				time.sleep(2)
				if timeout and (time.time() - start) > 4800:
					success = 0
					break
			else:
				print('Export completed')

#from pysmm.GEE_wrappers import GEE_extent
import numpy as np
import os
import math
import datetime as dt

def get_map(minlon, minlat, maxlon, maxlat, outpath, sampling=50, year=None, month=None, day=None, tracknr=None, overwrite=False, start=None, stop=None):

	if year is not None:
		# initialise GEE retrieval dataset
		GEE_interface = GEE_extent(minlon, minlat, maxlon, maxlat, outpath, sampling=sampling)
		GEE_interface.init_SM_retrieval(year, month, day, track=tracknr)
		if GEE_interface.ORBIT == 'ASCENDING':	orbit_prefix = 'A'
		else:	orbit_prefix = 'D'
		outname = 'SMCS1_' + str(GEE_interface.S1_DATE.year) + '{:02d}'.format(GEE_interface.S1_DATE.month) + '{:02d}'.format(GEE_interface.S1_DATE.day) + '_' + '{:02d}'.format(GEE_interface.S1_DATE.hour) + '{:02d}'.format(GEE_interface.S1_DATE.minute) + '{:02d}'.format(GEE_interface.S1_DATE.second) + '_' + '{:03d}'.format(math.trunc(GEE_interface.TRACK_NR)) + '_' + orbit_prefix
		# Estimate soil moisture
		GEE_interface.estimate_SM_GBR_1step()
		GEE_interface = None
	else:
		# if no specific date was specified extract entire time series
		GEE_interface = GEE_extent(minlon, minlat, maxlon, maxlat, outpath, sampling=sampling)
		# get list of S1 dates
		dates, orbits = GEE_interface.get_S1_dates(tracknr=tracknr, ascending=False, start=start, stop=stop)
		dates, unqidx = np.unique(dates, return_index=True)
		orbits = orbits[unqidx]
		todeldates = list()
		for i in range(1, len(dates)):	
			if ((dates[i] - dates[i - 1]) < dt.timedelta(minutes=10)) & (orbits[i] == orbits[i - 1]):
				todeldates.append(i)
		dates = np.delete(dates, todeldates)
		orbits = np.delete(orbits, todeldates)
		for dateI, orbitI in zip(dates, orbits):
			print(dateI, orbitI)
			if GEE_interface.ORBIT == 'ASCENDING':
				orbit_prefix = 'A'
			else:
				orbit_prefix = 'D'	
			outname = 'SMCS1_' + str(GEE_interface.S1_DATE.year) + '{:02d}'.format(GEE_interface.S1_DATE.month) + '{:02d}'.format(GEE_interface.S1_DATE.day) + '_' + '{:02d}'.format(GEE_interface.S1_DATE.hour) + '{:02d}'.format(GEE_interface.S1_DATE.minute) + '{:02d}'.format(GEE_interface.S1_DATE.second) + '_' + '{:03d}'.format(math.trunc(GEE_interface.TRACK_NR)) + '_' + orbit_prefix
			if overwrite == False and os.path.exists(outpath + outname + '.tif'):
				print(outname + ' already done')
			continue
		# Estimate soil moisture	GEE_interface.estimate_SM_GBR_1step()
		if GEE_interface.ESTIMATED_SM is not None:		
			GEE_interface.GEE_2_asset(name=outname, timeout=False)
			GEE_interface = None

### Add prediction algorithms

Convert the prediction algorithms:

In [4]:
def tree(feature_stack): 
  prediction = ee.Image(0.14341582)
  learning_rate = ee.Image(0.1)
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
  	  "(b('ndvi_med') <= 3272.0) ?  (b('l8b4') <= 1320.9143676757812) ?  (b('moss') <= 6.230724096298218) ?  (b('sand') <= 19.5) ?  (b('grass') <= 2.184100389480591) ?  (b('l8b11') <= 1278.2006225585938) ?  (b('dg0_v_vv') <= 1.160327672958374) ?  0.37258414924144745 :   0.3355841785669327 :   (b('l8b5') <= 1829.779541015625) ?  (b('l8b11') <= 1598.2060546875) ?  0.29058416187763214 :   (b('l8b11') <= 1926.0296630859375) ?  (b('l8b11_med') <= 1594.7315673828125) ?  0.31258417665958405 :   0.3455841690301895 :   0.30358417332172394 :   (b('l8b4_med') <= 852.7552185058594) ?  0.3075841814279556 :   0.19658417999744415 :   (b('dg0_v_vv') <= -0.04914283752441406) ?  (b('l8b11_med') <= 1341.2097778320312) ?  (b('l8b11') <= 761.8933715820312) ?  0.2895841747522354 :   (b('dg0_v_vv') <= -2.02449369430542) ?  (b('ndvi') <= 2378.5) ?  0.2325841635465622 :   0.21008417010307312 :   (b('l8b5') <= 1989.2023315429688) ?  0.24258418381214142 :   0.2415841668844223 :   (b('l8b11') <= 1616.835205078125) ?  (b('dg0_v_vv') <= -0.9127793312072754) ?  (b('dg0_v_vv') <= -2.186915874481201) ?  0.13058416545391083 :   0.15658417344093323 :   (b('dg0_v_vv') <= -0.4929652214050293) ?  0.08058418333530426 :   0.07658417522907257 :   (b('ndvi') <= 2730.0) ?  0.3105841726064682 :   (b('l8b11') <= 1951.6445922851562) ?  0.21358416974544525 :   0.17858417332172394 :   (b('ndvi') <= 1195.0) ?  0.13558419048786163 :   (b('l8b11') <= 2065.9537353515625) ?  (b('ndvi') <= 2489.0) ?  (b('dg0_v_vv') <= 0.5887031555175781) ?  0.24658416211605072 :   0.2575841695070267 :   (b('ndvi_med') <= 2876.0) ?  0.2690841754277547 :   0.3127841740846634 :   0.20358417928218842 :   (b('sand') <= 48.0) ?  (b('dg0_v_vv') <= 0.9420537948608398) ?  (b('l8b4') <= 941.6824035644531) ?  (b('grass') <= 8.039268493652344) ?  (b('moss') <= 1.3587230145931244) ?  (b('l8b5') <= 3214.1798095703125) ?  0.03858417540788651 :   0.15589917749166488 :   (b('bulk') <= 134.0) ?  0.021170034402548665 :   -0.0377015375665256 :   (b('l8b5_med') <= 2532.333740234375) ?  (b('bulk') <= 141.5) ?  0.15318417996168138 :   0.014256590383191561 :   (b('l8b4') <= 450.79103088378906) ?  0.21018417179584503 :   0.09348417619864145 :   (b('l8b5_med') <= 2995.0274658203125) ?  (b('crops') <= 77.67890167236328) ?  (b('grass') <= 98.81764602661133) ?  -0.026363192229090553 :   0.030280379366271103 :   (b('l8b5_med') <= 2878.876220703125) ?  0.12103154392618883 :   0.01562365036653845 :   (b('ndvi_med') <= 2444.25) ?  (b('l8b11_med') <= 2783.42626953125) ?  0.0618758432360159 :   -0.0389713810549842 :   (b('l8b5') <= 3677.837646484375) ?  0.16131145032969388 :   -0.0014158263802528381 :   (b('bulk') <= 149.0) ?  (b('moss') <= 1.849461317062378) ?  (b('bulk') <= 131.0) ?  (b('l8b5_med') <= 2037.2767333984375) ?  0.13908417150378227 :   0.24318417012691498 :   (b('dg0_v_vv') <= 3.3309643268585205) ?  0.09563484360774357 :   0.15375658972509976 :   (b('crops') <= 7.39507794380188) ?  (b('l8b4_med') <= 1228.8809814453125) ?  -0.029344394536955015 :   0.19658417999744415 :   (b('ndvi') <= 1189.5) ?  0.21598417460918426 :   0.04409211335910691 :   (b('l8b5_med') <= 2490.571044921875) ?  (b('sand') <= 37.5) ?  (b('ndvi') <= 2181.5) ?  -0.14241582353133708 :   -0.12441582418978214 :   (b('dg0_v_vv') <= 2.6190052032470703) ?  0.056869889476469586 :   0.144220539114692 :   (b('grass') <= 3.4026710987091064) ?  (b('ndvi') <= 2727.5) ?  -0.06941582262516022 :   -0.017915822565555573 :   (b('l8b4') <= 1167.5443115234375) ?  0.19314478315187222 :   0.12172703444957733 :   (b('l8b11') <= 1630.2577514648438) ?  (b('moss') <= 3.7832995653152466) ?  (b('l8b4_med') <= 1038.5774536132812) ?  (b('l8b11') <= 1208.4296264648438) ?  (b('dg0_v_vv') <= -0.1515493392944336) ?  0.025426282302329416 :   -0.013665823265910149 :   (b('dg0_v_vv') <= -0.2821469306945801) ?  -0.006082490086555481 :   -0.07449274574621366 :   (b('l8b5') <= 3951.0191650390625) ?  (b('dg0_v_vv') <= 0.0747075080871582) ?  0.010479629801755602 :   0.07395197438486552 :   (b('l8b11') <= 1012.7681579589844) ?  -0.09513248999913533 :   0.041454176604747775 :   (b('bulk') <= 132.0) ?  (b('l8b4') <= 518.1531066894531) ?  (b('l8b4_med') <= 572.0263366699219) ?  0.3366841822862625 :   0.4518841952085495 :   (b('l8b5_med') <= 3051.766357421875) ?  0.25123417377471924 :   0.28168417513370514 :   (b('l8b5') <= 4527.298095703125) ?  (b('ndvi') <= 1261.5) ?  0.0038341768085956573 :   0.09078943925468545 :   -0.08041582256555557 :   (b('l8b4') <= 1305.6856689453125) ?  (b('ndvi_med') <= 2854.0) ?  (b('ndvi_med') <= 2171.5) ?  (b('dg0_v_vv') <= -0.6566524505615234) ?  -0.04997108661030468 :   0.00643641607307676 :   (b('dg0_v_vv') <= 0.0025649070739746094) ?  -0.07144394809074583 :   -0.04216405832592179 :   (b('ndvi') <= 3005.0) ?  (b('l8b5_med') <= 3018.2542724609375) ?  0.01825084247522884 :   0.0697091743350029 :   (b('l8b11') <= 1814.3958129882812) ?  0.046584175527095796 :   -0.0395737183525374 :   (b('dg0_v_vv') <= 2.093641757965088) ?  (b('l8b4') <= 1316.4590454101562) ?  (b('moss') <= 2.3914284706115723) ?  -0.009665824472904205 :   0.12058417002360027 :   (b('ndvi_med') <= 2121.0) ?  -0.06941582262516022 :   -0.12481582351028919 :   (b('l8b11_med') <= 2520.0809326171875) ?  0.04938417673110962 :   (b('bulk') <= 144.0) ?  0.21748419106006622 :   0.2098841816186905 :   (b('l8b4') <= 624.9253540039062) ?  (b('crops') <= 17.920488357543945) ?  (b('l8b5') <= 2558.9141845703125) ?  (b('grass') <= 65.79678726196289) ?  (b('l8b5') <= 1440.6194458007812) ?  (b('grass') <= 57.12829399108887) ?  (b('l8b5') <= 1111.59814453125) ?  -0.12541582435369492 :   -0.06821582342187564 :   (b('l8b5_med') <= 1550.7833251953125) ?  0.015584174543619156 :   0.02958417683839798 :   (b('l8b5_med') <= 2048.4805297851562) ?  (b('l8b4') <= 579.9348449707031) ?  0.11463545740414889 :   0.052057861497527676 :   (b('l8b4') <= 474.6342315673828) ?  0.08510739056925688 :   0.028790371706791683 :   (b('sand') <= 37.5) ?  (b('l8b5_med') <= 2318.950927734375) ?  0.05858418345451355 :   (b('bulk') <= 146.5) ?  0.21858416497707367 :   0.14518417418003082 :   (b('grass') <= 72.45567321777344) ?  (b('grass') <= 70.59422302246094) ?  0.26105084518591565 :   0.22153417766094208 :   0.32358418405056 :   (b('dg0_v_vv') <= -0.8378076553344727) ?  (b('ndvi_med') <= 1808.25) ?  -0.12681582383811474 :   (b('grass') <= 51.25812911987305) ?  (b('l8b4') <= 587.7214965820312) ?  0.049817509949207306 :   -0.021415822207927704 :   (b('dg0_v_vv') <= -1.0722060203552246) ?  0.14345083137353262 :   0.06558418273925781 :   (b('sand') <= 26.0) ?  (b('ndvi') <= 3180.0) ?  (b('ndvi') <= 2968.0) ?  0.3545841723680496 :   0.3575841635465622 :   (b('dg0_v_vv') <= -0.3507380485534668) ?  0.35158418118953705 :   0.22008417546749115 :   (b('bulk') <= 131.0) ?  -0.05341582000255585 :   (b('ndvi') <= 3906.5) ?  0.1628457150207116 :   0.07198418080806732 :   (b('l8b5') <= 2573.7869873046875) ?  (b('dg0_v_vv') <= -0.16718149185180664) ?  (b('l8b5_med') <= 2741.2508544921875) ?  (b('l8b4') <= 549.2231750488281) ?  -0.03141582012176514 :   -0.07141582667827606 :   (b('l8b5_med') <= 3044.43896484375) ?  0.029584169387817383 :   0.024584174156188965 :   (b('ndvi_med') <= 2965.5) ?  (b('ndvi') <= 3713.0) ?  (b('l8b5') <= 1795.919921875) ?  0.05458417534828186 :   0.09938417375087738 :   0.040584176778793335 :   (b('ndvi') <= 3662.5) ?  (b('dg0_v_vv') <= 0.8081293106079102) ?  -0.004415825009346008 :   -0.04541582614183426 :   0.07258418202400208 :   (b('l8b5') <= 3564.1083984375) ?  (b('l8b5') <= 3466.7381591796875) ?  (b('l8b4') <= 535.7750549316406) ?  (b('ndvi_med') <= 2965.5) ?  -0.029149156560500464 :   0.017655605184180394 :   (b('l8b11') <= 1051.2119750976562) ?  -0.09681582301855088 :   -0.04055218669501218 :   (b('l8b4') <= 552.7584228515625) ?  (b('ndvi_med') <= 2473.25) ?  -0.03641582280397415 :   -0.02641582489013672 :   (b('dg0_v_vv') <= -0.6357183456420898) ?  -0.13941582362167537 :   -0.10241582291200757 :   (b('l8b11') <= 1171.0237426757812) ?  (b('l8b5') <= 3947.7930908203125) ?  (b('ndvi') <= 4630.0) ?  -0.015506733547557484 :   0.017473066846529644 :   (b('ndvi') <= 4710.5) ?  -0.034492747141764715 :   -0.10941582173109055 :   (b('l8b5_med') <= 2850.484619140625) ?  (b('ndvi') <= 3871.0) ?  0.08758418262004852 :   0.07391750812530518 :   (b('l8b11') <= 1198.7442016601562) ?  0.04933417588472366 :   0.0034932670268145475 :   (b('dg0_v_vv') <= 1.4022154808044434) ?  (b('bulk') <= 138.5) ?  (b('bulk') <= 130.0) ?  (b('l8b11') <= 2542.0272216796875) ?  (b('l8b11') <= 1251.7822265625) ?  (b('ndvi') <= 2771.5) ?  -0.04021582324057817 :   -0.12280218724415383 :   (b('ndvi') <= 3014.5) ?  -0.07880143499631675 :   -0.0420443947826113 :   (b('dg0_v_vv') <= 0.7835025787353516) ?  (b('l8b11') <= 3116.550048828125) ?  0.07183417677879333 :   -0.04441582411527634 :   0.2025841623544693 :   (b('l8b5_med') <= 2213.9940185546875) ?  (b('l8b11') <= 1873.5889282226562) ?  (b('l8b4_med') <= 1115.0663452148438) ?  -0.018062966210501536 :   0.049149562389804766 :   (b('dg0_v_vv') <= 0.8639883995056152) ?  -0.05608693465280036 :   0.04291751111547152 :   (b('dg0_v_vv') <= 0.327455997467041) ?  (b('l8b5_med') <= 2890.894775390625) ?  -0.044793435469714565 :   -0.06608890025482442 :   (b('l8b4') <= 1263.592041015625) ?  -0.03652799053705464 :   0.04325084388256073 :   (b('ndvi_med') <= 2726.0) ?  (b('grass') <= 3.7684255838394165) ?  (b('l8b11') <= 1164.012939453125) ?  (b('ndvi') <= 2986.0) ?  -0.0014158263802528381 :   0.01658417284488678 :   (b('ndvi_med') <= 1480.5) ?  0.026784178614616395 :   0.07574042771011591 :   (b('grass') <= 62.383872985839844) ?  (b('sand') <= 58.5) ?  -0.05433681999039886 :   0.028265426866710186 :   (b('l8b11') <= 1490.4927368164062) ?  0.051770252293518074 :   -0.02735988744539868 :   (b('grass') <= 44.10457229614258) ?  (b('crops') <= 29.692421913146973) ?  (b('ndvi') <= 2947.5) ?  -0.05712382361292839 :   0.0033993279617844205 :   (b('l8b5_med') <= 2588.2825927734375) ?  0.1061952883998553 :   0.03270917688496411 :   (b('dg0_v_vv') <= -1.153111457824707) ?  (b('grass') <= 49.792694091796875) ?  0.11958417668938637 :   0.017334173899143934 :   (b('l8b11') <= 1770.5822143554688) ?  0.2001995604771834 :   0.11764478299653891 :   (b('l8b4_med') <= 1103.789306640625) ?  (b('moss') <= 26.06871795654297) ?  (b('bulk') <= 137.5) ?  (b('grass') <= 73.11345672607422) ?  (b('sand') <= 38.5) ?  0.001089371910149401 :   -0.037625500466674566 :   (b('l8b11') <= 1657.3817749023438) ?  0.01786989133272852 :   0.08395917527377605 :   (b('bulk') <= 148.5) ?  (b('l8b5_med') <= 1838.5203247070312) ?  0.20644131941454752 :   0.08089872151613235 :   (b('l8b11_med') <= 1152.8079223632812) ?  -0.11231582344044 :   0.013350843451917171 :   (b('grass') <= 51.0879020690918) ?  (b('ndvi') <= 1651.0) ?  (b('ndvi') <= 961.0) ?  0.018584176898002625 :   -0.1264158240519464 :   (b('l8b4') <= 1145.5990600585938) ?  0.019584178924560547 :   0.08058418333530426 :   (b('l8b4_med') <= 1071.31494140625) ?  (b('ndvi') <= 1577.5) ?  -0.11659915706453224 :   -0.0800936010459231 :   0.046584174036979675 :   (b('l8b4') <= 1201.6109008789062) ?  (b('moss') <= 28.092727661132812) ?  (b('sand') <= 38.0) ?  (b('ndvi_med') <= 1312.25) ?  0.26591751476128894 :   0.1566023582761938 :   (b('ndvi_med') <= 2534.5) ?  0.13122306681341595 :   0.005917511880397797 :   (b('l8b5') <= 2248.20458984375) ?  (b('ndvi') <= 2006.0) ?  0.055311449549414894 :   0.21438418328762054 :   (b('l8b4') <= 994.65234375) ?  -0.011415824294090271 :   -0.024415824562311172 :   (b('l8b5_med') <= 3169.4300537109375) ?  (b('l8b5') <= 1970.3256225585938) ?  (b('l8b11_med') <= 2012.7874755859375) ?  -0.10551582276821136 :   -0.05091582238674164 :   (b('sand') <= 38.0) ?  -0.03808249160647392 :   0.05676599456505342 :   (b('moss') <= 12.622721195220947) ?  (b('ndvi') <= 1314.0) ?  -0.08441582322120667 :   -0.05341582000255585 :   (b('l8b11_med') <= 3332.7203369140625) ?  -0.12441582418978214 :   -0.14241582353133708 :   (b('moss') <= 8.85230016708374) ?  (b('l8b11_med') <= 2635.76513671875) ?  (b('moss') <= 0.0634004482999444) ?  (b('grass') <= 85.5868148803711) ?  (b('bulk') <= 149.0) ?  (b('bulk') <= 143.5) ?  (b('dg0_v_vv') <= 3.1727139949798584) ?  (b('ndvi_med') <= 1894.75) ?  -0.005257488538821538 :   0.062440427020192146 :   (b('sand') <= 32.0) ?  0.2965841740369797 :   0.24258418381214142 :   (b('dg0_v_vv') <= 1.1476354598999023) ?  (b('dg0_v_vv') <= -0.4414181709289551) ?  -0.025337562066219423 :   -0.06291582348704752 :   (b('crops') <= 58.6531925201416) ?  -0.021615825593471527 :   0.1088341698050499 :   (b('grass') <= 79.79425048828125) ?  (b('l8b11') <= 2311.5482177734375) ?  (b('bulk') <= 166.5) ?  -0.1327842445998134 :   -0.08405218747529117 :   (b('l8b5') <= 3353.4908447265625) ?  -0.061364803273154765 :   -0.10690673247657038 :   (b('l8b11') <= 2005.1611328125) ?  (b('l8b4') <= 1485.5) ?  -0.03533890212957676 :   -0.018415823578834534 :   (b('l8b4_med') <= 1244.2680053710938) ?  -0.040047402444638704 :   -0.06141582326520057 :   (b('dg0_v_vv') <= 1.21183443069458) ?  (b('ndvi_med') <= 2168.25) ?  (b('l8b5') <= 3361.9205322265625) ?  (b('bulk') <= 142.5) ?  -0.0662340045991269 :   -0.023567722379406797 :   0.22358419001102448 :   (b('ndvi') <= 1273.5) ?  (b('ndvi') <= 1171.0) ?  0.1505841761827469 :   0.10933417454361916 :   (b('bulk') <= 147.0) ?  -0.041815824806690216 :   0.03458417739186968 :   (b('ndvi_med') <= 2198.0) ?  (b('ndvi') <= 2351.5) ?  (b('l8b4_med') <= 1171.5706176757812) ?  -0.02241582175095876 :   0.10483417380601168 :   (b('l8b11') <= 2422.7069091796875) ?  0.22258417308330536 :   0.1855841726064682 :   (b('ndvi') <= 1304.0) ?  0.28758417069911957 :   (b('l8b4_med') <= 1055.1670227050781) ?  0.17458416521549225 :   0.1815841794013977 :   (b('l8b5_med') <= 3030.7413330078125) ?  (b('grass') <= 4.100144267082214) ?  (b('l8b11') <= 2080.4179077148438) ?  (b('l8b5') <= 2625.613037109375) ?  (b('dg0_v_vv') <= 0.9701080322265625) ?  0.13408417254686356 :   0.1899175097544988 :   (b('dg0_v_vv') <= -0.7952628135681152) ?  0.034141319138663154 :   0.08669042214751244 :   (b('dg0_v_vv') <= 0.9387078285217285) ?  (b('sand') <= 30.0) ?  0.054193266413428566 :   0.004217269924207986 :   (b('l8b5') <= 2678.9945068359375) ?  0.08606036610546566 :   0.03702862091638424 :   (b('dg0_v_vv') <= 1.240004062652588) ?  (b('sand') <= 24.0) ?  (b('dg0_v_vv') <= -0.45614099502563477) ?  0.03791750967502594 :   0.10758417596419652 :   (b('l8b4_med') <= 871.7990112304688) ?  -0.0965696697911391 :   -0.032938379729003237 :   (b('ndvi_med') <= 2111.0) ?  (b('l8b11') <= 2866.4813232421875) ?  0.11131494549604562 :   0.010750845074653625 :   (b('sand') <= 27.0) ?  0.05158416926860809 :   -0.029688551005992023 :   (b('l8b4') <= 1411.1295166015625) ?  (b('l8b5') <= 3335.5250244140625) ?  (b('bulk') <= 142.0) ?  (b('dg0_v_vv') <= 0.8479576110839844) ?  0.09038417637348176 :   0.1945841796696186 :   (b('l8b11') <= 2223.065185546875) ?  0.20649326524951242 :   0.245584174990654 :   (b('dg0_v_vv') <= -0.4390292167663574) ?  (b('l8b5_med') <= 3097.873046875) ?  -0.0074158161878585815 :   -0.03341582417488098 :   (b('dg0_v_vv') <= 0.24390554428100586) ?  0.09458418190479279 :   0.13458417356014252 :   (b('grass') <= 13.808833599090576) ?  (b('dg0_v_vv') <= 1.4927678108215332) ?  (b('l8b5') <= 2834.01611328125) ?  0.06919528709517585 :   0.003948882660445045 :   (b('ndvi') <= 3082.5) ?  0.08786988843764577 :   0.20858417451381683 :   (b('grass') <= 21.67441177368164) ?  (b('dg0_v_vv') <= -0.6033682823181152) ?  0.049030986555079197 :   0.1010072546509596 :   (b('dg0_v_vv') <= -2.008871555328369) ?  0.14991749823093414 :   0.20158417522907257 :   (b('bulk') <= 152.5) ?  (b('grass') <= 13.810791015625) ?  (b('dg0_v_vv') <= 2.3459343910217285) ?  (b('l8b11') <= 3318.1075439453125) ?  (b('ndvi') <= 2862.0) ?  (b('l8b5') <= 2604.697265625) ?  -0.07216000983621491 :   -0.09724211440740826 :   (b('l8b11') <= 2334.5906982421875) ?  -0.09584439492651395 :   -0.04029082250781357 :   (b('l8b4') <= 2363.0313720703125) ?  (b('dg0_v_vv') <= 2.196920871734619) ?  -0.118630109231786 :   -0.06891582533717155 :   (b('l8b4') <= 2395.3173828125) ?  -0.07461582310497761 :   -0.11696582343429326 :   (b('moss') <= 2.384734630584717) ?  (b('l8b11') <= 2860.9820556640625) ?  (b('l8b11') <= 2060.0965576171875) ?  -0.08241582289338112 :   -0.05474915603796641 :   (b('l8b11') <= 3413.206298828125) ?  -0.1040824901105629 :   -0.037415824830532074 :   (b('l8b4_med') <= 1830.4744262695312) ?  (b('l8b4') <= 2011.6565551757812) ?  -0.06313011103442737 :   0.025084175169467926 :   (b('l8b4') <= 2162.5101318359375) ?  0.019834176637232304 :   -0.05374915897846222 :   (b('dg0_v_vv') <= 1.8097028732299805) ?  (b('dg0_v_vv') <= 0.19105243682861328) ?  (b('l8b5') <= 2791.5035400390625) ?  (b('dg0_v_vv') <= -2.157388210296631) ?  0.04118417203426361 :   -0.0867686009603656 :   (b('l8b5') <= 2903.9736328125) ?  0.0011441772182782491 :   -0.06337323077712897 :   (b('ndvi_med') <= 1858.25) ?  (b('l8b5') <= 3706.9039306640625) ?  -0.032609157264232635 :   -0.08258249051868916 :   (b('dg0_v_vv') <= 0.759058952331543) ?  0.05850917845964432 :   -0.003215824564297994 :   (b('l8b11') <= 3213.1171875) ?  (b('l8b4') <= 1518.3990478515625) ?  (b('l8b5') <= 2923.236572265625) ?  0.18223418295383453 :   0.02833417057991028 :   (b('dg0_v_vv') <= 3.221949338912964) ?  0.02379846466439111 :   0.05195917375385761 :   (b('l8b11_med') <= 2830.743408203125) ?  -0.012415826320648193 :   (b('ndvi_med') <= 1520.75) ?  -0.0648658238351345 :   -0.059415824711322784 :   (b('l8b4') <= 1960.7346801757812) ?  (b('l8b11') <= 3091.4337158203125) ?  (b('dg0_v_vv') <= 2.329883098602295) ?  (b('l8b11_med') <= 2848.3275146484375) ?  (b('l8b4') <= 1860.0399169921875) ?  -0.09108249035974343 :   -0.04841582477092743 :   (b('dg0_v_vv') <= -0.052337646484375) ?  -0.008582489875455698 :   0.03694781186905774 :   (b('l8b11') <= 2798.9844970703125) ?  (b('l8b4') <= 1432.2578125) ?  0.18658418953418732 :   0.05629846294011388 :   (b('l8b11') <= 2836.7003173828125) ?  0.14608417451381683 :   0.11508417129516602 :   (b('l8b5_med') <= 2891.0013427734375) ?  (b('l8b5') <= 2982.88671875) ?  (b('ndvi') <= 1095.0) ?  0.08658417562643687 :   0.1190841756761074 :   0.17458416521549225 :   (b('l8b5_med') <= 2953.8138427734375) ?  0.06358417868614197 :   (b('dg0_v_vv') <= 1.707441806793213) ?  0.03558418154716492 :   0.0475841760635376 :   (b('crops') <= 36.73203182220459) ?  (b('dg0_v_vv') <= 3.4477806091308594) ?  (b('l8b5') <= 3013.663330078125) ?  (b('ndvi') <= 1207.0) ?  -0.06561582386493683 :   -0.029040824621915817 :   (b('l8b5') <= 3067.1474609375) ?  -0.08252693422966534 :   -0.06665972592990572 :   0.0995841771364212 :   (b('ndvi') <= 1720.0) ?  (b('grass') <= 20.817001670598984) ?  (b('l8b11') <= 3393.3631591796875) ?  -0.04914915636181831 :   -0.023229777119880498 :   (b('l8b4') <= 1972.2215576171875) ?  -0.0197491521636645 :   0.06958417408168316 :   (b('l8b5') <= 4246.0625) ?  (b('l8b4') <= 2370.552978515625) ?  -0.02229082491248846 :   0.045675084672190926 :   (b('ndvi_med') <= 1349.5) ?  0.10125084221363068 :   0.11358417570590973 :   (b('l8b5_med') <= 3303.5611572265625) ?  (b('dg0_v_vv') <= 1.2746853828430176) ?  (b('crops') <= 72.32853889465332) ?  (b('l8b4') <= 1694.9112548828125) ?  (b('ndvi') <= 808.0) ?  (b('ndvi_med') <= 899.0) ?  (b('l8b4') <= 1502.0194702148438) ?  0.04175084208448728 :   -0.08221582358237356 :   (b('dg0_v_vv') <= -0.24559879302978516) ?  0.0019841790199279785 :   0.16498417556285858 :   (b('grass') <= 23.900150299072266) ?  (b('l8b5') <= 2124.5980224609375) ?  -0.03399915682772795 :   0.04589326578107747 :   (b('ndvi_med') <= 1543.0) ?  -0.053809185262083434 :   -0.0737344441871191 :   (b('l8b5_med') <= 2627.36669921875) ?  (b('grass') <= 41.722944259643555) ?  (b('l8b11_med') <= 2682.1971435546875) ?  -0.11218926095898496 :   -0.07833890058100224 :   (b('l8b11_med') <= 2904.1224365234375) ?  -0.07056255240611314 :   -0.10013010931600418 :   (b('crops') <= 14.336705207824707) ?  (b('sand') <= 52.5) ?  -0.04938382348418236 :   -0.07048521114095133 :   (b('ndvi') <= 1928.0) ?  -0.10418052923427347 :   -0.06641582259908319 :   (b('l8b11') <= 2831.4981689453125) ?  (b('l8b5') <= 3633.9498291015625) ?  (b('dg0_v_vv') <= -2.34287691116333) ?  -0.03041582554578781 :   (b('l8b11') <= 2640.16845703125) ?  0.056050842503706615 :   0.021334178745746613 :   (b('ndvi') <= 1567.5) ?  (b('ndvi') <= 1509.5) ?  -0.028415821492671967 :   -0.0012158215045928954 :   (b('dg0_v_vv') <= -0.2057514190673828) ?  -0.014415819197893143 :   -0.03241582214832306 :   (b('ndvi') <= 2526.0) ?  (b('ndvi') <= 1262.5) ?  (b('ndvi') <= 1233.0) ?  -0.009709941990235272 :   0.12858419120311737 :   (b('ndvi') <= 1581.5) ?  -0.03414659230754925 :   -0.01888248821099599 :   (b('dg0_v_vv') <= 0.1292414665222168) ?  0.03258417546749115 :   0.07558417320251465 :   (b('bulk') <= 145.5) ?  (b('ndvi_med') <= 1373.25) ?  (b('ndvi') <= 2308.5) ?  (b('l8b11') <= 1879.8290405273438) ?  (b('dg0_v_vv') <= 2.4786877632141113) ?  0.19258417189121246 :   0.12758417427539825 :   (b('grass') <= 55.649017333984375) ?  0.01383417472243309 :   0.06924571555394393 :   0.2603841871023178 :   (b('l8b4') <= 1431.702392578125) ?  (b('bulk') <= 140.5) ?  (b('l8b4') <= 1338.0252685546875) ?  0.07958418130874634 :   -0.014182490607102713 :   (b('l8b11_med') <= 2453.065185546875) ?  0.1295841783285141 :   0.06858417391777039 :   (b('moss') <= 25.046354293823242) ?  (b('l8b5') <= 2709.19775390625) ?  -0.07155868144971984 :   -0.10141582414507866 :   0.023584172129631042 :   (b('moss') <= 15.701762199401855) ?  (b('dg0_v_vv') <= 2.6041290760040283) ?  (b('ndvi') <= 1819.0) ?  (b('l8b5') <= 2167.446533203125) ?  0.06258417665958405 :   -0.0304969053816151 :   (b('l8b5') <= 2873.254638671875) ?  0.012334179133176804 :   0.06538417935371399 :   (b('ndvi_med') <= 1431.25) ?  (b('l8b5_med') <= 2780.341064453125) ?  -0.06891582440584898 :   0.01508417601386706 :   (b('dg0_v_vv') <= 6.708409547805786) ?  0.0889478163285689 :   0.17358417809009552 :   (b('moss') <= 50.33349609375) ?  (b('moss') <= 38.56039237976074) ?  (b('l8b11_med') <= 2707.0025634765625) ?  -0.11380471206373638 :   -0.03991582337766886 :   (b('l8b5_med') <= 2429.1533203125) ?  0.040084175765514374 :   -0.05491582304239273 :   (b('l8b11') <= 2256.9927978515625) ?  (b('l8b4') <= 1576.0043334960938) ?  -0.10674915586908658 :   -0.1257491564999024 :   -0.062415823340415955 :   (b('bulk') <= 151.5) ?  (b('l8b4_med') <= 2066.49853515625) ?  (b('crops') <= 46.23920249938965) ?  (b('l8b5') <= 3717.3253173828125) ?  (b('ndvi') <= 1742.0) ?  (b('ndvi') <= 1185.5) ?  -0.006749155620733897 :   -0.06692801883190537 :   (b('dg0_v_vv') <= -0.28783273696899414) ?  -0.13316582384868525 :   -0.06252693422828559 :   (b('dg0_v_vv') <= -0.26917600631713867) ?  (b('dg0_v_vv') <= -1.0925307273864746) ?  -0.11341582424938679 :   -0.12041582353413105 :   (b('ndvi_med') <= 1569.0) ?  -0.1320824899400274 :   -0.1407491568243131 :   (b('ndvi_med') <= 1726.0) ?  (b('dg0_v_vv') <= 0.9063091278076172) ?  (b('l8b11') <= 4141.33837890625) ?  -0.1267052972045923 :   -0.09191582351922989 :   (b('ndvi') <= 3044.5) ?  -0.11533249033770214 :   -0.09581582359969616 :   (b('ndvi') <= 3524.5) ?  (b('l8b4') <= 2183.7149658203125) ?  -0.11241582343925019 :   -0.09401582330465316 :   (b('l8b11') <= 3198.6287841796875) ?  -0.09174915527304013 :   -0.0774158239364624 :   (b('dg0_v_vv') <= 1.3677406311035156) ?  (b('l8b4_med') <= 2523.0831298828125) ?  (b('l8b5') <= 3096.8826904296875) ?  (b('l8b4') <= 2057.93798828125) ?  -0.10603739221708156 :   -0.13598725213835547 :   (b('l8b11') <= 4367.96875) ?  -0.13136624365474986 :   -0.11075332341715693 :   (b('moss') <= 17.4368257522583) ?  (b('dg0_v_vv') <= -0.4319009780883789) ?  -0.06141582131385803 :   -0.0870158240199089 :   -0.03541582077741623 :   (b('l8b5') <= 3433.4608154296875) ?  (b('l8b11') <= 1916.8164672851562) ?  (b('l8b11_med') <= 2774.488037109375) ?  -0.0814158245921135 :   -0.08041582256555557 :   (b('l8b11') <= 2098.657470703125) ?  -0.1267491569839573 :   -0.10767898066459518 :   (b('ndvi_med') <= 1578.5) ?  (b('l8b11') <= 2258.0469970703125) ?  -0.08519360195431444 :   -0.05418505462316366 :   (b('dg0_v_vv') <= 1.4019618034362793) ?  -0.056082492073376976 :   -0.11998725200759336 :   (b('l8b11') <= 3204.9512939453125) ?  (b('dg0_v_vv') <= 1.8189005851745605) ?  (b('grass') <= 23.24074077606201) ?  (b('ndvi_med') <= 842.0) ?  (b('dg0_v_vv') <= -0.2555122375488281) ?  0.028584182262420654 :   0.04258418083190918 :   (b('dg0_v_vv') <= 1.6891093254089355) ?  0.0009925104677677155 :   -0.035715825855731964 :   (b('l8b11') <= 3125.280517578125) ?  (b('l8b4') <= 2345.03271484375) ?  -0.05441582202911377 :   -0.0319158211350441 :   -0.08441582322120667 :   (b('l8b11') <= 2707.9827880859375) ?  (b('dg0_v_vv') <= 3.188333749771118) ?  0.07858417928218842 :   0.09048417210578918 :   (b('l8b4') <= 2516.7257080078125) ?  (b('l8b4') <= 2448.8662109375) ?  0.02158418297767639 :   0.037584178149700165 :   0.0035841763019561768 :   (b('dg0_v_vv') <= 1.7529659271240234) ?  (b('grass') <= 23.24074077606201) ?  (b('l8b5') <= 3999.16943359375) ?  (b('l8b4_med') <= 3496.1298828125) ?  -0.05487036882815036 :   0.030584171414375305 :   (b('l8b11') <= 3991.739990234375) ?  -0.052945234977147156 :   -0.08606582367792726 :   (b('l8b5') <= 4250.3289794921875) ?  (b('l8b5') <= 3422.8250732421875) ?  -0.06541582383215427 :   -0.09638804564666417 :   -0.03241582214832306 :   (b('dg0_v_vv') <= 3.2638907432556152) ?  (b('l8b4') <= 2429.7333984375) ?  0.01658417284488678 :   (b('l8b4') <= 2941.072021484375) ?  -0.04701582491397858 :   -0.016415821388363838 :   (b('dg0_v_vv') <= 4.185548305511475) ?  (b('l8b11') <= 3770.3057861328125) ?  -0.0024158209562301636 :   -0.014415822923183441 :   (b('l8b11_med') <= 3982.898193359375) ?  0.024584174156188965 :   0.014584183692932129 :   (b('grass') <= 46.14527893066406) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('sand') <= 20.5) ?  (b('l8b5') <= 3486.4771728515625) ?  (b('l8b4') <= 871.57177734375) ?  (b('l8b11') <= 1197.610107421875) ?  (b('ndvi') <= 3588.5) ?  (b('dg0_v_vv') <= 1.6225152015686035) ?  0.19158418476581573 :   0.20958416163921356 :   (b('ndvi') <= 4861.0) ?  (b('dg0_v_vv') <= 2.825899600982666) ?  0.14491750299930573 :   0.11158417165279388 :   (b('ndvi_med') <= 3396.0) ?  0.18858416378498077 :   0.18058417737483978 :   (b('dg0_v_vv') <= 0.3637232780456543) ?  (b('dg0_v_vv') <= -1.5554656982421875) ?  0.10058417916297913 :   (b('l8b4') <= 799.0055541992188) ?  0.2252508451541265 :   0.19798417389392853 :   (b('l8b4') <= 852.0057067871094) ?  (b('ndvi') <= 4786.0) ?  0.3073619587553872 :   0.24258418381214142 :   (b('dg0_v_vv') <= 1.6866419315338135) ?  0.20958416163921356 :   0.23958416283130646 :   (b('l8b4') <= 897.3836059570312) ?  (b('dg0_v_vv') <= -0.8320989608764648) ?  0.10858418047428131 :   (b('dg0_v_vv') <= 0.07727575302124023) ?  0.06458418071269989 :   (b('l8b5_med') <= 3269.083740234375) ?  0.03158417344093323 :   0.03958417475223541 :   (b('dg0_v_vv') <= 0.908597469329834) ?  (b('dg0_v_vv') <= -2.699215888977051) ?  0.09158417582511902 :   (b('l8b5') <= 2693.2625732421875) ?  0.15158416330814362 :   0.257184174656868 :   0.04158417880535126 :   (b('l8b4') <= 450.18426513671875) ?  (b('l8b4_med') <= 571.3990478515625) ?  0.1765841692686081 :   (b('bulk') <= 146.5) ?  (b('dg0_v_vv') <= -0.5423331260681152) ?  0.245584174990654 :   0.2155841737985611 :   (b('moss') <= 3.2715134620666504) ?  0.2505841702222824 :   0.26458416879177094 :   (b('l8b4') <= 779.3079223632812) ?  (b('l8b4') <= 655.6082763671875) ?  (b('dg0_v_vv') <= -0.726778507232666) ?  (b('l8b11') <= 1124.3295288085938) ?  0.11878417134284973 :   0.01408417522907257 :   (b('ndvi') <= 4583.5) ?  0.1251995609356807 :   0.19358417093753816 :   (b('dg0_v_vv') <= 0.07375192642211914) ?  0.10658417642116547 :   (b('l8b4_med') <= 610.1589050292969) ?  0.03158417344093323 :   -0.010415822267532349 :   (b('l8b5_med') <= 3595.3917236328125) ?  (b('dg0_v_vv') <= -0.8319530487060547) ?  0.2505841702222824 :   0.31258417665958405 :   0.159584179520607 :   (b('ndvi_med') <= 4666.0) ?  (b('dg0_v_vv') <= -0.05260753631591797) ?  (b('moss') <= 11.730731010437012) ?  (b('bulk') <= 140.5) ?  (b('crops') <= 15.965596199035645) ?  (b('ndvi') <= 3135.5) ?  0.017293701214449748 :   -0.03934439454151659 :   (b('l8b4') <= 1651.916259765625) ?  0.032513280909112305 :   0.19525083899497986 :   (b('sand') <= 35.0) ?  (b('ndvi_med') <= 4051.5) ?  0.1330035324058225 :   0.06475809151711671 :   (b('l8b5') <= 3040.934326171875) ?  0.031721675768494606 :   -0.02941582351922989 :   (b('ndvi') <= 2930.0) ?  (b('l8b5') <= 3036.5240478515625) ?  (b('crops') <= 23.92689323425293) ?  -0.09541582316160202 :   0.014017510745260451 :   (b('ndvi_med') <= 4530.5) ?  0.07838417589664459 :   0.056584179401397705 :   (b('l8b11_med') <= 1179.0203857421875) ?  (b('ndvi') <= 4137.5) ?  0.05128417909145355 :   -0.05021582543849945 :   (b('ndvi_med') <= 4097.75) ?  0.11976482118329694 :   0.2209175080060959 :   (b('l8b11_med') <= 1026.959716796875) ?  (b('ndvi') <= 4270.5) ?  (b('ndvi') <= 2972.5) ?  (b('l8b11') <= 1057.0957641601562) ?  0.06508417640413557 :   0.21894417703151703 :   (b('l8b4') <= 1103.4029541015625) ?  0.027559176517220643 :   0.17958417534828186 :   (b('l8b11_med') <= 944.0569458007812) ?  (b('bulk') <= 131.5) ?  0.0905841737985611 :   0.04555084307988485 :   (b('l8b4_med') <= 459.2218933105469) ?  -0.05787653782005821 :   -0.003526933491230011 :   (b('bulk') <= 110.0) ?  (b('l8b4') <= 719.5686950683594) ?  (b('l8b5') <= 3824.29150390625) ?  0.13508417776652745 :   0.18753655964419955 :   (b('dg0_v_vv') <= 0.7878036499023438) ?  0.05058417717615763 :   0.02258417010307312 :   (b('sand') <= 27.5) ?  (b('crops') <= 58.59913444519043) ?  0.14732341638094262 :   0.06351400062180403 :   (b('l8b5') <= 2370.5924072265625) ?  0.11270639855000708 :   0.05929362945332753 :   (b('ndvi') <= 4524.0) ?  (b('ndvi') <= 3405.5) ?  (b('ndvi_med') <= 4697.0) ?  (b('ndvi') <= 2579.0) ?  0.32158417999744415 :   (b('dg0_v_vv') <= 0.9963421821594238) ?  0.3025841861963272 :   0.2935841828584671 :   (b('l8b5') <= 2874.64111328125) ?  0.3115841895341873 :   (b('dg0_v_vv') <= 3.1400158405303955) ?  0.3465841660896937 :   0.3310841768980026 :   (b('l8b11') <= 1240.7450256347656) ?  (b('l8b11_med') <= 1068.9169921875) ?  (b('l8b5') <= 3195.332275390625) ?  0.1465841680765152 :   0.03258417546749115 :   (b('dg0_v_vv') <= -0.08533143997192383) ?  0.18408417701721191 :   0.11358417570590973 :   (b('dg0_v_vv') <= -1.2602524757385254) ?  (b('dg0_v_vv') <= -2.155897617340088) ?  0.3355841785669327 :   0.19958417117595673 :   (b('l8b11_med') <= 1259.6744995117188) ?  0.3115841895341873 :   0.3495841771364212 :   (b('l8b4') <= 500.34800720214844) ?  (b('l8b4_med') <= 630.9979553222656) ?  (b('dg0_v_vv') <= -1.0223097801208496) ?  (b('ndvi') <= 4724.5) ?  0.2245841771364212 :   0.14958418905735016 :   (b('l8b5') <= 3105.357666015625) ?  0.30758417149384815 :   0.23158417642116547 :   (b('dg0_v_vv') <= 0.08380699157714844) ?  0.1465841680765152 :   0.15158416330814362 :   (b('dg0_v_vv') <= 0.7857785224914551) ?  (b('l8b4_med') <= 620.2109069824219) ?  (b('l8b11_med') <= 1139.5846557617188) ?  0.006250843405723572 :   -0.034415826201438904 :   (b('l8b5') <= 3202.7919921875) ?  -0.0024158209562301636 :   0.05114667396992445 :   (b('l8b4_med') <= 630.9979553222656) ?  (b('dg0_v_vv') <= 1.5339112281799316) ?  0.10858417302370071 :   0.08958417177200317 :   0.14458416402339935 :   (b('moss') <= 6.192884922027588) ?  (b('bulk') <= 154.0) ?  (b('dg0_v_vv') <= -0.5716390609741211) ?  (b('dg0_v_vv') <= -2.0733509063720703) ?  (b('sand') <= 41.0) ?  (b('dg0_v_vv') <= -3.665328025817871) ?  (b('l8b5') <= 3393.3226318359375) ?  -0.09791582450270653 :   -0.10941582173109055 :   (b('l8b5') <= 2730.2449951171875) ?  -0.03241582214832306 :   -0.07516582310199738 :   (b('l8b5') <= 3132.69287109375) ?  (b('l8b11') <= 2084.4052124023438) ?  0.024459177628159523 :   0.05338417589664459 :   (b('l8b5') <= 4000.214111328125) ?  -0.04127296805381775 :   0.04258417338132858 :   (b('l8b4') <= 1052.6575927734375) ?  (b('l8b5') <= 3734.87060546875) ?  (b('l8b4') <= 790.653076171875) ?  0.03502862072653241 :   -0.008349156379699707 :   (b('l8b4_med') <= 1216.8577270507812) ?  0.06738417446613312 :   0.1555841714143753 :   (b('l8b11_med') <= 1797.4715576171875) ?  (b('l8b4_med') <= 918.5065002441406) ?  0.11095917783677578 :   0.21108417212963104 :   (b('l8b11_med') <= 2095.1082763671875) ?  -0.02041582018136978 :   0.05658417567610741 :   (b('grass') <= 39.1515007019043) ?  (b('sand') <= 46.0) ?  (b('l8b5') <= 3882.23486328125) ?  (b('ndvi') <= 4541.5) ?  0.14267941599800474 :   0.09425084458457099 :   (b('dg0_v_vv') <= 1.1098873615264893) ?  0.02778417766094208 :   0.13808418065309525 :   (b('ndvi') <= 4223.5) ?  (b('l8b11_med') <= 1535.9295043945312) ?  0.043717510004838304 :   0.1125841783542259 :   (b('sand') <= 78.5) ?  0.059850842754046125 :   0.019250840776496463 :   (b('l8b5') <= 2625.9984130859375) ?  -0.08441582322120667 :   (b('l8b4') <= 1002.3343200683594) ?  (b('ndvi') <= 4092.5) ?  -0.028082492450873058 :   0.019984176754951476 :   (b('ndvi') <= 3493.5) ?  0.0475841760635376 :   0.07258418202400208 :   (b('ndvi') <= 3035.5) ?  (b('ndvi') <= 1879.0) ?  -0.013415828347206116 :   (b('dg0_v_vv') <= -0.19986295700073242) ?  (b('dg0_v_vv') <= -3.2769956588745117) ?  -0.09941582381725311 :   (b('ndvi') <= 2045.5) ?  -0.1034158244729042 :   -0.11529082339257002 :   -0.0814158245921135 :   (b('ndvi') <= 4567.5) ?  (b('l8b5') <= 3594.9215087890625) ?  (b('l8b4') <= 911.4842834472656) ?  (b('l8b5_med') <= 3211.497802734375) ?  -0.07591582369059324 :   -0.019165821373462677 :   (b('dg0_v_vv') <= 0.08088445663452148) ?  -0.028130108756678446 :   0.002084176056087017 :   (b('ndvi') <= 3892.5) ?  (b('l8b5') <= 3636.8193359375) ?  -0.09741582348942757 :   -0.08461582362651825 :   (b('dg0_v_vv') <= 1.7421860694885254) ?  -0.016415826976299286 :   -0.07641582190990448 :   (b('l8b5_med') <= 3211.497802734375) ?  (b('l8b4') <= 992.0172729492188) ?  0.029584169387817383 :   -0.0024158209562301636 :   (b('ndvi') <= 4607.5) ?  0.06158417463302612 :   (b('dg0_v_vv') <= 1.616377353668213) ?  0.004584178328514099 :   0.04858417809009552 :   (b('l8b4') <= 766.5037841796875) ?  (b('moss') <= 22.444746017456055) ?  (b('bulk') <= 136.0) ?  (b('l8b11') <= 1269.2608642578125) ?  (b('dg0_v_vv') <= 2.676393508911133) ?  (b('dg0_v_vv') <= -2.403744697570801) ?  -0.08861582428216934 :   -0.019322767517425947 :   (b('l8b4_med') <= 807.5645141601562) ?  -0.10574915632605553 :   -0.06769582256674767 :   (b('ndvi_med') <= 3361.5) ?  (b('l8b11') <= 1589.6637573242188) ?  -0.015241909448219381 :   0.028869892869676863 :   (b('l8b11') <= 1414.5289916992188) ?  0.012521672993898392 :   0.04896542523056269 :   (b('l8b4') <= 728.4005737304688) ?  (b('l8b5') <= 3851.60302734375) ?  (b('l8b5') <= 2696.0621337890625) ?  0.08608417585492134 :   -0.02185793024928946 :   (b('l8b5') <= 4466.429443359375) ?  0.08518417626619339 :   -0.012165822088718414 :   (b('dg0_v_vv') <= 1.3361997604370117) ?  (b('l8b5_med') <= 3120.56201171875) ?  0.13558418055375418 :   0.1645841747522354 :   (b('l8b4_med') <= 858.581787109375) ?  0.040584176778793335 :   0.0775841772556305 :   (b('l8b11') <= 1422.6717529296875) ?  (b('ndvi') <= 3466.5) ?  (b('l8b5_med') <= 3126.525146484375) ?  (b('l8b11_med') <= 1640.6630859375) ?  0.07958418130874634 :   0.05558417737483978 :   0.14158417284488678 :   (b('ndvi_med') <= 3554.25) ?  (b('l8b11') <= 1334.2658081054688) ?  0.13108417640129724 :   0.17158417403697968 :   (b('l8b4') <= 576.7095642089844) ?  0.19658416509628296 :   0.15845083693663278 :   (b('dg0_v_vv') <= -0.7684183120727539) ?  0.03958417475223541 :   0.0005841702222824097 :   (b('grass') <= 27.670451164245605) ?  (b('ndvi') <= 3210.0) ?  (b('dg0_v_vv') <= 0.5549821853637695) ?  (b('ndvi_med') <= 3352.75) ?  (b('ndvi') <= 1824.0) ?  -0.011415823362767696 :   -0.05774513314124839 :   (b('l8b5') <= 2746.50537109375) ?  0.0006610994728711935 :   -0.04141582248525487 :   (b('l8b11') <= 1347.9110107421875) ?  0.19258417189121246 :   (b('dg0_v_vv') <= 0.6067638397216797) ?  0.13158418238162994 :   -0.017922965543610708 :   (b('ndvi') <= 3287.0) ?  (b('moss') <= 24.7581205368042) ?  (b('ndvi_med') <= 3728.0) ?  0.009184175729751587 :   -0.024415820837020874 :   (b('l8b5') <= 3172.7989501953125) ?  0.19558417797088623 :   0.12008418142795563 :   (b('l8b5') <= 2699.52880859375) ?  (b('l8b5_med') <= 3104.2471923828125) ?  0.07958417385816574 :   0.03858418017625809 :   (b('l8b11') <= 2562.07373046875) ?  -0.016460063064702646 :   0.05375084529320399 :   (b('l8b11') <= 1774.69091796875) ?  (b('l8b11') <= 1753.0735473632812) ?  (b('dg0_v_vv') <= -1.9930438995361328) ?  (b('ndvi') <= 3162.5) ?  -0.08741582185029984 :   -0.12381582371890545 :   (b('l8b4') <= 807.423828125) ?  0.021484177220951427 :   -0.03502646183713953 :   (b('ndvi') <= 2955.5) ?  0.0605841726064682 :   (b('grass') <= 31.046818733215332) ?  0.11558417975902557 :   0.1505841761827469 :   (b('ndvi') <= 3617.5) ?  (b('crops') <= 38.579952239990234) ?  (b('l8b4_med') <= 896.9377746582031) ?  -0.09797296646450247 :   -0.07207371779766522 :   0.014584183692932129 :   (b('l8b4') <= 839.7635192871094) ?  (b('bulk') <= 116.5) ?  -0.10491582378745079 :   -0.07481582462787628 :   (b('ndvi') <= 4507.0) ?  -0.05609659320459916 :   -0.028708130694352664 :   (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 934.8659973144531) ?  (b('sand') <= 52.5) ?  (b('ndvi') <= 3695.0) ?  (b('l8b5_med') <= 3200.444580078125) ?  (b('l8b4') <= 441.41575622558594) ?  -0.0514158234000206 :   -0.04941582679748535 :   0.004584178328514099 :   (b('l8b4') <= 480.2507781982422) ?  (b('l8b4_med') <= 571.4405822753906) ?  (b('l8b5') <= 4649.17626953125) ?  (b('l8b4') <= 434.85899353027344) ?  0.08913417160511017 :   0.09158417582511902 :   0.11058418452739716 :   (b('l8b11_med') <= 1331.1044311523438) ?  (b('l8b5_med') <= 3047.50732421875) ?  0.19158418476581573 :   0.2075841873884201 :   (b('crops') <= 6.565573692321777) ?  0.1685841828584671 :   0.16358418762683868 :   (b('ndvi') <= 4414.0) ?  -0.021415822207927704 :   0.0035841763019561768 :   (b('l8b5') <= 1797.7119140625) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('ndvi_med') <= 3709.25) ?  0.33058418333530426 :   (b('dg0_v_vv') <= 1.2517390251159668) ?  (b('dg0_v_vv') <= 0.5880928039550781) ?  0.34258417785167694 :   0.3415841907262802 :   0.3405841737985611 :   (b('ndvi') <= 3953.0) ?  (b('moss') <= 7.54746855981648) ?  (b('dg0_v_vv') <= -0.23074960708618164) ?  0.26458418865998584 :   0.16908417890469232 :   (b('l8b11') <= 607.8457183837891) ?  0.1450841873884201 :   0.09025084475676219 :   (b('dg0_v_vv') <= 0.1938924789428711) ?  (b('dg0_v_vv') <= -0.211181640625) ?  0.0995841771364212 :   0.09558416903018951 :   (b('ndvi_med') <= 3709.25) ?  0.11508417129516602 :   0.1335841864347458 :   (b('l8b11') <= 802.6393737792969) ?  (b('dg0_v_vv') <= -0.4921112060546875) ?  (b('dg0_v_vv') <= -0.9455699920654297) ?  (b('dg0_v_vv') <= -1.787829875946045) ?  0.32658417522907257 :   0.33725084364414215 :   (b('dg0_v_vv') <= -0.5724129676818848) ?  0.3055841773748398 :   0.28758417069911957 :   (b('dg0_v_vv') <= 0.7278780937194824) ?  (b('ndvi') <= 3441.5) ?  0.34358416497707367 :   0.34891750911871594 :   (b('dg0_v_vv') <= 0.9823694229125977) ?  0.31658418476581573 :   0.33725085357824963 :   (b('dg0_v_vv') <= 0.0034008026123046875) ?  (b('dg0_v_vv') <= -1.44035005569458) ?  0.31258417665958405 :   (b('ndvi') <= 3765.5) ?  0.24595917388796806 :   0.1955841730038325 :   (b('dg0_v_vv') <= 0.4103517532348633) ?  (b('ndvi_med') <= 3450.0) ?  0.2515841871500015 :   0.3352508395910263 :   (b('ndvi') <= 2714.5) ?  0.2530841827392578 :   0.28686988992350443 :   (b('ndvi') <= 2661.0) ?  (b('sand') <= 58.5) ?  (b('l8b4') <= 873.3092651367188) ?  (b('bulk') <= 149.5) ?  (b('ndvi_med') <= 3680.0) ?  0.0035841763019561768 :   (b('bulk') <= 119.5) ?  -0.06341582536697388 :   -0.046415820717811584 :   (b('l8b4') <= 764.3272094726562) ?  0.04158417880535126 :   (b('dg0_v_vv') <= 0.054651737213134766) ?  0.07858417928218842 :   0.11058418452739716 :   (b('l8b11') <= 2043.0220947265625) ?  (b('dg0_v_vv') <= 1.7473945617675781) ?  (b('l8b11') <= 1792.0466918945312) ?  0.16040235622362656 :   0.08403872495347803 :   0.27358417212963104 :   (b('moss') <= 3.978449583053589) ?  (b('l8b5') <= 1697.7632446289062) ?  0.11458416283130646 :   0.23658417803900583 :   (b('ndvi') <= 1984.5) ?  0.19158418476581573 :   0.07208417356014252 :   (b('ndvi') <= 2597.0) ?  (b('ndvi') <= 2454.0) ?  (b('ndvi') <= 1964.0) ?  0.34758417308330536 :   (b('l8b5') <= 2344.6524658203125) ?  0.19058417528867722 :   0.23970917239785194 :   (b('l8b11') <= 1288.9684448242188) ?  0.20058418810367584 :   (b('l8b5_med') <= 2418.6700439453125) ?  0.336384180188179 :   0.2885841776927312 :   (b('l8b11') <= 1653.3384399414062) ?  (b('l8b11_med') <= 1198.8316650390625) ?  0.17058418691158295 :   0.13158418238162994 :   (b('ndvi_med') <= 3638.5) ?  (b('ndvi_med') <= 3447.5) ?  0.20708416402339935 :   0.22008417546749115 :   0.19758416712284088 :   (b('sand') <= 29.5) ?  (b('dg0_v_vv') <= 0.5034685134887695) ?  (b('sand') <= 18.0) ?  (b('l8b5') <= 2996.560302734375) ?  (b('l8b5') <= 2418.3916015625) ?  0.13258416950702667 :   0.0010841754930359976 :   (b('l8b4') <= 618.0915222167969) ?  0.049784180521965024 :   0.13858417868614198 :   (b('l8b4') <= 755.2341918945312) ?  (b('ndvi') <= 4802.0) ?  0.09619287433831589 :   0.18783417344093323 :   (b('dg0_v_vv') <= -1.289034366607666) ?  0.06791751086711884 :   0.2758698910474777 :   (b('ndvi') <= 3992.5) ?  (b('sand') <= 22.5) ?  (b('ndvi') <= 3102.5) ?  0.24758417904376984 :   0.06941750893990199 :   (b('l8b5') <= 2772.425537109375) ?  0.23083417862653732 :   0.2783341780304909 :   (b('dg0_v_vv') <= 2.0485310554504395) ?  (b('dg0_v_vv') <= 0.795811653137207) ?  0.3195841759443283 :   0.24841750661532083 :   (b('l8b4') <= 637.4120178222656) ?  0.331584170460701 :   0.33058418333530426 :   (b('dg0_v_vv') <= 0.1345682144165039) ?  (b('l8b11_med') <= 1198.8316650390625) ?  (b('l8b5') <= 2195.5146484375) ?  (b('dg0_v_vv') <= -0.09767913818359375) ?  0.11841751138369243 :   0.2545841783285141 :   (b('l8b11') <= 994.1633605957031) ?  0.17903417348861694 :   0.03613973284761111 :   (b('l8b5_med') <= 2867.99609375) ?  (b('dg0_v_vv') <= -1.0425753593444824) ?  -0.03749582260847092 :   0.007965128957515671 :   (b('l8b4') <= 583.8327026367188) ?  0.1505841761827469 :   0.03372121340146771 :   (b('l8b11_med') <= 1213.9009399414062) ?  (b('l8b5') <= 2431.5576171875) ?  (b('l8b11') <= 1203.1015625) ?  0.07708417624235153 :   0.226384174823761 :   (b('l8b11') <= 958.7872924804688) ?  0.23458416759967804 :   0.0807127473609788 :   (b('l8b4_med') <= 684.3815612792969) ?  (b('moss') <= 3.352852553129196) ?  0.07225084553162257 :   -0.030615824460983276 :   (b('l8b5') <= 3842.0255126953125) ?  0.07761703282594681 :   0.0015841759741306305 :   (b('l8b4_med') <= 636.951416015625) ?  (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b11') <= 1043.1750183105469) ?  (b('sand') <= 52.0) ?  (b('l8b4_med') <= 481.6563720703125) ?  (b('dg0_v_vv') <= 0.39547252655029297) ?  0.40508417785167694 :   0.30563417077064514 :   (b('l8b4_med') <= 490.6155700683594) ?  0.2780841737985611 :   0.29112703246729715 :   (b('grass') <= 59.51921463012695) ?  0.3526841849088669 :   0.34428416192531586 :   (b('l8b4') <= 383.7972869873047) ?  (b('dg0_v_vv') <= 0.4457864761352539) ?  0.4078841656446457 :   (b('grass') <= 53.71771430969238) ?  0.3729174981514613 :   0.38248418271541595 :   (b('crops') <= 25.986759185791016) ?  (b('ndvi') <= 4605.5) ?  0.3467270242316382 :   0.3577841967344284 :   (b('ndvi') <= 4376.0) ?  0.33124667778611183 :   0.3424716778099537 :   (b('l8b4') <= 491.7077331542969) ?  (b('grass') <= 59.88348388671875) ?  (b('grass') <= 54.61428642272949) ?  (b('ndvi') <= 3515.5) ?  0.44452416598796846 :   0.4292842000722885 :   (b('dg0_v_vv') <= -0.02643299102783203) ?  0.45381751159826916 :   0.44872861272758907 :   (b('dg0_v_vv') <= 1.5840044021606445) ?  (b('l8b4_med') <= 550.9379577636719) ?  0.38270916044712067 :   0.3671341687440872 :   0.402084156870842 :   (b('ndvi') <= 4492.0) ?  (b('l8b11_med') <= 1261.8388061523438) ?  (b('ndvi') <= 3879.5) ?  0.31098417937755585 :   0.35020918399095535 :   (b('dg0_v_vv') <= 1.4414210319519043) ?  0.36336988849299295 :   0.34088416397571564 :   (b('dg0_v_vv') <= 0.3932075500488281) ?  0.36968420445919037 :   (b('dg0_v_vv') <= 1.1399455070495605) ?  0.38593415915966034 :   0.3951842039823532 :   (b('l8b4_med') <= 585.992919921875) ?  (b('l8b4') <= 651.3680419921875) ?  (b('l8b11') <= 1454.7259521484375) ?  (b('bulk') <= 118.5) ?  (b('l8b5') <= 3385.19384765625) ?  0.2780341793383871 :   0.3160341680049896 :   (b('dg0_v_vv') <= 0.20115900039672852) ?  0.24521751701831818 :   0.25678418576717377 :   (b('dg0_v_vv') <= 0.3657493591308594) ?  (b('dg0_v_vv') <= -0.7485017776489258) ?  0.31738416850566864 :   0.30785084267457324 :   0.3496841639280319 :   (b('ndvi') <= 3319.5) ?  (b('dg0_v_vv') <= 0.540687084197998) ?  (b('l8b5') <= 2527.9896240234375) ?  0.2708841748535633 :   0.25766195522414315 :   (b('ndvi') <= 2853.5) ?  0.28575918823480606 :   0.26621751487255096 :   (b('l8b5') <= 2631.7169189453125) ?  (b('l8b5_med') <= 3028.1737060546875) ?  0.2523341774940491 :   0.2602841804424922 :   (b('dg0_v_vv') <= -0.632087230682373) ?  0.2395341694355011 :   0.23513417690992355 :   (b('dg0_v_vv') <= 0.012537002563476562) ?  (b('bulk') <= 123.5) ?  (b('dg0_v_vv') <= -0.4067220687866211) ?  (b('dg0_v_vv') <= -0.4750986099243164) ?  0.3114841878414154 :   0.2976841777563095 :   0.3334841877222061 :   (b('dg0_v_vv') <= 0.003139495849609375) ?  (b('dg0_v_vv') <= -0.0038509368896484375) ?  0.2911841720342636 :   0.24788416922092438 :   0.30698417127132416 :   (b('l8b11_med') <= 1261.8388061523438) ?  (b('crops') <= 27.774168014526367) ?  0.348884180188179 :   0.34998418390750885 :   0.3455841690301895 :   (b('moss') <= 5.497239112854004) ?  (b('dg0_v_vv') <= -0.29027223587036133) ?  (b('ndvi_med') <= 3684.75) ?  (b('l8b11') <= 1539.7706298828125) ?  (b('ndvi') <= 4379.0) ?  (b('ndvi') <= 4062.0) ?  0.056584179401397705 :   0.10583417490124702 :   (b('ndvi') <= 4596.5) ?  -0.0024158209562301636 :   0.027584180235862732 :   (b('l8b11_med') <= 1360.0833740234375) ?  0.13658417761325836 :   0.21458418667316437 :   (b('crops') <= 34.81739616394043) ?  (b('l8b4') <= 864.5421142578125) ?  -0.019415825605392456 :   -0.05841582268476486 :   (b('ndvi') <= 3011.5) ?  -0.03141582012176514 :   (b('dg0_v_vv') <= -2.1459336280822754) ?  0.011584177613258362 :   0.024917508165041607 :   (b('dg0_v_vv') <= 2.717564105987549) ?  (b('l8b5') <= 3386.87744140625) ?  (b('l8b4') <= 1073.6870727539062) ?  (b('l8b5_med') <= 3189.756591796875) ?  0.09368417561054229 :   0.1663841873407364 :   (b('l8b5') <= 3106.275634765625) ?  0.22683417797088623 :   0.06758417189121246 :   (b('ndvi') <= 4698.5) ?  (b('l8b11') <= 1312.8660888671875) ?  -0.0019158273935317993 :   0.03358417749404907 :   0.08858416974544525 :   (b('dg0_v_vv') <= 3.5686707496643066) ?  (b('ndvi') <= 3826.5) ?  0.284584179520607 :   (b('l8b4') <= 505.7168426513672) ?  0.24858416616916656 :   0.2595841735601425 :   0.12758417427539825 :   (b('dg0_v_vv') <= -2.049837112426758) ?  (b('l8b5') <= 2462.5084228515625) ?  (b('l8b4') <= 687.8755187988281) ?  (b('l8b4') <= 630.7738952636719) ?  (b('l8b11') <= 1286.971923828125) ?  -0.04761582240462303 :   -0.019782488544782 :   (b('ndvi_med') <= 3405.0) ?  0.05498418211936951 :   0.014750838279724121 :   0.32468418776988983 :   (b('ndvi') <= 3455.5) ?  (b('dg0_v_vv') <= -2.8641695976257324) ?  0.2147841900587082 :   0.21198417246341705 :   (b('ndvi') <= 3820.0) ?  0.28018416464328766 :   0.2709841877222061 :   (b('l8b5') <= 2483.6024169921875) ?  (b('l8b11') <= 1403.2102661132812) ?  (b('l8b11') <= 820.2210845947266) ?  (b('dg0_v_vv') <= -1.6547951698303223) ?  0.2424841672182083 :   0.338884174823761 :   (b('l8b4') <= 641.6177062988281) ?  0.021217510104179382 :   0.16226312831828468 :   (b('dg0_v_vv') <= 0.6892366409301758) ?  (b('l8b5') <= 2303.83642578125) ?  0.3504841774702072 :   0.28047417402267455 :   (b('ndvi_med') <= 3405.0) ?  0.16708417236804962 :   0.24341751635074615 :   (b('l8b11') <= 1285.9315795898438) ?  (b('l8b5') <= 3505.7174072265625) ?  (b('ndvi') <= 3822.5) ?  0.34133417904376984 :   0.31569845974445343 :   (b('ndvi') <= 4151.5) ?  0.28018416464328766 :   0.24788416922092438 :   (b('dg0_v_vv') <= -0.05058145523071289) ?  (b('l8b11') <= 1458.2535400390625) ?  0.2563841789960861 :   0.21708418428897858 :   (b('l8b4') <= 715.2341613769531) ?  0.28838416934013367 :   0.3050508499145508 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 3272.0) ?  (b('l8b11_med') <= 2649.172119140625) ?  (b('dg0_v_vv') <= 0.8454086780548096) ?  (b('l8b5_med') <= 2986.78076171875) ?  (b('l8b4') <= 612.5449829101562) ?  (b('l8b5_med') <= 2891.0401611328125) ?  (b('grass') <= 68.00817108154297) ?  (b('l8b4_med') <= 592.331787109375) ?  (b('l8b5') <= 2658.5054931640625) ?  (b('l8b11') <= 993.251708984375) ?  0.12585234915848587 :   0.033114115737852536 :   (b('l8b4') <= 391.063720703125) ?  0.03429960319055961 :   0.16341405678465057 :   (b('sand') <= 26.0) ?  (b('l8b11') <= 1193.3560791015625) ?  0.12208075761795045 :   0.27051907648642853 :   (b('l8b4_med') <= 715.5871276855469) ?  0.044461929398782074 :   0.0037318481691964766 :   (b('l8b4_med') <= 869.0109252929688) ?  (b('l8b5') <= 1682.4984130859375) ?  (b('l8b5') <= 1081.7807006835938) ?  0.04704155188642051 :   -0.04313904309498245 :   (b('l8b5') <= 3107.2479248046875) ?  0.07392262684666476 :   -0.034700395080905694 :   (b('l8b11_med') <= 2367.9005126953125) ?  (b('ndvi_med') <= 1547.75) ?  0.027730760077635447 :   0.16713842208686283 :   (b('l8b5') <= 2942.9573974609375) ?  0.02888898105692056 :   0.024158517144427738 :   (b('l8b4_med') <= 870.3021850585938) ?  (b('l8b11') <= 1030.0537109375) ?  (b('ndvi_med') <= 2491.5) ?  -0.013500903869668651 :   (b('l8b5_med') <= 2924.678955078125) ?  -0.08150090756515663 :   -0.09919969468123534 :   (b('l8b5') <= 3093.8304443359375) ?  (b('dg0_v_vv') <= 0.1530160903930664) ?  -0.004617228134498963 :   0.030479009132590384 :   (b('l8b11') <= 1208.8089599609375) ?  -0.06916772213458158 :   -0.028490475139447635 :   (b('l8b4') <= 394.69044494628906) ?  (b('dg0_v_vv') <= -0.46768999099731445) ?  0.040625765919685364 :   (b('l8b11') <= 567.0040435791016) ?  0.10462574660778046 :   0.055967171550399125 :   (b('grass') <= 15.36157488822937) ?  (b('l8b5_med') <= 2900.7904052734375) ?  -0.054274238914251316 :   0.015940348993280253 :   -0.056966545368329846 :   (b('crops') <= 75.65563201904297) ?  (b('ndvi') <= 1895.5) ?  (b('moss') <= 25.712456703186035) ?  (b('grass') <= 98.81764602661133) ?  (b('l8b5') <= 2037.701416015625) ?  -0.024527464378030578 :   -0.04816682954939657 :   (b('l8b5_med') <= 2662.191650390625) ?  -0.013476540221280711 :   0.07351488667406593 :   (b('grass') <= 42.69499397277832) ?  (b('ndvi') <= 1193.0) ?  -0.10955759428639976 :   -0.08173386717483203 :   (b('grass') <= 70.5883903503418) ?  -0.05287864339293999 :   -0.11299583864699608 :   (b('bulk') <= 138.5) ?  (b('moss') <= 6.610314607620239) ?  (b('l8b5') <= 3513.08984375) ?  -0.025504550929998416 :   0.017857647872988523 :   (b('bulk') <= 128.0) ?  -0.08089466566711437 :   -0.04253694237698346 :   (b('l8b11_med') <= 1967.9754028320312) ?  (b('sand') <= 33.0) ?  0.03565097966322478 :   -0.041495810381377465 :   (b('ndvi_med') <= 2973.25) ?  0.01055326855370599 :   0.13313232654423018 :   (b('bulk') <= 144.5) ?  (b('moss') <= 0.0533333346247673) ?  (b('l8b4_med') <= 742.3873596191406) ?  (b('bulk') <= 134.5) ?  0.0755156349509693 :   -0.032418716169501594 :   (b('l8b4') <= 1284.8598022460938) ?  0.19220741556198767 :   0.07759013231843709 :   (b('ndvi_med') <= 2577.0) ?  (b('dg0_v_vv') <= -0.20123577117919922) ?  0.045524450757723565 :   0.09590258488581052 :   (b('ndvi') <= 2925.0) ?  0.0036194073018753904 :   0.02748427191924046 :   (b('ndvi_med') <= 2542.0) ?  (b('l8b11') <= 1480.7471313476562) ?  (b('l8b5') <= 3389.13134765625) ?  0.045124644736448914 :   -0.042482141758608594 :   (b('l8b5') <= 2745.4835205078125) ?  -0.01888068991113043 :   -0.04519042419991496 :   (b('l8b11') <= 1128.1316528320312) ?  (b('dg0_v_vv') <= -0.5046505928039551) ?  0.14624426260590553 :   0.09399425978461902 :   (b('dg0_v_vv') <= -0.7285594940185547) ?  0.0075100445189396955 :   0.03556584598122889 :   (b('grass') <= 51.88082504272461) ?  (b('sand') <= 53.0) ?  (b('moss') <= 6.5066306591033936) ?  (b('ndvi_med') <= 2413.5) ?  (b('l8b11') <= 1748.8951416015625) ?  (b('l8b11') <= 1578.0667114257812) ?  0.0508353906445877 :   0.13852789896070916 :   (b('grass') <= 13.808833599090576) ?  0.016084580140632083 :   0.0636707918489061 :   (b('l8b4') <= 537.3472290039062) ?  (b('grass') <= 22.02735996246338) ?  0.254495761361387 :   0.31852575689554213 :   (b('l8b11') <= 2202.234619140625) ?  0.08470289391532883 :   0.17492288757799104 :   (b('dg0_v_vv') <= 0.5267102718353271) ?  (b('l8b4') <= 850.4696960449219) ?  (b('l8b4_med') <= 958.5111694335938) ?  -0.04564192736664595 :   0.04149858502652309 :   (b('l8b4') <= 929.7593383789062) ?  0.07485471211626674 :   0.006669677741053447 :   (b('l8b4') <= 1243.68212890625) ?  (b('l8b4') <= 1027.7614135742188) ?  -0.08470344184339046 :   -0.09974267713861032 :   (b('l8b11') <= 2366.544677734375) ?  -0.03663541472545176 :   0.004884926602244377 :   (b('l8b11_med') <= 2427.9947509765625) ?  (b('l8b11') <= 2080.4179077148438) ?  (b('l8b11') <= 2001.1566162109375) ?  (b('dg0_v_vv') <= 0.36762142181396484) ?  0.05175206894373118 :   0.0009382537566125365 :   (b('l8b11_med') <= 2393.7314453125) ?  0.1479151461273432 :   0.09491512812674047 :   (b('l8b4') <= 2335.6044921875) ?  (b('dg0_v_vv') <= -1.851346492767334) ?  0.05616245240897691 :   -0.016625429052982987 :   (b('l8b11') <= 2887.4873046875) ?  0.1047338793625651 :   -0.012837545862488392 :   (b('dg0_v_vv') <= -0.9660854339599609) ?  (b('l8b4') <= 1945.1007690429688) ?  (b('l8b4') <= 1451.638671875) ?  -0.035649325598413086 :   -0.08094907221530569 :   (b('l8b11') <= 3525.5) ?  -0.11848515017635443 :   -0.08739181701771237 :   (b('l8b4') <= 1735.4385986328125) ?  (b('l8b5') <= 2234.296875) ?  -0.027485421484762976 :   0.04358052021741034 :   (b('l8b5') <= 3757.8028564453125) ?  -0.0259685029627579 :   -0.1023918169928771 :   (b('l8b11') <= 1701.4208984375) ?  (b('dg0_v_vv') <= -0.2618403434753418) ?  (b('dg0_v_vv') <= -1.7041149139404297) ?  (b('l8b4_med') <= 601.5855407714844) ?  0.22166074812412262 :   0.14623908350865045 :   (b('l8b11') <= 1441.1064453125) ?  (b('l8b11') <= 1297.4808349609375) ?  0.31624787521476927 :   0.3416597361998776 :   (b('l8b4') <= 809.4548034667969) ?  0.2103480671078731 :   0.3067074851723961 :   (b('ndvi') <= 3106.0) ?  (b('l8b11') <= 1399.63037109375) ?  (b('ndvi') <= 2217.5) ?  0.05356421310168047 :   0.0977093472599219 :   (b('ndvi') <= 2366.5) ?  0.1795642151282384 :   0.1675642206118657 :   (b('dg0_v_vv') <= -0.023397445678710938) ?  (b('l8b5') <= 3581.7354736328125) ?  0.19757576435804367 :   0.21556422847967882 :   (b('l8b5') <= 3068.7623291015625) ?  0.24621575325727463 :   0.30331805986471666 :   (b('ndvi') <= 2056.5) ?  0.09556422371130724 :   (b('ndvi_med') <= 3065.5) ?  -0.007180299971139781 :   -0.02714924025349319 :   (b('sand') <= 28.0) ?  (b('l8b11_med') <= 1600.824462890625) ?  (b('crops') <= 6.198555946350098) ?  (b('l8b5') <= 2288.2791748046875) ?  (b('l8b5') <= 2050.68310546875) ?  0.20249430641531943 :   (b('l8b5') <= 2158.1939697265625) ?  (b('dg0_v_vv') <= 1.5481233596801758) ?  0.05256422597628374 :   0.000494299381971347 :   (b('l8b11_med') <= 1555.286865234375) ?  0.11449430689215659 :   0.09056422847967882 :   (b('dg0_v_vv') <= 2.9672021865844727) ?  (b('l8b5') <= 3015.627685546875) ?  (b('l8b5') <= 2349.2899169921875) ?  0.1626719111815477 :   0.24573206879864298 :   (b('l8b4_med') <= 810.0767517089844) ?  0.13049430951476096 :   0.0763857564330101 :   0.021494297236204135 :   (b('ndvi_med') <= 1933.0) ?  (b('l8b4') <= 701.7617797851562) ?  0.3020257607102394 :   0.3608257845044136 :   (b('ndvi') <= 4534.5) ?  (b('l8b4') <= 798.2325134277344) ?  (b('l8b5') <= 4517.8819580078125) ?  0.19859575333694615 :   0.25567575792471564 :   (b('l8b11') <= 1633.0470581054688) ?  0.24522043140605093 :   0.2743057599663734 :   (b('dg0_v_vv') <= 1.2419123649597168) ?  0.29557575434446337 :   0.32457576841115954 :   (b('l8b4_med') <= 945.9946594238281) ?  (b('ndvi') <= 2168.5) ?  (b('l8b11_med') <= 1668.19970703125) ?  (b('l8b11_med') <= 1661.7429809570312) ?  -0.002524758147245093 :   0.015174967588649863 :   (b('l8b4_med') <= 927.91552734375) ?  (b('l8b4_med') <= 868.1021118164062) ?  -0.01564389062093134 :   -0.02382503694130314 :   -0.03550569906830789 :   (b('dg0_v_vv') <= 4.199320316314697) ?  (b('l8b5') <= 2136.9558715820312) ?  (b('dg0_v_vv') <= 2.188020706176758) ?  0.11449430689215659 :   0.12249429330229758 :   (b('l8b4') <= 417.1304016113281) ?  0.10317495966123211 :   0.03828038198973032 :   0.15117496007846462 :   (b('l8b5') <= 3845.001708984375) ?  (b('l8b11') <= 1906.120361328125) ?  (b('l8b5_med') <= 2011.1242065429688) ?  (b('l8b4') <= 848.4983520507812) ?  0.16158054578877643 :   0.055705576710048166 :   (b('l8b11_med') <= 2003.1835327148438) ?  0.2078298754066159 :   0.17413807893454514 :   (b('l8b5_med') <= 3042.8221435546875) ?  (b('ndvi_med') <= 2474.0) ?  0.08580945676310327 :   0.1810581865161657 :   (b('ndvi') <= 2607.5) ?  0.181125770136714 :   0.24948344809504655 :   (b('l8b4') <= 1090.1581726074219) ?  (b('l8b5') <= 4961.71435546875) ?  (b('l8b5') <= 4132.183837890625) ?  0.009174970330463522 :   0.012174961508976095 :   (b('dg0_v_vv') <= 2.04666805267334) ?  0.02920536063197586 :   0.03750830147829321 :   (b('l8b4') <= 1223.6895141601562) ?  (b('dg0_v_vv') <= 1.9226315021514893) ?  0.08327495807574856 :   0.06795496398376095 :   (b('ndvi_med') <= 2107.5) ?  0.06677496189044582 :   0.14077496284412014 :   (b('crops') <= 78.23976135253906) ?  (b('grass') <= 76.3312759399414) ?  (b('l8b4_med') <= 1902.1754760742188) ?  (b('l8b5') <= 2016.1123657226562) ?  (b('moss') <= 26.208369255065918) ?  (b('moss') <= 20.914034843444824) ?  0.03968833476208672 :   0.116435991247833 :   (b('l8b4_med') <= 1329.599853515625) ?  -0.020996774844462128 :   0.10309581271693809 :   (b('moss') <= 6.230724096298218) ?  (b('moss') <= 3.171052575111389) ?  -3.4523734976111094e-05 :   0.06641471184893576 :   (b('l8b4') <= 1653.511962890625) ?  -0.01150919412296394 :   -0.07458379446328137 :   (b('ndvi') <= 1196.5) ?  (b('l8b5_med') <= 2595.2958984375) ?  (b('dg0_v_vv') <= 1.0258691310882568) ?  -0.10819689727431976 :   -0.12119689754254068 :   (b('l8b5_med') <= 2639.315185546875) ?  0.08845268258681666 :   0.012603003202149498 :   (b('l8b5') <= 3397.4146728515625) ?  (b('crops') <= 37.5662841796875) ?  0.02522576302289964 :   0.1837632568744131 :   (b('l8b11') <= 2248.0804443359375) ?  0.14688554220570202 :   0.04619773694283358 :   (b('l8b11_med') <= 1861.5925903320312) ?  (b('bulk') <= 149.0) ?  (b('l8b4_med') <= 1106.645263671875) ?  (b('ndvi_med') <= 2675.0) ?  -0.04512599639646577 :   0.009932825841243768 :   (b('dg0_v_vv') <= 2.1735920906066895) ?  0.04846186795168453 :   0.13446187832289272 :   (b('l8b4') <= 484.64031982421875) ?  (b('ndvi_med') <= 1734.0) ?  0.21826971026532577 :   0.20671592389769627 :   (b('l8b5') <= 2758.48193359375) ?  0.13752964867351503 :   0.025583445531562027 :   (b('l8b4') <= 892.8446350097656) ?  (b('l8b4') <= 648.1883850097656) ?  (b('l8b4') <= 496.4841766357422) ?  0.10220945529082298 :   0.2297046616104376 :   (b('grass') <= 80.0257339477539) ?  0.07838988564596124 :   0.1373919122906764 :   (b('dg0_v_vv') <= 3.001939296722412) ?  (b('sand') <= 41.5) ?  0.02529349320464217 :   0.08264326867325689 :   (b('l8b11') <= 2478.0625) ?  0.09904728574582507 :   0.19881377737379757 :   (b('l8b11_med') <= 1713.6244506835938) ?  (b('l8b4_med') <= 627.6808776855469) ?  (b('l8b4') <= 362.009765625) ?  (b('l8b11_med') <= 1191.3309936523438) ?  0.01895075719803571 :   (b('dg0_v_vv') <= 2.282353401184082) ?  -0.03704924453049899 :   -0.0460492404177785 :   (b('ndvi') <= 4215.5) ?  -0.06804924029856921 :   (b('ndvi_med') <= 2910.0) ?  -0.05845845150516221 :   -0.059458453531720135 :   (b('l8b4_med') <= 1027.3583679199219) ?  (b('dg0_v_vv') <= 0.9435324668884277) ?  (b('l8b4') <= 830.059814453125) ?  0.09423575272162754 :   0.13812878556974362 :   (b('ndvi') <= 1882.0) ?  -0.004048998950808139 :   0.05427816002311372 :   (b('l8b5') <= 3479.94677734375) ?  (b('l8b5') <= 2249.3121948242188) ?  -0.007479303648074476 :   0.029004212718631257 :   (b('dg0_v_vv') <= 2.05704402923584) ?  -0.012429312934478132 :   0.00822069389621416 :   (b('l8b4') <= 1255.9391479492188) ?  (b('crops') <= 89.0081558227539) ?  (b('dg0_v_vv') <= 3.0514883995056152) ?  (b('l8b5') <= 3977.1026611328125) ?  0.18157820638923855 :   0.24221372707039024 :   (b('l8b5_med') <= 2604.223876953125) ?  0.22326576203107834 :   0.03109949962533884 :   (b('l8b11') <= 1746.7815551757812) ?  (b('ndvi') <= 4844.0) ?  0.10699793189901302 :   0.06013098802997006 :   (b('l8b11') <= 1839.5359497070312) ?  0.15975949723876878 :   0.03972576531767846 :   (b('crops') <= 94.0407485961914) ?  (b('l8b11') <= 3156.31884765625) ?  (b('dg0_v_vv') <= 3.5670738220214844) ?  0.05305439798826302 :   0.1058361628136079 :   (b('ndvi_med') <= 2029.0) ?  0.16295752523138243 :   0.1887257441878319 :   (b('l8b5') <= 2539.575927734375) ?  (b('l8b5_med') <= 2088.3817138671875) ?  0.20567576475441457 :   0.1725924238562584 :   0.13097813218122437 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b11') <= 2630.368896484375) ?  (b('l8b4') <= 1297.9138793945312) ?  (b('dg0_v_vv') <= 3.451791286468506) ?  (b('ndvi') <= 1725.5) ?  (b('l8b4') <= 1115.95703125) ?  (b('dg0_v_vv') <= -0.8303613662719727) ?  (b('dg0_v_vv') <= -1.2997679710388184) ?  0.07695794115774333 :   0.0669579506944865 :   (b('dg0_v_vv') <= 3.1780474185943604) ?  0.09097180485964204 :   0.11746185877256923 :   (b('dg0_v_vv') <= 3.0029404163360596) ?  (b('dg0_v_vv') <= 1.5664966106414795) ?  0.04856788398463348 :   0.05418897719453958 :   0.08818897159170297 :   (b('l8b4') <= 1010.9631958007812) ?  (b('l8b4') <= 848.1984252929688) ?  (b('dg0_v_vv') <= -0.34941625595092773) ?  0.06962147269242754 :   0.10801013698801398 :   (b('dg0_v_vv') <= -0.4813389778137207) ?  0.06595794866792858 :   0.042459911945689884 :   (b('l8b11') <= 1818.9609375) ?  (b('ndvi_med') <= 1481.0) ?  -0.03009424477815628 :   -0.026989868376404047 :   (b('ndvi_med') <= 1481.5) ?  0.03701012721285224 :   0.03246186532908016 :   (b('ndvi') <= 1239.0) ?  0.17346188285284572 :   (b('l8b4') <= 977.6329345703125) ?  (b('l8b4_med') <= 1656.4095458984375) ?  0.1074618683093124 :   (b('l8b4') <= 882.14111328125) ?  0.1204618797534042 :   0.1254618749850326 :   0.07046186783247524 :   (b('l8b11') <= 2535.6141357421875) ?  (b('bulk') <= 149.0) ?  (b('dg0_v_vv') <= -0.3138742446899414) ?  -0.06971424356102943 :   -0.06558193322271108 :   (b('l8b5') <= 3561.4498291015625) ?  (b('l8b5') <= 2498.8812255859375) ?  (b('l8b5_med') <= 2515.5849609375) ?  0.06740909578899543 :   0.011046177680039616 :   (b('moss') <= 6.368875980377197) ?  0.11542185278633585 :   0.04158687210745283 :   (b('dg0_v_vv') <= -1.092806339263916) ?  (b('dg0_v_vv') <= -1.4408788681030273) ?  -0.021557575246940047 :   -0.02555757590259114 :   (b('dg0_v_vv') <= -0.960057258605957) ?  0.027762430384755135 :   -0.008351885210375333 :   (b('l8b11') <= 2581.6141357421875) ?  (b('l8b5_med') <= 3622.8421630859375) ?  (b('ndvi') <= 1306.5) ?  0.09388939189639958 :   0.0924424248126646 :   0.11195433751813003 :   (b('ndvi') <= 1754.5) ?  (b('dg0_v_vv') <= -1.5566277503967285) ?  (b('dg0_v_vv') <= -1.8346643447875977) ?  0.069979088952144 :   0.052312421252330155 :   (b('l8b5_med') <= 2812.056640625) ?  0.04997909312446913 :   0.026498284259378307 :   (b('dg0_v_vv') <= 0.8722143173217773) ?  (b('l8b11_med') <= 2890.52392578125) ?  0.09601667069576003 :   0.10501665913245894 :   (b('l8b4_med') <= 2271.3209228515625) ?  0.08301667415282943 :   0.08401666127822616 :   (b('ndvi') <= 1012.5) ?  (b('dg0_v_vv') <= 1.6424832344055176) ?  (b('ndvi_med') <= 864.5) ?  (b('ndvi') <= 910.5) ?  (b('ndvi') <= 626.0) ?  (b('l8b4') <= 2660.458984375) ?  -0.03307068239105803 :   -0.07247744158804417 :   (b('l8b5') <= 2987.8594970703125) ?  -0.06336540435591426 :   -0.04568770019128919 :   0.014418072439730173 :   (b('l8b5') <= 1982.8384399414062) ?  0.029984093892077596 :   (b('l8b11_med') <= 2956.887451171875) ?  (b('l8b4_med') <= 2012.2193603515625) ?  -0.04682386369608781 :   -0.06703155199513416 :   (b('l8b11') <= 4560.7978515625) ?  -0.08171416044348945 :   -0.0348092397209257 :   (b('ndvi_med') <= 889.5) ?  -0.04242423838004469 :   (b('l8b11') <= 3737.9757080078125) ?  0.07090715162975844 :   0.020878694316043583 :   (b('dg0_v_vv') <= 2.3334012031555176) ?  (b('ndvi') <= 2266.5) ?  (b('l8b5') <= 2910.119873046875) ?  (b('l8b5') <= 2637.677734375) ?  (b('l8b11') <= 2895.7447509765625) ?  0.02046516827392307 :   -0.025240931555084823 :   (b('l8b11') <= 2951.8863525390625) ?  -0.0051823128469196825 :   0.08839823734456684 :   (b('l8b11') <= 2700.478759765625) ?  (b('dg0_v_vv') <= -1.433309555053711) ?  -0.045500909462571154 :   0.01608459440618753 :   (b('ndvi') <= 1738.0) ?  -0.02794273216796537 :   -0.0014354053049970693 :   (b('l8b11') <= 2912.2064208984375) ?  (b('dg0_v_vv') <= 0.5113711357116699) ?  (b('dg0_v_vv') <= -2.321016311645508) ?  -0.041186739597469574 :   -0.02418673867359758 :   (b('l8b5') <= 3613.2978515625) ?  0.00988940047946843 :   -0.002186742518097168 :   (b('l8b11') <= 2952.25341796875) ?  (b('l8b5') <= 3489.388427734375) ?  0.06802575588226317 :   0.09920909032225608 :   0.01281325807794928 :   (b('l8b11_med') <= 2880.1024169921875) ?  (b('l8b5') <= 2449.6160888671875) ?  0.12668940885500474 :   0.09668939276175065 :   (b('dg0_v_vv') <= 3.7491743564605713) ?  (b('l8b11') <= 3087.64990234375) ?  0.08745908886194229 :   (b('ndvi_med') <= 1405.5) ?  0.04751666926524856 :   0.034689401225610206 :   (b('ndvi') <= 2193.5) ?  -0.05098332918503068 :   0.04507576227188109 :   (b('grass') <= 27.41930866241455) ?  (b('l8b4_med') <= 2187.779052734375) ?  (b('ndvi_med') <= 2106.5) ?  (b('l8b4_med') <= 2066.49853515625) ?  (b('l8b5') <= 2172.4903564453125) ?  (b('crops') <= 91.17935943603516) ?  (b('dg0_v_vv') <= 0.9863948822021484) ?  -0.025061068258110542 :   -0.000630725646479529 :   (b('l8b5_med') <= 3221.940673828125) ?  -0.09847424253821374 :   -0.09547424018383027 :   (b('dg0_v_vv') <= 1.7019867897033691) ?  (b('crops') <= 46.23920249938965) ?  -0.0721675000958475 :   -0.0921810873713549 :   (b('l8b4_med') <= 1784.3443603515625) ?  -0.10218464351818832 :   -0.045590788152799426 :   (b('l8b11_med') <= 3847.0084228515625) ?  (b('dg0_v_vv') <= 1.3677406311035156) ?  (b('l8b5') <= 3211.02294921875) ?  -0.1026854105409571 :   -0.11908091346037077 :   (b('crops') <= 50.06903839111328) ?  -0.11061683864021815 :   -0.08775775252371522 :   (b('l8b11') <= 3694.02978515625) ?  0.05722576081752778 :   (b('dg0_v_vv') <= -0.15767717361450195) ?  -0.010630797422150284 :   -0.038302047638813175 :   (b('dg0_v_vv') <= -0.926490306854248) ?  (b('l8b5') <= 2621.3082275390625) ?  (b('l8b11') <= 3183.2554931640625) ?  -0.13363896348875845 :   -0.13203896341276253 :   (b('l8b5') <= 3437.588134765625) ?  (b('l8b4') <= 1866.7098388671875) ?  0.03159323085536375 :   0.02062150169232929 :   -0.005274244397878658 :   (b('l8b11') <= 2079.6779174804688) ?  (b('l8b4') <= 900.1136779785156) ?  (b('l8b4') <= 671.4519958496094) ?  0.050136214418506075 :   0.06644053672120642 :   (b('l8b5') <= 3235.8082275390625) ?  -0.015311022090204685 :   -0.027094244460264844 :   (b('ndvi') <= 2671.0) ?  (b('l8b4') <= 1522.8529663085938) ?  0.14577371689542784 :   0.09427738266100211 :   (b('l8b5') <= 3505.9490966796875) ?  0.04371957375975341 :   0.09702576100826263 :   (b('l8b5') <= 2994.7655029296875) ?  (b('dg0_v_vv') <= 0.855919361114502) ?  (b('l8b5') <= 2529.6197509765625) ?  (b('grass') <= 21.033963203430176) ?  (b('l8b5_med') <= 3080.0504150390625) ?  0.01448131369219885 :   0.03213269628356305 :   -0.007422376615949439 :   (b('l8b11_med') <= 2960.3900146484375) ?  (b('ndvi_med') <= 762.5) ?  0.02663270003865567 :   0.004632700157864961 :   (b('l8b11') <= 2775.182373046875) ?  -0.07142868897501305 :   -0.040465217126585594 :   (b('l8b11') <= 3156.8052978515625) ?  (b('bulk') <= 153.5) ?  (b('l8b5_med') <= 3095.292724609375) ?  0.08695433155766555 :   0.06290275923286875 :   0.030075755839546525 :   (b('l8b5') <= 2846.27294921875) ?  0.04282575845718384 :   (b('moss') <= 8.797333002090454) ?  -0.02937424331903457 :   -0.024366136378533132 :   (b('grass') <= 21.365646362304688) ?  (b('dg0_v_vv') <= 0.6242423057556152) ?  (b('ndvi') <= 997.0) ?  (b('l8b5_med') <= 3519.1146240234375) ?  -0.05697958714508699 :   -0.03566822936770656 :   (b('l8b11') <= 3988.4112548828125) ?  -0.057096291512057175 :   -0.07039299094106705 :   (b('dg0_v_vv') <= 3.1303610801696777) ?  (b('ndvi_med') <= 1166.0) ?  -0.027946413403217147 :   -0.05333484153317761 :   (b('dg0_v_vv') <= 7.187874794006348) ?  0.0008657591417431832 :   -0.05487423837184907 :   (b('l8b5') <= 4320.913818359375) ?  (b('dg0_v_vv') <= 1.2214851379394531) ?  (b('dg0_v_vv') <= -0.01578807830810547) ?  -0.06921948440434694 :   -0.08382241126328306 :   (b('l8b11') <= 3013.23974609375) ?  0.016775763779878605 :   -0.04515467717211813 :   (b('dg0_v_vv') <= 0.3528175354003906) ?  (b('l8b5_med') <= 3708.87744140625) ?  -0.00817424207925796 :   -0.028174241632223124 :   0.011825753748416906 :   (b('dg0_v_vv') <= 1.686361312866211) ?  (b('ndvi_med') <= 1758.5) ?  (b('ndvi_med') <= 1343.5) ?  (b('l8b5_med') <= 3248.876953125) ?  (b('l8b11_med') <= 2908.095458984375) ?  (b('dg0_v_vv') <= 1.3158907890319824) ?  -0.0571838519567165 :   0.009300533943885081 :   (b('sand') <= 47.5) ?  -0.04553007267133185 :   -0.0827507611038925 :   0.20452255352437496 :   (b('ndvi_med') <= 1478.0) ?  (b('crops') <= 41.09989643096924) ?  (b('dg0_v_vv') <= 1.013615608215332) ?  -0.0353440822646223 :   0.04604140208246964 :   (b('l8b5') <= 2530.572265625) ?  0.08757251181714579 :   0.027051781689716285 :   (b('l8b11') <= 2663.07470703125) ?  (b('l8b11_med') <= 2865.619384765625) ?  -0.030054251123676812 :   0.01746323524950673 :   (b('ndvi') <= 1211.5) ?  0.02513329239596647 :   -0.05976259977771847 :   (b('ndvi') <= 2103.5) ?  (b('bulk') <= 146.5) ?  (b('l8b4_med') <= 1483.1878051757812) ?  (b('ndvi') <= 1901.0) ?  0.1163201644504496 :   0.04292150665143574 :   (b('l8b11') <= 3455.5108642578125) ?  -0.08013972779989698 :   0.13092150617459858 :   (b('dg0_v_vv') <= 1.4727296829223633) ?  (b('grass') <= 61.469181060791016) ?  -0.09011812516644889 :   -0.10939219599750574 :   (b('ndvi_med') <= 1831.5) ?  -0.010795536050917922 :   -0.10103535287910037 :   (b('l8b5_med') <= 2719.6295166015625) ?  (b('l8b5') <= 2751.84814453125) ?  (b('l8b11_med') <= 2660.5242919921875) ?  -0.0751798324799112 :   -0.0905388670179893 :   (b('l8b11') <= 2697.935546875) ?  -0.12704237954755282 :   -0.11404237927933192 :   (b('dg0_v_vv') <= 0.3999643325805664) ?  (b('ndvi') <= 2253.0) ?  -0.02399023980656195 :   -0.06739079753050911 :   (b('l8b4_med') <= 1889.65869140625) ?  0.026447359341199572 :   -0.016622228348835505 :   (b('moss') <= 13.497345924377441) ?  (b('moss') <= 7.676438808441162) ?  (b('ndvi') <= 3288.5) ?  (b('ndvi') <= 2937.5) ?  (b('l8b5') <= 2270.446044921875) ?  -0.04573271051794945 :   0.026177740632123914 :   0.18889576345682144 :   (b('l8b11') <= 1592.2201538085938) ?  -0.10100757678349814 :   (b('dg0_v_vv') <= 2.309558153152466) ?  -0.0681549109518528 :   -0.061825031994117624 :   (b('ndvi_med') <= 1470.0) ?  (b('dg0_v_vv') <= 2.242757797241211) ?  (b('moss') <= 10.17537784576416) ?  -0.04415490701794625 :   -0.05136613149095226 :   (b('l8b11') <= 2760.7073974609375) ?  0.008741340365309222 :   0.08026316435629542 :   (b('dg0_v_vv') <= 4.077725887298584) ?  (b('l8b5') <= 2970.21630859375) ?  0.0930568504369871 :   0.030097834990073596 :   (b('ndvi_med') <= 1688.5) ?  0.13045242537151683 :   0.24846187093191677 :   (b('l8b4_med') <= 1397.789794921875) ?  0.1504618809454971 :   (b('l8b4_med') <= 1537.77783203125) ?  (b('ndvi') <= 1364.0) ?  -0.1270353524842196 :   (b('dg0_v_vv') <= 4.0383806228637695) ?  -0.06536868471238348 :   -0.11203535281949573 :   (b('l8b11_med') <= 3099.6298828125) ?  (b('dg0_v_vv') <= 3.88191556930542) ?  -0.033672826147798186 :   0.011932901284169588 :   (b('l8b5') <= 2887.3997802734375) ?  0.076882572167299 :   0.004575755043576167 :   (b('grass') <= 46.14527893066406) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('bulk') <= 140.5) ?  (b('crops') <= 16.61120891571045) ?  (b('l8b4') <= 522.7384033203125) ?  (b('l8b5') <= 3832.9178466796875) ?  (b('ndvi') <= 2863.5) ?  (b('dg0_v_vv') <= 0.03835725784301758) ?  (b('ndvi_med') <= 4009.0) ?  0.036454810734306065 :   0.02705480062535831 :   0.08257576056889127 :   (b('l8b4') <= 457.01271057128906) ?  (b('l8b11') <= 984.209716796875) ?  -0.046636210674568246 :   -0.004013350732291933 :   (b('ndvi') <= 3522.0) ?  -0.014686462837788794 :   -0.07813151735765063 :   (b('ndvi') <= 4380.5) ?  (b('dg0_v_vv') <= 1.4444115161895752) ?  (b('l8b5') <= 4189.426025390625) ?  0.05362504599728022 :   0.07015481127046086 :   (b('l8b5_med') <= 2913.7916259765625) ?  0.06612826134436406 :   0.01572825755350865 :   (b('l8b11') <= 1007.6720886230469) ?  (b('grass') <= 33.51909637451172) ?  0.0055186117029920045 :   -0.022328170082931004 :   (b('dg0_v_vv') <= -0.3186488151550293) ?  -0.05548138525717111 :   -0.046345186404957994 :   (b('l8b4') <= 528.7445678710938) ?  (b('l8b4') <= 524.9793395996094) ?  0.01585480474999973 :   (b('dg0_v_vv') <= -0.38351964950561523) ?  (b('l8b4_med') <= 447.916259765625) ?  0.17425481879285404 :   0.162218611869885 :   (b('dg0_v_vv') <= -0.05716538429260254) ?  0.14451862517364172 :   0.15135900674101252 :   (b('l8b5_med') <= 2316.7889404296875) ?  (b('l8b11') <= 1329.8439331054688) ?  (b('ndvi') <= 3347.5) ?  0.18312576040625572 :   0.2223135406192806 :   0.11560769215706856 :   (b('moss') <= 7.815729141235352) ?  (b('l8b11') <= 1363.1504516601562) ?  0.04458634678149306 :   -0.0044074946833795765 :   (b('l8b5') <= 2381.392578125) ?  -0.0028569593571356844 :   -0.04857796339394088 :   (b('dg0_v_vv') <= -0.5962433815002441) ?  (b('ndvi_med') <= 4409.5) ?  (b('l8b5_med') <= 2510.6680908203125) ?  (b('crops') <= 56.30350303649902) ?  -0.028667150954490017 :   (b('dg0_v_vv') <= -0.636446475982666) ?  -0.07625048448153844 :   -0.03266715161014111 :   (b('l8b11_med') <= 1383.3305053710938) ?  (b('l8b4') <= 797.0771179199219) ?  0.033392078700226945 :   -0.00022166151565224736 :   (b('l8b4') <= 410.17127990722656) ?  0.31033283569406955 :   0.06129608081017391 :   (b('l8b5') <= 3078.330078125) ?  (b('dg0_v_vv') <= -0.8427643775939941) ?  (b('l8b5') <= 2993.352783203125) ?  0.25535909781853355 :   0.13990909258524578 :   (b('dg0_v_vv') <= -0.6678624153137207) ?  0.263825752834479 :   0.2988257492582003 :   (b('l8b11') <= 1157.4540405273438) ?  (b('l8b5_med') <= 3583.33154296875) ?  0.26949242800474166 :   0.27283284165453403 :   (b('l8b4') <= 630.9979553222656) ?  0.06747169748414307 :   0.0018085716059431378 :   (b('ndvi_med') <= 4059.0) ?  (b('moss') <= 11.788260459899902) ?  (b('ndvi') <= 1849.5) ?  (b('l8b11_med') <= 987.3240661621094) ?  0.24868976324796677 :   0.12014923604051349 :   (b('l8b4') <= 702.9306030273438) ?  0.06157643095010555 :   0.029905928697743934 :   (b('ndvi') <= 3595.5) ?  (b('ndvi') <= 3081.0) ?  0.025514892749855123 :   0.08389300034847243 :   (b('l8b4') <= 521.0) ?  0.12571850650421176 :   0.16848798079523397 :   (b('sand') <= 69.5) ?  (b('dg0_v_vv') <= 1.8423500061035156) ?  (b('l8b4') <= 1215.1187744140625) ?  0.1050854851803939 :   0.1992577721787189 :   (b('l8b4_med') <= 485.56373596191406) ?  0.00865481702230908 :   0.24115220120191871 :   (b('ndvi') <= 3556.0) ?  (b('l8b11') <= 1623.923828125) ?  0.09928305529246084 :   0.17768975347280502 :   (b('ndvi') <= 3751.0) ?  0.020741415420064434 :   0.05211060314481365 :   (b('sand') <= 24.0) ?  (b('l8b5') <= 3486.4771728515625) ?  (b('crops') <= 40.39838218688965) ?  (b('dg0_v_vv') <= 0.7361893653869629) ?  (b('ndvi_med') <= 4107.0) ?  (b('ndvi') <= 4381.5) ?  0.13456782575466147 :   0.05906783559794278 :   (b('l8b5_med') <= 3314.0533447265625) ?  0.03848010370847152 :   -0.024874241650104517 :   (b('l8b5') <= 2790.249267578125) ?  (b('dg0_v_vv') <= 1.129014492034912) ?  0.1728518350214898 :   0.09935183508109444 :   (b('l8b5') <= 3439.7222900390625) ?  0.20539096558018577 :   0.2888518316836297 :   (b('dg0_v_vv') <= -1.4781737327575684) ?  (b('l8b5_med') <= 2894.9593505859375) ?  0.02972575873136521 :   (b('crops') <= 45.261009216308594) ?  0.14142577201128007 :   0.08647575974464417 :   (b('l8b5') <= 3032.6806640625) ?  (b('l8b4') <= 1313.6944580078125) ?  0.1579375215955809 :   0.2929257690906525 :   (b('l8b11') <= 1084.3497314453125) ?  0.08859241902828216 :   0.24594217931046902 :   (b('l8b4') <= 474.3256072998047) ?  (b('l8b4') <= 420.0584716796875) ?  (b('ndvi_med') <= 3984.5) ?  0.05128382747211763 :   0.06128381793537446 :   (b('l8b5') <= 4300.952880859375) ?  (b('ndvi') <= 4126.0) ?  0.19902047709541223 :   0.14591546556806265 :   0.2721257612109184 :   (b('ndvi') <= 4003.0) ?  (b('l8b4') <= 516.0157318115234) ?  (b('l8b11') <= 1092.5159912109375) ?  0.04570576429367065 :   0.05411998681150949 :   (b('l8b11') <= 1257.048095703125) ?  0.21042402366143892 :   0.1319934339675761 :   (b('ndvi_med') <= 3717.0) ?  (b('ndvi') <= 4221.0) ?  0.046440754979848856 :   0.10647460381572063 :   (b('l8b11_med') <= 1231.8487548828125) ?  0.04252111634582741 :   -0.018674238771200183 :   (b('dg0_v_vv') <= 0.708594799041748) ?  (b('l8b11_med') <= 1364.9776611328125) ?  (b('dg0_v_vv') <= 0.6068377494812012) ?  (b('l8b11') <= 1094.1755981445312) ?  (b('ndvi') <= 4654.0) ?  0.0164325258464335 :   0.17110837704461554 :   (b('l8b5') <= 2635.011962890625) ?  0.1311617385919168 :   0.07855451972414891 :   (b('ndvi') <= 3952.5) ?  0.06455481333278157 :   -0.14934518652416728 :   (b('l8b5') <= 4023.05810546875) ?  (b('moss') <= 0.9812286496162415) ?  (b('dg0_v_vv') <= -1.4813494682312012) ?  -0.017474237829446793 :   -0.08464537020179615 :   -0.014891627366128174 :   (b('ndvi') <= 4385.5) ?  (b('crops') <= 64.6768798828125) ?  0.02452576532959938 :   -0.0038916348763134156 :   (b('bulk') <= 149.5) ?  0.05810837156098822 :   0.08652575686573982 :   (b('l8b5') <= 3915.0411376953125) ?  (b('ndvi') <= 3339.0) ?  (b('ndvi') <= 2072.0) ?  (b('l8b4') <= 670.8307495117188) ?  0.12831352893677023 :   0.07831353191700247 :   (b('ndvi') <= 3083.0) ?  0.18617682454701126 :   0.1324841629808605 :   (b('dg0_v_vv') <= 0.9552335739135742) ?  (b('dg0_v_vv') <= 0.9009885787963867) ?  0.13320772617940022 :   0.19765481633685567 :   (b('dg0_v_vv') <= 1.1147232055664062) ?  -0.03534518646456264 :   0.07097434116744664 :   (b('l8b5') <= 4752.34716796875) ?  (b('ndvi') <= 4951.0) ?  (b('grass') <= 22.16900062561035) ?  0.1816548137142513 :   0.19323277111377632 :   0.23965481204532124 :   0.2562327646764747 :   (b('moss') <= 6.192884922027588) ?  (b('bulk') <= 154.0) ?  (b('dg0_v_vv') <= -0.6801788806915283) ?  (b('crops') <= 41.514814376831055) ?  (b('ndvi') <= 4030.0) ?  (b('dg0_v_vv') <= -1.8294243812561035) ?  (b('l8b4') <= 970.627197265625) ?  0.07613826151937247 :   0.07424576371908187 :   (b('ndvi') <= 3100.5) ?  0.09271450905129314 :   0.11650221479125322 :   (b('dg0_v_vv') <= -0.8321223258972168) ?  -0.05958091109991073 :   0.015419099330902103 :   (b('dg0_v_vv') <= -2.9685163497924805) ?  (b('ndvi') <= 3471.0) ?  (b('l8b4') <= 875.6398620605469) ?  -0.0418992444872856 :   -0.08166872212155299 :   (b('dg0_v_vv') <= -4.5499982833862305) ?  0.054285759553313245 :   -0.005392987933009859 :   (b('ndvi') <= 4904.0) ?  (b('ndvi') <= 4807.5) ?  0.013801545630525176 :   -0.06849979500803682 :   (b('sand') <= 51.0) ?  0.12105353329744604 :   0.034847770083902614 :   (b('grass') <= 39.1515007019043) ?  (b('ndvi') <= 2714.0) ?  (b('l8b5_med') <= 2506.6820068359375) ?  (b('l8b5_med') <= 2170.44140625) ?  0.24531623196034205 :   0.18092575622539897 :   (b('dg0_v_vv') <= -0.4066448211669922) ?  0.01631622981457484 :   0.11663861915170037 :   (b('bulk') <= 150.5) ?  (b('ndvi') <= 4481.0) ?  0.07950506242565217 :   0.04134244756002996 :   (b('l8b5') <= 4018.6204833984375) ?  0.14101011441543257 :   -0.01919423729181291 :   (b('l8b4') <= 1013.3258972167969) ?  (b('dg0_v_vv') <= 0.691105842590332) ?  (b('l8b5') <= 2911.115234375) ?  -0.06285202047891088 :   -0.049107575168212264 :   (b('ndvi') <= 4092.5) ?  -0.012107578416665404 :   0.017585756282011682 :   (b('dg0_v_vv') <= -0.213287353515625) ?  0.03648825827986002 :   0.06302575767040253 :   (b('l8b5') <= 3476.2423095703125) ?  (b('l8b5') <= 2883.8438720703125) ?  (b('dg0_v_vv') <= 0.9910783767700195) ?  (b('dg0_v_vv') <= -2.0634984970092773) ?  -0.09688674109056591 :   (b('ndvi') <= 3268.5) ?  -0.07243852455701147 :   -0.08282424183562398 :   0.005375762749463331 :   (b('l8b4') <= 801.9378967285156) ?  (b('ndvi_med') <= 3405.25) ?  -0.07382424222305417 :   (b('ndvi') <= 3917.5) ?  -0.03249924033880233 :   -0.01649924144148826 :   (b('l8b4') <= 1250.1012573242188) ?  (b('ndvi') <= 3520.5) ?  -0.011774017779888311 :   0.019775754213333135 :   (b('ndvi') <= 3554.0) ?  -0.03262423975393175 :   -0.04562424374744295 :   (b('ndvi') <= 4170.0) ?  (b('ndvi') <= 2045.5) ?  (b('dg0_v_vv') <= -3.660003662109375) ?  -0.11607424020767212 :   (b('dg0_v_vv') <= -1.7851157188415527) ?  -0.05507423952221871 :   -0.0060742408037185724 :   (b('ndvi') <= 3157.5) ?  (b('l8b11') <= 2443.2498779296875) ?  -0.10388674037531018 :   -0.11388674201443791 :   (b('l8b5') <= 3636.8193359375) ?  -0.08965043219782058 :   -0.07390281228082521 :   (b('dg0_v_vv') <= 2.0657200813293457) ?  (b('l8b4_med') <= 1058.0714111328125) ?  0.026625752449035645 :   0.004125760495662684 :   -0.027774238586425776 :   (b('l8b4') <= 767.1305541992188) ?  (b('crops') <= 4.350515365600586) ?  (b('dg0_v_vv') <= 0.10925674438476562) ?  (b('dg0_v_vv') <= -0.8822212219238281) ?  (b('l8b4') <= 674.2375793457031) ?  (b('l8b4') <= 576.7095642089844) ?  0.17692574858665466 :   0.13773908366759618 :   0.035625757277011866 :   (b('dg0_v_vv') <= -0.5263495445251465) ?  (b('l8b5') <= 2679.60205078125) ?  0.0005257532000541576 :   0.025739095111687982 :   (b('ndvi') <= 3325.5) ?  -0.003974235057830816 :   0.07120075768066776 :   (b('ndvi') <= 4093.5) ?  (b('ndvi') <= 3441.0) ?  0.2058257445693016 :   (b('l8b11_med') <= 1654.46435546875) ?  0.14256426697377012 :   0.17777576893568037 :   (b('ndvi') <= 4221.5) ?  -0.018574236333370214 :   0.09868762781843543 :   (b('l8b5') <= 1724.2650146484375) ?  (b('l8b5') <= 1565.0074462890625) ?  (b('dg0_v_vv') <= 2.251969814300537) ?  0.11351644840453648 :   0.12351643886779332 :   0.1496876417659223 :   (b('l8b11') <= 1308.7908325195312) ?  (b('grass') <= 25.91626739501953) ?  (b('l8b11') <= 1075.6165771484375) ?  0.02817095411868363 :   -0.016528775357502266 :   (b('ndvi') <= 3558.5) ?  0.0244164526126244 :   -0.037536244835160666 :   (b('l8b5') <= 3686.2098388671875) ?  (b('dg0_v_vv') <= 1.3320093154907227) ?  -0.009960197389407007 :   0.0344390973337425 :   (b('l8b5') <= 3759.8682861328125) ?  0.11834242583976855 :   0.032112509662277744 :   (b('crops') <= 10.455060958862305) ?  (b('ndvi') <= 3856.0) ?  (b('l8b5') <= 2369.5615234375) ?  (b('l8b5') <= 2000.9340209960938) ?  -0.09491317560698123 :   (b('l8b11') <= 1743.5806274414062) ?  -0.08391317566658589 :   -0.08020845007053333 :   (b('l8b5') <= 2577.77392578125) ?  (b('l8b5_med') <= 2818.5750732421875) ?  -0.06491317441488834 :   0.104025761783123 :   (b('l8b11') <= 2623.1478271484375) ?  -0.06202476131257596 :   -0.07931956144986535 :   (b('bulk') <= 122.5) ?  (b('ndvi') <= 4840.0) ?  (b('ndvi_med') <= 4085.5) ?  -0.08791317632223697 :   -0.10991317620302768 :   -0.05391317447449298 :   (b('ndvi') <= 4583.5) ?  (b('l8b5') <= 3120.8621826171875) ?  -0.042319668462851755 :   -0.02413835616363182 :   (b('dg0_v_vv') <= -0.09417390823364258) ?  -0.04441317757393451 :   0.07508682245586781 :   (b('l8b4') <= 1116.8915405273438) ?  (b('moss') <= 17.72283124923706) ?  (b('ndvi') <= 3634.0) ?  (b('sand') <= 38.5) ?  -0.026032987420297797 :   -0.08305532868611339 :   (b('l8b4') <= 778.67822265625) ?  -0.04989481734687007 :   0.0013953596433531144 :   (b('dg0_v_vv') <= 0.08629894256591797) ?  (b('ndvi') <= 3067.5) ?  -0.067931985390829 :   0.04631581931692006 :   (b('dg0_v_vv') <= 1.3189353942871094) ?  0.06565073181679904 :   0.14815087488719395 :   (b('crops') <= 23.818787574768066) ?  (b('dg0_v_vv') <= 1.6960699558258057) ?  (b('l8b11') <= 2048.0275268554688) ?  -0.08969150325749535 :   -0.04196172595751069 :   (b('l8b11') <= 2414.3704833984375) ?  0.06800333111791568 :   -0.03226981888168967 :   (b('dg0_v_vv') <= -2.638676643371582) ?  (b('ndvi') <= 2909.5) ?  -0.01920845497301535 :   -0.1015710606293372 :   (b('ndvi') <= 3617.5) ?  -0.026491910349404846 :   0.002015104255949801 :   (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 958.7872924804688) ?  (b('sand') <= 52.5) ?  (b('dg0_v_vv') <= -0.2442154884338379) ?  (b('l8b11') <= 890.7442626953125) ?  0.011725763976573933 :   -0.044474244117736816 :   (b('ndvi') <= 4550.0) ?  (b('ndvi') <= 3930.0) ?  0.08077075630426406 :   (b('l8b11_med') <= 1264.818359375) ?  0.1927257500588894 :   (b('l8b11') <= 780.6007080078125) ?  0.14522576481103897 :   0.15372575707733632 :   (b('dg0_v_vv') <= 0.6559228897094727) ?  (b('l8b5_med') <= 3095.4654541015625) ?  (b('grass') <= 57.94600296020508) ?  0.003225758671760559 :   0.022345321236745166 :   (b('dg0_v_vv') <= 0.28925418853759766) ?  0.07967075258493422 :   0.06482575982809066 :   (b('dg0_v_vv') <= 1.6469488143920898) ?  0.1372257485985756 :   0.1472257688641548 :   (b('l8b5') <= 1878.2954711914062) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('ndvi') <= 3923.5) ?  (b('dg0_v_vv') <= 0.4310593605041504) ?  0.31752574592828753 :   (b('ndvi_med') <= 3709.25) ?  0.29752576500177386 :   0.306525756418705 :   0.2695257678627968 :   (b('l8b5') <= 1748.1115112304688) ?  (b('l8b5') <= 1591.57275390625) ?  (b('l8b5') <= 1479.5771484375) ?  0.1254090890288353 :   0.08255909134944281 :   (b('ndvi') <= 3677.0) ?  0.18248485121311564 :   0.09355075433850288 :   -0.11014090627431869 :   (b('l8b11') <= 802.6393737792969) ?  (b('ndvi') <= 4025.5) ?  (b('ndvi') <= 3696.5) ?  (b('ndvi_med') <= 3709.25) ?  0.29123409738143285 :   0.307859106361866 :   (b('dg0_v_vv') <= 0.3092193603515625) ?  0.26029242525498075 :   0.29185910274585086 :   (b('dg0_v_vv') <= -0.5529718399047852) ?  0.30285908132791517 :   (b('dg0_v_vv') <= 0.6683902740478516) ?  0.3146924262245496 :   0.3158590917785962 :   (b('dg0_v_vv') <= 1.0994482040405273) ?  (b('dg0_v_vv') <= -0.7820858955383301) ?  (b('dg0_v_vv') <= -1.387646198272705) ?  0.28132575899362566 :   0.19525700627515713 :   (b('ndvi_med') <= 3625.75) ?  0.2336296880564519 :   0.2677104292038296 :   (b('ndvi_med') <= 3671.5) ?  (b('bulk') <= 125.5) ?  0.27789717550788606 :   0.2742757469415665 :   (b('dg0_v_vv') <= 1.8667182922363281) ?  0.24912576824426652 :   0.26812574714422227 :   (b('ndvi') <= 2671.0) ?  (b('l8b5_med') <= 2779.658935546875) ?  (b('ndvi_med') <= 3960.5) ?  (b('dg0_v_vv') <= -3.400446891784668) ?  0.005180309171026409 :   (b('ndvi') <= 2554.0) ?  (b('ndvi') <= 2494.0) ?  0.21227689301682678 :   0.2759617937089701 :   (b('l8b4') <= 834.1351928710938) ?  0.14675075709819796 :   0.20138712287551228 :   (b('l8b5_med') <= 2441.1961669921875) ?  -0.057074242830276484 :   (b('l8b4') <= 809.7182006835938) ?  0.03722575753927232 :   0.06737575381994249 :   (b('grass') <= 57.94600296020508) ?  (b('dg0_v_vv') <= 1.6822466850280762) ?  (b('dg0_v_vv') <= 1.3969101905822754) ?  (b('ndvi') <= 2456.0) ?  0.12754394127563998 :   0.13118029629642314 :   0.18137574642896653 :   0.24622575491666793 :   (b('l8b5') <= 2611.140380859375) ?  (b('ndvi_med') <= 4049.0) ?  (b('ndvi') <= 2362.0) ?  0.14916991232056342 :   0.07582576125860215 :   -0.017819692736322224 :   (b('l8b5_med') <= 3095.4654541015625) ?  (b('l8b11_med') <= 1535.0466918945312) ?  0.018384851583025677 :   -0.03029697042974558 :   (b('ndvi') <= 2486.0) ?  0.06911121098252086 :   0.01954394592480227 :   (b('sand') <= 29.5) ?  (b('dg0_v_vv') <= -1.0708322525024414) ?  (b('ndvi') <= 4075.5) ?  (b('l8b11') <= 1959.6427612304688) ?  (b('dg0_v_vv') <= -1.2870864868164062) ?  -0.00911237485186191 :   0.03847575517637389 :   0.10132574588060378 :   (b('ndvi') <= 4402.5) ?  (b('bulk') <= 148.5) ?  0.09272575855255127 :   0.1839649013851 :   (b('l8b5') <= 2854.75341796875) ?  0.1427924379706383 :   0.04138074292112952 :   (b('dg0_v_vv') <= 0.5034685134887695) ?  (b('l8b4') <= 817.1256103515625) ?  (b('ndvi') <= 3318.5) ?  0.015471410035901929 :   0.1313208389760544 :   (b('dg0_v_vv') <= -0.833714485168457) ?  0.17866147458553314 :   0.26024719327688217 :   (b('l8b5') <= 2788.14404296875) ?  (b('dg0_v_vv') <= 1.280632495880127) ?  0.2392146492169963 :   0.11902242829402288 :   (b('l8b11') <= 1209.3847045898438) ?  0.14233687130941286 :   0.24024520450168182 :   (b('dg0_v_vv') <= 0.3038792610168457) ?  (b('l8b4_med') <= 468.779296875) ?  (b('l8b4') <= 592.5939025878906) ?  (b('ndvi') <= 4000.5) ?  0.06629456979629798 :   0.015236869826912868 :   (b('dg0_v_vv') <= -0.0711827278137207) ?  0.11799242397149404 :   0.28307391384409536 :   (b('dg0_v_vv') <= -0.702603816986084) ?  (b('grass') <= 51.157758712768555) ?  0.0556665990075178 :   -0.023489005256758167 :   (b('grass') <= 53.852949142456055) ?  -0.008015418092706108 :   0.05101255560977443 :   (b('l8b5') <= 2431.5576171875) ?  (b('l8b11_med') <= 1251.6475219726562) ?  (b('l8b5') <= 2030.6070556640625) ?  0.08389908681313196 :   0.23361242592334755 :   (b('dg0_v_vv') <= 1.4080986976623535) ?  0.06959690223137538 :   -0.09826588459312915 :   (b('l8b4_med') <= 617.7979125976562) ?  (b('l8b11') <= 1088.39404296875) ?  0.0696461687100177 :   -0.04684274365910056 :   (b('ndvi') <= 3006.0) ?  -0.06344419355193774 :   0.0852679358444823 :   (b('l8b4_med') <= 636.951416015625) ?  (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b11') <= 1029.7211608886719) ?  (b('dg0_v_vv') <= 0.8429303169250488) ?  (b('bulk') <= 123.0) ?  (b('l8b4') <= 479.28233337402344) ?  0.25916131221585803 :   0.24757575541734694 :   0.19531577080488205 :   (b('dg0_v_vv') <= 1.617750644683838) ?  0.3174157664179802 :   0.29237146015678134 :   (b('crops') <= 29.840476036071777) ?  (b('l8b4') <= 489.6418151855469) ?  (b('l8b11') <= 1167.4552001953125) ?  0.3122087922035939 :   0.298009517416358 :   (b('ndvi') <= 4033.5) ?  0.2698594953864813 :   0.28655949495732785 :   (b('l8b4') <= 394.7136993408203) ?  (b('dg0_v_vv') <= 0.2660188674926758) ?  0.35029576271772384 :   0.3357282549142837 :   (b('grass') <= 52.48655891418457) ?  0.35247144933257785 :   0.3563369881361723 :   (b('l8b11_med') <= 1220.7982788085938) ?  (b('moss') <= 2.254432260990143) ?  (b('ndvi') <= 3465.5) ?  (b('dg0_v_vv') <= -0.858790397644043) ?  0.4105024521549543 :   0.3965750025930228 :   (b('dg0_v_vv') <= 0.0943455696105957) ?  0.40393574883540473 :   0.4112113156252437 :   (b('dg0_v_vv') <= 0.9437251091003418) ?  (b('dg0_v_vv') <= 0.2007613182067871) ?  0.4088068715400166 :   0.40241128587060504 :   0.41141128920846515 :   (b('ndvi') <= 2667.5) ?  (b('moss') <= 0.7690265774726868) ?  0.38377573937177656 :   0.4037317606806755 :   (b('l8b11_med') <= 1261.8388061523438) ?  (b('dg0_v_vv') <= -0.19048500061035156) ?  0.3440632365643978 :   0.29243743121623994 :   (b('l8b5') <= 3052.2410888671875) ?  0.33466218537711484 :   0.36938957948060264 :   (b('l8b11') <= 1640.8772583007812) ?  (b('l8b5_med') <= 3089.992919921875) ?  (b('moss') <= 5.734982252120972) ?  (b('crops') <= 30.077006340026855) ?  (b('l8b4_med') <= 490.6155700683594) ?  0.23101681447573127 :   0.22220201525423264 :   (b('l8b4') <= 634.9986267089844) ?  0.25303631832911855 :   0.24677675425533266 :   (b('ndvi') <= 4735.0) ?  (b('l8b4_med') <= 585.992919921875) ?  0.25548689353303544 :   0.2819057601193587 :   0.18196575492620468 :   (b('moss') <= 1.760066032409668) ?  (b('dg0_v_vv') <= -0.5406818389892578) ?  (b('dg0_v_vv') <= -0.8772726058959961) ?  0.2383807481399604 :   0.23398076306496346 :   (b('dg0_v_vv') <= -0.06157970428466797) ?  0.2794307650199958 :   0.26513076137219155 :   (b('dg0_v_vv') <= -0.5597352981567383) ?  (b('l8b4') <= 640.1963195800781) ?  0.28564575165510175 :   0.27792242815097173 :   (b('ndvi') <= 3194.0) ?  0.3278074209888776 :   0.27825742413600285 :   (b('dg0_v_vv') <= -0.40502166748046875) ?  (b('dg0_v_vv') <= -0.5439906120300293) ?  (b('dg0_v_vv') <= -0.6491532325744629) ?  (b('l8b11') <= 1670.56884765625) ?  0.21153075098991395 :   0.2080179883374108 :   0.21963075399398804 :   0.2055707611143589 :   (b('ndvi') <= 2892.5) ?  (b('dg0_v_vv') <= 0.11155557632446289) ?  (b('dg0_v_vv') <= -0.18300294876098633) ?  0.23491798175705803 :   0.2323179675473107 :   (b('dg0_v_vv') <= 0.6391983032226562) ?  0.2493179870976342 :   0.24760826751589776 :   (b('l8b5') <= 2790.9544677734375) ?  0.21507075801491737 :   0.2119707651436329 :   (b('ndvi') <= 3669.5) ?  (b('ndvi') <= 3555.5) ?  (b('l8b5_med') <= 3137.2218017578125) ?  (b('l8b4') <= 641.6177062988281) ?  (b('ndvi') <= 3323.0) ?  (b('ndvi_med') <= 3442.5) ?  0.04886242300271987 :   0.0653624191880226 :   (b('l8b11') <= 1325.1373291015625) ?  -0.005037580430507671 :   -0.0005642436444759369 :   (b('l8b11') <= 1509.8101806640625) ?  (b('ndvi') <= 3236.0) ?  0.1440871629411714 :   0.20931979867188555 :   (b('dg0_v_vv') <= 0.8834905624389648) ?  0.22757925137877463 :   0.2742291003465652 :   (b('l8b4') <= 1051.2958374023438) ?  (b('l8b11') <= 1723.482666015625) ?  (b('dg0_v_vv') <= 0.07141256332397461) ?  0.010581318206257295 :   0.03022575974464417 :   (b('ndvi') <= 3361.5) ?  -0.05344424035400153 :   -0.0049075717727343204 :   0.10082574635744096 :   (b('l8b5') <= 3096.6072998046875) ?  (b('dg0_v_vv') <= 1.0319414138793945) ?  (b('l8b5') <= 2315.7037353515625) ?  (b('dg0_v_vv') <= -3.8604965209960938) ?  -0.03668757217625777 :   -0.029754241928458214 :   (b('l8b5') <= 2383.83544921875) ?  -0.007649094964328557 :   -0.018633408074577647 :   0.034457871004154816 :   (b('dg0_v_vv') <= -0.2818779945373535) ?  0.12292575985193252 :   0.13994576960802077 :   (b('bulk') <= 136.5) ?  (b('ndvi') <= 3939.0) ?  (b('dg0_v_vv') <= -2.0984559059143066) ?  (b('ndvi') <= 3820.0) ?  0.25216574817895887 :   0.2438857689499855 :   (b('ndvi_med') <= 3405.0) ?  (b('ndvi') <= 3909.5) ?  0.29721242934465414 :   0.25471431761980057 :   (b('ndvi') <= 3820.0) ?  0.3097007587552071 :   0.30520194818576174 :   (b('dg0_v_vv') <= 2.1213083267211914) ?  (b('l8b5') <= 3169.4764404296875) ?  (b('dg0_v_vv') <= -1.268153190612793) ?  0.24426400639116763 :   0.274142083923022 :   (b('dg0_v_vv') <= -0.6557927131652832) ?  0.25216574817895887 :   0.21484575420618057 :   (b('ndvi_med') <= 3457.25) ?  0.15037575513124465 :   0.27417909204959867 :   (b('l8b5_med') <= 3270.988525390625) ?  (b('l8b11') <= 1121.7677612304688) ?  (b('ndvi') <= 4658.5) ?  (b('dg0_v_vv') <= -0.9047336578369141) ?  0.12400075607001781 :   0.007442008014768367 :   0.15072576254606246 :   (b('l8b5') <= 4594.870361328125) ?  (b('ndvi') <= 3998.5) ?  0.027092418571313226 :   0.10608742474172361 :   0.19772575348615645 :   (b('l8b5') <= 3519.4412841796875) ?  (b('l8b4') <= 786.865478515625) ?  (b('l8b5') <= 3084.6798095703125) ?  0.1539457681775093 :   0.16709241916735967 :   (b('l8b11') <= 1625.4749755859375) ?  0.09522575736045838 :   0.08109242369731268 :   (b('l8b5') <= 3664.00634765625) ?  0.19577574580907822 :   (b('l8b5') <= 4119.95654296875) ?  0.241092420121034 :   0.22372574955224991 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1735.126708984375) ?  (b('l8b4_med') <= 658.2301025390625) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 35.5) ?  (b('crops') <= 7.542971134185791) ?  (b('l8b4') <= 740.1128845214844) ?  (b('l8b4') <= 473.8344268798828) ?  (b('dg0_v_vv') <= -0.14986038208007812) ?  (b('ndvi') <= 2892.5) ?  (b('l8b5_med') <= 2163.919677734375) ?  0.11482724566413818 :   0.1361272485966867 :   (b('l8b4') <= 432.7792510986328) ?  0.005413918940101556 :   0.06360781071023136 :   (b('l8b11') <= 844.3566284179688) ?  (b('l8b11') <= 838.6338195800781) ?  0.1476708136511053 :   0.0520824483594111 :   (b('dg0_v_vv') <= 1.0401699542999268) ?  0.1584812201554387 :   0.13670324066143325 :   (b('l8b5') <= 2673.60498046875) ?  (b('l8b11') <= 1051.3290405273438) ?  (b('bulk') <= 139.0) ?  0.006030445944835106 :   0.05873117648480874 :   (b('bulk') <= 144.5) ?  0.06674899813329985 :   0.02301226444668162 :   (b('l8b5') <= 2752.253662109375) ?  (b('l8b11') <= 1121.6900939941406) ?  0.16445821128076749 :   0.19973449753523795 :   (b('l8b5_med') <= 2149.7215576171875) ?  0.007412806870092908 :   0.09608902098095382 :   (b('l8b5') <= 3336.02734375) ?  (b('crops') <= 3.2411575317382812) ?  (b('l8b11') <= 1416.2500610351562) ?  (b('l8b4') <= 869.5136413574219) ?  -0.009674132443295524 :   0.16339515731523752 :   (b('l8b5') <= 2756.0557861328125) ?  -0.06706853968752342 :   -0.00892085400754945 :   0.12316766195757622 :   (b('dg0_v_vv') <= 0.06928825378417969) ?  (b('l8b5_med') <= 2724.859619140625) ?  0.17167915138071044 :   0.1306791427976416 :   0.03267915281122191 :   (b('l8b11_med') <= 1117.17626953125) ?  (b('l8b11') <= 924.6221313476562) ?  (b('crops') <= 97.81589889526367) ?  (b('l8b11') <= 644.5185241699219) ?  0.23517618114087316 :   0.2411761933002207 :   0.29976812402868863 :   (b('crops') <= 74.91983222961426) ?  (b('dg0_v_vv') <= 0.0067043304443359375) ?  (b('sand') <= 34.5) ?  0.17879935697346483 :   0.16932197956810063 :   0.146321977660752 :   (b('l8b11') <= 1242.0262451171875) ?  (b('l8b11') <= 978.7548828125) ?  0.19197617405984135 :   0.20353045887524351 :   (b('l8b11') <= 1498.7840576171875) ?  0.22115372432085373 :   0.22385545970772172 :   (b('l8b5_med') <= 3642.110595703125) ?  (b('dg0_v_vv') <= 0.4282188415527344) ?  (b('l8b11') <= 899.7489624023438) ?  (b('l8b4_med') <= 581.2989196777344) ?  0.11004846854859823 :   0.2591719602900372 :   (b('ndvi') <= 4003.0) ?  0.13008104436137483 :   0.06803759370877778 :   (b('crops') <= 30.905277252197266) ?  (b('l8b4_med') <= 602.2307434082031) ?  0.017842609487707495 :   0.11153349652296557 :   (b('l8b11') <= 1313.5629272460938) ?  0.20973868065787835 :   0.12315814590658664 :   (b('l8b5') <= 4764.901611328125) ?  (b('l8b4') <= 510.8207092285156) ?  (b('sand') <= 19.5) ?  0.19002977397923973 :   0.005325340819604047 :   (b('l8b4') <= 622.5991821289062) ?  0.1162046763304399 :   0.036691781715876184 :   (b('dg0_v_vv') <= 0.6997065544128418) ?  (b('l8b5') <= 5274.403564453125) ?  0.1132138238450186 :   0.14791385230082701 :   (b('ndvi') <= 4570.5) ?  0.2116438518407014 :   0.1593219593025214 :   (b('l8b5_med') <= 2962.950439453125) ?  (b('crops') <= 82.29873275756836) ?  (b('ndvi') <= 4116.0) ?  (b('l8b5') <= 1746.0) ?  (b('l8b4') <= 417.97357177734375) ?  (b('dg0_v_vv') <= 1.6739423274993896) ?  0.09991782946066603 :   0.004151799707936998 :   (b('l8b5_med') <= 1967.9053955078125) ?  -0.06382044249676815 :   0.05614491493523796 :   (b('l8b5_med') <= 1607.8214721679688) ?  (b('l8b4') <= 585.1986389160156) ?  0.18655829909495358 :   -0.011723843934959819 :   (b('l8b5') <= 2473.2083740234375) ?  -0.01623433631740141 :   0.028211699532789486 :   (b('l8b5_med') <= 2940.280029296875) ?  (b('l8b11') <= 702.65771484375) ?  -0.14700635951369143 :   (b('dg0_v_vv') <= -0.06558895111083984) ?  -0.027864434149591426 :   -0.05668176030469312 :   (b('ndvi_med') <= 3972.25) ?  -0.13441066787175054 :   (b('dg0_v_vv') <= 0.7336139678955078) ?  -0.15720063849658217 :   -0.15294262053287852 :   (b('l8b4') <= 567.8264465332031) ?  0.20286653125044285 :   (b('dg0_v_vv') <= 1.7386300563812256) ?  (b('dg0_v_vv') <= -1.0897772312164307) ?  0.16027957237780024 :   (b('ndvi') <= 2945.5) ?  0.11389867363322753 :   0.1322795752388232 :   0.06359287891910267 :   (b('ndvi_med') <= 4430.0) ?  (b('l8b5') <= 2042.3394775390625) ?  (b('l8b5') <= 1475.260986328125) ?  (b('dg0_v_vv') <= 0.18105745315551758) ?  (b('l8b5_med') <= 3401.141357421875) ?  0.033238053237701244 :   0.017657297071545203 :   (b('ndvi') <= 2046.0) ?  0.04997874446197341 :   0.03815589442482853 :   (b('l8b5_med') <= 3047.593017578125) ?  0.2238207869231701 :   (b('dg0_v_vv') <= 4.564294815063477) ?  0.13619259441247675 :   0.10237525293922412 :   (b('moss') <= 6.318181753158569) ?  (b('l8b4') <= 761.0018005371094) ?  (b('l8b5') <= 5076.612548828125) ?  0.03634395039783616 :   -0.01639516305643825 :   (b('ndvi_med') <= 4046.25) ?  -0.04067450856990958 :   0.0703550072100528 :   (b('l8b4') <= 926.9201049804688) ?  (b('ndvi') <= 4812.0) ?  0.05515449740048178 :   0.1782684988305792 :   (b('ndvi') <= 3487.5) ?  0.1585963429325579 :   0.1299058244756422 :   (b('l8b11') <= 1108.5331420898438) ?  (b('dg0_v_vv') <= 0.6208868026733398) ?  (b('bulk') <= 120.5) ?  (b('dg0_v_vv') <= -0.677299976348877) ?  0.24554955748908064 :   0.28288527389730267 :   (b('dg0_v_vv') <= 0.07370328903198242) ?  0.16593799528074202 :   0.08493907656208415 :   (b('ndvi') <= 3813.0) ?  (b('dg0_v_vv') <= 2.080751895904541) ?  0.257767217556618 :   0.27751053547464444 :   0.28991721789040403 :   (b('l8b11') <= 1660.2769775390625) ?  (b('l8b11') <= 1475.9852294921875) ?  (b('ndvi') <= 4942.0) ?  0.02402166875940257 :   0.058364486588927254 :   (b('l8b4') <= 662.5381164550781) ?  0.004450537959717116 :   -0.003753215524166853 :   (b('dg0_v_vv') <= -1.8872909545898438) ?  0.2600898504753908 :   (b('ndvi_med') <= 4697.0) ?  0.15008983617027602 :   0.1580898523827394 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11') <= 939.8609008789062) ?  (b('l8b5') <= 1878.9283447265625) ?  (b('l8b4_med') <= 522.1105041503906) ?  (b('l8b5') <= 1389.6187133789062) ?  (b('ndvi') <= 3923.5) ?  (b('ndvi_med') <= 3533.5) ?  0.2677731885015964 :   0.2763231883943081 :   0.2425731910765171 :   (b('ndvi') <= 2892.5) ?  (b('dg0_v_vv') <= -0.28155994415283203) ?  0.175377271480181 :   0.15398283235578225 :   (b('dg0_v_vv') <= -0.04656791687011719) ?  0.094316363493666 :   0.1450656453181397 :   (b('l8b5') <= 1771.8475341796875) ?  (b('l8b5') <= 1274.049072265625) ?  (b('dg0_v_vv') <= 2.002592086791992) ?  0.0053640099474684955 :   0.016463357552938013 :   (b('ndvi') <= 3614.5) ?  0.08155589296480913 :   0.10953484997153282 :   (b('l8b4_med') <= 646.2068786621094) ?  0.0484046051516496 :   (b('ndvi_med') <= 3447.5) ?  -0.0981268136203289 :   -0.09912681564688683 :   (b('ndvi_med') <= 2794.25) ?  (b('l8b5_med') <= 2280.4501953125) ?  -0.013795389770034666 :   (b('l8b5') <= 2320.5966796875) ?  0.05902723987056671 :   0.033588203068279504 :   (b('bulk') <= 113.5) ?  0.07170367732644081 :   (b('l8b11') <= 881.2326354980469) ?  (b('ndvi') <= 4069.0) ?  0.25491077553422464 :   0.2805031264021706 :   (b('dg0_v_vv') <= -0.006542205810546875) ?  0.17531393225389047 :   0.2249197765707521 :   (b('l8b5') <= 2117.5440673828125) ?  (b('l8b4') <= 693.2637939453125) ?  (b('l8b11_med') <= 1252.2130126953125) ?  (b('l8b4') <= 558.9656677246094) ?  (b('ndvi') <= 3452.5) ?  0.08648585691054661 :   0.061112973190747294 :   (b('sand') <= 52.0) ?  0.03914902091472269 :   0.05089262885784173 :   (b('l8b11') <= 1326.4383544921875) ?  -0.07204116621379214 :   -0.027317341388358063 :   (b('dg0_v_vv') <= -0.9729723930358887) ?  (b('dg0_v_vv') <= -1.2193865776062012) ?  0.22694317668676375 :   (b('dg0_v_vv') <= -1.0978169441223145) ?  0.021943178474903097 :   0.02394318252801894 :   (b('ndvi') <= 2454.0) ?  (b('dg0_v_vv') <= -0.03489971160888672) ?  0.25511875050603694 :   0.1394647330386309 :   (b('ndvi') <= 4615.0) ?  0.2516993431183593 :   0.1769223797355813 :   (b('ndvi_med') <= 3042.5) ?  (b('l8b4') <= 537.7270202636719) ?  (b('l8b5') <= 2472.4605712890625) ?  0.05774871612833704 :   0.10708821045925546 :   (b('moss') <= 14.317763328552246) ?  (b('bulk') <= 147.5) ?  -0.05764804878076881 :   -0.0896394592778345 :   (b('ndvi') <= 1896.0) ?  -0.06106546003152463 :   -0.03909878227967056 :   (b('ndvi') <= 2958.0) ?  (b('moss') <= 9.496762037277222) ?  (b('l8b11_med') <= 1219.7081909179688) ?  0.0436328131440398 :   0.002417753228952435 :   (b('l8b5') <= 2597.2200927734375) ?  0.14034744057113377 :   0.20487006828748883 :   (b('grass') <= 53.63877487182617) ?  (b('grass') <= 51.872941970825195) ?  0.014117933385636855 :   0.15804325353972762 :   (b('dg0_v_vv') <= -0.961205005645752) ?  -0.028595889761043197 :   0.041246570317688414 :   (b('l8b4') <= 550.6309814453125) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b5') <= 2831.73046875) ?  (b('dg0_v_vv') <= -1.8405475616455078) ?  (b('dg0_v_vv') <= -2.261984348297119) ?  (b('ndvi') <= 3820.0) ?  0.22694917336106302 :   0.21672718893736603 :   0.1877155556778113 :   (b('ndvi') <= 3909.5) ?  (b('ndvi_med') <= 3494.75) ?  0.28199026056461857 :   0.27417555744449296 :   (b('l8b4') <= 536.8708190917969) ?  0.2408216112942108 :   0.26840013440450033 :   (b('l8b5') <= 3621.7896728515625) ?  (b('l8b11_med') <= 1255.5039672851562) ?  (b('dg0_v_vv') <= 1.5718121528625488) ?  0.2767341852683633 :   0.3064440905298328 :   (b('ndvi') <= 2393.5) ?  0.3275743143780985 :   0.296861960448257 :   (b('l8b4_med') <= 537.5668334960938) ?  (b('dg0_v_vv') <= -0.04401540756225586) ?  0.252455330365707 :   0.24334483841818477 :   0.2631343141411032 :   (b('l8b4') <= 491.7077331542969) ?  (b('moss') <= 0.7690265774726868) ?  (b('ndvi') <= 2366.0) ?  0.3453981654345989 :   (b('dg0_v_vv') <= 0.7153482437133789) ?  0.3026328592234618 :   0.2955045311385161 :   (b('dg0_v_vv') <= -0.39860057830810547) ?  (b('l8b5') <= 3231.26171875) ?  0.36303009820205195 :   0.34194464321316237 :   (b('dg0_v_vv') <= 0.726557731628418) ?  0.363363424200212 :   0.3707941876454854 :   (b('crops') <= 25.502659797668457) ?  (b('dg0_v_vv') <= 0.11215686798095703) ?  (b('l8b11_med') <= 1271.4739990234375) ?  0.24023207557132484 :   0.24923207890918495 :   0.25243207347324137 :   (b('l8b5_med') <= 2592.1824951171875) ?  (b('dg0_v_vv') <= 0.07712221145629883) ?  0.25540821927643953 :   0.2886082150325888 :   (b('dg0_v_vv') <= 1.358168601989746) ?  0.30010978079195955 :   0.2690295307570464 :   (b('l8b5') <= 2405.21044921875) ?  (b('dg0_v_vv') <= -1.4954829216003418) ?  (b('l8b4') <= 630.7738952636719) ?  (b('dg0_v_vv') <= -2.439309597015381) ?  (b('dg0_v_vv') <= -2.6410932540893555) ?  -0.03408214847246806 :   -0.023278814367949957 :   (b('dg0_v_vv') <= -2.4059205055236816) ?  -0.05327881556004285 :   -0.04287881832569837 :   (b('ndvi') <= 3293.5) ?  0.044599521607160575 :   (b('dg0_v_vv') <= -1.910329818725586) ?  -0.006041906905409539 :   0.002318848296999909 :   (b('ndvi') <= 3523.5) ?  (b('ndvi_med') <= 3442.5) ?  (b('dg0_v_vv') <= 0.20822954177856445) ?  0.15282587988360932 :   0.1779501390146582 :   (b('dg0_v_vv') <= 0.7853989601135254) ?  0.15030915434287306 :   0.11648248131201981 :   (b('ndvi_med') <= 3438.5) ?  0.03101208390373933 :   0.021312083617637034 :   (b('dg0_v_vv') <= -3.9513120651245117) ?  -0.0155742359434565 :   (b('dg0_v_vv') <= -0.4701676368713379) ?  (b('l8b4') <= 632.1445007324219) ?  (b('l8b4') <= 602.4166259765625) ?  0.21531311599673786 :   -0.010443957755135172 :   (b('l8b5') <= 2546.05419921875) ?  0.23514828104107535 :   0.19877963495777068 :   (b('l8b11') <= 1194.320556640625) ?  (b('ndvi_med') <= 3646.0) ?  0.28166298704133613 :   0.23387517881890138 :   (b('crops') <= 26.48409938812256) ?  0.25479556319594193 :   0.22726398114155444 :   (b('moss') <= 6.230724096298218) ?  (b('sand') <= 20.5) ?  (b('ndvi') <= 3083.0) ?  (b('dg0_v_vv') <= -0.26369810104370117) ?  (b('ndvi') <= 1194.0) ?  0.26210501743742687 :   (b('l8b4_med') <= 765.9837646484375) ?  (b('l8b4') <= 713.5370788574219) ?  0.1566320141296761 :   0.058332013104476194 :   (b('dg0_v_vv') <= -2.5616278648376465) ?  (b('crops') <= 77.88888931274414) ?  0.11396065094219726 :   0.1024738387564818 :   (b('dg0_v_vv') <= -1.124680995941162) ?  0.17521706471903756 :   0.21024368253995218 :   (b('l8b4') <= 961.5106811523438) ?  (b('ndvi') <= 1817.0) ?  (b('dg0_v_vv') <= -0.09786748886108398) ?  0.2918050105709718 :   0.3142738686303298 :   (b('l8b11') <= 1481.193115234375) ?  (b('ndvi') <= 3000.0) ?  0.1987086137191472 :   0.2570781577783992 :   (b('dg0_v_vv') <= 0.761681079864502) ?  0.2878442185089079 :   0.2468751839697361 :   (b('bulk') <= 130.5) ?  0.09459519544243813 :   0.1444052294649259 :   (b('dg0_v_vv') <= -1.3960041999816895) ?  (b('l8b5') <= 2442.2066650390625) ?  (b('l8b4_med') <= 815.4116516113281) ?  (b('dg0_v_vv') <= -1.8581790924072266) ?  -0.010613004358207695 :   -0.023613000901138298 :   -0.029613005609905235 :   (b('dg0_v_vv') <= -2.255934715270996) ?  0.009367628031306807 :   (b('dg0_v_vv') <= -1.6275629997253418) ?  0.04038699468811799 :   (b('sand') <= 16.5) ?  0.022467681096623754 :   0.02689373581773706 :   (b('l8b11') <= 1254.086181640625) ?  (b('l8b11') <= 976.8113708496094) ?  (b('ndvi') <= 3947.0) ?  (b('crops') <= 77.88888931274414) ?  0.285097686111927 :   0.27182318463921545 :   (b('l8b4_med') <= 751.4978637695312) ?  0.028061677351593983 :   0.1632447773594142 :   (b('ndvi') <= 4894.5) ?  (b('crops') <= 41.00823211669922) ?  0.014059660074396858 :   0.08076217840958701 :   (b('bulk') <= 147.0) ?  0.19055318281054495 :   0.24213153233645873 :   (b('l8b4') <= 751.4978637695312) ?  (b('l8b5_med') <= 3017.598388671875) ?  (b('dg0_v_vv') <= -0.1810469627380371) ?  0.25821540794339465 :   0.21493096137727682 :   (b('ndvi_med') <= 3851.75) ?  0.12060702568426346 :   0.1503831772506237 :   (b('l8b4_med') <= 813.6318054199219) ?  (b('ndvi') <= 3637.0) ?  -0.006136534954389046 :   0.10685684452272759 :   (b('l8b4') <= 829.6798095703125) ?  0.1374668390201714 :   0.2397308045078338 :   (b('dg0_v_vv') <= 0.11562633514404297) ?  (b('l8b5_med') <= 2500.130126953125) ?  (b('crops') <= 71.7296028137207) ?  (b('ndvi_med') <= 3461.0) ?  (b('l8b11') <= 1024.0546875) ?  (b('dg0_v_vv') <= -0.011037826538085938) ?  0.012115542114871484 :   0.15129840847835013 :   (b('l8b5') <= 2975.7943115234375) ?  -0.047480803411073524 :   0.039185595453077475 :   (b('dg0_v_vv') <= -0.029304981231689453) ?  (b('ndvi') <= 2494.5) ?  0.1642331953609691 :   0.16309560083045258 :   0.17023317771799423 :   (b('bulk') <= 130.5) ?  (b('grass') <= 15.279504299163818) ?  (b('dg0_v_vv') <= -0.030740737915039062) ?  0.21559558655873154 :   0.17604501901456576 :   0.13689257742277494 :   (b('l8b4') <= 477.8813781738281) ?  0.035668359678524914 :   -0.018253774304986703 :   (b('bulk') <= 137.5) ?  (b('l8b11_med') <= 1964.0279541015625) ?  (b('l8b5_med') <= 3425.5689697265625) ?  (b('sand') <= 54.0) ?  0.00560854069198526 :   0.0601418603823226 :   (b('l8b5') <= 4485.472412109375) ?  0.05148666990944149 :   0.1798184886166977 :   (b('l8b4_med') <= 1205.6154174804688) ?  (b('l8b5') <= 2583.6232299804688) ?  0.19901500913887085 :   0.1781925691794526 :   (b('l8b11') <= 1420.1786499023438) ?  0.1468657285787998 :   0.11926028023392923 :   (b('sand') <= 28.0) ?  (b('ndvi') <= 2398.0) ?  (b('ndvi') <= 2193.5) ?  0.023423849502786465 :   0.07664319023489952 :   (b('l8b5') <= 2325.009033203125) ?  0.2878328012750312 :   0.12167681561674654 :   (b('l8b11_med') <= 2706.273681640625) ?  (b('ndvi_med') <= 1615.25) ?  0.13188391276630498 :   0.03510572618211427 :   (b('l8b5') <= 2535.334228515625) ?  -0.023522701482042933 :   -0.07149098078929224 :   (b('bulk') <= 150.5) ?  (b('grass') <= 8.456110954284668) ?  (b('l8b11_med') <= 1755.041015625) ?  (b('l8b11_med') <= 1722.6598510742188) ?  (b('ndvi') <= 1828.5) ?  0.1136075549346748 :   0.03518446450213129 :   (b('ndvi') <= 4844.0) ?  0.11498879111911378 :   0.06912830160124532 :   (b('l8b4') <= 1213.0177001953125) ?  (b('ndvi') <= 1016.5) ?  0.041709599048846795 :   -0.027777001917414605 :   (b('l8b5_med') <= 3694.775634765625) ?  0.03741620782349817 :   0.12119244613812843 :   (b('sand') <= 33.0) ?  (b('l8b5') <= 3845.001708984375) ?  (b('ndvi') <= 4629.0) ?  0.1494977152444363 :   0.07150962603637308 :   (b('l8b4') <= 1090.1581726074219) ?  0.020883017074005316 :   0.07642641873849143 :   (b('l8b5_med') <= 2160.37890625) ?  (b('ndvi') <= 2183.0) ?  0.030106195768457567 :   -0.052433075726468166 :   (b('l8b5') <= 1654.9096069335938) ?  0.12094371082934208 :   0.05041618098381749 :   (b('sand') <= 48.0) ?  (b('l8b5_med') <= 2500.130126953125) ?  (b('bulk') <= 161.0) ?  (b('l8b11') <= 914.2815551757812) ?  0.18849054370978566 :   0.05228856261562241 :   (b('dg0_v_vv') <= 1.68023681640625) ?  -0.10009879197685936 :   -0.002699130840277425 :   (b('dg0_v_vv') <= 2.7632620334625244) ?  (b('l8b5') <= 1984.1084594726562) ?  -0.03244992723203752 :   0.13450831991980566 :   (b('l8b4') <= 1024.8797607421875) ?  0.23077426528337577 :   0.15257363855108116 :   (b('l8b4') <= 878.9347839355469) ?  (b('l8b4') <= 579.0841979980469) ?  (b('dg0_v_vv') <= 0.7277927398681641) ?  -0.014849317297339432 :   -0.0767635286074311 :   (b('ndvi_med') <= 2450.75) ?  -0.014747946028750894 :   0.014036909329748909 :   (b('l8b5') <= 1992.6270141601562) ?  (b('dg0_v_vv') <= 1.6228179931640625) ?  0.07344990373297769 :   0.15803638817144292 :   (b('l8b5') <= 2556.0694580078125) ?  0.056057326485052975 :   0.012509089522106725 :   (b('bulk') <= 137.5) ?  (b('ndvi_med') <= 3240.5) ?  (b('bulk') <= 112.5) ?  (b('ndvi') <= 3392.5) ?  (b('l8b5') <= 2461.484375) ?  (b('l8b4') <= 960.7802734375) ?  (b('l8b11') <= 1049.226806640625) ?  -0.11654613815359728 :   -0.024407147426062068 :   (b('ndvi') <= 2229.0) ?  -0.08341548535743895 :   -0.12062223692346112 :   (b('dg0_v_vv') <= 1.3112549781799316) ?  (b('dg0_v_vv') <= -0.5085153579711914) ?  -0.07429960251367133 :   -0.11042942336452578 :   (b('ndvi_med') <= 3031.75) ?  0.11601532135280673 :   -0.13050235373989108 :   (b('ndvi') <= 3423.5) ?  (b('l8b4') <= 743.8332214355469) ?  0.12895386576540813 :   0.08693952935125851 :   (b('l8b11') <= 1238.3279418945312) ?  (b('dg0_v_vv') <= 0.5677127838134766) ?  -0.07004613705091135 :   -0.11804613839946644 :   (b('ndvi') <= 3481.5) ?  -0.12712191734387215 :   -0.014653805487499092 :   (b('grass') <= 57.093515396118164) ?  (b('dg0_v_vv') <= -0.043707847595214844) ?  (b('ndvi') <= 3318.5) ?  (b('grass') <= 16.96821689605713) ?  -0.06844676152686875 :   -0.04533379848509954 :   (b('l8b4_med') <= 706.6049499511719) ?  -0.07320184288938948 :   -0.024098279303369236 :   (b('l8b11') <= 979.98974609375) ?  (b('l8b5_med') <= 2631.6678466796875) ?  -0.14340662326993275 :   0.03109490173614397 :   (b('l8b11') <= 1028.5697631835938) ?  -0.06934370249216457 :   -0.024277951883112995 :   (b('moss') <= 12.66005277633667) ?  (b('moss') <= 6.591236591339111) ?  (b('ndvi') <= 3102.5) ?  -0.08849487203785221 :   0.03441397549029021 :   (b('l8b4') <= 1216.3544921875) ?  0.11499711671921803 :   0.0024391402570313886 :   (b('bulk') <= 134.5) ?  (b('l8b4') <= 730.2648315429688) ?  -0.001497121905482686 :   -0.060797243959521304 :   (b('l8b11') <= 1518.7025146484375) ?  0.029349591898213876 :   -0.024413333647179878 :   (b('l8b5') <= 2462.38037109375) ?  (b('l8b5_med') <= 2233.0537109375) ?  (b('l8b5') <= 2097.9619140625) ?  (b('moss') <= 7.090433120727539) ?  0.03468364994156192 :   (b('moss') <= 9.914189100265503) ?  0.1106648073787801 :   0.09186278291543326 :   (b('dg0_v_vv') <= -0.2659173011779785) ?  (b('l8b5') <= 2191.5975341796875) ?  0.16481318436563014 :   0.1470469355775464 :   (b('l8b11_med') <= 1300.2606811523438) ?  0.17408219347149134 :   0.18704692723289612 :   (b('l8b4') <= 632.7470397949219) ?  (b('dg0_v_vv') <= -0.7475500106811523) ?  (b('ndvi_med') <= 3329.5) ?  -0.07683066827806276 :   -0.07240136394269292 :   (b('l8b5') <= 2222.748046875) ?  -0.004345775975458821 :   -0.04813721768061319 :   (b('l8b11') <= 1436.123779296875) ?  (b('l8b4') <= 885.683837890625) ?  0.049884540565975695 :   0.19777812703989675 :   (b('crops') <= 8.186375617980957) ?  -0.04478083882630282 :   0.02999049269557153 :   (b('l8b5') <= 3021.619140625) ?  (b('ndvi_med') <= 4106.5) ?  (b('l8b11') <= 1709.3108520507812) ?  (b('l8b11_med') <= 1464.8991088867188) ?  -0.018576046749665412 :   -0.04666647786239207 :   0.1207360000499609 :   (b('dg0_v_vv') <= -3.0499353408813477) ?  (b('l8b4_med') <= 788.7772827148438) ?  -0.1068771347118766 :   -0.10837713495774576 :   (b('bulk') <= 119.0) ?  -0.10048739042319177 :   -0.08687713329626628 :   (b('bulk') <= 120.0) ?  (b('l8b11') <= 998.6707458496094) ?  (b('l8b4') <= 426.05743408203125) ?  0.247419572024896 :   0.13514625184477147 :   (b('dg0_v_vv') <= 0.685204029083252) ?  0.01629818197756718 :   0.11210741121678246 :   (b('bulk') <= 127.0) ?  (b('l8b4') <= 952.2659301757812) ?  -0.029124400850489382 :   -0.07966717502850708 :   (b('l8b4') <= 744.3202209472656) ?  0.02497458242934329 :   -0.0054856336718588725 :   (b('moss') <= 13.551528930664062) ?  (b('grass') <= 65.5498046875) ?  (b('ndvi') <= 3152.0) ?  (b('l8b5') <= 2065.5804443359375) ?  (b('moss') <= 7.853857040405273) ?  (b('l8b4') <= 1307.9089965820312) ?  -0.00924181366190635 :   -0.0639515437507374 :   (b('l8b11_med') <= 1248.767822265625) ?  0.00693944587037464 :   0.08463880394888476 :   (b('bulk') <= 150.5) ?  (b('l8b11') <= 1285.518798828125) ?  0.07093481900881146 :   0.0054478346608050725 :   (b('l8b11') <= 1041.333740234375) ?  0.14513990912655658 :   -0.06739435172282286 :   (b('moss') <= 12.015450477600098) ?  (b('ndvi_med') <= 2867.75) ?  (b('grass') <= 61.13550567626953) ?  -0.0014736584181114622 :   -0.14126804542636243 :   (b('dg0_v_vv') <= 1.8694133758544922) ?  0.05367779907454231 :   0.11209273671814059 :   (b('ndvi') <= 3595.5) ?  (b('l8b11_med') <= 1153.7015991210938) ?  -0.02469974814409373 :   0.07121670478371847 :   (b('ndvi') <= 3626.0) ?  0.18810920910891055 :   0.11665032656062177 :   (b('l8b5') <= 2804.08056640625) ?  (b('l8b4') <= 744.9050903320312) ?  (b('ndvi') <= 3221.5) ?  (b('grass') <= 68.20121002197266) ?  0.07704603301696024 :   0.20628969425454521 :   (b('l8b4') <= 734.4613037109375) ?  0.08905612623781045 :   0.24699911203591646 :   (b('dg0_v_vv') <= 2.829127788543701) ?  (b('ndvi') <= 2003.5) ?  0.011589862467441219 :   0.0957506930911264 :   (b('ndvi') <= 2417.0) ?  0.18878638857788824 :   0.2236156838387251 :   (b('l8b5_med') <= 2880.021728515625) ?  (b('dg0_v_vv') <= -0.2338271141052246) ?  (b('l8b4') <= 574.0109252929688) ?  0.14425938421345444 :   0.005068843225409628 :   (b('moss') <= 7.534638404846191) ?  0.048203644505976556 :   0.11891714917606228 :   (b('l8b5_med') <= 2941.370849609375) ?  -0.03570432247243538 :   -0.09090681910514833 :   (b('dg0_v_vv') <= 0.8984355926513672) ?  (b('ndvi') <= 3275.5) ?  (b('l8b5_med') <= 1780.4354248046875) ?  (b('l8b11') <= 1362.8778076171875) ?  (b('bulk') <= 145.0) ?  0.13371962487252326 :   0.042091873165738286 :   (b('l8b4') <= 614.6118469238281) ?  0.05803219896470174 :   -0.014825053602932398 :   (b('l8b4') <= 513.6836395263672) ?  (b('moss') <= 14.358450412750244) ?  0.1453002536440456 :   0.029178277588611808 :   (b('l8b4_med') <= 1072.5784912109375) ?  -0.04746179676407141 :   -0.013703967057402728 :   (b('l8b4_med') <= 887.5253601074219) ?  (b('l8b5_med') <= 2351.364990234375) ?  (b('l8b5_med') <= 2130.404296875) ?  -0.0258325577402386 :   0.08991383458792014 :   (b('ndvi') <= 4812.0) ?  -0.01582016494219194 :   0.04365733732110121 :   (b('l8b4') <= 776.326171875) ?  (b('l8b11_med') <= 1654.46435546875) ?  0.10624941428329251 :   0.011220167640695892 :   (b('l8b4_med') <= 1048.4745788574219) ?  0.004555317103462901 :   0.06580917540172404 :   (b('grass') <= 53.53939628601074) ?  (b('ndvi_med') <= 2876.5) ?  (b('dg0_v_vv') <= 1.8862566947937012) ?  (b('ndvi') <= 2494.0) ?  -0.05064019692649942 :   0.01676397683432054 :   (b('dg0_v_vv') <= 3.543779134750366) ?  0.03679643475771108 :   0.11381827586024931 :   (b('ndvi') <= 3826.5) ?  (b('l8b4') <= 913.9605407714844) ?  0.13655319771364025 :   0.0515826128306589 :   (b('l8b4_med') <= 942.4861145019531) ?  0.008258282091281486 :   0.10421152687036336 :   (b('ndvi_med') <= 2231.0) ?  (b('grass') <= 63.691490173339844) ?  (b('l8b5') <= 2928.1539306640625) ?  0.1399669882901258 :   0.2914505124104617 :   (b('l8b4') <= 711.8874206542969) ?  0.10668371519327804 :   0.03816970597624279 :   (b('l8b11') <= 1392.269775390625) ?  (b('ndvi') <= 3926.0) ?  -0.00048121334496066076 :   0.03325993808642046 :   (b('l8b4') <= 876.39599609375) ?  -0.037136579059949576 :   -0.05849898153322297 :   (b('l8b11_med') <= 2649.172119140625) ?  (b('crops') <= 58.653371810913086) ?  (b('dg0_v_vv') <= 1.3652663230895996) ?  (b('ndvi_med') <= 2722.75) ?  (b('grass') <= 90.26663208007812) ?  (b('sand') <= 36.5) ?  (b('l8b11_med') <= 2117.103271484375) ?  (b('crops') <= 31.142061233520508) ?  (b('l8b11') <= 1951.3969116210938) ?  -0.05427535687751113 :   -0.07976544704987289 :   (b('dg0_v_vv') <= -2.0880751609802246) ?  -0.09154308348441535 :   -0.11902994548087265 :   (b('dg0_v_vv') <= 0.6263210773468018) ?  (b('l8b4') <= 980.8529663085938) ?  0.0761164349437372 :   -0.0346646133859854 :   (b('l8b5') <= 2485.4310302734375) ?  0.1531310210605455 :   -0.04199729487002289 :   (b('moss') <= 49.34893798828125) ?  (b('l8b4') <= 1613.3408813476562) ?  (b('l8b11_med') <= 2417.81494140625) ?  -0.03766207380573541 :   -0.008869612999399016 :   (b('grass') <= 33.78372573852539) ?  -0.08533879960998464 :   -0.04814900322374253 :   (b('ndvi') <= 1193.0) ?  (b('l8b11') <= 2181.134033203125) ?  -0.11732914542282055 :   -0.1065973219083306 :   (b('l8b4') <= 1289.9705200195312) ?  -0.051558755540314305 :   -0.08379391693959164 :   (b('sand') <= 49.5) ?  (b('ndvi_med') <= 1752.0) ?  (b('ndvi') <= 1913.0) ?  (b('l8b11_med') <= 2457.32958984375) ?  -0.02667805905378753 :   -0.06054819029333209 :   (b('l8b11') <= 2399.5760498046875) ?  0.010832992414307685 :   0.03382268455846296 :   (b('dg0_v_vv') <= 0.4510078430175781) ?  (b('ndvi_med') <= 2064.25) ?  -0.0041564007473283695 :   0.03250052764766729 :   (b('l8b4_med') <= 1288.7588500976562) ?  0.04350328951756235 :   0.12607527706061752 :   (b('moss') <= 4.24605131149292) ?  (b('sand') <= 53.5) ?  (b('l8b4') <= 828.3163757324219) ?  -0.10954983625361919 :   -0.06491600276353288 :   (b('l8b11') <= 2006.5519409179688) ?  -0.011027407277757945 :   -0.05953106824209034 :   (b('l8b5') <= 1940.8349609375) ?  0.11239328935789253 :   (b('dg0_v_vv') <= -0.22980737686157227) ?  0.05039796839476346 :   0.02267620496629985 :   (b('sand') <= 24.5) ?  (b('dg0_v_vv') <= -1.5076565742492676) ?  (b('l8b5') <= 2306.631591796875) ?  (b('l8b4_med') <= 815.4116516113281) ?  (b('l8b5') <= 2006.9644165039062) ?  0.1601081186179687 :   0.1266980669313964 :   0.0911931712925434 :   (b('ndvi') <= 4525.0) ?  (b('dg0_v_vv') <= -2.722292423248291) ?  -0.05018317756826905 :   0.0038844777122447605 :   0.0737781822681427 :   (b('l8b5_med') <= 2941.4141845703125) ?  (b('l8b5') <= 2057.7791137695312) ?  (b('l8b4') <= 1054.5972290039062) ?  0.15564579944517098 :   0.09492533009405887 :   (b('bulk') <= 148.0) ?  0.21749187581829849 :   0.17829813277064813 :   (b('dg0_v_vv') <= -1.06022310256958) ?  -0.056730924804177546 :   (b('ndvi') <= 2610.5) ?  0.019832005117453783 :   0.12759536770944468 :   (b('grass') <= 33.23013496398926) ?  (b('l8b5_med') <= 2781.259521484375) ?  (b('crops') <= 41.12409019470215) ?  (b('ndvi') <= 3327.0) ?  0.044759069599499234 :   0.15708338233116617 :   (b('ndvi') <= 1454.5) ?  0.05655843191587858 :   -0.016066342854960853 :   (b('bulk') <= 120.5) ?  (b('l8b4') <= 1363.933837890625) ?  0.13620324477918985 :   0.17672902219126466 :   (b('grass') <= 28.193211555480957) ?  -0.021871551872729735 :   -0.04829268271235916 :   (b('moss') <= 23.058512687683105) ?  (b('bulk') <= 130.0) ?  (b('l8b11_med') <= 1553.487548828125) ?  0.023890473448918186 :   -0.0538472429101321 :   (b('ndvi_med') <= 3317.0) ?  -0.000837623841576305 :   0.05317479202152074 :   (b('l8b4') <= 1040.9805908203125) ?  (b('dg0_v_vv') <= 0.6086740493774414) ?  0.18153809679537541 :   -0.004493530002958435 :   (b('dg0_v_vv') <= 0.46020030975341797) ?  -0.0010652151519531417 :   0.1289606853827923 :   (b('grass') <= 78.23691940307617) ?  (b('sand') <= 25.5) ?  (b('l8b11') <= 1780.4437255859375) ?  (b('moss') <= 2.111313819885254) ?  0.17231274919230727 :   0.16121522767135615 :   (b('dg0_v_vv') <= 1.4309210777282715) ?  0.19771789100435047 :   (b('l8b11') <= 1797.3060302734375) ?  0.275831554437861 :   (b('ndvi') <= 2914.0) ?  0.24062083226052766 :   0.24604696871928963 :   (b('l8b5_med') <= 3209.6536865234375) ?  (b('l8b5') <= 1897.8151245117188) ?  (b('l8b4_med') <= 1266.5403442382812) ?  (b('l8b11_med') <= 1773.726318359375) ?  0.096822890514882 :   0.0035041265622852654 :   (b('l8b5') <= 1738.0552368164062) ?  0.21082595290744693 :   0.06893364724307682 :   (b('grass') <= 16.96821689605713) ?  (b('l8b11') <= 1995.265380859375) ?  -0.02269268706416539 :   -0.0643519810955811 :   (b('dg0_v_vv') <= 2.307797908782959) ?  -0.009844374035178027 :   0.033020388692297654 :   (b('grass') <= 31.963186264038086) ?  (b('dg0_v_vv') <= 1.522003173828125) ?  0.06402867254045932 :   (b('l8b5_med') <= 3322.8258056640625) ?  0.07187668076626597 :   0.17551353445332155 :   (b('l8b5') <= 2532.1771240234375) ?  (b('dg0_v_vv') <= 2.7911908626556396) ?  0.1326517666784895 :   0.17058453065156937 :   (b('ndvi') <= 3712.0) ?  0.02552531859417735 :   0.08494102720118718 :   (b('dg0_v_vv') <= 3.0835914611816406) ?  (b('l8b5_med') <= 2599.157958984375) ?  (b('l8b4') <= 1424.8674926757812) ?  (b('dg0_v_vv') <= 2.181826114654541) ?  (b('ndvi_med') <= 2250.0) ?  0.051689766560360104 :   -0.03338873564834112 :   (b('l8b11') <= 1749.0678100585938) ?  -0.10368681861323528 :   0.08495230743630888 :   (b('l8b11_med') <= 2170.646484375) ?  (b('dg0_v_vv') <= 1.5377264022827148) ?  -0.047940435873485704 :   -0.02811608938325802 :   (b('moss') <= 9.130302906036377) ?  -0.048693437261129846 :   -0.0667893078537683 :   (b('l8b4') <= 889.8487548828125) ?  -0.08071849624804894 :   (b('l8b11') <= 2108.6265869140625) ?  (b('sand') <= 33.5) ?  0.15199299359634008 :   0.08488971535633406 :   (b('dg0_v_vv') <= 1.768847942352295) ?  0.14166144448830326 :   0.17977131624093867 :   (b('l8b4_med') <= 1157.6909790039062) ?  (b('l8b4') <= 1246.6459350585938) ?  (b('ndvi') <= 2242.0) ?  (b('dg0_v_vv') <= 4.540953159332275) ?  0.043146192395057396 :   0.1314424778469393 :   (b('l8b5_med') <= 2269.953369140625) ?  0.13819289015710276 :   0.08806027212647251 :   (b('l8b11') <= 2384.6102294921875) ?  -0.08866163737321658 :   -0.05374260600528158 :   (b('ndvi_med') <= 2108.75) ?  (b('l8b5') <= 2514.88720703125) ?  (b('l8b11') <= 2092.08251953125) ?  0.25725740585604273 :   0.1537848511352465 :   (b('ndvi') <= 2242.0) ?  0.016249906987818363 :   0.10478706767906054 :   (b('ndvi') <= 2701.5) ?  (b('dg0_v_vv') <= 3.914259910583496) ?  0.20486033738855278 :   0.24363816376923592 :   0.16136496094832686 :   (b('l8b5_med') <= 3616.919921875) ?  (b('l8b5_med') <= 3030.7413330078125) ?  (b('bulk') <= 122.0) ?  (b('ndvi_med') <= 1681.0) ?  (b('l8b5') <= 2318.4764404296875) ?  (b('dg0_v_vv') <= -1.908799648284912) ?  0.25800502253958446 :   (b('l8b11') <= 2005.46728515625) ?  0.2501050229210542 :   0.25400501443335277 :   0.3053509122816225 :   (b('ndvi') <= 2937.0) ?  (b('dg0_v_vv') <= 0.21893310546875) ?  (b('ndvi_med') <= 2045.5) ?  0.05004501698800784 :   0.11534365870606088 :   (b('ndvi_med') <= 2045.5) ?  0.13870693771550754 :   0.16903718575157156 :   (b('bulk') <= 113.0) ?  0.17436573215507847 :   0.21526919819211257 :   (b('dg0_v_vv') <= 0.4116244316101074) ?  (b('bulk') <= 151.5) ?  (b('crops') <= 77.67890167236328) ?  (b('ndvi_med') <= 3859.25) ?  -0.009999571850588132 :   0.0770589311321445 :   (b('grass') <= 10.506257057189941) ?  0.020984074245587195 :   0.19477343732121485 :   (b('ndvi') <= 3414.5) ?  (b('l8b5') <= 2636.8311767578125) ?  -0.005977658632331659 :   -0.04295542195774921 :   (b('dg0_v_vv') <= -1.4819540977478027) ?  -0.011843977796801066 :   0.07457268890960203 :   (b('crops') <= 98.52266693115234) ?  (b('bulk') <= 141.0) ?  (b('ndvi_med') <= 2510.25) ?  0.1626061879889457 :   0.03311856539804497 :   (b('l8b5_med') <= 2893.3145751953125) ?  -0.017880903855622167 :   0.03153284907724261 :   (b('ndvi') <= 1388.0) ?  0.007320902685187064 :   (b('dg0_v_vv') <= 0.6335649490356445) ?  0.03889075972526826 :   0.14252338125383351 :   (b('dg0_v_vv') <= -0.20925283432006836) ?  (b('l8b5') <= 2900.8919677734375) ?  (b('bulk') <= 157.0) ?  (b('ndvi_med') <= 4305.25) ?  (b('ndvi_med') <= 2185.0) ?  0.0761348970939553 :   0.03049528011577788 :   0.21988983457287153 :   (b('l8b11') <= 2646.857666015625) ?  (b('dg0_v_vv') <= -0.8584904670715332) ?  0.14927590042318195 :   0.1864471094848696 :   0.09110012758827404 :   (b('crops') <= 83.83524322509766) ?  (b('ndvi') <= 3025.5) ?  (b('crops') <= 72.46576690673828) ?  -0.006955664836044696 :   0.03142764417098593 :   (b('ndvi_med') <= 4697.0) ?  0.07414494954632035 :   -0.020433278693570523 :   (b('grass') <= 5.914103746414185) ?  (b('ndvi') <= 2058.0) ?  -0.01315410390459204 :   0.019264473902973114 :   (b('l8b5') <= 3338.1546630859375) ?  -0.026978319725715322 :   -0.07680387577539775 :   (b('grass') <= 11.703174591064453) ?  (b('l8b4') <= 1400.3999633789062) ?  (b('l8b4_med') <= 1144.2802124023438) ?  (b('l8b11_med') <= 1938.9884033203125) ?  0.08461793197278832 :   -0.03061429639888974 :   (b('dg0_v_vv') <= 0.22638416290283203) ?  0.0705437995788886 :   0.16437750351238906 :   (b('l8b11') <= 2173.5) ?  (b('l8b5_med') <= 3097.873046875) ?  0.014966295654497731 :   -0.05264593259266031 :   (b('sand') <= 24.5) ?  0.2048509540334538 :   0.04965102204507241 :   (b('l8b5') <= 2487.81103515625) ?  (b('dg0_v_vv') <= -0.15535783767700195) ?  0.021116366049133 :   (b('crops') <= 72.93136215209961) ?  0.259850748331536 :   0.1645965745074709 :   (b('ndvi') <= 1147.5) ?  (b('l8b11') <= 2964.237060546875) ?  0.01528141476930426 :   0.04816976042865634 :   (b('l8b11') <= 2301.4681396484375) ?  0.0814274355650536 :   0.1323078321136102 :   (b('bulk') <= 149.5) ?  (b('dg0_v_vv') <= -0.36566638946533203) ?  0.027252917681224703 :   (b('l8b11') <= 2785.210693359375) ?  (b('ndvi_med') <= 3513.75) ?  0.10783766518441326 :   (b('ndvi') <= 4109.0) ?  (b('dg0_v_vv') <= 6.2452614307403564) ?  0.09155801648988007 :   0.08083767007199413 :   (b('dg0_v_vv') <= 0.20551252365112305) ?  0.07625291267443454 :   0.08137731699559422 :   0.11713533127498577 :   (b('l8b5') <= 3142.757080078125) ?  (b('dg0_v_vv') <= -1.3798432350158691) ?  (b('l8b5') <= 2521.1158447265625) ?  (b('ndvi') <= 1454.0) ?  (b('l8b11_med') <= 2518.2154541015625) ?  -0.08518443626840708 :   -0.0821844376393139 :   (b('dg0_v_vv') <= -4.267696380615234) ?  -0.09104249147263953 :   -0.1077924916570414 :   (b('l8b5') <= 2867.196533203125) ?  (b('l8b5') <= 2786.6824951171875) ?  -0.05382271734349031 :   0.02331556754629019 :   (b('ndvi') <= 1582.0) ?  -0.06718443704326746 :   -0.1161844363826493 :   (b('l8b4') <= 1175.1336669921875) ?  (b('dg0_v_vv') <= 3.9949934482574463) ?  0.08080403343516235 :   0.15094397407433355 :   (b('ndvi') <= 1538.0) ?  (b('dg0_v_vv') <= 0.5787863731384277) ?  -0.0761824917482492 :   -0.10368249346188274 :   (b('l8b11') <= 2592.2750244140625) ?  -0.02061561813370123 :   0.017680108829187583 :   (b('l8b11') <= 3078.45849609375) ?  (b('l8b11') <= 2212.74072265625) ?  (b('dg0_v_vv') <= 4.123589515686035) ?  (b('dg0_v_vv') <= 2.7205300331115723) ?  -0.07119596540254708 :   -0.08652929876568273 :   -0.04019596590918656 :   (b('l8b4') <= 1462.186767578125) ?  (b('ndvi') <= 4163.0) ?  -0.12464346817652527 :   -0.09205503083404427 :   (b('l8b4') <= 2003.2720336914062) ?  -0.09050232285420373 :   -0.11144806381964498 :   (b('dg0_v_vv') <= 0.4828362464904785) ?  (b('dg0_v_vv') <= -0.5864505767822266) ?  (b('l8b11') <= 3280.2891845703125) ?  -0.11212696819671923 :   -0.08693130197333679 :   (b('dg0_v_vv') <= 0.41527318954467773) ?  -0.0036282969950648114 :   -0.11812829993059357 :   (b('l8b5_med') <= 3694.775634765625) ?  -0.012128299319645958 :   (b('dg0_v_vv') <= 3.0347416400909424) ?  0.031278306572273123 :   0.06527830096943651 :   (b('grass') <= 1.0004997551441193) ?  (b('ndvi') <= 976.5) ?  (b('l8b11') <= 2886.304931640625) ?  (b('dg0_v_vv') <= 0.28336572647094727) ?  (b('ndvi') <= 937.5) ?  (b('ndvi') <= 626.0) ?  (b('l8b11') <= 2730.3548583984375) ?  (b('l8b5') <= 2549.483642578125) ?  -0.03506023678863332 :   -0.030274867188977123 :   -0.04306023064935491 :   (b('l8b5') <= 2005.5791015625) ?  (b('dg0_v_vv') <= -0.11111068725585938) ?  -0.0458444889310852 :   -0.02184449244775924 :   (b('l8b4') <= 1728.8726196289062) ?  -0.07094026295082466 :   -0.05681101808980031 :   0.012985685933381313 :   (b('ndvi') <= 937.5) ?  (b('l8b11') <= 2846.7366943359375) ?  (b('l8b11') <= 2693.22607421875) ?  0.06196923963978207 :   (b('dg0_v_vv') <= 0.3387565612792969) ?  0.02796924524261868 :   0.04086543983404911 :   -0.05941051378089737 :   (b('l8b5_med') <= 2254.820556640625) ?  0.10955304593694482 :   0.08216626874842964 :   (b('dg0_v_vv') <= 1.5957951545715332) ?  (b('ndvi') <= 717.5) ?  (b('l8b5') <= 3794.837158203125) ?  (b('l8b4') <= 2656.1912841796875) ?  (b('ndvi') <= 620.5) ?  -0.014670374660240543 :   -0.04557972049572388 :   -0.06506918518453092 :   (b('l8b11') <= 4233.6361083984375) ?  0.03905011572481057 :   -0.03132831574883313 :   (b('l8b5') <= 4004.40234375) ?  (b('l8b4') <= 1887.6387939453125) ?  (b('l8b11') <= 3024.7738037109375) ?  -0.06664872051308991 :   -0.10455988375057658 :   (b('ndvi') <= 836.5) ?  -0.0323540840415826 :   -0.061354363128682775 :   (b('ndvi') <= 825.5) ?  (b('l8b11') <= 4010.6466064453125) ?  -0.015949883977166207 :   -0.07513782510708822 :   (b('l8b5') <= 4598.046142578125) ?  -0.0804759562027222 :   -0.061637827550878654 :   (b('dg0_v_vv') <= 2.3834609985351562) ?  0.018790824884439228 :   0.060468181325122716 :   (b('l8b4') <= 1738.8670654296875) ?  (b('dg0_v_vv') <= 2.8061439990997314) ?  (b('l8b4') <= 1115.95703125) ?  (b('l8b5_med') <= 2812.056640625) ?  (b('l8b11') <= 1970.1572875976562) ?  0.05591295876702629 :   0.02921567879617215 :   (b('dg0_v_vv') <= -0.6070122718811035) ?  (b('dg0_v_vv') <= -1.2850861549377441) ?  0.07231287809927016 :   0.05627780877922972 :   (b('dg0_v_vv') <= 1.4126996994018555) ?  0.08124047479452055 :   0.10736469116120831 :   (b('l8b4') <= 1570.12158203125) ?  (b('l8b4') <= 1493.0738525390625) ?  (b('l8b11') <= 2506.1741943359375) ?  -0.008861872851255021 :   0.04489853881877899 :   (b('dg0_v_vv') <= 0.8842053413391113) ?  -0.013896669047145678 :   -0.037934659502431266 :   (b('bulk') <= 152.0) ?  (b('l8b4') <= 1656.4095458984375) ?  0.040391453008076984 :   -0.017162598095512432 :   (b('dg0_v_vv') <= 2.189052104949951) ?  0.07847757337617199 :   0.10841213864963745 :   (b('l8b4') <= 1205.6609191894531) ?  0.21341568700969218 :   (b('l8b11_med') <= 2890.52392578125) ?  (b('ndvi_med') <= 1424.5) ?  (b('dg0_v_vv') <= 3.823817014694214) ?  0.06571009456002641 :   0.057710093248724226 :   (b('l8b4_med') <= 1649.88818359375) ?  0.11402046796950427 :   0.09402045724066821 :   (b('ndvi') <= 1283.5) ?  0.10075890376631702 :   (b('l8b11') <= 3109.673095703125) ?  0.12477331764391296 :   0.1092204403606328 :   (b('dg0_v_vv') <= 1.200575828552246) ?  (b('ndvi') <= 1720.0) ?  (b('l8b5') <= 4264.43408203125) ?  (b('l8b11') <= 2685.8968505859375) ?  (b('ndvi') <= 1666.5) ?  0.0034225670956716065 :   -0.06610936642959714 :   (b('l8b11') <= 3393.3631591796875) ?  -0.02896178433109146 :   -0.00897810773178998 :   (b('dg0_v_vv') <= 0.6882758140563965) ?  (b('ndvi') <= 1282.0) ?  -0.07915676859361291 :   -0.07263190701653045 :   -0.06129857306728245 :   (b('l8b5') <= 4246.0625) ?  (b('l8b11') <= 2628.866455078125) ?  (b('ndvi') <= 1754.5) ?  0.03036683380596264 :   0.08748166450722651 :   (b('l8b5') <= 3741.14013671875) ?  0.01652332042051142 :   -0.02915982345365194 :   (b('dg0_v_vv') <= 0.3444066047668457) ?  (b('ndvi') <= 3561.0) ?  0.06830484516918658 :   0.0810381869226694 :   0.09653817363083361 :   (b('l8b11') <= 3656.909423828125) ?  (b('dg0_v_vv') <= 1.2369766235351562) ?  0.143349436207591 :   (b('dg0_v_vv') <= 4.195020437240601) ?  (b('l8b5') <= 3098.8421630859375) ?  -0.007331883294523487 :   0.04331585573392147 :   (b('dg0_v_vv') <= 4.811807632446289) ?  0.1750204679098996 :   0.038294320241971436 :   (b('l8b4') <= 2811.069091796875) ?  -0.011844509455511215 :   (b('l8b11') <= 3912.302001953125) ?  -0.05329857175598027 :   -0.06729857032546879 :   (b('grass') <= 27.41930866241455) ?  (b('bulk') <= 151.5) ?  (b('l8b5_med') <= 3339.2752685546875) ?  (b('grass') <= 16.03073215484619) ?  (b('l8b5') <= 2102.2525634765625) ?  (b('dg0_v_vv') <= 0.5253033638000488) ?  (b('dg0_v_vv') <= -0.028571605682373047) ?  -0.02600495521015779 :   -0.005097296427462761 :   (b('l8b4_med') <= 1803.0075073242188) ?  0.047003607020877886 :   0.09882656136960759 :   (b('ndvi_med') <= 1740.5) ?  (b('bulk') <= 150.0) ?  -0.06395340374175616 :   -0.08929622420982124 :   (b('dg0_v_vv') <= 2.3808393478393555) ?  -0.07103864125179632 :   -0.01056397263845662 :   (b('l8b4') <= 1352.3377685546875) ?  (b('ndvi') <= 2113.5) ?  0.1262631727564256 :   0.1415631486284654 :   (b('ndvi') <= 1418.5) ?  (b('ndvi') <= 1382.0) ?  -0.0498805759804424 :   -0.003581120351936221 :   (b('l8b11') <= 3049.818603515625) ?  -0.08004712670760798 :   -0.05214840258305806 :   (b('dg0_v_vv') <= 1.1337785720825195) ?  (b('l8b5') <= 3065.5802001953125) ?  (b('l8b4') <= 2049.66552734375) ?  (b('ndvi') <= 2326.5) ?  -0.08501616909741352 :   -0.06604354288952345 :   (b('grass') <= 4.434427618980408) ?  -0.056827716152551 :   -0.11254855719831675 :   (b('l8b4_med') <= 2458.094970703125) ?  (b('l8b5') <= 4116.728759765625) ?  -0.10705052285716997 :   -0.09077110792233256 :   (b('dg0_v_vv') <= -0.21473407745361328) ?  -0.04130741975641514 :   -0.04017628600523501 :   (b('ndvi') <= 4196.0) ?  (b('l8b5') <= 3431.8533935546875) ?  (b('l8b5_med') <= 3696.302001953125) ?  -0.09541569515237823 :   -0.05229844892033187 :   (b('moss') <= 30.879332542419434) ?  -0.06416562075376499 :   -0.10312705986239536 :   0.042648588883427874 :   (b('l8b4') <= 2081.0809326171875) ?  (b('ndvi_med') <= 1118.0) ?  (b('ndvi') <= 2005.0) ?  (b('l8b11') <= 2632.4132080078125) ?  (b('ndvi') <= 1616.5) ?  -0.02536325625740013 :   -0.014280147335181625 :   (b('l8b11') <= 2837.7303466796875) ?  -0.006625071598073962 :   -0.010320779381174011 :   0.03710318237543106 :   (b('dg0_v_vv') <= 0.07643890380859375) ?  (b('l8b11') <= 2730.9708251953125) ?  (b('bulk') <= 154.5) ?  0.013033182322978953 :   -0.004746819958090787 :   (b('l8b4') <= 1611.6276245117188) ?  0.032320471895861735 :   0.027653806016473076 :   (b('l8b4') <= 1794.298828125) ?  (b('l8b11') <= 2248.481689453125) ?  0.10487567376534945 :   0.08279104165466768 :   (b('l8b5_med') <= 3010.2784423828125) ?  0.051503184735775004 :   0.03854318261146544 :   (b('dg0_v_vv') <= 0.9582796096801758) ?  (b('grass') <= 21.365646362304688) ?  (b('l8b5') <= 3030.94482421875) ?  (b('bulk') <= 152.5) ?  0.05976917368741927 :   -0.018947192669683827 :   (b('ndvi') <= 1117.5) ?  -0.04265722162706875 :   -0.058959913685030486 :   (b('l8b5') <= 4320.913818359375) ?  (b('ndvi') <= 790.5) ?  -0.032659882616587185 :   -0.06642341528477984 :   (b('dg0_v_vv') <= 0.3528175354003906) ?  -0.025356817469000814 :   0.010643178373575202 :   (b('l8b11') <= 3108.5987548828125) ?  0.03438429281589707 :   (b('dg0_v_vv') <= 3.329953670501709) ?  (b('ndvi_med') <= 930.0) ?  -0.017486624016293754 :   -0.03863556635862797 :   (b('l8b4') <= 2591.18310546875) ?  0.012039190363138913 :   -0.006660817358642823 :   (b('dg0_v_vv') <= 0.6733798980712891) ?  (b('ndvi_med') <= 1772.0) ?  (b('sand') <= 52.5) ?  (b('l8b4_med') <= 1828.8604736328125) ?  (b('l8b5_med') <= 2817.5809326171875) ?  (b('ndvi') <= 2625.0) ?  -0.04895750286794795 :   0.05539630241886842 :   (b('l8b11') <= 3051.893798828125) ?  0.015495788292332327 :   0.06517777817831262 :   (b('l8b4') <= 1737.7548217773438) ?  (b('l8b11') <= 2202.611572265625) ?  -0.049185844197023154 :   -0.0014183884936948728 :   (b('l8b11_med') <= 3115.2999267578125) ?  -0.05194560678580159 :   -0.01223559215386955 :   (b('ndvi') <= 1674.5) ?  (b('ndvi') <= 864.0) ?  (b('l8b5') <= 2249.76953125) ?  -0.0015725117996041593 :   -0.018868637862847405 :   (b('moss') <= 13.875142574310303) ?  -0.08055107530345165 :   -0.05947202596497827 :   (b('ndvi') <= 2000.0) ?  (b('moss') <= 24.018625259399414) ?  -0.00737237980000562 :   -0.0452923885479025 :   (b('l8b11_med') <= 3149.922119140625) ?  0.05001130913666019 :   0.0049413494415183334 :   (b('bulk') <= 145.5) ?  (b('dg0_v_vv') <= -0.5900630950927734) ?  (b('dg0_v_vv') <= -1.3426823616027832) ?  -0.08137065250736972 :   (b('l8b5_med') <= 2879.4798583984375) ?  -0.08672499159687796 :   -0.09172499055379668 :   (b('l8b5') <= 3011.570068359375) ?  (b('ndvi') <= 2110.0) ?  0.015284444063663036 :   0.05686878160711023 :   (b('ndvi') <= 2103.0) ?  -0.0539586482842985 :   0.019320522645069532 :   (b('ndvi') <= 2342.0) ?  (b('l8b11_med') <= 2674.5888671875) ?  (b('l8b11') <= 2805.9588623046875) ?  -0.09647196303333162 :   -0.11106037216930394 :   (b('l8b11') <= 2860.5843505859375) ?  -0.05301718347217479 :   -0.09371848466729141 :   (b('dg0_v_vv') <= -0.8953323364257812) ?  (b('l8b11') <= 2803.3084716796875) ?  -0.04694988507228817 :   -0.08636895617331124 :   (b('bulk') <= 147.0) ?  -0.06769988371782615 :   -0.0333953891296449 :   (b('moss') <= 13.497345924377441) ?  (b('moss') <= 6.877649307250977) ?  (b('dg0_v_vv') <= 2.110391616821289) ?  (b('l8b11') <= 2238.541015625) ?  (b('ndvi') <= 2534.0) ?  0.03636128128684386 :   -0.04014947987902065 :   (b('l8b4') <= 2264.077392578125) ?  -0.036799862145334816 :   0.014222594152851204 :   (b('ndvi') <= 3112.0) ?  (b('l8b5_med') <= 3026.2239990234375) ?  -0.04628775561270125 :   0.018705704485985405 :   0.1700061871111393 :   (b('sand') <= 47.0) ?  (b('dg0_v_vv') <= 0.7173037528991699) ?  -0.05214093686056093 :   (b('l8b11') <= 3118.6649169921875) ?  0.06870089834463279 :   0.11350213868016501 :   (b('sand') <= 71.0) ?  (b('ndvi') <= 1118.0) ?  0.023924372569006633 :   -0.0074599100059769925 :   -0.08075770727688719 :   (b('sand') <= 48.5) ?  (b('l8b4_med') <= 1754.7330322265625) ?  (b('l8b11') <= 2529.3077392578125) ?  (b('l8b4') <= 1245.9231567382812) ?  -0.06562170401775283 :   -0.08589922603752365 :   (b('sand') <= 47.0) ?  -0.087906959115791 :   -0.11356834233122771 :   (b('l8b5') <= 2712.00146484375) ?  (b('moss') <= 16.791054725646973) ?  -0.002010860376413794 :   -0.04692443646667821 :   (b('ndvi') <= 2129.0) ?  -0.07152754792325411 :   0.00184988027563604 :   (b('ndvi') <= 1314.0) ?  (b('l8b4_med') <= 1957.8063354492188) ?  (b('moss') <= 24.955320358276367) ?  -0.07466842187422855 :   -0.04500478239690998 :   0.05044304310348863 :   (b('l8b11_med') <= 2830.9715576171875) ?  (b('l8b4') <= 1775.9574584960938) ?  0.1100433176551459 :   0.036677453970452606 :   (b('ndvi') <= 1370.0) ?  0.0651333127741809 :   -0.009198190147909817 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 3272.0) ?  (b('l8b4') <= 1430.482177734375) ?  (b('moss') <= 6.230724096298218) ?  (b('sand') <= 24.0) ?  (b('l8b11_med') <= 1670.345703125) ?  (b('ndvi') <= 1487.0) ?  (b('ndvi') <= 1341.5) ?  (b('ndvi_med') <= 2003.5) ?  (b('l8b11') <= 1598.2060546875) ?  0.21609450238992745 :   (b('l8b11') <= 1799.1797485351562) ?  0.2626245095138746 :   0.22751389939176067 :   0.17235855043828485 :   (b('dg0_v_vv') <= -0.3608245849609375) ?  0.28724945841466465 :   0.28284648176729676 :   (b('dg0_v_vv') <= -0.023137569427490234) ?  (b('l8b5') <= 2634.6373291015625) ?  (b('l8b4') <= 908.3641052246094) ?  (b('dg0_v_vv') <= -1.4745965003967285) ?  0.10503270869374631 :   0.16478332507990298 :   (b('l8b4_med') <= 854.3993225097656) ?  0.20233922824534112 :   0.15851981893014674 :   (b('l8b5') <= 3039.79541015625) ?  (b('l8b5') <= 2996.9404296875) ?  0.10875563392126829 :   0.04793629878145611 :   (b('l8b11') <= 1018.5015563964844) ?  0.10899320911985155 :   0.1541462203932417 :   (b('l8b11') <= 758.6168212890625) ?  0.2446408661752939 :   (b('ndvi') <= 2972.5) ?  (b('ndvi') <= 2489.0) ?  0.17304914744018499 :   0.19960286544559186 :   (b('dg0_v_vv') <= 0.1082005500793457) ?  0.20239110233111457 :   0.14463926337124447 :   (b('ndvi') <= 4829.5) ?  (b('crops') <= 38.22074508666992) ?  (b('ndvi') <= 2482.0) ?  (b('dg0_v_vv') <= -0.06833362579345703) ?  -0.012852955872811045 :   (b('dg0_v_vv') <= 1.2297320365905762) ?  0.06324565943749562 :   0.09907255260262732 :   (b('l8b4') <= 1212.5626831054688) ?  (b('l8b4') <= 942.2908935546875) ?  -0.0236782396096038 :   -0.012218645213739804 :   0.01980864189410722 :   (b('l8b11') <= 1168.5030517578125) ?  (b('dg0_v_vv') <= -1.8036465644836426) ?  (b('l8b4') <= 727.6373291015625) ?  0.19996651234928248 :   0.2225156219118554 :   (b('l8b4') <= 477.378173828125) ?  0.12965711025830717 :   0.16758299807124524 :   (b('l8b4') <= 1230.0103759765625) ?  (b('l8b11') <= 1681.078125) ?  0.05184137648483507 :   0.1330895921404146 :   (b('dg0_v_vv') <= 1.1887195110321045) ?  0.09939379848750377 :   0.14917263520271598 :   (b('l8b5') <= 4724.470947265625) ?  (b('l8b5_med') <= 3116.2115478515625) ?  (b('l8b5_med') <= 2844.591552734375) ?  0.0021691634837849305 :   (b('ndvi') <= 4892.0) ?  0.004565193704228421 :   0.00616917159001662 :   -0.00032282979423703395 :   (b('dg0_v_vv') <= 2.04666805267334) ?  (b('l8b4_med') <= 1334.3555297851562) ?  0.014166123023122229 :   0.022166124334424414 :   (b('dg0_v_vv') <= 3.078529119491577) ?  (b('dg0_v_vv') <= 2.7941391468048096) ?  0.031335835614210716 :   0.03233583764076864 :   0.020335828223234764 :   (b('dg0_v_vv') <= 0.5227217674255371) ?  (b('l8b5_med') <= 3084.6864013671875) ?  (b('crops') <= 98.06986236572266) ?  (b('bulk') <= 130.5) ?  (b('l8b11_med') <= 1719.9636840820312) ?  (b('l8b4_med') <= 577.529296875) ?  0.18243676625343708 :   0.00802634616169755 :   (b('l8b5_med') <= 2037.2767333984375) ?  0.07731124468138938 :   0.15964971448363297 :   (b('l8b11') <= 1374.8938598632812) ?  (b('crops') <= 3.3714100420475006) ?  0.10516929743951035 :   -0.0072675798001070714 :   (b('ndvi_med') <= 1862.75) ?  -0.03961838233675704 :   -0.013524362648950293 :   (b('l8b5_med') <= 1912.002197265625) ?  (b('ndvi') <= 2442.0) ?  (b('l8b11') <= 1892.9313354492188) ?  0.20079302017595563 :   0.1771618751935563 :   0.22667511493645023 :   (b('ndvi') <= 3840.5) ?  (b('l8b5_med') <= 2776.2713623046875) ?  0.0655882698190481 :   0.023691840424953068 :   (b('l8b4') <= 786.4572143554688) ?  0.058036280988567994 :   0.11519079432854713 :   (b('l8b4_med') <= 1096.3162231445312) ?  (b('l8b11') <= 2041.3314819335938) ?  (b('bulk') <= 130.5) ?  (b('dg0_v_vv') <= -2.7762346267700195) ?  0.2615975161681229 :   0.23059752785063325 :   (b('ndvi') <= 1724.5) ?  0.20822811730731428 :   0.04845152392732377 :   (b('l8b4') <= 1392.19970703125) ?  (b('ndvi') <= 2045.5) ?  0.07549132672864635 :   0.16245524531386973 :   0.041276401035405946 :   (b('l8b5_med') <= 3259.750732421875) ?  (b('moss') <= 6.159199953079224) ?  (b('ndvi') <= 4344.5) ?  -0.05232871471397363 :   -0.008476766809763878 :   (b('ndvi') <= 3415.0) ?  -0.005711122446903816 :   0.07993041057204657 :   (b('ndvi_med') <= 2029.0) ?  (b('l8b4') <= 919.5252075195312) ?  0.1811652034164938 :   0.060383419971435065 :   (b('dg0_v_vv') <= -1.6001253128051758) ?  0.06021172084241774 :   -0.0051567277732327416 :   (b('l8b5_med') <= 3661.0555419921875) ?  (b('l8b5_med') <= 3259.750732421875) ?  (b('bulk') <= 154.5) ?  (b('moss') <= 1.449996531009674) ?  (b('sand') <= 46.0) ?  0.08291099965670794 :   0.018150573858061216 :   (b('l8b4_med') <= 602.2307434082031) ?  0.23133049769918304 :   0.016768270727840186 :   (b('l8b5_med') <= 2506.0723876953125) ?  (b('grass') <= 73.97923278808594) ?  -0.09836261882736143 :   0.049315590376882 :   (b('sand') <= 40.5) ?  0.14127209448727093 :   0.04443601092535292 :   (b('sand') <= 52.0) ?  (b('l8b11') <= 1573.3331909179688) ?  (b('l8b4') <= 737.3588256835938) ?  0.08770246734376323 :   0.1719895140265966 :   (b('l8b11') <= 1612.715576171875) ?  -0.03011769507029312 :   0.11048487758246602 :   (b('dg0_v_vv') <= 3.911597490310669) ?  (b('ndvi') <= 1050.5) ?  0.07126434416821315 :   0.038712872662728055 :   (b('l8b4') <= 361.2332305908203) ?  -0.029304802633356553 :   -0.060789600669447705 :   (b('l8b11') <= 1923.7166748046875) ?  (b('ndvi') <= 1909.5) ?  (b('l8b5_med') <= 3694.775634765625) ?  (b('dg0_v_vv') <= 1.8656964302062988) ?  0.020450807593594467 :   0.04694318991871637 :   (b('l8b4') <= 1175.1336669921875) ?  0.07272363009164612 :   0.08286559524853246 :   (b('dg0_v_vv') <= 1.3138189315795898) ?  (b('ndvi') <= 4673.5) ?  0.023230154197285524 :   0.07051707382949801 :   (b('ndvi') <= 3895.5) ?  -0.010245993366628977 :   -0.056621421909373804 :   (b('l8b11') <= 2436.2935791015625) ?  (b('l8b11') <= 2231.518310546875) ?  (b('dg0_v_vv') <= 4.966431379318237) ?  -0.0798149032179527 :   -0.038176365920803146 :   (b('l8b11') <= 2387.977783203125) ?  -0.11393998217610167 :   -0.0905906839567871 :   (b('dg0_v_vv') <= 2.4095993041992188) ?  -0.002214327753349149 :   0.006785675584510958 :   (b('l8b4') <= 624.95166015625) ?  (b('grass') <= 33.973506927490234) ?  (b('l8b5_med') <= 2891.454833984375) ?  (b('l8b5') <= 1459.3858642578125) ?  (b('dg0_v_vv') <= 2.262361526489258) ?  (b('dg0_v_vv') <= -0.2295823097229004) ?  (b('dg0_v_vv') <= -0.701770544052124) ?  -0.056658392042813255 :   -0.037658387065825416 :   (b('l8b11') <= 897.0798950195312) ?  -0.05191971064204842 :   -0.0826537517524636 :   0.06381896846393759 :   (b('l8b5_med') <= 1590.654296875) ?  (b('l8b4') <= 579.9348449707031) ?  (b('ndvi') <= 2921.0) ?  0.17422965969422674 :   0.07239948187425183 :   (b('ndvi') <= 3283.5) ?  -0.05599974466171325 :   0.03910458136740261 :   (b('l8b11_med') <= 1102.9674682617188) ?  (b('ndvi') <= 2367.0) ?  0.270297874472054 :   0.022132636952825524 :   (b('ndvi') <= 3180.0) ?  -0.029448664258953802 :   0.004752326315652098 :   (b('l8b4') <= 608.9883728027344) ?  (b('l8b4') <= 559.0523681640625) ?  (b('ndvi') <= 3727.5) ?  (b('ndvi') <= 2381.5) ?  0.028643286785761824 :   -0.0014130078994614795 :   (b('dg0_v_vv') <= 0.18042564392089844) ?  -0.06400974939118784 :   -0.01573103105838868 :   (b('l8b5') <= 3708.8658447265625) ?  (b('l8b11') <= 1210.534423828125) ?  -0.08120321090917128 :   -0.018605043808480493 :   (b('dg0_v_vv') <= 1.534522533416748) ?  -0.0002741593456321578 :   0.014213567952807593 :   (b('bulk') <= 132.5) ?  0.05232088690086209 :   (b('ndvi_med') <= 3114.25) ?  (b('dg0_v_vv') <= 0.9176263809204102) ?  0.01591624375338721 :   0.04121811076415796 :   (b('dg0_v_vv') <= -0.5109472274780273) ?  0.024808733490330026 :   0.012117517124828328 :   (b('sand') <= 26.0) ?  (b('dg0_v_vv') <= -2.6565685272216797) ?  (b('dg0_v_vv') <= -3.0703907012939453) ?  0.03781085564506781 :   0.11581084980381262 :   (b('l8b4') <= 620.512939453125) ?  (b('dg0_v_vv') <= -0.5269103050231934) ?  (b('l8b5') <= 3227.5064697265625) ?  0.245996648287785 :   0.22978074765407286 :   (b('ndvi') <= 3856.0) ?  0.27568961585146157 :   0.2570107984649201 :   (b('l8b11_med') <= 1244.5859985351562) ?  0.16201386256826655 :   0.20191185532572603 :   (b('bulk') <= 121.5) ?  (b('l8b5') <= 1962.3838500976562) ?  (b('ndvi_med') <= 3045.75) ?  (b('l8b4_med') <= 746.8522644042969) ?  -0.10529513464165838 :   -0.10675877296986798 :   -0.1102951335985771 :   (b('l8b4') <= 542.8884887695312) ?  (b('dg0_v_vv') <= -0.9722409248352051) ?  -0.032227327366757735 :   -0.039227334102082595 :   (b('dg0_v_vv') <= -0.9430065155029297) ?  0.015879620917921486 :   -0.004336493983795532 :   (b('l8b4') <= 472.67169189453125) ?  (b('bulk') <= 147.5) ?  (b('ndvi') <= 3519.5) ?  0.12358410437104166 :   0.05855202048284336 :   (b('dg0_v_vv') <= 0.8193967342376709) ?  0.04763085948037668 :   0.12948520749648032 :   (b('moss') <= 10.802180290222168) ?  (b('l8b5') <= 2364.8363037109375) ?  0.17376093068405224 :   0.06595830681539021 :   (b('moss') <= 23.06063938140869) ?  0.04476970950863265 :   -0.0628799764956651 :   (b('dg0_v_vv') <= 1.6310529708862305) ?  (b('grass') <= 3.7684255838394165) ?  (b('l8b11') <= 1185.2977294921875) ?  (b('ndvi') <= 2986.0) ?  (b('dg0_v_vv') <= 0.6556625366210938) ?  -0.012913334428953926 :   -0.022437913929552578 :   0.006368049789848396 :   (b('l8b4') <= 1423.8170166015625) ?  (b('dg0_v_vv') <= -0.11284351348876953) ?  (b('l8b5') <= 2999.2117919921875) ?  0.06012408920979326 :   0.04675526611955162 :   (b('dg0_v_vv') <= 0.8625345230102539) ?  0.09028183331489337 :   0.05157238166586301 :   (b('dg0_v_vv') <= 1.549391746520996) ?  (b('dg0_v_vv') <= 1.418116569519043) ?  0.009132327190504275 :   -0.0035845690071788228 :   -0.01758456012608675 :   (b('grass') <= 86.5132827758789) ?  (b('dg0_v_vv') <= 0.3473520278930664) ?  (b('l8b4_med') <= 641.7774353027344) ?  (b('l8b5') <= 2816.5213623046875) ?  -0.008856715952591313 :   0.23216962831267351 :   (b('l8b5_med') <= 2943.271484375) ?  -0.03066885435927033 :   -0.056876737048704364 :   (b('l8b11_med') <= 1976.3319702148438) ?  (b('dg0_v_vv') <= 0.3527836799621582) ?  0.11610069283986074 :   -0.021599707126195397 :   (b('l8b4_med') <= 942.0953674316406) ?  0.13533409368783494 :   -0.004734726390478597 :   (b('l8b4') <= 1084.5861206054688) ?  (b('l8b11') <= 1438.046875) ?  (b('l8b11') <= 1370.0924682617188) ?  0.07237810322605015 :   0.20418652548640184 :   (b('l8b11') <= 1655.5532836914062) ?  0.024239697237013014 :   0.08643418429869462 :   (b('ndvi') <= 2939.5) ?  (b('l8b4') <= 1303.0376586914062) ?  -0.01484003248871304 :   -0.05841495562689967 :   (b('ndvi_med') <= 1936.5) ?  0.0020310462869547186 :   0.07192329522498064 :   (b('l8b4_med') <= 1057.7998657226562) ?  (b('moss') <= 24.86649990081787) ?  (b('l8b5_med') <= 2805.8912353515625) ?  (b('l8b5') <= 3252.0723876953125) ?  (b('l8b11') <= 1190.4302368164062) ?  -0.10499042624956353 :   0.026894018528833344 :   (b('l8b11_med') <= 1810.21044921875) ?  0.07225395471290696 :   0.2393566987995631 :   (b('l8b5') <= 2477.9954833984375) ?  (b('l8b5_med') <= 2952.895751953125) ?  -0.00745174076202882 :   0.07815114123671833 :   (b('l8b4_med') <= 694.1253356933594) ?  -0.09211785637805298 :   -0.018593766547176975 :   (b('dg0_v_vv') <= 1.6683030128479004) ?  0.17369652495635313 :   (b('crops') <= 12.621294021606445) ?  (b('dg0_v_vv') <= 3.967568874359131) ?  -0.08858344973562336 :   -0.04420726933748174 :   0.03809703887597843 :   (b('l8b5_med') <= 3062.42626953125) ?  (b('dg0_v_vv') <= 2.2276358604431152) ?  (b('l8b5_med') <= 1899.461669921875) ?  (b('l8b4') <= 1062.0275268554688) ?  0.018851638141636116 :   -0.05181135033603438 :   (b('l8b5_med') <= 2197.1087646484375) ?  0.12598571195807332 :   0.04383968155401671 :   (b('l8b11') <= 1836.40234375) ?  (b('moss') <= 26.61461353302002) ?  0.11085403210260056 :   0.03628770644317109 :   (b('bulk') <= 153.5) ?  0.07256328208748225 :   -0.013537831621806487 :   (b('bulk') <= 149.5) ?  (b('l8b4_med') <= 1889.65869140625) ?  -0.005416439967425191 :   0.0018931780077180993 :   (b('l8b4') <= 1194.8609619140625) ?  (b('l8b4') <= 1143.3157958984375) ?  -0.12194070244162811 :   -0.0995271775044014 :   (b('bulk') <= 153.5) ?  -0.07008724617265961 :   -0.03572222001636105 :   (b('moss') <= 8.85230016708374) ?  (b('l8b11_med') <= 2935.290283203125) ?  (b('crops') <= 86.48950958251953) ?  (b('crops') <= 77.47303771972656) ?  (b('l8b5_med') <= 3193.2320556640625) ?  (b('dg0_v_vv') <= 1.756382942199707) ?  (b('ndvi_med') <= 1953.0) ?  (b('moss') <= 8.482120990753174) ?  -0.036233052616480826 :   0.0201125467954435 :   (b('sand') <= 24.0) ?  0.06623765507157788 :   -0.00819559569723617 :   (b('ndvi') <= 2584.5) ?  (b('dg0_v_vv') <= 6.189727544784546) ?  0.021794150988895592 :   0.21834487369491656 :   (b('ndvi') <= 3068.5) ?  0.09305196672716828 :   0.16702923941569253 :   (b('crops') <= 68.03106307983398) ?  (b('dg0_v_vv') <= 0.7727665901184082) ?  (b('l8b5') <= 2814.14501953125) ?  0.14698413061713533 :   0.19116586141895298 :   0.22379919976718918 :   (b('dg0_v_vv') <= -1.3552403450012207) ?  (b('l8b5_med') <= 3259.750732421875) ?  -0.04872360254087274 :   0.0169645254921912 :   (b('l8b4') <= 1859.3367919921875) ?  0.02564205865881671 :   0.0768183008611646 :   (b('dg0_v_vv') <= 2.311645030975342) ?  (b('ndvi') <= 1138.0) ?  (b('l8b11') <= 2747.58349609375) ?  (b('dg0_v_vv') <= 1.0764970779418945) ?  -0.11358637357827595 :   -0.1202454035175162 :   (b('l8b5_med') <= 3066.3553466796875) ?  -0.10233923320292455 :   -0.1076926704135741 :   (b('l8b5') <= 2867.196533203125) ?  (b('l8b5') <= 2836.09375) ?  -0.05736568912217647 :   0.018094375962187794 :   (b('l8b5') <= 3122.2880859375) ?  -0.0839087051896843 :   -0.06676146748186508 :   (b('ndvi') <= 2452.5) ?  (b('l8b11') <= 2789.60205078125) ?  (b('sand') <= 46.5) ?  -0.028345111924767144 :   -0.09408371375958063 :   (b('l8b5') <= 3233.626953125) ?  -0.000751787193096217 :   0.08358798944672471 :   (b('l8b5') <= 3414.4903564453125) ?  (b('ndvi_med') <= 1766.0) ?  0.013512659431408669 :   -0.07848733702211497 :   (b('dg0_v_vv') <= 3.034999370574951) ?  -0.08583478408318615 :   -0.10508648861074205 :   (b('ndvi') <= 2217.0) ?  (b('dg0_v_vv') <= 1.1693487167358398) ?  (b('bulk') <= 152.0) ?  (b('l8b4') <= 2307.929443359375) ?  (b('l8b4') <= 2126.1290283203125) ?  0.017276656385993898 :   -0.03462260790714696 :   (b('l8b5') <= 3164.1209716796875) ?  0.09345366408743602 :   -0.004849854182822623 :   (b('l8b5') <= 2760.1500244140625) ?  (b('l8b4') <= 1537.0585327148438) ?  -0.006319289948554259 :   0.07288876263204597 :   (b('l8b5') <= 3375.09521484375) ?  -0.03888352569432811 :   -0.017949711378040696 :   (b('sand') <= 22.5) ?  0.23664314121984398 :   (b('l8b5') <= 2762.9464111328125) ?  (b('ndvi') <= 1859.5) ?  0.0796354260386102 :   9.336239596703193e-05 :   (b('l8b11') <= 3002.1644287109375) ?  0.03377328969852278 :   -0.00269863327907161 :   (b('l8b5') <= 2850.947998046875) ?  (b('dg0_v_vv') <= -0.9098787307739258) ?  (b('l8b5_med') <= 2877.812744140625) ?  (b('dg0_v_vv') <= -3.652944564819336) ?  0.01951399429925929 :   0.018013991259422407 :   (b('l8b5_med') <= 3025.82421875) ?  0.07444779282938171 :   0.01772658538632141 :   (b('l8b4') <= 1489.6334228515625) ?  0.1753012799521568 :   (b('dg0_v_vv') <= 1.3036751747131348) ?  0.09513771896559166 :   0.07103045504188539 :   (b('l8b11') <= 2239.3875732421875) ?  (b('l8b11') <= 1930.1786499023438) ?  (b('ndvi') <= 3077.5) ?  0.05839642869825121 :   0.03661077479551533 :   (b('l8b4_med') <= 1927.28564453125) ?  -0.04083441919529363 :   -0.023812151768225556 :   (b('dg0_v_vv') <= 0.7406196594238281) ?  (b('l8b4') <= 2171.085693359375) ?  0.03139895683336044 :   0.06855704164505666 :   (b('dg0_v_vv') <= 0.9035983085632324) ?  0.1597947042347797 :   0.059658955851196536 :   (b('crops') <= 58.6531925201416) ?  (b('dg0_v_vv') <= 0.6470198631286621) ?  (b('bulk') <= 144.0) ?  (b('dg0_v_vv') <= -0.004704475402832031) ?  0.11630091115077243 :   -0.046456902946151596 :   (b('ndvi') <= 1444.0) ?  (b('l8b11') <= 3367.1812744140625) ?  (b('ndvi') <= 673.5) ?  -0.04360803335136912 :   -0.08293124979681074 :   (b('l8b11') <= 3380.2904052734375) ?  -0.012392484128964043 :   -0.04860443424113386 :   (b('ndvi') <= 1894.0) ?  (b('dg0_v_vv') <= -0.2607851028442383) ?  -0.021600404122720643 :   0.009029580342453039 :   (b('l8b5_med') <= 3264.3426513671875) ?  -0.07114422906542606 :   -0.05564422745610065 :   (b('l8b4') <= 1715.7947387695312) ?  (b('ndvi') <= 1443.5) ?  -0.028037197967203803 :   -0.042884666723313786 :   (b('crops') <= 31.469197273254395) ?  (b('l8b11') <= 3216.3890380859375) ?  (b('l8b4') <= 2424.677001953125) ?  0.06254287000764883 :   0.06654287811388052 :   (b('ndvi') <= 956.0) ?  -0.03822346932516826 :   0.020267193629342102 :   0.10463675671870934 :   (b('l8b11') <= 2649.51416015625) ?  (b('l8b11') <= 2568.4312744140625) ?  (b('l8b4') <= 1539.8984375) ?  (b('l8b4') <= 1527.361572265625) ?  (b('ndvi') <= 1139.5) ?  -0.0551105881043291 :   -0.046368815804769536 :   -0.06754388056274359 :   (b('l8b11_med') <= 3108.688720703125) ?  (b('ndvi') <= 1157.0) ?  -0.09405209122847917 :   -0.08511439769627664 :   (b('ndvi') <= 1455.5) ?  -0.09405209122847917 :   -0.10454388103958075 :   (b('dg0_v_vv') <= -0.4905071258544922) ?  (b('l8b5_med') <= 3217.6090087890625) ?  -0.060239009484482145 :   -0.04423901058716807 :   (b('dg0_v_vv') <= 2.482607364654541) ?  (b('dg0_v_vv') <= 2.0523643493652344) ?  -0.02222096208560436 :   -0.0334522085865198 :   -0.046452205129450405 :   (b('l8b11') <= 3053.1263427734375) ?  (b('l8b5') <= 3760.158203125) ?  (b('ndvi') <= 1640.0) ?  (b('dg0_v_vv') <= 1.6982059478759766) ?  -0.09854388098742665 :   -0.10720291063361075 :   (b('l8b5_med') <= 3217.6090087890625) ?  -0.08916888073131299 :   -0.09569798376704192 :   (b('ndvi') <= 1412.5) ?  (b('dg0_v_vv') <= -1.64485502243042) ?  -0.06554388023491804 :   -0.09921054745292768 :   (b('dg0_v_vv') <= 1.9720072746276855) ?  -0.06045863901923354 :   -0.048202914001273175 :   (b('ndvi') <= 3591.0) ?  (b('l8b11') <= 3719.512451171875) ?  (b('l8b11') <= 3532.5390625) ?  -0.07926304270866745 :   -0.09694074458794225 :   (b('dg0_v_vv') <= -0.17497920989990234) ?  -0.08813025993278359 :   -0.0323709085301642 :   -0.02123900867981944 :   (b('l8b5_med') <= 3243.8726806640625) ?  (b('crops') <= 67.45035552978516) ?  (b('dg0_v_vv') <= 1.58306884765625) ?  (b('l8b5') <= 1987.6564331054688) ?  (b('dg0_v_vv') <= 0.8500161170959473) ?  (b('ndvi_med') <= 846.0) ?  (b('moss') <= 24.206185340881348) ?  -0.027854531337087013 :   -0.09043683768270161 :   (b('l8b11') <= 2295.302490234375) ?  0.0466368323737792 :   -0.005216214033806169 :   (b('sand') <= 56.0) ?  0.08191623509929222 :   0.18336313393401923 :   (b('ndvi') <= 1281.0) ?  (b('bulk') <= 149.5) ?  (b('ndvi') <= 849.0) ?  -0.03666828549029153 :   -0.054717129827774776 :   (b('l8b11_med') <= 2941.146240234375) ?  -0.07140886986991456 :   -0.052524017846633776 :   (b('ndvi_med') <= 1538.25) ?  (b('ndvi') <= 1465.5) ?  -0.03922347153956588 :   -0.011986857686539905 :   (b('grass') <= 23.610958099365234) ?  0.08037764297757292 :   -0.06324947093646673 :   (b('bulk') <= 144.5) ?  (b('ndvi_med') <= 1398.0) ?  (b('ndvi') <= 983.0) ?  -0.07412459995915285 :   (b('l8b4_med') <= 1841.61572265625) ?  0.08641071766724864 :   0.024440875865101658 :   (b('l8b5') <= 2827.76708984375) ?  (b('l8b11') <= 2066.3561401367188) ?  -0.07512459826042048 :   -0.056014281024281215 :   -0.015831426937085114 :   (b('l8b11_med') <= 2707.0025634765625) ?  (b('dg0_v_vv') <= 1.991711139678955) ?  (b('l8b4') <= 1634.3681030273438) ?  0.06294611536180895 :   -0.03905817963510384 :   (b('l8b11_med') <= 2255.6011962890625) ?  -0.04636400563306101 :   -0.08728551848770898 :   (b('l8b11') <= 2434.393310546875) ?  (b('dg0_v_vv') <= 2.587805986404419) ?  0.029165425538574596 :   0.06818082279383034 :   (b('dg0_v_vv') <= 4.572864532470703) ?  -0.016894161490319533 :   0.03761524056330882 :   (b('l8b4') <= 1656.4095458984375) ?  (b('dg0_v_vv') <= 0.900352954864502) ?  (b('dg0_v_vv') <= 0.40377283096313477) ?  (b('ndvi') <= 1046.0) ?  -0.0670579642902038 :   (b('dg0_v_vv') <= 0.38795995712280273) ?  0.036654476262369014 :   -0.03321874236731609 :   (b('dg0_v_vv') <= 0.49266815185546875) ?  0.10111987472288769 :   (b('dg0_v_vv') <= 0.5811104774475098) ?  0.09011986733191174 :   0.08203095655744645 :   (b('l8b4') <= 1556.4546508789062) ?  (b('ndvi') <= 1694.5) ?  (b('ndvi_med') <= 1481.5) ?  -0.031142921903911318 :   -0.026048574858793416 :   (b('ndvi_med') <= 1481.5) ?  -0.018493335484104367 :   -0.02461500084761481 :   (b('dg0_v_vv') <= 1.6173005104064941) ?  -0.032887045626277284 :   (b('l8b4_med') <= 1656.4095458984375) ?  -0.005707625513847223 :   -0.01499442450217553 :   (b('dg0_v_vv') <= 1.329604148864746) ?  (b('l8b5') <= 3169.7298583984375) ?  (b('ndvi') <= 1252.5) ?  (b('l8b4_med') <= 1656.4095458984375) ?  -0.03952996263854051 :   -0.04688946790956096 :   (b('dg0_v_vv') <= -0.9611120223999023) ?  -0.042080895292659376 :   -0.02387271769656616 :   (b('ndvi') <= 1591.0) ?  (b('l8b11_med') <= 2881.5589599609375) ?  -0.003639911278413256 :   -0.01849035044500133 :   (b('l8b5') <= 3495.8150634765625) ?  0.014052405459686413 :   0.1121628728390549 :   (b('l8b5') <= 2707.427490234375) ?  (b('dg0_v_vv') <= 4.747189044952393) ?  (b('ndvi') <= 1165.0) ?  0.031953649432501535 :   0.09603141706647692 :   (b('l8b4_med') <= 1649.88818359375) ?  0.007727383616295741 :   0.032191022415052756 :   (b('dg0_v_vv') <= 1.557413101196289) ?  (b('ndvi_med') <= 1480.5) ?  0.01209655376826635 :   0.05509655150328985 :   (b('dg0_v_vv') <= 1.8867926597595215) ?  -0.024742886725949084 :   0.0020965484038483195 :   (b('crops') <= 9.474358558654785) ?  (b('dg0_v_vv') <= 0.7344307899475098) ?  (b('l8b4') <= 3137.3089599609375) ?  (b('ndvi') <= 1035.5) ?  (b('l8b4') <= 2904.8929443359375) ?  (b('ndvi') <= 649.5) ?  0.053792256318677345 :   -0.022864585522302732 :   (b('l8b11') <= 3904.3516845703125) ?  -0.06967390007449821 :   -0.03598238880878655 :   (b('ndvi') <= 1592.5) ?  (b('moss') <= 20.744579315185547) ?  -0.07312176117476106 :   -0.04842479344094488 :   (b('ndvi') <= 1769.5) ?  0.009620626872971196 :   -0.03358034530609119 :   (b('ndvi') <= 717.5) ?  0.03514510415232952 :   (b('ndvi') <= 1262.5) ?  (b('l8b11') <= 3818.3399658203125) ?  -0.04716738746898516 :   -0.0689165844185684 :   (b('dg0_v_vv') <= 0.31328868865966797) ?  -0.0028739480066184153 :   -0.055873951106059944 :   (b('l8b11') <= 3467.724609375) ?  (b('l8b4') <= 2598.625) ?  (b('dg0_v_vv') <= 1.8650827407836914) ?  (b('l8b11_med') <= 3502.8905029296875) ?  0.01165975812030104 :   -0.02179574477067787 :   (b('l8b4') <= 2503.04833984375) ?  0.009745887686410127 :   0.030709751277687825 :   (b('l8b11') <= 3316.7025146484375) ?  0.03893257872144498 :   0.020614516211466566 :   (b('l8b5') <= 4201.893798828125) ?  (b('l8b11') <= 3581.0625) ?  (b('bulk') <= 152.5) ?  -0.09247122299533941 :   -0.06551719725841926 :   (b('l8b5') <= 4176.78369140625) ?  -0.03357545196937617 :   -0.08559022972523458 :   (b('l8b11') <= 4131.8162841796875) ?  (b('l8b11_med') <= 3982.898193359375) ?  -0.005037883847710219 :   0.014240538823152307 :   (b('dg0_v_vv') <= 1.0189199447631836) ?  -0.006157044013502297 :   -0.05551807734861737 :   (b('l8b5_med') <= 3339.2752685546875) ?  (b('crops') <= 46.23920249938965) ?  (b('ndvi') <= 1203.5) ?  (b('ndvi') <= 1090.0) ?  (b('l8b11') <= 2583.6632080078125) ?  0.015358786693160634 :   0.03946390137900718 :   (b('l8b11') <= 2854.3857421875) ?  -0.004755323246497421 :   0.007824120082098235 :   (b('l8b5') <= 3727.572021484375) ?  (b('l8b11') <= 2935.1094970703125) ?  -0.06419616703636302 :   -0.042216860893835195 :   (b('dg0_v_vv') <= 1.152116298675537) ?  -0.10385931761160529 :   -0.06777598357188318 :   (b('ndvi_med') <= 1726.0) ?  (b('l8b11') <= 2614.08984375) ?  (b('dg0_v_vv') <= 1.0325212478637695) ?  -0.08253752626267201 :   -0.0707292357952132 :   (b('l8b11') <= 3192.9940185546875) ?  -0.09964029388482153 :   -0.0878645292408498 :   (b('l8b11') <= 4075.3177490234375) ?  (b('l8b5') <= 3317.4764404296875) ?  -0.08222868487158513 :   -0.07470068766656722 :   (b('dg0_v_vv') <= -1.911313533782959) ?  -0.04986651044787947 :   -0.05886651006044928 :   (b('dg0_v_vv') <= 1.3677406311035156) ?  (b('l8b5') <= 3096.8826904296875) ?  (b('l8b5') <= 2992.95263671875) ?  (b('crops') <= 20.952744483947754) ?  -0.05999055800170705 :   -0.09124908528535179 :   (b('l8b11') <= 3751.3763427734375) ?  -0.06239266598801265 :   -0.09384016267116264 :   (b('l8b4_med') <= 2095.435791015625) ?  (b('l8b11') <= 4316.35546875) ?  -0.0923447701854026 :   -0.06655046509973245 :   (b('dg0_v_vv') <= 0.06026792526245117) ?  -0.10030265087775662 :   -0.09430898580154215 :   (b('dg0_v_vv') <= 1.3973312377929688) ?  (b('ndvi_med') <= 2040.5) ?  (b('crops') <= 20.952744483947754) ?  -0.028433184998072625 :   -0.02956715443162039 :   -0.03343318768028164 :   (b('ndvi_med') <= 1876.0) ?  (b('ndvi') <= 1742.0) ?  -0.0806318728719606 :   -0.05955356510138858 :   (b('ndvi') <= 1803.5) ?  -0.061148579834210764 :   -0.09812518235761325 :   (b('grass') <= 46.14527893066406) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('sand') <= 20.5) ?  (b('dg0_v_vv') <= 0.18925762176513672) ?  (b('l8b11_med') <= 1264.1098022460938) ?  (b('l8b4') <= 450.18426513671875) ?  (b('ndvi_med') <= 4103.5) ?  (b('ndvi') <= 4592.0) ?  0.16043123246397475 :   0.1427925227396757 :   0.18212073238412338 :   (b('ndvi') <= 3964.0) ?  (b('bulk') <= 146.5) ?  (b('ndvi') <= 3738.5) ?  0.11819570734232102 :   0.11615918472971473 :   0.11343201482019871 :   (b('ndvi') <= 4402.0) ?  (b('l8b11') <= 1438.0007934570312) ?  -0.0015838165460904385 :   -0.026055995656681707 :   (b('l8b4') <= 662.0132141113281) ?  0.028511048481546675 :   0.06450084642612566 :   (b('l8b4') <= 551.4732055664062) ?  (b('dg0_v_vv') <= -0.4490194320678711) ?  (b('l8b11') <= 1174.304931640625) ?  (b('l8b5') <= 3510.7490234375) ?  0.02525550961643458 :   -0.0375462272103045 :   0.13524216812453518 :   (b('l8b5_med') <= 2894.9593505859375) ?  0.09234053185306773 :   (b('dg0_v_vv') <= -0.04832315444946289) ?  0.09834391673665493 :   0.10234055211864695 :   (b('ndvi') <= 4651.0) ?  (b('l8b11') <= 1045.6509704589844) ?  0.25092420312020774 :   (b('l8b5_med') <= 2894.9593505859375) ?  0.1505964580557404 :   0.11047903690873309 :   (b('l8b4') <= 1117.5633239746094) ?  (b('l8b5') <= 3896.2171630859375) ?  0.2599926141597808 :   0.2722894214494681 :   0.21094462253759486 :   (b('ndvi') <= 4721.0) ?  (b('ndvi') <= 2905.0) ?  (b('l8b4') <= 867.5180358886719) ?  (b('l8b5') <= 3227.6739501953125) ?  (b('l8b5') <= 3017.7572021484375) ?  0.16601355023283976 :   0.186920705094415 :   (b('l8b11') <= 1507.9556274414062) ?  0.24624566246997678 :   0.2712083360590273 :   (b('l8b5') <= 2219.6649169921875) ?  0.21856991147250912 :   (b('l8b11') <= 1796.1363525390625) ?  0.0071914858187654696 :   0.017848804605708396 :   (b('l8b11') <= 983.6965942382812) ?  (b('crops') <= 37.60532188415527) ?  0.22589440843291678 :   (b('ndvi') <= 4302.5) ?  0.1250956532653935 :   0.0834902947437276 :   (b('ndvi') <= 4518.0) ?  (b('ndvi') <= 3548.5) ?  0.24380315877017056 :   0.20333539925557828 :   (b('l8b11') <= 1096.9569702148438) ?  0.27699978320331975 :   0.2542799014629308 :   (b('sand') <= 18.0) ?  (b('l8b4_med') <= 622.3268127441406) ?  (b('l8b11_med') <= 1215.2675170898438) ?  0.12560442372749436 :   0.13930447190542705 :   0.19589438935943046 :   (b('l8b5_med') <= 2894.9593505859375) ?  0.0891569649469365 :   0.028156960536192793 :   (b('crops') <= 16.61120891571045) ?  (b('moss') <= 0.3583333194255829) ?  (b('ndvi_med') <= 4183.0) ?  (b('dg0_v_vv') <= 0.4839167594909668) ?  (b('l8b5') <= 1960.4208374023438) ?  (b('l8b11') <= 1487.94677734375) ?  0.08186643098295507 :   0.12335088384088966 :   (b('l8b4') <= 501.0332946777344) ?  0.10158754368015743 :   0.015044362790737023 :   (b('l8b4') <= 991.4561767578125) ?  (b('l8b11') <= 1086.2581787109375) ?  0.12299666013993264 :   0.0816012693467027 :   (b('l8b4') <= 1118.171630859375) ?  0.15508147427307653 :   0.18764587678512273 :   (b('l8b5') <= 2677.8988037109375) ?  (b('l8b4') <= 727.2212829589844) ?  (b('l8b5') <= 2377.259765625) ?  0.03158545875420671 :   0.024328928294382046 :   0.14258546018471818 :   (b('dg0_v_vv') <= 0.6446273326873779) ?  (b('l8b5') <= 2852.0574951171875) ?  0.18289491802259428 :   0.20045304378889484 :   0.1644646135869044 :   (b('l8b5_med') <= 2316.7889404296875) ?  (b('dg0_v_vv') <= -0.11491060256958008) ?  (b('ndvi') <= 3143.5) ?  0.14833186592906714 :   0.13234224201979175 :   (b('ndvi') <= 3437.5) ?  0.1712768680199025 :   0.18137749383406293 :   (b('ndvi') <= 2919.5) ?  (b('l8b11') <= 1386.7245483398438) ?  (b('l8b5') <= 2844.4661865234375) ?  0.05334744112287712 :   0.1549243276293125 :   (b('dg0_v_vv') <= 1.7208151817321777) ?  -0.03682505830351869 :   0.12295317834797329 :   (b('grass') <= 32.56171989440918) ?  (b('dg0_v_vv') <= 0.37894582748413086) ?  -0.011889020915941578 :   0.038613059467813744 :   (b('l8b11_med') <= 1148.4629516601562) ?  -0.022308565165207992 :   -0.04990925494786086 :   (b('dg0_v_vv') <= -0.5553703308105469) ?  (b('ndvi_med') <= 4589.0) ?  (b('l8b5_med') <= 2510.6680908203125) ?  (b('l8b5') <= 3719.327880859375) ?  (b('ndvi') <= 4508.0) ?  -0.07558956402941719 :   -0.08134552317520641 :   (b('l8b4_med') <= 479.81861877441406) ?  -0.05134524113911267 :   -0.01913483017847316 :   (b('bulk') <= 153.0) ?  (b('l8b11_med') <= 1383.3305053710938) ?  0.0221089001945182 :   0.052727588085437704 :   (b('dg0_v_vv') <= -0.9685029983520508) ?  0.08535617253425117 :   0.1912306326077822 :   (b('l8b4') <= 853.5129089355469) ?  (b('dg0_v_vv') <= -1.2602524757385254) ?  (b('dg0_v_vv') <= -2.7786102294921875) ?  0.25048086588084695 :   0.1231609642547409 :   (b('dg0_v_vv') <= -0.6678624153137207) ?  0.23036512760242228 :   0.2523493748043061 :   (b('dg0_v_vv') <= -0.7733373641967773) ?  -0.01166801490264624 :   -0.019668016213948425 :   (b('ndvi_med') <= 3930.75) ?  (b('l8b5') <= 4743.040771484375) ?  (b('ndvi') <= 2074.0) ?  (b('moss') <= 7.712578773498535) ?  0.12678828791850433 :   0.05356549385747912 :   (b('moss') <= 11.788260459899902) ?  0.044883875151191355 :   0.09014202880834678 :   (b('l8b4') <= 516.1160736083984) ?  (b('ndvi') <= 3852.0) ?  -0.008341899000929751 :   -0.03634208994752684 :   (b('dg0_v_vv') <= 1.724630355834961) ?  -0.08387060994386458 :   -0.09571210096641075 :   (b('sand') <= 53.0) ?  (b('dg0_v_vv') <= 1.0617618560791016) ?  (b('l8b5') <= 3491.671142578125) ?  0.11360319536664526 :   0.0741638347517113 :   (b('crops') <= 27.214961051940918) ?  -0.1411483585876241 :   0.1447077496137603 :   (b('ndvi') <= 2510.5) ?  (b('ndvi_med') <= 4384.0) ?  0.20143870823085308 :   0.288823399606735 :   (b('dg0_v_vv') <= 2.91914701461792) ?  0.048681297048623354 :   0.1273404409518345 :   (b('moss') <= 6.192884922027588) ?  (b('bulk') <= 154.0) ?  (b('dg0_v_vv') <= -0.9828248023986816) ?  (b('dg0_v_vv') <= -2.518155574798584) ?  (b('sand') <= 62.5) ?  (b('l8b5_med') <= 3229.48779296875) ?  (b('dg0_v_vv') <= -3.283809185028076) ?  -0.060487945167864855 :   -0.02934002508003805 :   (b('l8b4') <= 1337.1279296875) ?  -0.07799566921727719 :   -0.09176180261161804 :   (b('l8b5') <= 3307.3251953125) ?  (b('ndvi') <= 3628.5) ?  0.02031797399119282 :   0.06704956527140081 :   (b('l8b5') <= 4213.56494140625) ?  -0.06521606950214777 :   0.009338832029907068 :   (b('grass') <= 39.1515007019043) ?  (b('ndvi') <= 2476.0) ?  0.14409730675617183 :   (b('ndvi') <= 4914.0) ?  0.020830122743066132 :   0.07280764638319277 :   (b('dg0_v_vv') <= -1.2747316360473633) ?  (b('ndvi') <= 3192.0) ?  -0.041554439278437716 :   -0.00420706490354196 :   -0.092521915500232 :   (b('grass') <= 39.1515007019043) ?  (b('l8b11_med') <= 1535.9295043945312) ?  (b('l8b4') <= 818.8710327148438) ?  (b('ndvi') <= 4063.5) ?  -0.02610988153127906 :   0.04464032291109131 :   (b('bulk') <= 148.0) ?  0.04408147914154118 :   0.08735196190414096 :   (b('ndvi') <= 4389.0) ?  (b('bulk') <= 140.0) ?  0.07621924196422514 :   0.12666286377031005 :   (b('dg0_v_vv') <= -0.7383499145507812) ?  -0.045390911255052624 :   0.04009982322204619 :   (b('l8b5') <= 2691.6300048828125) ?  (b('dg0_v_vv') <= 0.7856841087341309) ?  (b('ndvi') <= 3406.0) ?  0.03752313987520242 :   0.03383938963693284 :   0.05571133151475713 :   (b('l8b5') <= 2812.7457275390625) ?  (b('dg0_v_vv') <= 1.826866626739502) ?  -0.06536398943056357 :   -0.10778867274697498 :   (b('ndvi') <= 3667.5) ?  0.023119514073554764 :   -0.02297501306677474 :   (b('dg0_v_vv') <= 1.2097764015197754) ?  (b('ndvi') <= 3206.5) ?  (b('l8b4') <= 1111.6660766601562) ?  (b('l8b4') <= 900.4649658203125) ?  (b('l8b5') <= 3192.10595703125) ?  -0.08509120885149771 :   -0.07760629744069447 :   (b('dg0_v_vv') <= -1.3356990814208984) ?  -0.10088007081782374 :   -0.09797649738711015 :   (b('dg0_v_vv') <= -0.9894304275512695) ?  (b('dg0_v_vv') <= -3.1257119178771973) ?  -0.09051004855228384 :   -0.0643900483174788 :   (b('dg0_v_vv') <= -0.2641324996948242) ?  -0.000637548452110781 :   -0.06373434573154228 :   (b('l8b5') <= 3627.0184326171875) ?  (b('l8b11') <= 1986.397216796875) ?  (b('l8b5') <= 3568.254150390625) ?  -0.05088079074583114 :   0.021955236431849046 :   (b('dg0_v_vv') <= -1.0160093307495117) ?  0.007211369112648913 :   -0.023091265869892953 :   (b('ndvi') <= 3624.0) ?  (b('dg0_v_vv') <= -0.24608325958251953) ?  -0.0637346913592364 :   -0.06573469168706195 :   (b('l8b4_med') <= 1058.0714111328125) ?  -0.06073469273014323 :   -0.06289902762536492 :   (b('l8b4') <= 1368.5966796875) ?  (b('l8b5') <= 3670.959716796875) ?  (b('dg0_v_vv') <= 1.7113986015319824) ?  (b('l8b4') <= 1085.3671569824219) ?  -0.015530925437755527 :   0.01095413460336521 :   (b('ndvi') <= 4213.0) ?  0.023834136514327378 :   0.04206227527930903 :   (b('l8b11') <= 1929.9658813476562) ?  (b('l8b5') <= 3861.6141357421875) ?  -0.05370760563008839 :   -0.017320461867040088 :   -0.008399522249604274 :   0.10144407844362444 :   (b('l8b11') <= 1795.4749755859375) ?  (b('l8b4_med') <= 940.2608337402344) ?  (b('l8b5') <= 2999.9476318359375) ?  (b('l8b5') <= 2488.8875732421875) ?  (b('l8b11_med') <= 1483.2841186523438) ?  (b('l8b11') <= 1257.8451538085938) ?  0.09909833903866822 :   0.09665238683787175 :   (b('l8b11') <= 1336.7672729492188) ?  -0.03381572493563116 :   0.016081975175986817 :   (b('l8b4') <= 1036.4037475585938) ?  (b('l8b11_med') <= 1664.699462890625) ?  -0.02293111769331272 :   -0.04317977126983174 :   (b('moss') <= 15.511473417282104) ?  -0.09182692145694205 :   -0.07165365894145387 :   (b('l8b5_med') <= 2780.8529052734375) ?  (b('l8b11') <= 1103.2518920898438) ?  (b('l8b5') <= 4447.6416015625) ?  -0.06724776745014742 :   -0.030226437224081294 :   (b('dg0_v_vv') <= 0.5060367584228516) ?  -0.05181748541757311 :   0.05000383789716027 :   (b('l8b4') <= 771.7037658691406) ?  (b('l8b4_med') <= 858.581787109375) ?  0.001730335945641139 :   0.03322758894404858 :   (b('ndvi_med') <= 3806.0) ?  -0.004569522710569499 :   -0.06753615539907365 :   (b('dg0_v_vv') <= 0.6985321044921875) ?  (b('l8b4') <= 907.0552673339844) ?  (b('sand') <= 34.5) ?  (b('ndvi') <= 4677.0) ?  0.03926025287206321 :   0.15117900483784844 :   (b('l8b5') <= 3240.1141357421875) ?  -0.04124712932986501 :   0.021733889338648515 :   (b('l8b11') <= 1674.4719848632812) ?  (b('ndvi') <= 3710.0) ?  -0.07412742500273473 :   -0.03796969173259224 :   (b('ndvi') <= 3221.5) ?  0.0767930445939836 :   -0.02413226246436584 :   (b('l8b11_med') <= 1509.7184448242188) ?  (b('l8b11') <= 1638.4361572265625) ?  (b('sand') <= 48.5) ?  -0.035366875216369414 :   -0.0694446705773123 :   0.08156019717638066 :   (b('l8b5') <= 3082.4012451171875) ?  (b('moss') <= 22.444746017456055) ?  0.05256911539064284 :   0.13282224147113966 :   (b('l8b11') <= 1669.0426025390625) ?  0.007633645360743356 :   0.09814715541437347 :   (b('crops') <= 32.00249671936035) ?  (b('l8b5') <= 3214.5106201171875) ?  (b('ndvi') <= 3133.0) ?  (b('ndvi') <= 2971.0) ?  (b('dg0_v_vv') <= -0.6246762275695801) ?  -0.03466308147479149 :   -0.06730980563158889 :   (b('l8b11') <= 1940.7933349609375) ?  -0.10386900115070728 :   -0.0890421779393355 :   (b('l8b5_med') <= 2987.221435546875) ?  (b('dg0_v_vv') <= -0.7714099884033203) ?  -0.06917936791827055 :   -0.03346121374148883 :   (b('dg0_v_vv') <= 1.6204586029052734) ?  -0.006668158214701252 :   0.08421367633176563 :   (b('l8b11') <= 2013.7197265625) ?  (b('ndvi') <= 3129.0) ?  (b('ndvi_med') <= 3524.0) ?  -0.10012488822445234 :   -0.09455852272309227 :   (b('l8b4') <= 1229.5523681640625) ?  -0.08756309507532586 :   -0.05759268173843033 :   (b('l8b5') <= 3252.8468017578125) ?  (b('l8b11_med') <= 1974.8834228515625) ?  -0.09024019745331008 :   -0.08589091515661043 :   (b('dg0_v_vv') <= 0.23404169082641602) ?  -0.06903973412137451 :   -0.05389141782150959 :   (b('ndvi') <= 3141.5) ?  (b('l8b5_med') <= 3233.3369140625) ?  (b('l8b4') <= 1249.0056762695312) ?  (b('dg0_v_vv') <= 2.2899744510650635) ?  -0.019348819339422285 :   0.03367773189465223 :   (b('l8b11_med') <= 1692.3551635742188) ?  0.058562099853656235 :   0.056562110701701585 :   (b('dg0_v_vv') <= 1.1872448921203613) ?  (b('ndvi') <= 2942.0) ?  -0.061027962639194806 :   -0.0393186662735197 :   0.03352280018964865 :   (b('ndvi') <= 3287.0) ?  (b('ndvi_med') <= 3314.75) ?  (b('ndvi') <= 3192.5) ?  0.1715743064166516 :   0.0943417754452546 :   -0.012322271920045036 :   (b('dg0_v_vv') <= -1.8444533348083496) ?  (b('crops') <= 45.14968299865723) ?  -0.05593347081173222 :   -0.10704242452089241 :   (b('l8b11') <= 1930.3327026367188) ?  0.01430008111173799 :   -0.013877331524552023 :   (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 958.7872924804688) ?  (b('l8b11_med') <= 1198.8316650390625) ?  (b('l8b4') <= 449.68695068359375) ?  (b('dg0_v_vv') <= 0.07591009140014648) ?  (b('l8b11') <= 898.7603149414062) ?  (b('l8b11') <= 802.6393737792969) ?  (b('ndvi') <= 4271.5) ?  0.22180114418686017 :   0.25117287278975337 :   (b('dg0_v_vv') <= 0.0039272308349609375) ?  0.18154125233463841 :   0.24779607879874344 :   (b('ndvi_med') <= 3520.75) ?  0.18057082197228097 :   (b('l8b4_med') <= 439.1380310058594) ?  0.07788679624061054 :   0.1119686717051791 :   (b('dg0_v_vv') <= 1.8202075958251953) ?  (b('l8b5') <= 2276.63427734375) ?  (b('ndvi') <= 3093.5) ?  0.25474461366614576 :   0.24712039915678183 :   (b('dg0_v_vv') <= 0.6498103141784668) ?  0.2677960597252571 :   0.2744926207879656 :   (b('dg0_v_vv') <= 2.411674976348877) ?  0.2401408696621656 :   0.22761548030002227 :   (b('grass') <= 53.3387336730957) ?  (b('ndvi') <= 4492.0) ?  0.20840886606586712 :   (b('dg0_v_vv') <= 1.6628389358520508) ?  0.23140886797321575 :   (b('ndvi_med') <= 3910.5) ?  0.22450884504926938 :   0.22650884910238522 :   (b('ndvi') <= 3677.0) ?  (b('dg0_v_vv') <= -0.23407888412475586) ?  (b('dg0_v_vv') <= -0.27887392044067383) ?  0.14764260183123154 :   0.2944456496743477 :   (b('dg0_v_vv') <= 0.0044307708740234375) ?  0.11367341594765352 :   0.14787692485653767 :   (b('crops') <= 6.565573692321777) ?  (b('dg0_v_vv') <= 0.018620014190673828) ?  0.11203904172678143 :   0.07948911709677088 :   (b('l8b5') <= 3303.6009521484375) ?  0.1559217818276114 :   0.13066121362627003 :   (b('l8b4') <= 424.16847229003906) ?  (b('ndvi_med') <= 3671.5) ?  (b('dg0_v_vv') <= -0.8944525718688965) ?  (b('crops') <= 3.734968423843384) ?  (b('ndvi') <= 4009.0) ?  -0.004217183634638799 :   0.07558757623692086 :   (b('ndvi_med') <= 3447.5) ?  -0.08831413225829601 :   -0.08921413408219814 :   (b('l8b11') <= 796.1546325683594) ?  (b('dg0_v_vv') <= 0.1146383285522461) ?  0.06342148889061158 :   0.07661447846936141 :   0.13579233479568822 :   (b('l8b5_med') <= 2987.147216796875) ?  -0.04094172661459802 :   -0.05491217075964347 :   (b('ndvi') <= 4741.0) ?  (b('l8b4') <= 452.011474609375) ?  (b('dg0_v_vv') <= -0.31488609313964844) ?  (b('ndvi') <= 3225.0) ?  0.19009388603407978 :   0.13937297932095974 :   (b('dg0_v_vv') <= 0.9670624732971191) ?  0.18741358539537725 :   0.2267315300207949 :   (b('l8b11') <= 921.1899108886719) ?  0.12129391830592526 :   0.13498539396686438 :   (b('dg0_v_vv') <= 0.29659366607666016) ?  0.020449601475960838 :   0.06877281452715395 :   (b('dg0_v_vv') <= 0.0008573532104492188) ?  (b('ndvi') <= 2661.0) ?  (b('l8b5_med') <= 3107.796630859375) ?  (b('ndvi_med') <= 3599.5) ?  (b('grass') <= 65.67167663574219) ?  (b('l8b11') <= 1814.461669921875) ?  0.1799606476306324 :   0.2410219431749517 :   0.043975414272269114 :   (b('l8b5') <= 2052.0783081054688) ?  -0.020717185512493713 :   (b('l8b11_med') <= 1434.456787109375) ?  0.05335971734693435 :   0.1430569971504169 :   (b('l8b4') <= 1202.4754028320312) ?  (b('l8b5') <= 2675.833251953125) ?  (b('sand') <= 37.0) ?  0.017945831107423454 :   0.03926953182963583 :   (b('dg0_v_vv') <= -1.476236343383789) ?  -0.008048293155935693 :   0.0041222420755491485 :   (b('ndvi') <= 1883.0) ?  0.12230945653631681 :   (b('l8b5') <= 2736.734375) ?  0.04947570326030176 :   0.07895170434066923 :   (b('moss') <= 13.440124988555908) ?  (b('l8b11') <= 1440.6341552734375) ?  (b('l8b4') <= 639.4686584472656) ?  (b('ndvi') <= 4553.0) ?  0.03977031673807029 :   -0.014233991408626377 :   (b('l8b11_med') <= 1298.455078125) ?  0.16317270512182097 :   0.048276639110529464 :   (b('l8b5_med') <= 2571.1346435546875) ?  (b('l8b5') <= 2191.525634765625) ?  -0.08452606998687207 :   -0.04893329198395447 :   (b('l8b11') <= 1830.6926879882812) ?  0.0011401373317004688 :   0.06508289035460836 :   (b('l8b5') <= 2607.827392578125) ?  (b('ndvi_med') <= 3447.5) ?  0.08556174354169166 :   (b('l8b11_med') <= 1198.8316650390625) ?  0.026744288874728017 :   0.028561747237179635 :   (b('ndvi') <= 3336.0) ?  0.050561747117970346 :   (b('ndvi') <= 4234.0) ?  -0.052929302072306755 :   -0.004195668820461758 :   (b('sand') <= 29.5) ?  (b('l8b4') <= 527.5891571044922) ?  (b('ndvi') <= 4541.0) ?  (b('l8b5') <= 2955.1961669921875) ?  0.1036651287732625 :   (b('l8b11') <= 976.9484558105469) ?  0.039381969879097034 :   -0.003987190420144185 :   0.10906772324587571 :   (b('l8b11') <= 1652.4683837890625) ?  (b('dg0_v_vv') <= 1.6870956420898438) ?  (b('dg0_v_vv') <= 1.5659003257751465) ?  0.1413867951081473 :   -0.033832927749438524 :   (b('l8b5') <= 2788.14404296875) ?  0.13705536718588796 :   0.21176937324204495 :   (b('l8b11') <= 2219.4361572265625) ?  (b('ndvi') <= 4055.0) ?  0.19926425824869295 :   0.23838111986794625 :   (b('grass') <= 67.80655670166016) ?  0.17258039023402655 :   0.10494887934956654 :   (b('l8b5') <= 2180.29736328125) ?  (b('l8b5') <= 2094.093017578125) ?  (b('ndvi') <= 3692.0) ?  (b('ndvi') <= 3034.5) ?  0.1625797138577485 :   0.059579834345265315 :   0.2994445422095654 :   (b('l8b4') <= 743.2679443359375) ?  (b('ndvi') <= 4198.5) ?  0.16154836937066172 :   0.23822305363007323 :   (b('ndvi_med') <= 3520.75) ?  0.2551796590247294 :   0.2601796393551966 :   (b('ndvi') <= 2939.0) ?  (b('l8b4_med') <= 743.4407653808594) ?  (b('l8b5') <= 2486.3482666015625) ?  0.24455166956433752 :   0.14988038597624342 :   (b('ndvi_med') <= 3667.0) ?  0.0908737580152347 :   -0.06159909924947789 :   (b('dg0_v_vv') <= 0.5106964111328125) ?  (b('l8b11') <= 1214.87353515625) ?  0.055439744816550666 :   0.0073576502327937655 :   (b('l8b5') <= 3936.1026611328125) ?  0.08112729791561911 :   -0.025914512463281723 :   (b('l8b4_med') <= 636.951416015625) ?  (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b11') <= 1043.1750183105469) ?  (b('l8b4') <= 315.82281494140625) ?  (b('ndvi') <= 4890.5) ?  (b('l8b5_med') <= 2911.697265625) ?  0.2069854580610662 :   0.2555807480194739 :   (b('dg0_v_vv') <= 0.39547252655029297) ?  0.3145774677155023 :   0.2192970093181669 :   (b('l8b4_med') <= 592.59375) ?  (b('l8b4_med') <= 490.34466552734375) ?  0.17978371327145704 :   0.21319702709941568 :   0.14836663797994457 :   (b('ndvi') <= 4498.5) ?  (b('bulk') <= 124.5) ?  (b('dg0_v_vv') <= 1.5718121528625488) ?  0.24591243138481408 :   0.2674316364971201 :   0.21545599010338387 :   (b('l8b4') <= 413.8631896972656) ?  (b('dg0_v_vv') <= 0.29305362701416016) ?  0.28557999040112575 :   0.275282066667335 :   0.26131959223996387 :   (b('l8b4') <= 491.7077331542969) ?  (b('moss') <= 0.7690265774726868) ?  (b('ndvi') <= 2366.0) ?  0.31085834889113906 :   (b('dg0_v_vv') <= -0.12167787551879883) ?  0.2863837279737383 :   0.2682939075291544 :   (b('l8b5_med') <= 3217.3944091796875) ?  (b('dg0_v_vv') <= 0.726557731628418) ?  0.32613728951022214 :   0.3299798456079798 :   (b('dg0_v_vv') <= -0.6841945648193359) ?  0.32030366691199563 :   0.30685073826167336 :   (b('dg0_v_vv') <= 0.48209667205810547) ?  (b('l8b5_med') <= 2786.8970947265625) ?  (b('ndvi') <= 3879.5) ?  0.21879975521678116 :   0.2502673243939172 :   (b('dg0_v_vv') <= -0.4210939407348633) ?  0.2729699969631069 :   0.2579928663854292 :   (b('dg0_v_vv') <= 1.4414210319519043) ?  (b('l8b5_med') <= 2725.892578125) ?  0.30094740549794263 :   0.279874777276071 :   0.24212657768134171 :   (b('l8b4_med') <= 573.9899597167969) ?  (b('l8b4') <= 651.3680419921875) ?  (b('l8b11') <= 1454.7259521484375) ?  (b('grass') <= 55.45222854614258) ?  (b('dg0_v_vv') <= -0.15011835098266602) ?  0.20255466224560134 :   0.22395920328830807 :   (b('l8b5') <= 2981.356201171875) ?  0.2034900778285249 :   0.1828295963126622 :   (b('l8b5') <= 2562.859130859375) ?  (b('dg0_v_vv') <= 0.3657493591308594) ?  0.2237653369074237 :   0.24508610928675462 :   (b('dg0_v_vv') <= -1.7164154052734375) ?  0.2580255854734944 :   0.260125576318221 :   (b('ndvi') <= 3319.5) ?  (b('dg0_v_vv') <= 1.3369603157043457) ?  (b('grass') <= 59.07160568237305) ?  0.1913534830782614 :   0.19986035491432907 :   (b('ndvi_med') <= 3932.5) ?  0.22611867857230622 :   0.18863435300157633 :   (b('grass') <= 52.18928527832031) ?  (b('l8b5') <= 2541.8841552734375) ?  0.19065168364513466 :   0.18652590386303053 :   (b('dg0_v_vv') <= -0.5439906120300293) ?  0.17414471374697882 :   0.16960403713372943 :   (b('dg0_v_vv') <= -0.001407623291015625) ?  (b('dg0_v_vv') <= -0.005527496337890625) ?  (b('ndvi') <= 2892.5) ?  (b('dg0_v_vv') <= -1.9145331382751465) ?  0.22823034326898162 :   0.23443035881387297 :   (b('l8b5') <= 2640.412841796875) ?  0.20415144374370064 :   0.21927724933370749 :   0.17217877817274063 :   (b('l8b4') <= 707.9406433105469) ?  (b('ndvi') <= 3094.5) ?  0.20006752723424012 :   (b('dg0_v_vv') <= 0.5953421592712402) ?  0.22611458191266462 :   0.23905250431565817 :   (b('dg0_v_vv') <= 0.005893707275390625) ?  (b('dg0_v_vv') <= -0.000247955322265625) ?  0.2444727753065122 :   0.27477878642202347 :   (b('l8b11') <= 1540.9984130859375) ?  0.20706878316522567 :   0.2550833482796118 :   (b('sand') <= 55.5) ?  (b('dg0_v_vv') <= -1.4237165451049805) ?  (b('dg0_v_vv') <= -2.4582605361938477) ?  (b('l8b11') <= 2243.4617309570312) ?  (b('dg0_v_vv') <= -3.3520350456237793) ?  -0.06954729278162203 :   -0.05254729558304033 :   -0.08897429210841737 :   (b('l8b4') <= 958.1712951660156) ?  (b('ndvi_med') <= 3757.0) ?  0.008048840903474908 :   0.024344915195471717 :   (b('dg0_v_vv') <= -1.7249770164489746) ?  -0.004805262366685381 :   (b('l8b5_med') <= 3218.4537353515625) ?  -0.02331826184541297 :   -0.014405263940247992 :   (b('dg0_v_vv') <= 2.717564105987549) ?  (b('l8b4') <= 1148.9852294921875) ?  (b('l8b11') <= 1754.746337890625) ?  (b('l8b4') <= 489.19317626953125) ?  0.1731735494684255 :   0.0847364520219661 :   (b('ndvi') <= 3457.5) ?  0.06372950943958489 :   -0.02183171347429434 :   (b('dg0_v_vv') <= -0.36136627197265625) ?  0.19005033982396047 :   (b('ndvi') <= 3109.0) ?  0.23082536620497623 :   0.20682534736990849 :   (b('dg0_v_vv') <= 3.250490427017212) ?  (b('dg0_v_vv') <= 2.935457468032837) ?  0.21596674282255332 :   0.23846674818697136 :   0.1850286968610835 :   (b('l8b5') <= 2405.21044921875) ?  (b('l8b5') <= 2241.7022705078125) ?  (b('l8b11') <= 1012.0128936767578) ?  (b('ndvi_med') <= 3442.5) ?  (b('ndvi_med') <= 3405.0) ?  0.19228446692600848 :   0.1409765421168672 :   (b('ndvi_med') <= 3494.75) ?  0.25767653572724925 :   0.2467580017000437 :   (b('dg0_v_vv') <= -0.31525659561157227) ?  0.2773952465969481 :   0.2748828206838432 :   (b('ndvi') <= 3523.5) ?  (b('l8b4') <= 641.6177062988281) ?  (b('ndvi_med') <= 3442.5) ?  0.043704635149848306 :   -0.0046711821764224 :   (b('dg0_v_vv') <= 1.74916410446167) ?  0.12172016613958962 :   0.15969250239399965 :   (b('l8b5') <= 2315.7037353515625) ?  (b('ndvi') <= 3587.5) ?  -0.03859093649312853 :   -0.022446268989394114 :   (b('dg0_v_vv') <= 0.571326732635498) ?  -0.005561261332701689 :   0.027910875513365407 :   (b('ndvi') <= 3265.5) ?  (b('ndvi') <= 2898.5) ?  (b('ndvi_med') <= 3405.0) ?  0.14703301781085235 :   (b('l8b11') <= 1540.9984130859375) ?  0.2789132102814552 :   0.21551918602125167 :   (b('l8b5') <= 2520.982421875) ?  (b('l8b11') <= 1310.3475341796875) ?  -0.0067794212093783834 :   0.03592057144732938 :   (b('dg0_v_vv') <= -1.0735430717468262) ?  0.08463908089650335 :   0.16108908607018652 :   (b('ndvi') <= 3820.0) ?  (b('l8b11') <= 1283.6010131835938) ?  (b('ndvi') <= 3515.0) ?  0.25695248048588176 :   0.2459130061059715 :   (b('dg0_v_vv') <= -0.8300981521606445) ?  0.23884961647719405 :   0.21157679885013986 :   (b('l8b4') <= 577.5890808105469) ?  (b('dg0_v_vv') <= -0.9074625968933105) ?  0.21165451681398353 :   0.24516419977726583 :   (b('dg0_v_vv') <= 1.8636789321899414) ?  0.19791176369583857 :   0.2617115594577927 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 3272.0) ?  (b('l8b4') <= 1287.954833984375) ?  (b('moss') <= 1.7430431246757507) ?  (b('sand') <= 48.5) ?  (b('bulk') <= 133.0) ?  (b('sand') <= 37.5) ?  (b('ndvi') <= 931.5) ?  (b('l8b11') <= 1686.4376220703125) ?  (b('dg0_v_vv') <= -2.701472282409668) ?  0.2112320739452539 :   0.1944850521509347 :   (b('dg0_v_vv') <= 0.35985374450683594) ?  0.23636205856248715 :   0.23558838345175442 :   (b('l8b11') <= 1814.6522827148438) ?  (b('l8b11') <= 1655.0140380859375) ?  (b('l8b11_med') <= 989.9714660644531) ?  0.0715744569058891 :   0.15172792563050838 :   (b('dg0_v_vv') <= 0.47463560104370117) ?  0.12973392517543184 :   0.08840418347154595 :   (b('l8b11') <= 1994.1318969726562) ?  (b('dg0_v_vv') <= -2.659235954284668) ?  0.2354377645513106 :   0.1984587826964257 :   (b('ndvi_med') <= 1969.25) ?  0.15425995311211077 :   0.1772880187994951 :   (b('ndvi') <= 2238.5) ?  (b('l8b11') <= 1994.1546020507812) ?  (b('ndvi') <= 1919.5) ?  (b('ndvi') <= 1097.5) ?  0.09996629670939504 :   0.11406539731679588 :   (b('ndvi') <= 2017.0) ?  0.0919030581722538 :   0.08176165662033039 :   (b('l8b4_med') <= 827.3872680664062) ?  0.06351569137022497 :   0.029309389509765915 :   (b('dg0_v_vv') <= -1.414680004119873) ?  0.09143394893899634 :   (b('crops') <= 69.70147705078125) ?  0.14793648990322178 :   (b('bulk') <= 123.0) ?  0.18447018947812932 :   0.17428877815972776 :   (b('dg0_v_vv') <= 1.7982110977172852) ?  (b('l8b5_med') <= 2500.130126953125) ?  (b('grass') <= 91.17486190795898) ?  (b('l8b4') <= 672.2311706542969) ?  (b('grass') <= 69.83709335327148) ?  -0.018459502784366873 :   0.08539637016581864 :   (b('l8b5') <= 1999.9428100585938) ?  -0.032138968186861204 :   -0.0712647814518085 :   (b('l8b4') <= 971.8255920410156) ?  (b('dg0_v_vv') <= -0.027410507202148438) ?  0.03132951339850617 :   0.06581260364532983 :   (b('l8b11') <= 2063.260009765625) ?  -0.03389894390239611 :   0.02274881663203208 :   (b('ndvi') <= 2874.5) ?  (b('bulk') <= 152.5) ?  (b('crops') <= 93.49007415771484) ?  0.0705537975781628 :   0.02549201688473722 :   (b('l8b11') <= 1852.0067749023438) ?  -0.04985558459076391 :   0.0173479332893182 :   (b('bulk') <= 147.0) ?  (b('l8b11') <= 1127.6461791992188) ?  0.0854377960934144 :   0.0394509734398845 :   (b('dg0_v_vv') <= -0.40169382095336914) ?  0.06565528136301392 :   0.14582286894226923 :   (b('dg0_v_vv') <= 4.038148880004883) ?  (b('l8b11') <= 2456.398681640625) ?  (b('l8b5') <= 1542.5994873046875) ?  -0.10659295965840848 :   (b('grass') <= 0.5440000146627426) ?  0.010829961611079797 :   0.10312382120684999 :   (b('l8b5_med') <= 2088.8604125976562) ?  -0.08434658754387173 :   -0.05018340279055955 :   (b('l8b5') <= 4106.8990478515625) ?  (b('l8b11') <= 1684.7173461914062) ?  (b('l8b11_med') <= 2141.5521240234375) ?  0.14624442132330875 :   0.23018252788580457 :   (b('ndvi_med') <= 2246.0) ?  0.1012323648897627 :   0.15178523971268063 :   -0.015116630145180776 :   (b('sand') <= 51.0) ?  (b('l8b4') <= 964.2301940917969) ?  (b('dg0_v_vv') <= 0.5436849594116211) ?  (b('l8b4') <= 860.3297729492188) ?  (b('l8b11') <= 1498.2390747070312) ?  -0.07171644995936388 :   (b('dg0_v_vv') <= 0.15793657302856445) ?  -0.0969205407831548 :   -0.10571835879360432 :   (b('l8b4') <= 894.3937072753906) ?  (b('l8b5') <= 2072.7994384765625) ?  -0.05907973451089557 :   -0.034277807614095185 :   (b('dg0_v_vv') <= -1.2107172012329102) ?  -0.06307973516654666 :   -0.08799180647582719 :   (b('ndvi') <= 3317.0) ?  (b('dg0_v_vv') <= 0.6439967155456543) ?  -0.06967364517879798 :   (b('l8b4') <= 738.1320190429688) ?  -0.05967363981437995 :   -0.05140913584944288 :   -0.10631062332883603 :   (b('l8b5') <= 2440.62890625) ?  (b('dg0_v_vv') <= 1.026376724243164) ?  (b('ndvi') <= 2414.0) ?  (b('dg0_v_vv') <= -0.15350723266601562) ?  -0.04217877311885562 :   -0.04601027634853688 :   (b('l8b5_med') <= 2405.40380859375) ?  -0.029277812382466767 :   -0.03627781166721103 :   (b('dg0_v_vv') <= 2.2440643310546875) ?  -0.021010277838653002 :   -0.021816491990094 :   (b('l8b11') <= 1978.9064331054688) ?  -0.04201027569288579 :   -0.0782778111009669 :   (b('sand') <= 53.0) ?  (b('l8b4') <= 661.2547912597656) ?  (b('l8b5') <= 3037.6492919921875) ?  0.24553575639650954 :   0.2346357655994667 :   (b('l8b5') <= 2283.9320068359375) ?  (b('l8b11') <= 2184.1658325195312) ?  (b('l8b4') <= 928.8358764648438) ?  0.06269706741746559 :   0.11433266694325053 :   0.17952460919496824 :   (b('dg0_v_vv') <= 0.8785409927368164) ?  (b('ndvi') <= 2796.0) ?  0.014961560290279346 :   -0.009623017730201078 :   (b('l8b5') <= 2600.025390625) ?  0.10112582770674777 :   0.045011532368987556 :   (b('l8b5') <= 2522.3018798828125) ?  (b('l8b11') <= 1734.7022705078125) ?  (b('ndvi') <= 3205.0) ?  (b('ndvi') <= 1880.0) ?  0.033389868359965394 :   -0.0027017428346029215 :   (b('ndvi_med') <= 1596.25) ?  0.1578996015745693 :   0.061806626637569306 :   (b('dg0_v_vv') <= 1.0763273239135742) ?  (b('l8b5') <= 2452.9532470703125) ?  -0.04307776254019325 :   0.0789265969444876 :   (b('l8b11_med') <= 2318.158935546875) ?  -0.013297237219543942 :   0.06189540310463593 :   (b('l8b11') <= 696.46142578125) ?  (b('moss') <= 0.010067113675177097) ?  (b('dg0_v_vv') <= 2.321920871734619) ?  0.07503576104567183 :   0.08553575997278823 :   (b('dg0_v_vv') <= 3.7106337547302246) ?  0.042335758852220906 :   0.037935758876062764 :   (b('ndvi_med') <= 1596.25) ?  (b('ndvi') <= 3767.5) ?  -0.06852930270662637 :   0.058189085592526835 :   (b('bulk') <= 149.0) ?  -0.004882834846291041 :   -0.0313409258288546 :   (b('bulk') <= 138.5) ?  (b('l8b4_med') <= 638.7227478027344) ?  (b('ndvi_med') <= 2956.25) ?  (b('l8b5') <= 1650.4305419921875) ?  (b('dg0_v_vv') <= 1.7294659614562988) ?  (b('l8b11') <= 1239.1609497070312) ?  (b('dg0_v_vv') <= -0.654778242111206) ?  -0.03680558018960473 :   -0.07130121205230792 :   0.043911878118059716 :   (b('l8b11') <= 874.7621765136719) ?  -0.003503328450281895 :   (b('l8b4') <= 559.6546020507812) ?  0.05743707161754383 :   0.07819905101175831 :   (b('l8b4') <= 580.3703308105469) ?  (b('l8b11') <= 1107.7598266601562) ?  (b('l8b11') <= 1054.6875) ?  0.030011517127607305 :   0.07561355243709285 :   (b('bulk') <= 130.0) ?  0.057703311214816144 :   0.1650950157967995 :   (b('l8b5') <= 1726.0) ?  (b('ndvi') <= 1764.5) ?  0.027110929722117982 :   -0.06698466366411686 :   (b('ndvi') <= 1871.0) ?  0.11296047081167519 :   0.015703783795892535 :   (b('l8b4') <= 527.0768127441406) ?  (b('l8b11') <= 1215.0491943359375) ?  (b('l8b5_med') <= 2813.5113525390625) ?  0.21339974495485386 :   0.27168383317037037 :   0.3423448219163374 :   (b('l8b4_med') <= 581.5994567871094) ?  (b('dg0_v_vv') <= -0.020769119262695312) ?  (b('dg0_v_vv') <= -0.11250114440917969) ?  0.16922777905080097 :   0.16383310133572096 :   (b('dg0_v_vv') <= 0.15629911422729492) ?  0.19388927336046616 :   0.1714241113698053 :   0.08797547889510976 :   (b('grass') <= 6.58846640586853) ?  (b('dg0_v_vv') <= 0.6403923034667969) ?  (b('ndvi') <= 3431.5) ?  (b('l8b4') <= 1175.5679321289062) ?  (b('dg0_v_vv') <= -2.851531982421875) ?  -0.03794078894063152 :   0.005762320084193776 :   (b('ndvi') <= 2234.5) ?  -0.034232724614133764 :   -0.00523409484051654 :   (b('l8b4') <= 968.9979248046875) ?  (b('l8b4_med') <= 980.9122009277344) ?  0.02171977519091654 :   -0.003982395735392369 :   (b('ndvi') <= 3910.0) ?  0.02992878501450074 :   0.06687740373616073 :   (b('l8b5') <= 4668.3271484375) ?  (b('dg0_v_vv') <= 2.7819955348968506) ?  (b('ndvi') <= 2027.5) ?  0.05347216454416881 :   0.025959353090336867 :   (b('dg0_v_vv') <= 4.853342771530151) ?  0.05034660896122539 :   0.13086219054762102 :   (b('ndvi') <= 4519.0) ?  (b('ndvi_med') <= 2662.75) ?  -0.03901685600464011 :   -0.058350188781663426 :   0.10098315204198693 :   (b('ndvi_med') <= 2118.0) ?  (b('moss') <= 12.493750095367432) ?  (b('l8b11') <= 1738.0487670898438) ?  (b('l8b5_med') <= 2034.357177734375) ?  0.04152896985112247 :   0.15976440881009754 :   (b('ndvi') <= 1796.5) ?  -0.09478155237653973 :   -0.01263793261659003 :   (b('sand') <= 41.5) ?  (b('ndvi') <= 2008.0) ?  -0.06313019460796994 :   -0.011074383881549705 :   (b('ndvi') <= 2619.5) ?  0.003807445931543227 :   0.1255426458088375 :   (b('dg0_v_vv') <= 0.988492488861084) ?  (b('bulk') <= 128.0) ?  (b('l8b5') <= 2548.525390625) ?  -0.04693145191897974 :   -0.08336605767406803 :   (b('l8b5') <= 3552.1968994140625) ?  -0.0333500792361126 :   -0.0038481907290027887 :   (b('grass') <= 66.39847946166992) ?  (b('l8b5') <= 2447.3394775390625) ?  0.004285483746260205 :   -0.0249395534116132 :   (b('bulk') <= 128.0) ?  -0.05549471321499112 :   0.05308911070005584 :   (b('moss') <= 26.28097152709961) ?  (b('l8b4') <= 1134.1126708984375) ?  (b('l8b5_med') <= 2383.0186767578125) ?  (b('ndvi_med') <= 1479.5) ?  (b('dg0_v_vv') <= 0.5575966835021973) ?  (b('dg0_v_vv') <= 0.2819023132324219) ?  0.08015946390358891 :   -0.022892364351680468 :   (b('l8b11') <= 2224.3969116210938) ?  0.16759129010637694 :   -0.06979084929484258 :   (b('l8b4') <= 628.1376647949219) ?  (b('grass') <= 69.06616592407227) ?  0.04263306533963145 :   0.139120085189622 :   (b('dg0_v_vv') <= 0.9428491592407227) ?  -0.016688310805307024 :   0.02775440291248247 :   (b('grass') <= 45.16307258605957) ?  (b('moss') <= 3.173942804336548) ?  (b('dg0_v_vv') <= -2.8366427421569824) ?  -0.0767940315508259 :   -0.03269677874783289 :   (b('grass') <= 37.80600547790527) ?  0.04944550377152817 :   -0.03242911395939504 :   (b('l8b4') <= 892.4676818847656) ?  (b('ndvi_med') <= 2973.25) ?  0.09016550572142079 :   0.15228225084615568 :   (b('moss') <= 17.20577621459961) ?  0.027644903669879924 :   0.09988660906811632 :   (b('l8b11_med') <= 2749.34619140625) ?  (b('dg0_v_vv') <= 0.8740842342376709) ?  (b('ndvi') <= 2181.0) ?  (b('crops') <= 0.36066467873752117) ?  -0.04364157288488537 :   -0.014739333397046417 :   (b('l8b4_med') <= 1289.8043212890625) ?  -5.966992791129948e-05 :   0.07708001505914149 :   (b('moss') <= 20.18179416656494) ?  (b('sand') <= 28.0) ?  0.127727925213637 :   0.034937046222624724 :   (b('bulk') <= 145.5) ?  0.10683152328536108 :   -0.05759405203496006 :   (b('l8b11') <= 1850.147216796875) ?  (b('l8b5_med') <= 3153.067626953125) ?  (b('ndvi') <= 2135.0) ?  -0.09471893669539744 :   -0.07728489909013694 :   (b('ndvi') <= 1582.5) ?  -0.05745030651281732 :   -0.10924273790256667 :   (b('ndvi') <= 2264.0) ?  (b('moss') <= 18.607559204101562) ?  -0.03886285123085724 :   -0.07166711064150376 :   (b('moss') <= 5.308910250663757) ?  0.07022435163189451 :   -0.0029787528965512233 :   (b('l8b4_med') <= 1071.31494140625) ?  (b('l8b4') <= 753.6303100585938) ?  (b('dg0_v_vv') <= 0.8675751686096191) ?  (b('l8b4') <= 606.1407165527344) ?  (b('ndvi_med') <= 1674.75) ?  -0.08086124125930061 :   -0.05203495745822527 :   (b('ndvi') <= 2771.0) ?  -0.04002378274296595 :   0.023412217644443206 :   (b('l8b11') <= 1545.4685668945312) ?  (b('l8b4') <= 686.1468811035156) ?  0.02662279692646804 :   -0.06192506808298362 :   0.1563268724607178 :   (b('sand') <= 55.5) ?  (b('moss') <= 32.7097749710083) ?  (b('ndvi') <= 2577.5) ?  -0.058704009071359003 :   -0.030305572680901834 :   (b('l8b4') <= 1041.0762939453125) ?  -0.010551695337369113 :   -0.050205263881076045 :   (b('dg0_v_vv') <= 4.334393501281738) ?  (b('l8b11') <= 1840.2525024414062) ?  -0.04246671894585152 :   -0.09994666022402746 :   (b('dg0_v_vv') <= 4.7211713790893555) ?  -0.05313685778556679 :   -0.01386138667977023 :   (b('dg0_v_vv') <= 0.6740884780883789) ?  (b('l8b5_med') <= 1752.2700805664062) ?  (b('ndvi') <= 1136.0) ?  (b('ndvi') <= 869.0) ?  -0.04808046228637433 :   -0.01157906207885274 :   (b('ndvi') <= 1207.5) ?  -0.10586118487103491 :   -0.08659417054633932 :   (b('l8b5') <= 2864.8524169921875) ?  (b('l8b5') <= 1678.0108032226562) ?  0.02000473049082138 :   -0.019630462364372395 :   (b('moss') <= 28.28955841064453) ?  0.13566031879828824 :   0.04768638627494157 :   (b('bulk') <= 139.5) ?  (b('ndvi') <= 1934.0) ?  0.23678981832600743 :   0.18248255057886537 :   (b('bulk') <= 153.5) ?  (b('ndvi') <= 2222.5) ?  0.005206370342391968 :   0.07416113197098653 :   0.23668773105191057 :   (b('moss') <= 8.85230016708374) ?  (b('l8b11_med') <= 2935.290283203125) ?  (b('dg0_v_vv') <= 1.8657013177871704) ?  (b('crops') <= 86.48950958251953) ?  (b('moss') <= 3.33070170879364) ?  (b('crops') <= 36.34514808654785) ?  (b('ndvi_med') <= 1882.0) ?  (b('l8b11') <= 2193.60693359375) ?  -0.0134798612361894 :   -0.03657559089507897 :   (b('ndvi') <= 1251.5) ?  0.08305241778004467 :   -0.014293429362455925 :   (b('bulk') <= 147.5) ?  (b('l8b4') <= 1926.7157592773438) ?  -0.02055875848912862 :   0.10988931887220843 :   (b('l8b11') <= 2436.2935791015625) ?  -0.09323685518492296 :   -0.05601665170137753 :   (b('moss') <= 3.989913582801819) ?  (b('ndvi') <= 1686.5) ?  (b('l8b5') <= 2210.69287109375) ?  0.11567229747735441 :   0.03735222901481167 :   (b('l8b5') <= 3269.670166015625) ?  0.11812101130637004 :   0.060785795855523336 :   (b('l8b4_med') <= 1832.1223754882812) ?  (b('sand') <= 59.5) ?  -0.009374963237448589 :   -0.06470553225267371 :   (b('l8b4_med') <= 1948.0820922851562) ?  -0.05520089758017572 :   -0.03469011047091109 :   (b('bulk') <= 152.0) ?  (b('ndvi') <= 3850.5) ?  (b('dg0_v_vv') <= 0.651463508605957) ?  (b('l8b5') <= 3664.1351318359375) ?  0.02144308215041766 :   -0.016981996099522067 :   (b('l8b4') <= 2337.050537109375) ?  0.06109516014591074 :   -0.02136596142454201 :   (b('l8b11') <= 1677.4415893554688) ?  (b('l8b5_med') <= 3378.561279296875) ?  0.167830903312008 :   0.1755286254666405 :   (b('l8b4_med') <= 1536.2318115234375) ?  0.10877223363256153 :   -0.022201322129000972 :   (b('bulk') <= 160.5) ?  (b('l8b5') <= 3008.7415771484375) ?  (b('ndvi') <= 1032.5) ?  -0.012085888761020645 :   0.04458070014839292 :   (b('ndvi') <= 1720.0) ?  -0.020436075665197513 :   0.010495302004694186 :   (b('ndvi') <= 1745.5) ?  (b('ndvi') <= 1658.0) ?  -0.03641040485348373 :   -0.06020400240163283 :   (b('ndvi') <= 1821.5) ?  -0.02304281532841506 :   -0.045096751136990736 :   (b('l8b5_med') <= 3660.6177978515625) ?  (b('l8b4') <= 1458.1231689453125) ?  (b('grass') <= 8.057114601135254) ?  (b('moss') <= 0.0533333346247673) ?  -0.10165516464697091 :   (b('l8b5_med') <= 2687.2421875) ?  0.1385432035056704 :   0.029884351744950927 :   (b('l8b4') <= 1297.1775512695312) ?  -0.0759348781347701 :   (b('l8b4_med') <= 1367.7310180664062) ?  0.07574375096985476 :   0.12566984728387529 :   (b('sand') <= 50.5) ?  (b('bulk') <= 149.0) ?  (b('dg0_v_vv') <= 1.9335920810699463) ?  0.16560141186823785 :   0.024824920468895492 :   (b('ndvi') <= 2055.0) ?  0.09523264155209762 :   0.02599302722739517 :   (b('sand') <= 56.5) ?  (b('l8b11') <= 3343.1099853515625) ?  -0.032475988610046226 :   0.04984824480732617 :   (b('l8b5') <= 2724.8516845703125) ?  0.06812591138803366 :   0.022287622814990353 :   (b('l8b11') <= 2674.1968994140625) ?  (b('l8b5') <= 4101.63623046875) ?  (b('dg0_v_vv') <= 3.352621078491211) ?  (b('l8b11') <= 2293.6363525390625) ?  -0.09527560176969753 :   -0.10646524601502204 :   (b('dg0_v_vv') <= 4.053211688995361) ?  -0.057160239149163995 :   -0.09827560185398222 :   (b('dg0_v_vv') <= 2.859360694885254) ?  (b('l8b5') <= 4140.56640625) ?  -0.07999531844723584 :   -0.05873060507407732 :   (b('dg0_v_vv') <= 3.9135684967041016) ?  -0.08377495749912117 :   -0.06156154448892781 :   (b('l8b5') <= 3769.918212890625) ?  (b('ndvi') <= 1752.5) ?  0.06836446854213427 :   (b('l8b11') <= 2931.4639892578125) ?  0.024146667199351962 :   0.03513233004937999 :   (b('ndvi') <= 3222.5) ?  -0.008995316122654695 :   -0.05808798723413856 :   (b('l8b11_med') <= 3132.103515625) ?  (b('ndvi') <= 3967.5) ?  (b('l8b4') <= 1919.3078002929688) ?  (b('dg0_v_vv') <= -0.17208290100097656) ?  (b('l8b11') <= 2274.7880859375) ?  (b('ndvi') <= 1172.5) ?  -0.017319220144511996 :   -0.04341520955859525 :   (b('l8b4') <= 1884.6140747070312) ?  -0.08155238740738285 :   -0.06221289174711532 :   (b('ndvi') <= 2323.5) ?  (b('l8b4_med') <= 1935.1727294921875) ?  -0.040557844593046066 :   -0.0566467707409028 :   (b('l8b5') <= 3602.804931640625) ?  -0.05697701871832324 :   -0.09477141552181703 :   (b('l8b4') <= 2121.2952880859375) ?  (b('l8b5') <= 3568.9945068359375) ?  (b('dg0_v_vv') <= 3.147535562515259) ?  -0.08552669587110637 :   -0.047592566020812535 :   -0.036617574384528215 :   (b('ndvi') <= 1689.5) ?  (b('ndvi') <= 1125.5) ?  -0.058514066755089814 :   -0.07989309688569814 :   (b('l8b11') <= 3780.9508056640625) ?  -0.05792273538607952 :   -0.019667989780254552 :   (b('ndvi') <= 4198.0) ?  (b('dg0_v_vv') <= 0.7131190299987793) ?  (b('l8b5_med') <= 3217.6090087890625) ?  -0.054215108536033924 :   -0.05901691530247963 :   (b('l8b5_med') <= 3217.6090087890625) ?  -0.011016911159956816 :   (b('l8b4_med') <= 1935.1727294921875) ?  -0.018675947161486114 :   -0.017675945134928192 :   (b('l8b5_med') <= 3217.6090087890625) ?  0.0006086658553897395 :   (b('ndvi_med') <= 1482.0) ?  0.022608673186761047 :   0.015084159300211022 :   (b('l8b5') <= 3023.612548828125) ?  (b('l8b5') <= 2981.4073486328125) ?  (b('ndvi') <= 1849.5) ?  (b('dg0_v_vv') <= -0.2680182456970215) ?  (b('l8b5') <= 2071.9451904296875) ?  0.015691702324356765 :   -0.01637232341611236 :   (b('l8b4_med') <= 2132.427001953125) ?  -0.04975135881504974 :   -0.03126887492947067 :   (b('l8b11') <= 2968.7689208984375) ?  0.010876935589173492 :   0.03266214498738468 :   (b('l8b4_med') <= 2286.5834350585938) ?  0.1046708200356952 :   (b('dg0_v_vv') <= 1.0143752098083496) ?  0.05988859030249247 :   0.0438885876798881 :   (b('ndvi') <= 2056.5) ?  (b('dg0_v_vv') <= 0.6470198631286621) ?  (b('l8b11') <= 4024.6578369140625) ?  (b('ndvi_med') <= 604.0) ?  -0.038075423647900675 :   -0.050843185743872484 :   (b('l8b4_med') <= 2158.3577880859375) ?  -0.021793482542491013 :   -0.02492572382349205 :   (b('dg0_v_vv') <= 0.895599365234375) ?  0.038601064689672365 :   (b('dg0_v_vv') <= 1.2941350936889648) ?  -0.03173445528888405 :   -0.05534572893543791 :   0.000676194996317464 :   (b('l8b5_med') <= 3339.2752685546875) ?  (b('crops') <= 72.32853889465332) ?  (b('dg0_v_vv') <= 1.729705810546875) ?  (b('bulk') <= 146.5) ?  (b('bulk') <= 138.5) ?  (b('bulk') <= 137.5) ?  (b('l8b5') <= 2484.858154296875) ?  -0.03871888107901301 :   -0.0645061526442464 :   (b('l8b5') <= 2067.166015625) ?  -0.09432828779174965 :   -0.07288414425405805 :   (b('ndvi') <= 2272.5) ?  (b('l8b5_med') <= 1934.02001953125) ?  -0.07018120936083326 :   -0.027532829736668978 :   (b('grass') <= 88.07605361938477) ?  0.04036382214742679 :   -0.06412745505091377 :   (b('bulk') <= 152.5) ?  (b('ndvi_med') <= 1593.5) ?  (b('l8b11_med') <= 2687.3529052734375) ?  -0.061184866446679406 :   -0.040618792414235265 :   (b('dg0_v_vv') <= 0.4095492362976074) ?  -0.07109850048462804 :   -0.05216992599356239 :   (b('ndvi') <= 2310.5) ?  (b('bulk') <= 153.5) ?  -0.018528965378577424 :   -0.04467254908384535 :   (b('l8b5_med') <= 2764.910400390625) ?  0.06654733229092899 :   -0.011982530578764876 :   (b('grass') <= 44.73517036437988) ?  (b('crops') <= 54.62039375305176) ?  (b('grass') <= 21.917519569396973) ?  (b('l8b4_med') <= 2109.5477294921875) ?  0.06131738851417564 :   0.013498115759177656 :   (b('sand') <= 56.5) ?  -0.023549468867753703 :   -0.0799665376430058 :   (b('l8b5') <= 3395.5906982421875) ?  (b('l8b11') <= 3390.0531005859375) ?  -0.06623096307395412 :   -0.09594803446819879 :   (b('l8b5') <= 3711.9561767578125) ?  -0.09083507189859473 :   -0.07167243964909467 :   (b('ndvi_med') <= 1739.0) ?  (b('l8b5_med') <= 2387.6632080078125) ?  (b('l8b11_med') <= 2132.2347412109375) ?  0.07010007812594693 :   0.20285530649000738 :   (b('l8b11') <= 1880.5628662109375) ?  0.15923196500636644 :   0.01951728647203018 :   (b('l8b4') <= 1559.1403198242188) ?  (b('l8b11_med') <= 1951.6283569335938) ?  -0.05280964857650716 :   -0.08919116164577667 :   (b('grass') <= 58.683271408081055) ?  -0.04652316825064373 :   -0.030413094839096627 :   (b('dg0_v_vv') <= 2.4484176635742188) ?  (b('l8b11') <= 2831.4981689453125) ?  (b('l8b11') <= 2450.076171875) ?  (b('l8b11') <= 2108.4068603515625) ?  0.034942542419923145 :   (b('dg0_v_vv') <= -0.37197160720825195) ?  0.0003579673423136673 :   -0.02571992603754685 :   (b('l8b11_med') <= 2881.5589599609375) ?  (b('ndvi') <= 1046.0) ?  -0.060352167861183414 :   0.0229784685929837 :   (b('ndvi') <= 1103.0) ?  0.10099591487306966 :   0.028328281128927914 :   (b('ndvi') <= 1474.5) ?  (b('dg0_v_vv') <= -1.6081600189208984) ?  (b('ndvi') <= 1196.0) ?  0.010609613278726726 :   -0.00514786094824661 :   (b('l8b5') <= 3265.4012451171875) ?  -0.028953995404120993 :   -0.02052956981098351 :   (b('ndvi') <= 1481.0) ?  (b('l8b5') <= 2535.2183837890625) ?  0.05902066826954816 :   0.08605788392465974 :   (b('l8b4') <= 2444.7628173828125) ?  -0.012532819847680333 :   0.1009465855551494 :   (b('l8b4') <= 1782.654296875) ?  (b('l8b4_med') <= 1656.4095458984375) ?  (b('l8b5_med') <= 2812.056640625) ?  0.07997191921987318 :   0.06715050761856547 :   0.028758284489251384 :   0.006954645254666159 :   (b('l8b4_med') <= 2435.798095703125) ?  (b('dg0_v_vv') <= 2.394139051437378) ?  (b('l8b5') <= 3069.4146728515625) ?  (b('l8b11') <= 2732.7313232421875) ?  (b('l8b4') <= 1518.3820190429688) ?  (b('dg0_v_vv') <= 1.347372055053711) ?  -0.0585829283138415 :   -0.11140923978310051 :   (b('ndvi') <= 1267.0) ?  -0.09519664650050008 :   -0.08491202379092973 :   (b('dg0_v_vv') <= 1.210721492767334) ?  (b('l8b11') <= 3556.587646484375) ?  -0.07099407680672254 :   -0.055302659570970636 :   (b('grass') <= 13.035483121871948) ?  -0.05526739441124391 :   -0.038111509812653516 :   (b('l8b5') <= 4116.728759765625) ?  (b('dg0_v_vv') <= 0.7612195014953613) ?  (b('l8b4') <= 1591.796875) ?  -0.07451254353459678 :   -0.087738400296768 :   (b('l8b11_med') <= 2774.488037109375) ?  -0.06310359036630103 :   -0.08342722553766958 :   (b('l8b5') <= 4199.140869140625) ?  (b('dg0_v_vv') <= 0.4880528450012207) ?  -0.06606185741763367 :   -0.04195015533214506 :   (b('dg0_v_vv') <= 1.5245521068572998) ?  -0.09656341509863192 :   -0.07069540200463323 :   (b('l8b5') <= 3177.0390625) ?  (b('dg0_v_vv') <= 4.281494140625) ?  (b('l8b4') <= 1950.2202758789062) ?  (b('l8b4') <= 1449.7473754882812) ?  -0.06672658003905961 :   -0.09058997202356751 :   (b('l8b11') <= 2663.9453125) ?  -0.07526739396420908 :   -0.040267393815197464 :   -0.04689371007809674 :   (b('ndvi_med') <= 2030.0) ?  (b('l8b4') <= 2170.8605346679688) ?  (b('l8b11') <= 1957.2428588867188) ?  -0.05069540245166806 :   -0.03231482456684031 :   (b('dg0_v_vv') <= 3.188278913497925) ?  -0.06129435114438331 :   -0.043748792366549376 :   (b('l8b11') <= 3867.8369140625) ?  -0.09723019026597549 :   -0.10223215389735404 :   (b('dg0_v_vv') <= 0.30910205841064453) ?  (b('sand') <= 58.0) ?  (b('l8b4') <= 2565.861328125) ?  (b('l8b11') <= 3076.068603515625) ?  0.018416142748227504 :   -0.00039606966260130605 :   (b('ndvi') <= 943.5) ?  (b('grass') <= 15.222972869873047) ?  -0.05501198143437199 :   -0.027560193276062548 :   (b('ndvi') <= 979.5) ?  -0.06655034271803015 :   -0.04424338050605624 :   (b('ndvi') <= 1691.5) ?  (b('l8b5') <= 3457.89892578125) ?  (b('ndvi') <= 961.5) ?  -0.05429878045533427 :   0.0369765145972905 :   (b('bulk') <= 151.5) ?  -0.04369024394706421 :   -0.06566362403383513 :   (b('ndvi_med') <= 1098.25) ?  (b('l8b5') <= 4510.243896484375) ?  -0.007463098852009298 :   -0.0014631015938229575 :   -0.018463102890223987 :   (b('l8b5') <= 3654.93359375) ?  (b('l8b4') <= 2493.0772705078125) ?  (b('ndvi') <= 1511.5) ?  (b('grass') <= 25.494921684265137) ?  -0.03172422711911757 :   -0.07066980360995956 :   0.0223301911448317 :   (b('ndvi_med') <= 1094.5) ?  (b('l8b5_med') <= 3483.153076171875) ?  -0.009620680854939057 :   0.016856720876479176 :   (b('ndvi') <= 928.0) ?  -0.023809780898544464 :   -0.002248488467713685 :   (b('l8b11') <= 3991.546630859375) ?  (b('l8b5') <= 4206.704345703125) ?  (b('dg0_v_vv') <= 3.179013729095459) ?  -0.03715101316408351 :   0.003211506862213123 :   (b('l8b5') <= 4363.705810546875) ?  0.020887937552545324 :   -0.01885296992538235 :   (b('l8b11') <= 4386.4912109375) ?  (b('l8b11') <= 4018.4351806640625) ?  -0.06799053879932826 :   -0.05392346212828531 :   (b('l8b5') <= 4560.805908203125) ?  -0.027579779772599594 :   -0.04169857080654059 :   (b('grass') <= 46.14527893066406) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('sand') <= 20.5) ?  (b('l8b5') <= 3333.8504638671875) ?  (b('dg0_v_vv') <= 1.034651279449463) ?  (b('bulk') <= 153.0) ?  (b('dg0_v_vv') <= 0.7778067588806152) ?  (b('l8b5_med') <= 3746.0811767578125) ?  (b('ndvi') <= 1454.5) ?  0.24055973288648297 :   0.13156346865590754 :   -0.006875563172262983 :   (b('l8b5') <= 2887.925537109375) ?  (b('dg0_v_vv') <= 0.8956403732299805) ?  0.016063924145137548 :   0.00647233723688892 :   0.0788889045126499 :   (b('ndvi') <= 4662.5) ?  (b('l8b5') <= 3145.30517578125) ?  (b('ndvi_med') <= 4372.5) ?  0.20360967714252332 :   0.20616939968747058 :   0.20060441930403655 :   (b('l8b5') <= 3287.74267578125) ?  0.18985016028383536 :   0.19085189391208146 :   (b('l8b4') <= 751.1295776367188) ?  (b('bulk') <= 153.0) ?  (b('dg0_v_vv') <= 3.3004541397094727) ?  (b('dg0_v_vv') <= 2.1333377361297607) ?  0.1665820186665902 :   0.13822759709115473 :   0.22618082876909004 :   0.22885191131663774 :   (b('ndvi_med') <= 4372.5) ?  (b('ndvi') <= 3144.0) ?  (b('dg0_v_vv') <= 1.506026268005371) ?  0.21678503101193827 :   0.19699083151916735 :   (b('dg0_v_vv') <= 2.0112357139587402) ?  0.22670222039107188 :   0.21547344417741449 :   0.1867986908619804 :   (b('l8b5') <= 3753.6854248046875) ?  (b('dg0_v_vv') <= 0.5148234367370605) ?  (b('ndvi_med') <= 3451.5) ?  (b('ndvi') <= 4103.5) ?  (b('l8b11') <= 1134.0706176757812) ?  0.011263794776780994 :   -0.013789911123608473 :   (b('grass') <= 32.82162952423096) ?  0.05902445784105592 :   0.051210086492205736 :   (b('dg0_v_vv') <= -0.8484559059143066) ?  (b('sand') <= 16.0) ?  -0.033058485889209455 :   -0.030403066745214202 :   (b('dg0_v_vv') <= -0.6823153495788574) ?  -0.05379329785493964 :   -0.04290482909422916 :   (b('l8b11') <= 1280.4808959960938) ?  (b('l8b11') <= 970.870849609375) ?  0.11258608793885416 :   (b('l8b5_med') <= 2894.9593505859375) ?  0.08024126845224286 :   0.04501725507681174 :   0.17859011839911249 :   (b('l8b5') <= 3801.125244140625) ?  (b('ndvi') <= 4639.5) ?  (b('dg0_v_vv') <= -0.11318683624267578) ?  (b('dg0_v_vv') <= -0.7864599227905273) ?  0.18827040167628578 :   0.19889398572278175 :   0.17367051204329617 :   0.23399335274380273 :   (b('l8b4') <= 627.0768737792969) ?  (b('ndvi') <= 4581.0) ?  (b('ndvi') <= 4268.0) ?  0.12240931109061388 :   0.03747698478767336 :   (b('dg0_v_vv') <= 1.0438337326049805) ?  0.15300550758933004 :   0.24929980488298778 :   (b('l8b11') <= 1260.7633056640625) ?  (b('ndvi') <= 4642.5) ?  -0.0025273625242616025 :   0.03167392556401463 :   (b('dg0_v_vv') <= -0.17710447311401367) ?  0.10634990826345664 :   0.10941137191381117 :   (b('ndvi_med') <= 4217.25) ?  (b('dg0_v_vv') <= -0.1481766700744629) ?  (b('l8b4_med') <= 852.34228515625) ?  (b('dg0_v_vv') <= -1.4964489936828613) ?  (b('ndvi') <= 1836.5) ?  (b('dg0_v_vv') <= -2.7053322792053223) ?  0.04733893234909588 :   0.10633224670283369 :   (b('l8b4') <= 1532.2200317382812) ?  -0.012614089186560483 :   0.1675507443103917 :   (b('l8b5') <= 3084.451904296875) ?  (b('l8b5') <= 1985.8095092773438) ?  0.06756280370136306 :   0.0037039822330662683 :   (b('ndvi_med') <= 3998.5) ?  0.04510547567907465 :   0.015686484134588296 :   (b('l8b5') <= 4572.207763671875) ?  (b('l8b11') <= 2953.329345703125) ?  (b('l8b11') <= 1989.1065063476562) ?  0.05681836188973981 :   0.027997238907665284 :   0.17895083394940334 :   (b('dg0_v_vv') <= -0.7861218452453613) ?  (b('dg0_v_vv') <= -1.2531218528747559) ?  0.1359486290822175 :   0.17544863710897632 :   0.07187202182550462 :   (b('l8b11_med') <= 1290.6134033203125) ?  (b('l8b5_med') <= 2926.2784423828125) ?  (b('l8b5') <= 3051.0675048828125) ?  (b('ndvi') <= 3627.5) ?  0.03568813667542626 :   -0.008509816467629354 :   (b('dg0_v_vv') <= -0.019226551055908203) ?  0.0018024086805203121 :   -0.04911113495546019 :   (b('bulk') <= 148.5) ?  (b('l8b5') <= 2932.7662353515625) ?  0.06339580400122155 :   0.028935666428500737 :   (b('l8b4') <= 595.9152221679688) ?  0.14909387617618639 :   0.07125433028246198 :   (b('l8b5') <= 2375.9698486328125) ?  (b('sand') <= 31.0) ?  (b('dg0_v_vv') <= 0.5437178611755371) ?  0.06179460030667718 :   0.1036821122675209 :   (b('ndvi') <= 3452.5) ?  0.16820574947125064 :   0.11936579039318831 :   (b('l8b4') <= 1536.620849609375) ?  (b('bulk') <= 131.5) ?  0.036625515057100023 :   0.06284180285744718 :   (b('l8b4') <= 1886.2599487304688) ?  0.16857382031777715 :   0.2292263831715249 :   (b('ndvi') <= 3108.5) ?  (b('bulk') <= 131.0) ?  (b('dg0_v_vv') <= 0.5787291526794434) ?  (b('dg0_v_vv') <= -0.06584024429321289) ?  0.23906401752471326 :   0.2599410596460615 :   (b('ndvi') <= 2579.0) ?  (b('l8b11_med') <= 1222.7625122070312) ?  0.20772714867931402 :   0.2301771387928903 :   (b('dg0_v_vv') <= 2.080751895904541) ?  0.22307236459405685 :   0.2436004344755928 :   (b('dg0_v_vv') <= 0.14699411392211914) ?  (b('ndvi') <= 2582.0) ?  0.025152557818536814 :   0.05801143491912275 :   0.18620931814639 :   (b('l8b11') <= 1345.4435424804688) ?  (b('l8b4') <= 411.5653839111328) ?  (b('ndvi_med') <= 4522.5) ?  (b('l8b5') <= 3580.295166015625) ?  0.024387301010695528 :   -0.00964458625719384 :   0.15378421972890946 :   (b('sand') <= 60.0) ?  (b('l8b4') <= 468.2355194091797) ?  0.021731544780319675 :   0.10738784107029899 :   (b('ndvi') <= 4357.0) ?  0.05506534186260713 :   0.19579971043315625 :   (b('dg0_v_vv') <= 0.7363452911376953) ?  (b('dg0_v_vv') <= -1.6112675666809082) ?  (b('ndvi') <= 4423.5) ?  0.13515208451617922 :   -0.012472494423960562 :   (b('l8b5') <= 3332.621337890625) ?  0.010362652772795656 :   0.04340174029170155 :   (b('dg0_v_vv') <= 1.8393750190734863) ?  (b('l8b11') <= 1746.3754272460938) ?  0.17576433120975765 :   0.09170248568952999 :   (b('l8b5') <= 2690.209716796875) ?  -0.00271881058334969 :   0.07629223499757432 :   (b('moss') <= 6.192884922027588) ?  (b('bulk') <= 154.0) ?  (b('dg0_v_vv') <= -1.2323670387268066) ?  (b('sand') <= 46.0) ?  (b('l8b4') <= 1243.1409912109375) ?  (b('l8b5') <= 2836.626220703125) ?  (b('dg0_v_vv') <= -2.1686325073242188) ?  -0.027182495892397553 :   -0.06515117221014613 :   (b('crops') <= 52.261112213134766) ?  -0.0488397128544414 :   -0.07108887636139435 :   (b('sand') <= 23.5) ?  -0.009962528564464945 :   0.13363663973736986 :   (b('dg0_v_vv') <= -2.4942054748535156) ?  (b('ndvi') <= 4826.0) ?  (b('l8b5') <= 3103.5440673828125) ?  0.0017150016239493998 :   -0.05634167015945111 :   (b('ndvi_med') <= 3656.75) ?  0.019949121039035678 :   0.003916995906877274 :   (b('l8b11') <= 726.1044921875) ?  -0.05994270840578941 :   (b('l8b4') <= 1101.98681640625) ?  0.02111455418918211 :   0.07216170470417733 :   (b('grass') <= 39.1515007019043) ?  (b('ndvi') <= 4011.5) ?  (b('l8b4_med') <= 967.0794677734375) ?  (b('bulk') <= 150.5) ?  0.06493936209604767 :   0.1082697422207187 :   (b('l8b11') <= 1695.9063110351562) ?  0.15515477806931757 :   0.0711985678143135 :   (b('sand') <= 46.0) ?  (b('dg0_v_vv') <= 0.5420589447021484) ?  0.04114677490518313 :   0.09882445781337582 :   (b('grass') <= 12.78446912765503) ?  0.05481322400401492 :   0.002778126128533849 :   (b('ndvi') <= 2556.0) ?  (b('l8b4') <= 1040.9045715332031) ?  -0.07844139663713295 :   -0.0970098054722775 :   (b('l8b4') <= 859.8863525390625) ?  (b('ndvi') <= 4682.5) ?  -0.03610787777912134 :   0.009171751827883871 :   (b('ndvi') <= 2854.5) ?  0.00597936864662052 :   0.03892263286591993 :   (b('dg0_v_vv') <= 0.17691421508789062) ?  (b('ndvi') <= 3252.0) ?  (b('ndvi') <= 2045.5) ?  (b('ndvi') <= 1879.0) ?  -0.006573798315666651 :   -0.038298542467013044 :   (b('ndvi') <= 3108.5) ?  (b('dg0_v_vv') <= -0.19986295700073242) ?  -0.08652140984788023 :   -0.060894137510523166 :   (b('l8b4') <= 1183.0180053710938) ?  -0.015317282247415825 :   -0.07428405216999041 :   (b('dg0_v_vv') <= -1.1741695404052734) ?  (b('dg0_v_vv') <= -2.1977791786193848) ?  -0.005317284333578393 :   (b('dg0_v_vv') <= -1.5744061470031738) ?  0.00810523511338758 :   0.017413727850501756 :   (b('ndvi') <= 4491.0) ?  (b('ndvi') <= 3300.5) ?  -0.0012870188087663048 :   -0.05416068742180469 :   0.0029865805762847586 :   (b('l8b5') <= 3642.8980712890625) ?  (b('l8b11') <= 2923.432861328125) ?  (b('dg0_v_vv') <= 0.22945594787597656) ?  0.0605865766066154 :   (b('l8b4') <= 568.6746978759766) ?  -0.008276306493022384 :   0.017810011723208168 :   0.091299670599262 :   (b('dg0_v_vv') <= 2.2645740509033203) ?  (b('l8b4') <= 746.1207733154297) ?  -0.05815556228175757 :   -0.05466122345712891 :   -0.015588415680336073 :   (b('l8b11') <= 1451.3238525390625) ?  (b('moss') <= 19.52681541442871) ?  (b('l8b11') <= 1419.4006958007812) ?  (b('bulk') <= 123.5) ?  (b('l8b4') <= 477.0339813232422) ?  (b('l8b11') <= 1069.7172241210938) ?  -0.026699595402329752 :   0.028273202620888782 :   (b('l8b11_med') <= 1719.2047729492188) ?  -0.09012612024325255 :   -0.050490393221462246 :   (b('dg0_v_vv') <= -1.0201306343078613) ?  (b('ndvi') <= 4415.5) ?  -0.03432648760236823 :   -0.0006826410116833603 :   (b('ndvi') <= 4816.0) ?  0.001915630313994876 :   0.034795857203821244 :   (b('l8b5_med') <= 3030.43359375) ?  (b('dg0_v_vv') <= -0.28499794006347656) ?  (b('l8b5_med') <= 2864.0498046875) ?  0.047818802597725 :   -0.0460525455827103 :   (b('ndvi') <= 3768.0) ?  0.02955033353731357 :   0.05852324939960609 :   (b('l8b11') <= 1434.2569580078125) ?  (b('l8b4_med') <= 858.581787109375) ?  0.03231564633099396 :   0.01722600891589754 :   (b('l8b4') <= 673.1319885253906) ?  0.07743509032034954 :   0.12457563810976317 :   (b('dg0_v_vv') <= -0.05314826965332031) ?  (b('l8b4') <= 576.7095642089844) ?  (b('l8b11_med') <= 1638.81787109375) ?  0.13949032907406145 :   0.12749033455768877 :   (b('grass') <= 19.980335235595703) ?  -0.05535300178887681 :   (b('ndvi') <= 3932.0) ?  0.05590167728868984 :   -0.013724273257818581 :   (b('l8b4') <= 814.5948181152344) ?  (b('l8b4') <= 677.7054748535156) ?  (b('l8b4') <= 583.3910522460938) ?  0.12342695923399988 :   0.06660592085610831 :   (b('l8b4') <= 754.0293884277344) ?  0.16326484635324398 :   0.12841097227475784 :   (b('l8b11') <= 1405.0650024414062) ?  (b('bulk') <= 152.0) ?  0.11762384061465483 :   0.041158169698809366 :   -0.04584363096668756 :   (b('crops') <= 32.00249671936035) ?  (b('l8b11') <= 1795.4749755859375) ?  (b('l8b4') <= 929.7593383789062) ?  (b('ndvi_med') <= 3613.25) ?  (b('ndvi') <= 4585.5) ?  0.029184080956506455 :   0.20220293688529328 :   (b('dg0_v_vv') <= 0.09462881088256836) ?  -0.030992255793783285 :   -0.09579806156839979 :   (b('l8b11') <= 1711.5758666992188) ?  (b('moss') <= 25.96159553527832) ?  -0.05037838299378977 :   0.041479448370033534 :   (b('moss') <= 19.49218511581421) ?  0.08425129221630555 :   -0.01673511472005068 :   (b('l8b5') <= 3214.2169189453125) ?  (b('l8b4') <= 1334.4517822265625) ?  (b('ndvi') <= 3252.0) ?  -0.0585800110733529 :   -0.028024591233490492 :   (b('ndvi') <= 3062.0) ?  0.006010049996858384 :   0.1751694049176684 :   (b('ndvi') <= 3364.0) ?  (b('ndvi_med') <= 3541.75) ?  -0.07973973747479798 :   -0.061902514090928916 :   (b('dg0_v_vv') <= 0.05282115936279297) ?  -0.07050549676500899 :   -0.04365888436519129 :   (b('l8b5') <= 2998.5814208984375) ?  (b('l8b5') <= 2460.3826904296875) ?  (b('sand') <= 75.5) ?  (b('l8b4') <= 814.8666687011719) ?  0.02010430964536365 :   -0.015066395278404586 :   (b('l8b5') <= 1890.1865844726562) ?  0.11398715816766489 :   0.06379710141792433 :   (b('l8b11') <= 1631.7040405273438) ?  (b('l8b5') <= 2943.72607421875) ?  -0.05756166616902159 :   -0.021867601191017445 :   (b('grass') <= 20.20370578765869) ?  0.014416888463055038 :   -0.02339491308097968 :   (b('l8b11') <= 1711.3966674804688) ?  (b('l8b5_med') <= 3233.3369140625) ?  (b('l8b4') <= 792.8507995605469) ?  -0.012174974433901744 :   0.01247431760664005 :   (b('l8b11') <= 1697.6556396484375) ?  0.09956300579484423 :   0.1594120181531585 :   (b('l8b11') <= 2465.0108642578125) ?  (b('l8b4') <= 1157.0153198242188) ?  -0.013141043450290497 :   -0.039171403129626386 :   (b('l8b4') <= 1683.7824096679688) ?  0.0754279693941821 :   -0.030940618857353313 :   (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 958.7872924804688) ?  (b('sand') <= 53.5) ?  (b('ndvi') <= 3695.0) ?  (b('dg0_v_vv') <= -0.6455874443054199) ?  -0.0037954652711749137 :   -0.05752399144400991 :   (b('l8b11') <= 855.1739196777344) ?  (b('l8b5_med') <= 3095.4654541015625) ?  (b('sand') <= 39.5) ?  0.1222131013161194 :   0.1065940498829431 :   0.1905835552586865 :   (b('ndvi') <= 4146.0) ?  (b('ndvi_med') <= 3823.0) ?  (b('l8b4') <= 415.65440368652344) ?  0.0700981166165495 :   0.029048477886951335 :   (b('l8b4_med') <= 653.1116027832031) ?  0.11759509226364304 :   0.12148685457017794 :   (b('ndvi') <= 4415.0) ?  -0.05214398743291532 :   (b('ndvi') <= 4718.0) ?  0.10600600200067126 :   0.008982186257883225 :   (b('l8b5') <= 1878.2954711914062) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('ndvi_med') <= 3520.75) ?  0.19613575755017937 :   (b('ndvi_med') <= 3709.25) ?  0.23342881258034975 :   0.22442883725667268 :   (b('l8b5') <= 1748.1115112304688) ?  (b('dg0_v_vv') <= -0.6967501640319824) ?  (b('dg0_v_vv') <= -0.885399341583252) ?  0.13552396190643018 :   0.24068138496127156 :   (b('ndvi') <= 2886.0) ?  0.12907479963799015 :   0.08489095220097413 :   -0.08029272067397833 :   (b('l8b5') <= 2194.4620361328125) ?  (b('l8b4') <= 374.4420166015625) ?  (b('dg0_v_vv') <= -0.11535072326660156) ?  (b('dg0_v_vv') <= -0.5873222351074219) ?  0.17332532800077222 :   0.18512531913160107 :   (b('ndvi_med') <= 3520.75) ?  0.2304608310461994 :   0.21247007500216514 :   (b('dg0_v_vv') <= 0.03709745407104492) ?  (b('dg0_v_vv') <= -0.5522427558898926) ?  0.23073324776529092 :   0.2014311584516249 :   (b('ndvi_med') <= 3520.75) ?  0.22845347826573043 :   0.23077014788878747 :   (b('dg0_v_vv') <= -0.0022830963134765625) ?  (b('dg0_v_vv') <= -1.44035005569458) ?  0.20954798030737673 :   (b('ndvi_med') <= 3338.5) ?  0.16314881439056564 :   0.13158780714948642 :   (b('l8b5') <= 2922.29052734375) ?  (b('ndvi') <= 3225.0) ?  0.17119214980741074 :   0.20693920508332117 :   0.09156796372365417 :   (b('dg0_v_vv') <= -0.49141502380371094) ?  (b('ndvi') <= 2661.0) ?  (b('ndvi_med') <= 3623.0) ?  (b('dg0_v_vv') <= -2.0301947593688965) ?  -0.018645466961244334 :   (b('ndvi_med') <= 3515.5) ?  (b('l8b4') <= 1225.7032470703125) ?  0.1365527463079035 :   0.0679450130293485 :   (b('l8b4_med') <= 795.2744140625) ?  0.17353875022921805 :   0.20988494680179506 :   (b('dg0_v_vv') <= -1.7138681411743164) ?  (b('moss') <= 4.178937166929245) ?  (b('l8b4') <= 909.0767517089844) ?  -0.004509552290433982 :   0.05814501459415583 :   (b('l8b5') <= 2220.409912109375) ?  -0.0832569595184301 :   -0.06119443437966027 :   (b('grass') <= 65.37767791748047) ?  (b('grass') <= 53.56487274169922) ?  -0.044817016113369204 :   0.036497335821824184 :   (b('l8b5') <= 2356.1842041015625) ?  0.10863185873091104 :   0.04802472476657549 :   (b('dg0_v_vv') <= -1.2092328071594238) ?  (b('l8b5_med') <= 3186.5877685546875) ?  (b('ndvi') <= 3031.0) ?  (b('l8b11') <= 1666.310546875) ?  0.04395315435434112 :   -0.024029022985663666 :   (b('ndvi') <= 3560.5) ?  -0.05613374692195245 :   -0.02056481588090977 :   (b('l8b11') <= 1401.9918212890625) ?  (b('ndvi') <= 4295.5) ?  0.023038539586657275 :   0.0994389801764887 :   (b('l8b5') <= 3512.5986328125) ?  0.013901560984363653 :   -0.0056731221983909175 :   (b('l8b5') <= 2412.87890625) ?  (b('ndvi') <= 3330.0) ?  (b('l8b4') <= 674.9803466796875) ?  0.034932048626268086 :   0.0609355233884028 :   (b('ndvi') <= 4597.5) ?  0.1383358966683214 :   0.047166502861174575 :   (b('dg0_v_vv') <= -1.1405062675476074) ?  (b('crops') <= 10.516666650772095) ?  0.020901914545979387 :   0.19143847442904646 :   (b('ndvi_med') <= 3823.0) ?  -0.013848287669261494 :   0.03617595810321286 :   (b('l8b11_med') <= 1654.3424682617188) ?  (b('sand') <= 29.5) ?  (b('sand') <= 22.5) ?  (b('dg0_v_vv') <= 0.5264005661010742) ?  (b('l8b11') <= 997.695068359375) ?  0.210967415858049 :   0.0659115473548822 :   (b('l8b5') <= 3167.9388427734375) ?  0.1702931834919466 :   0.08637558640096106 :   (b('l8b4') <= 604.6152038574219) ?  (b('l8b5') <= 2582.930419921875) ?  0.027490836521879974 :   0.10469710666028159 :   (b('l8b4') <= 1414.56982421875) ?  0.19039248746433954 :   0.09445399141460989 :   (b('l8b5') <= 2218.475341796875) ?  (b('ndvi') <= 3938.5) ?  (b('l8b4') <= 595.28173828125) ?  0.02977295319008148 :   0.14964076394052084 :   (b('ndvi') <= 4656.0) ?  0.23315024932356426 :   0.10111875145820397 :   (b('ndvi') <= 3135.0) ?  (b('moss') <= 10.938619136810303) ?  0.08311307523346469 :   0.18179651013505652 :   (b('sand') <= 41.0) ?  0.07656512812969057 :   0.028899973103289456 :   (b('ndvi') <= 3925.5) ?  (b('l8b5') <= 2622.3404541015625) ?  (b('ndvi') <= 3182.0) ?  -0.031573465938614764 :   (b('ndvi') <= 3632.0) ?  -0.07327591414532245 :   -0.10027875726601856 :   (b('dg0_v_vv') <= -0.20381832122802734) ?  (b('l8b11') <= 1883.2224731445312) ?  0.05119880796395243 :   0.038860642707055776 :   (b('dg0_v_vv') <= 4.261830806732178) ?  -0.04518068004925663 :   0.00937692696065709 :   (b('dg0_v_vv') <= 0.4391441345214844) ?  (b('ndvi') <= 4540.0) ?  (b('l8b11_med') <= 1707.0121459960938) ?  -0.05348170011046918 :   -0.03695324933762306 :   (b('bulk') <= 129.0) ?  0.0006671626755129223 :   -0.020400768254725976 :   (b('l8b5') <= 3272.864501953125) ?  (b('l8b5') <= 2697.677490234375) ?  0.11250340599987459 :   0.06735412556505548 :   -0.041858671077634774 :   (b('l8b4_med') <= 636.951416015625) ?  (b('l8b4') <= 513.1579742431641) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b11') <= 1043.1750183105469) ?  (b('dg0_v_vv') <= 0.9202713966369629) ?  (b('l8b4_med') <= 481.6563720703125) ?  (b('dg0_v_vv') <= 0.04053020477294922) ?  0.17127893273359923 :   0.16557891075140624 :   (b('dg0_v_vv') <= 0.04627799987792969) ?  0.18435844855887543 :   0.19689529664603 :   (b('l8b5') <= 2363.4988403320312) ?  0.20100290734656945 :   (b('l8b11') <= 1016.0818786621094) ?  0.2262426628749285 :   0.2155011800170513 :   (b('crops') <= 29.68074893951416) ?  (b('l8b11') <= 1124.5382080078125) ?  (b('l8b5_med') <= 2984.57470703125) ?  0.2306131468205314 :   0.23540852346013086 :   (b('dg0_v_vv') <= 0.24115419387817383) ?  0.22528191966023217 :   0.22816946900884885 :   (b('dg0_v_vv') <= 1.3156061172485352) ?  (b('ndvi_med') <= 4027.0) ?  0.2521268698015125 :   0.25702199136101317 :   (b('ndvi_med') <= 4027.0) ?  0.2467051786619734 :   0.24210519020143262 :   (b('ndvi') <= 2667.5) ?  (b('ndvi') <= 2550.0) ?  (b('moss') <= 0.7690265774726868) ?  0.27977251400202513 :   (b('dg0_v_vv') <= 1.1222176551818848) ?  0.2942708951088194 :   0.302043427686891 :   (b('dg0_v_vv') <= -0.4175567626953125) ?  (b('dg0_v_vv') <= -0.5687241554260254) ?  0.28983373433587095 :   0.2929790278684752 :   (b('dg0_v_vv') <= 0.5246090888977051) ?  0.26437905037518905 :   0.27079408944697136 :   (b('ndvi') <= 4350.5) ?  (b('dg0_v_vv') <= 1.7923049926757812) ?  (b('dg0_v_vv') <= 1.3470120429992676) ?  0.2488356642200781 :   0.21791391991320752 :   0.27202967877927925 :   (b('l8b11_med') <= 1270.8560180664062) ?  (b('dg0_v_vv') <= -1.0201325416564941) ?  0.29189208538052697 :   0.2962186352283537 :   0.2758419635432958 :   (b('l8b4_med') <= 585.992919921875) ?  (b('l8b4') <= 651.3680419921875) ?  (b('l8b5_med') <= 3089.992919921875) ?  (b('ndvi') <= 4406.5) ?  (b('l8b4') <= 541.5288391113281) ?  0.23687072968415263 :   0.17497015682087141 :   (b('ndvi_med') <= 3831.25) ?  0.2034939364716677 :   0.17115533332547972 :   (b('l8b5') <= 3218.955322265625) ?  (b('dg0_v_vv') <= 1.3989896774291992) ?  0.2202037235131322 :   0.26331571661985975 :   (b('dg0_v_vv') <= 0.33684682846069336) ?  0.18406406465928352 :   0.22644536576678626 :   (b('ndvi') <= 3319.5) ?  (b('dg0_v_vv') <= 0.6217918395996094) ?  (b('moss') <= 0.7690265774726868) ?  0.1796919896613494 :   0.16922965797612785 :   (b('grass') <= 53.03794288635254) ?  0.1859178935247841 :   0.16891953806088683 :   (b('l8b5_med') <= 3155.5751953125) ?  0.16426374884455242 :   (b('dg0_v_vv') <= -0.6143622398376465) ?  0.16037524372411427 :   0.15226396875728956 :   (b('ndvi') <= 2956.5) ?  (b('l8b11_med') <= 1261.8388061523438) ?  (b('dg0_v_vv') <= 1.963045597076416) ?  0.23856045656270547 :   0.23481729500986986 :   (b('dg0_v_vv') <= 0.6374268531799316) ?  0.2318472849676793 :   0.23084729784228258 :   (b('dg0_v_vv') <= -0.0035457611083984375) ?  (b('l8b11_med') <= 1216.3350219726562) ?  (b('l8b5') <= 2474.0494384765625) ?  0.18723365158916494 :   0.1935900871063913 :   (b('dg0_v_vv') <= -0.4067220687866211) ?  0.19880720553218537 :   0.2146192048531405 :   (b('dg0_v_vv') <= -4.38690185546875e-05) ?  (b('dg0_v_vv') <= -0.000247955322265625) ?  0.2161506415488483 :   0.2473009077798211 :   (b('l8b5') <= 2866.849853515625) ?  0.1953444543423243 :   0.22278181224967136 :   (b('dg0_v_vv') <= -1.9941816329956055) ?  (b('l8b11') <= 1118.4485473632812) ?  (b('l8b11') <= 778.0353546142578) ?  0.17305602023340763 :   0.17966295541435956 :   (b('ndvi') <= 3351.5) ?  (b('bulk') <= 136.5) ?  (b('dg0_v_vv') <= -2.2544708251953125) ?  (b('dg0_v_vv') <= -2.5997462272644043) ?  0.13470685319571507 :   0.1772595850961637 :   0.03576910593145968 :   (b('l8b11_med') <= 1330.5008544921875) ?  (b('ndvi') <= 2612.0) ?  -0.08007686289757562 :   -0.05494256476409806 :   -0.0069567738341288765 :   (b('l8b4') <= 630.7738952636719) ?  (b('dg0_v_vv') <= -2.360713005065918) ?  (b('ndvi') <= 3587.5) ?  -0.0385051754793773 :   -0.032923256649835725 :   -0.021446310690914572 :   (b('ndvi_med') <= 3442.5) ?  -0.007344884905725907 :   (b('dg0_v_vv') <= -2.390312671661377) ?  0.010136823338586898 :   -0.001465684901260228 :   (b('sand') <= 55.5) ?  (b('l8b4') <= 1148.9852294921875) ?  (b('dg0_v_vv') <= 2.6256213188171387) ?  (b('l8b11') <= 822.0127563476562) ?  (b('l8b11') <= 793.8027954101562) ?  0.10996060910227964 :   0.1691560504977795 :   (b('l8b11') <= 1245.9922485351562) ?  0.030076349363107626 :   0.07620811260423707 :   (b('l8b5') <= 3565.392578125) ?  0.19437006854029798 :   0.16652582717497516 :   (b('dg0_v_vv') <= 0.2501964569091797) ?  0.2077428295844786 :   0.18614281263291763 :   (b('l8b5') <= 2377.837890625) ?  (b('l8b4') <= 491.52301025390625) ?  (b('dg0_v_vv') <= -0.45203351974487305) ?  (b('dg0_v_vv') <= -1.4032788276672363) ?  0.23838705765335924 :   0.22208220153003932 :   0.12687888790518048 :   (b('ndvi') <= 3523.5) ?  (b('ndvi') <= 3319.0) ?  0.10440445288554762 :   0.14164941487430424 :   (b('dg0_v_vv') <= 0.4966468811035156) ?  -0.004219199884085543 :   0.016389787704536807 :   (b('dg0_v_vv') <= -1.2122888565063477) ?  (b('ndvi') <= 3265.5) ?  (b('l8b5') <= 2434.0947265625) ?  -0.006101479088440553 :   0.10124076667187192 :   (b('dg0_v_vv') <= -1.3315086364746094) ?  0.1874598710531068 :   0.11651655335314551 :   (b('ndvi') <= 4735.0) ?  (b('ndvi') <= 2579.5) ?  0.25102188925330965 :   0.2038345123592736 :   (b('ndvi_med') <= 3442.5) ?  0.08747563793028441 :   0.20371782918040998 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 3272.0) ?  (b('l8b4') <= 1320.0479125976562) ?  (b('moss') <= 6.230724096298218) ?  (b('sand') <= 24.0) ?  (b('ndvi') <= 4639.5) ?  (b('l8b4') <= 1030.06640625) ?  (b('l8b4_med') <= 1975.3472900390625) ?  (b('ndvi_med') <= 2107.5) ?  (b('dg0_v_vv') <= -1.0674409866333008) ?  (b('ndvi_med') <= 1383.5) ?  0.1249063794643704 :   0.16965547397672223 :   (b('l8b4') <= 669.49072265625) ?  0.23152330251718428 :   0.18918656619321314 :   (b('l8b5') <= 1613.44677734375) ?  0.05905940250721123 :   (b('dg0_v_vv') <= 0.6089358329772949) ?  0.12863649220702883 :   0.16643121243010936 :   -0.006582101826453551 :   (b('l8b4') <= 1230.0103759765625) ?  (b('grass') <= 12.003249406814575) ?  (b('l8b5_med') <= 1973.9232177734375) ?  0.19042712773963033 :   (b('dg0_v_vv') <= 0.17711305618286133) ?  0.1739898551576719 :   0.17285775588025296 :   (b('dg0_v_vv') <= 3.2206082344055176) ?  (b('l8b11') <= 2078.722412109375) ?  0.052465484006651825 :   -0.021309162813050814 :   (b('ndvi') <= 1685.0) ?  0.18712431377685412 :   0.0900217598346546 :   (b('l8b11') <= 1680.04443359375) ?  (b('ndvi') <= 3574.0) ?  (b('l8b11') <= 1644.1467895507812) ?  0.1344413905077964 :   0.17091467362485604 :   (b('dg0_v_vv') <= 0.1743783950805664) ?  0.11011124765750493 :   0.127016464790816 :   (b('ndvi_med') <= 2632.5) ?  (b('grass') <= 53.824214935302734) ?  0.08646389336676435 :   0.05003949775909228 :   0.15828569447602206 :   (b('l8b4') <= 541.6111450195312) ?  0.17637332341350745 :   (b('dg0_v_vv') <= 0.2480335235595703) ?  (b('l8b4') <= 716.015625) ?  -0.025650874877427343 :   (b('ndvi_med') <= 2273.75) ?  -0.013576928884149464 :   -0.0021335237320871947 :   (b('l8b5') <= 4132.183837890625) ?  (b('l8b11_med') <= 1794.3097534179688) ?  -0.005235097191966159 :   0.0006077040538621237 :   (b('dg0_v_vv') <= 1.3947269916534424) ?  (b('dg0_v_vv') <= 1.0841631889343262) ?  0.014098440975726165 :   0.007037441077159673 :   (b('l8b5_med') <= 3116.2115478515625) ?  0.02308133215405618 :   0.01335769502375847 :   (b('sand') <= 49.5) ?  (b('moss') <= 1.4706422090530396) ?  (b('dg0_v_vv') <= 0.428497314453125) ?  (b('bulk') <= 133.0) ?  (b('l8b11_med') <= 1768.0971069335938) ?  (b('l8b5_med') <= 2652.38818359375) ?  0.0816707293144464 :   0.14162997683159897 :   (b('dg0_v_vv') <= -2.6601510047912598) ?  0.12566672311657673 :   0.14601873757736786 :   (b('ndvi') <= 2099.0) ?  (b('ndvi_med') <= 1877.5) ?  -0.0446251080087836 :   0.019329764257951684 :   (b('l8b5') <= 3109.2716064453125) ?  0.023775827576088985 :   0.06964226170907141 :   (b('ndvi_med') <= 1670.5) ?  (b('l8b4_med') <= 1496.5256958007812) ?  (b('ndvi_med') <= 1615.25) ?  0.15477233539559396 :   0.22354791271857405 :   (b('bulk') <= 158.5) ?  -0.13108893115270742 :   0.08609093450028145 :   (b('sand') <= 36.5) ?  (b('bulk') <= 144.0) ?  0.0719167797181387 :   0.014207220218084435 :   (b('ndvi_med') <= 1881.5) ?  0.05642823091702402 :   0.09839105413950561 :   (b('bulk') <= 154.0) ?  (b('moss') <= 2.9430052042007446) ?  (b('grass') <= 1.1757924556732178) ?  (b('l8b4') <= 1107.545166015625) ?  0.024767668142261667 :   -0.014400502116862663 :   (b('ndvi') <= 1016.5) ?  0.035550260815188243 :   -0.02395763262591354 :   (b('l8b4') <= 1085.0819702148438) ?  (b('ndvi') <= 4951.0) ?  0.028619284776288875 :   0.17567939811575453 :   (b('dg0_v_vv') <= 1.180875301361084) ?  -0.008136317412727226 :   0.04147954849684263 :   (b('l8b11') <= 889.6414794921875) ?  (b('l8b11') <= 761.616455078125) ?  0.0708481307750615 :   (b('ndvi') <= 3545.0) ?  -0.0060892990255528995 :   -0.060124506077263364 :   (b('dg0_v_vv') <= 0.4560837745666504) ?  (b('ndvi') <= 2564.0) ?  0.08589306780059207 :   -0.00519438696301715 :   (b('l8b5') <= 3816.8292236328125) ?  0.14868902026695913 :   -0.03435824240841567 :   (b('l8b5_med') <= 2434.85009765625) ?  (b('sand') <= 52.0) ?  (b('ndvi') <= 2116.5) ?  (b('dg0_v_vv') <= -1.2133145332336426) ?  (b('l8b5') <= 1942.087158203125) ?  -0.05163650755326845 :   -0.08592430007870486 :   (b('l8b4') <= 794.6246948242188) ?  -0.023024395076385692 :   -0.04426825989545688 :   (b('l8b5') <= 2358.9609375) ?  (b('l8b4') <= 712.4212341308594) ?  -0.028343258721549275 :   -0.09318178500969826 :   (b('l8b5') <= 2480.25390625) ?  -0.031463145611896524 :   -0.06795934168520736 :   (b('dg0_v_vv') <= 0.4045555591583252) ?  (b('sand') <= 57.5) ?  (b('l8b4') <= 761.1083068847656) ?  -0.07715373650998918 :   -0.034394018468452095 :   (b('dg0_v_vv') <= -0.05324745178222656) ?  -0.002957825419764801 :   -0.050842124988470405 :   (b('sand') <= 55.0) ?  (b('ndvi') <= 1369.0) ?  -0.04566330171585262 :   0.09319100645076987 :   (b('l8b4') <= 373.39622497558594) ?  0.06499103832218421 :   -0.016997254941231034 :   (b('bulk') <= 125.5) ?  (b('ndvi') <= 2542.5) ?  (b('ndvi') <= 2393.5) ?  0.2445154498533333 :   0.20575143325801754 :   (b('dg0_v_vv') <= 0.04746246337890625) ?  (b('ndvi_med') <= 3109.75) ?  0.14740517866589026 :   0.15523250935089628 :   (b('l8b11') <= 1211.5787353515625) ?  0.18558570756758547 :   0.16735486377709247 :   (b('dg0_v_vv') <= -0.9759063720703125) ?  (b('bulk') <= 142.5) ?  (b('dg0_v_vv') <= -4.127198696136475) ?  -0.044464569173539165 :   0.03127436903225677 :   (b('l8b11') <= 1623.68017578125) ?  -0.007099499662734659 :   -0.053729629353394315 :   (b('l8b5') <= 3741.7059326171875) ?  (b('l8b11') <= 1320.1732788085938) ?  0.0717648013326946 :   0.019327388697894757 :   (b('dg0_v_vv') <= 0.8041057586669922) ?  0.03274377135538673 :   -0.038152903698900675 :   (b('l8b4') <= 624.9120483398438) ?  (b('crops') <= 17.920488357543945) ?  (b('l8b5') <= 1440.6194458007812) ?  (b('grass') <= 59.192848205566406) ?  (b('dg0_v_vv') <= 2.460822343826294) ?  (b('l8b4') <= 513.6013641357422) ?  (b('ndvi') <= 1648.5) ?  -0.0607628942020223 :   -0.08049936797176856 :   (b('l8b4') <= 607.9132080078125) ?  -0.0488689578185339 :   -0.06636395407928297 :   0.05169336445578945 :   (b('dg0_v_vv') <= -0.6717696189880371) ?  (b('ndvi_med') <= 1589.0) ?  (b('dg0_v_vv') <= -1.2991819381713867) ?  -0.002869903364032561 :   -0.005369893529266173 :   0.010230095989257032 :   (b('dg0_v_vv') <= 0.21098566055297852) ?  (b('l8b5_med') <= 1538.142578125) ?  0.06163010813966388 :   0.04142187537751982 :   0.006230102784186536 :   (b('l8b4_med') <= 1287.0316772460938) ?  (b('l8b5_med') <= 2715.992919921875) ?  (b('ndvi_med') <= 2188.75) ?  (b('moss') <= 31.57449436187744) ?  0.06983938495928092 :   -0.07813551550196414 :   (b('ndvi') <= 2193.0) ?  -0.027123314641790523 :   0.03978790342223123 :   (b('ndvi_med') <= 3035.0) ?  (b('ndvi') <= 3780.0) ?  0.10383547219703411 :   -0.012156067483659536 :   (b('l8b4') <= 620.512939453125) ?  0.2132764723785704 :   0.13058425122682432 :   (b('ndvi') <= 3126.5) ?  (b('l8b4') <= 529.1823120117188) ?  0.23533986115647337 :   (b('l8b11_med') <= 2415.8519287109375) ?  0.2535946961316999 :   0.25957690824684 :   (b('l8b11_med') <= 2081.80810546875) ?  0.18449029773463121 :   0.197896386139633 :   (b('l8b4') <= 534.963134765625) ?  (b('l8b5') <= 3368.5230712890625) ?  (b('dg0_v_vv') <= -0.4777650833129883) ?  (b('l8b11_med') <= 1543.1768188476562) ?  (b('ndvi') <= 2381.5) ?  0.009175113217402248 :   -0.012919401330577254 :   (b('ndvi') <= 3843.0) ?  0.030622538925494894 :   -0.023669189159624626 :   (b('l8b11_med') <= 1304.4703979492188) ?  (b('dg0_v_vv') <= -0.01298379898071289) ?  0.16887619476291332 :   0.05086831462853512 :   (b('dg0_v_vv') <= 2.052385091781616) ?  0.028828894975607946 :   0.10118590561920895 :   (b('sand') <= 36.5) ?  (b('l8b4') <= 461.6372375488281) ?  (b('l8b4_med') <= 751.5858154296875) ?  0.009081083264580125 :   -0.01448372793183958 :   (b('l8b5_med') <= 2920.932373046875) ?  -0.04351396086958102 :   -0.02040829423203696 :   (b('l8b5_med') <= 2887.1971435546875) ?  (b('dg0_v_vv') <= 1.4853129386901855) ?  0.0062799592914108355 :   -0.029823093725764763 :   (b('dg0_v_vv') <= 2.1096174716949463) ?  0.0014952991267696836 :   0.04149530568328061 :   (b('l8b11') <= 1030.0537109375) ?  (b('sand') <= 43.5) ?  (b('l8b5_med') <= 2740.2227783203125) ?  (b('dg0_v_vv') <= -0.5637664794921875) ?  -0.022029450616031815 :   -0.05099825164447635 :   (b('dg0_v_vv') <= 1.0414190292358398) ?  -0.0714324611305815 :   -0.08352556105631233 :   0.10527972752236284 :   (b('ndvi') <= 4271.0) ?  (b('l8b11') <= 1167.6231079101562) ?  (b('grass') <= 19.2280330657959) ?  -0.0219594224551069 :   0.008100358869221343 :   (b('l8b11') <= 1206.1718139648438) ?  -0.06791035550712671 :   -0.0211583622080062 :   (b('l8b5') <= 3413.706787109375) ?  (b('l8b11') <= 1231.0350341796875) ?  -0.08456070664529151 :   -0.010273045550479677 :   (b('ndvi') <= 4916.0) ?  0.03397012563913175 :   -0.0005422197492094827 :   (b('dg0_v_vv') <= 0.9517905712127686) ?  (b('bulk') <= 138.5) ?  (b('moss') <= 21.374034881591797) ?  (b('ndvi') <= 1364.0) ?  (b('ndvi') <= 1344.0) ?  (b('moss') <= 17.901039123535156) ?  4.1840953380322346e-05 :   -0.06416564932366593 :   (b('l8b5') <= 2465.8319091796875) ?  0.051872841449598764 :   0.07891491329351494 :   (b('l8b11') <= 1275.856201171875) ?  (b('sand') <= 38.5) ?  -0.047645461736048486 :   -0.07760669992206722 :   (b('moss') <= 13.945783138275146) ?  -0.02938314743702239 :   -0.05052175663190374 :   (b('moss') <= 22.54153347015381) ?  (b('dg0_v_vv') <= 0.029250144958496094) ?  (b('dg0_v_vv') <= -0.022337913513183594) ?  0.04189863584482476 :   0.2675981219765124 :   (b('l8b11') <= 1935.59326171875) ?  -0.001958155493301274 :   0.053419385596075786 :   (b('ndvi_med') <= 1076.25) ?  (b('l8b4') <= 1169.206298828125) ?  0.12898556004615042 :   -0.0029064360074546186 :   (b('moss') <= 24.9157075881958) ?  -0.05332421064385907 :   -0.022139710795737854 :   (b('ndvi_med') <= 2810.25) ?  (b('ndvi') <= 2700.0) ?  (b('grass') <= 3.7684255838394165) ?  (b('ndvi') <= 2316.0) ?  0.05525350537686649 :   -0.03314530232721899 :   (b('grass') <= 62.383872985839844) ?  -0.036561402965585 :   -0.012914035688467457 :   (b('l8b4_med') <= 1048.9940795898438) ?  (b('l8b4_med') <= 773.3999633789062) ?  -0.09659919425781019 :   -0.01390023982180685 :   (b('ndvi') <= 2762.0) ?  0.1598068502775798 :   0.04152934657615344 :   (b('l8b4') <= 929.7593383789062) ?  (b('grass') <= 44.10457229614258) ?  (b('crops') <= 38.36996650695801) ?  0.017489285695667028 :   0.11613631630201286 :   (b('bulk') <= 145.5) ?  0.03833144234985787 :   0.15493232318181113 :   (b('ndvi') <= 1751.0) ?  0.26119686989931 :   (b('l8b5') <= 3383.685791015625) ?  -0.020571013778758432 :   0.06058281010587152 :   (b('l8b4_med') <= 1071.0055541992188) ?  (b('moss') <= 26.06871795654297) ?  (b('bulk') <= 137.5) ?  (b('sand') <= 54.5) ?  (b('l8b4') <= 1029.2115478515625) ?  -0.008699489984808913 :   0.0320118414014415 :   (b('l8b4') <= 740.9545593261719) ?  -0.12166975641276867 :   -0.09897974362125274 :   (b('bulk') <= 156.5) ?  (b('l8b4') <= 888.1572570800781) ?  0.08077011493207122 :   0.01666578506865195 :   (b('dg0_v_vv') <= 3.332456588745117) ?  -0.020622382103490353 :   0.04831480177894228 :   (b('l8b4') <= 714.9935302734375) ?  (b('l8b5') <= 2543.5635986328125) ?  (b('l8b4') <= 666.0565490722656) ?  -0.004245253865672816 :   0.00999007510294897 :   (b('l8b5') <= 2775.501708984375) ?  0.02129402575324746 :   0.031625055295733784 :   (b('moss') <= 32.7097749710083) ?  (b('dg0_v_vv') <= 4.7211713790893555) ?  -0.06807436349111401 :   -0.01490040329865582 :   (b('ndvi') <= 1855.5) ?  -0.04015057586229154 :   0.014772004806326755 :   (b('l8b4') <= 1223.9721069335938) ?  (b('l8b5_med') <= 3062.42626953125) ?  (b('sand') <= 37.5) ?  (b('ndvi_med') <= 1278.0) ?  -0.08896459571702317 :   0.106958062087446 :   (b('bulk') <= 150.5) ?  0.03859092115430913 :   0.09061535490522209 :   (b('l8b5') <= 3256.42919921875) ?  (b('l8b11_med') <= 2502.7966918945312) ?  -0.06235028151878645 :   -0.003240690170206534 :   (b('l8b5') <= 3523.491943359375) ?  -0.11469118257461813 :   -0.14349399747863237 :   (b('ndvi') <= 3315.0) ?  (b('l8b11_med') <= 2503.0577392578125) ?  (b('l8b11_med') <= 1906.191162109375) ?  -0.07462822647750178 :   0.015679143195472298 :   (b('ndvi_med') <= 1597.0) ?  -0.01721399944962917 :   -0.07170117965348242 :   (b('ndvi_med') <= 1668.0) ?  (b('l8b4_med') <= 1877.4243774414062) ?  0.08765621342695379 :   0.08352248382527042 :   0.21991475942874372 :   (b('moss') <= 8.85230016708374) ?  (b('moss') <= 3.33070170879364) ?  (b('grass') <= 1.5864591300487518) ?  (b('sand') <= 30.0) ?  (b('ndvi_med') <= 1662.5) ?  (b('ndvi_med') <= 1383.5) ?  0.21049633505097004 :   0.27094384719981907 :   (b('l8b5') <= 2644.2236328125) ?  (b('l8b5_med') <= 2687.2421875) ?  (b('l8b5_med') <= 2601.205078125) ?  0.11003476244153813 :   0.12065119884174078 :   (b('l8b5') <= 2539.575927734375) ?  0.07700532082961467 :   0.1348181149607551 :   (b('l8b11') <= 2536.2760009765625) ?  (b('dg0_v_vv') <= -1.65861177444458) ?  0.03558501339009526 :   0.13452904698439103 :   (b('l8b4') <= 1908.1124877929688) ?  0.03648200611912617 :   0.020759852193396964 :   (b('ndvi') <= 2175.5) ?  (b('dg0_v_vv') <= 0.5129237174987793) ?  (b('dg0_v_vv') <= -1.822436809539795) ?  (b('l8b5') <= 2768.6463623046875) ?  -0.09427882590222716 :   -0.02273883650869804 :   (b('l8b4') <= 2840.0167236328125) ?  -0.010323340408477315 :   -0.05125632050826697 :   (b('l8b4') <= 2579.587158203125) ?  (b('l8b4_med') <= 2271.3209228515625) ?  0.015848619563610722 :   0.049486711914006866 :   (b('l8b4') <= 2840.0167236328125) ?  0.003667093582447233 :   -0.03688827746169505 :   (b('ndvi') <= 2268.5) ?  (b('dg0_v_vv') <= 0.3700399398803711) ?  0.11385332090540987 :   0.11165501954178503 :   (b('l8b4') <= 2171.085693359375) ?  (b('l8b5') <= 3865.78125) ?  0.02687131812048821 :   -0.011624343055801603 :   (b('l8b5_med') <= 3622.8421630859375) ?  0.06738702419401543 :   0.04552912414987026 :   (b('l8b4_med') <= 1787.4124145507812) ?  (b('grass') <= 4.636483192443848) ?  (b('l8b11') <= 2235.6070556640625) ?  (b('l8b4_med') <= 1536.2318115234375) ?  (b('dg0_v_vv') <= -2.3858132362365723) ?  0.2147455294133938 :   0.23374552321451075 :   (b('ndvi') <= 1523.5) ?  0.1810815592533882 :   0.12922187248906525 :   (b('dg0_v_vv') <= -1.358811855316162) ?  (b('dg0_v_vv') <= -3.5708518028259277) ?  -0.027870228061349776 :   0.00025443244093578676 :   (b('dg0_v_vv') <= 3.1294119358062744) ?  0.04655772923301253 :   -0.036277531176955605 :   (b('l8b11_med') <= 1878.9872436523438) ?  (b('crops') <= 21.614031553268433) ?  (b('l8b11') <= 2058.9573364257812) ?  0.0004675900692010991 :   -0.044496874197757594 :   (b('dg0_v_vv') <= -3.478720188140869) ?  -0.053606946647077505 :   -0.0944569567560984 :   (b('l8b5_med') <= 3117.32177734375) ?  (b('ndvi_med') <= 1849.25) ?  -0.02581793093342617 :   0.006756618442811721 :   (b('l8b11') <= 2436.2935791015625) ?  -0.0834112578339341 :   -0.05302755498836561 :   (b('dg0_v_vv') <= 3.452453136444092) ?  (b('ndvi') <= 2962.0) ?  (b('l8b5') <= 2774.091064453125) ?  (b('ndvi') <= 1347.0) ?  -0.05151338405808082 :   -0.023366040836422356 :   (b('dg0_v_vv') <= 0.8273868560791016) ?  -0.06491615818956314 :   -0.051852475324162865 :   (b('ndvi') <= 3366.0) ?  (b('dg0_v_vv') <= 1.5773720741271973) ?  0.021329131716180366 :   -0.010945987035513458 :   (b('l8b5') <= 4027.55126953125) ?  -0.04249500311382407 :   0.007061771320020345 :   (b('ndvi_med') <= 1766.0) ?  (b('l8b4_med') <= 1909.0737915039062) ?  (b('l8b4') <= 2113.01806640625) ?  0.01574082217136969 :   0.03230659014802845 :   -0.03499287240412739 :   0.08883543391228138 :   (b('moss') <= 3.989913582801819) ?  (b('dg0_v_vv') <= 0.053775787353515625) ?  (b('grass') <= 21.67441177368164) ?  (b('ndvi') <= 2671.5) ?  (b('l8b4') <= 1461.91748046875) ?  (b('l8b4_med') <= 1708.0615234375) ?  0.06023574383835037 :   0.12505733588931706 :   (b('dg0_v_vv') <= -0.3589649200439453) ?  0.01536551165011757 :   0.03967182756935487 :   (b('l8b11') <= 2502.7515869140625) ?  (b('l8b11_med') <= 2589.752685546875) ?  0.026430572408936367 :   0.15325671912837524 :   (b('l8b5_med') <= 2844.591552734375) ?  0.1845375824566589 :   0.11133307921094573 :   (b('l8b11') <= 2464.4984130859375) ?  (b('l8b4') <= 1585.0731811523438) ?  (b('dg0_v_vv') <= -0.5099167823791504) ?  0.13270036103404168 :   0.16170034529841545 :   (b('dg0_v_vv') <= -1.0198516845703125) ?  0.16477722829594285 :   0.19077722138180406 :   (b('l8b4') <= 1515.0216674804688) ?  0.05547960063709609 :   (b('l8b5') <= 2595.552734375) ?  0.11149535732272153 :   0.09497959872974746 :   (b('ndvi') <= 2184.0) ?  (b('ndvi_med') <= 1542.25) ?  (b('ndvi') <= 1448.5) ?  (b('l8b11') <= 2646.8963623046875) ?  -0.023387499915203353 :   0.042205895958293174 :   (b('dg0_v_vv') <= 2.3322720527648926) ?  0.058920295708177084 :   0.15754476411721843 :   (b('dg0_v_vv') <= 1.200279712677002) ?  (b('dg0_v_vv') <= 0.2826824188232422) ?  0.17746852412812847 :   0.0698272138957984 :   (b('dg0_v_vv') <= 1.7162439823150635) ?  0.16411593798685334 :   0.11425844084812402 :   (b('l8b5') <= 4052.642578125) ?  (b('l8b11') <= 2162.038330078125) ?  (b('grass') <= 18.937304496765137) ?  0.11913906404242443 :   0.13931482811419243 :   (b('l8b11') <= 2690.410888671875) ?  0.1844717342759553 :   0.11436147075846301 :   (b('ndvi') <= 4352.5) ?  0.05787282920476744 :   0.009058034132617854 :   (b('dg0_v_vv') <= 2.327847480773926) ?  (b('moss') <= 8.321518421173096) ?  (b('ndvi') <= 2048.0) ?  (b('l8b11') <= 1664.3709106445312) ?  0.17254010001449058 :   (b('dg0_v_vv') <= 0.5550479888916016) ?  -0.03416503493663119 :   -0.010297999398025984 :   (b('bulk') <= 151.0) ?  (b('sand') <= 66.0) ?  -0.013900723389997848 :   -0.10038467883525751 :   (b('l8b5') <= 3164.393310546875) ?  0.021650174311859074 :   0.07222383156075339 :   (b('l8b11_med') <= 2051.5857543945312) ?  (b('l8b4') <= 1636.1359252929688) ?  (b('dg0_v_vv') <= 0.2775735855102539) ?  -0.06653764752376062 :   -0.04427565686463003 :   (b('l8b4') <= 1692.6876831054688) ?  0.06378937585555998 :   -0.06621061937606844 :   (b('l8b4') <= 1995.4308471679688) ?  (b('dg0_v_vv') <= 0.29706668853759766) ?  0.0160990596213077 :   0.0487746246586842 :   (b('l8b4') <= 2291.93603515625) ?  -0.025204252291554343 :   0.04051073480061877 :   (b('crops') <= 63.926958084106445) ?  (b('l8b4') <= 1469.6049194335938) ?  (b('dg0_v_vv') <= 3.145423412322998) ?  (b('grass') <= 72.20706558227539) ?  0.12536168413876186 :   0.07479797597635959 :   (b('dg0_v_vv') <= 3.7030813694000244) ?  0.22107159535704196 :   0.19561355902678587 :   (b('l8b4_med') <= 1255.1715698242188) ?  -0.06047129015636074 :   (b('grass') <= 82.46515655517578) ?  0.07394709218473605 :   0.15045274773151973 :   (b('sand') <= 50.0) ?  (b('l8b4') <= 1500.5209350585938) ?  (b('crops') <= 71.0881233215332) ?  -0.0728739834118877 :   -0.07481018307970896 :   (b('l8b5') <= 3293.1539306640625) ?  0.028343876822806955 :   -0.015544746291745322 :   (b('dg0_v_vv') <= 3.509432077407837) ?  (b('dg0_v_vv') <= 3.28745698928833) ?  0.06235098957050107 :   -0.025235535623223775 :   (b('l8b5') <= 2776.079833984375) ?  0.12224573448975865 :   0.06638670511891738 :   (b('l8b5_med') <= 3287.4775390625) ?  (b('dg0_v_vv') <= 1.1927642822265625) ?  (b('crops') <= 72.32853889465332) ?  (b('moss') <= 50.33349609375) ?  (b('l8b4') <= 1873.9694213867188) ?  (b('ndvi_med') <= 1476.0) ?  (b('ndvi') <= 1497.5) ?  -0.03327903327687849 :   0.00499178959805876 :   (b('moss') <= 15.359296321868896) ?  -0.03504608068429422 :   -0.052684504913381745 :   (b('l8b5_med') <= 2627.36669921875) ?  (b('dg0_v_vv') <= -0.36457252502441406) ?  -0.04408488246030348 :   -0.0692341132497109 :   (b('crops') <= 39.207810401916504) ?  -0.03812637451305058 :   -0.063927674200151 :   (b('ndvi') <= 1172.0) ?  (b('l8b5') <= 2168.384521484375) ?  -0.042086858791149276 :   (b('l8b5') <= 2660.3890380859375) ?  -0.07565672443662784 :   -0.08594637731492587 :   (b('l8b5') <= 2165.2298583984375) ?  (b('l8b5') <= 2129.3165283203125) ?  -0.09682789258671519 :   -0.08633202279938199 :   (b('l8b5') <= 2471.190185546875) ?  -0.04703384324788216 :   -0.06732055826765442 :   (b('l8b4') <= 1656.4095458984375) ?  (b('dg0_v_vv') <= 0.6935839653015137) ?  (b('l8b11') <= 2984.068115234375) ?  (b('dg0_v_vv') <= 0.38795995712280273) ?  0.03592407028493164 :   -0.005201064160333405 :   (b('l8b5_med') <= 2846.0980224609375) ?  0.07485654244182385 :   0.05311860144259334 :   (b('l8b11') <= 3108.0413818359375) ?  (b('dg0_v_vv') <= 0.9605884552001953) ?  -0.02999019230334496 :   -0.02622882476184124 :   -0.03591674071237891 :   (b('l8b4') <= 1782.654296875) ?  (b('l8b4') <= 1762.5487060546875) ?  (b('ndvi_med') <= 1480.5) ?  -0.0381700332462811 :   -0.02364264395120296 :   (b('dg0_v_vv') <= -0.664736270904541) ?  -0.0333878137616815 :   -0.03877268734322191 :   (b('l8b4') <= 1808.9067993164062) ?  0.11663052258572426 :   (b('l8b5') <= 3169.7298583984375) ?  -0.019373674788940823 :   0.0057925274955455295 :   (b('ndvi_med') <= 1554.5) ?  (b('grass') <= 50.46748924255371) ?  (b('dg0_v_vv') <= 2.8280601501464844) ?  (b('l8b11_med') <= 2760.4342041015625) ?  (b('ndvi') <= 1219.5) ?  -0.06930531668598655 :   -0.024092757100738903 :   (b('l8b5') <= 3483.3876953125) ?  -0.008221461527218152 :   0.1314314406957716 :   (b('ndvi_med') <= 1480.5) ?  (b('dg0_v_vv') <= 3.5662667751312256) ?  0.005835690484655971 :   0.09347468023060301 :   (b('dg0_v_vv') <= 4.588886737823486) ?  0.10616045008431639 :   0.04214338191612786 :   (b('l8b4') <= 1749.3042602539062) ?  (b('moss') <= 35.18026161193848) ?  (b('dg0_v_vv') <= 1.2355046272277832) ?  0.1349516270033902 :   0.0394037252023646 :   (b('dg0_v_vv') <= 2.2404685020446777) ?  0.07166664723622894 :   0.19362277781119974 :   (b('dg0_v_vv') <= 2.3908677101135254) ?  (b('l8b5') <= 2860.52685546875) ?  -0.04295792913209684 :   0.005885161548645689 :   (b('l8b5_med') <= 2799.6318359375) ?  0.07718647234834103 :   -0.0005837596283122471 :   (b('l8b5') <= 2094.848876953125) ?  (b('bulk') <= 155.5) ?  0.058947976555384224 :   0.16048676160460962 :   (b('moss') <= 33.88349628448486) ?  (b('ndvi') <= 2356.0) ?  (b('crops') <= 39.207810401916504) ?  -0.0778502194284398 :   -0.03555325219567164 :   (b('ndvi') <= 2448.5) ?  0.10088599327387165 :   -0.016988931232987795 :   (b('dg0_v_vv') <= 2.209486484527588) ?  (b('ndvi_med') <= 1616.75) ?  -0.00021539056642341792 :   0.05299338399949674 :   0.11585146223758111 :   (b('bulk') <= 151.5) ?  (b('l8b5_med') <= 3339.2752685546875) ?  (b('grass') <= 18.531929969787598) ?  (b('ndvi') <= 1575.5) ?  (b('ndvi') <= 1320.0) ?  (b('l8b5') <= 3597.01220703125) ?  -0.0663392517361525 :   -0.09170125983946861 :   (b('dg0_v_vv') <= -1.7027440071105957) ?  -0.06009572535186378 :   -0.08240565468108035 :   (b('ndvi') <= 4159.5) ?  (b('ndvi') <= 3177.5) ?  -0.063257020558763 :   -0.07596801618057525 :   (b('l8b5_med') <= 3311.2408447265625) ?  -0.040944247407929686 :   -0.05027660649608344 :   (b('l8b5') <= 2348.0582275390625) ?  (b('ndvi') <= 1866.0) ?  (b('dg0_v_vv') <= -0.10222387313842773) ?  -0.11176254153234703 :   -0.11382643060614571 :   -0.10176254175586445 :   (b('l8b5') <= 3725.3699951171875) ?  (b('l8b5') <= 3541.3212890625) ?  -0.044605947004103245 :   -0.009855703784304577 :   (b('ndvi_med') <= 1569.0) ?  -0.07829622888652878 :   -0.10113928438488334 :   (b('dg0_v_vv') <= 2.394139051437378) ?  (b('l8b4_med') <= 2523.0831298828125) ?  (b('l8b5') <= 3069.4146728515625) ?  (b('l8b5') <= 2992.95263671875) ?  -0.07026345319581055 :   -0.04366577914598441 :   (b('l8b5') <= 4005.8782958984375) ?  -0.07772842901151553 :   -0.06568874059226695 :   (b('l8b5') <= 3536.8404541015625) ?  (b('l8b4') <= 2828.0234375) ?  -0.030521194833836918 :   0.007293364967874347 :   (b('l8b11') <= 3866.191650390625) ?  -0.06456269326525649 :   -0.04474457662597371 :   (b('l8b11') <= 3581.6116943359375) ?  (b('dg0_v_vv') <= 3.1970629692077637) ?  (b('l8b4') <= 1529.4222412109375) ?  -0.041510935806828776 :   -0.06452807766947434 :   (b('l8b5') <= 3905.39892578125) ?  -0.03681809285970287 :   -0.06224215617703562 :   (b('l8b5') <= 3559.726318359375) ?  -0.06550078746454664 :   (b('dg0_v_vv') <= 4.104727268218994) ?  -0.09336957076858032 :   -0.06585531172594983 :   (b('l8b11') <= 3208.2916259765625) ?  (b('l8b11') <= 2337.451416015625) ?  (b('ndvi') <= 1676.5) ?  -0.0800634462134864 :   (b('ndvi') <= 3073.5) ?  -0.03789311408227025 :   -0.012229652247971401 :   (b('l8b11_med') <= 2365.364013671875) ?  (b('l8b4') <= 1560.3987426757812) ?  (b('dg0_v_vv') <= 3.188333749771118) ?  0.06555575425170296 :   0.07568688408309879 :   (b('l8b4') <= 1617.72119140625) ?  0.00461403799466939 :   0.027719882153757577 :   (b('l8b4') <= 2516.7257080078125) ?  (b('l8b4') <= 1617.72119140625) ?  -0.004730544113610446 :   0.014812072890424032 :   (b('bulk') <= 152.5) ?  -0.008658612769445156 :   -0.03465329849452095 :   (b('dg0_v_vv') <= 2.935779571533203) ?  (b('ndvi') <= 1039.0) ?  (b('l8b4') <= 3014.6832275390625) ?  (b('dg0_v_vv') <= -0.6462430953979492) ?  -0.03672337567142031 :   -0.013027040511832604 :   (b('dg0_v_vv') <= 0.6504669189453125) ?  -0.0496013811385082 :   -0.019106174980447775 :   (b('l8b11') <= 3587.3641357421875) ?  (b('l8b4_med') <= 2785.3529052734375) ?  -0.05519851949070087 :   -0.06401350792126333 :   (b('l8b5_med') <= 3900.0040283203125) ?  -0.02740831294747026 :   -0.049162960526327425 :   (b('moss') <= 23.845518112182617) ?  (b('dg0_v_vv') <= 3.0064029693603516) ?  (b('l8b4_med') <= 2874.1834716796875) ?  0.020097172030348526 :   0.005811892117467632 :   (b('dg0_v_vv') <= 3.1402664184570312) ?  -0.029258507519687 :   -0.003414776343725223 :   0.06149400970595642 :   (b('grass') <= 46.14527893066406) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('bulk') <= 140.5) ?  (b('crops') <= 15.965596199035645) ?  (b('ndvi') <= 2919.5) ?  (b('ndvi') <= 2650.5) ?  (b('l8b5') <= 1948.04248046875) ?  (b('l8b4') <= 833.5028076171875) ?  (b('l8b4') <= 758.850830078125) ?  -0.03598674114176559 :   -0.052186785535757035 :   -0.09257632932494791 :   (b('ndvi') <= 2193.0) ?  (b('l8b5') <= 2104.5379638671875) ?  0.018717289469981496 :   0.005005361059723694 :   -0.031864867273510206 :   (b('l8b11') <= 1057.0957641601562) ?  (b('l8b4') <= 485.29347229003906) ?  (b('dg0_v_vv') <= 0.18603944778442383) ?  0.03110823697292754 :   0.07940405172902436 :   (b('l8b4_med') <= 447.916259765625) ?  -0.013241204155759911 :   -0.030589441266361816 :   (b('l8b4_med') <= 447.916259765625) ?  (b('dg0_v_vv') <= -0.5066707134246826) ?  0.1397771431160145 :   0.1793714507677494 :   (b('dg0_v_vv') <= -0.48138999938964844) ?  0.12963861147478345 :   0.1137444245019153 :   (b('grass') <= 33.51909637451172) ?  (b('ndvi') <= 4141.0) ?  (b('l8b5') <= 3306.5584716796875) ?  (b('grass') <= 32.576393127441406) ?  0.11873761959450596 :   0.07639772545748508 :   (b('l8b4') <= 551.1476287841797) ?  0.046957595721124464 :   0.003261198081057959 :   (b('ndvi') <= 4832.5) ?  (b('l8b5') <= 3306.5584716796875) ?  0.027614737165639523 :   -0.03806704766803423 :   (b('dg0_v_vv') <= 0.22580862045288086) ?  -0.07330995604051851 :   -0.0894919061837372 :   (b('l8b11') <= 966.4153747558594) ?  (b('dg0_v_vv') <= -0.3677973747253418) ?  (b('dg0_v_vv') <= -0.3761286735534668) ?  -0.035378347644930155 :   0.022016435571739323 :   (b('grass') <= 39.78398895263672) ?  -0.056344060043548125 :   -0.018921924437212828 :   (b('ndvi') <= 4160.0) ?  (b('ndvi') <= 4093.0) ?  -0.005431460898381931 :   0.0904653211376718 :   (b('l8b5') <= 3358.6680908203125) ?  -0.05604404418825507 :   -0.01733825623687175 :   (b('dg0_v_vv') <= -0.5682883262634277) ?  (b('l8b5_med') <= 2510.6680908203125) ?  (b('l8b5') <= 3719.327880859375) ?  (b('ndvi') <= 4508.0) ?  (b('l8b5_med') <= 2463.5460205078125) ?  -0.06783422442943725 :   -0.07281486469746448 :   (b('l8b5_med') <= 2463.5460205078125) ?  -0.07864618405529833 :   -0.07085969684192137 :   (b('l8b5_med') <= 2410.511962890625) ?  0.019919229268847677 :   (b('ndvi_med') <= 3916.25) ?  -0.05254694148743255 :   -0.02383910116322957 :   (b('l8b11') <= 1076.944580078125) ?  (b('ndvi_med') <= 4430.0) ?  (b('l8b11_med') <= 1437.3794555664062) ?  0.03657096375948973 :   0.2477161877001704 :   (b('l8b5_med') <= 3413.9039306640625) ?  0.09424164339065795 :   0.17023429164490758 :   (b('l8b4') <= 1729.2342529296875) ?  (b('l8b5') <= 3603.5684814453125) ?  0.00633718280557253 :   0.04132836544060089 :   (b('l8b5') <= 3486.552734375) ?  0.20241517285375382 :   0.14829567226353835 :   (b('moss') <= 7.155266046524048) ?  (b('l8b5') <= 4743.040771484375) ?  (b('ndvi') <= 2789.5) ?  (b('ndvi_med') <= 4407.25) ?  0.061650914611772176 :   0.21028865431082688 :   (b('l8b4') <= 671.5080261230469) ?  0.045691527030036316 :   0.017716486281550647 :   (b('crops') <= 49.725759506225586) ?  (b('dg0_v_vv') <= 1.0210566520690918) ?  0.027534616792869573 :   0.08670282605830631 :   (b('dg0_v_vv') <= 2.550539016723633) ?  -0.08588074593311178 :   -0.026124229138044 :   (b('l8b5') <= 3666.6407470703125) ?  (b('ndvi') <= 4812.0) ?  (b('dg0_v_vv') <= 1.9611015319824219) ?  0.04468071465707082 :   0.10313572082296284 :   (b('moss') <= 10.41492509841919) ?  0.16718114287232275 :   0.2372042637059643 :   (b('l8b5_med') <= 2952.3045654296875) ?  (b('dg0_v_vv') <= 1.2019729614257812) ?  0.042863133188223775 :   0.09876894978337555 :   (b('l8b11_med') <= 1098.6852416992188) ?  0.0525271539618751 :   0.11413928601591579 :   (b('sand') <= 35.5) ?  (b('crops') <= 62.37131881713867) ?  (b('dg0_v_vv') <= 1.4949016571044922) ?  (b('l8b5_med') <= 2399.0399169921875) ?  (b('dg0_v_vv') <= -0.5217533111572266) ?  (b('l8b11') <= 1136.8828125) ?  0.0030565035222769876 :   -0.08140008118820556 :   (b('l8b4') <= 1023.2243041992188) ?  0.0674188570039826 :   0.13023867081005602 :   (b('ndvi') <= 3011.5) ?  (b('ndvi') <= 2784.0) ?  0.12218519125858276 :   0.17784124275600635 :   (b('l8b5') <= 2578.120361328125) ?  0.02420777035736255 :   0.10159298634567862 :   (b('l8b11') <= 1030.84912109375) ?  (b('l8b11_med') <= 1301.8933715820312) ?  0.17941989759979693 :   (b('ndvi_med') <= 3396.0) ?  0.07221714160701859 :   0.020839538974892324 :   (b('sand') <= 25.5) ?  (b('ndvi') <= 4086.5) ?  0.1937030323928345 :   0.1644249641674918 :   (b('l8b5') <= 4164.2515869140625) ?  0.13259613421642752 :   0.17396050752788056 :   (b('l8b5') <= 4664.878662109375) ?  (b('dg0_v_vv') <= 1.8820610046386719) ?  (b('ndvi') <= 3632.0) ?  (b('moss') <= 3.325710356235504) ?  -0.09507386658915347 :   0.03621668994030971 :   (b('l8b4_med') <= 673.5367126464844) ?  0.015941861857873003 :   0.10534955894280228 :   (b('ndvi') <= 4682.0) ?  (b('l8b5') <= 3052.5333251953125) ?  0.1475933046957756 :   0.11845515004330849 :   0.047651454832609796 :   (b('dg0_v_vv') <= -0.016918659210205078) ?  (b('l8b11_med') <= 1177.0497436523438) ?  0.158194049590758 :   0.1362896961629172 :   0.19128993544255601 :   (b('ndvi') <= 4902.5) ?  (b('l8b5') <= 2905.0257568359375) ?  (b('bulk') <= 147.5) ?  (b('l8b5') <= 2576.404052734375) ?  (b('dg0_v_vv') <= -0.062235355377197266) ?  0.07021686389726028 :   0.1046246509416357 :   0.14901755709039072 :   (b('grass') <= 38.69702911376953) ?  (b('l8b5') <= 2834.5682373046875) ?  0.008115769860050437 :   0.06340371746648313 :   -0.05431688490519368 :   (b('ndvi') <= 4442.0) ?  (b('dg0_v_vv') <= 0.870457649230957) ?  (b('dg0_v_vv') <= -0.0833749771118164) ?  -0.015915979057550766 :   -0.09518806222265165 :   0.0651504179489521 :   (b('l8b11') <= 1048.4972839355469) ?  0.06751734855202293 :   0.0781504144918827 :   (b('dg0_v_vv') <= 1.0395183563232422) ?  -0.12835155524254468 :   -0.1229270891556449 :   (b('moss') <= 6.192884922027588) ?  (b('bulk') <= 154.0) ?  (b('dg0_v_vv') <= -1.2381086349487305) ?  (b('sand') <= 39.0) ?  (b('l8b5') <= 2231.1935424804688) ?  0.13620269330156926 :   (b('l8b4') <= 760.3129272460938) ?  (b('l8b4_med') <= 749.7178344726562) ?  0.10433443175129525 :   -0.004054560394315586 :   (b('dg0_v_vv') <= -2.3816895484924316) ?  -0.05923402451915825 :   -0.021329752149013247 :   (b('l8b4') <= 1530.1837158203125) ?  (b('l8b11') <= 1860.661376953125) ?  (b('l8b4_med') <= 903.0169677734375) ?  0.018850081049752304 :   -0.02270318882074437 :   (b('l8b4_med') <= 916.4306945800781) ?  0.03962746178574053 :   0.07374069342969937 :   (b('l8b11') <= 2715.3499755859375) ?  -0.04259349998797926 :   (b('ndvi_med') <= 3656.75) ?  -0.04528549337522568 :   -0.046785492689772265 :   (b('grass') <= 39.1515007019043) ?  (b('ndvi') <= 4096.0) ?  (b('ndvi_med') <= 3910.75) ?  (b('moss') <= 2.014968693256378) ?  0.07424066283146308 :   0.04587213250003651 :   (b('ndvi') <= 2262.0) ?  0.0032397226015263014 :   0.10164921819172228 :   (b('l8b11') <= 1613.47021484375) ?  (b('l8b11') <= 1257.7529296875) ?  0.025625101263241155 :   0.06923210396856729 :   (b('l8b11') <= 1716.5633544921875) ?  -0.0626052728966913 :   0.020667036868625033 :   (b('l8b11') <= 1884.1138305664062) ?  (b('ndvi') <= 4697.0) ?  (b('dg0_v_vv') <= -1.0430450439453125) ?  0.04534106382770331 :   -0.04423305228542357 :   (b('l8b4') <= 648.9554443359375) ?  0.04731599475448375 :   0.01816589090455792 :   (b('l8b11') <= 2273.8206176757812) ?  (b('ndvi') <= 3808.5) ?  0.028220874993868855 :   0.04624793507668945 :   0.06367180564812863 :   (b('dg0_v_vv') <= -0.09801959991455078) ?  (b('ndvi') <= 3252.0) ?  (b('l8b11_med') <= 2053.1215209960938) ?  (b('ndvi') <= 2045.5) ?  -0.07736869059555682 :   (b('l8b5') <= 3390.773681640625) ?  -0.07915957761614702 :   -0.08256825130668247 :   (b('dg0_v_vv') <= -3.660003662109375) ?  -0.08992916322887393 :   (b('ndvi') <= 1879.0) ?  0.006083590933433886 :   -0.0551300913178134 :   (b('l8b4') <= 1132.4074096679688) ?  (b('l8b4') <= 550.2295227050781) ?  (b('l8b4_med') <= 1058.0714111328125) ?  -0.04244823815303429 :   0.018018340003613964 :   (b('dg0_v_vv') <= -2.1876139640808105) ?  -0.07418869531353316 :   -0.050753263804948885 :   (b('dg0_v_vv') <= -1.2807602882385254) ?  0.005872201572616725 :   -0.0011583169278896743 :   (b('l8b5') <= 3461.7615966796875) ?  (b('l8b4') <= 1368.5966796875) ?  (b('ndvi') <= 3405.0) ?  (b('l8b4') <= 1090.0593872070312) ?  -0.020751366876139148 :   0.004636940782256728 :   (b('l8b11') <= 1367.9501647949219) ?  -0.007448675843720137 :   0.025275958831991432 :   0.08216970353933581 :   (b('ndvi') <= 4478.5) ?  (b('dg0_v_vv') <= 0.07938957214355469) ?  -0.004991424839194819 :   (b('l8b5') <= 3636.8193359375) ?  -0.07333047647250639 :   -0.048812363031137396 :   0.0360750465790573 :   (b('l8b4') <= 767.1305541992188) ?  (b('crops') <= 4.350515365600586) ?  (b('l8b11') <= 1367.6683349609375) ?  (b('l8b11') <= 1083.822998046875) ?  (b('ndvi') <= 3478.5) ?  (b('dg0_v_vv') <= -0.01659393310546875) ?  0.07669413222069998 :   0.0432941286086585 :   (b('l8b4') <= 524.6513671875) ?  0.10754622608147357 :   0.10382404085643201 :   (b('ndvi') <= 4466.5) ?  (b('ndvi') <= 3940.5) ?  0.005843449294713958 :   -0.06539599290134838 :   (b('dg0_v_vv') <= -0.2658267021179199) ?  0.0249128980894065 :   0.10677306483492441 :   (b('dg0_v_vv') <= -0.1450362205505371) ?  (b('l8b11_med') <= 1635.6578369140625) ?  0.009986646210765476 :   (b('l8b11') <= 1422.6717529296875) ?  0.13010581582065794 :   0.06980934733400591 :   (b('l8b5') <= 2920.6778564453125) ?  (b('l8b4') <= 756.4701538085938) ?  0.16442173908812485 :   0.1265329816934826 :   (b('l8b5') <= 3183.467529296875) ?  0.1077037151194633 :   0.0696915812883146 :   (b('l8b5') <= 2622.435546875) ?  (b('ndvi') <= 3085.5) ?  (b('l8b11') <= 1176.593505859375) ?  (b('dg0_v_vv') <= 2.685582160949707) ?  -0.014938110868792974 :   0.010203668646906167 :   (b('l8b4_med') <= 759.933837890625) ?  0.012585759360475256 :   0.03765490208313422 :   (b('dg0_v_vv') <= -0.17564725875854492) ?  0.018804239191045208 :   (b('ndvi_med') <= 3830.0) ?  0.09291978878486348 :   0.028633954315037707 :   (b('moss') <= 8.678699970245361) ?  (b('dg0_v_vv') <= 2.611720561981201) ?  (b('l8b5') <= 3851.60302734375) ?  -0.025616003606995504 :   -0.00424971606706191 :   (b('dg0_v_vv') <= 8.116326808929443) ?  -0.07545238241516944 :   -0.03280369241894533 :   (b('l8b5') <= 3483.03466796875) ?  (b('dg0_v_vv') <= 1.2603175640106201) ?  -0.011009913403290064 :   0.020666249049304673 :   (b('ndvi') <= 3881.5) ?  0.08083505937490088 :   0.018802892627327344 :   (b('crops') <= 10.455060958862305) ?  (b('ndvi') <= 4518.5) ?  (b('l8b11') <= 2113.9287109375) ?  (b('l8b11') <= 2060.95751953125) ?  (b('l8b5') <= 2242.8656005859375) ?  -0.06025027839165039 :   -0.032055553922063384 :   (b('dg0_v_vv') <= -0.38855504989624023) ?  0.010776816690996135 :   0.15765246442590156 :   (b('ndvi') <= 3626.0) ?  (b('dg0_v_vv') <= 0.11444950103759766) ?  -0.0551480273055104 :   -0.03952812597520457 :   -0.08102583957295988 :   (b('dg0_v_vv') <= 1.4779818058013916) ?  (b('dg0_v_vv') <= -0.8273563385009766) ?  (b('dg0_v_vv') <= -1.5387964248657227) ?  -0.03875419361890112 :   0.15092627918577345 :   (b('dg0_v_vv') <= 0.41457700729370117) ?  -0.06945888933778661 :   0.011699130409768934 :   0.17598898241520244 :   (b('ndvi') <= 3217.0) ?  (b('l8b5') <= 3233.586181640625) ?  (b('l8b4') <= 1451.6693115234375) ?  (b('bulk') <= 132.5) ?  0.014776503632181352 :   -0.023588059387224497 :   (b('l8b11_med') <= 1872.1339111328125) ?  0.05091699539640182 :   0.14687407883556824 :   (b('l8b5') <= 3544.9710693359375) ?  (b('crops') <= 29.669858932495117) ?  -0.07337433521908021 :   -0.052554771762296754 :   (b('ndvi') <= 2845.0) ?  -0.0349440152490528 :   0.0031766898463803836 :   (b('ndvi') <= 3287.0) ?  (b('ndvi') <= 3275.5) ?  (b('dg0_v_vv') <= 0.13550710678100586) ?  0.012993942073372572 :   0.06465542226438042 :   (b('dg0_v_vv') <= -0.13358068466186523) ?  0.11728529073589056 :   0.16024266565664189 :   (b('l8b4_med') <= 1146.3720703125) ?  (b('l8b4') <= 1085.6044311523438) ?  -0.004353208454528274 :   -0.022929125367626188 :   (b('l8b5') <= 3385.3167724609375) ?  0.04541990268799291 :   -0.015003699234755048 :   (b('sand') <= 50.5) ?  (b('bulk') <= 144.5) ?  (b('l8b4') <= 1124.726806640625) ?  (b('l8b5_med') <= 3092.5577392578125) ?  (b('dg0_v_vv') <= 0.7944493293762207) ?  (b('l8b5') <= 1744.6207275390625) ?  0.1102890333737542 :   (b('l8b4') <= 749.9579467773438) ?  (b('l8b5') <= 3840.787841796875) ?  -0.035046484802454925 :   0.021589280071030597 :   (b('moss') <= 6.563378095626831) ?  0.008504484871381286 :   -0.03980581981260685 :   (b('l8b5') <= 3379.8458251953125) ?  (b('grass') <= 63.029747009277344) ?  (b('dg0_v_vv') <= 0.9839110374450684) ?  0.06069863800189617 :   -0.022801373662879286 :   (b('dg0_v_vv') <= 2.776118755340576) ?  0.06709581554264542 :   0.0012679047978413533 :   (b('dg0_v_vv') <= 1.1116118431091309) ?  -0.10579734911234648 :   (b('grass') <= 67.60559463500977) ?  -0.02444526103719187 :   -0.037709288274398914 :   (b('ndvi') <= 3703.0) ?  (b('dg0_v_vv') <= 1.8851728439331055) ?  (b('l8b4') <= 992.394775390625) ?  (b('l8b11') <= 1568.970458984375) ?  -0.04100755145160702 :   0.006227425060497967 :   (b('l8b11') <= 1835.360107421875) ?  0.13206489335765634 :   0.05425041573435286 :   0.22679255262058656 :   (b('l8b5_med') <= 3312.4837646484375) ?  (b('dg0_v_vv') <= -0.28708791732788086) ?  (b('l8b4') <= 903.6016845703125) ?  0.15799128016171476 :   0.039330994485921714 :   (b('dg0_v_vv') <= -0.09836435317993164) ?  -0.03919390371756859 :   0.07607277893590833 :   (b('ndvi') <= 3895.0) ?  (b('dg0_v_vv') <= -0.5569291114807129) ?  0.03551140679327591 :   0.11858221681418077 :   (b('ndvi') <= 4031.5) ?  -0.0232751220041963 :   0.03588394293060466 :   (b('l8b5') <= 3540.3916015625) ?  (b('dg0_v_vv') <= -1.1103782653808594) ?  (b('moss') <= 6.438619136810303) ?  0.11247027782161825 :   0.0896578585238289 :   (b('l8b4_med') <= 785.3102722167969) ?  (b('ndvi_med') <= 3704.0) ?  0.11261512090051506 :   (b('l8b11') <= 2240.3543701171875) ?  0.12723369678134486 :   0.141193073180614 :   (b('dg0_v_vv') <= -0.4791259765625) ?  0.1291304627511744 :   0.1989987981790438 :   -0.0077724031549532 :   (b('dg0_v_vv') <= -1.8383870124816895) ?  (b('l8b5') <= 1901.5988159179688) ?  0.0779684674263946 :   (b('dg0_v_vv') <= -1.9867329597473145) ?  (b('l8b11_med') <= 1434.456787109375) ?  (b('l8b5') <= 2216.750244140625) ?  0.04359385041422509 :   (b('l8b4') <= 700.5178833007812) ?  0.02820781230717223 :   0.022244492690396273 :   (b('l8b11') <= 1670.9224853515625) ?  (b('sand') <= 25.5) ?  -0.004512862840472365 :   0.005731223106841113 :   (b('l8b5') <= 2712.918701171875) ?  -0.021808515203619937 :   -0.036535749606168025 :   -0.08186435680346668 :   (b('moss') <= 3.0389994382858276) ?  (b('l8b11') <= 1708.9014892578125) ?  (b('dg0_v_vv') <= 0.0018677711486816406) ?  (b('l8b11') <= 1205.639404296875) ?  (b('crops') <= 10.516666650772095) ?  0.14207293444945046 :   0.014554768560644145 :   (b('dg0_v_vv') <= -0.5277738571166992) ?  0.07079928956501147 :   0.12803983508407396 :   (b('l8b4') <= 714.7695922851562) ?  (b('dg0_v_vv') <= 0.8233375549316406) ?  0.11917751977479396 :   0.18928490088897007 :   (b('ndvi') <= 3000.0) ?  0.12934999817087775 :   0.051628710257273745 :   (b('l8b5') <= 2057.7791137695312) ?  (b('l8b4') <= 1058.5663452148438) ?  (b('dg0_v_vv') <= 2.086695671081543) ?  0.11473547187895275 :   0.14968023094821084 :   0.026177932057195485 :   (b('ndvi') <= 2019.0) ?  (b('l8b4') <= 1255.6373901367188) ?  0.07209968225281557 :   0.15121211152065187 :   (b('sand') <= 19.5) ?  0.20544263829573256 :   0.16440319750026325 :   (b('dg0_v_vv') <= 2.847309112548828) ?  (b('ndvi') <= 4489.0) ?  (b('l8b5') <= 2274.349609375) ?  (b('dg0_v_vv') <= -1.1957588195800781) ?  0.07327971789515944 :   0.13835935500301622 :   (b('l8b5') <= 2355.9151611328125) ?  -0.03532865627634855 :   0.05371543949843478 :   (b('l8b4') <= 899.1400756835938) ?  (b('ndvi') <= 4873.0) ?  0.06637259672723372 :   0.16434242241109928 :   (b('ndvi_med') <= 3725.0) ?  0.23345256138124326 :   0.22335954124921392 :   (b('moss') <= 8.197460174560547) ?  (b('l8b5_med') <= 3028.8795166015625) ?  0.15647257536544934 :   0.14944900040767367 :   0.19500179443480053 :   (b('crops') <= 16.23762822151184) ?  (b('l8b5') <= 2243.9810791015625) ?  (b('l8b5') <= 1908.4381713867188) ?  (b('l8b5') <= 1794.5233154296875) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('ndvi_med') <= 3533.5) ?  0.19667025398074067 :   (b('dg0_v_vv') <= 1.2517390251159668) ?  0.2019859535310054 :   0.2010859382960582 :   (b('ndvi') <= 2416.0) ?  (b('l8b5') <= 1609.0025024414062) ?  0.2582867469861936 :   0.1316963844256376 :   (b('dg0_v_vv') <= -0.7956352233886719) ?  0.13034799484413478 :   0.07353976302410836 :   (b('ndvi_med') <= 3807.5) ?  -0.07226344860658049 :   -0.04033531450203229 :   (b('grass') <= 58.827531814575195) ?  (b('ndvi') <= 4657.0) ?  (b('l8b5') <= 2085.4517822265625) ?  (b('l8b5') <= 2011.3902587890625) ?  0.19474377456173142 :   0.05176898414404624 :   (b('dg0_v_vv') <= -0.4921112060546875) ?  0.1601425736085817 :   0.20580107577112658 :   (b('dg0_v_vv') <= -1.0146560668945312) ?  (b('dg0_v_vv') <= -1.2193865776062012) ?  0.18998807009399624 :   -0.0012850601708867537 :   (b('dg0_v_vv') <= -0.4122004508972168) ?  0.20121702935899788 :   0.14919195062886728 :   (b('ndvi') <= 2948.0) ?  (b('dg0_v_vv') <= 0.503242015838623) ?  0.08485579907093518 :   0.12069703619525682 :   0.029442839151270106 :   (b('l8b11') <= 958.7872924804688) ?  (b('dg0_v_vv') <= -0.07867813110351562) ?  (b('ndvi') <= 3225.0) ?  0.159925720768839 :   (b('l8b4_med') <= 589.7862548828125) ?  (b('ndvi_med') <= 3709.25) ?  0.08761302381971256 :   0.08837290430199948 :   (b('l8b5_med') <= 2629.4852294921875) ?  0.1241057120323209 :   0.10701772564837672 :   (b('l8b5') <= 2922.29052734375) ?  (b('dg0_v_vv') <= 0.33847713470458984) ?  (b('dg0_v_vv') <= -0.0008087158203125) ?  0.16068953827281204 :   0.12622944512746836 :   (b('ndvi') <= 3225.0) ?  0.1564429503713433 :   0.1777407321714755 :   0.08241116735128876 :   (b('l8b11') <= 1387.8487548828125) ?  (b('ndvi') <= 3355.0) ?  (b('dg0_v_vv') <= 0.4252181053161621) ?  (b('dg0_v_vv') <= -0.08604717254638672) ?  0.049019471787358986 :   0.03552776310375644 :   (b('dg0_v_vv') <= 0.7631731033325195) ?  0.07885873207319136 :   0.06385871657598371 :   (b('l8b5') <= 2304.115966796875) ?  (b('dg0_v_vv') <= -1.3460636138916016) ?  -0.1118488802828298 :   0.08149323842123228 :   (b('dg0_v_vv') <= 0.4556846618652344) ?  -0.03633927846178399 :   0.031393737091870676 :   (b('l8b11_med') <= 1269.3097534179688) ?  (b('ndvi') <= 3128.5) ?  (b('dg0_v_vv') <= 2.4640579223632812) ?  0.12708892426082652 :   0.27416746635027067 :   (b('dg0_v_vv') <= 0.6448264122009277) ?  0.07692759913582908 :   0.03135872149336691 :   (b('ndvi') <= 3637.5) ?  (b('crops') <= 2.891176462173462) ?  -0.04658003103519121 :   -0.0759218013102892 :   0.031121543209366803 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b11') <= 1029.7211608886719) ?  (b('dg0_v_vv') <= -0.0663909912109375) ?  (b('dg0_v_vv') <= -0.08670473098754883) ?  (b('l8b5_med') <= 2553.574951171875) ?  0.19336151519251055 :   0.16892505346631065 :   0.26599182767059215 :   (b('l8b4_med') <= 579.2226257324219) ?  (b('l8b5') <= 3967.3460693359375) ?  0.17119587590500812 :   0.1446898086117634 :   0.11509412932606256 :   (b('l8b11_med') <= 1348.42724609375) ?  (b('dg0_v_vv') <= -0.936497688293457) ?  (b('ndvi_med') <= 3494.75) ?  0.16265424084199864 :   0.19726652783154874 :   (b('crops') <= 25.986759185791016) ?  0.20663771778910758 :   0.19499616909088857 :   (b('l8b4_med') <= 527.48388671875) ?  (b('l8b5') <= 3197.041259765625) ?  0.23131979222491184 :   0.24466828764284765 :   (b('l8b4_med') <= 548.9114074707031) ?  0.22559251638198696 :   0.21329250188613735 :   (b('l8b4') <= 491.7077331542969) ?  (b('grass') <= 59.88348388671875) ?  (b('l8b5') <= 3231.26171875) ?  (b('dg0_v_vv') <= -0.8103389739990234) ?  0.2632707744411396 :   0.268212268052107 :   (b('ndvi') <= 3515.5) ?  0.26120035937640496 :   0.24825776718896625 :   (b('dg0_v_vv') <= 1.5840044021606445) ?  (b('l8b4_med') <= 537.5668334960938) ?  0.23577077451814488 :   0.21794672770767684 :   (b('ndvi') <= 2366.0) ?  0.2517952626018226 :   0.24482671090135133 :   (b('dg0_v_vv') <= 1.0754718780517578) ?  (b('dg0_v_vv') <= -0.04107856750488281) ?  (b('ndvi_med') <= 3562.5) ?  0.2325844251114115 :   0.20870436257464234 :   (b('ndvi') <= 3805.5) ?  0.17647288349233503 :   0.20011156235213678 :   (b('l8b4_med') <= 579.6047058105469) ?  (b('bulk') <= 112.0) ?  0.22446913466038032 :   0.2224989338353731 :   0.24187447461559153 :   (b('l8b5') <= 2405.21044921875) ?  (b('dg0_v_vv') <= -1.429443359375) ?  (b('ndvi') <= 3293.5) ?  (b('dg0_v_vv') <= -2.0139780044555664) ?  0.03219219533831372 :   0.055443491234990594 :   (b('ndvi') <= 3587.5) ?  (b('ndvi_med') <= 3405.0) ?  -0.021029487152995574 :   -0.00044832818006607794 :   (b('ndvi_med') <= 3494.75) ?  -0.023467664099924804 :   -0.015940315612851197 :   (b('l8b5') <= 2361.711669921875) ?  (b('ndvi') <= 2895.5) ?  0.23921527664869852 :   (b('ndvi') <= 3523.5) ?  0.10822614236005886 :   0.01475080893408312 :   -0.038928402492074604 :   (b('dg0_v_vv') <= -2.387979030609131) ?  (b('ndvi') <= 3449.0) ?  (b('dg0_v_vv') <= -2.549759864807129) ?  (b('ndvi') <= 2686.5) ?  0.17561038780616312 :   0.18875804434518023 :   (b('l8b5_med') <= 2670.971923828125) ?  0.13030920882791447 :   0.15193300431212103 :   -0.038522474100279036 :   (b('ndvi') <= 3265.5) ?  (b('l8b4') <= 636.6922607421875) ?  (b('dg0_v_vv') <= -0.3463625907897949) ?  -0.0054913311795964925 :   0.006374279238264169 :   (b('ndvi') <= 3130.0) ?  0.16599488748575109 :   0.09712448451724556 :   (b('l8b11_med') <= 1212.746337890625) ?  (b('ndvi') <= 4735.0) ?  0.18371369141325902 :   0.1518464721624932 :   (b('l8b5_med') <= 2974.984375) ?  0.17006630110657506 :   0.1511832107079782 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1735.2150268554688) ?  (b('l8b4_med') <= 688.1059875488281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 24.0) ?  (b('l8b4') <= 622.2225646972656) ?  (b('crops') <= 36.96536636352539) ?  (b('l8b4') <= 453.53761291503906) ?  (b('dg0_v_vv') <= 1.0288739204406738) ?  (b('l8b11_med') <= 1335.7188110351562) ?  (b('ndvi_med') <= 4094.5) ?  0.12198787947394804 :   0.138597064258848 :   0.16035377211407972 :   0.08489926350255758 :   (b('ndvi') <= 4831.0) ?  (b('dg0_v_vv') <= 0.10362434387207031) ?  (b('ndvi') <= 4296.0) ?  0.06910979637887027 :   0.027595112105597452 :   (b('l8b4_med') <= 622.3268127441406) ?  0.011082826701995147 :   -0.00037208380666482743 :   0.10459301539654894 :   (b('l8b11') <= 1157.3131713867188) ?  (b('l8b4_med') <= 561.1583557128906) ?  (b('ndvi') <= 3977.0) ?  (b('dg0_v_vv') <= -1.6712470054626465) ?  -0.00561634779748299 :   -3.4439158880197884e-05 :   (b('l8b11') <= 1062.044189453125) ?  0.0613416814461151 :   0.11477659012553434 :   (b('grass') <= 32.294264793395996) ?  (b('l8b5_med') <= 3617.75048828125) ?  0.240191629902516 :   0.15061371720964706 :   (b('dg0_v_vv') <= 0.8027663230895996) ?  0.09221734049662701 :   0.1692757233040369 :   (b('dg0_v_vv') <= -0.9400548934936523) ?  (b('ndvi') <= 3848.0) ?  0.04874657183185244 :   (b('l8b5') <= 4105.5416259765625) ?  0.13679980821687202 :   0.133486082377677 :   (b('grass') <= 42.213863372802734) ?  (b('ndvi') <= 2269.5) ?  0.1486045665048153 :   0.20662605955481314 :   (b('ndvi') <= 2668.5) ?  0.12723687048693902 :   0.15942278936897755 :   (b('ndvi') <= 3970.0) ?  (b('l8b5_med') <= 3503.7686767578125) ?  (b('dg0_v_vv') <= 1.4454069137573242) ?  (b('dg0_v_vv') <= 0.059375762939453125) ?  (b('ndvi_med') <= 4102.5) ?  0.07901708386183721 :   0.1135931392585344 :   (b('l8b11') <= 1447.2579345703125) ?  0.13420734657817865 :   0.13068266201400242 :   0.047148292667162195 :   (b('l8b5') <= 3732.5875244140625) ?  (b('l8b11') <= 1514.5681762695312) ?  0.13365601491792792 :   0.14486969737580152 :   (b('l8b5') <= 4427.926513671875) ?  0.11730136245449024 :   0.12476170965630493 :   (b('bulk') <= 143.5) ?  (b('dg0_v_vv') <= 0.07375192642211914) ?  (b('dg0_v_vv') <= -0.0024137496948242188) ?  0.0883109360878622 :   0.06822769313929664 :   -0.017924168983105587 :   (b('l8b5') <= 4384.23046875) ?  (b('dg0_v_vv') <= -0.34594106674194336) ?  (b('ndvi_med') <= 4276.0) ?  0.017496182145891312 :   -0.017421588602894233 :   (b('dg0_v_vv') <= 0.11186361312866211) ?  -0.01959347256512267 :   -0.03928665619856647 :   -0.06109046409253152 :   (b('ndvi_med') <= 4666.0) ?  (b('sand') <= 36.5) ?  (b('l8b4') <= 741.6185913085938) ?  (b('dg0_v_vv') <= 1.4752388000488281) ?  (b('l8b5') <= 4764.901611328125) ?  (b('ndvi') <= 4906.0) ?  0.03869209909400321 :   0.11600294654598327 :   (b('ndvi_med') <= 4364.5) ?  0.10517207626658985 :   0.20916914807652137 :   (b('l8b5') <= 2929.21533203125) ?  (b('dg0_v_vv') <= 2.3980941772460938) ?  0.058106404935822434 :   0.12018540687788064 :   (b('l8b11_med') <= 1273.6918334960938) ?  0.12903317978661444 :   0.1760819896160385 :   (b('l8b11_med') <= 1138.7337036132812) ?  (b('l8b11') <= 1394.990478515625) ?  (b('l8b4') <= 869.5136413574219) ?  -0.005938446708283516 :   0.05669512613681753 :   (b('l8b5') <= 2756.0557861328125) ?  -0.05647809969093466 :   -0.006910363475425091 :   (b('ndvi') <= 4268.0) ?  (b('l8b4') <= 908.4690551757812) ?  0.15329229816260181 :   0.06732801481707341 :   (b('l8b5_med') <= 3886.60986328125) ?  0.01817754209329256 :   0.030438489871274232 :   (b('crops') <= 40.08180618286133) ?  (b('ndvi') <= 4319.0) ?  (b('l8b5') <= 2464.9544677734375) ?  (b('l8b11_med') <= 1107.5009155273438) ?  0.0009666565724132654 :   -0.03755276724182718 :   (b('l8b11') <= 1128.01513671875) ?  0.005551339571295854 :   0.056906851933378196 :   (b('l8b4') <= 648.1369018554688) ?  (b('ndvi') <= 4944.0) ?  -0.015856971894806438 :   -0.0735240911318829 :   (b('dg0_v_vv') <= 1.1310234069824219) ?  -0.11998423330196586 :   -0.09602112817175223 :   (b('l8b5_med') <= 3841.7591552734375) ?  (b('l8b5') <= 2982.2440185546875) ?  (b('bulk') <= 125.5) ?  0.08614526073327873 :   0.04278612415351919 :   (b('crops') <= 47.465087890625) ?  0.030771970056951353 :   0.0033587089373322814 :   (b('grass') <= 31.54545497894287) ?  (b('l8b4') <= 858.407958984375) ?  0.059431868267883665 :   0.10704756711361313 :   (b('dg0_v_vv') <= -0.2522101402282715) ?  0.18218031151291436 :   0.21744047972997962 :   (b('l8b4') <= 417.38026428222656) ?  (b('dg0_v_vv') <= 0.26052141189575195) ?  0.16780615740096302 :   (b('dg0_v_vv') <= 0.7419066429138184) ?  0.23190824265018206 :   (b('ndvi_med') <= 4697.0) ?  0.20190824145808917 :   0.21443563496161777 :   (b('ndvi') <= 3256.0) ?  (b('dg0_v_vv') <= 0.40630531311035156) ?  0.21291808825037267 :   (b('l8b11_med') <= 1068.9169921875) ?  0.16592556838029993 :   (b('dg0_v_vv') <= 0.9026055335998535) ?  0.18783626570764256 :   0.18429356547341477 :   (b('l8b5') <= 3078.330078125) ?  (b('l8b11') <= 1319.9922790527344) ?  (b('dg0_v_vv') <= -0.883854866027832) ?  0.07381297111723868 :   0.13494145252713777 :   (b('dg0_v_vv') <= -1.6890454292297363) ?  0.20944984741539474 :   0.12829478716115456 :   (b('l8b4') <= 455.1209259033203) ?  (b('l8b5_med') <= 3413.9039306640625) ?  0.127262475749632 :   0.14030616276538108 :   (b('dg0_v_vv') <= 0.8198938369750977) ?  -0.013250972607674954 :   0.05399042954930353 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11') <= 935.7751159667969) ?  (b('l8b11_med') <= 990.374267578125) ?  (b('dg0_v_vv') <= -1.41066312789917) ?  (b('ndvi_med') <= 2083.75) ?  0.10164784704759644 :   0.06896647266128458 :   (b('ndvi') <= 2981.5) ?  (b('dg0_v_vv') <= -0.7527174949645996) ?  -0.06778058067977308 :   (b('l8b11') <= 809.0) ?  0.011670910869633816 :   0.04964786087587403 :   (b('dg0_v_vv') <= -0.0516810417175293) ?  (b('l8b11_med') <= 936.5092468261719) ?  0.01738684151503292 :   0.048918047032168566 :   (b('dg0_v_vv') <= 0.15077829360961914) ?  0.07566646686771311 :   0.07850160093476896 :   (b('l8b5') <= 1878.2954711914062) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('dg0_v_vv') <= 0.2931399345397949) ?  0.18988733596921425 :   (b('ndvi_med') <= 3533.5) ?  0.1770032285826666 :   0.18183234552860714 :   (b('l8b5') <= 1748.1115112304688) ?  (b('ndvi') <= 2886.0) ?  0.09250171969865169 :   0.061017336546068515 :   -0.06422710210441052 :   (b('bulk') <= 113.5) ?  (b('l8b4') <= 419.28321838378906) ?  0.0006848364011032837 :   0.05339003655505939 :   (b('grass') <= 54.38536262512207) ?  (b('ndvi') <= 4328.0) ?  0.15718361403229988 :   0.18511021437823066 :   (b('ndvi') <= 3225.0) ?  0.14086602791183214 :   0.1111075131344649 :   (b('sand') <= 32.0) ?  (b('ndvi') <= 2959.5) ?  (b('l8b5') <= 2412.263671875) ?  (b('l8b5_med') <= 2944.8780517578125) ?  (b('l8b5') <= 2145.9061279296875) ?  0.03923446537280259 :   0.11642569790791515 :   (b('l8b11_med') <= 1239.6309204101562) ?  -0.017199581713978257 :   0.00569983179747216 :   (b('l8b11') <= 1701.4208984375) ?  (b('ndvi') <= 2793.5) ?  0.2155028726791169 :   0.1440129712080039 :   0.04159644269104812 :   (b('l8b4') <= 811.9796447753906) ?  (b('dg0_v_vv') <= 2.8931326866149902) ?  (b('ndvi') <= 3059.0) ?  0.0038485197799979567 :   0.07349959714284873 :   0.20768695675565657 :   (b('dg0_v_vv') <= 0.562321662902832) ?  0.2157636600187779 :   0.14524950856176191 :   (b('l8b5') <= 2117.5440673828125) ?  (b('l8b11_med') <= 1180.7840576171875) ?  (b('l8b11') <= 1197.364990234375) ?  (b('l8b4') <= 558.9656677246094) ?  0.05549786389661819 :   0.030007410070813784 :   (b('l8b4') <= 667.0793762207031) ?  0.23441390148259825 :   0.11417535804849473 :   (b('crops') <= 6.950881481170654) ?  (b('ndvi') <= 2310.0) ?  -0.08897613713013636 :   -0.06500020712224762 :   0.041599404087918984 :   (b('ndvi_med') <= 3216.0) ?  (b('l8b4') <= 556.1734924316406) ?  (b('l8b5_med') <= 2542.5650634765625) ?  -0.02163667807565431 :   0.026621100023072775 :   (b('l8b11') <= 1313.9087524414062) ?  -0.07289851275368585 :   -0.04778635658621859 :   (b('ndvi') <= 3029.5) ?  (b('moss') <= 10.430890083312988) ?  0.03031395897117976 :   0.10844397769906139 :   (b('dg0_v_vv') <= -0.90130615234375) ?  -0.042489902600013014 :   0.018928060508734794 :   (b('l8b5') <= 2434.0947265625) ?  (b('dg0_v_vv') <= -1.8737859725952148) ?  (b('l8b4') <= 477.61962890625) ?  0.1364142666908158 :   (b('ndvi') <= 3351.5) ?  (b('l8b5') <= 2405.21044921875) ?  (b('ndvi_med') <= 3438.5) ?  0.028972975804482354 :   0.049899142111491546 :   0.10473750060757353 :   (b('l8b4') <= 630.7738952636719) ?  (b('ndvi') <= 3587.5) ?  -0.03460982511343297 :   -0.02397699217829288 :   (b('l8b11') <= 1245.7349243164062) ?  -0.0053689684133521876 :   0.00916797382273482 :   (b('ndvi') <= 3739.0) ?  (b('l8b11') <= 1403.2102661132812) ?  (b('l8b4') <= 641.6177062988281) ?  (b('dg0_v_vv') <= -1.3131413459777832) ?  -0.004942198061636843 :   -0.035035562242867135 :   (b('ndvi') <= 3523.5) ?  0.09524537594870784 :   0.0036315976490832367 :   (b('ndvi_med') <= 3494.75) ?  (b('dg0_v_vv') <= -1.5166988372802734) ?  0.17668762256746096 :   0.21230698028161615 :   (b('dg0_v_vv') <= -0.19624805450439453) ?  0.10496587600691515 :   0.14845609027314477 :   (b('dg0_v_vv') <= 0.7927160263061523) ?  (b('l8b4_med') <= 481.6563720703125) ?  (b('dg0_v_vv') <= -0.08648824691772461) ?  0.13725853411360828 :   0.23939264490353296 :   (b('dg0_v_vv') <= -0.5416131019592285) ?  0.17149525016252554 :   0.14609456888605393 :   (b('dg0_v_vv') <= 1.2780194282531738) ?  (b('dg0_v_vv') <= 1.1388568878173828) ?  0.08725582396466558 :   0.06457843929310558 :   0.17195581232983892 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b5') <= 4120.966552734375) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('moss') <= 0.9910394549369812) ?  (b('dg0_v_vv') <= 0.9595227241516113) ?  0.151543862561544 :   0.17683147442484537 :   (b('dg0_v_vv') <= 1.950265884399414) ?  0.18235683511966072 :   0.2066056277673113 :   (b('l8b4') <= 491.7077331542969) ?  (b('moss') <= 0.7690265774726868) ?  0.1952586251459752 :   0.23584761629190557 :   (b('l8b5_med') <= 2592.1824951171875) ?  0.16012030191294868 :   0.19109425782648182 :   -0.03562882172454712 :   (b('dg0_v_vv') <= 0.3190345764160156) ?  (b('l8b4_med') <= 636.951416015625) ?  (b('grass') <= 63.13859176635742) ?  (b('bulk') <= 117.5) ?  0.14500496094006785 :   0.16889171854091892 :   (b('dg0_v_vv') <= -0.5621681213378906) ?  0.12069883055871655 :   0.12720097445022968 :   (b('dg0_v_vv') <= -1.03255033493042) ?  (b('l8b11') <= 1201.6112670898438) ?  0.13103935081264922 :   0.05600350018137497 :   (b('grass') <= 54.28562355041504) ?  0.14109442295112037 :   0.04220740556253014 :   (b('ndvi') <= 4542.5) ?  (b('l8b4_med') <= 584.9970397949219) ?  (b('ndvi') <= 4182.5) ?  0.14171508512833178 :   0.1645206201078056 :   (b('ndvi_med') <= 3494.75) ?  0.1861046883271859 :   0.16420734606028559 :   0.05244311109554939 :   (b('bulk') <= 137.5) ?  (b('crops') <= 65.88730239868164) ?  (b('ndvi_med') <= 3431.5) ?  (b('dg0_v_vv') <= 1.408236026763916) ?  (b('l8b5') <= 3584.6781005859375) ?  (b('bulk') <= 112.5) ?  (b('l8b5') <= 2548.525390625) ?  (b('ndvi') <= 1788.5) ?  0.06720656912910526 :   -0.029215407827605795 :   (b('l8b11') <= 1345.26220703125) ?  -0.08300011939677 :   -0.029381039757958725 :   (b('l8b5_med') <= 1639.760498046875) ?  (b('l8b5') <= 1751.51025390625) ?  -0.0007342639163917299 :   0.17683976008000943 :   (b('l8b4') <= 437.4970397949219) ?  0.17250674000473523 :   -0.023448941196024347 :   (b('l8b11') <= 1446.93408203125) ?  (b('l8b11') <= 1225.1600952148438) ?  (b('moss') <= 6.923041105270386) ?  0.04392317641369356 :   -0.0010670748821415472 :   (b('l8b4_med') <= 709.5168762207031) ?  -0.0872903557854765 :   -0.012450568055011251 :   (b('l8b4_med') <= 770.6791687011719) ?  (b('l8b5_med') <= 2962.1964111328125) ?  0.03979226221986557 :   0.06500362821255888 :   (b('l8b5') <= 3814.8470458984375) ?  -0.004850958621092814 :   0.02398822590592565 :   (b('ndvi') <= 1317.0) ?  (b('dg0_v_vv') <= 1.4776527881622314) ?  -0.022487336571861916 :   (b('grass') <= 48.80607795715332) ?  (b('dg0_v_vv') <= 1.5710058212280273) ?  0.11972583362806163 :   0.05076970833516489 :   (b('l8b4') <= 959.3878173828125) ?  0.09255808557307005 :   0.13494896366339354 :   (b('l8b5_med') <= 2812.8599853515625) ?  (b('bulk') <= 113.0) ?  (b('dg0_v_vv') <= 2.939746618270874) ?  -0.05847358174265752 :   -0.0021264327766567317 :   (b('dg0_v_vv') <= 3.2697534561157227) ?  0.014779458627138267 :   0.06610110938628071 :   (b('sand') <= 36.5) ?  (b('dg0_v_vv') <= 3.757176160812378) ?  -0.0033685246473431707 :   0.13324054331276788 :   (b('ndvi') <= 4256.0) ?  -0.01707230532437752 :   -0.06924446810419903 :   (b('l8b5') <= 2580.7315673828125) ?  (b('l8b5_med') <= 2239.5936279296875) ?  (b('l8b4') <= 784.6617736816406) ?  (b('l8b11') <= 1195.8831481933594) ?  0.09926013003637879 :   (b('crops') <= 22.95339584350586) ?  0.059863845833335705 :   0.026338080529223123 :   (b('dg0_v_vv') <= -0.14313077926635742) ?  (b('dg0_v_vv') <= -0.48138999938964844) ?  0.11819449621314107 :   0.1085304569849597 :   (b('l8b4_med') <= 955.3427429199219) ?  0.1350615287928812 :   0.12026881444364586 :   (b('l8b11') <= 1315.856689453125) ?  (b('ndvi') <= 1101.5) ?  -0.0758128490244123 :   (b('ndvi') <= 4498.5) ?  0.05727565176421792 :   0.10400665199349637 :   (b('moss') <= 7.961389780044556) ?  (b('l8b11') <= 1652.7327880859375) ?  0.025778498966571358 :   0.12717904667436483 :   (b('l8b11') <= 1517.4390258789062) ?  -0.05016310427110086 :   0.0033965876998336836 :   (b('sand') <= 57.0) ?  (b('l8b11') <= 808.5489807128906) ?  (b('dg0_v_vv') <= 0.3725457191467285) ?  0.22294456893015335 :   0.16125037965396216 :   (b('l8b4') <= 909.6919555664062) ?  (b('dg0_v_vv') <= 1.5264387130737305) ?  0.00907344124626856 :   0.06804766517975412 :   (b('moss') <= 3.969032496213913) ?  0.07727110388425218 :   -0.03580589597499702 :   (b('l8b5_med') <= 2810.554443359375) ?  (b('l8b5') <= 3444.8939208984375) ?  (b('dg0_v_vv') <= -0.7260298728942871) ?  -0.05255125389075354 :   -0.0779960111080685 :   (b('dg0_v_vv') <= -1.804603099822998) ?  0.010764745389452872 :   -0.039739471991190545 :   (b('ndvi') <= 3756.5) ?  (b('dg0_v_vv') <= -1.4006781578063965) ?  -0.02614319819750871 :   0.020186372831555752 :   (b('ndvi') <= 4041.0) ?  -0.04196428031765588 :   -0.01597160120485973 :   (b('ndvi_med') <= 2598.0) ?  (b('ndvi_med') <= 1552.75) ?  (b('l8b5') <= 2240.047607421875) ?  (b('l8b11') <= 1504.9881591796875) ?  0.16682058773587272 :   0.17106615377676548 :   0.23093972183183747 :   (b('l8b5') <= 4386.0) ?  (b('sand') <= 41.0) ?  (b('ndvi_med') <= 1945.0) ?  (b('l8b4') <= 624.7539672851562) ?  0.1508804740274348 :   0.12270086980721512 :   (b('l8b5') <= 2313.91259765625) ?  0.12938919250231828 :   0.10603454182837498 :   0.18552956387735706 :   (b('dg0_v_vv') <= 3.330624580383301) ?  (b('l8b5_med') <= 1603.275146484375) ?  0.03209809589130119 :   0.007010538129962246 :   0.16346111564133686 :   (b('sand') <= 26.5) ?  (b('l8b4') <= 305.94541931152344) ?  (b('ndvi') <= 3342.0) ?  0.06005425326218977 :   (b('l8b4_med') <= 813.8113098144531) ?  0.1475631868773365 :   (b('l8b11_med') <= 1257.730224609375) ?  0.11860557160813831 :   0.11283659151364994 :   (b('dg0_v_vv') <= -1.4527249336242676) ?  (b('l8b4') <= 928.6112365722656) ?  (b('l8b11_med') <= 1220.2855834960938) ?  0.09782620009280085 :   -0.017795697568106793 :   (b('ndvi') <= 3108.5) ?  -0.03944698999426353 :   -0.06012178384195108 :   (b('dg0_v_vv') <= 2.1668331623077393) ?  (b('l8b11_med') <= 1295.4417114257812) ?  -0.025029823152035398 :   0.01448266046291164 :   (b('l8b4') <= 908.4755554199219) ?  0.022692624171754466 :   0.1140859444325608 :   (b('ndvi') <= 2381.5) ?  (b('grass') <= 0.7789046764373779) ?  0.17561862975486323 :   (b('l8b4_med') <= 955.183837890625) ?  (b('dg0_v_vv') <= -0.11034679412841797) ?  0.09570753523273863 :   0.10760446656228422 :   (b('l8b5_med') <= 3490.31982421875) ?  0.12330654849240255 :   0.11364348746497954 :   (b('l8b5') <= 2919.025146484375) ?  (b('moss') <= 0.06590257585048676) ?  (b('dg0_v_vv') <= 1.6418936252593994) ?  0.11507711853065401 :   0.13421742649793028 :   (b('l8b11') <= 1257.3109741210938) ?  0.03248864437501536 :   0.06996483428695385 :   (b('l8b5') <= 4595.557861328125) ?  (b('l8b11') <= 1388.8997802734375) ?  0.026955546130344864 :   -0.0014328083165267116 :   (b('ndvi') <= 4456.5) ?  0.06742444404480846 :   0.00787441984291179 :   (b('sand') <= 28.0) ?  (b('moss') <= 5.450023412704468) ?  (b('l8b5') <= 2411.615478515625) ?  (b('dg0_v_vv') <= -0.7852997779846191) ?  (b('ndvi') <= 914.5) ?  (b('l8b5') <= 1708.9805908203125) ?  -0.015404563098266294 :   -0.016325674122165373 :   (b('bulk') <= 142.0) ?  0.04036890980913796 :   (b('ndvi_med') <= 3422.5) ?  0.1034116426279931 :   0.08191566496762942 :   (b('l8b11_med') <= 2090.8768310546875) ?  (b('l8b5') <= 1942.27978515625) ?  (b('bulk') <= 146.5) ?  0.12401345378636795 :   0.09695158511539295 :   (b('ndvi_med') <= 3869.0) ?  0.17030178778206018 :   0.07110620062491577 :   (b('ndvi') <= 2445.5) ?  (b('dg0_v_vv') <= 1.2060365676879883) ?  0.07566328755099119 :   -0.024252066509779055 :   (b('dg0_v_vv') <= 2.307697296142578) ?  0.0830620144234871 :   0.12089373664188226 :   (b('l8b5') <= 2529.4814453125) ?  (b('l8b4') <= 875.0882873535156) ?  (b('dg0_v_vv') <= -1.2870864868164062) ?  (b('sand') <= 17.5) ?  -0.02354510985922348 :   -0.006922284290926228 :   (b('ndvi') <= 3315.0) ?  -0.007253868360436122 :   0.010442822307384636 :   (b('ndvi') <= 4436.5) ?  (b('l8b11_med') <= 1493.6405029296875) ?  0.04399615061186593 :   0.031723626021903345 :   (b('l8b11_med') <= 1512.6978759765625) ?  0.12343526061884391 :   0.033704755323737234 :   (b('l8b5') <= 5284.332275390625) ?  (b('l8b11') <= 1722.6476440429688) ?  (b('ndvi') <= 4903.0) ?  0.08903673376520004 :   0.050050921492338606 :   (b('ndvi_med') <= 3803.0) ?  -0.01852156069964908 :   -0.07472271628793647 :   (b('l8b5_med') <= 2852.2147216796875) ?  (b('dg0_v_vv') <= 0.9791553020477295) ?  -0.023085787389684614 :   -0.00927069070773448 :   (b('dg0_v_vv') <= 1.6867907047271729) ?  0.006333696969443692 :   0.021573690305875155 :   (b('l8b4') <= 786.5494995117188) ?  (b('ndvi') <= 3714.0) ?  (b('l8b4') <= 749.1124877929688) ?  (b('ndvi') <= 2899.5) ?  (b('l8b5') <= 2431.8848876953125) ?  0.002333986719775453 :   -0.0375407298229208 :   (b('ndvi') <= 2994.0) ?  0.14158740403278952 :   0.03075942506726508 :   (b('dg0_v_vv') <= 0.6866369247436523) ?  (b('ndvi') <= 1935.0) ?  0.05307098534576815 :   0.09823494593459806 :   (b('l8b5') <= 2012.7752685546875) ?  0.1941503959581588 :   0.14703153445801015 :   (b('dg0_v_vv') <= -0.12453746795654297) ?  (b('dg0_v_vv') <= -0.6160588264465332) ?  (b('ndvi') <= 4740.5) ?  0.0720176894158277 :   0.0026279035562030983 :   (b('dg0_v_vv') <= -0.4734187126159668) ?  -0.04219223346290109 :   0.030406514251497832 :   (b('l8b11') <= 1134.0049438476562) ?  (b('l8b5_med') <= 2663.0615234375) ?  0.03641584208408072 :   0.11230038581539308 :   (b('ndvi') <= 4063.5) ?  0.11858251290332844 :   0.20352837880328856 :   (b('l8b11') <= 1667.943115234375) ?  (b('l8b4') <= 919.2506103515625) ?  (b('ndvi') <= 4533.0) ?  (b('l8b4') <= 908.2957458496094) ?  -0.014987554235923154 :   0.03035990955032236 :   (b('ndvi_med') <= 3554.25) ?  -0.00404170806129759 :   0.1481912414030521 :   (b('l8b11') <= 1534.6617431640625) ?  (b('l8b4_med') <= 940.2608337402344) ?  -0.03765055347807274 :   0.012323154515789712 :   (b('crops') <= 23.92689323425293) ?  -0.04873968124726592 :   -0.008525656455546454 :   (b('l8b11') <= 1670.1726684570312) ?  (b('dg0_v_vv') <= -0.3435344696044922) ?  0.0432174834063995 :   (b('l8b11_med') <= 1555.286865234375) ?  0.18767897704069197 :   0.2061459860516175 :   (b('ndvi') <= 3975.0) ?  (b('l8b11') <= 1695.6295166015625) ?  0.06689598474864739 :   0.00342030317171671 :   0.15617154812773887 :   (b('moss') <= 12.857846736907959) ?  (b('dg0_v_vv') <= 0.05479907989501953) ?  (b('grass') <= 64.03558540344238) ?  (b('grass') <= 57.61799430847168) ?  (b('l8b4') <= 1385.1483154296875) ?  (b('l8b5') <= 2064.8687744140625) ?  0.03618952865212486 :   0.009105129703566242 :   (b('dg0_v_vv') <= -1.5378198623657227) ?  0.18289850767872803 :   0.06075727357172831 :   (b('ndvi') <= 1577.0) ?  (b('dg0_v_vv') <= -0.18686723709106445) ?  -0.13503848073031732 :   -0.09034141656266644 :   (b('dg0_v_vv') <= -3.316023826599121) ?  -0.06556085325143286 :   -0.032407778899269594 :   (b('ndvi_med') <= 2115.0) ?  (b('moss') <= 10.696201801300049) ?  (b('l8b4') <= 759.1402282714844) ?  0.09093944233857794 :   0.02816675451099545 :   (b('bulk') <= 145.0) ?  0.1333806089238303 :   0.23493801510828813 :   (b('sand') <= 44.5) ?  (b('dg0_v_vv') <= -1.482609748840332) ?  -0.031111940234777567 :   0.02008907046412597 :   (b('l8b11') <= 1658.3245849609375) ?  -0.07575914596147945 :   -0.038269088594043495 :   (b('l8b11_med') <= 2749.34619140625) ?  (b('ndvi_med') <= 2076.5) ?  (b('l8b5_med') <= 2499.1395263671875) ?  (b('ndvi') <= 2018.5) ?  -0.028958168879007113 :   0.058813747730733605 :   (b('dg0_v_vv') <= 2.0022871494293213) ?  0.07437430195118068 :   0.12445863247652311 :   (b('sand') <= 46.0) ?  (b('bulk') <= 160.5) ?  0.04516145272857265 :   0.12506800184879271 :   (b('moss') <= 1.4057907313108444) ?  -0.022332629637692576 :   0.019230247520180394 :   (b('moss') <= 11.310256481170654) ?  (b('ndvi_med') <= 1394.5) ?  (b('l8b5') <= 2141.7821044921875) ?  0.05913321470049251 :   0.01070959510041023 :   (b('l8b5') <= 1934.3297729492188) ?  -0.0849731863366775 :   -0.010936573193590548 :   (b('ndvi_med') <= 1472.0) ?  0.10064122494760311 :   (b('dg0_v_vv') <= 1.8223285675048828) ?  0.030577223260941033 :   0.06392786167517436 :   (b('moss') <= 24.19915199279785) ?  (b('moss') <= 21.619216918945312) ?  (b('bulk') <= 156.5) ?  (b('l8b11') <= 1517.71044921875) ?  (b('moss') <= 20.837696075439453) ?  0.024436612720486275 :   0.11581657942848504 :   (b('l8b11_med') <= 2333.6634521484375) ?  -0.0011474930631940065 :   -0.09820970344186819 :   (b('l8b5') <= 2208.591552734375) ?  (b('l8b5') <= 1853.8900756835938) ?  -0.03760255516880916 :   0.05026671415199794 :   (b('l8b5_med') <= 2387.0047607421875) ?  -0.05545986945240691 :   -0.09404114917169598 :   (b('l8b4') <= 1014.5798034667969) ?  (b('dg0_v_vv') <= 0.7775285243988037) ?  (b('sand') <= 48.0) ?  0.27834258064660966 :   0.185995587776541 :   (b('dg0_v_vv') <= 3.1401472091674805) ?  0.1336247424760051 :   0.08588478701560216 :   (b('l8b5') <= 2251.9012451171875) ?  (b('l8b5') <= 2145.240478515625) ?  -0.062310120842684974 :   -0.006070306307644657 :   0.061596797278087284 :   (b('l8b11_med') <= 2235.078125) ?  (b('dg0_v_vv') <= 0.41887855529785156) ?  (b('l8b11') <= 1686.253662109375) ?  (b('l8b4_med') <= 963.2676696777344) ?  -0.047264287456123204 :   -0.0266098137089405 :   (b('l8b11') <= 1689.2788696289062) ?  0.08187643799598188 :   0.0007881612776044935 :   (b('sand') <= 35.5) ?  (b('l8b4') <= 712.2126770019531) ?  0.008718644664479244 :   -0.031660754337369576 :   (b('ndvi') <= 2461.0) ?  -0.0009960111672325256 :   0.05106112688393607 :   (b('l8b5') <= 2168.28466796875) ?  (b('l8b11_med') <= 2298.01025390625) ?  (b('dg0_v_vv') <= -0.3997035026550293) ?  -0.04745889567126668 :   -0.017618725886564518 :   (b('ndvi_med') <= 2577.0) ?  0.052148594702198466 :   0.007131014858848761 :   (b('l8b4') <= 661.4141235351562) ?  (b('sand') <= 38.0) ?  0.08908643558889706 :   0.22647486019517438 :   (b('bulk') <= 152.5) ?  0.10064094938370383 :   0.06404000177144648 :   (b('l8b11_med') <= 2649.172119140625) ?  (b('crops') <= 58.930376052856445) ?  (b('dg0_v_vv') <= 1.108428955078125) ?  (b('ndvi_med') <= 2722.75) ?  (b('grass') <= 90.26663208007812) ?  (b('sand') <= 36.5) ?  (b('crops') <= 34.1732873916626) ?  (b('l8b4_med') <= 1233.99462890625) ?  (b('l8b11') <= 1915.749267578125) ?  -0.03198915772124045 :   -0.05412935988544273 :   (b('dg0_v_vv') <= 0.9091026782989502) ?  -0.007393613091764382 :   0.1523522201091967 :   (b('l8b4') <= 1282.3992309570312) ?  (b('l8b11_med') <= 1839.635009765625) ?  -0.10007008123832443 :   -0.08606470032203171 :   (b('l8b5') <= 2524.6605224609375) ?  -0.07894834630919986 :   -0.08846297494728997 :   (b('grass') <= 41.722944259643555) ?  (b('bulk') <= 146.5) ?  (b('moss') <= 23.168834686279297) ?  -0.02364406519781339 :   -0.05525797966868827 :   (b('l8b5') <= 2888.3411865234375) ?  -0.06990910637391418 :   -0.03725104925466254 :   (b('crops') <= 0.36066467873752117) ?  (b('l8b4_med') <= 1366.960205078125) ?  -0.03609035986391004 :   -0.017888476111888606 :   (b('sand') <= 46.0) ?  0.033093621898113754 :   -0.009760029146454095 :   (b('bulk') <= 145.5) ?  (b('sand') <= 30.5) ?  (b('l8b4') <= 1161.9364013671875) ?  (b('l8b11') <= 2262.390869140625) ?  0.016027026870392733 :   0.07419105704104328 :   (b('ndvi') <= 1564.0) ?  -0.03538132185025568 :   -0.016877217912946715 :   (b('bulk') <= 141.5) ?  (b('ndvi') <= 1768.5) ?  -0.029791329084834096 :   0.01038224224246837 :   (b('l8b11') <= 1813.1629028320312) ?  -0.06641799723617196 :   -0.03591891630788984 :   (b('l8b5_med') <= 2662.191650390625) ?  (b('dg0_v_vv') <= 0.4199104309082031) ?  (b('sand') <= 51.5) ?  0.011924804729939224 :   -0.03128351908967921 :   (b('l8b5') <= 2207.7777099609375) ?  0.1260734357443671 :   0.026383749797306812 :   (b('l8b5') <= 2200.9254150390625) ?  (b('l8b4_med') <= 1241.2111206054688) ?  0.12252931512281055 :   0.1670974717397708 :   (b('dg0_v_vv') <= 0.6125521659851074) ?  0.055651719550886304 :   0.11691701129277451 :   (b('sand') <= 24.5) ?  (b('dg0_v_vv') <= -1.4583592414855957) ?  (b('l8b11') <= 2126.08251953125) ?  (b('l8b5') <= 1997.84375) ?  0.12258242397141234 :   (b('ndvi') <= 3954.0) ?  -0.013847536272948699 :   -0.06273390907884971 :   (b('l8b4') <= 1350.6481323242188) ?  (b('l8b11_med') <= 1554.8241577148438) ?  0.10376795873970149 :   0.12240595097853421 :   0.03080596473576447 :   (b('l8b4_med') <= 725.9989624023438) ?  (b('dg0_v_vv') <= 0.19002628326416016) ?  (b('ndvi') <= 3936.5) ?  0.10656326211052196 :   0.17330571609654413 :   (b('l8b5_med') <= 3595.3917236328125) ?  0.15129330853400166 :   0.21871263646167555 :   (b('ndvi_med') <= 3672.75) ?  (b('l8b11') <= 2053.037353515625) ?  0.09670067480879951 :   0.14352400936442863 :   (b('ndvi') <= 3235.5) ?  -0.08008586166623313 :   0.05571165265306418 :   (b('grass') <= 32.961503982543945) ?  (b('bulk') <= 120.0) ?  (b('l8b5') <= 3224.37451171875) ?  (b('l8b4') <= 1450.0838012695312) ?  0.12302177683119153 :   0.12637034693798888 :   0.18217365556837845 :   (b('l8b5_med') <= 2781.259521484375) ?  (b('crops') <= 37.226152420043945) ?  0.13727874965048445 :   0.01636333088618554 :   (b('l8b11') <= 1736.593017578125) ?  -0.0850429986172405 :   -0.019022849634611507 :   (b('moss') <= 23.950027465820312) ?  (b('l8b11_med') <= 1626.2047119140625) ?  (b('sand') <= 45.5) ?  0.02357433539522745 :   0.08336470988785122 :   (b('ndvi') <= 1919.5) ?  0.09106132636197854 :   -0.02679120093855507 :   (b('l8b4') <= 1030.9885864257812) ?  (b('dg0_v_vv') <= -0.024332523345947266) ?  0.1295773731931955 :   0.22515571519091257 :   (b('dg0_v_vv') <= 0.6176028251647949) ?  -0.010615963164042643 :   0.10674881670616676 :   (b('sand') <= 24.5) ?  (b('grass') <= 69.29146575927734) ?  (b('grass') <= 37.87437438964844) ?  (b('l8b4') <= 1000.2957458496094) ?  (b('l8b4') <= 922.3747253417969) ?  0.12543552105543396 :   0.12913457311381582 :   (b('ndvi') <= 3036.5) ?  0.1139188433207407 :   0.10116763162692863 :   (b('l8b11_med') <= 1563.6657104492188) ?  (b('l8b11') <= 1797.3060302734375) ?  (b('moss') <= 3.2715134620666504) ?  0.18182755783866722 :   0.1888088714851619 :   (b('dg0_v_vv') <= 1.2005271911621094) ?  0.19026089708293237 :   0.19090591091229614 :   (b('l8b4') <= 978.7127685546875) ?  0.1511351052154789 :   0.16191780462137295 :   (b('l8b4') <= 952.8914489746094) ?  (b('l8b11_med') <= 1645.0632934570312) ?  0.028396930826449118 :   (b('l8b11_med') <= 1970.0264282226562) ?  (b('ndvi_med') <= 2376.0) ?  0.1350847544348675 :   0.14140896302774686 :   0.09438011095499009 :   (b('l8b11_med') <= 1971.0542602539062) ?  (b('l8b4') <= 1065.8452758789062) ?  -0.06593753284303078 :   -0.04593463983073695 :   0.0323486231606597 :   (b('grass') <= 78.29829025268555) ?  (b('ndvi_med') <= 996.5) ?  (b('l8b4') <= 1507.215087890625) ?  (b('dg0_v_vv') <= 1.8538198471069336) ?  (b('dg0_v_vv') <= 1.6054177284240723) ?  0.09486058161968443 :   0.025382875283525036 :   (b('ndvi_med') <= 956.5) ?  0.1728328578509158 :   0.21126372250972128 :   (b('l8b4_med') <= 2046.2477416992188) ?  (b('moss') <= 7.032404899597168) ?  0.07936554650036465 :   -0.0048293950624547725 :   (b('l8b5') <= 2776.7088623046875) ?  -0.05408001852960062 :   -0.07336054358591339 :   (b('moss') <= 26.56121253967285) ?  (b('l8b5') <= 2470.179931640625) ?  (b('l8b5') <= 2447.85400390625) ?  0.02449870285990759 :   0.1801231168621623 :   (b('l8b5_med') <= 2982.3580322265625) ?  -0.01654052508113413 :   0.021201152696617903 :   (b('l8b11_med') <= 2411.768798828125) ?  (b('l8b4_med') <= 1071.31494140625) ?  -0.052004673488955044 :   -0.01765744983017333 :   (b('l8b11') <= 2385.7490234375) ?  0.104266316013823 :   0.16323048776387578 :   (b('sand') <= 26.0) ?  (b('l8b4') <= 1213.0125122070312) ?  (b('l8b11_med') <= 2118.291015625) ?  (b('l8b11') <= 2188.0157470703125) ?  0.018338768567569566 :   0.062424773782246396 :   -0.04585063845725529 :   (b('dg0_v_vv') <= 3.1834635734558105) ?  (b('ndvi_med') <= 2225.5) ?  -0.08281885250228559 :   -0.08549155604986677 :   -0.0506646076739741 :   (b('dg0_v_vv') <= 2.1524605751037598) ?  (b('bulk') <= 162.5) ?  (b('l8b5_med') <= 2662.191650390625) ?  0.03015253261395856 :   0.10482407892210134 :   (b('ndvi') <= 1099.5) ?  0.10909793652767513 :   -0.050504928283877565 :   (b('sand') <= 49.5) ?  (b('ndvi') <= 1378.0) ?  0.11917646532480901 :   0.06326710054630978 :   (b('l8b11_med') <= 1786.3392333984375) ?  0.028500230002242517 :   -0.06656574676401965 :   (b('l8b5_med') <= 3602.328857421875) ?  (b('l8b5_med') <= 3030.7413330078125) ?  (b('bulk') <= 129.0) ?  (b('l8b4') <= 984.677978515625) ?  (b('l8b11') <= 1994.1546020507812) ?  (b('l8b5') <= 2857.278076171875) ?  (b('ndvi_med') <= 2239.5) ?  0.08976601001101964 :   0.05401567569383495 :   0.16955985141542587 :   (b('l8b11') <= 2095.6897583007812) ?  (b('dg0_v_vv') <= -0.14661550521850586) ?  0.037211376395515354 :   0.018211377627344677 :   -0.042337451695474315 :   (b('l8b4') <= 1537.769775390625) ?  (b('ndvi_med') <= 1383.5) ?  (b('ndvi') <= 1170.5) ?  0.17318441861347159 :   0.17905454741861282 :   (b('ndvi') <= 2812.0) ?  0.1553036685874623 :   0.13476331167857045 :   0.08321156187339018 :   (b('dg0_v_vv') <= 0.3862314224243164) ?  (b('l8b5') <= 3054.810791015625) ?  (b('moss') <= 0.9145320579409599) ?  (b('bulk') <= 133.0) ?  0.11557590102114727 :   0.005247908146684182 :   (b('moss') <= 2.9057151079177856) ?  -0.04011696161060078 :   -0.007183878349932703 :   (b('l8b4') <= 2302.7088623046875) ?  (b('ndvi') <= 3686.0) ?  0.0065771993407969295 :   0.04569900756141303 :   (b('l8b4') <= 2351.68359375) ?  0.09038740752692631 :   -0.07311974065917118 :   (b('crops') <= 98.52266693115234) ?  (b('ndvi_med') <= 3697.5) ?  (b('ndvi_med') <= 2152.0) ?  0.03752521619724918 :   0.004188925886733602 :   (b('l8b4') <= 1706.4485473632812) ?  0.07120235535231001 :   0.14552268389067072 :   (b('dg0_v_vv') <= 2.4773271083831787) ?  (b('ndvi') <= 2949.0) ?  0.0715387810027979 :   0.10647059277468114 :   (b('l8b4_med') <= 1303.0054931640625) ?  0.10716708165370269 :   0.10231892459120101 :   (b('dg0_v_vv') <= -0.4060969352722168) ?  (b('ndvi_med') <= 3819.0) ?  (b('ndvi_med') <= 2416.25) ?  (b('l8b11') <= 1972.5216064453125) ?  (b('ndvi') <= 1066.0) ?  0.07503499971032723 :   -0.03496178723012047 :   (b('dg0_v_vv') <= -1.2147579193115234) ?  0.008814196112106404 :   0.03467151138017303 :   (b('l8b4') <= 1550.9585571289062) ?  (b('l8b5') <= 3545.6173095703125) ?  0.04472210928870693 :   -0.031126209400915587 :   (b('l8b4_med') <= 767.7357177734375) ?  -0.04160694613280638 :   0.1296375777314239 :   (b('grass') <= 20.025272369384766) ?  (b('l8b4_med') <= 630.9979553222656) ?  (b('l8b5_med') <= 3356.888427734375) ?  -0.04260694070878371 :   -0.030554297571739233 :   -0.015475105722109028 :   (b('l8b5_med') <= 3452.132568359375) ?  -0.08941757927040314 :   (b('dg0_v_vv') <= -3.4179587364196777) ?  -0.05692994703234652 :   -0.06520494443060379 :   (b('grass') <= 11.703174591064453) ?  (b('ndvi') <= 4925.0) ?  (b('dg0_v_vv') <= 2.170043706893921) ?  (b('l8b5_med') <= 3353.6900634765625) ?  0.008943464149409639 :   0.04619057160470975 :   (b('l8b5') <= 2929.2213134765625) ?  0.03168349021388433 :   0.10074535380811311 :   (b('l8b5') <= 3023.139404296875) ?  0.06444505126663772 :   0.30775320342951556 :   (b('ndvi') <= 1147.5) ?  (b('l8b11') <= 1987.6410522460938) ?  (b('ndvi') <= 886.5) ?  0.06869040312511372 :   0.12818340526009575 :   (b('l8b11') <= 2807.1087646484375) ?  -0.0038012000376268923 :   0.019700659624755684 :   (b('ndvi') <= 4739.5) ?  (b('l8b11') <= 1890.2849731445312) ?  0.04317858938549905 :   0.08814415102197347 :   (b('moss') <= 1.7780540585517883) ?  -0.037584526493249046 :   -0.005410841745962583 :   (b('ndvi_med') <= 3208.75) ?  (b('l8b5') <= 3142.757080078125) ?  (b('dg0_v_vv') <= -0.8383293151855469) ?  (b('l8b5_med') <= 3694.775634765625) ?  (b('dg0_v_vv') <= -4.238973617553711) ?  -0.015270704362826987 :   (b('l8b5') <= 2793.2647705078125) ?  -0.05991729756735943 :   -0.08621682935807407 :   (b('l8b5') <= 2582.291015625) ?  (b('l8b5') <= 2449.0430908203125) ?  -0.07408686203689059 :   -0.05343167124631775 :   (b('l8b5') <= 2867.196533203125) ?  0.0027347758448316058 :   -0.03657076533732168 :   (b('ndvi') <= 1530.0) ?  (b('l8b11') <= 3266.9671630859375) ?  -0.04676895190702217 :   (b('dg0_v_vv') <= 0.8486013412475586) ?  -0.07642325235148238 :   -0.08770476382598079 :   (b('dg0_v_vv') <= 8.23637056350708) ?  (b('dg0_v_vv') <= 3.3455636501312256) ?  0.01711782992741153 :   0.0800041922819001 :   -0.06860772081752389 :   (b('dg0_v_vv') <= 4.72709059715271) ?  (b('l8b11') <= 3078.45849609375) ?  (b('l8b11') <= 2265.3204345703125) ?  (b('l8b5') <= 4052.5994873046875) ?  -0.033252174026446354 :   -0.06602737082721966 :   (b('l8b5') <= 3362.3563232421875) ?  -0.0920754403404679 :   -0.07369026388106278 :   (b('dg0_v_vv') <= -0.5080380439758301) ?  (b('ndvi') <= 1790.5) ?  0.0026021772405559404 :   -0.07483641182528282 :   (b('ndvi') <= 1790.5) ?  0.0504013712426168 :   0.0051839646355480535 :   (b('l8b5') <= 4009.6563720703125) ?  0.052181194421718924 :   (b('ndvi') <= 4070.5) ?  0.005797614818914526 :   0.048435160176584086 :   (b('l8b4_med') <= 560.0) ?  (b('l8b11_med') <= 1010.9424743652344) ?  (b('dg0_v_vv') <= 1.6726787090301514) ?  (b('dg0_v_vv') <= -1.0318655967712402) ?  0.08304652070973961 :   0.06958683393023538 :   (b('dg0_v_vv') <= 4.687073230743408) ?  0.03670756794237129 :   (b('dg0_v_vv') <= 7.326849460601807) ?  0.0597449085937279 :   0.055154276388608786 :   (b('l8b4') <= 1468.9552612304688) ?  (b('l8b11') <= 2530.994140625) ?  0.01811156983834805 :   0.0058888513810679555 :   0.05594490554441281 :   (b('dg0_v_vv') <= 1.5584378242492676) ?  (b('l8b5') <= 3431.4776611328125) ?  (b('l8b4') <= 1399.27587890625) ?  (b('dg0_v_vv') <= 1.0078816413879395) ?  0.07985638049303026 :   0.07815910562312722 :   0.08638236852613557 :   0.07087076617285029 :   0.1050340342803455 :   (b('grass') <= 1.0004997551441193) ?  (b('ndvi') <= 976.5) ?  (b('dg0_v_vv') <= 0.9475173950195312) ?  (b('l8b11') <= 2940.610595703125) ?  (b('l8b11') <= 2935.29150390625) ?  (b('l8b4') <= 1672.8639526367188) ?  (b('l8b4') <= 1639.4662475585938) ?  (b('ndvi') <= 790.0) ?  -0.05628661548106531 :   -0.0053457310736250985 :   0.08659182622736145 :   (b('ndvi_med') <= 864.5) ?  (b('l8b4') <= 2146.4608154296875) ?  -0.0172317908882696 :   -0.04133276249160036 :   (b('dg0_v_vv') <= 0.39745473861694336) ?  -0.05135067757676241 :   -0.06910346026663891 :   (b('dg0_v_vv') <= 0.11856937408447266) ?  0.05006259055059757 :   0.07822770123277174 :   (b('ndvi_med') <= 880.75) ?  (b('dg0_v_vv') <= 0.3164482116699219) ?  (b('ndvi_med') <= 631.75) ?  -0.013343900750903068 :   (b('l8b4') <= 1821.7059936523438) ?  -0.05105934935021078 :   -0.04128209440248976 :   (b('l8b4') <= 3386.688232421875) ?  (b('dg0_v_vv') <= 0.42833375930786133) ?  0.0017261796353373118 :   -0.04932771340478929 :   (b('l8b5') <= 4396.07421875) ?  0.036459516515317814 :   -0.022911184297294857 :   (b('l8b5') <= 2226.9453125) ?  (b('l8b11') <= 2978.9263916015625) ?  -0.06810346196537129 :   (b('dg0_v_vv') <= 0.4848356246948242) ?  -0.07631234388328559 :   -0.07910346190576664 :   (b('l8b4_med') <= 3478.349365234375) ?  (b('ndvi') <= 955.5) ?  -0.05030902253837218 :   -0.0605466365348178 :   (b('dg0_v_vv') <= 0.4943079948425293) ?  -0.07944525669854358 :   -0.0473460853771753 :   (b('l8b11') <= 3030.60693359375) ?  (b('ndvi_med') <= 872.25) ?  (b('ndvi') <= 856.0) ?  0.056703806648642774 :   0.10873416852205905 :   0.16489654083604702 :   (b('l8b11_med') <= 4007.7548828125) ?  (b('l8b5') <= 2936.91748046875) ?  -0.04368107358014797 :   -0.05587847393342908 :   0.0013613263347283938 :   (b('l8b4') <= 1409.993896484375) ?  (b('dg0_v_vv') <= 2.4147143363952637) ?  (b('l8b5_med') <= 2812.056640625) ?  (b('l8b4') <= 1168.9111633300781) ?  (b('l8b11') <= 1912.4689331054688) ?  0.011674764001483512 :   0.010234140215222953 :   -0.0024014292441117135 :   (b('ndvi') <= 1245.0) ?  (b('dg0_v_vv') <= 1.4126996994018555) ?  (b('dg0_v_vv') <= 1.0790929794311523) ?  0.0694629312936773 :   0.045723654822698384 :   0.08344061139710204 :   (b('ndvi') <= 1654.0) ?  (b('l8b5') <= 1863.3602294921875) ?  0.03916945950800687 :   0.050308530883297085 :   (b('l8b4_med') <= 1652.7982177734375) ?  0.03623591891824626 :   0.016037641361399874 :   (b('dg0_v_vv') <= 3.655099630355835) ?  (b('ndvi') <= 1236.0) ?  0.06831413349301113 :   (b('l8b11') <= 2467.1976928710938) ?  0.07371343006680103 :   0.07803099719351744 :   (b('dg0_v_vv') <= 4.86641788482666) ?  0.11871415516525996 :   0.09583077722215114 :   (b('dg0_v_vv') <= 3.5695295333862305) ?  (b('l8b11') <= 2630.368896484375) ?  (b('dg0_v_vv') <= 1.678795337677002) ?  (b('l8b4') <= 2254.5) ?  (b('l8b5') <= 3497.5291748046875) ?  0.026403738672824643 :   -0.004965902930730186 :   (b('ndvi') <= 2418.0) ?  0.02260142451159375 :   0.0715467518665899 :   (b('dg0_v_vv') <= 2.8872671127319336) ?  (b('dg0_v_vv') <= 2.3733108043670654) ?  0.06657643880210495 :   0.07784611985060863 :   (b('dg0_v_vv') <= 3.352047920227051) ?  0.04885340755051795 :   0.04399810902726303 :   (b('ndvi') <= 981.0) ?  0.11937902879790166 :   (b('ndvi') <= 3498.0) ?  (b('l8b5') <= 2231.039306640625) ?  0.021619472423530985 :   -0.009089702211684038 :   (b('l8b11') <= 2881.7080078125) ?  -0.034029074261125225 :   0.06367226414448526 :   (b('l8b4') <= 1865.9760131835938) ?  (b('ndvi_med') <= 1481.0) ?  (b('bulk') <= 151.5) ?  (b('dg0_v_vv') <= 3.6865854263305664) ?  0.08757454230280162 :   0.08412721220754271 :   0.07554203731292458 :   0.1255140077186287 :   (b('dg0_v_vv') <= 3.7491743564605713) ?  0.039607808095867086 :   (b('l8b11') <= 3268.2208251953125) ?  0.025856964341828326 :   -0.052542190151756374 :   (b('l8b5_med') <= 3211.2489013671875) ?  (b('dg0_v_vv') <= 2.304236888885498) ?  (b('l8b11_med') <= 2666.28271484375) ?  (b('dg0_v_vv') <= -0.36904382705688477) ?  (b('l8b4_med') <= 1714.1714477539062) ?  (b('ndvi') <= 1582.5) ?  (b('ndvi') <= 1377.0) ?  -0.07904817029214922 :   -0.09569087518314716 :   (b('ndvi') <= 1765.0) ?  -0.06106990635141259 :   -0.052752715862009816 :   (b('ndvi') <= 703.0) ?  (b('ndvi') <= 680.5) ?  -0.04033087242819168 :   -0.07203572780462958 :   (b('l8b4') <= 2136.1529541015625) ?  -0.027403249704419236 :   -0.04653087158180572 :   (b('ndvi') <= 2620.5) ?  (b('l8b4') <= 1383.4856567382812) ?  (b('ndvi') <= 1419.0) ?  -0.046335677264663144 :   -0.0700971337899708 :   (b('l8b11') <= 2570.8212890625) ?  -0.0859816987723296 :   -0.07541298131972976 :   -0.024538896583980202 :   (b('l8b5') <= 2428.4473876953125) ?  (b('sand') <= 50.5) ?  (b('l8b11') <= 2549.776611328125) ?  (b('l8b4_med') <= 1491.0311889648438) ?  0.10394709166732452 :   -0.004479661436758335 :   (b('l8b4_med') <= 1841.5863647460938) ?  0.06096272106401549 :   0.019843524039897455 :   (b('bulk') <= 141.5) ?  0.10063420892308794 :   (b('l8b5_med') <= 2409.5438232421875) ?  -0.04524351453622769 :   -0.018952239358417628 :   (b('dg0_v_vv') <= 0.3655891418457031) ?  (b('ndvi_med') <= 1658.5) ?  (b('ndvi') <= 1261.0) ?  -0.046156228771278306 :   -0.031517451245959865 :   (b('l8b5_med') <= 2740.7989501953125) ?  0.02545883622930587 :   -0.05266792402152852 :   (b('l8b5') <= 2920.841064453125) ?  (b('sand') <= 44.0) ?  0.02500942275640761 :   -0.014886229498668887 :   (b('moss') <= 5.99336838722229) ?  -0.04892374746859302 :   -0.0252312402388013 :   (b('moss') <= 13.497345924377441) ?  (b('ndvi_med') <= 1819.0) ?  (b('moss') <= 8.057602167129517) ?  (b('l8b4') <= 1233.6731567382812) ?  (b('moss') <= 4.424629330635071) ?  0.08140032422772771 :   0.0705860418223449 :   (b('grass') <= 53.71356391906738) ?  0.010293735496046967 :   -0.08083975562287399 :   (b('crops') <= 25.207279205322266) ?  (b('l8b11') <= 2782.2869873046875) ?  0.04412638043292694 :   0.022832312719821946 :   (b('l8b5') <= 3067.1680908203125) ?  0.08289342521324813 :   0.11363290381620195 :   (b('l8b11_med') <= 2741.1431884765625) ?  0.028155866015225656 :   (b('l8b4_med') <= 1480.4217529296875) ?  (b('sand') <= 61.0) ?  0.14466603667250233 :   0.13788251076823013 :   (b('ndvi') <= 1830.0) ?  0.09509193489009678 :   0.11021844830234699 :   (b('l8b11') <= 3311.6610107421875) ?  (b('ndvi') <= 1811.0) ?  (b('l8b4_med') <= 1853.0592041015625) ?  (b('l8b5') <= 3057.7310791015625) ?  -0.08690661865428904 :   -0.019586763412342972 :   (b('dg0_v_vv') <= 2.7634897232055664) ?  -0.026027926029098144 :   0.019535881543189276 :   (b('ndvi') <= 2513.5) ?  (b('l8b4') <= 1704.9218139648438) ?  -0.06362709604813274 :   -0.10940160312607361 :   (b('sand') <= 47.0) ?  -0.03151020606590932 :   -0.038795086802917725 :   (b('dg0_v_vv') <= 3.22707462310791) ?  0.0025013108165568287 :   (b('ndvi_med') <= 1578.5) ?  0.040219291796788764 :   (b('ndvi') <= 3169.0) ?  0.030489800371392317 :   0.022501306644231694 :   (b('crops') <= 9.474358558654785) ?  (b('l8b11_med') <= 3434.5712890625) ?  (b('ndvi') <= 642.5) ?  (b('l8b5') <= 3471.580322265625) ?  (b('ndvi') <= 609.5) ?  (b('ndvi') <= 553.5) ?  0.040529528851701885 :   0.05210340431324682 :   (b('dg0_v_vv') <= 1.488652229309082) ?  0.015093321184362352 :   0.010413686129260497 :   -0.03140221673730581 :   (b('l8b11') <= 3222.0543212890625) ?  (b('ndvi') <= 1269.5) ?  (b('l8b11') <= 3124.818115234375) ?  -0.03261880088218875 :   -0.06369864215096838 :   (b('ndvi') <= 1313.0) ?  0.016628942855411943 :   0.0014110625421521056 :   (b('l8b4') <= 2470.8028564453125) ?  (b('dg0_v_vv') <= -0.7662224769592285) ?  0.005336760686411174 :   0.1313367627129691 :   (b('ndvi') <= 730.0) ?  -0.04115327626085378 :   -0.006805538551020001 :   (b('dg0_v_vv') <= 1.038175106048584) ?  (b('moss') <= 20.744579315185547) ?  (b('ndvi') <= 2007.0) ?  (b('l8b5_med') <= 3683.5443115234375) ?  -0.04166181458353486 :   -0.057448196402194396 :   (b('dg0_v_vv') <= -0.43936920166015625) ?  0.0014240398603063698 :   -0.01711988401033095 :   (b('l8b5') <= 3199.4306640625) ?  (b('l8b5') <= 3095.320556640625) ?  -0.044643319822280925 :   -0.08533711742246133 :   (b('l8b5') <= 3448.11181640625) ?  -0.0102622310114843 :   -0.034116751653953274 :   (b('dg0_v_vv') <= 3.329953670501709) ?  (b('l8b11') <= 3158.8448486328125) ?  (b('l8b4') <= 2516.7257080078125) ?  0.019612752688367915 :   -0.031187968645068842 :   (b('l8b11') <= 4095.0716552734375) ?  -0.02413458280484655 :   -0.04820603599748184 :   (b('dg0_v_vv') <= 8.078087329864502) ?  (b('dg0_v_vv') <= 4.185548305511475) ?  -0.006744336081255908 :   0.015399281210979304 :   -0.030629364887149457 :   (b('l8b11_med') <= 2736.5032958984375) ?  (b('l8b5') <= 3586.195068359375) ?  (b('dg0_v_vv') <= -2.2668237686157227) ?  0.05381808069840113 :   (b('ndvi_med') <= 3548.25) ?  (b('ndvi') <= 3443.5) ?  0.019377084028752156 :   0.02023933200806341 :   0.028417289498946646 :   (b('l8b5') <= 3642.061279296875) ?  0.07019498804375746 :   0.06076271509795786 :   (b('l8b4_med') <= 2066.49853515625) ?  (b('l8b4_med') <= 2000.5802612304688) ?  (b('ndvi') <= 4289.5) ?  (b('l8b11') <= 2646.4464111328125) ?  -0.049802039943509 :   -0.05986481136961968 :   (b('l8b5_med') <= 3219.7747802734375) ?  -0.00015837786215126826 :   0.01964162873608291 :   (b('l8b4') <= 2455.4144287109375) ?  (b('l8b11') <= 2157.4681396484375) ?  -0.002751870134121369 :   -0.04180190007719123 :   (b('dg0_v_vv') <= -1.3112750053405762) ?  -0.06308090992501447 :   -0.08444298203321592 :   (b('dg0_v_vv') <= 1.0047030448913574) ?  (b('l8b5') <= 3211.02294921875) ?  (b('dg0_v_vv') <= -0.16893863677978516) ?  -0.06636869573094505 :   -0.0546099023915936 :   (b('l8b4') <= 1412.5259399414062) ?  -0.09225502601639946 :   -0.07069931933117059 :   (b('l8b4') <= 2062.5195922851562) ?  (b('l8b11') <= 2074.1766357421875) ?  -0.06037083386111525 :   -0.03934227327794526 :   (b('l8b4') <= 2388.5601806640625) ?  -0.06784537713509628 :   -0.05347864163516459 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1148.1754760742188) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 35.5) ?  (b('moss') <= 13.872385501861572) ?  (b('dg0_v_vv') <= -0.7113780975341797) ?  (b('sand') <= 26.5) ?  (b('crops') <= 82.48814010620117) ?  (b('l8b4') <= 450.73085021972656) ?  (b('l8b11') <= 963.3289794921875) ?  0.050364462138500356 :   -0.05625735483541414 :   (b('dg0_v_vv') <= -1.330897331237793) ?  0.07865574845011522 :   0.045011762701031015 :   (b('l8b4') <= 402.54888916015625) ?  0.163061439553495 :   (b('l8b11') <= 1363.7578125) ?  0.11237434924440266 :   0.13871257939108156 :   (b('l8b11_med') <= 1355.3932495117188) ?  (b('ndvi') <= 2521.0) ?  (b('l8b5_med') <= 2442.7811279296875) ?  0.07788230551704861 :   0.04158320390728548 :   (b('dg0_v_vv') <= -1.687577724456787) ?  -0.05361750558969339 :   -0.0024385524326639013 :   (b('l8b5') <= 2442.173828125) ?  -0.02850737999217251 :   (b('grass') <= 24.1593017578125) ?  0.10412955442551151 :   0.04468085451460327 :   (b('l8b11') <= 1206.1826782226562) ?  (b('l8b11') <= 1162.5672607421875) ?  (b('l8b11') <= 917.5054321289062) ?  (b('sand') <= 32.0) ?  0.15145289706838827 :   0.04481767474705857 :   (b('l8b11') <= 1003.1830139160156) ?  0.03859427792354773 :   0.08376944408692485 :   (b('ndvi_med') <= 4290.75) ?  (b('l8b4') <= 575.67578125) ?  0.11862044187383902 :   0.14301804463325743 :   (b('ndvi') <= 4508.0) ?  0.15317469317337584 :   0.23476553252327959 :   (b('ndvi') <= 3639.5) ?  (b('l8b5') <= 2024.6710815429688) ?  (b('ndvi') <= 2839.5) ?  0.04143006061644788 :   -0.09861950431482794 :   (b('l8b5') <= 3440.2813720703125) ?  0.0870561002398222 :   0.14213405309346222 :   (b('dg0_v_vv') <= 1.5212135314941406) ?  (b('crops') <= 48.76013374328613) ?  0.03653286924907532 :   -0.044576939771638895 :   (b('l8b11_med') <= 1292.2822875976562) ?  0.135096828999259 :   0.04087659763353305 :   (b('l8b4') <= 474.72686767578125) ?  (b('l8b5') <= 3327.8951416015625) ?  (b('l8b4') <= 422.8126983642578) ?  (b('l8b4') <= 416.0238952636719) ?  (b('dg0_v_vv') <= 0.24152421951293945) ?  0.08786109321795563 :   0.10422448065295659 :   (b('dg0_v_vv') <= 0.44741153717041016) ?  0.12539158237728482 :   0.14839158428463345 :   (b('ndvi') <= 2599.0) ?  (b('l8b5_med') <= 2124.7242431640625) ?  0.08043694887339056 :   0.1013462303116538 :   (b('l8b11') <= 846.2605895996094) ?  0.053002656958408 :   0.09021635614511445 :   (b('l8b5_med') <= 2724.859619140625) ?  0.16005757172366034 :   0.11805757601519476 :   (b('l8b11') <= 1416.2500610351562) ?  (b('grass') <= 35.03351974487305) ?  (b('l8b4') <= 740.258544921875) ?  (b('dg0_v_vv') <= -0.6305975914001465) ?  0.013134387720717356 :   0.09887618982317091 :   (b('dg0_v_vv') <= 0.06849002838134766) ?  0.12519463913548087 :   -0.036004999418149886 :   (b('ndvi') <= 3282.0) ?  (b('ndvi') <= 2520.0) ?  0.03139419785960113 :   -0.0009796017132216689 :   (b('l8b5') <= 2371.383544921875) ?  0.064425306227416 :   5.797180822839261e-05 :   (b('l8b4') <= 1001.8004150390625) ?  (b('ndvi') <= 2786.5) ?  (b('l8b5') <= 2611.00146484375) ?  -0.056805932860040896 :   -0.05072168372996702 :   (b('l8b5') <= 2645.9599609375) ?  -0.06845300584899018 :   -0.0667216789019908 :   (b('dg0_v_vv') <= 0.2151017189025879) ?  (b('ndvi') <= 2932.5) ?  -0.05482597427356125 :   -0.038239507881090984 :   (b('l8b5') <= 2642.7059326171875) ?  0.004881600717223683 :   0.02653478208695645 :   (b('l8b5_med') <= 2940.280029296875) ?  (b('grass') <= 8.827258110046387) ?  (b('ndvi') <= 2571.5) ?  (b('ndvi') <= 1452.0) ?  (b('ndvi') <= 991.5) ?  (b('l8b4_med') <= 620.2438659667969) ?  0.055795666247759584 :   0.0763421062270432 :   0.051195989929846336 :   (b('l8b4') <= 620.2438659667969) ?  0.11384056307963927 :   0.07374289867442627 :   0.025534919260113226 :   (b('l8b5') <= 1697.7359619140625) ?  (b('ndvi') <= 2597.5) ?  (b('dg0_v_vv') <= 1.7294659614562988) ?  (b('ndvi') <= 1505.5) ?  -0.012463900704848416 :   -0.05500893158449715 :   (b('l8b4_med') <= 610.1719055175781) ?  0.05878989580239562 :   -0.03576403534488131 :   (b('dg0_v_vv') <= -0.02045440673828125) ?  (b('ndvi') <= 2759.5) ?  0.01654650655967016 :   0.00645324788231047 :   (b('l8b5') <= 1453.7935791015625) ?  0.0663284408134751 :   0.11162313500399401 :   (b('ndvi') <= 3185.0) ?  (b('l8b11') <= 1200.05517578125) ?  (b('l8b11') <= 1005.138916015625) ?  0.011126746851562218 :   0.0734515332317147 :   (b('bulk') <= 148.5) ?  -0.015602720007392401 :   -0.0657022243683562 :   (b('l8b11') <= 939.8026123046875) ?  (b('ndvi_med') <= 2482.25) ?  0.06166016504298594 :   -0.03913637038791428 :   (b('ndvi_med') <= 3681.25) ?  -0.020807994363901187 :   0.0069313127442110775 :   (b('ndvi_med') <= 4666.0) ?  (b('ndvi') <= 4821.5) ?  (b('moss') <= 4.990872859954834) ?  (b('ndvi') <= 4279.5) ?  (b('l8b4') <= 774.0072631835938) ?  0.02551382390544789 :   -0.00576772602109015 :   (b('l8b5') <= 1854.09521484375) ?  0.09026132743662489 :   -0.00743971968917887 :   (b('bulk') <= 119.0) ?  0.20619324813419126 :   (b('l8b5') <= 2685.5068359375) ?  0.11327479336073845 :   0.03914067282680796 :   (b('l8b4') <= 585.3011779785156) ?  (b('l8b4_med') <= 564.0519409179688) ?  (b('crops') <= 44.69799613952637) ?  0.13503808449636026 :   0.046560824112827075 :   (b('l8b11') <= 970.9723205566406) ?  0.027346631306466664 :   -0.03323366379623225 :   (b('l8b5') <= 3236.3077392578125) ?  (b('dg0_v_vv') <= -0.1966848373413086) ?  0.04285517303238967 :   0.06131679842013488 :   (b('dg0_v_vv') <= 1.1350011825561523) ?  0.19799615192227368 :   0.10331138717690785 :   (b('l8b5') <= 3078.330078125) ?  (b('dg0_v_vv') <= 0.624910831451416) ?  (b('dg0_v_vv') <= -1.8872909545898438) ?  0.18667085752866291 :   (b('dg0_v_vv') <= -1.2586750984191895) ?  0.09818561719897853 :   0.14132404230789233 :   (b('l8b4') <= 693.3630523681641) ?  0.22181407547885357 :   0.2503431392179506 :   (b('l8b4') <= 500.34800720214844) ?  (b('dg0_v_vv') <= -0.1562938690185547) ?  0.10046218225872833 :   (b('dg0_v_vv') <= 0.5884723663330078) ?  0.15837542739345412 :   0.1944756357209667 :   (b('dg0_v_vv') <= 0.771763801574707) ?  (b('l8b5') <= 3202.7919921875) ?  -0.018193128632025254 :   0.020908306794534665 :   (b('dg0_v_vv') <= 1.0215387344360352) ?  0.2214649650298572 :   0.04623013679240502 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1274.3564453125) ?  (b('l8b5') <= 2205.009521484375) ?  (b('l8b4_med') <= 485.4912109375) ?  (b('l8b5') <= 2085.4517822265625) ?  (b('l8b11') <= 734.0272827148438) ?  (b('dg0_v_vv') <= 0.9823694229125977) ?  0.15708195967648853 :   0.16400686076116294 :   (b('ndvi') <= 1914.5) ?  0.2294573312804929 :   0.08155332262993682 :   (b('ndvi') <= 4657.0) ?  (b('dg0_v_vv') <= -0.5884246826171875) ?  0.13221314594004174 :   0.17360766301410666 :   (b('dg0_v_vv') <= -0.5626401901245117) ?  0.15897576457809748 :   -0.016514938229765574 :   (b('l8b11') <= 1247.9188232421875) ?  (b('ndvi_med') <= 3660.5) ?  (b('dg0_v_vv') <= -1.2867546081542969) ?  0.08189684213964112 :   0.03284695203037101 :   -0.05354259057038241 :   0.24131173970196992 :   (b('dg0_v_vv') <= 1.012650489807129) ?  (b('l8b11') <= 939.8609008789062) ?  (b('crops') <= 3.734968423843384) ?  (b('dg0_v_vv') <= 0.19574737548828125) ?  0.03324087072742642 :   0.11866820956481877 :   (b('l8b11_med') <= 1221.8770751953125) ?  0.11424845942770559 :   0.17321175049072754 :   (b('l8b11') <= 1394.8807373046875) ?  (b('l8b5_med') <= 2884.7431640625) ?  -0.004613112952664431 :   0.05070376982495286 :   (b('ndvi') <= 3326.0) ?  0.07994371599989585 :   0.022811281202321166 :   (b('l8b4') <= 438.2935485839844) ?  (b('ndvi') <= 4453.5) ?  0.030826539676946124 :   0.004826546591084918 :   (b('grass') <= 57.94600296020508) ?  (b('l8b11') <= 954.4361572265625) ?  0.07227724456528642 :   0.16300092048850104 :   (b('ndvi') <= 3648.0) ?  0.13910348360255972 :   0.07745879198832495 :   (b('l8b11') <= 1235.6378173828125) ?  (b('l8b5_med') <= 2532.8045654296875) ?  -0.01947301026808887 :   (b('grass') <= 60.62055587768555) ?  0.024748953728701478 :   0.02060026864693354 :   (b('ndvi') <= 4570.0) ?  (b('l8b11') <= 2194.7716064453125) ?  (b('l8b5') <= 3136.4246826171875) ?  (b('sand') <= 43.5) ?  -0.054844411994920406 :   -0.03136818153839649 :   (b('dg0_v_vv') <= -0.23987388610839844) ?  -0.06827354739459399 :   -0.0971104202519853 :   0.03248650050183299 :   (b('l8b11') <= 1586.7207641601562) ?  (b('dg0_v_vv') <= -0.2812333106994629) ?  -0.011793226461229184 :   -0.013760024923874042 :   (b('dg0_v_vv') <= -2.285740375518799) ?  0.02312072135118587 :   0.010120724808255266 :   (b('l8b4') <= 550.6309814453125) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b5') <= 2533.8240966796875) ?  (b('ndvi_med') <= 3442.5) ?  (b('l8b5') <= 2010.19580078125) ?  0.08807212929345082 :   0.12419161834225012 :   (b('l8b4_med') <= 636.951416015625) ?  (b('ndvi') <= 4890.5) ?  0.13160208211999774 :   0.1476498990994552 :   (b('l8b5') <= 2010.19580078125) ?  0.17369518811978996 :   0.12719555227675766 :   (b('dg0_v_vv') <= 0.8470621109008789) ?  (b('ndvi_med') <= 3948.5) ?  (b('ndvi') <= 3822.5) ?  0.16906909476952528 :   0.15858173983561433 :   (b('ndvi_med') <= 4175.5) ?  0.13378370992304178 :   0.13813948357715627 :   (b('l8b5_med') <= 3027.6649169921875) ?  (b('ndvi') <= 4720.5) ?  0.16415436389835208 :   0.17825964205756215 :   0.19932609331404386 :   (b('l8b4') <= 491.7077331542969) ?  (b('moss') <= 0.7690265774726868) ?  (b('dg0_v_vv') <= 0.5910487174987793) ?  (b('ndvi_med') <= 4027.0) ?  0.19266783455173286 :   0.17879574439585316 :   (b('ndvi') <= 3502.5) ?  0.16942057102084268 :   0.17208070913116247 :   (b('dg0_v_vv') <= 1.1222176551818848) ?  (b('dg0_v_vv') <= -0.8860135078430176) ?  0.2068964048858115 :   0.21292788701332124 :   (b('l8b4_med') <= 532.5971221923828) ?  0.21958242575918174 :   0.22398495807598978 :   (b('ndvi') <= 4492.0) ?  (b('crops') <= 29.1966495513916) ?  (b('ndvi') <= 3979.0) ?  0.13574685848639884 :   0.16378380041313637 :   (b('dg0_v_vv') <= 0.9620800018310547) ?  0.17357095480999565 :   0.1563574190957839 :   (b('dg0_v_vv') <= 0.8995957374572754) ?  (b('ndvi_med') <= 3541.0) ?  0.19172201640551845 :   0.19675609170827224 :   0.18291279541169408 :   (b('l8b5') <= 2405.21044921875) ?  (b('ndvi') <= 3523.5) ?  (b('l8b5') <= 2361.711669921875) ?  (b('l8b11') <= 1352.4260864257812) ?  (b('ndvi') <= 3319.0) ?  0.06550232068649109 :   0.10153133958049261 :   0.19180186763387258 :   -0.031532006018580416 :   (b('l8b4') <= 640.9915161132812) ?  (b('ndvi_med') <= 3405.0) ?  -0.011889183009809373 :   (b('dg0_v_vv') <= -2.730191230773926) ?  -0.02288918295020473 :   -0.031148842602089666 :   (b('dg0_v_vv') <= 0.4966468811035156) ?  (b('dg0_v_vv') <= -1.504101276397705) ?  -0.002476031417268154 :   -0.005918627440366919 :   0.012912568275766478 :   (b('dg0_v_vv') <= -2.438610553741455) ?  (b('l8b11') <= 1388.4204711914062) ?  (b('ndvi_med') <= 3442.5) ?  -0.03227252747242185 :   -0.026572520391390048 :   0.15940848984990486 :   (b('dg0_v_vv') <= 1.0416145324707031) ?  (b('dg0_v_vv') <= 0.9879283905029297) ?  (b('l8b5') <= 2427.334228515625) ?  0.15151941494195306 :   0.12687000441846932 :   (b('l8b11_med') <= 1204.6583251953125) ?  -0.012873617518280839 :   0.10458936646859063 :   (b('crops') <= 28.65452480316162) ?  (b('grass') <= 61.095558166503906) ?  0.11864974481060146 :   0.13372986843295953 :   (b('dg0_v_vv') <= 3.189753532409668) ?  0.14885527321709446 :   0.16568467002930515 :   (b('sand') <= 28.5) ?  (b('bulk') <= 138.5) ?  (b('grass') <= 5.332152366638184) ?  (b('l8b5') <= 3417.40283203125) ?  (b('l8b4_med') <= 828.212158203125) ?  (b('l8b11') <= 1629.7060546875) ?  (b('ndvi') <= 1428.5) ?  0.1931427385926141 :   0.2078457496486537 :   (b('l8b4') <= 966.0611877441406) ?  0.15192369611069928 :   (b('l8b5') <= 1884.6951293945312) ?  0.14425790135545646 :   0.14631925424404824 :   (b('ndvi') <= 4564.5) ?  (b('ndvi') <= 986.5) ?  0.1767005807092406 :   (b('ndvi') <= 1333.5) ?  0.04021454300625826 :   0.09815574196106513 :   (b('ndvi') <= 4819.0) ?  (b('dg0_v_vv') <= 0.5179178714752197) ?  0.0446607814072314 :   0.05166078069197566 :   (b('dg0_v_vv') <= -3.387554168701172) ?  0.05474732647706291 :   0.0733959207354257 :   (b('l8b5') <= 4572.207763671875) ?  (b('l8b11') <= 1385.4227294921875) ?  (b('l8b4') <= 513.4217834472656) ?  (b('l8b5_med') <= 3490.31982421875) ?  -0.0017051380712882942 :   0.004563082719741263 :   (b('dg0_v_vv') <= 1.2988815307617188) ?  0.027145656806243517 :   0.05620612114503779 :   (b('dg0_v_vv') <= 2.059311866760254) ?  (b('ndvi') <= 4778.5) ?  -0.015045798926476409 :   0.06811041921050742 :   (b('l8b4') <= 935.1096496582031) ?  0.08886646900744369 :   -0.02663353409199784 :   (b('l8b11') <= 785.4035949707031) ?  (b('dg0_v_vv') <= 1.5561585426330566) ?  (b('l8b4_med') <= 955.183837890625) ?  0.021516190782527672 :   0.024516189411620842 :   (b('l8b4_med') <= 955.183837890625) ?  0.0290161836299703 :   0.05751618923280691 :   (b('ndvi') <= 4290.0) ?  (b('dg0_v_vv') <= 2.149768114089966) ?  0.11324405345517369 :   0.14711500407720318 :   0.017435744880197668 :   (b('l8b4_med') <= 788.9470825195312) ?  (b('l8b5') <= 3006.8961181640625) ?  (b('ndvi') <= 3644.5) ?  (b('l8b5') <= 2373.5045166015625) ?  (b('ndvi') <= 3602.5) ?  0.03391825532845899 :   -0.002664865143415938 :   (b('ndvi') <= 2757.0) ?  0.002202968767141031 :   -0.032660132843077845 :   (b('l8b5') <= 1828.1383666992188) ?  (b('dg0_v_vv') <= 0.7951436042785645) ?  0.1052577849607436 :   0.07120109287580131 :   (b('l8b11') <= 1253.7177734375) ?  0.011534429467521895 :   0.03732602234120477 :   (b('ndvi') <= 3570.5) ?  (b('dg0_v_vv') <= 0.3670840263366699) ?  (b('dg0_v_vv') <= 0.029772281646728516) ?  0.04279794366029762 :   0.11688481283163096 :   (b('dg0_v_vv') <= 0.5601515769958496) ?  0.03638193955609467 :   0.02159216463833509 :   (b('ndvi') <= 4084.5) ?  (b('dg0_v_vv') <= 1.6927199363708496) ?  0.11181251542679972 :   0.06363172810642134 :   (b('l8b5') <= 4103.4417724609375) ?  0.058649922996111534 :   0.09038502303188717 :   (b('l8b4') <= 813.4707336425781) ?  (b('l8b4_med') <= 925.0691223144531) ?  (b('l8b4') <= 305.94541931152344) ?  (b('dg0_v_vv') <= -0.48251914978027344) ?  0.048434803854671415 :   0.13191444637711747 :   (b('l8b4_med') <= 827.9517517089844) ?  -0.013172942422190883 :   0.021122038236926453 :   (b('l8b4') <= 799.6823120117188) ?  (b('l8b11') <= 1421.2685546875) ?  0.07919524484417521 :   0.03133942295433496 :   (b('sand') <= 25.0) ?  0.1049456287408016 :   0.2064494589491401 :   (b('dg0_v_vv') <= 0.8231806755065918) ?  (b('bulk') <= 135.5) ?  (b('ndvi') <= 1885.5) ?  0.06955313841448402 :   -0.0049915773108428945 :   (b('ndvi') <= 2871.5) ?  -0.05427700755020182 :   -0.014297134437894437 :   (b('l8b5') <= 1775.1478881835938) ?  (b('l8b11_med') <= 1949.941162109375) ?  -0.05934377955872769 :   -0.13864752209438352 :   (b('l8b5') <= 2651.979248046875) ?  0.07360116142908481 :   0.017094635295154488 :   (b('dg0_v_vv') <= -0.3313426971435547) ?  (b('crops') <= 50.7027587890625) ?  (b('dg0_v_vv') <= -2.0845131874084473) ?  (b('ndvi') <= 2019.0) ?  (b('l8b11_med') <= 1799.8099365234375) ?  (b('grass') <= 50.667964935302734) ?  0.11184431995996647 :   0.09339116286573135 :   -0.033620787786308004 :   (b('ndvi') <= 3596.0) ?  (b('l8b5') <= 2555.652099609375) ?  -0.04690829318892537 :   -0.02083151208429716 :   (b('l8b4') <= 690.1231994628906) ?  -5.45613186087035e-06 :   -0.011825871212129219 :   (b('ndvi_med') <= 3396.0) ?  (b('dg0_v_vv') <= -1.0646638870239258) ?  (b('dg0_v_vv') <= -1.4658069610595703) ?  -0.004077223374629446 :   0.0823939595449511 :   (b('grass') <= 64.01420211791992) ?  -0.07471397274526584 :   0.00419731098587939 :   (b('ndvi') <= 2244.5) ?  (b('l8b11') <= 1364.4932250976562) ?  0.19609367397521343 :   0.11120223558419885 :   (b('crops') <= 11.156686067581177) ?  0.0769027252462511 :   0.03148451525843468 :   (b('dg0_v_vv') <= -0.38315725326538086) ?  (b('l8b4') <= 1042.072509765625) ?  (b('l8b11') <= 1612.4677124023438) ?  (b('bulk') <= 143.5) ?  0.13209867151334173 :   0.08109571437561638 :   (b('ndvi') <= 2793.5) ?  0.20150874886078818 :   0.15935221991151752 :   0.021937067368309027 :   -0.010003187023672533 :   (b('l8b5') <= 3803.18603515625) ?  (b('sand') <= 24.5) ?  (b('l8b4_med') <= 798.7330017089844) ?  (b('dg0_v_vv') <= 2.177379608154297) ?  (b('moss') <= 0.994081437587738) ?  0.10153294616492213 :   0.05573729944561213 :   (b('grass') <= 53.64360809326172) ?  0.09528984723627221 :   0.18419841127797018 :   (b('l8b11_med') <= 2366.1443481445312) ?  (b('l8b11') <= 1292.0450439453125) ?  0.1027201056452913 :   0.14907972808878567 :   (b('dg0_v_vv') <= 0.10813283920288086) ?  0.06516717355612872 :   -0.035115983788315985 :   (b('ndvi') <= 4380.0) ?  (b('l8b11') <= 1246.8001098632812) ?  (b('dg0_v_vv') <= 0.6279149055480957) ?  -0.006732661453814659 :   0.06598307605789837 :   (b('l8b11') <= 1718.9332275390625) ?  0.08931367992342987 :   0.047241040371907295 :   (b('l8b5_med') <= 2814.132568359375) ?  (b('dg0_v_vv') <= 0.2652773857116699) ?  0.12598788633337954 :   0.19228234696594684 :   (b('l8b11') <= 1552.0748901367188) ?  0.05823161442962235 :   0.015496250910766313 :   (b('l8b4') <= 500.8457489013672) ?  (b('dg0_v_vv') <= 0.13854408264160156) ?  0.09841329224929818 :   (b('ndvi') <= 4079.5) ?  0.1947743742811841 :   (b('l8b5') <= 4050.150146484375) ?  0.12177501871643145 :   0.1333622931873668 :   (b('crops') <= 23.79574942588806) ?  (b('bulk') <= 152.5) ?  0.0489975590889275 :   0.10978803933915415 :   (b('ndvi') <= 4828.0) ?  (b('crops') <= 61.03461456298828) ?  -0.05006182962292165 :   -0.004054829379955299 :   (b('l8b5') <= 3888.620361328125) ?  0.05980503942171411 :   0.00917672913860447 :   (b('l8b4_med') <= 1141.771240234375) ?  (b('crops') <= 64.48217391967773) ?  (b('bulk') <= 138.5) ?  (b('ndvi_med') <= 3353.25) ?  (b('grass') <= 3.4404144287109375) ?  (b('ndvi') <= 2476.0) ?  (b('l8b4') <= 896.9746398925781) ?  0.06108239217700955 :   0.014715575559105524 :   (b('l8b11') <= 1472.3165893554688) ?  -0.012316654236895482 :   0.01869567839109632 :   (b('bulk') <= 130.0) ?  (b('l8b11') <= 2294.1025390625) ?  -0.0393978818984805 :   0.14432833215961674 :   (b('grass') <= 57.02948760986328) ?  -0.019326290161088833 :   0.006187193462296893 :   (b('l8b5') <= 2531.8682861328125) ?  (b('bulk') <= 128.0) ?  (b('l8b4') <= 854.4761657714844) ?  0.03905795343505102 :   0.09550873566450563 :   (b('sand') <= 52.5) ?  -0.024244983105890423 :   0.05835794880768386 :   (b('l8b11_med') <= 1136.3778076171875) ?  (b('dg0_v_vv') <= -1.1037836074829102) ?  0.05166979755392923 :   0.10610643182517766 :   (b('l8b5_med') <= 3095.4654541015625) ?  -0.011886642703441976 :   0.007717054079676115 :   (b('moss') <= 24.19915199279785) ?  (b('moss') <= 23.217729568481445) ?  (b('l8b4') <= 711.8874206542969) ?  (b('l8b4_med') <= 1060.195556640625) ?  0.028573997929628957 :   0.08266180279839738 :   (b('sand') <= 46.0) ?  0.011801923072429835 :   -0.017221260976635403 :   (b('l8b4') <= 1111.2385864257812) ?  (b('ndvi') <= 3517.5) ?  0.13755612665546735 :   0.03411260724685332 :   (b('dg0_v_vv') <= -0.7980227470397949) ?  0.012615972220580418 :   -0.037304996005553405 :   (b('sand') <= 55.5) ?  (b('dg0_v_vv') <= 3.067309856414795) ?  (b('l8b4') <= 492.1104431152344) ?  0.12313897102178141 :   -0.02784299902322713 :   (b('ndvi_med') <= 1874.25) ?  0.1134053402617144 :   0.07317668668632396 :   (b('dg0_v_vv') <= 3.323352813720703) ?  (b('l8b4') <= 1129.1240234375) ?  -0.09233310598617002 :   -0.039318719838167086 :   (b('l8b4') <= 896.2048645019531) ?  -0.07508490063757027 :   -0.011468652776780806 :   (b('sand') <= 30.0) ?  (b('dg0_v_vv') <= -0.8250458240509033) ?  (b('ndvi_med') <= 2229.5) ?  (b('ndvi') <= 2764.0) ?  -0.09029722871945459 :   -0.0444220606200281 :   (b('l8b11') <= 2617.3677978515625) ?  (b('l8b5_med') <= 3029.82763671875) ?  -0.022426088019538257 :   -0.04867838308275696 :   (b('dg0_v_vv') <= -2.1130828857421875) ?  -0.04884297128757997 :   -0.053859474467816054 :   (b('l8b4') <= 537.4624938964844) ?  (b('l8b5') <= 3449.665771484375) ?  (b('l8b11_med') <= 2104.2247314453125) ?  -0.09036509881469271 :   -0.07647904026079377 :   (b('l8b11') <= 943.1977233886719) ?  -0.016656136073387735 :   -0.03707141471012925 :   (b('l8b5') <= 2204.340576171875) ?  (b('dg0_v_vv') <= 2.30357027053833) ?  -0.03315629993774589 :   0.019456300354102368 :   (b('dg0_v_vv') <= -0.06566786766052246) ?  -0.03483702222670906 :   -0.006527909183147574 :   (b('l8b11_med') <= 1742.99267578125) ?  (b('ndvi_med') <= 2519.25) ?  (b('l8b5') <= 3341.7647705078125) ?  (b('dg0_v_vv') <= -0.0035886764526367188) ?  -0.02815004180668561 :   0.0072004640243007656 :   (b('ndvi') <= 3268.0) ?  -0.027360353621779205 :   -0.044651494788468 :   (b('l8b5') <= 3900.3150634765625) ?  (b('l8b11') <= 1476.6392822265625) ?  0.04660516126447135 :   0.009806916791783742 :   (b('sand') <= 79.0) ?  0.006006272680656661 :   -0.06779856761601646 :   (b('l8b5') <= 3686.15576171875) ?  (b('sand') <= 43.0) ?  (b('dg0_v_vv') <= 0.3815302848815918) ?  0.10700154698022367 :   0.14695882823322007 :   (b('dg0_v_vv') <= -2.2219061851501465) ?  -0.018559265756190112 :   0.04788474114349404 :   (b('ndvi') <= 2490.5) ?  (b('ndvi') <= 2178.0) ?  0.1309653718870702 :   0.13198775217838934 :   (b('l8b11') <= 1701.1583862304688) ?  0.040715266362591374 :   -0.021425066867850413 :   (b('dg0_v_vv') <= 0.11537933349609375) ?  (b('bulk') <= 169.5) ?  (b('bulk') <= 141.5) ?  (b('ndvi_med') <= 2074.0) ?  (b('ndvi_med') <= 1843.0) ?  (b('bulk') <= 140.5) ?  0.04513643204242246 :   0.1345591982077245 :   (b('l8b5') <= 2136.697265625) ?  0.2604968609822388 :   0.14779574406439072 :   (b('l8b5') <= 1856.3228149414062) ?  (b('l8b5_med') <= 2127.3282470703125) ?  0.04656022402155949 :   0.07856022926676823 :   (b('l8b4_med') <= 1193.1287231445312) ?  -0.1106740412742725 :   -0.031100938244919597 :   (b('ndvi') <= 4638.5) ?  (b('grass') <= 9.950793743133545) ?  (b('ndvi_med') <= 1462.5) ?  -0.11575040181106824 :   0.029859309192978564 :   (b('crops') <= 25.62586212158203) ?  0.006650476811946871 :   -0.02808494153965158 :   (b('l8b5_med') <= 2462.8555908203125) ?  -0.0009232010556427528 :   (b('dg0_v_vv') <= -0.5955538749694824) ?  0.13345098092694205 :   0.06987070877524745 :   (b('l8b11') <= 1264.88671875) ?  (b('dg0_v_vv') <= -0.23114538192749023) ?  (b('ndvi') <= 4472.5) ?  0.13942129179444682 :   (b('dg0_v_vv') <= -1.362370491027832) ?  0.15819290990557044 :   0.1521928977462229 :   0.1604212896486796 :   0.16943806533432423 :   (b('dg0_v_vv') <= 2.271103858947754) ?  (b('moss') <= 22.585250854492188) ?  (b('l8b11') <= 1254.7141723632812) ?  (b('l8b5') <= 1622.349365234375) ?  (b('ndvi_med') <= 1787.0) ?  -0.05410952046035333 :   3.755715813334448e-05 :   (b('grass') <= 80.97905349731445) ?  0.06376753171360003 :   0.1401667299464486 :   (b('l8b5_med') <= 2601.3017578125) ?  (b('l8b4_med') <= 1147.43994140625) ?  0.11050490460662281 :   -0.0031350997429988106 :   (b('l8b5_med') <= 2846.0980224609375) ?  0.07344740520116198 :   0.007563728305292427 :   (b('l8b11') <= 1742.5233764648438) ?  (b('l8b5') <= 2900.3350830078125) ?  (b('grass') <= 71.69305419921875) ?  0.023507654964114326 :   0.10037038168322224 :   0.12474062786945447 :   (b('moss') <= 27.39853858947754) ?  (b('l8b11_med') <= 2019.272705078125) ?  0.11314720030881699 :   0.1966832912732895 :   (b('sand') <= 54.0) ?  0.05021638219884948 :   0.17959458273001608 :   (b('crops') <= 0.3571428656578064) ?  (b('l8b4') <= 716.3681640625) ?  (b('moss') <= 5.73104453086853) ?  (b('sand') <= 38.5) ?  0.20336270157498526 :   0.06343737373307917 :   (b('ndvi_med') <= 1727.5) ?  0.20810281367074787 :   0.1420615608936186 :   (b('grass') <= 81.02410125732422) ?  (b('dg0_v_vv') <= 4.212535381317139) ?  0.03359524996005894 :   0.08867686391360853 :   (b('l8b5_med') <= 2226.137451171875) ?  0.07456727662782757 :   0.1368263188377598 :   (b('l8b5') <= 4343.3203125) ?  (b('l8b5_med') <= 3248.3341064453125) ?  (b('l8b11') <= 2252.490966796875) ?  0.03678608398628128 :   -0.1162229572859434 :   (b('l8b11') <= 1573.3331909179688) ?  0.13773748262243463 :   0.06984586460566437 :   (b('l8b11') <= 1147.6455383300781) ?  (b('ndvi') <= 3919.0) ?  -0.07524750943815649 :   -0.058776632863118475 :   (b('dg0_v_vv') <= 3.407113552093506) ?  0.03242715228760301 :   0.033427154314160934 :   (b('moss') <= 9.19554853439331) ?  (b('ndvi_med') <= 2346.25) ?  (b('moss') <= 3.484117269515991) ?  (b('l8b5_med') <= 3302.02978515625) ?  (b('grass') <= 97.49250793457031) ?  (b('bulk') <= 150.5) ?  (b('l8b4_med') <= 997.6832885742188) ?  (b('l8b5') <= 2799.2406005859375) ?  (b('l8b11_med') <= 2009.24560546875) ?  0.12417610846630239 :   0.10917609296909475 :   (b('bulk') <= 123.0) ?  0.18112554535853403 :   0.15894996438269146 :   (b('l8b11_med') <= 2472.0782470703125) ?  (b('l8b11_med') <= 2402.645751953125) ?  -0.018344555868964014 :   -0.057443998416376624 :   (b('ndvi_med') <= 1644.75) ?  -0.038394596172289124 :   0.011750861256166848 :   (b('grass') <= 79.79425048828125) ?  (b('l8b11_med') <= 1945.7068481445312) ?  (b('dg0_v_vv') <= -1.8976106643676758) ?  -0.038457720251952765 :   -0.07816292706586314 :   (b('l8b4') <= 1919.8523559570312) ?  -0.03830250600630466 :   -0.05207762626686639 :   (b('l8b5') <= 3504.0377197265625) ?  (b('dg0_v_vv') <= 1.7875685691833496) ?  -0.026465444279838563 :   0.05507337537403321 :   0.214238924246825 :   (b('dg0_v_vv') <= 0.92816162109375) ?  (b('sand') <= 41.5) ?  (b('l8b4') <= 1700.3731079101562) ?  (b('l8b4') <= 1222.7479248046875) ?  0.02587150360067177 :   -0.03364895420077003 :   (b('l8b4') <= 1789.1576538085938) ?  0.03185484434186519 :   -0.07998639952978853 :   (b('l8b5_med') <= 2420.2574462890625) ?  (b('l8b5') <= 2238.0670166015625) ?  0.025037856479473465 :   -0.03275306054383381 :   (b('ndvi') <= 1251.5) ?  0.10092439359545942 :   0.02430532478384746 :   (b('bulk') <= 144.0) ?  (b('ndvi') <= 1415.5) ?  -0.08433395305355981 :   (b('l8b11') <= 2514.67626953125) ?  -0.04126557461152977 :   -0.04559814690657668 :   (b('l8b5') <= 2768.7281494140625) ?  (b('l8b5_med') <= 2465.26171875) ?  0.053312350935001064 :   0.12668755119245492 :   (b('l8b5_med') <= 2587.6973876953125) ?  -0.018771112356161374 :   0.06567450027125 :   (b('bulk') <= 150.5) ?  (b('l8b5') <= 3142.757080078125) ?  (b('dg0_v_vv') <= -1.3798432350158691) ?  (b('ndvi') <= 1661.0) ?  (b('dg0_v_vv') <= -1.7149124145507812) ?  -0.051829049953361954 :   -0.07235923205664874 :   (b('ndvi') <= 1786.0) ?  0.038700676800224126 :   -0.04074201842876826 :   (b('l8b5') <= 2486.617919921875) ?  (b('l8b5_med') <= 3694.775634765625) ?  0.02643444225962833 :   0.06556741841715277 :   (b('ndvi') <= 1685.5) ?  -0.04493440197016185 :   0.0060046443576299725 :   (b('ndvi') <= 1822.5) ?  (b('ndvi') <= 1710.0) ?  (b('l8b11_med') <= 3230.607421875) ?  -0.07148061397916507 :   -0.015055153714589112 :   (b('l8b5') <= 3675.962646484375) ?  -0.024228630190011327 :   0.052792958619888884 :   (b('ndvi') <= 2036.0) ?  (b('dg0_v_vv') <= 3.6595005989074707) ?  -0.0777690676625185 :   -0.0657378896884711 :   (b('ndvi') <= 2560.0) ?  -0.028155948337401832 :   -0.06066445703829148 :   (b('l8b11') <= 1619.9957275390625) ?  (b('ndvi') <= 2474.0) ?  0.03833135021846412 :   (b('ndvi') <= 4180.5) ?  (b('l8b4_med') <= 1536.2318115234375) ?  0.15848337489523012 :   0.1267637249031971 :   0.12023583046423808 :   (b('dg0_v_vv') <= 1.529259204864502) ?  (b('l8b5') <= 2834.01611328125) ?  (b('l8b5') <= 1955.299560546875) ?  0.1121561617049413 :   0.031515108182724294 :   (b('ndvi') <= 1860.5) ?  -0.014936845047111728 :   0.015871498378249867 :   (b('ndvi') <= 1816.5) ?  (b('l8b11') <= 3472.1177978515625) ?  0.03819941702129758 :   0.016744305426488006 :   (b('l8b4') <= 2186.47314453125) ?  0.0684164731501881 :   0.12213586172435216 :   (b('moss') <= 4.223352074623108) ?  (b('dg0_v_vv') <= 0.4428677558898926) ?  (b('ndvi') <= 2671.5) ?  (b('l8b5') <= 1921.9046630859375) ?  0.13197692698879854 :   (b('ndvi') <= 1429.5) ?  (b('l8b4') <= 2727.7564697265625) ?  0.010108292913267243 :   0.048735687626174494 :   (b('l8b5') <= 3005.55322265625) ?  0.0704886878183156 :   0.021441203901249036 :   (b('l8b11') <= 2452.679931640625) ?  (b('l8b5') <= 3292.90283203125) ?  (b('dg0_v_vv') <= -1.0405373573303223) ?  0.1068139339903259 :   0.12111828708778913 :   (b('crops') <= 75.78702926635742) ?  0.022917386285407654 :   0.01051532678936179 :   (b('l8b5_med') <= 2844.591552734375) ?  0.03642611107184282 :   (b('ndvi_med') <= 2107.5) ?  0.11060301552182884 :   0.15475298595406395 :   (b('l8b5') <= 3642.251220703125) ?  (b('ndvi') <= 2190.5) ?  (b('l8b5') <= 2428.5419921875) ?  (b('dg0_v_vv') <= 0.5741424560546875) ?  0.1500933832333114 :   0.09530077828414973 :   (b('l8b4') <= 2098.8707275390625) ?  0.026508161618862693 :   0.0932862149995558 :   (b('l8b4') <= 1682.5559692382812) ?  (b('l8b4') <= 1261.9492797851562) ?  0.1374567339375679 :   0.10453782944636714 :   (b('ndvi_med') <= 2107.5) ?  0.1670728406819839 :   0.16385732843053247 :   (b('ndvi') <= 1176.5) ?  -0.015989579498176615 :   (b('ndvi') <= 1302.0) ?  0.14518082706647625 :   (b('ndvi') <= 1658.5) ?  -0.02681916387361774 :   0.03484727375737194 :   (b('moss') <= 8.321518421173096) ?  (b('dg0_v_vv') <= 0.7383213043212891) ?  (b('l8b11') <= 1402.69677734375) ?  0.20642132920507617 :   (b('ndvi') <= 3170.0) ?  (b('sand') <= 27.0) ?  -0.06557202675272297 :   -0.024368459254919445 :   (b('l8b11') <= 2474.0677490234375) ?  0.004901754511938987 :   0.08009182833984782 :   (b('ndvi_med') <= 1731.0) ?  (b('l8b11') <= 2609.6944580078125) ?  (b('l8b4') <= 1340.5200805664062) ?  -0.052409863910256614 :   0.03901051994650768 :   (b('dg0_v_vv') <= 0.8087539672851562) ?  0.0957893228438322 :   -0.028395685146945654 :   (b('dg0_v_vv') <= 1.7154903411865234) ?  (b('l8b4') <= 1175.314697265625) ?  -0.07353161049635654 :   0.03646163658439703 :   (b('l8b4') <= 1723.1982421875) ?  0.09962935841499407 :   0.21788892955857217 :   (b('sand') <= 50.0) ?  (b('l8b11') <= 2235.1251220703125) ?  (b('ndvi') <= 1917.5) ?  (b('ndvi_med') <= 1841.5) ?  -0.06758730351281367 :   -0.05089857973513551 :   (b('l8b4_med') <= 1132.2922668457031) ?  0.004667437556841875 :   -0.0070526221276973056 :   (b('l8b4_med') <= 1805.8067626953125) ?  (b('l8b11') <= 3349.4808349609375) ?  0.06528489903392436 :   -0.005320613665039951 :   (b('l8b11') <= 2798.9844970703125) ?  0.06250961521504422 :   -0.01772033060189764 :   (b('l8b4') <= 2351.68359375) ?  (b('l8b4') <= 2335.6044921875) ?  (b('dg0_v_vv') <= 1.02064847946167) ?  -0.0008945347768733525 :   0.033072950857377235 :   (b('ndvi') <= 1344.0) ?  0.10201116503257562 :   0.04684450329322626 :   (b('l8b5') <= 4130.2208251953125) ?  (b('dg0_v_vv') <= 0.24332237243652344) ?  -0.05521054079664405 :   -0.06978508336102936 :   (b('ndvi_med') <= 1751.0) ?  -0.04054979596975862 :   -0.02224863453534978 :   (b('l8b5_med') <= 2983.2811279296875) ?  (b('moss') <= 2.3882676362991333) ?  (b('l8b11_med') <= 1650.3992309570312) ?  (b('dg0_v_vv') <= -2.048003673553467) ?  -0.06059273660240494 :   (b('l8b11') <= 1949.5026245117188) ?  0.052697602262803345 :   (b('ndvi') <= 2783.5) ?  (b('l8b11_med') <= 1478.4027099609375) ?  0.1355641352705549 :   0.1552742873364272 :   (b('ndvi') <= 3357.0) ?  0.09035467081793744 :   0.07557909952494934 :   (b('l8b4_med') <= 1067.40771484375) ?  (b('crops') <= 10.979857355356216) ?  (b('l8b5') <= 3209.90576171875) ?  (b('ndvi') <= 1782.5) ?  -0.05946638857750711 :   -0.020970654825264983 :   0.1073702981256632 :   (b('l8b5') <= 2627.04296875) ?  (b('bulk') <= 122.0) ?  0.1212869805107134 :   0.06997012339308617 :   (b('l8b4') <= 1187.9185791015625) ?  -0.08847685991632323 :   0.011240445622789676 :   (b('dg0_v_vv') <= 2.4130806922912598) ?  (b('l8b11_med') <= 2677.41357421875) ?  (b('grass') <= 5.0273668467998505) ?  0.028789560028212573 :   0.06857978790528577 :   (b('l8b5') <= 3187.5052490234375) ?  -0.06487950511066194 :   0.085957726152934 :   (b('grass') <= 93.53741455078125) ?  (b('l8b5_med') <= 2503.153076171875) ?  0.03549653587664209 :   0.09895359665731257 :   (b('dg0_v_vv') <= 4.487156867980957) ?  0.15976236676490524 :   0.166507486473414 :   (b('dg0_v_vv') <= 1.7777624130249023) ?  (b('ndvi') <= 3596.5) ?  (b('crops') <= 24.796006202697754) ?  (b('dg0_v_vv') <= 0.5695085525512695) ?  (b('l8b11_med') <= 2358.4903564453125) ?  -0.05839313330739959 :   0.014153240288431615 :   (b('ndvi_med') <= 2422.75) ?  0.09770224991311621 :   -0.021810312792639844 :   (b('dg0_v_vv') <= -2.7410712242126465) ?  (b('ndvi') <= 1902.5) ?  -0.008314327094012697 :   -0.08524459349964375 :   (b('l8b4_med') <= 1263.2464599609375) ?  -0.005305987875265532 :   -0.03481860720894669 :   (b('bulk') <= 137.0) ?  (b('ndvi_med') <= 3720.25) ?  (b('l8b5') <= 2649.3251953125) ?  0.06244045881858437 :   0.0026874835953882978 :   (b('l8b11') <= 2555.119384765625) ?  -0.07730943572472851 :   -0.02405326037972814 :   (b('l8b11') <= 1869.597900390625) ?  -0.025137883840487762 :   (b('dg0_v_vv') <= -0.25945138931274414) ?  0.1433135096099743 :   0.0908631567061841 :   (b('l8b4') <= 1228.4132690429688) ?  (b('bulk') <= 132.5) ?  0.10420674628151094 :   (b('l8b11') <= 1977.1075439453125) ?  (b('l8b11') <= 1840.0867309570312) ?  -0.08441579323815532 :   -0.050391053807062866 :   (b('bulk') <= 133.5) ?  0.004867553658783913 :   -0.028671429730545893 :   (b('l8b11_med') <= 1466.6646118164062) ?  (b('l8b5') <= 4018.9212646484375) ?  (b('l8b4_med') <= 797.8888549804688) ?  0.17715566749943387 :   0.19940515373260156 :   0.1304704219389053 :   (b('l8b11') <= 2349.8255615234375) ?  (b('dg0_v_vv') <= 2.563194751739502) ?  0.09720161237162894 :   0.04676288841280813 :   (b('dg0_v_vv') <= 3.107292890548706) ?  0.009805462421066271 :   0.05126372933924162 :   (b('moss') <= 6.359084129333496) ?  (b('l8b5') <= 3016.0538330078125) ?  (b('bulk') <= 157.5) ?  (b('ndvi') <= 1832.0) ?  (b('l8b11') <= 2327.5711669921875) ?  (b('dg0_v_vv') <= -0.4641118049621582) ?  0.04891903748346528 :   0.09422778840878589 :   0.16782344834270257 :   (b('crops') <= 39.061086654663086) ?  (b('sand') <= 39.5) ?  0.1487307780800768 :   0.07835993972643886 :   (b('dg0_v_vv') <= -2.566953182220459) ?  -0.04116923771290594 :   0.027622958970822408 :   (b('ndvi') <= 2278.5) ?  (b('dg0_v_vv') <= -0.8286228179931641) ?  (b('dg0_v_vv') <= -1.7974591255187988) ?  0.08176874003484033 :   0.11438207383088733 :   (b('ndvi') <= 1839.5) ?  0.09403822586997243 :   0.15150973585916597 :   (b('l8b4') <= 1518.189697265625) ?  (b('dg0_v_vv') <= -1.7243843078613281) ?  0.019186717463867076 :   0.059967958970503575 :   0.13151241446463308 :   (b('ndvi_med') <= 3384.75) ?  (b('ndvi') <= 1581.0) ?  (b('l8b11') <= 2087.0767211914062) ?  0.10613068012130605 :   (b('ndvi_med') <= 2883.75) ?  0.13533574769320617 :   0.13395690824187986 :   (b('l8b4') <= 1387.3922119140625) ?  (b('l8b11') <= 2045.965087890625) ?  0.031182525089205635 :   0.10334859963606811 :   (b('grass') <= 14.509827136993408) ?  0.0037531470377099017 :   0.05996163973507006 :   (b('dg0_v_vv') <= 2.007081985473633) ?  (b('l8b11') <= 1879.5122680664062) ?  (b('ndvi_med') <= 3795.0) ?  0.055201639938911785 :   -0.0001618789055122527 :   (b('ndvi') <= 4100.0) ?  -0.01999266069568249 :   -0.07916417336831068 :   0.15817167953763236 :   (b('l8b11_med') <= 1237.39453125) ?  (b('crops') <= 50.78729438781738) ?  0.02772536826218802 :   (b('l8b11_med') <= 1032.7852478027344) ?  (b('ndvi') <= 3770.5) ?  (b('l8b11_med') <= 948.3958435058594) ?  0.09664379939761797 :   0.13577184138844745 :   0.08007647999465678 :   (b('ndvi') <= 3770.5) ?  0.07945962550557656 :   0.08645962479032082 :   (b('dg0_v_vv') <= -0.2930917739868164) ?  (b('l8b11') <= 2404.8184814453125) ?  0.14337527401122957 :   (b('grass') <= 35.65878105163574) ?  0.1639562900115406 :   0.16000462019437156 :   (b('l8b11') <= 1855.7593994140625) ?  0.11071959914807239 :   (b('l8b5') <= 3844.5611572265625) ?  0.12112603237018135 :   0.11851032487205537 :   (b('l8b4_med') <= 2066.49853515625) ?  (b('dg0_v_vv') <= 2.5316896438598633) ?  (b('l8b4') <= 1292.480224609375) ?  (b('crops') <= 35.54349327087402) ?  (b('l8b11') <= 2335.0052490234375) ?  (b('l8b11') <= 1791.7130126953125) ?  (b('ndvi') <= 2004.0) ?  (b('l8b4') <= 1183.7979736328125) ?  0.022788037699286008 :   -0.03534145713044557 :   (b('moss') <= 14.337447166442871) ?  0.12338085941262465 :   0.00994004716146966 :   (b('ndvi_med') <= 1493.5) ?  (b('ndvi_med') <= 1350.5) ?  -0.020594335332505348 :   0.022407970550102893 :   (b('l8b4') <= 1176.7952880859375) ?  -0.019133435110243947 :   -0.0385835209722966 :   (b('moss') <= 20.18179416656494) ?  (b('sand') <= 37.5) ?  (b('l8b11') <= 2433.039306640625) ?  -0.08159038727596124 :   -0.04481350092494559 :   (b('ndvi') <= 1192.5) ?  0.21724516590135004 :   0.053695884345354945 :   (b('grass') <= 52.32510185241699) ?  (b('dg0_v_vv') <= -1.4873676300048828) ?  -0.05840246600729877 :   0.02405051258736316 :   (b('l8b11') <= 2584.1453857421875) ?  -0.05725602748035811 :   -0.02360298994535335 :   (b('sand') <= 38.5) ?  (b('dg0_v_vv') <= 0.13236236572265625) ?  (b('ndvi') <= 3358.5) ?  (b('l8b11') <= 2244.2845458984375) ?  -0.020398964878837356 :   0.04884106723518933 :   (b('l8b11_med') <= 1625.8787841796875) ?  0.059325850980353166 :   -0.007588059859795465 :   (b('bulk') <= 140.5) ?  (b('ndvi') <= 1694.5) ?  0.06462755046825085 :   0.026778311522497846 :   (b('dg0_v_vv') <= 1.3097338676452637) ?  0.08921741586814459 :   0.03773891126152926 :   (b('ndvi') <= 2127.5) ?  (b('l8b11') <= 2488.3687744140625) ?  (b('l8b4_med') <= 1348.3570556640625) ?  -0.029411125572858243 :   -0.03463744855295753 :   -0.040990547176276496 :   -0.07221196539989473 :   (b('ndvi_med') <= 1568.5) ?  (b('ndvi') <= 1465.5) ?  (b('l8b4_med') <= 1089.1637573242188) ?  (b('l8b5') <= 2110.091552734375) ?  (b('dg0_v_vv') <= 0.3711981773376465) ?  -0.037439791358044296 :   -0.09124522554530021 :   (b('dg0_v_vv') <= -1.125007152557373) ?  -0.07510342654703808 :   -0.09041393995806679 :   (b('moss') <= 47.06789016723633) ?  (b('grass') <= 70.30375289916992) ?  -0.024346224160239432 :   -0.044860336527985024 :   (b('dg0_v_vv') <= -0.7592005729675293) ?  -0.018748917202656794 :   -0.05802547353348114 :   (b('ndvi_med') <= 1483.0) ?  (b('ndvi') <= 1466.5) ?  (b('dg0_v_vv') <= 0.4820365905761719) ?  0.02769292859943484 :   0.12798836809064906 :   (b('dg0_v_vv') <= -1.207209587097168) ?  0.053870689350452 :   -0.005157202276510762 :   (b('bulk') <= 124.5) ?  (b('l8b5') <= 2371.425537109375) ?  -0.02939167215416666 :   0.17060832337548498 :   (b('l8b11') <= 1681.1227416992188) ?  -0.08857809490779979 :   -0.02979650478247396 :   (b('ndvi_med') <= 2267.75) ?  (b('bulk') <= 160.0) ?  (b('l8b11_med') <= 1615.6271362304688) ?  (b('ndvi') <= 1841.0) ?  -0.030076583976082094 :   0.12305630988710717 :   (b('l8b11') <= 1586.0474243164062) ?  0.0790718878937747 :   -0.04721477172236332 :   (b('l8b11') <= 2291.784912109375) ?  (b('l8b11') <= 2181.489990234375) ?  -0.027015704397385654 :   -0.052341424578698816 :   (b('l8b5') <= 2047.9649047851562) ?  0.03346633563549491 :   -0.010789712990239064 :   (b('moss') <= 11.354184627532959) ?  (b('dg0_v_vv') <= -1.8483991622924805) ?  -0.004232594246101062 :   (b('dg0_v_vv') <= 0.4505176544189453) ?  0.06856431654646794 :   0.02083803679489593 :   (b('l8b11_med') <= 2247.2081298828125) ?  (b('l8b4') <= 1342.6486206054688) ?  -0.0021895706115993055 :   -0.03919150657224474 :   (b('ndvi') <= 3642.0) ?  -0.01886767599096743 :   0.03578804483028881 :   (b('l8b11_med') <= 2896.2930908203125) ?  (b('ndvi_med') <= 1389.0) ?  (b('ndvi') <= 2058.0) ?  (b('l8b4') <= 1499.5454711914062) ?  (b('l8b11') <= 1508.805908203125) ?  -0.02159652197637979 :   (b('l8b11') <= 1951.8944091796875) ?  0.09088630543959113 :   0.0547074221006695 :   (b('bulk') <= 142.0) ?  0.19214932839049467 :   0.1469074389874882 :   (b('ndvi') <= 2301.5) ?  0.20209193454077123 :   0.2423024260310783 :   (b('l8b5_med') <= 2043.4984130859375) ?  (b('l8b5') <= 2079.6448974609375) ?  (b('dg0_v_vv') <= 4.832881927490234) ?  0.005539106049879311 :   0.0004907309515783664 :   (b('l8b5') <= 2301.2353515625) ?  -0.09740965384021216 :   -0.11362816882404081 :   (b('l8b11') <= 1901.4890747070312) ?  (b('ndvi') <= 2528.5) ?  (b('l8b4_med') <= 963.3899230957031) ?  -0.007579580666394248 :   -0.028105925410617665 :   (b('ndvi_med') <= 3524.0) ?  -0.05971347025476348 :   -0.027919876737294222 :   (b('ndvi_med') <= 1401.75) ?  -0.05615161734772839 :   (b('ndvi') <= 1027.0) ?  0.1419882151581579 :   0.05295827446131109 :   (b('l8b4_med') <= 1813.3069458007812) ?  (b('dg0_v_vv') <= 3.470470428466797) ?  (b('l8b4') <= 1548.97705078125) ?  (b('dg0_v_vv') <= 2.6933846473693848) ?  (b('dg0_v_vv') <= 2.651063919067383) ?  -0.04187428182918808 :   -0.040874283527920455 :   -0.04887428483922264 :   (b('l8b5') <= 3246.209228515625) ?  (b('l8b4') <= 1737.0136108398438) ?  -0.05824216931154364 :   -0.07054929822226517 :   (b('l8b5') <= 3480.80078125) ?  -0.028012755318142388 :   -0.058774550861417064 :   (b('l8b11') <= 2751.249267578125) ?  (b('dg0_v_vv') <= 4.229308366775513) ?  -0.03573256076223584 :   (b('dg0_v_vv') <= 4.99390435218811) ?  -0.050566062049309246 :   -0.04452695496114589 :   (b('l8b11') <= 2918.80224609375) ?  -0.027515745965364588 :   -0.02038240177255568 :   (b('l8b5') <= 2876.8597412109375) ?  (b('l8b11_med') <= 2917.3968505859375) ?  -0.033068286029142024 :   (b('l8b4') <= 1797.6257934570312) ?  (b('l8b5') <= 2427.774658203125) ?  0.027361079519937714 :   0.01336978245542924 :   (b('ndvi') <= 1324.0) ?  0.037955552503372056 :   0.05478459586083276 :   (b('l8b5') <= 3179.962890625) ?  (b('ndvi') <= 1531.0) ?  (b('dg0_v_vv') <= 3.594062089920044) ?  -0.018338782175649444 :   -0.055412034451924504 :   (b('ndvi') <= 1683.5) ?  -0.07374388208987605 :   -0.09936435532558521 :   (b('l8b5') <= 3404.8311767578125) ?  (b('dg0_v_vv') <= 3.0057826042175293) ?  0.002251179734901132 :   0.029359114546383246 :   (b('dg0_v_vv') <= 2.9380178451538086) ?  -0.047731681897933004 :   -0.004749826350114653 :   (b('l8b4_med') <= 2194.7454833984375) ?  (b('grass') <= 30.21126651763916) ?  (b('dg0_v_vv') <= 1.811309814453125) ?  (b('l8b5') <= 3211.02294921875) ?  (b('dg0_v_vv') <= -0.6050124168395996) ?  (b('crops') <= 9.474358558654785) ?  (b('l8b4') <= 1705.1132202148438) ?  -0.07081926891432393 :   -0.03224164703036611 :   (b('ndvi') <= 1517.5) ?  -0.06043794986697859 :   -0.046478395679620615 :   (b('grass') <= 16.151529788970947) ?  (b('l8b5') <= 2896.5206298828125) ?  -0.06931784086475563 :   -0.05594257552680445 :   (b('l8b4') <= 1857.331787109375) ?  -0.05022793945705258 :   -0.02861537096272445 :   (b('l8b4') <= 1412.5259399414062) ?  (b('l8b5') <= 4043.866455078125) ?  (b('l8b5') <= 3775.2972412109375) ?  -0.08349434013449535 :   -0.09608205554158677 :   -0.050968354371998814 :   (b('l8b5') <= 4116.728759765625) ?  (b('l8b11') <= 4395.508056640625) ?  -0.06414050833975625 :   -0.028955686335754238 :   (b('dg0_v_vv') <= 0.9829311370849609) ?  -0.05545082466139962 :   -0.009951425482442315 :   (b('l8b5_med') <= 3696.302001953125) ?  (b('l8b5') <= 3177.0390625) ?  (b('moss') <= 30.879332542419434) ?  (b('ndvi') <= 1321.5) ?  -0.05568600285353856 :   -0.07597182295341459 :   (b('l8b5') <= 2760.4580078125) ?  -0.034277966874931357 :   -0.0420204512053326 :   (b('l8b11') <= 3653.9498291015625) ?  (b('l8b11') <= 3574.7095947265625) ?  -0.04296694979728222 :   0.0007927347577541405 :   (b('dg0_v_vv') <= 1.8874573707580566) ?  -0.02346442056832991 :   -0.06903701749464035 :   (b('ndvi') <= 2023.0) ?  -0.0074644179457255405 :   0.01523323610773164 :   (b('dg0_v_vv') <= 0.8269762992858887) ?  (b('dg0_v_vv') <= -0.22994613647460938) ?  (b('l8b4') <= 2022.0881958007812) ?  (b('l8b5_med') <= 2680.7239990234375) ?  0.014980047520265133 :   0.02998004811631158 :   (b('ndvi') <= 1681.5) ?  (b('l8b11_med') <= 2996.6781005859375) ?  -0.028636348757276317 :   -0.03809183910591269 :   (b('ndvi_med') <= 1758.5) ?  -0.013573768891755822 :   0.008480494425651339 :   (b('ndvi') <= 1679.5) ?  (b('dg0_v_vv') <= 0.2708592414855957) ?  (b('ndvi_med') <= 1115.0) ?  -0.05330301450007099 :   -0.03654358538417752 :   (b('l8b4') <= 2190.8997802734375) ?  -0.0824143862216145 :   -0.04303006416066659 :   (b('dg0_v_vv') <= 0.5759320259094238) ?  (b('ndvi') <= 1758.5) ?  0.00377280224580287 :   -0.026087001829567434 :   -0.0670269122407082 :   (b('dg0_v_vv') <= 1.0494298934936523) ?  -0.07202441467758881 :   (b('l8b5') <= 3128.144287109375) ?  (b('ndvi') <= 1184.0) ?  -0.09130579032178798 :   -0.10135172431892561 :   -0.11273287224998477 :   (b('l8b4') <= 2068.473876953125) ?  (b('moss') <= 12.847553730010986) ?  -0.026911240203775513 :   (b('bulk') <= 151.5) ?  (b('l8b4') <= 2038.6134643554688) ?  0.06772824373863195 :   (b('dg0_v_vv') <= -0.14704561233520508) ?  (b('dg0_v_vv') <= -1.301168441772461) ?  0.0499226222112823 :   0.05397174021466275 :   0.0371282487811849 :   (b('l8b4') <= 2051.4620361328125) ?  0.02494790336268779 :   (b('dg0_v_vv') <= 0.14130830764770508) ?  0.012185352583214237 :   0.006913561762293324 :   (b('dg0_v_vv') <= 1.4952983856201172) ?  (b('sand') <= 58.5) ?  (b('l8b11') <= 3016.333251953125) ?  (b('dg0_v_vv') <= -1.4902610778808594) ?  0.026721738635139644 :   (b('sand') <= 54.5) ?  -0.052879971980064625 :   -0.03435171431901678 :   (b('l8b11') <= 3993.4464111328125) ?  (b('l8b5_med') <= 3139.6427001953125) ?  -0.033839638519147064 :   -0.015147798917733777 :   (b('l8b11') <= 4230.129638671875) ?  -0.046350280370248936 :   -0.023624105538799997 :   (b('ndvi') <= 772.5) ?  (b('l8b11') <= 3192.3726806640625) ?  (b('l8b11') <= 2966.567138671875) ?  -0.004970373644432445 :   0.00787950119643208 :   (b('l8b4_med') <= 2433.415283203125) ?  -0.024031449873653438 :   -0.04795371909726176 :   (b('ndvi') <= 1299.0) ?  (b('l8b4_med') <= 2433.415283203125) ?  -0.053686977395140244 :   -0.04317375302972141 :   (b('l8b4') <= 2172.876220703125) ?  -0.03969641582965247 :   -0.005871844703271886 :   (b('l8b11') <= 3940.5704345703125) ?  (b('ndvi_med') <= 888.0) ?  (b('moss') <= 19.314571857452393) ?  0.03977841009139646 :   0.05520847610188795 :   (b('bulk') <= 152.5) ?  (b('ndvi') <= 1410.0) ?  0.00973649024746235 :   0.03542763119024056 :   (b('l8b11_med') <= 3982.898193359375) ?  -0.01622883209041419 :   0.003188577184827099 :   (b('l8b11') <= 4219.205322265625) ?  (b('l8b5') <= 4033.69140625) ?  (b('dg0_v_vv') <= 2.3858189582824707) ?  -0.034858543210258025 :   -0.025335838184771066 :   (b('ndvi') <= 1287.0) ?  -0.04016524689315759 :   -0.05100433817228215 :   -8.529010098072554e-05 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4_med') <= 681.2565307617188) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 26.5) ?  (b('l8b11') <= 978.0977783203125) ?  (b('ndvi') <= 4793.5) ?  (b('crops') <= 97.81589889526367) ?  (b('l8b4_med') <= 662.5958251953125) ?  (b('l8b4') <= 453.3358154296875) ?  (b('l8b11') <= 765.9326782226562) ?  (b('l8b4') <= 376.64190673828125) ?  0.1440552901264391 :   0.15876465892648572 :   (b('l8b5') <= 4641.697265625) ?  0.12072333985300625 :   0.15189974171327225 :   (b('ndvi') <= 4505.5) ?  (b('ndvi') <= 3605.5) ?  0.10182786476978684 :   0.12723357803152266 :   0.060935051315777006 :   0.20806289547470275 :   0.21919220463576508 :   (b('crops') <= 54.55145454406738) ?  (b('dg0_v_vv') <= -1.349484920501709) ?  -0.01734000783928602 :   0.01058544648722258 :   (b('l8b4_med') <= 560.0) ?  0.055656959819760266 :   0.11224710210511893 :   (b('l8b5') <= 3499.9541015625) ?  (b('l8b5') <= 2734.6817626953125) ?  (b('l8b5_med') <= 3503.7686767578125) ?  (b('ndvi') <= 2226.5) ?  (b('ndvi_med') <= 4073.25) ?  0.04765588963265835 :   0.09352821530869876 :   (b('dg0_v_vv') <= 0.8106398582458496) ?  (b('dg0_v_vv') <= 0.26244592666625977) ?  -0.024465361423221643 :   0.003974904727070627 :   0.04107145979519505 :   (b('ndvi_med') <= 4153.75) ?  (b('ndvi') <= 2889.0) ?  (b('l8b11') <= 1283.6634826660156) ?  0.1106378975823114 :   0.12325921953310628 :   0.09141320596764121 :   (b('l8b11') <= 1777.7081298828125) ?  0.08465645982885678 :   (b('l8b11_med') <= 1177.0497436523438) ?  0.07185614901538295 :   0.06834467285518862 :   (b('l8b11') <= 1122.6760864257812) ?  (b('ndvi') <= 4318.0) ?  (b('l8b4') <= 499.3129425048828) ?  0.0485635969070107 :   0.04239350442456402 :   0.028251784505426092 :   (b('dg0_v_vv') <= -3.6686739921569824) ?  0.024952831435969214 :   (b('sand') <= 17.0) ?  (b('l8b4') <= 645.5318908691406) ?  0.17415924287994652 :   0.12688957073812682 :   (b('dg0_v_vv') <= -1.2108306884765625) ?  0.13596131155004473 :   0.09711431651748002 :   (b('l8b11') <= 1306.8628540039062) ?  (b('l8b11_med') <= 1210.298583984375) ?  (b('dg0_v_vv') <= 1.195732593536377) ?  (b('dg0_v_vv') <= 0.034494876861572266) ?  (b('l8b11') <= 1029.928466796875) ?  0.15675242040226617 :   0.07707582393336176 :   (b('dg0_v_vv') <= 0.7943596839904785) ?  0.008958604182763438 :   0.1002997781288989 :   0.19342352563736265 :   (b('l8b4') <= 498.2988586425781) ?  (b('l8b5') <= 4404.368896484375) ?  (b('ndvi_med') <= 4103.5) ?  0.010374780004390384 :   0.054406308758189074 :   (b('ndvi_med') <= 3875.75) ?  -0.01292028786274621 :   -0.06731660916226759 :   (b('ndvi') <= 4493.0) ?  (b('dg0_v_vv') <= -1.7099637985229492) ?  0.005326313947815187 :   0.09000813350731546 :   (b('l8b11') <= 1216.5958862304688) ?  0.04069333255618852 :   -0.015476696137528098 :   (b('l8b4') <= 665.2812805175781) ?  -0.0528718382048706 :   (b('l8b4') <= 881.8447265625) ?  (b('crops') <= 63.68690490722656) ?  0.03671191893769024 :   0.05792341471019158 :   (b('dg0_v_vv') <= 0.18782424926757812) ?  (b('dg0_v_vv') <= -0.6876311302185059) ?  0.0037716992413201755 :   -0.005383092776897927 :   (b('dg0_v_vv') <= 1.5158257484436035) ?  0.009463461066101797 :   0.0006641917589699786 :   (b('ndvi_med') <= 4455.25) ?  (b('sand') <= 36.5) ?  (b('l8b4') <= 741.6185913085938) ?  (b('ndvi') <= 4906.0) ?  (b('l8b4') <= 738.7237243652344) ?  (b('dg0_v_vv') <= 1.0274624824523926) ?  (b('l8b4') <= 432.018310546875) ?  0.07560412591621957 :   0.027891818853924093 :   (b('l8b5_med') <= 2163.919677734375) ?  0.007934609411942165 :   0.08953965239988709 :   (b('dg0_v_vv') <= -0.8939356803894043) ?  0.19144080292062696 :   0.24244080196695264 :   (b('l8b11') <= 1156.272216796875) ?  (b('ndvi_med') <= 4314.0) ?  (b('l8b4') <= 513.2666625976562) ?  0.08630090240435513 :   0.074100775188188 :   0.05265749478632023 :   (b('dg0_v_vv') <= -0.03731107711791992) ?  (b('dg0_v_vv') <= -0.16209006309509277) ?  0.1915728192383244 :   0.16518980302440475 :   (b('dg0_v_vv') <= 1.1154894828796387) ?  0.1041626151673905 :   0.17801176232872803 :   (b('l8b11_med') <= 1138.7337036132812) ?  (b('l8b11') <= 1394.990478515625) ?  (b('l8b4') <= 869.5136413574219) ?  (b('l8b4') <= 744.8042907714844) ?  -0.036566451609979345 :   -0.0025841916329804636 :   (b('ndvi') <= 2850.5) ?  -0.010604834979380917 :   0.11893106790401298 :   (b('ndvi') <= 2939.5) ?  (b('sand') <= 31.0) ?  0.0053642029129513935 :   -0.05273233757533891 :   (b('l8b4') <= 938.5943603515625) ?  -0.02810910338269921 :   0.0050336592003606545 :   (b('l8b11') <= 1498.8012084960938) ?  (b('l8b5_med') <= 2643.7559814453125) ?  (b('dg0_v_vv') <= 0.49393224716186523) ?  0.07030523488041746 :   0.03152909624917763 :   (b('ndvi_med') <= 4314.0) ?  0.0066927468777811805 :   0.012706500959055761 :   (b('ndvi') <= 1667.0) ?  0.048953595629566415 :   (b('grass') <= 27.143917083740234) ?  0.06938468286928942 :   0.12498117252335188 :   (b('crops') <= 17.43150043487549) ?  (b('ndvi') <= 4319.0) ?  (b('l8b4') <= 585.3588562011719) ?  (b('l8b11') <= 1111.0009155273438) ?  (b('ndvi') <= 2085.5) ?  -0.05589419227403224 :   0.0019165160064418367 :   (b('l8b11') <= 1191.7877807617188) ?  0.10624056964119462 :   -0.05653145506485678 :   (b('moss') <= 4.658638209104538) ?  (b('dg0_v_vv') <= -0.6839079856872559) ?  -0.008044508009240822 :   0.05385441584240217 :   (b('ndvi') <= 1695.5) ?  0.014691875860837929 :   -0.03701289409284219 :   (b('ndvi_med') <= 3472.5) ?  (b('ndvi') <= 4633.0) ?  (b('dg0_v_vv') <= 0.7134585380554199) ?  -0.030911138719490427 :   -0.06571982203538097 :   (b('l8b5') <= 3658.0679931640625) ?  -0.0735423272907684 :   -0.09274013111995161 :   (b('dg0_v_vv') <= -0.5566427707672119) ?  (b('ndvi') <= 4690.0) ?  0.009458035026289346 :   -0.03012900683980431 :   (b('l8b4') <= 577.0565795898438) ?  -0.02826895595075264 :   -0.07182530680503334 :   (b('dg0_v_vv') <= -1.0402941703796387) ?  (b('ndvi') <= 4371.5) ?  (b('l8b5_med') <= 2340.0615234375) ?  (b('l8b11_med') <= 1351.4182739257812) ?  0.10245650677167534 :   0.06191198927099071 :   (b('moss') <= 5.335443019866943) ?  -0.0039981428828024055 :   0.026726096149921243 :   (b('ndvi') <= 4615.0) ?  (b('l8b11') <= 1162.4904174804688) ?  -0.1000955860934228 :   -0.03094171051879531 :   (b('l8b5_med') <= 2638.134033203125) ?  -0.04864796500776469 :   0.003796084018378658 :   (b('l8b5') <= 5077.538818359375) ?  (b('sand') <= 44.0) ?  (b('l8b5') <= 2873.183349609375) ?  0.09539898865845753 :   0.03799587578559988 :   (b('l8b5_med') <= 2848.7379150390625) ?  -0.0006219310854257774 :   0.028370503987555345 :   (b('l8b11') <= 733.4683227539062) ?  0.06506175478981863 :   (b('l8b5') <= 5776.425537109375) ?  -0.04456046859427906 :   -0.017205098955666537 :   (b('l8b11') <= 1108.5331420898438) ?  (b('crops') <= 37.08962059020996) ?  (b('ndvi_med') <= 4621.0) ?  0.02786837946843468 :   (b('l8b11') <= 1027.7081604003906) ?  0.0011448759157734467 :   0.012253190394379415 :   (b('dg0_v_vv') <= -0.27655839920043945) ?  (b('l8b11_med') <= 1049.894775390625) ?  -0.013486140138184194 :   (b('crops') <= 60.368648529052734) ?  (b('l8b4') <= 456.64051818847656) ?  0.1433429555482038 :   0.1781533548190289 :   (b('dg0_v_vv') <= -0.3760390281677246) ?  0.10831450885647251 :   0.04817960028806548 :   (b('dg0_v_vv') <= 0.6208868026733398) ?  (b('dg0_v_vv') <= 0.47183847427368164) ?  (b('l8b4_med') <= 585.4060668945312) ?  0.1871717958564937 :   0.14175860404455132 :   (b('l8b4') <= 368.88348388671875) ?  0.11041086369909578 :   0.06077948786839685 :   (b('l8b11') <= 935.7955932617188) ?  (b('dg0_v_vv') <= 0.7419066429138184) ?  0.18926985481306718 :   0.17020953226790575 :   (b('crops') <= 60.368648529052734) ?  0.18557392332077216 :   0.19693266941313706 :   (b('ndvi') <= 4529.0) ?  (b('l8b4_med') <= 585.4060668945312) ?  (b('moss') <= 1.0019225031137466) ?  -0.014996008195851923 :   (b('grass') <= 17.338309288024902) ?  0.09764673051267786 :   (b('l8b5_med') <= 3313.158935546875) ?  0.04796028696927668 :   0.03366962603464649 :   (b('l8b4') <= 754.5669250488281) ?  0.2517307944198504 :   (b('l8b11') <= 2049.476318359375) ?  (b('dg0_v_vv') <= -2.084359645843506) ?  0.17167178206618128 :   0.10564674672514124 :   0.18632069457439687 :   (b('l8b4_med') <= 630.9979553222656) ?  (b('dg0_v_vv') <= 0.6897683143615723) ?  (b('ndvi_med') <= 4697.0) ?  (b('l8b11') <= 1778.9485473632812) ?  0.010334391794929232 :   0.09866126788450036 :   (b('l8b11') <= 1475.9852294921875) ?  0.010876093771079723 :   -0.01512134377312887 :   (b('l8b11_med') <= 1259.6744995117188) ?  -0.036600123748658314 :   -0.028494111535339234 :   (b('dg0_v_vv') <= 0.1925806999206543) ?  (b('l8b5') <= 3202.7919921875) ?  -0.020541869484381248 :   0.02288523169708373 :   (b('dg0_v_vv') <= 0.649540901184082) ?  0.05893301475267404 :   (b('dg0_v_vv') <= 1.0993890762329102) ?  0.08510295833270323 :   0.08222699986310641 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11') <= 935.7751159667969) ?  (b('l8b11_med') <= 990.374267578125) ?  (b('l8b4') <= 385.4004821777344) ?  (b('l8b4') <= 306.60760498046875) ?  0.0648151249779047 :   (b('grass') <= 47.60150337219238) ?  0.053880141181192026 :   0.05878461988481021 :   (b('dg0_v_vv') <= -0.9166197776794434) ?  -0.06432660968453843 :   (b('l8b11') <= 729.9971923828125) ?  (b('ndvi') <= 1575.0) ?  0.0069109167799641935 :   -0.03136713290246382 :   (b('l8b11') <= 809.0) ?  (b('dg0_v_vv') <= 1.072892665863037) ?  0.011120355903577489 :   0.024612862527052026 :   (b('dg0_v_vv') <= -0.6521978378295898) ?  0.049500869229091554 :   0.04139837958524953 :   (b('l8b5') <= 1891.419921875) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('dg0_v_vv') <= 0.5799064636230469) ?  (b('ndvi_med') <= 3709.25) ?  0.155190406404644 :   0.1487959160702524 :   (b('ndvi_med') <= 3520.75) ?  0.1435947097567511 :   0.13809138697933127 :   (b('ndvi') <= 3026.5) ?  (b('ndvi_med') <= 3709.25) ?  (b('ndvi') <= 2357.0) ?  0.07609217637291976 :   0.09120939744688004 :   (b('dg0_v_vv') <= 0.08632087707519531) ?  0.13409266437084105 :   0.08275868339358133 :   (b('l8b5') <= 1748.1115112304688) ?  (b('ndvi_med') <= 3709.25) ?  0.04372934330885236 :   0.11052816160142265 :   (b('ndvi_med') <= 3447.5) ?  -0.061089087097006556 :   -0.06189908873851849 :   (b('grass') <= 57.87235069274902) ?  (b('dg0_v_vv') <= 0.07591009140014648) ?  (b('dg0_v_vv') <= -1.0679688453674316) ?  (b('l8b11_med') <= 1132.9754638671875) ?  0.15612091349172616 :   0.13819871417442103 :   (b('ndvi') <= 4839.0) ?  0.10870189431279835 :   0.17081149789751437 :   (b('l8b5') <= 2276.63427734375) ?  (b('l8b11') <= 802.6393737792969) ?  0.15270205460056469 :   0.13556986266617715 :   (b('ndvi_med') <= 3374.75) ?  0.17607218086651016 :   0.18772308455392225 :   (b('ndvi') <= 4313.0) ?  (b('l8b11_med') <= 1198.8316650390625) ?  (b('dg0_v_vv') <= -0.00142669677734375) ?  0.16255228399930033 :   0.10387149758681924 :   (b('ndvi') <= 3225.0) ?  0.11710904787565933 :   0.09832747291052102 :   0.028132956572423096 :   (b('ndvi') <= 3029.0) ?  (b('ndvi_med') <= 3042.5) ?  (b('l8b4') <= 572.7074279785156) ?  (b('l8b11') <= 1089.5679321289062) ?  (b('ndvi') <= 2100.0) ?  0.02426082791669759 :   (b('l8b11') <= 988.431640625) ?  0.05582589275463609 :   0.05780684499382066 :   -0.017525709241279996 :   (b('l8b4') <= 620.2667236328125) ?  -0.08804501401342055 :   (b('ndvi') <= 2303.0) ?  (b('l8b5_med') <= 2014.856689453125) ?  -0.035452074403713646 :   -0.06869025740003923 :   (b('grass') <= 47.60150337219238) ?  -0.026729474104572624 :   -0.01423251182511552 :   (b('dg0_v_vv') <= -1.747446060180664) ?  (b('ndvi') <= 2258.5) ?  (b('l8b5') <= 2676.691650390625) ?  -0.026524075066049424 :   (b('ndvi') <= 2113.5) ?  -0.05843292807937006 :   -0.044480571986171474 :   (b('l8b4_med') <= 620.4314270019531) ?  0.024678603220984724 :   0.027620746310897226 :   (b('l8b4') <= 596.6692504882812) ?  (b('dg0_v_vv') <= -0.2027416229248047) ?  (b('l8b5') <= 2323.0999755859375) ?  0.04068208751032548 :   0.03225314214209768 :   (b('dg0_v_vv') <= 0.08734607696533203) ?  0.059237949664846384 :   0.03023795049931141 :   (b('bulk') <= 113.5) ?  -0.02172158610621011 :   (b('ndvi_med') <= 4049.0) ?  0.12802183307351214 :   -0.020550000525075718 :   (b('dg0_v_vv') <= -0.7886061668395996) ?  (b('bulk') <= 144.0) ?  (b('ndvi') <= 4967.5) ?  (b('l8b11') <= 1228.2862548828125) ?  (b('l8b5') <= 2304.115966796875) ?  0.09428074069374551 :   -0.015369451590482117 :   (b('ndvi') <= 3743.0) ?  -0.05646085007326874 :   -0.008416840069693687 :   (b('ndvi_med') <= 3709.25) ?  -0.02847166641645732 :   0.1436741508219374 :   (b('ndvi') <= 3806.0) ?  0.13640688346705537 :   (b('dg0_v_vv') <= -1.5929198265075684) ?  0.13021777117319988 :   (b('l8b4') <= 614.8463134765625) ?  0.04207654349768855 :   -0.013931825679299209 :   (b('l8b11_med') <= 1368.5714111328125) ?  (b('ndvi_med') <= 3205.0) ?  (b('l8b5') <= 3040.34423828125) ?  -0.0014046949807507148 :   (b('ndvi') <= 3888.5) ?  0.1184778166394194 :   0.1618051802931473 :   (b('ndvi') <= 4151.5) ?  (b('ndvi') <= 4147.0) ?  0.03174424550270378 :   -0.05831669155976511 :   (b('ndvi_med') <= 4003.5) ?  0.07536505856446239 :   -0.016061209701410797 :   (b('ndvi') <= 3817.5) ?  (b('dg0_v_vv') <= 0.03081655502319336) ?  -0.057730063178050225 :   (b('l8b11') <= 1410.700927734375) ?  -0.03732270821581343 :   -0.04432262048461354 :   (b('l8b11') <= 1516.2619018554688) ?  (b('l8b5_med') <= 2485.4915771484375) ?  -0.005460504819681605 :   0.017968055796108384 :   (b('l8b4') <= 769.4023742675781) ?  -0.017771405092338405 :   -0.039036542036447075 :   (b('l8b4') <= 582.22607421875) ?  (b('sand') <= 41.5) ?  (b('l8b11') <= 1246.2892456054688) ?  (b('l8b11') <= 1232.1015014648438) ?  0.035141811668589346 :   -0.034923339345055315 :   (b('ndvi') <= 4151.5) ?  0.09440822278578684 :   (b('l8b5') <= 4059.4266357421875) ?  0.14240820085127756 :   0.13846602519908574 :   (b('l8b5') <= 3065.4569091796875) ?  (b('ndvi') <= 3137.5) ?  (b('l8b5_med') <= 2879.3673095703125) ?  0.19769554809818457 :   (b('grass') <= 57.495262145996094) ?  (b('dg0_v_vv') <= -0.212371826171875) ?  0.1571127354954566 :   0.1427528178875871 :   (b('dg0_v_vv') <= 1.5840044021606445) ?  0.1599052431769083 :   0.18387612019453436 :   (b('l8b4') <= 550.6309814453125) ?  (b('l8b5') <= 2533.8240966796875) ?  (b('ndvi_med') <= 3442.5) ?  0.07926491636410571 :   0.12340884255293273 :   (b('l8b11_med') <= 1190.1539916992188) ?  0.19110545833739034 :   0.14895290388695898 :   (b('l8b4') <= 556.8037414550781) ?  (b('ndvi_med') <= 3311.0) ?  0.08744080398012766 :   0.10185797482891415 :   (b('dg0_v_vv') <= 0.8886899948120117) ?  0.12132690868841173 :   0.1359299078201346 :   (b('l8b5') <= 3119.6812744140625) ?  (b('l8b5') <= 3109.942626953125) ?  (b('dg0_v_vv') <= 0.5745325088500977) ?  (b('dg0_v_vv') <= 0.03293895721435547) ?  0.1603493131342429 :   0.14019624581998238 :   (b('grass') <= 59.05731964111328) ?  0.19408477164218632 :   0.17130178907387175 :   (b('dg0_v_vv') <= 1.1924476623535156) ?  (b('dg0_v_vv') <= 0.2624478340148926) ?  0.19549175112765474 :   0.20018499550515412 :   0.1873675908473189 :   (b('l8b11') <= 1339.4357299804688) ?  (b('ndvi') <= 3810.5) ?  0.08583314517511764 :   (b('moss') <= 10.536637783050537) ?  0.14846736609222239 :   0.17253879419049983 :   (b('ndvi') <= 3515.5) ?  0.18835276414370322 :   0.18543621101801017 :   (b('dg0_v_vv') <= -2.424285888671875) ?  (b('ndvi') <= 2276.5) ?  0.13650815002241878 :   (b('l8b4_med') <= 607.0223693847656) ?  0.10629835880191169 :   (b('l8b5') <= 2427.334228515625) ?  (b('l8b4') <= 687.8755187988281) ?  (b('dg0_v_vv') <= -4.142661094665527) ?  -0.022206929585965 :   -0.005760851710109184 :   0.0783229015618257 :   (b('l8b11') <= 2276.8761596679688) ?  (b('l8b5') <= 3035.451904296875) ?  -0.027173642163288336 :   -0.047560384664131844 :   -0.07465179334681218 :   (b('l8b11') <= 1347.9652709960938) ?  (b('l8b5') <= 2756.1441650390625) ?  (b('dg0_v_vv') <= -0.18450593948364258) ?  (b('l8b4') <= 627.7143249511719) ?  (b('ndvi') <= 3323.0) ?  -0.012143605007351552 :   -0.09019836961116959 :   (b('ndvi') <= 3523.5) ?  0.041885764153451924 :   -0.005285131104074535 :   (b('ndvi') <= 3523.5) ?  (b('dg0_v_vv') <= 0.2667231559753418) ?  0.05123137361423978 :   0.10097502266650564 :   (b('ndvi_med') <= 3438.5) ?  0.020351311705681913 :   0.011621311448189842 :   (b('ndvi') <= 4327.5) ?  (b('dg0_v_vv') <= 1.9329605102539062) ?  (b('dg0_v_vv') <= -0.47915172576904297) ?  0.1146874927095743 :   0.13068098035474754 :   (b('l8b5') <= 3374.4283447265625) ?  0.15287227499645473 :   0.16328958698033202 :   (b('dg0_v_vv') <= -0.16571474075317383) ?  0.07823742960944124 :   0.08810243869028417 :   (b('sand') <= 41.5) ?  -0.046343966949333876 :   (b('dg0_v_vv') <= 1.375166416168213) ?  (b('l8b5') <= 2538.2679443359375) ?  (b('ndvi') <= 3830.0) ?  0.1322991604758444 :   0.10920724751379182 :   (b('ndvi') <= 3530.5) ?  0.1048087280394184 :   0.12755965121213217 :   (b('dg0_v_vv') <= 3.4245176315307617) ?  (b('ndvi') <= 3199.5) ?  0.15011027621899106 :   0.1378656233881534 :   (b('l8b4') <= 733.3843383789062) ?  0.18012927468016188 :   0.16426063344652903 :   (b('l8b4') <= 1330.6592407226562) ?  (b('moss') <= 6.230724096298218) ?  (b('sand') <= 48.5) ?  (b('dg0_v_vv') <= 0.4156646728515625) ?  (b('l8b4_med') <= 1046.5274047851562) ?  (b('ndvi_med') <= 2139.0) ?  (b('bulk') <= 115.0) ?  (b('dg0_v_vv') <= -0.9424290657043457) ?  0.11970694748812366 :   (b('l8b4') <= 766.3434143066406) ?  0.0022379576807068646 :   0.018090083142679004 :   (b('l8b11_med') <= 1594.7315673828125) ?  (b('l8b4') <= 939.1285400390625) ?  0.17226578599189735 :   0.12983211121991084 :   (b('ndvi') <= 1087.5) ?  0.13139677130076588 :   0.09704539821565533 :   (b('bulk') <= 147.5) ?  (b('dg0_v_vv') <= -1.199720859527588) ?  (b('grass') <= 3.9269566535949707) ?  0.03367375403335339 :   -0.008003562988919218 :   (b('crops') <= 20.988574981689453) ?  0.045894546317882284 :   0.017216694175899692 :   (b('moss') <= 3.684385895729065) ?  (b('l8b5') <= 3456.25537109375) ?  0.07431152453344121 :   0.0253757137399361 :   (b('dg0_v_vv') <= -0.5148558616638184) ?  -0.006119357670097854 :   0.05293264751929993 :   (b('ndvi') <= 1871.5) ?  (b('moss') <= 6.159199953079224) ?  (b('crops') <= 85.60601806640625) ?  (b('dg0_v_vv') <= -0.4707474708557129) ?  -0.015007374065233225 :   -0.04368780440918898 :   (b('l8b4') <= 1164.89013671875) ?  -0.0018595108086461554 :   0.07974728738789474 :   (b('l8b4') <= 921.7945861816406) ?  0.2937322164351791 :   -0.007809125486641172 :   (b('grass') <= 63.74846267700195) ?  (b('ndvi_med') <= 2346.25) ?  (b('sand') <= 26.0) ?  0.07992872014200941 :   -0.02022762662024489 :   (b('ndvi') <= 1874.0) ?  0.24442163107272263 :   0.01323639923044445 :   (b('l8b4') <= 795.4068908691406) ?  (b('l8b4_med') <= 1672.4807739257812) ?  0.10487729336812457 :   -0.04249789115189867 :   (b('dg0_v_vv') <= -0.957280158996582) ?  0.07919346202809768 :   0.00047857257207976623 :   (b('grass') <= 11.046688079833984) ?  (b('moss') <= 1.3587230145931244) ?  (b('ndvi_med') <= 2525.0) ?  (b('l8b5') <= 4360.988037109375) ?  (b('bulk') <= 138.5) ?  0.11285560407986235 :   0.05313723155109135 :   (b('ndvi') <= 3397.0) ?  0.148979593913625 :   0.11578315461552662 :   (b('l8b5_med') <= 3320.2362060546875) ?  (b('l8b4') <= 305.94541931152344) ?  0.11149758767428705 :   0.015212010489316353 :   (b('l8b5') <= 3035.6180419921875) ?  0.1007377732298717 :   0.022181714065044496 :   (b('ndvi_med') <= 2346.25) ?  (b('ndvi') <= 957.0) ?  (b('l8b4_med') <= 1132.8245239257812) ?  0.03105329320688531 :   0.05454636928867185 :   (b('l8b5') <= 2659.843994140625) ?  -0.04343147871048662 :   -0.01960100392636556 :   (b('dg0_v_vv') <= 3.3001174926757812) ?  (b('l8b5') <= 1691.8463745117188) ?  0.060154700692712115 :   0.014766239021063398 :   (b('l8b4_med') <= 1263.2464599609375) ?  0.05390911138427294 :   0.20648646607096077 :   (b('grass') <= 25.906103134155273) ?  (b('l8b5') <= 3892.8204345703125) ?  (b('l8b4') <= 1152.8848876953125) ?  (b('ndvi') <= 2665.5) ?  0.11559635349741064 :   0.15894097976295304 :   (b('l8b11_med') <= 2237.0929565429688) ?  0.10916172578188675 :   0.030773664523845126 :   (b('l8b11') <= 1683.0321655273438) ?  (b('grass') <= 21.18791961669922) ?  0.009545433915812473 :   0.0787988508963259 :   (b('ndvi') <= 2282.5) ?  0.04329288725918287 :   0.09336960698980899 :   (b('crops') <= 52.261112213134766) ?  (b('moss') <= 5.037330627441406) ?  (b('sand') <= 24.5) ?  0.08159194796141252 :   0.0470962126206354 :   (b('ndvi') <= 3688.5) ?  0.11996456297993163 :   -0.058802041327518845 :   (b('l8b4') <= 510.31988525390625) ?  (b('l8b5') <= 3550.4454345703125) ?  0.04989386742563284 :   0.09911000773520655 :   (b('l8b4') <= 684.80224609375) ?  -0.026513408180000423 :   0.020670023664132467 :   (b('ndvi_med') <= 3422.25) ?  (b('ndvi_med') <= 2164.0) ?  (b('dg0_v_vv') <= -2.343411445617676) ?  (b('ndvi') <= 3987.5) ?  (b('ndvi') <= 2135.0) ?  (b('dg0_v_vv') <= -2.7755370140075684) ?  -0.07019300400553447 :   -0.06919300384162169 :   (b('l8b11') <= 1742.8350830078125) ?  -0.0667675846757409 :   -0.05205307825382284 :   -0.09696768529530125 :   (b('l8b11') <= 1191.6804809570312) ?  (b('l8b4_med') <= 1386.1749267578125) ?  (b('l8b5') <= 1543.0519409179688) ?  -0.029697398964715026 :   0.032780008686879566 :   (b('dg0_v_vv') <= 2.836556911468506) ?  0.07884581509532435 :   0.1524003285931246 :   (b('dg0_v_vv') <= 3.046320915222168) ?  (b('ndvi_med') <= 2049.5) ?  -0.0040461464584290495 :   0.03561204343585834 :   (b('ndvi') <= 2125.0) ?  0.012092633347758258 :   0.07623603844629304 :   (b('l8b11_med') <= 1553.6473388671875) ?  (b('l8b5') <= 4265.2586669921875) ?  (b('l8b4_med') <= 813.8887939453125) ?  (b('l8b11') <= 909.3287353515625) ?  0.030319859585220373 :   -0.0033842792392297966 :   (b('l8b4') <= 1068.308837890625) ?  0.03342080141199362 :   0.07434884073706424 :   -0.09720276788567558 :   (b('crops') <= 55.03236198425293) ?  (b('l8b4') <= 1210.509765625) ?  (b('l8b4') <= 1031.6658325195312) ?  -0.038345668380295676 :   -0.0034703962336746497 :   (b('dg0_v_vv') <= 2.1109328269958496) ?  -0.07429646928254303 :   -0.1309123079525581 :   (b('l8b5') <= 2759.36083984375) ?  (b('dg0_v_vv') <= 0.05061674118041992) ?  -0.002476331922361963 :   0.03998822194904083 :   (b('l8b11') <= 1197.57666015625) ?  -0.05601155377036235 :   -0.018307774387032783 :   (b('ndvi') <= 2726.5) ?  (b('l8b4_med') <= 879.5944519042969) ?  (b('ndvi') <= 2090.5) ?  (b('l8b5') <= 2373.0792236328125) ?  (b('dg0_v_vv') <= 6.3220155239105225) ?  0.04891308461427702 :   0.0645402093041296 :   (b('grass') <= 22.634614944458008) ?  0.029621985762679243 :   0.03853178906489163 :   (b('ndvi') <= 2718.0) ?  (b('grass') <= 35.82088851928711) ?  0.08124227893711669 :   0.060185218684240434 :   0.09508293505889504 :   (b('l8b4_med') <= 955.3427429199219) ?  (b('dg0_v_vv') <= 0.03249835968017578) ?  0.10321292268995991 :   (b('dg0_v_vv') <= 0.33271074295043945) ?  0.12059460648470022 :   0.11159460314684011 :   (b('dg0_v_vv') <= -0.3566093444824219) ?  0.08999183164431102 :   (b('dg0_v_vv') <= -0.01484823226928711) ?  0.09838712737368874 :   0.10119105704864491 :   (b('ndvi') <= 3211.0) ?  (b('l8b11') <= 1383.9494018554688) ?  (b('l8b11') <= 1301.6073608398438) ?  (b('l8b5') <= 2982.054931640625) ?  -0.05403028745393579 :   -0.06464100528380125 :   -0.11073802757037024 :   (b('ndvi') <= 3024.0) ?  (b('grass') <= 24.875121116638184) ?  0.0214055484792553 :   -0.009796939409945082 :   0.056910512435127625 :   (b('l8b11') <= 647.292236328125) ?  (b('l8b5_med') <= 2830.92822265625) ?  -0.048347099798192406 :   -0.06460057557177654 :   (b('ndvi') <= 4200.0) ?  (b('l8b4_med') <= 879.5944519042969) ?  0.044177411344201466 :   -0.018869239676437017 :   (b('dg0_v_vv') <= -0.49779653549194336) ?  0.033244987374110496 :   0.0024713600220413784 :   (b('bulk') <= 137.5) ?  (b('dg0_v_vv') <= 1.496492862701416) ?  (b('ndvi_med') <= 3167.0) ?  (b('bulk') <= 134.5) ?  (b('l8b4') <= 727.8829345703125) ?  (b('l8b4_med') <= 875.34814453125) ?  (b('ndvi') <= 2719.0) ?  -0.003931475271360861 :   -0.032408402335696265 :   (b('l8b5') <= 3309.6239013671875) ?  0.06294240246328653 :   -0.005158162411801512 :   (b('l8b11') <= 1343.283447265625) ?  (b('dg0_v_vv') <= 0.38025903701782227) ?  -0.056838079924688 :   -0.0865378743924356 :   (b('l8b5') <= 3744.6102294921875) ?  -0.029226852851402016 :   0.04726662536607292 :   (b('l8b11_med') <= 1721.4924926757812) ?  (b('ndvi') <= 3766.5) ?  (b('dg0_v_vv') <= -1.0815229415893555) ?  -0.037041468859044656 :   -0.01828872598421643 :   (b('l8b11') <= 1343.7316284179688) ?  0.026743859027880942 :   -0.0033338903003868655 :   (b('l8b11_med') <= 1736.9934692382812) ?  (b('l8b4') <= 789.1884765625) ?  0.16219629595972512 :   0.23273898916721003 :   (b('l8b5') <= 2110.1883544921875) ?  0.03646114904284122 :   -0.025648215868647236 :   (b('l8b11_med') <= 1347.6688232421875) ?  (b('dg0_v_vv') <= -0.17006874084472656) ?  (b('dg0_v_vv') <= -0.24150466918945312) ?  (b('l8b11_med') <= 1300.2606811523438) ?  0.07969139007060333 :   0.09537164682558767 :   0.04997166590649704 :   (b('l8b4_med') <= 753.7964477539062) ?  0.07341582810080294 :   (b('l8b5') <= 2115.5643310546875) ?  0.11685355434134931 :   0.10293481472668213 :   (b('sand') <= 48.0) ?  (b('l8b4') <= 1133.5214233398438) ?  (b('l8b5') <= 3127.7288818359375) ?  -0.013259432020991344 :   0.0017337211187590429 :   (b('l8b4') <= 1168.7904663085938) ?  0.09039327329284416 :   0.011597458898400288 :   (b('l8b4') <= 1295.2246704101562) ?  (b('l8b4') <= 1003.8405151367188) ?  -0.026464083376176967 :   -0.06384612739037744 :   0.09964763923326514 :   (b('ndvi') <= 2414.0) ?  (b('l8b11') <= 1048.8137512207031) ?  (b('l8b11_med') <= 1380.0347290039062) ?  -0.07204226003584978 :   -0.10071887305206408 :   (b('dg0_v_vv') <= 3.418597459793091) ?  (b('l8b11') <= 2430.1014404296875) ?  (b('l8b11_med') <= 1658.15625) ?  0.028000665432010307 :   -0.003906480142925652 :   0.14787611061934008 :   (b('moss') <= 7.48991584777832) ?  0.12184911799759998 :   (b('l8b5_med') <= 2581.0853271484375) ?  0.07632282056754554 :   0.031140983680295956 :   (b('dg0_v_vv') <= 5.526454210281372) ?  (b('ndvi_med') <= 3873.5) ?  (b('l8b5') <= 3341.6258544921875) ?  (b('grass') <= 21.682384490966797) ?  0.015558452108822266 :   -0.01059347024443034 :   (b('ndvi') <= 2971.5) ?  -0.12330298705251796 :   -0.02030973166259093 :   (b('l8b5_med') <= 2971.7447509765625) ?  (b('l8b11') <= 1023.921142578125) ?  -0.024642429731312074 :   0.027318715092160464 :   (b('ndvi') <= 4461.0) ?  0.1120379052707785 :   0.040224815410900394 :   (b('l8b4_med') <= 825.6941528320312) ?  (b('dg0_v_vv') <= 7.03738808631897) ?  (b('dg0_v_vv') <= 6.490777492523193) ?  -0.05272960310920445 :   -0.05477439102989147 :   (b('l8b4_med') <= 788.7772827148438) ?  -0.03869053171014613 :   -0.02176888396766414 :   (b('l8b5') <= 3064.6373291015625) ?  -0.11411418897013134 :   -0.11911418978969521 :   (b('l8b4') <= 901.3296508789062) ?  (b('moss') <= 13.551528930664062) ?  (b('l8b4_med') <= 988.8267517089844) ?  (b('l8b4') <= 594.8707580566406) ?  (b('grass') <= 59.75464057922363) ?  (b('l8b11_med') <= 1248.767822265625) ?  -0.007451282248551105 :   0.056082871552630746 :   (b('l8b4') <= 592.6436767578125) ?  0.12859395420317568 :   0.01767092852319202 :   (b('bulk') <= 145.5) ?  (b('dg0_v_vv') <= 0.6816802024841309) ?  0.019414848484892843 :   0.05288337952260469 :   (b('bulk') <= 150.5) ?  -0.12525486385794235 :   -0.01953489787142199 :   (b('ndvi_med') <= 1550.25) ?  (b('l8b4_med') <= 1936.8309326171875) ?  (b('l8b11') <= 1189.82177734375) ?  -0.005105509820409887 :   0.04623439987660518 :   (b('dg0_v_vv') <= -0.20451688766479492) ?  -0.11007989908562793 :   -0.1288073132419393 :   (b('l8b11_med') <= 1919.4158325195312) ?  (b('l8b11') <= 1156.2919311523438) ?  -0.016635965811718403 :   0.1793413316626923 :   (b('l8b4') <= 539.9447631835938) ?  0.15486164659340254 :   0.07455518297946986 :   (b('dg0_v_vv') <= 0.9374656677246094) ?  (b('ndvi_med') <= 2972.0) ?  (b('l8b4') <= 478.9473114013672) ?  (b('sand') <= 38.5) ?  0.11317473918339252 :   0.03498276513455054 :   (b('l8b5') <= 1985.5722045898438) ?  -0.0028375201566019003 :   -0.028160131015326512 :   (b('grass') <= 42.41777038574219) ?  (b('crops') <= 26.900809288024902) ?  0.03875328605541636 :   -0.001499892213145246 :   (b('dg0_v_vv') <= 0.041251182556152344) ?  0.1743694594997803 :   0.05390881561603818 :   (b('moss') <= 16.488059997558594) ?  (b('l8b11') <= 898.5862426757812) ?  -0.15245535501375004 :   (b('sand') <= 33.5) ?  0.05890900007113161 :   -0.004369685941204432 :   (b('moss') <= 26.28097152709961) ?  (b('l8b4_med') <= 881.8549499511719) ?  0.040557233499024674 :   0.10513431424831132 :   (b('l8b5') <= 2365.2808837890625) ?  0.04594130364429602 :   -0.02345547164367002 :   (b('ndvi') <= 2318.0) ?  (b('l8b4_med') <= 1234.7986450195312) ?  (b('l8b11_med') <= 2202.55712890625) ?  (b('l8b11_med') <= 1989.0631713867188) ?  (b('l8b5') <= 1608.130126953125) ?  0.0005819096145262431 :   -0.03617562818800143 :   (b('ndvi_med') <= 2124.25) ?  -0.024571101569485814 :   0.036880664922638993 :   (b('ndvi') <= 2287.0) ?  (b('grass') <= 64.08544158935547) ?  -0.0460935028126858 :   -0.08146589904313789 :   (b('l8b5_med') <= 1992.9571533203125) ?  0.011145810931160974 :   -0.021521648016817677 :   (b('l8b4') <= 1182.69482421875) ?  (b('ndvi_med') <= 1850.0) ?  (b('sand') <= 39.5) ?  0.10820899795141789 :   0.029468959588466046 :   (b('ndvi_med') <= 1908.0) ?  -0.06804155971448268 :   0.03393365487073626 :   (b('l8b5_med') <= 3234.30810546875) ?  (b('dg0_v_vv') <= 0.46616649627685547) ?  -0.0179047386702829 :   0.025857923590063624 :   (b('ndvi') <= 1848.0) ?  -0.0414883170982763 :   -0.07108601217467647 :   (b('bulk') <= 153.5) ?  (b('dg0_v_vv') <= 0.6668763160705566) ?  (b('l8b11_med') <= 1975.468017578125) ?  (b('crops') <= 35.54349327087402) ?  -0.015286594652724297 :   0.02575865659979932 :   (b('ndvi_med') <= 1895.5) ?  -0.0026168615460708663 :   0.044494624466953646 :   (b('grass') <= 52.166311264038086) ?  (b('grass') <= 14.177669525146484) ?  0.0530250941778471 :   0.008290750687768429 :   (b('bulk') <= 145.5) ?  0.09740703235950031 :   0.038550964107285556 :   (b('l8b5') <= 3256.722900390625) ?  (b('ndvi') <= 3252.0) ?  (b('l8b5') <= 2185.668212890625) ?  0.07440090731483737 :   -0.03944225641596569 :   (b('l8b11_med') <= 1950.8327026367188) ?  -0.019818886168097738 :   0.022512070990305864 :   (b('ndvi_med') <= 3305.25) ?  (b('l8b5') <= 3395.2108154296875) ?  -0.10652237189406111 :   -0.07966911344122951 :   (b('ndvi') <= 3450.0) ?  -0.06552769217557054 :   -0.045570756362417635 :   (b('moss') <= 10.304576396942139) ?  (b('dg0_v_vv') <= 1.620556354522705) ?  (b('ndvi_med') <= 1953.0) ?  (b('l8b5_med') <= 3250.251708984375) ?  (b('l8b5_med') <= 3040.2669677734375) ?  (b('l8b11_med') <= 1958.6300659179688) ?  (b('sand') <= 36.0) ?  (b('dg0_v_vv') <= -1.8976106643676758) ?  -0.03461194822675749 :   -0.06931873421545721 :   (b('moss') <= 9.691657066345215) ?  -0.02790271348329942 :   0.11075067889839646 :   (b('ndvi') <= 2093.5) ?  (b('l8b4') <= 2273.6734619140625) ?  -0.017789055605190396 :   0.02856278249208914 :   (b('ndvi') <= 2140.0) ?  0.06963573032836429 :   0.010283111830908322 :   (b('l8b4_med') <= 1950.225341796875) ?  (b('l8b4_med') <= 1830.4744262695312) ?  (b('l8b11') <= 2850.732177734375) ?  -0.04753507583971345 :   -0.02198088523628834 :   (b('l8b5') <= 2316.269775390625) ?  -0.020633593855638826 :   -0.0502671621313664 :   (b('dg0_v_vv') <= -1.8800983428955078) ?  (b('l8b5') <= 2488.87939453125) ?  0.013801931635253534 :   -0.05265594888725464 :   (b('l8b11') <= 2623.0478515625) ?  -0.009168455408362983 :   -0.02884918033042122 :   (b('l8b4_med') <= 2143.6115112304688) ?  (b('l8b4') <= 1556.1575317382812) ?  (b('l8b4') <= 1460.6211547851562) ?  (b('ndvi_med') <= 1542.25) ?  0.07902298642209007 :   0.09154327175167913 :   0.20126304753049548 :   (b('ndvi') <= 1245.0) ?  (b('l8b5') <= 2686.695556640625) ?  -0.0074451452059606125 :   0.07988938060054637 :   (b('l8b5') <= 2839.4482421875) ?  0.09815401717938416 :   0.01131096850244392 :   (b('l8b5') <= 3038.888916015625) ?  (b('dg0_v_vv') <= -1.296532154083252) ?  (b('ndvi') <= 1939.0) ?  -0.015000173293645308 :   0.015245487842162397 :   (b('l8b4') <= 2102.1524658203125) ?  0.027846732074024556 :   0.06127877035465559 :   (b('l8b5') <= 3395.3304443359375) ?  (b('ndvi') <= 1544.0) ?  -0.043190811343629315 :   -0.018215652334174777 :   (b('l8b4') <= 2840.0167236328125) ?  0.0026619600093929934 :   -0.03432130475617649 :   (b('sand') <= 49.5) ?  (b('crops') <= 64.63016700744629) ?  (b('l8b5') <= 2141.669921875) ?  (b('l8b11_med') <= 2212.833251953125) ?  (b('l8b11_med') <= 2197.9912109375) ?  0.11491744376350488 :   0.127282350213143 :   0.14795719208740898 :   (b('l8b11_med') <= 1560.9974365234375) ?  (b('l8b4') <= 1472.570068359375) ?  0.11544172840671874 :   0.04435553278501486 :   (b('l8b4') <= 2289.8673095703125) ?  -0.011532073079969287 :   0.18949319457346966 :   (b('crops') <= 68.03106307983398) ?  (b('l8b5') <= 2794.379638671875) ?  (b('l8b4') <= 1470.6766967773438) ?  0.04354075789812906 :   0.09210548693300952 :   (b('l8b4') <= 1466.7079467773438) ?  0.14364827467784294 :   0.12008167297102766 :   (b('l8b11') <= 1723.6815795898438) ?  (b('dg0_v_vv') <= -2.514301300048828) ?  0.06540561902671335 :   0.15187480927007835 :   (b('moss') <= 3.8244681358337402) ?  0.020688918466293445 :   -0.01144108521680874 :   (b('ndvi_med') <= 2164.0) ?  (b('l8b5') <= 2925.792236328125) ?  (b('ndvi') <= 2282.0) ?  (b('l8b5') <= 2413.6229248046875) ?  -0.05098981254400514 :   0.03858594380052129 :   -0.11658142305576877 :   (b('dg0_v_vv') <= -2.3545947074890137) ?  (b('dg0_v_vv') <= -3.908644676208496) ?  0.022252995607210224 :   0.033655825671139006 :   (b('dg0_v_vv') <= -0.056795597076416016) ?  0.16427700079106597 :   0.07597260040059095 :   (b('bulk') <= 143.5) ?  (b('ndvi') <= 2751.5) ?  (b('ndvi_med') <= 3288.0) ?  0.05055323875506227 :   0.10162070900717185 :   (b('l8b11') <= 2384.972412109375) ?  -0.08801732687381181 :   -0.027181572528909643 :   (b('ndvi') <= 1984.5) ?  (b('ndvi') <= 1948.5) ?  -0.04640409258275222 :   0.004744443996325776 :   (b('l8b11_med') <= 2420.6748046875) ?  -0.11162949371090597 :   -0.0570273159410369 :   (b('crops') <= 76.88049697875977) ?  (b('l8b11') <= 2093.4111328125) ?  (b('l8b11') <= 1954.8650512695312) ?  (b('l8b11') <= 1851.0606689453125) ?  (b('l8b5_med') <= 3212.814697265625) ?  (b('bulk') <= 149.0) ?  0.02701510926947291 :   0.0041980848370765655 :   (b('ndvi_med') <= 1542.25) ?  0.07180481913259462 :   0.09672166900348178 :   (b('crops') <= 20.387500762939453) ?  (b('l8b5_med') <= 2260.5576171875) ?  0.05336273944446224 :   0.030261730721133584 :   (b('dg0_v_vv') <= 2.5934832096099854) ?  0.08946965575499972 :   0.10462485813671656 :   (b('moss') <= 5.0774842500686646) ?  (b('l8b4') <= 1370.1926879882812) ?  (b('l8b5_med') <= 3289.2808837890625) ?  0.11530127766096765 :   0.12830128910505945 :   (b('l8b4') <= 1404.94091796875) ?  0.08137625872621115 :   0.0988180496776967 :   (b('grass') <= 31.920185565948486) ?  0.18851458573675545 :   0.1594401007494905 :   (b('ndvi') <= 2585.5) ?  (b('l8b4_med') <= 1388.7922973632812) ?  (b('l8b5_med') <= 3097.873046875) ?  (b('dg0_v_vv') <= 2.9816648960113525) ?  -0.02740334890512282 :   0.020436925264261524 :   (b('l8b5') <= 2892.4383544921875) ?  0.19932186663049375 :   -0.023449479568792453 :   (b('l8b4_med') <= 2060.2488403320312) ?  (b('dg0_v_vv') <= 3.3224406242370605) ?  0.03888931838288652 :   0.08438346151490615 :   (b('l8b5') <= 3405.0804443359375) ?  0.018858025701226057 :   -0.021359995873763478 :   (b('l8b11') <= 2384.50830078125) ?  (b('bulk') <= 139.5) ?  (b('l8b11') <= 2317.327880859375) ?  0.09956131378208499 :   0.10901342913316323 :   (b('ndvi_med') <= 2868.5) ?  0.13953635375004306 :   0.16445913397743506 :   (b('bulk') <= 135.5) ?  (b('l8b11_med') <= 1912.1572265625) ?  0.018067335755704055 :   -0.03743752042968504 :   (b('l8b11') <= 2517.0809326171875) ?  -0.004987333978559365 :   0.10242144940298226 :   (b('l8b11_med') <= 2427.9947509765625) ?  (b('l8b5') <= 2489.0257568359375) ?  (b('dg0_v_vv') <= 2.962953805923462) ?  (b('dg0_v_vv') <= 1.7036032676696777) ?  0.01639517733738577 :   (b('dg0_v_vv') <= 2.5013463497161865) ?  0.0027748843251097707 :   0.005845890960095634 :   (b('ndvi') <= 1545.5) ?  (b('moss') <= 4.646369904279709) ?  -0.0695988725315469 :   -0.060792979039922646 :   -0.027865074381351718 :   (b('bulk') <= 154.5) ?  (b('l8b4_med') <= 1303.0054931640625) ?  (b('grass') <= 0.41066667437553406) ?  0.16301299082268061 :   0.10907207060934317 :   (b('l8b5') <= 3543.8658447265625) ?  0.0583134950984388 :   -0.01697322393294151 :   (b('l8b11_med') <= 2002.0396118164062) ?  -0.06872860360251806 :   -0.08883012610802565 :   (b('grass') <= 1.900430142879486) ?  (b('ndvi_med') <= 1678.75) ?  (b('ndvi') <= 1164.0) ?  (b('ndvi') <= 1106.5) ?  0.023283916389662793 :   -0.052771559758386974 :   (b('ndvi') <= 1260.5) ?  0.0825326150411835 :   0.03530855881370501 :   (b('l8b4') <= 1781.0494384765625) ?  (b('dg0_v_vv') <= 1.871337890625) ?  -0.07870421735422056 :   -0.10478444709206784 :   (b('ndvi') <= 1888.0) ?  -0.00811211000689517 :   0.003271870024609491 :   (b('dg0_v_vv') <= 3.1260650157928467) ?  (b('l8b11') <= 2438.413330078125) ?  (b('l8b11') <= 2265.3204345703125) ?  -0.03760820281905314 :   -0.06606060033532417 :   (b('l8b4') <= 1922.0938110351562) ?  -0.01770857073451717 :   -0.04804118695963531 :   (b('l8b11') <= 2646.1943359375) ?  (b('crops') <= 86.48950958251953) ?  -0.05771057665188957 :   -0.005744543999463066 :   (b('ndvi') <= 1473.0) ?  -0.031434787514661625 :   0.02755264915217256 :   (b('ndvi_med') <= 1568.5) ?  (b('moss') <= 11.336488246917725) ?  (b('dg0_v_vv') <= 1.3374433517456055) ?  (b('dg0_v_vv') <= -0.16148042678833008) ?  (b('l8b5') <= 3961.3912353515625) ?  (b('l8b11') <= 3294.5179443359375) ?  (b('dg0_v_vv') <= -1.7690000534057617) ?  -0.05620498616295122 :   -0.06262264435644789 :   (b('dg0_v_vv') <= -1.0772705078125) ?  -0.060635650420647315 :   -0.036593780872647544 :   -0.02526944713780173 :   (b('dg0_v_vv') <= 0.6829452514648438) ?  (b('l8b4') <= 2436.899169921875) ?  (b('l8b5') <= 2916.5810546875) ?  -0.037177195935786965 :   -0.0563705429852035 :   (b('dg0_v_vv') <= 0.6036195755004883) ?  -0.030701966220364352 :   -0.027400911732655922 :   (b('l8b4') <= 1699.4860229492188) ?  (b('l8b5') <= 3462.8228759765625) ?  -0.06812470571621923 :   -0.08082571966353189 :   (b('l8b11') <= 3072.552734375) ?  -0.05056424058220388 :   -0.0644108384688584 :   (b('l8b11') <= 2725.757568359375) ?  (b('dg0_v_vv') <= 2.736262321472168) ?  (b('ndvi') <= 1000.0) ?  -0.07707059521441832 :   (b('l8b4') <= 1512.8534545898438) ?  -0.05275073844701414 :   -0.06470653686830172 :   (b('l8b5_med') <= 3359.630615234375) ?  (b('ndvi') <= 2112.0) ?  -0.022949697889162443 :   -0.012934408826849472 :   (b('dg0_v_vv') <= 3.837535858154297) ?  -0.05767775787175826 :   -0.03685964840908359 :   (b('l8b11') <= 3140.4788818359375) ?  (b('l8b5_med') <= 3359.630615234375) ?  (b('l8b4') <= 2010.4581909179688) ?  -0.022163049373345043 :   -0.03718582542571458 :   (b('ndvi') <= 2233.5) ?  -0.015256262539023147 :   -0.0037178509076240865 :   (b('dg0_v_vv') <= 1.8641867637634277) ?  -0.013773029696624112 :   (b('l8b4') <= 2405.9683837890625) ?  -0.037759476697276174 :   -0.05814842134572899 :   (b('ndvi_med') <= 1351.75) ?  (b('dg0_v_vv') <= 1.7920925617218018) ?  (b('l8b5') <= 2040.6199951171875) ?  (b('l8b4_med') <= 1310.5563354492188) ?  (b('ndvi_med') <= 805.25) ?  -0.014164379764164093 :   -0.07233919368659995 :   (b('dg0_v_vv') <= 0.5298552513122559) ?  -0.008800359964302232 :   0.06631860404133559 :   (b('l8b11_med') <= 2731.180908203125) ?  (b('ndvi') <= 1199.5) ?  -0.04670972934310511 :   -0.02512634024034269 :   (b('l8b11') <= 2672.194580078125) ?  0.009774955945511315 :   -0.028928942022731448 :   (b('ndvi') <= 806.0) ?  0.1729343955514452 :   (b('moss') <= 46.925031661987305) ?  (b('dg0_v_vv') <= 2.8190581798553467) ?  0.015074113035224037 :   -0.018309005430813367 :   (b('ndvi') <= 1457.5) ?  -0.07217406083014692 :   -0.0621896062516233 :   (b('l8b4') <= 1538.3041381835938) ?  (b('l8b5') <= 2374.1441650390625) ?  (b('l8b4') <= 1417.951171875) ?  (b('grass') <= 58.784751892089844) ?  -0.008406941961088188 :   0.07843845424175958 :   (b('l8b5') <= 2050.48486328125) ?  0.08701183432066013 :   -0.019629648129917895 :   (b('l8b5_med') <= 2358.859375) ?  (b('ndvi_med') <= 1515.5) ?  0.11076156650971716 :   0.008843434331725464 :   (b('grass') <= 65.67251586914062) ?  -0.01574545662193677 :   0.04794457361086007 :   (b('crops') <= 59.06916427612305) ?  (b('dg0_v_vv') <= 1.115966796875) ?  (b('l8b5') <= 2927.041015625) ?  -0.03077224165643581 :   -0.01889517243870079 :   (b('l8b4_med') <= 1827.1918334960938) ?  0.04499142088760995 :   -0.004643948962655134 :   (b('ndvi') <= 1219.0) ?  (b('ndvi') <= 1213.5) ?  0.009180323053769143 :   0.06913792697259871 :   (b('l8b5') <= 2637.677734375) ?  -0.02438804851579356 :   -2.834235390643977e-05 :   (b('sand') <= 47.0) ?  (b('ndvi') <= 2916.0) ?  (b('dg0_v_vv') <= 4.66080904006958) ?  (b('moss') <= 15.805471897125244) ?  (b('l8b4') <= 1416.302001953125) ?  (b('moss') <= 10.552303314208984) ?  -0.006930812034794051 :   -0.0493986079157773 :   (b('l8b5') <= 3058.159423828125) ?  -0.0037430346395119587 :   -0.0381935560198554 :   (b('ndvi') <= 2109.0) ?  (b('l8b11') <= 3110.629638671875) ?  -0.040449775717265456 :   -0.05997156131881235 :   (b('l8b11') <= 3300.0927734375) ?  -0.0367124661212386 :   0.0038745854572508403 :   0.12778939364234213 :   (b('ndvi') <= 3172.0) ?  (b('ndvi') <= 3136.0) ?  (b('l8b11') <= 1969.4210815429688) ?  0.07041722041054502 :   (b('dg0_v_vv') <= 0.2417764663696289) ?  0.012153446066296363 :   0.03630694044255763 :   0.1358391835341255 :   (b('ndvi') <= 3642.0) ?  (b('l8b4') <= 1342.5259399414062) ?  (b('crops') <= 37.8488073348999) ?  -0.008380558114901071 :   0.07028778165056401 :   (b('l8b11') <= 3219.234375) ?  -0.026977062792125275 :   0.03357534493587333 :   (b('dg0_v_vv') <= 0.748314380645752) ?  (b('ndvi') <= 3828.5) ?  0.019310295359100657 :   -0.01672223227872573 :   0.12172698925048844 :   (b('sand') <= 59.5) ?  (b('l8b11') <= 2143.916259765625) ?  (b('l8b4') <= 1420.04931640625) ?  (b('ndvi') <= 1866.0) ?  -0.08790134002910799 :   -0.08604383979283999 :   (b('l8b4') <= 1684.4205932617188) ?  (b('l8b4_med') <= 1179.6764526367188) ?  0.04965959497809004 :   0.022581045330995822 :   -0.024002911035216387 :   (b('ndvi_med') <= 1588.5) ?  (b('dg0_v_vv') <= 0.10228633880615234) ?  (b('ndvi') <= 1759.5) ?  -0.044136818938182526 :   -0.0935853909537137 :   (b('ndvi') <= 1526.0) ?  0.0223192296467397 :   -0.025634051190650237 :   (b('l8b5') <= 3627.1275634765625) ?  (b('l8b5') <= 3570.4554443359375) ?  -0.054560548776968545 :   -0.04095175319576926 :   (b('crops') <= 20.952744483947754) ?  -0.061676961311984434 :   -0.05474240977435079 :   (b('dg0_v_vv') <= 0.4332752227783203) ?  (b('ndvi') <= 1719.5) ?  (b('dg0_v_vv') <= -0.6063408851623535) ?  (b('l8b5_med') <= 3020.435791015625) ?  -0.01659041859510052 :   -0.032195506461181286 :   (b('ndvi') <= 1523.0) ?  -0.03888411009787529 :   -0.050800812775793895 :   (b('dg0_v_vv') <= 0.2182145118713379) ?  (b('ndvi') <= 1768.5) ?  0.01735393160004465 :   -0.01496185960849549 :   (b('ndvi') <= 1849.5) ?  -0.03888310911081847 :   -0.06691080700071733 :   (b('dg0_v_vv') <= 0.45774078369140625) ?  0.05274646155475153 :   (b('l8b5') <= 2247.2535400390625) ?  -0.06391948315568707 :   (b('ndvi_med') <= 1607.5) ?  0.009108753917432955 :   -0.0206982993019966 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1756.63623046875) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 35.5) ?  (b('l8b4') <= 688.0215454101562) ?  (b('l8b5_med') <= 2482.2919921875) ?  (b('l8b4') <= 580.3703308105469) ?  (b('l8b11') <= 1159.826416015625) ?  (b('ndvi') <= 3199.5) ?  (b('l8b11') <= 843.0230407714844) ?  0.057143450447748635 :   0.005218408377004294 :   (b('l8b11') <= 1063.29736328125) ?  0.0318606823753507 :   0.08952022055682973 :   (b('l8b4') <= 545.7338562011719) ?  (b('l8b11') <= 1180.5170288085938) ?  -0.11021864649687099 :   -0.08758019315548687 :   (b('dg0_v_vv') <= 0.9181256294250488) ?  -0.002756713890623666 :   0.0351158737644896 :   (b('l8b4') <= 657.126708984375) ?  (b('dg0_v_vv') <= 0.545250415802002) ?  (b('crops') <= 3.2411575317382812) ?  -0.0020230880569395565 :   -0.08301011001575069 :   (b('dg0_v_vv') <= 0.7688164710998535) ?  0.0824712048242874 :   -0.005784643990700784 :   (b('ndvi') <= 3385.0) ?  0.07094454452557064 :   0.1126328015491902 :   (b('ndvi') <= 3695.0) ?  (b('dg0_v_vv') <= -1.1541147232055664) ?  (b('l8b4') <= 468.29310607910156) ?  -0.060584948246040826 :   (b('l8b4') <= 525.0195617675781) ?  0.005132175629907123 :   0.05114275661853532 :   (b('l8b11') <= 1157.9054565429688) ?  (b('l8b5_med') <= 2654.993408203125) ?  -0.012075255880083041 :   0.08230818243136823 :   (b('ndvi_med') <= 3961.5) ?  0.14065854328252794 :   0.07408848829586655 :   (b('l8b5') <= 4847.33154296875) ?  (b('dg0_v_vv') <= -0.03124523162841797) ?  (b('sand') <= 34.0) ?  0.035205156000988974 :   0.000618615335150743 :   (b('l8b11') <= 915.1075134277344) ?  0.10301477179648945 :   0.04831534367085588 :   (b('l8b5') <= 5101.5966796875) ?  (b('dg0_v_vv') <= -0.5173344612121582) ?  0.10128131151370035 :   0.16420004681995012 :   (b('ndvi_med') <= 4246.75) ?  0.101646681434627 :   0.08142458369747659 :   (b('moss') <= 6.94382119178772) ?  (b('ndvi') <= 3996.0) ?  (b('l8b5_med') <= 3552.709228515625) ?  (b('ndvi') <= 3351.5) ?  (b('dg0_v_vv') <= 0.5055680274963379) ?  0.06313674068815509 :   0.017870796343154444 :   (b('l8b4_med') <= 585.3415832519531) ?  0.08993740803964939 :   0.13070121608227667 :   (b('l8b5_med') <= 3828.1124267578125) ?  (b('crops') <= 53.75650405883789) ?  0.13725196136820633 :   0.1754468057177131 :   (b('ndvi') <= 2430.5) ?  0.11904341263613594 :   0.08140851136861793 :   (b('l8b11') <= 1418.590087890625) ?  (b('l8b4') <= 790.0234985351562) ?  0.08367253290797252 :   0.12859395821602493 :   (b('l8b11') <= 1494.3541870117188) ?  (b('ndvi') <= 4491.5) ?  0.005189668917487317 :   0.03725044620215737 :   (b('dg0_v_vv') <= 0.10745811462402344) ?  -0.051188689212926436 :   -0.042464933858592985 :   (b('l8b4') <= 889.3902893066406) ?  (b('dg0_v_vv') <= 1.9385967254638672) ?  (b('l8b11') <= 1206.848876953125) ?  (b('ndvi') <= 2301.0) ?  -0.02659560187469224 :   0.036815165615700134 :   (b('l8b4') <= 878.0329895019531) ?  -0.00514653388713325 :   0.09184399126351808 :   (b('l8b11_med') <= 1178.7727661132812) ?  0.05782127777649693 :   0.0988958463283402 :   (b('l8b5') <= 2800.0650634765625) ?  (b('grass') <= 35.03351974487305) ?  (b('l8b4') <= 1061.0851440429688) ?  -0.048762220071240314 :   -0.06993136296924116 :   (b('dg0_v_vv') <= 0.6864027976989746) ?  0.004827782621656257 :   0.0007141668847652216 :   (b('dg0_v_vv') <= -0.4979071617126465) ?  0.06458108118910952 :   (b('dg0_v_vv') <= 0.46820497512817383) ?  -0.03641892605285482 :   0.023377937958224765 :   (b('l8b5_med') <= 2962.950439453125) ?  (b('l8b4') <= 839.5362854003906) ?  (b('ndvi') <= 4916.5) ?  (b('l8b4') <= 649.6515808105469) ?  (b('l8b4_med') <= 598.2109985351562) ?  (b('l8b11') <= 1116.3103637695312) ?  -0.017323473263102126 :   0.008096453127461961 :   (b('ndvi') <= 4781.0) ?  0.022833277388153616 :   0.1403318445314999 :   (b('l8b5_med') <= 2927.793212890625) ?  (b('ndvi') <= 1637.5) ?  -0.003182802737710952 :   -0.05001417899606575 :   (b('dg0_v_vv') <= -0.18362140655517578) ?  -0.00529595202431285 :   0.1163338349316233 :   (b('ndvi') <= 4950.5) ?  (b('ndvi_med') <= 3972.25) ?  -0.1374098332479484 :   (b('dg0_v_vv') <= 1.759988784790039) ?  -0.12058536715508814 :   -0.12758536737115497 :   (b('dg0_v_vv') <= -0.6748523712158203) ?  (b('crops') <= 40.07795333862305) ?  -0.04735408213278533 :   -0.05494184015122572 :   (b('dg0_v_vv') <= 0.29697513580322266) ?  -0.03293833817478216 :   -0.0352871851271999 :   (b('ndvi_med') <= 3855.0) ?  (b('l8b4_med') <= 609.0743103027344) ?  (b('l8b11_med') <= 1253.0780639648438) ?  0.07773046391042512 :   0.06071186229390234 :   (b('dg0_v_vv') <= -0.26943159103393555) ?  -0.07787169555204028 :   -0.037171695772577465 :   0.1918453559820179 :   (b('ndvi_med') <= 4430.0) ?  (b('l8b5') <= 5077.538818359375) ?  (b('dg0_v_vv') <= 1.0889458656311035) ?  (b('l8b11') <= 1123.1968994140625) ?  (b('grass') <= 23.161325454711914) ?  0.0362646733178636 :   0.014775144272301931 :   (b('moss') <= 4.994444370269775) ?  -0.004494175375112784 :   0.041021186705617434 :   (b('l8b5') <= 3533.6507568359375) ?  (b('l8b11') <= 1031.932861328125) ?  0.031776937779956715 :   0.0804918415002559 :   (b('l8b4_med') <= 462.4842834472656) ?  -0.05496908954580014 :   0.03116685347188111 :   (b('ndvi') <= 3805.5) ?  (b('dg0_v_vv') <= 3.9234039783477783) ?  (b('dg0_v_vv') <= 3.6462883949279785) ?  -0.014303570721214659 :   -0.028303561840122587 :   (b('dg0_v_vv') <= 4.136626720428467) ?  0.018696436550552004 :   0.010696435239249819 :   (b('l8b4') <= 362.009765625) ?  (b('l8b5') <= 5776.425537109375) ?  -0.043459905072669225 :   -0.025975150948975814 :   (b('l8b11_med') <= 1115.5166625976562) ?  -0.05553805967200348 :   -0.04862166125885134 :   (b('l8b11') <= 1108.5331420898438) ?  (b('dg0_v_vv') <= 0.6406669616699219) ?  (b('ndvi_med') <= 4607.5) ?  (b('dg0_v_vv') <= -0.992304801940918) ?  0.07092775358994241 :   0.13692790070359018 :   (b('ndvi') <= 3049.0) ?  0.17081826916023252 :   0.04245465494175981 :   (b('ndvi') <= 3623.5) ?  (b('l8b4_med') <= 630.9979553222656) ?  0.1287871170216704 :   0.15173522102152628 :   (b('l8b4') <= 417.38026428222656) ?  0.15652355466656875 :   0.17077796747134355 :   (b('l8b4') <= 707.109619140625) ?  (b('l8b5') <= 3091.146240234375) ?  -0.10522299003467109 :   (b('l8b11_med') <= 1198.20263671875) ?  -0.012279598297267828 :   0.018774795769926474 :   (b('dg0_v_vv') <= -1.2602524757385254) ?  (b('dg0_v_vv') <= -1.496300220489502) ?  0.0672876082616477 :   0.09508207205262709 :   (b('l8b4') <= 754.5669250488281) ?  0.22655771497786537 :   0.17764782324438977 :   (b('crops') <= 9.933524370193481) ?  (b('l8b11') <= 958.7872924804688) ?  (b('sand') <= 52.5) ?  (b('l8b4') <= 436.92864990234375) ?  (b('l8b5_med') <= 2308.12939453125) ?  0.05833361248011423 :   (b('ndvi') <= 3621.0) ?  -0.01268843797002489 :   (b('l8b11_med') <= 1071.2799072265625) ?  0.011569476965641812 :   0.025319660915180786 :   (b('ndvi_med') <= 3203.75) ?  (b('l8b4') <= 581.1234436035156) ?  -0.05789394871608458 :   -0.07924051261207848 :   -0.03618773251893748 :   (b('l8b5') <= 1863.9479370117188) ?  (b('ndvi_med') <= 3709.25) ?  (b('l8b4_med') <= 522.1105041503906) ?  (b('ndvi') <= 3821.0) ?  0.07898862582380242 :   0.04318106060851688 :   (b('dg0_v_vv') <= -2.5135726928710938) ?  0.08972603909732654 :   0.020623968179528693 :   (b('dg0_v_vv') <= -0.23407888412475586) ?  (b('dg0_v_vv') <= -0.29335546493530273) ?  0.12068339793375693 :   0.2238481311066619 :   (b('l8b4') <= 367.0625305175781) ?  0.1332503048013847 :   0.05730012127189816 :   (b('dg0_v_vv') <= 0.9269256591796875) ?  (b('l8b5') <= 2194.4620361328125) ?  (b('ndvi_med') <= 3520.75) ?  0.1389038617425224 :   0.1225527242848042 :   (b('ndvi_med') <= 3625.75) ?  0.10137212095068646 :   0.07159126800264495 :   (b('dg0_v_vv') <= 1.0339317321777344) ?  0.17853275922800949 :   (b('ndvi') <= 4492.0) ?  0.12059021337014224 :   0.1483670504059131 :   (b('sand') <= 76.5) ?  (b('ndvi_med') <= 3182.0) ?  (b('l8b4') <= 537.7270202636719) ?  (b('moss') <= 11.153125286102295) ?  0.021834745125027838 :   0.08041659641647425 :   (b('l8b5') <= 2091.6347045898438) ?  (b('moss') <= 7.0015058517456055) ?  -0.10121197828706238 :   -0.06772505647762675 :   (b('l8b5_med') <= 2280.4501953125) ?  -0.06275423208638292 :   -0.03681751899820712 :   (b('l8b4') <= 702.8229370117188) ?  (b('moss') <= 13.496818542480469) ?  (b('l8b4') <= 446.27130126953125) ?  0.127878741539858 :   0.02152757465391848 :   (b('l8b5') <= 3001.1319580078125) ?  -0.042098036011317896 :   -0.0016717254632746612 :   (b('dg0_v_vv') <= -1.3634610176086426) ?  (b('sand') <= 57.5) ?  -0.0058846581248737095 :   -0.05408009627999665 :   (b('l8b5') <= 2509.8441162109375) ?  0.0868469728560026 :   0.05398759693780194 :   (b('l8b5') <= 2842.8968505859375) ?  (b('l8b4') <= 726.2369079589844) ?  -0.09030760569195218 :   -0.1131810604315227 :   (b('ndvi') <= 3727.5) ?  -0.1413944662962091 :   -0.13670023570869452 :   (b('l8b5') <= 2490.3359375) ?  (b('dg0_v_vv') <= -2.049837112426758) ?  (b('ndvi') <= 3351.5) ?  (b('dg0_v_vv') <= -2.3700757026672363) ?  0.07049061140564311 :   0.025040302410546944 :   (b('ndvi') <= 3490.0) ?  0.011980462213330295 :   (b('dg0_v_vv') <= -2.455948829650879) ?  (b('dg0_v_vv') <= -5.165910720825195) ?  -0.007438973580119815 :   -0.022136623861572363 :   (b('ndvi_med') <= 3442.5) ?  -0.01091381662338492 :   -0.006978545308190309 :   (b('l8b11') <= 1403.3554077148438) ?  (b('l8b4') <= 457.4508056640625) ?  (b('ndvi_med') <= 3438.5) ?  (b('grass') <= 48.067684173583984) ?  0.117570008777924 :   0.161328970786764 :   (b('l8b4_med') <= 585.5948486328125) ?  0.10274683162541136 :   0.06318276325835959 :   (b('dg0_v_vv') <= -0.4263801574707031) ?  (b('l8b5') <= 2411.2080078125) ?  0.03177208335965329 :   -0.012024080787542968 :   (b('ndvi') <= 3523.5) ?  0.07314281872097521 :   0.014387680419242294 :   (b('dg0_v_vv') <= -0.6049928665161133) ?  (b('ndvi_med') <= 3259.75) ?  0.09725901367334305 :   (b('dg0_v_vv') <= -0.9162955284118652) ?  0.1327594345482044 :   0.15503489412821758 :   (b('ndvi') <= 4735.0) ?  (b('ndvi') <= 4504.5) ?  0.09540519812908146 :   0.12895397902376643 :   0.032314343290706427 :   (b('l8b4') <= 485.2424774169922) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('ndvi') <= 4931.0) ?  (b('l8b11_med') <= 1348.42724609375) ?  (b('l8b11_med') <= 1212.746337890625) ?  0.13918064702498764 :   0.12922076818137274 :   0.15888430171368895 :   (b('l8b5') <= 3361.4019775390625) ?  (b('dg0_v_vv') <= 2.6869654655456543) ?  0.1489227049250079 :   0.16446105247702558 :   0.18325037698529223 :   (b('sand') <= 51.5) ?  (b('grass') <= 59.88348388671875) ?  (b('dg0_v_vv') <= -0.8103389739990234) ?  0.16611063348611105 :   0.17616877171958295 :   (b('dg0_v_vv') <= 0.4085807800292969) ?  0.13842916116418183 :   0.16737019290376723 :   0.23912622394157967 :   (b('l8b5') <= 3589.2022705078125) ?  (b('ndvi_med') <= 3948.5) ?  (b('ndvi') <= 2806.0) ?  (b('l8b5') <= 2836.0579833984375) ?  0.12919672326638582 :   0.17253390521589732 :   (b('l8b4') <= 773.0056762695312) ?  0.11607066527378448 :   0.04040865842535033 :   (b('sand') <= 50.5) ?  (b('dg0_v_vv') <= 1.1184616088867188) ?  -0.011255629153434735 :   -0.029278792616893617 :   (b('l8b4') <= 575.9626770019531) ?  0.11370844061636039 :   0.09779908031963964 :   (b('crops') <= 13.149437427520752) ?  (b('dg0_v_vv') <= -3.0703907012939453) ?  -0.02796821605601174 :   (b('ndvi') <= 4251.5) ?  0.08807079529570996 :   0.04393990654648952 :   -0.07582774089543035 :   (b('bulk') <= 137.5) ?  (b('moss') <= 6.460667610168457) ?  (b('crops') <= 92.60075759887695) ?  (b('dg0_v_vv') <= -0.4847736358642578) ?  (b('l8b5') <= 3097.281494140625) ?  (b('l8b11_med') <= 1837.4970092773438) ?  (b('l8b5_med') <= 3359.4439697265625) ?  (b('l8b11') <= 931.7568664550781) ?  0.026497400627139775 :   -0.010833413994565865 :   (b('l8b5_med') <= 3382.257568359375) ?  -0.09552000810055018 :   -0.05418082925007686 :   (b('l8b4_med') <= 1141.1287841796875) ?  (b('l8b5') <= 1821.885498046875) ?  -0.03281881692549669 :   -0.0647641619500312 :   0.05673344707455438 :   (b('ndvi') <= 1844.0) ?  (b('dg0_v_vv') <= -2.305248737335205) ?  0.10787961922922296 :   (b('dg0_v_vv') <= -1.5887274742126465) ?  0.08168918877569059 :   0.10157499128970235 :   (b('crops') <= 83.83524322509766) ?  (b('l8b5') <= 4391.015380859375) ?  0.019825089156143765 :   0.09167530688561795 :   (b('l8b11_med') <= 1220.2855834960938) ?  0.05633245242071294 :   -0.018968964235040693 :   (b('grass') <= 8.456110954284668) ?  (b('l8b11') <= 1538.1957397460938) ?  (b('ndvi') <= 2491.5) ?  (b('moss') <= 2.703668713569641) ?  0.06112397334124734 :   0.010258916151616167 :   (b('l8b5') <= 3363.351318359375) ?  0.029329817596375395 :   0.006792085526452226 :   (b('l8b5_med') <= 2942.0115966796875) ?  (b('l8b5') <= 2616.8179931640625) ?  -0.015802766093913713 :   0.010037887172892921 :   (b('l8b5') <= 3214.4698486328125) ?  -0.05437449126307672 :   -0.004927714393503289 :   (b('ndvi') <= 4603.0) ?  (b('l8b4_med') <= 725.7943725585938) ?  (b('dg0_v_vv') <= -0.08287858963012695) ?  0.06570442511474854 :   -0.021637102752586432 :   (b('ndvi') <= 2669.5) ?  0.08631281124784872 :   0.035950128157796954 :   (b('moss') <= 0.6213347911834717) ?  (b('dg0_v_vv') <= 0.28251218795776367) ?  0.04124001521082926 :   0.0009933505585007958 :   (b('l8b5_med') <= 3095.4654541015625) ?  -0.08030947084791303 :   0.025910427880273326 :   (b('l8b4_med') <= 723.2454223632812) ?  -0.01569330970912816 :   (b('l8b5') <= 1445.4461669921875) ?  (b('ndvi') <= 1285.5) ?  (b('dg0_v_vv') <= 0.3658313751220703) ?  (b('dg0_v_vv') <= -1.0473637580871582) ?  0.11201117260672999 :   0.11101115567901088 :   0.09360084003990454 :   (b('l8b4_med') <= 843.4451293945312) ?  0.08555665369864543 :   (b('ndvi_med') <= 1531.25) ?  0.01785758071071175 :   0.045166226612943144 :   (b('dg0_v_vv') <= 4.345206260681152) ?  (b('grass') <= 2.184100389480591) ?  (b('dg0_v_vv') <= -0.8359527587890625) ?  0.12268356254609028 :   0.14628615883427729 :   (b('ndvi') <= 1946.5) ?  0.10971606908003656 :   0.11634777714780192 :   (b('crops') <= 97.81589889526367) ?  0.09945060744860579 :   0.06084655244704029 :   (b('ndvi_med') <= 3406.5) ?  (b('bulk') <= 128.0) ?  (b('ndvi') <= 1058.0) ?  (b('dg0_v_vv') <= -0.96295166015625) ?  0.04203352971550596 :   (b('l8b4') <= 921.5459899902344) ?  0.0514735901875272 :   0.05503354115959776 :   (b('l8b4') <= 728.8764343261719) ?  (b('l8b11') <= 1219.3206787109375) ?  (b('l8b4') <= 622.568603515625) ?  -0.022160994425181938 :   -0.06388556231061256 :   (b('moss') <= 18.676668643951416) ?  -0.0076648162983273555 :   0.07356572592998312 :   (b('l8b11') <= 1341.9823608398438) ?  (b('l8b11') <= 1158.8320922851562) ?  -0.03881060256777063 :   -0.07078091677242147 :   (b('l8b5_med') <= 2437.6734619140625) ?  -0.052991810543894985 :   -0.013553208196372039 :   (b('l8b4') <= 551.8664855957031) ?  (b('ndvi_med') <= 1841.5) ?  (b('dg0_v_vv') <= -0.5289568901062012) ?  0.13392771724111197 :   0.036318130962816025 :   (b('dg0_v_vv') <= -0.03522968292236328) ?  (b('grass') <= 16.96821689605713) ?  -0.025328559490152763 :   0.004876350554939531 :   (b('dg0_v_vv') <= -0.027335166931152344) ?  0.1535917125195452 :   0.017973974689168767 :   (b('dg0_v_vv') <= -0.05087089538574219) ?  (b('grass') <= 16.96821689605713) ?  (b('ndvi') <= 3382.0) ?  -0.047479780423565376 :   -0.01473565039556148 :   (b('l8b5') <= 2818.356201171875) ?  -0.02690209600306643 :   -0.010167450898157826 :   (b('grass') <= 56.6933650970459) ?  (b('l8b11') <= 1030.0537109375) ?  -0.06130366136927046 :   -0.011488617413002842 :   (b('moss') <= 8.884375095367432) ?  0.17328010495020016 :   0.01732047574713502 :   (b('l8b4') <= 917.9808959960938) ?  (b('sand') <= 51.5) ?  (b('dg0_v_vv') <= 2.275409698486328) ?  (b('l8b4_med') <= 798.931640625) ?  (b('dg0_v_vv') <= 0.9409642219543457) ?  0.07185213800085383 :   -0.1077294977942169 :   (b('ndvi') <= 2378.0) ?  0.04771832205711993 :   0.01243277969348705 :   (b('bulk') <= 119.0) ?  0.19774552337816198 :   (b('l8b11_med') <= 1448.5018920898438) ?  0.07893692079212136 :   0.0021266893043326205 :   (b('l8b4') <= 885.683837890625) ?  (b('l8b5') <= 1714.9334106445312) ?  (b('ndvi') <= 3467.5) ?  0.08083833516412164 :   0.06724683491662514 :   (b('grass') <= 32.49230194091797) ?  -0.010254360525719082 :   -0.05626841111463715 :   0.15664273753609678 :   (b('l8b11') <= 1709.3108520507812) ?  (b('ndvi_med') <= 4323.0) ?  (b('dg0_v_vv') <= -2.9269070625305176) ?  (b('l8b11_med') <= 1698.3795776367188) ?  -0.06391377293005199 :   -0.04043696882710471 :   (b('l8b4') <= 960.1868286132812) ?  -0.02331632183860744 :   -0.04122350101670347 :   (b('dg0_v_vv') <= -2.472285270690918) ?  -0.061436968543982645 :   -0.07911743432193552 :   (b('l8b11_med') <= 1509.7184448242188) ?  0.08484987286096543 :   0.06684988108640641 :   (b('dg0_v_vv') <= 0.8203530311584473) ?  (b('sand') <= 28.0) ?  (b('moss') <= 4.076430082321167) ?  (b('dg0_v_vv') <= -1.0265154838562012) ?  (b('l8b5') <= 4527.396240234375) ?  (b('l8b11') <= 1371.5348510742188) ?  (b('ndvi') <= 4910.5) ?  -0.016953750184165742 :   0.0828176096835016 :   (b('l8b11') <= 1472.638671875) ?  0.12476556256521117 :   0.029936322402384176 :   (b('dg0_v_vv') <= -4.6602888107299805) ?  0.06107326979353864 :   (b('dg0_v_vv') <= -2.6576266288757324) ?  0.15861164927007765 :   0.08173083726253634 :   (b('l8b4_med') <= 859.3739318847656) ?  (b('l8b11') <= 1585.397705078125) ?  (b('l8b4') <= 569.447998046875) ?  0.11895691092869934 :   0.07178017523661617 :   (b('l8b5') <= 2482.41357421875) ?  0.09526331954569996 :   0.1536560111031049 :   (b('l8b4') <= 1073.6870727539062) ?  (b('l8b4') <= 749.2815246582031) ?  0.05394629153831298 :   0.009518421286273249 :   (b('ndvi') <= 991.5) ?  0.0573293828924934 :   0.11413602468223093 :   (b('ndvi') <= 3483.0) ?  (b('l8b11') <= 1731.347412109375) ?  (b('l8b4_med') <= 749.7178344726562) ?  (b('dg0_v_vv') <= -1.5664772987365723) ?  -0.07300385351617325 :   -0.08264383871507783 :   (b('l8b4') <= 883.2559204101562) ?  0.012196811600195695 :   -0.018829791528754695 :   (b('l8b11_med') <= 1551.15087890625) ?  0.060597561074666406 :   0.10437241800788463 :   (b('l8b4') <= 788.9912414550781) ?  (b('l8b5') <= 2383.8009033203125) ?  (b('ndvi') <= 3644.5) ?  0.010332779042164527 :   0.04337654769639536 :   (b('dg0_v_vv') <= -0.22356557846069336) ?  0.051645635580739334 :   0.10082944964143858 :   (b('ndvi') <= 3588.0) ?  (b('dg0_v_vv') <= -0.47748374938964844) ?  0.024902048474496397 :   0.08142469204556123 :   (b('l8b4') <= 814.3976440429688) ?  -0.07002748133466548 :   0.011077685856559254 :   (b('ndvi') <= 2641.0) ?  (b('l8b5') <= 3813.2708740234375) ?  (b('l8b4') <= 653.7662963867188) ?  (b('grass') <= 67.24948501586914) ?  (b('l8b4') <= 645.2193603515625) ?  0.011857720729918813 :   0.12490401973203154 :   (b('crops') <= 10.701711654663086) ?  0.12816184091963337 :   -0.07731257871528296 :   (b('l8b5_med') <= 2613.8431396484375) ?  (b('l8b5') <= 2278.7554931640625) ?  -0.013887428028156287 :   -0.04433930278315157 :   (b('l8b5') <= 1712.7002563476562) ?  0.06028089182650741 :   0.00291235190945729 :   (b('l8b5') <= 4472.28369140625) ?  (b('l8b4') <= 1055.8176879882812) ?  (b('l8b5') <= 4226.907470703125) ?  0.1273117689458616 :   0.11533126332436958 :   0.10526126806110334 :   0.21714423128131632 :   (b('l8b11_med') <= 1965.692626953125) ?  (b('l8b11') <= 1077.5028076171875) ?  (b('l8b11_med') <= 1531.8209228515625) ?  (b('l8b4_med') <= 685.1024169921875) ?  -0.03325844279332127 :   0.06329612342710093 :   (b('l8b4') <= 815.7243041992188) ?  0.007707941338924707 :   0.11954425099980776 :   (b('l8b5') <= 3669.0870361328125) ?  (b('sand') <= 41.5) ?  0.0049423993915903896 :   -0.02049570602516416 :   (b('ndvi_med') <= 2378.75) ?  0.20401361560659734 :   0.026178401764560225 :   (b('l8b5') <= 2630.2864990234375) ?  (b('sand') <= 52.5) ?  (b('l8b4_med') <= 1227.092529296875) ?  0.06366529590902624 :   0.12152350453871034 :   (b('l8b11_med') <= 2393.7314453125) ?  -0.06512781865393352 :   0.016027089918298997 :   (b('crops') <= 48.69628715515137) ?  (b('l8b5') <= 3305.698486328125) ?  0.03283471136526815 :   0.12365485092846878 :   (b('crops') <= 72.46576690673828) ?  -0.03377066678714628 :   0.031096840074658776 :   (b('dg0_v_vv') <= 3.1747543811798096) ?  (b('l8b11_med') <= 2728.3125) ?  (b('ndvi_med') <= 1737.5) ?  (b('moss') <= 28.092727661132812) ?  (b('l8b11') <= 889.6414794921875) ?  (b('l8b11') <= 798.6510314941406) ?  0.05410634362810823 :   -0.06513900002489964 :   (b('l8b11') <= 1256.836181640625) ?  0.1044649640899754 :   0.06514853267329448 :   (b('bulk') <= 153.0) ?  (b('l8b4') <= 1228.7866821289062) ?  0.004327457034837771 :   -0.1205004299147889 :   0.1375067553886297 :   (b('sand') <= 30.0) ?  (b('l8b11_med') <= 2078.8038330078125) ?  (b('l8b5') <= 3198.235595703125) ?  0.06243647351076926 :   0.03772159082836935 :   (b('l8b4_med') <= 1013.0897827148438) ?  0.08438574989366807 :   -0.020405720818108587 :   (b('l8b11_med') <= 1886.2213134765625) ?  (b('l8b4') <= 518.7847595214844) ?  0.0705208160249332 :   0.0029690394178351312 :   (b('moss') <= 0.4869457269087434) ?  0.0014545516329715817 :   0.05945333686421879 :   (b('l8b5') <= 2529.839599609375) ?  (b('l8b4_med') <= 1794.1739501953125) ?  (b('sand') <= 49.5) ?  (b('l8b11') <= 1457.1358642578125) ?  0.08886968153570579 :   0.06701884540877176 :   (b('moss') <= 30.395100116729736) ?  0.04080177088036248 :   0.02950811161912742 :   (b('l8b11') <= 1131.2290649414062) ?  (b('l8b5') <= 1531.071044921875) ?  -0.028261517975689432 :   -0.05579179276554219 :   (b('l8b5_med') <= 3153.067626953125) ?  -0.006355095815523299 :   0.027897006045513865 :   (b('l8b11') <= 1348.8751220703125) ?  -0.11592658191774538 :   (b('crops') <= 69.43515014648438) ?  (b('l8b5_med') <= 2950.2667236328125) ?  -0.06861866788910487 :   -0.07452245655814649 :   (b('dg0_v_vv') <= 1.138467788696289) ?  -0.08047026419247283 :   -0.02357071169161438 :   (b('l8b5') <= 3529.3551025390625) ?  (b('bulk') <= 154.5) ?  (b('ndvi') <= 4498.5) ?  (b('sand') <= 24.0) ?  (b('l8b4') <= 944.7452697753906) ?  0.15879471287981411 :   0.1341788911062617 :   (b('l8b11') <= 1666.6775512695312) ?  0.05554447763166363 :   0.09305412710420258 :   (b('l8b5_med') <= 3025.82421875) ?  (b('l8b4_med') <= 1326.1889038085938) ?  -0.03292410690233172 :   0.04741137816303978 :   (b('l8b4_med') <= 852.275390625) ?  -0.04260130674343887 :   -0.03222904284669832 :   (b('grass') <= 73.00110626220703) ?  (b('l8b11') <= 1317.75927734375) ?  (b('dg0_v_vv') <= 5.387262344360352) ?  0.1153856358169994 :   0.12538565608257862 :   (b('bulk') <= 156.0) ?  0.09343581403261303 :   0.06031963532449384 :   (b('ndvi') <= 2530.5) ?  0.05398834828663879 :   (b('l8b11') <= 1078.8698120117188) ?  0.11300143937503278 :   0.1722651708694909 :   (b('ndvi') <= 3813.0) ?  (b('l8b4_med') <= 1531.5534057617188) ?  (b('l8b11_med') <= 2319.3582763671875) ?  -0.07298150064213343 :   -0.11164112682352181 :   (b('dg0_v_vv') <= 3.7531585693359375) ?  (b('dg0_v_vv') <= 3.5366766452789307) ?  -0.024959265634583477 :   0.031915216321448114 :   (b('dg0_v_vv') <= 4.903790473937988) ?  -0.048208779824123936 :   -0.011959269091652874 :   (b('sand') <= 37.5) ?  0.07182079837068325 :   (b('l8b5_med') <= 2860.499755859375) ?  -0.002172582063851236 :   (b('ndvi') <= 4011.5) ?  0.022394484077816412 :   0.02720218404189259 :   (b('l8b11_med') <= 2649.172119140625) ?  (b('crops') <= 37.20681381225586) ?  (b('dg0_v_vv') <= 1.1941642761230469) ?  (b('l8b4') <= 1030.6765747070312) ?  (b('bulk') <= 151.5) ?  (b('moss') <= 6.610314607620239) ?  (b('l8b4_med') <= 1214.2190551757812) ?  (b('l8b11') <= 2086.565673828125) ?  (b('dg0_v_vv') <= 0.7485194206237793) ?  -0.004062575324339546 :   0.039564284207964984 :   (b('l8b11_med') <= 2268.7845458984375) ?  0.04349458740373052 :   -0.050668313837774176 :   (b('l8b11_med') <= 2150.0435791015625) ?  0.15628204894801515 :   0.09865707994210784 :   (b('l8b11_med') <= 2131.842041015625) ?  (b('l8b4_med') <= 1057.7998657226562) ?  (b('l8b11_med') <= 1384.2262573242188) ?  0.046372521301868204 :   -0.019044106507200183 :   (b('l8b5') <= 2365.5994873046875) ?  0.009841977767263841 :   0.09251846331593565 :   (b('ndvi') <= 2899.5) ?  (b('l8b11_med') <= 2496.2672119140625) ?  -0.07726703519578647 :   0.03713316663687384 :   (b('ndvi') <= 3269.5) ?  0.020626888525790074 :   0.0693680426195181 :   (b('l8b11_med') <= 1970.5457763671875) ?  (b('l8b5_med') <= 2735.3013916015625) ?  (b('dg0_v_vv') <= 0.8442330360412598) ?  (b('ndvi_med') <= 2256.75) ?  0.005042632340132783 :   -0.04496410341742062 :   (b('ndvi') <= 2393.5) ?  0.04296396310763042 :   0.13722965736938414 :   (b('l8b5_med') <= 2889.6197509765625) ?  (b('dg0_v_vv') <= 0.25821876525878906) ?  0.11329278359622218 :   0.19333645383916173 :   (b('l8b4') <= 993.3842468261719) ?  0.024931100288868042 :   0.08423878156053617 :   (b('dg0_v_vv') <= -0.9320588111877441) ?  (b('l8b5_med') <= 2569.76123046875) ?  (b('dg0_v_vv') <= -1.5509510040283203) ?  0.0957600325918985 :   0.01856828536563525 :   (b('ndvi') <= 2649.5) ?  -0.051428200870137275 :   0.007606437795576015 :   (b('sand') <= 44.0) ?  (b('dg0_v_vv') <= -0.7140040397644043) ?  0.2507748322290636 :   0.11786833166226962 :   (b('l8b5') <= 2309.8641357421875) ?  0.07447338454504616 :   -0.02338549061266361 :   (b('grass') <= 90.26663208007812) ?  (b('crops') <= 36.54450607299805) ?  (b('l8b4_med') <= 826.7156066894531) ?  (b('moss') <= 4.864360570907593) ?  (b('sand') <= 34.5) ?  0.09792487803128218 :   0.03131260295459523 :   (b('l8b4_med') <= 805.7397155761719) ?  -0.02915261293837638 :   0.034991125558861746 :   (b('bulk') <= 133.5) ?  (b('l8b11') <= 2547.2735595703125) ?  -0.041531278692291326 :   -0.0012670362129198708 :   (b('ndvi') <= 1467.5) ?  -0.02674597649885257 :   -0.01632987495776972 :   (b('l8b5') <= 2304.61669921875) ?  (b('l8b5_med') <= 2487.8336181640625) ?  (b('l8b5') <= 2251.0281982421875) ?  -0.07371745606132156 :   -0.07171745573349601 :   (b('dg0_v_vv') <= 0.5409951210021973) ?  -0.08076178602329964 :   -0.08540412950195218 :   (b('l8b4') <= 1404.7400512695312) ?  (b('dg0_v_vv') <= -1.0910534858703613) ?  -0.05310626809547723 :   -0.05903784792097979 :   (b('ndvi') <= 2453.5) ?  -0.06472270209089141 :   -0.06109767335436539 :   (b('bulk') <= 145.5) ?  (b('l8b11') <= 1857.8810424804688) ?  0.05773556526596582 :   (b('ndvi') <= 2081.5) ?  (b('l8b11_med') <= 2036.2794799804688) ?  -0.052081878506057264 :   -0.025945985509576656 :   (b('ndvi_med') <= 1887.75) ?  0.018525231017861105 :   -0.020549381050899502 :   (b('dg0_v_vv') <= 0.33675670623779297) ?  (b('sand') <= 51.5) ?  (b('sand') <= 41.5) ?  -0.0038738886160775165 :   0.02550311302127877 :   (b('dg0_v_vv') <= -0.2128314971923828) ?  -0.03711668158654544 :   -0.08077519141820651 :   (b('l8b11') <= 2765.211669921875) ?  (b('ndvi') <= 1447.5) ?  0.0985202309245674 :   0.04964770304236418 :   -0.08146951895255855 :   (b('grass') <= 41.722944259643555) ?  (b('l8b5_med') <= 2360.1275634765625) ?  (b('ndvi_med') <= 2523.5) ?  0.16663724454490905 :   0.11357294032981347 :   (b('l8b4') <= 970.9587707519531) ?  (b('l8b11') <= 1941.3433837890625) ?  (b('l8b11_med') <= 1570.8994750976562) ?  0.07697920031224692 :   0.09582595327014518 :   0.010174844852089415 :   (b('sand') <= 51.5) ?  (b('dg0_v_vv') <= 1.2568683624267578) ?  0.10015081496962325 :   (b('ndvi_med') <= 3220.5) ?  -0.012809329155748294 :   -0.04433418552188668 :   (b('l8b4') <= 2144.4844970703125) ?  (b('dg0_v_vv') <= 2.6199986934661865) ?  -0.06831853427823166 :   -0.0075501356580860435 :   0.026685220075848393 :   (b('moss') <= 26.56121253967285) ?  (b('moss') <= 22.585250854492188) ?  (b('dg0_v_vv') <= 3.0700578689575195) ?  (b('l8b11_med') <= 2310.958251953125) ?  (b('l8b5_med') <= 2604.3846435546875) ?  0.017741232843725196 :   0.05415815034981316 :   (b('ndvi') <= 2670.5) ?  -0.011803095007117898 :   -0.11625360707627637 :   (b('l8b5_med') <= 2201.696044921875) ?  (b('l8b4') <= 939.7157897949219) ?  0.06796357645671108 :   -0.02910231929242504 :   (b('l8b11_med') <= 2131.676513671875) ?  0.04065208385409957 :   0.08661479262380616 :   (b('l8b11') <= 2392.1025390625) ?  (b('bulk') <= 147.0) ?  (b('l8b11_med') <= 2064.3157958984375) ?  0.10018230882699 :   0.025983358348933262 :   (b('l8b4') <= 1159.2129516601562) ?  0.16168823928774284 :   0.21724310835919364 :   (b('l8b5') <= 1696.3011474609375) ?  -0.09693839522804687 :   (b('l8b11') <= 2614.5118408203125) ?  -0.03648442142248521 :   0.023027586500260294 :   (b('l8b11_med') <= 2311.08447265625) ?  (b('l8b4_med') <= 1071.31494140625) ?  (b('l8b5') <= 1803.3695068359375) ?  (b('l8b11') <= 1900.7877197265625) ?  0.0022691951178314546 :   -0.022678554504858256 :   (b('dg0_v_vv') <= 4.334393501281738) ?  -0.052711322438696584 :   -0.019551557859433084 :   (b('ndvi') <= 1335.5) ?  (b('sand') <= 46.0) ?  -0.056263254539536504 :   -0.003352265620984453 :   (b('dg0_v_vv') <= 2.69279146194458) ?  0.00988886737995966 :   0.059837098550175834 :   (b('l8b4_med') <= 1537.3861694335938) ?  (b('l8b4') <= 1331.047607421875) ?  (b('l8b4_med') <= 1507.4218139648438) ?  0.009890235214388365 :   0.012049076308854073 :   -0.02025266158267694 :   (b('moss') <= 33.66784191131592) ?  (b('dg0_v_vv') <= 2.346001625061035) ?  0.1672190178951554 :   0.15564095599630068 :   (b('l8b4_med') <= 1709.3284301757812) ?  0.10381535630831276 :   0.1340475956318207 :   (b('sand') <= 23.5) ?  (b('sand') <= 20.5) ?  (b('l8b11') <= 1767.937255859375) ?  (b('dg0_v_vv') <= -2.5615177154541016) ?  -0.002615577822149273 :   -0.030193331027001863 :   (b('l8b4') <= 1504.0353393554688) ?  (b('dg0_v_vv') <= 1.0512943267822266) ?  (b('l8b5') <= 2837.2484130859375) ?  (b('sand') <= 15.0) ?  0.12334285111964363 :   0.0961444424756529 :   (b('ndvi_med') <= 3451.5) ?  0.07663749989622301 :   0.11582850638101172 :   (b('ndvi') <= 3144.0) ?  (b('l8b11') <= 2062.9310913085938) ?  0.12079627581801317 :   0.11683733451963169 :   (b('dg0_v_vv') <= 2.0112357139587402) ?  0.14875595039462142 :   0.1319327291497953 :   0.2208769465793205 :   (b('l8b11') <= 2505.304443359375) ?  (b('dg0_v_vv') <= 1.591029405593872) ?  (b('ndvi') <= 4211.0) ?  (b('dg0_v_vv') <= -1.8576502799987793) ?  (b('ndvi') <= 1115.5) ?  -0.011156175742980418 :   -0.026871146925376593 :   (b('l8b5') <= 2178.821044921875) ?  0.07987750071926533 :   0.02900316988285723 :   (b('l8b4') <= 1446.400146484375) ?  (b('ndvi_med') <= 3351.25) ?  -0.05279540610230982 :   -0.07991111826065087 :   0.009552256798127129 :   (b('l8b4') <= 1574.78662109375) ?  (b('dg0_v_vv') <= 1.9636685848236084) ?  (b('l8b11') <= 2009.4659423828125) ?  0.09804287021378413 :   0.11087656679377668 :   (b('l8b5_med') <= 2925.356689453125) ?  0.024073995116166008 :   0.07831301000012808 :   -0.007027775039428863 :   (b('dg0_v_vv') <= -1.778393268585205) ?  (b('l8b11_med') <= 2095.1530151367188) ?  0.026053233668400816 :   0.05243358858658714 :   (b('l8b5_med') <= 3151.007568359375) ?  (b('ndvi') <= 1938.5) ?  (b('dg0_v_vv') <= -0.36789560317993164) ?  0.07784384070790154 :   0.10833187775861308 :   (b('dg0_v_vv') <= 1.818221092224121) ?  0.1506690914798644 :   0.1343383775850579 :   (b('dg0_v_vv') <= -0.450803279876709) ?  (b('ndvi') <= 2935.5) ?  0.1111815460627475 :   0.08671220302074073 :   0.03213326776535966 :   (b('l8b5_med') <= 3560.2987060546875) ?  (b('l8b5_med') <= 2995.778076171875) ?  (b('l8b4_med') <= 812.4763488769531) ?  (b('ndvi_med') <= 1855.0) ?  (b('l8b4') <= 874.8910217285156) ?  0.06962155997786035 :   0.06688518712218175 :   (b('ndvi') <= 1026.5) ?  (b('l8b5_med') <= 2962.1964111328125) ?  -0.08092902784349013 :   -0.09127079796711582 :   (b('dg0_v_vv') <= -1.4773564338684082) ?  -0.03635485443269893 :   -0.015394106116922025 :   (b('bulk') <= 143.5) ?  (b('l8b5_med') <= 2741.945556640625) ?  (b('l8b5') <= 3636.6375732421875) ?  0.03325086986715114 :   0.09276933964505937 :   (b('moss') <= 0.02757793851196766) ?  0.11523179551293669 :   0.0007210824016495675 :   (b('l8b5') <= 3173.2542724609375) ?  (b('l8b4') <= 2080.8897705078125) ?  -0.012878497787155824 :   0.06719224363716297 :   (b('dg0_v_vv') <= 0.3651595115661621) ?  -0.002652056942421959 :   0.05226025718209516 :   (b('dg0_v_vv') <= 0.5278596878051758) ?  (b('l8b11_med') <= 1982.345947265625) ?  (b('l8b11') <= 2038.3331298828125) ?  (b('bulk') <= 135.5) ?  0.037197937785965425 :   -0.004031726368669483 :   (b('bulk') <= 145.5) ?  0.028288051474285345 :   0.08800917854942898 :   (b('ndvi_med') <= 2152.5) ?  (b('dg0_v_vv') <= -2.3784475326538086) ?  -0.013155657080902616 :   0.021298544911415782 :   (b('ndvi') <= 3533.5) ?  -0.028926909758681996 :   0.029715601964867108 :   (b('l8b5') <= 3664.334228515625) ?  (b('bulk') <= 159.0) ?  (b('ndvi') <= 4401.0) ?  0.03844796874322659 :   0.08546188454468809 :   (b('dg0_v_vv') <= 0.7727665901184082) ?  0.09227921900610014 :   0.15930496142483108 :   (b('dg0_v_vv') <= 1.300309181213379) ?  (b('ndvi') <= 2675.0) ?  0.031556843480541566 :   -0.04026452249169342 :   (b('dg0_v_vv') <= 1.3755202293395996) ?  0.18113674277744593 :   0.04150399217939452 :   (b('l8b11_med') <= 1260.1935424804688) ?  (b('ndvi') <= 4262.5) ?  (b('l8b4') <= 1468.9552612304688) ?  (b('dg0_v_vv') <= 5.6453306674957275) ?  (b('ndvi') <= 3656.0) ?  0.07596898449785557 :   0.05848384137884788 :   0.05191951786537907 :   0.08681846527013118 :   (b('dg0_v_vv') <= 1.0600881576538086) ?  (b('l8b11') <= 2785.210693359375) ?  (b('dg0_v_vv') <= -2.502865791320801) ?  0.02806846845573646 :   0.011283270012410557 :   -0.00773339111555027 :   (b('crops') <= 82.11072540283203) ?  -0.0464714303129555 :   -0.05991026192745871 :   (b('l8b5') <= 3428.2880859375) ?  (b('l8b11') <= 2436.2935791015625) ?  (b('dg0_v_vv') <= 4.242205381393433) ?  (b('ndvi') <= 1983.0) ?  -0.0654713540891001 :   -0.02875499917077336 :   0.07410418784430467 :   (b('l8b11') <= 3110.9288330078125) ?  (b('l8b4') <= 1700.1217041015625) ?  0.017502276726352763 :   0.09106564596302331 :   (b('dg0_v_vv') <= -0.08240127563476562) ?  -0.040513574835505814 :   0.040757342452065096 :   (b('ndvi') <= 1764.5) ?  (b('ndvi') <= 1710.0) ?  (b('l8b11_med') <= 2518.2154541015625) ?  -0.05051742790001423 :   -0.038351549565018975 :   (b('dg0_v_vv') <= 0.44811582565307617) ?  0.025136080029510482 :   0.04472234751464145 :   (b('dg0_v_vv') <= 4.72709059715271) ?  (b('l8b4') <= 1369.8638305664062) ?  -0.03309352904621577 :   -0.05491939854795832 :   (b('l8b11') <= 2725.14306640625) ?  -0.023541906854817513 :   0.05078159903317686 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b4') <= 1672.8639526367188) ?  (b('ndvi') <= 1489.0) ?  (b('ndvi') <= 1463.0) ?  (b('l8b4') <= 1606.9500732421875) ?  (b('l8b11') <= 2350.42724609375) ?  (b('ndvi') <= 1271.0) ?  (b('l8b11_med') <= 2890.706298828125) ?  0.05617765500659997 :   0.07139328146801599 :   (b('dg0_v_vv') <= 1.9351000785827637) ?  0.031217143309556987 :   0.05971658270264618 :   (b('dg0_v_vv') <= 1.2568726539611816) ?  (b('dg0_v_vv') <= 0.07752037048339844) ?  0.024684858408391103 :   0.058715121416243056 :   (b('l8b11') <= 2695.1473388671875) ?  -0.009122733435901226 :   -0.03140602665184744 :   (b('l8b4') <= 1661.8778686523438) ?  (b('ndvi_med') <= 1480.5) ?  (b('dg0_v_vv') <= 0.01640796661376953) ?  -0.042386063402255716 :   -0.01901381726655562 :   (b('ndvi') <= 1215.5) ?  0.011234957024317094 :   -0.00895745650062496 :   0.08326016022292237 :   (b('dg0_v_vv') <= 6.8364434242248535) ?  (b('ndvi_med') <= 1481.5) ?  (b('l8b11') <= 3109.673095703125) ?  0.08179256624991374 :   (b('dg0_v_vv') <= 0.49266815185546875) ?  0.07634396929816067 :   0.06779823602450963 :   0.04882400442770479 :   0.10766961373602535 :   (b('l8b4') <= 977.6329345703125) ?  (b('ndvi') <= 2019.0) ?  (b('l8b11_med') <= 2881.5589599609375) ?  0.024144325943924794 :   0.010730608435884026 :   (b('ndvi') <= 2845.5) ?  0.09622679499520129 :   (b('dg0_v_vv') <= 1.3444066047668457) ?  0.0497253217916826 :   0.05083253974977753 :   (b('l8b4') <= 1588.02099609375) ?  (b('l8b11') <= 2695.1473388671875) ?  (b('dg0_v_vv') <= -0.03405904769897461) ?  (b('moss') <= 6.368875980377197) ?  -0.017654124377320203 :   0.0016426050078561238 :   (b('dg0_v_vv') <= 0.38744163513183594) ?  0.04785628303600978 :   -0.007809295805164818 :   (b('l8b11_med') <= 2881.5589599609375) ?  (b('dg0_v_vv') <= 1.9242985248565674) ?  -0.022371173575548558 :   -0.011980632990136297 :   (b('ndvi') <= 1510.5) ?  -0.028106226791612787 :   -0.023187272494399794 :   (b('dg0_v_vv') <= 0.5788631439208984) ?  (b('l8b5') <= 3497.5291748046875) ?  (b('l8b4_med') <= 1649.88818359375) ?  0.04669006650880668 :   0.019047345827438434 :   0.061502747065930646 :   (b('l8b4_med') <= 1656.4095458984375) ?  -0.023813335021564186 :   -0.04605066919881662 :   (b('ndvi') <= 1012.5) ?  (b('ndvi') <= 717.5) ?  (b('ndvi') <= 650.5) ?  (b('l8b11') <= 3240.8087158203125) ?  (b('ndvi_med') <= 631.75) ?  (b('l8b11') <= 3073.788330078125) ?  -0.0009923388038403137 :   -0.04824757480649598 :   (b('l8b5') <= 2629.95458984375) ?  -0.0248513690824786 :   -0.03734471704437253 :   (b('l8b4_med') <= 2589.50146484375) ?  0.017065296267754437 :   (b('dg0_v_vv') <= -0.006103515625) ?  -0.0059347056395941955 :   -0.007934702242129443 :   (b('moss') <= 25.531142234802246) ?  (b('l8b11') <= 4233.6361083984375) ?  (b('ndvi_med') <= 888.0) ?  0.03742628533074998 :   0.0177474161067879 :   -0.012931799755566048 :   (b('l8b4') <= 2656.1912841796875) ?  -0.014582065818953355 :   -0.0396331219130724 :   (b('l8b5_med') <= 2254.820556640625) ?  (b('ndvi') <= 907.0) ?  (b('dg0_v_vv') <= 0.08590507507324219) ?  (b('dg0_v_vv') <= -0.0597834587097168) ?  -0.03159689053806171 :   -0.010913249045408836 :   (b('l8b11') <= 2877.694091796875) ?  -0.021871408950287538 :   -0.03799327764749345 :   (b('dg0_v_vv') <= 0.3202676773071289) ?  (b('l8b5_med') <= 2236.06005859375) ?  0.01792930626004463 :   0.0448051199182295 :   -0.04039032110216814 :   (b('l8b5') <= 2240.372802734375) ?  (b('l8b4') <= 1688.1171875) ?  -0.037074853454841475 :   (b('l8b11') <= 2978.9263916015625) ?  -0.05641559838610749 :   -0.0646095959867764 :   (b('ndvi') <= 820.0) ?  (b('dg0_v_vv') <= 0.14824199676513672) ?  -0.03030402216683064 :   0.0412878547865051 :   (b('dg0_v_vv') <= 1.4952983856201172) ?  -0.03998254589344816 :   -0.015231251412709262 :   (b('ndvi') <= 1720.0) ?  (b('ndvi') <= 1088.5) ?  (b('l8b11_med') <= 2928.825439453125) ?  (b('dg0_v_vv') <= 0.2584238052368164) ?  (b('ndvi_med') <= 1349.5) ?  0.07210331856009229 :   0.0686508983259802 :   0.05474432924178013 :   (b('dg0_v_vv') <= 0.12394285202026367) ?  0.020717008545867846 :   -0.04128299044085319 :   (b('l8b11_med') <= 2880.1024169921875) ?  (b('dg0_v_vv') <= 1.2086248397827148) ?  (b('l8b5') <= 3118.744140625) ?  -0.01937043066437181 :   0.0020793468825641343 :   (b('l8b4') <= 1762.5487060546875) ?  -0.025583481488729604 :   0.07274961715423481 :   (b('l8b11') <= 3393.3631591796875) ?  (b('l8b5') <= 2872.5596923828125) ?  0.007678980503625709 :   -0.02867158661323337 :   (b('l8b4') <= 2487.7574462890625) ?  0.07791269356525621 :   -0.012439305740731906 :   (b('l8b11') <= 2628.866455078125) ?  (b('ndvi') <= 1754.5) ?  (b('l8b5_med') <= 3622.8421630859375) ?  0.028075271456161288 :   0.020075270144859103 :   (b('crops') <= 91.9686164855957) ?  0.022685163277481574 :   (b('l8b11_med') <= 2890.52392578125) ?  0.07521341467323839 :   0.05469794339299558 :   (b('ndvi') <= 3498.0) ?  (b('l8b5') <= 3741.14013671875) ?  (b('dg0_v_vv') <= 3.340824604034424) ?  0.018296481976520337 :   -0.01386690456805414 :   (b('l8b5') <= 3923.5264892578125) ?  -0.04221298432978418 :   -0.012357840234978646 :   (b('dg0_v_vv') <= -1.1218409538269043) ?  (b('ndvi') <= 3741.0) ?  0.05275565021050646 :   0.06756984995409304 :   (b('dg0_v_vv') <= -0.5816340446472168) ?  0.03860985161467845 :   0.04956985817953402 :   (b('grass') <= 27.41930866241455) ?  (b('l8b4_med') <= 2187.779052734375) ?  (b('ndvi_med') <= 2106.5) ?  (b('l8b4_med') <= 2066.49853515625) ?  (b('dg0_v_vv') <= 2.077484130859375) ?  (b('l8b5') <= 2241.1700439453125) ?  (b('l8b4_med') <= 1780.755615234375) ?  -0.04670420320248822 :   -0.014088363086055081 :   (b('l8b5_med') <= 3317.5343017578125) ?  -0.04164888276204307 :   -0.030990127101621245 :   (b('l8b5_med') <= 3148.1982421875) ?  (b('ndvi_med') <= 1766.0) ?  0.0202966574149326 :   -0.023622232701168006 :   (b('l8b5_med') <= 3322.7132568359375) ?  -0.035853004645588056 :   0.017818319003954954 :   (b('dg0_v_vv') <= 2.9971072673797607) ?  (b('l8b5') <= 2309.8245849609375) ?  (b('l8b11') <= 2266.612548828125) ?  -0.0406025143992545 :   -0.002551253956119948 :   (b('l8b4') <= 2473.560546875) ?  -0.04843950138371709 :   -0.05774925320273526 :   (b('l8b5_med') <= 3155.236083984375) ?  0.020629613452708917 :   (b('l8b11_med') <= 3566.801025390625) ?  -0.027357847150409075 :   -0.05978977875075823 :   (b('dg0_v_vv') <= -0.2501988410949707) ?  (b('l8b5') <= 2621.3082275390625) ?  (b('dg0_v_vv') <= -1.7871603965759277) ?  -0.10492328075019189 :   (b('l8b4') <= 1590.5604858398438) ?  -0.11554001737770637 :   -0.11900001736742458 :   (b('ndvi') <= 2867.5) ?  (b('l8b11') <= 3114.1199951171875) ?  0.0324313589639752 :   0.014638632454200745 :   -0.022073290135010942 :   (b('l8b5') <= 2513.6807861328125) ?  (b('l8b11') <= 2538.85986328125) ?  0.0978817190893067 :   0.1251354150413468 :   (b('l8b4') <= 2000.8531494140625) ?  (b('l8b11') <= 3177.4505615234375) ?  0.0450783214947607 :   0.0625616748009566 :   0.11134534011561989 :   (b('l8b5') <= 2915.3016357421875) ?  (b('dg0_v_vv') <= 1.142526626586914) ?  (b('dg0_v_vv') <= 0.6464033126831055) ?  (b('ndvi') <= 736.5) ?  (b('l8b11_med') <= 3090.7059326171875) ?  0.05943309814794037 :   0.012071066659869761 :   (b('l8b4') <= 2405.6190185546875) ?  -0.003792468820913554 :   -0.042583969635522925 :   (b('ndvi') <= 1641.5) ?  (b('l8b4_med') <= 2612.810302734375) ?  -0.05829958187026485 :   -0.07267685251006342 :   0.005326523039649672 :   (b('l8b11') <= 2975.2486572265625) ?  (b('l8b5') <= 2724.4693603515625) ?  (b('moss') <= 10.968353748321533) ?  0.061953577571448495 :   0.05821763151252024 :   0.038693463284529966 :   (b('crops') <= 11.524752616882324) ?  -0.01189896304794974 :   0.021925613665139898 :   (b('grass') <= 21.365646362304688) ?  (b('dg0_v_vv') <= 0.7344307899475098) ?  (b('ndvi') <= 1117.5) ?  (b('ndvi') <= 801.5) ?  -0.03025958968050871 :   -0.01333301072316333 :   (b('l8b11') <= 3988.4112548828125) ?  -0.021766304747890795 :   -0.03787397094325501 :   (b('l8b11') <= 3580.158935546875) ?  (b('dg0_v_vv') <= 1.0077781677246094) ?  0.0271995451822593 :   -0.0051859175122570935 :   (b('ndvi') <= 1222.5) ?  -0.02908626072141999 :   -0.013826965452782801 :   (b('l8b4') <= 2821.695068359375) ?  (b('l8b11') <= 3265.115966796875) ?  (b('l8b4') <= 2421.828125) ?  -0.039716221686848345 :   -0.019094031819362887 :   (b('dg0_v_vv') <= -0.10975933074951172) ?  -0.03691962350405234 :   -0.04830475694557188 :   (b('l8b4') <= 2895.4779052734375) ?  (b('l8b5_med') <= 3683.5443115234375) ?  0.02440144757912079 :   -0.01725098624886596 :   (b('dg0_v_vv') <= -1.0903968811035156) ?  -0.05376374922609492 :   -0.03460044124092826 :   (b('dg0_v_vv') <= 0.6688103675842285) ?  (b('ndvi_med') <= 1772.0) ?  (b('grass') <= 49.75533485412598) ?  (b('bulk') <= 152.5) ?  (b('l8b11_med') <= 2796.288330078125) ?  (b('l8b4') <= 1762.9371948242188) ?  -0.0326985172211611 :   0.01919965240798641 :   (b('l8b11') <= 2425.6220703125) ?  0.00772786495287308 :   -0.026434103690367503 :   (b('l8b5') <= 2533.476806640625) ?  (b('dg0_v_vv') <= -0.6631364822387695) ?  0.07371105972620602 :   0.04563300406547866 :   (b('l8b5') <= 2637.0560302734375) ?  -0.05728676549363885 :   0.026822742632704198 :   (b('ndvi') <= 1279.5) ?  (b('ndvi') <= 843.0) ?  (b('l8b5') <= 2247.0693359375) ?  0.017044501260619543 :   0.011116351329746324 :   (b('moss') <= 25.32264995574951) ?  -0.04552346568411432 :   -0.01705778091536829 :   (b('ndvi') <= 1283.0) ?  (b('dg0_v_vv') <= 0.10226726531982422) ?  0.04902791711722847 :   0.09202791485225197 :   (b('ndvi') <= 2625.0) ?  -0.024572711942358324 :   0.040458270784602096 :   (b('l8b11_med') <= 3088.0281982421875) ?  (b('grass') <= 68.11360549926758) ?  (b('ndvi') <= 2088.0) ?  (b('l8b4') <= 1881.6530151367188) ?  -0.030108523701006685 :   -0.0557383889771199 :   (b('l8b11') <= 3294.7308349609375) ?  -0.03143903221377768 :   0.026195980953465363 :   (b('dg0_v_vv') <= -0.11232709884643555) ?  (b('l8b5') <= 3145.286865234375) ?  -0.061369887097254154 :   -0.09912058084367754 :   (b('sand') <= 56.5) ?  -0.05044623747784631 :   0.05081106083038808 :   (b('l8b5') <= 3059.030029296875) ?  (b('l8b4') <= 1521.8128051757812) ?  0.014984192023030196 :   (b('ndvi') <= 1973.5) ?  0.09939868802489973 :   0.05753393825177777 :   -0.03153194408437514 :   (b('moss') <= 14.482890129089355) ?  (b('l8b4_med') <= 1818.0885620117188) ?  (b('l8b5') <= 2122.4185791015625) ?  -0.1042536219339816 :   (b('l8b4') <= 1515.0272216796875) ?  (b('l8b4') <= 1140.8831176757812) ?  0.05028051980204748 :   0.0972959334698676 :   (b('l8b11') <= 2771.5474853515625) ?  0.00028031793948245926 :   0.06289334387103804 :   (b('l8b11_med') <= 2997.412353515625) ?  (b('dg0_v_vv') <= 3.5240867137908936) ?  (b('ndvi') <= 1314.5) ?  -0.04051661107404767 :   -0.007541539010034633 :   (b('l8b11') <= 3294.03955078125) ?  0.03683010943594486 :   -0.04619760080227828 :   (b('l8b4') <= 1720.6055297851562) ?  (b('l8b5') <= 2595.9569091796875) ?  -0.01291872385406434 :   0.08901984649210147 :   (b('grass') <= 70.26726150512695) ?  0.029261131574541074 :   -0.0354286363506735 :   (b('grass') <= 65.2627010345459) ?  (b('l8b11_med') <= 2979.07373046875) ?  (b('l8b5') <= 2684.28173828125) ?  (b('l8b4') <= 1757.4463500976562) ?  0.07243642872284338 :   0.00199236737852633 :   (b('sand') <= 44.0) ?  0.2080657690657032 :   0.12040811649734662 :   (b('l8b11_med') <= 3387.9322509765625) ?  (b('l8b4') <= 1302.6473388671875) ?  0.04071585770683605 :   -0.01723044463053048 :   (b('dg0_v_vv') <= 3.7747859954833984) ?  -0.0677507613905827 :   -0.09938975509478662 :   (b('l8b11') <= 2955.65576171875) ?  (b('moss') <= 16.294450283050537) ?  (b('l8b4') <= 1712.7339477539062) ?  -0.032510703297434156 :   -0.050936887719121984 :   (b('ndvi') <= 2459.5) ?  -0.06303462712042444 :   -0.024738046072692754 :   (b('l8b5') <= 2518.1431884765625) ?  -0.04494832067045079 :   (b('l8b5') <= 2578.575439453125) ?  -0.02569350833115988 :   -0.017942346619204574 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1723.2078247070312) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 35.5) ?  (b('l8b5') <= 2835.8304443359375) ?  (b('l8b4') <= 459.3279571533203) ?  (b('ndvi') <= 3507.5) ?  (b('l8b11') <= 900.6961975097656) ?  (b('dg0_v_vv') <= 0.7800068855285645) ?  (b('ndvi') <= 3453.0) ?  0.06354402894158226 :   0.09706119330479757 :   (b('dg0_v_vv') <= 1.442713975906372) ?  0.10465913724599188 :   0.08273071863400983 :   (b('ndvi') <= 3059.5) ?  0.13981937074512724 :   0.0766172491045354 :   (b('ndvi') <= 4521.5) ?  (b('dg0_v_vv') <= -1.0291624069213867) ?  -0.003975345143487963 :   -0.007907630633180673 :   -0.023948609147406402 :   (b('crops') <= 17.18335199356079) ?  (b('ndvi') <= 3199.5) ?  (b('ndvi') <= 2576.0) ?  (b('l8b5') <= 2127.99560546875) ?  0.0425696315320279 :   -0.014124373617484613 :   (b('ndvi') <= 2927.5) ?  -0.020944761710630304 :   0.00476219478436245 :   (b('ndvi') <= 3249.5) ?  (b('l8b5') <= 2507.9656982421875) ?  0.041341518095826535 :   0.13001077717928355 :   (b('l8b5') <= 2169.8046875) ?  0.05510194322568108 :   0.014745198310353754 :   (b('ndvi') <= 3321.5) ?  (b('l8b11') <= 1405.104248046875) ?  (b('ndvi') <= 2935.5) ?  0.09858499077406835 :   0.15758833353911045 :   (b('l8b11') <= 1426.87841796875) ?  -0.03551332798219642 :   0.06835422070446202 :   (b('dg0_v_vv') <= -1.4659099578857422) ?  (b('ndvi') <= 4134.0) ?  0.036910369614759825 :   0.11592397498653007 :   (b('sand') <= 30.5) ?  -0.06575138676996192 :   -0.01190539611543856 :   (b('l8b11_med') <= 1355.3932495117188) ?  (b('dg0_v_vv') <= -0.03996086120605469) ?  (b('ndvi_med') <= 3984.5) ?  (b('bulk') <= 133.5) ?  (b('sand') <= 23.5) ?  0.10095729889819643 :   0.1972071835537993 :   (b('l8b4') <= 498.2988586425781) ?  0.0201201219831691 :   0.07079210720333191 :   (b('ndvi') <= 4880.5) ?  (b('l8b4') <= 449.30381774902344) ?  -0.03872001195825341 :   0.01741011839372475 :   (b('l8b11') <= 1186.7662963867188) ?  0.012257822455360168 :   0.1824390554879521 :   (b('l8b4') <= 700.5472412109375) ?  (b('l8b5_med') <= 3756.3685302734375) ?  (b('sand') <= 14.0) ?  0.14684854084672488 :   0.08142638771263523 :   (b('ndvi') <= 4263.0) ?  0.11188172639090004 :   0.008019417621495576 :   (b('l8b11') <= 1451.7606201171875) ?  (b('ndvi_med') <= 2809.5) ?  0.11573456239442242 :   -0.011568389526109252 :   (b('crops') <= 58.690534591674805) ?  0.10956443385236363 :   -0.041274606355002214 :   (b('l8b5') <= 4023.661376953125) ?  (b('l8b5_med') <= 2643.7559814453125) ?  (b('dg0_v_vv') <= -0.8939356803894043) ?  0.16330298182459932 :   0.20920298096629242 :   (b('l8b11') <= 1097.1131591796875) ?  0.16010619603211335 :   0.13712218609081112 :   (b('dg0_v_vv') <= 1.1973342895507812) ?  (b('ndvi') <= 4295.0) ?  0.09334509720880987 :   0.0799454671557478 :   0.10202174275020448 :   (b('crops') <= 16.61120891571045) ?  (b('l8b5') <= 1727.986083984375) ?  (b('l8b4') <= 417.97357177734375) ?  (b('l8b5_med') <= 1572.5333862304688) ?  0.09672256233282878 :   (b('dg0_v_vv') <= 1.6062326431274414) ?  (b('ndvi') <= 2961.5) ?  0.016432551629369185 :   0.0004872628683182667 :   -0.019855217533156472 :   (b('dg0_v_vv') <= 1.766922950744629) ?  (b('ndvi') <= 1505.5) ?  0.013406800311176956 :   (b('dg0_v_vv') <= -1.9850811958312988) ?  -0.010802292817643534 :   -0.044912514720387195 :   (b('ndvi_med') <= 2072.25) ?  (b('ndvi') <= 1750.5) ?  0.04787013932644307 :   0.06816508376812053 :   -0.05741137126939995 :   (b('moss') <= 15.61111068725586) ?  (b('l8b4') <= 732.7281799316406) ?  (b('ndvi') <= 3627.5) ?  (b('l8b11_med') <= 1133.1121215820312) ?  0.03657313278470546 :   -0.014020580336403844 :   (b('ndvi_med') <= 4100.75) ?  -0.0235562337814256 :   0.024502640638929585 :   (b('l8b5') <= 2260.1475830078125) ?  (b('dg0_v_vv') <= -1.7489862442016602) ?  -0.0428034655695721 :   -0.057937158303419105 :   (b('l8b11') <= 1459.5136108398438) ?  -0.07293452581206185 :   -0.08967509112982708 :   (b('dg0_v_vv') <= 0.7275066375732422) ?  (b('ndvi') <= 3126.5) ?  (b('ndvi') <= 1557.0) ?  0.002818110659358289 :   0.12303231017057173 :   (b('ndvi') <= 3695.0) ?  -0.002769470565098736 :   0.06196254452716271 :   (b('l8b5') <= 2346.5665283203125) ?  (b('l8b11') <= 853.3075866699219) ?  -0.016059968705496447 :   0.06494382111529899 :   (b('ndvi') <= 2444.5) ?  -0.0893001304549925 :   -0.03783471660744999 :   (b('ndvi_med') <= 4666.0) ?  (b('l8b5') <= 2674.1826171875) ?  (b('bulk') <= 134.0) ?  (b('l8b4') <= 770.9285278320312) ?  (b('ndvi') <= 4555.5) ?  0.040353980615464274 :   -0.05034681319832954 :   (b('l8b4_med') <= 573.5225219726562) ?  -0.038641695244477296 :   -0.027149678355984495 :   (b('l8b11') <= 1087.4679565429688) ?  (b('dg0_v_vv') <= -0.5180959701538086) ?  0.09497177353652067 :   0.041450522912957206 :   (b('ndvi_med') <= 3972.25) ?  0.1570333101343956 :   0.1062710528541441 :   (b('moss') <= 4.324254751205444) ?  (b('grass') <= 4.055993676185608) ?  (b('l8b4') <= 314.03736877441406) ?  -0.03413970095302832 :   0.031162907266884757 :   (b('l8b5') <= 2873.183349609375) ?  0.19591909624169812 :   -0.013874784554791228 :   (b('ndvi') <= 4906.0) ?  (b('l8b11') <= 1074.874267578125) ?  0.03459346910539828 :   0.0099485071532612 :   (b('grass') <= 18.683167457580566) ?  0.12157559074342031 :   0.033412783832540115 :   (b('l8b4') <= 700.7987670898438) ?  (b('l8b11') <= 1222.7625122070312) ?  (b('l8b11_med') <= 1068.9169921875) ?  (b('l8b4') <= 438.09165954589844) ?  0.038963278373825516 :   -0.01534740398717191 :   (b('dg0_v_vv') <= -1.0340423583984375) ?  0.044531868695054655 :   0.10847649263775566 :   (b('ndvi') <= 4942.0) ?  (b('dg0_v_vv') <= -0.9091582298278809) ?  -0.02436515531800626 :   -0.001821609689533395 :   (b('l8b5_med') <= 3413.9039306640625) ?  0.028254163021913484 :   0.049448564535024386 :   (b('dg0_v_vv') <= -1.2602524757385254) ?  (b('l8b4_med') <= 585.4060668945312) ?  0.0783738502561474 :   0.0855738648473644 :   (b('l8b4') <= 754.5669250488281) ?  0.20390194348007884 :   0.15988304091995081 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1274.3564453125) ?  (b('l8b5') <= 2225.67822265625) ?  (b('l8b5') <= 2085.4517822265625) ?  (b('grass') <= 58.827531814575195) ?  (b('ndvi') <= 1914.5) ?  (b('bulk') <= 135.0) ?  0.1915566573797005 :   0.20120117408845634 :   (b('l8b4') <= 556.854248046875) ?  0.08677470680627052 :   0.032461682804343535 :   (b('l8b5') <= 1808.5505981445312) ?  (b('dg0_v_vv') <= -2.3233518600463867) ?  0.0807534351875939 :   0.023612424559777367 :   -0.09109078045835614 :   (b('dg0_v_vv') <= -0.9349837303161621) ?  (b('dg0_v_vv') <= -1.3912005424499512) ?  (b('dg0_v_vv') <= -1.4974184036254883) ?  0.11788994547430554 :   0.13988993045393505 :   (b('l8b5') <= 2159.42578125) ?  -0.034309197060411856 :   0.0542990542985203 :   (b('l8b5') <= 2207.9786376953125) ?  (b('grass') <= 58.827531814575195) ?  0.1286488986753838 :   0.04401134494314024 :   0.23249954206753445 :   (b('crops') <= 9.933524370193481) ?  (b('l8b11') <= 939.8609008789062) ?  (b('l8b11') <= 882.5259704589844) ?  (b('l8b4') <= 502.24266052246094) ?  0.010807929867822402 :   -0.052104553844476126 :   (b('ndvi') <= 4517.0) ?  0.09355592786418965 :   0.15993304496543004 :   (b('l8b11') <= 1402.0955810546875) ?  (b('l8b11') <= 1272.1600952148438) ?  0.010678261591724515 :   -0.05696914621265697 :   (b('sand') <= 39.5) ?  -0.022803212365760224 :   0.06311521188401709 :   (b('dg0_v_vv') <= -1.2283859252929688) ?  (b('ndvi') <= 4521.0) ?  (b('l8b11') <= 1249.2156372070312) ?  -0.025171394450410556 :   -0.030716006852304356 :   (b('l8b11_med') <= 1244.5859985351562) ?  0.11145676593884163 :   0.041254043751471514 :   (b('dg0_v_vv') <= -0.21826505661010742) ?  (b('l8b5') <= 2555.1011962890625) ?  0.08548075631359886 :   0.15043863465696353 :   (b('l8b5') <= 3021.815673828125) ?  0.07348354142315264 :   0.11936402201035372 :   (b('ndvi') <= 3066.0) ?  (b('dg0_v_vv') <= 0.8869194984436035) ?  (b('l8b5') <= 2180.6176147460938) ?  -0.060952550829864074 :   -0.05209583120785498 :   -0.10302725920614875 :   (b('l8b5') <= 3071.4813232421875) ?  (b('ndvi') <= 3817.5) ?  (b('l8b5') <= 2361.545166015625) ?  -0.009435254202420584 :   (b('l8b5_med') <= 2532.8045654296875) ?  -0.0318848942645627 :   -0.05306756841448318 :   (b('l8b5_med') <= 2656.3057861328125) ?  (b('ndvi') <= 4588.0) ?  -0.0203501576566975 :   -0.008067320849695794 :   0.019575620833101604 :   (b('dg0_v_vv') <= 1.1854348182678223) ?  (b('moss') <= 9.900750160217285) ?  (b('l8b5') <= 3274.1270751953125) ?  -0.09775173931271602 :   -0.07930712477769494 :   (b('l8b5_med') <= 2583.02294921875) ?  -0.03582543752871267 :   -0.05794019262740194 :   0.0001502370034761058 :   (b('l8b4') <= 537.6677856445312) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b5') <= 2588.2802734375) ?  (b('dg0_v_vv') <= -0.04287385940551758) ?  (b('dg0_v_vv') <= -0.12109136581420898) ?  (b('dg0_v_vv') <= -1.6654491424560547) ?  0.09056702926731251 :   0.11299456304655398 :   0.17807282308539973 :   (b('l8b5_med') <= 2553.574951171875) ?  0.05715589071172189 :   (b('l8b4_med') <= 481.6563720703125) ?  0.07218639196557736 :   0.09897306477266442 :   (b('dg0_v_vv') <= 0.40665769577026367) ?  (b('dg0_v_vv') <= 0.1475353240966797) ?  (b('ndvi') <= 3822.5) ?  0.12404027713909437 :   0.11565268859516382 :   (b('l8b11_med') <= 1297.9468994140625) ?  0.10383093452938626 :   0.09331066280998068 :   (b('dg0_v_vv') <= 1.5718121528625488) ?  (b('l8b5_med') <= 2949.3199462890625) ?  0.13491135491107453 :   0.12216450668198311 :   (b('l8b5_med') <= 3027.6649169921875) ?  0.12326437567527845 :   0.1466823354525526 :   (b('l8b4') <= 447.9172668457031) ?  (b('dg0_v_vv') <= -0.39939117431640625) ?  (b('ndvi') <= 3465.5) ?  0.16393988518739885 :   0.16023116128701967 :   (b('dg0_v_vv') <= 0.04723978042602539) ?  (b('ndvi') <= 3465.5) ?  0.1487117994097198 :   0.1544522851243769 :   (b('dg0_v_vv') <= 0.28346872329711914) ?  0.16374003685626 :   0.15775936678570202 :   (b('ndvi') <= 2667.5) ?  (b('dg0_v_vv') <= -0.4175567626953125) ?  (b('dg0_v_vv') <= -0.5687241554260254) ?  0.15190061055737458 :   0.15623138522713587 :   (b('dg0_v_vv') <= -0.2849302291870117) ?  0.13049140548317834 :   0.15137037055332303 :   (b('dg0_v_vv') <= 1.7600140571594238) ?  (b('dg0_v_vv') <= 0.06099224090576172) ?  0.1242831875502279 :   0.13583677458764848 :   0.1107872152159397 :   (b('dg0_v_vv') <= -2.438610553741455) ?  (b('l8b4') <= 630.7738952636719) ?  (b('dg0_v_vv') <= -3.164905548095703) ?  (b('ndvi_med') <= 3457.25) ?  (b('dg0_v_vv') <= -3.8604965209960938) ?  -0.01777257424121126 :   -0.007910517151660287 :   -0.006695076222107826 :   (b('dg0_v_vv') <= -2.730191230773926) ?  -0.017810517098016107 :   (b('dg0_v_vv') <= -2.46519136428833) ?  -0.023772563333809678 :   -0.018984241749764985 :   (b('dg0_v_vv') <= -3.306211471557617) ?  -0.0011773253838489595 :   0.010782415991997252 :   (b('l8b5') <= 2405.21044921875) ?  (b('l8b5') <= 2260.337646484375) ?  0.14985124500999275 :   (b('ndvi') <= 3523.5) ?  (b('ndvi') <= 3442.5) ?  0.05122980725717183 :   0.08517466903778757 :   (b('dg0_v_vv') <= -1.8746628761291504) ?  -0.0035819691468172155 :   -0.005950485153663254 :   (b('dg0_v_vv') <= -0.36297178268432617) ?  (b('bulk') <= 119.0) ?  (b('grass') <= 57.840450286865234) ?  0.08913196198914464 :   0.11152743754684454 :   (b('l8b5') <= 2455.188720703125) ?  0.09977966188079122 :   0.05604016368401946 :   (b('dg0_v_vv') <= 1.3603038787841797) ?  (b('ndvi') <= 4406.5) ?  0.09660359431853252 :   0.11384116461076117 :   (b('l8b5') <= 2487.0224609375) ?  0.09623380485116015 :   0.1214456398196531 :   (b('bulk') <= 137.5) ?  (b('crops') <= 74.177001953125) ?  (b('ndvi_med') <= 3174.5) ?  (b('bulk') <= 128.0) ?  (b('l8b4') <= 816.8327331542969) ?  (b('l8b5') <= 2471.67919921875) ?  (b('ndvi') <= 3676.0) ?  (b('l8b11') <= 1627.6461181640625) ?  -0.007364809813288416 :   0.0919862828299931 :   (b('l8b5_med') <= 2634.7596435546875) ?  -0.10643299697802158 :   -0.08173299767912123 :   (b('l8b4') <= 796.7146301269531) ?  (b('l8b11') <= 1097.1507568359375) ?  0.03330164418242909 :   -0.051112209979916355 :   (b('l8b5') <= 2746.8800048828125) ?  0.024693481423059473 :   0.14374033745897785 :   (b('grass') <= 24.99612331390381) ?  0.10627646703400825 :   (b('ndvi') <= 1099.5) ?  (b('ndvi') <= 1040.0) ?  -0.01546981802275682 :   0.049530187043637985 :   (b('moss') <= 11.080983638763428) ?  -0.030187016207445082 :   -0.06632528176866932 :   (b('grass') <= 52.39274787902832) ?  (b('dg0_v_vv') <= 1.1375489234924316) ?  (b('l8b4') <= 535.7750549316406) ?  (b('dg0_v_vv') <= 0.17119073867797852) ?  -0.012479985264190486 :   0.023530928606828965 :   (b('l8b4_med') <= 737.3608703613281) ?  -0.025050776430380915 :   -0.013281380810601514 :   (b('l8b5') <= 2987.7646484375) ?  (b('ndvi_med') <= 2191.25) ?  -0.03325463937123346 :   0.01949682510864529 :   (b('ndvi') <= 4610.5) ?  -0.01579555353580128 :   0.016117410860374624 :   (b('moss') <= 12.153802871704102) ?  (b('moss') <= 11.76934003829956) ?  (b('l8b11_med') <= 1872.5770874023438) ?  0.042350476016588735 :   -0.034922979573727704 :   (b('l8b4') <= 832.9972839355469) ?  0.11211227636383211 :   0.22553618312791096 :   (b('dg0_v_vv') <= 1.7912530899047852) ?  (b('ndvi') <= 2315.0) ?  -0.029662664202219413 :   0.008841637594546824 :   (b('moss') <= 19.492186546325684) ?  0.029797306897779503 :   0.1032557835211676 :   (b('l8b5') <= 2441.902099609375) ?  (b('l8b5_med') <= 2239.5936279296875) ?  (b('grass') <= 42.7893180847168) ?  (b('l8b4') <= 756.1703491210938) ?  (b('l8b5') <= 1788.438720703125) ?  0.05431788414415459 :   0.023440935703083976 :   (b('dg0_v_vv') <= -0.14383840560913086) ?  0.07337922071826489 :   0.08721174489971774 :   -0.07122443228106973 :   (b('l8b11_med') <= 1436.123779296875) ?  (b('l8b11') <= 1436.123779296875) ?  (b('l8b4') <= 888.5449523925781) ?  0.05142515028654495 :   0.1409784637824871 :   (b('moss') <= 7.812680244445801) ?  0.013144378874183474 :   -0.037610512294523765 :   (b('l8b11') <= 1608.5706787109375) ?  (b('dg0_v_vv') <= 2.138211488723755) ?  -0.013265869270561488 :   0.019653894446013905 :   0.10469607606543702 :   (b('l8b5') <= 3132.4952392578125) ?  (b('l8b4_med') <= 747.5508728027344) ?  (b('ndvi') <= 4687.0) ?  (b('ndvi') <= 4190.0) ?  0.04988492214878615 :   0.008289107142297882 :   (b('l8b11_med') <= 1502.0938720703125) ?  -0.07797892887150687 :   -0.07159500018139645 :   (b('dg0_v_vv') <= 0.5879464149475098) ?  (b('ndvi_med') <= 3994.0) ?  -0.020726319406074292 :   -0.05787479697731732 :   (b('moss') <= 7.656388282775879) ?  0.045000400286018 :   -0.010507569823046872 :   (b('ndvi') <= 3130.5) ?  (b('grass') <= 23.426127433776855) ?  (b('ndvi') <= 3012.0) ?  0.02555890772459964 :   -0.008223717687314605 :   (b('dg0_v_vv') <= 2.1422228813171387) ?  0.06406767076093058 :   0.17782162011921174 :   (b('ndvi') <= 3200.5) ?  (b('l8b4') <= 867.7798767089844) ?  -0.06852687528055865 :   -0.04518139322189561 :   (b('sand') <= 70.5) ?  0.014754085528138113 :   -0.013358780774839714 :   (b('l8b5_med') <= 2896.5753173828125) ?  (b('sand') <= 51.5) ?  (b('l8b4_med') <= 802.5662536621094) ?  (b('dg0_v_vv') <= 1.5857255458831787) ?  (b('dg0_v_vv') <= -0.3608245849609375) ?  0.14593594923336622 :   0.1419732702507352 :   0.12722289615389376 :   (b('bulk') <= 113.0) ?  (b('l8b5') <= 2412.4403076171875) ?  (b('dg0_v_vv') <= -0.23485946655273438) ?  0.12105128766889511 :   0.13100632678033686 :   0.14199721690074596 :   (b('l8b5') <= 4208.7008056640625) ?  (b('dg0_v_vv') <= -2.218214988708496) ?  0.06024487569453609 :   0.09281459959348394 :   0.032378107276673146 :   (b('l8b11_med') <= 1612.4236450195312) ?  (b('l8b5') <= 2054.71728515625) ?  -0.004196857101810114 :   -0.01412397873821536 :   (b('dg0_v_vv') <= -0.47145986557006836) ?  0.047128048304770714 :   0.019271650027358783 :   (b('l8b4') <= 1052.549560546875) ?  (b('dg0_v_vv') <= 1.4974970817565918) ?  (b('ndvi') <= 1902.0) ?  (b('dg0_v_vv') <= -2.919558525085449) ?  0.022553185779531887 :   (b('dg0_v_vv') <= 0.16421127319335938) ?  0.1075030891369894 :   0.05810995581784603 :   (b('l8b4') <= 305.94541931152344) ?  (b('ndvi_med') <= 3351.25) ?  0.10657506894771956 :   0.11927876388680747 :   (b('crops') <= 88.03737258911133) ?  0.018973098976992557 :   -0.007000725732829402 :   (b('l8b11_med') <= 1432.5072631835938) ?  (b('ndvi') <= 3782.0) ?  (b('ndvi_med') <= 3341.25) ?  0.14309826668158238 :   0.08840553094206188 :   (b('l8b4') <= 305.94541931152344) ?  0.08700943868272193 :   0.008162914108161029 :   (b('ndvi') <= 3503.5) ?  (b('l8b4_med') <= 733.9454345703125) ?  -0.06929157261408134 :   0.02000477956206248 :   (b('ndvi') <= 3650.5) ?  0.09451455091383398 :   0.036779181874172194 :   (b('dg0_v_vv') <= 1.723301887512207) ?  (b('dg0_v_vv') <= 1.3874130249023438) ?  (b('l8b4_med') <= 813.8113098144531) ?  (b('ndvi') <= 2745.0) ?  -0.04246148304975683 :   -0.030295550378294123 :   (b('dg0_v_vv') <= 0.272676944732666) ?  -0.059712432332258 :   -0.036828597591400336 :   (b('ndvi_med') <= 3351.25) ?  -0.07930398621246687 :   -0.07602421812551935 :   (b('l8b4_med') <= 860.6597900390625) ?  (b('l8b11_med') <= 1220.2855834960938) ?  0.08269602916553148 :   0.0537844369098642 :   (b('l8b4_med') <= 955.183837890625) ?  (b('l8b5_med') <= 3474.926513671875) ?  -0.016695577312092064 :   -0.022695574570278404 :   -0.03469558398781228 :   (b('moss') <= 13.551528930664062) ?  (b('sand') <= 28.0) ?  (b('dg0_v_vv') <= -0.5446658134460449) ?  (b('crops') <= 46.74076271057129) ?  (b('ndvi') <= 2005.5) ?  (b('ndvi') <= 1685.0) ?  0.16187513660068636 :   0.11642530030344822 :   (b('ndvi') <= 4618.5) ?  (b('ndvi') <= 4273.5) ?  0.009990878395337733 :   -0.042785630548724654 :   (b('ndvi') <= 4715.5) ?  0.14195861643892793 :   0.040331082688044376 :   (b('l8b5') <= 3257.240234375) ?  (b('ndvi') <= 876.5) ?  0.11873538601232472 :   (b('dg0_v_vv') <= -0.6580452919006348) ?  0.04034818810597369 :   -0.00750222636404288 :   (b('dg0_v_vv') <= -0.945713996887207) ?  (b('l8b5_med') <= 3032.838623046875) ?  0.14947368921959728 :   0.08647234623388106 :   0.006787715215783335 :   (b('moss') <= 2.898351311683655) ?  (b('moss') <= 1.7632771134376526) ?  (b('l8b5') <= 3728.753662109375) ?  (b('bulk') <= 149.0) ?  0.0730968195365854 :   0.03934082319075907 :   (b('l8b4') <= 480.1725311279297) ?  0.10117547610061534 :   0.17581727089259178 :   (b('l8b4') <= 577.8113708496094) ?  0.16609578879422454 :   (b('l8b4') <= 923.8953552246094) ?  0.11831585254874873 :   0.14545684002056936 :   (b('ndvi') <= 4649.5) ?  (b('ndvi') <= 4059.0) ?  (b('ndvi') <= 1512.0) ?  0.09663913283440628 :   0.0411627808052184 :   (b('l8b4') <= 523.7588806152344) ?  0.025810560890416584 :   0.08618022671836018 :   (b('l8b5') <= 3390.1072998046875) ?  (b('l8b5') <= 3252.1834716796875) ?  0.10359170849686943 :   0.08480100852009331 :   (b('dg0_v_vv') <= 0.3984818458557129) ?  -0.038643904594963134 :   -0.003749305988280024 :   (b('grass') <= 69.81828308105469) ?  (b('bulk') <= 163.5) ?  (b('dg0_v_vv') <= 0.19564390182495117) ?  (b('l8b11_med') <= 1248.767822265625) ?  (b('l8b4') <= 432.4931640625) ?  -0.0015882124001849092 :   -0.075027265764597 :   (b('l8b4_med') <= 1011.082763671875) ?  0.003765826122423786 :   0.02229145452229382 :   (b('bulk') <= 161.5) ?  (b('ndvi_med') <= 4013.5) ?  0.025245113403209608 :   0.06574135008403852 :   (b('l8b4') <= 776.1520690917969) ?  0.1733813268594772 :   0.111707291092872 :   (b('l8b5') <= 1514.955322265625) ?  (b('dg0_v_vv') <= 4.848642110824585) ?  (b('crops') <= 64.45532989501953) ?  0.008158397280261192 :   0.03418798743809412 :   -0.034050466877070584 :   (b('l8b11') <= 941.069091796875) ?  (b('l8b5_med') <= 2487.8336181640625) ?  0.012070137513163381 :   -0.008429859327790445 :   (b('l8b5_med') <= 2487.8336181640625) ?  -0.025772402974851685 :   -0.0596541460890632 :   (b('ndvi_med') <= 2437.5) ?  (b('grass') <= 71.74169921875) ?  (b('ndvi') <= 1933.5) ?  -0.05006370554378642 :   (b('ndvi_med') <= 1909.5) ?  0.12209130444334187 :   0.19960607164507374 :   (b('dg0_v_vv') <= 0.4793086051940918) ?  (b('ndvi') <= 3477.0) ?  0.01510389079047429 :   0.07872626171158723 :   (b('dg0_v_vv') <= 4.757458686828613) ?  0.06860078302247923 :   0.1481509594521491 :   (b('ndvi') <= 3615.5) ?  (b('l8b5_med') <= 2370.6107177734375) ?  (b('dg0_v_vv') <= -0.36174869537353516) ?  -0.007252038013678935 :   0.07259990266191489 :   (b('l8b4') <= 543.7127380371094) ?  0.042501359451678156 :   -0.04013557465880814 :   (b('l8b11_med') <= 1650.5877685546875) ?  (b('dg0_v_vv') <= 1.6955714225769043) ?  0.04368440772424607 :   0.17848014752597083 :   (b('l8b11') <= 1402.8045654296875) ?  -0.024147762899637774 :   0.04190283419881994 :   (b('dg0_v_vv') <= 0.9214684963226318) ?  (b('moss') <= 24.19915199279785) ?  (b('moss') <= 21.619216918945312) ?  (b('l8b4') <= 649.4373779296875) ?  (b('dg0_v_vv') <= -1.0236334800720215) ?  (b('ndvi') <= 2854.0) ?  -0.0016959982839472088 :   -0.08106888587158441 :   (b('ndvi') <= 4044.5) ?  0.042963560590832756 :   -0.001106153317648478 :   (b('l8b5') <= 2874.925048828125) ?  (b('l8b11_med') <= 1304.1707763671875) ?  -0.06446326244607106 :   -0.009639743315209537 :   (b('l8b4') <= 885.3081970214844) ?  0.013371707764651403 :   0.05407743432330353 :   (b('ndvi') <= 1135.5) ?  -0.05644375990184154 :   (b('ndvi') <= 1455.5) ?  (b('dg0_v_vv') <= 0.1346268653869629) ?  0.1907708691393038 :   0.19502791598529715 :   (b('dg0_v_vv') <= -0.2993149757385254) ?  0.21294923437552132 :   0.08729597381642265 :   (b('ndvi') <= 2860.0) ?  (b('moss') <= 28.85543727874756) ?  (b('l8b11') <= 1699.373046875) ?  (b('ndvi_med') <= 1505.75) ?  0.06422089729295494 :   -0.047771403552512776 :   (b('dg0_v_vv') <= 0.20862197875976562) ?  0.05868345156057232 :   0.10725811852773949 :   (b('l8b11_med') <= 2979.461669921875) ?  (b('ndvi_med') <= 1200.5) ?  -0.04919550767760865 :   -0.01662089724723682 :   (b('ndvi_med') <= 1535.5) ?  0.06041757014478141 :   0.04790125281941138 :   (b('ndvi') <= 2968.0) ?  (b('l8b11') <= 1334.0712280273438) ?  (b('l8b5_med') <= 2527.20849609375) ?  0.0013566006633545236 :   0.018968943213288857 :   (b('ndvi_med') <= 2213.5) ?  -0.010867596680569414 :   0.08536100405036932 :   (b('dg0_v_vv') <= -0.909515380859375) ?  (b('ndvi') <= 4705.5) ?  0.07824992374915535 :   -0.04039924920485771 :   (b('l8b11_med') <= 1703.7199096679688) ?  -0.014660267445877794 :   0.08204629986121066 :   (b('l8b11_med') <= 1360.8564453125) ?  (b('l8b11_med') <= 1064.426025390625) ?  (b('l8b11') <= 981.4255981445312) ?  (b('l8b4') <= 427.57713317871094) ?  0.07635490764369796 :   (b('l8b11_med') <= 1018.512451171875) ?  0.056359920935383806 :   0.046751610766646606 :   -0.06711457554819089 :   (b('dg0_v_vv') <= 1.569852352142334) ?  (b('l8b11') <= 908.4990539550781) ?  0.01827940576867132 :   (b('l8b5') <= 2111.8399658203125) ?  -0.045424547258641854 :   -0.07839755577695373 :   (b('l8b11') <= 1025.6437683105469) ?  (b('dg0_v_vv') <= 1.991347312927246) ?  -0.12609565823694455 :   -0.14426190111486836 :   -0.09970112271121409 :   (b('moss') <= 26.70722484588623) ?  (b('l8b4') <= 876.6275329589844) ?  (b('moss') <= 14.83458662033081) ?  (b('l8b5') <= 3648.9593505859375) ?  -0.01657100443946512 :   0.042269552823668836 :   (b('ndvi') <= 1294.5) ?  -0.09080160680214458 :   0.07407245366189959 :   (b('ndvi') <= 1394.5) ?  0.056950998399209446 :   (b('l8b5_med') <= 2818.5750732421875) ?  -0.05089112073809431 :   -0.011368890379966278 :   (b('ndvi_med') <= 2747.5) ?  (b('ndvi') <= 4250.5) ?  (b('bulk') <= 142.0) ?  0.030947010808151144 :   -0.022608150433024086 :   0.12261389819771132 :   (b('l8b5') <= 2187.796630859375) ?  0.098539492281702 :   (b('l8b4_med') <= 1105.0031433105469) ?  0.03184434033145561 :   0.05636416243112389 :   (b('l8b11_med') <= 2570.105224609375) ?  (b('crops') <= 58.930376052856445) ?  (b('dg0_v_vv') <= 1.6657252311706543) ?  (b('ndvi_med') <= 2760.75) ?  (b('l8b11_med') <= 1988.9616088867188) ?  (b('sand') <= 35.5) ?  (b('l8b11_med') <= 1640.2098388671875) ?  (b('l8b5') <= 2926.5931396484375) ?  (b('ndvi') <= 1485.5) ?  0.10596807287386686 :   -0.008774441278885282 :   (b('dg0_v_vv') <= -0.15875625610351562) ?  0.09106120116111932 :   0.23495966966922624 :   (b('ndvi') <= 2884.5) ?  (b('moss') <= 6.990145683288574) ?  -0.057487898202817835 :   -0.04102769393740675 :   (b('l8b5') <= 1876.4083251953125) ?  0.07680344265255128 :   -0.02745652141352161 :   (b('bulk') <= 132.5) ?  (b('ndvi_med') <= 2556.75) ?  (b('l8b5') <= 1776.0261840820312) ?  0.07055850158230165 :   -0.03345392500298616 :   (b('l8b11_med') <= 1747.6846923828125) ?  -0.059470139377880295 :   -0.08558651361137067 :   (b('l8b11') <= 1948.2685546875) ?  (b('sand') <= 38.5) ?  0.0006864514159374265 :   -0.01873113913758845 :   (b('grass') <= 55.9560432434082) ?  -0.04016799616228806 :   -0.023596202606168785 :   (b('grass') <= 77.98130798339844) ?  (b('l8b11_med') <= 2417.81494140625) ?  (b('l8b11_med') <= 2050.369873046875) ?  (b('l8b4') <= 1106.0482177734375) ?  0.04484957650218699 :   -0.011042413193615382 :   (b('l8b4_med') <= 1233.010986328125) ?  -0.03600527511728433 :   -0.020201580661714222 :   (b('l8b4') <= 1185.8775634765625) ?  (b('ndvi_med') <= 1702.0) ?  0.09974754922730482 :   0.0034681875827980552 :   (b('moss') <= 38.18636512756348) ?  -0.009652424859821988 :   0.06331740439939561 :   (b('ndvi_med') <= 2109.75) ?  (b('l8b4') <= 903.9888610839844) ?  (b('grass') <= 83.9271240234375) ?  0.15162885751201205 :   0.016959456160434247 :   (b('l8b5') <= 3361.9205322265625) ?  -0.010429280196977147 :   0.1962269248784385 :   (b('dg0_v_vv') <= 0.10889816284179688) ?  (b('ndvi') <= 1221.0) ?  0.05981776001792645 :   0.010685335381780644 :   (b('l8b5') <= 2212.196044921875) ?  0.103627046475456 :   0.031049143851093006 :   (b('grass') <= 32.961503982543945) ?  (b('l8b5_med') <= 2790.2403564453125) ?  (b('bulk') <= 140.5) ?  (b('l8b5') <= 2571.0272216796875) ?  (b('moss') <= 7.090433120727539) ?  -0.00024027340275791648 :   0.03755025324449145 :   (b('l8b5') <= 3061.4375) ?  -0.023753124909401194 :   0.02032232609157742 :   (b('l8b11') <= 1867.7252807617188) ?  (b('l8b4') <= 1005.1083984375) ?  0.12365156035505731 :   0.11666414705945394 :   (b('l8b5') <= 2633.826171875) ?  0.09439069467912863 :   0.06837386480505786 :   (b('bulk') <= 128.5) ?  (b('ndvi') <= 4113.5) ?  (b('dg0_v_vv') <= 0.556304931640625) ?  0.07820377445884694 :   0.15284849272774392 :   (b('l8b11_med') <= 1065.25048828125) ?  0.016520263787593813 :   0.043410070885744695 :   (b('l8b4') <= 691.6106567382812) ?  (b('dg0_v_vv') <= 0.2948336601257324) ?  -0.07017249623984859 :   -0.07964782133164745 :   (b('grass') <= 25.49624538421631) ?  -0.006643698791304947 :   -0.021733404312259022 :   (b('bulk') <= 143.5) ?  (b('l8b11_med') <= 1269.7872924804688) ?  (b('grass') <= 56.4937686920166) ?  (b('crops') <= 37.04895210266113) ?  0.005150450905657083 :   0.055009596997398866 :   (b('dg0_v_vv') <= -2.751765251159668) ?  0.1410114265832919 :   0.09448449540190956 :   (b('l8b5') <= 3394.5250244140625) ?  (b('l8b5_med') <= 3307.21044921875) ?  -0.012129839060503652 :   0.05752197023128009 :   (b('dg0_v_vv') <= -0.5902910232543945) ?  -0.06566766796327486 :   -0.035866116448558964 :   (b('ndvi') <= 2284.5) ?  (b('dg0_v_vv') <= -2.5135178565979004) ?  (b('ndvi') <= 1566.5) ?  -0.06352559444640074 :   -0.025876858693925864 :   (b('moss') <= 6.059974908828735) ?  0.07466981222675333 :   0.13953542038796934 :   (b('dg0_v_vv') <= 0.3162083625793457) ?  (b('l8b11_med') <= 1548.8165283203125) ?  0.04673373432652881 :   0.0030653828450367755 :   (b('dg0_v_vv') <= 1.192558765411377) ?  0.07072681436146724 :   0.023782926821721728 :   (b('grass') <= 79.11454772949219) ?  (b('sand') <= 25.5) ?  (b('l8b11') <= 1762.2595825195312) ?  0.08896014207904518 :   (b('ndvi_med') <= 4372.5) ?  (b('l8b11') <= 1797.3060302734375) ?  0.13049639452531625 :   (b('dg0_v_vv') <= 1.971078872680664) ?  0.1135339366755278 :   0.10812763014507085 :   0.09340352532179891 :   (b('l8b5') <= 2385.921630859375) ?  (b('dg0_v_vv') <= 5.223286867141724) ?  (b('bulk') <= 144.5) ?  (b('ndvi_med') <= 1015.0) ?  0.1452869882511552 :   0.04488023374842072 :   (b('grass') <= 43.9757080078125) ?  0.14319160565020486 :   -0.023870141970058854 :   (b('l8b5_med') <= 2418.692138671875) ?  (b('l8b5_med') <= 2267.0) ?  -0.07597634719666084 :   -0.061507121468777515 :   (b('crops') <= 50.504770278930664) ?  -0.026346908105160164 :   0.028066384888550677 :   (b('l8b5_med') <= 3330.356689453125) ?  (b('l8b5') <= 2634.805419921875) ?  (b('l8b11') <= 2588.4783935546875) ?  -0.035863327231302894 :   0.0568774473719297 :   (b('l8b11_med') <= 2421.9002685546875) ?  -0.009774156411684198 :   0.06429591024885649 :   (b('l8b5') <= 3059.3392333984375) ?  0.032914695953922296 :   (b('bulk') <= 128.5) ?  0.09396168700190743 :   0.10020983779668735 :   (b('l8b5_med') <= 2643.062255859375) ?  (b('l8b4') <= 1137.1513671875) ?  (b('bulk') <= 137.5) ?  (b('l8b5_med') <= 2065.3483276367188) ?  0.0038586562271245295 :   (b('l8b4_med') <= 1254.5942993164062) ?  -0.00868967941828891 :   -0.015553516435835718 :   (b('l8b11_med') <= 1945.9461059570312) ?  (b('l8b4') <= 941.5990295410156) ?  0.07317233776599638 :   -0.04245292252463839 :   (b('l8b5_med') <= 2465.26171875) ?  0.05883892960261322 :   0.09096468334623176 :   (b('sand') <= 32.0) ?  (b('ndvi') <= 1575.0) ?  (b('ndvi') <= 1473.0) ?  -0.06189817329646416 :   -0.07780599083375336 :   (b('l8b5_med') <= 2108.4293823242188) ?  -0.04419953931896643 :   -0.03967996029697263 :   (b('bulk') <= 142.5) ?  (b('l8b4') <= 1503.0380249023438) ?  0.10795718211753544 :   0.12865524851791602 :   (b('dg0_v_vv') <= 3.6609911918640137) ?  0.030813030577120013 :   -0.026795957739123022 :   (b('l8b11') <= 1909.5586547851562) ?  (b('l8b4_med') <= 1274.1759033203125) ?  0.07305420604875021 :   0.07853929303975365 :   (b('dg0_v_vv') <= 3.07393741607666) ?  (b('dg0_v_vv') <= 2.788949966430664) ?  (b('l8b11') <= 2307.181396484375) ?  0.10330905495151904 :   0.11901622923736685 :   0.0840155852210133 :   (b('ndvi_med') <= 2303.25) ?  (b('bulk') <= 159.5) ?  0.13305591727347482 :   0.13849809449114064 :   0.1514645283747737 :   (b('crops') <= 99.79462814331055) ?  (b('dg0_v_vv') <= -1.1364617347717285) ?  (b('crops') <= 78.23976135253906) ?  (b('moss') <= 2.241892158985138) ?  (b('grass') <= 24.250466346740723) ?  (b('dg0_v_vv') <= -3.656632900238037) ?  (b('l8b11') <= 3078.45849609375) ?  -0.05908777101624395 :   -0.05369047374663721 :   (b('l8b11') <= 2502.25732421875) ?  -0.052995010130757886 :   -0.03645069929915171 :   (b('l8b5') <= 3283.8468017578125) ?  (b('l8b11') <= 3159.4447021484375) ?  -0.09547835284583003 :   -0.07001808723159521 :   -0.05548165498221627 :   (b('grass') <= 17.282055854797363) ?  (b('ndvi') <= 3421.5) ?  (b('l8b11') <= 2186.64453125) ?  -0.015884469479470135 :   -0.03751010093846392 :   (b('l8b4_med') <= 769.0651245117188) ?  -0.02647831302700481 :   0.021566659705038446 :   (b('l8b5_med') <= 3042.8221435546875) ?  (b('ndvi') <= 2577.5) ?  -0.02144795944566759 :   0.016596899414559767 :   (b('dg0_v_vv') <= -1.548701286315918) ?  0.020290266356851303 :   0.07762637409299422 :   (b('l8b5') <= 1854.1934814453125) ?  (b('l8b11') <= 2238.3591918945312) ?  (b('ndvi_med') <= 2029.0) ?  (b('ndvi') <= 1672.5) ?  0.08094633719529468 :   0.07539175405666643 :   (b('dg0_v_vv') <= -1.5584993362426758) ?  0.08796350639294068 :   0.08450033098892654 :   0.06812155707936932 :   (b('moss') <= 0.34493397921323776) ?  (b('ndvi') <= 1776.5) ?  (b('ndvi') <= 1410.5) ?  -0.004991358938350336 :   -0.026168256385908518 :   (b('l8b4') <= 1108.9265747070312) ?  0.07748370640082167 :   0.02203750164742986 :   (b('l8b5') <= 2453.0882568359375) ?  (b('l8b5') <= 2404.6258544921875) ?  0.0040076073427173925 :   0.05981469477963023 :   (b('l8b11') <= 2943.702880859375) ?  -0.005402008401748197 :   0.022722493892501822 :   (b('bulk') <= 145.5) ?  (b('moss') <= 1.4308947920799255) ?  (b('ndvi') <= 4341.5) ?  (b('ndvi') <= 2436.5) ?  (b('l8b11') <= 3051.122314453125) ?  0.0696589203710526 :   0.0058779350807379805 :   (b('dg0_v_vv') <= 3.4263625144958496) ?  0.033065339175763854 :   0.08411173566770709 :   (b('bulk') <= 130.5) ?  (b('l8b5') <= 3121.7799072265625) ?  0.04601967374193766 :   -0.0027082077308960097 :   (b('ndvi_med') <= 2910.0) ?  -0.055884118353788065 :   -0.06566817148008189 :   (b('l8b4') <= 1940.3153686523438) ?  (b('l8b11_med') <= 1991.1105346679688) ?  (b('bulk') <= 137.0) ?  0.013275613729821233 :   0.0334022061976506 :   (b('dg0_v_vv') <= 0.12292122840881348) ?  -0.023269639529647675 :   0.005266062970665988 :   (b('ndvi') <= 2347.5) ?  (b('l8b4_med') <= 1115.8173828125) ?  -0.011068104331565964 :   -0.01409453751278722 :   (b('grass') <= 14.831022262573242) ?  0.1638303147116157 :   0.10063332902127155 :   (b('l8b5_med') <= 2893.3145751953125) ?  (b('l8b4') <= 1033.9186401367188) ?  (b('ndvi') <= 1440.0) ?  (b('l8b4_med') <= 1530.0742797851562) ?  -0.020745172031914616 :   -0.031906214596969684 :   (b('dg0_v_vv') <= 0.08331871032714844) ?  0.026071358470412153 :   0.008591198335390703 :   (b('l8b11') <= 1761.3284301757812) ?  (b('l8b5_med') <= 2574.4525146484375) ?  0.05813770021058487 :   0.01563757666164013 :   (b('l8b5') <= 2467.034423828125) ?  -0.04207546075618755 :   -0.020755156209619215 :   (b('moss') <= 0.0533333346247673) ?  (b('l8b5') <= 3048.988037109375) ?  (b('dg0_v_vv') <= 1.593216896057129) ?  -0.00737150283117282 :   0.0639004518080907 :   (b('l8b4') <= 2059.5665893554688) ?  -0.04963346672498341 :   0.017649410396206457 :   (b('crops') <= 75.33199691772461) ?  (b('l8b4') <= 943.7453308105469) ?  -0.02689026339386976 :   0.051805207140387964 :   (b('dg0_v_vv') <= 2.2326669692993164) ?  0.012546875868228294 :   0.04855193112314039 :   (b('l8b11') <= 1856.8114624023438) ?  (b('dg0_v_vv') <= 0.05078387260437012) ?  0.0626594039800743 :   (b('ndvi_med') <= 1806.25) ?  0.04564959918327727 :   0.04876910354148689 :   (b('l8b5_med') <= 3165.2183837890625) ?  (b('bulk') <= 96.0) ?  0.063475785159818 :   (b('dg0_v_vv') <= -5.865630626678467) ?  0.07995697846936761 :   (b('l8b5') <= 2652.5208740234375) ?  (b('ndvi_med') <= 1383.5) ?  0.11954625832430849 :   0.10824966804538615 :   (b('dg0_v_vv') <= 0.4863743782043457) ?  0.12887408476758022 :   0.12462399708248051 :   (b('dg0_v_vv') <= -2.7762346267700195) ?  0.1754714556041947 :   0.15126936430394247 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b11') <= 2665.5185546875) ?  (b('l8b5') <= 3576.2919921875) ?  (b('dg0_v_vv') <= 2.4147143363952637) ?  (b('dg0_v_vv') <= 1.1679191589355469) ?  (b('sand') <= 54.0) ?  (b('ndvi') <= 1046.0) ?  -0.06150162770103279 :   (b('l8b5') <= 3497.5291748046875) ?  0.02761616668947697 :   0.05040108117545418 :   (b('l8b5_med') <= 3622.8421630859375) ?  (b('l8b5') <= 2083.7440185546875) ?  0.04710483488403566 :   0.05637238900368167 :   0.033068950407147274 :   (b('ndvi') <= 1341.0) ?  (b('bulk') <= 151.5) ?  0.030231689680245255 :   0.018231425629486958 :   (b('ndvi') <= 2230.5) ?  (b('l8b11') <= 2222.8779296875) ?  0.0033661514167424023 :   -0.012703284854050999 :   (b('ndvi_med') <= 1481.5) ?  0.00697421536795606 :   0.004216971360791438 :   (b('l8b4') <= 1005.4320373535156) ?  (b('dg0_v_vv') <= 3.655099630355835) ?  0.04923945028557525 :   0.09459946979059922 :   (b('l8b11_med') <= 2890.52392578125) ?  (b('l8b4') <= 1402.6660766601562) ?  0.053744924432381574 :   (b('dg0_v_vv') <= 3.73034930229187) ?  0.032345300530192106 :   0.021125208481935775 :   (b('dg0_v_vv') <= 4.219031810760498) ?  0.07074409251241642 :   0.05636506731635807 :   (b('l8b4') <= 2273.11865234375) ?  (b('dg0_v_vv') <= -0.11760950088500977) ?  (b('moss') <= 6.368875980377197) ?  (b('dg0_v_vv') <= -1.0903396606445312) ?  (b('dg0_v_vv') <= -1.3023428916931152) ?  -0.01951928722809594 :   0.0018952895914152723 :   (b('ndvi') <= 2683.0) ?  -0.017649763845263566 :   -0.023614297434897503 :   (b('ndvi') <= 1709.0) ?  (b('dg0_v_vv') <= -0.24473905563354492) ?  -0.0037614539238402006 :   -0.021917366482034628 :   0.020416646949733414 :   (b('l8b4') <= 1859.49951171875) ?  (b('ndvi') <= 3888.5) ?  0.009621688917230864 :   0.011465982516374151 :   (b('ndvi_med') <= 1349.5) ?  (b('ndvi') <= 2413.5) ?  -0.0035770899082875895 :   -0.009587674842113564 :   0.007412329807048729 :   (b('ndvi') <= 1754.5) ?  (b('dg0_v_vv') <= -0.5014300346374512) ?  0.01806774313037318 :   0.025267744310545154 :   (b('dg0_v_vv') <= 1.553025245666504) ?  (b('ndvi_med') <= 1349.5) ?  (b('l8b11_med') <= 2890.52392578125) ?  0.06449352967289665 :   0.058830864623805434 :   0.07089061673893246 :   0.04302271445640779 :   (b('ndvi') <= 937.5) ?  (b('dg0_v_vv') <= 1.6345438957214355) ?  (b('l8b5') <= 2320.261962890625) ?  (b('l8b11') <= 3020.302978515625) ?  (b('ndvi') <= 788.5) ?  (b('l8b11') <= 2932.763671875) ?  -0.042086449634684796 :   -0.050324039311926305 :   (b('ndvi') <= 873.5) ?  -0.027983892039360785 :   -0.05576489629990061 :   -0.06495567091354477 :   (b('l8b5') <= 2358.0760498046875) ?  (b('l8b4') <= 1869.5948486328125) ?  (b('dg0_v_vv') <= 0.37734222412109375) ?  -0.01968426805525879 :   -0.02459116081472708 :   (b('l8b11') <= 2996.432373046875) ?  0.014438092490608262 :   0.04032460792640655 :   (b('ndvi') <= 826.5) ?  (b('ndvi') <= 811.5) ?  -0.022090471918085487 :   0.013535153665342992 :   (b('l8b4_med') <= 3496.1298828125) ?  -0.028534738544553472 :   -0.04192406903291026 :   (b('l8b5_med') <= 4239.44873046875) ?  0.028265403457904154 :   0.01597267449610912 :   (b('l8b4') <= 2840.0167236328125) ?  (b('l8b11') <= 2735.2174072265625) ?  (b('ndvi') <= 1060.0) ?  (b('l8b11_med') <= 2890.52392578125) ?  (b('l8b4_med') <= 2271.3209228515625) ?  0.06544055967504148 :   0.061785808493382194 :   (b('l8b11') <= 2688.1181640625) ?  0.029134145647906423 :   -0.07652935198901098 :   (b('l8b4_med') <= 2271.3209228515625) ?  (b('dg0_v_vv') <= -0.6556463241577148) ?  -0.03799263042656708 :   -0.005871303506947451 :   (b('ndvi') <= 1703.5) ?  -0.045501859747398905 :   -0.0201327100300858 :   (b('l8b5') <= 4178.792724609375) ?  (b('l8b4') <= 2441.6597900390625) ?  (b('l8b5') <= 3259.9996337890625) ?  0.0016250627663424737 :   -0.024011625368384668 :   (b('l8b4_med') <= 1649.88818359375) ?  0.12089659994483895 :   0.00863107301827403 :   (b('ndvi') <= 2713.5) ?  0.08922848905657896 :   (b('dg0_v_vv') <= -0.5025768280029297) ?  0.052768245626326385 :   0.04459469610719792 :   (b('dg0_v_vv') <= -0.7740259170532227) ?  (b('dg0_v_vv') <= -1.7912588119506836) ?  -0.030339970392455964 :   (b('ndvi_med') <= 1481.5) ?  (b('crops') <= 91.9686164855957) ?  0.0027439860855065917 :   -0.005357838684010102 :   (b('dg0_v_vv') <= -0.9120087623596191) ?  0.016300947206945388 :   0.011300937074155776 :   (b('l8b4') <= 3078.5592041015625) ?  (b('l8b5_med') <= 3622.8421630859375) ?  (b('ndvi') <= 1291.0) ?  -0.028800800235528716 :   -0.021014766412436145 :   (b('dg0_v_vv') <= 0.4701666831970215) ?  -0.03702450246978561 :   -0.022794638160691078 :   (b('ndvi') <= 1310.0) ?  (b('l8b11') <= 3994.556640625) ?  -0.050147397144463604 :   -0.02587682088881335 :   (b('ndvi_med') <= 1349.5) ?  -0.05218250143674591 :   -0.04449245410637377 :   (b('dg0_v_vv') <= 1.680880069732666) ?  (b('l8b5_med') <= 3339.2752685546875) ?  (b('grass') <= 16.03073215484619) ?  (b('l8b4_med') <= 2324.4437255859375) ?  (b('ndvi') <= 3253.0) ?  (b('l8b5') <= 2309.8245849609375) ?  (b('crops') <= 70.93189811706543) ?  -0.03637445761250782 :   -0.016232627619612652 :   (b('ndvi') <= 1150.5) ?  -0.04694987821699787 :   -0.03897975699089151 :   (b('ndvi') <= 3315.5) ?  (b('l8b11_med') <= 2939.19091796875) ?  0.07493572352374613 :   0.05143045415607293 :   (b('l8b4') <= 2005.8223876953125) ?  -0.02872765235345928 :   0.01629994286830888 :   (b('ndvi') <= 642.5) ?  (b('ndvi') <= 634.0) ?  0.07690910509998403 :   (b('ndvi_med') <= 747.5) ?  0.034824116956076956 :   0.041906499115603665 :   (b('l8b11') <= 2397.4219970703125) ?  0.041593708673942624 :   (b('ndvi') <= 878.0) ?  -0.02659279010663732 :   0.004665480817802899 :   (b('grass') <= 18.961012840270996) ?  (b('ndvi') <= 1177.5) ?  (b('ndvi') <= 573.0) ?  0.042719654265494095 :   (b('ndvi') <= 730.0) ?  -0.027410093369179336 :   -0.0036587670844772766 :   (b('dg0_v_vv') <= -0.3774423599243164) ?  (b('ndvi') <= 1220.0) ?  0.13256048288182684 :   0.0006036372693259067 :   (b('l8b4') <= 2253.0789794921875) ?  0.04775072192827307 :   0.13011034522602474 :   (b('l8b5_med') <= 2619.382568359375) ?  (b('l8b5_med') <= 2513.924560546875) ?  (b('bulk') <= 142.5) ?  -0.0020873863457877904 :   -0.03264371530962035 :   (b('moss') <= 26.398873329162598) ?  -0.04951379284267563 :   -0.03355394624821802 :   (b('l8b4') <= 1394.4181518554688) ?  (b('bulk') <= 147.0) ?  0.06293806950960729 :   0.00734038284276052 :   (b('l8b5_med') <= 2647.819091796875) ?  0.019482149188363287 :   -0.02413592726874808 :   (b('bulk') <= 151.5) ?  (b('ndvi') <= 2465.5) ?  (b('l8b5') <= 3211.02294921875) ?  (b('l8b11') <= 2745.2608642578125) ?  (b('l8b4') <= 1516.482177734375) ?  -0.03580425727286208 :   -0.05088511898441091 :   (b('l8b11') <= 2748.1201171875) ?  0.02426751834547916 :   -0.035714353823350684 :   (b('l8b11') <= 3613.6263427734375) ?  (b('l8b4') <= 1432.6884765625) ?  -0.06354961775957456 :   -0.04925163114055948 :   (b('l8b11') <= 3627.4031982421875) ?  -0.02429259683897551 :   -0.044669026546363705 :   (b('l8b5') <= 3153.229248046875) ?  (b('l8b5') <= 3067.9737548828125) ?  (b('sand') <= 53.0) ?  -0.024418325477796265 :   0.019217474421611536 :   0.07745908714837152 :   (b('l8b5_med') <= 3523.59814453125) ?  (b('l8b4') <= 1669.9845581054688) ?  -0.05281448331686611 :   -0.022417138802246318 :   (b('l8b5') <= 3416.221923828125) ?  -0.0033051183791310557 :   -0.047934675641343766 :   (b('l8b5') <= 3445.42138671875) ?  (b('l8b11') <= 2983.216796875) ?  (b('crops') <= 37.01948165893555) ?  (b('l8b4_med') <= 2810.8577880859375) ?  -0.06754465233255533 :   -0.04894159971562058 :   -0.020698581348967016 :   (b('l8b5') <= 3048.0740966796875) ?  0.05567964264791414 :   (b('dg0_v_vv') <= -0.34925222396850586) ?  -0.018348703981872404 :   0.006544723467212492 :   (b('l8b4_med') <= 2736.4166259765625) ?  (b('ndvi') <= 976.5) ?  (b('l8b4') <= 2894.7744140625) ?  -0.035083583279587605 :   -0.023204452371665817 :   (b('ndvi') <= 1067.0) ?  -0.0011713020593518975 :   0.01816485274437262 :   (b('l8b11') <= 3522.10546875) ?  (b('dg0_v_vv') <= 0.8322906494140625) ?  -0.03737735319424475 :   -0.0584510167144875 :   (b('ndvi') <= 875.0) ?  0.011762269053238867 :   -0.02558524617422293 :   (b('grass') <= 17.66883087158203) ?  (b('l8b5') <= 2088.29052734375) ?  (b('bulk') <= 149.5) ?  0.06216231137103767 :   0.06470104320835285 :   (b('l8b11') <= 3245.4498291015625) ?  (b('l8b5_med') <= 3056.480712890625) ?  (b('dg0_v_vv') <= 2.43143892288208) ?  (b('ndvi') <= 1255.0) ?  -0.037411445186016634 :   -0.009289963015178093 :   (b('ndvi') <= 2507.5) ?  0.09312917738365276 :   0.02974380937991375 :   (b('l8b4') <= 2020.48583984375) ?  (b('l8b11') <= 2793.9603271484375) ?  -0.036642984807163895 :   -0.018639856508710816 :   (b('bulk') <= 149.5) ?  -0.0960525149462472 :   -0.0520096797637195 :   (b('l8b11') <= 3291.9000244140625) ?  (b('ndvi') <= 1760.5) ?  (b('l8b11') <= 3263.0826416015625) ?  0.05771812410601704 :   0.07954781406143605 :   (b('l8b4_med') <= 1803.0075073242188) ?  0.03797352991355152 :   0.004781423894176125 :   (b('dg0_v_vv') <= 3.1211788654327393) ?  (b('l8b4_med') <= 2008.2191772460938) ?  -0.04692389144285484 :   -0.023338164262265933 :   (b('l8b4_med') <= 2006.9535522460938) ?  0.02134601768118785 :   -0.027341882854403157 :   (b('moss') <= 8.695244312286377) ?  (b('l8b5_med') <= 3245.567626953125) ?  (b('l8b4') <= 1928.8226928710938) ?  (b('l8b4') <= 1172.0656127929688) ?  (b('ndvi_med') <= 2114.0) ?  -0.031211721470824683 :   -0.09382825974058345 :   (b('dg0_v_vv') <= 4.260805606842041) ?  0.023814982545944097 :   0.06830002841295693 :   (b('l8b11') <= 2604.5220947265625) ?  (b('dg0_v_vv') <= 2.5756616592407227) ?  -0.03243171573210002 :   0.04548633284534838 :   (b('l8b5') <= 2878.4345703125) ?  0.0070843741358957045 :   -0.0589220714264461 :   (b('ndvi') <= 1176.5) ?  (b('l8b4') <= 2090.8731689453125) ?  0.05970275848033768 :   (b('l8b4') <= 2547.8612060546875) ?  -0.018947055791842637 :   -0.02242995260458705 :   (b('l8b4') <= 2056.7996826171875) ?  (b('ndvi') <= 3314.0) ?  0.05800928742338481 :   0.09762636475074232 :   (b('ndvi') <= 1642.5) ?  0.1232099867659903 :   0.15604001204991935 :   (b('ndvi_med') <= 1679.25) ?  (b('ndvi') <= 1577.5) ?  (b('ndvi') <= 1426.0) ?  (b('bulk') <= 139.5) ?  0.04746027311786358 :   -0.008080847300062876 :   (b('ndvi_med') <= 1521.5) ?  -0.013551803587589431 :   -0.055204522868371284 :   (b('l8b5') <= 2644.8984375) ?  0.13927489726931433 :   (b('moss') <= 11.728094577789307) ?  -0.05678480822128042 :   0.018178198892434626 :   (b('moss') <= 18.607559204101562) ?  (b('ndvi') <= 1324.5) ?  (b('l8b5') <= 2431.6337890625) ?  0.0021138020641267574 :   -0.08830914231845435 :   (b('l8b5') <= 2991.1405029296875) ?  -0.014296629782203108 :   0.011340346102305351 :   (b('dg0_v_vv') <= 5.3809814453125) ?  (b('l8b5') <= 3677.83203125) ?  -0.03291556867480848 :   -0.055393505239939314 :   -0.08945077958530795 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1735.2388305664062) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('crops') <= 17.562152862548828) ?  (b('sand') <= 36.5) ?  (b('ndvi') <= 3130.0) ?  (b('ndvi') <= 2587.0) ?  (b('l8b5') <= 2897.7911376953125) ?  (b('dg0_v_vv') <= 2.311717987060547) ?  (b('ndvi') <= 1948.0) ?  0.07255102209121926 :   0.016387557108595817 :   (b('ndvi') <= 2262.0) ?  0.08581266224073456 :   0.08895578244122371 :   (b('dg0_v_vv') <= -0.75457763671875) ?  0.13017300447263877 :   (b('ndvi_med') <= 3569.0) ?  0.03748464440358096 :   0.059824974954455955 :   (b('l8b11') <= 858.7087707519531) ?  (b('dg0_v_vv') <= 0.7447218894958496) ?  (b('dg0_v_vv') <= -0.5115189552307129) ?  0.01920432882246188 :   0.045728908968424374 :   (b('l8b11') <= 844.2808532714844) ?  0.12703141798198228 :   0.07445764677060884 :   (b('moss') <= 4.940035104751587) ?  (b('l8b4_med') <= 573.5225219726562) ?  -0.03159921772776207 :   -0.042443780970965864 :   (b('ndvi') <= 2959.0) ?  -0.012175986885344778 :   0.02303997202158684 :   (b('l8b11') <= 1074.8532104492188) ?  (b('bulk') <= 128.0) ?  (b('l8b4_med') <= 599.630859375) ?  0.17748646519841937 :   0.12067204009500782 :   (b('dg0_v_vv') <= -0.8401851654052734) ?  (b('l8b4') <= 565.0333251953125) ?  0.11168362707354298 :   0.02787421869943954 :   (b('l8b5') <= 3660.7122802734375) ?  0.039463020530023506 :   0.08080784678851415 :   (b('ndvi') <= 3825.0) ?  (b('ndvi') <= 3199.5) ?  (b('l8b11_med') <= 1212.3648071289062) ?  -0.014359483372221801 :   0.015015317844667753 :   (b('ndvi') <= 3249.5) ?  0.0953091163463913 :   0.03280365509147431 :   (b('l8b11') <= 1089.1432495117188) ?  (b('l8b5') <= 3239.618896484375) ?  -0.025076616478422815 :   -0.08451434153047041 :   (b('l8b4') <= 527.1549987792969) ?  -0.10733942061844742 :   0.016089841018386605 :   (b('ndvi') <= 1582.0) ?  (b('ndvi') <= 1557.0) ?  (b('ndvi') <= 1493.0) ?  (b('l8b4_med') <= 595.4280090332031) ?  0.003610327031803612 :   0.012066120280059267 :   0.08161032119054842 :   0.23186321370822793 :   (b('l8b4') <= 585.3588562011719) ?  (b('ndvi') <= 3227.0) ?  (b('l8b4') <= 565.6223449707031) ?  (b('ndvi') <= 2143.0) ?  -0.020437692664452334 :   0.02697619578963292 :   (b('l8b5_med') <= 1590.654296875) ?  0.11014370659860756 :   0.14664322667293855 :   (b('bulk') <= 147.0) ?  (b('l8b4') <= 520.2550354003906) ?  -0.026804695006919534 :   -0.004464377016628754 :   (b('dg0_v_vv') <= 0.17944955825805664) ?  0.006493051992521842 :   0.05213515472911543 :   (b('bulk') <= 158.5) ?  (b('bulk') <= 147.0) ?  (b('l8b11') <= 1239.1609497070312) ?  -0.036168404032623516 :   -0.015264922277576178 :   (b('l8b5') <= 2457.1446533203125) ?  -0.06597546148624654 :   -0.08070758201684437 :   (b('dg0_v_vv') <= -0.6101760864257812) ?  (b('l8b4') <= 713.0617065429688) ?  0.05879355088360211 :   0.1527591616436583 :   (b('ndvi') <= 2390.5) ?  -0.07206182694338385 :   0.02618050736793613 :   (b('sand') <= 44.0) ?  (b('dg0_v_vv') <= 0.7859745025634766) ?  (b('ndvi_med') <= 4009.75) ?  (b('l8b11') <= 1491.2034912109375) ?  (b('l8b11') <= 1399.472412109375) ?  (b('l8b5') <= 2873.183349609375) ?  0.09633007856894081 :   0.046662291309808604 :   (b('dg0_v_vv') <= 0.6044206619262695) ?  0.005095632913359929 :   0.0651276976398634 :   (b('crops') <= 32.760751724243164) ?  (b('l8b11_med') <= 1267.6890258789062) ?  0.15690154621659266 :   0.18828268286966318 :   (b('ndvi') <= 3487.5) ?  0.09493067938507092 :   0.0668685914118556 :   (b('ndvi') <= 4382.5) ?  (b('l8b5') <= 4818.233154296875) ?  (b('ndvi') <= 4276.0) ?  -0.0030761142361679645 :   -0.10379504971214103 :   (b('bulk') <= 127.5) ?  0.05977151138719759 :   0.08941216852295783 :   (b('l8b4') <= 622.5991821289062) ?  (b('l8b11') <= 1174.21826171875) ?  0.034875900780879814 :   0.0953870064461323 :   (b('l8b5') <= 3600.916259765625) ?  0.11980903249754896 :   -0.0060472061559535585 :   (b('sand') <= 13.0) ?  (b('l8b5') <= 4337.11962890625) ?  (b('l8b4') <= 583.2243957519531) ?  (b('l8b11') <= 946.5736389160156) ?  0.17228665290786715 :   0.15883219896125572 :   0.14267019683233595 :   0.09234638055295263 :   (b('ndvi_med') <= 4364.5) ?  (b('dg0_v_vv') <= 1.2059030532836914) ?  (b('ndvi_med') <= 3984.5) ?  0.081745503341242 :   0.10586581048261309 :   (b('l8b4_med') <= 588.2196960449219) ?  0.0725521474254878 :   -0.0007086380440178702 :   (b('dg0_v_vv') <= 2.058229923248291) ?  (b('l8b11') <= 1307.303955078125) ?  -0.06087602956364033 :   -0.05321813051361868 :   0.10683166243954928 :   (b('ndvi_med') <= 4666.0) ?  (b('sand') <= 62.5) ?  (b('l8b5') <= 3140.85693359375) ?  (b('l8b11') <= 1189.4674072265625) ?  (b('ndvi') <= 4633.0) ?  0.0013929430869916621 :   -0.04950756934970359 :   (b('l8b11') <= 1377.6149291992188) ?  -0.05635504765447086 :   -0.09633689423924474 :   (b('l8b11') <= 1620.3944091796875) ?  (b('l8b11') <= 1350.5661010742188) ?  -0.011311604517093425 :   0.047274656012902684 :   -0.10061152390220887 :   (b('l8b5') <= 4104.932373046875) ?  (b('dg0_v_vv') <= -3.0536842346191406) ?  (b('l8b11') <= 1154.7627563476562) ?  -0.05325834921125934 :   -0.007185946038958432 :   (b('l8b4') <= 772.3627319335938) ?  0.030223699938468976 :   -0.011922547672973067 :   (b('l8b11_med') <= 1153.4663696289062) ?  (b('ndvi') <= 3805.5) ?  0.009210500127349239 :   -0.02960352106744078 :   (b('ndvi') <= 4804.5) ?  -0.006044355100876038 :   0.02912241497732154 :   (b('dg0_v_vv') <= 0.9826679229736328) ?  (b('l8b5') <= 3078.330078125) ?  (b('dg0_v_vv') <= 0.624910831451416) ?  (b('dg0_v_vv') <= -0.5036697387695312) ?  0.09996987539323644 :   0.033938474912456496 :   0.15201395497874465 :   (b('ndvi') <= 3256.0) ?  (b('l8b4_med') <= 585.4060668945312) ?  0.15527118264292644 :   0.10615882844115937 :   (b('l8b4') <= 455.1209259033203) ?  0.09185441077795709 :   -0.0021154058608813093 :   (b('ndvi') <= 3601.5) ?  (b('dg0_v_vv') <= 3.1400158405303955) ?  0.14935602667911163 :   (b('l8b4_med') <= 592.6703796386719) ?  0.11634507333278898 :   0.13139203464797833 :   0.18351174913207097 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1274.3564453125) ?  (b('grass') <= 51.872941970825195) ?  (b('l8b11') <= 1278.1806030273438) ?  (b('ndvi') <= 2613.0) ?  (b('l8b5') <= 2097.6336059570312) ?  (b('dg0_v_vv') <= 1.749094009399414) ?  0.033177657344425376 :   -0.0038019023825102416 :   (b('dg0_v_vv') <= -0.2777118682861328) ?  -0.059581558771253026 :   -0.016038364021290216 :   (b('dg0_v_vv') <= 0.04989957809448242) ?  (b('l8b4') <= 457.6837615966797) ?  0.014466504606354555 :   -0.014654560544264339 :   (b('bulk') <= 158.0) ?  0.08095560962156632 :   0.04490739210650564 :   (b('ndvi') <= 2467.0) ?  (b('l8b5') <= 2292.17236328125) ?  -0.03137032871913757 :   (b('l8b4') <= 957.5859680175781) ?  -0.052890604312662924 :   -0.06090496039288777 :   (b('l8b11') <= 1536.4625244140625) ?  (b('grass') <= 51.46572303771973) ?  -0.02948933108528412 :   -0.03415164504341239 :   (b('dg0_v_vv') <= -0.7680964469909668) ?  -0.0030158710758103335 :   -0.016672825531244898 :   (b('grass') <= 57.94600296020508) ?  (b('l8b4') <= 450.03575134277344) ?  (b('dg0_v_vv') <= -0.40523242950439453) ?  (b('ndvi_med') <= 3520.75) ?  0.10643430856583018 :   0.05986920081501752 :   (b('dg0_v_vv') <= 1.139845848083496) ?  0.1033903479331561 :   0.11878451079868305 :   (b('l8b4') <= 464.1109619140625) ?  (b('ndvi') <= 4080.0) ?  0.04385183731779484 :   -0.020354969589339522 :   (b('dg0_v_vv') <= -1.3657855987548828) ?  -0.00812637764885707 :   0.08390713742457874 :   (b('dg0_v_vv') <= -0.2964811325073242) ?  (b('ndvi') <= 3724.0) ?  (b('ndvi') <= 3638.5) ?  0.02011117693716174 :   0.0862765770849621 :   (b('l8b11') <= 487.0253448486328) ?  0.08473760106534617 :   -0.035209524298271996 :   (b('ndvi') <= 4845.0) ?  (b('moss') <= 6.887476682662964) ?  -0.0034213684746598524 :   0.054113483638310755 :   (b('l8b4') <= 537.8892364501953) ?  0.1594016326799308 :   0.10834512895764115 :   (b('l8b5') <= 2213.302734375) ?  (b('l8b11') <= 1377.208740234375) ?  (b('l8b11') <= 1322.887451171875) ?  (b('ndvi') <= 2931.5) ?  0.02580196774081797 :   0.02203159243574279 :   (b('l8b11') <= 1353.986328125) ?  0.039531925506314014 :   0.03785443104439534 :   (b('l8b11') <= 1417.676025390625) ?  -0.012640727681681133 :   -0.030881622369469175 :   (b('ndvi') <= 3066.0) ?  -0.09272453328553387 :   (b('ndvi') <= 4949.0) ?  (b('ndvi_med') <= 3776.0) ?  (b('l8b5_med') <= 2583.02294921875) ?  -0.030603323666581327 :   -0.0414997372082072 :   (b('dg0_v_vv') <= 1.4125561714172363) ?  -0.006762978244786088 :   -0.027081755012561423 :   -0.06601584273631816 :   (b('l8b4') <= 581.8955993652344) ?  (b('ndvi') <= 2667.5) ?  (b('moss') <= 3.9983750581741333) ?  (b('l8b4') <= 454.26145935058594) ?  (b('l8b11') <= 1194.6172485351562) ?  (b('dg0_v_vv') <= 0.2007613182067871) ?  0.1441964713759152 :   0.14169683256422777 :   0.1474407375054393 :   (b('dg0_v_vv') <= 0.7740201950073242) ?  (b('dg0_v_vv') <= -0.4347834587097168) ?  0.1403195255249586 :   0.13771250272690722 :   0.12002219648618323 :   (b('dg0_v_vv') <= 0.7107014656066895) ?  0.10866245557106674 :   0.10040559431893559 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b4') <= 216.6506118774414) ?  (b('dg0_v_vv') <= -0.020409107208251953) ?  (b('dg0_v_vv') <= -0.1702251434326172) ?  0.07861160066471284 :   0.07100383394486748 :   (b('dg0_v_vv') <= 0.6195993423461914) ?  0.09128583570909797 :   0.11036505079990455 :   (b('ndvi') <= 4424.5) ?  (b('moss') <= 0.9910394549369812) ?  0.11924673983260245 :   0.10769171474111787 :   (b('l8b11') <= 1029.7211608886719) ?  0.10365449598083461 :   0.12713050078043184 :   (b('l8b5') <= 3189.2669677734375) ?  (b('ndvi_med') <= 3208.25) ?  0.10983892170686937 :   (b('l8b5') <= 2981.356201171875) ?  0.0889590561339716 :   0.08104533273681781 :   (b('l8b4_med') <= 636.951416015625) ?  (b('dg0_v_vv') <= -0.05034971237182617) ?  0.11648781883422019 :   0.10120556080104888 :   (b('dg0_v_vv') <= -0.7770786285400391) ?  0.1232974130800033 :   0.12868109393570296 :   (b('l8b5') <= 2405.21044921875) ?  (b('dg0_v_vv') <= -1.36433744430542) ?  (b('ndvi') <= 3293.5) ?  (b('ndvi_med') <= 3438.5) ?  0.01741329144377507 :   0.0355736630251727 :   (b('dg0_v_vv') <= -2.3849072456359863) ?  (b('dg0_v_vv') <= -2.470510959625244) ?  -0.004470202620262288 :   0.03540535850403562 :   (b('ndvi_med') <= 3405.0) ?  -0.027767883506702007 :   -0.012391295703280934 :   (b('ndvi') <= 3523.5) ?  (b('dg0_v_vv') <= 0.06996679306030273) ?  (b('ndvi_med') <= 3494.75) ?  0.035525414278033435 :   0.0757470551737042 :   (b('dg0_v_vv') <= 0.3058357238769531) ?  0.08396437121619432 :   0.06979967026445656 :   -0.00843041427946567 :   (b('dg0_v_vv') <= -0.3883686065673828) ?  (b('l8b5') <= 2427.334228515625) ?  (b('ndvi') <= 3806.5) ?  (b('ndvi_med') <= 3405.0) ?  0.10738309312603561 :   0.1299788257632431 :   (b('ndvi') <= 4504.5) ?  0.0631713146410225 :   0.09689690429223312 :   (b('l8b4_med') <= 624.4176635742188) ?  (b('l8b4') <= 651.3680419921875) ?  0.09578659740469843 :   0.07538423881240702 :   (b('l8b11') <= 1192.5197143554688) ?  0.07372225899545715 :   0.011478163707563688 :   (b('l8b4_med') <= 585.992919921875) ?  (b('l8b4') <= 651.3680419921875) ?  (b('crops') <= 28.55040740966797) ?  0.07677280927471744 :   0.110297430409245 :   (b('ndvi') <= 2970.5) ?  0.08500655738566605 :   0.07617127358670599 :   (b('ndvi') <= 4307.0) ?  (b('dg0_v_vv') <= 3.0452075004577637) ?  0.11069455577795659 :   0.08585251957455164 :   (b('dg0_v_vv') <= 0.3353610038757324) ?  0.07127744575680535 :   0.13025087804470403 :   (b('bulk') <= 137.5) ?  (b('moss') <= 2.015839397907257) ?  (b('l8b4_med') <= 864.026123046875) ?  (b('ndvi') <= 4148.5) ?  (b('ndvi') <= 1749.0) ?  (b('l8b4') <= 1017.5579833984375) ?  (b('ndvi') <= 1502.5) ?  0.148424708776985 :   0.13395248034203294 :   0.08454268633037887 :   (b('dg0_v_vv') <= -1.633157730102539) ?  (b('ndvi') <= 1885.5) ?  (b('bulk') <= 116.5) ?  0.03591706664962502 :   0.02790354531124986 :   (b('ndvi_med') <= 3671.5) ?  -0.021163260040358584 :   0.020081294244941195 :   (b('l8b4') <= 870.8105163574219) ?  (b('l8b5') <= 2866.06103515625) ?  -0.005583414470369983 :   0.028704108887155435 :   (b('dg0_v_vv') <= -0.9756293296813965) ?  -0.01665136622953567 :   0.05858361006858863 :   (b('l8b5') <= 4637.2001953125) ?  (b('ndvi') <= 4619.5) ?  (b('l8b5') <= 2994.716552734375) ?  (b('crops') <= 75.03485870361328) ?  0.024694829481485048 :   -0.04446978062700743 :   (b('l8b11') <= 1211.9463500976562) ?  -0.09115061176524668 :   -0.0443779934884679 :   (b('ndvi_med') <= 3931.0) ?  (b('ndvi_med') <= 3391.25) ?  0.04146099768025186 :   -0.002032376034436615 :   (b('l8b11') <= 1000.2472839355469) ?  -0.006219367524696678 :   -0.06143556587722714 :   0.09845430732273999 :   (b('l8b5_med') <= 2578.979248046875) ?  (b('l8b5') <= 2742.910888671875) ?  (b('l8b11_med') <= 1572.3937377929688) ?  (b('ndvi_med') <= 2297.5) ?  0.04493221643316922 :   (b('dg0_v_vv') <= -0.3974442481994629) ?  0.0683314704923551 :   0.0670181979179485 :   (b('ndvi') <= 2186.5) ?  (b('l8b4') <= 730.0816650390625) ?  0.08194846510545856 :   0.11138132673816442 :   (b('l8b4_med') <= 955.3427429199219) ?  0.08639622485410987 :   0.07077324779576469 :   (b('dg0_v_vv') <= 0.4021570682525635) ?  (b('grass') <= 7.9687488079071045) ?  0.0773199594292075 :   0.09185280485247216 :   (b('ndvi') <= 2818.5) ?  (b('dg0_v_vv') <= 1.3532543182373047) ?  0.1104716633825629 :   0.09914088808314013 :   0.11118185296333666 :   (b('l8b11') <= 1340.7396850585938) ?  (b('l8b11_med') <= 1891.1978149414062) ?  (b('l8b4') <= 669.0605163574219) ?  (b('l8b5') <= 3539.82421875) ?  0.0511060845324042 :   0.019029969554641913 :   (b('ndvi') <= 4373.5) ?  0.07800926465225738 :   0.002914028297499968 :   (b('l8b5') <= 3085.065673828125) ?  (b('dg0_v_vv') <= 1.1629705429077148) ?  0.048276691148484235 :   0.10096775145147943 :   (b('dg0_v_vv') <= 0.34115004539489746) ?  0.08925234020549394 :   0.12367558580981697 :   (b('ndvi_med') <= 3428.5) ?  (b('l8b5_med') <= 2729.0048828125) ?  (b('dg0_v_vv') <= -0.061344146728515625) ?  -0.045267362873823686 :   -0.09371698818676313 :   (b('ndvi_med') <= 2241.25) ?  0.09403067841921436 :   0.002253261231462842 :   (b('grass') <= 17.76924228668213) ?  0.07857413467931834 :   0.1121574696931528 :   (b('bulk') <= 110.5) ?  (b('l8b4') <= 839.9110717773438) ?  (b('l8b5') <= 2458.689208984375) ?  (b('l8b5') <= 2192.0823974609375) ?  (b('dg0_v_vv') <= 0.2699613571166992) ?  (b('l8b5_med') <= 2437.6734619140625) ?  -0.06048747832678601 :   -0.07486024386434785 :   -0.09578969728021942 :   (b('l8b5_med') <= 2634.7596435546875) ?  (b('dg0_v_vv') <= -1.1041841506958008) ?  -0.03833541726811916 :   0.05622643616915894 :   (b('l8b4') <= 721.6745300292969) ?  -0.027466791211458952 :   0.009191048770546642 :   (b('ndvi') <= 3403.5) ?  (b('l8b5') <= 2706.499267578125) ?  (b('dg0_v_vv') <= 1.1340854167938232) ?  -0.06090634242907177 :   -0.08935406941625765 :   (b('l8b11') <= 1160.970703125) ?  -0.017928142207198204 :   -0.05273148442762937 :   (b('l8b11') <= 1082.634521484375) ?  0.15285475783701338 :   (b('l8b5') <= 2725.7630615234375) ?  0.011826381100087002 :   -0.06599878148632511 :   (b('dg0_v_vv') <= 4.304002285003662) ?  (b('dg0_v_vv') <= 1.1504957675933838) ?  (b('ndvi') <= 2229.0) ?  (b('dg0_v_vv') <= -0.2957916259765625) ?  -0.06110355726076849 :   -0.03163445599333943 :   (b('dg0_v_vv') <= 0.22052001953125) ?  -0.07486029550131178 :   -0.06257490382225668 :   (b('l8b11_med') <= 1613.077880859375) ?  -0.0976483076773902 :   -0.1183074061010084 :   -0.012187509998091123 :   (b('dg0_v_vv') <= 0.9618024826049805) ?  (b('ndvi_med') <= 3361.5) ?  (b('ndvi') <= 3836.5) ?  (b('l8b4_med') <= 896.981689453125) ?  (b('l8b4') <= 788.3464050292969) ?  -0.013255322413016048 :   -0.031057700447333955 :   (b('bulk') <= 129.5) ?  -0.03765946034092402 :   0.002547165441573085 :   (b('ndvi_med') <= 3116.0) ?  (b('l8b11') <= 1326.6142578125) ?  0.021371078126832314 :   0.0008461887344397735 :   (b('l8b4') <= 874.6788940429688) ?  -0.007968973086497706 :   -0.05437009710626028 :   (b('l8b5') <= 2534.7431640625) ?  (b('l8b5_med') <= 2287.4644775390625) ?  (b('l8b5_med') <= 2156.5162353515625) ?  0.09209290769121782 :   0.06731537600942641 :   (b('dg0_v_vv') <= -0.07884883880615234) ?  0.03836953137085833 :   0.011797814840370754 :   (b('l8b4') <= 915.791015625) ?  (b('l8b5_med') <= 2982.940185546875) ?  -0.002835814849573369 :   0.01635205023865775 :   (b('l8b4') <= 967.3245239257812) ?  -0.02190841414930423 :   -0.041752604162905445 :   (b('bulk') <= 116.0) ?  (b('dg0_v_vv') <= 1.281590461730957) ?  (b('l8b11') <= 1508.1181640625) ?  (b('l8b11') <= 1240.775146484375) ?  -0.07515779977001513 :   -0.07332576623202221 :   -0.11328814172668153 :   (b('ndvi_med') <= 4008.5) ?  (b('dg0_v_vv') <= 3.0924601554870605) ?  -0.03730098757296897 :   -0.047807273363481895 :   -0.059848814633060105 :   (b('l8b5_med') <= 3114.77099609375) ?  (b('l8b5_med') <= 2788.2109375) ?  (b('ndvi') <= 599.5) ?  -0.05512615160890527 :   0.015828159945679546 :   (b('dg0_v_vv') <= 8.096093654632568) ?  6.828212385458517e-05 :   -0.09100359158774052 :   (b('dg0_v_vv') <= 1.93461275100708) ?  (b('ndvi') <= 2245.0) ?  0.19525127937480347 :   0.09946142346352889 :   (b('l8b4') <= 687.4669494628906) ?  0.05065455478153529 :   -0.02953664086906805 :   (b('moss') <= 13.551528930664062) ?  (b('sand') <= 26.0) ?  (b('l8b5') <= 3419.1944580078125) ?  (b('dg0_v_vv') <= -1.1704630851745605) ?  (b('l8b4') <= 665.9411315917969) ?  (b('dg0_v_vv') <= -1.4591917991638184) ?  (b('ndvi') <= 3584.5) ?  -0.019955862005699862 :   -0.005495720394064106 :   (b('sand') <= 18.0) ?  -0.03834417593410286 :   -0.04269277100091637 :   (b('dg0_v_vv') <= -1.5911850929260254) ?  (b('l8b11') <= 1367.681640625) ?  0.11931003488626689 :   0.04758979895064504 :   (b('sand') <= 23.5) ?  -0.01702244001990666 :   0.0650490877606791 :   (b('l8b11') <= 1259.7227172851562) ?  (b('l8b4_med') <= 995.5265502929688) ?  (b('l8b5') <= 3138.1307373046875) ?  0.014900705861672697 :   0.07669677130626923 :   (b('dg0_v_vv') <= 2.2853167057037354) ?  0.1143681692777994 :   0.09414946274791258 :   (b('moss') <= 4.076430082321167) ?  (b('l8b5') <= 2520.4481201171875) ?  0.05991589542192006 :   0.11256604902266161 :   (b('dg0_v_vv') <= -0.3440675735473633) ?  -0.031078704893705705 :   0.05163102557768191 :   (b('l8b4') <= 548.9988403320312) ?  (b('grass') <= 55.14389419555664) ?  (b('ndvi_med') <= 3966.5) ?  (b('l8b4_med') <= 751.4978637695312) ?  0.14833217148064254 :   0.09124814744028631 :   -0.0008474014822610898 :   (b('l8b11') <= 1035.7786560058594) ?  (b('l8b11_med') <= 1512.6978759765625) ?  -0.05164761073332627 :   -0.08582029514939538 :   0.12762640565503963 :   (b('dg0_v_vv') <= -0.7015347480773926) ?  (b('dg0_v_vv') <= -1.2216687202453613) ?  (b('ndvi') <= 4103.5) ?  -0.05141546780273493 :   0.016945425682291376 :   (b('l8b4') <= 787.4713745117188) ?  -0.054812044369962704 :   -0.02494986511792316 :   (b('moss') <= 0.3046875) ?  (b('l8b11') <= 1218.966552734375) ?  0.030401858605020843 :   0.11362986433520525 :   (b('dg0_v_vv') <= -0.5935206413269043) ?  0.06925044817862532 :   0.0019281907682231246 :   (b('dg0_v_vv') <= -0.2807464599609375) ?  (b('grass') <= 64.03558540344238) ?  (b('grass') <= 58.87520980834961) ?  (b('ndvi_med') <= 1462.5) ?  (b('l8b11_med') <= 2844.9417724609375) ?  -0.07501757024753718 :   0.009363976518775935 :   (b('l8b4') <= 1330.9874877929688) ?  0.0079900778559075 :   0.07948979308337603 :   (b('ndvi_med') <= 1654.0) ?  (b('dg0_v_vv') <= -1.4012188911437988) ?  -0.14321928570901735 :   -0.10809471933130921 :   (b('l8b11') <= 1380.3159790039062) ?  -0.01862984261470305 :   -0.058088085385901406 :   (b('grass') <= 78.29829025268555) ?  (b('l8b5') <= 2465.691650390625) ?  (b('l8b4_med') <= 1112.7555541992188) ?  0.061631825424819923 :   -0.024926177270626265 :   (b('moss') <= 10.623213291168213) ?  0.05870508618530289 :   0.1538619410239704 :   (b('l8b4') <= 1253.0075073242188) ?  (b('l8b11_med') <= 2171.7244873046875) ?  -0.005756755412315252 :   0.05939395530731943 :   0.18392971475736739 :   (b('sand') <= 47.5) ?  (b('l8b4_med') <= 2059.6664428710938) ?  (b('l8b4_med') <= 1379.1030883789062) ?  (b('l8b4') <= 916.5168762207031) ?  0.038998502253702716 :   0.008752319569885689 :   (b('grass') <= 3.4026710987091064) ?  0.0016124378533123188 :   0.08218067009771214 :   (b('ndvi') <= 2535.0) ?  (b('l8b4') <= 1260.7611083984375) ?  0.0007150720932026397 :   -0.08528601058590378 :   (b('dg0_v_vv') <= 3.7688052654266357) ?  -0.07326658174102256 :   -0.022520421668720203 :   (b('grass') <= 93.00495910644531) ?  (b('l8b5') <= 3134.4471435546875) ?  (b('grass') <= 68.83333206176758) ?  0.024066942406394686 :   0.06932716180096878 :   (b('bulk') <= 144.5) ?  -0.05225161082224762 :   0.004747592548953389 :   (b('l8b4') <= 836.424072265625) ?  (b('l8b5') <= 2842.50732421875) ?  -0.033139844809188794 :   -0.08558085828963484 :   (b('l8b4') <= 894.8955993652344) ?  -0.07723242815992962 :   -0.049595125309909144 :   (b('dg0_v_vv') <= 0.03626537322998047) ?  (b('l8b4') <= 441.4360809326172) ?  (b('l8b4_med') <= 818.1032409667969) ?  (b('l8b11') <= 880.5991821289062) ?  (b('l8b5') <= 1886.753173828125) ?  (b('dg0_v_vv') <= -0.4116053581237793) ?  0.09130278680208007 :   0.06858750115912535 :   (b('ndvi') <= 3804.0) ?  0.013484232565375445 :   0.04838820916530959 :   (b('moss') <= 15.332817554473877) ?  0.11422938731097676 :   0.15517402783425535 :   (b('l8b4_med') <= 865.6775207519531) ?  -0.03963197195166174 :   (b('ndvi') <= 3843.0) ?  (b('dg0_v_vv') <= -0.43505096435546875) ?  0.017172421961786484 :   0.05963736655900245 :   -0.03252230137636364 :   (b('l8b11_med') <= 1440.0303955078125) ?  (b('l8b4') <= 907.9395446777344) ?  (b('l8b5_med') <= 2400.5274658203125) ?  (b('l8b5') <= 2614.9105224609375) ?  -0.06257503134013766 :   -0.027881220190418592 :   (b('dg0_v_vv') <= -0.14905786514282227) ?  -0.06690583461039988 :   -0.09345025146984691 :   (b('l8b11') <= 1505.1705932617188) ?  (b('dg0_v_vv') <= -0.2868189811706543) ?  -0.014086598650203189 :   -0.01671326315385059 :   -0.024613262772380864 :   (b('moss') <= 24.19915199279785) ?  (b('moss') <= 20.837696075439453) ?  (b('ndvi') <= 1960.5) ?  -0.028792863045360023 :   0.007118675826185054 :   (b('l8b4') <= 765.5046081542969) ?  0.17302847197947555 :   0.017933354565250848 :   (b('ndvi') <= 4677.0) ?  (b('l8b11') <= 1696.1181640625) ?  -0.028602280980526425 :   0.011962948860048188 :   (b('ndvi') <= 4710.5) ?  0.09744893755860115 :   -0.036047472749674396 :   (b('l8b4_med') <= 1093.2515258789062) ?  (b('l8b11_med') <= 1687.7662353515625) ?  (b('moss') <= 14.975155353546143) ?  (b('l8b5') <= 3654.60009765625) ?  (b('l8b11') <= 1559.9835815429688) ?  -0.02545751506321084 :   0.0003455580579550836 :   (b('ndvi') <= 4763.5) ?  0.048789504034020346 :   -0.001661659836886896 :   (b('l8b4') <= 948.5784606933594) ?  (b('ndvi') <= 4564.0) ?  0.02846688446588603 :   0.12436291602481324 :   (b('l8b4_med') <= 961.3511962890625) ?  -0.02304991838213317 :   0.05406893496486303 :   (b('l8b5_med') <= 2358.4151611328125) ?  (b('l8b11') <= 1478.531494140625) ?  (b('l8b5_med') <= 2245.4676513671875) ?  -0.042314915531262 :   -0.014569583040020552 :   (b('l8b4') <= 768.0749206542969) ?  0.12601689489306914 :   -0.007125110305963664 :   (b('ndvi_med') <= 2656.0) ?  0.0813508753149963 :   0.10614157209150776 :   (b('l8b11_med') <= 1903.0198974609375) ?  (b('l8b4') <= 727.9677429199219) ?  (b('dg0_v_vv') <= 2.225700855255127) ?  (b('dg0_v_vv') <= 0.6213016510009766) ?  -0.027598148612499568 :   0.017013042773105835 :   0.07111362875593036 :   (b('l8b11') <= 1627.3833618164062) ?  (b('ndvi_med') <= 1616.5) ?  0.17552512438676743 :   0.0999090957595113 :   (b('l8b4') <= 1062.1260986328125) ?  0.058033994986856366 :   0.03226603792271274 :   (b('l8b4_med') <= 1481.015380859375) ?  (b('l8b11_med') <= 2060.6083984375) ?  (b('l8b11') <= 1252.6304931640625) ?  -0.10191787457926521 :   0.011985851896800582 :   (b('l8b4') <= 1249.1597290039062) ?  0.0468936246941593 :   0.16867434740982112 :   (b('dg0_v_vv') <= 0.5784621238708496) ?  (b('grass') <= 34.96886157989502) ?  -0.07843846688562196 :   -0.030084950954481052 :   (b('sand') <= 51.0) ?  -0.010862113225756295 :   0.04690144005460639 :   (b('l8b11_med') <= 2635.76513671875) ?  (b('dg0_v_vv') <= 1.0458025932312012) ?  (b('crops') <= 38.36996650695801) ?  (b('ndvi') <= 2336.5) ?  (b('crops') <= 23.221932411193848) ?  (b('moss') <= 26.236469268798828) ?  (b('ndvi_med') <= 2918.25) ?  (b('l8b11_med') <= 2014.972412109375) ?  (b('l8b4') <= 1072.6869506835938) ?  -0.006145179299011881 :   -0.027099360281144624 :   (b('sand') <= 50.5) ?  -0.004177660241140851 :   -0.026939838648498596 :   (b('l8b5_med') <= 2865.7408447265625) ?  (b('bulk') <= 145.5) ?  0.01079920806169751 :   0.09578948379647512 :   (b('l8b5_med') <= 3104.2471923828125) ?  -0.05321958657738505 :   0.03706389632551046 :   (b('sand') <= 58.5) ?  (b('moss') <= 28.417470932006836) ?  (b('l8b11_med') <= 2199.0439453125) ?  -0.015703516521935218 :   -0.06474572437571328 :   (b('ndvi') <= 1792.5) ?  -0.02822093064704647 :   0.007523812868755628 :   (b('ndvi') <= 943.5) ?  (b('l8b5') <= 1975.8766479492188) ?  0.0442748401952418 :   0.16534855634214154 :   (b('l8b5') <= 2036.9194946289062) ?  0.014340441475419772 :   -0.04316173403247435 :   (b('l8b11') <= 2138.478271484375) ?  (b('ndvi') <= 1461.0) ?  (b('l8b11') <= 1919.4877319335938) ?  (b('l8b11_med') <= 1644.19921875) ?  -0.017561035194668617 :   0.021930235767650946 :   (b('grass') <= 43.24021911621094) ?  -0.10124701158544185 :   -0.04760579565593488 :   (b('moss') <= 14.452311992645264) ?  (b('ndvi') <= 2121.0) ?  -0.047923554292284475 :   -0.06863842805707743 :   (b('l8b11') <= 1949.8665161132812) ?  -0.016186594160902158 :   -0.05372118114827091 :   (b('l8b4') <= 1245.6098022460938) ?  (b('l8b5') <= 2082.145263671875) ?  (b('l8b5_med') <= 2882.512939453125) ?  0.04650623888876641 :   0.04011710273817033 :   (b('dg0_v_vv') <= -1.0583100318908691) ?  -0.0051166688038433855 :   0.020617752062196404 :   (b('l8b11') <= 2264.1912841796875) ?  (b('l8b11') <= 2148.58203125) ?  0.02465666980629902 :   -0.06284943448107311 :   (b('l8b5') <= 3113.4261474609375) ?  -0.0070040785485356045 :   -0.03475959850594567 :   (b('grass') <= 45.12700653076172) ?  (b('l8b5') <= 3214.5106201171875) ?  (b('l8b5') <= 3118.333251953125) ?  (b('l8b11') <= 1798.0419311523438) ?  (b('l8b4_med') <= 1256.9429321289062) ?  0.013931838008020699 :   0.1755295746715987 :   (b('grass') <= 24.99324893951416) ?  0.0009347125687542918 :   -0.026836134196011135 :   (b('ndvi') <= 3571.0) ?  (b('l8b11') <= 2169.2943115234375) ?  0.037999479122421645 :   0.14228697964171103 :   (b('l8b4') <= 1282.219482421875) ?  -0.03459559567173317 :   0.022717152794064305 :   (b('crops') <= 28.946438789367676) ?  (b('l8b4') <= 1375.1340942382812) ?  (b('ndvi') <= 3450.0) ?  -0.06054340385022274 :   -0.043056287459913196 :   (b('l8b5') <= 3488.389404296875) ?  -0.03538552612963392 :   -0.05296300965669685 :   (b('l8b11') <= 1801.3228759765625) ?  (b('ndvi') <= 3994.0) ?  -0.0293802532944267 :   -0.005946289978005753 :   (b('l8b11') <= 2100.591796875) ?  0.06191199510557993 :   0.009756752245055997 :   (b('l8b5_med') <= 2607.319580078125) ?  (b('bulk') <= 132.0) ?  (b('l8b11') <= 1809.8004760742188) ?  (b('l8b11') <= 1761.7882690429688) ?  -0.09526477072850535 :   -0.05927291087422604 :   (b('l8b11') <= 1845.4203491210938) ?  0.06402723136760431 :   -0.02230365835124547 :   (b('bulk') <= 156.5) ?  (b('grass') <= 78.81625747680664) ?  0.04490385329868187 :   0.00766672476643609 :   (b('l8b5') <= 2205.2020263671875) ?  -0.014514204874048475 :   -0.051901193610033654 :   (b('sand') <= 64.0) ?  (b('l8b11_med') <= 1271.7744750976562) ?  (b('l8b4') <= 1174.9598388671875) ?  -0.039424606590584554 :   -0.0017505869147912319 :   (b('l8b4') <= 1285.1192016601562) ?  0.08128762121370885 :   0.0223965169931927 :   (b('bulk') <= 137.5) ?  (b('ndvi_med') <= 3336.0) ?  0.06884059507598606 :   0.08073128909763355 :   (b('dg0_v_vv') <= -0.07224225997924805) ?  -0.03166746601547405 :   -0.10253504220860872 :   (b('sand') <= 17.0) ?  (b('dg0_v_vv') <= -2.488344192504883) ?  0.18590184344463045 :   (b('l8b4') <= 1145.2271728515625) ?  (b('l8b5') <= 3019.7684326171875) ?  (b('dg0_v_vv') <= 0.4251112937927246) ?  (b('bulk') <= 132.0) ?  0.11261593072922249 :   0.11128640431955159 :   (b('moss') <= 3.2715134620666504) ?  0.12155316702374425 :   0.11752779019138593 :   0.15856649502507225 :   (b('ndvi') <= 2423.5) ?  (b('ndvi_med') <= 3738.5) ?  0.07988776820026838 :   0.09527396440439273 :   (b('sand') <= 14.0) ?  0.0593869148126715 :   0.07041125948362487 :   (b('dg0_v_vv') <= -1.6278276443481445) ?  (b('crops') <= 78.23976135253906) ?  (b('ndvi') <= 4588.0) ?  (b('l8b4_med') <= 1819.289306640625) ?  (b('bulk') <= 121.0) ?  0.07049107277328993 :   -0.023719446524170757 :   (b('l8b4') <= 2163.775634765625) ?  0.039356948436257824 :   -0.0007244307308995853 :   (b('l8b4_med') <= 862.096435546875) ?  (b('l8b11') <= 1986.7250366210938) ?  -0.054889287750847254 :   0.006473593017486987 :   (b('ndvi_med') <= 2471.75) ?  0.10427126912613296 :   0.05226500399323083 :   (b('l8b4') <= 1105.4560546875) ?  (b('l8b5') <= 3441.713623046875) ?  (b('l8b4') <= 945.9223022460938) ?  0.018145416571049133 :   0.05204231540087889 :   (b('l8b5_med') <= 2520.3472900390625) ?  0.11598667629082221 :   0.15792431004377522 :   (b('l8b5') <= 3685.09521484375) ?  (b('crops') <= 78.62849044799805) ?  0.08832702808206999 :   -0.0004667830086010619 :   (b('l8b4_med') <= 938.6988220214844) ?  0.10193635458849612 :   0.014475089242815264 :   (b('l8b4') <= 1035.16357421875) ?  (b('ndvi_med') <= 3430.25) ?  (b('l8b5') <= 2767.4312744140625) ?  (b('l8b5') <= 2396.479248046875) ?  -0.00847664156124475 :   0.032594393546646894 :   (b('l8b4') <= 762.3768920898438) ?  -0.06322406203418919 :   -0.019502018166068245 :   (b('ndvi') <= 4739.5) ?  (b('ndvi_med') <= 4344.75) ?  0.014278067711994833 :   0.0658061367389388 :   (b('sand') <= 76.5) ?  -0.03140959733777669 :   -0.0054200245462315555 :   (b('crops') <= 39.53758239746094) ?  (b('dg0_v_vv') <= -1.476801872253418) ?  -0.014060712279875126 :   (b('l8b5') <= 3148.9908447265625) ?  0.05818178848141218 :   0.02863198532668379 :   (b('moss') <= 3.989913582801819) ?  (b('moss') <= 3.095021605491638) ?  0.008776964061776072 :   0.03824993970032076 :   (b('l8b5_med') <= 2630.8795166015625) ?  -0.015726804284540204 :   0.0053625198217392344 :   (b('grass') <= 85.01520538330078) ?  (b('ndvi_med') <= 2184.5) ?  (b('l8b4_med') <= 1697.1256713867188) ?  (b('sand') <= 58.5) ?  (b('bulk') <= 147.5) ?  (b('ndvi') <= 921.5) ?  (b('bulk') <= 145.5) ?  0.06197058975080556 :   0.1618203192689382 :   (b('crops') <= 77.44355773925781) ?  -0.001952749696684016 :   0.12040253481274776 :   (b('grass') <= 4.636483192443848) ?  (b('l8b11') <= 2235.6070556640625) ?  0.11363593221348357 :   0.011272737697324314 :   (b('l8b4') <= 908.9608459472656) ?  0.01935704152959829 :   -0.026368857844205906 :   (b('l8b5') <= 2189.7073974609375) ?  (b('dg0_v_vv') <= 1.9359917640686035) ?  0.10666797782549658 :   (b('l8b4') <= 1323.7211303710938) ?  0.13596841728052433 :   0.12554816157155507 :   (b('l8b4') <= 1152.6708374023438) ?  -0.021834963818531783 :   (b('dg0_v_vv') <= 1.3798828125) ?  0.08201087093075428 :   0.031125773383988708 :   (b('grass') <= 75.34359741210938) ?  (b('ndvi_med') <= 774.75) ?  (b('l8b11_med') <= 2514.2159423828125) ?  (b('sand') <= 56.0) ?  -0.055494290613335606 :   -0.05206583019746541 :   -0.04117187548316162 :   (b('dg0_v_vv') <= 3.5563220977783203) ?  (b('ndvi') <= 1206.0) ?  0.00036587563161346627 :   0.03763069866118984 :   (b('ndvi') <= 1146.0) ?  0.11350930005806115 :   0.06088034730357011 :   (b('l8b11_med') <= 2110.28173828125) ?  0.16463513901026655 :   0.18777270537061344 :   (b('grass') <= 36.288320541381836) ?  (b('crops') <= 55.03236198425293) ?  (b('l8b5_med') <= 2805.8912353515625) ?  (b('l8b4') <= 881.3539123535156) ?  (b('l8b11') <= 1916.0184326171875) ?  0.06512871062759153 :   -0.0301010321699538 :   (b('l8b11') <= 2029.6538696289062) ?  0.0024516974825248517 :   0.04356648219467504 :   (b('l8b5_med') <= 3330.356689453125) ?  (b('l8b4_med') <= 744.5271606445312) ?  0.05670378343659722 :   -0.0164286235678672 :   (b('sand') <= 61.0) ?  0.09964303558272956 :   0.010425844540951051 :   (b('l8b11') <= 2329.6783447265625) ?  (b('l8b11') <= 2317.6646728515625) ?  (b('bulk') <= 161.5) ?  0.03188839151200237 :   0.13092206890485367 :   0.1999088324382787 :   (b('ndvi') <= 4769.0) ?  (b('l8b5') <= 3402.5203857421875) ?  0.0010916222330801138 :   -0.06698107949393878 :   0.09456408022044549 :   (b('l8b11_med') <= 1573.7101440429688) ?  (b('moss') <= 5.447983980178833) ?  (b('crops') <= 40.11230659484863) ?  (b('l8b4') <= 916.2154846191406) ?  -0.009587884210642328 :   0.05485910463032236 :   (b('ndvi_med') <= 3396.0) ?  0.12633421360638464 :   0.13097404148639685 :   (b('ndvi') <= 1527.0) ?  -0.006521493427341402 :   (b('l8b11') <= 1991.772705078125) ?  0.08977609635291027 :   0.1326627755724771 :   (b('l8b5') <= 1844.2811279296875) ?  (b('l8b11') <= 2052.1300659179688) ?  (b('l8b4_med') <= 1055.79296875) ?  0.12038255313383307 :   0.19190343501663285 :   (b('l8b4_med') <= 953.8369445800781) ?  -0.04732066981596347 :   0.08473493175325242 :   (b('bulk') <= 116.0) ?  (b('grass') <= 62.045658111572266) ?  -0.055990112824433724 :   -0.0868572114299496 :   (b('ndvi') <= 2770.0) ?  0.006573883544023001 :   0.04346435792290737 :   (b('l8b5_med') <= 2485.7818603515625) ?  (b('l8b5') <= 2453.04296875) ?  (b('sand') <= 49.5) ?  (b('l8b4_med') <= 1377.0902709960938) ?  (b('l8b11') <= 2412.227294921875) ?  (b('l8b5') <= 2426.4798583984375) ?  0.05327731428834125 :   0.12526788340728076 :   (b('l8b5') <= 2328.5501708984375) ?  0.02061411074836173 :   -0.061585624310245835 :   (b('dg0_v_vv') <= 1.9377975463867188) ?  (b('dg0_v_vv') <= 1.2016587257385254) ?  0.10966390681485597 :   0.031077323539170743 :   (b('l8b4') <= 1099.0731201171875) ?  0.16080131260436653 :   0.10525215854719659 :   -0.07607497215809184 :   (b('dg0_v_vv') <= 4.89862585067749) ?  (b('l8b11') <= 2415.835205078125) ?  (b('sand') <= 40.5) ?  (b('l8b11_med') <= 2262.3712158203125) ?  0.04203322207630008 :   0.014754545728093177 :   (b('ndvi') <= 2687.5) ?  -0.0017489834964428488 :   -0.029943949879458653 :   (b('l8b5_med') <= 2282.89404296875) ?  (b('ndvi_med') <= 2093.0) ?  -0.06617151020641965 :   -0.07108573121728315 :   -0.052646722161763304 :   0.11997715439113829 :   (b('l8b4') <= 1205.1002807617188) ?  (b('l8b4') <= 1168.2620239257812) ?  (b('ndvi') <= 2232.0) ?  0.004411959485489669 :   (b('l8b11') <= 2000.4853515625) ?  (b('l8b5_med') <= 2738.65673828125) ?  0.064345211761258 :   0.07068536373577827 :   (b('dg0_v_vv') <= 1.561859130859375) ?  0.06556103870362404 :   0.08522807870338898 :   (b('l8b4_med') <= 1199.1158447265625) ?  0.0011946870446184754 :   0.021804107460300814 :   (b('l8b5_med') <= 2845.646240234375) ?  (b('l8b5') <= 2164.1356201171875) ?  (b('l8b11') <= 2414.0556640625) ?  (b('ndvi_med') <= 1989.0) ?  0.08911151406064671 :   0.09014711458450225 :   0.07561402669891198 :   (b('bulk') <= 157.0) ?  (b('l8b5') <= 2918.8343505859375) ?  0.12622146416673002 :   0.15564353330102423 :   (b('grass') <= 93.0868148803711) ?  0.0998425917104109 :   0.07295134933575242 :   0.019216592753437067 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b4') <= 1672.8639526367188) ?  (b('ndvi') <= 1489.0) ?  (b('ndvi') <= 1463.0) ?  (b('l8b11') <= 2895.7447509765625) ?  (b('l8b11') <= 2857.86865234375) ?  (b('l8b11_med') <= 2956.887451171875) ?  (b('ndvi') <= 1205.5) ?  0.04162116411491289 :   0.019237913131537162 :   (b('ndvi') <= 1075.0) ?  -0.01519707962846793 :   0.02175057508159288 :   (b('dg0_v_vv') <= 1.5648832321166992) ?  (b('l8b5_med') <= 2254.820556640625) ?  0.09469974598922373 :   0.07477163792399588 :   0.051902064974510026 :   (b('l8b4') <= 1610.2670288085938) ?  (b('dg0_v_vv') <= 0.7235713005065918) ?  -0.037035507984884336 :   (b('l8b5_med') <= 2846.0980224609375) ?  -0.030552603537012113 :   -0.028427930263296944 :   (b('ndvi') <= 1252.5) ?  (b('l8b4_med') <= 1649.88818359375) ?  -0.0224761778630488 :   -0.016337096555912776 :   (b('l8b4_med') <= 1649.88818359375) ?  0.0012623174253295205 :   -0.012901526893607258 :   (b('dg0_v_vv') <= 6.323261737823486) ?  (b('dg0_v_vv') <= 4.200683355331421) ?  (b('l8b4_med') <= 1649.88818359375) ?  (b('l8b11') <= 3109.673095703125) ?  0.0574310013225178 :   0.0543105265864573 :   (b('dg0_v_vv') <= 2.7770001888275146) ?  0.0613000285535821 :   0.06658176498269577 :   0.043779097708300074 :   0.09674014608578857 :   (b('l8b11') <= 2092.53515625) ?  (b('dg0_v_vv') <= 0.6969428062438965) ?  (b('dg0_v_vv') <= -0.2910895347595215) ?  (b('grass') <= 0.27586206793785095) ?  (b('l8b5_med') <= 2812.056640625) ?  0.02524098308681938 :   0.008253157337332 :   (b('dg0_v_vv') <= -1.0903396606445312) ?  0.001705760632273734 :   -0.019411141273775318 :   (b('dg0_v_vv') <= -0.15210676193237305) ?  0.0485361540803807 :   (b('l8b11_med') <= 2890.52392578125) ?  0.015451103704160252 :   0.04210848584068569 :   0.10379105991559154 :   (b('dg0_v_vv') <= -1.0531153678894043) ?  (b('dg0_v_vv') <= -1.4899101257324219) ?  (b('dg0_v_vv') <= -2.1433067321777344) ?  0.03740957579435161 :   (b('crops') <= 91.9686164855957) ?  -0.066590418900835 :   -0.018157448602956844 :   (b('ndvi_med') <= 1481.0) ?  0.04040958187402538 :   0.05031236424179217 :   (b('l8b4') <= 1609.9188842773438) ?  (b('ndvi') <= 1906.0) ?  (b('dg0_v_vv') <= 0.963325023651123) ?  -0.024804649672120427 :   -0.017837572485582077 :   (b('l8b5_med') <= 2812.056640625) ?  0.00698107198959752 :   -0.010208518112544465 :   (b('l8b5_med') <= 2812.056640625) ?  -0.04868762777118543 :   -0.03698000717882019 :   (b('dg0_v_vv') <= 0.7469959259033203) ?  (b('ndvi') <= 1918.0) ?  (b('ndvi_med') <= 1349.5) ?  (b('l8b11') <= 2656.592529296875) ?  (b('l8b5') <= 3299.971923828125) ?  (b('l8b11_med') <= 2893.19287109375) ?  0.04965411791289673 :   0.05059805633677447 :   0.016260968817335852 :   (b('l8b5') <= 4151.032958984375) ?  (b('l8b4') <= 3388.4295654296875) ?  -0.022593799199810343 :   0.04235680390256932 :   (b('dg0_v_vv') <= 0.5343871116638184) ?  -0.03975437345416206 :   -0.0022531883631411176 :   (b('l8b5') <= 2847.659423828125) ?  (b('ndvi') <= 1340.5) ?  (b('l8b4') <= 1866.6240844726562) ?  -0.024378442061068244 :   -0.029716063489105157 :   (b('l8b11_med') <= 2881.5589599609375) ?  -0.002806771922946677 :   -0.01495068983866013 :   (b('dg0_v_vv') <= -0.6581101417541504) ?  (b('l8b11') <= 3656.909423828125) ?  -0.0010187705377574826 :   0.08398743441520745 :   (b('l8b5') <= 3252.6292724609375) ?  0.005685334963276852 :   -0.017812750965043693 :   (b('l8b5') <= 4246.0625) ?  (b('ndvi') <= 2063.0) ?  (b('l8b5_med') <= 3622.8421630859375) ?  0.051338553318978464 :   0.0483385472393047 :   (b('l8b5') <= 3418.539306640625) ?  (b('l8b11') <= 2926.0838623046875) ?  0.018098721656811048 :   0.002154273674723567 :   (b('dg0_v_vv') <= -0.7348213195800781) ?  -0.028671103704867362 :   -0.012099482957559415 :   (b('l8b4_med') <= 2271.3209228515625) ?  (b('dg0_v_vv') <= -0.9485692977905273) ?  0.05553604039605109 :   0.053153399293791426 :   (b('ndvi') <= 3741.0) ?  0.04249369555253721 :   0.02072182992362845 :   (b('l8b5') <= 2418.2491455078125) ?  0.16299375217335313 :   (b('l8b11') <= 2772.6373291015625) ?  (b('dg0_v_vv') <= 1.8240609169006348) ?  (b('l8b11') <= 2652.8525390625) ?  (b('ndvi') <= 1725.5) ?  0.019934558646926236 :   0.038539645660796235 :   (b('l8b11') <= 2723.215576171875) ?  0.060992090476576 :   0.02389658301734543 :   (b('ndvi') <= 1466.5) ?  (b('l8b4') <= 1936.6969604492188) ?  0.04354267011220572 :   0.04690558363042702 :   0.06366968326117478 :   (b('dg0_v_vv') <= 0.7699775695800781) ?  0.10258838684378985 :   (b('ndvi') <= 1283.5) ?  (b('ndvi') <= 1147.0) ?  -0.013459085123333688 :   0.03803632598719721 :   (b('l8b11') <= 3656.909423828125) ?  -0.01545848056457662 :   -0.043542043556758114 :   (b('grass') <= 27.41930866241455) ?  (b('l8b4_med') <= 1631.4669799804688) ?  (b('l8b4') <= 1849.7748413085938) ?  (b('l8b11') <= 2079.6779174804688) ?  -0.0521275911903086 :   (b('dg0_v_vv') <= -1.0712971687316895) ?  (b('l8b11') <= 2563.544677734375) ?  (b('l8b11') <= 2379.28662109375) ?  -0.021682446395947103 :   -0.018628723214070303 :   (b('l8b11_med') <= 2698.7501220703125) ?  0.05396849704440443 :   0.013796979289614625 :   (b('l8b4') <= 1605.5573120117188) ?  (b('dg0_v_vv') <= 0.5070362091064453) ?  0.025612560861170114 :   0.041604446448398016 :   (b('l8b11') <= 3298.13671875) ?  0.05025078103302266 :   0.05661571406789223 :   0.16374163211729328 :   (b('l8b4_med') <= 2187.779052734375) ?  (b('l8b5_med') <= 3339.2752685546875) ?  (b('grass') <= 17.6483736038208) ?  (b('l8b5') <= 2309.328369140625) ?  (b('grass') <= 13.11876630783081) ?  -0.005948121024536734 :   -0.038283140338612735 :   (b('ndvi') <= 1150.5) ?  -0.04282606450251093 :   -0.03082448549254472 :   (b('l8b11') <= 2935.1094970703125) ?  (b('l8b4') <= 1838.1134033203125) ?  -0.004781962554599891 :   -0.04248148947250815 :   (b('l8b4') <= 2176.9434814453125) ?  0.011351209639779194 :   -0.028699111535865202 :   (b('dg0_v_vv') <= 1.8571224212646484) ?  (b('l8b11') <= 1821.9047241210938) ?  (b('l8b5_med') <= 3359.630615234375) ?  -0.03005088477221239 :   -0.06219563190267698 :   (b('l8b5') <= 3191.970947265625) ?  -0.0350432301045174 :   -0.041409122272242235 :   (b('l8b4') <= 2417.26904296875) ?  (b('l8b5_med') <= 3359.630615234375) ?  -0.019484145096532478 :   -0.03670873836335409 :   (b('ndvi_med') <= 2030.0) ?  -0.015880679291701654 :   0.01572665049833223 :   (b('l8b5') <= 2928.7486572265625) ?  (b('ndvi') <= 736.5) ?  (b('l8b5_med') <= 3098.2930908203125) ?  0.06921819458998563 :   (b('grass') <= 9.98997974395752) ?  0.011216592229194244 :   0.031060427275453245 :   (b('l8b4') <= 2353.625732421875) ?  (b('l8b11') <= 2985.32421875) ?  0.0009948222948499867 :   0.02181531186371869 :   (b('l8b11') <= 2777.8636474609375) ?  -0.030305531878316957 :   -0.033911670135953656 :   (b('grass') <= 21.365646362304688) ?  (b('l8b5') <= 3010.65234375) ?  (b('ndvi_med') <= 718.5) ?  0.04695621049304724 :   -0.01184451349146029 :   (b('dg0_v_vv') <= 2.534661293029785) ?  -0.016989726496632326 :   0.0028101737206015142 :   (b('dg0_v_vv') <= 1.0609712600708008) ?  (b('l8b11_med') <= 3224.09033203125) ?  -0.03993368248690222 :   -0.028025128077374513 :   (b('l8b5') <= 3409.4462890625) ?  0.017243037386667 :   -0.02046560468401167 :   (b('dg0_v_vv') <= 0.6673226356506348) ?  (b('ndvi_med') <= 1772.0) ?  (b('l8b4') <= 1199.0728149414062) ?  (b('l8b11') <= 2018.2509155273438) ?  (b('ndvi') <= 3712.0) ?  0.03593551065834846 :   0.017393726154402628 :   (b('l8b5') <= 2981.535888671875) ?  (b('l8b5_med') <= 2769.3907470703125) ?  0.05225754179329534 :   0.061257545131155444 :   0.07532011528907584 :   (b('l8b4') <= 1979.1570434570312) ?  (b('crops') <= 41.09989643096924) ?  (b('grass') <= 36.37481498718262) ?  0.015545443092962435 :   -0.02124529367941064 :   (b('l8b11') <= 3115.0638427734375) ?  -0.008849773191741267 :   0.040882117085786186 :   (b('l8b11_med') <= 3088.060546875) ?  (b('l8b11_med') <= 2824.77783203125) ?  -0.017507682712706935 :   -0.03354568116326884 :   (b('l8b11_med') <= 3457.2012939453125) ?  0.009074831339912723 :   -0.02275208572798394 :   (b('bulk') <= 145.5) ?  (b('ndvi') <= 2067.5) ?  (b('l8b11') <= 2830.903564453125) ?  (b('l8b11') <= 2769.515869140625) ?  -0.06343454719647452 :   -0.0551681983729948 :   (b('dg0_v_vv') <= -1.277618408203125) ?  -0.05215003788437646 :   0.004277677963865906 :   (b('ndvi') <= 2211.5) ?  (b('l8b5') <= 3014.8006591796875) ?  0.06232166831879706 :   0.035572167397761825 :   (b('ndvi_med') <= 1920.75) ?  0.001101780602414032 :   -0.010716699475986746 :   (b('dg0_v_vv') <= -2.1942973136901855) ?  (b('l8b5') <= 2587.2435302734375) ?  (b('l8b5') <= 2069.7350463867188) ?  -0.04065593308232919 :   -0.028657304287443026 :   (b('l8b4') <= 1853.2401733398438) ?  0.0005468297872731898 :   0.004041211401627998 :   (b('dg0_v_vv') <= 0.592921257019043) ?  (b('ndvi_med') <= 2262.75) ?  -0.042008245373432763 :   -0.0691549186036201 :   (b('dg0_v_vv') <= 0.6128630638122559) ?  0.02076106408283962 :   -0.028612992959411204 :   (b('moss') <= 13.497345924377441) ?  (b('l8b4_med') <= 1893.3539428710938) ?  (b('l8b5') <= 3327.68505859375) ?  (b('ndvi') <= 2633.0) ?  (b('ndvi') <= 1756.5) ?  0.05472287902118831 :   0.015575999380199951 :   (b('l8b5') <= 3133.8519287109375) ?  0.06761010790879801 :   0.1101737781470476 :   (b('ndvi_med') <= 2060.25) ?  (b('ndvi') <= 1819.5) ?  -0.05507758972060672 :   -0.03572138685255992 :   0.04199914927050641 :   (b('l8b11_med') <= 2775.4326171875) ?  -0.08787211650025603 :   (b('dg0_v_vv') <= 1.7871766090393066) ?  (b('dg0_v_vv') <= 0.6774797439575195) ?  0.05447644154446983 :   -0.01587834584615978 :   (b('sand') <= 26.0) ?  0.11918170088210167 :   0.006205694180564005 :   (b('grass') <= 65.2627010345459) ?  (b('dg0_v_vv') <= 0.7601408958435059) ?  (b('l8b11') <= 2814.3055419921875) ?  (b('sand') <= 43.0) ?  -0.03054616210252857 :   0.03609215424903026 :   0.1896727848860077 :   (b('l8b4_med') <= 1757.4463500976562) ?  (b('l8b11') <= 2909.050048828125) ?  0.07525637812562604 :   -0.0003150108355904936 :   (b('l8b11') <= 3122.3123779296875) ?  -0.022308754048004746 :   0.00010476587708377338 :   (b('dg0_v_vv') <= 4.281482696533203) ?  (b('l8b4_med') <= 1853.0592041015625) ?  (b('grass') <= 82.49162292480469) ?  -0.035736517306098885 :   -0.0815150421806624 :   (b('ndvi') <= 1301.0) ?  -0.06287103002754141 :   0.0038161734019489927 :   0.05847431927283128 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1735.2388305664062) ?  (b('l8b4_med') <= 658.2301025390625) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 26.5) ?  (b('ndvi') <= 4759.5) ?  (b('l8b4') <= 601.80908203125) ?  (b('dg0_v_vv') <= -0.9139800071716309) ?  (b('ndvi') <= 4433.0) ?  (b('ndvi') <= 3431.5) ?  -0.05034684080199461 :   (b('l8b5') <= 4067.6409912109375) ?  0.03508178164126025 :   0.006763243203141353 :   (b('bulk') <= 132.0) ?  0.10866728102118939 :   (b('l8b5') <= 4892.111572265625) ?  0.06437892741979295 :   0.0841590512827789 :   (b('sand') <= 13.0) ?  (b('l8b4_med') <= 632.78857421875) ?  0.1384782564980668 :   0.16264301993003294 :   (b('ndvi') <= 4313.0) ?  (b('l8b4') <= 589.0153198242188) ?  0.07456970188886611 :   0.11893930624172334 :   (b('dg0_v_vv') <= -0.039232730865478516) ?  -0.08031713874787183 :   0.03490270950483541 :   (b('ndvi_med') <= 4372.5) ?  (b('crops') <= 53.75650405883789) ?  (b('l8b11') <= 1257.4755859375) ?  (b('moss') <= 6.94382119178772) ?  -0.007518654008208475 :   -0.03178058915009074 :   (b('l8b4') <= 644.9564819335938) ?  0.12985782234657794 :   0.029608501908603112 :   (b('ndvi') <= 4601.5) ?  (b('dg0_v_vv') <= 2.194567918777466) ?  0.08802021927907519 :   0.06795089282734403 :   0.021083700840540093 :   (b('l8b4') <= 643.4403686523438) ?  (b('dg0_v_vv') <= -0.2898707389831543) ?  -0.029761186798989592 :   -0.016458103413537678 :   -0.10368708524539666 :   (b('l8b11') <= 1097.049072265625) ?  (b('grass') <= 42.213863372802734) ?  (b('dg0_v_vv') <= 0.09928178787231445) ?  (b('l8b5') <= 4412.750732421875) ?  (b('crops') <= 29.014663696289062) ?  -0.0002545145772412938 :   0.028006649749460294 :   (b('dg0_v_vv') <= -1.2346720695495605) ?  0.03723543314441255 :   0.08816656120123537 :   (b('bulk') <= 150.0) ?  0.07267923474653565 :   0.08311174249765735 :   (b('dg0_v_vv') <= -1.660301685333252) ?  -0.0008047580242895869 :   -0.025804763984754064 :   (b('crops') <= 43.283342361450195) ?  (b('dg0_v_vv') <= 0.36792945861816406) ?  (b('l8b5') <= 3476.7996826171875) ?  -0.01695676630808951 :   -0.04373188363371169 :   -0.0040232739770331705 :   (b('ndvi_med') <= 4604.75) ?  (b('l8b4') <= 603.0020446777344) ?  (b('l8b4_med') <= 560.0) ?  -0.04789631746225681 :   -0.0707568324695341 :   (b('l8b4') <= 727.8005065917969) ?  -0.03492025527286549 :   -0.04440236046205079 :   -0.08666888142126469 :   (b('ndvi_med') <= 4455.25) ?  (b('sand') <= 36.5) ?  (b('l8b4') <= 628.4984436035156) ?  (b('l8b5') <= 3827.189453125) ?  (b('ndvi') <= 2599.0) ?  (b('ndvi') <= 2305.5) ?  0.02228535917715867 :   0.06923406743781028 :   (b('ndvi') <= 2959.0) ?  -0.004680363713916907 :   0.025748753851795068 :   (b('l8b11') <= 1081.505615234375) ?  (b('ndvi') <= 3892.5) ?  -0.03509566690173411 :   0.05462276257483297 :   (b('ndvi') <= 4800.5) ?  0.07897261788504169 :   0.14575392313027838 :   (b('l8b11_med') <= 1355.3932495117188) ?  (b('l8b11') <= 1315.790771484375) ?  (b('ndvi') <= 3595.5) ?  0.006822932584204016 :   0.06330554359343384 :   (b('ndvi') <= 2060.0) ?  0.032860051210311346 :   -0.01901119936100271 :   (b('ndvi') <= 3955.5) ?  (b('dg0_v_vv') <= -0.5735077857971191) ?  0.16945441458269683 :   0.08575009749060125 :   (b('l8b4') <= 672.8038635253906) ?  0.09469856963098297 :   -0.004288172065881973 :   (b('l8b11') <= 1467.5543212890625) ?  (b('l8b5') <= 1711.1781616210938) ?  (b('ndvi') <= 2942.0) ?  (b('l8b11') <= 1239.1609497070312) ?  -0.034716144293620146 :   0.0571251693513322 :   (b('ndvi_med') <= 2975.25) ?  0.09186844565634966 :   0.014449001319828403 :   (b('ndvi') <= 4171.5) ?  (b('l8b11_med') <= 1364.7742919921875) ?  0.014050455430872029 :   -0.060918837752403365 :   (b('moss') <= 11.730731010437012) ?  -0.007778273927002454 :   0.07069706005413169 :   (b('l8b5') <= 3534.43408203125) ?  (b('ndvi_med') <= 3559.5) ?  (b('l8b4') <= 706.5252075195312) ?  0.013851977281409817 :   -0.01008368585134409 :   (b('grass') <= 34.64734649658203) ?  0.03954830003646089 :   0.15180433063536786 :   (b('crops') <= 58.70992088317871) ?  0.05436552575560155 :   (b('bulk') <= 125.5) ?  -0.0438437490322272 :   -0.0774316476650167 :   (b('l8b11') <= 1198.20263671875) ?  (b('l8b11_med') <= 1127.5553588867188) ?  (b('l8b5') <= 3979.475341796875) ?  (b('ndvi') <= 4093.5) ?  (b('ndvi') <= 3256.0) ?  0.11155797853591799 :   0.05155358483212384 :   (b('bulk') <= 120.5) ?  0.14827888413584633 :   0.1210109261794184 :   -0.01256553537099675 :   (b('ndvi') <= 3256.0) ?  0.11825283118318047 :   (b('l8b4') <= 513.3248901367188) ?  (b('l8b5') <= 4845.569580078125) ?  0.07117140720138009 :   -0.0006964382627914212 :   (b('dg0_v_vv') <= 1.0545611381530762) ?  -0.042445772560165546 :   0.01067703285301791 :   (b('l8b4') <= 707.109619140625) ?  (b('sand') <= 60.0) ?  -0.10069732435536438 :   (b('l8b4') <= 630.9979553222656) ?  (b('ndvi_med') <= 4697.0) ?  0.02557239101980495 :   0.018516834914676372 :   (b('dg0_v_vv') <= -0.2099318504333496) ?  -0.01312810257497285 :   0.006040804988730247 :   (b('dg0_v_vv') <= -1.3317904472351074) ?  0.06053947769120899 :   (b('l8b4') <= 754.5669250488281) ?  0.16516057421886388 :   0.13389774928863207 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1274.3564453125) ?  (b('ndvi') <= 3033.5) ?  (b('ndvi_med') <= 2702.75) ?  (b('l8b4') <= 588.0639953613281) ?  (b('dg0_v_vv') <= -0.2820720672607422) ?  (b('dg0_v_vv') <= -0.6498451232910156) ?  0.06573565870635878 :   0.03743062329739458 :   (b('l8b5') <= 2131.7212524414062) ?  -0.004344272497526294 :   0.013172096089521523 :   (b('dg0_v_vv') <= 0.3432788848876953) ?  (b('dg0_v_vv') <= 0.18480849266052246) ?  -0.050728599515109475 :   -0.07078045110791406 :   (b('dg0_v_vv') <= 0.6509404182434082) ?  -0.0059978271910091085 :   -0.019894533266468784 :   (b('dg0_v_vv') <= 1.2383151054382324) ?  (b('ndvi_med') <= 3599.0) ?  (b('dg0_v_vv') <= -1.4312076568603516) ?  -0.026831768402188205 :   0.07643554403629461 :   (b('l8b11_med') <= 1201.6178588867188) ?  0.052899977946635396 :   -0.0068883531190691944 :   (b('ndvi_med') <= 3342.5) ?  0.05424155751174109 :   (b('sand') <= 70.5) ?  0.1242748726852977 :   0.1887862234398972 :   (b('l8b11') <= 958.7872924804688) ?  (b('ndvi') <= 3630.0) ?  (b('dg0_v_vv') <= 0.7684192657470703) ?  (b('dg0_v_vv') <= -0.6445045471191406) ?  0.0030261226066671857 :   0.023869198506097165 :   0.11721507943381576 :   (b('grass') <= 54.38536262512207) ?  (b('ndvi') <= 4289.5) ?  0.06532495942125718 :   0.10972784339940325 :   (b('ndvi') <= 3724.0) ?  0.0838633627146342 :   0.02729299319631743 :   (b('dg0_v_vv') <= -0.42188549041748047) ?  (b('sand') <= 76.5) ?  (b('l8b5') <= 2304.115966796875) ?  0.040888392141829655 :   -0.013825259990997797 :   (b('ndvi_med') <= 3910.5) ?  -0.12572401125901722 :   -0.08153203351704369 :   (b('ndvi') <= 4424.0) ?  (b('l8b11') <= 1305.0191040039062) ?  0.022115923157517856 :   0.07490122753522978 :   (b('l8b4') <= 650.0836486816406) ?  0.0053221810682484655 :   -0.03497301975840125 :   (b('l8b11_med') <= 1307.5338745117188) ?  (b('dg0_v_vv') <= 0.627079963684082) ?  (b('l8b4') <= 663.8805847167969) ?  -0.047996199643841025 :   -0.04060931244233129 :   -0.08345207995698048 :   (b('ndvi') <= 4954.5) ?  (b('l8b4') <= 706.3503723144531) ?  (b('ndvi') <= 3156.5) ?  (b('l8b11_med') <= 1347.3683471679688) ?  0.014167306848784395 :   -0.0032640421564200106 :   (b('ndvi') <= 3648.0) ?  -0.041133082823327755 :   -0.010323701988632663 :   (b('l8b4') <= 727.4757080078125) ?  -0.05864273885131721 :   (b('dg0_v_vv') <= -1.3991456031799316) ?  -0.018776157600266123 :   -0.0336207167035639 :   -0.05941425846268636 :   (b('l8b5') <= 2478.9398193359375) ?  (b('dg0_v_vv') <= -1.4459495544433594) ?  (b('l8b4') <= 687.8755187988281) ?  (b('l8b5') <= 1895.348876953125) ?  0.08842692344129272 :   (b('l8b4') <= 630.7738952636719) ?  (b('dg0_v_vv') <= -2.4020309448242188) ?  -0.01819409449133778 :   -0.008225108476036608 :   (b('l8b11') <= 1245.7349243164062) ?  -0.001255679737868498 :   0.012911578477110663 :   (b('dg0_v_vv') <= -2.175748825073242) ?  0.11451016539264347 :   0.0134852269052258 :   (b('l8b5') <= 2455.188720703125) ?  (b('ndvi') <= 2895.5) ?  (b('l8b4') <= 768.2122802734375) ?  (b('l8b11') <= 1509.9014282226562) ?  0.12122831783223709 :   0.12983933876107234 :   (b('ndvi_med') <= 3405.0) ?  0.10672211866390757 :   0.10943013019732417 :   (b('ndvi') <= 4504.5) ?  (b('dg0_v_vv') <= -0.8719205856323242) ?  0.07250117544758573 :   0.04284772705425698 :   (b('l8b4_med') <= 481.6563720703125) ?  0.15316515738237302 :   0.08327795527053135 :   -0.04972785491484977 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b4') <= 383.7972869873047) ?  (b('dg0_v_vv') <= 0.2773747444152832) ?  (b('grass') <= 53.71771430969238) ?  0.11871755260476052 :   0.14064702034920346 :   (b('grass') <= 57.495262145996094) ?  0.1040183334446314 :   0.11385116082650382 :   (b('dg0_v_vv') <= 1.996042251586914) ?  (b('grass') <= 57.840450286865234) ?  0.09886000682276139 :   0.08143827826021362 :   (b('crops') <= 25.986759185791016) ?  0.09891308339542759 :   0.12173661070276696 :   (b('ndvi_med') <= 3208.25) ?  0.1920161113182871 :   (b('ndvi') <= 2667.5) ?  (b('dg0_v_vv') <= 0.03272056579589844) ?  0.12409850655202177 :   0.1313181844233602 :   (b('l8b5_med') <= 2592.1824951171875) ?  0.131364271461129 :   0.10465359648335265 :   (b('dg0_v_vv') <= 0.41835927963256836) ?  (b('l8b5') <= 2502.41357421875) ?  (b('dg0_v_vv') <= -0.4305291175842285) ?  (b('dg0_v_vv') <= -0.5597352981567383) ?  0.09781258813130435 :   0.1516396768328817 :   (b('dg0_v_vv') <= 0.22918081283569336) ?  0.0835680778830723 :   0.06714391731130509 :   (b('crops') <= 26.48409938812256) ?  (b('sand') <= 52.5) ?  0.09609249655303345 :   0.08029493428656602 :   (b('ndvi') <= 3530.5) ?  0.06481130424089482 :   0.07756008119487727 :   (b('ndvi_med') <= 3494.75) ?  (b('l8b5_med') <= 2931.880126953125) ?  (b('dg0_v_vv') <= 3.398371696472168) ?  0.10279296008600215 :   0.12977946474534674 :   (b('dg0_v_vv') <= 1.0079193115234375) ?  0.07458297250610887 :   0.090123909389764 :   (b('l8b11_med') <= 1320.1442260742188) ?  (b('l8b4_med') <= 490.6155700683594) ?  0.06403281833753477 :   0.08004496111646339 :   (b('dg0_v_vv') <= 3.0883212089538574) ?  0.10145168842924929 :   0.07371169672711264 :   (b('bulk') <= 137.5) ?  (b('crops') <= 65.88730239868164) ?  (b('dg0_v_vv') <= 0.9536998271942139) ?  (b('l8b5') <= 3561.4698486328125) ?  (b('ndvi_med') <= 3356.5) ?  (b('ndvi_med') <= 2147.0) ?  (b('moss') <= 12.437694549560547) ?  (b('l8b11_med') <= 1881.4260864257812) ?  0.0382084579606122 :   -0.052045856486937575 :   (b('bulk') <= 134.5) ?  -0.04058775146170942 :   0.0059167953901823256 :   (b('sand') <= 36.5) ?  (b('ndvi') <= 1247.5) ?  -0.08973691755557159 :   -0.012818267340474815 :   (b('l8b4') <= 732.4737243652344) ?  -0.012791441626254235 :   -0.029312868190951986 :   (b('l8b5') <= 2458.8193359375) ?  (b('moss') <= 15.687782287597656) ?  (b('l8b4_med') <= 998.1137084960938) ?  0.03568263126238843 :   -0.07331127982208452 :   -0.0885594782484525 :   (b('l8b4') <= 410.92982482910156) ?  0.1922289875900231 :   (b('ndvi') <= 4389.5) ?  -0.0010643882417194334 :   -0.024537847089924656 :   (b('l8b11') <= 1432.9409790039062) ?  (b('l8b4_med') <= 852.8804321289062) ?  (b('l8b11') <= 1223.8272094726562) ?  (b('l8b11_med') <= 1454.9094848632812) ?  0.014234834794260518 :   -0.011468282290068597 :   (b('dg0_v_vv') <= -0.8169784545898438) ?  -0.004647898717087238 :   -0.03705290829399885 :   (b('ndvi') <= 2854.5) ?  (b('moss') <= 2.703668713569641) ?  0.06868439499895299 :   0.03348212687666852 :   (b('l8b4_med') <= 887.6435241699219) ?  0.04994165432522931 :   -0.0015442357753869553 :   (b('grass') <= 47.533498764038086) ?  (b('l8b4_med') <= 749.0808715820312) ?  (b('ndvi_med') <= 3134.25) ?  0.07129852775950254 :   0.0413451129224775 :   (b('l8b5_med') <= 3419.3935546875) ?  0.016785376609444014 :   -0.06519506989242459 :   0.17478749407183258 :   (b('l8b11_med') <= 2326.0118408203125) ?  (b('crops') <= 55.25153923034668) ?  (b('grass') <= 51.26084899902344) ?  (b('l8b5') <= 1278.1651611328125) ?  0.14062779756350258 :   (b('crops') <= 3.538277119398117) ?  -0.03494670760660154 :   -0.0044948375515070395 :   (b('ndvi') <= 3348.0) ?  (b('l8b4_med') <= 775.0222473144531) ?  -0.024999131605219514 :   0.015596368552820311 :   (b('ndvi_med') <= 2451.0) ?  0.17127892492461486 :   0.0492067381435113 :   (b('l8b11') <= 1301.3427124023438) ?  (b('l8b4') <= 711.9574279785156) ?  (b('l8b11') <= 723.5576477050781) ?  -0.035324779073000225 :   0.011561108477025944 :   (b('moss') <= 5.675629615783691) ?  -0.014132866720661663 :   -0.1018051987941509 :   (b('l8b11') <= 1311.5980224609375) ?  (b('l8b4_med') <= 721.4329223632812) ?  0.08485487380293094 :   0.12787031269979784 :   (b('sand') <= 37.0) ?  0.027090475565749998 :   0.07195306656861308 :   0.2639972752330057 :   (b('ndvi_med') <= 2598.0) ?  (b('l8b5') <= 4386.0) ?  (b('l8b4_med') <= 802.5662536621094) ?  (b('l8b11') <= 1367.9440307617188) ?  0.11293347234796314 :   0.13358223789928647 :   (b('dg0_v_vv') <= -2.5748510360717773) ?  0.0018524885596410967 :   (b('l8b5') <= 3731.90966796875) ?  (b('ndvi') <= 1918.5) ?  0.07620421183450055 :   0.048598006360560664 :   (b('ndvi') <= 2179.5) ?  0.11130802722883526 :   0.08184049622242168 :   (b('dg0_v_vv') <= 3.6872618198394775) ?  (b('l8b11') <= 855.785400390625) ?  (b('dg0_v_vv') <= -0.06127500534057617) ?  -0.007689072519526385 :   0.0013082570838979235 :   -0.019065811632879265 :   0.026269885660290287 :   (b('l8b5') <= 2276.7049560546875) ?  (b('moss') <= 0.8611987233161926) ?  (b('ndvi') <= 1783.0) ?  (b('l8b5_med') <= 3483.843994140625) ?  0.046781394517351305 :   (b('l8b11_med') <= 1594.2955322265625) ?  0.06074771206418539 :   0.06358728957643095 :   (b('ndvi') <= 2026.5) ?  (b('dg0_v_vv') <= -0.23485946655273438) ?  0.09780802622818918 :   0.10676756142848676 :   (b('l8b4') <= 932.8193969726562) ?  0.06718738606564315 :   0.101076786210686 :   (b('ndvi') <= 3213.0) ?  (b('dg0_v_vv') <= 2.06626558303833) ?  (b('l8b4') <= 341.0372772216797) ?  0.017803078371498138 :   -0.008896462363620239 :   0.025760035273382664 :   (b('l8b11_med') <= 1375.4410400390625) ?  (b('dg0_v_vv') <= 0.24116182327270508) ?  0.01433032214093602 :   0.024851297415742796 :   (b('dg0_v_vv') <= -0.3076133728027344) ?  0.035714984463427045 :   0.04836772506792912 :   (b('ndvi') <= 1816.0) ?  (b('grass') <= 7.951443672180176) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.1265232163018369 :   0.11239109651740803 :   (b('dg0_v_vv') <= 0.37515783309936523) ?  0.07608841769734098 :   0.06745432606882384 :   (b('l8b4') <= 993.8001098632812) ?  (b('ndvi') <= 3958.5) ?  (b('ndvi') <= 3940.0) ?  0.009867793865172605 :   -0.06302907389600408 :   (b('ndvi') <= 4199.0) ?  0.05475851703104856 :   0.015387957612984678 :   (b('dg0_v_vv') <= 1.2970271110534668) ?  (b('l8b5') <= 3208.818603515625) ?  -0.05215393343734997 :   -0.010831440035690823 :   (b('l8b5_med') <= 3474.926513671875) ?  0.060898633303850286 :   -0.026051347474287098 :   (b('dg0_v_vv') <= 1.0708136558532715) ?  (b('sand') <= 28.0) ?  (b('l8b5') <= 3112.9217529296875) ?  (b('ndvi') <= 2582.5) ?  (b('ndvi') <= 2327.5) ?  (b('ndvi') <= 2252.5) ?  (b('l8b11_med') <= 1535.1817016601562) ?  0.08681442610733323 :   0.02637161028055124 :   (b('dg0_v_vv') <= -0.5750946998596191) ?  -0.03937062454470052 :   -0.004273757730697936 :   (b('dg0_v_vv') <= 0.2618880271911621) ?  (b('bulk') <= 143.5) ?  0.007719713731058356 :   0.06999167533524091 :   (b('ndvi_med') <= 2987.75) ?  0.14625587476077043 :   0.15788002381095273 :   (b('l8b4') <= 791.6678161621094) ?  (b('ndvi') <= 3677.5) ?  (b('ndvi_med') <= 3563.5) ?  0.033311596739606944 :   -0.0011289706879855103 :   (b('l8b5_med') <= 2765.5899658203125) ?  0.0004199050418414679 :   0.05208888423226446 :   (b('l8b4_med') <= 725.9989624023438) ?  (b('ndvi_med') <= 3834.5) ?  0.10740645391898651 :   0.13158995867152284 :   (b('grass') <= 27.141289710998535) ?  0.02748324257144887 :   -0.0176744238362542 :   (b('l8b4_med') <= 859.3739318847656) ?  (b('l8b5') <= 3183.68212890625) ?  (b('ndvi') <= 4349.0) ?  (b('ndvi_med') <= 3786.5) ?  0.10433700452269298 :   0.05486248403727412 :   (b('l8b11_med') <= 1512.6978759765625) ?  0.13946855722645188 :   0.17549345361486454 :   (b('grass') <= 55.14389419555664) ?  (b('l8b4_med') <= 743.82470703125) ?  0.013439907305125145 :   0.07269303826152916 :   (b('l8b5') <= 3202.947021484375) ?  0.04144511234146181 :   -0.04375020717639603 :   (b('ndvi') <= 3486.0) ?  (b('l8b11') <= 929.9408264160156) ?  (b('l8b5_med') <= 3042.8221435546875) ?  0.1054490827761892 :   0.16042876766448733 :   (b('dg0_v_vv') <= -0.4218878746032715) ?  0.04821665031021169 :   0.08995084190434681 :   (b('dg0_v_vv') <= -2.4714226722717285) ?  0.11255290418110137 :   (b('l8b11_med') <= 1543.727783203125) ?  0.01826727102494607 :   -0.02065567912139596 :   (b('l8b5_med') <= 2521.7769775390625) ?  (b('bulk') <= 141.5) ?  (b('ndvi_med') <= 1325.25) ?  (b('dg0_v_vv') <= 0.6909375190734863) ?  (b('ndvi') <= 1144.0) ?  -0.0495749405695052 :   0.0010133139213482107 :   (b('moss') <= 23.54700231552124) ?  0.05941372341751203 :   0.04038872696696713 :   (b('ndvi') <= 3810.0) ?  (b('l8b11') <= 1244.6018676757812) ?  0.05733448006110038 :   0.10986942584427063 :   (b('dg0_v_vv') <= -0.7442121505737305) ?  0.07210802709547198 :   -0.034177966321372225 :   (b('l8b4') <= 654.14599609375) ?  (b('l8b11_med') <= 1555.9805908203125) ?  (b('ndvi_med') <= 2637.5) ?  0.02629852512849473 :   -0.02819488462932417 :   (b('l8b5_med') <= 2208.2332763671875) ?  -0.06514686568693757 :   -0.0013627429876616427 :   (b('moss') <= 18.907434463500977) ?  (b('l8b11_med') <= 1970.4664306640625) ?  -0.04343005502599312 :   -0.001168065029667527 :   (b('ndvi') <= 1379.0) ?  -0.03950008197322517 :   -0.004172608607617299 :   (b('l8b4_med') <= 2052.0275268554688) ?  (b('ndvi_med') <= 2171.5) ?  (b('l8b5_med') <= 2577.157470703125) ?  (b('l8b5') <= 2751.3546142578125) ?  0.16492560747885815 :   0.0639392971806333 :   (b('ndvi') <= 2652.5) ?  0.005032205656792622 :   0.04727399435769182 :   (b('ndvi') <= 2235.5) ?  (b('dg0_v_vv') <= -0.5081653594970703) ?  0.007166991394236999 :   0.06374115208680195 :   (b('l8b11') <= 1717.4613037109375) ?  0.0027394484154646148 :   0.0643106064768035 :   (b('moss') <= 8.284094095230103) ?  (b('ndvi') <= 2933.0) ?  (b('l8b5') <= 2141.7821044921875) ?  0.024596627535391807 :   -0.0015700649193893523 :   (b('l8b5_med') <= 3169.1014404296875) ?  -0.03918857524358605 :   -0.05099255323287806 :   (b('ndvi') <= 2688.5) ?  (b('ndvi') <= 1866.0) ?  -0.07755071821696906 :   -0.07146766575915385 :   -0.09690898161200667 :   (b('l8b5') <= 3306.1719970703125) ?  (b('l8b4') <= 715.1031188964844) ?  (b('l8b11') <= 868.146484375) ?  (b('l8b4_med') <= 894.5899047851562) ?  (b('dg0_v_vv') <= 1.3848977088928223) ?  (b('l8b4') <= 324.6738739013672) ?  0.06454624067840911 :   0.07238504985331735 :   0.08454625140724517 :   (b('moss') <= 0.9351777359843254) ?  (b('ndvi') <= 4761.5) ?  0.023261298383793358 :   -0.009849152286538938 :   (b('l8b5') <= 2633.2252197265625) ?  -0.05949166420795437 :   -0.008263416145656649 :   (b('l8b4_med') <= 956.3034973144531) ?  (b('l8b4_med') <= 809.0887145996094) ?  (b('l8b4') <= 709.6019897460938) ?  0.05411047229074907 :   0.1227125923127192 :   (b('grass') <= 55.07127571105957) ?  0.0027366919926818094 :   0.058204979384099784 :   (b('sand') <= 52.5) ?  (b('l8b4') <= 665.0191345214844) ?  0.10975199960460909 :   0.04990099163168566 :   (b('l8b4_med') <= 1372.2369384765625) ?  0.004190773910824286 :   0.110776595438514 :   (b('l8b5_med') <= 3167.522705078125) ?  (b('dg0_v_vv') <= 2.2850708961486816) ?  (b('ndvi') <= 4212.0) ?  (b('bulk') <= 158.5) ?  0.004234961030901555 :   -0.04914612812971172 :   (b('l8b11') <= 1642.5775146484375) ?  0.09027785454781548 :   -0.004840575337896946 :   (b('ndvi_med') <= 3835.5) ?  (b('dg0_v_vv') <= 4.425183296203613) ?  0.02849471385240456 :   0.0786858702523538 :   (b('dg0_v_vv') <= 2.5914194583892822) ?  0.14203222196516213 :   0.14622367399729072 :   (b('l8b11_med') <= 2578.2747802734375) ?  (b('ndvi') <= 4282.0) ?  (b('l8b5_med') <= 3264.6268310546875) ?  0.13008160633981883 :   0.0820930060671521 :   -0.04881516130286909 :   (b('l8b11') <= 1671.0731811523438) ?  (b('l8b5') <= 2271.8082275390625) ?  0.035860017757788375 :   0.007663637314999283 :   0.10590779403625067 :   (b('sand') <= 67.0) ?  (b('sand') <= 16.5) ?  0.206330758400655 :   (b('dg0_v_vv') <= 1.3046965599060059) ?  (b('ndvi') <= 3431.5) ?  (b('l8b4') <= 751.7041320800781) ?  0.14049939155008356 :   0.1094984012223109 :   (b('dg0_v_vv') <= 1.232253074645996) ?  0.021434768633922335 :   0.08497192750799172 :   (b('bulk') <= 158.5) ?  (b('l8b4') <= 628.3688049316406) ?  -0.0069027091715144215 :   0.01684787018669739 :   (b('sand') <= 34.0) ?  0.11746842905630606 :   0.06451398882189625 :   (b('grass') <= 69.92729187011719) ?  (b('crops') <= 39.15159511566162) ?  -0.1514955964898215 :   (b('ndvi') <= 4098.0) ?  -0.06355740115447317 :   (b('dg0_v_vv') <= 1.9314196109771729) ?  -0.08828255432909174 :   -0.08218470590776236 :   0.040337586213911786 :   (b('ndvi_med') <= 2346.25) ?  (b('dg0_v_vv') <= 1.039083480834961) ?  (b('moss') <= 8.85230016708374) ?  (b('bulk') <= 133.5) ?  (b('crops') <= 77.11273193359375) ?  (b('dg0_v_vv') <= -0.7128691673278809) ?  (b('dg0_v_vv') <= -0.8862833976745605) ?  -0.013059379916436895 :   -0.02875590330592319 :   (b('ndvi_med') <= 1996.5) ?  (b('l8b11') <= 1910.3478393554688) ?  -0.0014105590766181203 :   -0.005562238015375948 :   (b('l8b4') <= 1111.6979675292969) ?  (b('l8b4_med') <= 771.2719421386719) ?  0.017016206046180787 :   0.009216755362604245 :   -0.0008418338181140211 :   (b('l8b11') <= 1856.8114624023438) ?  (b('l8b11') <= 1803.1948852539062) ?  (b('l8b11_med') <= 1916.2466430664062) ?  0.034400196838321156 :   0.036932308189445595 :   (b('l8b11_med') <= 1351.4182739257812) ?  0.04473985734346267 :   0.057241127738191344 :   (b('l8b4') <= 1309.2120361328125) ?  (b('dg0_v_vv') <= 0.35843467712402344) ?  (b('ndvi') <= 2823.0) ?  0.08221964396748292 :   0.10438800866173997 :   (b('l8b5') <= 1723.814453125) ?  0.1198397753481753 :   0.10939785032136984 :   (b('l8b11') <= 2373.6317138671875) ?  0.027307522423038355 :   0.05625051023765859 :   (b('grass') <= 89.59642791748047) ?  (b('l8b5_med') <= 2564.462158203125) ?  (b('sand') <= 44.0) ?  (b('l8b4') <= 846.3193054199219) ?  (b('crops') <= 1.0771889686584473) ?  0.10416328294707053 :   0.02798244700545592 :   (b('dg0_v_vv') <= -0.9091572761535645) ?  -0.011053790619722626 :   -0.04185130177047737 :   (b('dg0_v_vv') <= 0.6367411613464355) ?  (b('l8b5') <= 3290.997314453125) ?  -0.009461292376591409 :   0.13901604577651597 :   (b('l8b11_med') <= 2354.7825927734375) ?  -0.012698510445167946 :   -0.0938174720744714 :   (b('l8b11_med') <= 3011.9715576171875) ?  (b('ndvi') <= 2339.5) ?  (b('moss') <= 8.42107343673706) ?  -0.010945128981828003 :   0.028416505640737773 :   (b('crops') <= 87.66040420532227) ?  -0.0025361783389583957 :   0.0292411300077389 :   (b('ndvi') <= 3172.0) ?  (b('grass') <= 4.107848167419434) ?  -0.03283325913057725 :   -0.014155173302251045 :   (b('l8b11_med') <= 3108.688720703125) ?  0.0017728942128400343 :   0.026061489005400743 :   (b('ndvi') <= 2241.0) ?  (b('bulk') <= 145.5) ?  (b('l8b11_med') <= 2036.2794799804688) ?  (b('l8b5') <= 1904.5177612304688) ?  -0.05063827780198826 :   -0.07254386944549726 :   (b('l8b11') <= 2330.8486328125) ?  -0.0007028443779546009 :   -0.027217638536462675 :   (b('ndvi') <= 1404.0) ?  (b('l8b5') <= 1801.2013549804688) ?  -0.06053365756201872 :   0.03627020994181982 :   (b('l8b4') <= 1068.0910034179688) ?  0.060269740187713756 :   -0.007482533030122885 :   (b('dg0_v_vv') <= 0.06958770751953125) ?  (b('sand') <= 53.0) ?  (b('l8b4') <= 1368.9437255859375) ?  0.0311724679205219 :   -0.023253708107349252 :   (b('l8b11_med') <= 2497.6190185546875) ?  -0.042131343078730885 :   -0.027259200958455715 :   (b('ndvi_med') <= 2198.0) ?  (b('ndvi') <= 2626.0) ?  0.03906434468124517 :   0.08576814213487392 :   (b('dg0_v_vv') <= 0.7568812370300293) ?  0.07440768848470233 :   0.14301558743744663 :   (b('crops') <= 81.16990280151367) ?  (b('l8b5_med') <= 2409.4822998046875) ?  (b('crops') <= 0.339643657207489) ?  (b('l8b4_med') <= 1233.2196044921875) ?  (b('l8b4_med') <= 959.9854736328125) ?  (b('ndvi_med') <= 2266.25) ?  -0.008267017376512963 :   0.055774174276276456 :   (b('l8b11') <= 3400.300048828125) ?  -0.02944730045881545 :   0.14398104651835306 :   (b('l8b4') <= 1116.3886108398438) ?  (b('l8b5_med') <= 2075.4801025390625) ?  0.09919175244051658 :   0.010449300807875784 :   (b('l8b5') <= 1680.5130004882812) ?  0.039654631892561104 :   -0.014024768797007188 :   (b('l8b11') <= 1959.969482421875) ?  (b('l8b11') <= 1894.7297973632812) ?  (b('l8b4_med') <= 1213.42236328125) ?  0.034350953991839635 :   0.11073922396918054 :   (b('l8b11') <= 1911.5865478515625) ?  0.12685324382139404 :   0.09697864348583578 :   (b('ndvi') <= 1196.5) ?  (b('ndvi_med') <= 1891.5) ?  0.04629361925754536 :   0.09347884780524646 :   (b('l8b4') <= 1314.9439697265625) ?  0.010525161015069843 :   -0.012197727323384948 :   (b('crops') <= 0.021021021530032158) ?  (b('ndvi') <= 2720.5) ?  (b('l8b5_med') <= 2603.133056640625) ?  (b('ndvi_med') <= 1550.25) ?  -0.025485861238906237 :   -0.039040196201007384 :   (b('l8b4') <= 1241.80908203125) ?  0.032975255429136326 :   -0.019445257301077882 :   (b('l8b5_med') <= 2764.910400390625) ?  (b('ndvi_med') <= 1588.5) ?  0.07405720973628173 :   0.14712192317434725 :   (b('l8b11') <= 2761.2496337890625) ?  0.02718546093666213 :   -0.013148318455070404 :   (b('l8b5') <= 2699.42822265625) ?  (b('l8b11_med') <= 2207.8406982421875) ?  (b('l8b11') <= 1819.8712158203125) ?  -0.045340595883333855 :   -0.08508954258567723 :   (b('moss') <= 16.67528247833252) ?  -0.026299332586630152 :   -0.040442633422761054 :   (b('l8b5') <= 3648.1920166015625) ?  (b('grass') <= 17.99287700653076) ?  -0.037177782034852426 :   -0.026133712322096436 :   (b('bulk') <= 147.5) ?  -0.003223711818214875 :   -0.040374664261605984 :   (b('l8b5') <= 2336.48583984375) ?  (b('ndvi') <= 1436.5) ?  (b('ndvi') <= 1046.0) ?  (b('l8b5_med') <= 2814.7022705078125) ?  0.07576593386222794 :   0.08569969311840697 :   (b('dg0_v_vv') <= -1.0873446464538574) ?  (b('dg0_v_vv') <= -1.5898337364196777) ?  0.03057078067204018 :   0.012779789390274915 :   (b('ndvi') <= 1354.0) ?  0.03606408743046782 :   0.051378276544170154 :   (b('dg0_v_vv') <= 0.10035037994384766) ?  (b('dg0_v_vv') <= -0.7696323394775391) ?  -0.0021085878956103865 :   0.012891412700436061 :   (b('dg0_v_vv') <= 0.9225168228149414) ?  -0.024382335331197075 :   -0.015447122853927395 :   (b('ndvi') <= 1474.5) ?  (b('ndvi') <= 1217.5) ?  (b('ndvi') <= 1130.0) ?  (b('dg0_v_vv') <= 0.6168398857116699) ?  -0.026206725838481496 :   -0.02075288612853718 :   (b('dg0_v_vv') <= -0.34369373321533203) ?  -0.00557063040970694 :   0.0317397585843202 :   (b('dg0_v_vv') <= 0.4434318542480469) ?  (b('l8b5') <= 2521.85400390625) ?  -0.009875819694118946 :   -0.019277234086043177 :   (b('l8b11') <= 3074.33251953125) ?  -0.033331957186395894 :   -0.035514580612727054 :   (b('l8b4') <= 2444.7628173828125) ?  (b('l8b11') <= 3167.180419921875) ?  (b('ndvi') <= 1763.5) ?  -0.0051950303128660115 :   0.01601138711147074 :   (b('l8b4_med') <= 1652.7982177734375) ?  0.048879473927811556 :   0.055545424011761935 :   0.08118387934805407 :   (b('l8b5_med') <= 3056.480712890625) ?  (b('moss') <= 26.56121253967285) ?  (b('moss') <= 24.824777603149414) ?  (b('dg0_v_vv') <= 3.085139274597168) ?  (b('ndvi_med') <= 2306.5) ?  (b('grass') <= 90.26663208007812) ?  (b('l8b5_med') <= 1832.1904907226562) ?  0.07217374407377598 :   0.006933610707839521 :   (b('dg0_v_vv') <= 1.217090129852295) ?  -0.06734139370757003 :   0.04357041501482189 :   (b('l8b4') <= 1057.5670471191406) ?  (b('l8b4_med') <= 1078.64892578125) ?  -0.11153435505945591 :   -0.10190768963253762 :   (b('l8b11') <= 2460.9508056640625) ?  -0.0353664511063738 :   -0.06397715809555482 :   (b('bulk') <= 158.0) ?  (b('moss') <= 0.010067113675177097) ?  (b('l8b11') <= 2608.368408203125) ?  0.06376658417942632 :   -0.04890978352200963 :   (b('crops') <= 41.09989643096924) ?  0.0042872588551880935 :   0.04162555913778476 :   (b('ndvi') <= 1644.0) ?  (b('l8b5_med') <= 2359.6533203125) ?  0.11137287499715376 :   0.05848195455580518 :   (b('bulk') <= 163.0) ?  0.17954342304791576 :   0.15661837887659866 :   (b('moss') <= 25.738844871520996) ?  (b('dg0_v_vv') <= 1.7376608848571777) ?  (b('dg0_v_vv') <= 1.6696791648864746) ?  (b('l8b4_med') <= 1524.6758422851562) ?  0.09589722699111769 :   0.10115268462464486 :   0.10996265157844931 :   (b('ndvi') <= 1382.0) ?  (b('l8b5_med') <= 2012.2393188476562) ?  0.1196178460019478 :   0.09910552728789618 :   0.14285762933830676 :   (b('l8b11_med') <= 2330.10009765625) ?  (b('l8b11') <= 2109.4277954101562) ?  0.05153734280867517 :   0.05477522975934232 :   (b('ndvi_med') <= 1189.0) ?  0.04385558770409087 :   (b('l8b4_med') <= 1477.233154296875) ?  -0.019107383079514262 :   -0.015598304591867063 :   (b('ndvi_med') <= 996.5) ?  (b('l8b4') <= 1561.0130004882812) ?  (b('dg0_v_vv') <= 1.9359917640686035) ?  (b('grass') <= 54.434064865112305) ?  0.09600118004294694 :   (b('ndvi_med') <= 859.0) ?  0.05888348654078274 :   0.07671996237534728 :   (b('l8b5') <= 2305.484375) ?  (b('l8b4') <= 1323.7211303710938) ?  0.12237157555247191 :   0.11299334541439954 :   0.16650040408205624 :   (b('dg0_v_vv') <= 1.21653413772583) ?  -0.04245658099555341 :   (b('l8b4_med') <= 1696.0219116210938) ?  -0.058452824979133594 :   -0.053712342908008054 :   (b('l8b5_med') <= 1934.97314453125) ?  (b('l8b5') <= 1834.6284790039062) ?  (b('l8b4') <= 1103.4021606445312) ?  (b('l8b4') <= 991.5465698242188) ?  0.03551088628005087 :   -0.017324274517251482 :   (b('moss') <= 34.76172637939453) ?  0.10936598367126385 :   0.05682507318906799 :   (b('l8b5_med') <= 1915.1431884765625) ?  (b('l8b4_med') <= 1071.31494140625) ?  -0.014081847974577049 :   -0.07276082656732523 :   (b('l8b5') <= 1947.1851806640625) ?  0.002640278972485255 :   0.09385365364395312 :   (b('dg0_v_vv') <= 3.568903923034668) ?  (b('moss') <= 39.32973289489746) ?  (b('l8b11') <= 2266.37646484375) ?  -0.05029677254966328 :   -0.017684644260251483 :   (b('ndvi_med') <= 1243.0) ?  -0.06313511200128119 :   -0.0032056386006220146 :   (b('ndvi') <= 1412.0) ?  (b('ndvi') <= 991.0) ?  0.11287823878596281 :   0.047885615944042184 :   (b('bulk') <= 139.0) ?  0.030603008165110673 :   -0.026053615676223385 :   (b('l8b11_med') <= 2953.3192138671875) ?  (b('crops') <= 76.15693664550781) ?  (b('crops') <= 69.99465560913086) ?  (b('l8b11') <= 3131.513671875) ?  (b('l8b5') <= 3346.6671142578125) ?  (b('l8b5') <= 2859.5760498046875) ?  -0.0021240358150356672 :   0.030997389043550636 :   (b('dg0_v_vv') <= 1.0942516326904297) ?  0.03112725726818987 :   -0.02246978491982907 :   (b('l8b4') <= 2252.67431640625) ?  (b('dg0_v_vv') <= 1.1572060585021973) ?  -0.020693216891412715 :   -0.0429760904763009 :   (b('dg0_v_vv') <= 1.5550107955932617) ?  0.05752440658398349 :   -0.00043390012680343676 :   (b('ndvi') <= 1182.5) ?  (b('l8b4') <= 1817.0479736328125) ?  -0.06407030796497584 :   (b('l8b11') <= 2736.241455078125) ?  0.002698950334003039 :   -0.020343419192663203 :   (b('dg0_v_vv') <= 1.5879182815551758) ?  (b('dg0_v_vv') <= 1.3171048164367676) ?  0.06447876095504489 :   -0.012312542636009097 :   (b('l8b4') <= 1989.5501708984375) ?  0.07519594552641383 :   0.10673421154409339 :   (b('crops') <= 84.41960525512695) ?  (b('l8b4') <= 1132.597900390625) ?  (b('moss') <= 2.921739101409912) ?  (b('l8b4') <= 1058.1379699707031) ?  0.010018467457560604 :   0.04113913603108943 :   0.0785409071183395 :   (b('l8b11') <= 2745.2608642578125) ?  (b('dg0_v_vv') <= 3.1316516399383545) ?  -0.03714644706972832 :   -0.016149515173361422 :   (b('ndvi_med') <= 2171.5) ?  -0.011614924032273715 :   0.061855047718299444 :   (b('dg0_v_vv') <= 1.8244318962097168) ?  (b('ndvi') <= 1179.0) ?  (b('l8b4') <= 2669.446533203125) ?  0.015345311594420768 :   0.10409456048590414 :   (b('l8b5') <= 3195.38134765625) ?  0.015784080945409434 :   -0.021906248019529406 :   (b('ndvi_med') <= 2029.0) ?  (b('dg0_v_vv') <= 4.04228663444519) ?  0.015462177020698444 :   -0.059529946813461465 :   0.12200388755491831 :   (b('ndvi_med') <= 1642.5) ?  (b('grass') <= 5.898765325546265) ?  (b('dg0_v_vv') <= 2.9646213054656982) ?  (b('ndvi') <= 3243.5) ?  (b('l8b11') <= 3719.512451171875) ?  -0.03587468024880798 :   -0.012847348081297337 :   (b('dg0_v_vv') <= 2.2982912063598633) ?  0.009384095752604224 :   -0.010946423270327868 :   (b('ndvi_med') <= 1181.5) ?  0.026784771624447118 :   0.02316248944614449 :   (b('l8b5') <= 3316.3179931640625) ?  (b('l8b5') <= 3229.839111328125) ?  (b('l8b11') <= 2682.8580322265625) ?  0.020056237656269613 :   -0.0032022824628853584 :   (b('l8b11') <= 2550.49658203125) ?  0.06815057226710461 :   0.03463365479279438 :   (b('ndvi') <= 1621.5) ?  (b('ndvi') <= 1195.5) ?  -0.015893693063541853 :   -0.03227339952919464 :   (b('l8b5') <= 3609.1689453125) ?  0.010968720214745759 :   -0.018790163492416728 :   (b('dg0_v_vv') <= 1.0402865409851074) ?  0.041758598526617216 :   (b('l8b5') <= 2642.8504638671875) ?  (b('l8b11_med') <= 3020.3779296875) ?  (b('dg0_v_vv') <= 2.7032470703125) ?  -0.004503353365465035 :   0.0099656374865075 :   (b('dg0_v_vv') <= 4.364159345626831) ?  -0.03658197305152185 :   -0.0009536182925051762 :   (b('l8b4') <= 2388.5601806640625) ?  (b('dg0_v_vv') <= 3.35945987701416) ?  -0.038879436686645355 :   -0.021429282977012465 :   (b('l8b4') <= 2433.3560791015625) ?  -0.00668156338457047 :   -0.05259146417662558 :   (b('moss') <= 4.821818828582764) ?  (b('sand') <= 48.5) ?  (b('dg0_v_vv') <= 0.0980062484741211) ?  (b('ndvi_med') <= 3715.75) ?  (b('l8b4_med') <= 858.9129943847656) ?  (b('l8b5') <= 3117.5186767578125) ?  (b('moss') <= 1.1430021822452545) ?  (b('l8b5') <= 3024.2484130859375) ?  0.063602260512627 :   0.13487601914862013 :   (b('l8b11') <= 2040.73291015625) ?  -0.02162772640438998 :   0.03405518295717247 :   (b('l8b5_med') <= 3440.5325927734375) ?  (b('l8b11') <= 2012.0780029296875) ?  -0.030047858407999183 :   0.04962825031994347 :   (b('dg0_v_vv') <= -1.67547607421875) ?  0.1203499968691707 :   0.040779940811583315 :   (b('ndvi') <= 3989.0) ?  (b('bulk') <= 156.0) ?  (b('l8b5') <= 2861.705078125) ?  -0.005566845122157161 :   0.01566897522857439 :   (b('l8b5') <= 3325.7315673828125) ?  0.05053856843466464 :   -0.030612548442331862 :   (b('l8b5') <= 3964.480712890625) ?  (b('l8b11') <= 2287.09423828125) ?  0.05009862217827484 :   0.016544196123114464 :   (b('crops') <= 68.47074508666992) ?  0.003279057953065573 :   0.0013780216582801597 :   (b('moss') <= 2.3150565028190613) ?  (b('ndvi') <= 3403.5) ?  (b('sand') <= 27.0) ?  (b('ndvi') <= 2471.0) ?  -0.03205165207076975 :   0.0012855591438247261 :   (b('l8b4') <= 1246.8564453125) ?  -0.048909452756842005 :   -0.08552295024354277 :   (b('dg0_v_vv') <= -0.4128761291503906) ?  (b('ndvi') <= 4029.5) ?  0.04555715682803879 :   -0.038204349802987675 :   -0.052974154897481224 :   (b('l8b4') <= 1074.7539672851562) ?  (b('l8b11') <= 1924.559814453125) ?  (b('l8b4') <= 863.61279296875) ?  0.0059731536039971875 :   0.06732184594122652 :   (b('dg0_v_vv') <= -2.658262252807617) ?  -0.009614931347213189 :   -0.06346717892567061 :   (b('l8b4_med') <= 667.822021484375) ?  (b('crops') <= 52.695980072021484) ?  0.08992201096528774 :   0.06413454282692156 :   (b('l8b5_med') <= 2996.3297119140625) ?  0.13166646271279164 :   0.10127948426357888 :   (b('bulk') <= 149.5) ?  (b('crops') <= 39.17510986328125) ?  (b('dg0_v_vv') <= 1.260115623474121) ?  (b('crops') <= 27.450624465942383) ?  (b('ndvi_med') <= 3693.0) ?  0.06105480088132042 :   0.10192148766128835 :   (b('l8b5') <= 2333.801513671875) ?  0.07961866341689156 :   0.1939404175244165 :   (b('bulk') <= 139.0) ?  (b('grass') <= 63.7784538269043) ?  -0.0016161474298521938 :   -0.09705188319977107 :   (b('ndvi') <= 2657.0) ?  0.007578483117705662 :   0.060226861108820404 :   (b('dg0_v_vv') <= 2.573171615600586) ?  (b('l8b5_med') <= 3474.926513671875) ?  (b('l8b11_med') <= 1295.4417114257812) ?  0.07185730365998272 :   0.013296463691637284 :   (b('l8b4') <= 1165.7374877929688) ?  0.09849874855156651 :   0.027022976799039915 :   (b('crops') <= 59.41643714904785) ?  -0.07566887663952623 :   (b('ndvi_med') <= 2570.75) ?  0.06667269187176118 :   0.033882800759422196 :   (b('l8b5') <= 3665.3919677734375) ?  (b('grass') <= 62.437679290771484) ?  (b('l8b5_med') <= 3416.5760498046875) ?  (b('dg0_v_vv') <= 0.3002505302429199) ?  0.1322895554510827 :   0.09578748855456655 :   (b('dg0_v_vv') <= 0.6068181991577148) ?  0.06457012623962116 :   -0.06213220921000617 :   (b('l8b4') <= 1332.0014038085938) ?  (b('ndvi') <= 2242.0) ?  -0.029411335001734255 :   0.045471022262368536 :   (b('l8b5') <= 2820.5374755859375) ?  0.09980576166998315 :   0.12075372220719394 :   (b('ndvi') <= 2654.0) ?  0.017535186580686257 :   (b('bulk') <= 157.0) ?  -0.06901594220801976 :   (b('dg0_v_vv') <= 0.6810741424560547) ?  -0.05449700396867127 :   -0.04098239287852684 :   (b('ndvi_med') <= 4454.5) ?  (b('crops') <= 0.3571428656578064) ?  (b('l8b4') <= 1124.0375366210938) ?  (b('l8b5') <= 1958.1295166015625) ?  (b('l8b5') <= 1865.1142578125) ?  (b('ndvi') <= 1739.5) ?  -0.028064619200168846 :   -0.03576274806944553 :   0.004792831893419211 :   (b('ndvi') <= 3420.5) ?  (b('l8b11') <= 1737.1468505859375) ?  -0.017700580619918485 :   -0.059466166784064384 :   -0.008699587985222967 :   (b('dg0_v_vv') <= 2.804490089416504) ?  (b('bulk') <= 145.5) ?  (b('ndvi') <= 1396.0) ?  -0.03051851439124816 :   -0.011101899262632959 :   -0.04650345591889074 :   -0.06795381308687273 :   (b('dg0_v_vv') <= 0.20965194702148438) ?  (b('ndvi_med') <= 3602.0) ?  (b('l8b5') <= 1870.1956176757812) ?  (b('ndvi') <= 2156.5) ?  -0.027789733348908992 :   0.02289543278792126 :   (b('l8b4') <= 1107.8487548828125) ?  -0.03654416233887415 :   -0.01278647122416412 :   (b('l8b11') <= 2058.0170288085938) ?  (b('l8b5') <= 2537.197998046875) ?  0.010091321205652934 :   -0.05909330892644623 :   (b('l8b11') <= 2206.5771484375) ?  0.05143591701489807 :   0.01501236856193088 :   (b('crops') <= 45.43688774108887) ?  (b('ndvi') <= 2589.0) ?  (b('dg0_v_vv') <= 1.533146858215332) ?  0.03034961047366947 :   -0.035319308468147056 :   (b('l8b5') <= 2480.8616943359375) ?  0.10821463329615075 :   0.05684982973281785 :   (b('l8b11') <= 2977.4132080078125) ?  (b('l8b5') <= 1754.125) ?  0.029776573447112542 :   -0.01358224642434259 :   (b('l8b11_med') <= 2089.0799560546875) ?  0.054115905322103024 :   0.11761967532116227 :   (b('l8b5') <= 2968.4638671875) ?  (b('dg0_v_vv') <= 0.9692111015319824) ?  (b('ndvi') <= 3751.0) ?  0.1682465043131136 :   0.15695164492265262 :   0.19669384378795685 :   (b('dg0_v_vv') <= -0.44847965240478516) ?  (b('dg0_v_vv') <= -0.7733373641967773) ?  -0.020213344416860957 :   -0.02716454054955071 :   (b('dg0_v_vv') <= 0.905341625213623) ?  (b('l8b4_med') <= 630.9979553222656) ?  (b('dg0_v_vv') <= -0.33357667922973633) ?  0.06516717775420511 :   0.08223697246491843 :   (b('dg0_v_vv') <= -0.15181970596313477) ?  0.07301651989647018 :   0.050077413261976414 :   0.03742443273049911 :   (b('dg0_v_vv') <= 1.8259549140930176) ?  (b('ndvi') <= 1727.5) ?  (b('ndvi') <= 1015.0) ?  (b('l8b5_med') <= 2962.1964111328125) ?  (b('dg0_v_vv') <= -1.2860450744628906) ?  -0.06929981137695973 :   -0.07320800631281836 :   (b('dg0_v_vv') <= 1.3352231979370117) ?  (b('dg0_v_vv') <= 0.7409825325012207) ?  (b('dg0_v_vv') <= 0.4677891731262207) ?  -0.07865762326448365 :   -0.07765762496321603 :   -0.08232338680712689 :   -0.0875857407998977 :   (b('l8b4') <= 1453.2037353515625) ?  (b('moss') <= 12.024037837982178) ?  (b('l8b5_med') <= 2952.4937744140625) ?  (b('ndvi') <= 1037.5) ?  0.04192626825561513 :   -0.015489361971841056 :   (b('ndvi') <= 1666.0) ?  0.04445397924891363 :   -0.011671678602051538 :   (b('l8b5') <= 1846.836181640625) ?  (b('sand') <= 45.5) ?  0.13149751770059231 :   0.12973522703869442 :   (b('l8b11') <= 2029.6538696289062) ?  0.0072780115443595705 :   0.053692173613103494 :   (b('l8b5') <= 3104.8206787109375) ?  (b('dg0_v_vv') <= -0.8454809188842773) ?  (b('dg0_v_vv') <= -1.02711820602417) ?  0.1383585934003971 :   0.15964837767720674 :   (b('ndvi_med') <= 3501.0) ?  0.10179418073743013 :   0.08458490269029939 :   (b('ndvi_med') <= 2378.75) ?  0.05473238769109898 :   -0.01049754588058327 :   (b('l8b11_med') <= 1566.9320068359375) ?  (b('bulk') <= 120.0) ?  (b('bulk') <= 115.5) ?  (b('ndvi') <= 3488.5) ?  0.18223272983228525 :   (b('ndvi_med') <= 3920.5) ?  0.14357134293471718 :   0.13261185604079453 :   (b('l8b11') <= 2374.2521362304688) ?  (b('l8b4') <= 1234.8654174804688) ?  0.09605941907457308 :   0.07420145533294675 :   0.13702739147279558 :   (b('grass') <= 53.32517623901367) ?  (b('l8b11') <= 1736.593017578125) ?  (b('dg0_v_vv') <= -1.0642147064208984) ?  -0.05944637891722343 :   -0.056554987603379105 :   (b('ndvi') <= 2356.5) ?  -0.021741791793047926 :   0.006028217905512838 :   (b('l8b4') <= 975.5419006347656) ?  (b('ndvi') <= 2358.0) ?  -0.04746101673036785 :   0.025956336350457392 :   (b('dg0_v_vv') <= 0.20125579833984375) ?  0.05017808312383811 :   0.12622793409344904 :   (b('bulk') <= 138.5) ?  (b('crops') <= 37.672115325927734) ?  (b('l8b11') <= 1825.7084350585938) ?  (b('bulk') <= 134.5) ?  -0.052974765222497375 :   -0.024705400772232735 :   (b('l8b5') <= 1799.4483642578125) ?  0.0587097379867235 :   -0.027456025002182423 :   (b('l8b11') <= 1968.7843017578125) ?  (b('bulk') <= 129.5) ?  0.02494874478882115 :   -0.018827029147521414 :   (b('l8b4') <= 1040.1928100585938) ?  0.05311550702615121 :   -0.008505357437678544 :   (b('l8b5') <= 2088.583251953125) ?  (b('ndvi') <= 2088.5) ?  (b('dg0_v_vv') <= -0.5551495552062988) ?  -0.028942651985986705 :   0.02250710958919975 :   (b('l8b4') <= 941.1997680664062) ?  0.11230041113313188 :   0.19683807168982886 :   (b('l8b4') <= 1102.9276733398438) ?  (b('l8b11') <= 2038.0872192382812) ?  -0.004674996107812315 :   0.05193577525363543 :   (b('l8b5_med') <= 2431.5496826171875) ?  0.020790846332940358 :   -0.015048518024211328 :   (b('grass') <= 34.88662147521973) ?  (b('dg0_v_vv') <= 1.8643059730529785) ?  (b('l8b5_med') <= 3215.934814453125) ?  0.17209550025004866 :   (b('dg0_v_vv') <= 1.8430547714233398) ?  0.0984251809684486 :   0.1303624458303041 :   (b('l8b5') <= 3139.7510986328125) ?  (b('ndvi') <= 2424.0) ?  (b('l8b11') <= 2190.3193359375) ?  (b('dg0_v_vv') <= 2.1687467098236084) ?  0.044208463647361466 :   0.018907011967703995 :   (b('ndvi_med') <= 3226.0) ?  -0.055563661820479435 :   -0.006279325336579267 :   (b('l8b4') <= 957.8826599121094) ?  (b('dg0_v_vv') <= 4.123125314712524) ?  0.03564804628897315 :   -0.05234630610490437 :   (b('l8b11') <= 2055.42236328125) ?  -0.05809691768708906 :   -0.021021823975001932 :   (b('l8b4') <= 1230.5919799804688) ?  (b('ndvi_med') <= 2820.0) ?  (b('crops') <= 70.15833854675293) ?  0.04966151340987658 :   0.01608154695409167 :   (b('crops') <= 37.000967025756836) ?  -0.046152219492888606 :   -0.0008835063444489685 :   (b('l8b4_med') <= 1377.5476684570312) ?  (b('l8b11_med') <= 2365.364013671875) ?  0.08099915111286224 :   0.04331877033683956 :   0.14466794831214647 :   (b('sand') <= 50.0) ?  (b('dg0_v_vv') <= 2.2566041946411133) ?  (b('ndvi_med') <= 3454.5) ?  (b('l8b5') <= 2057.8837280273438) ?  -0.005869344084607264 :   -0.012400365336282887 :   0.07692257645977091 :   (b('l8b11') <= 1975.8248291015625) ?  (b('l8b4') <= 846.7496948242188) ?  (b('l8b4') <= 805.6737976074219) ?  0.11191725883530598 :   0.10850882236010495 :   (b('l8b4') <= 1192.494384765625) ?  0.05182458621547687 :   0.11861629211651126 :   (b('moss') <= 18.995800971984863) ?  (b('ndvi_med') <= 2431.75) ?  0.08253698136802898 :   0.12446828983740993 :   (b('bulk') <= 143.5) ?  0.07581767300602937 :   0.07828605647094275 :   (b('bulk') <= 137.5) ?  0.07363913368872094 :   (b('dg0_v_vv') <= 2.168341636657715) ?  -0.04600695130020252 :   (b('l8b5') <= 3024.3089599609375) ?  (b('l8b4') <= 1182.5333862304688) ?  0.021856704861973858 :   0.009041236210037884 :   -0.00912161936630082 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1495.1371459960938) ?  (b('grass') <= 46.070945739746094) ?  (b('l8b5_med') <= 3250.9730224609375) ?  (b('bulk') <= 137.5) ?  (b('crops') <= 95.2543830871582) ?  (b('l8b11') <= 1327.829833984375) ?  (b('l8b4_med') <= 829.7794189453125) ?  (b('grass') <= 34.11734199523926) ?  (b('l8b5_med') <= 3062.6536865234375) ?  (b('dg0_v_vv') <= -0.037058353424072266) ?  -0.01632922031115545 :   -0.0009644750214369627 :   (b('ndvi') <= 2634.0) ?  0.03461124750337177 :   0.004350972236539175 :   (b('dg0_v_vv') <= 2.38649320602417) ?  (b('l8b5') <= 1740.0596923828125) ?  0.0433496678278695 :   0.01342026336286247 :   (b('l8b5_med') <= 2780.81591796875) ?  0.0731367974533203 :   0.126824722218769 :   (b('l8b5_med') <= 2795.55322265625) ?  (b('ndvi') <= 715.5) ?  (b('l8b4') <= 1016.6167602539062) ?  -0.021059291939541563 :   -0.05921148610808917 :   (b('l8b11') <= 1109.1182861328125) ?  0.0108962982064958 :   0.054051345184840525 :   (b('l8b11_med') <= 1889.7191772460938) ?  (b('l8b4_med') <= 887.6435241699219) ?  -0.0036833259270957434 :   0.01591562088425803 :   (b('sand') <= 39.5) ?  0.07072907203866506 :   0.046705663277576956 :   (b('l8b11') <= 1341.8060302734375) ?  (b('l8b4') <= 750.5197448730469) ?  (b('moss') <= 6.923041105270386) ?  (b('ndvi') <= 4468.5) ?  -0.051795245651599965 :   -0.024163128159332907 :   (b('dg0_v_vv') <= -1.0034055709838867) ?  -0.07616651638306338 :   0.004793993238815566 :   (b('ndvi_med') <= 3624.75) ?  (b('moss') <= 8.253445863723755) ?  -0.0675384455063343 :   -0.0877560107437189 :   (b('ndvi') <= 4601.0) ?  -0.07841451271271604 :   -0.005062192432216062 :   (b('dg0_v_vv') <= 1.4497613906860352) ?  (b('bulk') <= 120.0) ?  (b('bulk') <= 114.5) ?  -0.021623130024385515 :   0.06398601223947195 :   (b('l8b5_med') <= 2925.8314208984375) ?  -0.013798454579557036 :   -0.0009140945690237836 :   (b('moss') <= 8.555930137634277) ?  (b('l8b4_med') <= 592.7463684082031) ?  -0.016290943242207256 :   0.03438543632224391 :   (b('ndvi') <= 3567.0) ?  -0.01901066820212304 :   0.03318122072451729 :   (b('l8b5_med') <= 2556.873291015625) ?  (b('l8b4') <= 563.7236633300781) ?  (b('l8b5_med') <= 2134.86181640625) ?  (b('dg0_v_vv') <= 3.883657217025757) ?  (b('dg0_v_vv') <= -1.0473637580871582) ?  0.07571332769271266 :   0.07473225973129904 :   0.060921851825820994 :   (b('l8b11_med') <= 1441.8287353515625) ?  0.11578564641023747 :   0.10520653619753478 :   (b('l8b4') <= 766.8971862792969) ?  (b('ndvi_med') <= 1832.5) ?  (b('sand') <= 21.5) ?  0.0016672397036769926 :   0.030413228753024726 :   0.0553062796429436 :   (b('l8b5') <= 3932.2777099609375) ?  (b('grass') <= 2.184100389480591) ?  0.07998248230285743 :   0.09520386703094691 :   0.06140391386404459 :   (b('l8b11') <= 903.7995910644531) ?  (b('l8b11') <= 826.1931457519531) ?  0.008425358479978162 :   (b('dg0_v_vv') <= -0.6291108131408691) ?  0.07608623346344145 :   (b('ndvi_med') <= 3819.0) ?  0.0656416132978898 :   0.06251718476432164 :   (b('l8b11') <= 1059.0491333007812) ?  (b('l8b5') <= 3064.9818115234375) ?  (b('l8b4_med') <= 625.1705322265625) ?  0.053955600136840004 :   0.011098924229428892 :   (b('ndvi') <= 4357.5) ?  -0.04032728045895656 :   0.005009749681824898 :   (b('dg0_v_vv') <= 0.08458566665649414) ?  0.06301717832702 :   0.06801720336097081 :   (b('sand') <= 35.5) ?  (b('l8b11_med') <= 2037.5724487304688) ?  (b('l8b5') <= 3567.6153564453125) ?  (b('l8b4') <= 769.0118408203125) ?  (b('l8b11_med') <= 1835.3823852539062) ?  (b('l8b5') <= 3482.0833740234375) ?  0.02559451047167367 :   -0.011289613224950335 :   (b('dg0_v_vv') <= 0.032056331634521484) ?  0.02467014222858753 :   0.1150180204402371 :   (b('sand') <= 25.5) ?  (b('dg0_v_vv') <= -1.261289119720459) ?  0.016287537172173638 :   0.07639975953689719 :   (b('l8b11') <= 1103.6746215820312) ?  0.08442189996295489 :   -0.014521738771175779 :   (b('bulk') <= 151.0) ?  (b('ndvi_med') <= 2264.5) ?  (b('dg0_v_vv') <= -1.8210744857788086) ?  0.031944366075481095 :   -0.007962617856842786 :   (b('ndvi') <= 4975.0) ?  0.06912923290457137 :   -0.056605625334705256 :   (b('ndvi') <= 2775.5) ?  (b('dg0_v_vv') <= 1.2691712379455566) ?  0.04397590904579324 :   0.10271166141448035 :   (b('dg0_v_vv') <= -0.04488039016723633) ?  0.01936305587865028 :   -0.02550033050198844 :   (b('l8b5') <= 3464.9844970703125) ?  (b('dg0_v_vv') <= 0.03229188919067383) ?  (b('bulk') <= 155.5) ?  (b('dg0_v_vv') <= -0.7653698921203613) ?  -0.03828641009141667 :   -0.042243723464653965 :   -0.054508292272789166 :   (b('ndvi_med') <= 2320.5) ?  (b('ndvi_med') <= 2255.0) ?  -0.06400857824053138 :   -0.06081084588013008 :   -0.08268972826910995 :   (b('dg0_v_vv') <= 1.9202170372009277) ?  (b('dg0_v_vv') <= 1.3678994178771973) ?  (b('ndvi') <= 3332.0) ?  -0.012600698200480848 :   -0.024425907433122104 :   -0.05459768871113149 :   (b('ndvi_med') <= 2295.0) ?  0.02340230544761332 :   -0.0375976915125498 :   (b('l8b11') <= 1462.3347778320312) ?  (b('ndvi_med') <= 4047.25) ?  (b('moss') <= 16.057286262512207) ?  (b('crops') <= 10.56101381778717) ?  (b('l8b4') <= 657.3594360351562) ?  -0.011597799909454339 :   -0.058274447007297205 :   (b('grass') <= 43.151580810546875) ?  0.0010860967066340294 :   0.12512478243782363 :   (b('ndvi') <= 2362.5) ?  (b('l8b4') <= 841.9873046875) ?  -0.030413007925369467 :   0.06856008337616308 :   (b('l8b4') <= 412.60081481933594) ?  0.03621939962657344 :   0.13834467139363738 :   (b('l8b4') <= 747.5465698242188) ?  (b('l8b5') <= 2834.5682373046875) ?  (b('l8b5') <= 2562.4219970703125) ?  -0.059641014688022465 :   -0.0028185213817705224 :   (b('dg0_v_vv') <= -2.080982208251953) ?  -0.04182872731016812 :   0.06196055506302865 :   (b('l8b11') <= 1244.5252685546875) ?  (b('dg0_v_vv') <= 1.3429896831512451) ?  0.15530560703669266 :   0.12911627179359006 :   (b('l8b4_med') <= 841.58056640625) ?  0.1078436170817463 :   0.09730560870562271 :   (b('l8b5') <= 1824.3919677734375) ?  (b('dg0_v_vv') <= 0.5877137184143066) ?  (b('dg0_v_vv') <= 0.11457204818725586) ?  0.25989698526357985 :   0.245532428834639 :   0.18930586917378114 :   (b('dg0_v_vv') <= 0.9345049858093262) ?  (b('l8b4') <= 1032.3085327148438) ?  (b('bulk') <= 154.5) ?  0.015926834029902952 :   -0.014728585746227818 :   0.07487553977712802 :   -0.027680464786249 :   (b('l8b4') <= 916.2767639160156) ?  (b('sand') <= 26.5) ?  (b('ndvi') <= 4872.5) ?  (b('l8b11') <= 905.5500793457031) ?  (b('grass') <= 42.213863372802734) ?  (b('l8b11') <= 863.8023986816406) ?  (b('sand') <= 16.0) ?  0.11624894078926021 :   0.09246547923486836 :   (b('l8b11_med') <= 1218.8040771484375) ?  0.06672223809907701 :   0.07527185720119631 :   (b('ndvi') <= 3407.5) ?  0.023056165945826268 :   0.06056857097732343 :   (b('l8b5') <= 3510.3348388671875) ?  (b('dg0_v_vv') <= -1.3421907424926758) ?  (b('dg0_v_vv') <= -2.485621452331543) ?  0.056817723397427855 :   0.013486103877394656 :   (b('crops') <= 44.62105369567871) ?  0.06205053006741598 :   0.12073095338597226 :   (b('ndvi') <= 4003.0) ?  (b('l8b5') <= 4404.047119140625) ?  0.06861347193572762 :   0.011612541621362374 :   (b('l8b5') <= 3888.1695556640625) ?  -0.00460473209402811 :   0.03420742701642803 :   (b('l8b5_med') <= 3940.474609375) ?  -0.023224287586278647 :   -0.07800199327913823 :   (b('ndvi_med') <= 4666.0) ?  (b('sand') <= 52.5) ?  (b('l8b5') <= 4988.789306640625) ?  (b('l8b11') <= 1400.6526489257812) ?  (b('l8b4_med') <= 2059.6664428710938) ?  0.025896738586721927 :   -0.10121656391085927 :   (b('ndvi') <= 4654.5) ?  -0.008343226312619294 :   0.06379969688941156 :   (b('crops') <= 68.44170188903809) ?  (b('dg0_v_vv') <= 1.2022638320922852) ?  0.061631528950762585 :   0.013226205516076395 :   (b('dg0_v_vv') <= 0.775639533996582) ?  0.07329609701155387 :   0.12427695068984565 :   (b('l8b5') <= 2789.8455810546875) ?  (b('ndvi') <= 4379.0) ?  (b('sand') <= 78.5) ?  0.04682699074762937 :   0.08110883387875038 :   (b('bulk') <= 142.0) ?  -0.04424077329284769 :   -0.01298709297536596 :   (b('l8b5') <= 3570.3087158203125) ?  (b('l8b5_med') <= 3266.9854736328125) ?  0.1330463716422422 :   -0.030918385341053834 :   (b('l8b5') <= 3856.5413818359375) ?  0.02366420743551927 :   -0.0011324661426817932 :   (b('l8b4') <= 585.4060668945312) ?  (b('dg0_v_vv') <= 0.6208868026733398) ?  (b('dg0_v_vv') <= 0.07370328903198242) ?  (b('dg0_v_vv') <= -0.7755880355834961) ?  0.04144963245645361 :   0.0755385009787249 :   (b('dg0_v_vv') <= 0.3388195037841797) ?  -0.035474467055457926 :   0.02909743611222193 :   (b('l8b11_med') <= 1068.9169921875) ?  0.12326462615760869 :   (b('dg0_v_vv') <= 0.7419066429138184) ?  0.12255633090559043 :   0.09653549399221573 :   (b('ndvi') <= 4942.0) ?  -0.005226355393183349 :   (b('dg0_v_vv') <= 0.9246907234191895) ?  0.014253669742811864 :   0.0047396651611376905 :   (b('bulk') <= 131.0) ?  (b('l8b4') <= 949.4273071289062) ?  -0.035169113956786055 :   (b('dg0_v_vv') <= -0.7960805892944336) ?  -0.0031286749428137695 :   (b('dg0_v_vv') <= 3.388674259185791) ?  (b('l8b5_med') <= 3505.713134765625) ?  (b('dg0_v_vv') <= 0.623755931854248) ?  0.013430671521603682 :   0.05095281225026591 :   (b('l8b5') <= 2757.404296875) ?  0.06567294824874281 :   0.05701417705534842 :   0.014577371857031646 :   (b('l8b11') <= 1220.762939453125) ?  (b('dg0_v_vv') <= -3.7598185539245605) ?  -0.008410981085242242 :   (b('dg0_v_vv') <= -2.0968809127807617) ?  0.0045718455447866235 :   0.014571850909204653 :   (b('dg0_v_vv') <= 3.684619903564453) ?  (b('dg0_v_vv') <= -0.7852129936218262) ?  (b('l8b11') <= 1348.9382934570312) ?  0.14558197693024885 :   0.10535996270675504 :   (b('ndvi') <= 3276.5) ?  (b('l8b5_med') <= 3378.561279296875) ?  0.09038273416826967 :   0.060686929218983186 :   (b('ndvi_med') <= 2029.0) ?  0.09413614673072498 :   0.11306669400274912 :   (b('l8b5') <= 2984.5278930664062) ?  0.12598635443963463 :   0.15318352852828931 :   (b('l8b4_med') <= 652.0127868652344) ?  (b('crops') <= 19.45354127883911) ?  (b('l8b4') <= 450.03575134277344) ?  (b('l8b11_med') <= 976.31103515625) ?  (b('ndvi') <= 2609.5) ?  (b('l8b4') <= 408.7028503417969) ?  (b('dg0_v_vv') <= 0.907170295715332) ?  -0.010907529396344401 :   (b('bulk') <= 130.5) ?  -0.004317316625499473 :   -0.0029872848945065866 :   -0.05706723675376443 :   (b('l8b5') <= 2264.0440673828125) ?  (b('l8b4_med') <= 625.94091796875) ?  0.03496876850875924 :   0.03694512149021548 :   (b('l8b5_med') <= 2439.7489013671875) ?  0.017852010209111535 :   0.025322022422065743 :   (b('ndvi_med') <= 3966.5) ?  (b('bulk') <= 158.0) ?  (b('ndvi') <= 2711.0) ?  (b('moss') <= 7.54746855981648) ?  0.09747641187615239 :   0.13015964718756745 :   (b('l8b4') <= 355.29173278808594) ?  0.08973690517493083 :   0.06795974369280765 :   0.1381574525998266 :   -0.03602394789838195 :   (b('l8b11_med') <= 1274.3564453125) ?  (b('grass') <= 51.78411865234375) ?  (b('dg0_v_vv') <= 0.3432788848876953) ?  (b('l8b11') <= 1131.77587890625) ?  (b('l8b11') <= 950.8148193359375) ?  -0.048030025698077926 :   -0.06332076323404072 :   (b('l8b5') <= 2485.471923828125) ?  -0.03453294486873189 :   -0.008901232476930213 :   (b('ndvi_med') <= 2397.0) ?  (b('l8b4_med') <= 625.94091796875) ?  -0.005398044471908198 :   -0.017905079939821905 :   0.03104570714662075 :   (b('l8b4') <= 459.5747985839844) ?  (b('ndvi') <= 3940.0) ?  (b('ndvi') <= 3012.5) ?  0.01901849055878908 :   0.03246933279966525 :   (b('ndvi') <= 4266.5) ?  -0.0618559689746499 :   -0.008159646351859007 :   (b('l8b5') <= 3249.1973876953125) ?  (b('ndvi') <= 4138.0) ?  0.03419652469442285 :   0.08185516737313858 :   (b('l8b5') <= 3300.427001953125) ?  -0.1255585246424262 :   0.034652350176552646 :   (b('l8b5') <= 2545.029296875) ?  (b('dg0_v_vv') <= -0.34859132766723633) ?  (b('l8b4') <= 537.0124816894531) ?  -0.015246229159526017 :   (b('l8b5') <= 2320.2138671875) ?  0.031132700720991374 :   -0.002937637940778004 :   (b('l8b4') <= 653.6947021484375) ?  (b('ndvi') <= 4138.0) ?  -0.010209454213882585 :   -0.0059086547569904835 :   (b('ndvi') <= 2235.0) ?  -0.05277846496618549 :   -0.0342517230881597 :   (b('dg0_v_vv') <= -0.5773186683654785) ?  (b('ndvi') <= 4100.5) ?  -0.053966047401340284 :   -0.08626789735810256 :   (b('l8b11') <= 1392.2593994140625) ?  (b('dg0_v_vv') <= -0.1469879150390625) ?  -0.04319657967945692 :   -0.015545587801796937 :   -0.07502831384019086 :   (b('l8b5') <= 2623.2069091796875) ?  (b('ndvi') <= 3668.5) ?  (b('l8b4') <= 635.8302612304688) ?  (b('dg0_v_vv') <= -1.8072123527526855) ?  (b('l8b4') <= 630.7738952636719) ?  (b('dg0_v_vv') <= -5.034023284912109) ?  -0.0037591388887370375 :   -0.0235081251360347 :   0.03064940184043269 :   (b('ndvi') <= 3323.0) ?  (b('l8b11') <= 1310.3475341796875) ?  -0.02502148182906247 :   -0.0065145099855669075 :   (b('dg0_v_vv') <= -1.3065624237060547) ?  -0.026282253003742873 :   -0.06872378288742537 :   (b('ndvi') <= 3555.5) ?  (b('ndvi') <= 3236.0) ?  (b('l8b11_med') <= 1198.4310302734375) ?  0.031641092997601795 :   0.06105762673563904 :   (b('dg0_v_vv') <= -1.8058862686157227) ?  0.10364821484929651 :   0.06412664524661861 :   (b('dg0_v_vv') <= 0.4966468811035156) ?  (b('dg0_v_vv') <= -1.1584281921386719) ?  -0.003990739094181989 :   -0.010822142684000946 :   0.006173729499333813 :   (b('dg0_v_vv') <= 0.6395821571350098) ?  (b('dg0_v_vv') <= 0.32501935958862305) ?  (b('dg0_v_vv') <= 0.1851954460144043) ?  (b('ndvi_med') <= 3547.5) ?  0.08149337642411078 :   0.06744546342473405 :   0.03443599439830908 :   (b('l8b5') <= 2481.111328125) ?  0.10889799471318046 :   (b('dg0_v_vv') <= 0.48818063735961914) ?  0.10239952041376005 :   0.10012472051680615 :   (b('l8b5_med') <= 2629.7320556640625) ?  (b('dg0_v_vv') <= 0.9995365142822266) ?  0.012762962077186246 :   (b('dg0_v_vv') <= 1.7241182327270508) ?  -0.0036540908309639075 :   -0.017566954490311204 :   0.07724001214380816 :   (b('l8b4') <= 513.1579742431641) ?  (b('l8b5_med') <= 3051.766357421875) ?  (b('l8b11_med') <= 1198.2420043945312) ?  (b('dg0_v_vv') <= 0.2007613182067871) ?  (b('l8b4_med') <= 490.6155700683594) ?  0.11989908220045542 :   0.11589995892043048 :   0.11163034890678014 :   (b('l8b5') <= 3621.7896728515625) ?  (b('moss') <= 2.7349822521209717) ?  0.10390996561943396 :   0.09007707534569959 :   (b('dg0_v_vv') <= -0.42009401321411133) ?  0.08170582446597957 :   0.06320412722197932 :   (b('l8b4_med') <= 569.4422302246094) ?  (b('ndvi') <= 2379.0) ?  0.12486323353577555 :   (b('ndvi') <= 2550.0) ?  0.10393691500270892 :   0.11278403152772049 :   0.1728145001864584 :   (b('grass') <= 63.13859176635742) ?  (b('dg0_v_vv') <= -0.37967777252197266) ?  (b('l8b5') <= 2640.0189208984375) ?  (b('dg0_v_vv') <= -1.197784423828125) ?  0.11368333739467229 :   0.1096833292884406 :   (b('l8b4') <= 562.7259216308594) ?  0.059789768965180665 :   0.06949750073143551 :   (b('l8b4_med') <= 573.9899597167969) ?  (b('dg0_v_vv') <= 1.556044101715088) ?  0.07756729593871631 :   0.11928119922342534 :   (b('l8b5') <= 3505.7174072265625) ?  0.09610129185709926 :   0.07083620707644028 :   (b('dg0_v_vv') <= 0.7554092407226562) ?  (b('l8b4_med') <= 561.1671142578125) ?  (b('dg0_v_vv') <= -0.23162126541137695) ?  0.056569590898946955 :   0.048259585917637016 :   0.04274574039615975 :   0.06715961409265847 :   (b('dg0_v_vv') <= 1.0562186241149902) ?  (b('bulk') <= 130.0) ?  (b('ndvi_med') <= 3664.0) ?  (b('ndvi') <= 1991.0) ?  (b('dg0_v_vv') <= -1.176496982574463) ?  (b('l8b11') <= 1146.5726318359375) ?  0.028504514188601282 :   0.0400417032304959 :   (b('l8b5') <= 1672.3843994140625) ?  (b('dg0_v_vv') <= -0.5723180770874023) ?  -0.053653194032446214 :   -0.005012884873899115 :   (b('l8b4') <= 752.7579956054688) ?  -0.03473441597680449 :   -0.07798463605110081 :   (b('l8b4') <= 785.6981201171875) ?  (b('l8b11') <= 1049.2786865234375) ?  (b('dg0_v_vv') <= -0.022231578826904297) ?  -0.00896929744174282 :   0.13884842621593746 :   (b('l8b5') <= 2493.0177001953125) ?  -0.010038440704035883 :   -0.03833121191209335 :   (b('dg0_v_vv') <= 0.264371395111084) ?  (b('grass') <= 62.66605758666992) ?  0.03590059551711953 :   -0.02370578111053111 :   (b('l8b4_med') <= 814.7207336425781) ?  -0.04299712092259654 :   -0.000822986025947034 :   (b('ndvi') <= 4734.5) ?  (b('dg0_v_vv') <= -0.5682010650634766) ?  (b('dg0_v_vv') <= -1.429464340209961) ?  (b('ndvi') <= 2617.0) ?  -0.03944135777785948 :   -0.023180604483682872 :   (b('dg0_v_vv') <= -1.3771495819091797) ?  0.11684718645340561 :   0.061009806599758534 :   (b('l8b11') <= 1106.2200012207031) ?  (b('ndvi') <= 3535.5) ?  -0.05520061929523526 :   -0.06506375043611477 :   (b('ndvi_med') <= 3834.5) ?  0.041406941952917296 :   -0.01948695110625786 :   (b('l8b11') <= 1043.4605407714844) ?  0.026243419763323728 :   (b('ndvi_med') <= 4085.5) ?  0.07736764555689082 :   0.08639350350694947 :   (b('moss') <= 23.60063934326172) ?  (b('ndvi_med') <= 1484.5) ?  (b('ndvi') <= 2183.5) ?  (b('dg0_v_vv') <= 0.7825222015380859) ?  (b('l8b5') <= 1756.6851806640625) ?  0.07614600321906388 :   0.0001949764571602212 :   -0.0819993042206624 :   (b('l8b5') <= 1581.1138916015625) ?  0.23037058250224784 :   (b('l8b4') <= 663.6840515136719) ?  0.14200044013920837 :   0.16592713408847107 :   (b('dg0_v_vv') <= -1.6724920272827148) ?  (b('ndvi') <= 1216.0) ?  -0.12876108283934945 :   (b('l8b11') <= 891.5256958007812) ?  0.03174320998227854 :   -0.024304112109125166 :   (b('moss') <= 12.215838432312012) ?  (b('grass') <= 84.40863800048828) ?  0.03691703546646939 :   -0.015623192652155227 :   (b('l8b4') <= 619.8653259277344) ?  0.02382501633875323 :   -0.002089327316836489 :   (b('l8b11_med') <= 1655.7363891601562) ?  (b('dg0_v_vv') <= -0.7420506477355957) ?  (b('l8b4') <= 880.1321716308594) ?  (b('l8b5') <= 2008.895263671875) ?  0.053761832563247944 :   0.067779706748514 :   0.10127006260471251 :   (b('l8b5_med') <= 1784.8324584960938) ?  0.07435057000056852 :   (b('l8b4') <= 520.5371856689453) ?  -0.08830155805922249 :   -0.03739887044141975 :   (b('sand') <= 51.5) ?  (b('l8b11') <= 1147.9226684570312) ?  (b('l8b11') <= 1124.8286743164062) ?  -0.04095016447321853 :   -0.06355508887739818 :   (b('l8b11') <= 1360.8916015625) ?  -0.012389341206478334 :   -0.029533629888840878 :   (b('dg0_v_vv') <= -0.027993202209472656) ?  (b('l8b5') <= 1689.4564819335938) ?  -0.04591482674636552 :   -0.05585770542315906 :   (b('l8b4') <= 813.7760925292969) ?  -0.0861239019698698 :   -0.07971545210909375 :   (b('l8b11_med') <= 1194.9595947265625) ?  (b('dg0_v_vv') <= 1.3754768371582031) ?  (b('dg0_v_vv') <= 1.167633056640625) ?  (b('dg0_v_vv') <= 1.1441588401794434) ?  -0.05404275366811909 :   -0.04921206154801236 :   -0.0843323871871825 :   (b('l8b4') <= 506.2979278564453) ?  (b('l8b5') <= 1429.2807006835938) ?  -0.022057871559018227 :   (b('l8b5_med') <= 2157.6514892578125) ?  0.05469439196735673 :   (b('l8b5') <= 2080.0675659179688) ?  0.03921115396722813 :   0.040399553501622526 :   (b('ndvi') <= 2034.0) ?  (b('sand') <= 52.5) ?  -0.02571945992469521 :   -0.04767708571925937 :   (b('ndvi') <= 2229.0) ?  0.014440459572655412 :   (b('dg0_v_vv') <= 1.848484992980957) ?  -0.0031708988878914046 :   -0.010988259018400462 :   (b('l8b11_med') <= 1632.7683715820312) ?  (b('l8b11') <= 1369.4396362304688) ?  (b('l8b5') <= 1759.2536010742188) ?  (b('l8b5_med') <= 1549.84619140625) ?  0.0807783503088203 :   (b('grass') <= 61.86792182922363) ?  0.005058881361057629 :   0.03674742695881685 :   (b('l8b5_med') <= 3372.1890869140625) ?  (b('l8b4_med') <= 664.9232482910156) ?  0.02249672433150261 :   0.09048437598006256 :   0.22012473999096518 :   (b('ndvi') <= 3028.0) ?  (b('l8b11_med') <= 1473.3522338867188) ?  (b('l8b11') <= 1410.0800170898438) ?  0.062414547431555716 :   0.01214569755652988 :   (b('dg0_v_vv') <= 2.2989535331726074) ?  0.08904319930953661 :   0.009449787699448403 :   (b('l8b5_med') <= 2715.071533203125) ?  (b('l8b4') <= 818.4091796875) ?  -0.02304013643857966 :   -0.04455618314236087 :   (b('dg0_v_vv') <= 2.7907323837280273) ?  0.03144163942757454 :   -0.015328009394379033 :   (b('sand') <= 67.0) ?  (b('l8b11_med') <= 2063.5125732421875) ?  (b('l8b4') <= 402.7866516113281) ?  (b('dg0_v_vv') <= 3.776826858520508) ?  0.10743378937678588 :   0.09837504913532419 :   (b('bulk') <= 137.5) ?  0.03317429887236295 :   -0.005347325073335606 :   (b('l8b5') <= 2380.0198974609375) ?  (b('l8b11') <= 1231.25) ?  0.10885032249943645 :   0.06190363807946846 :   (b('l8b11') <= 1246.0986328125) ?  0.019309588605971855 :   0.06521177390954686 :   (b('dg0_v_vv') <= 1.614199161529541) ?  -0.13634603684083935 :   (b('l8b4') <= 412.5377655029297) ?  -0.04074581848813977 :   -0.058734150317258704 :   (b('l8b11_med') <= 2649.172119140625) ?  (b('l8b5_med') <= 2986.78076171875) ?  (b('sand') <= 22.0) ?  (b('sand') <= 13.5) ?  (b('l8b4') <= 1363.0614624023438) ?  (b('dg0_v_vv') <= -0.9758691787719727) ?  (b('l8b4') <= 986.7598571777344) ?  (b('l8b11') <= 1559.184326171875) ?  0.07716833199871309 :   (b('ndvi') <= 2303.5) ?  0.056228210509544095 :   0.029748164581544 :   0.09353357784424515 :   (b('ndvi') <= 3018.0) ?  (b('dg0_v_vv') <= -0.43169355392456055) ?  0.15215985512748342 :   (b('ndvi_med') <= 1383.5) ?  0.10024487133279253 :   0.11534760821949011 :   (b('dg0_v_vv') <= 0.9491028785705566) ?  (b('l8b5') <= 2901.34228515625) ?  0.09790030434631014 :   0.04708799728014168 :   (b('l8b11') <= 1827.0198364257812) ?  0.028070993691862334 :   0.04408424681741918 :   0.16458090685786358 :   (b('l8b5') <= 2474.6077880859375) ?  (b('l8b5') <= 2418.3916015625) ?  (b('l8b4') <= 981.0661315917969) ?  (b('dg0_v_vv') <= 0.5347118377685547) ?  (b('ndvi_med') <= 3599.5) ?  0.06068521005918037 :   0.006297309630513406 :   0.16046563204167275 :   (b('l8b11') <= 2043.0220947265625) ?  (b('l8b11') <= 1697.273681640625) ?  0.01054270395929785 :   -0.056508282748366126 :   (b('sand') <= 17.5) ?  0.052949962167943204 :   0.0754744496288981 :   (b('dg0_v_vv') <= 0.009906768798828125) ?  (b('ndvi_med') <= 3396.0) ?  (b('ndvi') <= 2791.0) ?  -0.0038463819576281533 :   -0.013427195811390708 :   (b('sand') <= 17.5) ?  -0.025003432058512576 :   -0.035200977048507054 :   -0.06376726477446631 :   (b('dg0_v_vv') <= -2.305209159851074) ?  (b('ndvi') <= 3690.5) ?  -0.05320763283798771 :   -0.016244809261275694 :   (b('l8b4_med') <= 818.8702392578125) ?  (b('l8b5') <= 2556.3958740234375) ?  -0.013176198015814933 :   (b('ndvi') <= 3178.0) ?  0.06743680029878883 :   0.09849867437936421 :   (b('dg0_v_vv') <= -1.2005181312561035) ?  0.1594061976837397 :   0.146311319926831 :   (b('dg0_v_vv') <= 2.2672359943389893) ?  (b('ndvi_med') <= 2729.5) ?  (b('crops') <= 72.66519546508789) ?  (b('grass') <= 85.5868148803711) ?  (b('sand') <= 36.5) ?  (b('ndvi') <= 3310.5) ?  -0.02723181600702409 :   0.0035443825578472168 :   (b('l8b5') <= 2129.727783203125) ?  -0.0046089254475361755 :   -0.014101492667438372 :   (b('sand') <= 49.5) ?  (b('dg0_v_vv') <= 0.45981740951538086) ?  0.0007406024063571753 :   0.031177519096645612 :   (b('l8b4') <= 1164.4922485351562) ?  -0.03813955198021412 :   0.0010198539444309872 :   (b('bulk') <= 144.5) ?  (b('l8b11_med') <= 1832.9356079101562) ?  (b('ndvi') <= 1596.5) ?  -0.00779156257548331 :   0.020355727636923244 :   (b('dg0_v_vv') <= 1.9594039916992188) ?  0.061160458177412586 :   0.13175758105579316 :   (b('ndvi') <= 1188.0) ?  (b('l8b5') <= 2633.9835205078125) ?  0.017860778136997938 :   -0.0813087868976053 :   (b('l8b5_med') <= 2893.3145751953125) ?  -0.012886425245643785 :   0.005347517514352314 :   (b('grass') <= 45.82082176208496) ?  (b('l8b5_med') <= 2438.36474609375) ?  (b('l8b4') <= 787.0558776855469) ?  (b('l8b5') <= 1989.5031127929688) ?  -0.006542050369844563 :   -0.05422329004549784 :   (b('l8b11') <= 1747.4979248046875) ?  0.07510852923611006 :   0.01399922445426637 :   (b('grass') <= 21.708049774169922) ?  (b('ndvi_med') <= 3858.75) ?  0.0008193968190678206 :   0.03062134617763535 :   (b('l8b11_med') <= 1512.97607421875) ?  -0.032580507230450396 :   -0.007426539008129751 :   (b('grass') <= 63.621238708496094) ?  (b('ndvi') <= 2853.0) ?  (b('dg0_v_vv') <= 0.0081939697265625) ?  0.05591349551789384 :   0.1005240003835841 :   (b('l8b4') <= 828.9872436523438) ?  0.05640331305444011 :   -0.0007269983127554409 :   (b('moss') <= 1.4770330786705017) ?  (b('l8b11') <= 2027.3270263671875) ?  0.006204646263061953 :   0.06425817520804775 :   (b('l8b5_med') <= 2446.29052734375) ?  -0.033087334636949135 :   -0.0010139497710799352 :   (b('l8b4_med') <= 1145.7929077148438) ?  (b('l8b5_med') <= 2812.1878662109375) ?  (b('sand') <= 45.5) ?  (b('dg0_v_vv') <= 3.7030696868896484) ?  (b('ndvi_med') <= 2866.75) ?  0.006039071779501103 :   0.04342879045866851 :   (b('crops') <= 0.339643657207489) ?  0.02037906806014058 :   0.07025888077845836 :   (b('l8b11_med') <= 2321.2286376953125) ?  (b('l8b5') <= 2022.5587768554688) ?  0.0026803636555585227 :   -0.03333798294775899 :   (b('l8b11') <= 2219.4700927734375) ?  0.11945408761656723 :   0.13203262215016798 :   (b('sand') <= 26.0) ?  (b('dg0_v_vv') <= 3.6313180923461914) ?  (b('ndvi') <= 3037.5) ?  0.02242562035233775 :   0.04763371182940229 :   (b('l8b4') <= 1079.5279846191406) ?  0.0023149387523414555 :   -0.0369511217200508 :   (b('l8b4') <= 1196.3527221679688) ?  (b('l8b4') <= 1163.0091552734375) ?  -0.01818235525255641 :   0.025939044733096522 :   (b('l8b5') <= 2785.1695556640625) ?  -0.02782112637961842 :   -0.0648610995912141 :   (b('sand') <= 56.5) ?  (b('l8b5_med') <= 2983.9554443359375) ?  (b('l8b5_med') <= 2573.8905029296875) ?  (b('ndvi') <= 1978.5) ?  0.017312161021386923 :   0.05546978038340873 :   (b('l8b5') <= 2703.7869873046875) ?  0.07726331395681282 :   0.0320434544631447 :   (b('l8b4') <= 1425.74658203125) ?  0.019012282313492224 :   (b('l8b4') <= 1688.4573364257812) ?  -0.10285534001285687 :   -0.08584055110715721 :   (b('ndvi_med') <= 1078.0) ?  0.1016940108729596 :   (b('l8b5') <= 2957.5718994140625) ?  (b('dg0_v_vv') <= 2.519279718399048) ?  -0.08524524757791942 :   -0.0166107458061122 :   0.0946139566408628 :   (b('l8b5_med') <= 3571.581298828125) ?  (b('l8b5') <= 2731.8978271484375) ?  (b('l8b4') <= 2024.0022583007812) ?  (b('l8b11') <= 2505.126708984375) ?  (b('moss') <= 4.218456745147705) ?  (b('l8b4') <= 1512.739501953125) ?  (b('l8b5') <= 1711.2403564453125) ?  0.0068664981069886475 :   0.05779573118786977 :   (b('ndvi') <= 1209.0) ?  -0.027643139393946747 :   0.04089166883084049 :   (b('dg0_v_vv') <= -0.3385610580444336) ?  (b('ndvi') <= 2608.5) ?  -0.012036612034313372 :   0.0165297704377836 :   (b('grass') <= 47.533498764038086) ?  0.03569705988858663 :   -0.1487216322708399 :   (b('l8b4') <= 1993.0022583007812) ?  (b('l8b5') <= 2664.146728515625) ?  (b('l8b5') <= 2311.9725341796875) ?  -0.04490631466774283 :   -0.0016128094099668738 :   (b('crops') <= 60.74416923522949) ?  -0.03110288288022032 :   0.06027131212216396 :   (b('ndvi_med') <= 1751.0) ?  (b('dg0_v_vv') <= -0.46534156799316406) ?  -0.0708644950235251 :   -0.07186449705008302 :   (b('l8b5_med') <= 3025.82421875) ?  -0.06381498284104879 :   -0.05706961208015479 :   (b('ndvi') <= 1210.0) ?  (b('l8b5_med') <= 3025.82421875) ?  0.2059303844627758 :   (b('dg0_v_vv') <= -1.0147271156311035) ?  0.15813549222108092 :   (b('dg0_v_vv') <= -0.9463763236999512) ?  0.20413549603577819 :   0.16513549150582518 :   0.05830195624170012 :   (b('moss') <= 13.856780052185059) ?  (b('l8b11_med') <= 1982.345947265625) ?  (b('dg0_v_vv') <= 2.386629581451416) ?  (b('l8b11_med') <= 1879.1678466796875) ?  (b('l8b5') <= 3672.85498046875) ?  0.013222117303736226 :   0.03992747525463625 :   (b('l8b4') <= 753.927490234375) ?  -0.08720129218420167 :   0.04683306413990528 :   (b('l8b5_med') <= 3317.088623046875) ?  (b('l8b4') <= 1209.8782958984375) ?  0.031545050422684566 :   0.08372336123878595 :   (b('dg0_v_vv') <= 2.6867287158966064) ?  0.16726422762524767 :   0.09399798866744387 :   (b('l8b5') <= 3501.5458984375) ?  (b('l8b11') <= 2030.467529296875) ?  (b('l8b11') <= 2006.69140625) ?  0.03236054102592737 :   0.08253401239284965 :   (b('sand') <= 48.0) ?  0.015700785830432152 :   -0.007581232868037262 :   (b('l8b11_med') <= 2488.0540771484375) ?  (b('ndvi') <= 2618.5) ?  -0.0087326743507802 :   -0.04960704592974006 :   (b('ndvi') <= 3311.0) ?  0.014531083856129874 :   0.08640006767818867 :   (b('l8b11_med') <= 1493.65966796875) ?  (b('l8b5') <= 3040.501708984375) ?  (b('ndvi_med') <= 2739.5) ?  (b('l8b11') <= 1624.8284301757812) ?  0.08715668994836666 :   -0.03284330736942433 :   (b('dg0_v_vv') <= 1.2633740901947021) ?  0.12991299626875613 :   0.07806155025510193 :   (b('ndvi') <= 3273.5) ?  (b('l8b11') <= 1609.2401733398438) ?  0.025902703659987328 :   -0.01539481540007935 :   (b('dg0_v_vv') <= 0.7143068313598633) ?  0.05778334368387557 :   0.10614757640949729 :   (b('grass') <= 21.71052646636963) ?  (b('ndvi') <= 3141.5) ?  (b('l8b4_med') <= 1415.7860107421875) ?  0.0004997441735703767 :   -0.03210111765797153 :   (b('l8b5') <= 3524.34375) ?  0.06028849574824813 :   0.0060938317625612816 :   (b('l8b5') <= 3214.5106201171875) ?  (b('l8b5') <= 3034.1453857421875) ?  -0.02365884114921099 :   0.008667664278184403 :   (b('grass') <= 24.99324893951416) ?  -0.00702259486539041 :   -0.04641681805775584 :   (b('l8b11_med') <= 1157.8265380859375) ?  (b('l8b11') <= 1796.5224609375) ?  (b('l8b4_med') <= 539.290283203125) ?  (b('crops') <= 50.06374740600586) ?  0.11713532492620826 :   0.09009057312247887 :   0.039409578827895114 :   (b('l8b5') <= 3846.040283203125) ?  (b('dg0_v_vv') <= 5.551367521286011) ?  (b('dg0_v_vv') <= 2.537583351135254) ?  (b('dg0_v_vv') <= -2.426581859588623) ?  0.01585484460287831 :   0.04025318339834432 :   (b('l8b11') <= 2230.5882568359375) ?  0.006919091376031528 :   -7.355249901422334e-05 :   0.07417937719631484 :   (b('l8b4_med') <= 493.5237579345703) ?  0.022359414139103445 :   -0.04345303334203199 :   (b('l8b11') <= 1629.1834716796875) ?  (b('dg0_v_vv') <= 1.837204933166504) ?  (b('dg0_v_vv') <= 0.8399829864501953) ?  (b('bulk') <= 128.5) ?  -0.05137354112015291 :   (b('dg0_v_vv') <= -0.49582672119140625) ?  0.005576229976829543 :   0.04676426426937698 :   (b('l8b5') <= 2959.180908203125) ?  (b('dg0_v_vv') <= 1.5968585014343262) ?  -0.007339493513479137 :   -0.021093359839214898 :   (b('l8b5') <= 3730.650146484375) ?  -0.03020633644853063 :   -0.03996212441584571 :   (b('l8b11_med') <= 2518.2154541015625) ?  (b('sand') <= 66.5) ?  0.0739066538698534 :   0.05800303598383877 :   (b('dg0_v_vv') <= 2.9267122745513916) ?  0.034232744261406184 :   0.015823740217445204 :   (b('l8b5') <= 2867.196533203125) ?  (b('l8b11') <= 1855.6644287109375) ?  (b('l8b5') <= 2082.4375610351562) ?  (b('ndvi') <= 2001.0) ?  0.006998128587023089 :   0.008548406941435771 :   (b('crops') <= 73.68918991088867) ?  0.06816423446761272 :   0.09882600248871254 :   (b('l8b5') <= 2521.1158447265625) ?  (b('ndvi') <= 1906.5) ?  -0.037731545852775036 :   -0.05002513728660762 :   (b('l8b5') <= 2812.03466796875) ?  -0.001733193949274627 :   0.033425652107127626 :   (b('dg0_v_vv') <= 4.242205381393433) ?  (b('l8b5_med') <= 4025.0614013671875) ?  (b('l8b11') <= 2961.228271484375) ?  -0.03619621731615831 :   -0.02218215456775652 :   (b('l8b4') <= 1094.2437133789062) ?  0.0022907353183800605 :   0.05056915500452183 :   (b('dg0_v_vv') <= 5.188971757888794) ?  (b('ndvi_med') <= 2171.5) ?  0.039443443415438734 :   0.06455556118082187 :   -0.012786264591336458 :   (b('grass') <= 1.0004997551441193) ?  (b('ndvi') <= 1011.0) ?  (b('dg0_v_vv') <= 0.9475173950195312) ?  (b('l8b5') <= 1970.1157836914062) ?  (b('dg0_v_vv') <= 0.07752037048339844) ?  0.018057121183087915 :   0.0466051145643952 :   (b('ndvi') <= 1005.5) ?  (b('crops') <= 91.9686164855957) ?  (b('ndvi') <= 917.0) ?  (b('l8b5') <= 2320.261962890625) ?  -0.02702687432924407 :   -0.013403866977009259 :   (b('l8b4') <= 1688.5248413085938) ?  -0.06709017472864359 :   -0.03228477471146264 :   (b('l8b11') <= 3580.6287841796875) ?  (b('ndvi') <= 1000.5) ?  -0.0283644163626425 :   -0.00012943713444324434 :   0.11515765452756463 :   (b('dg0_v_vv') <= 0.5558047294616699) ?  -0.05149243986792827 :   (b('l8b4_med') <= 2271.3209228515625) ?  -0.061306624021507744 :   -0.06513287495215489 :   (b('dg0_v_vv') <= 1.3153109550476074) ?  (b('l8b5_med') <= 2254.820556640625) ?  0.039176377208188506 :   (b('grass') <= 0.27586206793785095) ?  0.15094003505557313 :   0.06287963122329637 :   (b('l8b4') <= 1779.4755859375) ?  (b('l8b4_med') <= 2271.3209228515625) ?  -0.028163707488197798 :   -0.03852751600798168 :   (b('ndvi') <= 954.0) ?  (b('dg0_v_vv') <= 1.428715705871582) ?  -0.025902550726783546 :   (b('crops') <= 49.389366149902344) ?  0.01080647250123537 :   -0.0019255985245138496 :   0.037096264336699714 :   (b('l8b5') <= 2872.5596923828125) ?  (b('dg0_v_vv') <= 3.515869379043579) ?  (b('sand') <= 55.0) ?  (b('l8b11') <= 2060.2054443359375) ?  (b('ndvi') <= 2019.0) ?  (b('dg0_v_vv') <= 0.8523678779602051) ?  0.018608798232414417 :   0.043690494453562406 :   (b('dg0_v_vv') <= -0.48689889907836914) ?  0.03956471849868634 :   0.08069624974517034 :   (b('dg0_v_vv') <= 3.3052978515625) ?  (b('l8b11') <= 3156.693603515625) ?  -0.005744268523297751 :   0.02978175065263836 :   -0.08175305915450176 :   (b('l8b11') <= 2861.87890625) ?  (b('l8b5') <= 1998.9990844726562) ?  (b('dg0_v_vv') <= -0.28357791900634766) ?  -0.0017993376278478042 :   -0.013942629223419725 :   (b('l8b11') <= 2794.69384765625) ?  0.02719740977389885 :   0.005792340168580129 :   (b('l8b4') <= 1852.2540893554688) ?  (b('ndvi') <= 1060.5) ?  0.022144658214328553 :   0.05505905477163162 :   0.08994016444135385 :   (b('l8b4_med') <= 1649.88818359375) ?  -0.027064528398463933 :   (b('l8b4') <= 1782.654296875) ?  (b('l8b11') <= 1804.6836547851562) ?  0.005079862973132593 :   (b('dg0_v_vv') <= 7.207872629165649) ?  0.044388615612205105 :   0.08290357556343123 :   0.10870520004638182 :   (b('dg0_v_vv') <= -1.742262840270996) ?  (b('ndvi') <= 2282.5) ?  (b('dg0_v_vv') <= -2.256289005279541) ?  (b('ndvi_med') <= 1424.0) ?  0.017546298442126435 :   (b('l8b5') <= 3497.5291748046875) ?  0.026558408310750153 :   0.034188121246203046 :   (b('l8b11') <= 3040.9764404296875) ?  (b('l8b4_med') <= 1953.6048583984375) ?  -0.0594118739794649 :   -0.031171316429192863 :   (b('ndvi_med') <= 1481.5) ?  0.0005715738412395421 :   -0.008441392793757885 :   (b('l8b5') <= 3600.725341796875) ?  -0.06428585041302573 :   (b('dg0_v_vv') <= -2.321016311645508) ?  (b('l8b11_med') <= 2890.52392578125) ?  -0.03782045911686102 :   -0.0422174271267427 :   -0.03074233852408431 :   (b('ndvi') <= 1074.0) ?  (b('l8b4') <= 2220.5069580078125) ?  0.04066880756531449 :   0.06478589198649787 :   (b('ndvi') <= 4222.5) ?  (b('dg0_v_vv') <= -1.5603184700012207) ?  (b('dg0_v_vv') <= -1.5983772277832031) ?  0.0056693218978680834 :   0.09468319564642852 :   (b('l8b5') <= 4034.74951171875) ?  -0.0025334719948178147 :   -0.016928237682391517 :   (b('l8b4_med') <= 2271.3209228515625) ?  (b('dg0_v_vv') <= 1.0780887603759766) ?  0.031913949820707804 :   0.033291679431212445 :   0.0472024156559995 :   (b('grass') <= 27.41930866241455) ?  (b('l8b4_med') <= 1631.4669799804688) ?  (b('l8b4') <= 1878.9908447265625) ?  (b('l8b5') <= 2517.72216796875) ?  (b('ndvi') <= 2081.0) ?  (b('l8b4') <= 1374.9554443359375) ?  0.08033299704148109 :   0.01731207376395935 :   (b('dg0_v_vv') <= 2.377394676208496) ?  (b('dg0_v_vv') <= 0.03300142288208008) ?  -0.10621941558955073 :   -0.1253171001449811 :   0.030160658367524712 :   (b('l8b4') <= 1438.9642333984375) ?  (b('l8b11_med') <= 2807.767822265625) ?  (b('l8b4') <= 1113.55712890625) ?  0.027274954777376487 :   0.029902630733274052 :   (b('l8b4') <= 1009.784912109375) ?  0.02240012210739481 :   0.02469348206417496 :   (b('ndvi') <= 2521.5) ?  (b('l8b5') <= 2759.9107666015625) ?  0.034004767113375245 :   0.02575208484470312 :   0.05026078159031905 :   (b('dg0_v_vv') <= -0.6167559623718262) ?  0.14846198180374676 :   0.08298294087167507 :   (b('l8b4_med') <= 2187.779052734375) ?  (b('l8b4_med') <= 2066.49853515625) ?  (b('dg0_v_vv') <= 2.304236888885498) ?  (b('l8b11') <= 1731.3468017578125) ?  (b('grass') <= 16.311988830566406) ?  -0.0992977604275749 :   -0.06609353317170505 :   (b('l8b11') <= 2625.5745849609375) ?  -0.018644429989184478 :   -0.027085151697844187 :   (b('l8b11') <= 2816.7159423828125) ?  (b('l8b11') <= 2004.5325317382812) ?  0.013193663709423781 :   -0.02755517267266873 :   (b('l8b5') <= 3173.16845703125) ?  -0.0174777947822767 :   0.016741539284679433 :   (b('dg0_v_vv') <= 1.3613271713256836) ?  (b('l8b5') <= 2499.7274169921875) ?  (b('ndvi') <= 822.5) ?  -0.033480101372730414 :   -0.01102708458591446 :   (b('l8b4') <= 1381.897705078125) ?  -0.05385502427210751 :   -0.03363943068316547 :   (b('dg0_v_vv') <= 1.4726572036743164) ?  (b('l8b5') <= 2963.49658203125) ?  -0.04522341489019362 :   0.01475397103200494 :   (b('l8b5') <= 3416.955810546875) ?  -0.03375040151664632 :   -0.022754625875842597 :   (b('l8b5') <= 2792.2857666015625) ?  (b('dg0_v_vv') <= 0.2840561866760254) ?  (b('l8b4') <= 2216.074462890625) ?  (b('ndvi') <= 737.5) ?  0.018497594636754905 :   0.013513003780453038 :   (b('l8b11_med') <= 2956.1988525390625) ?  -0.012137015883792415 :   -0.003066404201899256 :   (b('l8b11_med') <= 2960.072265625) ?  (b('ndvi') <= 722.0) ?  0.06424090086109485 :   0.0621140412079115 :   (b('bulk') <= 149.5) ?  0.03921932919726248 :   0.052645303046118336 :   (b('dg0_v_vv') <= 0.8203220367431641) ?  (b('grass') <= 24.79003620147705) ?  (b('l8b4') <= 3146.07666015625) ?  -0.015674942494951796 :   -0.03068760721119477 :   (b('ndvi') <= 875.0) ?  0.0034411299342839985 :   -0.030506819371301674 :   (b('l8b11') <= 3142.228271484375) ?  (b('l8b11') <= 3124.43994140625) ?  0.0034695810623306123 :   0.0521846284280702 :   (b('ndvi') <= 1605.0) ?  -0.014193364945869511 :   0.005326266265768764 :   (b('dg0_v_vv') <= 0.666295051574707) ?  (b('grass') <= 50.4460563659668) ?  (b('ndvi_med') <= 1480.0) ?  (b('ndvi_med') <= 1458.5) ?  (b('crops') <= 52.13633346557617) ?  (b('ndvi') <= 1351.5) ?  -0.007449395256312366 :   0.016814837867267323 :   (b('l8b4') <= 1151.4386291503906) ?  -0.026251495301133645 :   -0.06206831967622911 :   (b('l8b11') <= 3384.9942626953125) ?  (b('dg0_v_vv') <= 0.27710390090942383) ?  0.05155893270594262 :   0.09304206587577477 :   -0.02808335637151438 :   (b('l8b11') <= 2682.099853515625) ?  (b('dg0_v_vv') <= -2.135037422180176) ?  (b('ndvi_med') <= 1636.25) ?  -0.05226123769636476 :   -0.037984914822717715 :   (b('ndvi') <= 2565.5) ?  0.015950492240311615 :   -0.020199200742851413 :   (b('l8b11') <= 3333.1025390625) ?  (b('l8b11') <= 3093.4615478515625) ?  -0.016712723985737164 :   -0.029883086058645925 :   (b('bulk') <= 144.0) ?  0.09286016148312307 :   -0.009682650676565995 :   (b('sand') <= 47.5) ?  (b('l8b4') <= 1414.2357788085938) ?  (b('dg0_v_vv') <= 0.03553915023803711) ?  (b('l8b11') <= 2198.9493408203125) ?  0.017624802596267482 :   0.04933654881123371 :   (b('bulk') <= 155.5) ?  -0.044309863868980504 :   -0.040750279986515814 :   (b('ndvi_med') <= 1727.75) ?  (b('l8b11') <= 2444.9000244140625) ?  -0.03644502772364481 :   -0.01418963886355053 :   (b('l8b4') <= 1632.829833984375) ?  -0.051442771860095285 :   -0.08382141739376804 :   (b('ndvi_med') <= 1489.0) ?  (b('ndvi') <= 1424.0) ?  (b('l8b4') <= 1611.5009155273438) ?  0.03819527229986459 :   -0.030996139027301257 :   (b('l8b5') <= 2647.16259765625) ?  -0.03466079873503064 :   0.004425774100987839 :   (b('l8b11') <= 2805.9588623046875) ?  (b('l8b4') <= 1370.1571655273438) ?  -0.03957257128693668 :   -0.027619728480417823 :   (b('dg0_v_vv') <= -1.7416677474975586) ?  -0.07860079104965656 :   -0.044365552331416806 :   (b('moss') <= 12.43514108657837) ?  (b('moss') <= 7.676438808441162) ?  (b('l8b5') <= 2313.1558837890625) ?  (b('l8b4_med') <= 1450.0427856445312) ?  -0.0824711028577304 :   (b('dg0_v_vv') <= 0.8393425941467285) ?  -0.06682814795482442 :   -0.03828916409170136 :   (b('l8b4') <= 1563.4301147460938) ?  (b('l8b11') <= 1600.4156494140625) ?  -0.0650994559175006 :   0.0363520270007426 :   (b('ndvi_med') <= 1607.0) ?  -0.04851556744670772 :   0.008356524017290772 :   (b('l8b5') <= 3327.68505859375) ?  (b('l8b11') <= 2631.7899169921875) ?  (b('ndvi') <= 3877.5) ?  0.06558116213249689 :   -0.010606272850518184 :   (b('l8b11') <= 3065.0093994140625) ?  0.01920008431670276 :   0.057364101903445625 :   (b('l8b4_med') <= 1795.3252563476562) ?  -0.03841043563825616 :   -0.007936054546150567 :   (b('dg0_v_vv') <= 0.6786432266235352) ?  0.1091162078796915 :   (b('l8b11_med') <= 2657.93896484375) ?  (b('ndvi') <= 1727.5) ?  (b('l8b5') <= 2460.494140625) ?  -0.05119463155769647 :   -0.023957153106631303 :   (b('dg0_v_vv') <= 1.0589303970336914) ?  -0.046290483579571946 :   -0.05755149059583056 :   (b('l8b4') <= 1386.4633178710938) ?  (b('l8b4_med') <= 1676.150634765625) ?  0.04422457165610254 :   -0.002104631718206275 :   (b('sand') <= 63.5) ?  -0.009711548559984953 :   -0.042705749536592 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1735.9608764648438) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('crops') <= 35.47955322265625) ?  (b('sand') <= 36.5) ?  (b('l8b4') <= 468.3511047363281) ?  (b('l8b11_med') <= 1370.9267578125) ?  (b('l8b11') <= 783.1763610839844) ?  (b('dg0_v_vv') <= -0.24941778182983398) ?  -0.046683942079043306 :   -0.053909617364238216 :   (b('ndvi') <= 3229.0) ?  (b('l8b5') <= 2716.4249267578125) ?  0.048792279084542874 :   -0.0014714630086994362 :   (b('dg0_v_vv') <= -1.2100319862365723) ?  0.020201694684194832 :   0.0897995986283584 :   (b('l8b4') <= 396.27464294433594) ?  0.0004976136753146454 :   (b('l8b5') <= 3008.967041015625) ?  -0.03488409272639534 :   -0.06509458864263329 :   (b('l8b4') <= 718.6219177246094) ?  (b('l8b11') <= 1264.8095703125) ?  (b('ndvi') <= 4906.0) ?  (b('l8b11') <= 1124.9669189453125) ?  0.015414881232138912 :   -0.009598223714709845 :   (b('ndvi_med') <= 3584.5) ?  0.05435223128181407 :   0.0760022212130702 :   (b('l8b11') <= 1300.4547119140625) ?  (b('ndvi_med') <= 3645.5) ?  0.10539526828102913 :   0.03058598855687536 :   (b('ndvi_med') <= 3818.75) ?  0.009707512647549222 :   0.06314814838576197 :   (b('l8b11_med') <= 1347.2418823242188) ?  (b('l8b11') <= 1394.990478515625) ?  (b('l8b4') <= 833.972900390625) ?  -0.003282188209046512 :   0.0548196105996961 :   (b('l8b4_med') <= 614.0503845214844) ?  -0.03331414601379545 :   0.02288661648537029 :   (b('l8b5') <= 2559.340576171875) ?  (b('ndvi') <= 2186.5) ?  0.035661360030331324 :   0.03464244098877067 :   (b('ndvi_med') <= 3735.5) ?  0.10726647614635543 :   0.15576702384747215 :   (b('ndvi') <= 4238.5) ?  (b('l8b4') <= 585.3588562011719) ?  (b('l8b11') <= 1084.552001953125) ?  (b('bulk') <= 141.0) ?  (b('l8b11') <= 1042.0650634765625) ?  -0.005589446484694849 :   -0.06759924540983105 :   (b('l8b5') <= 2342.4378662109375) ?  0.005008648639679467 :   0.04118770754607007 :   (b('moss') <= 10.962814331054688) ?  (b('ndvi') <= 3939.5) ?  -0.0072848904117689085 :   -0.0745985233664238 :   (b('l8b11_med') <= 1102.9674682617188) ?  0.0988256284875448 :   0.05085401455895999 :   (b('l8b4') <= 769.7160949707031) ?  (b('l8b4') <= 753.5161437988281) ?  (b('dg0_v_vv') <= -1.09181547164917) ?  -0.05162707179715515 :   -0.010517011920798505 :   0.1391195007287422 :   (b('l8b4_med') <= 629.5668029785156) ?  (b('dg0_v_vv') <= -0.46628260612487793) ?  -0.05486416977617821 :   -0.028616351895382176 :   (b('l8b11') <= 1181.1840209960938) ?  -0.09232581191192618 :   -0.11014740713204219 :   (b('bulk') <= 141.0) ?  (b('l8b5') <= 4460.89208984375) ?  (b('l8b4') <= 683.847900390625) ?  (b('dg0_v_vv') <= -0.6394481658935547) ?  0.002608663204454397 :   -0.029261452272596716 :   (b('ndvi_med') <= 4525.5) ?  -0.08582375426231328 :   -0.08753575338572256 :   (b('ndvi') <= 4522.0) ?  (b('l8b11_med') <= 1128.1526489257812) ?  -0.008805880449320291 :   -0.021395490879191875 :   (b('l8b5') <= 4845.569580078125) ?  0.04863420529375134 :   0.007894166605890152 :   (b('dg0_v_vv') <= 0.33362674713134766) ?  (b('sand') <= 39.5) ?  0.01189507910499149 :   (b('dg0_v_vv') <= -1.132319450378418) ?  -0.019122264445227882 :   -0.017914671633618373 :   (b('ndvi') <= 4615.5) ?  -0.1309982280894705 :   (b('dg0_v_vv') <= 0.8997001647949219) ?  -0.11433593626196802 :   -0.12117913409448207 :   (b('sand') <= 69.0) ?  (b('dg0_v_vv') <= 0.36918067932128906) ?  (b('ndvi') <= 3207.5) ?  (b('ndvi_med') <= 3969.5) ?  (b('ndvi') <= 3027.5) ?  (b('l8b5') <= 2270.1546630859375) ?  0.08355209228564868 :   0.05386534149383781 :   (b('bulk') <= 145.5) ?  0.11734118440293231 :   0.14326301652169332 :   (b('dg0_v_vv') <= 0.0595250129699707) ?  (b('l8b5') <= 3070.1356201171875) ?  0.07796860827351254 :   0.027790275919474545 :   (b('l8b5') <= 3070.1356201171875) ?  -0.05196803340188244 :   -0.01953872089753822 :   (b('grass') <= 44.62780952453613) ?  (b('l8b4_med') <= 625.6583251953125) ?  (b('l8b4_med') <= 555.4220581054688) ?  0.010623867806631715 :   0.03673911735568419 :   (b('l8b4') <= 431.52093505859375) ?  0.045478131853042676 :   -0.013307935295827585 :   (b('dg0_v_vv') <= -0.677299976348877) ?  0.09411951121932122 :   0.14565770608920886 :   (b('l8b4') <= 617.4010314941406) ?  (b('ndvi_med') <= 4388.0) ?  (b('sand') <= 51.0) ?  (b('l8b11') <= 1003.1830139160156) ?  0.024441828169833334 :   0.06538340275989402 :   (b('ndvi_med') <= 3163.0) ?  -0.029489131357984394 :   -0.08379877186574292 :   (b('l8b4_med') <= 518.2619934082031) ?  (b('l8b11_med') <= 1157.4540405273438) ?  0.1251065770608542 :   0.1539858431336089 :   (b('l8b4_med') <= 633.909912109375) ?  0.08847259894745627 :   0.06360623874468355 :   (b('ndvi') <= 4064.5) ?  (b('sand') <= 62.5) ?  (b('ndvi_med') <= 4338.25) ?  0.04750526690475421 :   0.14009785855147716 :   (b('l8b5_med') <= 2807.138427734375) ?  -0.01118295460401425 :   -0.02576667333142149 :   (b('ndvi_med') <= 3785.75) ?  (b('dg0_v_vv') <= 1.0890748500823975) ?  0.06611805188580094 :   0.08468312678288115 :   (b('ndvi') <= 4942.0) ?  -0.04046188977431972 :   0.019753058132747265 :   (b('l8b5') <= 2980.4603271484375) ?  (b('dg0_v_vv') <= -0.8736944198608398) ?  (b('l8b5_med') <= 3460.2845458984375) ?  (b('grass') <= 2.715542569756508) ?  (b('dg0_v_vv') <= -2.7215981483459473) ?  0.03237957881629533 :   0.02887451271657493 :   (b('crops') <= 68.10383796691895) ?  0.007961683511240525 :   -0.004328787508478149 :   (b('dg0_v_vv') <= -1.8563666343688965) ?  -0.039000792392581585 :   (b('dg0_v_vv') <= -1.2957119941711426) ?  -0.04563259476617268 :   -0.04063259953454426 :   (b('dg0_v_vv') <= -0.4158177375793457) ?  (b('crops') <= 55.91495704650879) ?  0.15115061438440222 :   0.08764626262043482 :   (b('ndvi_med') <= 3656.75) ?  (b('dg0_v_vv') <= 0.5162110328674316) ?  0.007532856390061349 :   -0.009916985010774685 :   (b('l8b4_med') <= 530.8816223144531) ?  0.025212569446520845 :   0.048981692991131275 :   (b('l8b4') <= 367.22666931152344) ?  (b('ndvi_med') <= 3585.0) ?  (b('l8b5') <= 5558.252197265625) ?  (b('grass') <= 11.388772249221802) ?  -0.03768185848212177 :   -0.057066148042740156 :   (b('ndvi_med') <= 2910.0) ?  0.007162712739124111 :   -0.0212554013639602 :   (b('ndvi') <= 4437.0) ?  (b('ndvi') <= 4184.0) ?  -0.09414518415098717 :   -0.06765237334953625 :   -0.13250932883616773 :   (b('dg0_v_vv') <= -3.492854595184326) ?  (b('l8b11') <= 1154.7627563476562) ?  (b('bulk') <= 130.5) ?  -0.045226146795523434 :   -0.07023504457601643 :   (b('dg0_v_vv') <= -4.340610980987549) ?  -0.04126373680762305 :   -0.006091050716085604 :   (b('l8b11') <= 1327.5875854492188) ?  (b('ndvi') <= 3854.0) ?  0.027344625711111334 :   0.008404938690824306 :   (b('l8b11_med') <= 941.7888488769531) ?  0.022611872018744192 :   -0.02359048446893872 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11') <= 958.7872924804688) ?  (b('bulk') <= 150.5) ?  (b('l8b5') <= 1797.7119140625) ?  (b('l8b4') <= 335.7366180419922) ?  (b('l8b5') <= 1277.6231079101562) ?  (b('dg0_v_vv') <= 1.10563325881958) ?  0.08606860204952721 :   0.0831971464392896 :   0.07115204000530448 :   (b('dg0_v_vv') <= -0.19228458404541016) ?  (b('dg0_v_vv') <= -0.29335546493530273) ?  0.026532716952782337 :   0.18259409126353715 :   (b('ndvi') <= 2886.0) ?  0.037237576578102255 :   0.007515842146062564 :   (b('l8b5') <= 4495.730224609375) ?  (b('l8b11') <= 954.4361572265625) ?  (b('ndvi_med') <= 3877.5) ?  0.0701403850354039 :   0.02444665932089446 :   (b('l8b5_med') <= 2682.068603515625) ?  0.09165587846825049 :   0.10395563193939994 :   (b('dg0_v_vv') <= -1.4115853309631348) ?  0.03294363407114281 :   -0.0214621351162452 :   (b('l8b4') <= 404.1337890625) ?  (b('l8b11_med') <= 1040.0068969726562) ?  (b('ndvi') <= 2790.5) ?  0.025180716468990394 :   0.03325060934119392 :   0.04743480319768259 :   (b('dg0_v_vv') <= -0.04373502731323242) ?  -0.023129931840330575 :   (b('crops') <= 3.3066861629486084) ?  (b('l8b11') <= 835.87744140625) ?  -0.05136051307838799 :   -0.05737032967371858 :   -0.08113332992700603 :   (b('ndvi') <= 2770.5) ?  (b('ndvi_med') <= 2781.75) ?  (b('l8b11') <= 1238.0838623046875) ?  (b('l8b11') <= 1099.8031616210938) ?  (b('bulk') <= 148.5) ?  0.008435234011127085 :   0.0077347725502752845 :   -0.008011109229237187 :   (b('ndvi') <= 2467.0) ?  (b('ndvi_med') <= 2083.75) ?  -0.020803626252019675 :   -0.04201486159065728 :   -0.009888393071208645 :   (b('l8b5') <= 2758.0596923828125) ?  (b('l8b11_med') <= 1244.5859985351562) ?  (b('ndvi') <= 1869.5) ?  0.157287103545141 :   0.06076172455600817 :   (b('ndvi') <= 1725.0) ?  -0.07869182855629597 :   0.003024481024763326 :   (b('l8b4') <= 619.7152404785156) ?  0.12129197380596762 :   0.13167523790611965 :   (b('dg0_v_vv') <= 0.7888178825378418) ?  (b('l8b5') <= 2294.676513671875) ?  (b('l8b5') <= 2103.5667724609375) ?  (b('l8b5') <= 1695.2687377929688) ?  -0.056607044147554714 :   -0.0016261487137742598 :   (b('ndvi') <= 4313.5) ?  0.060437599586149554 :   0.02192404170138787 :   (b('ndvi') <= 3336.0) ?  (b('ndvi_med') <= 3755.0) ?  0.0189077919842105 :   -0.12488308907706072 :   (b('bulk') <= 147.5) ?  -0.025298172024837053 :   0.015305349206763352 :   (b('bulk') <= 144.0) ?  (b('l8b11_med') <= 1254.4337158203125) ?  (b('dg0_v_vv') <= 1.4443011283874512) ?  0.04038753894449567 :   0.00314627845285928 :   (b('l8b4') <= 536.4166107177734) ?  -0.005633514179505139 :   -0.07080570449803297 :   (b('l8b11') <= 1109.7821655273438) ?  0.0535297704921035 :   (b('l8b4') <= 595.1662902832031) ?  0.1087366937700348 :   0.08813396438755883 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('dg0_v_vv') <= -1.9178524017333984) ?  (b('ndvi_med') <= 3442.5) ?  0.05437768561870576 :   0.02004658254514019 :   (b('l8b4') <= 322.4314422607422) ?  (b('dg0_v_vv') <= -0.08648824691772461) ?  (b('moss') <= 6.607478976249695) ?  0.04783386659643463 :   0.06222783290753614 :   (b('dg0_v_vv') <= -0.0663909912109375) ?  0.13110409530166234 :   0.06580250394758891 :   (b('l8b4') <= 351.8328857421875) ?  (b('dg0_v_vv') <= -0.7209587097167969) ?  0.09913485590159543 :   0.11546764459438863 :   (b('moss') <= 2.254432260990143) ?  0.06694731319918915 :   0.08054627498259627 :   (b('moss') <= 4.263392806053162) ?  (b('moss') <= 0.7690265774726868) ?  (b('ndvi') <= 2366.0) ?  0.10436111370689827 :   (b('l8b4_med') <= 550.9379577636719) ?  0.09477019659229653 :   0.0868189791108719 :   (b('dg0_v_vv') <= 1.2769227027893066) ?  (b('dg0_v_vv') <= 0.09958887100219727) ?  0.10536144559973815 :   0.10074340253842799 :   0.08360975524245684 :   (b('l8b11_med') <= 1261.8388061523438) ?  (b('ndvi') <= 3879.5) ?  (b('dg0_v_vv') <= -0.21280956268310547) ?  0.05549651791857385 :   0.06000977266242652 :   (b('dg0_v_vv') <= -0.41936254501342773) ?  0.11222353144236646 :   0.06725798955139163 :   (b('dg0_v_vv') <= 0.49697303771972656) ?  (b('ndvi') <= 4492.0) ?  0.08838352549843588 :   0.1026839997736193 :   (b('dg0_v_vv') <= 1.004955768585205) ?  0.12695753840434099 :   0.10938122896281605 :   (b('l8b5') <= 2478.9398193359375) ?  (b('l8b4') <= 641.6177062988281) ?  (b('dg0_v_vv') <= -1.3131413459777832) ?  (b('ndvi') <= 3207.5) ?  0.02766019869090111 :   (b('ndvi') <= 3587.5) ?  -0.01894193157343123 :   -0.008248873725423628 :   (b('ndvi') <= 3323.0) ?  (b('ndvi_med') <= 3438.5) ?  -0.044103618424808116 :   -0.005863058987010228 :   (b('l8b4') <= 622.6875915527344) ?  -0.07488279353022911 :   -0.048820015667136535 :   (b('l8b11') <= 1318.109619140625) ?  (b('ndvi') <= 3523.5) ?  (b('dg0_v_vv') <= 0.7986822128295898) ?  0.052324201055777515 :   0.033709113392689775 :   (b('ndvi_med') <= 3461.25) ?  -0.0007739657394175564 :   -0.010789931288544719 :   (b('ndvi') <= 3806.5) ?  (b('l8b5') <= 2428.09716796875) ?  0.10639948475778833 :   0.07630835414776035 :   (b('dg0_v_vv') <= -0.00666046142578125) ?  0.06724534603438212 :   0.04197105280297355 :   (b('l8b4_med') <= 573.9899597167969) ?  (b('l8b4') <= 651.3680419921875) ?  (b('l8b11') <= 1454.7259521484375) ?  (b('dg0_v_vv') <= 0.6353654861450195) ?  0.05829118827493022 :   0.0732226165590724 :   (b('dg0_v_vv') <= -2.114851474761963) ?  0.0745255873814229 :   0.08878089847054169 :   (b('l8b11') <= 1598.3849487304688) ?  (b('bulk') <= 112.0) ?  0.06578341252554899 :   0.05705800369722243 :   (b('l8b4_med') <= 569.4422302246094) ?  0.046604768737901585 :   0.0522973145097365 :   (b('dg0_v_vv') <= -0.5081462860107422) ?  (b('l8b11') <= 1543.2174072265625) ?  (b('ndvi_med') <= 3494.75) ?  0.04725588703797902 :   0.076616711109893 :   (b('l8b11') <= 1623.0953979492188) ?  0.0908129195009004 :   0.08382512572973289 :   (b('ndvi_med') <= 3494.75) ?  (b('dg0_v_vv') <= 3.317589282989502) ?  0.08579599720203163 :   0.10586616137058022 :   (b('l8b5') <= 3137.4453125) ?  0.06390529620802576 :   0.07749942758624917 :   (b('bulk') <= 137.5) ?  (b('moss') <= 5.807085037231445) ?  (b('crops') <= 92.60075759887695) ?  (b('ndvi_med') <= 2020.75) ?  (b('l8b4_med') <= 886.9541931152344) ?  0.022553267273611677 :   (b('ndvi') <= 2685.5) ?  (b('dg0_v_vv') <= -0.6396505832672119) ?  0.0708911435761016 :   (b('dg0_v_vv') <= 0.4623758792877197) ?  0.04518203847940677 :   0.059053339463056166 :   (b('dg0_v_vv') <= 1.9022581577301025) ?  (b('dg0_v_vv') <= 0.4167928695678711) ?  0.07459256596741559 :   0.08344132580522712 :   0.069441301754753 :   (b('l8b5_med') <= 3258.8597412109375) ?  (b('sand') <= 74.0) ?  (b('moss') <= 3.064139246940613) ?  (b('ndvi') <= 2796.5) ?  0.004757599440374481 :   -0.012470092935977953 :   (b('l8b5_med') <= 1801.5562133789062) ?  -0.07453648421755914 :   0.013215028853857379 :   (b('l8b5') <= 2750.31591796875) ?  (b('l8b4_med') <= 705.83837890625) ?  -0.0001987458343930687 :   0.0456648090520076 :   (b('l8b4') <= 550.787841796875) ?  0.0027778827265913567 :   -0.05591499983844633 :   (b('l8b4') <= 578.3419189453125) ?  (b('l8b5') <= 2725.8026123046875) ?  (b('l8b11') <= 1086.1966857910156) ?  -0.08249959361669648 :   -0.09356169122196756 :   (b('l8b5') <= 4566.603515625) ?  -0.00875475619104117 :   0.027211403906836233 :   (b('l8b5') <= 2938.66845703125) ?  (b('sand') <= 61.5) ?  0.06722636140815419 :   0.02422260351864006 :   (b('l8b11') <= 1401.7719116210938) ?  0.03213736166897763 :   -0.006455006066050926 :   (b('l8b5_med') <= 2132.814453125) ?  (b('ndvi') <= 2680.5) ?  (b('l8b11') <= 1425.9469604492188) ?  (b('ndvi_med') <= 1531.25) ?  0.0015005157333092822 :   (b('dg0_v_vv') <= 2.259322166442871) ?  0.06156990688864461 :   0.08003929251841152 :   (b('dg0_v_vv') <= -1.1733903884887695) ?  (b('dg0_v_vv') <= -2.701472282409668) ?  0.08418022005982062 :   0.07858954613043129 :   0.0931977286749181 :   (b('ndvi_med') <= 1531.25) ?  0.015984186127671718 :   -0.024632456442721218 :   (b('l8b5') <= 1728.9381713867188) ?  0.054638886760115635 :   (b('bulk') <= 114.0) ?  (b('dg0_v_vv') <= -0.23485946655273438) ?  0.08002897537508452 :   0.08809255705535235 :   (b('ndvi') <= 1428.5) ?  (b('dg0_v_vv') <= -0.3608245849609375) ?  0.09468588257778132 :   0.09111947149341337 :   (b('bulk') <= 122.0) ?  0.10868925328740886 :   0.10601952412241894 :   (b('dg0_v_vv') <= 0.8503141403198242) ?  (b('ndvi') <= 3310.0) ?  (b('l8b4') <= 537.6180114746094) ?  (b('l8b11_med') <= 1417.859375) ?  (b('dg0_v_vv') <= -0.043032169342041016) ?  (b('l8b5_med') <= 2962.1964111328125) ?  -0.01037294581943052 :   0.010342701811560542 :   (b('l8b11_med') <= 1307.5980834960938) ?  0.11059572546164745 :   0.03955391901374992 :   (b('dg0_v_vv') <= -0.3908414840698242) ?  (b('l8b11') <= 1033.6028747558594) ?  0.1130226318494239 :   0.1085326320284764 :   0.061853568698095385 :   (b('l8b5') <= 2448.494384765625) ?  (b('moss') <= 7.166860342025757) ?  (b('grass') <= 70.44684600830078) ?  0.014395650108359833 :   0.1852608677024775 :   (b('sand') <= 53.0) ?  -0.015980379080526987 :   0.01449030267219548 :   (b('ndvi_med') <= 2623.75) ?  (b('l8b11') <= 1553.0513916015625) ?  -0.02480700136265266 :   -0.07834297119414448 :   (b('l8b5') <= 2821.8489990234375) ?  -0.023591020004223792 :   -0.008363380571123142 :   (b('l8b5') <= 2286.073486328125) ?  (b('l8b4') <= 688.0476379394531) ?  (b('sand') <= 60.5) ?  -0.05947377679626459 :   0.0013529757758744543 :   (b('dg0_v_vv') <= 0.06332063674926758) ?  (b('sand') <= 44.0) ?  0.053708875619485064 :   0.04402771241728305 :   (b('l8b4') <= 756.1085205078125) ?  0.12328661939813924 :   0.08561423224133086 :   (b('ndvi') <= 3346.0) ?  (b('dg0_v_vv') <= 0.27484941482543945) ?  (b('dg0_v_vv') <= -1.8104805946350098) ?  0.004133242274324507 :   0.04527989129753914 :   (b('ndvi_med') <= 3342.25) ?  9.07648527267435e-05 :   -0.010302835483715633 :   (b('ndvi') <= 3415.5) ?  (b('sand') <= 38.5) ?  -0.019709567885217258 :   -0.04498916974591964 :   (b('sand') <= 70.5) ?  -0.003795928848724325 :   -0.031128861180215423 :   (b('l8b11_med') <= 2198.6363525390625) ?  (b('l8b11_med') <= 1251.4487915039062) ?  (b('l8b4') <= 737.9379577636719) ?  (b('l8b5') <= 2393.375244140625) ?  -0.03167873633970046 :   (b('l8b5') <= 2544.821533203125) ?  -0.08476291538500084 :   -0.07152368274846746 :   (b('dg0_v_vv') <= 0.9512579441070557) ?  -0.04074655370408936 :   (b('ndvi') <= 3467.0) ?  -0.036781781967835714 :   -0.04010056482812478 :   (b('moss') <= 13.163265228271484) ?  (b('l8b5_med') <= 3157.514404296875) ?  (b('l8b5_med') <= 2736.3499755859375) ?  0.02088933414660703 :   0.000801642594292648 :   (b('dg0_v_vv') <= 1.69757080078125) ?  -0.015386488621040648 :   0.14372521322292306 :   (b('l8b11') <= 1325.6389770507812) ?  (b('l8b11') <= 1225.7869262695312) ?  -0.019405770482419808 :   0.04102152651206692 :   (b('moss') <= 24.614730834960938) ?  -0.031239777670169187 :   0.031067970494824137 :   (b('l8b4') <= 794.9459533691406) ?  0.23219241319122108 :   0.10741903905372 :   (b('l8b5_med') <= 2499.1395263671875) ?  (b('bulk') <= 141.5) ?  (b('l8b5_med') <= 2201.04638671875) ?  (b('ndvi_med') <= 2457.0) ?  (b('l8b5') <= 2187.8914794921875) ?  (b('dg0_v_vv') <= 0.16370820999145508) ?  (b('grass') <= 76.36701965332031) ?  -0.024317770217109617 :   0.027331259604102598 :   (b('dg0_v_vv') <= 2.0542736053466797) ?  0.021397448910730532 :   -0.00048627315403433163 :   (b('l8b11_med') <= 1884.3165283203125) ?  0.0653747933940097 :   0.05942214320868974 :   (b('l8b11') <= 1510.1045532226562) ?  -0.08156693163884687 :   -0.03642678207543529 :   (b('l8b11') <= 1565.51318359375) ?  (b('l8b4') <= 694.8933715820312) ?  (b('l8b11_med') <= 1771.86474609375) ?  (b('l8b4') <= 440.50563049316406) ?  0.06222525271207768 :   0.10386265722199364 :   (b('l8b11') <= 1193.1138916015625) ?  -0.011204546686447707 :   0.06576195665528757 :   (b('ndvi_med') <= 1364.25) ?  0.1674582607809068 :   (b('l8b5_med') <= 2251.0975341796875) ?  0.08491734044488738 :   0.12327812826635921 :   (b('dg0_v_vv') <= 2.8270649909973145) ?  (b('l8b11') <= 1655.759033203125) ?  (b('l8b5') <= 2648.31591796875) ?  -0.02751356182908805 :   0.03597216215681397 :   (b('dg0_v_vv') <= 1.0700325965881348) ?  0.07937369434841118 :   0.016442132687911948 :   0.1515532570585953 :   (b('l8b4') <= 652.2040710449219) ?  (b('dg0_v_vv') <= 0.30109214782714844) ?  (b('dg0_v_vv') <= 0.2459392547607422) ?  (b('l8b4') <= 428.8229217529297) ?  (b('l8b5_med') <= 2269.1644287109375) ?  0.08690779701564226 :   0.026542569282424367 :   (b('l8b11') <= 1136.5359497070312) ?  -0.016563667882729956 :   0.023264383647874497 :   (b('moss') <= 21.31038761138916) ?  -0.17767753406625042 :   -0.04388764701285941 :   (b('grass') <= 38.52004623413086) ?  (b('dg0_v_vv') <= 4.719754219055176) ?  (b('dg0_v_vv') <= 0.7358322143554688) ?  -0.039408764301469945 :   -0.003984197186246058 :   -0.09149470352232661 :   (b('ndvi') <= 2403.5) ?  (b('moss') <= 13.651904582977295) ?  -0.033579166612491516 :   0.032354220898340266 :   (b('l8b5') <= 3229.7000732421875) ?  0.07060457401227663 :   -0.009778047372171014 :   (b('bulk') <= 158.5) ?  (b('dg0_v_vv') <= 1.546877384185791) ?  (b('l8b11') <= 1684.354248046875) ?  (b('ndvi') <= 3667.0) ?  -0.012525715586824788 :   0.032065393190297456 :   (b('l8b4') <= 788.7470703125) ?  0.057857661122721726 :   0.006241746683612861 :   (b('dg0_v_vv') <= 1.6566133499145508) ?  (b('sand') <= 31.5) ?  0.14613345613560963 :   0.10461415308725174 :   (b('l8b11_med') <= 2033.97314453125) ?  0.0348852811584477 :   -0.013708961163586848 :   (b('l8b11') <= 1716.4255981445312) ?  (b('dg0_v_vv') <= 0.7956254482269287) ?  (b('dg0_v_vv') <= 0.3980998992919922) ?  -0.034450879602653986 :   0.017166088570054614 :   (b('dg0_v_vv') <= 1.859506607055664) ?  -0.08326178876451167 :   0.014728097244575689 :   (b('moss') <= 17.25117826461792) ?  0.06673669842222385 :   0.0701491666477479 :   (b('grass') <= 36.312795639038086) ?  (b('l8b11_med') <= 2739.7578125) ?  (b('ndvi_med') <= 2141.0) ?  (b('l8b5') <= 1639.3827514648438) ?  (b('ndvi') <= 2616.5) ?  (b('sand') <= 49.5) ?  -0.0034333359270907306 :   0.07631687764193831 :   (b('ndvi') <= 2986.0) ?  -0.072561065311433 :   -0.019763684645858065 :   (b('l8b11') <= 1273.9500732421875) ?  (b('dg0_v_vv') <= 4.309821367263794) ?  0.07543035627766917 :   -0.06269629637283897 :   (b('l8b4') <= 1078.3367919921875) ?  0.008083041524465512 :   0.05418074208032936 :   (b('l8b11_med') <= 1966.1676025390625) ?  (b('l8b11') <= 1118.8828125) ?  (b('grass') <= 0.5440000146627426) ?  0.09249927388205587 :   0.02955428965763155 :   (b('l8b5') <= 1895.51123046875) ?  0.07628502506280774 :   0.010983730409221487 :   (b('l8b11') <= 1700.2625122070312) ?  (b('dg0_v_vv') <= -1.8253049850463867) ?  0.03257801272744246 :   -0.01584251120045054 :   (b('dg0_v_vv') <= -0.13358068466186523) ?  0.07524414561738246 :   0.10064928767269987 :   (b('sand') <= 52.5) ?  (b('dg0_v_vv') <= -0.11966085433959961) ?  (b('ndvi') <= 3964.0) ?  (b('l8b4_med') <= 1803.0075073242188) ?  -0.02925562754210459 :   -0.06451453928909805 :   (b('ndvi') <= 4217.5) ?  -0.02705377640909469 :   -0.033847570627580786 :   (b('l8b4') <= 689.1914978027344) ?  (b('dg0_v_vv') <= 2.3180294036865234) ?  -0.054602194154452124 :   -0.08711978962207184 :   (b('l8b11') <= 1494.0662841796875) ?  -0.007494960614412059 :   -0.040647580651585816 :   (b('l8b4') <= 864.2594604492188) ?  (b('ndvi') <= 1568.5) ?  (b('dg0_v_vv') <= 3.4954562187194824) ?  0.03684747262602464 :   0.04822420263927554 :   (b('l8b11') <= 1223.7142944335938) ?  0.043322692433013926 :   0.021238509180054203 :   (b('l8b5') <= 2818.8438720703125) ?  (b('dg0_v_vv') <= 3.051032066345215) ?  0.013568424345372273 :   -0.008499116440989635 :   (b('l8b11') <= 1564.655029296875) ?  -0.03519904603475993 :   -0.03717077453962547 :   (b('dg0_v_vv') <= 3.0359063148498535) ?  (b('ndvi') <= 4540.5) ?  (b('ndvi') <= 4489.5) ?  (b('ndvi') <= 4472.5) ?  (b('l8b11') <= 911.2364807128906) ?  -0.016729368043064782 :   0.02681382234604059 :   (b('l8b4_med') <= 815.4116516113281) ?  0.1553774708873943 :   0.13452488017797737 :   (b('grass') <= 62.437679290771484) ?  -0.02832879299898508 :   (b('dg0_v_vv') <= 0.9631190299987793) ?  -0.06983030306752944 :   -0.06154201651906652 :   (b('l8b4_med') <= 1461.6143188476562) ?  (b('dg0_v_vv') <= 0.27183961868286133) ?  (b('bulk') <= 149.5) ?  -0.014803402420410202 :   0.062089179045845135 :   (b('l8b11_med') <= 1589.8713989257812) ?  0.09917466585786668 :   0.046610546139852854 :   (b('grass') <= 69.39340209960938) ?  (b('l8b11_med') <= 2259.6182861328125) ?  0.1818664232014026 :   0.15355745348720762 :   0.22481602948018695 :   (b('moss') <= 0.010067113675177097) ?  (b('bulk') <= 149.5) ?  (b('l8b5_med') <= 2734.7528076171875) ?  (b('l8b11') <= 929.6260986328125) ?  0.1685367184359493 :   0.16994058181182897 :   (b('dg0_v_vv') <= 3.711705446243286) ?  0.15416973408645385 :   0.15607784788589837 :   (b('dg0_v_vv') <= 3.753917932510376) ?  0.06385422340906405 :   (b('bulk') <= 152.0) ?  0.11855993297104339 :   0.09336216032565964 :   (b('bulk') <= 151.0) ?  (b('l8b5') <= 2826.4471435546875) ?  (b('l8b4') <= 949.9939270019531) ?  -0.01446027257266328 :   0.04729206165888675 :   (b('bulk') <= 147.5) ?  0.07715846076596176 :   -0.007179511702884722 :   (b('bulk') <= 156.0) ?  (b('moss') <= 6.652481317520142) ?  0.13489029525178842 :   0.09236307521162873 :   (b('l8b5') <= 2992.9625244140625) ?  0.018959885551030564 :   0.061741816076708256 :   (b('l8b11_med') <= 2628.4404296875) ?  (b('dg0_v_vv') <= 0.7229645252227783) ?  (b('crops') <= 38.36996650695801) ?  (b('l8b11_med') <= 2014.972412109375) ?  (b('l8b11_med') <= 1640.2098388671875) ?  (b('l8b11') <= 2138.9461669921875) ?  (b('grass') <= 71.33245849609375) ?  (b('ndvi') <= 1222.0) ?  (b('l8b5') <= 1642.7349243164062) ?  0.026590402454203948 :   -0.03779435087560088 :   (b('grass') <= 25.737481117248535) ?  -0.0018940836591452173 :   -0.02128265384987569 :   (b('l8b4') <= 976.2763977050781) ?  (b('ndvi') <= 2115.5) ?  0.021503990734692703 :   -0.014618566913601498 :   (b('dg0_v_vv') <= -0.24839544296264648) ?  0.017359721227872653 :   0.08235228076705639 :   (b('crops') <= 29.038859367370605) ?  (b('l8b11') <= 2364.8590087890625) ?  (b('sand') <= 30.5) ?  0.04064504860889594 :   0.005668514084153493 :   (b('moss') <= 2.013333320617676) ?  -0.07424733326245228 :   -0.019832450808105192 :   (b('dg0_v_vv') <= -0.36136627197265625) ?  0.0951107820207378 :   (b('dg0_v_vv') <= 0.2501964569091797) ?  0.13180830576365204 :   0.10467866236243467 :   (b('bulk') <= 158.0) ?  (b('l8b4_med') <= 1080.6199951171875) ?  (b('l8b4') <= 1000.9952697753906) ?  (b('l8b5') <= 1891.9058227539062) ?  0.019663131412931363 :   -0.01973430762936208 :   (b('crops') <= 33.12485313415527) ?  -0.028368644719832282 :   0.032095692852638115 :   (b('l8b4') <= 1057.55712890625) ?  (b('dg0_v_vv') <= -1.4211974143981934) ?  -0.05047404044384877 :   0.050618346369928034 :   (b('l8b5') <= 2809.052734375) ?  -0.01940367857521271 :   -0.0016883588274831354 :   (b('l8b5_med') <= 2783.7923583984375) ?  (b('dg0_v_vv') <= -1.8976106643676758) ?  0.03144694938910816 :   (b('l8b11') <= 2297.389404296875) ?  -0.04270002524784892 :   0.011297276607762155 :   (b('l8b4') <= 1760.9891967773438) ?  (b('dg0_v_vv') <= -1.187917709350586) ?  -0.007012793302014722 :   0.011773855469101847 :   (b('l8b4') <= 1837.8548583984375) ?  -0.021428125620831615 :   -0.029428130657424098 :   (b('l8b4') <= 1186.0213623046875) ?  (b('sand') <= 43.5) ?  (b('dg0_v_vv') <= 0.37277650833129883) ?  (b('ndvi_med') <= 1679.5) ?  (b('moss') <= 25.70225238800049) ?  0.08249377736101701 :   -0.0024831590413150494 :   (b('l8b5') <= 2502.4766845703125) ?  0.019106989396547532 :   -0.008930793406168997 :   (b('l8b4') <= 1003.7868957519531) ?  (b('dg0_v_vv') <= 0.5200939178466797) ?  0.11970446782732508 :   0.19667026435746932 :   (b('l8b11_med') <= 2023.069091796875) ?  0.1460479915656515 :   0.03478874890441657 :   (b('l8b5_med') <= 2113.6082763671875) ?  (b('ndvi_med') <= 1283.0) ?  (b('l8b5_med') <= 1983.588134765625) ?  0.04793384895406814 :   -0.010333948690873007 :   (b('l8b5') <= 2327.3695068359375) ?  -0.057468503437579505 :   -0.0031205133674515515 :   (b('grass') <= 41.695701599121094) ?  (b('l8b4') <= 1009.7288208007812) ?  -0.041458397493117326 :   -0.08800944966050858 :   (b('bulk') <= 144.5) ?  -0.02556064645016762 :   0.03015067564230347 :   (b('l8b5') <= 3520.865966796875) ?  (b('l8b4_med') <= 1123.7163696289062) ?  (b('bulk') <= 154.5) ?  (b('l8b4_med') <= 1055.3640747070312) ?  -0.017300400433940533 :   0.021776151031415118 :   0.16860949838762196 :   (b('grass') <= 41.722944259643555) ?  (b('ndvi') <= 2295.5) ?  -0.02816171755340175 :   0.0538540992612107 :   (b('ndvi_med') <= 2245.0) ?  -0.009177538294174286 :   -0.039469645649489966 :   (b('grass') <= 41.695701599121094) ?  (b('ndvi') <= 2843.0) ?  (b('l8b11') <= 3173.1776123046875) ?  -6.143615103374606e-06 :   -0.03386319900439026 :   (b('l8b11_med') <= 2365.364013671875) ?  0.016531279364326107 :   0.037226666501553596 :   (b('ndvi') <= 3275.5) ?  (b('l8b11_med') <= 2272.5635986328125) ?  0.16520252679640363 :   0.17804245107225583 :   (b('dg0_v_vv') <= -0.11464118957519531) ?  0.11620253180319379 :   0.10403295908684732 :   (b('dg0_v_vv') <= -1.6519694328308105) ?  (b('bulk') <= 131.5) ?  (b('moss') <= 0.02757793851196766) ?  (b('l8b4') <= 1108.9265747070312) ?  (b('crops') <= 89.40567779541016) ?  (b('dg0_v_vv') <= -5.437074661254883) ?  0.06625549466798586 :   0.04834258982116885 :   (b('dg0_v_vv') <= -3.5215916633605957) ?  0.08158977515546034 :   0.10699278545188001 :   (b('l8b11') <= 2321.4666748046875) ?  (b('l8b5_med') <= 2950.891845703125) ?  0.06515631505937347 :   0.0039371056003115095 :   (b('l8b5_med') <= 3177.6944580078125) ?  0.0790998585697047 :   0.06366879436101573 :   (b('bulk') <= 120.0) ?  0.08140038432573096 :   (b('l8b5') <= 2199.164306640625) ?  (b('ndvi') <= 4761.5) ?  0.035360589456258765 :   0.007364332558329453 :   (b('l8b5') <= 2772.3079833984375) ?  -0.01845042168582692 :   0.004062317875870613 :   (b('sand') <= 48.0) ?  (b('sand') <= 41.5) ?  (b('sand') <= 27.0) ?  (b('moss') <= 0.3046875) ?  0.09082272137817937 :   0.005973508716824989 :   (b('crops') <= 95.00122833251953) ?  -0.02515231488380908 :   0.0018960843769436696 :   (b('dg0_v_vv') <= -2.2878427505493164) ?  (b('ndvi_med') <= 1786.5) ?  0.012335853325616312 :   -0.015795432561269265 :   (b('l8b4') <= 1618.0145263671875) ?  0.05739536542179451 :   0.014627621261460869 :   (b('l8b5') <= 3449.048095703125) ?  (b('l8b4') <= 2272.7607421875) ?  (b('moss') <= 5.28290581703186) ?  -0.016608566303997593 :   -0.056032584721082995 :   0.0734290503360791 :   (b('dg0_v_vv') <= -1.9822020530700684) ?  (b('dg0_v_vv') <= -4.388134002685547) ?  -0.05125423814904965 :   -0.04002480849294699 :   (b('crops') <= 73.83032989501953) ?  -0.05967617963652211 :   -0.05607837776832389 :   (b('sand') <= 20.5) ?  (b('l8b11') <= 2104.6336669921875) ?  (b('l8b4') <= 980.2715759277344) ?  (b('l8b5_med') <= 2847.531494140625) ?  0.03548048876102711 :   (b('l8b4') <= 915.3010559082031) ?  0.05744243435371538 :   0.07480700618625974 :   (b('dg0_v_vv') <= -0.29067230224609375) ?  (b('dg0_v_vv') <= -0.5855059623718262) ?  0.03138801784158813 :   -0.017148160141418567 :   (b('grass') <= 33.36697959899902) ?  0.04237919755212749 :   0.04344082822629075 :   (b('ndvi') <= 2992.0) ?  (b('l8b11_med') <= 1156.7976684570312) ?  0.10576603314223287 :   (b('ndvi') <= 1396.5) ?  0.08310788612627273 :   0.07425552684737456 :   0.14464712172459288 :   (b('l8b5_med') <= 2996.1737060546875) ?  (b('l8b4') <= 1891.0691528320312) ?  (b('l8b5_med') <= 2811.5986328125) ?  (b('crops') <= 77.88256454467773) ?  0.0014640612246887693 :   0.041639746236952165 :   (b('l8b4') <= 1699.0471801757812) ?  -0.004992133303658074 :   -0.028991600157758684 :   (b('ndvi') <= 2686.5) ?  (b('ndvi') <= 1365.5) ?  0.08471128621672319 :   -0.004252505227669916 :   (b('ndvi') <= 3460.0) ?  0.13938177502702334 :   0.04633384761625581 :   (b('l8b5') <= 3357.5439453125) ?  (b('l8b11') <= 2955.5087890625) ?  (b('l8b4') <= 2351.68359375) ?  0.025375219648509483 :   -0.06589074275584582 :   (b('ndvi') <= 2305.0) ?  -0.02239105558638555 :   0.035637135345963335 :   (b('l8b5') <= 3381.640869140625) ?  (b('l8b4_med') <= 1050.762451171875) ?  -0.08444699467190829 :   -0.024373330014846283 :   (b('l8b4') <= 1422.0134887695312) ?  0.016291738812617782 :   -0.014835763263073088 :   (b('sand') <= 24.5) ?  (b('l8b11') <= 2481.8310546875) ?  (b('ndvi') <= 4355.5) ?  (b('l8b4') <= 968.0950622558594) ?  (b('ndvi') <= 2759.5) ?  (b('ndvi') <= 1538.5) ?  0.09119544370240415 :   (b('l8b4_med') <= 952.4891967773438) ?  -0.04751207785964105 :   -0.11824469802352584 :   (b('l8b4') <= 852.3948669433594) ?  0.10338548894330635 :   (b('moss') <= 5.450023412704468) ?  0.06005162948908399 :   0.06786037974717993 :   (b('l8b5_med') <= 2939.2430419921875) ?  (b('ndvi') <= 1122.0) ?  0.026959559529385974 :   (b('sand') <= 16.5) ?  0.06139937439829114 :   0.08065431364704698 :   (b('dg0_v_vv') <= 0.7746751308441162) ?  (b('l8b4_med') <= 1057.4125671386719) ?  -0.01788208984725298 :   -0.06162183473666505 :   (b('l8b5') <= 2643.174072265625) ?  0.05767753884852625 :   0.011046028203891018 :   (b('sand') <= 22.0) ?  -0.07043425503230727 :   (b('l8b11') <= 2036.3236694335938) ?  -0.0634365597175914 :   -0.06529939278318134 :   (b('l8b11_med') <= 1257.730224609375) ?  0.1525424578708258 :   (b('l8b5_med') <= 3085.2413330078125) ?  (b('ndvi_med') <= 3136.5) ?  (b('l8b4') <= 1740.3795166015625) ?  0.11656533600549523 :   0.12257412719677568 :   0.10822440764445287 :   (b('ndvi_med') <= 3351.25) ?  (b('l8b5_med') <= 3151.007568359375) ?  0.08679242229336404 :   0.09631096340900941 :   0.10804370018367537 :   (b('sand') <= 52.5) ?  (b('dg0_v_vv') <= 2.7136411666870117) ?  (b('l8b11_med') <= 2467.304443359375) ?  (b('ndvi') <= 4896.5) ?  (b('l8b5_med') <= 3115.2471923828125) ?  (b('l8b5') <= 1761.7288208007812) ?  0.043869127696252154 :   -0.00331318164791173 :   (b('bulk') <= 160.5) ?  0.01863056269816863 :   0.0708119602262217 :   (b('l8b4_med') <= 982.061279296875) ?  (b('l8b4') <= 1416.7532348632812) ?  0.030769113767636996 :   0.07394562246935116 :   0.2574903985221505 :   (b('l8b11') <= 2248.0804443359375) ?  (b('l8b4') <= 1058.3867797851562) ?  (b('moss') <= 9.0) ?  0.10023741813663267 :   0.14043050707009536 :   (b('l8b4') <= 1401.9832153320312) ?  0.04737797568462759 :   0.1324299396248539 :   (b('l8b11') <= 2828.923095703125) ?  (b('l8b4_med') <= 1245.3859252929688) ?  -0.10839138448755373 :   0.011729755660825571 :   (b('ndvi_med') <= 1542.25) ?  0.021701161663538214 :   0.08853820000757942 :   (b('l8b4_med') <= 1157.4581298828125) ?  (b('ndvi') <= 2480.0) ?  (b('l8b4') <= 1140.0979614257812) ?  (b('l8b5') <= 2244.0550537109375) ?  0.034710227111532606 :   -0.01953628102447421 :   (b('grass') <= 3.1480519771575928) ?  0.023551816237947443 :   -0.034378410117571326 :   (b('l8b4') <= 1203.6040649414062) ?  (b('l8b5_med') <= 2331.9173583984375) ?  0.05244775344721375 :   0.008078525924326658 :   (b('l8b5_med') <= 2735.39697265625) ?  0.11000447120481677 :   0.052644021781050085 :   (b('moss') <= 29.549626350402832) ?  (b('dg0_v_vv') <= 5.027825355529785) ?  (b('l8b11') <= 2202.3494873046875) ?  0.06210512780982707 :   0.022546554107901615 :   (b('grass') <= 87.12417221069336) ?  0.14762711073937346 :   0.10823011149063788 :   (b('l8b5') <= 2046.94580078125) ?  (b('bulk') <= 140.5) ?  -0.0666025062143055 :   -0.0687784277828507 :   (b('moss') <= 30.33520221710205) ?  -0.011524675492742761 :   0.040017895267830755 :   (b('l8b5') <= 3737.7353515625) ?  (b('l8b11') <= 1962.759033203125) ?  (b('l8b11') <= 1962.1190185546875) ?  (b('ndvi_med') <= 1514.5) ?  (b('l8b11') <= 1777.2052001953125) ?  0.15806260761179716 :   0.032939530804171606 :   (b('moss') <= 17.525090217590332) ?  0.0124049351501706 :   -0.03547224281741065 :   (b('dg0_v_vv') <= 1.1729941368103027) ?  0.15147749849912634 :   0.15327710274731723 :   (b('l8b4') <= 1342.6871337890625) ?  (b('ndvi_med') <= 1471.5) ?  (b('grass') <= 53.20034217834473) ?  -0.061208820319046654 :   0.02910324520071357 :   (b('l8b5') <= 1947.7936401367188) ?  0.010554340401327387 :   -0.019374240256417638 :   (b('dg0_v_vv') <= 1.4067480564117432) ?  (b('l8b5') <= 1962.8308715820312) ?  0.169727936164156 :   -0.01745701550964564 :   (b('ndvi') <= 1929.0) ?  0.008290299986423522 :   0.04653093916461268 :   (b('l8b11') <= 1906.5609741210938) ?  0.015580560035628327 :   (b('ndvi') <= 2567.5) ?  (b('bulk') <= 149.0) ?  -0.07654116973399641 :   (b('dg0_v_vv') <= 3.5287649631500244) ?  -0.04539401008541755 :   -0.033456830947083765 :   (b('l8b5') <= 3957.1788330078125) ?  (b('dg0_v_vv') <= 2.162809371948242) ?  0.020240311999600316 :   -0.024736733934231498 :   (b('l8b4_med') <= 1555.0704345703125) ?  -0.044241011307551675 :   -0.028950928731356282 :   (b('grass') <= 1.900430142879486) ?  (b('dg0_v_vv') <= 3.5116987228393555) ?  (b('l8b11') <= 3917.4981689453125) ?  (b('l8b11') <= 2292.020263671875) ?  (b('ndvi') <= 1371.5) ?  (b('ndvi') <= 1271.0) ?  (b('ndvi') <= 1127.0) ?  (b('dg0_v_vv') <= 0.4293675422668457) ?  0.035369091589798285 :   0.018322548807671923 :   (b('dg0_v_vv') <= 0.525634765625) ?  0.044942447014424514 :   0.0386416123555055 :   (b('ndvi') <= 1354.0) ?  (b('dg0_v_vv') <= -1.3072333335876465) ?  0.008194451733215757 :   0.016890022868454375 :   0.04138423015984952 :   (b('l8b11') <= 1852.0209350585938) ?  0.06556809046748467 :   (b('grass') <= 0.27586206793785095) ?  (b('l8b11') <= 2072.1531982421875) ?  0.011705180153815609 :   -0.0014655783337329835 :   (b('l8b4_med') <= 1850.4474487304688) ?  -0.03374194483107676 :   -0.01063543930345227 :   (b('l8b5') <= 4259.192138671875) ?  (b('l8b4') <= 2452.17919921875) ?  (b('l8b11') <= 3210.026611328125) ?  (b('l8b4') <= 2113.3668212890625) ?  -0.004132419795051289 :   -0.014766278302986803 :   (b('dg0_v_vv') <= 2.2101023197174072) ?  -0.028882709308905605 :   -0.07357775323905158 :   (b('l8b4') <= 2501.752685546875) ?  (b('ndvi') <= 1040.0) ?  0.10364188907480817 :   0.0399242273436525 :   (b('dg0_v_vv') <= -0.6798315048217773) ?  0.014461044628554425 :   -0.005403813630956257 :   (b('ndvi') <= 2771.5) ?  0.08573493677757082 :   (b('l8b5_med') <= 3622.8421630859375) ?  (b('ndvi') <= 3929.0) ?  0.03868488177463206 :   0.029962511488091204 :   (b('dg0_v_vv') <= -0.6064085960388184) ?  0.03810401192320398 :   0.05669522167809882 :   (b('dg0_v_vv') <= 1.3916206359863281) ?  (b('ndvi') <= 927.5) ?  (b('dg0_v_vv') <= 0.04475212097167969) ?  (b('ndvi') <= 818.5) ?  (b('ndvi') <= 767.0) ?  -0.022389607562218902 :   -0.005803473451634844 :   (b('l8b5') <= 4371.70947265625) ?  -0.02267660102211679 :   -0.0322912118501678 :   (b('dg0_v_vv') <= 0.8298931121826172) ?  (b('l8b11') <= 4254.87255859375) ?  -0.01420903345019858 :   -0.002027968455231894 :   (b('l8b4') <= 3068.6898193359375) ?  -0.023312295654105197 :   -0.031006143080990603 :   (b('dg0_v_vv') <= -0.2454395294189453) ?  (b('l8b11_med') <= 2890.52392578125) ?  -0.03466736128067227 :   (b('crops') <= 49.389366149902344) ?  -0.049017106868794236 :   -0.04401143090453534 :   (b('dg0_v_vv') <= 0.24415159225463867) ?  (b('bulk') <= 156.0) ?  -0.031974597290437945 :   -0.03016133327534483 :   (b('ndvi_med') <= 1349.5) ?  -0.02820280236871811 :   -0.01462048293146806 :   (b('ndvi_med') <= 1103.0) ?  -0.05150674093998567 :   -0.04482020081640138 :   (b('l8b4') <= 1750.8133544921875) ?  (b('l8b5') <= 2003.7764892578125) ?  0.1384432139139563 :   (b('l8b4') <= 1023.1703186035156) ?  0.021239690593089 :   (b('l8b5') <= 2764.3023681640625) ?  (b('l8b4_med') <= 1656.4095458984375) ?  0.030799770462471093 :   (b('l8b4_med') <= 1850.4474487304688) ?  0.06491946306901114 :   0.05008834413902034 :   0.07461321800708809 :   (b('l8b5') <= 3711.4227294921875) ?  (b('dg0_v_vv') <= 10.80087661743164) ?  (b('bulk') <= 151.5) ?  -0.01669784741814684 :   -0.014606564442096265 :   -0.024358075558617548 :   (b('dg0_v_vv') <= 3.6534085273742676) ?  0.03121830157110872 :   0.029218297517992875 :   (b('bulk') <= 149.5) ?  (b('dg0_v_vv') <= 0.2692990303039551) ?  (b('l8b11_med') <= 2966.3846435546875) ?  (b('dg0_v_vv') <= -1.8903694152832031) ?  (b('l8b4') <= 1519.4933471679688) ?  (b('sand') <= 66.5) ?  0.023554346406625182 :   0.04434291500603327 :   (b('l8b11') <= 3315.7080078125) ?  (b('l8b5_med') <= 2791.6810302734375) ?  -0.07665064888472027 :   -0.035364523820982464 :   (b('l8b4') <= 1866.7098388671875) ?  0.0177249442125139 :   -0.014130738266721649 :   (b('l8b5') <= 2791.5035400390625) ?  (b('sand') <= 44.0) ?  (b('l8b11') <= 2781.741943359375) ?  -0.06959196117362902 :   -0.033063803847499504 :   (b('l8b11') <= 2863.0003662109375) ?  -0.008471368802523223 :   -0.04446105714444668 :   (b('l8b5') <= 3121.2481689453125) ?  (b('dg0_v_vv') <= -1.819751262664795) ?  0.08864809093161329 :   0.004089656229999563 :   (b('l8b11') <= 2233.510986328125) ?  0.09188501968253622 :   -0.01564097820451132 :   (b('ndvi') <= 2088.0) ?  (b('sand') <= 42.5) ?  (b('ndvi') <= 1418.0) ?  (b('dg0_v_vv') <= -0.7600011825561523) ?  -0.039097462526243895 :   0.001067514682319009 :   0.05223581609772854 :   (b('dg0_v_vv') <= -0.5569477081298828) ?  (b('l8b5') <= 3491.087158203125) ?  -0.020707839926263755 :   -0.04157467543066046 :   (b('dg0_v_vv') <= 0.06553411483764648) ?  -0.043330578855630845 :   -0.01853775469534496 :   (b('ndvi') <= 2297.0) ?  (b('l8b4_med') <= 1889.65869140625) ?  (b('ndvi_med') <= 1601.25) ?  0.01727227456279773 :   0.03534261802006331 :   -0.009998536202689007 :   (b('l8b5') <= 3133.279296875) ?  (b('dg0_v_vv') <= -0.9412326812744141) ?  -0.019551183639440098 :   -0.03882028606612039 :   (b('l8b4') <= 1749.9016723632812) ?  -0.015202581950224045 :   -0.004114754730345657 :   (b('ndvi_med') <= 1819.0) ?  (b('l8b5_med') <= 3248.876953125) ?  (b('bulk') <= 148.5) ?  (b('l8b4') <= 2318.5709228515625) ?  (b('ndvi') <= 2121.0) ?  -0.015832337640150476 :   0.004059316142787306 :   (b('ndvi') <= 1186.0) ?  0.09602046064305242 :   -5.585503736488945e-05 :   (b('l8b4_med') <= 1782.2893676757812) ?  (b('l8b11') <= 2833.626708984375) ?  0.053394549486864694 :   0.09504637397634451 :   (b('l8b4') <= 1750.488525390625) ?  -0.04237131123178288 :   0.02738177402753621 :   0.1736211869835132 :   (b('dg0_v_vv') <= 5.026772499084473) ?  (b('bulk') <= 145.5) ?  (b('l8b5') <= 2923.236572265625) ?  (b('l8b4') <= 1140.8831176757812) ?  0.03172619231834347 :   0.055162279552203736 :   (b('dg0_v_vv') <= 2.8396137952804565) ?  0.015295870367013548 :   0.0030307204966439905 :   (b('l8b4') <= 1536.0271606445312) ?  (b('dg0_v_vv') <= 0.8637871742248535) ?  0.11747789462130673 :   0.09531519545645911 :   (b('l8b4') <= 1969.4534301757812) ?  0.038256651199866015 :   0.09263576495733195 :   -0.07422399257195736 :   (b('bulk') <= 151.5) ?  (b('ndvi_med') <= 1789.5) ?  (b('dg0_v_vv') <= -0.06850528717041016) ?  (b('l8b5') <= 2309.8245849609375) ?  (b('dg0_v_vv') <= -0.8766603469848633) ?  (b('dg0_v_vv') <= -1.2730116844177246) ?  -0.0055992814758372995 :   0.01610155293328378 :   (b('l8b11') <= 2381.7786865234375) ?  -0.027896581731753297 :   0.019676570694133297 :   (b('crops') <= 46.23920249938965) ?  (b('dg0_v_vv') <= -1.0715680122375488) ?  -0.012933715362232456 :   -0.027238921896335935 :   (b('l8b5') <= 2548.6468505859375) ?  -0.00463749940507145 :   -0.030508267372180035 :   (b('l8b5_med') <= 2678.3040771484375) ?  (b('l8b5') <= 2291.4835205078125) ?  0.06752364231861682 :   (b('l8b11') <= 2937.369140625) ?  -0.03929425226416194 :   -0.08677172809736328 :   (b('l8b5_med') <= 3069.832275390625) ?  (b('ndvi') <= 1313.0) ?  -0.03536340494599356 :   -0.004887475435612441 :   (b('l8b4') <= 2350.5592041015625) ?  -0.02095142136927866 :   -0.00811899048484161 :   (b('l8b11') <= 2108.4381103515625) ?  (b('bulk') <= 150.5) ?  -0.09343402988117461 :   (b('dg0_v_vv') <= 0.481842041015625) ?  -0.05560935432587802 :   -0.0540362414758616 :   (b('ndvi_med') <= 2041.5) ?  (b('l8b5') <= 4000.20751953125) ?  (b('l8b11') <= 2218.7430419921875) ?  -0.017631682053720163 :   -0.03623961933244983 :   (b('dg0_v_vv') <= 0.9298076629638672) ?  -0.02856499147658479 :   0.017930811076734377 :   (b('ndvi') <= 1764.5) ?  (b('ndvi') <= 1561.0) ?  -0.029389463383137124 :   0.007848246818531333 :   (b('dg0_v_vv') <= -1.622171401977539) ?  -0.006955908106000446 :   -0.02989627348621901 :   (b('ndvi') <= 3026.5) ?  (b('dg0_v_vv') <= 0.7724108695983887) ?  (b('l8b4_med') <= 1781.6423950195312) ?  (b('dg0_v_vv') <= -0.542574405670166) ?  (b('bulk') <= 153.0) ?  -0.04445568876357462 :   -0.022940467319863135 :   (b('l8b5') <= 3294.6832275390625) ?  -0.020326494968318894 :   0.01930085194669827 :   (b('crops') <= 11.524752616882324) ?  (b('l8b5') <= 2588.2322998046875) ?  0.004856113303052837 :   -0.017330831111099367 :   (b('dg0_v_vv') <= -0.9090137481689453) ?  -0.009911275415945177 :   0.021913789918831507 :   (b('l8b4') <= 1922.0614624023438) ?  (b('l8b11') <= 3003.94580078125) ?  (b('ndvi') <= 1268.0) ?  -0.038004396531382115 :   0.013962540681204609 :   (b('l8b5') <= 2631.1744384765625) ?  0.09201182849952057 :   0.03695922294714572 :   (b('crops') <= 49.290465354919434) ?  (b('ndvi') <= 1777.5) ?  -0.012859448106570498 :   0.03075766213373818 :   (b('ndvi') <= 1237.0) ?  -0.09482459537121582 :   -0.03944572878486011 :   (b('l8b11') <= 2925.2049560546875) ?  (b('l8b11') <= 2482.563232421875) ?  (b('dg0_v_vv') <= -1.0938358306884766) ?  (b('sand') <= 45.0) ?  0.01570203674609022 :   0.012183696799206112 :   (b('l8b11_med') <= 3052.07763671875) ?  0.06567364732998558 :   0.04728529528732345 :   (b('dg0_v_vv') <= -0.2647976875305176) ?  0.09507896371642546 :   0.11595786889193554 :   (b('l8b5') <= 3505.9490966796875) ?  (b('ndvi') <= 4267.5) ?  (b('dg0_v_vv') <= 0.019097328186035156) ?  0.0009698393059221488 :   0.0025937416007491138 :   -0.012063427292877987 :   0.041704476573109606 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1320.8862915039062) ?  (b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 36.5) ?  (b('l8b11') <= 1127.6405639648438) ?  (b('ndvi') <= 3967.0) ?  (b('ndvi') <= 3815.5) ?  (b('l8b5') <= 2736.470458984375) ?  (b('l8b4') <= 580.3703308105469) ?  (b('ndvi') <= 3296.5) ?  0.01143829865604277 :   0.044075284701345564 :   (b('dg0_v_vv') <= -0.7316131591796875) ?  0.012822051580937655 :   -0.020466974719785626 :   (b('ndvi_med') <= 4280.75) ?  (b('ndvi') <= 3502.5) ?  0.06918113418661477 :   0.029493490989115693 :   (b('ndvi') <= 3523.5) ?  -0.03091827144934577 :   0.036727340855709 :   (b('dg0_v_vv') <= -0.8089427947998047) ?  (b('moss') <= 13.872385501861572) ?  (b('bulk') <= 125.0) ?  -0.005285727820837582 :   -0.02903262569277147 :   0.11771888558101937 :   (b('crops') <= 53.03295707702637) ?  (b('dg0_v_vv') <= 0.06531906127929688) ?  -0.03919729414953016 :   -0.07665131624776686 :   0.06293449073564783 :   (b('l8b5_med') <= 2399.0399169921875) ?  (b('ndvi') <= 4697.0) ?  (b('ndvi_med') <= 2906.5) ?  (b('dg0_v_vv') <= -0.5159258842468262) ?  -0.027329228287330698 :   -0.03053048309261347 :   (b('l8b5_med') <= 2163.919677734375) ?  0.0010595565851339722 :   -0.018295077947981478 :   0.056999432955660845 :   (b('l8b5') <= 2732.12060546875) ?  (b('dg0_v_vv') <= 1.350947380065918) ?  (b('dg0_v_vv') <= -0.3064250946044922) ?  0.09773274378779305 :   0.06720721469087451 :   0.1354703590425042 :   (b('ndvi_med') <= 3569.0) ?  (b('ndvi') <= 4186.5) ?  0.13825140936119376 :   0.0637073106162057 :   (b('l8b4') <= 429.8273468017578) ?  -0.0030212557870485817 :   0.04267396091640933 :   (b('l8b11_med') <= 1138.7337036132812) ?  (b('ndvi') <= 4906.0) ?  (b('sand') <= 31.0) ?  (b('dg0_v_vv') <= -0.21158313751220703) ?  (b('l8b5_med') <= 2163.919677734375) ?  0.04848193238218395 :   0.004548177253056469 :   (b('l8b4') <= 1049.2434692382812) ?  0.00141322187385897 :   0.07140266317492266 :   (b('ndvi') <= 2913.5) ?  (b('ndvi') <= 2803.5) ?  -0.03792114674214082 :   -0.06175975357133974 :   (b('l8b5_med') <= 2724.859619140625) ?  0.02087851201831441 :   -0.013183351601745054 :   (b('dg0_v_vv') <= 1.1282014846801758) ?  (b('dg0_v_vv') <= 0.936964750289917) ?  (b('l8b4_med') <= 577.5997924804688) ?  0.07016818795336438 :   0.1247435023531544 :   0.16544966792021498 :   (b('l8b11') <= 1236.1396484375) ?  0.042753150169997534 :   -0.04391953457056047 :   (b('l8b4') <= 534.4433898925781) ?  (b('l8b4') <= 520.1318054199219) ?  (b('l8b5') <= 3476.37353515625) ?  (b('bulk') <= 129.0) ?  0.06174026579236788 :   0.06583744151595258 :   (b('dg0_v_vv') <= -0.6237044334411621) ?  -0.023522667172735684 :   -0.050716060794802155 :   (b('l8b5') <= 3127.17626953125) ?  -0.10717502811713095 :   -0.09168487272298592 :   (b('ndvi') <= 3391.0) ?  (b('l8b5') <= 4118.5904541015625) ?  (b('sand') <= 34.0) ?  0.05940389503181687 :   -0.058375694903109815 :   -0.07204724298344123 :   (b('l8b4') <= 624.9400329589844) ?  (b('l8b11_med') <= 1211.5642700195312) ?  0.07789379059828865 :   0.01377841551519717 :   (b('ndvi') <= 4033.5) ?  0.032811861266463224 :   -0.0016183135343240025 :   (b('l8b5_med') <= 2940.280029296875) ?  (b('l8b11') <= 973.0213623046875) ?  (b('l8b4') <= 350.34710693359375) ?  (b('ndvi') <= 3525.5) ?  (b('ndvi') <= 2821.5) ?  0.03669571820050496 :   (b('l8b11_med') <= 1208.174560546875) ?  -0.0028930264162924935 :   0.0008148338078043244 :   (b('l8b4_med') <= 598.2109985351562) ?  (b('dg0_v_vv') <= -0.5451240539550781) ?  -0.039459884374639556 :   -0.06789490842460333 :   -0.11925839595255096 :   (b('l8b4') <= 403.7890930175781) ?  (b('l8b4_med') <= 595.4280090332031) ?  (b('dg0_v_vv') <= 0.6916754245758057) ?  -0.04830791856172731 :   -0.030295817257803725 :   (b('l8b4') <= 391.9961700439453) ?  0.017197054810381935 :   0.07130293313190132 :   (b('dg0_v_vv') <= -0.20435333251953125) ?  (b('l8b4_med') <= 611.1411743164062) ?  -0.009206101110804395 :   -0.07065499828267958 :   (b('dg0_v_vv') <= 2.626246452331543) ?  -0.03469794593457527 :   0.025764018253134804 :   (b('l8b11_med') <= 948.916259765625) ?  (b('bulk') <= 157.5) ?  0.010705571194492336 :   (b('l8b4') <= 721.4338989257812) ?  0.15265329898734015 :   0.125207550655868 :   (b('l8b4') <= 616.8479309082031) ?  (b('grass') <= 40.928409576416016) ?  (b('l8b11') <= 1084.552001953125) ?  0.004202442407610552 :   0.06056598502139271 :   (b('l8b4_med') <= 573.5060119628906) ?  -0.01641783444775314 :   -0.04158430154076416 :   (b('ndvi') <= 1596.0) ?  (b('ndvi') <= 1557.0) ?  -0.006049561833099657 :   0.20841999548854145 :   (b('l8b5_med') <= 2569.871337890625) ?  -0.02973806198214742 :   -0.003711496250311637 :   (b('l8b5') <= 2829.8966064453125) ?  (b('moss') <= 0.6001937836408615) ?  (b('l8b5') <= 2310.955810546875) ?  (b('dg0_v_vv') <= 0.9857649803161621) ?  (b('ndvi') <= 3227.0) ?  0.006055846629725793 :   -0.019404158319014354 :   (b('l8b4') <= 986.5016174316406) ?  -0.059645896492844686 :   -0.04407158896405569 :   (b('bulk') <= 113.0) ?  (b('l8b11_med') <= 1142.9754028320312) ?  0.04800348957624828 :   0.044237930008020676 :   (b('dg0_v_vv') <= 2.3780922889709473) ?  0.024850478245247047 :   0.011298096294478569 :   (b('l8b5_med') <= 3025.3770751953125) ?  (b('ndvi') <= 4169.0) ?  (b('dg0_v_vv') <= 0.2003326416015625) ?  0.15486678918871646 :   0.10418654698185843 :   (b('dg0_v_vv') <= -0.5180959701538086) ?  0.08233265457391523 :   -0.01846465274196407 :   (b('ndvi') <= 4536.0) ?  (b('ndvi_med') <= 4438.0) ?  0.04346637344225502 :   0.173182310316016 :   (b('ndvi') <= 4567.5) ?  0.0017156551538697329 :   -0.037509930743362646 :   (b('moss') <= 4.990872859954834) ?  (b('l8b4') <= 549.9536743164062) ?  (b('ndvi') <= 2850.0) ?  (b('dg0_v_vv') <= 1.7309260368347168) ?  0.06079778626250416 :   0.08391098496193759 :   (b('sand') <= 47.0) ?  -0.022077448784393185 :   0.013777386939013519 :   (b('ndvi') <= 3224.5) ?  (b('grass') <= 13.976126208901405) ?  -0.06986899932343603 :   -0.08128999878012391 :   (b('moss') <= 0.2564399056136608) ?  0.029626164273854116 :   -0.013948393378168113 :   (b('l8b11_med') <= 972.2954711914062) ?  (b('ndvi') <= 2764.5) ?  (b('dg0_v_vv') <= 0.2891559600830078) ?  -0.030259989341777482 :   -0.047429018518124255 :   (b('l8b4') <= 732.9202880859375) ?  0.012120681434554416 :   0.03176812363208557 :   (b('bulk') <= 119.0) ?  (b('dg0_v_vv') <= -0.2522101402282715) ?  0.08470756009738911 :   0.11259591935476876 :   (b('l8b5') <= 3613.8023681640625) ?  0.05969847846613652 :   0.02772122041914562 :   (b('crops') <= 19.45354127883911) ?  (b('l8b5') <= 2205.009521484375) ?  (b('l8b5') <= 2085.4517822265625) ?  (b('l8b4_med') <= 634.6933898925781) ?  (b('l8b5') <= 1937.2792358398438) ?  (b('l8b11_med') <= 1230.8153686523438) ?  (b('l8b11_med') <= 1180.7840576171875) ?  0.044032620013187695 :   -0.050946339732799245 :   0.14589842570956701 :   (b('dg0_v_vv') <= 0.4570136070251465) ?  (b('ndvi_med') <= 3533.5) ?  0.0001933568546371306 :   0.00916642171791307 :   (b('l8b11') <= 1429.556396484375) ?  -0.01820709419700675 :   -0.0029963572376097147 :   (b('l8b5') <= 1748.7725830078125) ?  (b('l8b5') <= 1479.5771484375) ?  0.06403683600477403 :   (b('dg0_v_vv') <= -0.6762237548828125) ?  0.004900062883501297 :   0.020234710829592478 :   (b('dg0_v_vv') <= -0.28078794479370117) ?  (b('ndvi_med') <= 3447.5) ?  -0.056538384186643406 :   -0.05726738566400415 :   -0.07082264570066638 :   (b('dg0_v_vv') <= -0.9729723930358887) ?  (b('ndvi_med') <= 3533.5) ?  -0.051935747570659885 :   (b('ndvi') <= 4397.0) ?  (b('l8b11') <= 1266.5304565429688) ?  0.02401122408848988 :   -0.019765366202188356 :   0.1011954842960911 :   (b('sand') <= 57.5) ?  -0.016435365671173346 :   (b('ndvi') <= 4657.0) ?  (b('l8b11') <= 1092.9508972167969) ?  0.0689271094808369 :   0.09678655934663885 :   (b('dg0_v_vv') <= -0.5152311325073242) ?  0.11222031978485247 :   -0.03622086073251507 :   (b('l8b11') <= 958.7872924804688) ?  (b('bulk') <= 150.5) ?  (b('bulk') <= 113.5) ?  (b('dg0_v_vv') <= -1.4115853309631348) ?  0.029649270664028532 :   (b('dg0_v_vv') <= 0.23027658462524414) ?  -0.015044247614093079 :   0.011041360094961483 :   (b('ndvi') <= 4323.5) ?  (b('ndvi') <= 3585.5) ?  0.06610242362291803 :   0.04109900764316112 :   (b('ndvi_med') <= 3520.75) ?  0.06015151803283643 :   0.09326967466791572 :   (b('crops') <= 3.3066861629486084) ?  (b('l8b5') <= 2239.8275146484375) ?  -0.01242499107216502 :   (b('l8b4') <= 623.1063842773438) ?  -0.040393512699698236 :   -0.04143454718877593 :   -0.07301999693430544 :   (b('sand') <= 76.5) ?  (b('ndvi') <= 3336.0) ?  (b('moss') <= 14.74414587020874) ?  (b('l8b11') <= 1044.7664184570312) ?  0.02083257374069224 :   -0.02248218133531212 :   (b('l8b5') <= 2984.0155029296875) ?  0.0591751547892474 :   0.022319192381832338 :   (b('sand') <= 41.0) ?  (b('l8b11') <= 1377.3652954101562) ?  0.05279509911747421 :   -0.00900727775147552 :   (b('l8b5') <= 2607.827392578125) ?  0.0028567776770756907 :   -0.029206802649963125 :   (b('dg0_v_vv') <= -0.19624805450439453) ?  (b('l8b5') <= 2353.6640625) ?  -0.08284224259339656 :   -0.08470394051141605 :   (b('ndvi_med') <= 3910.5) ?  -0.11239478016935464 :   -0.11756792368306665 :   (b('l8b4') <= 550.6309814453125) ?  (b('ndvi') <= 2726.5) ?  (b('ndvi') <= 2659.0) ?  (b('crops') <= 27.364456176757812) ?  (b('dg0_v_vv') <= 1.950265884399414) ?  (b('ndvi_med') <= 3311.0) ?  0.061676013387660766 :   0.0755741692547984 :   0.08382130817424893 :   (b('dg0_v_vv') <= 0.7328968048095703) ?  (b('dg0_v_vv') <= 0.016338825225830078) ?  0.09574125712656006 :   0.08925373531199646 :   (b('dg0_v_vv') <= 2.0905141830444336) ?  0.0973143580917506 :   0.09392500233620843 :   0.14499690560783873 :   (b('l8b5') <= 2533.8240966796875) ?  (b('dg0_v_vv') <= -0.0663909912109375) ?  (b('l8b4_med') <= 481.6563720703125) ?  0.1179936857714961 :   (b('ndvi') <= 4394.0) ?  0.07192421527524077 :   0.05446098522639035 :   (b('dg0_v_vv') <= 0.370729923248291) ?  (b('ndvi') <= 4890.5) ?  0.021579511293593163 :   0.05030972975557879 :   (b('ndvi') <= 4897.5) ?  0.06293576053466843 :   0.05539374722391793 :   (b('sand') <= 52.5) ?  (b('l8b11') <= 1351.4340209960938) ?  (b('grass') <= 57.840450286865234) ?  0.08593323328840094 :   0.07699344185602999 :   (b('ndvi_med') <= 4027.0) ?  0.05983351607094689 :   0.06690590811843955 :   (b('l8b11') <= 1167.4552001953125) ?  (b('dg0_v_vv') <= -0.9425535202026367) ?  0.04049613790459 :   0.0746395808484067 :   (b('ndvi') <= 4349.0) ?  0.0540375662926285 :   0.06279029479464607 :   (b('l8b5') <= 2405.21044921875) ?  (b('l8b4') <= 641.6177062988281) ?  (b('dg0_v_vv') <= -1.3065624237060547) ?  (b('ndvi') <= 3587.5) ?  (b('dg0_v_vv') <= -2.4059205055236816) ?  -0.029071080199521163 :   -0.018867861238019243 :   (b('ndvi_med') <= 3438.5) ?  -0.0061152615294863474 :   -0.010587335790908647 :   -0.04393801410042289 :   (b('ndvi') <= 3523.5) ?  (b('l8b4') <= 700.7951049804688) ?  (b('dg0_v_vv') <= 0.23594284057617188) ?  0.020626516306346945 :   0.04976341271786963 :   0.10062410685739692 :   (b('dg0_v_vv') <= -0.14490795135498047) ?  (b('dg0_v_vv') <= -3.136234760284424) ?  -1.0534126291011914e-05 :   -0.0013826042046605758 :   0.013490753355085039 :   (b('crops') <= 26.48409938812256) ?  (b('moss') <= 7.655141830444336) ?  (b('dg0_v_vv') <= 0.42270898818969727) ?  (b('l8b5_med') <= 2592.1824951171875) ?  0.079854426468806 :   0.07051861724118312 :   (b('dg0_v_vv') <= 2.247805595397949) ?  0.08497489929673702 :   0.09395431751900024 :   (b('ndvi') <= 3979.0) ?  (b('dg0_v_vv') <= 0.17544031143188477) ?  0.056755810281832034 :   0.043420849946716855 :   (b('dg0_v_vv') <= 0.49387073516845703) ?  0.07072342988305919 :   0.07318845120859913 :   (b('crops') <= 28.65452480316162) ?  (b('grass') <= 61.095558166503906) ?  (b('dg0_v_vv') <= 0.510249137878418) ?  0.051552255668456844 :   0.042834335654657135 :   (b('dg0_v_vv') <= -0.7295513153076172) ?  0.032642047529050755 :   0.04408659810629035 :   (b('grass') <= 55.426164627075195) ?  (b('ndvi') <= 3427.0) ?  0.047826334666138774 :   0.05935200226048061 :   (b('moss') <= 0.9910394549369812) ?  0.05921014112310254 :   0.08111797179895486 :   (b('sand') <= 28.5) ?  (b('bulk') <= 138.5) ?  (b('bulk') <= 122.0) ?  (b('l8b5') <= 1589.9111938476562) ?  (b('ndvi') <= 1032.5) ?  (b('dg0_v_vv') <= -2.5600504875183105) ?  (b('l8b11_med') <= 1594.7315673828125) ?  0.053355392327900875 :   0.05824285021335096 :   (b('ndvi') <= 835.5) ?  0.07073059151738814 :   0.0850357649298572 :   -0.020041822808372356 :   (b('ndvi') <= 1909.0) ?  (b('ndvi') <= 1333.5) ?  (b('l8b4_med') <= 852.7552185058594) ?  (b('dg0_v_vv') <= 0.8285527229309082) ?  0.05843341013458453 :   0.06708886121148044 :   (b('l8b4') <= 1087.4130859375) ?  0.07576219805383855 :   0.08419565840074084 :   (b('l8b4') <= 715.778564453125) ?  0.08521729432000319 :   (b('bulk') <= 115.5) ?  0.0951894298280096 :   0.09782032795866796 :   (b('l8b11') <= 873.8999633789062) ?  (b('l8b4') <= 403.1565246582031) ?  0.07599576402690655 :   0.07894726946649816 :   (b('dg0_v_vv') <= 0.24123024940490723) ?  0.049106531788775665 :   (b('l8b4') <= 828.212158203125) ?  0.05920411051567093 :   0.06807496250623424 :   (b('l8b4') <= 814.7949523925781) ?  (b('ndvi_med') <= 3341.25) ?  (b('l8b4') <= 305.94541931152344) ?  (b('dg0_v_vv') <= 1.0422091484069824) ?  0.09676891495801512 :   0.05804345280735823 :   (b('l8b4') <= 800.2589721679688) ?  (b('l8b5_med') <= 1946.91943359375) ?  -0.0734102384749622 :   0.004663948263658722 :   (b('dg0_v_vv') <= 0.46256065368652344) ?  0.08126490234580735 :   0.06608115952891869 :   (b('dg0_v_vv') <= -1.2466864585876465) ?  (b('sand') <= 27.5) ?  (b('ndvi') <= 4133.5) ?  -0.010659208840623675 :   -0.06975021122802198 :   (b('l8b4') <= 589.5198669433594) ?  -0.015407614880312781 :   0.027830036303860013 :   (b('l8b5') <= 2785.548095703125) ?  (b('l8b11') <= 1526.8865356445312) ?  0.016187676231882286 :   0.15801428522234015 :   (b('ndvi_med') <= 3568.0) ?  0.02923685775533697 :   0.061822127058711394 :   (b('grass') <= 25.327675819396973) ?  (b('l8b5') <= 2420.1474609375) ?  (b('ndvi') <= 3179.5) ?  (b('ndvi_med') <= 3341.25) ?  0.020582900262903686 :   0.045984874272118975 :   (b('l8b11') <= 1397.5734252929688) ?  0.0749565606325884 :   0.14423174791939838 :   (b('dg0_v_vv') <= 3.4839234352111816) ?  (b('ndvi') <= 1876.5) ?  0.0886228819711935 :   0.0022957154369720035 :   (b('l8b11_med') <= 1366.441650390625) ?  0.030335289625530226 :   0.08255951334338245 :   (b('l8b5') <= 1903.53662109375) ?  (b('l8b4') <= 952.8914489746094) ?  (b('ndvi') <= 2238.5) ?  -0.0007823239367151513 :   0.08094158503778186 :   (b('l8b11') <= 1933.724365234375) ?  -0.08059171410968405 :   0.011810306213639097 :   (b('ndvi') <= 3342.5) ?  (b('l8b11') <= 2062.47607421875) ?  -0.03221900901428175 :   -0.06411016750284285 :   (b('l8b4') <= 864.3087463378906) ?  0.014564954248476791 :   -0.01245909699313713 :   (b('dg0_v_vv') <= -0.5595097541809082) ?  (b('l8b11') <= 916.2740783691406) ?  (b('dg0_v_vv') <= -2.165222644805908) ?  (b('l8b5_med') <= 3108.58837890625) ?  0.09056014152767622 :   0.10059092711824225 :   0.1352141015530543 :   (b('ndvi') <= 2613.0) ?  (b('ndvi') <= 2327.5) ?  (b('l8b11') <= 1615.0256958007812) ?  (b('ndvi_med') <= 3578.0) ?  0.03101964389788485 :   0.1111643269348781 :   (b('l8b11') <= 2344.526611328125) ?  -0.017993652062608678 :   0.03233723807324206 :   (b('moss') <= 0.689393937587738) ?  (b('dg0_v_vv') <= -1.2521390914916992) ?  0.06557736058968688 :   -0.035013277633839344 :   (b('ndvi_med') <= 3599.5) ?  0.10386103846182869 :   0.03730246336969721 :   (b('l8b4') <= 532.8299255371094) ?  (b('grass') <= 55.14389419555664) ?  (b('grass') <= 38.4017391204834) ?  0.034768648264122216 :   0.08528319677581749 :   (b('grass') <= 69.29146575927734) ?  -0.04848091472360505 :   -0.02400941993855138 :   (b('l8b4') <= 829.2710266113281) ?  (b('l8b4') <= 789.310791015625) ?  -0.015690921981506813 :   -0.05690178009443132 :   (b('dg0_v_vv') <= -2.2753448486328125) ?  -0.02588474210491154 :   0.03142286707348663 :   (b('l8b5') <= 4189.72021484375) ?  (b('l8b5') <= 2294.992919921875) ?  (b('l8b5') <= 2248.405029296875) ?  (b('l8b4') <= 1095.331298828125) ?  (b('l8b5_med') <= 3058.4775390625) ?  0.026694491665503063 :   -0.05795669171317862 :   (b('dg0_v_vv') <= -0.4226255416870117) ?  -0.01490700496102583 :   0.07208651964130985 :   (b('dg0_v_vv') <= 0.7059202194213867) ?  (b('l8b5') <= 2271.35009765625) ?  -0.026526357426673308 :   -0.010978676805537657 :   (b('l8b11') <= 2029.6705322265625) ?  -0.06636079938523781 :   -0.04126648309627187 :   (b('ndvi') <= 4754.5) ?  (b('l8b5') <= 2314.76025390625) ?  (b('ndvi_med') <= 3508.5) ?  0.10336449564305866 :   0.15544143515434888 :   (b('dg0_v_vv') <= -0.10373878479003906) ?  0.06696224644078326 :   0.04007849182896458 :   (b('l8b4') <= 701.6230773925781) ?  (b('l8b5_med') <= 3012.455322265625) ?  0.012872825346844252 :   0.0626336456934922 :   (b('bulk') <= 142.5) ?  -0.019872321863838693 :   -0.0672673528996733 :   (b('sand') <= 20.5) ?  -0.09859131635330082 :   (b('bulk') <= 153.0) ?  (b('dg0_v_vv') <= 1.860952377319336) ?  (b('ndvi') <= 4753.5) ?  -0.018237135353849562 :   -0.003187657215378892 :   (b('l8b5_med') <= 2844.591552734375) ?  0.03570194557238235 :   0.007293896209224264 :   0.04147165403753972 :   (b('crops') <= 64.8629264831543) ?  (b('bulk') <= 137.5) ?  (b('l8b5') <= 2445.4755859375) ?  (b('l8b5') <= 2378.5692138671875) ?  (b('ndvi_med') <= 3103.0) ?  (b('l8b4_med') <= 1215.56201171875) ?  (b('l8b5') <= 2098.865234375) ?  -0.0008312624791646956 :   -0.014615763484553667 :   (b('l8b4_med') <= 1263.2452392578125) ?  0.04227871898550459 :   0.0012688371661460798 :   (b('l8b5') <= 2120.342529296875) ?  (b('l8b4_med') <= 676.5486145019531) ?  0.048956161562466266 :   -0.0034476560296233644 :   (b('ndvi') <= 1737.0) ?  0.05953652123858574 :   0.023516914266210588 :   (b('l8b5') <= 2387.1395263671875) ?  (b('ndvi') <= 2189.0) ?  (b('l8b5_med') <= 2734.0740966796875) ?  0.013601948741525158 :   0.03829492162355084 :   (b('ndvi_med') <= 3045.75) ?  0.07743172472192206 :   0.022644822750535785 :   (b('moss') <= 8.555930137634277) ?  (b('l8b5') <= 2390.5189208984375) ?  -0.027605250465509646 :   0.039916337434744945 :   (b('grass') <= 49.2135066986084) ?  -0.031421640323943366 :   0.008092425486002818 :   (b('l8b5_med') <= 2994.1806640625) ?  (b('crops') <= 58.930376052856445) ?  (b('l8b5') <= 2672.6378173828125) ?  (b('sand') <= 76.5) ?  -0.024226508268535736 :   0.03077234227046991 :   (b('l8b11') <= 2003.9058227539062) ?  -0.0090161287821941 :   -0.02939208207995142 :   (b('bulk') <= 125.0) ?  (b('l8b5') <= 2742.910888671875) ?  0.1287437947399452 :   0.07462512128917026 :   (b('ndvi') <= 4193.0) ?  -0.004226688687884899 :   0.01599627951425932 :   (b('crops') <= 32.88705635070801) ?  (b('ndvi') <= 4724.0) ?  (b('ndvi') <= 3703.0) ?  -0.006654335237747069 :   0.02657375720042503 :   (b('l8b11') <= 1135.6019287109375) ?  -0.016315564366745437 :   -0.0636381357858791 :   (b('dg0_v_vv') <= 1.8010706901550293) ?  (b('l8b4') <= 795.8892822265625) ?  0.01607523846151189 :   0.05092217430027134 :   (b('l8b4_med') <= 754.373779296875) ?  0.13091547047318003 :   0.043779812513839333 :   (b('ndvi') <= 1923.5) ?  (b('ndvi_med') <= 2152.25) ?  (b('ndvi_med') <= 1727.5) ?  (b('bulk') <= 152.5) ?  (b('l8b5') <= 2939.0072021484375) ?  -0.011546932668654435 :   0.11121616029705542 :   (b('dg0_v_vv') <= 0.9813480377197266) ?  0.006725981907122676 :   0.09351349258661984 :   (b('ndvi') <= 1404.0) ?  (b('grass') <= 98.81764602661133) ?  -0.010713162674396643 :   0.049797362999396966 :   (b('ndvi') <= 1479.0) ?  -0.05487859365093834 :   -0.023348051627274522 :   (b('crops') <= 0.339643657207489) ?  (b('ndvi') <= 1832.5) ?  (b('grass') <= 51.918222427368164) ?  0.0955107053059701 :   -0.0009269755371118624 :   (b('l8b5_med') <= 2203.0350341796875) ?  0.024631084573899537 :   -0.041128890392879085 :   (b('ndvi') <= 1817.0) ?  (b('grass') <= 14.690678119659424) ?  0.2262787842309411 :   0.04455475336898376 :   (b('l8b11') <= 1788.4483642578125) ?  0.022141525716403147 :   -0.029830365972909844 :   (b('dg0_v_vv') <= 0.07637453079223633) ?  (b('grass') <= 62.97923278808594) ?  (b('moss') <= 0.6168657392263412) ?  (b('l8b11_med') <= 2472.0782470703125) ?  -0.02984731847003809 :   0.007885682164129739 :   (b('sand') <= 39.5) ?  -0.005829253593836625 :   0.009207132422403976 :   (b('ndvi_med') <= 2255.0) ?  (b('ndvi') <= 3033.0) ?  0.015385081973115694 :   0.054522814197621276 :   (b('crops') <= 0.29916672222316265) ?  -0.016582156589175317 :   0.015075682583658967 :   (b('l8b4_med') <= 1240.6671142578125) ?  (b('l8b5') <= 2014.2413330078125) ?  (b('ndvi') <= 1961.5) ?  0.12869716616410623 :   0.031109551827941638 :   (b('ndvi') <= 4250.5) ?  0.0026118071285288294 :   0.027048327667467835 :   (b('grass') <= 73.0641860961914) ?  (b('l8b4_med') <= 1890.9815673828125) ?  0.0221460657813296 :   -0.02268541119745908 :   (b('l8b5_med') <= 2643.5938720703125) ?  0.0352653157686272 :   0.08380912659675405 :   (b('bulk') <= 160.0) ?  (b('moss') <= 1.338077336549759) ?  (b('ndvi_med') <= 2105.5) ?  (b('ndvi') <= 3440.0) ?  (b('ndvi') <= 2322.0) ?  (b('l8b5') <= 4360.988037109375) ?  0.036943650319483684 :   0.17963920246445833 :   (b('l8b4') <= 1081.2952270507812) ?  0.07093583343574579 :   0.10926953169842098 :   (b('ndvi') <= 3528.0) ?  (b('dg0_v_vv') <= 0.006649494171142578) ?  -0.038928352369062565 :   -0.06530981003067748 :   (b('dg0_v_vv') <= -1.7352008819580078) ?  0.008699024524711663 :   0.06861916838754892 :   (b('bulk') <= 144.5) ?  (b('l8b4') <= 702.0782165527344) ?  (b('l8b5') <= 3879.9412841796875) ?  0.00825804234574628 :   -0.04635446574413936 :   (b('l8b11') <= 1346.6690673828125) ?  0.057184305560524576 :   0.030502748399820832 :   (b('l8b11') <= 1128.1316528320312) ?  (b('dg0_v_vv') <= 2.238008499145508) ?  0.06420984717249419 :   -0.0495402745312018 :   (b('l8b4') <= 1317.830322265625) ?  -0.005339924515433228 :   0.054866985317680146 :   (b('moss') <= 3.4928038120269775) ?  (b('l8b11_med') <= 1949.8594360351562) ?  (b('l8b5') <= 3706.8031005859375) ?  (b('ndvi') <= 3806.0) ?  -0.0026124031801747273 :   0.029370354495769198 :   (b('dg0_v_vv') <= 0.9487085342407227) ?  -0.012528736305844862 :   -0.11470120793916966 :   (b('dg0_v_vv') <= 3.159214973449707) ?  (b('l8b11') <= 551.5433959960938) ?  -0.07041608487011575 :   -0.02309289841216912 :   (b('crops') <= 78.13647842407227) ?  0.033249976737778075 :   0.05644891446052028 :   (b('l8b4_med') <= 2059.6664428710938) ?  (b('l8b5_med') <= 2630.8795166015625) ?  (b('dg0_v_vv') <= 2.3773903846740723) ?  -0.006447487454790307 :   0.03209310995253208 :   (b('sand') <= 47.0) ?  0.047262880778418975 :   0.014660113574333286 :   (b('l8b11') <= 1638.060791015625) ?  (b('l8b4') <= 1020.0674438476562) ?  -0.08049519229457121 :   -0.05467688084328915 :   (b('l8b4') <= 1224.9555053710938) ?  -0.040844789851286985 :   -0.01488431005864263 :   (b('dg0_v_vv') <= -0.8688292503356934) ?  (b('l8b5') <= 3239.089599609375) ?  (b('ndvi') <= 2480.0) ?  (b('l8b5_med') <= 2900.7904052734375) ?  0.0016640838272371228 :   (b('dg0_v_vv') <= -2.0539169311523438) ?  0.024219932343632955 :   0.03604198205834207 :   -0.03211575052379945 :   (b('ndvi') <= 2631.5) ?  (b('ndvi') <= 2288.5) ?  -0.04131019187958013 :   -0.04864735725421834 :   -0.01802256288874554 :   (b('l8b4') <= 1304.9414672851562) ?  (b('ndvi') <= 2896.0) ?  (b('ndvi') <= 2737.5) ?  (b('l8b11') <= 1934.11669921875) ?  0.07487553258238758 :   0.03726474051659268 :   (b('dg0_v_vv') <= 0.802093505859375) ?  0.1940586313903941 :   0.12301391405742415 :   (b('ndvi') <= 3128.5) ?  (b('l8b5') <= 3728.1553955078125) ?  0.02881960025896362 :   -0.05733871756556072 :   (b('l8b11') <= 1538.334228515625) ?  0.14334830001378268 :   0.08676771089920053 :   -0.04684829723655576 :   (b('moss') <= 9.157812595367432) ?  (b('ndvi') <= 2859.0) ?  (b('moss') <= 3.33070170879364) ?  (b('l8b11_med') <= 2397.37158203125) ?  (b('bulk') <= 162.5) ?  (b('l8b5') <= 2178.477783203125) ?  (b('l8b11') <= 2567.0069580078125) ?  (b('ndvi_med') <= 2370.25) ?  (b('bulk') <= 150.5) ?  0.06644470895679798 :   0.03174685892985479 :   (b('l8b11') <= 2433.8182373046875) ?  -0.02607252794990321 :   -0.004861769136556338 :   (b('sand') <= 41.0) ?  (b('l8b11_med') <= 2267.8272705078125) ?  0.1731113162635451 :   0.11680081704996248 :   (b('l8b11_med') <= 2197.9912109375) ?  0.0900281158220729 :   0.11134387974771186 :   (b('sand') <= 22.0) ?  (b('dg0_v_vv') <= -2.978026866912842) ?  0.14160718466613986 :   (b('crops') <= 65.30402755737305) ?  0.06459659275744764 :   0.08475406701890235 :   (b('l8b4_med') <= 1209.7080688476562) ?  (b('l8b5') <= 2849.3699951171875) ?  -0.022176587455886975 :   -0.003098798035692123 :   (b('l8b11_med') <= 2380.7177734375) ?  0.0035364060910423076 :   0.04513075095983997 :   (b('l8b11_med') <= 2116.2254638671875) ?  (b('l8b4') <= 1335.621337890625) ?  -0.0750966556822953 :   (b('l8b4_med') <= 1460.6707153320312) ?  (b('l8b5') <= 2503.6080322265625) ?  -0.030890511290058074 :   -0.03771622543241202 :   (b('l8b5') <= 2596.2528076171875) ?  -0.058934605961055514 :   -0.05993460426232314 :   (b('l8b4') <= 1398.736572265625) ?  (b('l8b11') <= 2365.8685302734375) ?  (b('bulk') <= 168.0) ?  0.026924784242746858 :   0.013815138662436 :   (b('l8b5_med') <= 2572.6190185546875) ?  -0.0072489219916646655 :   0.0026389224880199644 :   (b('l8b4') <= 1501.7689208984375) ?  (b('dg0_v_vv') <= 2.2311363220214844) ?  -0.02955219591758984 :   -0.08203523661479908 :   (b('dg0_v_vv') <= 1.571380615234375) ?  -0.015275908112738039 :   0.08192842516577822 :   (b('l8b4_med') <= 2150.42578125) ?  (b('l8b11_med') <= 2422.781982421875) ?  (b('l8b5') <= 2435.7550048828125) ?  (b('grass') <= 61.42880630493164) ?  (b('ndvi') <= 1615.5) ?  0.05164707302303807 :   0.010046498534033665 :   (b('l8b11_med') <= 2404.7958984375) ?  -0.03769312480721422 :   -0.0736931251201386 :   (b('l8b11') <= 2630.5906982421875) ?  (b('l8b4_med') <= 1299.1873168945312) ?  -0.02168355282343944 :   -0.0558372009418117 :   (b('dg0_v_vv') <= 0.09192371368408203) ?  -0.09122742878861147 :   -0.07044640280284174 :   (b('dg0_v_vv') <= 6.049469709396362) ?  (b('l8b4') <= 2244.079345703125) ?  (b('ndvi') <= 1326.5) ?  -0.023323945861582263 :   -0.013820175701681309 :   (b('ndvi') <= 1989.5) ?  -0.03282396628349337 :   -0.014245806323462345 :   (b('dg0_v_vv') <= 6.5324132442474365) ?  0.04915065115707218 :   0.08363249339258505 :   (b('l8b4') <= 2134.131103515625) ?  (b('l8b4') <= 1537.0585327148438) ?  (b('dg0_v_vv') <= 3.382887125015259) ?  (b('l8b5') <= 2923.624267578125) ?  0.00025923821019948614 :   -0.009032271689535687 :   (b('l8b4_med') <= 2271.3209228515625) ?  -0.02449742509527908 :   -0.033824852763084556 :   (b('l8b11') <= 2651.9771728515625) ?  (b('dg0_v_vv') <= 3.8941707611083984) ?  0.02971175436462755 :   0.009151018013848322 :   (b('ndvi_med') <= 1349.5) ?  0.017251791067009865 :   -0.004941800666357916 :   (b('dg0_v_vv') <= 1.5991158485412598) ?  (b('l8b5') <= 3366.4415283203125) ?  (b('dg0_v_vv') <= -0.5379109382629395) ?  0.007752751753616546 :   -0.03360868707424734 :   (b('ndvi') <= 1075.0) ?  0.053092145437699446 :   -0.008923008764592488 :   (b('l8b4') <= 2487.7574462890625) ?  (b('l8b4') <= 2433.1993408203125) ?  0.03834241778276587 :   0.0728270999208046 :   (b('ndvi') <= 1702.0) ?  -0.004649088889917437 :   0.027893769643597993 :   (b('dg0_v_vv') <= 1.604611873626709) ?  (b('ndvi_med') <= 2914.0) ?  (b('moss') <= 4.233438491821289) ?  (b('crops') <= 70.6978645324707) ?  (b('dg0_v_vv') <= -2.008871555328369) ?  (b('dg0_v_vv') <= -2.708132743835449) ?  0.03360670031240237 :   0.04260670365026248 :   (b('dg0_v_vv') <= -0.6954250335693359) ?  0.07503674641868624 :   0.046912626276589625 :   (b('ndvi') <= 2671.5) ?  (b('l8b5') <= 2210.69287109375) ?  0.043980273875374974 :   0.005003832743139539 :   (b('dg0_v_vv') <= -0.3535118103027344) ?  0.07991822605032528 :   0.00530909052647506 :   (b('l8b4') <= 1366.4600219726562) ?  (b('l8b5_med') <= 3057.3515625) ?  (b('ndvi_med') <= 1929.5) ?  0.04060617343190841 :   -0.0131875063552686 :   (b('l8b4') <= 1351.1170654296875) ?  0.17818441317280448 :   0.1653143371410137 :   (b('moss') <= 8.482120990753174) ?  (b('ndvi') <= 1170.5) ?  -0.022444726316103142 :   -0.006854118798915558 :   (b('l8b11') <= 2432.469970703125) ?  -0.054532915596444104 :   0.021555968583981067 :   (b('l8b11') <= 2529.1826171875) ?  (b('l8b11_med') <= 1563.3502197265625) ?  (b('l8b4_med') <= 751.1001586914062) ?  (b('l8b11_med') <= 1237.39453125) ?  0.06621698273250431 :   0.06854031327981247 :   0.07620701089544926 :   (b('l8b11') <= 2201.6973266601562) ?  (b('l8b5') <= 2792.00927734375) ?  0.05085042762254946 :   0.034513674372330216 :   (b('dg0_v_vv') <= -0.8690061569213867) ?  -0.010161789575909297 :   0.014856406642441033 :   (b('l8b5') <= 3010.419677734375) ?  0.13615141632311248 :   0.11342731021516755 :   (b('l8b11_med') <= 2561.252685546875) ?  (b('ndvi') <= 2160.0) ?  (b('sand') <= 46.5) ?  (b('moss') <= 5.321009159088135) ?  (b('l8b5') <= 2532.17041015625) ?  0.05597325038806351 :   0.08710130903530547 :   (b('l8b4') <= 1450.64306640625) ?  0.12921259870362342 :   0.15389505665946135 :   (b('ndvi_med') <= 1751.0) ?  (b('grass') <= 45.86714172363281) ?  0.09024207752564584 :   0.02517190911861779 :   (b('dg0_v_vv') <= 2.980302095413208) ?  0.008028896022630396 :   0.042876437488110805 :   (b('ndvi') <= 2178.0) ?  -0.09899316681576595 :   (b('l8b5') <= 3261.453369140625) ?  (b('ndvi') <= 2369.5) ?  0.02314680264521815 :   0.06056704537934069 :   (b('dg0_v_vv') <= 1.6155915260314941) ?  0.07704584364364497 :   -0.019564367015574152 :   (b('l8b11') <= 2597.8944091796875) ?  (b('l8b11') <= 2524.9757080078125) ?  (b('dg0_v_vv') <= 1.716209888458252) ?  (b('l8b5') <= 2484.9351806640625) ?  0.06489678598504206 :   0.08709587528882945 :   (b('ndvi') <= 2534.0) ?  0.029572483074835294 :   0.0035539569604736104 :   (b('crops') <= 69.99465560913086) ?  (b('ndvi_med') <= 1636.25) ?  0.04238302123774107 :   0.04938302052248533 :   (b('l8b4') <= 1972.302978515625) ?  0.09910201903231092 :   0.13509442550132733 :   (b('ndvi') <= 2372.0) ?  (b('dg0_v_vv') <= 2.310182571411133) ?  (b('crops') <= 36.73203182220459) ?  0.029855771153912303 :   -0.0301017197281876 :   (b('bulk') <= 154.0) ?  0.027337096679637695 :   -0.09687286209909374 :   (b('bulk') <= 150.5) ?  (b('dg0_v_vv') <= 3.3587117195129395) ?  -0.08392676014536243 :   -0.07466729919296858 :   -0.027877431451425044 :   (b('l8b5_med') <= 3661.0555419921875) ?  (b('sand') <= 62.0) ?  (b('l8b11_med') <= 2761.9261474609375) ?  (b('l8b11') <= 2598.7742919921875) ?  (b('ndvi') <= 4361.0) ?  (b('ndvi') <= 4050.0) ?  (b('dg0_v_vv') <= 3.47413969039917) ?  0.020519694507130246 :   0.11276245270814554 :   (b('dg0_v_vv') <= 1.1629905700683594) ?  0.03717369519727506 :   0.0863826960030285 :   (b('l8b5_med') <= 3269.548828125) ?  (b('l8b11') <= 2228.921630859375) ?  -0.059720826288511296 :   -0.02057980206776657 :   (b('dg0_v_vv') <= -1.0001897811889648) ?  0.10218933123280832 :   0.02501953153464928 :   (b('l8b5_med') <= 3052.3477783203125) ?  (b('dg0_v_vv') <= -3.5184569358825684) ?  0.022361760606622316 :   (b('moss') <= 4.000938653945923) ?  0.11665717458531435 :   0.07876608437792978 :   (b('dg0_v_vv') <= 0.7567763328552246) ?  (b('ndvi_med') <= 2029.0) ?  0.05115676468316895 :   -0.01216421566103476 :   (b('ndvi') <= 3324.5) ?  0.08667986706810848 :   0.09909146679279096 :   (b('sand') <= 46.5) ?  (b('l8b4') <= 2228.3857421875) ?  (b('l8b5') <= 3594.5457763671875) ?  (b('l8b11_med') <= 2825.4913330078125) ?  -0.015484276500314823 :   -0.02470689417266119 :   0.015358516158057736 :   (b('l8b11_med') <= 2830.743408203125) ?  (b('ndvi_med') <= 1636.25) ?  -0.030155021710321042 :   -0.029541804173280284 :   (b('dg0_v_vv') <= -1.018064022064209) ?  -0.04611417483634549 :   -0.04705122210368344 :   (b('dg0_v_vv') <= -2.1699905395507812) ?  (b('l8b11_med') <= 2867.9974365234375) ?  (b('ndvi_med') <= 1766.0) ?  -0.049561638596682736 :   -0.03983484744936544 :   (b('l8b4_med') <= 2271.3209228515625) ?  -0.03273731460577872 :   -0.027254862692170756 :   (b('l8b4') <= 2179.832275390625) ?  (b('l8b4') <= 1384.6357421875) ?  -0.05372168577313366 :   0.011936249576860269 :   (b('l8b4') <= 2284.967041015625) ?  0.03687951233621449 :   0.05551178775827845 :   (b('ndvi') <= 3128.0) ?  (b('l8b11') <= 2929.51708984375) ?  (b('sand') <= 71.0) ?  -0.06243353042596114 :   -0.04666277245214755 :   -0.08006030543999101 :   (b('moss') <= 5.2718119621276855) ?  (b('dg0_v_vv') <= -3.773306369781494) ?  -0.02400599410523696 :   (b('l8b11_med') <= 1637.8905639648438) ?  -0.04027475544664235 :   -0.034998659882549674 :   (b('dg0_v_vv') <= 2.241042137145996) ?  (b('dg0_v_vv') <= 0.3809547424316406) ?  (b('l8b4') <= 1479.1551513671875) ?  -0.020478485830823845 :   -0.019609000175519004 :   -0.008478483863870567 :   (b('l8b4_med') <= 814.8666687011719) ?  -0.025479089441112962 :   -0.027894063896176408 :   (b('l8b5') <= 3351.1748046875) ?  0.05509612591315605 :   (b('ndvi') <= 3187.0) ?  (b('l8b4_med') <= 1555.0704345703125) ?  0.018216280799640286 :   0.028259492697348162 :   (b('l8b4') <= 1334.1885375976562) ?  (b('l8b5_med') <= 3694.775634765625) ?  (b('dg0_v_vv') <= 1.9412508010864258) ?  -0.050583159749817974 :   -0.05188220794159992 :   -0.05280932727225661 :   (b('l8b4') <= 1383.240234375) ?  (b('dg0_v_vv') <= 3.9627246856689453) ?  (b('l8b11_med') <= 2518.2154541015625) ?  -0.0015578045103938239 :   -0.00812029490445085 :   (b('l8b5_med') <= 3694.775634765625) ?  -0.055183756597492535 :   -0.0387466336934614 :   (b('dg0_v_vv') <= 2.5214295387268066) ?  (b('l8b11') <= 2795.7503662109375) ?  -0.04372125785513618 :   -0.03676890444095787 :   (b('dg0_v_vv') <= 2.776526927947998) ?  -0.015923911302987315 :   -0.03622913820167456 :   (b('ndvi_med') <= 1608.75) ?  (b('bulk') <= 150.5) ?  (b('l8b11_med') <= 2103.8870849609375) ?  (b('ndvi') <= 800.5) ?  (b('ndvi_med') <= 998.5) ?  (b('ndvi_med') <= 805.25) ?  -0.04247655880273847 :   (b('l8b4') <= 1469.3471069335938) ?  0.07145555483013122 :   0.03908317264432348 :   0.17722124655460303 :   (b('dg0_v_vv') <= 1.722038745880127) ?  (b('l8b4_med') <= 1266.5403442382812) ?  (b('l8b5') <= 2205.435302734375) ?  (b('ndvi_med') <= 1244.5) ?  -0.054064898689985624 :   -0.03192771541903258 :   (b('l8b4') <= 1511.107177734375) ?  -0.0057021232970699675 :   -0.029615983188490694 :   (b('l8b11_med') <= 1736.4746704101562) ?  0.1054042457096805 :   (b('l8b5') <= 2557.080078125) ?  -0.022642647873894663 :   -0.004657868084653029 :   (b('sand') <= 49.5) ?  (b('l8b4_med') <= 1367.7449340820312) ?  0.13055725708651045 :   0.1017847129957474 :   (b('dg0_v_vv') <= 2.0594801902770996) ?  (b('grass') <= 50.51819610595703) ?  0.026683097688952717 :   0.04455869548559968 :   (b('l8b5') <= 2316.1458740234375) ?  0.013592038567502018 :   -0.020947253943898708 :   (b('l8b11') <= 2381.750244140625) ?  (b('ndvi') <= 967.0) ?  (b('sand') <= 56.0) ?  (b('l8b5') <= 2207.91259765625) ?  (b('dg0_v_vv') <= 0.625159740447998) ?  0.010575003329418908 :   0.11904415415554633 :   (b('ndvi') <= 581.5) ?  -0.04061371832318786 :   -0.0605719854587312 :   (b('l8b11') <= 2131.597900390625) ?  (b('l8b4_med') <= 1473.6810302734375) ?  0.13861912955572492 :   0.1591378131649624 :   (b('sand') <= 57.5) ?  0.10898243368911716 :   0.037199173707283574 :   (b('ndvi') <= 1466.5) ?  (b('ndvi') <= 987.5) ?  (b('ndvi') <= 979.5) ?  -0.08859218211561667 :   -0.05733888019648446 :   (b('l8b11_med') <= 2936.2467041015625) ?  -0.014508678095860155 :   0.06378142496917194 :   (b('ndvi') <= 1468.5) ?  (b('l8b5') <= 2339.488037109375) ?  0.086428394823067 :   0.15793738479342667 :   (b('sand') <= 48.5) ?  0.044125796004553255 :   -0.003952632709700591 :   (b('grass') <= 49.56411170959473) ?  (b('sand') <= 50.0) ?  (b('l8b11_med') <= 2538.8292236328125) ?  0.1567125692532652 :   (b('ndvi') <= 1128.0) ?  0.15625906828516187 :   0.012934186549746079 :   (b('l8b5_med') <= 2732.0045166015625) ?  (b('dg0_v_vv') <= 3.2526309490203857) ?  -0.01601563330333862 :   -0.09673619717510223 :   (b('l8b11') <= 2835.87109375) ?  0.011824681549012582 :   -0.006914606704654578 :   (b('l8b5_med') <= 2116.1029052734375) ?  (b('ndvi_med') <= 1230.5) ?  (b('bulk') <= 145.5) ?  -0.09469566643299637 :   -0.08996223257220315 :   (b('l8b4_med') <= 1026.2471618652344) ?  -0.07594714592398681 :   -0.07098700817713025 :   (b('dg0_v_vv') <= 2.5216188430786133) ?  (b('l8b5') <= 2308.426513671875) ?  0.007790623425531625 :   -0.02098567792544878 :   (b('l8b4') <= 1672.2215576171875) ?  -0.017617458944905656 :   0.0414649822489645 :   (b('sand') <= 44.5) ?  (b('dg0_v_vv') <= 3.6072521209716797) ?  (b('moss') <= 24.391379356384277) ?  (b('dg0_v_vv') <= 2.1015501022338867) ?  (b('l8b11') <= 2355.2420654296875) ?  (b('l8b5') <= 2398.553466796875) ?  -0.003485838014075282 :   0.11595155099087659 :   (b('ndvi') <= 2736.5) ?  -0.0049877772410632975 :   0.02749853209858379 :   (b('l8b11_med') <= 2392.1243896484375) ?  -0.08172182279541901 :   (b('grass') <= 45.37925148010254) ?  -0.027062933678336926 :   -0.04709221351805981 :   (b('l8b5') <= 2740.912841796875) ?  (b('ndvi') <= 1269.5) ?  (b('ndvi') <= 1182.5) ?  -0.029999382426783468 :   -0.010585054584958977 :   (b('dg0_v_vv') <= 0.18481159210205078) ?  -0.032437059588446816 :   -0.013259977978486178 :   (b('l8b5') <= 2784.9854736328125) ?  -0.047575515283934555 :   -0.05130451032296156 :   (b('grass') <= 60.42243766784668) ?  0.09624971598450952 :   (b('l8b4_med') <= 1807.5634765625) ?  0.059789830984891934 :   0.03955584013616062 :   (b('l8b11_med') <= 2941.146240234375) ?  (b('ndvi') <= 1504.5) ?  (b('l8b11') <= 1837.336181640625) ?  (b('l8b5_med') <= 2456.864013671875) ?  -0.07710368084928551 :   (b('ndvi') <= 1052.5) ?  -0.05520703179803055 :   -0.06466891248989465 :   (b('l8b11_med') <= 2925.1917724609375) ?  (b('l8b4') <= 2020.0953369140625) ?  -0.02314908284826521 :   -0.029827893533326284 :   (b('l8b11') <= 3028.69677734375) ?  -0.05742087395818663 :   -0.02373611049938737 :   (b('grass') <= 69.96735191345215) ?  (b('ndvi') <= 1939.5) ?  (b('l8b5') <= 2478.867919921875) ?  -0.012579537033397451 :   -0.030832422336178254 :   (b('l8b4') <= 1464.6121826171875) ?  -0.03501029468446091 :   -0.0043628187734575125 :   (b('ndvi') <= 1837.0) ?  (b('dg0_v_vv') <= -0.06878423690795898) ?  0.0003782635189230471 :   0.02207521798041137 :   -0.0500625471244186 :   (b('l8b4') <= 1840.1677856445312) ?  (b('l8b4') <= 1411.9867553710938) ?  (b('dg0_v_vv') <= -0.3401470184326172) ?  (b('dg0_v_vv') <= -0.8934731483459473) ?  -0.056698908835148276 :   -0.05079502161438249 :   -0.06376279779718824 :   (b('dg0_v_vv') <= 1.8280186653137207) ?  (b('dg0_v_vv') <= 0.8459749221801758) ?  -0.003953044980600985 :   0.04520697111398658 :   (b('l8b5_med') <= 2489.433349609375) ?  -0.04511149588888236 :   -0.08485558504452001 :   (b('l8b11') <= 2929.8604736328125) ?  (b('ndvi') <= 1434.5) ?  (b('l8b4') <= 2172.876220703125) ?  -0.01755446961678018 :   0.033664516209836126 :   (b('l8b5') <= 3330.6439208984375) ?  -0.054572685590576465 :   -0.011068592582969963 :   (b('l8b11') <= 2964.803466796875) ?  (b('sand') <= 54.5) ?  0.036781812815500574 :   0.08780073482388415 :   (b('ndvi') <= 2018.0) ?  -0.015551287869631471 :   0.016394834985474907 :   (b('ndvi') <= 2289.0) ?  (b('ndvi') <= 1763.0) ?  (b('l8b4_med') <= 943.834716796875) ?  0.05792033311414571 :   (b('ndvi') <= 1742.0) ?  (b('l8b5') <= 3065.1339111328125) ?  (b('l8b5') <= 3006.6519775390625) ?  (b('bulk') <= 152.5) ?  -0.024033755608794734 :   -0.01078606447548985 :   (b('dg0_v_vv') <= -0.05316734313964844) ?  -0.01940423958808509 :   0.031708263121653986 :   (b('ndvi') <= 1574.5) ?  (b('l8b11') <= 4316.35546875) ?  -0.03155320423749303 :   -0.007219704564666042 :   (b('l8b5_med') <= 3272.1044921875) ?  -0.03394439776060113 :   -0.014180856581104483 :   (b('l8b11') <= 2496.5709228515625) ?  (b('sand') <= 49.0) ?  (b('l8b11_med') <= 2390.048095703125) ?  -0.04449380544971513 :   -0.057532869422569785 :   (b('dg0_v_vv') <= -0.042310237884521484) ?  -0.021408585847621238 :   0.0035914126622626427 :   (b('grass') <= 34.123403549194336) ?  (b('l8b11') <= 3294.760009765625) ?  0.0027581624909896107 :   -0.015573207902352072 :   (b('dg0_v_vv') <= 0.40226173400878906) ?  0.04024247121584168 :   0.01972850497878273 :   (b('dg0_v_vv') <= 0.3738255500793457) ?  (b('moss') <= 10.249449729919434) ?  (b('grass') <= 74.7491340637207) ?  0.07726537875218224 :   0.004588156160609119 :   (b('ndvi_med') <= 2115.0) ?  (b('bulk') <= 129.0) ?  (b('l8b5') <= 2467.630615234375) ?  0.0696568587030698 :   -0.008926934401005407 :   (b('dg0_v_vv') <= 0.37239646911621094) ?  -0.02724401885472128 :   0.045943722766655114 :   (b('ndvi') <= 2095.5) ?  (b('grass') <= 22.04929542541504) ?  0.05094905507160913 :   0.008059718265866666 :   (b('l8b11') <= 2402.7215576171875) ?  -0.032076307280090026 :   -0.006931397419279876 :   (b('l8b11') <= 3195.002685546875) ?  (b('grass') <= 74.62616729736328) ?  (b('grass') <= 17.99287700653076) ?  (b('l8b4') <= 1685.0548706054688) ?  -0.020367830528447255 :   -0.036842371107650526 :   (b('dg0_v_vv') <= 2.3360791206359863) ?  -0.038902743619741526 :   -0.054175870994473393 :   (b('ndvi') <= 1826.0) ?  -0.07151607879651967 :   -0.06862964434551723 :   (b('l8b11_med') <= 3020.3779296875) ?  0.05091524876545797 :   (b('dg0_v_vv') <= 2.417602062225342) ?  (b('l8b11') <= 3216.0655517578125) ?  0.005359652346258598 :   -0.031047396101242873 :   (b('ndvi') <= 2163.0) ?  0.01430568923733511 :   -0.051397632295798405 :   (b('moss') <= 14.337447166442871) ?  (b('ndvi') <= 2444.5) ?  (b('l8b5') <= 3047.445556640625) ?  0.10615017393471476 :   0.10904354308946183 :   (b('l8b5_med') <= 2401.4825439453125) ?  (b('l8b11') <= 2172.7640380859375) ?  0.013382970235697761 :   0.009957382722173572 :   (b('ndvi') <= 2847.0) ?  0.051862836568600926 :   0.04349569733972414 :   (b('l8b5') <= 3266.7855224609375) ?  (b('l8b4') <= 1336.0352783203125) ?  (b('dg0_v_vv') <= -1.2682743072509766) ?  (b('l8b11') <= 1937.4465942382812) ?  -0.04302161723809621 :   0.003848309705676775 :   (b('ndvi_med') <= 3554.25) ?  (b('l8b11_med') <= 2365.364013671875) ?  0.0427536552854376 :   -0.0072766468630026115 :   0.13675604254229506 :   (b('bulk') <= 127.5) ?  (b('l8b4_med') <= 1556.397216796875) ?  (b('l8b5_med') <= 2862.447998046875) ?  0.07028312977002503 :   0.0659551551361289 :   0.009283125359281316 :   (b('ndvi_med') <= 3006.25) ?  (b('ndvi') <= 2382.5) ?  -0.04241851109617925 :   -0.0024068683241662368 :   (b('l8b11_med') <= 1606.0435180664062) ?  -0.010286761912616627 :   -0.038713205349548266 :   (b('bulk') <= 152.0) ?  (b('l8b4') <= 1664.3740844726562) ?  (b('l8b11_med') <= 2247.2081298828125) ?  (b('ndvi') <= 3667.5) ?  -0.036939187523093435 :   -0.10019075485319656 :   (b('l8b11') <= 2867.850341796875) ?  -0.02780520900485678 :   -0.051209006668432595 :   (b('l8b4') <= 1697.0965576171875) ?  (b('l8b11_med') <= 2247.2081298828125) ?  0.037426027507000276 :   0.022104950921119 :   (b('l8b11_med') <= 1509.7184448242188) ?  0.03859363829414801 :   -0.025290478832008547 :   (b('l8b5') <= 3631.7523193359375) ?  (b('ndvi_med') <= 2577.0) ?  (b('l8b5') <= 3382.9552001953125) ?  -0.014774439664492905 :   -0.003193101286042714 :   (b('dg0_v_vv') <= 1.5003032684326172) ?  -0.02698141748321528 :   -0.024286231370625078 :   (b('ndvi_med') <= 2577.0) ?  (b('dg0_v_vv') <= 0.33780908584594727) ?  0.003020309937154797 :   0.025783426672712884 :   0.03424154249381292 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1475.7420043945312) ?  (b('grass') <= 51.88082504272461) ?  (b('sand') <= 28.5) ?  (b('ndvi_med') <= 2660.5) ?  (b('l8b11') <= 1243.6622924804688) ?  (b('bulk') <= 136.5) ?  (b('bulk') <= 95.5) ?  0.006982820211170804 :   (b('l8b11_med') <= 1322.5) ?  (b('l8b4') <= 385.20416259765625) ?  (b('l8b4') <= 297.1484832763672) ?  0.0683961876242159 :   0.06458383583527577 :   (b('dg0_v_vv') <= 2.1738808155059814) ?  0.04502131585334949 :   0.056319983745777374 :   (b('dg0_v_vv') <= 0.46256065368652344) ?  (b('grass') <= 4.616700172424316) ?  0.07669556488800289 :   0.07313841211122663 :   (b('l8b5_med') <= 2925.356689453125) ?  0.05947304357602684 :   0.052301181082954507 :   (b('dg0_v_vv') <= -3.445002555847168) ?  (b('l8b5_med') <= 3108.58837890625) ?  0.03906089103655169 :   0.0806671370145608 :   (b('l8b11') <= 1191.9859619140625) ?  (b('l8b5_med') <= 3196.53857421875) ?  (b('dg0_v_vv') <= -2.664226531982422) ?  0.08601798089066331 :   0.1051817088177626 :   0.13868504773641133 :   (b('dg0_v_vv') <= 3.4352505207061768) ?  (b('ndvi_med') <= 2273.75) ?  0.06133245391858355 :   0.07441742660549819 :   0.08886657952463217 :   (b('l8b11') <= 1425.9469604492188) ?  (b('l8b11') <= 1301.2376098632812) ?  (b('ndvi_med') <= 2107.5) ?  -0.009479671930878175 :   (b('l8b4_med') <= 1334.3555297851562) ?  -0.016395793267687808 :   -0.019374386087221862 :   (b('l8b5_med') <= 3042.8221435546875) ?  (b('l8b5_med') <= 2844.591552734375) ?  (b('l8b11') <= 1344.96630859375) ?  -0.0008526596954723797 :   0.003354646440815623 :   -0.018992321603267415 :   (b('dg0_v_vv') <= 3.078529119491577) ?  (b('ndvi_med') <= 2273.75) ?  0.011353717468967406 :   0.014733319661676514 :   -0.006023180346449247 :   0.0636575323656493 :   (b('bulk') <= 132.5) ?  (b('dg0_v_vv') <= 1.172614336013794) ?  (b('l8b4_med') <= 897.87841796875) ?  (b('dg0_v_vv') <= -1.4263381958007812) ?  (b('l8b5') <= 3172.2752685546875) ?  (b('dg0_v_vv') <= -1.716383457183838) ?  -0.03361973268461007 :   0.003447306544903439 :   (b('dg0_v_vv') <= -3.5238375663757324) ?  -0.01920453185023266 :   -0.043199156621085574 :   (b('l8b11') <= 1359.4191284179688) ?  (b('ndvi') <= 2996.5) ?  0.02495035304037529 :   0.002990398364449158 :   (b('dg0_v_vv') <= 0.20231914520263672) ?  -0.027880938090444743 :   0.002447291875036034 :   (b('l8b11') <= 1340.7396850585938) ?  (b('l8b4') <= 635.1471862792969) ?  (b('ndvi') <= 2900.0) ?  0.029023662835417966 :   0.0021718130001353282 :   (b('ndvi') <= 4445.5) ?  0.052852440582277005 :   0.01396037761520617 :   (b('ndvi') <= 3774.5) ?  (b('ndvi_med') <= 3373.0) ?  0.016051664090105777 :   -0.029125292792173546 :   (b('ndvi') <= 4778.5) ?  0.0063862919502620545 :   0.05127021829634887 :   (b('l8b11_med') <= 1651.1475219726562) ?  (b('l8b11') <= 1115.4427490234375) ?  (b('ndvi') <= 3045.5) ?  (b('dg0_v_vv') <= 1.675920009613037) ?  0.0043134662219676085 :   0.013133893670678992 :   (b('ndvi_med') <= 2736.25) ?  0.10221764455856253 :   0.06641660731029637 :   (b('l8b11') <= 1384.8170166015625) ?  (b('l8b5') <= 2352.737060546875) ?  0.04150493378968983 :   0.00819289779355069 :   (b('l8b5_med') <= 3490.31982421875) ?  0.08178886875063435 :   0.012867305119590125 :   (b('l8b5') <= 2255.2860107421875) ?  (b('l8b5_med') <= 2931.823974609375) ?  -0.047194621843656304 :   -0.012094617766698595 :   (b('dg0_v_vv') <= 1.3954906463623047) ?  0.024155094534000332 :   (b('l8b5_med') <= 2931.823974609375) ?  0.010780328135414538 :   -0.0023309742791753282 :   (b('l8b5') <= 4509.390869140625) ?  (b('l8b5') <= 4261.120849609375) ?  (b('l8b5') <= 2535.232666015625) ?  (b('l8b5') <= 2469.85205078125) ?  (b('ndvi_med') <= 3545.75) ?  0.07265665843847068 :   0.016731639484351638 :   (b('ndvi') <= 3921.5) ?  -0.04291599918009096 :   0.03625304070957877 :   (b('crops') <= 41.00823211669922) ?  (b('ndvi_med') <= 3937.75) ?  0.010283882723182385 :   0.036476176661512504 :   (b('dg0_v_vv') <= -1.1324048042297363) ?  0.022912224268918306 :   0.05414960858320656 :   (b('l8b5_med') <= 3595.3917236328125) ?  (b('l8b4_med') <= 721.0103149414062) ?  0.0605618036191477 :   0.10731596028270024 :   (b('dg0_v_vv') <= 0.002559661865234375) ?  (b('l8b5') <= 4425.248046875) ?  -0.025461753292555202 :   0.02448458947821998 :   (b('dg0_v_vv') <= 0.07872533798217773) ?  -0.0648425186850971 :   -0.037212855583300154 :   (b('moss') <= 1.0738831758499146) ?  (b('bulk') <= 152.0) ?  -0.028431334250975798 :   -0.004739128543550741 :   (b('dg0_v_vv') <= -1.2346720695495605) ?  (b('l8b4_med') <= 560.0) ?  0.040551827955926795 :   0.06484417427751782 :   (b('l8b5') <= 4698.57470703125) ?  (b('crops') <= 41.01304626464844) ?  0.08451655591726276 :   0.07657364351378063 :   (b('l8b11') <= 809.0003967285156) ?  0.08377831825216209 :   0.11246504292647502 :   (b('grass') <= 49.844093322753906) ?  (b('grass') <= 49.750457763671875) ?  (b('dg0_v_vv') <= -1.0382766723632812) ?  (b('l8b4') <= 543.4092407226562) ?  (b('ndvi_med') <= 3556.75) ?  (b('bulk') <= 130.5) ?  (b('grass') <= 28.945043087005615) ?  0.06575308706632954 :   0.07490753971327219 :   (b('l8b5') <= 1600.5772094726562) ?  -0.04187924446028826 :   0.01234511130714051 :   (b('l8b4') <= 374.45973205566406) ?  (b('l8b5_med') <= 2999.88916015625) ?  -0.04099114777913956 :   -0.004373626339995562 :   (b('ndvi') <= 4316.0) ?  -0.007714765731977684 :   0.011526061707372106 :   (b('l8b4_med') <= 568.6681213378906) ?  (b('moss') <= 3.128513753414154) ?  (b('l8b4_med') <= 503.0681610107422) ?  -0.07409021641380137 :   -0.09763217967931656 :   (b('ndvi') <= 4679.5) ?  -0.02767663712761648 :   0.07880416005600657 :   (b('dg0_v_vv') <= -1.0847878456115723) ?  (b('crops') <= 83.01785278320312) ?  -0.010765009899117007 :   0.011337699967734923 :   (b('grass') <= 16.96821689605713) ?  -0.07987417980912244 :   -0.027212650469970606 :   (b('l8b5_med') <= 3250.9730224609375) ?  (b('bulk') <= 117.5) ?  (b('l8b5_med') <= 2579.135986328125) ?  (b('ndvi_med') <= 3498.5) ?  0.023552848479989275 :   -0.03647688383609104 :   (b('bulk') <= 114.5) ?  0.034325468972886956 :   0.07289904754671697 :   (b('sand') <= 34.5) ?  (b('grass') <= 19.02888584136963) ?  0.0020766185721804737 :   0.015801344535444593 :   (b('moss') <= 7.416376113891602) ?  0.009897834455900919 :   -0.0029026900166590312 :   (b('l8b4') <= 893.3348388671875) ?  (b('ndvi_med') <= 4411.25) ?  (b('l8b4') <= 418.2057342529297) ?  -0.0010173741753635801 :   0.019449325850162437 :   (b('l8b11') <= 1123.0560302734375) ?  0.06958538802507373 :   0.010359255064402351 :   (b('l8b4') <= 971.3243103027344) ?  (b('dg0_v_vv') <= 0.12436676025390625) ?  0.1271034637320629 :   0.04345983262879988 :   (b('dg0_v_vv') <= -0.008449077606201172) ?  0.038569631276895666 :   0.08024472525762827 :   (b('ndvi') <= 3661.5) ?  (b('l8b4') <= 641.6177062988281) ?  (b('dg0_v_vv') <= -1.1488666534423828) ?  (b('ndvi') <= 3207.5) ?  0.026780964945612912 :   (b('dg0_v_vv') <= -2.245967388153076) ?  -0.0018573467704043056 :   -0.019737150024554607 :   (b('ndvi') <= 3323.0) ?  -0.044475890048941175 :   -0.07217714764382008 :   (b('ndvi') <= 3561.0) ?  (b('dg0_v_vv') <= -0.6542487144470215) ?  (b('ndvi_med') <= 3494.75) ?  0.08389527011645861 :   0.060635628881529846 :   (b('ndvi') <= 3385.0) ?  0.02667626652246449 :   0.05144161615648718 :   (b('dg0_v_vv') <= 1.6990995407104492) ?  (b('dg0_v_vv') <= -1.2922272682189941) ?  -9.480713661913498e-06 :   0.005286274342746394 :   0.012141678019576524 :   (b('l8b5') <= 3505.7174072265625) ?  (b('ndvi_med') <= 3664.75) ?  (b('l8b11') <= 984.0259399414062) ?  (b('ndvi_med') <= 3494.75) ?  0.08711524477085897 :   0.07278563536480304 :   (b('dg0_v_vv') <= -2.206328868865967) ?  0.04260862464407644 :   0.06238547174371824 :   (b('dg0_v_vv') <= -0.00254058837890625) ?  (b('dg0_v_vv') <= -0.0049839019775390625) ?  0.04282795621219214 :   0.0022868252511097364 :   0.09559572052884774 :   (b('dg0_v_vv') <= -0.9552197456359863) ?  0.04382981010077652 :   0.020463830730693744 :   (b('l8b5') <= 2391.3360595703125) ?  (b('l8b5') <= 1839.9698486328125) ?  (b('l8b11_med') <= 1203.5457763671875) ?  -0.09278592208362288 :   (b('moss') <= 9.913345098495483) ?  -0.04585170575951933 :   (b('l8b5_med') <= 2186.9771728515625) ?  -0.06933664207263646 :   -0.057867095690431705 :   (b('ndvi') <= 2926.5) ?  (b('ndvi') <= 2566.0) ?  (b('l8b5') <= 2054.2109985351562) ?  -0.0055784495349682495 :   (b('l8b5_med') <= 2675.4471435546875) ?  -0.0003509301381788149 :   0.0036490705174722776 :   (b('l8b5') <= 2143.0535888671875) ?  0.018021857752803516 :   (b('ndvi_med') <= 2981.0) ?  0.06502330120235184 :   0.08574523977381135 :   (b('l8b5') <= 2327.5904541015625) ?  (b('l8b4') <= 527.4883575439453) ?  0.009485640715174853 :   (b('dg0_v_vv') <= -0.42427825927734375) ?  -0.005773440370739326 :   -0.0358032526356382 :   (b('dg0_v_vv') <= -0.4937019348144531) ?  0.018573731026737497 :   (b('dg0_v_vv') <= 0.1476893424987793) ?  0.0027985789734136374 :   0.01347670396135684 :   (b('ndvi') <= 3575.0) ?  (b('l8b11') <= 1049.2786865234375) ?  (b('l8b5') <= 2623.8504638671875) ?  0.011832190133704429 :   0.049793855768871076 :   (b('dg0_v_vv') <= 1.018202304840088) ?  (b('l8b11') <= 1341.9823608398438) ?  (b('l8b11') <= 1284.2098999023438) ?  -0.040980256535206004 :   -0.05975386401841295 :   (b('l8b5_med') <= 2910.012939453125) ?  0.018964406981226353 :   -0.020343903031935964 :   (b('l8b5') <= 2906.39306640625) ?  (b('l8b4') <= 548.240234375) ?  -0.09976963650055451 :   -0.07090156492314661 :   (b('l8b4') <= 782.3010559082031) ?  -0.05451330037081156 :   -0.03518889546709289 :   (b('l8b5') <= 2583.3065185546875) ?  (b('l8b4_med') <= 740.747314453125) ?  0.028156514960599913 :   (b('ndvi_med') <= 3031.75) ?  0.06415652831204034 :   0.06315652628548242 :   (b('dg0_v_vv') <= 1.743204116821289) ?  (b('ndvi') <= 4708.5) ?  (b('l8b11') <= 1096.751220703125) ?  -0.058759700846680446 :   0.0016891015051380186 :   (b('ndvi_med') <= 3834.5) ?  -0.053181649812466314 :   -0.04181587476585391 :   -0.06431647141203731 :   (b('ndvi') <= 1465.0) ?  (b('moss') <= 22.596078872680664) ?  (b('l8b4') <= 984.2905578613281) ?  (b('grass') <= 63.95784568786621) ?  (b('l8b4') <= 565.8685913085938) ?  (b('moss') <= 11.811776161193848) ?  -0.11307391450027607 :   -0.08931904301280795 :   (b('dg0_v_vv') <= -0.7577319145202637) ?  -0.046803406781624424 :   (b('ndvi') <= 1109.5) ?  -0.059541112542966484 :   (b('ndvi_med') <= 1869.25) ?  -0.08156431112480228 :   -0.0644879006826794 :   (b('ndvi_med') <= 1622.5) ?  (b('dg0_v_vv') <= 1.2782182693481445) ?  (b('dg0_v_vv') <= -0.06186103820800781) ?  -0.04211594755816536 :   -0.04506054087266039 :   -0.10121733657752663 :   (b('l8b4_med') <= 1380.681884765625) ?  (b('l8b5_med') <= 2098.8546752929688) ?  (b('l8b11_med') <= 959.292724609375) ?  -0.01201260462207851 :   -0.0299971635799114 :   (b('ndvi') <= 1392.5) ?  -0.017364255091335545 :   0.0062114599900747955 :   -0.06094270195530577 :   0.03771869706341548 :   (b('dg0_v_vv') <= 1.1630597114562988) ?  (b('dg0_v_vv') <= -0.384890079498291) ?  0.007480857541510649 :   (b('bulk') <= 143.0) ?  (b('l8b5_med') <= 2044.6566162109375) ?  -0.03665231756166998 :   -0.03260178420024645 :   -0.02159731277944528 :   (b('l8b11') <= 1419.58740234375) ?  (b('dg0_v_vv') <= 1.894693374633789) ?  0.01954911464386702 :   (b('l8b5_med') <= 2053.8475341796875) ?  0.044307971851486994 :   0.06606624747610204 :   -0.020173137164132632 :   (b('l8b4') <= 442.47039794921875) ?  (b('sand') <= 36.0) ?  0.1918138883857577 :   (b('grass') <= 61.2738151550293) ?  (b('ndvi') <= 3796.0) ?  (b('grass') <= 56.258867263793945) ?  (b('l8b11') <= 691.341064453125) ?  (b('dg0_v_vv') <= 1.1951336860656738) ?  0.07802619049409507 :   0.07044626957054409 :   (b('moss') <= 12.15508222579956) ?  0.05679400926924664 :   0.0870608150939682 :   (b('l8b4') <= 415.1496887207031) ?  0.10923878969767012 :   (b('l8b11_med') <= 1194.6172485351562) ?  0.08347487466811704 :   0.08937774177196894 :   (b('ndvi') <= 4328.0) ?  (b('dg0_v_vv') <= 0.657958984375) ?  (b('dg0_v_vv') <= -0.821162223815918) ?  0.005473764302273402 :   0.028103862863627316 :   (b('l8b5') <= 2276.63427734375) ?  0.0696856157864165 :   0.033359097375016245 :   (b('ndvi') <= 4830.5) ?  (b('ndvi_med') <= 3618.25) ?  0.06536332434672028 :   0.09091304432790777 :   (b('dg0_v_vv') <= 0.6254019737243652) ?  0.04581587262703281 :   0.06499028535755026 :   (b('l8b5') <= 2077.009033203125) ?  (b('l8b5') <= 1672.1563110351562) ?  (b('dg0_v_vv') <= -1.153569221496582) ?  (b('grass') <= 81.67167663574219) ?  0.05763315240429662 :   0.038692223899791184 :   (b('l8b11') <= 451.41485595703125) ?  -0.02024471809923739 :   0.015231040965896964 :   (b('ndvi_med') <= 3447.5) ?  -0.05088454576797907 :   -0.051540647097603726 :   (b('l8b5') <= 3093.0123291015625) ?  (b('l8b11') <= 808.8252868652344) ?  (b('l8b4') <= 384.78814697265625) ?  0.1271414684445595 :   0.08936877232502677 :   (b('dg0_v_vv') <= 0.9670624732971191) ?  0.05080735912611911 :   0.08089712018363357 :   (b('ndvi') <= 4367.5) ?  (b('l8b4_med') <= 707.0129089355469) ?  -0.007788775173072485 :   -0.04776811454833921 :   (b('ndvi_med') <= 3671.5) ?  0.02081352204297783 :   0.13821424058913687 :   (b('l8b5_med') <= 2931.2874755859375) ?  (b('ndvi_med') <= 1534.0) ?  (b('ndvi_med') <= 1398.25) ?  (b('ndvi') <= 3636.0) ?  (b('l8b5_med') <= 2282.9473266601562) ?  (b('l8b11') <= 1418.0984497070312) ?  -0.06967888377897169 :   0.045442489762391874 :   (b('l8b11') <= 853.1332702636719) ?  -0.05669674040488376 :   0.08753403626000658 :   (b('l8b4') <= 520.9586181640625) ?  -0.07743340289924044 :   (b('l8b11_med') <= 2028.5079956054688) ?  -0.10910344965353602 :   -0.09783689408918028 :   (b('l8b4') <= 779.9220886230469) ?  (b('l8b4') <= 725.3379516601562) ?  (b('dg0_v_vv') <= -0.9906682968139648) ?  0.06264708895956372 :   0.1241792680345162 :   0.20547514062791136 :   (b('l8b11') <= 1471.9641723632812) ?  (b('dg0_v_vv') <= -0.6426968574523926) ?  0.007904868368275753 :   0.0625553090109703 :   0.1643352807345122 :   (b('l8b4_med') <= 1379.1030883789062) ?  (b('dg0_v_vv') <= 0.6690688133239746) ?  (b('sand') <= 40.5) ?  (b('moss') <= 11.660176277160645) ?  0.007093322397595154 :   -0.020106902250966334 :   (b('sand') <= 41.5) ?  0.07503265917743038 :   0.010747420204333051 :   (b('sand') <= 33.5) ?  (b('l8b4') <= 509.6878204345703) ?  0.10924848856713125 :   0.04109905140990276 :   (b('l8b5') <= 3379.8458251953125) ?  0.026947016475152876 :   -0.03043665606534238 :   (b('l8b4') <= 781.9871520996094) ?  (b('dg0_v_vv') <= 0.26814985275268555) ?  (b('l8b5') <= 1837.78515625) ?  0.006069595590076415 :   0.07395262521662554 :   (b('dg0_v_vv') <= 1.5952913761138916) ?  0.15549965228594487 :   0.10518312803581426 :   (b('l8b5_med') <= 2541.6783447265625) ?  -0.020086683636673847 :   (b('bulk') <= 159.5) ?  0.036327255148164705 :   0.011025165700561057 :   (b('crops') <= 9.505242109298706) ?  (b('l8b4') <= 538.868408203125) ?  (b('dg0_v_vv') <= 0.15514612197875977) ?  (b('l8b4') <= 471.89219665527344) ?  -0.0657179972408749 :   (b('dg0_v_vv') <= 0.034595489501953125) ?  -0.028578001914317655 :   -0.03592720080441092 :   (b('l8b4_med') <= 661.7697448730469) ?  0.009937224085465324 :   0.005368375974076134 :   (b('dg0_v_vv') <= -0.5609011650085449) ?  (b('l8b5') <= 3225.3880615234375) ?  (b('l8b11') <= 1367.3671875) ?  -0.04402795778764679 :   -0.061623437112964224 :   (b('sand') <= 37.0) ?  0.031643685305565744 :   -0.009985527174783432 :   (b('dg0_v_vv') <= 0.1679396629333496) ?  (b('dg0_v_vv') <= -0.5511517524719238) ?  0.16132089887095943 :   0.05594888634273265 :   (b('l8b11') <= 1397.5353393554688) ?  0.020493106073361314 :   -0.040990032739080216 :   (b('dg0_v_vv') <= 0.29318714141845703) ?  (b('l8b4_med') <= 656.7872619628906) ?  (b('ndvi') <= 2726.5) ?  (b('grass') <= 52.195669174194336) ?  0.1309028990928326 :   0.05987249058156574 :   (b('l8b4') <= 720.6612854003906) ?  0.05473291157577579 :   -0.04066468092827341 :   (b('l8b11') <= 1268.533203125) ?  (b('dg0_v_vv') <= 0.09204864501953125) ?  -0.024773312026377548 :   0.0412218986427225 :   (b('grass') <= 56.60784149169922) ?  0.12174388694564511 :   0.11822457336595993 :   (b('ndvi_med') <= 3098.25) ?  0.14560853789045064 :   (b('l8b11_med') <= 1257.0541381835938) ?  (b('l8b5') <= 2785.60693359375) ?  0.034116875006267144 :   0.06264815844076524 :   (b('dg0_v_vv') <= 1.2997794151306152) ?  0.07102746728871477 :   0.12086503031281386 :   (b('l8b11_med') <= 2649.172119140625) ?  (b('dg0_v_vv') <= 0.9097990989685059) ?  (b('l8b5_med') <= 2986.78076171875) ?  (b('ndvi_med') <= 3126.75) ?  (b('crops') <= 72.66519546508789) ?  (b('l8b5_med') <= 2982.138916015625) ?  (b('l8b11_med') <= 2014.972412109375) ?  (b('grass') <= 15.049099922180176) ?  (b('l8b5') <= 2773.38427734375) ?  -0.01667913693903762 :   0.009583437454514981 :   (b('l8b4') <= 1044.7940063476562) ?  -0.010315623352015267 :   -0.019987707973304444 :   (b('sand') <= 55.5) ?  (b('l8b4') <= 1097.212158203125) ?  0.01853970435871074 :   -0.00571835892020541 :   (b('grass') <= 69.92729187011719) ?  -0.012078977704413898 :   -0.0360398395355869 :   (b('l8b5') <= 2659.0950927734375) ?  (b('l8b4') <= 1009.7288208007812) ?  -0.02239183768133643 :   (b('l8b4') <= 1307.6296997070312) ?  -0.07839294841706106 :   -0.05781888901050832 :   (b('dg0_v_vv') <= -1.2574992179870605) ?  0.03721844950415097 :   (b('l8b11') <= 2452.2388916015625) ?  -0.03952402986463929 :   -0.019365361465870187 :   (b('bulk') <= 144.5) ?  (b('l8b5_med') <= 2843.04541015625) ?  (b('bulk') <= 131.0) ?  (b('l8b5') <= 1826.195556640625) ?  0.045161772519686366 :   0.0684326745869069 :   (b('dg0_v_vv') <= -0.19750356674194336) ?  0.01145661187069121 :   0.03875972971877718 :   (b('l8b5') <= 2725.9671630859375) ?  (b('l8b5') <= 2404.795166015625) ?  0.022233139837070426 :   -0.00819496762200711 :   (b('ndvi') <= 4211.0) ?  0.00436750570425965 :   0.01988128824855544 :   (b('dg0_v_vv') <= -0.7056980133056641) ?  (b('ndvi') <= 3414.5) ?  (b('l8b11') <= 1534.6119384765625) ?  -0.07111205880211621 :   -0.015449558550317186 :   (b('ndvi') <= 3715.5) ?  0.0324543380983132 :   -0.014694738199333604 :   (b('ndvi') <= 1256.5) ?  (b('l8b5') <= 2692.028564453125) ?  -0.008021256990712839 :   -0.05772347219780153 :   (b('l8b4_med') <= 1703.289306640625) ?  -0.0003977346516994181 :   0.035043588219528604 :   (b('grass') <= 30.832636833190918) ?  (b('moss') <= 0.8611987233161926) ?  (b('l8b4') <= 856.8493347167969) ?  (b('bulk') <= 146.0) ?  -0.048771907779370544 :   (b('dg0_v_vv') <= -1.6044535636901855) ?  0.019117239023498767 :   0.03797309961216572 :   (b('l8b5') <= 2192.069091796875) ?  0.13934682433719686 :   (b('ndvi') <= 2378.0) ?  0.045685994785168715 :   0.0795287189882345 :   (b('l8b5') <= 3579.9757080078125) ?  (b('l8b5') <= 2502.948486328125) ?  (b('l8b4_med') <= 918.0328063964844) ?  0.009043870460495438 :   -0.02391714893051179 :   (b('l8b5') <= 2822.4598388671875) ?  -0.018175540344001376 :   -0.0029033738645696714 :   (b('l8b11') <= 2765.3160400390625) ?  (b('ndvi') <= 3708.0) ?  0.005139050307706533 :   0.03140545981573264 :   0.11602148386946243 :   (b('l8b4_med') <= 721.4874267578125) ?  (b('crops') <= 4.610949158668518) ?  (b('ndvi') <= 4039.0) ?  (b('ndvi_med') <= 3542.5) ?  -0.00993247954931243 :   -0.03807319544123002 :   (b('l8b4') <= 772.7867431640625) ?  -0.02137518216193124 :   0.12189640125594053 :   (b('l8b4') <= 834.8306884765625) ?  (b('bulk') <= 130.0) ?  0.052668460382038505 :   0.005810806062755167 :   (b('l8b5_med') <= 2962.950439453125) ?  0.07243732559600793 :   -0.005715680636855208 :   (b('dg0_v_vv') <= 0.2762947082519531) ?  (b('sand') <= 26.5) ?  (b('dg0_v_vv') <= -2.126140594482422) ?  -0.028714875959707453 :   0.0408558320406212 :   (b('grass') <= 32.17122459411621) ?  0.03246757937636373 :   -0.017321679259432513 :   (b('l8b4') <= 770.4174194335938) ?  (b('l8b5') <= 2966.93798828125) ?  -0.04236172115003393 :   0.031091138127690217 :   (b('l8b4') <= 961.5106811523438) ?  0.09333599770663924 :   0.03691406372100046 :   (b('l8b5') <= 2731.8978271484375) ?  (b('l8b4') <= 2024.0022583007812) ?  (b('l8b11_med') <= 2314.19921875) ?  (b('bulk') <= 150.5) ?  (b('bulk') <= 132.5) ?  (b('l8b11_med') <= 1303.4545288085938) ?  0.05265725676110006 :   0.024473051023199896 :   (b('l8b11_med') <= 2012.5054321289062) ?  0.009007494338300337 :   -0.03818203911723918 :   (b('sand') <= 48.0) ?  (b('dg0_v_vv') <= 0.2710719108581543) ?  0.04442431203077456 :   0.07845508381460452 :   (b('crops') <= 60.5658073425293) ?  -0.05507306793165566 :   -0.022400126513088703 :   (b('ndvi_med') <= 1637.5) ?  (b('ndvi') <= 1196.5) ?  (b('l8b4') <= 1772.0249633789062) ?  -0.030747695882889073 :   -0.017024727887029953 :   (b('l8b11') <= 2418.456787109375) ?  0.05903988270851146 :   0.0497746902828791 :   (b('l8b4') <= 1993.0022583007812) ?  (b('l8b4') <= 1905.7645263671875) ?  -0.019786793937278835 :   0.09020035409228823 :   (b('dg0_v_vv') <= -0.5870742797851562) ?  -0.052989329182929916 :   -0.06324117344926512 :   (b('dg0_v_vv') <= -0.7069764137268066) ?  (b('ndvi_med') <= 1751.0) ?  0.18186983634724097 :   0.18348523593153884 :   0.17876983751549202 :   (b('l8b11') <= 2893.3167724609375) ?  (b('moss') <= 13.856780052185059) ?  (b('l8b5') <= 2784.703369140625) ?  (b('l8b11_med') <= 1199.9916381835938) ?  (b('ndvi') <= 3763.5) ?  0.01696357248967752 :   0.048416836594703416 :   (b('ndvi_med') <= 3681.75) ?  -0.05843964626365392 :   -0.09562859196487744 :   (b('l8b5') <= 3494.469970703125) ?  (b('dg0_v_vv') <= -1.5534472465515137) ?  0.0017528281498598568 :   0.019544841206220524 :   (b('l8b5') <= 3880.521728515625) ?  -0.010596716867646886 :   0.02210175750162442 :   (b('ndvi') <= 3553.5) ?  (b('l8b4_med') <= 918.6410827636719) ?  (b('ndvi') <= 2580.0) ?  -0.04120508926518394 :   0.0024430855525316366 :   (b('l8b4') <= 1057.55712890625) ?  -0.04571680845175739 :   -0.018807470494664396 :   (b('ndvi_med') <= 3355.5) ?  (b('l8b5') <= 3434.999755859375) ?  0.06310823321161374 :   -0.001027660009893788 :   (b('l8b11') <= 1495.5664672851562) ?  0.08958953515685542 :   -0.016574943520503992 :   (b('l8b5') <= 2832.7657470703125) ?  (b('dg0_v_vv') <= -1.2805604934692383) ?  (b('moss') <= 4.646369904279709) ?  (b('ndvi_med') <= 2029.0) ?  0.07094322788668095 :   0.07194322991323887 :   0.09102394600468366 :   (b('l8b4') <= 2000.2980346679688) ?  -0.007376694241946896 :   (b('dg0_v_vv') <= -0.9073705673217773) ?  0.003023946481520823 :   0.08504150852236678 :   (b('sand') <= 48.0) ?  (b('ndvi') <= 3982.0) ?  (b('l8b4') <= 2422.990478515625) ?  -0.00984125664761479 :   0.015612347000851448 :   0.09100167610439658 :   (b('ndvi') <= 1862.0) ?  (b('ndvi') <= 1544.0) ?  -0.03226190641044448 :   0.0003209856975701503 :   (b('l8b5') <= 3051.73046875) ?  -0.012239249704688626 :   -0.037321130489938105 :   (b('sand') <= 28.5) ?  (b('bulk') <= 143.5) ?  (b('l8b4') <= 1718.5924072265625) ?  (b('l8b5') <= 2118.9830322265625) ?  (b('ndvi') <= 1956.5) ?  (b('ndvi') <= 1714.0) ?  (b('bulk') <= 115.5) ?  -0.018037640527535093 :   0.04893054984176221 :   -0.035708309660603815 :   (b('dg0_v_vv') <= 2.2088961601257324) ?  (b('crops') <= 1.0771889686584473) ?  0.09353370269359687 :   0.08495278154519753 :   (b('l8b4_med') <= 987.94384765625) ?  0.0731005775486421 :   0.062461157296126224 :   (b('l8b5') <= 2304.0) ?  (b('dg0_v_vv') <= 2.6450510025024414) ?  (b('ndvi_med') <= 2154.25) ?  0.007596925475563149 :   -0.021051427779816834 :   (b('dg0_v_vv') <= 3.422964096069336) ?  -0.059771667604674464 :   -0.02832263692664494 :   (b('l8b5') <= 3409.7127685546875) ?  (b('l8b5') <= 3362.96923828125) ?  0.015014001565350822 :   0.14154761880613728 :   (b('l8b4') <= 1002.9585571289062) ?  0.06417243302464921 :   -0.01655559978244168 :   (b('l8b11_med') <= 1257.730224609375) ?  0.12881280538185297 :   (b('ndvi') <= 1931.5) ?  0.06963777336213739 :   (b('l8b4') <= 1857.12158203125) ?  0.10033092214541578 :   (b('l8b4_med') <= 1334.3555297851562) ?  0.07801188036129764 :   0.08462775729323307 :   (b('l8b11') <= 1577.4417114257812) ?  (b('l8b11_med') <= 1560.470458984375) ?  (b('l8b4') <= 709.1162719726562) ?  -0.06833259706167805 :   (b('ndvi_med') <= 3984.5) ?  (b('grass') <= 54.08838081359863) ?  0.007290245336939366 :   -0.021675688967553758 :   -0.05036853366679944 :   (b('l8b4') <= 786.3685607910156) ?  (b('l8b5') <= 3407.584716796875) ?  (b('l8b11') <= 1529.9969482421875) ?  0.03401160004379178 :   0.020157672098857493 :   -0.009774584816184095 :   0.07222797787112462 :   (b('ndvi_med') <= 2776.5) ?  (b('dg0_v_vv') <= 1.9486083984375) ?  (b('l8b4') <= 1111.1354675292969) ?  0.12751226133915217 :   0.19127995388854518 :   (b('ndvi') <= 2344.5) ?  0.05172488356398808 :   0.09078928210480702 :   (b('ndvi') <= 2837.0) ?  (b('l8b4') <= 834.4081420898438) ?  -0.01829142691640437 :   (b('l8b5') <= 1862.6451416015625) ?  0.02530349389772145 :   0.05079263647292158 :   (b('moss') <= 5.615995168685913) ?  (b('sand') <= 23.5) ?  0.07993407220822071 :   0.051898862481363224 :   (b('ndvi') <= 3626.0) ?  0.1238812505205796 :   0.0915417741094762 :   (b('l8b4_med') <= 1689.2672119140625) ?  (b('grass') <= 84.42839050292969) ?  (b('ndvi_med') <= 3276.5) ?  (b('l8b11') <= 2016.07373046875) ?  (b('sand') <= 40.5) ?  (b('crops') <= 59.452327728271484) ?  -0.008947890470965392 :   0.015662708616439716 :   (b('l8b11') <= 2013.57666015625) ?  0.01978735527038079 :   0.1426348459150617 :   (b('l8b4') <= 922.0498352050781) ?  (b('l8b4_med') <= 1146.26904296875) ?  -0.036056665522570044 :   -0.11505494114132739 :   (b('l8b5_med') <= 2846.39599609375) ?  0.0011918470369526896 :   -0.017696689764903684 :   (b('l8b11_med') <= 1421.9089965820312) ?  (b('bulk') <= 134.0) ?  (b('grass') <= 43.76112365722656) ?  0.009794954535097643 :   0.048153418546275514 :   (b('dg0_v_vv') <= 2.978693962097168) ?  0.06855390837162113 :   0.1044909877519904 :   (b('l8b4') <= 963.7568054199219) ?  (b('l8b4') <= 963.52294921875) ?  0.01825034507284901 :   0.16148962237264095 :   (b('l8b11_med') <= 1554.9989624023438) ?  -0.028236760614105193 :   0.008114358714467538 :   (b('dg0_v_vv') <= 3.1187210083007812) ?  (b('l8b5') <= 2521.1304931640625) ?  (b('bulk') <= 146.5) ?  (b('l8b4') <= 1635.6199340820312) ?  0.003421229477023799 :   0.14076201384185902 :   (b('l8b11_med') <= 2019.0858764648438) ?  0.02519368444079969 :   0.05961469912965689 :   (b('l8b4_med') <= 1339.08447265625) ?  (b('l8b11') <= 1905.7978515625) ?  -0.06308570460806086 :   7.41422962840077e-05 :   (b('l8b4_med') <= 1392.3910522460938) ?  -0.12550823194434776 :   -0.09213904344278243 :   (b('l8b11') <= 1684.7173461914062) ?  (b('ndvi') <= 3463.0) ?  (b('bulk') <= 143.0) ?  0.13287139977587303 :   0.11927133058794934 :   (b('l8b5') <= 2748.618896484375) ?  0.0807332993241881 :   0.051961320272252365 :   (b('l8b11') <= 2439.397705078125) ?  (b('ndvi') <= 2025.0) ?  -0.02071407327420305 :   0.05036970328459191 :   (b('l8b5') <= 3189.5743408203125) ?  0.09333656504053496 :   0.0399059552477225 :   (b('moss') <= 7.898109674453735) ?  (b('ndvi') <= 1206.0) ?  (b('l8b11') <= 2572.177978515625) ?  (b('sand') <= 50.0) ?  0.1215849829511946 :   (b('moss') <= 3.7176480293273926) ?  0.08536100771357516 :   0.05840710738653784 :   (b('l8b5_med') <= 3144.026611328125) ?  (b('dg0_v_vv') <= 1.102858543395996) ?  0.014590325947870963 :   -0.07899352926763811 :   (b('l8b11') <= 2807.1087646484375) ?  -0.007908030822708037 :   0.04021706312832002 :   (b('l8b4') <= 1214.0650634765625) ?  (b('dg0_v_vv') <= 4.574630498886108) ?  (b('l8b5_med') <= 3289.2808837890625) ?  0.08629557156149091 :   0.07627944737923542 :   (b('crops') <= 75.33199691772461) ?  0.10265414488395802 :   0.11183633825430189 :   (b('l8b4') <= 1341.6664428710938) ?  (b('l8b4') <= 1247.5433959960938) ?  0.032449341781035465 :   -0.014655351550771169 :   (b('l8b11') <= 3363.7882080078125) ?  0.05037540945480832 :   0.13536208365381205 :   (b('l8b4') <= 2196.444580078125) ?  (b('dg0_v_vv') <= 0.9348249435424805) ?  0.13684136123124416 :   (b('ndvi') <= 2506.5) ?  (b('l8b11') <= 2822.8070068359375) ?  0.0015114909847991642 :   0.06686622765151239 :   (b('dg0_v_vv') <= 1.8769066333770752) ?  0.05661286947913175 :   0.03725078826028931 :   (b('l8b5_med') <= 3025.82421875) ?  (b('dg0_v_vv') <= 0.9830055236816406) ?  -0.03031607917580203 :   (b('crops') <= 44.584747314453125) ?  -0.03690288012282818 :   -0.042374681979484685 :   (b('ndvi') <= 1573.5) ?  -0.06570804400771818 :   -0.05706066282860231 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b4') <= 1672.8639526367188) ?  (b('l8b4') <= 1661.8778686523438) ?  (b('l8b5') <= 3576.2919921875) ?  (b('l8b11') <= 3156.693603515625) ?  (b('l8b11') <= 2868.1038818359375) ?  (b('ndvi') <= 1075.0) ?  (b('ndvi_med') <= 1481.0) ?  0.004178257927621719 :   -0.06115038343117535 :   (b('ndvi') <= 1372.0) ?  0.02431396019093837 :   0.006715319363689402 :   (b('ndvi') <= 2526.0) ?  (b('l8b4') <= 1423.039306640625) ?  0.030050525109037163 :   -0.017498784694460128 :   (b('l8b4_med') <= 1649.88818359375) ?  0.05997415198986508 :   0.01357911432371553 :   (b('ndvi') <= 1463.0) ?  -0.01729475893652252 :   (b('dg0_v_vv') <= 5.800390005111694) ?  (b('dg0_v_vv') <= 2.7770001888275146) ?  0.0542911937617994 :   0.04455230987918388 :   0.06784335687684476 :   (b('dg0_v_vv') <= -0.06596183776855469) ?  (b('crops') <= 91.9686164855957) ?  (b('l8b4_med') <= 1656.4095458984375) ?  (b('l8b4_med') <= 1649.88818359375) ?  -0.03125502681577427 :   -0.03256546648250741 :   -0.03491970844309625 :   (b('ndvi') <= 3888.5) ?  (b('dg0_v_vv') <= -0.4891328811645508) ?  -0.008838036654506637 :   -0.01797631170483885 :   (b('dg0_v_vv') <= -1.057988166809082) ?  -0.027301231269156206 :   -0.014409032352345252 :   (b('dg0_v_vv') <= 0.662447452545166) ?  0.00997886581611565 :   0.01406395721062842 :   (b('ndvi') <= 982.0) ?  (b('ndvi_med') <= 872.25) ?  0.08257591282838977 :   0.0762145149617266 :   (b('l8b4_med') <= 1784.99560546875) ?  0.051501578859090014 :   0.042510410888407596 :   (b('ndvi') <= 1064.5) ?  (b('ndvi') <= 1002.5) ?  (b('dg0_v_vv') <= 0.7729263305664062) ?  (b('ndvi_med') <= 880.75) ?  (b('dg0_v_vv') <= -0.27872514724731445) ?  (b('ndvi') <= 605.5) ?  -0.0010169376550819564 :   -0.020490976863983098 :   (b('ndvi') <= 927.5) ?  -0.004502699773415042 :   -0.03361098638467579 :   (b('l8b5') <= 3944.27197265625) ?  (b('dg0_v_vv') <= 0.25734758377075195) ?  -0.025558067849720877 :   -0.04058459765675538 :   (b('dg0_v_vv') <= -0.23316383361816406) ?  -0.0313821644299354 :   -0.01141677046911105 :   (b('ndvi_med') <= 880.75) ?  (b('dg0_v_vv') <= 0.7836918830871582) ?  0.04804342390357608 :   (b('ndvi') <= 709.0) ?  -0.011581794393344719 :   -0.036417229035370854 :   (b('dg0_v_vv') <= 1.413558006286621) ?  (b('l8b11') <= 3271.1011962890625) ?  0.1317385764181223 :   0.08262830813430977 :   (b('l8b5') <= 3826.8687744140625) ?  0.023487357884540677 :   0.018020913266896532 :   (b('dg0_v_vv') <= 0.5558047294616699) ?  (b('l8b4_med') <= 1799.5505981445312) ?  -0.033901435928684956 :   (b('crops') <= 49.389366149902344) ?  -0.046236634800571774 :   -0.04150569934341203 :   (b('dg0_v_vv') <= 0.7182517051696777) ?  -0.050338465081633546 :   -0.053782090919216 :   (b('l8b11') <= 3799.934326171875) ?  (b('l8b4') <= 2437.00439453125) ?  (b('l8b5') <= 2872.5596923828125) ?  (b('l8b11_med') <= 2880.1024169921875) ?  (b('dg0_v_vv') <= 0.3479499816894531) ?  -0.022703481984465076 :   0.004362500112339334 :   (b('l8b5') <= 2678.7000732421875) ?  0.07007008268714215 :   0.02114502359827048 :   (b('ndvi') <= 1342.0) ?  (b('moss') <= 6.368875980377197) ?  -0.035284860724660284 :   -0.00265330371046775 :   (b('l8b5') <= 4246.0625) ?  -0.006706529395400188 :   0.030602252488725817 :   (b('l8b4') <= 2539.9510498046875) ?  (b('l8b5') <= 3416.531005859375) ?  (b('l8b11_med') <= 2890.52392578125) ?  -0.0018361795773195466 :   -0.03125101287149251 :   (b('dg0_v_vv') <= -0.424471378326416) ?  0.005312170556162639 :   0.0801720404246228 :   (b('l8b11_med') <= 2890.52392578125) ?  (b('l8b5') <= 3972.4658203125) ?  0.006381296948367075 :   0.05159950653494102 :   (b('l8b5') <= 3812.0435791015625) ?  0.007347903004727965 :   -0.018102706023347997 :   (b('dg0_v_vv') <= -0.09879302978515625) ?  (b('dg0_v_vv') <= -0.47196435928344727) ?  (b('dg0_v_vv') <= -0.5149469375610352) ?  (b('dg0_v_vv') <= -0.6870570182800293) ?  -0.037400455954709796 :   -0.03871798693762256 :   -0.04369606746380074 :   (b('l8b5_med') <= 3622.8421630859375) ?  -0.034615559570655735 :   -0.030717985626320374 :   (b('dg0_v_vv') <= 0.13411998748779297) ?  -0.0028997551167448954 :   (b('ndvi') <= 1359.0) ?  (b('dg0_v_vv') <= 1.8369555473327637) ?  -0.026157696905835814 :   -0.020992372601622233 :   -0.03439453585112121 :   (b('dg0_v_vv') <= 3.676583766937256) ?  (b('grass') <= 27.41930866241455) ?  (b('l8b4_med') <= 1631.4669799804688) ?  (b('ndvi') <= 1589.0) ?  0.13553921258653034 :   (b('l8b4') <= 1541.1029663085938) ?  (b('l8b5_med') <= 3594.4241943359375) ?  (b('ndvi') <= 2350.5) ?  0.06892553071280516 :   0.02825408113868626 :   (b('dg0_v_vv') <= -1.443953514099121) ?  -0.014856748962834593 :   0.01325380648879837 :   (b('l8b11') <= 3213.130859375) ?  -0.07783622367147941 :   (b('l8b11') <= 3295.8280029296875) ?  0.03875045048190212 :   0.010457565638151312 :   (b('l8b4_med') <= 2187.779052734375) ?  (b('l8b11') <= 1849.5628051757812) ?  (b('l8b5') <= 4221.24072265625) ?  (b('dg0_v_vv') <= -2.426753520965576) ?  -0.03788412556274994 :   -0.049686840112618605 :   (b('dg0_v_vv') <= 2.3911070823669434) ?  -0.018545072960599394 :   -0.0033038743631163825 :   (b('l8b4') <= 1843.02880859375) ?  (b('dg0_v_vv') <= -1.6259245872497559) ?  -0.030245076468218415 :   -0.012885220100420473 :   (b('l8b11') <= 2959.348388671875) ?  -0.0279555291645946 :   -0.021625693182573177 :   (b('ndvi_med') <= 1154.75) ?  (b('l8b11_med') <= 3968.727294921875) ?  (b('dg0_v_vv') <= -0.017019271850585938) ?  -0.005272728678311549 :   -0.016606954751431185 :   (b('l8b5') <= 3322.173828125) ?  0.028362658425023966 :   0.001960709492896189 :   (b('ndvi') <= 941.5) ?  (b('dg0_v_vv') <= -0.9625406265258789) ?  -0.03317104297755567 :   0.0021657680871112004 :   (b('l8b5') <= 4363.705810546875) ?  -0.021951263754385334 :   0.011606096646225701 :   (b('grass') <= 50.4460563659668) ?  (b('bulk') <= 152.5) ?  (b('dg0_v_vv') <= 0.31621408462524414) ?  (b('bulk') <= 148.5) ?  (b('l8b11_med') <= 2830.743408203125) ?  -0.008051422409683035 :   -0.02224086455604897 :   (b('l8b4') <= 1800.875732421875) ?  -0.0172851827505289 :   0.010659448251840058 :   (b('l8b4_med') <= 1834.5802001953125) ?  (b('l8b11') <= 3535.2872314453125) ?  0.03463317160387244 :   -0.045908162226817796 :   (b('l8b11') <= 2044.1732177734375) ?  -0.033936497753483535 :   -0.002260560572282684 :   (b('l8b5') <= 2544.435302734375) ?  (b('ndvi') <= 1703.5) ?  (b('ndvi') <= 1166.0) ?  0.04462089656973026 :   0.059648675721879894 :   0.02157154557260743 :   (b('l8b4') <= 1887.017822265625) ?  (b('dg0_v_vv') <= 0.21279525756835938) ?  -0.04197795760493427 :   0.017985054092324575 :   (b('l8b11') <= 3387.837890625) ?  0.04359697503039926 :   -0.037278617462419214 :   (b('moss') <= 16.738183975219727) ?  (b('l8b4') <= 1423.8971557617188) ?  (b('l8b5') <= 3439.8223876953125) ?  (b('dg0_v_vv') <= 3.165626049041748) ?  0.019743865292932247 :   -0.08227654638419804 :   -0.09552231446454146 :   (b('l8b5') <= 2402.17236328125) ?  (b('l8b11_med') <= 3237.8294677734375) ?  0.012606807130844248 :   -0.044719117986305575 :   (b('l8b5') <= 2569.518310546875) ?  -0.041125119484607904 :   -0.016131698570076403 :   (b('moss') <= 22.263896942138672) ?  (b('dg0_v_vv') <= -0.9171600341796875) ?  (b('l8b5') <= 2141.2406005859375) ?  -0.027408904176793394 :   -0.04238271681570027 :   (b('l8b11') <= 3259.83154296875) ?  -0.02787987957906095 :   -0.008640885881378207 :   (b('ndvi') <= 1964.5) ?  (b('ndvi') <= 1207.0) ?  -0.0044122761516241565 :   -0.03912826180110569 :   0.0626292500842682 :   (b('l8b5_med') <= 3137.0338134765625) ?  (b('l8b5') <= 2928.394287109375) ?  (b('l8b11_med') <= 2889.1270751953125) ?  (b('l8b5_med') <= 3025.76806640625) ?  (b('l8b4') <= 1061.2191162109375) ?  0.11724760326734635 :   (b('l8b11_med') <= 2767.9241943359375) ?  0.0567846883053878 :   0.07859475854116313 :   (b('ndvi') <= 2216.5) ?  (b('l8b4_med') <= 1850.4474487304688) ?  0.05814921565082015 :   0.044596519166079936 :   0.024642790366325218 :   (b('ndvi') <= 2581.5) ?  (b('l8b4') <= 1671.0615844726562) ?  (b('crops') <= 50.07838439941406) ?  0.006306596013351762 :   0.016869333877946802 :   0.03421833721081863 :   0.05381084788640275 :   (b('bulk') <= 152.0) ?  (b('l8b4_med') <= 1803.0075073242188) ?  (b('l8b4') <= 1886.1797485351562) ?  (b('l8b5') <= 3092.7073974609375) ?  0.0003847502013280546 :   -0.005477522240529331 :   -0.04212483033080254 :   (b('l8b4') <= 1929.2559204101562) ?  (b('dg0_v_vv') <= 5.847688674926758) ?  0.015505117267262648 :   -0.010921224215577521 :   (b('ndvi') <= 2177.0) ?  0.07526924405332655 :   0.00979563235798131 :   (b('l8b5') <= 3067.1680908203125) ?  (b('l8b11_med') <= 2980.5115966796875) ?  0.027162317739299058 :   0.035600256122544555 :   (b('moss') <= 8.057602167129517) ?  0.04615148962785587 :   0.052382554182379026 :   (b('sand') <= 62.0) ?  (b('ndvi') <= 1729.5) ?  (b('ndvi') <= 1418.0) ?  (b('ndvi') <= 1310.5) ?  (b('l8b4') <= 1456.3582153320312) ?  -0.024146019340132555 :   -0.001106664561119437 :   (b('ndvi_med') <= 1636.25) ?  0.03339617704054204 :   0.016542269959089362 :   (b('l8b5') <= 3116.0333251953125) ?  (b('l8b5_med') <= 3253.813720703125) ?  -0.05418161124331283 :   -0.03417041576930126 :   (b('dg0_v_vv') <= 5.985781669616699) ?  -0.024831047584317725 :   0.000547188424867423 :   (b('l8b5') <= 3636.106689453125) ?  (b('ndvi') <= 2208.0) ?  (b('l8b5') <= 3447.144287109375) ?  0.058498471021605825 :   0.059804157363103044 :   0.036346396540075376 :   (b('l8b5') <= 3776.9403076171875) ?  0.018202119874187503 :   (b('dg0_v_vv') <= 5.010555982589722) ?  -0.017059500182670755 :   0.0060584158448585695 :   (b('l8b5') <= 3469.0350341796875) ?  -0.07343810174993073 :   -0.036525959399398775 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi') <= 2321.5) ?  (b('dg0_v_vv') <= 1.6021161079406738) ?  (b('ndvi_med') <= 3209.0) ?  (b('moss') <= 8.823604106903076) ?  (b('l8b11') <= 1660.2322998046875) ?  (b('crops') <= 61.46778106689453) ?  (b('l8b4') <= 766.312255859375) ?  (b('l8b4') <= 763.8227233886719) ?  (b('moss') <= 7.979443788528442) ?  (b('ndvi') <= 2284.5) ?  0.00948480188116313 :   -0.03321622797093087 :   (b('moss') <= 8.63595724105835) ?  0.04908100919149881 :   0.09025922915580223 :   (b('moss') <= 5.0035587549209595) ?  (b('l8b5_med') <= 2932.6890869140625) ?  0.04879399189812342 :   0.03601117644828464 :   0.20356436029015437 :   (b('l8b4') <= 773.0325622558594) ?  (b('dg0_v_vv') <= 0.6950974464416504) ?  (b('crops') <= 54.7174129486084) ?  -0.05448153289986731 :   -0.03807970663233279 :   -0.0762217255052703 :   (b('l8b4') <= 779.4426879882812) ?  (b('ndvi') <= 1188.0) ?  0.07510927890973587 :   0.047278303116063056 :   (b('dg0_v_vv') <= 1.2982754707336426) ?  -0.009996792050175654 :   0.07095058087863192 :   (b('crops') <= 68.03106307983398) ?  (b('l8b5') <= 1691.8463745117188) ?  (b('l8b4_med') <= 1115.8173828125) ?  0.21399492247611068 :   0.14888727968812154 :   (b('l8b4') <= 883.7261657714844) ?  (b('l8b5') <= 2841.511962890625) ?  0.16523554775707705 :   0.03540444178260822 :   (b('crops') <= 65.55995178222656) ?  -0.012351906967279244 :   0.1583013604946353 :   (b('bulk') <= 141.0) ?  (b('moss') <= 1.890370711684227) ?  (b('dg0_v_vv') <= 0.021382808685302734) ?  0.03989929750410838 :   0.07131848315600658 :   (b('dg0_v_vv') <= -0.04791402816772461) ?  -0.02654161378758127 :   0.031006882480620373 :   (b('l8b5') <= 4222.4788818359375) ?  (b('grass') <= 17.282055854797363) ?  -0.002412124157708899 :   0.030693870323807503 :   0.15946510646785 :   (b('moss') <= 8.482120990753174) ?  (b('grass') <= 98.81764602661133) ?  (b('grass') <= 62.12185096740723) ?  (b('l8b4') <= 1707.683837890625) ?  (b('crops') <= 24.796006202697754) ?  -0.013038902959206409 :   0.003598609529431461 :   (b('l8b5_med') <= 3226.0240478515625) ?  -0.014045466151580217 :   -0.002649165012135444 :   (b('ndvi_med') <= 2879.5) ?  (b('ndvi_med') <= 1874.75) ?  -0.01173491135380514 :   -0.02790223330072468 :   (b('sand') <= 37.5) ?  0.12310811058282176 :   0.08835372108146465 :   (b('bulk') <= 145.5) ?  (b('ndvi') <= 1555.0) ?  (b('l8b5_med') <= 2006.7608642578125) ?  0.025070043327851167 :   -0.027960236139217443 :   (b('ndvi_med') <= 2581.75) ?  -0.00015847080196112565 :   -0.04042540117502477 :   (b('l8b5_med') <= 2675.7364501953125) ?  (b('l8b5') <= 2278.9857177734375) ?  0.03773460171204516 :   0.005509243138700363 :   (b('dg0_v_vv') <= -0.7363905906677246) ?  0.12473091548946177 :   0.056726117940320266 :   (b('l8b11') <= 2190.7646484375) ?  (b('l8b4') <= 1169.5552368164062) ?  (b('grass') <= 84.5565185546875) ?  (b('dg0_v_vv') <= -1.1691994667053223) ?  0.025029921433102148 :   0.006029927631985205 :   0.07497481905444847 :   (b('ndvi') <= 1616.5) ?  (b('dg0_v_vv') <= 0.18263912200927734) ?  -0.047250580796987685 :   -0.06454845658947395 :   (b('l8b11') <= 1770.6920776367188) ?  -0.06698041350696668 :   -0.022716857320845732 :   (b('l8b4') <= 1621.3695678710938) ?  (b('dg0_v_vv') <= 0.4158205986022949) ?  (b('l8b4') <= 1563.6166381835938) ?  0.02260514600719941 :   0.10668177542359324 :   (b('l8b4_med') <= 1484.9597778320312) ?  0.1476087990893873 :   0.08754085988594953 :   (b('l8b11_med') <= 2873.516357421875) ?  (b('l8b4') <= 1887.017822265625) ?  -0.03887723368056674 :   0.022421562786393852 :   (b('l8b4_med') <= 1795.3252563476562) ?  0.05480171042194103 :   0.01989125093597198 :   (b('l8b5') <= 2059.4749755859375) ?  (b('ndvi') <= 1960.5) ?  (b('ndvi_med') <= 956.5) ?  (b('dg0_v_vv') <= 0.31555891036987305) ?  (b('l8b4') <= 1138.8262329101562) ?  (b('dg0_v_vv') <= -0.5494356155395508) ?  -0.041631791130307697 :   -0.06178325146743341 :   (b('ndvi') <= 1012.0) ?  0.018190489893971956 :   -0.005245257779010423 :   (b('l8b4') <= 1361.2604370117188) ?  (b('l8b11') <= 1354.5609741210938) ?  0.08819969151808389 :   -0.00580841973557419 :   (b('dg0_v_vv') <= 0.9408774375915527) ?  0.11868945176073249 :   0.028223847424653906 :   (b('grass') <= 49.411293029785156) ?  (b('l8b5') <= 1621.0892944335938) ?  (b('ndvi_med') <= 1755.5) ?  -0.10425847123789622 :   -0.012716698797630191 :   (b('dg0_v_vv') <= 0.04060077667236328) ?  0.005309224004326183 :   0.05237867695497773 :   (b('l8b11') <= 2141.2447509765625) ?  (b('l8b11') <= 1790.1416015625) ?  -0.020636728127796527 :   -0.003528238582536207 :   (b('ndvi_med') <= 1866.5) ?  -0.03344738964810946 :   0.011122357060705747 :   (b('l8b4_med') <= 893.5168151855469) ?  (b('l8b5_med') <= 1607.8214721679688) ?  (b('l8b5') <= 1731.5941772460938) ?  (b('l8b11') <= 1436.08935546875) ?  -0.017724365397520883 :   0.036126509269350716 :   (b('dg0_v_vv') <= -0.21488475799560547) ?  0.1648550557361867 :   0.07128165876753459 :   (b('moss') <= 16.825237274169922) ?  (b('l8b5') <= 1869.3552856445312) ?  -0.011948777664389969 :   0.006053780547581838 :   (b('l8b4_med') <= 835.6808471679688) ?  -0.0424878933263207 :   -0.07320453021168258 :   (b('ndvi') <= 1965.5) ?  (b('grass') <= 64.58584976196289) ?  0.22274440682605878 :   0.17486931052492974 :   (b('l8b4_med') <= 958.1580810546875) ?  (b('bulk') <= 139.0) ?  0.006432999889223338 :   0.09802573169004304 :   (b('l8b4_med') <= 1215.56201171875) ?  -0.00276170426299315 :   0.056994504372355946 :   (b('ndvi_med') <= 1571.75) ?  (b('ndvi') <= 1465.5) ?  (b('l8b11') <= 2222.6761474609375) ?  (b('bulk') <= 143.5) ?  (b('ndvi') <= 995.0) ?  -0.05519008146084599 :   -0.006240148538287021 :   (b('dg0_v_vv') <= -1.668217658996582) ?  0.026047747821490146 :   -0.03309596135778295 :   (b('l8b5') <= 2204.72412109375) ?  (b('l8b4') <= 1644.19384765625) ?  -0.014101275495434102 :   0.061217838964508865 :   (b('l8b11_med') <= 2827.15966796875) ?  -0.017627122913704052 :   -0.010621959430000529 :   (b('l8b5_med') <= 3322.7132568359375) ?  (b('ndvi_med') <= 913.0) ?  (b('ndvi_med') <= 899.0) ?  0.02272223435556709 :   0.23170486990156125 :   (b('l8b11_med') <= 1604.3845825195312) ?  0.14790175266106098 :   0.001343325190243854 :   (b('l8b11') <= 3443.1575927734375) ?  (b('dg0_v_vv') <= -0.39380455017089844) ?  -0.035654740929078874 :   -0.014960105300375243 :   (b('dg0_v_vv') <= -0.23285865783691406) ?  0.022370673274744856 :   -0.0061342480882537005 :   (b('l8b11') <= 1388.7745361328125) ?  (b('dg0_v_vv') <= -1.1724247932434082) ?  (b('sand') <= 34.0) ?  0.12291000700905652 :   0.08893840821826551 :   (b('l8b11') <= 1330.8753662109375) ?  (b('l8b4') <= 779.0618591308594) ?  -0.027658606304192333 :   -0.047283609630126545 :   (b('l8b11_med') <= 1304.4703979492188) ?  -0.05839922462884194 :   -0.07628752037489744 :   (b('ndvi_med') <= 2569.25) ?  (b('bulk') <= 160.0) ?  (b('bulk') <= 158.5) ?  -0.020093385137455223 :   -0.03511125790433365 :   (b('ndvi_med') <= 1704.0) ?  -0.005418223629259836 :   0.034725674322518184 :   (b('ndvi') <= 2234.0) ?  (b('ndvi_med') <= 2619.0) ?  0.15205920372802742 :   0.004683262207451475 :   (b('l8b4_med') <= 972.0348205566406) ?  -0.037930913691113295 :   0.037054415877073144 :   (b('dg0_v_vv') <= -1.105031967163086) ?  (b('ndvi') <= 1860.5) ?  (b('l8b11') <= 1604.5377807617188) ?  (b('dg0_v_vv') <= -1.8820624351501465) ?  (b('l8b4_med') <= 765.4933166503906) ?  0.04088364041681794 :   -0.001666890288398809 :   (b('dg0_v_vv') <= -1.1857576370239258) ?  (b('ndvi') <= 1790.5) ?  (b('l8b11_med') <= 1462.4194946289062) ?  0.06514260363555932 :   0.06112784252466136 :   0.07812783227266246 :   0.09832992325487036 :   (b('l8b4') <= 1201.1568603515625) ?  (b('l8b11') <= 2321.4449462890625) ?  (b('ndvi') <= 1566.5) ?  -0.05213926427891338 :   (b('l8b11') <= 1824.854248046875) ?  -0.025879392919097563 :   -0.0028165534301251988 :   0.04488672988784442 :   (b('l8b11_med') <= 1553.0325927734375) ?  (b('dg0_v_vv') <= -3.942993640899658) ?  (b('dg0_v_vv') <= -4.308004856109619) ?  0.00997042611102462 :   0.003803756881358228 :   (b('l8b5_med') <= 3001.805908203125) ?  0.0035086418436590106 :   0.021825897885314232 :   0.047806631176004366 :   (b('l8b11_med') <= 1356.5550537109375) ?  (b('dg0_v_vv') <= -2.8289804458618164) ?  (b('sand') <= 50.0) ?  0.08364871668280857 :   0.07123133838235768 :   (b('l8b5_med') <= 2987.578369140625) ?  (b('l8b5_med') <= 2789.7708740234375) ?  (b('sand') <= 32.5) ?  0.03319695908565651 :   0.01847475675173968 :   0.07182946350070407 :   (b('ndvi') <= 1980.5) ?  -0.0372235483836513 :   (b('dg0_v_vv') <= -2.4753060340881348) ?  -0.03436134513094763 :   -0.012909624330986039 :   (b('l8b11_med') <= 1498.3541870117188) ?  (b('l8b4_med') <= 705.1431579589844) ?  (b('dg0_v_vv') <= -2.1465067863464355) ?  (b('dg0_v_vv') <= -2.203223705291748) ?  -0.009762845531519349 :   -0.01904770881515036 :   (b('l8b11') <= 1493.1549072265625) ?  -0.020764512717039264 :   -0.03250554788765093 :   (b('l8b4') <= 770.9387817382812) ?  (b('dg0_v_vv') <= -1.4103584289550781) ?  0.009329109037350514 :   0.02832911773962865 :   (b('grass') <= 11.059110403060913) ?  -0.004882896547144312 :   -0.01904770881515036 :   (b('l8b5') <= 2351.189697265625) ?  -0.07389247695895315 :   (b('dg0_v_vv') <= -3.064650058746338) ?  -0.04985090492010544 :   (b('dg0_v_vv') <= -2.5595626831054688) ?  -0.006266556428640754 :   -0.03913896251206185 :   (b('ndvi') <= 2294.0) ?  (b('l8b5') <= 1427.7767333984375) ?  -0.09463871904475274 :   (b('ndvi_med') <= 4121.5) ?  (b('l8b11') <= 1345.7085571289062) ?  (b('l8b4_med') <= 584.3911743164062) ?  (b('l8b5') <= 1644.5070190429688) ?  0.16366474541676984 :   0.051115491298914996 :   (b('grass') <= 4.1314942836761475) ?  0.054157103053633665 :   0.02046220231292331 :   (b('sand') <= 22.0) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.019517691134447544 :   0.06098436948165639 :   (b('ndvi') <= 1859.0) ?  0.03242508251615837 :   0.011701174565050074 :   -0.09449630106648335 :   (b('grass') <= 23.562928199768066) ?  (b('ndvi_med') <= 3279.5) ?  -0.019945756169880258 :   -0.02082113693098034 :   (b('dg0_v_vv') <= 0.016761302947998047) ?  -0.05981696547801462 :   -0.06264009374612646 :   (b('sand') <= 45.5) ?  (b('dg0_v_vv') <= 2.2162301540374756) ?  (b('l8b5_med') <= 1780.7888793945312) ?  (b('l8b11') <= 1199.4290771484375) ?  0.12452177523229468 :   (b('sand') <= 41.5) ?  0.061574774578037034 :   0.06454731686288667 :   (b('dg0_v_vv') <= 2.1410412788391113) ?  (b('dg0_v_vv') <= 2.100377321243286) ?  (b('dg0_v_vv') <= 1.9372038841247559) ?  (b('ndvi_med') <= 1385.75) ?  (b('dg0_v_vv') <= 1.8760499954223633) ?  0.07471487164922731 :   0.0965328149007442 :   (b('l8b5') <= 2170.443115234375) ?  -0.005657896942973324 :   0.020934736684254388 :   (b('bulk') <= 129.0) ?  (b('ndvi') <= 2057.0) ?  -0.06311284842040388 :   -0.1034889986545603 :   (b('ndvi') <= 1992.0) ?  -0.02511638673268965 :   0.021965473454236835 :   (b('bulk') <= 131.0) ?  0.12270263816468771 :   (b('l8b5_med') <= 2400.0955200195312) ?  0.0697975475518108 :   (b('l8b5') <= 2885.171630859375) ?  0.060555423295577865 :   0.05263568258947793 :   (b('bulk') <= 133.0) ?  -0.00029537935377321745 :   (b('l8b5_med') <= 2761.732421875) ?  (b('dg0_v_vv') <= 2.146782398223877) ?  -0.04898095924836951 :   (b('dg0_v_vv') <= 2.2084455490112305) ?  -0.05480611310088925 :   -0.05145718060769561 :   (b('l8b4_med') <= 1217.834716796875) ?  -0.041865926838729334 :   (b('ndvi') <= 1627.5) ?  -0.04602647507283536 :   -0.04484990071757908 :   (b('l8b11') <= 2780.76123046875) ?  (b('bulk') <= 128.0) ?  (b('crops') <= 67.33716201782227) ?  (b('sand') <= 37.5) ?  (b('moss') <= 6.7599149364978075) ?  -0.07090966070555448 :   -0.08038713871152715 :   -0.09067263067414652 :   (b('ndvi_med') <= 2209.0) ?  0.033418735414265655 :   0.055635467881033074 :   (b('l8b4') <= 1822.9525756835938) ?  (b('ndvi_med') <= 2205.75) ?  (b('bulk') <= 138.5) ?  (b('l8b4') <= 904.7775573730469) ?  0.018818651274735754 :   -0.010011791728960683 :   (b('ndvi_med') <= 1291.75) ?  0.10151683171242176 :   0.02237694552526021 :   (b('l8b11_med') <= 2159.5281982421875) ?  (b('l8b11') <= 2302.889892578125) ?  0.035249393484596894 :   -0.041630518060041435 :   (b('sand') <= 27.0) ?  0.04398893378309891 :   0.08810450396613116 :   (b('moss') <= 6.009309768676758) ?  (b('moss') <= 4.182402610778809) ?  (b('moss') <= 1.2606635093688965) ?  -0.032742510452463125 :   -0.019277457723899133 :   (b('dg0_v_vv') <= 2.725799322128296) ?  -0.0029648153794418486 :   0.0161959048106409 :   -0.10296929201716187 :   (b('moss') <= 6.727759122848511) ?  (b('l8b4_med') <= 1906.8161010742188) ?  (b('bulk') <= 158.5) ?  (b('l8b5_med') <= 2627.387939453125) ?  0.101551263399739 :   0.09673466717935691 :   0.08937094448399957 :   0.11392642447912113 :   (b('l8b5_med') <= 2767.893310546875) ?  0.03627562012284713 :   (b('l8b4_med') <= 1811.2288818359375) ?  0.07774641854682798 :   (b('ndvi_med') <= 1365.75) ?  0.06817079375106441 :   0.06468600209262176 :   (b('l8b11') <= 1749.1813354492188) ?  (b('l8b5') <= 1647.63818359375) ?  (b('dg0_v_vv') <= 2.0937116146087646) ?  (b('l8b5_med') <= 1538.142578125) ?  (b('l8b5') <= 1514.5205078125) ?  (b('bulk') <= 144.0) ?  0.11743112588704396 :   0.08437992640400288 :   0.039510628336068526 :   (b('l8b5_med') <= 1691.6533813476562) ?  (b('ndvi_med') <= 1478.0) ?  -0.04690882824067344 :   -0.09109560291977398 :   (b('ndvi') <= 1666.0) ?  (b('l8b11') <= 1524.5765380859375) ?  -0.0075121898820751515 :   0.010098215900401553 :   -0.04452153682796184 :   (b('l8b11_med') <= 2651.708740234375) ?  (b('l8b4') <= 978.4250793457031) ?  (b('dg0_v_vv') <= 5.1379945278167725) ?  (b('l8b11') <= 1589.1052856445312) ?  -0.04330952258586835 :   -0.05603031799003322 :   -0.01753848270478603 :   (b('dg0_v_vv') <= 2.34958553314209) ?  -0.08340493898004786 :   -0.06575591145751572 :   0.0194143910153467 :   (b('l8b11_med') <= 2586.0220947265625) ?  (b('grass') <= 93.4565200805664) ?  (b('ndvi') <= 2297.5) ?  (b('l8b4_med') <= 1540.5338134765625) ?  (b('l8b5') <= 2057.3917236328125) ?  0.043615716686120104 :   0.09094146819828475 :   (b('l8b11_med') <= 2326.037109375) ?  0.01541867176900713 :   0.044656083952964755 :   -0.07706545854846039 :   (b('ndvi') <= 2215.5) ?  (b('dg0_v_vv') <= 2.45184326171875) ?  -0.009983987233542296 :   (b('dg0_v_vv') <= 3.586395025253296) ?  0.011909074874288855 :   0.004452793376479164 :   -0.0594590454648644 :   (b('l8b5') <= 2247.9952392578125) ?  (b('sand') <= 53.5) ?  (b('dg0_v_vv') <= 4.253085136413574) ?  (b('ndvi') <= 1568.5) ?  0.03534999967512795 :   0.025015692373086762 :   0.07176513449004024 :   (b('l8b4') <= 1388.4542236328125) ?  (b('ndvi') <= 1050.5) ?  0.024298891781897168 :   0.00771084588279729 :   (b('ndvi_med') <= 1349.5) ?  -0.022465508378513355 :   -0.001249123169418026 :   (b('ndvi') <= 1314.0) ?  (b('sand') <= 50.0) ?  (b('dg0_v_vv') <= 4.415253400802612) ?  -0.019472555424247542 :   -0.023472556079898635 :   -0.04241800866301812 :   (b('ndvi') <= 1576.5) ?  (b('l8b11') <= 1641.7232666015625) ?  -0.010216471483247627 :   -0.000785263799703409 :   (b('l8b11_med') <= 2825.4913330078125) ?  0.0107116680559494 :   0.0008000260565766137 :   (b('dg0_v_vv') <= 1.6269431114196777) ?  (b('l8b4') <= 1355.206298828125) ?  0.1360537831934639 :   (b('sand') <= 50.0) ?  (b('bulk') <= 147.5) ?  0.030132634215988476 :   0.026485557936570425 :   0.04568863520018546 :   (b('dg0_v_vv') <= 4.271538019180298) ?  (b('bulk') <= 142.5) ?  (b('moss') <= 31.207828521728516) ?  (b('ndvi') <= 2290.0) ?  (b('l8b5') <= 3109.827880859375) ?  0.011262430999710783 :   0.07549133347982437 :   -0.06605766308924402 :   (b('l8b11') <= 2026.974609375) ?  0.10742657192337879 :   0.07466261376682976 :   (b('l8b4') <= 1722.0859985351562) ?  (b('l8b11_med') <= 2221.01025390625) ?  (b('l8b11_med') <= 2154.2769775390625) ?  0.0013593643045301214 :   0.08313455693031604 :   (b('ndvi_med') <= 2248.75) ?  -0.01598151978012947 :   -0.08166172395178586 :   (b('moss') <= 1.3587230145931244) ?  (b('ndvi') <= 1772.5) ?  0.004370959071233451 :   0.04842513271087362 :   (b('grass') <= 31.331445693969727) ?  -0.00839293084889058 :   0.016593594328258927 :   (b('crops') <= 77.60760116577148) ?  (b('l8b4') <= 992.0476379394531) ?  (b('ndvi') <= 2199.0) ?  (b('l8b5') <= 2212.9378662109375) ?  0.024323122251227514 :   0.07556489133458794 :   0.13804925174435656 :   (b('ndvi_med') <= 2138.75) ?  (b('l8b11_med') <= 2810.475341796875) ?  -0.013078211362093762 :   0.01270631910878323 :   (b('dg0_v_vv') <= 7.305660009384155) ?  -0.06458335474553924 :   -0.006096854498311571 :   (b('ndvi') <= 1229.0) ?  (b('l8b11_med') <= 2984.7359619140625) ?  -0.01477285201710063 :   0.02371028975226014 :   (b('ndvi') <= 1732.5) ?  (b('l8b5') <= 2723.280029296875) ?  0.09300892054605965 :   0.06178318281058307 :   (b('dg0_v_vv') <= 6.709964752197266) ?  0.03345476406832688 :   0.06697056979675574 :   (b('grass') <= 34.08266639709473) ?  (b('bulk') <= 129.5) ?  (b('ndvi_med') <= 3065.0) ?  (b('l8b4_med') <= 899.8749694824219) ?  (b('l8b11_med') <= 1645.1785888671875) ?  (b('dg0_v_vv') <= -1.4598159790039062) ?  (b('l8b5_med') <= 3253.04833984375) ?  (b('ndvi') <= 2512.0) ?  0.03328421675725618 :   0.03393597047369662 :   0.03125325045874189 :   (b('dg0_v_vv') <= 4.6497814655303955) ?  (b('dg0_v_vv') <= -0.39136266708374023) ?  0.07957182679118319 :   (b('l8b5') <= 2573.1109619140625) ?  0.05485943304818214 :   0.06326040728961776 :   0.08100376507741935 :   (b('l8b4_med') <= 843.4451293945312) ?  (b('l8b4') <= 836.2725830078125) ?  (b('l8b4_med') <= 822.0557250976562) ?  (b('dg0_v_vv') <= 0.24711298942565918) ?  0.09645485021346856 :   0.09256575825336633 :   0.08775420856910832 :   0.11608478592370164 :   0.06319703263037146 :   (b('l8b11') <= 1225.7555541992188) ?  -0.012803910070729674 :   0.008129891819255886 :   (b('ndvi') <= 4731.0) ?  (b('ndvi') <= 4719.5) ?  (b('ndvi_med') <= 4519.0) ?  (b('l8b5') <= 2305.5380859375) ?  (b('l8b5') <= 2106.40478515625) ?  (b('sand') <= 78.5) ?  0.007698337503025612 :   0.04166330966230279 :   (b('ndvi_med') <= 3671.0) ?  0.05696879612850606 :   0.04266123214079975 :   (b('l8b5') <= 5354.774658203125) ?  (b('l8b11_med') <= 1454.9094848632812) ?  0.01249315532228358 :   -0.017469132188257817 :   (b('dg0_v_vv') <= 4.583444118499756) ?  0.05091321159924887 :   0.022612439373659133 :   (b('l8b4') <= 700.4657287597656) ?  (b('ndvi') <= 3256.0) ?  (b('dg0_v_vv') <= 0.40630531311035156) ?  0.10521707140646425 :   0.06671916030058153 :   (b('l8b11_med') <= 1068.9169921875) ?  -0.017329299477310437 :   0.03217625949474818 :   (b('l8b4') <= 853.5129089355469) ?  (b('l8b4') <= 754.5669250488281) ?  0.15178383792834288 :   0.1211411794584992 :   (b('l8b5_med') <= 3413.9039306640625) ?  0.05696717819407094 :   0.010352524146226977 :   (b('crops') <= 44.40579414367676) ?  0.16803587784033264 :   (b('dg0_v_vv') <= 0.360288143157959) ?  0.07544319134200453 :   0.0899672485755143 :   (b('grass') <= 31.182425498962402) ?  (b('l8b4') <= 672.3958740234375) ?  (b('ndvi_med') <= 3896.0) ?  (b('l8b11') <= 1004.0027465820312) ?  (b('ndvi') <= 4956.0) ?  -0.018533111529111662 :   0.0055834800805745904 :   (b('dg0_v_vv') <= -0.04886150360107422) ?  0.01631879030400504 :   0.04026160093471301 :   (b('l8b11') <= 922.1272277832031) ?  (b('l8b4') <= 418.13441467285156) ?  0.08404349305947251 :   0.05575409811430995 :   (b('dg0_v_vv') <= -0.5036697387695312) ?  0.04945198319363032 :   0.016416731054745955 :   (b('l8b11_med') <= 1421.1503295898438) ?  (b('l8b5') <= 3064.659912109375) ?  (b('dg0_v_vv') <= -3.453171730041504) ?  -0.0056658548898466205 :   0.006284751756223506 :   (b('ndvi') <= 4777.5) ?  -0.009058432413929778 :   -0.027581880979151704 :   (b('l8b4_med') <= 658.6383361816406) ?  0.021008809424481523 :   (b('l8b11') <= 2167.2970581054688) ?  0.03845920665328966 :   0.050098359106292664 :   (b('ndvi') <= 4905.0) ?  (b('ndvi') <= 4869.0) ?  (b('l8b11') <= 1044.0164184570312) ?  (b('ndvi_med') <= 4323.0) ?  -0.03830080246392444 :   0.03788233743919467 :   (b('ndvi_med') <= 4343.0) ?  -0.005538609509340282 :   -0.032641087751498994 :   (b('l8b4_med') <= 927.03662109375) ?  (b('dg0_v_vv') <= -0.49001646041870117) ?  -0.06492920566566668 :   -0.05885956721435209 :   -0.03565870411214768 :   (b('dg0_v_vv') <= 0.486328125) ?  (b('sand') <= 53.5) ?  (b('dg0_v_vv') <= -0.5049071311950684) ?  0.012695881475793924 :   0.003096583123549712 :   (b('ndvi_med') <= 4323.0) ?  -0.03969202676870911 :   -0.06197799961240044 :   (b('dg0_v_vv') <= 0.8813338279724121) ?  0.08737296997228641 :   (b('dg0_v_vv') <= 3.508380651473999) ?  0.03131520264303716 :   0.06012952366934761 :   (b('bulk') <= 137.5) ?  (b('grass') <= 13.839268207550049) ?  (b('l8b11_med') <= 2009.24560546875) ?  (b('ndvi') <= 2506.0) ?  (b('ndvi_med') <= 3569.0) ?  (b('ndvi') <= 2335.5) ?  (b('l8b5_med') <= 2923.4842529296875) ?  -0.046369908930214074 :   -0.041781956813091985 :   (b('ndvi') <= 2344.0) ?  0.07452828798835509 :   0.010071218913584552 :   (b('dg0_v_vv') <= -0.14647531509399414) ?  (b('dg0_v_vv') <= -0.27724695205688477) ?  0.05146922000371026 :   0.15172928292540433 :   (b('l8b11_med') <= 1132.2691650390625) ?  0.025977715925265044 :   0.04632296185102586 :   (b('ndvi') <= 3210.5) ?  (b('crops') <= 71.58008575439453) ?  (b('ndvi') <= 2536.5) ?  -0.08594547115479212 :   -0.019430943624906192 :   (b('dg0_v_vv') <= -2.2138099670410156) ?  -0.03181313668992032 :   0.007805719785783449 :   (b('l8b5') <= 3793.0899658203125) ?  (b('l8b4') <= 1464.9367065429688) ?  0.007440463265460808 :   0.045990395757315596 :   (b('l8b5_med') <= 3099.957763671875) ?  0.0036561370137257435 :   -0.023268633286758524 :   (b('ndvi') <= 3155.0) ?  (b('l8b11_med') <= 2380.4537353515625) ?  -0.012046432447275879 :   (b('l8b4') <= 1277.8174438476562) ?  -0.0027125637714622286 :   0.00613896206571915 :   (b('dg0_v_vv') <= 1.8979146480560303) ?  (b('l8b4') <= 1140.8088684082031) ?  (b('dg0_v_vv') <= 1.5958671569824219) ?  0.06493851004511189 :   0.04707106297465907 :   (b('l8b5_med') <= 2996.1737060546875) ?  0.019807612852836054 :   0.025408358466991438 :   (b('l8b4') <= 1050.7095642089844) ?  0.10816670889665447 :   0.08726229712316966 :   (b('grass') <= 17.926023483276367) ?  (b('l8b5') <= 2694.989013671875) ?  (b('l8b11_med') <= 1977.55224609375) ?  (b('ndvi') <= 4504.0) ?  (b('dg0_v_vv') <= -1.6122417449951172) ?  -0.007909001335748478 :   0.02556715587889747 :   (b('l8b4_med') <= 719.25048828125) ?  -0.02507195400209107 :   -0.00820420878492565 :   (b('l8b4') <= 917.1578063964844) ?  (b('dg0_v_vv') <= 0.8416538238525391) ?  -0.04476662313035165 :   -0.013069463695963927 :   (b('dg0_v_vv') <= 1.470750093460083) ?  -0.008464732303035221 :   0.05786333135384625 :   (b('l8b11') <= 1455.145263671875) ?  (b('dg0_v_vv') <= -0.6724534034729004) ?  (b('dg0_v_vv') <= -1.7370495796203613) ?  -0.01025453891793563 :   -0.05337835161280801 :   (b('ndvi') <= 4607.0) ?  0.004742723756079076 :   -0.04822947029217609 :   (b('ndvi') <= 2839.0) ?  (b('l8b11') <= 1646.7377319335938) ?  -0.08306468461987099 :   -0.054415119409923364 :   (b('dg0_v_vv') <= 6.263915300369263) ?  -0.031565019589476974 :   -0.0935330457529368 :   (b('l8b5_med') <= 3086.213134765625) ?  (b('l8b5') <= 2437.158935546875) ?  (b('dg0_v_vv') <= 0.17549967765808105) ?  (b('l8b11') <= 1161.5805053710938) ?  -0.027923011699576592 :   0.00491706143494623 :   (b('l8b5_med') <= 1590.654296875) ?  -0.023154351012125905 :   0.03576550560027252 :   (b('sand') <= 36.5) ?  (b('l8b11') <= 1191.67138671875) ?  0.02634167001849098 :   -0.0047423469972224346 :   (b('l8b5') <= 3667.31005859375) ?  -0.015695706860621238 :   0.001292905789426198 :   (b('dg0_v_vv') <= -1.4944982528686523) ?  (b('moss') <= 9.370570421218872) ?  (b('dg0_v_vv') <= -3.3313660621643066) ?  -0.002789599294962017 :   -0.028177331128270127 :   (b('ndvi') <= 4276.0) ?  -0.034690081109705895 :   0.07576009127886349 :   (b('l8b5_med') <= 3763.3551025390625) ?  (b('ndvi') <= 4012.5) ?  0.030559984917784593 :   0.004618349457695049 :   (b('dg0_v_vv') <= 0.6034369468688965) ?  -0.09053221211375985 :   -0.013909353993018375 :   (b('l8b11_med') <= 2736.30615234375) ?  (b('ndvi_med') <= 2141.0) ?  (b('l8b5') <= 3326.60107421875) ?  (b('l8b5') <= 1765.1270141601562) ?  (b('ndvi') <= 2616.5) ?  0.06750944752197705 :   (b('dg0_v_vv') <= 6.490261077880859) ?  -0.04404576121071803 :   0.05339722775061803 :   (b('l8b11') <= 1624.0656127929688) ?  (b('dg0_v_vv') <= 2.8313658237457275) ?  0.052260539114152245 :   0.09840028019828766 :   (b('l8b4') <= 1600.2325439453125) ?  0.025261099308433704 :   0.06902108922486433 :   (b('dg0_v_vv') <= 3.3169045448303223) ?  (b('dg0_v_vv') <= 3.0025455951690674) ?  (b('ndvi') <= 2659.0) ?  0.00034985695251699015 :   0.02547992638841768 :   (b('l8b4') <= 1569.9780883789062) ?  -0.07925988987214688 :   -0.0025321888062403153 :   (b('moss') <= 2.466792196035385) ?  0.11349567577182251 :   0.07732352628753297 :   (b('ndvi_med') <= 2346.25) ?  (b('l8b4') <= 787.1499938964844) ?  (b('l8b4') <= 457.5613555908203) ?  (b('dg0_v_vv') <= 1.8869898319244385) ?  -0.0008994169351878717 :   -0.07312843706095969 :   (b('l8b4') <= 475.9568786621094) ?  -0.07361469690262756 :   -0.03771210986580376 :   (b('l8b4') <= 1138.7021484375) ?  (b('dg0_v_vv') <= 1.1744987964630127) ?  -0.006044812642151574 :   0.02069873294091308 :   (b('ndvi') <= 3187.0) ?  -0.015793785819247196 :   -0.03346245096619812 :   (b('grass') <= 26.036283493041992) ?  (b('sand') <= 36.0) ?  (b('dg0_v_vv') <= 1.5917339324951172) ?  0.01390950146678359 :   0.03614669319393281 :   (b('ndvi_med') <= 4013.5) ?  0.0005660360252206186 :   0.03212253400872524 :   (b('l8b5') <= 3223.3177490234375) ?  (b('ndvi') <= 4551.0) ?  -0.0008529741817086276 :   0.057635861680804215 :   (b('ndvi') <= 4371.5) ?  -0.029424846986608116 :   0.018420314101253533 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b4') <= 2171.085693359375) ?  (b('dg0_v_vv') <= -3.2804417610168457) ?  (b('l8b4') <= 1220.5791015625) ?  (b('dg0_v_vv') <= -4.818332672119141) ?  0.033964133789130185 :   0.02191194666113766 :   0.05184142397711425 :   (b('l8b11') <= 1244.9990844726562) ?  (b('ndvi') <= 3871.0) ?  0.03836979799340333 :   0.01501589450315024 :   (b('l8b11') <= 2917.9676513671875) ?  -0.008934818622453882 :   0.05397673679087858 :   (b('l8b11') <= 2779.307373046875) ?  (b('dg0_v_vv') <= -0.6468896865844727) ?  0.05317946119408168 :   (b('dg0_v_vv') <= -0.31043386459350586) ?  0.035083066106968985 :   0.04808306264989959 :   (b('l8b11_med') <= 2890.52392578125) ?  (b('ndvi') <= 3561.0) ?  0.013374718442832223 :   0.030781735743395877 :   0.008241190108085716 :   (b('l8b4') <= 856.8479614257812) ?  (b('dg0_v_vv') <= 2.411346673965454) ?  (b('dg0_v_vv') <= 1.3396039009094238) ?  -0.02358716045639997 :   (b('l8b11') <= 1555.193359375) ?  -0.08772773843588241 :   -0.11720065671081376 :   (b('dg0_v_vv') <= 3.0925941467285156) ?  (b('crops') <= 84.94914245605469) ?  0.02272932595044147 :   -0.014913477936360137 :   -0.07235598561844317 :   (b('dg0_v_vv') <= -1.6917109489440918) ?  (b('l8b11') <= 3160.9521484375) ?  (b('l8b5') <= 2754.9298095703125) ?  -0.07005260130433147 :   -0.03199052199905181 :   (b('moss') <= 8.29210078716278) ?  0.020528304172595054 :   -0.023112415995643204 :   (b('l8b5') <= 3221.3533935546875) ?  (b('moss') <= 5.358757972717285) ?  0.011138783540563224 :   -0.012261926242837388 :   (b('sand') <= 49.0) ?  -0.007444120706600515 :   -0.02316013209733482 :   (b('l8b4_med') <= 597.48828125) ?  (b('grass') <= 46.14527893066406) ?  (b('ndvi') <= 4264.0) ?  (b('l8b11') <= 1365.3912353515625) ?  (b('bulk') <= 141.0) ?  (b('l8b4_med') <= 573.8701782226562) ?  (b('ndvi') <= 4216.5) ?  (b('dg0_v_vv') <= -0.732243537902832) ?  -0.000690186787926524 :   0.013659936483675653 :   (b('dg0_v_vv') <= 0.5850048065185547) ?  0.022682728409347094 :   0.03793455554454833 :   (b('sand') <= 36.5) ?  -0.023258519613140738 :   -0.04618762149743952 :   (b('ndvi') <= 2931.5) ?  (b('l8b11') <= 1324.2808227539062) ?  (b('l8b4') <= 719.8872985839844) ?  0.013282527191696043 :   -0.09718398971867051 :   0.12590445395166458 :   (b('l8b5') <= 2641.568359375) ?  (b('l8b11') <= 838.6338195800781) ?  0.06920432955737062 :   0.011558219122399849 :   (b('l8b5') <= 2899.501708984375) ?  0.08817341549229733 :   0.042172713153581017 :   (b('moss') <= 5.550050735473633) ?  (b('sand') <= 34.5) ?  (b('ndvi') <= 2889.0) ?  (b('bulk') <= 153.0) ?  0.0691379423578915 :   0.05919060504664067 :   (b('ndvi') <= 3351.5) ?  -0.01793913819561438 :   0.036492305073250686 :   (b('bulk') <= 121.5) ?  0.10031689507269817 :   0.13060611104637637 :   (b('ndvi') <= 3082.5) ?  (b('l8b11') <= 1395.4365844726562) ?  0.07078501862059847 :   (b('l8b4') <= 661.4244995117188) ?  -0.09049191160255401 :   -0.03019106232304437 :   (b('moss') <= 8.800304889678955) ?  (b('dg0_v_vv') <= -0.27457618713378906) ?  -0.06736363152821917 :   -0.032371795718685814 :   (b('l8b4') <= 694.840087890625) ?  0.06073122793856406 :   0.009493864698802432 :   (b('l8b5') <= 3140.7200927734375) ?  (b('l8b5') <= 2904.5263671875) ?  (b('ndvi') <= 4472.0) ?  (b('l8b5_med') <= 2434.426513671875) ?  -0.025062215815765093 :   -0.049745360271880756 :   (b('moss') <= 0.20509709417819977) ?  0.05562839329339944 :   (b('l8b11') <= 1868.044677734375) ?  -0.001004146566869011 :   0.021947704526377676 :   (b('grass') <= 35.97805595397949) ?  (b('dg0_v_vv') <= 0.3461427688598633) ?  -0.14032597097495836 :   (b('dg0_v_vv') <= 1.243851661682129) ?  -0.11656563654256928 :   -0.1035394742595448 :   (b('l8b5') <= 2961.1390380859375) ?  -0.09748591357773609 :   (b('dg0_v_vv') <= -0.042484283447265625) ?  -0.06024364099525615 :   -0.037873589111557936 :   (b('dg0_v_vv') <= 0.7126283645629883) ?  (b('moss') <= 14.592564582824707) ?  (b('l8b5') <= 3449.541015625) ?  (b('dg0_v_vv') <= -1.6075644493103027) ?  -0.07221598204081567 :   -0.01887689046724487 :   (b('ndvi_med') <= 4321.0) ?  0.004520364808657316 :   0.04788335601893279 :   -0.09504305969143895 :   (b('dg0_v_vv') <= 1.2742810249328613) ?  (b('l8b11') <= 1102.4774780273438) ?  (b('bulk') <= 127.5) ?  0.09801358709378777 :   0.10409743298209587 :   0.07784043153490165 :   (b('ndvi') <= 4632.0) ?  0.037059117997260305 :   0.0030515544715020437 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1225.8602905273438) ?  (b('l8b5') <= 2320.642822265625) ?  (b('l8b5') <= 2085.4517822265625) ?  (b('dg0_v_vv') <= -0.7515559196472168) ?  (b('ndvi') <= 3677.0) ?  0.152133389290319 :   0.05595375507367223 :   (b('l8b4') <= 439.3976745605469) ?  0.06664543832825014 :   0.01682088510633526 :   (b('ndvi') <= 4355.5) ?  (b('l8b4') <= 545.7312469482422) ?  0.05261152467639784 :   0.09287924443537073 :   (b('ndvi_med') <= 3709.25) ?  0.012890464037169933 :   0.09153987734095781 :   (b('l8b4') <= 571.4405822753906) ?  (b('l8b4') <= 445.03187561035156) ?  (b('dg0_v_vv') <= -0.04849576950073242) ?  -0.0028586847652289638 :   0.052389386583451915 :   (b('ndvi') <= 4095.5) ?  0.01862908507094892 :   -0.012991489517784699 :   (b('l8b4') <= 651.7890014648438) ?  (b('l8b11_med') <= 1146.024658203125) ?  0.04779132858048818 :   0.0872962969300179 :   (b('ndvi') <= 2934.0) ?  0.0016362961963784922 :   0.04231630240014792 :   (b('dg0_v_vv') <= 1.4339194297790527) ?  (b('ndvi') <= 4038.5) ?  (b('l8b4') <= 524.2643127441406) ?  (b('crops') <= 6.950881481170654) ?  -0.041266535183567404 :   -0.018863571602402845 :   (b('ndvi_med') <= 3897.0) ?  0.03897434831377419 :   0.006890048477738169 :   (b('dg0_v_vv') <= 1.1116118431091309) ?  (b('l8b5') <= 3190.3148193359375) ?  -0.016115836309015006 :   -0.05842947889975828 :   (b('ndvi') <= 4892.5) ?  0.004187770142081604 :   0.000894183109975924 :   -0.06745183960407497 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('dg0_v_vv') <= -0.08648824691772461) ?  (b('crops') <= 25.986759185791016) ?  (b('dg0_v_vv') <= -0.6748180389404297) ?  0.053599806601700106 :   0.06155823808953285 :   (b('dg0_v_vv') <= -0.11155176162719727) ?  0.04389523748011796 :   0.019623834789179506 :   (b('dg0_v_vv') <= -0.0663909912109375) ?  0.10161272993164316 :   (b('l8b4') <= 226.28527069091797) ?  0.03803924775985338 :   0.06349589589101796 :   (b('ndvi_med') <= 3208.25) ?  0.11740692513777162 :   (b('ndvi') <= 4344.0) ?  (b('ndvi') <= 3502.5) ?  0.07136585219522071 :   0.057292962811675105 :   (b('l8b5_med') <= 2592.1824951171875) ?  0.09506442033886528 :   0.07854784035645139 :   (b('l8b4_med') <= 585.992919921875) ?  (b('l8b5') <= 2502.41357421875) ?  (b('ndvi') <= 2720.5) ?  (b('ndvi_med') <= 3281.75) ?  0.06144367602177203 :   0.03803232366755541 :   (b('dg0_v_vv') <= -0.9435973167419434) ?  0.053369978397693085 :   0.0845777885265168 :   (b('dg0_v_vv') <= -0.5114340782165527) ?  (b('l8b5_med') <= 3217.3944091796875) ?  0.04231259786694177 :   0.02974153142047765 :   (b('ndvi') <= 4406.5) ?  0.04536081159691774 :   0.05462690897397996 :   (b('l8b5_med') <= 2592.1824951171875) ?  (b('dg0_v_vv') <= -1.100924015045166) ?  0.06219032447167511 :   (b('dg0_v_vv') <= -0.25833988189697266) ?  0.07659396508617011 :   0.06495225426546003 :   (b('dg0_v_vv') <= -0.4750986099243164) ?  0.06511520514966884 :   (b('dg0_v_vv') <= 0.6114373207092285) ?  0.05462315854773079 :   0.047699897029746585 :   (b('dg0_v_vv') <= 1.116894245147705) ?  (b('bulk') <= 128.0) ?  (b('bulk') <= 124.5) ?  (b('l8b5') <= 4144.524169921875) ?  (b('l8b11') <= 1526.1018676757812) ?  (b('dg0_v_vv') <= 0.09142541885375977) ?  (b('l8b11') <= 1014.4495239257812) ?  -0.04326524316293974 :   0.0008997971208684361 :   (b('l8b5') <= 2922.3779296875) ?  -0.025426416910720697 :   -0.06489310641759442 :   (b('l8b11') <= 1552.2391967773438) ?  (b('ndvi') <= 3834.5) ?  0.0721471704607353 :   0.016844425620880676 :   (b('l8b5') <= 2307.4288330078125) ?  0.03809933897341792 :   -0.004019942346520789 :   (b('dg0_v_vv') <= -0.19509315490722656) ?  0.12439281653022319 :   (b('l8b5') <= 4593.343994140625) ?  0.08486970811373662 :   0.06980883185870368 :   (b('moss') <= 15.054561138153076) ?  (b('dg0_v_vv') <= 0.616490364074707) ?  (b('l8b4') <= 632.2872924804688) ?  0.016427937325093123 :   (b('dg0_v_vv') <= -2.1734914779663086) ?  -0.012018725792914409 :   -0.04148495105752518 :   (b('l8b4') <= 1139.8180847167969) ?  0.02549314147028106 :   -0.016905039216419657 :   (b('ndvi') <= 2789.5) ?  (b('l8b4') <= 919.2722473144531) ?  (b('l8b4_med') <= 866.0950317382812) ?  -0.06601813637776417 :   -0.0724708477542869 :   (b('dg0_v_vv') <= 0.22866487503051758) ?  -0.07967979539529443 :   -0.07633549497668064 :   (b('l8b4_med') <= 866.0950317382812) ?  (b('dg0_v_vv') <= 0.19718265533447266) ?  -0.05608893714917192 :   -0.06062158985492532 :   -0.016552526285486327 :   (b('dg0_v_vv') <= -0.5460472106933594) ?  (b('l8b4_med') <= 652.6907958984375) ?  (b('ndvi') <= 2804.5) ?  (b('l8b11_med') <= 1143.9313354492188) ?  (b('ndvi') <= 2481.0) ?  0.08659521320169816 :   0.022580870840170367 :   (b('dg0_v_vv') <= -1.0933570861816406) ?  0.0950455590517009 :   0.07482047946044443 :   (b('ndvi') <= 3680.0) ?  (b('dg0_v_vv') <= -0.7803606986999512) ?  0.00477623792987296 :   -0.021802172989527243 :   (b('l8b5') <= 3572.396728515625) ?  0.049054890360181395 :   -0.016229247170484958 :   (b('ndvi_med') <= 4011.0) ?  (b('l8b4') <= 1014.7101745605469) ?  (b('l8b4') <= 1008.8448791503906) ?  -0.005352673210211856 :   -0.0869400154027688 :   (b('l8b4') <= 1543.7664184570312) ?  0.013773377799544476 :   -0.0403939212049323 :   (b('ndvi') <= 4864.0) ?  (b('dg0_v_vv') <= -2.852837085723877) ?  -0.04044480957087657 :   -0.07381448920288144 :   0.011195466642130364 :   (b('l8b5_med') <= 2417.5970458984375) ?  (b('l8b5') <= 2186.8734130859375) ?  (b('ndvi_med') <= 2494.25) ?  (b('ndvi') <= 2694.5) ?  0.023280938017916558 :   0.06309121519399084 :   (b('dg0_v_vv') <= 0.8418896198272705) ?  0.0008485290619401759 :   0.10248334127987163 :   (b('l8b5_med') <= 2355.943115234375) ?  (b('l8b4') <= 689.0125427246094) ?  0.018131045703221157 :   -0.005941836437681071 :   (b('l8b11') <= 1036.6906127929688) ?  -0.16568889998621558 :   -0.032021404853211595 :   (b('ndvi') <= 4689.0) ?  (b('l8b5') <= 2449.74853515625) ?  (b('ndvi_med') <= 2598.25) ?  0.058827522388688026 :   0.025640940675745332 :   (b('bulk') <= 158.0) ?  0.01827451215097105 :   -0.008769557266226413 :   (b('l8b11_med') <= 1437.5504150390625) ?  (b('moss') <= 3.5762009620666504) ?  0.10319213713559865 :   0.06359032247522688 :   (b('ndvi') <= 4773.5) ?  0.06387378500376417 :   0.006607341468693628 :   (b('l8b4') <= 541.0198974609375) ?  (b('l8b4_med') <= 1364.1295776367188) ?  (b('grass') <= 60.25079154968262) ?  (b('l8b4_med') <= 602.3245849609375) ?  -0.07203804466678335 :   (b('l8b11') <= 980.2834777832031) ?  (b('l8b11') <= 968.330322265625) ?  0.05775473990514275 :   -0.01379156653008104 :   (b('grass') <= 50.81816482543945) ?  0.05717401607831135 :   0.1500023491646856 :   (b('ndvi') <= 3679.0) ?  (b('moss') <= 17.301149368286133) ?  (b('ndvi') <= 2531.5) ?  0.14934959726745323 :   0.06571937792140335 :   (b('l8b4') <= 458.0093688964844) ?  0.019278526807376295 :   -0.01313391322166188 :   (b('l8b11') <= 968.5401611328125) ?  (b('grass') <= 68.83333206176758) ?  0.004491682398384437 :   -0.04720563597378381 :   (b('l8b4_med') <= 717.3295288085938) ?  -0.0703822683638829 :   0.040830076388855274 :   (b('dg0_v_vv') <= 5.587636470794678) ?  (b('ndvi_med') <= 1850.25) ?  (b('l8b11') <= 917.1139221191406) ?  0.0742942786486519 :   (b('dg0_v_vv') <= 2.0925045013427734) ?  0.13376112452501865 :   0.11362094236462228 :   0.14847865656863132 :   -0.009305854856135304 :   (b('sand') <= 29.5) ?  (b('l8b5_med') <= 3068.690185546875) ?  (b('ndvi') <= 4588.5) ?  (b('ndvi_med') <= 2157.75) ?  (b('l8b5') <= 1978.7594604492188) ?  0.05937086802757328 :   -0.022196465733730974 :   (b('l8b4') <= 954.4468994140625) ?  0.06683549999837918 :   0.0377639134384008 :   (b('ndvi') <= 4620.5) ?  -0.08026726324693426 :   (b('l8b5_med') <= 2941.4141845703125) ?  0.060174332575950645 :   0.007319253191601205 :   (b('l8b5') <= 3514.3836669921875) ?  (b('l8b5_med') <= 3238.644287109375) ?  (b('dg0_v_vv') <= 1.4344077110290527) ?  0.017662662065661066 :   0.0001809124622350222 :   (b('moss') <= 3.2715134620666504) ?  0.04534236249632867 :   0.08630922070306557 :   (b('ndvi') <= 4425.5) ?  -0.09406484403891047 :   -0.09802318012645383 :   (b('ndvi') <= 2362.5) ?  (b('grass') <= 48.13426971435547) ?  0.1732723721261963 :   (b('grass') <= 81.95861053466797) ?  (b('moss') <= 10.0180983543396) ?  0.08904351389652619 :   0.06050232650815861 :   (b('dg0_v_vv') <= 2.5961101055145264) ?  0.03949217328788025 :   0.0008013080525116623 :   (b('l8b5_med') <= 2411.1597900390625) ?  (b('dg0_v_vv') <= 2.8522109985351562) ?  (b('l8b11_med') <= 2336.0272216796875) ?  -0.00976046205706983 :   0.12840134344994736 :   (b('l8b11') <= 1180.498291015625) ?  -0.06100812956859002 :   0.036895677179011155 :   (b('moss') <= 0.48118618316948414) ?  (b('ndvi') <= 2632.0) ?  -0.03282358260351225 :   0.016256404873136693 :   (b('l8b4_med') <= 1893.3539428710938) ?  0.03392817755320689 :   -0.08000948738193278 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi') <= 2321.5) ?  (b('dg0_v_vv') <= 0.9694757461547852) ?  (b('ndvi_med') <= 2985.5) ?  (b('moss') <= 8.823604106903076) ?  (b('moss') <= 8.321518421173096) ?  (b('l8b11_med') <= 1765.5895385742188) ?  (b('moss') <= 7.05361008644104) ?  (b('moss') <= 6.545327425003052) ?  (b('ndvi_med') <= 2921.0) ?  (b('sand') <= 20.5) ?  0.051113333271566475 :   0.004183544379103124 :   (b('ndvi') <= 2233.5) ?  0.0658444274465669 :   0.035474886516472556 :   (b('ndvi') <= 2013.0) ?  (b('dg0_v_vv') <= 0.6199007034301758) ?  0.07553916355397972 :   0.03654825737255475 :   (b('l8b11') <= 1519.3341674804688) ?  0.22623991939498864 :   0.17020335019876212 :   (b('ndvi') <= 1940.5) ?  (b('dg0_v_vv') <= 0.06161832809448242) ?  (b('l8b11_med') <= 1644.19921875) ?  0.010876704750696572 :   -0.01786668119935999 :   (b('dg0_v_vv') <= 0.11005687713623047) ?  0.10194986435694668 :   0.0019543147471324795 :   (b('dg0_v_vv') <= 0.5890116691589355) ?  (b('l8b4') <= 698.6427307128906) ?  -0.015830219639762975 :   -0.04241974168306026 :   (b('l8b4') <= 956.169677734375) ?  0.012571070634654055 :   0.10001292519474049 :   (b('bulk') <= 131.0) ?  (b('l8b4_med') <= 848.7766723632812) ?  (b('ndvi_med') <= 2079.0) ?  -0.012943667315619078 :   (b('l8b11') <= 1408.0502319335938) ?  -0.005130829704861062 :   0.0016471363693640328 :   (b('l8b4') <= 1309.2120361328125) ?  (b('l8b11') <= 2531.9183349609375) ?  0.045290532075742206 :   0.0739756331477126 :   0.007894549084202174 :   (b('l8b5_med') <= 2500.130126953125) ?  (b('l8b11_med') <= 1970.0264282226562) ?  (b('sand') <= 39.5) ?  -0.03275790844039382 :   -0.01487559788499043 :   (b('ndvi') <= 1232.5) ?  -0.03049913477300986 :   -0.007833142641806878 :   (b('l8b4') <= 1713.463134765625) ?  (b('l8b4_med') <= 1230.4055786132812) ?  -0.011070963047925377 :   0.002588540063161004 :   (b('l8b11_med') <= 2629.3411865234375) ?  -0.006089229305900948 :   -0.015240356994095337 :   (b('l8b5') <= 2085.9556884765625) ?  (b('l8b4_med') <= 1484.9597778320312) ?  (b('ndvi') <= 1403.0) ?  0.14060159963762514 :   (b('l8b5') <= 2034.8182373046875) ?  (b('bulk') <= 139.0) ?  0.08123330624022201 :   0.07813409304322091 :   0.10294237310961057 :   (b('l8b4') <= 1512.1868896484375) ?  (b('dg0_v_vv') <= -0.17928123474121094) ?  (b('moss') <= 8.51813268661499) ?  0.04751984010400928 :   0.028178711358053193 :   -0.0038485855361567595 :   -0.024994412742973843 :   (b('dg0_v_vv') <= -1.4593729972839355) ?  (b('ndvi') <= 1701.5) ?  (b('ndvi') <= 1336.5) ?  0.004911121428815501 :   (b('dg0_v_vv') <= -1.581005573272705) ?  0.06971285486853664 :   0.10960822087401778 :   (b('ndvi') <= 2168.5) ?  (b('ndvi') <= 2024.0) ?  -0.019542615208542745 :   -0.047320909809677264 :   0.06516934672501504 :   (b('ndvi') <= 1234.5) ?  (b('ndvi') <= 1184.0) ?  (b('ndvi') <= 1161.0) ?  0.02001891933961755 :   -0.05620430883830868 :   (b('l8b11') <= 3006.3726806640625) ?  0.1132444715463257 :   -0.00462596391659006 :   (b('l8b11') <= 2014.8851928710938) ?  (b('bulk') <= 139.0) ?  -0.043703943048697215 :   0.03437401245171922 :   (b('l8b4') <= 1905.257080078125) ?  -0.030673847253169417 :   0.0010596087215027073 :   (b('l8b5') <= 2040.5491943359375) ?  (b('moss') <= 12.857846736907959) ?  (b('l8b11') <= 1846.4996948242188) ?  (b('moss') <= 11.825395584106445) ?  (b('l8b11') <= 1071.207763671875) ?  -0.05378885525841992 :   (b('dg0_v_vv') <= 0.06593465805053711) ?  0.02569647239904427 :   0.09167765066030857 :   (b('bulk') <= 150.0) ?  (b('l8b5') <= 1832.6546020507812) ?  0.18583797458431378 :   0.20046996614345292 :   (b('dg0_v_vv') <= -0.21834468841552734) ?  0.07483008917205178 :   0.062371691711051475 :   (b('l8b4_med') <= 1289.7028198242188) ?  (b('moss') <= 11.158973217010498) ?  (b('l8b5') <= 1886.3258056640625) ?  -0.04906866820988576 :   -0.03365654443870298 :   (b('l8b4_med') <= 1038.4496765136719) ?  0.010065908865424467 :   -0.018241070370228346 :   (b('dg0_v_vv') <= 0.1305837631225586) ?  (b('dg0_v_vv') <= -0.5910954475402832) ?  0.0012857949537687166 :   0.025758867191120347 :   (b('crops') <= 10.164965629577637) ?  0.04032940554679389 :   0.16793488499234516 :   (b('l8b4_med') <= 1216.4403076171875) ?  (b('moss') <= 21.596078872680664) ?  (b('l8b5') <= 1445.3436889648438) ?  (b('dg0_v_vv') <= -0.5540766716003418) ?  -0.0023756583637085014 :   -0.02813175112675257 :   (b('bulk') <= 133.5) ?  -0.026638968978521933 :   0.006902098475112324 :   (b('moss') <= 31.57449436187744) ?  (b('l8b11') <= 1629.18505859375) ?  -0.04584168159428294 :   -0.02465233890369355 :   (b('ndvi_med') <= 1607.0) ?  -0.02081370774836464 :   -0.0002034966816361923 :   (b('ndvi') <= 1625.5) ?  (b('sand') <= 65.5) ?  (b('ndvi') <= 1040.0) ?  0.01608734809861945 :   -0.00651765996454383 :   (b('l8b4_med') <= 1431.794677734375) ?  0.006469597121184112 :   0.19336875396019565 :   (b('dg0_v_vv') <= -0.4184994697570801) ?  (b('sand') <= 39.0) ?  0.013760974338786683 :   -0.034858957127921905 :   (b('dg0_v_vv') <= -0.03199148178100586) ?  0.1834718110396543 :   0.14151128818403658 :   (b('ndvi_med') <= 1568.5) ?  (b('l8b11') <= 1476.4822387695312) ?  (b('ndvi') <= 1287.0) ?  0.08554482613476844 :   (b('ndvi') <= 1920.0) ?  0.11863620725079263 :   0.13311157739495488 :   (b('ndvi') <= 1281.5) ?  (b('crops') <= 81.16990280151367) ?  (b('l8b11_med') <= 2369.3162841796875) ?  -0.026786079612256063 :   -0.01297096152859822 :   (b('ndvi') <= 1233.0) ?  0.015365347144602994 :   0.11816454934835063 :   (b('l8b5_med') <= 3322.7132568359375) ?  (b('l8b5') <= 2235.470703125) ?  -0.015368237340202805 :   -0.0002514803544960271 :   (b('l8b5') <= 4218.92138671875) ?  -0.021180115436814525 :   0.007232340819186703 :   (b('l8b11') <= 1382.740966796875) ?  (b('l8b11') <= 1069.0128173828125) ?  (b('dg0_v_vv') <= -0.18716812133789062) ?  0.11574292522236632 :   0.007562429357732908 :   (b('l8b11_med') <= 2161.71044921875) ?  (b('grass') <= 31.318479537963867) ?  -0.06875037780502606 :   -0.03655078323095993 :   0.08004456739643895 :   (b('ndvi_med') <= 2902.75) ?  (b('sand') <= 33.0) ?  (b('dg0_v_vv') <= 0.2810359001159668) ?  0.006822422719517269 :   0.062322580288547404 :   (b('bulk') <= 120.0) ?  -0.06196729762087801 :   -0.01823320278693624 :   (b('dg0_v_vv') <= -1.111867904663086) ?  (b('l8b4_med') <= 811.2398986816406) ?  -0.06842664535307809 :   -0.002879681505754533 :   (b('l8b5_med') <= 2868.5509033203125) ?  0.01593135958431575 :   0.1105003074416799 :   (b('ndvi_med') <= 2987.25) ?  (b('dg0_v_vv') <= 0.2475438117980957) ?  (b('ndvi') <= 2056.5) ?  (b('l8b11') <= 2005.752685546875) ?  0.12073540030869878 :   0.12582338364651635 :   (b('l8b4') <= 1064.5002136230469) ?  0.1587950053593264 :   0.14732790894883308 :   (b('ndvi') <= 2082.0) ?  0.053497620361702525 :   0.03867615931537771 :   (b('moss') <= 7.5638086795806885) ?  (b('dg0_v_vv') <= -3.4830188751220703) ?  (b('l8b11_med') <= 1530.5140991210938) ?  (b('dg0_v_vv') <= -4.921833515167236) ?  0.022306709084651682 :   (b('l8b5_med') <= 3474.926513671875) ?  (b('bulk') <= 137.0) ?  -0.0024506168550221252 :   -0.006733351958882666 :   (b('dg0_v_vv') <= -4.308004856109619) ?  0.008306717965743754 :   0.0034233811932224134 :   (b('bulk') <= 138.5) ?  -0.02970699179654898 :   (b('dg0_v_vv') <= -3.9551162719726562) ?  -0.06566905601683787 :   -0.06650322926305785 :   (b('ndvi') <= 1816.0) ?  (b('ndvi_med') <= 3440.75) ?  (b('l8b5') <= 3901.105224609375) ?  (b('dg0_v_vv') <= -2.3658299446105957) ?  0.06275383488861358 :   0.012313777733577997 :   0.08849693092938332 :   (b('dg0_v_vv') <= -0.449307918548584) ?  (b('l8b5_med') <= 2863.166748046875) ?  0.06327408463130846 :   0.0974867923686203 :   (b('dg0_v_vv') <= 0.6573052406311035) ?  0.024964187238836876 :   0.10343786553780795 :   (b('sand') <= 18.0) ?  (b('sand') <= 15.5) ?  0.0703150490453962 :   0.12428156845759869 :   (b('grass') <= 19.251008987426758) ?  (b('ndvi_med') <= 3391.25) ?  0.016503034196055647 :   -0.017726660516787042 :   (b('ndvi') <= 1897.5) ?  -0.01940907747259239 :   0.030421970629026632 :   (b('l8b4') <= 758.2536315917969) ?  (b('l8b11') <= 1674.8615112304688) ?  (b('moss') <= 15.207282066345215) ?  (b('l8b4_med') <= 694.1253356933594) ?  (b('l8b5') <= 2774.484130859375) ?  -0.011591617598496546 :   0.05793606081648203 :   (b('ndvi_med') <= 3318.5) ?  0.03260633348148945 :   0.009776013565496747 :   0.11061900630815086 :   -0.04708796364507792 :   (b('l8b4') <= 1722.0333862304688) ?  (b('sand') <= 38.0) ?  (b('ndvi') <= 1922.0) ?  (b('dg0_v_vv') <= -0.10828256607055664) ?  0.0009307860313338086 :   0.034809679891367386 :   (b('l8b5_med') <= 2946.6160888671875) ?  -0.014048583191954981 :   -0.03236219379847639 :   (b('dg0_v_vv') <= 0.6531004905700684) ?  (b('l8b4') <= 834.520263671875) ?  -0.0299639314100491 :   -0.06289594975534847 :   0.06759835101876228 :   0.10464657478613953 :   (b('l8b11_med') <= 3076.66943359375) ?  (b('moss') <= 26.208369255065918) ?  (b('moss') <= 24.824777603149414) ?  (b('dg0_v_vv') <= 2.3763551712036133) ?  (b('l8b5_med') <= 2164.5704345703125) ?  (b('l8b11_med') <= 2477.956787109375) ?  (b('l8b11_med') <= 1666.5599365234375) ?  (b('l8b11') <= 1098.372802734375) ?  -0.036527732493873165 :   0.035650297413395246 :   (b('l8b11_med') <= 1945.9461059570312) ?  -0.03416143042184935 :   0.015243823829403194 :   (b('dg0_v_vv') <= 1.6870465278625488) ?  0.14552678390880452 :   (b('l8b11_med') <= 2514.0128173828125) ?  0.10316715455853365 :   0.11043237434821895 :   (b('crops') <= 30.22932243347168) ?  (b('l8b11') <= 892.42236328125) ?  (b('l8b4') <= 426.6035919189453) ?  -0.04977166507590472 :   -0.13060696858976314 :   (b('dg0_v_vv') <= 0.9705376625061035) ?  0.15318082019870885 :   -0.009104658393307954 :   (b('moss') <= 4.868765592575073) ?  (b('grass') <= 11.044764041900635) ?  0.005018391583133145 :   0.034618108210787917 :   (b('ndvi') <= 1006.5) ?  -0.06383018993949513 :   -0.00040722912549119263 :   (b('l8b5') <= 1948.9157104492188) ?  (b('bulk') <= 123.5) ?  (b('l8b11') <= 1119.6739501953125) ?  -0.07234842484037443 :   (b('ndvi_med') <= 1710.0) ?  -0.045548287417517125 :   -0.034948195096768 :   (b('ndvi') <= 2110.0) ?  (b('ndvi_med') <= 1591.75) ?  0.019140999673765233 :   0.035451757569623946 :   (b('dg0_v_vv') <= 4.210267543792725) ?  0.03834291053006632 :   0.11098888862421366 :   (b('sand') <= 43.5) ?  (b('bulk') <= 144.5) ?  (b('crops') <= 59.41643714904785) ?  -0.007843833666914803 :   0.025841681574505535 :   (b('grass') <= 69.01767349243164) ?  0.020178129561449792 :   0.07561442643048866 :   (b('l8b11') <= 2781.6649169921875) ?  (b('sand') <= 53.0) ?  -0.012123693229299973 :   0.00912824531752549 :   (b('ndvi') <= 1296.5) ?  -0.0003429103091199509 :   0.03154651380291669 :   (b('l8b4_med') <= 1434.68408203125) ?  (b('ndvi_med') <= 2150.75) ?  (b('l8b4_med') <= 1292.9146118164062) ?  (b('ndvi_med') <= 1450.75) ?  0.07359017937117518 :   0.06799375753737821 :   (b('dg0_v_vv') <= 3.596621036529541) ?  (b('dg0_v_vv') <= 2.016721248626709) ?  0.08961836134477208 :   0.1085014339573766 :   0.07315104481870505 :   0.15628678968167453 :   (b('l8b11_med') <= 2378.07421875) ?  (b('ndvi') <= 1261.0) ?  -0.009202611303884195 :   (b('l8b11') <= 2574.059326171875) ?  0.011673890423865907 :   0.008295884030113415 :   0.07198889106586026 :   (b('l8b4_med') <= 1068.3740844726562) ?  (b('l8b5') <= 1735.3119506835938) ?  (b('l8b5_med') <= 1784.8324584960938) ?  (b('l8b5_med') <= 1755.5365600585938) ?  (b('l8b4') <= 1234.8936157226562) ?  -0.07787901181992338 :   -0.08056354998987664 :   -0.08662000181134276 :   (b('grass') <= 66.89338874816895) ?  (b('l8b4') <= 892.0914611816406) ?  -0.04363940740893571 :   -0.018535196273035848 :   -0.0641342048348979 :   (b('ndvi_med') <= 1779.75) ?  (b('l8b5') <= 1803.3695068359375) ?  (b('l8b5') <= 1740.5905151367188) ?  -0.005644287030727363 :   (b('ndvi') <= 1747.0) ?  0.005963264119211206 :   0.0075113371269434626 :   (b('dg0_v_vv') <= 1.3383855819702148) ?  (b('l8b11') <= 1989.5497436523438) ?  0.040397426803574846 :   -0.023006400187230523 :   (b('l8b5') <= 2187.371826171875) ?  -0.032853697582525564 :   -0.054875106979450604 :   0.09880023560782122 :   (b('bulk') <= 154.0) ?  (b('l8b4') <= 1147.3919677734375) ?  (b('l8b4') <= 1146.73681640625) ?  (b('l8b11_med') <= 1940.4482421875) ?  (b('grass') <= 60.64468955993652) ?  0.03638609446873381 :   0.04988466876240899 :   (b('l8b5') <= 1831.4260864257812) ?  0.018470382396558613 :   -0.007602488805443374 :   0.09668391473104093 :   (b('dg0_v_vv') <= 1.127265214920044) ?  (b('l8b11_med') <= 2944.6937255859375) ?  (b('l8b5_med') <= 2403.3905029296875) ?  -0.020717960453774373 :   0.02208108190258375 :   0.11962691471931014 :   (b('l8b5') <= 1977.0068969726562) ?  (b('l8b5') <= 1635.0026245117188) ?  0.05616345824202218 :   -0.04647712450172223 :   (b('bulk') <= 146.5) ?  0.0081359577434669 :   -0.025358613845160437 :   (b('dg0_v_vv') <= 1.1980962753295898) ?  0.14783883232037248 :   0.04683601912916732 :   (b('grass') <= 43.20292663574219) ?  (b('l8b11') <= 3343.7020263671875) ?  (b('ndvi') <= 1037.0) ?  (b('dg0_v_vv') <= 1.0575742721557617) ?  -0.06335798687917638 :   (b('ndvi_med') <= 1166.0) ?  (b('dg0_v_vv') <= 1.260735034942627) ?  (b('l8b11') <= 2886.666748046875) ?  0.05378905347051194 :   0.03504505208677036 :   (b('l8b5') <= 2999.02099609375) ?  -0.0013154256919944152 :   0.018243623202937733 :   (b('l8b11') <= 3087.8839111328125) ?  (b('l8b4') <= 2078.77734375) ?  -0.01484900138379907 :   -0.008668070176640813 :   (b('l8b5_med') <= 3219.7747802734375) ?  0.008073156560376417 :   0.002132253719193039 :   (b('grass') <= 36.800336837768555) ?  (b('l8b5') <= 2120.3907470703125) ?  (b('sand') <= 51.5) ?  0.04385170875846969 :   0.012861854369875775 :   (b('l8b5_med') <= 3322.7132568359375) ?  (b('l8b4') <= 1825.0419921875) ?  -0.012225264548279893 :   -0.027880861024605096 :   (b('ndvi') <= 1389.5) ?  -0.032035372799511366 :   0.0005692457793833746 :   0.0582293779473457 :   (b('l8b5_med') <= 3130.3966064453125) ?  (b('sand') <= 52.5) ?  -0.03478351287980386 :   (b('l8b4_med') <= 1992.8049926757812) ?  -0.05765973005402632 :   -0.06168421821380529 :   (b('l8b4') <= 1905.058349609375) ?  (b('ndvi_med') <= 1716.5) ?  (b('l8b11') <= 3641.236572265625) ?  -0.048475055504420064 :   -0.044673242404193535 :   (b('crops') <= 41.22767448425293) ?  -0.029977978220837663 :   -0.03819534666567243 :   (b('l8b4') <= 2067.5098266601562) ?  (b('l8b5_med') <= 3217.6090087890625) ?  -0.05122366559854473 :   (b('ndvi_med') <= 1482.0) ?  -0.024205671179761266 :   0.0046040494115218945 :   (b('dg0_v_vv') <= 2.972266674041748) ?  (b('l8b4') <= 2182.2012939453125) ?  -0.02935777945155287 :   -0.020537325573653824 :   (b('l8b11') <= 3883.044677734375) ?  0.00233740275093521 :   -0.01930235066193942 :   (b('l8b5') <= 2885.4024658203125) ?  (b('l8b11_med') <= 3419.625) ?  0.02279458157735112 :   0.003720107534705433 :   (b('moss') <= 6.990653991699219) ?  0.06794220013184193 :   0.05285630902786215 :   (b('l8b4_med') <= 658.7252502441406) ?  (b('grass') <= 46.14527893066406) ?  (b('ndvi_med') <= 4408.5) ?  (b('l8b4') <= 726.9224243164062) ?  (b('sand') <= 36.5) ?  (b('dg0_v_vv') <= -0.20614027976989746) ?  (b('ndvi') <= 2478.5) ?  (b('ndvi') <= 2424.5) ?  (b('ndvi_med') <= 2906.5) ?  0.06622302843296562 :   -0.02342528268087407 :   (b('dg0_v_vv') <= -0.36102867126464844) ?  0.08273749967618821 :   0.05422502649440999 :   (b('l8b5') <= 3347.6810302734375) ?  (b('sand') <= 29.0) ?  -0.04690284768205009 :   0.0018256710051205373 :   (b('l8b4_med') <= 587.9795837402344) ?  0.014995236383406427 :   0.07414451066090973 :   (b('dg0_v_vv') <= -0.12671566009521484) ?  (b('l8b4') <= 439.1753387451172) ?  (b('l8b4_med') <= 566.6458129882812) ?  0.0027869248111947464 :   0.02158801096760618 :   (b('l8b11') <= 1187.52392578125) ?  0.11400502536315749 :   0.04892047932506451 :   (b('ndvi') <= 4906.0) ?  (b('ndvi_med') <= 4364.5) ?  0.01956022172606019 :   -0.042823833656728316 :   (b('l8b5_med') <= 3886.60986328125) ?  0.0670812798637075 :   -0.02598488561722398 :   (b('sand') <= 74.5) ?  (b('ndvi') <= 2335.5) ?  (b('ndvi_med') <= 2072.25) ?  0.16851187066517825 :   0.09051908154235916 :   (b('ndvi_med') <= 4053.25) ?  (b('ndvi') <= 4879.5) ?  -0.005503615545354916 :   -0.03847739826607512 :   (b('ndvi') <= 4149.0) ?  0.020665168102911288 :   -0.0036197053825796433 :   (b('l8b11') <= 1033.1356811523438) ?  (b('l8b5_med') <= 2945.5799560546875) ?  (b('l8b11') <= 702.65771484375) ?  -0.10340075654927823 :   -0.011576035518434856 :   (b('l8b4') <= 407.5164337158203) ?  0.02244139539498955 :   -0.007320445940399679 :   (b('dg0_v_vv') <= -2.133314609527588) ?  (b('dg0_v_vv') <= -4.293484210968018) ?  -0.061346021285124896 :   0.004471158613657833 :   (b('l8b11') <= 1536.97119140625) ?  0.046900333061609435 :   0.023935329195714942 :   (b('l8b4_med') <= 436.96240234375) ?  (b('dg0_v_vv') <= 1.3574450016021729) ?  (b('ndvi_med') <= 3840.5) ?  0.10217378969694535 :   (b('l8b5_med') <= 3658.046875) ?  (b('dg0_v_vv') <= 0.17258596420288086) ?  0.027000133838291793 :   0.01917342641700494 :   (b('dg0_v_vv') <= -1.0026535987854004) ?  0.02089637030772175 :   0.06305068004976735 :   (b('ndvi') <= 3960.0) ?  -0.0012598029425825719 :   -0.04552739391349461 :   (b('l8b11') <= 1197.2069091796875) ?  0.09009196296595345 :   (b('l8b5') <= 1921.1044311523438) ?  (b('ndvi_med') <= 3513.75) ?  0.05000636396746805 :   (b('dg0_v_vv') <= -0.612983226776123) ?  0.039165749240065675 :   0.021396812083561822 :   (b('l8b11_med') <= 1085.2537231445312) ?  (b('dg0_v_vv') <= 1.1033577919006348) ?  -0.03004880945761312 :   0.026462400629510614 :   (b('l8b4') <= 732.9578247070312) ?  -0.028474309914570173 :   -0.0026318848008981834 :   (b('l8b4') <= 933.4629821777344) ?  (b('l8b11') <= 1211.626953125) ?  (b('ndvi') <= 4458.0) ?  (b('l8b4_med') <= 585.4060668945312) ?  (b('ndvi') <= 4322.5) ?  (b('l8b11_med') <= 1090.6433715820312) ?  0.0011581728041913375 :   -0.008150242902700944 :   (b('crops') <= 15.85194206237793) ?  -0.04477082424469267 :   -0.029453649145104427 :   (b('dg0_v_vv') <= -2.6878747940063477) ?  0.00978597480208307 :   (b('dg0_v_vv') <= -0.08533143997192383) ?  0.04591930951112289 :   0.02531579460121468 :   (b('dg0_v_vv') <= 0.39916086196899414) ?  (b('l8b4') <= 513.3248901367188) ?  (b('dg0_v_vv') <= -0.7755880355834961) ?  0.015442628477204762 :   0.0651832134460198 :   -0.04715483553136407 :   (b('l8b5') <= 4367.649169921875) ?  (b('ndvi') <= 4811.5) ?  0.0882961031501804 :   0.12519703722535752 :   (b('l8b11_med') <= 1198.20263671875) ?  0.06570380173357859 :   0.046710424641133064 :   (b('sand') <= 60.0) ?  (b('l8b4') <= 643.4403686523438) ?  (b('l8b11') <= 1320.8679809570312) ?  -0.032505222078990426 :   -0.007599269591952076 :   (b('ndvi') <= 4548.5) ?  (b('l8b11') <= 1389.1729736328125) ?  -0.08465835963501941 :   -0.07239890011624811 :   -0.05761810410105393 :   (b('ndvi') <= 4739.5) ?  (b('l8b11') <= 1660.2769775390625) ?  (b('ndvi_med') <= 4638.5) ?  0.008319921844898557 :   -0.011483424994747063 :   (b('dg0_v_vv') <= 0.47589683532714844) ?  0.06313867928630033 :   0.027289402464627254 :   (b('ndvi') <= 4942.0) ?  (b('dg0_v_vv') <= -0.4678936004638672) ?  -0.015984087638112474 :   -0.03802501019597829 :   (b('dg0_v_vv') <= -0.8677473068237305) ?  0.035290428474658325 :   0.0187227061186969 :   (b('grass') <= 26.803903579711914) ?  (b('dg0_v_vv') <= 0.9692111015319824) ?  (b('l8b11_med') <= 1259.6744995117188) ?  0.10465871081963074 :   0.11764250484483563 :   0.14918786601149747 :   0.019752934073739903 :   (b('crops') <= 19.45354127883911) ?  (b('l8b11_med') <= 1274.3564453125) ?  (b('ndvi') <= 2943.5) ?  (b('bulk') <= 113.5) ?  (b('moss') <= 0.4536082446575165) ?  -0.017763282489534765 :   -0.03667377653002485 :   (b('bulk') <= 150.5) ?  (b('ndvi') <= 2478.0) ?  (b('dg0_v_vv') <= 0.4135909080505371) ?  0.046771425455389225 :   0.008997297369771149 :   (b('grass') <= 58.827531814575195) ?  0.08457481952702407 :   0.047733292802161445 :   (b('l8b4') <= 559.8279571533203) ?  (b('l8b4') <= 418.5820617675781) ?  0.023364990949229558 :   0.04134414244476958 :   (b('l8b5') <= 2404.5439453125) ?  -0.021544331419368325 :   -0.005344868814931403 :   (b('dg0_v_vv') <= 1.0087261199951172) ?  (b('sand') <= 76.5) ?  (b('crops') <= 9.933524370193481) ?  (b('l8b4_med') <= 455.5856628417969) ?  0.08257258035844944 :   0.01346317417069884 :   (b('l8b5') <= 3249.9671630859375) ?  0.0809362483572579 :   0.026338017518948455 :   (b('ndvi_med') <= 3910.5) ?  (b('dg0_v_vv') <= -0.3737149238586426) ?  -0.09076026814776772 :   -0.09574073353109305 :   -0.10201652232051583 :   (b('ndvi') <= 3390.5) ?  (b('l8b11') <= 1582.8406982421875) ?  (b('grass') <= 58.827531814575195) ?  -0.013508151986799222 :   0.017878912922578677 :   0.08671538193408879 :   (b('l8b4_med') <= 642.8293762207031) ?  (b('l8b11') <= 954.4361572265625) ?  0.03581444627119405 :   0.08068756815226383 :   (b('dg0_v_vv') <= 1.105506420135498) ?  0.11707170902448666 :   -0.0020241766223302177 :   (b('moss') <= 10.665639638900757) ?  (b('l8b5') <= 3386.314208984375) ?  (b('dg0_v_vv') <= -0.8108916282653809) ?  (b('l8b11') <= 1377.208740234375) ?  (b('ndvi') <= 4286.5) ?  -0.04488269206193336 :   -0.031054179564755757 :   (b('l8b5') <= 2632.0823974609375) ?  -0.005399708449698115 :   0.015673052249167208 :   (b('dg0_v_vv') <= -0.6039056777954102) ?  (b('l8b11') <= 1823.1958618164062) ?  0.014384302013927505 :   -0.001802954930591405 :   (b('l8b5') <= 2196.3948974609375) ?  -0.012393601397900106 :   0.0010649598980106312 :   (b('dg0_v_vv') <= 0.31055498123168945) ?  0.004973220769167688 :   (b('l8b11') <= 1304.4785766601562) ?  (b('l8b4') <= 476.93682861328125) ?  -0.019483518012108514 :   0.0008047647989783274 :   (b('l8b4') <= 618.8165893554688) ?  -0.048637618244862096 :   -0.04271391656060618 :   (b('l8b11') <= 1260.1450805664062) ?  -0.06070665564366748 :   (b('l8b5') <= 2195.4736328125) ?  -0.03658234025472494 :   (b('l8b5_med') <= 2583.02294921875) ?  (b('l8b5') <= 2507.557861328125) ?  -0.025170264234663245 :   -0.02194363177096874 :   (b('l8b11') <= 1295.4886474609375) ?  -0.03199137999945098 :   -0.032889582633563036 :   (b('l8b5') <= 2478.9398193359375) ?  (b('l8b5') <= 2455.188720703125) ?  (b('dg0_v_vv') <= -1.968216896057129) ?  (b('l8b11') <= 1403.2102661132812) ?  (b('ndvi_med') <= 3405.0) ?  (b('l8b5') <= 1941.3568115234375) ?  0.04518768985225369 :   0.007280910234806454 :   (b('l8b4') <= 630.7738952636719) ?  -0.01349168992199537 :   0.007387469509201984 :   (b('ndvi_med') <= 3442.5) ?  0.06840459462142362 :   0.07734561434372805 :   (b('ndvi') <= 2895.5) ?  (b('ndvi_med') <= 3438.5) ?  0.06529619438008916 :   0.06964631139678185 :   (b('dg0_v_vv') <= 1.0336246490478516) ?  (b('dg0_v_vv') <= 0.768944263458252) ?  0.02975081118372638 :   -0.009849909636886284 :   (b('dg0_v_vv') <= 1.4419317245483398) ?  0.06139814398720342 :   0.04014225694102269 :   (b('ndvi_med') <= 3405.0) ?  (b('ndvi') <= 3323.0) ?  -0.041855752259144174 :   -0.06277921558048533 :   0.006879212209092378 :   (b('l8b4') <= 544.4970397949219) ?  (b('ndvi') <= 2667.5) ?  (b('l8b11') <= 1187.7406616210938) ?  (b('l8b5') <= 3051.2576904296875) ?  0.058725180960133905 :   0.05696239168247874 :   (b('l8b4') <= 454.26145935058594) ?  (b('dg0_v_vv') <= 0.1822962760925293) ?  0.0627393932357411 :   0.07000018357265383 :   (b('dg0_v_vv') <= 0.46289968490600586) ?  0.07107374987348564 :   0.07427033659897009 :   (b('ndvi') <= 3537.0) ?  (b('l8b4_med') <= 537.5668334960938) ?  (b('grass') <= 53.71771430969238) ?  0.06320669388473138 :   0.06566303572816823 :   (b('l8b5_med') <= 2931.880126953125) ?  0.03425709144257729 :   0.046842041017347937 :   (b('l8b11') <= 1176.05859375) ?  (b('dg0_v_vv') <= 1.567528247833252) ?  0.05278145430244164 :   0.07052383098264381 :   (b('ndvi_med') <= 3682.75) ?  0.0691014495952489 :   0.05697851709617678 :   (b('l8b4_med') <= 573.9899597167969) ?  (b('bulk') <= 118.5) ?  (b('bulk') <= 106.5) ?  (b('dg0_v_vv') <= -0.5640087127685547) ?  0.026247375436480445 :   0.034123437131871846 :   (b('l8b5_med') <= 3089.992919921875) ?  0.040186419284910145 :   0.05158158080361413 :   (b('l8b11') <= 1292.0343017578125) ?  (b('ndvi') <= 3958.0) ?  0.017320397114710007 :   0.02656105219266089 :   (b('ndvi_med') <= 3311.0) ?  0.03437819954250907 :   0.02719510243198686 :   (b('dg0_v_vv') <= -0.5347347259521484) ?  (b('l8b4') <= 577.5890808105469) ?  0.07152100603194433 :   (b('dg0_v_vv') <= -1.332906723022461) ?  0.048065473148720894 :   0.015377847046276774 :   (b('ndvi') <= 3326.0) ?  (b('l8b11') <= 1376.3753662109375) ?  0.08403007657070746 :   0.059218298814762815 :   (b('dg0_v_vv') <= 2.061504364013672) ?  0.049407821981599334 :   0.06579768190850754 :   (b('bulk') <= 137.5) ?  (b('crops') <= 59.452327728271484) ?  (b('l8b5_med') <= 1700.2011108398438) ?  (b('l8b11') <= 1631.9219360351562) ?  (b('dg0_v_vv') <= 0.34363412857055664) ?  0.02061607074557631 :   0.06929304078602183 :   0.20924432144425562 :   (b('ndvi_med') <= 3318.5) ?  (b('ndvi') <= 2780.5) ?  (b('l8b11_med') <= 2249.6854248046875) ?  (b('moss') <= 19.307927131652832) ?  (b('ndvi') <= 2390.0) ?  -0.018815665261933283 :   0.0012039336245846745 :   (b('sand') <= 40.5) ?  -0.056423851258356646 :   -0.023331387630048488 :   (b('l8b4') <= 674.2841796875) ?  0.21188920881275342 :   (b('l8b5') <= 2266.7244262695312) ?  -0.014081947105508676 :   0.06644490216328561 :   (b('ndvi') <= 2796.5) ?  (b('ndvi') <= 2785.0) ?  (b('l8b11_med') <= 2150.8240966796875) ?  -0.032500326873959595 :   -0.053786813109502195 :   (b('dg0_v_vv') <= 0.7058005332946777) ?  -0.06693598772874179 :   -0.08988628844951443 :   (b('l8b11_med') <= 2002.45849609375) ?  (b('grass') <= 50.95400047302246) ?  -0.012089849843977719 :   0.0025673278189720216 :   (b('l8b11') <= 2716.8878173828125) ?  -0.034383014823199325 :   0.0378878522970996 :   (b('l8b11_med') <= 1347.6688232421875) ?  (b('l8b11') <= 855.1739196777344) ?  -0.04997218627937566 :   (b('l8b4_med') <= 746.6159973144531) ?  (b('l8b4') <= 726.9024353027344) ?  0.041164398762010584 :   -0.01526080768018559 :   (b('l8b11') <= 1388.5983276367188) ?  0.053698884485250585 :   0.03169226134315817 :   (b('l8b5_med') <= 3340.3853759765625) ?  (b('ndvi_med') <= 4106.5) ?  (b('l8b11') <= 789.1821899414062) ?  0.12154013128766672 :   5.3100502812630375e-05 :   (b('l8b5') <= 2402.41064453125) ?  0.004844681973101061 :   -0.02541944320467298 :   (b('l8b4') <= 1622.350341796875) ?  (b('l8b4') <= 741.1357116699219) ?  -0.0802865458279065 :   -0.04216210499979909 :   0.04842441418397983 :   (b('dg0_v_vv') <= 0.8434832096099854) ?  (b('sand') <= 22.0) ?  (b('ndvi') <= 2712.0) ?  (b('ndvi_med') <= 3351.25) ?  (b('l8b11_med') <= 1295.4417114257812) ?  (b('dg0_v_vv') <= -0.42728519439697266) ?  0.018079821929385037 :   -0.0332064924209389 :   (b('dg0_v_vv') <= -1.3548297882080078) ?  -0.032952106644624224 :   -0.06806953556252796 :   (b('l8b11') <= 650.7136840820312) ?  0.023459330139980028 :   (b('dg0_v_vv') <= -1.5102615356445312) ?  0.037025987426812235 :   0.0734742706009789 :   (b('ndvi_med') <= 3391.25) ?  (b('ndvi') <= 4619.5) ?  (b('l8b11') <= 1470.8193969726562) ?  -0.025281199550415755 :   -0.04912076368588021 :   0.037457419316457796 :   (b('dg0_v_vv') <= 0.48157739639282227) ?  (b('l8b11') <= 1150.8447265625) ?  -0.0023451039050389894 :   -0.02499561052386228 :   0.06191190251285966 :   (b('ndvi') <= 3219.5) ?  (b('l8b5') <= 3076.441650390625) ?  (b('l8b5_med') <= 3474.926513671875) ?  (b('l8b5') <= 3039.614990234375) ?  -0.0025330187869959217 :   -0.046311912443878896 :   (b('ndvi') <= 2596.5) ?  -0.17137237736346625 :   0.006973685753707826 :   (b('l8b5') <= 3747.8695068359375) ?  (b('l8b4') <= 1485.5625610351562) ?  0.025327946472388882 :   -0.004539040488878569 :   (b('dg0_v_vv') <= -0.7872519493103027) ?  0.008943686901306844 :   -0.027977636813849294 :   (b('l8b11_med') <= 1868.24072265625) ?  (b('ndvi') <= 3259.0) ?  (b('ndvi_med') <= 2965.5) ?  0.06421502197603633 :   -0.006806820179900919 :   (b('l8b4') <= 1834.26708984375) ?  0.007163139750745806 :   0.08799598557314108 :   (b('l8b4') <= 1521.8001708984375) ?  (b('dg0_v_vv') <= 0.43007326126098633) ?  0.056585811010465864 :   0.02286752262029229 :   (b('ndvi') <= 3840.0) ?  0.005839452220077895 :   0.03418095636819113 :   (b('grass') <= 9.215554237365723) ?  (b('dg0_v_vv') <= 3.4839234352111816) ?  (b('l8b11_med') <= 1220.2855834960938) ?  (b('ndvi') <= 3845.5) ?  (b('l8b5_med') <= 2623.8328857421875) ?  0.043982637611875264 :   0.0646800740868188 :   0.11233821885675543 :   (b('ndvi_med') <= 3383.0) ?  (b('l8b5') <= 4652.053955078125) ?  0.012310221298940332 :   -0.043787390079125105 :   (b('l8b5') <= 3373.61669921875) ?  -0.011401217800855637 :   -0.04098956999340387 :   (b('dg0_v_vv') <= 3.793283700942993) ?  (b('dg0_v_vv') <= 3.663762092590332) ?  (b('moss') <= 1.2606635093688965) ?  0.07907734225406038 :   0.029561563718248536 :   (b('l8b11') <= 975.8413696289062) ?  0.05734733693665078 :   0.08553123299263714 :   (b('dg0_v_vv') <= 4.845574140548706) ?  (b('l8b5_med') <= 2792.08935546875) ?  0.04291196338297573 :   -0.013432491782498201 :   (b('l8b5_med') <= 2528.884765625) ?  0.03764012635362429 :   0.07097263487606825 :   (b('moss') <= 0.15536722540855408) ?  (b('l8b5') <= 3668.396484375) ?  (b('l8b4') <= 1059.09814453125) ?  (b('l8b5') <= 2214.556396484375) ?  0.09735003800698905 :   0.056866305090084 :   (b('l8b5_med') <= 2591.1446533203125) ?  0.0726707069284443 :   0.10422980060124828 :   (b('l8b11_med') <= 2028.6973876953125) ?  0.0020736514977661713 :   0.04236395667719317 :   (b('dg0_v_vv') <= 2.1722700595855713) ?  (b('dg0_v_vv') <= 0.8686833381652832) ?  (b('l8b4') <= 693.2548828125) ?  0.06634057132231955 :   0.09068324977960932 :   (b('l8b11') <= 1146.258544921875) ?  0.0012046051018817794 :   0.02124573646577564 :   (b('crops') <= 74.0476188659668) ?  (b('ndvi') <= 3464.0) ?  0.07085935001247427 :   0.10827064211157678 :   (b('dg0_v_vv') <= 6.22850775718689) ?  0.03196279353694182 :   0.010539102698354341 :   (b('sand') <= 47.5) ?  (b('ndvi') <= 3511.0) ?  (b('l8b5') <= 2151.3306884765625) ?  (b('l8b11') <= 2153.1478271484375) ?  (b('l8b11_med') <= 2724.4326171875) ?  (b('sand') <= 39.5) ?  (b('dg0_v_vv') <= -0.3288764953613281) ?  -0.0035742041544301923 :   0.02531412474393284 :   (b('ndvi') <= 3417.5) ?  0.05083379262323386 :   -0.026255142992246988 :   (b('dg0_v_vv') <= -0.017299652099609375) ?  (b('ndvi') <= 2652.0) ?  -0.07509354601477425 :   -0.057603511889776915 :   (b('ndvi') <= 2476.0) ?  -0.021228444410759967 :   -0.035863960198012544 :   (b('bulk') <= 148.0) ?  0.1247593614015893 :   0.1559451349135767 :   (b('l8b11') <= 1150.9000244140625) ?  (b('dg0_v_vv') <= 0.5893826484680176) ?  (b('l8b5_med') <= 2575.6273193359375) ?  (b('dg0_v_vv') <= -0.09951639175415039) ?  -0.04498737239075224 :   0.027433772631413472 :   (b('l8b5') <= 2921.5146484375) ?  -0.001818879225498833 :   0.05157499359593836 :   (b('l8b5_med') <= 2367.96337890625) ?  (b('grass') <= 55.13003158569336) ?  0.16521156978818163 :   0.11580151221146773 :   (b('moss') <= 12.247491359710693) ?  0.055455323506765145 :   -0.023512440311744452 :   (b('bulk') <= 166.0) ?  (b('grass') <= 37.80600547790527) ?  (b('bulk') <= 160.0) ?  -0.004572542693360285 :   0.04188528444013892 :   (b('l8b5_med') <= 2216.328125) ?  -0.016973030123154664 :   0.014051416874315551 :   (b('l8b5_med') <= 2601.6121826171875) ?  (b('l8b11_med') <= 2170.646484375) ?  -0.1381451140618219 :   -0.09401781593866945 :   0.008297008643191273 :   (b('l8b4') <= 629.4564514160156) ?  (b('l8b5_med') <= 2166.23095703125) ?  -0.1626933981825599 :   (b('l8b5_med') <= 2351.24560546875) ?  (b('dg0_v_vv') <= 0.5672051906585693) ?  (b('sand') <= 42.5) ?  0.034763222744754484 :   0.07155489164644897 :   (b('l8b5_med') <= 2289.8765869140625) ?  0.07257420925165714 :   0.13016666343427724 :   (b('dg0_v_vv') <= 2.1936821937561035) ?  (b('l8b5_med') <= 2364.94482421875) ?  -0.14912000998759403 :   0.002798935925290095 :   (b('dg0_v_vv') <= 2.3592615127563477) ?  0.10428559549583498 :   0.012877040083680385 :   (b('moss') <= 13.93380355834961) ?  (b('l8b4') <= 635.7025146484375) ?  (b('bulk') <= 153.5) ?  0.1394877627662497 :   0.18972757468175488 :   (b('ndvi') <= 3557.5) ?  (b('l8b5') <= 3914.2958984375) ?  0.04824463245432617 :   0.1911234377877032 :   (b('dg0_v_vv') <= -1.4710783958435059) ?  -0.007588510876260015 :   0.02612094518446122 :   (b('grass') <= 31.046818733215332) ?  (b('l8b11') <= 2562.07373046875) ?  (b('dg0_v_vv') <= 2.1621174812316895) ?  -0.00038445167101302304 :   -0.037421200219694216 :   (b('l8b4') <= 1500.5227661132812) ?  0.10806649664704045 :   0.024674382662719097 :   (b('l8b4_med') <= 955.7552490234375) ?  (b('l8b5') <= 2639.00439453125) ?  0.11962311044285023 :   0.038108977827574324 :   (b('l8b5') <= 3269.421630859375) ?  0.010634992464534566 :   0.08323094369750014 :   (b('dg0_v_vv') <= 4.220325469970703) ?  (b('l8b11_med') <= 1589.20361328125) ?  (b('l8b4') <= 1027.0681457519531) ?  (b('l8b5') <= 4511.73828125) ?  (b('l8b5') <= 3086.48291015625) ?  (b('ndvi') <= 2363.5) ?  0.07469504854127329 :   -0.01497571844465586 :   (b('dg0_v_vv') <= -2.2814979553222656) ?  -0.05514407585616862 :   0.029987730545120557 :   (b('dg0_v_vv') <= 3.097120761871338) ?  (b('l8b4_med') <= 685.351806640625) ?  -0.03331528659710292 :   -0.08001099193503104 :   -0.021084577509634855 :   (b('dg0_v_vv') <= 0.7441830635070801) ?  (b('l8b11') <= 1803.0143432617188) ?  (b('dg0_v_vv') <= 0.11150121688842773) ?  0.05701874175281332 :   0.041743083747231735 :   (b('l8b4') <= 1332.3839111328125) ?  0.03638120892273102 :   0.06480437447018517 :   (b('ndvi') <= 3023.0) ?  (b('dg0_v_vv') <= 1.2677102088928223) ?  0.06693037774954738 :   0.06346089674992073 :   0.07293991317303786 :   (b('ndvi_med') <= 2112.0) ?  (b('ndvi_med') <= 2058.0) ?  (b('bulk') <= 151.5) ?  (b('grass') <= 6.505878329277039) ?  0.010756755354715906 :   -0.013976925048534728 :   (b('l8b11_med') <= 1796.7609252929688) ?  -0.045798952990729815 :   0.01476296113325972 :   (b('dg0_v_vv') <= 1.9635086059570312) ?  (b('dg0_v_vv') <= -0.43289709091186523) ?  0.11320866731236717 :   0.08963682672984859 :   0.047994812714208385 :   (b('l8b11') <= 643.557861328125) ?  0.10854456936123474 :   (b('l8b4') <= 903.1581115722656) ?  (b('l8b11_med') <= 1784.0094604492188) ?  -0.008178854528442329 :   -0.03564324056323939 :   (b('moss') <= 16.259236335754395) ?  -0.014288043449581808 :   0.1474895177363581 :   (b('bulk') <= 143.0) ?  (b('l8b5') <= 3037.6492919921875) ?  0.13722529526059174 :   0.13363079091176822 :   (b('ndvi_med') <= 1724.5) ?  (b('bulk') <= 151.5) ?  (b('ndvi') <= 3109.0) ?  (b('l8b4') <= 945.27392578125) ?  -0.07106441106339938 :   -0.0017834156770502596 :   (b('l8b4') <= 775.4584655761719) ?  0.05327614948221118 :   -0.023223841181434172 :   0.09081267640904292 :   (b('ndvi') <= 2624.0) ?  (b('l8b11') <= 1399.0490112304688) ?  (b('sand') <= 56.0) ?  0.06434336758338366 :   0.06075850276977934 :   0.1260163646900514 :   (b('grass') <= 73.09062576293945) ?  (b('dg0_v_vv') <= 5.678764343261719) ?  0.00948612121058362 :   0.032768969778902805 :   (b('ndvi_med') <= 2483.0) ?  0.05408839723801484 :   0.06797470448130566 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4_med') <= 658.7252502441406) ?  (b('grass') <= 45.98488426208496) ?  (b('grass') <= 45.3359375) ?  (b('l8b4') <= 585.81201171875) ?  (b('l8b4') <= 519.4012756347656) ?  (b('l8b4') <= 514.9747009277344) ?  (b('grass') <= 41.9486198425293) ?  (b('ndvi_med') <= 4408.5) ?  (b('sand') <= 32.5) ?  (b('l8b5') <= 4423.28857421875) ?  0.01720442887062018 :   0.0656898564292443 :   (b('l8b5_med') <= 2926.2784423828125) ?  -0.011091229005069056 :   0.008598347677499861 :   (b('l8b5') <= 3862.753173828125) ?  (b('ndvi_med') <= 4576.5) ?  0.10103339270942657 :   0.034705351253636975 :   (b('dg0_v_vv') <= 1.095479965209961) ?  0.0021076810287894107 :   0.07373489458347879 :   (b('ndvi') <= 4538.0) ?  (b('ndvi') <= 3726.5) ?  (b('dg0_v_vv') <= 0.820930004119873) ?  0.06364695071040274 :   0.03929727080958079 :   (b('dg0_v_vv') <= -0.03513479232788086) ?  0.11535153316069077 :   0.08430221144527904 :   (b('ndvi') <= 4913.5) ?  (b('l8b5_med') <= 3503.7686767578125) ?  -0.01862282008766293 :   0.006252691596125182 :   0.04640478745707116 :   (b('sand') <= 40.5) ?  (b('l8b11_med') <= 1109.44775390625) ?  (b('dg0_v_vv') <= -1.0549955368041992) ?  (b('ndvi') <= 2934.5) ?  -0.020547410398956502 :   -0.015480472998674166 :   (b('l8b4_med') <= 577.3431701660156) ?  -0.03784652358847487 :   -0.020739114835132896 :   (b('l8b11_med') <= 1120.615478515625) ?  0.020981089023499683 :   (b('l8b5') <= 3281.892333984375) ?  0.001564180737298887 :   -0.014474440766322566 :   (b('l8b5_med') <= 2940.280029296875) ?  (b('bulk') <= 123.0) ?  -0.03831833713979653 :   -0.0628626421463674 :   (b('ndvi') <= 4615.5) ?  -0.12574301232292703 :   (b('dg0_v_vv') <= 1.759988784790039) ?  -0.08618393231590514 :   -0.09248778710421288 :   (b('l8b11_med') <= 1215.2675170898438) ?  (b('l8b11') <= 1185.5816040039062) ?  (b('dg0_v_vv') <= 0.6607151031494141) ?  (b('l8b4_med') <= 614.1442565917969) ?  (b('l8b4') <= 527.4898681640625) ?  0.05909706498290168 :   0.019918683175569864 :   (b('l8b11') <= 1085.9926147460938) ?  -0.016468532601404327 :   0.016835368841108006 :   (b('dg0_v_vv') <= 1.2783284187316895) ?  (b('l8b11') <= 1114.3367309570312) ?  0.04662340670929889 :   0.10293324339727139 :   (b('moss') <= 4.940035104751587) ?  0.03672462827827531 :   -0.03030843951798967 :   (b('bulk') <= 110.0) ?  (b('dg0_v_vv') <= 0.2230362892150879) ?  (b('ndvi') <= 4508.0) ?  0.1161468059752469 :   0.15707999814452195 :   0.07254071891570901 :   (b('ndvi') <= 2540.5) ?  (b('dg0_v_vv') <= -0.4498865604400635) ?  0.15166068359866042 :   0.06946454450237496 :   (b('ndvi') <= 4263.5) ?  0.05361392068434122 :   0.005014932402657123 :   (b('crops') <= 91.79462814331055) ?  (b('l8b5') <= 4092.561767578125) ?  (b('ndvi') <= 3156.5) ?  (b('bulk') <= 140.5) ?  0.09176850889322688 :   -0.0035137314462939107 :   (b('ndvi') <= 3761.0) ?  -0.01957313649109737 :   0.011844447899569255 :   (b('ndvi') <= 4877.5) ?  -0.08378000692933765 :   -0.06137764648948546 :   (b('l8b11') <= 1231.8994750976562) ?  0.06006810458805156 :   0.09080710739865794 :   (b('l8b5_med') <= 1607.8214721679688) ?  (b('ndvi') <= 1822.0) ?  (b('dg0_v_vv') <= 0.33765649795532227) ?  (b('l8b4') <= 738.5) ?  (b('dg0_v_vv') <= 0.10703778266906738) ?  (b('ndvi') <= 1804.5) ?  0.006290607964310878 :   -0.00018181514223981898 :   0.019230192649026617 :   0.04353503682879874 :   -0.064207885121393 :   (b('ndvi') <= 2832.5) ?  (b('l8b4') <= 590.1642150878906) ?  (b('l8b4_med') <= 595.4280090332031) ?  -0.06533954139285951 :   -0.05636743560241114 :   (b('dg0_v_vv') <= -0.4601900577545166) ?  (b('l8b5') <= 1566.5389404296875) ?  -0.016868299668379397 :   -0.010795161419912783 :   (b('ndvi') <= 2188.5) ?  -0.024257430833127428 :   -0.03773162028068775 :   (b('ndvi') <= 2957.0) ?  0.051860883059521745 :   (b('l8b4') <= 858.7998352050781) ?  (b('l8b4_med') <= 595.4280090332031) ?  0.006333661772874283 :   -0.01679063723546211 :   (b('l8b11_med') <= 1102.9674682617188) ?  -0.02972913520346057 :   -0.045769177052764926 :   (b('ndvi') <= 2493.0) ?  (b('ndvi') <= 2431.0) ?  (b('ndvi') <= 2183.5) ?  (b('l8b5') <= 1721.7799682617188) ?  (b('dg0_v_vv') <= 0.033354759216308594) ?  -0.00015404567445036365 :   0.028357064238745656 :   (b('moss') <= 18.525145530700684) ?  0.036251704994213235 :   0.1819401873983442 :   (b('ndvi_med') <= 2482.25) ?  (b('moss') <= 18.62898826599121) ?  -0.07000353736309411 :   -0.03126170481208318 :   (b('l8b5_med') <= 2124.7242431640625) ?  0.07176971563521574 :   0.00421817240222165 :   (b('l8b4') <= 758.3311157226562) ?  (b('l8b11') <= 1261.174560546875) ?  (b('dg0_v_vv') <= -0.6288471221923828) ?  0.07366194008603377 :   0.06787490444901687 :   (b('crops') <= 9.191222190856934) ?  0.09994313120975251 :   0.11386437011103362 :   (b('bulk') <= 116.5) ?  0.019118729116272326 :   0.029128871112510446 :   (b('l8b4_med') <= 655.749755859375) ?  (b('l8b5_med') <= 3712.1297607421875) ?  (b('l8b5') <= 1827.215087890625) ?  (b('dg0_v_vv') <= 3.4388787746429443) ?  0.036091002562040994 :   0.015872978922013458 :   (b('l8b5') <= 1917.8048706054688) ?  -0.05695264405229731 :   0.0003836356116208578 :   (b('l8b5_med') <= 4136.123046875) ?  (b('l8b4_med') <= 581.9038696289062) ?  0.03477847110097139 :   0.0014085071435354245 :   (b('sand') <= 63.5) ?  -0.0651590101046233 :   -0.034045283436594576 :   (b('ndvi') <= 3330.0) ?  (b('dg0_v_vv') <= 1.0007078647613525) ?  0.07008454540109016 :   0.06691033457682544 :   (b('l8b4') <= 676.3401794433594) ?  (b('l8b4') <= 654.9471740722656) ?  0.038035566394050424 :   0.10226027674859894 :   (b('moss') <= 5.019837498664856) ?  0.011316495285598785 :   0.030032306211386417 :   (b('ndvi_med') <= 3577.75) ?  (b('l8b5') <= 3137.499755859375) ?  (b('dg0_v_vv') <= -1.589578628540039) ?  -0.08089235888776312 :   (b('dg0_v_vv') <= 1.0296573638916016) ?  (b('l8b11') <= 592.4971618652344) ?  (b('l8b4_med') <= 581.4598999023438) ?  -0.022055959686343593 :   -0.03712023312021155 :   (b('dg0_v_vv') <= 0.7110428810119629) ?  (b('l8b11') <= 2030.5311279296875) ?  -0.007877001879862918 :   0.0059724880051831575 :   0.014975888471896098 :   (b('dg0_v_vv') <= 1.4027690887451172) ?  (b('l8b5') <= 2723.0118408203125) ?  -0.06387388195587652 :   -0.0823261339367872 :   (b('l8b5') <= 2820.38330078125) ?  -0.014507600322503839 :   0.007742168463616694 :   (b('l8b5') <= 4066.7396240234375) ?  (b('dg0_v_vv') <= 0.1937103271484375) ?  (b('l8b4') <= 462.04295349121094) ?  -0.0591741217514669 :   (b('l8b5') <= 3669.69873046875) ?  -0.035730723015666016 :   -0.04625066399662241 :   (b('l8b11') <= 992.97021484375) ?  -0.05022217315572672 :   (b('l8b5') <= 3676.688232421875) ?  (b('l8b4') <= 583.3661193847656) ?  -0.06428387864556953 :   -0.06070797182025392 :   -0.07319389672915126 :   (b('dg0_v_vv') <= -0.3186488151550293) ?  -0.03326112798661125 :   -0.004250664562866532 :   (b('l8b5') <= 3220.4420166015625) ?  (b('dg0_v_vv') <= -0.5739626884460449) ?  (b('dg0_v_vv') <= -0.6898221969604492) ?  (b('dg0_v_vv') <= -0.732243537902832) ?  (b('l8b5_med') <= 3019.57470703125) ?  0.0022847151674526833 :   0.01472561345776452 :   0.02596476295652181 :   0.057909009201834366 :   (b('ndvi') <= 3448.0) ?  (b('l8b4') <= 891.6674499511719) ?  -0.03229837464656239 :   -0.03910157191229943 :   (b('l8b4') <= 679.9635009765625) ?  (b('l8b5') <= 3152.664306640625) ?  (b('l8b5') <= 2662.5489501953125) ?  -0.0037914341629768367 :   -0.010826542473452966 :   -0.038158278578022814 :   (b('l8b11') <= 1736.0618896484375) ?  0.0389549616275027 :   0.009276344711600076 :   0.09083556711996385 :   (b('crops') <= 19.45354127883911) ?  (b('dg0_v_vv') <= 0.13691329956054688) ?  (b('l8b11') <= 988.2427062988281) ?  (b('dg0_v_vv') <= -0.009121894836425781) ?  (b('l8b11_med') <= 935.4756774902344) ?  (b('l8b5') <= 2239.8275146484375) ?  (b('bulk') <= 130.5) ?  0.01416295951683913 :   -0.012716164173339817 :   (b('l8b5') <= 2260.92138671875) ?  -0.04245639375655652 :   -0.04580925536004983 :   (b('l8b5') <= 2019.7839965820312) ?  (b('ndvi_med') <= 3625.75) ?  (b('dg0_v_vv') <= -1.5373401641845703) ?  -0.029987934036067587 :   0.006880345980351366 :   (b('dg0_v_vv') <= -0.8458223342895508) ?  -0.03268907169810137 :   0.04437963918202059 :   (b('grass') <= 57.87235069274902) ?  (b('dg0_v_vv') <= -1.3912005424499512) ?  0.0728310023855067 :   0.03906509702314834 :   (b('l8b5') <= 2824.03857421875) ?  0.024227298151066123 :   -0.009578534015953682 :   (b('dg0_v_vv') <= -0.0008087158203125) ?  (b('l8b5') <= 2195.1582641601562) ?  0.1638044469422812 :   (b('ndvi') <= 3225.0) ?  0.08684737695228734 :   0.08311909956784691 :   (b('dg0_v_vv') <= 0.000385284423828125) ?  -0.013566818675253667 :   (b('bulk') <= 145.0) ?  (b('sand') <= 70.5) ?  0.04396701845511042 :   0.047201043755299266 :   0.020536907389312214 :   (b('ndvi') <= 1716.0) ?  0.12018888743897332 :   (b('l8b4') <= 1080.5400390625) ?  (b('l8b11') <= 1292.7738647460938) ?  (b('l8b11') <= 1259.20751953125) ?  (b('ndvi') <= 3521.5) ?  0.014248360161520432 :   -0.01504656764040965 :   (b('l8b11') <= 1270.3040771484375) ?  -0.07571053748776949 :   -0.03920198214119567 :   (b('l8b5') <= 2984.0155029296875) ?  (b('ndvi_med') <= 3744.0) ?  0.03047401628913891 :   -0.01704537418066103 :   (b('ndvi_med') <= 4056.0) ?  -0.03493401415428793 :   0.003695964870668656 :   (b('l8b11_med') <= 1244.5859985351562) ?  (b('l8b11') <= 2113.2613525390625) ?  -0.03692557742247268 :   -0.046501765682270046 :   (b('bulk') <= 146.5) ?  -0.06555845324120357 :   -0.07712914805150373 :   (b('l8b11_med') <= 1307.5338745117188) ?  (b('ndvi') <= 4394.0) ?  (b('ndvi_med') <= 2397.0) ?  (b('ndvi') <= 2390.5) ?  (b('l8b5_med') <= 2023.093505859375) ?  -0.006407351922275845 :   -0.014265463460995019 :   -0.04696234786309629 :   (b('dg0_v_vv') <= 0.23805904388427734) ?  (b('ndvi') <= 3611.0) ?  (b('l8b4') <= 788.5829162597656) ?  0.14428716639862252 :   0.04588928817891702 :   (b('grass') <= 50.690229415893555) ?  0.04448267469944253 :   0.06842620874266198 :   (b('dg0_v_vv') <= 0.9762048721313477) ?  (b('l8b5') <= 2085.4517822265625) ?  0.01968147251277856 :   0.04548620444408884 :   (b('dg0_v_vv') <= 1.060258388519287) ?  0.10957966774914078 :   0.05649494637980989 :   (b('ndvi_med') <= 3682.5) ?  (b('crops') <= 9.933524370193481) ?  (b('l8b5') <= 2382.87744140625) ?  -0.050733672917476436 :   (b('dg0_v_vv') <= 1.1982264518737793) ?  -0.002134685916848409 :   -0.01812047031495332 :   0.061808712382230874 :   (b('ndvi') <= 4492.0) ?  (b('l8b11_med') <= 1147.8057250976562) ?  0.0021565250989700746 :   0.04766323169485959 :   (b('dg0_v_vv') <= 1.27186918258667) ?  (b('sand') <= 52.0) ?  0.07875522603673668 :   0.07607946741655452 :   (b('dg0_v_vv') <= 1.987194538116455) ?  0.06655049314158051 :   0.05937552907634458 :   (b('l8b5') <= 2560.079833984375) ?  0.00028748939438943966 :   (b('dg0_v_vv') <= 1.1681451797485352) ?  (b('crops') <= 6.950881481170654) ?  -0.02386508546999422 :   (b('l8b5') <= 3766.4395751953125) ?  (b('ndvi') <= 4760.5) ?  -0.04377385642037589 :   -0.04180606648901655 :   -0.038442524904545555 :   (b('l8b11') <= 1147.638671875) ?  -0.02399497178729451 :   -0.01753516621089765 :   (b('l8b4') <= 550.6309814453125) ?  (b('l8b5_med') <= 3051.766357421875) ?  (b('ndvi') <= 4931.0) ?  (b('ndvi') <= 4907.0) ?  (b('sand') <= 52.5) ?  (b('l8b5_med') <= 2592.1824951171875) ?  (b('dg0_v_vv') <= 0.49697303771972656) ?  0.05459137312823309 :   0.08655717259414009 :   (b('l8b5_med') <= 2761.9703369140625) ?  0.0426837980714501 :   0.05256426714020823 :   (b('ndvi') <= 2393.5) ?  0.07081816413779901 :   (b('ndvi_med') <= 3442.5) ?  0.036687508470864494 :   0.04571252260638171 :   (b('ndvi_med') <= 3541.0) ?  0.04134757739347977 :   (b('dg0_v_vv') <= 1.4038753509521484) ?  (b('l8b11_med') <= 1332.1425170898438) ?  0.029107384855244778 :   0.033239797039380015 :   0.03807251166251607 :   (b('dg0_v_vv') <= 0.4457864761352539) ?  0.08073589082524857 :   (b('l8b4') <= 394.7136993408203) ?  (b('moss') <= 5.344086170196533) ?  0.05470217598176719 :   0.049776139689237386 :   0.06445426270976773 :   (b('grass') <= 52.18928527832031) ?  0.10098202852225963 :   (b('l8b11') <= 1180.7108764648438) ?  (b('dg0_v_vv') <= 1.3513526916503906) ?  (b('dg0_v_vv') <= 0.1486525535583496) ?  0.06214090527003985 :   (b('dg0_v_vv') <= 0.6758866310119629) ?  0.045031542877096764 :   0.042723751649138164 :   0.07329221526411817 :   (b('dg0_v_vv') <= 1.4671530723571777) ?  (b('dg0_v_vv') <= -0.24639320373535156) ?  (b('l8b11') <= 1270.8560180664062) ?  0.06616704127240151 :   0.06080351569535322 :   (b('dg0_v_vv') <= 1.102283000946045) ?  0.06932573527174057 :   0.07362463586652557 :   0.05580164513150254 :   (b('l8b5') <= 2405.21044921875) ?  (b('ndvi') <= 3523.5) ?  (b('l8b11') <= 1437.5267333984375) ?  (b('l8b4') <= 641.6177062988281) ?  (b('dg0_v_vv') <= -1.800633430480957) ?  (b('ndvi') <= 3293.5) ?  0.02065016353969168 :   0.00956576173349763 :   -0.021352038331128004 :   (b('ndvi_med') <= 3494.75) ?  (b('dg0_v_vv') <= -0.012296676635742188) ?  0.029122310236262355 :   0.03813395524959752 :   (b('ndvi') <= 3319.0) ?  0.005494108878303011 :   0.034068237449037864 :   0.06952378782652538 :   (b('l8b11') <= 1216.6610717773438) ?  (b('ndvi_med') <= 3494.75) ?  (b('dg0_v_vv') <= 1.6990995407104492) ?  (b('dg0_v_vv') <= -2.2203359603881836) ?  0.0028603832761550374 :   -0.0025545814988435084 :   0.0035204667681959234 :   -0.010896168992918731 :   (b('ndvi_med') <= 3442.5) ?  (b('dg0_v_vv') <= -2.4059205055236816) ?  -0.025106692303316372 :   -0.013031821929958429 :   -0.010875177989491044 :   (b('dg0_v_vv') <= -1.5577630996704102) ?  (b('ndvi_med') <= 3442.5) ?  (b('dg0_v_vv') <= -3.170614719390869) ?  0.01773303142250332 :   (b('l8b5') <= 2742.911376953125) ?  (b('ndvi') <= 2261.5) ?  0.07039779951965675 :   0.054919684242713686 :   (b('dg0_v_vv') <= -1.9025611877441406) ?  0.050734150550783064 :   0.03340980149756828 :   (b('dg0_v_vv') <= -1.7164154052734375) ?  (b('l8b11') <= 1469.739013671875) ?  0.06857241584336832 :   0.06961105290935526 :   0.07046240760362227 :   (b('l8b5') <= 2487.0224609375) ?  (b('l8b4') <= 668.9454040527344) ?  (b('ndvi_med') <= 3405.0) ?  (b('ndvi') <= 3323.0) ?  -0.03767017703322975 :   -0.05650129402243678 :   0.006191290988183135 :   (b('dg0_v_vv') <= 0.6850171089172363) ?  (b('ndvi') <= 4735.0) ?  0.038384388968441985 :   0.07193500088913568 :   (b('l8b11') <= 1540.2808837890625) ?  -0.023025722543277893 :   0.03522688745278324 :   (b('dg0_v_vv') <= -0.49304676055908203) ?  (b('ndvi_med') <= 3423.0) ?  (b('dg0_v_vv') <= -1.309615135192871) ?  -0.026474191391811452 :   0.027186556450520902 :   (b('l8b5') <= 3505.7174072265625) ?  0.0340892452658838 :   0.011974173916978581 :   (b('l8b11') <= 1184.98681640625) ?  (b('ndvi_med') <= 3612.5) ?  0.06486008971366974 :   0.03762620499892766 :   (b('grass') <= 63.13859176635742) ?  0.039050350463657346 :   0.0253360168881 :   (b('l8b11_med') <= 2628.4404296875) ?  (b('dg0_v_vv') <= 0.8229169845581055) ?  (b('l8b5_med') <= 2986.78076171875) ?  (b('sand') <= 22.0) ?  (b('l8b5') <= 3330.0013427734375) ?  (b('dg0_v_vv') <= -2.2220849990844727) ?  (b('ndvi_med') <= 3373.0) ?  (b('l8b11') <= 1498.7047729492188) ?  (b('dg0_v_vv') <= -2.759805202484131) ?  -0.007653041211809486 :   0.031776007440283305 :   (b('l8b4') <= 1014.8195495605469) ?  0.028147812888592083 :   0.05224144771219652 :   (b('ndvi_med') <= 3599.5) ?  (b('l8b5') <= 2294.8575439453125) ?  -0.006060016762994391 :   -0.025657591725895046 :   (b('l8b4') <= 1133.7440795898438) ?  -0.04059702162868435 :   -0.05325274631415289 :   (b('l8b4_med') <= 841.0370788574219) ?  (b('l8b4') <= 500.4179229736328) ?  -0.02206243645596359 :   (b('l8b5') <= 2839.0357666015625) ?  0.04682996139760152 :   0.07554187757885597 :   (b('l8b4') <= 777.1054077148438) ?  (b('ndvi') <= 4937.5) ?  -0.023384891822686948 :   -0.07293905255647884 :   (b('dg0_v_vv') <= -0.20522642135620117) ?  0.09934949074905124 :   0.03155780587099915 :   (b('l8b5') <= 3651.0595703125) ?  (b('dg0_v_vv') <= -0.32991456985473633) ?  (b('l8b4') <= 529.7083129882812) ?  (b('ndvi_med') <= 3459.0) ?  -0.045753812417430845 :   -0.04408678176251188 :   (b('l8b5_med') <= 2894.9593505859375) ?  -0.07500053180092747 :   -0.08883575829273127 :   (b('bulk') <= 150.5) ?  (b('dg0_v_vv') <= -0.028347015380859375) ?  0.021824331111285566 :   0.0070097931574135686 :   -0.008497070656971806 :   (b('ndvi') <= 3489.0) ?  (b('l8b11') <= 1024.5054626464844) ?  0.014323243094354399 :   0.030592483946995652 :   (b('l8b11') <= 921.9379272460938) ?  0.07579461796671455 :   (b('l8b11_med') <= 1467.1681518554688) ?  0.06399840423888253 :   0.05908160500325349 :   (b('l8b4') <= 654.3262939453125) ?  (b('crops') <= 18.473238945007324) ?  (b('l8b4_med') <= 685.7174072265625) ?  (b('l8b11') <= 760.6183166503906) ?  0.039778120448005355 :   (b('l8b5') <= 2242.548095703125) ?  -0.06426271099367474 :   -0.11668111101891324 :   (b('sand') <= 43.5) ?  (b('dg0_v_vv') <= 0.08334064483642578) ?  0.018967863348157973 :   0.05164509954869304 :   (b('ndvi') <= 1961.5) ?  0.046692727664671585 :   -0.0017921138925320078 :   (b('dg0_v_vv') <= -0.9854488372802734) ?  (b('l8b5') <= 4447.6416015625) ?  (b('l8b4_med') <= 701.7364501953125) ?  0.018188523861835233 :   -0.020393622163545462 :   (b('sand') <= 47.5) ?  0.10065381253178918 :   0.038563388423918626 :   (b('l8b5') <= 2633.3861083984375) ?  (b('l8b5') <= 2537.8040771484375) ?  0.005979756363709086 :   0.045317353953864443 :   (b('l8b5') <= 3593.8751220703125) ?  -0.008290778013424122 :   0.008215572738498284 :   (b('l8b11_med') <= 1360.62841796875) ?  (b('l8b5_med') <= 2278.0772705078125) ?  (b('ndvi') <= 721.5) ?  (b('ndvi') <= 690.5) ?  -0.05735983999338862 :   -0.026596532061254195 :   (b('ndvi') <= 1294.5) ?  0.06610891848021783 :   -0.0013383140024109735 :   (b('l8b5') <= 2822.079345703125) ?  (b('ndvi') <= 3519.0) ?  -0.028688324054757373 :   0.008891992825905814 :   (b('l8b5') <= 2937.2640380859375) ?  0.01824042278457137 :   -0.016976316809625747 :   (b('l8b11') <= 1318.9013671875) ?  (b('l8b4_med') <= 895.3853759765625) ?  (b('bulk') <= 135.0) ?  0.0057892902834949895 :   -0.02697162874408744 :   (b('l8b11') <= 1306.4591064453125) ?  0.01758613163941242 :   0.06911918347840752 :   (b('ndvi_med') <= 1866.75) ?  (b('l8b11_med') <= 2472.848876953125) ?  -0.010243926835629696 :   -0.00033271899363975654 :   (b('l8b11_med') <= 1975.9132690429688) ?  -0.005226372716675227 :   0.005596370211095571 :   (b('l8b5_med') <= 3535.697509765625) ?  (b('moss') <= 13.856780052185059) ?  (b('l8b5') <= 2565.8309326171875) ?  (b('l8b4') <= 903.4913330078125) ?  (b('l8b4') <= 511.0576934814453) ?  (b('ndvi') <= 3717.0) ?  -0.002388542293303141 :   0.04850631877669679 :   (b('l8b5') <= 2122.6624755859375) ?  -0.042072804745271704 :   -0.007252133176503247 :   (b('ndvi') <= 1791.0) ?  (b('crops') <= 75.78702926635742) ?  -0.01787474399788258 :   0.015368461563871757 :   (b('l8b11') <= 1679.9190063476562) ?  0.07835916622634473 :   0.034379037560849125 :   (b('l8b5') <= 2708.6300048828125) ?  (b('l8b5_med') <= 3338.6287841796875) ?  (b('ndvi_med') <= 3685.5) ?  -0.0342602290813286 :   0.06072758998310193 :   0.07342706448334252 :   (b('l8b5') <= 2734.4859619140625) ?  (b('l8b5_med') <= 3162.0750732421875) ?  0.12757217154035283 :   0.05456277589702276 :   (b('grass') <= 0.7789046764373779) ?  0.08800019275980318 :   0.007034633385593531 :   (b('l8b5') <= 2848.5947265625) ?  (b('crops') <= 28.946438789367676) ?  (b('ndvi_med') <= 3524.0) ?  (b('ndvi_med') <= 3347.75) ?  0.0531976343707203 :   0.027941186069949605 :   -0.03654855588197459 :   (b('l8b4') <= 943.1292419433594) ?  (b('l8b4') <= 752.0089416503906) ?  -0.0026310950355522174 :   -0.02776578790155624 :   (b('dg0_v_vv') <= 0.45308351516723633) ?  0.004289839797186783 :   0.06923693659583537 :   (b('l8b5') <= 2870.5125732421875) ?  (b('l8b11_med') <= 1966.1676025390625) ?  (b('l8b5_med') <= 3104.2471923828125) ?  -0.027007621571925253 :   -0.046056873534177 :   (b('ndvi') <= 2633.0) ?  -0.05540571534233858 :   -0.06596060057810577 :   (b('l8b5') <= 3722.599609375) ?  (b('l8b11') <= 1205.5081176757812) ?  -0.1063177796289562 :   -0.007999304417541056 :   (b('l8b5') <= 3753.6925048828125) ?  0.0782250670405799 :   -0.0035299357302104664 :   (b('dg0_v_vv') <= -1.6994519233703613) ?  (b('l8b4') <= 2003.2720336914062) ?  (b('l8b4') <= 1857.5347900390625) ?  (b('ndvi') <= 2497.5) ?  (b('l8b5') <= 2885.8753662109375) ?  -0.02642980964741226 :   -0.046830998625211465 :   (b('dg0_v_vv') <= -2.865184783935547) ?  -0.029056514438842766 :   -0.003189605765595502 :   (b('ndvi') <= 2556.0) ?  (b('l8b5') <= 3323.6634521484375) ?  0.01627901597886644 :   0.005148507331503294 :   (b('l8b5_med') <= 3694.775634765625) ?  -0.011600645076663196 :   -0.013600641679198444 :   (b('l8b11') <= 3622.0931396484375) ?  (b('l8b4') <= 2363.8314208984375) ?  (b('l8b11') <= 3078.45849609375) ?  -0.03531210488020113 :   -0.0395238928402631 :   (b('dg0_v_vv') <= -5.652923583984375) ?  -0.03767268750980655 :   -0.0299104576387319 :   (b('dg0_v_vv') <= -2.3974618911743164) ?  -0.014162909238077045 :   -0.018311305501389416 :   (b('l8b4') <= 1735.4385986328125) ?  (b('l8b11') <= 2934.3756103515625) ?  (b('l8b4_med') <= 1555.0704345703125) ?  (b('dg0_v_vv') <= -1.5829639434814453) ?  0.0506801872828985 :   -0.00838523002304896 :   (b('dg0_v_vv') <= -0.593381404876709) ?  0.011359226983512919 :   0.054039417571862966 :   0.09757515915351118 :   (b('ndvi') <= 2350.5) ?  (b('ndvi') <= 1902.0) ?  (b('l8b5') <= 2793.2647705078125) ?  -0.04838553895300197 :   -0.012831756100900168 :   (b('l8b4') <= 2143.2957763671875) ?  0.02880619632174027 :   -0.020492488428630704 :   (b('l8b5') <= 3343.1597900390625) ?  (b('dg0_v_vv') <= -1.2687091827392578) ?  -0.03762234429260891 :   -0.06565573258562689 :   (b('l8b5') <= 3779.8072509765625) ?  -0.031056840859005688 :   -0.022250247754738606 :   (b('bulk') <= 137.5) ?  (b('crops') <= 58.930376052856445) ?  (b('l8b11_med') <= 2526.8916015625) ?  (b('ndvi_med') <= 3342.25) ?  (b('l8b11') <= 2977.4132080078125) ?  (b('l8b11') <= 1653.801025390625) ?  (b('l8b4') <= 809.0575561523438) ?  -0.005873935995944896 :   -0.025923444329265275 :   (b('moss') <= 13.518618106842041) ?  -0.007543379532375477 :   0.014480495205232808 :   (b('l8b4_med') <= 807.3660278320312) ?  0.048570531834136954 :   0.10228479583848168 :   (b('dg0_v_vv') <= 3.0252444744110107) ?  (b('l8b5_med') <= 3095.4654541015625) ?  (b('dg0_v_vv') <= 0.8819351196289062) ?  0.04660495312691163 :   0.002412481975109175 :   (b('ndvi') <= 3134.5) ?  0.13867244953553304 :   0.031129395696026595 :   (b('bulk') <= 133.5) ?  (b('l8b5') <= 3582.579345703125) ?  -0.012154153392116174 :   -0.036404533312338096 :   0.06674942686243515 :   (b('l8b5') <= 1940.7014770507812) ?  0.19070028793147809 :   0.09938913691339707 :   (b('sand') <= 36.0) ?  (b('l8b4') <= 1478.3203735351562) ?  (b('l8b5') <= 4652.053955078125) ?  (b('l8b11') <= 954.1026611328125) ?  (b('l8b11') <= 540.7763977050781) ?  -0.037955229321622944 :   0.0402412608367519 :   (b('dg0_v_vv') <= 3.355390787124634) ?  0.003325562593038211 :   0.02523406899404334 :   (b('ndvi') <= 4519.0) ?  (b('dg0_v_vv') <= 3.4168474674224854) ?  -0.042489057440358566 :   -0.0703797893785777 :   (b('dg0_v_vv') <= 4.3161940574646) ?  0.06069920165067111 :   0.011699191756300076 :   (b('l8b5') <= 3426.4393310546875) ?  (b('dg0_v_vv') <= 2.587428092956543) ?  (b('l8b4') <= 1710.8235473632812) ?  0.06415706791851611 :   0.026194700988037778 :   (b('bulk') <= 133.5) ?  0.018463600322245338 :   -0.016846412257765064 :   (b('l8b11_med') <= 1257.730224609375) ?  0.10933124873718869 :   (b('l8b11') <= 2391.667724609375) ?  0.06934888974362613 :   0.06425287967447724 :   (b('ndvi') <= 2853.5) ?  (b('l8b11') <= 1538.509765625) ?  (b('sand') <= 41.5) ?  0.06826432326778342 :   (b('l8b5_med') <= 2990.6875) ?  0.10368744018792636 :   0.12110230672606594 :   (b('l8b11_med') <= 2009.24560546875) ?  (b('l8b11') <= 2293.9544677734375) ?  0.033294505958862755 :   0.010013993641292379 :   (b('l8b5') <= 1440.2705078125) ?  0.04366930144566622 :   0.08377807484824723 :   (b('ndvi') <= 4452.5) ?  (b('l8b5') <= 3232.6578369140625) ?  (b('l8b11') <= 1098.9818420410156) ?  0.008372748945728342 :   0.028641391068762432 :   (b('ndvi_med') <= 3704.5) ?  0.02357595611831612 :   0.06028272616006608 :   (b('dg0_v_vv') <= 1.0400171279907227) ?  0.03526268455253767 :   (b('bulk') <= 128.0) ?  0.02394617505471444 :   -0.005912251854888066 :   (b('moss') <= 26.28097152709961) ?  (b('l8b5_med') <= 3472.3001708984375) ?  (b('dg0_v_vv') <= 4.2002809047698975) ?  (b('moss') <= 0.7815661132335663) ?  (b('l8b11_med') <= 1347.2548828125) ?  (b('dg0_v_vv') <= 2.6947414875030518) ?  0.04774872806228817 :   0.10236209311733134 :   (b('l8b4') <= 1659.0186157226562) ?  -0.00048202618040189366 :   0.05192517917139598 :   (b('l8b11') <= 2102.6766357421875) ?  (b('ndvi_med') <= 1534.0) ?  0.05832164107247865 :   0.020331919375248433 :   (b('sand') <= 64.0) ?  0.00489076425684233 :   -0.05993931112009151 :   (b('ndvi') <= 1079.0) ?  (b('ndvi_med') <= 1151.0) ?  -0.08728725800293541 :   -0.06277237445143113 :   (b('l8b11_med') <= 2496.0) ?  (b('l8b11') <= 1141.177001953125) ?  0.003225862208162075 :   0.045007902401159795 :   (b('l8b11_med') <= 2559.8609619140625) ?  0.13062966951217997 :   0.0753533034704387 :   (b('l8b5') <= 2894.3790283203125) ?  (b('ndvi') <= 1511.0) ?  (b('ndvi_med') <= 2171.5) ?  -0.06574753919269086 :   (b('dg0_v_vv') <= 1.5641732215881348) ?  -0.003948133026265704 :   -0.0238119633643529 :   (b('dg0_v_vv') <= 4.175759792327881) ?  (b('dg0_v_vv') <= 3.7185704708099365) ?  0.02410958164979455 :   0.04345115555029719 :   0.07604444906297095 :   (b('dg0_v_vv') <= 0.9390630722045898) ?  (b('l8b5') <= 3882.3067626953125) ?  (b('l8b5_med') <= 3599.1788330078125) ?  0.032557547922035496 :   0.03610721841947928 :   0.05799542031249971 :   (b('moss') <= 0.9812286496162415) ?  (b('ndvi') <= 1663.5) ?  -0.06866565872549403 :   -0.018632900225479258 :   (b('dg0_v_vv') <= 1.3939342498779297) ?  -0.10251342862413712 :   -0.11377422013765953 :   (b('l8b4_med') <= 1071.31494140625) ?  (b('ndvi') <= 2498.5) ?  (b('l8b11') <= 1545.1506958007812) ?  (b('grass') <= 56.34734344482422) ?  (b('l8b11') <= 1124.4516906738281) ?  -0.05807339622187657 :   -0.05318766297874933 :   -0.07795800163020848 :   (b('dg0_v_vv') <= 4.313644886016846) ?  (b('l8b11') <= 2076.0426025390625) ?  -0.02287157584407778 :   -0.04012259646737747 :   0.025824254965976193 :   (b('ndvi') <= 2963.5) ?  (b('dg0_v_vv') <= 1.1903622150421143) ?  (b('ndvi') <= 2649.5) ?  0.04748696318457768 :   0.02614323243486843 :   (b('ndvi_med') <= 1353.5) ?  0.05320492186417791 :   -0.011632552898976073 :   (b('l8b5') <= 3288.876708984375) ?  (b('dg0_v_vv') <= 4.583925247192383) ?  -0.05466765058669736 :   -0.06407604104054604 :   (b('l8b11') <= 1058.3283386230469) ?  -0.04505160573914746 :   0.03456224879867245 :   (b('sand') <= 58.5) ?  (b('l8b11_med') <= 1982.0604248046875) ?  (b('l8b4') <= 1196.2337646484375) ?  (b('ndvi') <= 1853.0) ?  0.06001349561315483 :   0.009062879514603375 :   0.13966885704319804 :   (b('l8b11_med') <= 2374.2691650390625) ?  (b('dg0_v_vv') <= 2.4795081615448) ?  -0.01709372879592571 :   0.016437942154710956 :   (b('grass') <= 48.874887466430664) ?  -0.012894609250773181 :   0.08630904884474158 :   (b('ndvi_med') <= 1353.25) ?  (b('grass') <= 53.85307312011719) ?  (b('l8b11') <= 1890.0121459960938) ?  0.13305494908833526 :   0.13065140709140582 :   (b('ndvi_med') <= 924.0) ?  0.03365007134181769 :   0.07669913993686844 :   -0.02822769261206004 :   (b('l8b5_med') <= 2986.9957275390625) ?  (b('l8b5_med') <= 2619.382568359375) ?  (b('l8b11_med') <= 2674.5888671875) ?  (b('dg0_v_vv') <= -0.917539119720459) ?  (b('l8b4') <= 1762.5742797851562) ?  (b('dg0_v_vv') <= -1.156747817993164) ?  (b('l8b4') <= 1277.1495361328125) ?  -0.0429525875116958 :   (b('dg0_v_vv') <= -1.4367051124572754) ?  -0.017490534764876588 :   -0.029240303121508493 :   (b('l8b11') <= 2755.1663818359375) ?  0.008291915391367893 :   -0.0076675508914282875 :   (b('ndvi') <= 1251.0) ?  (b('dg0_v_vv') <= -1.5327668190002441) ?  (b('l8b5_med') <= 2576.9456787109375) ?  0.009918768059421545 :   -0.006346145021883277 :   (b('ndvi') <= 768.0) ?  0.029814708231882814 :   0.008720382250040687 :   -0.025908696406538513 :   (b('dg0_v_vv') <= 2.4067234992980957) ?  (b('dg0_v_vv') <= 0.2299060821533203) ?  (b('l8b11') <= 2503.8541259765625) ?  (b('l8b4') <= 1444.0556640625) ?  -0.020361275525018253 :   -0.03652755873287731 :   (b('l8b11') <= 3215.1953125) ?  -0.016023754317242313 :   0.005690280660801614 :   (b('dg0_v_vv') <= 2.024923801422119) ?  (b('l8b4') <= 1269.6292114257812) ?  -0.014591498810375043 :   -0.027198666557860962 :   0.009628060936123048 :   (b('ndvi') <= 1598.0) ?  (b('ndvi') <= 1317.5) ?  -0.049586667340318115 :   -0.041135474271465966 :   -0.07615731247148064 :   (b('dg0_v_vv') <= 1.2278523445129395) ?  (b('l8b11') <= 2761.0087890625) ?  (b('bulk') <= 141.5) ?  (b('dg0_v_vv') <= 0.10226726531982422) ?  0.04795982024009207 :   0.08739591508537589 :   (b('sand') <= 66.5) ?  (b('l8b5_med') <= 2281.6424560546875) ?  0.013989930684730109 :   -0.018907062058426415 :   (b('ndvi') <= 864.5) ?  0.046214616787937375 :   0.003234829487362384 :   (b('ndvi') <= 1318.0) ?  (b('ndvi') <= 1268.5) ?  (b('l8b11_med') <= 2956.887451171875) ?  -0.00860220803313061 :   -0.0226813088939857 :   0.06150138011775125 :   (b('ndvi') <= 1403.5) ?  -0.08247801492054745 :   -0.04575728522451941 :   (b('sand') <= 56.5) ?  (b('l8b5') <= 2386.6171875) ?  -0.029385259208792695 :   (b('ndvi_med') <= 1046.5) ?  -0.008927418002185106 :   (b('l8b5') <= 2562.1995849609375) ?  0.051148657094279376 :   0.02599948451629505 :   (b('dg0_v_vv') <= 4.098867416381836) ?  (b('l8b11_med') <= 2956.887451171875) ?  (b('sand') <= 65.5) ?  0.05048543433570998 :   0.04473058380867853 :   (b('l8b11_med') <= 3032.515625) ?  0.07073239903361594 :   0.06951784448046933 :   0.011575668932888195 :   (b('ndvi') <= 1707.5) ?  (b('l8b5') <= 2335.41796875) ?  (b('l8b4_med') <= 1874.479248046875) ?  (b('l8b11') <= 3061.0208740234375) ?  (b('l8b11') <= 1271.1591796875) ?  (b('dg0_v_vv') <= 0.23308420181274414) ?  0.05930112150357894 :   0.04162175396908338 :   (b('ndvi_med') <= 1498.0) ?  0.02239306311496279 :   -0.005290997033400702 :   (b('l8b5') <= 2282.983154296875) ?  (b('dg0_v_vv') <= 0.6815071105957031) ?  -0.040816025243176665 :   -0.015435123703499525 :   (b('l8b5_med') <= 2723.5404052734375) ?  -0.008923267027584891 :   0.0383715248802481 :   (b('l8b4') <= 1635.1865234375) ?  (b('l8b11') <= 2253.4625244140625) ?  (b('dg0_v_vv') <= -0.07635021209716797) ?  -0.06529153035248116 :   -0.03353411677709577 :   (b('ndvi') <= 1616.5) ?  -0.014533299784555961 :   0.0055180926299881194 :   (b('sand') <= 41.5) ?  0.010564249086295066 :   0.05811526354150115 :   (b('sand') <= 60.5) ?  (b('l8b11') <= 3029.3798828125) ?  (b('ndvi') <= 1286.0) ?  (b('grass') <= 9.842857360839844) ?  0.0017492408569353135 :   -0.02452686219059644 :   (b('l8b4_med') <= 1741.3679809570312) ?  -0.011184845295269608 :   0.010772347138488592 :   (b('grass') <= 70.93289947509766) ?  (b('moss') <= 0.010067113675177097) ?  0.06878742083379999 :   0.011878174511038574 :   (b('dg0_v_vv') <= 0.4697542190551758) ?  -0.016654213445630034 :   0.021536587393459348 :   (b('dg0_v_vv') <= 1.8102235794067383) ?  (b('l8b5') <= 3153.210693359375) ?  (b('dg0_v_vv') <= 0.9147429466247559) ?  -0.021643860580326763 :   -0.04454087658512764 :   (b('l8b5') <= 3417.0654296875) ?  0.047707583700885486 :   -0.002025413079780214 :   (b('l8b4') <= 1731.3177490234375) ?  0.052406440152611145 :   (b('l8b11') <= 3216.0501708984375) ?  -0.0040532680647862585 :   -0.0026313775402185408 :   (b('dg0_v_vv') <= -1.1270875930786133) ?  (b('ndvi') <= 2688.5) ?  (b('ndvi') <= 2114.5) ?  (b('dg0_v_vv') <= -1.8887977600097656) ?  (b('l8b4') <= 1665.7903442382812) ?  -0.038169789942946876 :   -0.005846481412546747 :   (b('ndvi') <= 1905.5) ?  -0.0028959731798959033 :   0.04126734529904033 :   (b('ndvi') <= 2431.5) ?  (b('l8b4_med') <= 1470.9869995117188) ?  -0.06161853682894014 :   -0.06707287221163742 :   (b('l8b4_med') <= 1656.4095458984375) ?  -0.029053689370795093 :   -0.03469460780037195 :   (b('bulk') <= 151.0) ?  (b('l8b11') <= 1924.4166870117188) ?  (b('l8b5') <= 3888.3819580078125) ?  0.020074686956386697 :   0.011703822295840799 :   0.045581606043931244 :   (b('ndvi') <= 3693.5) ?  -0.005784627582475055 :   (b('l8b4_med') <= 1788.063232421875) ?  -0.012408245414552738 :   -0.015091494609166728 :   (b('l8b4') <= 1145.997314453125) ?  (b('l8b4') <= 1118.3089599609375) ?  (b('l8b5') <= 2973.2457275390625) ?  (b('l8b4_med') <= 1841.5863647460938) ?  0.004466808738439172 :   0.0327954751900367 :   (b('l8b11') <= 1564.5322875976562) ?  -0.024157026640131203 :   -0.05095669583285045 :   -0.07749647088575197 :   (b('l8b11') <= 1625.6854858398438) ?  -0.06691502914401384 :   (b('l8b4') <= 1207.4856567382812) ?  (b('l8b4') <= 1166.4990844726562) ?  0.019838678563334017 :   0.06537094922119832 :   (b('dg0_v_vv') <= 1.8135042190551758) ?  0.028789463747261088 :   0.006783295291119064 :   (b('l8b4_med') <= 2187.779052734375) ?  (b('dg0_v_vv') <= 3.547208786010742) ?  (b('bulk') <= 150.5) ?  (b('l8b4') <= 1741.2278442382812) ?  (b('ndvi') <= 1082.5) ?  (b('l8b4_med') <= 1850.4474487304688) ?  -0.08400112093597044 :   -0.0567404585656979 :   (b('l8b4') <= 1723.8724975585938) ?  (b('l8b4') <= 1697.8351440429688) ?  0.00041535416916654286 :   -0.03521230272312766 :   (b('ndvi') <= 1309.0) ?  0.10973334531425054 :   0.013904890661028001 :   (b('l8b11') <= 2966.25830078125) ?  (b('ndvi') <= 1235.0) ?  (b('dg0_v_vv') <= 0.27808237075805664) ?  -0.04233580887339317 :   -0.06780581445648844 :   (b('ndvi') <= 1540.5) ?  -0.011833395955522262 :   -0.025054980638358582 :   (b('l8b5') <= 2900.451416015625) ?  (b('l8b5') <= 2791.5035400390625) ?  -0.016588175883868583 :   0.02807361395408212 :   (b('l8b11') <= 3437.5062255859375) ?  -0.015627801361180443 :   0.003727537972007711 :   (b('l8b11') <= 1655.5509643554688) ?  (b('l8b5_med') <= 3221.940673828125) ?  (b('l8b5') <= 3067.36962890625) ?  (b('dg0_v_vv') <= 0.6011505126953125) ?  -0.02654309987298724 :   -0.015696331532075534 :   (b('l8b11_med') <= 2831.5748291015625) ?  -0.04725212391241028 :   -0.0448642273964637 :   (b('l8b11') <= 1494.0662841796875) ?  (b('l8b11') <= 1348.8751220703125) ?  -0.06508259011141 :   -0.08198462394494932 :   (b('l8b5_med') <= 3322.7132568359375) ?  -0.04616951842394637 :   -0.04009019091053318 :   (b('dg0_v_vv') <= -0.2211298942565918) ?  (b('l8b5') <= 2072.508056640625) ?  (b('ndvi_med') <= 1766.0) ?  0.020386323529241947 :   -0.004301952921295427 :   (b('l8b11') <= 1849.5628051757812) ?  -0.03300034271445404 :   -0.019211946894209322 :   (b('l8b11') <= 3348.8629150390625) ?  (b('l8b11') <= 3150.8740234375) ?  -0.014503291240170668 :   -0.0008329956765891622 :   (b('l8b11') <= 3449.38134765625) ?  -0.030807161309877342 :   -0.01578310376210083 :   (b('moss') <= 8.284094095230103) ?  (b('dg0_v_vv') <= 3.9790637493133545) ?  (b('l8b11') <= 2907.2152099609375) ?  (b('l8b11') <= 2517.7357177734375) ?  (b('l8b4_med') <= 1946.1463012695312) ?  0.048623558128632866 :   0.04525439336082676 :   (b('ndvi_med') <= 1636.25) ?  0.03685676381773184 :   0.03290458669509095 :   0.07059005641236105 :   (b('l8b4_med') <= 1803.0075073242188) ?  -0.047188398734663906 :   (b('ndvi_med') <= 1766.0) ?  (b('l8b11') <= 2604.5220947265625) ?  0.027263710707188367 :   0.008152567209721701 :   0.05840934998664393 :   (b('moss') <= 19.390490531921387) ?  (b('l8b11') <= 2890.6590576171875) ?  (b('l8b11_med') <= 3129.853515625) ?  (b('ndvi') <= 1124.5) ?  -0.018769085354871703 :   -0.004022887337026055 :   -0.03776413802295306 :   (b('l8b5') <= 3350.5311279296875) ?  (b('l8b11') <= 3066.4068603515625) ?  0.056276003106794814 :   0.022742495194698148 :   (b('l8b11') <= 3695.9931640625) ?  -0.013176755821317283 :   0.02712865103169547 :   (b('bulk') <= 150.5) ?  -0.06759866376090952 :   (b('ndvi') <= 1485.0) ?  -0.014949760694241525 :   (b('l8b11') <= 3766.29833984375) ?  -0.04020591816377418 :   -0.03866842027480471 :   (b('l8b11') <= 3084.9400634765625) ?  (b('sand') <= 65.0) ?  (b('ndvi') <= 801.5) ?  (b('l8b11') <= 2359.6021728515625) ?  -0.05727406111079218 :   (b('l8b11') <= 2694.018310546875) ?  (b('l8b4') <= 2296.58642578125) ?  0.011469709964910771 :   0.05791615278833351 :   (b('bulk') <= 151.0) ?  -0.019272885413823684 :   0.005009934938304238 :   (b('ndvi') <= 819.5) ?  (b('dg0_v_vv') <= -0.852637767791748) ?  (b('ndvi_med') <= 751.0) ?  0.0255567245769059 :   0.03817930304564204 :   (b('ndvi_med') <= 713.5) ?  0.048410148123460756 :   0.06408664823376933 :   (b('ndvi') <= 1014.5) ?  (b('l8b11') <= 2800.557861328125) ?  -0.02112240019354433 :   0.00820930961140034 :   (b('dg0_v_vv') <= -0.821141242980957) ?  -0.006030889626078633 :   0.012319650743375033 :   (b('dg0_v_vv') <= 1.2369332313537598) ?  (b('bulk') <= 148.5) ?  -0.03231120727483951 :   -0.02952724125266122 :   -0.08710154165905307 :   (b('sand') <= 57.5) ?  (b('l8b11') <= 3097.0919189453125) ?  (b('dg0_v_vv') <= 1.1466684341430664) ?  (b('l8b5_med') <= 3622.8421630859375) ?  (b('grass') <= 16.54544100165367) ?  -0.036657599400184857 :   -0.04808346815307998 :   -0.0186300342700825 :   -0.07152520697721901 :   (b('l8b11') <= 3297.080078125) ?  (b('l8b11_med') <= 3264.05029296875) ?  (b('ndvi') <= 1161.0) ?  0.013077551874564313 :   -0.003368161409338299 :   (b('l8b5') <= 3125.1754150390625) ?  0.056020900235134174 :   0.01707428939650475 :   (b('l8b11') <= 3394.9947509765625) ?  (b('dg0_v_vv') <= 0.6518311500549316) ?  -0.019699353022401785 :   -0.07602421406793813 :   (b('l8b11') <= 3401.80078125) ?  0.05755602118295233 :   -0.007190629035458124 :   (b('ndvi') <= 902.0) ?  (b('l8b4') <= 2853.9566650390625) ?  (b('ndvi') <= 779.0) ?  (b('l8b11_med') <= 3224.09033203125) ?  0.005502347154696584 :   -0.021602934820321187 :   (b('l8b5_med') <= 3130.3966064453125) ?  -0.036515208320047655 :   -0.04692031368996339 :   (b('l8b11') <= 3380.650390625) ?  0.04559837360907837 :   (b('ndvi') <= 876.0) ?  -0.008777019467656924 :   -0.024248427779570335 :   (b('dg0_v_vv') <= -0.3388223648071289) ?  (b('l8b5_med') <= 3130.3966064453125) ?  (b('l8b5') <= 3352.7010498046875) ?  0.032201360597779366 :   0.05613530215603983 :   (b('dg0_v_vv') <= -0.3649177551269531) ?  -0.007935292283499592 :   0.06454664367463295 :   (b('dg0_v_vv') <= 0.5756058692932129) ?  (b('l8b11_med') <= 3224.09033203125) ?  -0.03049399415509946 :   -0.015594224866778436 :   (b('bulk') <= 151.5) ?  0.05086156658390416 :   -0.0015191560416737777 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 738.9322509765625) ?  (b('grass') <= 46.231340408325195) ?  (b('sand') <= 24.5) ?  (b('ndvi_med') <= 3316.75) ?  (b('crops') <= 92.60075759887695) ?  (b('l8b11') <= 1340.0193481445312) ?  (b('ndvi_med') <= 2107.5) ?  (b('dg0_v_vv') <= -2.6576266288757324) ?  (b('dg0_v_vv') <= -3.1387853622436523) ?  0.004987122232837982 :   (b('l8b4') <= 480.4256286621094) ?  0.06937097971591216 :   0.09058843127861027 :   (b('l8b5_med') <= 2844.591552734375) ?  (b('ndvi') <= 4029.0) ?  -0.012723459148824667 :   -0.019463858626690478 :   -0.0013660811909367576 :   (b('l8b4') <= 348.14906311035156) ?  (b('ndvi') <= 3342.0) ?  0.014012523442954367 :   0.039598252255477445 :   (b('ndvi') <= 3271.5) ?  (b('ndvi') <= 2559.0) ?  -0.02937764910522625 :   0.018945068584672653 :   (b('ndvi_med') <= 2791.25) ?  -0.005166056503329568 :   -0.03161274501959634 :   (b('dg0_v_vv') <= 0.18208932876586914) ?  0.07151362663224409 :   0.051954108552333955 :   (b('dg0_v_vv') <= -1.5238447189331055) ?  (b('l8b5_med') <= 2922.309814453125) ?  0.05052900523378562 :   0.06728194622927813 :   (b('dg0_v_vv') <= 3.1284258365631104) ?  (b('l8b4_med') <= 721.6394348144531) ?  0.02803310924387581 :   (b('bulk') <= 115.5) ?  0.035560247767012554 :   0.034375802412232925 :   0.05797731073239276 :   (b('dg0_v_vv') <= -1.4297451972961426) ?  (b('l8b11') <= 1262.7715454101562) ?  (b('ndvi') <= 3272.0) ?  (b('dg0_v_vv') <= -1.7900390625) ?  (b('dg0_v_vv') <= -2.782949924468994) ?  (b('l8b11') <= 953.9575500488281) ?  -0.03832585435654107 :   -0.019256525580177236 :   (b('l8b4') <= 463.9608154296875) ?  0.05761992385545553 :   0.020095015969928087 :   -0.05938006738266169 :   (b('ndvi_med') <= 3729.5) ?  (b('grass') <= 32.82162952423096) ?  (b('ndvi_med') <= 3391.25) ?  -0.030052093679069344 :   -0.03581719324566508 :   -0.04667882643169924 :   (b('l8b4_med') <= 632.78857421875) ?  (b('l8b4_med') <= 606.2927856445312) ?  -0.006939696049322097 :   -0.021318206770499726 :   0.04500902306201446 :   (b('sand') <= 23.5) ?  0.0674709780119363 :   0.04578161844378337 :   (b('l8b4') <= 445.79673767089844) ?  (b('l8b11') <= 540.7763977050781) ?  (b('dg0_v_vv') <= 0.2561492919921875) ?  -0.010727453518652286 :   -0.014484579557566463 :   (b('bulk') <= 123.5) ?  0.11951039903408067 :   (b('dg0_v_vv') <= -1.063124179840088) ?  0.10369745466567387 :   (b('dg0_v_vv') <= -0.8063335418701172) ?  0.01868209060869433 :   0.0660194757270652 :   (b('ndvi') <= 3939.5) ?  (b('l8b4') <= 503.5918731689453) ?  (b('l8b4') <= 483.272705078125) ?  0.027520706549478358 :   (b('dg0_v_vv') <= -0.5115666389465332) ?  -0.0245367338736944 :   -0.06325374162054082 :   (b('l8b4_med') <= 567.5329284667969) ?  (b('dg0_v_vv') <= -0.07290124893188477) ?  0.11465252716184049 :   0.08598540426639434 :   (b('l8b11_med') <= 1457.8485717773438) ?  0.03546571648619152 :   0.0681287230493657 :   (b('ndvi') <= 4921.5) ?  (b('ndvi_med') <= 3412.25) ?  (b('dg0_v_vv') <= -1.1745233535766602) ?  -0.013857988060021836 :   -0.03848462090613162 :   (b('dg0_v_vv') <= -0.41097545623779297) ?  0.060390493208977006 :   0.006658284417759448 :   (b('dg0_v_vv') <= -0.1945948600769043) ?  (b('ndvi') <= 4977.0) ?  0.04417612293761147 :   0.08548069220737492 :   0.09838647942082024 :   (b('l8b5_med') <= 2994.1806640625) ?  (b('ndvi') <= 3621.5) ?  (b('l8b4_med') <= 562.5854187011719) ?  (b('dg0_v_vv') <= -0.34255313873291016) ?  (b('ndvi') <= 2935.0) ?  (b('l8b11') <= 1062.8591613769531) ?  (b('l8b4') <= 508.53662109375) ?  0.04051297144526172 :   -0.019198180292273367 :   (b('l8b5') <= 2940.280029296875) ?  0.09245535919708342 :   -0.002184099812427953 :   (b('l8b5') <= 2534.0521240234375) ?  (b('l8b5') <= 2433.8773193359375) ?  0.007027636328697212 :   0.10808270923152227 :   (b('l8b4') <= 410.6142272949219) ?  -0.08280614576415873 :   -0.02448020960354197 :   (b('l8b5') <= 2424.0858154296875) ?  (b('l8b5') <= 2389.42529296875) ?  (b('l8b4_med') <= 447.916259765625) ?  -0.026875467114629847 :   0.051953876489950125 :   -0.0665776360092501 :   (b('l8b5') <= 2749.788818359375) ?  (b('dg0_v_vv') <= 0.5204377174377441) ?  0.07520904088985521 :   0.026134071134886187 :   (b('l8b4') <= 639.5362548828125) ?  0.039599874819193345 :   -0.03666902718591167 :   (b('ndvi') <= 3341.0) ?  (b('dg0_v_vv') <= 6.733710289001465) ?  (b('l8b11') <= 1707.5287475585938) ?  (b('bulk') <= 155.5) ?  0.002812042979743609 :   -0.02314170770756476 :   (b('dg0_v_vv') <= -0.22703218460083008) ?  -0.04168627042110863 :   -0.015029978003755906 :   (b('bulk') <= 156.0) ?  (b('dg0_v_vv') <= 7.052134037017822) ?  -0.07548341004794204 :   -0.08334027164368277 :   -0.0482460972665133 :   (b('l8b5') <= 2169.8046875) ?  (b('l8b5') <= 2100.849853515625) ?  (b('l8b5_med') <= 2149.7215576171875) ?  0.05526370287583302 :   0.029422037518305383 :   (b('dg0_v_vv') <= -0.38053035736083984) ?  0.10235215779423468 :   0.07567151298024678 :   (b('grass') <= 36.396745681762695) ?  (b('l8b11') <= 899.4151000976562) ?  0.050023595524961384 :   0.010345918210339977 :   (b('dg0_v_vv') <= -1.0536179542541504) ?  -0.11538496244358559 :   -0.010877569544473801 :   (b('sand') <= 38.5) ?  (b('ndvi') <= 4251.5) ?  (b('l8b11') <= 751.5029907226562) ?  (b('grass') <= 36.078264236450195) ?  (b('dg0_v_vv') <= 3.2471835613250732) ?  -0.04323493905910081 :   -0.05949834680315018 :   (b('l8b4_med') <= 581.4598999023438) ?  -0.019850363717709235 :   -0.03258927556668227 :   (b('l8b4') <= 527.1026000976562) ?  (b('crops') <= 70.03668594360352) ?  0.005424038603730743 :   0.04007686807824425 :   (b('ndvi') <= 3937.5) ?  -0.003134499230639826 :   -0.017323686160671776 :   (b('ndvi_med') <= 3318.5) ?  (b('l8b4_med') <= 713.3016967773438) ?  (b('ndvi') <= 4359.0) ?  0.019425135514213623 :   0.04699908773452857 :   (b('l8b4') <= 737.5427856445312) ?  0.00546093215992227 :   0.05599986708267345 :   (b('ndvi') <= 4925.0) ?  (b('dg0_v_vv') <= 2.223212718963623) ?  -0.019028707249320435 :   0.031224523304667743 :   (b('ndvi') <= 4975.5) ?  0.031521256429953994 :   -0.027767063693600436 :   (b('ndvi_med') <= 3577.0) ?  (b('l8b4') <= 622.3432922363281) ?  (b('l8b4') <= 616.2509460449219) ?  (b('l8b5') <= 2364.66064453125) ?  0.030882722253397433 :   -0.029026596011366576 :   (b('l8b5_med') <= 2962.1964111328125) ?  0.09092598615198236 :   0.03677595270120946 :   (b('dg0_v_vv') <= 0.23943519592285156) ?  (b('l8b5') <= 3571.749755859375) ?  -0.040543437912699175 :   -0.07775916100305452 :   (b('l8b11') <= 1695.8831176757812) ?  -0.06662312891201091 :   -0.0533720400831879 :   (b('sand') <= 42.5) ?  (b('l8b11') <= 1137.41552734375) ?  (b('ndvi') <= 3846.5) ?  0.07155382257081844 :   0.010157025970385966 :   0.14590506520681304 :   (b('ndvi') <= 3666.0) ?  (b('dg0_v_vv') <= 1.0486037731170654) ?  -0.03754697379114327 :   -0.06829331046722839 :   (b('ndvi') <= 4255.5) ?  0.003031008567042853 :   -0.013181976395996775 :   (b('ndvi') <= 2525.5) ?  (b('ndvi') <= 2478.5) ?  (b('ndvi_med') <= 2306.25) ?  -0.09175926917884036 :   (b('l8b5') <= 2810.139892578125) ?  (b('l8b5_med') <= 3090.0008544921875) ?  (b('dg0_v_vv') <= 0.7283611297607422) ?  0.01585110573745213 :   0.08402750038165042 :   (b('dg0_v_vv') <= -1.077986240386963) ?  -0.004310332316853936 :   0.014665587925462777 :   (b('l8b4_med') <= 573.5225219726562) ?  (b('dg0_v_vv') <= -0.27724695205688477) ?  0.05336971684579597 :   0.10151013899266921 :   (b('ndvi_med') <= 2907.75) ?  0.06360058680049922 :   0.026268902868136823 :   0.18038784726889878 :   (b('ndvi') <= 3220.0) ?  (b('ndvi') <= 3177.5) ?  (b('crops') <= 65.37240409851074) ?  (b('l8b4') <= 532.2679443359375) ?  (b('dg0_v_vv') <= -0.0184783935546875) ?  0.02523308122942933 :   -0.007786679662219183 :   (b('l8b4_med') <= 567.5584106445312) ?  0.008485805706643867 :   -0.037501515249026326 :   (b('crops') <= 95.3808479309082) ?  (b('moss') <= 0.542781189084053) ?  0.010318827124601365 :   0.042714887267939546 :   (b('crops') <= 97.94236373901367) ?  -0.05502573202565661 :   -0.03948772374466297 :   (b('dg0_v_vv') <= 0.15579986572265625) ?  (b('grass') <= 17.025083541870117) ?  (b('l8b5') <= 3586.845703125) ?  -0.05194524905150079 :   -0.06117402032606847 :   (b('moss') <= 2.4998489320278168) ?  -0.09323718396033803 :   -0.0776965829306262 :   -0.13353210281162037 :   (b('ndvi_med') <= 3309.0) ?  (b('ndvi') <= 3840.0) ?  (b('l8b5') <= 2826.3795166015625) ?  (b('l8b5') <= 2388.8016357421875) ?  -0.017735845326644496 :   0.049972028209382374 :   (b('dg0_v_vv') <= 0.3040783405303955) ?  -0.06319278102103705 :   -0.03582378594009982 :   (b('bulk') <= 147.0) ?  (b('ndvi') <= 4842.0) ?  -0.0033803221527930294 :   -0.04000575180160195 :   (b('l8b11') <= 1104.2649841308594) ?  0.02955470275817266 :   -0.014017198036596235 :   (b('dg0_v_vv') <= -1.1794862747192383) ?  (b('ndvi') <= 3792.5) ?  (b('l8b5') <= 2312.16357421875) ?  0.04621315110600522 :   0.01156863260881315 :   (b('ndvi') <= 4647.5) ?  -0.0161428786161899 :   0.0036048864540125006 :   (b('ndvi_med') <= 3384.75) ?  (b('l8b11') <= 1147.2174682617188) ?  0.03571279838366864 :   0.12667202824776352 :   (b('moss') <= 0.7406862676143646) ?  -0.0006247207387080874 :   0.015363864358471108 :   (b('moss') <= 12.215838432312012) ?  (b('sand') <= 36.5) ?  (b('l8b11_med') <= 2072.46923828125) ?  (b('l8b4') <= 682.9836730957031) ?  (b('l8b4') <= 675.1580810546875) ?  (b('dg0_v_vv') <= 2.695847511291504) ?  (b('l8b5_med') <= 2944.8780517578125) ?  (b('dg0_v_vv') <= 1.4988517761230469) ?  -0.016708188941577055 :   0.027720545544767116 :   (b('dg0_v_vv') <= -0.000499725341796875) ?  0.03273184927993915 :   -0.004470631230569119 :   (b('ndvi') <= 3919.0) ?  (b('ndvi_med') <= 3799.5) ?  0.12515374869631601 :   0.08385288591081175 :   (b('ndvi') <= 4167.0) ?  -0.0028941894948585345 :   0.05387741077930887 :   (b('sand') <= 28.5) ?  (b('ndvi_med') <= 4049.0) ?  (b('dg0_v_vv') <= 0.795811653137207) ?  0.10584696004841158 :   0.06599319047059737 :   (b('ndvi') <= 4202.5) ?  0.051911895099819416 :   0.04536974753009537 :   -0.010525333217053151 :   (b('l8b5_med') <= 3024.054931640625) ?  (b('l8b4_med') <= 815.4116516113281) ?  (b('l8b11') <= 1409.3512573242188) ?  (b('sand') <= 18.0) ?  -0.10512562100471778 :   -0.1066588178149025 :   (b('dg0_v_vv') <= -0.2320270538330078) ?  -0.04995934958845094 :   -0.0753040987305826 :   (b('dg0_v_vv') <= 1.192842960357666) ?  (b('l8b5_med') <= 2620.4422607421875) ?  -0.05141711611492104 :   -0.073061387292659 :   -0.002563556859748517 :   (b('l8b5') <= 3408.5096435546875) ?  (b('l8b11_med') <= 1426.9572143554688) ?  (b('dg0_v_vv') <= -4.5299530029296875e-05) ?  0.026710604185120612 :   0.03372066242091801 :   0.019992595334950847 :   (b('l8b11') <= 1412.0072631835938) ?  -0.03005074095703794 :   -0.0031158635551138403 :   (b('l8b4_med') <= 1282.8197631835938) ?  (b('l8b11') <= 1417.0043334960938) ?  (b('l8b4') <= 639.5804443359375) ?  (b('ndvi') <= 4875.5) ?  (b('l8b5_med') <= 2572.6190185546875) ?  0.04582087367310566 :   0.0064514382883208155 :   (b('dg0_v_vv') <= 0.42106056213378906) ?  0.16432839176995717 :   0.002052268343242347 :   (b('dg0_v_vv') <= 3.9447684288024902) ?  (b('dg0_v_vv') <= 1.864516258239746) ?  -0.03349978305626272 :   -0.06900029176921552 :   0.02977183019864288 :   (b('l8b4') <= 732.9704284667969) ?  (b('l8b11') <= 1606.631103515625) ?  (b('grass') <= 74.82279586791992) ?  0.07552725527970733 :   0.07432789149046665 :   (b('crops') <= 0.2781457006931305) ?  0.11394902640980847 :   0.10177188773093615 :   (b('l8b4') <= 736.1998291015625) ?  0.018713657352422475 :   (b('l8b11_med') <= 2118.291015625) ?  0.04249928885216647 :   0.04352310386032365 :   (b('ndvi') <= 4618.5) ?  (b('l8b11_med') <= 2290.540283203125) ?  0.10617758267494082 :   0.10347118983814813 :   (b('ndvi_med') <= 1602.25) ?  0.11685672579361106 :   0.12628953593343967 :   (b('grass') <= 79.0) ?  (b('grass') <= 68.66118240356445) ?  (b('l8b11_med') <= 1682.8125610351562) ?  (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 1382.3003540039062) ?  (b('l8b5') <= 2467.399658203125) ?  0.0377258933992936 :   0.018843829605493998 :   (b('moss') <= 8.13626503944397) ?  -0.02623291842794716 :   0.06344286778381676 :   (b('l8b5') <= 2405.21044921875) ?  (b('dg0_v_vv') <= -1.8863086700439453) ?  -0.003230163600310706 :   0.029586903055288195 :   (b('l8b4') <= 544.4970397949219) ?  0.046198968404481275 :   0.03553294675450199 :   (b('l8b4_med') <= 1395.4902954101562) ?  (b('ndvi_med') <= 1617.0) ?  -0.14416135463667223 :   (b('l8b11') <= 1628.7042846679688) ?  -0.005227070242334777 :   0.0515552127221034 :   (b('l8b11_med') <= 2259.6182861328125) ?  0.013813881692376684 :   (b('l8b11') <= 1254.5314331054688) ?  0.11472881545600888 :   0.04841758016472139 :   (b('l8b11') <= 1072.9926147460938) ?  (b('ndvi') <= 3855.0) ?  (b('l8b4_med') <= 1089.9920654296875) ?  (b('ndvi') <= 2815.5) ?  0.07563622865840747 :   0.09760160618446498 :   (b('ndvi_med') <= 1912.5) ?  0.05963929298026718 :   -0.03106383863640667 :   (b('l8b5_med') <= 2407.56005859375) ?  (b('l8b5_med') <= 2308.3662109375) ?  -0.043331135438404944 :   -0.05392746442951947 :   -0.028290687089375655 :   (b('moss') <= 10.623213291168213) ?  (b('dg0_v_vv') <= -0.17641258239746094) ?  (b('grass') <= 74.67932510375977) ?  0.022488220892129374 :   0.07636894915827895 :   (b('grass') <= 77.0) ?  0.09983836546649803 :   0.16382500507193318 :   (b('l8b4_med') <= 1105.3286743164062) ?  (b('dg0_v_vv') <= -1.6161761283874512) ?  0.1413299128970682 :   0.13636854287113928 :   0.12923251595021462 :   (b('l8b4') <= 692.7525634765625) ?  (b('l8b4_med') <= 1557.8544311523438) ?  (b('sand') <= 44.5) ?  (b('l8b4') <= 463.4264221191406) ?  (b('dg0_v_vv') <= 2.2241530418395996) ?  -0.00189525515861462 :   -0.017536276334061185 :   (b('ndvi') <= 2366.0) ?  0.1268359132525066 :   0.041248511021858295 :   (b('l8b4') <= 685.0) ?  (b('l8b4') <= 595.4923095703125) ?  0.012188080127479522 :   -0.02154653073035518 :   (b('dg0_v_vv') <= 1.7865188121795654) ?  0.11750944916710909 :   0.008392739852166653 :   (b('ndvi') <= 3636.0) ?  (b('dg0_v_vv') <= 2.8036131858825684) ?  0.0307455837168798 :   0.011337807881382 :   (b('l8b4_med') <= 1636.6956787109375) ?  -0.11274240066866353 :   -0.08935292888098442 :   (b('l8b5') <= 1822.29052734375) ?  -0.07953303489796781 :   (b('dg0_v_vv') <= 0.9607319831848145) ?  (b('l8b5_med') <= 2370.6107177734375) ?  (b('ndvi') <= 2051.0) ?  3.5240166692818153e-06 :   0.03494616206398117 :   (b('ndvi') <= 4212.0) ?  -0.02280378951130802 :   0.007557889924968628 :   (b('dg0_v_vv') <= 2.90329647064209) ?  (b('bulk') <= 123.0) ?  -0.03817048633846237 :   -0.02298436222329711 :   -0.07474677412782954 :   (b('l8b11') <= 1620.0960693359375) ?  (b('sand') <= 29.5) ?  (b('l8b4') <= 703.8435668945312) ?  (b('l8b4_med') <= 810.0767517089844) ?  (b('dg0_v_vv') <= -3.0703907012939453) ?  (b('moss') <= 14.804993152618408) ?  -0.0339146491016101 :   -0.035713882519845974 :   (b('dg0_v_vv') <= 0.4241056442260742) ?  (b('dg0_v_vv') <= 0.050867557525634766) ?  0.06277388489969009 :   0.1230742798501494 :   (b('dg0_v_vv') <= 0.6863269805908203) ?  -0.040033142281913514 :   0.042606602112311966 :   (b('l8b5') <= 2282.3214111328125) ?  -0.026876591392963278 :   (b('ndvi') <= 4489.0) ?  (b('dg0_v_vv') <= 0.9855685234069824) ?  -0.013410240945347512 :   -0.012028966745642578 :   -0.00675441505659885 :   (b('ndvi') <= 3679.0) ?  0.12511500834219536 :   0.1341769784386851 :   (b('l8b4_med') <= 1136.119873046875) ?  (b('l8b4') <= 529.5194396972656) ?  (b('bulk') <= 122.5) ?  (b('sand') <= 50.5) ?  -0.04951266974013688 :   -0.07957649181053755 :   (b('dg0_v_vv') <= -0.8158531188964844) ?  (b('sand') <= 65.5) ?  -0.021991990498408946 :   0.05037075246978312 :   (b('l8b11_med') <= 1472.2579956054688) ?  0.01824932570054895 :   0.05602325396854023 :   (b('ndvi') <= 2381.5) ?  (b('ndvi') <= 2141.0) ?  (b('moss') <= 13.23233699798584) ?  0.0243696904258351 :   -0.010284543546364973 :   (b('ndvi_med') <= 1571.5) ?  0.07348779993144802 :   0.0011213412103820708 :   (b('l8b4') <= 715.6927185058594) ?  (b('l8b5_med') <= 2314.54345703125) ?  -0.038954704926390187 :   -0.01632831641391012 :   (b('l8b5') <= 2414.0198974609375) ?  0.014583164532120019 :   -0.029276676177356912 :   (b('moss') <= 24.150099754333496) ?  (b('dg0_v_vv') <= 2.9946327209472656) ?  0.038653111974422005 :   (b('l8b11_med') <= 2552.1595458984375) ?  0.055369501047822234 :   0.05037370047800144 :   (b('l8b11_med') <= 2415.8519287109375) ?  0.08510895786188788 :   0.11563202693086477 :   (b('l8b5') <= 1361.23095703125) ?  (b('ndvi') <= 1042.0) ?  -0.03237817242633528 :   -0.0026522011918847233 :   (b('l8b5') <= 2392.9095458984375) ?  (b('ndvi') <= 2797.5) ?  (b('l8b11_med') <= 1557.0966796875) ?  (b('l8b11_med') <= 1472.2579956054688) ?  (b('dg0_v_vv') <= 0.5151247978210449) ?  0.0947321454487178 :   0.07846858185886513 :   (b('ndvi') <= 1482.0) ?  0.08895600554824437 :   0.05919080344569251 :   0.14953929706632751 :   (b('l8b4_med') <= 799.2757568359375) ?  (b('ndvi_med') <= 2523.25) ?  0.005252860534897791 :   0.007601232426241097 :   0.012579811522813913 :   (b('ndvi') <= 3367.5) ?  0.16307740839979468 :   0.0698966118369519 :   (b('moss') <= 8.65673303604126) ?  (b('l8b11') <= 1263.4005737304688) ?  (b('ndvi_med') <= 4007.25) ?  (b('grass') <= 41.3468074798584) ?  (b('l8b5') <= 3214.1798095703125) ?  (b('bulk') <= 147.5) ?  (b('l8b11') <= 1090.5578002929688) ?  (b('ndvi') <= 2026.5) ?  0.055127735544614476 :   (b('crops') <= 89.40567779541016) ?  0.03892741409065763 :   0.006794569562341313 :   (b('l8b11_med') <= 1844.76220703125) ?  (b('ndvi') <= 4018.0) ?  -0.021563101067939405 :   0.04852713618646004 :   (b('ndvi') <= 2360.5) ?  0.0058897738583600125 :   0.049550652461509435 :   (b('dg0_v_vv') <= 1.3602666854858398) ?  (b('l8b11') <= 1149.2780151367188) ?  (b('ndvi') <= 2986.0) ?  -0.03972985590642948 :   0.020931714273192223 :   (b('l8b5') <= 2968.0274658203125) ?  0.040980105552911476 :   2.2390232690822076e-05 :   (b('ndvi_med') <= 2029.0) ?  (b('dg0_v_vv') <= 1.7287585735321045) ?  0.055979704704163025 :   0.03883936470222411 :   (b('l8b11') <= 1138.47021484375) ?  0.09371134524875119 :   0.0749809192391816 :   (b('dg0_v_vv') <= 3.975674867630005) ?  (b('l8b11') <= 1080.60498046875) ?  (b('dg0_v_vv') <= -3.445002555847168) ?  0.02406778532332815 :   (b('l8b11_med') <= 1676.4300537109375) ?  0.09033311898262307 :   0.06621503801473147 :   (b('ndvi') <= 2236.5) ?  (b('l8b4_med') <= 1088.479736328125) ?  0.08687869811410609 :   0.07544627493600228 :   (b('ndvi_med') <= 2585.5) ?  0.044343915047625755 :   0.023696173213082344 :   -0.07027032807023437 :   (b('l8b11') <= 1191.4013671875) ?  (b('l8b11') <= 1168.328369140625) ?  (b('ndvi') <= 3764.0) ?  (b('grass') <= 58.67845153808594) ?  -0.06369404343233215 :   -0.0030464695170287492 :   0.008717584230947067 :   0.09042737404340415 :   (b('crops') <= 26.929054260253906) ?  (b('l8b4') <= 848.9505310058594) ?  -0.08191188932600058 :   -0.09648678475158465 :   -0.1308397100374901 :   (b('dg0_v_vv') <= 0.8795747756958008) ?  (b('l8b11_med') <= 1426.873779296875) ?  (b('l8b5') <= 3402.2991943359375) ?  0.12983341672375937 :   0.15265896661645556 :   0.09428832260063136 :   (b('dg0_v_vv') <= 1.9440944194793701) ?  0.02470818276863096 :   (b('l8b11') <= 1155.7753295898438) ?  0.09838833224324828 :   0.10281993221797758 :   (b('dg0_v_vv') <= 0.2229013442993164) ?  (b('l8b11_med') <= 2386.9385986328125) ?  (b('crops') <= 51.889204025268555) ?  (b('moss') <= 6.610314607620239) ?  (b('bulk') <= 162.5) ?  (b('dg0_v_vv') <= 0.21463966369628906) ?  (b('dg0_v_vv') <= -3.1430373191833496) ?  -0.03294406908664073 :   0.0023551242221408627 :   (b('ndvi_med') <= 3156.75) ?  -0.053212963190377625 :   -0.10392841710161715 :   (b('dg0_v_vv') <= -0.694758415222168) ?  (b('l8b5') <= 2872.513427734375) ?  -0.0037261094689782733 :   0.06812808032652115 :   (b('dg0_v_vv') <= 0.21149778366088867) ?  -0.026571046845480274 :   0.037322632811982795 :   (b('ndvi') <= 1801.5) ?  (b('moss') <= 7.1306469440460205) ?  (b('l8b5_med') <= 2259.166259765625) ?  0.0019149406901397316 :   0.04629142684303338 :   (b('ndvi_med') <= 3512.0) ?  -0.01018487597964696 :   0.08721905598482754 :   (b('ndvi') <= 2436.5) ?  (b('ndvi_med') <= 2103.25) ?  -0.010730109711068346 :   -0.037086223056221196 :   (b('l8b5') <= 2434.3780517578125) ?  0.0030911150747653005 :   -0.02101706775999 :   (b('bulk') <= 122.0) ?  (b('l8b4') <= 1558.82373046875) ?  (b('l8b5_med') <= 1747.1142578125) ?  -0.01112666331238965 :   (b('l8b11') <= 1591.3037109375) ?  0.08479920781655031 :   0.041553282324574364 :   0.10990785968951577 :   (b('bulk') <= 125.5) ?  (b('l8b11') <= 2269.2666625976562) ?  (b('l8b4') <= 846.3642578125) ?  -0.016862729848839134 :   -0.047512975535555384 :   0.05457548969137427 :   (b('l8b5_med') <= 3508.0341796875) ?  (b('l8b11') <= 1377.9127197265625) ?  0.028939255788515732 :   0.0023451071719884555 :   (b('l8b5') <= 3250.922607421875) ?  0.025291443502916938 :   0.06875483054911684 :   (b('l8b5_med') <= 3250.251708984375) ?  (b('ndvi') <= 4304.5) ?  (b('l8b4_med') <= 1247.2589721679688) ?  (b('ndvi') <= 3421.0) ?  (b('ndvi') <= 2071.5) ?  -0.023704064558678975 :   -0.0717523015317719 :   -0.01633795936885299 :   (b('dg0_v_vv') <= -1.4760246276855469) ?  (b('l8b5_med') <= 2651.7388916015625) ?  -0.047665539425906366 :   -0.01715054816137163 :   (b('grass') <= 2.4215551614761353) ?  -0.020398659776494136 :   -0.005334513504096538 :   (b('dg0_v_vv') <= -0.34304189682006836) ?  0.11427439192230623 :   0.08844125067555811 :   (b('l8b5_med') <= 3622.8421630859375) ?  (b('l8b4') <= 2185.171142578125) ?  (b('ndvi_med') <= 1542.25) ?  (b('l8b5') <= 2928.12646484375) ?  0.03461367463691419 :   0.004398943191489637 :   (b('l8b4') <= 2094.00244140625) ?  0.030069624703819353 :   0.14375893211542853 :   (b('ndvi') <= 3310.0) ?  (b('l8b11') <= 3555.549072265625) ?  -0.004947267874480651 :   0.008587214378211874 :   0.0760379579981097 :   (b('dg0_v_vv') <= -2.0577850341796875) ?  (b('dg0_v_vv') <= -2.711087226867676) ?  (b('ndvi') <= 1883.0) ?  -0.023066109652795302 :   -0.03338805311626459 :   (b('l8b5_med') <= 3694.775634765625) ?  -0.023305508565202405 :   -0.001272908128559862 :   (b('dg0_v_vv') <= 0.2029886245727539) ?  (b('l8b11') <= 3117.7891845703125) ?  0.0004904998640380597 :   -0.016665933043567133 :   -0.09351704679013265 :   (b('ndvi_med') <= 3497.75) ?  (b('dg0_v_vv') <= 3.143301248550415) ?  (b('bulk') <= 163.5) ?  (b('ndvi_med') <= 1508.5) ?  (b('moss') <= 8.229613304138184) ?  (b('l8b11_med') <= 2403.567626953125) ?  0.0240999503144416 :   -0.011666622481590597 :   (b('l8b11') <= 2207.14453125) ?  -0.03993981308265554 :   0.022709756498520695 :   (b('l8b4') <= 775.3285522460938) ?  (b('l8b4') <= 765.7528686523438) ?  -0.0052494230778387715 :   -0.05897319137293236 :   (b('ndvi_med') <= 1574.25) ?  0.018096198891351476 :   0.0031943314728144065 :   (b('l8b11') <= 2271.6971435546875) ?  (b('l8b4') <= 1786.7650146484375) ?  (b('l8b4') <= 1044.06103515625) ?  0.033312412342796714 :   -0.03355694854318287 :   0.11466957906620043 :   (b('l8b5') <= 3394.36279296875) ?  (b('l8b4') <= 1307.1531982421875) ?  0.10753805330186524 :   -0.0008603401224737475 :   0.16570429126143715 :   (b('l8b11') <= 1971.9109497070312) ?  (b('l8b4_med') <= 2060.1482543945312) ?  (b('l8b11_med') <= 2186.1248779296875) ?  (b('sand') <= 24.0) ?  0.05698390755292428 :   0.01975223535721497 :   (b('l8b11_med') <= 2326.037109375) ?  0.0918483645634928 :   0.039924621848525646 :   (b('l8b4') <= 1364.1302490234375) ?  (b('ndvi') <= 1050.5) ?  0.023295303476464135 :   0.001134102914072338 :   (b('dg0_v_vv') <= 3.3340113162994385) ?  -0.07792714297625297 :   -0.006231508546960515 :   (b('l8b5_med') <= 2201.696044921875) ?  (b('sand') <= 36.0) ?  (b('ndvi') <= 1560.0) ?  0.025628750646186638 :   -0.04414451692623392 :   (b('sand') <= 51.5) ?  -0.0716579441807416 :   -0.05426777153597468 :   (b('grass') <= 57.5) ?  (b('l8b4_med') <= 1629.4097900390625) ?  -0.012583832903575908 :   0.009699462976417838 :   (b('bulk') <= 149.0) ?  0.020431302047163452 :   0.06660878901461996 :   (b('ndvi') <= 4588.5) ?  (b('l8b4_med') <= 635.4568176269531) ?  (b('l8b4_med') <= 593.9647827148438) ?  (b('crops') <= 15.223477363586426) ?  (b('l8b5_med') <= 2708.1832275390625) ?  0.0762634515441784 :   0.050751464110862926 :   (b('crops') <= 21.252469062805176) ?  -0.03468795742736744 :   0.019151525033744732 :   (b('dg0_v_vv') <= 1.6939568519592285) ?  (b('ndvi_med') <= 3694.5) ?  -0.0033279049839740893 :   -0.05636438205472287 :   (b('dg0_v_vv') <= 4.306506633758545) ?  0.03834994676976644 :   -0.026444377238581313 :   (b('l8b4_med') <= 646.9902954101562) ?  (b('l8b5_med') <= 3465.753662109375) ?  0.1342307158491856 :   0.07790312355168671 :   (b('ndvi_med') <= 4127.5) ?  (b('l8b4_med') <= 721.2606506347656) ?  0.06910536712891033 :   0.03201252555037737 :   (b('l8b11') <= 2826.782470703125) ?  -0.027382597116390087 :   0.021146196587032134 :   (b('ndvi') <= 4644.5) ?  (b('dg0_v_vv') <= 1.3249616622924805) ?  -0.1452862621468057 :   -0.07688582337821182 :   (b('ndvi') <= 4756.0) ?  (b('l8b5_med') <= 3372.728759765625) ?  (b('l8b11_med') <= 1485.1502075195312) ?  0.04584861409075935 :   0.059430635537279175 :   (b('l8b11_med') <= 1486.1631469726562) ?  0.006549967923979494 :   -0.00906692179605037 :   (b('grass') <= 11.703174591064453) ?  (b('ndvi_med') <= 3966.0) ?  0.01667371186877252 :   0.055440225247027686 :   (b('bulk') <= 127.0) ?  -0.010400492103862445 :   -0.04626942169129171 :   (b('l8b4') <= 1225.7894287109375) ?  (b('l8b4_med') <= 1057.7998657226562) ?  (b('l8b11_med') <= 2201.8699951171875) ?  (b('l8b11_med') <= 1971.49267578125) ?  (b('dg0_v_vv') <= 1.4379472732543945) ?  (b('l8b5_med') <= 2956.10791015625) ?  (b('sand') <= 33.0) ?  (b('grass') <= 36.396745681762695) ?  -0.008343495816364652 :   0.01542801222709602 :   (b('sand') <= 40.5) ?  -0.017156718739540288 :   -0.0055728756927611095 :   (b('l8b11_med') <= 1533.0357666015625) ?  (b('dg0_v_vv') <= -2.049665927886963) ?  -0.022247750028784003 :   0.022943964077650147 :   (b('l8b5') <= 2799.662841796875) ?  -0.01479580674107976 :   -0.0002449214431230851 :   (b('l8b11_med') <= 1646.4898681640625) ?  (b('grass') <= 44.90012741088867) ?  (b('grass') <= 27.58390235900879) ?  0.02037141176137093 :   -0.010062161102026502 :   (b('l8b4') <= 1040.7618408203125) ?  0.026957433313804745 :   0.06260581359749025 :   (b('l8b5_med') <= 1639.760498046875) ?  0.12441649839577917 :   (b('l8b4') <= 754.749755859375) ?  0.10403981286382591 :   -0.01696925024300707 :   (b('l8b5') <= 2712.08447265625) ?  (b('l8b5') <= 1929.0255126953125) ?  (b('dg0_v_vv') <= 0.2645554542541504) ?  (b('ndvi') <= 2623.0) ?  0.10834429204259025 :   0.13289804758338147 :   (b('bulk') <= 154.0) ?  0.04091856383357653 :   0.07050259726699266 :   (b('l8b4') <= 889.278076171875) ?  (b('ndvi') <= 2426.5) ?  0.0502810253630166 :   -0.07015121338401059 :   (b('l8b4') <= 1012.6246643066406) ?  0.09852982604789338 :   -0.006145383271380873 :   (b('l8b11') <= 2329.9390869140625) ?  (b('dg0_v_vv') <= -0.024332523345947266) ?  (b('l8b4') <= 1079.51025390625) ?  0.09597844150632977 :   0.0008511948271817948 :   (b('moss') <= 21.098512649536133) ?  0.11489119997085397 :   0.21501709578241784 :   -0.041873930637655266 :   (b('dg0_v_vv') <= 1.7502036094665527) ?  (b('l8b11') <= 1903.4097900390625) ?  (b('l8b11') <= 1619.3409423828125) ?  -0.08774090648017564 :   (b('dg0_v_vv') <= 0.40165042877197266) ?  -0.06699291552543657 :   (b('dg0_v_vv') <= 1.0727009773254395) ?  -0.07825340534897132 :   -0.07668878721424494 :   (b('dg0_v_vv') <= -1.8449411392211914) ?  0.0032230598791178344 :   (b('l8b4') <= 925.5934448242188) ?  -0.010172999000197566 :   (b('l8b5') <= 2299.555419921875) ?  -0.04861679547043727 :   -0.07283722553999551 :   (b('l8b5') <= 1974.91748046875) ?  (b('l8b5') <= 1680.9161376953125) ?  -0.053708524704670364 :   (b('l8b5') <= 1763.017578125) ?  (b('l8b5_med') <= 1942.32666015625) ?  0.0027845122106924663 :   -0.0010675986809168814 :   (b('dg0_v_vv') <= 2.54301118850708) ?  0.012318714692972382 :   0.02181379559669687 :   (b('ndvi') <= 1912.0) ?  (b('l8b4_med') <= 1034.0540161132812) ?  -0.05330030623574297 :   -0.057500004647879716 :   (b('l8b11') <= 2017.3257446289062) ?  (b('l8b4_med') <= 1009.4158020019531) ?  -0.04052944126900063 :   -0.0468530292278718 :   (b('l8b5_med') <= 1942.32666015625) ?  -0.01391945701556905 :   -0.0016681155925176994 :   (b('dg0_v_vv') <= 0.07027339935302734) ?  (b('ndvi') <= 3062.0) ?  (b('l8b5_med') <= 2913.352294921875) ?  (b('sand') <= 62.5) ?  (b('l8b5_med') <= 2666.2344970703125) ?  (b('l8b11') <= 2657.2774658203125) ?  -0.006276418240823433 :   0.07995297582471572 :   (b('ndvi') <= 2900.0) ?  0.014157402227968722 :   0.12310113358122833 :   (b('l8b4_med') <= 1447.8766479492188) ?  (b('dg0_v_vv') <= -0.23477888107299805) ?  0.02709640173309824 :   0.08979723644200806 :   0.17505627124773904 :   (b('l8b4') <= 1208.3624877929688) ?  (b('l8b11') <= 1796.7037353515625) ?  (b('l8b11') <= 1761.5242309570312) ?  -0.032388795922869425 :   0.037538946793963995 :   (b('l8b4') <= 1077.6035766601562) ?  -0.06266544105621318 :   -0.039668530794555505 :   (b('ndvi') <= 2383.0) ?  (b('ndvi') <= 2096.5) ?  -0.0229952190102471 :   -0.03785780054363373 :   (b('dg0_v_vv') <= -0.5998377799987793) ?  0.08889047928251742 :   -0.05384484988132593 :   (b('l8b4') <= 1103.8444213867188) ?  (b('dg0_v_vv') <= -0.8625984191894531) ?  (b('ndvi') <= 3545.5) ?  (b('l8b5') <= 3106.4237060546875) ?  0.011269262330512937 :   -0.047035706828788604 :   (b('ndvi') <= 3934.5) ?  0.05794951486801928 :   0.014157778263711493 :   (b('moss') <= 14.440980911254883) ?  (b('l8b4') <= 1061.6661987304688) ?  0.040460778056582374 :   -0.021674761287082633 :   (b('l8b11_med') <= 2162.3863525390625) ?  0.047965059834196605 :   0.10963882366102756 :   (b('l8b5_med') <= 3370.5687255859375) ?  (b('dg0_v_vv') <= -0.3183126449584961) ?  (b('ndvi_med') <= 2392.5) ?  -0.04197380856993246 :   -0.024000182034824215 :   (b('ndvi_med') <= 3355.5) ?  -0.00012988245191367329 :   0.07808261689079907 :   0.0700496380289061 :   (b('l8b4') <= 1124.3387451171875) ?  (b('l8b11') <= 2301.5303955078125) ?  (b('moss') <= 16.849438667297363) ?  (b('grass') <= 73.00110626220703) ?  (b('moss') <= 12.006288528442383) ?  -0.05320520734466996 :   0.0064647705122011885 :   (b('l8b5_med') <= 2276.8875732421875) ?  0.002978636032971248 :   0.05383087659722448 :   (b('moss') <= 18.696696281433105) ?  (b('l8b5_med') <= 2664.7177734375) ?  0.12069895684022758 :   0.2115773103647429 :   (b('moss') <= 29.535037994384766) ?  0.04251065786922151 :   0.01514459631095713 :   (b('l8b5') <= 2015.1369018554688) ?  (b('bulk') <= 156.0) ?  -0.09704601458285714 :   (b('ndvi_med') <= 1790.0) ?  -0.09167851066047167 :   -0.08846014726751392 :   0.08945022322205737 :   (b('ndvi_med') <= 1385.5) ?  (b('dg0_v_vv') <= 0.1325855255126953) ?  (b('l8b5') <= 2012.9694213867188) ?  (b('sand') <= 48.5) ?  0.07873229715371477 :   0.04182542317605101 :   (b('ndvi') <= 1395.0) ?  0.16302090948506145 :   0.15443956615215815 :   (b('ndvi') <= 1792.5) ?  (b('l8b11_med') <= 2036.1627807617188) ?  0.033290117983478404 :   -0.004146163609515338 :   (b('l8b5') <= 2565.1815185546875) ?  0.05039959924024041 :   -0.01442079903691297 :   (b('l8b11') <= 2139.7032470703125) ?  (b('l8b11_med') <= 1675.779541015625) ?  0.06496687171821228 :   (b('l8b11') <= 2079.0308837890625) ?  -0.020894221418117812 :   -0.04614489040630691 :   (b('ndvi') <= 1209.5) ?  0.15216578917667362 :   (b('l8b11') <= 2469.3536376953125) ?  -0.002790795788237501 :   0.04622657243812626 :   (b('bulk') <= 150.5) ?  (b('ndvi') <= 2134.0) ?  (b('ndvi_med') <= 1483.75) ?  (b('ndvi') <= 1467.5) ?  (b('ndvi') <= 827.5) ?  (b('grass') <= 48.92453384399414) ?  (b('ndvi') <= 808.0) ?  -0.011389255263161072 :   0.033638329023837524 :   (b('moss') <= 30.787456512451172) ?  0.16942586778371152 :   0.05236889762868268 :   (b('moss') <= 25.738844871520996) ?  (b('l8b5') <= 2021.322509765625) ?  0.015688842207001293 :   -0.006554523970151971 :   (b('ndvi_med') <= 1223.0) ?  -0.02633993233113138 :   -0.009269405498439713 :   (b('l8b5_med') <= 2563.07861328125) ?  (b('l8b11') <= 2325.2679443359375) ?  (b('ndvi') <= 1470.0) ?  0.08428790065331988 :   -0.011058059168722063 :   (b('l8b11') <= 2410.779052734375) ?  0.09426790914145916 :   0.02027309062557641 :   (b('moss') <= 23.909934997558594) ?  (b('dg0_v_vv') <= 2.832256555557251) ?  -0.0038940622371284746 :   0.03079265585671027 :   (b('dg0_v_vv') <= -0.04702615737915039) ?  -0.0217465362255513 :   -0.042046094038765545 :   (b('ndvi_med') <= 2686.0) ?  (b('grass') <= 74.81922149658203) ?  (b('dg0_v_vv') <= 5.457340240478516) ?  (b('l8b11_med') <= 1590.11767578125) ?  0.02322327961465959 :   -0.013033346559393455 :   -0.10212586079054745 :   (b('l8b4') <= 1622.1401977539062) ?  (b('l8b11_med') <= 1943.598876953125) ?  -0.051927520133444934 :   -0.025321391610715956 :   (b('ndvi_med') <= 1569.5) ?  -0.0237296302764503 :   0.008826272571078911 :   (b('ndvi') <= 1785.0) ?  (b('moss') <= 9.037735939025879) ?  0.09997291396917943 :   (b('dg0_v_vv') <= 1.09004545211792) ?  0.04039626194657123 :   0.04932813279662722 :   (b('l8b11_med') <= 1890.1565551757812) ?  (b('dg0_v_vv') <= -1.7656373977661133) ?  -0.029945258037928135 :   -0.002317451238970816 :   -0.05895779922842043 :   (b('l8b11') <= 1799.1677856445312) ?  (b('bulk') <= 144.0) ?  (b('ndvi') <= 2759.5) ?  (b('l8b4_med') <= 1400.7770385742188) ?  (b('l8b4_med') <= 1202.2705688476562) ?  0.03421576885583341 :   0.0604479439189507 :   -0.010733596114173544 :   (b('l8b5_med') <= 3108.3231201171875) ?  (b('dg0_v_vv') <= 0.7965888977050781) ?  -0.02533439509852681 :   -0.03943662910248141 :   -0.004474507562095137 :   (b('l8b4_med') <= 1362.9207763671875) ?  (b('dg0_v_vv') <= 0.538823127746582) ?  0.1705576463933884 :   0.17993015381066102 :   (b('l8b4') <= 1255.437255859375) ?  0.14138833455236902 :   0.1566273070196326 :   (b('dg0_v_vv') <= 0.6607427597045898) ?  (b('ndvi') <= 2163.0) ?  (b('l8b11_med') <= 2223.251953125) ?  0.11843075703036289 :   (b('ndvi') <= 2137.0) ?  0.0013452446872141144 :   0.037782772188584605 :   (b('l8b4') <= 1262.664306640625) ?  (b('sand') <= 39.5) ?  -0.006462360447998375 :   -0.04913014236823957 :   (b('ndvi') <= 4096.0) ?  -0.005689372172856762 :   0.06849461015248327 :   (b('ndvi_med') <= 805.25) ?  0.12570197133887823 :   (b('l8b5_med') <= 2660.25146484375) ?  (b('sand') <= 52.0) ?  -0.030764332102241724 :   0.030659198457920367 :   (b('l8b4') <= 1570.0701293945312) ?  0.03681746686568676 :   0.0035553117119106912 :   (b('l8b11') <= 1997.1546020507812) ?  (b('l8b5_med') <= 2338.9254150390625) ?  (b('l8b11') <= 1840.1363525390625) ?  (b('l8b11_med') <= 1388.5218505859375) ?  -0.08741156062027661 :   (b('dg0_v_vv') <= -0.3211946487426758) ?  -0.08507582452032776 :   (b('l8b5') <= 2306.8460693359375) ?  -0.08286174841982245 :   -0.0837588480299234 :   -0.019109126209546454 :   (b('l8b5') <= 4043.866455078125) ?  (b('l8b11') <= 1837.336181640625) ?  (b('l8b11_med') <= 2273.3721923828125) ?  (b('l8b5_med') <= 2394.9208984375) ?  -0.07164778156467635 :   -0.06552866204927349 :   (b('ndvi') <= 1866.0) ?  -0.040340644950370694 :   -0.03531297484229806 :   (b('l8b11') <= 1884.0814819335938) ?  (b('l8b11_med') <= 2273.3721923828125) ?  -0.033549724187994276 :   0.000809836484538562 :   (b('grass') <= 48.047285079956055) ?  -0.029087488835199432 :   -0.00978428493594614 :   (b('ndvi') <= 1849.5) ?  (b('dg0_v_vv') <= 2.2746877670288086) ?  0.0007843146168331067 :   (b('dg0_v_vv') <= 4.415253400802612) ?  -0.01751982691181317 :   -0.019510642415276183 :   (b('dg0_v_vv') <= 4.386989593505859) ?  (b('dg0_v_vv') <= 2.4925873279571533) ?  0.006189837853284454 :   -0.004176608911928541 :   0.008375053453214681 :   (b('ndvi_med') <= 1696.5) ?  (b('l8b4') <= 1607.5054931640625) ?  (b('l8b11') <= 2283.105712890625) ?  (b('l8b4') <= 1586.1580810546875) ?  (b('l8b4') <= 1300.1709594726562) ?  0.02714249319096454 :   -0.01908659533214223 :   (b('ndvi') <= 862.5) ?  0.036690829430508595 :   0.01974259408097717 :   (b('l8b5_med') <= 2758.6209716796875) ?  (b('dg0_v_vv') <= 3.304295539855957) ?  0.00226370060870331 :   0.08713742215907672 :   (b('dg0_v_vv') <= -0.9138264656066895) ?  -0.0015337391335517453 :   0.036864959148840255 :   (b('l8b11_med') <= 3028.1297607421875) ?  (b('ndvi') <= 1840.0) ?  (b('dg0_v_vv') <= 0.17332172393798828) ?  -0.01226326738801182 :   -0.021079711519835998 :   (b('l8b11_med') <= 2917.3968505859375) ?  0.002286133429407312 :   0.03160853161547367 :   (b('l8b4') <= 1859.11279296875) ?  (b('l8b4') <= 1727.0196533203125) ?  -0.01569568930120893 :   0.017760339740061876 :   (b('l8b11') <= 2935.1094970703125) ?  -0.02523013716933466 :   -0.004914098148191149 :   (b('sand') <= 35.0) ?  (b('l8b4') <= 1236.310302734375) ?  (b('ndvi_med') <= 3305.25) ?  -0.051763854407674545 :   (b('dg0_v_vv') <= 0.019924640655517578) ?  -0.028264501270284972 :   -0.03658363764215174 :   (b('dg0_v_vv') <= 0.9322292804718018) ?  (b('l8b4') <= 1266.0748291015625) ?  0.14948010056167602 :   0.015209118527004354 :   (b('l8b11_med') <= 1983.5992431640625) ?  -0.037846567835639666 :   0.01045681942299619 :   (b('l8b5') <= 2281.5511474609375) ?  (b('dg0_v_vv') <= 3.830577850341797) ?  (b('l8b4') <= 1260.10986328125) ?  -0.049282555907215995 :   -0.0257556964877971 :   0.02299651887498924 :   (b('dg0_v_vv') <= 3.2996907234191895) ?  (b('ndvi') <= 1574.5) ?  -0.01966364758821985 :   -0.014601198928863012 :   (b('dg0_v_vv') <= 4.968780517578125) ?  -0.035658205697152626 :   -0.011487192709931988 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1390.6193237304688) ?  (b('l8b4_med') <= 658.7252502441406) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 35.5) ?  (b('l8b11') <= 867.2576904296875) ?  (b('ndvi') <= 4226.5) ?  (b('dg0_v_vv') <= -0.34763383865356445) ?  (b('l8b5_med') <= 2350.8095703125) ?  (b('dg0_v_vv') <= -1.0025568008422852) ?  0.011390291289155419 :   (b('dg0_v_vv') <= -0.4685978889465332) ?  0.03687878278367101 :   0.06040377546280418 :   (b('l8b11') <= 857.4646606445312) ?  (b('l8b11') <= 845.1726684570312) ?  -0.007914625888195984 :   0.002893419737345748 :   -0.033705748735335994 :   (b('ndvi') <= 3717.5) ?  (b('dg0_v_vv') <= -0.3356752395629883) ?  0.08140377331703696 :   (b('dg0_v_vv') <= 0.7853989601135254) ?  0.029763170311896132 :   0.051539939648612046 :   (b('l8b11') <= 863.1797180175781) ?  0.056720673621941387 :   0.08743883233565936 :   (b('dg0_v_vv') <= -1.2346720695495605) ?  (b('ndvi_med') <= 3486.5) ?  0.058977492420662414 :   0.026676697086989443 :   (b('bulk') <= 152.5) ?  (b('sand') <= 27.0) ?  (b('l8b5') <= 4898.854736328125) ?  0.06322407274884739 :   0.06440464056315531 :   0.06033422179683223 :   0.07455748651975719 :   (b('l8b4_med') <= 632.78857421875) ?  (b('l8b5') <= 2026.3069458007812) ?  (b('ndvi') <= 3551.5) ?  (b('ndvi') <= 2564.5) ?  (b('l8b5') <= 1964.8894653320312) ?  0.017514100195025174 :   -0.03710729824776074 :   (b('sand') <= 29.0) ?  -0.06679693737874871 :   -0.020117412314073616 :   0.08272175636629919 :   (b('l8b4') <= 718.6219177246094) ?  (b('ndvi') <= 3406.0) ?  (b('ndvi') <= 3398.5) ?  0.026162552126185613 :   0.0803974606674967 :   (b('l8b5') <= 3954.1021728515625) ?  0.0018792145880512357 :   0.022090906910139645 :   (b('l8b5') <= 2165.831298828125) ?  (b('dg0_v_vv') <= -0.17443132400512695) ?  0.038282286389541696 :   -0.03220511376834843 :   (b('ndvi') <= 2092.0) ?  0.10790880796513602 :   -0.008987348267810615 :   (b('crops') <= 42.76557922363281) ?  (b('l8b4') <= 432.2246856689453) ?  0.10830065992163818 :   (b('l8b4') <= 476.43138122558594) ?  (b('bulk') <= 143.5) ?  -0.03998392056315124 :   -0.016151195100790883 :   (b('l8b11') <= 1210.1322631835938) ?  0.05702586570744003 :   0.01790433907203612 :   (b('l8b5') <= 3961.734130859375) ?  (b('ndvi') <= 4406.0) ?  (b('l8b4') <= 831.1228942871094) ?  0.09910433885522546 :   0.09796477828248518 :   0.07011281119651802 :   0.040508120755813004 :   (b('crops') <= 17.562152862548828) ?  (b('l8b4') <= 585.81201171875) ?  (b('l8b11') <= 1107.4000854492188) ?  (b('l8b5') <= 3365.1431884765625) ?  (b('l8b5') <= 3224.4141845703125) ?  (b('dg0_v_vv') <= 0.6271510124206543) ?  -0.013684480402517836 :   0.011920402471535974 :   (b('ndvi') <= 4324.0) ?  -0.06540243195905696 :   -0.04471255967630344 :   (b('l8b4') <= 536.0966796875) ?  (b('ndvi') <= 4393.5) ?  0.033217478332605674 :   -0.009401593924139362 :   (b('dg0_v_vv') <= 0.5567944049835205) ?  -0.06338904177468252 :   -0.06505952700265868 :   (b('l8b11_med') <= 1119.14599609375) ?  (b('ndvi') <= 1868.0) ?  -0.03449052952219667 :   (b('dg0_v_vv') <= -1.0047695636749268) ?  0.011022388494032476 :   0.06908041343302472 :   (b('ndvi') <= 4906.0) ?  (b('ndvi') <= 3700.0) ?  0.02446808450332398 :   -0.015464525115017532 :   (b('l8b4_med') <= 577.5997924804688) ?  0.04287970062394675 :   0.10982510962721925 :   (b('l8b5_med') <= 2550.665283203125) ?  (b('l8b11_med') <= 1059.1539611816406) ?  (b('l8b5') <= 2572.0859375) ?  (b('ndvi') <= 2479.0) ?  -0.0840623872447617 :   -0.07276403059333014 :   -0.04766364106068438 :   (b('dg0_v_vv') <= 0.5238783359527588) ?  (b('dg0_v_vv') <= 0.42589688301086426) ?  -0.02316891159547434 :   0.04839046662752358 :   (b('l8b11') <= 1043.0648803710938) ?  -0.036671035233543425 :   -0.052866912498623526 :   (b('ndvi_med') <= 4138.5) ?  (b('l8b11') <= 1011.268310546875) ?  (b('l8b5_med') <= 2918.44873046875) ?  -0.03865988979855987 :   -0.07583569370064183 :   (b('l8b11') <= 1111.3642578125) ?  0.0405688053599948 :   -0.011012645146486017 :   0.08001358921466903 :   (b('l8b5') <= 2758.500732421875) ?  (b('ndvi') <= 4062.5) ?  (b('bulk') <= 134.0) ?  (b('ndvi_med') <= 4556.5) ?  (b('l8b11') <= 1359.1075439453125) ?  0.027803705789891445 :   0.011556089016236008 :   0.10560538008199116 :   (b('l8b11') <= 1730.0643920898438) ?  (b('l8b11') <= 1087.4679565429688) ?  0.018089171653835973 :   0.10004551329045776 :   -0.004266483806661919 :   (b('ndvi_med') <= 4454.5) ?  (b('bulk') <= 129.5) ?  (b('l8b5_med') <= 2629.1593017578125) ?  -0.02749833058260337 :   0.014324694928864573 :   (b('l8b4_med') <= 584.2852783203125) ?  -0.021016993857812243 :   -0.049242012935035884 :   0.09391996545930675 :   (b('bulk') <= 114.0) ?  (b('l8b11') <= 1066.85107421875) ?  (b('ndvi') <= 4633.0) ?  (b('dg0_v_vv') <= -0.7126121520996094) ?  0.008909894398460252 :   0.026809428233048417 :   -0.05794423430235501 :   (b('dg0_v_vv') <= -0.677299976348877) ?  0.049874127083612885 :   (b('ndvi') <= 4651.0) ?  0.08654946322573424 :   0.11848965119963265 :   (b('dg0_v_vv') <= 1.2645158767700195) ?  (b('crops') <= 70.61468124389648) ?  (b('moss') <= 2.8072200417518616) ?  -0.020674958485898377 :   0.006926767663331563 :   (b('l8b11') <= 853.0417175292969) ?  -0.03452019158923401 :   0.019533330796969187 :   (b('crops') <= 36.517295837402344) ?  (b('l8b11') <= 1216.4293823242188) ?  -0.0949947075566582 :   -0.07962816368962114 :   (b('ndvi_med') <= 3902.75) ?  -0.015031663589099273 :   0.006577020562166231 :   (b('crops') <= 19.45354127883911) ?  (b('ndvi_med') <= 3042.5) ?  (b('l8b5') <= 1922.4028930664062) ?  (b('bulk') <= 129.5) ?  (b('l8b5') <= 1452.2807006835938) ?  (b('dg0_v_vv') <= 2.002592086791992) ?  -0.0069854247427962035 :   -9.53581920409563e-05 :   -0.018622058205013142 :   (b('ndvi') <= 2302.5) ?  (b('l8b5_med') <= 2308.12939453125) ?  0.022953959773760718 :   0.017264616479991834 :   0.034967932436959365 :   (b('ndvi') <= 3745.0) ?  (b('l8b5_med') <= 2813.5404052734375) ?  (b('dg0_v_vv') <= 1.3197104930877686) ?  (b('l8b4') <= 978.8956298828125) ?  -0.019860597788647415 :   -0.0387869087194638 :   -0.04519288479226963 :   0.021238611440789534 :   0.03218833122426237 :   (b('l8b4') <= 450.03575134277344) ?  (b('l8b5') <= 2909.980224609375) ?  (b('l8b4') <= 442.0557098388672) ?  (b('dg0_v_vv') <= -0.006542205810546875) ?  (b('dg0_v_vv') <= -1.3384966850280762) ?  0.08189803660066058 :   0.019115327341298772 :   (b('ndvi') <= 4518.0) ?  0.03891829439600222 :   0.07017201031867096 :   0.1047279105645822 :   (b('dg0_v_vv') <= -0.1426987648010254) ?  (b('ndvi_med') <= 3491.5) ?  (b('l8b11') <= 918.5289001464844) ?  0.04752302095753669 :   0.04379356669911322 :   0.022616012893826865 :   (b('dg0_v_vv') <= 0.738584041595459) ?  (b('dg0_v_vv') <= 0.6887941360473633) ?  -0.038507657385082536 :   -0.0498914575116631 :   (b('dg0_v_vv') <= 0.9790582656860352) ?  0.0031085455877784274 :   0.0013737392294398132 :   (b('ndvi') <= 3336.0) ?  (b('ndvi') <= 3129.0) ?  (b('ndvi') <= 3033.5) ?  (b('dg0_v_vv') <= 2.4640579223632812) ?  0.02223489539258933 :   0.14245078124225447 :   (b('sand') <= 44.5) ?  -0.041090868301995584 :   -0.0034468698043052756 :   (b('l8b5') <= 2444.320556640625) ?  (b('bulk') <= 113.5) ?  0.10743646667552345 :   0.16762458222039073 :   (b('ndvi') <= 3180.0) ?  0.10427055962115883 :   0.03647550034931733 :   (b('grass') <= 55.15210723876953) ?  (b('dg0_v_vv') <= 0.8255691528320312) ?  (b('l8b11') <= 1380.9150390625) ?  0.011841798404538658 :   -0.02212216802790437 :   (b('l8b11') <= 954.4361572265625) ?  5.6489628523670454e-05 :   0.0558154862133382 :   (b('dg0_v_vv') <= 3.0962343215942383) ?  (b('ndvi') <= 4617.5) ?  -0.01911202134249066 :   0.010596250021406042 :   (b('sand') <= 30.0) ?  0.057286795637621735 :   0.0471445098487073 :   (b('l8b5') <= 2478.9398193359375) ?  (b('l8b11') <= 1403.2102661132812) ?  (b('l8b5') <= 1895.348876953125) ?  (b('dg0_v_vv') <= -0.2799191474914551) ?  (b('dg0_v_vv') <= -0.9946990013122559) ?  0.06805272641892868 :   0.0742241392202746 :   (b('ndvi') <= 4890.5) ?  (b('l8b5_med') <= 2553.574951171875) ?  -0.00250263964478048 :   0.021708829630075654 :   (b('dg0_v_vv') <= 0.39547252655029297) ?  0.08026125880055657 :   0.046635324282543966 :   (b('l8b4') <= 627.088134765625) ?  (b('ndvi_med') <= 3405.0) ?  (b('ndvi') <= 3323.0) ?  -0.03745645400535699 :   -0.054404459295643304 :   (b('l8b5') <= 2454.42578125) ?  -0.0316778806797596 :   0.0020188672139146158 :   (b('dg0_v_vv') <= 0.7986822128295898) ?  (b('dg0_v_vv') <= 0.37900543212890625) ?  0.004102826734432457 :   0.038078984786313684 :   (b('dg0_v_vv') <= 0.8771858215332031) ?  -0.014085391974822226 :   -0.017183396003043583 :   (b('dg0_v_vv') <= -2.5997462272644043) ?  -0.014989106082423598 :   (b('ndvi_med') <= 3494.75) ?  (b('ndvi') <= 2903.5) ?  (b('dg0_v_vv') <= 0.05087137222290039) ?  0.05788813937585485 :   0.0655415400807075 :   (b('l8b11') <= 1502.1250610351562) ?  0.05080747185864831 :   0.03831181675304868 :   (b('ndvi') <= 3806.5) ?  (b('dg0_v_vv') <= -0.8300981521606445) ?  0.07345835559757374 :   0.03802511129619427 :   -0.009746824744585214 :   (b('l8b4') <= 544.4970397949219) ?  (b('ndvi_med') <= 3208.25) ?  0.08626392882958545 :   (b('sand') <= 52.5) ?  (b('ndvi') <= 2667.5) ?  (b('dg0_v_vv') <= 1.196232795715332) ?  0.052687511001732815 :   0.057620890789410094 :   (b('ndvi') <= 4424.5) ?  0.04028874774163449 :   0.049918558408739704 :   (b('dg0_v_vv') <= -0.9544544219970703) ?  (b('l8b4') <= 522.62255859375) ?  0.033236415803102305 :   0.011735694785962314 :   (b('l8b11') <= 1110.660400390625) ?  0.04304161515544833 :   0.03646018021111356 :   (b('l8b4_med') <= 573.9899597167969) ?  (b('crops') <= 28.946664810180664) ?  (b('ndvi') <= 4406.5) ?  (b('l8b11') <= 1236.5640869140625) ?  0.014692395154409213 :   0.0234724473764684 :   (b('l8b11') <= 1271.4739990234375) ?  0.03322614951900826 :   0.04364613442903387 :   (b('l8b11_med') <= 1221.92578125) ?  (b('dg0_v_vv') <= -0.1659102439880371) ?  0.06010726501282606 :   0.03380187022384638 :   (b('l8b11') <= 1623.4761962890625) ?  0.031245323137758258 :   0.02404425184608423 :   (b('dg0_v_vv') <= -0.744509220123291) ?  (b('bulk') <= 123.5) ?  (b('dg0_v_vv') <= -1.3801922798156738) ?  0.03932947461471287 :   0.03120896277991031 :   (b('dg0_v_vv') <= -1.0735430717468262) ?  -0.0054315255392757 :   0.02654039621749571 :   (b('ndvi') <= 3326.0) ?  (b('dg0_v_vv') <= 0.3011479377746582) ?  0.07372621443070321 :   0.05117498679866157 :   (b('dg0_v_vv') <= 0.5356860160827637) ?  0.04599535951102054 :   0.035009590266193366 :   (b('sand') <= 20.5) ?  (b('ndvi_med') <= 3676.25) ?  (b('l8b11') <= 1263.0814208984375) ?  (b('l8b11') <= 1192.164306640625) ?  (b('ndvi') <= 3566.0) ?  (b('bulk') <= 147.0) ?  (b('ndvi') <= 2477.5) ?  (b('dg0_v_vv') <= 1.9403748512268066) ?  0.02133623607277968 :   0.04173994358752381 :   (b('l8b5') <= 2697.4437255859375) ?  0.032004222990311304 :   0.053232008416920856 :   (b('l8b11') <= 986.6701354980469) ?  (b('l8b4') <= 486.6945037841797) ?  0.12050239502846127 :   0.12753620849858244 :   (b('dg0_v_vv') <= -0.1788010597229004) ?  0.11765384606534307 :   0.01649077734466614 :   (b('l8b5') <= 3651.0595703125) ?  (b('l8b4') <= 605.5425720214844) ?  (b('ndvi') <= 4899.0) ?  -0.012035830469542283 :   0.026652625730574604 :   (b('ndvi') <= 4226.5) ?  0.01855395218511091 :   0.06865150453012059 :   (b('l8b5') <= 3830.9512939453125) ?  0.05633471900488779 :   0.05453800113000146 :   (b('l8b4_med') <= 810.1732177734375) ?  -0.09461305890424601 :   (b('dg0_v_vv') <= 0.10917043685913086) ?  -0.05833900857156507 :   (b('ndvi') <= 3391.5) ?  -0.045678293358928396 :   -0.038739759660183004 :   (b('l8b5') <= 2474.6077880859375) ?  (b('l8b5') <= 2411.615478515625) ?  (b('dg0_v_vv') <= -2.5135178565979004) ?  (b('l8b4_med') <= 810.1732177734375) ?  -0.04816298410495168 :   (b('l8b5') <= 2217.534423828125) ?  -0.006264642930531 :   0.025704481117403366 :   (b('l8b5') <= 2243.821533203125) ?  (b('ndvi') <= 1405.0) ?  0.056683382099797716 :   0.030798155959221963 :   (b('l8b11') <= 1455.5) ?  0.10008456554782796 :   0.06924006799782277 :   (b('dg0_v_vv') <= -0.5804476737976074) ?  (b('l8b11_med') <= 1347.2548828125) ?  (b('ndvi') <= 3755.5) ?  -0.027782111304651108 :   -0.0324286662871289 :   (b('bulk') <= 148.0) ?  -0.0049001440768198234 :   0.01971796006929688 :   (b('l8b4_med') <= 788.0063781738281) ?  (b('sand') <= 17.5) ?  -0.037341232350768194 :   -0.026837408582931488 :   -0.0525066705813306 :   (b('ndvi_med') <= 3396.0) ?  (b('ndvi') <= 2094.0) ?  -0.0029921086760389404 :   (b('l8b4') <= 802.6297912597656) ?  (b('l8b5') <= 2937.384521484375) ?  0.045520364424693996 :   0.05976582473691446 :   (b('grass') <= 33.36697959899902) ?  0.0204735121123694 :   0.04353020230473853 :   (b('dg0_v_vv') <= -1.6620793342590332) ?  (b('l8b11') <= 1653.4921264648438) ?  -0.005016918816254601 :   0.024955889286489175 :   (b('dg0_v_vv') <= -1.0592565536499023) ?  (b('ndvi') <= 4652.5) ?  0.12657441925008234 :   0.06465639375764987 :   (b('dg0_v_vv') <= -0.9896993637084961) ?  0.012681440976861802 :   0.07346211809249743 :   (b('l8b5') <= 3859.33642578125) ?  (b('ndvi') <= 4230.0) ?  (b('dg0_v_vv') <= -0.2493281364440918) ?  (b('ndvi') <= 3610.5) ?  -0.0026451464932227275 :   (b('l8b11_med') <= 1423.1534423828125) ?  -0.027278205084152435 :   -0.029462574386293766 :   (b('l8b11_med') <= 1423.1534423828125) ?  -0.09353557539145543 :   -0.06494109100090156 :   (b('l8b11') <= 1329.8082275390625) ?  (b('dg0_v_vv') <= 1.9587626457214355) ?  (b('l8b5_med') <= 3218.4537353515625) ?  0.009090493188572857 :   0.03564600677784241 :   0.04482937200222148 :   -0.0075910352041402285 :   (b('l8b5') <= 4637.3543701171875) ?  -0.08901330038107935 :   -0.09197304436705256 :   (b('dg0_v_vv') <= 0.8236193656921387) ?  (b('moss') <= 6.230724096298218) ?  (b('l8b5_med') <= 2556.8916015625) ?  (b('crops') <= 56.336843490600586) ?  (b('l8b11') <= 2121.3948974609375) ?  (b('l8b5_med') <= 2091.7357177734375) ?  (b('ndvi') <= 2783.0) ?  0.0034997793128315195 :   0.08364490334738353 :   (b('sand') <= 49.5) ?  -0.009688881659640365 :   -0.02693723612509648 :   (b('l8b11_med') <= 2390.1497802734375) ?  (b('l8b11_med') <= 2325.04833984375) ?  0.0010726855436360148 :   0.04739118727032712 :   (b('l8b11') <= 2693.1431884765625) ?  -0.003841061127748323 :   -0.03667742434923782 :   (b('ndvi') <= 1782.0) ?  (b('l8b11') <= 2095.6897583007812) ?  (b('l8b11') <= 1434.6983642578125) ?  0.05039604416625021 :   -0.0003739236778195182 :   (b('sand') <= 44.0) ?  -0.10262234999689906 :   -0.02901692540047602 :   (b('dg0_v_vv') <= -0.7202606201171875) ?  (b('crops') <= 65.65124130249023) ?  -0.014494188952530417 :   0.017480430906159375 :   (b('l8b4') <= 777.2853088378906) ?  0.01839568062656943 :   0.04872551851754581 :   (b('dg0_v_vv') <= -0.95989990234375) ?  (b('bulk') <= 169.5) ?  (b('l8b4') <= 1355.8049926757812) ?  (b('ndvi') <= 3353.5) ?  -0.0065825580556136 :   0.0046478406938179695 :   (b('dg0_v_vv') <= -1.114889144897461) ?  0.032125827884727266 :   -0.024821851826291028 :   (b('dg0_v_vv') <= -1.2679190635681152) ?  (b('l8b5') <= 2706.2354736328125) ?  0.0720273362383766 :   0.07520942226737987 :   0.09791898640604937 :   (b('grass') <= 8.456110954284668) ?  (b('moss') <= 1.3587230145931244) ?  (b('l8b11') <= 1302.1392211914062) ?  0.03374044494779223 :   0.004178557601484958 :   (b('l8b11_med') <= 1929.2610473632812) ?  0.0005493129751842183 :   -0.019794933361642227 :   (b('sand') <= 32.5) ?  (b('ndvi_med') <= 3976.5) ?  0.03646208605905589 :   0.001892460586152105 :   (b('l8b5') <= 1661.5052490234375) ?  0.08397868307005334 :   0.008869445418596496 :   (b('bulk') <= 130.5) ?  (b('l8b5_med') <= 3257.319091796875) ?  (b('l8b11') <= 2367.1060791015625) ?  (b('ndvi_med') <= 2733.75) ?  (b('l8b11_med') <= 1771.62060546875) ?  -0.038803337196292365 :   -0.01453502233580187 :   (b('l8b5') <= 2566.1632080078125) ?  -0.0016121302595352679 :   -0.01686382336411365 :   (b('l8b4') <= 1256.6170043945312) ?  (b('ndvi_med') <= 3594.5) ?  0.05762095118706007 :   -0.009714561433540374 :   (b('l8b11') <= 2687.253662109375) ?  -0.05442483934923403 :   -0.01476988271940297 :   (b('ndvi') <= 3164.0) ?  (b('grass') <= 47.533498764038086) ?  (b('ndvi_med') <= 3524.0) ?  0.05228265525764498 :   0.07812986664190838 :   -0.005420577742263649 :   (b('ndvi') <= 3267.0) ?  (b('dg0_v_vv') <= 0.1381549835205078) ?  -0.05157297841670419 :   -0.08730484643983075 :   (b('ndvi') <= 3895.0) ?  0.029503577670712027 :   -0.007676156588254173 :   (b('ndvi_med') <= 3804.0) ?  (b('l8b4') <= 529.5152893066406) ?  (b('ndvi') <= 3145.0) ?  (b('moss') <= 14.068111419677734) ?  0.04248638119058837 :   0.0016502288832938572 :   (b('l8b4_med') <= 685.1024169921875) ?  -0.09772930961579795 :   0.004576768130892152 :   (b('l8b4_med') <= 1090.3720703125) ?  (b('ndvi_med') <= 1403.0) ?  -0.03406538293059176 :   -0.004195812760805717 :   (b('l8b4') <= 1185.5440673828125) ?  0.012923621277178365 :   -0.005893826506368736 :   (b('dg0_v_vv') <= -0.9993653297424316) ?  (b('dg0_v_vv') <= -1.0345611572265625) ?  (b('l8b11_med') <= 1435.263427734375) ?  0.01224232698356358 :   -0.04798905389183076 :   -0.09412152900759121 :   (b('ndvi') <= 4590.0) ?  (b('ndvi') <= 4177.5) ?  0.04321230780173285 :   -0.017474539544395526 :   (b('l8b11') <= 1770.9428100585938) ?  0.0701435285959662 :   0.12390503527650894 :   (b('ndvi_med') <= 2152.25) ?  (b('moss') <= 26.28097152709961) ?  (b('moss') <= 19.290181159973145) ?  (b('ndvi') <= 3276.5) ?  (b('grass') <= 52.981109619140625) ?  (b('ndvi_med') <= 1862.75) ?  0.008578569354401405 :   0.037743362057301125 :   (b('l8b5') <= 2420.0577392578125) ?  0.00554720338024719 :   -0.022242792313180552 :   (b('dg0_v_vv') <= 1.2884445190429688) ?  (b('l8b11') <= 769.2667541503906) ?  -0.10121168163395733 :   0.06892931799955246 :   (b('grass') <= 82.72326278686523) ?  0.017740374373189937 :   0.07913392713730817 :   (b('sand') <= 47.0) ?  (b('l8b11') <= 2369.9501953125) ?  (b('l8b11') <= 1623.2939453125) ?  0.04658538036052674 :   0.08363217498660502 :   (b('ndvi') <= 1039.5) ?  0.047695534178950755 :   -0.04136491146639176 :   (b('l8b4') <= 1209.4842529296875) ?  (b('sand') <= 50.0) ?  -0.051032966504603726 :   0.03836696201745643 :   (b('l8b5_med') <= 2321.138916015625) ?  -0.07943645522146113 :   -0.012448112423987585 :   (b('bulk') <= 145.0) ?  (b('grass') <= 56.823978424072266) ?  (b('dg0_v_vv') <= 2.2279391288757324) ?  (b('dg0_v_vv') <= 0.9019341468811035) ?  0.1131317742049904 :   0.02850915518055803 :   0.1145203465364732 :   (b('grass') <= 57.13207244873047) ?  -0.10703382410977566 :   (b('l8b11_med') <= 1940.4482421875) ?  0.06889376326494012 :   0.002099693152119651 :   (b('l8b4') <= 438.1578369140625) ?  0.10378953739172758 :   (b('ndvi') <= 993.5) ?  (b('dg0_v_vv') <= 2.376511573791504) ?  0.007175910039162496 :   0.05123986085665286 :   (b('l8b11') <= 2535.5020751953125) ?  -0.020346947506778068 :   0.023839801566635285 :   (b('sand') <= 49.5) ?  (b('ndvi_med') <= 4053.5) ?  (b('l8b11_med') <= 1975.9132690429688) ?  (b('moss') <= 19.307927131652832) ?  (b('bulk') <= 160.5) ?  0.00470788549561156 :   0.04959845801346686 :   (b('ndvi') <= 1814.0) ?  0.07078809639992753 :   -0.019416143794944652 :   (b('l8b4') <= 1105.7637939453125) ?  (b('crops') <= 51.44186973571777) ?  0.04214012012216367 :   -0.01046450102697295 :   (b('l8b5') <= 2217.909423828125) ?  -0.07521725115806152 :   0.006220049689429066 :   (b('dg0_v_vv') <= 3.5594003200531006) ?  (b('dg0_v_vv') <= 1.500047206878662) ?  (b('l8b4') <= 990.9330139160156) ?  0.035370978447952006 :   0.054124572233672286 :   (b('l8b4_med') <= 846.61181640625) ?  0.0763869490604613 :   0.1665293305391472 :   -0.05672100713194206 :   (b('l8b11') <= 2460.0693359375) ?  (b('crops') <= 81.01694869995117) ?  (b('l8b5') <= 4438.25244140625) ?  (b('bulk') <= 119.0) ?  -0.03734470678866954 :   -0.00822389301853249 :   (b('dg0_v_vv') <= 1.2596700191497803) ?  0.038247143243822834 :   -0.04811676608069999 :   (b('ndvi') <= 3443.5) ?  (b('ndvi') <= 3122.5) ?  0.014005253652176272 :   -0.08039902611207272 :   (b('dg0_v_vv') <= 2.5169405937194824) ?  0.009342178834523462 :   0.02656553958911092 :   (b('l8b11') <= 2657.7843017578125) ?  (b('l8b5') <= 2391.7308349609375) ?  0.1313224871725217 :   0.11819465481521313 :   (b('ndvi') <= 3191.0) ?  (b('ndvi_med') <= 2483.0) ?  -0.02268047870828578 :   0.06196705890526305 :   (b('dg0_v_vv') <= 4.594395399093628) ?  -0.00814464360427175 :   0.0211275706952129 :   (b('ndvi_med') <= 2220.5) ?  (b('moss') <= 25.738844871520996) ?  (b('grass') <= 1.0004997551441193) ?  (b('l8b4_med') <= 1284.1814880371094) ?  (b('dg0_v_vv') <= -2.1527557373046875) ?  0.09891707372056419 :   (b('dg0_v_vv') <= 1.0148215293884277) ?  0.035904776944732014 :   0.050834573695253904 :   (b('l8b5') <= 4265.47412109375) ?  (b('l8b5') <= 4139.89306640625) ?  (b('l8b11') <= 2665.5185546875) ?  (b('l8b11') <= 2535.6141357421875) ?  (b('l8b4') <= 1718.9151611328125) ?  0.008267440169152743 :   -0.0049434333940617204 :   (b('dg0_v_vv') <= -1.0296378135681152) ?  0.007792528221339805 :   0.038649832812828494 :   (b('l8b4') <= 2452.17919921875) ?  (b('l8b4') <= 2157.1539306640625) ?  0.0007991251434523981 :   -0.014297462155592432 :   (b('l8b4') <= 2487.7574462890625) ?  0.06594003189285254 :   0.001991112080587099 :   (b('dg0_v_vv') <= 0.31720781326293945) ?  (b('grass') <= 0.27586206793785095) ?  -0.019530607428006108 :   (b('l8b4') <= 2171.085693359375) ?  0.005348738057150693 :   0.024943372608708048 :   (b('dg0_v_vv') <= 1.1181540489196777) ?  (b('l8b11') <= 3880.1253662109375) ?  0.04622850502522286 :   0.06129014958946136 :   0.07469129324412219 :   (b('dg0_v_vv') <= -0.09918832778930664) ?  (b('dg0_v_vv') <= -0.38329410552978516) ?  (b('l8b5') <= 4516.5517578125) ?  (b('dg0_v_vv') <= -0.9239721298217773) ?  -0.022990731147327026 :   -0.013588138808113524 :   -0.023471578655162723 :   (b('l8b5') <= 4311.6318359375) ?  (b('ndvi') <= 995.0) ?  -0.024235150333660527 :   -0.01810814820085832 :   (b('l8b11') <= 4143.648681640625) ?  -0.0350119265546013 :   -0.029825940954759378 :   (b('dg0_v_vv') <= 0.7923831939697266) ?  (b('l8b5') <= 4345.333984375) ?  (b('l8b4_med') <= 3496.1298828125) ?  -0.014631453267251723 :   -0.016952582595698013 :   (b('dg0_v_vv') <= 0.17461013793945312) ?  -0.007621828801718633 :   0.005691725210080689 :   (b('dg0_v_vv') <= 0.8560419082641602) ?  -0.027376088231784404 :   (b('l8b4') <= 2446.7137451171875) ?  -0.01958001419346922 :   -0.01819977421247551 :   (b('l8b11_med') <= 2575.8380126953125) ?  (b('l8b11_med') <= 2467.304443359375) ?  (b('ndvi') <= 2630.0) ?  (b('sand') <= 41.5) ?  (b('l8b5') <= 3489.5054931640625) ?  (b('dg0_v_vv') <= 3.889493465423584) ?  -0.01470079730354712 :   0.07563015617494291 :   0.14913386213529345 :   (b('grass') <= 98.81764602661133) ?  (b('l8b4') <= 1401.0753784179688) ?  -0.04480400714500807 :   -0.00523099588309459 :   (b('ndvi') <= 2046.5) ?  0.03814035863403136 :   -0.039014200643350865 :   (b('l8b5') <= 3752.165771484375) ?  (b('l8b5') <= 3523.06005859375) ?  (b('l8b11') <= 1950.6644897460938) ?  0.060462056119049576 :   0.012190643958335393 :   (b('dg0_v_vv') <= -2.06105375289917) ?  0.020203716548592432 :   0.07754930947152142 :   (b('l8b11') <= 2024.5573120117188) ?  (b('l8b4_med') <= 1536.2318115234375) ?  -0.06078363069411133 :   -0.06548301415572534 :   (b('l8b5') <= 4376.672119140625) ?  -0.024188566994753524 :   0.030093585797676736 :   (b('ndvi') <= 1535.5) ?  (b('l8b4') <= 2098.8707275390625) ?  (b('grass') <= 76.885498046875) ?  (b('moss') <= 17.717455863952637) ?  -0.009952933296968013 :   0.006159679997187615 :   (b('ndvi') <= 1053.0) ?  -0.02886279091033006 :   0.06064105131738336 :   (b('l8b4') <= 2116.661865234375) ?  (b('l8b11') <= 3182.1270751953125) ?  0.11590345865663157 :   0.059464743503030465 :   (b('l8b11') <= 2550.810302734375) ?  -0.02443691471397969 :   0.015582811770461199 :   (b('ndvi_med') <= 1327.0) ?  (b('ndvi') <= 1784.0) ?  (b('bulk') <= 147.0) ?  0.14192664185946122 :   0.11055633617365362 :   0.17549714295457167 :   (b('dg0_v_vv') <= 0.5947918891906738) ?  (b('ndvi') <= 1558.5) ?  0.06219367553061261 :   0.005479977428744746 :   (b('l8b5_med') <= 2517.19384765625) ?  0.08951397074511575 :   0.036057753913383866 :   (b('grass') <= 16.03073215484619) ?  (b('l8b4_med') <= 2324.4437255859375) ?  (b('l8b5') <= 3343.612060546875) ?  (b('l8b5') <= 3308.4671630859375) ?  (b('grass') <= 1.900430142879486) ?  -0.026401108438642008 :   -0.014067395309130823 :   (b('l8b4') <= 1858.718017578125) ?  -0.002738970977952902 :   -0.034635155536196754 :   (b('l8b5_med') <= 3148.1982421875) ?  (b('l8b4') <= 2470.2301025390625) ?  0.014663433286220245 :   -0.012058944766738867 :   (b('l8b4') <= 1443.2019653320312) ?  -0.030071022515152188 :   -0.013132919074453764 :   (b('l8b11') <= 2880.49267578125) ?  (b('sand') <= 57.0) ?  -0.0490236412827795 :   (b('ndvi') <= 747.5) ?  0.03792407150699858 :   0.009989654639347879 :   (b('ndvi') <= 997.5) ?  (b('l8b11') <= 2890.9833984375) ?  -0.02970211695893421 :   -0.0035619720915899826 :   (b('l8b11') <= 3844.948974609375) ?  0.02061260042293202 :   -0.015089342422730072 :   (b('dg0_v_vv') <= 1.4107933044433594) ?  (b('crops') <= 50.07838439941406) ?  (b('l8b4_med') <= 1716.1549682617188) ?  (b('ndvi_med') <= 1565.75) ?  -0.03280189993371098 :   -0.0179162676289607 :   (b('ndvi_med') <= 1821.25) ?  -0.009743044257532378 :   0.037566617796863086 :   (b('l8b5') <= 2483.3045654296875) ?  (b('sand') <= 56.5) ?  0.03495388692734854 :   -0.04001129589558626 :   (b('l8b5') <= 2605.5657958984375) ?  -0.04757929621827014 :   -0.0013290250011339527 :   (b('l8b4') <= 2081.8043212890625) ?  (b('l8b5_med') <= 2586.519287109375) ?  (b('l8b4_med') <= 1466.2388916015625) ?  0.06373272228058147 :   -0.027360259558915564 :   (b('ndvi') <= 1136.5) ?  0.03547918265824677 :   0.01109589187262099 :   (b('ndvi') <= 1717.5) ?  (b('ndvi') <= 1354.0) ?  -0.0011575628872181809 :   -0.023461159747332313 :   (b('l8b5') <= 3039.7635498046875) ?  -0.040140591472427184 :   0.02152233456508374 :   (b('ndvi_med') <= 913.0) ?  (b('ndvi_med') <= 885.25) ?  (b('ndvi') <= 1066.0) ?  (b('l8b11') <= 2240.00244140625) ?  (b('l8b5_med') <= 1962.0574951171875) ?  0.0038286964340566443 :   (b('l8b5_med') <= 2060.6375732421875) ?  0.10282430565578268 :   0.08158309118803579 :   (b('l8b11') <= 2532.225830078125) ?  (b('ndvi') <= 767.5) ?  (b('l8b11') <= 2280.062255859375) ?  0.018512244135230227 :   -0.028067713311784974 :   (b('ndvi') <= 831.5) ?  -0.0898158451156105 :   -0.04709835295713893 :   (b('l8b11') <= 2892.8363037109375) ?  (b('l8b11') <= 2811.8531494140625) ?  -0.005985197600543874 :   0.036692502350635155 :   (b('dg0_v_vv') <= 0.8767380714416504) ?  -0.017545615229844585 :   0.01729449647404302 :   (b('l8b4') <= 1609.8237915039062) ?  (b('ndvi_med') <= 866.75) ?  (b('l8b4') <= 1531.70166015625) ?  -0.0019041971913921485 :   (b('l8b5') <= 1998.9990844726562) ?  0.020845439316727554 :   0.023926897482461323 :   (b('grass') <= 52.54391670227051) ?  -0.031376998371666937 :   -0.007908722849362032 :   (b('l8b4_med') <= 1799.5505981445312) ?  (b('dg0_v_vv') <= -0.3745427131652832) ?  0.0565775688447773 :   (b('dg0_v_vv') <= 0.9615426063537598) ?  0.042330665635106235 :   0.04754486205412259 :   0.08176126021903216 :   (b('dg0_v_vv') <= -0.7115159034729004) ?  0.19885388534739665 :   (b('dg0_v_vv') <= -0.5178718566894531) ?  -0.10534610198244984 :   (b('l8b11') <= 2383.1998291015625) ?  (b('dg0_v_vv') <= -0.17517852783203125) ?  0.1777057762241131 :   (b('l8b5') <= 1969.879638671875) ?  0.05787761623899504 :   0.07692157878443376 :   (b('l8b4') <= 1765.06689453125) ?  (b('moss') <= 41.09561538696289) ?  0.01020945749140878 :   -0.0055952447569818375 :   -0.06648947139954564 :   (b('ndvi') <= 756.5) ?  0.14134564963955065 :   (b('l8b5_med') <= 2012.2635498046875) ?  (b('l8b11') <= 1979.1448364257812) ?  (b('ndvi') <= 1041.0) ?  -0.009331279598630088 :   0.03166872153385816 :   (b('l8b5') <= 2305.4376220703125) ?  (b('grass') <= 52.54879951477051) ?  (b('moss') <= 38.71941947937012) ?  -0.03424920816035171 :   -0.024151729171873693 :   (b('l8b11_med') <= 2062.9906005859375) ?  -0.040018917881616746 :   -0.054233738298005554 :   (b('l8b11') <= 2443.8194580078125) ?  0.0029496453748437818 :   (b('l8b11_med') <= 2062.9906005859375) ?  -0.004333517468261133 :   -0.022456140676045414 :   (b('ndvi') <= 1024.0) ?  (b('ndvi') <= 821.5) ?  (b('l8b5') <= 2030.1107788085938) ?  (b('l8b5_med') <= 2043.7534790039062) ?  0.01326684595690894 :   0.015420724351183523 :   (b('sand') <= 58.0) ?  0.03473975552839434 :   0.027083644589409178 :   (b('l8b4') <= 1954.1344604492188) ?  (b('dg0_v_vv') <= 0.41098880767822266) ?  -0.026590037719707315 :   -0.04274627701908169 :   (b('dg0_v_vv') <= 1.2663788795471191) ?  -0.017223644100893214 :   0.10839291425755135 :   (b('l8b11') <= 3051.55859375) ?  (b('l8b11') <= 2988.813720703125) ?  (b('dg0_v_vv') <= 2.192779541015625) ?  -0.0122810281221657 :   0.013002320756009085 :   (b('l8b11') <= 3011.5509033203125) ?  0.042326869740227084 :   0.001924198702478792 :   (b('dg0_v_vv') <= 0.5583019256591797) ?  (b('dg0_v_vv') <= 0.5362772941589355) ?  -0.014321572500552951 :   0.02426902826669452 :   (b('l8b5') <= 3621.52734375) ?  -0.036009676933090366 :   -0.017875755338575005 :   (b('moss') <= 4.653807640075684) ?  (b('l8b4') <= 1421.1156005859375) ?  (b('ndvi') <= 2088.0) ?  (b('ndvi_med') <= 2662.75) ?  (b('l8b11') <= 2303.70654296875) ?  (b('dg0_v_vv') <= 3.289245128631592) ?  0.04887377721171268 :   0.05536823275806391 :   (b('l8b5') <= 2685.5552978515625) ?  (b('l8b5_med') <= 3029.82763671875) ?  (b('l8b4') <= 1407.907470703125) ?  -0.015260890522639861 :   -0.02075318903232706 :   -0.06264832836349898 :   (b('dg0_v_vv') <= -0.22892427444458008) ?  -0.08552015279141853 :   -0.049339685770699276 :   (b('l8b5_med') <= 2797.3056640625) ?  -0.1400299344393111 :   (b('l8b11') <= 2334.3419189453125) ?  -0.09035277461402816 :   -0.10231913468861961 :   (b('l8b4_med') <= 850.6034545898438) ?  (b('ndvi') <= 3911.0) ?  (b('dg0_v_vv') <= -2.061121940612793) ?  0.004064714512734269 :   (b('l8b4') <= 1415.0947875976562) ?  -0.019815297687886063 :   -0.030112865213629852 :   (b('dg0_v_vv') <= -0.25640153884887695) ?  (b('ndvi_med') <= 3351.5) ?  -0.059366074666440116 :   -0.05377668060878503 :   -0.07080492526557045 :   (b('l8b4') <= 1416.111083984375) ?  (b('l8b11') <= 2321.4666748046875) ?  (b('ndvi') <= 4323.5) ?  (b('ndvi') <= 4313.0) ?  0.016787989882746497 :   -0.0047273643238203615 :   (b('l8b4_med') <= 1303.0054931640625) ?  0.039343101245420564 :   0.05105629260680561 :   (b('sand') <= 36.0) ?  0.04502704616434888 :   (b('l8b5') <= 2726.331298828125) ?  0.06715601323253456 :   0.06096655346520796 :   (b('dg0_v_vv') <= 0.7669758796691895) ?  -0.031616915712785826 :   (b('dg0_v_vv') <= 2.3839473724365234) ?  (b('ndvi_med') <= 2662.75) ?  -0.0049747722989854914 :   0.020040027985402123 :   (b('ndvi') <= 3867.0) ?  -0.03789493828048329 :   -0.024784985285133482 :   (b('sand') <= 22.0) ?  (b('l8b11_med') <= 1220.2855834960938) ?  (b('dg0_v_vv') <= -0.6269793510437012) ?  0.07611853701044821 :   0.09807869071618835 :   (b('ndvi') <= 2306.0) ?  (b('ndvi') <= 1991.5) ?  (b('l8b11') <= 2186.4622802734375) ?  0.06620863404941674 :   (b('l8b5_med') <= 2931.249267578125) ?  0.059631171654680626 :   0.05750815855974806 :   0.0851319924646986 :   (b('ndvi') <= 2760.0) ?  (b('l8b5_med') <= 3177.6944580078125) ?  (b('l8b4') <= 1604.5703735351562) ?  0.04748639596642745 :   0.04997681967511125 :   0.055665102229092184 :   0.03234499291821008 :   (b('l8b5') <= 3620.7364501953125) ?  (b('bulk') <= 148.0) ?  (b('grass') <= 74.13162231445312) ?  (b('crops') <= 64.08968353271484) ?  (b('dg0_v_vv') <= -1.601163387298584) ?  0.010362545584313494 :   0.037575698581769595 :   (b('l8b4_med') <= 1173.0162963867188) ?  -0.004351229020610438 :   0.01977144617762366 :   (b('dg0_v_vv') <= -0.5417356491088867) ?  (b('l8b11_med') <= 1844.1785888671875) ?  -0.0076075079773297305 :   -0.051145097236523884 :   (b('l8b4') <= 1650.705810546875) ?  -0.013688640917575648 :   0.04938310750948863 :   (b('dg0_v_vv') <= -0.8525576591491699) ?  (b('ndvi') <= 1362.5) ?  (b('l8b11') <= 2852.9302978515625) ?  0.039078093418027426 :   0.07984185745322925 :   (b('bulk') <= 157.5) ?  -0.01222385750382549 :   0.02912057198340341 :   (b('l8b4_med') <= 1042.0471801757812) ?  -0.034790775772974306 :   (b('l8b11') <= 2243.0712890625) ?  0.10599403996131135 :   0.0683916873623495 :   (b('l8b5') <= 3907.7401123046875) ?  (b('l8b5_med') <= 2966.51904296875) ?  -0.07447647646307018 :   (b('ndvi') <= 1879.0) ?  (b('grass') <= 19.244840621948242) ?  -0.05280439359191977 :   -0.06004012447091228 :   (b('l8b11_med') <= 2053.1215209960938) ?  -0.051255181506454896 :   -0.009620071904437547 :   (b('ndvi') <= 4469.5) ?  -0.010894421386231062 :   -0.002448063548898205 :   (b('ndvi') <= 1724.5) ?  (b('l8b4_med') <= 1004.7339477539062) ?  (b('bulk') <= 141.5) ?  (b('l8b5_med') <= 2465.3048095703125) ?  -0.007897058973509716 :   0.016986101788671104 :   (b('crops') <= 24.741840362548828) ?  (b('l8b11_med') <= 1784.0094604492188) ?  0.04702936501227459 :   0.03893318859071146 :   0.05557206577936008 :   (b('l8b5') <= 3025.8671875) ?  (b('l8b5_med') <= 3097.873046875) ?  (b('dg0_v_vv') <= -1.02711820602417) ?  0.09296754894222542 :   (b('dg0_v_vv') <= -0.82562255859375) ?  0.13683854452691935 :   0.10011787534987515 :   0.13491024316366812 :   0.04571880558776123 :   (b('dg0_v_vv') <= 2.724613666534424) ?  (b('sand') <= 62.0) ?  (b('ndvi') <= 1832.0) ?  (b('l8b5') <= 3845.208251953125) ?  (b('l8b5') <= 3495.288330078125) ?  (b('l8b4_med') <= 1254.496826171875) ?  -0.04098414117744292 :   -0.05073491309706968 :   (b('ndvi_med') <= 2378.75) ?  -0.03577665817707035 :   -0.0269613857537155 :   -0.09226850306548137 :   (b('bulk') <= 120.5) ?  0.09718707627458834 :   (b('moss') <= 28.85543727874756) ?  (b('ndvi') <= 4081.5) ?  -0.012228253621076327 :   0.025867258332551147 :   (b('l8b5') <= 3474.2979736328125) ?  0.043931085264620785 :   0.010811037023045885 :   (b('dg0_v_vv') <= -0.15805816650390625) ?  (b('crops') <= 64.95339584350586) ?  (b('l8b5') <= 2863.3316650390625) ?  (b('l8b11') <= 2584.680419921875) ?  -0.05438726304937319 :   -0.058429813826438845 :   (b('dg0_v_vv') <= -0.5741887092590332) ?  -0.06268371984795189 :   -0.06811339868261758 :   -0.03369598294297883 :   -0.02362664126595991 :   (b('l8b11') <= 2958.3758544921875) ?  (b('l8b11') <= 2077.772705078125) ?  (b('ndvi') <= 2714.5) ?  0.08999109278593015 :   0.10247445373905834 :   (b('ndvi') <= 3007.0) ?  (b('ndvi') <= 2157.5) ?  0.055738276927452535 :   0.06072573160998526 :   (b('l8b11_med') <= 2365.364013671875) ?  0.05096707490859548 :   0.02980010805916805 :   (b('l8b4_med') <= 814.8666687011719) ?  -0.0183913465474492 :   -0.020564823557006312 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 3319.5) ?  (b('l8b11') <= 2100.7135009765625) ?  (b('dg0_v_vv') <= 2.725463628768921) ?  (b('l8b11') <= 2100.443115234375) ?  (b('l8b4') <= 549.7540588378906) ?  (b('sand') <= 17.0) ?  (b('ndvi') <= 3642.5) ?  (b('bulk') <= 149.0) ?  0.11478258764872418 :   0.10845215552561516 :   0.0977251365787507 :   (b('l8b11') <= 1157.0029296875) ?  (b('ndvi') <= 2119.0) ?  (b('bulk') <= 156.0) ?  (b('l8b5') <= 2710.0645751953125) ?  -0.010783763762801107 :   0.03781826932315707 :   (b('ndvi') <= 1977.0) ?  -0.04199334219400738 :   -0.13129496512361472 :   (b('crops') <= 22.789156913757324) ?  (b('dg0_v_vv') <= 0.12810516357421875) ?  0.004877182130087483 :   0.03552103504409112 :   (b('l8b11_med') <= 2054.3936767578125) ?  0.005015764747717078 :   -0.019438531432597633 :   (b('sand') <= 56.0) ?  (b('l8b4_med') <= 1447.3806762695312) ?  (b('moss') <= 15.113589763641357) ?  0.030765671029714278 :   0.06082311765003931 :   0.12444830196416573 :   (b('l8b4') <= 528.6606292724609) ?  -0.06416962331055492 :   -0.02412109778252025 :   (b('l8b4_med') <= 895.6596069335938) ?  (b('l8b4') <= 1123.6094360351562) ?  (b('sand') <= 33.0) ?  (b('grass') <= 16.96821689605713) ?  (b('crops') <= 92.60075759887695) ?  -0.01984778288490958 :   0.03851503378678109 :   (b('grass') <= 36.50906562805176) ?  -0.0006593213312192684 :   0.01524576098653875 :   (b('l8b4_med') <= 586.141845703125) ?  (b('l8b4_med') <= 432.3870086669922) ?  -0.029731100345907452 :   0.02525354827715492 :   (b('ndvi_med') <= 2233.25) ?  -0.003446151940627362 :   -0.013335261518593341 :   (b('l8b11_med') <= 1336.6654052734375) ?  (b('l8b4') <= 1278.7171020507812) ?  (b('l8b5_med') <= 3177.6944580078125) ?  -0.005169096300220499 :   0.05346429273202982 :   (b('dg0_v_vv') <= -0.2693467140197754) ?  -0.05837038252360955 :   -0.017833767919097465 :   (b('l8b11') <= 2092.361328125) ?  (b('ndvi_med') <= 2196.25) ?  0.09895713900046677 :   0.025650158952789652 :   0.1694918183203002 :   (b('l8b11') <= 1506.2876586914062) ?  (b('l8b5') <= 1607.140380859375) ?  (b('ndvi') <= 734.5) ?  (b('dg0_v_vv') <= 0.9207758903503418) ?  -0.031134186835536398 :   -0.0637276459431069 :   (b('ndvi_med') <= 1453.5) ?  0.04130256165568907 :   -0.01670002230892377 :   (b('l8b11') <= 934.2075500488281) ?  (b('l8b5') <= 2654.2439575195312) ?  0.16187827399244775 :   0.054551121026235036 :   (b('grass') <= 62.97923278808594) ?  0.0061352100915288595 :   0.025648387390077607 :   (b('bulk') <= 155.5) ?  (b('l8b11') <= 1578.9525756835938) ?  (b('sand') <= 34.5) ?  0.0015406582176851387 :   -0.019951227422427695 :   (b('l8b11') <= 1580.0833740234375) ?  0.10568758975673676 :   0.0020251939243785145 :   (b('l8b4') <= 816.7687683105469) ?  (b('l8b11') <= 1620.2086181640625) ?  -0.0554471711033576 :   0.06612507451083101 :   (b('l8b5') <= 3430.3507080078125) ?  -0.01370517567343087 :   -0.049583007550654576 :   (b('l8b4_med') <= 1070.2862548828125) ?  0.1264826458363983 :   0.22433303061883592 :   (b('l8b5') <= 2576.121826171875) ?  (b('bulk') <= 137.5) ?  (b('dg0_v_vv') <= 3.173226833343506) ?  (b('dg0_v_vv') <= 2.8025875091552734) ?  0.061176820130716736 :   (b('moss') <= 20.007519721984863) ?  (b('l8b11') <= 1712.9712524414062) ?  (b('moss') <= 16.32582187652588) ?  -0.02264035013423734 :   -0.07898360885547048 :   (b('l8b11_med') <= 2227.3177490234375) ?  0.01970806216932569 :   -0.022185145368264714 :   (b('sand') <= 41.5) ?  0.04117660588751454 :   0.02886105004608583 :   (b('ndvi') <= 2810.0) ?  (b('l8b5_med') <= 2942.0115966796875) ?  (b('dg0_v_vv') <= 5.720979690551758) ?  (b('bulk') <= 136.0) ?  0.02835268112079212 :   0.05779543807609182 :   0.07180916918267796 :   -0.029216615114225214 :   (b('ndvi') <= 3332.0) ?  (b('ndvi') <= 3024.0) ?  (b('l8b5') <= 2354.9598388671875) ?  -0.016225578641760677 :   -0.0034572328570871835 :   (b('l8b5_med') <= 2802.52197265625) ?  0.03743593562104722 :   0.02447450205178267 :   (b('l8b11_med') <= 1555.2938232421875) ?  -0.051091413995420276 :   (b('ndvi') <= 4942.0) ?  -0.030251202683803713 :   -0.010683023387320856 :   (b('l8b11_med') <= 2649.4033203125) ?  (b('l8b5') <= 2282.4739990234375) ?  (b('l8b5_med') <= 2296.1966552734375) ?  (b('l8b4_med') <= 1220.440185546875) ?  (b('l8b5_med') <= 2241.7164306640625) ?  0.01038562826539551 :   -0.048219373584519845 :   (b('ndvi') <= 1936.0) ?  0.02235792391012149 :   0.06662328819229779 :   (b('ndvi') <= 3299.5) ?  (b('ndvi_med') <= 2508.75) ?  0.033843509467112766 :   0.09692827684042843 :   (b('dg0_v_vv') <= 5.671208143234253) ?  -0.028117236690828502 :   0.03986522906110035 :   (b('l8b4') <= 529.8005065917969) ?  (b('l8b4_med') <= 1208.7486572265625) ?  0.11359760158923121 :   0.1059225889788055 :   (b('l8b4') <= 669.529541015625) ?  (b('l8b5_med') <= 2191.5059814453125) ?  0.04308001501204131 :   -0.006616177638240198 :   (b('ndvi_med') <= 2054.75) ?  0.07335668908965735 :   0.043796889480846736 :   (b('l8b5') <= 2095.160400390625) ?  (b('l8b11') <= 2009.6427612304688) ?  (b('l8b11') <= 919.488037109375) ?  -0.0503810157585311 :   (b('crops') <= 62.21405792236328) ?  0.01260252669251792 :   0.02840321745772666 :   0.12019438637889063 :   (b('l8b11_med') <= 2708.5074462890625) ?  -0.06768935036079432 :   (b('ndvi') <= 1108.0) ?  (b('dg0_v_vv') <= 3.517228126525879) ?  -0.022018454141090014 :   -0.028824853809861984 :   (b('l8b5') <= 2406.0882568359375) ?  0.0013873949489667254 :   -0.019815557153974872 :   (b('dg0_v_vv') <= 6.641358375549316) ?  (b('l8b5') <= 2680.17529296875) ?  (b('l8b5_med') <= 2562.8753662109375) ?  (b('l8b5_med') <= 2365.254638671875) ?  -0.04275956877353698 :   (b('l8b4') <= 989.5246276855469) ?  (b('ndvi_med') <= 2190.5) ?  -0.05987598336097588 :   -0.061752051008552605 :   -0.06610891939814233 :   (b('l8b5') <= 2597.3336181640625) ?  (b('l8b4_med') <= 1080.891357421875) ?  0.027492767926992026 :   0.01571700998782294 :   (b('sand') <= 31.5) ?  -0.04742043969168003 :   -0.03049115753008934 :   (b('bulk') <= 154.5) ?  (b('l8b11_med') <= 2411.9676513671875) ?  (b('l8b4_med') <= 813.8113098144531) ?  (b('dg0_v_vv') <= 2.7325119972229004) ?  0.08316326814646799 :   -0.014436805895799487 :   (b('l8b5_med') <= 2355.994140625) ?  0.05275164312782373 :   0.014499760662534432 :   (b('ndvi') <= 1759.0) ?  (b('ndvi') <= 1596.0) ?  -0.015150772539238317 :   0.06110907221581616 :   (b('grass') <= 69.00047874450684) ?  -0.013436399317580585 :   -0.07489977369035178 :   (b('ndvi') <= 3323.0) ?  (b('l8b4_med') <= 1758.8311767578125) ?  (b('l8b11_med') <= 1914.0880126953125) ?  0.052392128579275964 :   0.0916213058255124 :   0.029290690986849 :   (b('sand') <= 43.5) ?  (b('l8b5') <= 3440.5072021484375) ?  0.004495558634485489 :   -0.021684648239117546 :   (b('ndvi_med') <= 1768.75) ?  0.04516566228974861 :   0.05923042684126867 :   (b('moss') <= 7.48991584777832) ?  (b('dg0_v_vv') <= 13.441749572753906) ?  (b('dg0_v_vv') <= 7.052134037017822) ?  -0.06879292597858798 :   (b('l8b5_med') <= 2657.87060546875) ?  -0.07370723944946511 :   -0.07140635296352232 :   -0.04640635445363844 :   -0.03418781675100366 :   (b('bulk') <= 124.0) ?  (b('l8b4') <= 1226.53125) ?  (b('dg0_v_vv') <= 0.435011625289917) ?  (b('dg0_v_vv') <= -1.878342628479004) ?  (b('dg0_v_vv') <= -2.9868955612182617) ?  0.03367818494932345 :   0.02582630671355121 :   (b('dg0_v_vv') <= -0.2626676559448242) ?  (b('l8b5_med') <= 2018.583740234375) ?  (b('ndvi') <= 1661.0) ?  0.07710294074183138 :   0.0613771850918723 :   (b('dg0_v_vv') <= -1.046877384185791) ?  0.05851302240985545 :   0.061141463602995466 :   (b('sand') <= 27.0) ?  0.05383446123287716 :   0.01914146789452989 :   (b('l8b5') <= 2047.1611938476562) ?  0.09427760979023495 :   0.11123231256893851 :   (b('l8b5') <= 2382.5284423828125) ?  (b('dg0_v_vv') <= -0.3033022880554199) ?  (b('l8b5_med') <= 2018.583740234375) ?  0.017472725727713898 :   (b('dg0_v_vv') <= -0.43898916244506836) ?  (b('dg0_v_vv') <= -0.8590388298034668) ?  0.032314299250258804 :   0.05181472446823937 :   (b('ndvi') <= 1343.5) ?  0.024814729355820242 :   0.030256999368063253 :   (b('l8b5') <= 1932.9437255859375) ?  (b('dg0_v_vv') <= 0.1382899284362793) ?  -0.04898235541431063 :   -0.03298571048298339 :   (b('ndvi') <= 1411.5) ?  (b('dg0_v_vv') <= 0.04117727279663086) ?  (b('ndvi_med') <= 1670.5) ?  0.0005852810425445443 :   -0.004176324965456918 :   -0.014644163932871046 :   (b('sand') <= 46.5) ?  (b('l8b4') <= 1481.2631225585938) ?  -0.02209155885016344 :   -0.01861371457252388 :   -0.03214946341431481 :   (b('ndvi_med') <= 1383.5) ?  0.045751116325728536 :   (b('l8b4') <= 1512.5305786132812) ?  0.13031660866859132 :   0.08902536634850777 :   (b('moss') <= 10.338469982147217) ?  (b('moss') <= 9.916528701782227) ?  (b('l8b5_med') <= 2597.39697265625) ?  (b('dg0_v_vv') <= -0.46715784072875977) ?  (b('dg0_v_vv') <= -0.7403535842895508) ?  (b('l8b11_med') <= 2390.1497802734375) ?  (b('l8b11_med') <= 2280.2601318359375) ?  -0.007757439224185315 :   0.026656460580370675 :   (b('l8b11') <= 2693.12451171875) ?  -0.004137887196469075 :   -0.041379827440806406 :   (b('dg0_v_vv') <= -0.7209467887878418) ?  0.25923142159180723 :   (b('l8b11') <= 2300.6461181640625) ?  -0.024784844899229566 :   0.020401003544600792 :   (b('ndvi') <= 2074.5) ?  (b('l8b4') <= 1354.525146484375) ?  (b('l8b4_med') <= 1486.3300170898438) ?  -0.031404917587904806 :   0.08093458996802583 :   (b('l8b11') <= 3061.0054931640625) ?  -0.013756131693423882 :   -0.06108741935880448 :   (b('l8b5') <= 1845.9402465820312) ?  (b('l8b5_med') <= 2000.6098022460938) ?  0.19457117470943908 :   0.05946189993985107 :   (b('grass') <= 93.00495910644531) ?  -0.013246273446370376 :   0.01721152582993187 :   (b('grass') <= 96.0947380065918) ?  (b('l8b5') <= 1988.8370971679688) ?  (b('l8b5') <= 1786.7101440429688) ?  (b('grass') <= 43.40882062911987) ?  0.04859508744404574 :   -0.037456054824329704 :   (b('l8b11') <= 2337.49560546875) ?  -0.09757576449370112 :   -0.04267446729326364 :   (b('l8b5') <= 2449.1190185546875) ?  (b('l8b5') <= 2446.7840576171875) ?  0.008868690996370795 :   0.1337040970842147 :   (b('bulk') <= 150.5) ?  -0.007310881208112548 :   -0.0010248313979424337 :   (b('l8b4') <= 1279.5938110351562) ?  (b('ndvi') <= 909.0) ?  0.09675381023114168 :   (b('l8b4') <= 1213.6495361328125) ?  0.06280414462667945 :   0.03702370555398973 :   (b('ndvi') <= 1335.0) ?  (b('dg0_v_vv') <= 0.32150745391845703) ?  0.04149627412799548 :   0.12381402078948661 :   (b('ndvi_med') <= 1995.75) ?  -0.014494478002610629 :   0.01929991831641231 :   (b('dg0_v_vv') <= 0.32656192779541016) ?  (b('ndvi') <= 2812.0) ?  (b('dg0_v_vv') <= -1.0689482688903809) ?  0.04477548830166153 :   (b('ndvi') <= 2376.0) ?  (b('dg0_v_vv') <= -0.3725910186767578) ?  0.03689438693446817 :   0.017996644971680048 :   0.011462613425161139 :   (b('dg0_v_vv') <= -0.5605316162109375) ?  (b('l8b5_med') <= 2401.4825439453125) ?  0.011055076431294969 :   0.04144332808402357 :   (b('ndvi_med') <= 2984.0) ?  0.0680550578346458 :   0.07044332724955854 :   0.14729650703774938 :   (b('dg0_v_vv') <= 3.491380453109741) ?  (b('l8b5_med') <= 2016.0888061523438) ?  (b('moss') <= 24.224138259887695) ?  (b('moss') <= 22.97331428527832) ?  (b('l8b5') <= 2055.61376953125) ?  (b('dg0_v_vv') <= 0.8702497482299805) ?  0.021801581523193503 :   -0.07712283641308787 :   (b('ndvi_med') <= 976.0) ?  -0.06445250866775329 :   -0.017405249986968765 :   (b('l8b5') <= 2329.660400390625) ?  (b('dg0_v_vv') <= -0.4753694534301758) ?  0.048578177812145434 :   0.05712601204162235 :   0.13164610374544322 :   (b('l8b11_med') <= 1980.844482421875) ?  (b('l8b11_med') <= 1596.5263671875) ?  0.08870658508470561 :   (b('l8b4_med') <= 1071.31494140625) ?  -0.02522687773409702 :   0.026459210025459017 :   (b('sand') <= 59.5) ?  (b('dg0_v_vv') <= 0.5413403511047363) ?  -0.030944868983175746 :   -0.04930064027246657 :   (b('l8b4_med') <= 1214.5320434570312) ?  0.03263403426308008 :   0.014375527274374772 :   (b('l8b5') <= 1629.3587036132812) ?  (b('bulk') <= 155.0) ?  (b('l8b11') <= 2413.0155029296875) ?  0.07634163327281496 :   0.06956670531062548 :   0.13155322444917011 :   (b('ndvi_med') <= 1696.5) ?  (b('l8b11') <= 2111.16162109375) ?  (b('l8b4') <= 1449.1803588867188) ?  0.003014095580893323 :   0.0548735738206136 :   (b('l8b4') <= 1553.0040893554688) ?  -0.0006808370710114498 :   -0.007814604166333722 :   (b('ndvi_med') <= 2149.0) ?  (b('l8b5') <= 2227.7791748046875) ?  -0.026166915570307186 :   -0.012074756690011925 :   (b('ndvi') <= 1745.5) ?  0.02982728216272738 :   -0.0054931698415161875 :   (b('bulk') <= 156.0) ?  (b('dg0_v_vv') <= 3.507617712020874) ?  0.12645836757385026 :   (b('grass') <= 2.522662878036499) ?  (b('dg0_v_vv') <= 11.14775562286377) ?  (b('l8b4') <= 1782.654296875) ?  0.035319896087319606 :   0.08899815642008913 :   -0.03053304257024181 :   (b('sand') <= 37.0) ?  0.045820273810649015 :   (b('moss') <= 32.801971435546875) ?  0.0013364435154322545 :   -0.0322195731303739 :   (b('l8b4') <= 1406.1349487304688) ?  (b('ndvi') <= 1031.0) ?  0.0775658230077289 :   (b('l8b11_med') <= 2490.1583251953125) ?  0.11177397657734678 :   0.09752234537178789 :   0.03313014272272857 :   (b('grass') <= 45.82082176208496) ?  (b('l8b11_med') <= 1439.7363891601562) ?  (b('l8b11_med') <= 1437.3794555664062) ?  (b('l8b11_med') <= 1205.32763671875) ?  (b('l8b5_med') <= 2702.0235595703125) ?  (b('l8b5') <= 2098.4482421875) ?  (b('l8b5') <= 1995.0030517578125) ?  (b('ndvi') <= 4118.0) ?  (b('dg0_v_vv') <= 2.9761877059936523) ?  -0.005752316823379781 :   -0.0301689911701781 :   0.03654183686289228 :   (b('dg0_v_vv') <= -0.7253715991973877) ?  0.019394203241075908 :   (b('bulk') <= 131.5) ?  0.03170584803568435 :   0.03927666074204836 :   (b('l8b11') <= 1173.4019165039062) ?  (b('ndvi') <= 4307.0) ?  (b('l8b11') <= 880.1746826171875) ?  -0.02196300028624812 :   0.023818692615299654 :   (b('l8b11') <= 891.3624267578125) ?  0.004788048325648229 :   -0.02582795050169081 :   (b('l8b11') <= 2088.9275512695312) ?  (b('dg0_v_vv') <= 0.31402587890625) ?  -0.04364204200045696 :   -0.014238816106935566 :   0.014414256156877525 :   (b('grass') <= 42.728538513183594) ?  (b('ndvi') <= 4884.5) ?  (b('crops') <= 39.00959014892578) ?  (b('bulk') <= 137.5) ?  -0.004626770347888313 :   0.03520321858971723 :   (b('l8b11') <= 540.7763977050781) ?  -0.03285167150979122 :   0.007611070181655936 :   (b('sand') <= 38.5) ?  (b('dg0_v_vv') <= 1.1350011825561523) ?  0.060303380865342295 :   -0.013951942410283358 :   (b('bulk') <= 137.5) ?  0.005331821556954948 :   -0.0826604294431777 :   (b('l8b5') <= 3415.1365966796875) ?  (b('ndvi') <= 3085.0) ?  (b('ndvi') <= 2264.0) ?  0.04069005208443127 :   0.0003115871008226023 :   (b('l8b5_med') <= 3503.7686767578125) ?  -0.054472089624897246 :   -0.020538375403467535 :   (b('dg0_v_vv') <= -1.5862045288085938) ?  (b('dg0_v_vv') <= -2.236295223236084) ?  0.031567504703948646 :   -0.013654599000314302 :   (b('l8b4') <= 746.9760437011719) ?  0.053728269695470184 :   0.010688883721439547 :   (b('ndvi') <= 3613.5) ?  (b('ndvi') <= 2431.0) ?  (b('l8b11') <= 1182.5386352539062) ?  (b('l8b11') <= 1113.388427734375) ?  (b('ndvi_med') <= 3373.0) ?  0.021763495956191925 :   0.036594952562699054 :   (b('l8b5_med') <= 2785.7506103515625) ?  0.061593058775594435 :   0.05054252545216295 :   (b('ndvi') <= 2015.5) ?  (b('ndvi') <= 1816.0) ?  0.020525549183599674 :   0.0016492941599510693 :   (b('sand') <= 29.5) ?  0.03512538447746504 :   -0.01043716816358714 :   (b('dg0_v_vv') <= 1.1070456504821777) ?  (b('crops') <= 1.3697916269302368) ?  (b('ndvi_med') <= 4170.0) ?  0.06447446894725622 :   -0.02902503769821395 :   (b('ndvi') <= 2590.5) ?  -0.04575022437258174 :   -0.004866406422276227 :   (b('bulk') <= 131.5) ?  (b('l8b5') <= 2427.883544921875) ?  0.06202828428531246 :   -0.004044049338476046 :   (b('ndvi') <= 2690.5) ?  -0.011387704907804701 :   0.038315800588835366 :   (b('l8b4') <= 733.7358093261719) ?  (b('dg0_v_vv') <= -0.02687358856201172) ?  (b('ndvi') <= 4917.0) ?  (b('l8b11_med') <= 1321.7329711914062) ?  0.014359247004811365 :   -0.010020015437437555 :   (b('l8b11') <= 1268.0344848632812) ?  0.03863302335443677 :   0.004839633540175271 :   (b('l8b5') <= 4534.626708984375) ?  (b('bulk') <= 130.0) ?  0.040002980754078835 :   0.014032968770237556 :   (b('l8b5') <= 4830.469482421875) ?  -0.053139002898950696 :   0.05373526419714314 :   (b('bulk') <= 116.0) ?  (b('l8b5') <= 2599.884521484375) ?  -0.04372384814780031 :   -0.04595721260840555 :   (b('l8b5_med') <= 2711.7161865234375) ?  (b('l8b5') <= 2841.0328369140625) ?  0.029917443572359807 :   0.09988345758128943 :   (b('ndvi') <= 4418.5) ?  0.030718961271355424 :   0.010201401786284663 :   (b('ndvi') <= 4084.0) ?  (b('l8b4') <= 1017.5652770996094) ?  (b('dg0_v_vv') <= -0.6485958099365234) ?  0.007810866545854639 :   -0.0024942166548895806 :   (b('l8b4') <= 1834.738037109375) ?  (b('l8b5') <= 3573.624755859375) ?  0.05230814409849377 :   0.05415358142893273 :   0.08746836864712948 :   (b('l8b5') <= 3605.1396484375) ?  (b('ndvi') <= 4441.0) ?  0.12803090556635466 :   0.1376060428119516 :   (b('ndvi') <= 4459.0) ?  0.08868053959060207 :   0.022201310503370264 :   (b('l8b11') <= 1448.3333129882812) ?  (b('l8b11') <= 1365.820556640625) ?  (b('dg0_v_vv') <= 1.6166620254516602) ?  (b('moss') <= 15.002391815185547) ?  (b('crops') <= 85.25251388549805) ?  (b('l8b5_med') <= 3109.781005859375) ?  (b('dg0_v_vv') <= -0.44593095779418945) ?  -0.01078597960578729 :   0.004742510792302597 :   (b('ndvi') <= 3233.0) ?  -0.06092219010536741 :   -0.00843685724622953 :   0.05477490374693378 :   (b('ndvi') <= 3494.0) ?  (b('l8b5_med') <= 3104.2471923828125) ?  (b('ndvi_med') <= 3613.25) ?  0.0016818752411415828 :   -0.025683908712768905 :   (b('l8b11') <= 1255.9173583984375) ?  0.04482184054894986 :   0.03998608432823336 :   (b('dg0_v_vv') <= -0.9585909843444824) ?  0.052082338919007265 :   (b('ndvi') <= 3790.5) ?  0.05451404977817413 :   0.06371212813700346 :   (b('l8b11') <= 1323.4650268554688) ?  (b('ndvi') <= 4216.0) ?  (b('l8b4') <= 687.4669494628906) ?  (b('l8b4') <= 636.928466796875) ?  0.021580546540384767 :   0.09524223336809896 :   (b('l8b5') <= 3477.86572265625) ?  0.002477267519465689 :   -0.061662074294963704 :   (b('ndvi') <= 4853.0) ?  (b('l8b4') <= 514.6905364990234) ?  -0.04068872706159501 :   -0.01262881541384899 :   (b('l8b5_med') <= 2956.624755859375) ?  0.05047476408272686 :   -0.003884753730713684 :   (b('l8b4_med') <= 858.581787109375) ?  0.1069091269533056 :   0.093759411190208 :   (b('ndvi') <= 3415.0) ?  (b('l8b4') <= 976.3093872070312) ?  (b('l8b4_med') <= 940.2608337402344) ?  (b('l8b5') <= 2585.1951904296875) ?  (b('ndvi_med') <= 3450.25) ?  0.02064135294138113 :   0.04699604000618443 :   (b('dg0_v_vv') <= -0.2851109504699707) ?  -0.0027780100662252575 :   0.016068833092752848 :   (b('l8b4') <= 845.6395568847656) ?  (b('bulk') <= 142.0) ?  -0.0072890485831028085 :   -0.030351378638081822 :   (b('l8b4') <= 955.85107421875) ?  -0.05088363740946911 :   -0.04187654825226088 :   (b('grass') <= 26.79197883605957) ?  0.1435751842293735 :   0.03412799972233957 :   (b('ndvi') <= 3881.5) ?  (b('ndvi') <= 3668.5) ?  (b('dg0_v_vv') <= 2.4571549892425537) ?  (b('dg0_v_vv') <= -0.019999980926513672) ?  0.04546791894945816 :   0.01440989601301807 :   (b('moss') <= 8.03786039352417) ?  0.052413706047798436 :   0.04726486672722652 :   (b('l8b4') <= 756.4701538085938) ?  (b('crops') <= 39.189809799194336) ?  0.08345807342610627 :   0.02623258645760182 :   (b('dg0_v_vv') <= -0.419130802154541) ?  -0.007444924877964207 :   0.06813125965598546 :   (b('l8b5_med') <= 3014.6224365234375) ?  (b('l8b11') <= 1421.2685546875) ?  (b('dg0_v_vv') <= 0.9120192527770996) ?  0.010107672969043236 :   0.06092458202962123 :   (b('dg0_v_vv') <= 0.05994272232055664) ?  0.0006830794121207226 :   -0.025471349991401566 :   (b('dg0_v_vv') <= -0.2990899085998535) ?  0.086406978453027 :   (b('l8b4') <= 803.8678894042969) ?  0.0293967918594051 :   0.06744126967355296 :   (b('l8b5') <= 2274.6668701171875) ?  (b('l8b4') <= 1100.6493530273438) ?  (b('sand') <= 25.5) ?  0.11209021229778973 :   (b('moss') <= 13.726855278015137) ?  (b('l8b4_med') <= 732.3951110839844) ?  (b('ndvi') <= 3918.0) ?  0.022784763760350144 :   0.008247285457012976 :   (b('dg0_v_vv') <= 4.685951232910156) ?  0.004887644517438527 :   -0.04639365239933391 :   (b('ndvi') <= 3109.5) ?  (b('sand') <= 42.5) ?  -0.019901274765465715 :   -0.029355543837658454 :   -0.032260863041178936 :   (b('ndvi') <= 3122.5) ?  (b('l8b11_med') <= 1712.3707885742188) ?  (b('l8b4_med') <= 917.1773986816406) ?  (b('l8b5') <= 2227.2305908203125) ?  0.03743495211051526 :   0.03915332920000353 :   (b('dg0_v_vv') <= 0.10858964920043945) ?  0.04090930344617385 :   0.051012406456336756 :   (b('dg0_v_vv') <= 0.15320444107055664) ?  0.03456138133589853 :   0.03625131717679031 :   (b('dg0_v_vv') <= -0.15935325622558594) ?  0.0735453956259432 :   (b('l8b5') <= 2185.2996826171875) ?  0.06313901115411641 :   0.061170384300987146 :   (b('sand') <= 26.0) ?  (b('dg0_v_vv') <= -0.6310453414916992) ?  (b('l8b4') <= 712.7687683105469) ?  0.07622803838053668 :   (b('ndvi') <= 4406.0) ?  (b('l8b5') <= 2814.7371826171875) ?  -0.040966594467378534 :   0.006322419362576574 :   (b('l8b5_med') <= 3012.455322265625) ?  -0.057447502056718266 :   -0.03677112639576638 :   (b('l8b11_med') <= 1570.7933349609375) ?  (b('l8b4') <= 703.2799072265625) ?  0.01838472704498509 :   (b('ndvi') <= 3031.5) ?  0.08057304425823378 :   0.05002975061067905 :   (b('l8b5') <= 3488.5477294921875) ?  (b('dg0_v_vv') <= -0.1661081314086914) ?  -0.0021636706300811417 :   0.032216469231457254 :   (b('l8b5_med') <= 3036.0072021484375) ?  -0.013542606138777558 :   -0.04885136164302317 :   (b('l8b11') <= 1621.4673461914062) ?  (b('l8b5') <= 2933.578369140625) ?  (b('l8b11') <= 1592.6176147460938) ?  (b('l8b4_med') <= 942.4861145019531) ?  -0.029521820397095895 :   0.004587112843729137 :   (b('moss') <= 22.444746017456055) ?  -0.04236661959903857 :   -0.07191057674468357 :   (b('grass') <= 27.670451164245605) ?  (b('l8b4') <= 780.0580139160156) ?  -0.023619378658693336 :   0.011252199842916652 :   (b('dg0_v_vv') <= 1.0692944526672363) ?  -0.04200509466271912 :   0.003156534634849872 :   (b('l8b11') <= 1748.0862426757812) ?  (b('ndvi') <= 3164.5) ?  (b('ndvi') <= 3107.5) ?  0.01920375059081615 :   0.07231881646121334 :   (b('bulk') <= 132.0) ?  -0.024309544317548098 :   0.0024047226459953776 :   (b('l8b5') <= 3137.462890625) ?  (b('l8b5') <= 3118.9793701171875) ?  -0.005410186900581288 :   0.025871594976013698 :   (b('ndvi') <= 3714.5) ?  -0.01887456100721883 :   0.0019225313204864116 :   (b('l8b4_med') <= 729.5171508789062) ?  (b('dg0_v_vv') <= -0.12773942947387695) ?  (b('bulk') <= 130.0) ?  (b('l8b5') <= 2591.3096923828125) ?  (b('dg0_v_vv') <= -0.22228336334228516) ?  (b('l8b4') <= 886.5632629394531) ?  (b('l8b4') <= 788.6523742675781) ?  (b('l8b4') <= 636.1902465820312) ?  0.00929885451150093 :   0.023549826968187772 :   (b('l8b5') <= 2365.7987060546875) ?  -0.026848967145760037 :   -0.08123617917760867 :   (b('l8b11') <= 2104.8966064453125) ?  (b('l8b11') <= 1710.7086791992188) ?  0.0644584596908947 :   0.10205083046385621 :   0.006957588916602364 :   (b('l8b5') <= 2408.3494873046875) ?  (b('l8b4') <= 681.5120239257812) ?  (b('ndvi_med') <= 3586.25) ?  -0.018268598925927818 :   -0.0014265689624685107 :   (b('grass') <= 50.14073944091797) ?  -0.05664301847940681 :   -0.040813059117989325 :   (b('dg0_v_vv') <= -0.1690235137939453) ?  0.017762310943149284 :   0.03347397947957803 :   (b('ndvi_med') <= 4309.25) ?  (b('sand') <= 42.0) ?  (b('ndvi') <= 3535.0) ?  -0.021281199114787847 :   -0.017902569196619866 :   (b('l8b4') <= 544.4970397949219) ?  (b('l8b11') <= 1044.5388488769531) ?  0.016388249692954153 :   0.041983645615846686 :   (b('l8b5_med') <= 2592.1824951171875) ?  0.03864331758998458 :   0.025080178208139584 :   0.08911715353055635 :   (b('ndvi') <= 4392.5) ?  (b('ndvi') <= 3549.0) ?  (b('l8b11_med') <= 1388.9777221679688) ?  (b('l8b4') <= 653.58056640625) ?  (b('ndvi_med') <= 3336.0) ?  -0.013397840236646308 :   0.0288392406428799 :   (b('l8b5_med') <= 3090.990234375) ?  -0.010558651208041576 :   -0.049659789397047384 :   (b('ndvi') <= 3293.0) ?  (b('dg0_v_vv') <= -0.6774506568908691) ?  0.038658081648426955 :   -0.06664449092019284 :   (b('l8b4') <= 714.7623291015625) ?  0.11482782869345048 :   0.08305344868783243 :   (b('l8b4_med') <= 722.1411437988281) ?  (b('crops') <= 10.300542116165161) ?  (b('grass') <= 55.64550971984863) ?  0.012697186778894642 :   -0.021741075231585215 :   (b('l8b11') <= 1424.4198608398438) ?  0.02811633181331376 :   0.004741384140737048 :   (b('ndvi') <= 3911.0) ?  -0.08497037741523242 :   -0.07716053151041394 :   (b('dg0_v_vv') <= -1.1208972930908203) ?  (b('crops') <= 10.257567644119263) ?  -0.020237234695412665 :   -0.02753936161456877 :   (b('l8b4') <= 558.6485290527344) ?  0.039414210029201896 :   (b('bulk') <= 141.0) ?  0.067299726113011 :   (b('dg0_v_vv') <= -0.6779356002807617) ?  0.09116598642260071 :   0.07952574518723654 :   (b('ndvi') <= 1964.0) ?  (b('ndvi') <= 1878.5) ?  0.0985433192226669 :   0.12383127829313967 :   (b('l8b4') <= 702.8281860351562) ?  (b('l8b5_med') <= 2775.3719482421875) ?  (b('l8b11_med') <= 1286.7654418945312) ?  (b('bulk') <= 158.0) ?  (b('crops') <= 16.734968423843384) ?  0.013153892090260091 :   0.028873927788153356 :   0.09425511950812399 :   (b('ndvi') <= 3059.0) ?  -0.06087614261514965 :   (b('l8b11') <= 1116.4811401367188) ?  0.013781747426516056 :   -0.013082587946277585 :   (b('l8b4_med') <= 703.1324157714844) ?  (b('l8b5') <= 3506.5750732421875) ?  (b('ndvi') <= 4890.5) ?  0.03363897552122563 :   0.05914864881551894 :   (b('ndvi') <= 4853.0) ?  -0.004367188255779933 :   0.02334228188609781 :   (b('ndvi') <= 4077.5) ?  (b('ndvi') <= 3269.5) ?  0.08445194604534925 :   0.12152215162937577 :   (b('l8b4') <= 483.2058868408203) ?  0.0318047302773391 :   0.05922279530335117 :   (b('l8b11') <= 1640.8772583007812) ?  (b('ndvi') <= 4428.5) ?  (b('ndvi') <= 3498.0) ?  (b('ndvi') <= 2574.5) ?  0.05405421991209731 :   0.02875072607570573 :   (b('ndvi_med') <= 3534.25) ?  0.035750588302781 :   0.0756056752545888 :   (b('ndvi') <= 4598.0) ?  (b('l8b11') <= 1450.5381469726562) ?  -0.053591645403419264 :   -0.013381657666602043 :   (b('dg0_v_vv') <= 2.1213083267211914) ?  0.055025686039950776 :   -0.0459196247784075 :   (b('ndvi') <= 4560.0) ?  (b('ndvi') <= 3764.0) ?  (b('l8b5') <= 2379.657958984375) ?  -0.04785195388067273 :   0.019230538608375602 :   (b('ndvi') <= 3985.5) ?  0.062066040094822694 :   0.09225812104891845 :   (b('moss') <= 16.323646068572998) ?  (b('l8b5_med') <= 2937.431640625) ?  -0.10431720448711021 :   -0.06883120778477678 :   -0.007464444378765922 :   (b('grass') <= 51.157758712768555) ?  (b('l8b11') <= 1907.8300170898438) ?  (b('bulk') <= 152.0) ?  (b('l8b11') <= 1739.5468139648438) ?  (b('l8b11') <= 1504.287353515625) ?  (b('ndvi') <= 4162.0) ?  (b('dg0_v_vv') <= -0.11744403839111328) ?  -0.036738753415600574 :   -0.0007792370869484763 :   (b('ndvi') <= 4210.0) ?  0.10152341125442785 :   0.01557318643945448 :   (b('l8b11') <= 1563.2974243164062) ?  (b('dg0_v_vv') <= -1.643235206604004) ?  0.01876602939593955 :   0.07711640637020459 :   (b('dg0_v_vv') <= 0.6201796531677246) ?  0.030988092988725856 :   -0.04650211206967766 :   (b('l8b11') <= 1852.0308227539062) ?  (b('dg0_v_vv') <= -0.29472875595092773) ?  (b('dg0_v_vv') <= -1.052393913269043) ?  -0.042726652557265155 :   -0.026516316947664376 :   (b('crops') <= 35.19958972930908) ?  0.005450436213020438 :   -0.011870102160920992 :   -0.0867488107597564 :   (b('dg0_v_vv') <= -0.5180168151855469) ?  (b('l8b5') <= 2788.613525390625) ?  0.044943968957775 :   (b('l8b4') <= 643.0916442871094) ?  0.005959857122246948 :   (b('ndvi') <= 3602.5) ?  -0.03295898270308245 :   -0.025159673346072053 :   (b('l8b5') <= 2512.5262451171875) ?  0.12029022293228864 :   (b('l8b5') <= 4068.4437255859375) ?  (b('l8b11') <= 1855.3721313476562) ?  0.06306286323459176 :   0.039238509825971546 :   0.09231434823536694 :   (b('l8b5') <= 2531.3734130859375) ?  0.06519124076243157 :   (b('dg0_v_vv') <= -0.36136627197265625) ?  (b('l8b5_med') <= 3047.4815673828125) ?  0.09827290858007048 :   0.07313526915662821 :   (b('moss') <= 6.438619136810303) ?  0.10484736461456062 :   0.12589373110414404 :   (b('dg0_v_vv') <= 0.8344135284423828) ?  (b('l8b11') <= 2123.6156005859375) ?  (b('dg0_v_vv') <= 0.6999602317810059) ?  (b('dg0_v_vv') <= -1.600236415863037) ?  (b('l8b5_med') <= 2925.1749267578125) ?  (b('l8b5') <= 2710.6158447265625) ?  -0.031466982828715524 :   0.003824973917651042 :   (b('dg0_v_vv') <= -1.9304594993591309) ?  -0.07922023444370022 :   -0.03014296232442786 :   (b('l8b5') <= 3739.3780517578125) ?  (b('l8b11_med') <= 1601.371826171875) ?  -0.018414614151168695 :   0.008118715719156707 :   (b('l8b5') <= 3804.9630126953125) ?  -0.09535757425328699 :   -0.022439276514444797 :   (b('l8b5') <= 2943.1351318359375) ?  -0.07899573828629691 :   -0.09590565536350479 :   (b('l8b4_med') <= 829.5044555664062) ?  (b('l8b4') <= 1142.006103515625) ?  (b('ndvi_med') <= 3704.0) ?  (b('crops') <= 18.551711559295654) ?  0.09847066818000888 :   0.0792712553038055 :   0.053075753632126105 :   (b('dg0_v_vv') <= -1.558356761932373) ?  0.033492072435150566 :   0.04262944861322521 :   (b('l8b11_med') <= 1787.6991577148438) ?  0.0036325105037150196 :   -0.0265688776425822 :   (b('ndvi') <= 4969.0) ?  (b('l8b5') <= 3857.8671875) ?  (b('ndvi') <= 4681.0) ?  (b('dg0_v_vv') <= 1.0965237617492676) ?  (b('l8b4') <= 761.7809448242188) ?  0.12389744468387776 :   0.0338852496781006 :   (b('moss') <= 10.728179454803467) ?  0.011859212262226576 :   -0.07959803584221209 :   (b('dg0_v_vv') <= 2.063441753387451) ?  (b('l8b11') <= 1130.05859375) ?  -0.023590972908200225 :   0.052876347934401824 :   0.15563513686812874 :   -0.08874564228202764 :   -0.10619723393399133 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11_med') <= 1629.9594116210938) ?  (b('ndvi_med') <= 3209.0) ?  (b('l8b4') <= 534.2102966308594) ?  (b('bulk') <= 143.0) ?  (b('l8b5') <= 1192.3328247070312) ?  (b('l8b11_med') <= 1102.9674682617188) ?  (b('dg0_v_vv') <= -0.17096543312072754) ?  -0.015158132039943456 :   (b('ndvi') <= 1477.5) ?  -0.018967555393529845 :   (b('l8b11_med') <= 1052.5210571289062) ?  -0.023834677181666902 :   -0.026142457570864336 :   (b('l8b4') <= 475.3157653808594) ?  -0.04913334206944167 :   -0.04982454424309919 :   (b('sand') <= 54.5) ?  (b('l8b4') <= 439.96424865722656) ?  (b('ndvi') <= 3621.5) ?  (b('ndvi_med') <= 2020.5) ?  -0.02682934553966171 :   (b('ndvi_med') <= 2292.75) ?  0.07740787175095461 :   0.02299334845637292 :   (b('ndvi') <= 4206.0) ?  (b('ndvi') <= 3898.0) ?  0.049377035391073944 :   0.09749447605359313 :   (b('moss') <= 2.0) ?  0.052578166703824425 :   0.005562345772194194 :   (b('dg0_v_vv') <= 0.0586390495300293) ?  (b('ndvi_med') <= 3118.5) ?  (b('l8b11_med') <= 1304.4703979492188) ?  -0.019916035325347338 :   -0.005284080231446477 :   (b('l8b5_med') <= 2967.7503662109375) ?  0.015348745079570486 :   0.03191692255204029 :   (b('dg0_v_vv') <= 0.22594785690307617) ?  (b('ndvi') <= 2579.5) ?  0.021875003558994327 :   0.08279066958784011 :   (b('bulk') <= 135.0) ?  0.024059470032323376 :   0.003094195085961002 :   (b('dg0_v_vv') <= 2.4881479740142822) ?  (b('dg0_v_vv') <= -1.7051639556884766) ?  0.02641029548526508 :   (b('crops') <= 78.25687408447266) ?  (b('l8b11_med') <= 959.292724609375) ?  -0.01724757059752058 :   -0.033719261137024004 :   (b('l8b4_med') <= 574.1911315917969) ?  -0.01773690386315908 :   0.004051177828069297 :   (b('dg0_v_vv') <= 2.679196357727051) ?  (b('dg0_v_vv') <= 2.6133148670196533) ?  0.050859510969146426 :   0.04129906739317499 :   (b('l8b5') <= 3477.318359375) ?  0.03542388340912858 :   (b('dg0_v_vv') <= 3.135655641555786) ?  -0.010292071134300848 :   -0.026560015746986403 :   (b('l8b11') <= 933.3890380859375) ?  (b('ndvi') <= 4229.0) ?  (b('grass') <= 28.702491760253906) ?  0.10413915428838749 :   (b('l8b11') <= 857.2205200195312) ?  (b('dg0_v_vv') <= 0.5316352844238281) ?  (b('l8b5') <= 3023.0755615234375) ?  0.017749865913556245 :   -0.06277816722458937 :   (b('ndvi') <= 3738.5) ?  -0.030587426222578984 :   -0.09150848215862725 :   (b('l8b4_med') <= 986.0304260253906) ?  (b('l8b4') <= 443.0957794189453) ?  0.002963864130643738 :   0.024858780711131234 :   (b('ndvi') <= 2834.0) ?  0.10223784143030812 :   0.051385674851146645 :   (b('crops') <= 6.198555946350098) ?  (b('dg0_v_vv') <= 0.04111123085021973) ?  0.06251917412501432 :   0.08985848014814574 :   0.04747834150089508 :   (b('ndvi') <= 3505.5) ?  (b('l8b5') <= 2593.5477294921875) ?  (b('l8b4_med') <= 738.962646484375) ?  (b('l8b4') <= 523.948974609375) ?  (b('l8b11') <= 958.7750854492188) ?  0.03797338059447054 :   0.004382274389590949 :   (b('ndvi') <= 1286.0) ?  0.06168563788303405 :   0.04654031305399489 :   (b('l8b11_med') <= 1458.7056274414062) ?  (b('l8b4') <= 472.9878845214844) ?  0.0898890751411805 :   0.12310117551871333 :   0.034901172610006664 :   (b('l8b4') <= 460.0452880859375) ?  (b('dg0_v_vv') <= -0.677708625793457) ?  0.0651620849628094 :   (b('l8b4') <= 447.06190490722656) ?  0.0629160369748134 :   0.0034202191971421936 :   (b('l8b4') <= 514.3752899169922) ?  (b('l8b4') <= 462.3018035888672) ?  0.14074994585731898 :   0.08795849916200382 :   (b('sand') <= 28.5) ?  0.08075920060342184 :   0.036077690005525465 :   (b('grass') <= 55.46148681640625) ?  (b('ndvi') <= 3788.5) ?  (b('moss') <= 12.170807361602783) ?  (b('dg0_v_vv') <= 0.6012587547302246) ?  0.08795262292087563 :   0.10282847589291083 :   (b('dg0_v_vv') <= -0.10300207138061523) ?  0.07956787947182492 :   0.03119878864931824 :   (b('bulk') <= 157.0) ?  (b('dg0_v_vv') <= 0.18252325057983398) ?  0.026662613391967366 :   -0.02898589328840274 :   0.05035564385539701 :   (b('l8b11_med') <= 1423.35498046875) ?  (b('ndvi_med') <= 2140.75) ?  0.02528325918652241 :   0.03663511675588843 :   (b('l8b5') <= 2599.762939453125) ?  -0.06788599162276332 :   -0.14214847594087912 :   (b('sand') <= 27.5) ?  (b('ndvi') <= 4353.0) ?  (b('l8b5') <= 2475.6044921875) ?  (b('l8b5') <= 2474.261962890625) ?  (b('l8b4') <= 1022.1873779296875) ?  (b('dg0_v_vv') <= 1.1852636337280273) ?  (b('l8b4_med') <= 634.6933898925781) ?  0.05195624565179372 :   0.006473914016255177 :   (b('l8b11') <= 1568.3739013671875) ?  0.0736719869327502 :   0.02496753079937343 :   (b('l8b11_med') <= 1570.2359619140625) ?  (b('crops') <= 97.81589889526367) ?  0.05720394939181649 :   0.040225287179708025 :   0.09014204112616714 :   0.1451739303188535 :   (b('l8b5') <= 2586.1357421875) ?  (b('l8b4_med') <= 634.6933898925781) ?  0.027806985150224284 :   (b('ndvi') <= 3239.0) ?  (b('l8b11') <= 1989.2775268554688) ?  -0.056441200646881004 :   -0.07093952388138774 :   -0.033250441215406865 :   (b('l8b4') <= 1273.0355834960938) ?  (b('moss') <= 2.0) ?  (b('l8b11') <= 1193.3560791015625) ?  0.004061790667184688 :   0.05141205492408353 :   (b('l8b11') <= 1046.2805786132812) ?  -0.0581657242593793 :   0.0021363150063257673 :   (b('ndvi_med') <= 2729.0) ?  (b('ndvi') <= 3294.5) ?  0.15254263648827016 :   0.07016651252623582 :   (b('l8b5') <= 2766.5687255859375) ?  -0.015482213288837976 :   0.00600737383278351 :   (b('dg0_v_vv') <= -0.7575778961181641) ?  (b('moss') <= 14.804993152618408) ?  (b('moss') <= 6.599216938018799) ?  0.027662817893996383 :   -0.030591321128377258 :   -0.1152522641157887 :   (b('grass') <= 38.84922981262207) ?  (b('ndvi') <= 4558.5) ?  -0.003438363180426063 :   (b('l8b5') <= 4091.380126953125) ?  (b('l8b4_med') <= 866.570068359375) ?  0.023874972183394072 :   0.028899425180623356 :   0.006725719216308018 :   (b('ndvi') <= 4465.5) ?  (b('dg0_v_vv') <= 0.34613037109375) ?  (b('ndvi') <= 4425.0) ?  0.12907372167635212 :   0.12501604542688416 :   (b('l8b11_med') <= 1555.286865234375) ?  0.08895628155364133 :   0.11965428577224324 :   (b('l8b4_med') <= 634.6933898925781) ?  0.04446041821731739 :   (b('dg0_v_vv') <= 0.5500240325927734) ?  0.07417574832974196 :   0.10700200757589423 :   (b('l8b11') <= 1052.3379516601562) ?  (b('bulk') <= 132.0) ?  (b('l8b5') <= 1608.0519409179688) ?  (b('l8b4') <= 608.9571838378906) ?  -0.03812074535789363 :   -0.09019865392816759 :   (b('sand') <= 30.0) ?  -0.09368914718564747 :   (b('ndvi') <= 1670.5) ?  0.10575314151317775 :   (b('l8b11_med') <= 1353.8103637695312) ?  0.01632288352931983 :   0.08918102108193904 :   (b('l8b4_med') <= 595.4280090332031) ?  (b('ndvi') <= 3813.0) ?  (b('dg0_v_vv') <= -0.2910137176513672) ?  (b('dg0_v_vv') <= -0.6446409225463867) ?  0.08080907618498706 :   0.07315416115589396 :   (b('l8b4') <= 535.2744140625) ?  -0.007883315407087949 :   0.013353814867462246 :   -0.07916225091677132 :   (b('sand') <= 38.5) ?  (b('l8b4') <= 616.4539184570312) ?  (b('ndvi') <= 2873.0) ?  -0.015382677183561648 :   -0.03177557410213273 :   0.16620667191709196 :   (b('moss') <= 14.504502773284912) ?  (b('l8b11') <= 1032.7630615234375) ?  -0.041050287315479474 :   -0.008666907828154446 :   (b('l8b5') <= 2387.960693359375) ?  -0.07086073806731355 :   -0.03636110849604261 :   (b('l8b4') <= 574.0782470703125) ?  (b('moss') <= 20.18353843688965) ?  (b('l8b5') <= 3593.8751220703125) ?  (b('crops') <= 33.66267108917236) ?  (b('l8b5') <= 1666.0775756835938) ?  0.013144476906708225 :   -0.005652265963434695 :   (b('dg0_v_vv') <= 0.6765313148498535) ?  0.022626651210048976 :   0.043001193495857064 :   (b('l8b4_med') <= 748.6678771972656) ?  (b('dg0_v_vv') <= 1.7715480327606201) ?  0.039650608740934845 :   0.04908408387528898 :   (b('l8b5_med') <= 2866.581298828125) ?  0.03903062965443499 :   0.02711075971202237 :   (b('l8b4_med') <= 731.3841857910156) ?  (b('dg0_v_vv') <= 0.23597145080566406) ?  (b('dg0_v_vv') <= -0.19818973541259766) ?  0.06352479022531483 :   0.1137859889338394 :   (b('l8b11') <= 1152.4080200195312) ?  -0.030696861375914264 :   0.05071346089967818 :   (b('dg0_v_vv') <= -0.1825118064880371) ?  -0.005302729708860379 :   (b('dg0_v_vv') <= 0.21882104873657227) ?  0.01506110975486051 :   0.01354795768085243 :   (b('l8b4_med') <= 873.3128662109375) ?  (b('sand') <= 33.0) ?  (b('l8b11') <= 1183.4380493164062) ?  (b('dg0_v_vv') <= 0.8080384731292725) ?  0.02096322601958501 :   -0.018761195888650453 :   (b('ndvi') <= 3179.0) ?  -0.004940765359433337 :   0.006138934594483134 :   (b('ndvi_med') <= 1934.5) ?  (b('l8b4') <= 711.8874206542969) ?  0.018497884560527846 :   -0.0036900244603957754 :   (b('ndvi') <= 2421.0) ?  -0.01600310404217205 :   -0.005994061880707481 :   (b('l8b4') <= 1039.5153198242188) ?  (b('grass') <= 61.696245193481445) ?  (b('l8b4_med') <= 894.6989440917969) ?  0.0389156991300318 :   0.00043215105892988786 :   (b('l8b5') <= 1413.27197265625) ?  0.17004559887362092 :   0.046840876503309904 :   (b('grass') <= 42.84832191467285) ?  (b('l8b5') <= 1426.255859375) ?  -0.03407628118268902 :   0.022457701075689562 :   (b('l8b11') <= 1764.94091796875) ?  -0.030916209580964738 :   0.0021036596184174734 :   (b('dg0_v_vv') <= -0.8428945541381836) ?  (b('bulk') <= 129.5) ?  (b('moss') <= 13.82375431060791) ?  (b('ndvi') <= 3058.5) ?  (b('ndvi') <= 2040.0) ?  (b('l8b11') <= 1401.494140625) ?  (b('l8b4_med') <= 679.3181762695312) ?  0.030584855562284796 :   0.034985707963633955 :   (b('ndvi') <= 1915.0) ?  (b('dg0_v_vv') <= -4.921833515167236) ?  0.019408257606494256 :   0.003439681126346424 :   (b('dg0_v_vv') <= -1.9043316841125488) ?  -0.006005889287236355 :   -0.015876807528430393 :   (b('l8b4') <= 581.4467468261719) ?  (b('ndvi') <= 2889.0) ?  (b('l8b4_med') <= 634.5110168457031) ?  0.04450676370097009 :   0.06836424163699104 :   (b('ndvi_med') <= 3533.5) ?  0.006039023964497903 :   0.03234658867514781 :   (b('ndvi_med') <= 3474.5) ?  (b('sand') <= 65.5) ?  0.03433106396897266 :   0.05288447156143122 :   (b('l8b11') <= 1420.9696044921875) ?  -0.024419735083352573 :   0.01896560389132067 :   (b('crops') <= 0.44501277804374695) ?  (b('l8b4_med') <= 578.9661865234375) ?  (b('dg0_v_vv') <= -1.2193865776062012) ?  (b('ndvi_med') <= 3494.0) ?  -0.007873076470008789 :   0.05968027511823612 :   (b('dg0_v_vv') <= -0.8934478759765625) ?  -0.057040810983004564 :   0.08692902845903117 :   (b('l8b4') <= 582.3166809082031) ?  (b('ndvi') <= 4714.0) ?  -0.0038333901832505804 :   0.01515354657692991 :   (b('l8b4_med') <= 661.6227416992188) ?  -0.0746631925677898 :   -0.042818697095270926 :   (b('ndvi') <= 3259.5) ?  (b('ndvi') <= 3181.5) ?  (b('l8b5') <= 2563.687744140625) ?  0.023036863142316705 :   -0.0018610874471524559 :   (b('l8b4_med') <= 729.3392944335938) ?  -0.030484289337961722 :   -0.08261532081891942 :   (b('l8b5_med') <= 3613.2467041015625) ?  (b('dg0_v_vv') <= -1.450453758239746) ?  0.007859955876305722 :   0.026196597006131323 :   (b('ndvi') <= 3553.0) ?  0.08014929677774324 :   -0.011640423577924764 :   (b('ndvi') <= 2796.0) ?  (b('l8b11') <= 1877.8550415039062) ?  (b('dg0_v_vv') <= -1.9246549606323242) ?  0.042249902324509675 :   0.025370488326638324 :   (b('dg0_v_vv') <= -2.5715417861938477) ?  -0.032430728243602325 :   -0.016851024169563855 :   (b('l8b11') <= 1448.5728759765625) ?  0.014044773069745026 :   (b('dg0_v_vv') <= -1.439957618713379) ?  (b('dg0_v_vv') <= -3.11221981048584) ?  (b('dg0_v_vv') <= -3.387143135070801) ?  -0.03473410215350452 :   -0.031229168214410852 :   (b('dg0_v_vv') <= -1.5662493705749512) ?  -0.04653763906945463 :   -0.03743837943782924 :   (b('l8b5_med') <= 3257.319091796875) ?  (b('dg0_v_vv') <= -0.9304060935974121) ?  -0.031315717168689916 :   -0.03537548991234438 :   -0.0041492570182964095 :   (b('dg0_v_vv') <= -2.149505138397217) ?  (b('l8b4') <= 1890.5612182617188) ?  (b('dg0_v_vv') <= -3.4176764488220215) ?  (b('l8b4') <= 341.1834259033203) ?  0.031015090089441588 :   (b('dg0_v_vv') <= -4.403416156768799) ?  (b('l8b5') <= 2575.082763671875) ?  -0.05328831707268879 :   -0.07799493569888959 :   (b('l8b5_med') <= 3447.9315185546875) ?  -0.02583952851791373 :   -0.06006599870009729 :   (b('ndvi') <= 4517.5) ?  (b('moss') <= 11.87805700302124) ?  (b('ndvi') <= 4424.0) ?  -0.01723376372789866 :   -0.07004336021334634 :   (b('dg0_v_vv') <= -2.9357738494873047) ?  0.00496003504517778 :   0.03724315903072617 :   (b('ndvi_med') <= 4185.5) ?  (b('l8b4') <= 652.6153564453125) ?  -0.023073936795256225 :   0.00043981988891257477 :   (b('ndvi') <= 4650.0) ?  0.04469437721150285 :   0.0033325131459137625 :   0.09348853932771711 :   (b('ndvi') <= 1949.5) ?  (b('dg0_v_vv') <= -0.9920125007629395) ?  (b('dg0_v_vv') <= -2.0462746620178223) ?  -0.026266052490653696 :   (b('crops') <= 11.09115743637085) ?  (b('ndvi') <= 1756.5) ?  0.004787786573300451 :   0.016859561520581188 :   (b('l8b11_med') <= 1345.9015502929688) ?  0.07805371107333667 :   0.04451425713737912 :   (b('l8b5') <= 2456.8065795898438) ?  0.08598575232514227 :   0.09584346526114293 :   (b('sand') <= 27.5) ?  (b('ndvi_med') <= 3451.5) ?  (b('ndvi') <= 4773.0) ?  (b('ndvi') <= 3286.5) ?  0.005901166274947573 :   -0.028305512076550403 :   0.066064079111956 :   (b('ndvi_med') <= 3567.5) ?  (b('l8b5') <= 3059.8157958984375) ?  0.08002266713323077 :   0.027928870665407488 :   (b('crops') <= 11.156686067581177) ?  -0.0068163928896968675 :   0.01659131034710179 :   (b('ndvi') <= 4650.0) ?  (b('l8b11_med') <= 1121.5840454101562) ?  (b('dg0_v_vv') <= -1.483107566833496) ?  0.024415942069425822 :   -0.0016536411976740832 :   (b('grass') <= 28.106820106506348) ?  -0.004873012055725657 :   -0.02026193551940992 :   (b('l8b5_med') <= 2816.65380859375) ?  (b('ndvi_med') <= 3846.25) ?  -0.026251614694835158 :   -0.0017524548962834137 :   (b('ndvi_med') <= 4306.0) ?  0.03462059481264562 :   -0.036441634282487034 :   (b('grass') <= 45.82082176208496) ?  (b('crops') <= 35.10989189147949) ?  (b('dg0_v_vv') <= -0.7719550132751465) ?  (b('l8b11') <= 1392.667724609375) ?  (b('ndvi') <= 4689.5) ?  (b('l8b11') <= 1082.4345092773438) ?  (b('dg0_v_vv') <= -0.7843749523162842) ?  0.004861228941602273 :   0.05962807945898323 :   (b('l8b4_med') <= 692.8524169921875) ?  0.07212578332789159 :   0.04984848263391429 :   -0.018145711023400957 :   (b('dg0_v_vv') <= -0.8179383277893066) ?  -0.021808977446310426 :   -0.011609981784513293 :   (b('dg0_v_vv') <= -0.4138526916503906) ?  (b('l8b4') <= 520.1811218261719) ?  (b('moss') <= 10.874892711639404) ?  (b('l8b11_med') <= 1148.0034790039062) ?  0.032771074174288026 :   -0.00047922722246510063 :   (b('l8b4_med') <= 447.916259765625) ?  0.0005913456430383275 :   -0.02522825965859693 :   (b('l8b5_med') <= 3581.0770263671875) ?  (b('bulk') <= 127.5) ?  0.02517177564273175 :   -0.01735238009632671 :   (b('l8b11_med') <= 1238.279541015625) ?  -0.05797724294964853 :   -0.038662517864564316 :   (b('ndvi') <= 4246.0) ?  (b('l8b5') <= 1580.3441162109375) ?  -0.09885670471848368 :   (b('sand') <= 25.0) ?  0.037262301315191045 :   0.007906176873302794 :   (b('grass') <= 24.133636474609375) ?  (b('ndvi') <= 4906.0) ?  2.6892267382952672e-05 :   0.04172223212831882 :   (b('ndvi') <= 4289.5) ?  -0.0428232435039233 :   -0.00823264471370437 :   (b('sand') <= 18.0) ?  (b('l8b5') <= 4089.228271484375) ?  (b('l8b5') <= 3925.1773681640625) ?  (b('l8b5') <= 3569.522705078125) ?  (b('l8b11') <= 1765.124267578125) ?  0.09415121889627355 :   0.05263696666244981 :   (b('dg0_v_vv') <= -0.5471911430358887) ?  0.0186237202926684 :   0.034805416877957795 :   (b('l8b11') <= 1034.4730529785156) ?  0.10458916142149305 :   0.11239942156349553 :   (b('ndvi') <= 4402.0) ?  -0.02504888032730604 :   -0.03897712814464627 :   (b('l8b5') <= 2235.029296875) ?  (b('l8b11_med') <= 1309.96630859375) ?  (b('dg0_v_vv') <= -0.7169070243835449) ?  (b('l8b4_med') <= 778.4898681640625) ?  0.06094605666243991 :   0.08250470437894952 :   (b('dg0_v_vv') <= 0.2533435821533203) ?  -0.01582420674033281 :   0.014084227832756492 :   (b('l8b11_med') <= 1555.2052612304688) ?  (b('l8b11') <= 1649.7256469726562) ?  0.03874296548359742 :   0.015801780531685624 :   (b('crops') <= 53.20339584350586) ?  0.012300150290864287 :   -0.07280145642676353 :   (b('l8b4') <= 1645.605712890625) ?  (b('sand') <= 71.5) ?  (b('l8b11') <= 1205.1296997070312) ?  0.016985316820921168 :   0.006627160351778859 :   (b('l8b5') <= 3533.629638671875) ?  0.006890183766095525 :   -0.014128189710387838 :   (b('ndvi') <= 4173.0) ?  (b('ndvi_med') <= 3351.5) ?  0.07734988133903997 :   0.0859300694598335 :   (b('l8b5') <= 3176.1689453125) ?  0.041685442469367895 :   0.03779873200706957 :   (b('ndvi') <= 2964.5) ?  (b('ndvi') <= 2939.5) ?  (b('ndvi') <= 1998.5) ?  (b('ndvi') <= 1347.0) ?  0.007121901802506547 :   (b('l8b5') <= 2367.8419189453125) ?  (b('moss') <= 4.4147228598594666) ?  0.09071917198113719 :   0.06991297474484903 :   (b('l8b4') <= 1277.3194580078125) ?  0.0416254611997571 :   0.05513712575948654 :   (b('ndvi') <= 2123.0) ?  (b('dg0_v_vv') <= -0.2426128387451172) ?  0.010467319745317949 :   (b('l8b4_med') <= 667.341064453125) ?  -0.010001494197258709 :   -0.025828508698975405 :   (b('grass') <= 61.2738151550293) ?  (b('l8b4') <= 746.1508483886719) ?  0.032357166328747744 :   0.04659684701292221 :   (b('l8b5_med') <= 3200.444580078125) ?  0.025874549758234604 :   -0.01274120894423451 :   0.10871362857844585 :   (b('l8b4_med') <= 652.0127868652344) ?  (b('ndvi') <= 3112.5) ?  (b('l8b4_med') <= 574.9132690429688) ?  (b('l8b5') <= 2128.2236328125) ?  (b('dg0_v_vv') <= 1.2711820602416992) ?  -0.01608204929594796 :   -0.026504833538190137 :   (b('bulk') <= 118.5) ?  0.018101267436146717 :   0.00547706724420483 :   (b('dg0_v_vv') <= 0.8290748596191406) ?  -0.0825803525469003 :   -0.05478852835363468 :   (b('sand') <= 50.5) ?  (b('l8b11') <= 1169.213623046875) ?  (b('ndvi_med') <= 4166.5) ?  0.045357699368763466 :   -0.0297608028242074 :   (b('ndvi') <= 4088.5) ?  0.01322197414594076 :   -0.01720704225983156 :   (b('dg0_v_vv') <= 1.244431972503662) ?  (b('l8b4') <= 330.76019287109375) ?  0.03940814334818429 :   0.02032563025410503 :   (b('ndvi_med') <= 3560.75) ?  0.032290590363128445 :   0.0574182322508604 :   (b('l8b4') <= 993.9336853027344) ?  (b('ndvi') <= 4449.0) ?  (b('dg0_v_vv') <= 2.6066856384277344) ?  (b('ndvi') <= 4321.5) ?  -0.0025652376188112745 :   -0.05887767636439334 :   (b('sand') <= 16.5) ?  -0.04830316121477568 :   0.05956539815995271 :   (b('l8b11') <= 1726.14208984375) ?  (b('l8b11') <= 1642.4111938476562) ?  0.024902773009746217 :   -0.10084686192724758 :   0.17892535326697648 :   (b('ndvi') <= 3173.5) ?  (b('dg0_v_vv') <= 1.4250431060791016) ?  0.008893207363316918 :   0.019988563719753216 :   (b('moss') <= 10.910750389099121) ?  (b('l8b4') <= 1018.73193359375) ?  0.07878353267020938 :   0.058364191792313884 :   0.11330435799372962 :   (b('dg0_v_vv') <= 2.697053909301758) ?  (b('l8b11_med') <= 2631.7816162109375) ?  (b('crops') <= 61.748586654663086) ?  (b('l8b11_med') <= 2014.0931396484375) ?  (b('crops') <= 37.853599548339844) ?  (b('bulk') <= 158.0) ?  (b('bulk') <= 130.0) ?  (b('moss') <= 19.46947479248047) ?  (b('ndvi') <= 1322.0) ?  -0.03934962013589243 :   -0.013491041245308715 :   (b('l8b4') <= 830.3696899414062) ?  -0.024641945374219317 :   -0.05040757380645043 :   (b('l8b11') <= 1965.3640747070312) ?  (b('l8b4_med') <= 1002.3865661621094) ?  -0.008980753211064778 :   0.0011637750281505886 :   (b('ndvi') <= 4263.5) ?  -0.01464116625360511 :   0.03734186902855569 :   (b('ndvi_med') <= 2006.5) ?  (b('ndvi') <= 4414.5) ?  (b('ndvi') <= 4164.0) ?  -0.018627899661040073 :   -0.1132134755723412 :   0.07853049082466379 :   (b('l8b4') <= 723.67236328125) ?  (b('grass') <= 61.13550567626953) ?  -0.07418799551323531 :   -0.15321111875786692 :   (b('ndvi') <= 2137.5) ?  -0.06510909700976354 :   -0.02322268558424833 :   (b('crops') <= 41.514814376831055) ?  (b('l8b11') <= 2015.8541870117188) ?  (b('l8b11') <= 1738.9981689453125) ?  (b('l8b11') <= 1517.5471801757812) ?  0.020665119531178687 :   0.06745939913711853 :   (b('dg0_v_vv') <= -0.002353668212890625) ?  -0.0139260294638325 :   0.01343004678981396 :   (b('l8b4') <= 1073.7342834472656) ?  0.11141819171669828 :   (b('ndvi') <= 3586.5) ?  0.021584670755576196 :   0.059361790907174077 :   (b('l8b5') <= 3055.1976318359375) ?  (b('l8b5') <= 2560.824951171875) ?  (b('l8b5') <= 2378.5692138671875) ?  -0.0014449033789895036 :   0.01834911061262057 :   (b('l8b5') <= 2886.635986328125) ?  -0.008825630372533833 :   -0.025898044050275928 :   (b('l8b4_med') <= 887.6435241699219) ?  (b('l8b5') <= 3312.6405029296875) ?  -0.01551604168392826 :   -0.00027333938793813343 :   (b('l8b4') <= 621.2730712890625) ?  -0.026841937956325935 :   0.010011084866094472 :   (b('ndvi_med') <= 1877.5) ?  (b('ndvi') <= 1636.5) ?  (b('bulk') <= 146.5) ?  (b('ndvi_med') <= 949.0) ?  (b('moss') <= 38.41355323791504) ?  0.006538055505155142 :   0.061908717030488655 :   (b('l8b4_med') <= 1123.564697265625) ?  0.022044460562970915 :   -0.007086769175244134 :   (b('ndvi_med') <= 1833.5) ?  (b('ndvi') <= 1294.0) ?  -0.01665198710202739 :   -0.005948829636275174 :   (b('l8b5') <= 2631.0428466796875) ?  -0.056725818806347715 :   -0.01659853918203417 :   (b('l8b4_med') <= 1428.0526123046875) ?  (b('ndvi_med') <= 1311.0) ?  (b('ndvi') <= 1683.0) ?  0.11351090579729652 :   0.05516356638654096 :   (b('ndvi') <= 1693.0) ?  0.01368599499651608 :   -0.009539171085664586 :   (b('l8b4') <= 1177.7713012695312) ?  (b('l8b4') <= 583.1094360351562) ?  -0.049092680396889826 :   0.04451571207047307 :   (b('l8b4_med') <= 1715.7022094726562) ?  0.005437825597392677 :   0.06681684541662462 :   (b('ndvi') <= 1321.5) ?  (b('dg0_v_vv') <= -0.6453547477722168) ?  (b('dg0_v_vv') <= -2.5051236152648926) ?  (b('l8b11') <= 2548.4049072265625) ?  -0.014276274102782816 :   -0.06371721239738974 :   (b('l8b11') <= 2537.3004150390625) ?  0.014583843129366876 :   0.06275647684287523 :   (b('l8b4') <= 1766.3403930664062) ?  (b('l8b4') <= 1118.234619140625) ?  0.044137099915847446 :   0.08743692906384364 :   -0.003815811700533356 :   (b('dg0_v_vv') <= 0.0728311538696289) ?  (b('ndvi_med') <= 2244.0) ?  (b('ndvi') <= 1941.0) ?  -0.007183231100301054 :   0.020990643148538612 :   (b('dg0_v_vv') <= 0.06526374816894531) ?  -0.00630236675724605 :   -0.15349049175032808 :   (b('moss') <= 16.575261116027832) ?  (b('l8b5_med') <= 2420.321533203125) ?  0.0004860413361141154 :   0.01895509604785003 :   (b('moss') <= 18.696696281433105) ?  0.12360137347502072 :   0.020996473119771538 :   (b('l8b5_med') <= 3595.85302734375) ?  (b('l8b5_med') <= 2946.720703125) ?  (b('crops') <= 77.88256454467773) ?  (b('l8b11') <= 3051.7969970703125) ?  (b('l8b11') <= 3041.1407470703125) ?  (b('l8b11_med') <= 1915.0007934570312) ?  0.0009074873249727911 :   -0.017912338005161968 :   0.20005847419642875 :   (b('ndvi') <= 1363.5) ?  (b('l8b5') <= 2392.793701171875) ?  -0.06443309051214292 :   -0.00826911952374132 :   (b('l8b4') <= 1848.3402099609375) ?  -0.1007351942708411 :   -0.10325346019828788 :   (b('l8b5_med') <= 2871.976318359375) ?  (b('moss') <= 0.0533333346247673) ?  (b('ndvi') <= 2343.0) ?  0.05108498133535283 :   0.02162350203256324 :   (b('dg0_v_vv') <= -0.02345895767211914) ?  0.006734059977797169 :   0.01961739100977425 :   (b('l8b11') <= 1114.3903198242188) ?  (b('ndvi') <= 2127.5) ?  -0.027097990369646403 :   0.033751224954365944 :   (b('l8b5') <= 2514.9949951171875) ?  -0.01315068319228301 :   0.001226979087734866 :   (b('l8b5') <= 2726.69921875) ?  (b('l8b4') <= 2024.0022583007812) ?  (b('ndvi') <= 1287.0) ?  (b('l8b5_med') <= 3378.561279296875) ?  -0.0017745224194998328 :   0.06279315808647937 :   (b('l8b5') <= 2666.98193359375) ?  0.027214169907405317 :   0.07002626099463037 :   (b('ndvi_med') <= 1751.0) ?  0.1421647690975204 :   0.13860680808133785 :   (b('grass') <= 14.618155479431152) ?  (b('l8b5') <= 2745.566162109375) ?  (b('l8b5_med') <= 3025.82421875) ?  -0.05932010045999429 :   -0.04382932113712091 :   (b('l8b4') <= 2307.929443359375) ?  0.0008254797041136784 :   0.01912947050071476 :   (b('ndvi') <= 2677.0) ?  (b('l8b11') <= 1662.4651489257812) ?  0.07252407038177841 :   0.002042746840057117 :   (b('l8b5') <= 3816.8292236328125) ?  0.05103324977393491 :   0.005123969982691639 :   (b('l8b11') <= 1623.68017578125) ?  (b('l8b11_med') <= 2518.2154541015625) ?  (b('ndvi') <= 3206.5) ?  (b('l8b5') <= 3284.79736328125) ?  (b('ndvi') <= 1525.5) ?  -0.013882754756356317 :   0.010870293266971282 :   0.0783222003400122 :   (b('ndvi') <= 4178.5) ?  -0.036804933971265996 :   (b('l8b5_med') <= 3694.775634765625) ?  -0.02070655672041355 :   -0.02211870844230937 :   (b('l8b11') <= 1087.1801452636719) ?  0.053685433435165325 :   (b('ndvi') <= 1525.5) ?  0.04054161868591097 :   (b('l8b4') <= 1019.1031494140625) ?  0.03419507580389575 :   0.0316449291309695 :   (b('l8b11') <= 2436.2935791015625) ?  (b('l8b4') <= 1104.8088989257812) ?  (b('dg0_v_vv') <= -0.08815908432006836) ?  -0.06201862693590085 :   -0.04794661126322097 :   (b('l8b11') <= 1670.64892578125) ?  (b('l8b5_med') <= 3694.775634765625) ?  -0.018297624879026985 :   -0.09531679273497692 :   (b('l8b11') <= 2155.2484130859375) ?  -0.013875833332939432 :   -0.025161747202670344 :   (b('dg0_v_vv') <= -1.7073373794555664) ?  (b('l8b4') <= 2003.2720336914062) ?  (b('dg0_v_vv') <= -3.7371668815612793) ?  0.00537533373018433 :   -0.013405900766454851 :   (b('l8b11') <= 3667.5931396484375) ?  -0.028357085291707034 :   -0.010633561907551 :   (b('ndvi') <= 1483.0) ?  (b('dg0_v_vv') <= 0.38545703887939453) ?  -0.027461821057354965 :   -0.05374799763420227 :   (b('l8b5') <= 3757.8028564453125) ?  0.01833806982177125 :   -0.016576903538548633 :   (b('grass') <= 1.0004997551441193) ?  (b('l8b11') <= 3861.611328125) ?  (b('l8b4') <= 972.0349731445312) ?  (b('ndvi') <= 3871.0) ?  (b('ndvi') <= 2019.0) ?  (b('l8b5') <= 2438.5631713867188) ?  (b('ndvi') <= 1127.0) ?  0.028590897784187514 :   0.019349199746253196 :   (b('dg0_v_vv') <= -0.4813389778137207) ?  0.0032563811633744277 :   -0.02135910331511759 :   (b('dg0_v_vv') <= -0.048386573791503906) ?  (b('l8b11') <= 1244.9990844726562) ?  0.030969706183353013 :   0.013040464576339941 :   0.061235048501308065 :   (b('dg0_v_vv') <= -2.1837453842163086) ?  -0.018118750907784337 :   -0.014223113123631115 :   (b('dg0_v_vv') <= -1.893341064453125) ?  (b('l8b4') <= 2115.582763671875) ?  (b('l8b5') <= 3600.725341796875) ?  (b('dg0_v_vv') <= -2.1785788536071777) ?  -0.05390937133380208 :   -0.051980290598029055 :   (b('dg0_v_vv') <= -3.3057355880737305) ?  -0.03413576876328728 :   -0.029037684453504936 :   (b('l8b4') <= 2245.55517578125) ?  (b('dg0_v_vv') <= -1.967085361480713) ?  0.022961366320840434 :   -0.013038632129438801 :   (b('ndvi') <= 1595.0) ?  -0.00645689445866796 :   -0.025568727139735577 :   (b('ndvi') <= 1738.0) ?  (b('l8b4') <= 1738.8670654296875) ?  (b('l8b11') <= 2895.7447509765625) ?  0.011240834471766317 :   -0.004907834376046737 :   (b('dg0_v_vv') <= 2.090515613555908) ?  -0.005997736994831202 :   0.032844426710805244 :   (b('l8b4') <= 2370.552978515625) ?  (b('l8b4') <= 2189.6463623046875) ?  0.0025252748254085008 :   -0.017864636135413756 :   (b('dg0_v_vv') <= -0.7756118774414062) ?  0.06558008763502252 :   0.036004454920142044 :   (b('grass') <= 0.27586206793785095) ?  (b('l8b4') <= 3168.4632568359375) ?  (b('dg0_v_vv') <= -0.22273826599121094) ?  (b('l8b11') <= 4086.1729736328125) ?  (b('l8b11_med') <= 3904.9847412109375) ?  0.006275571598231278 :   -0.0010242986144633898 :   0.014200160634786366 :   (b('l8b11') <= 3931.603271484375) ?  -0.008153487394663789 :   (b('dg0_v_vv') <= -0.04386758804321289) ?  -0.025346982699800055 :   -0.014524397445316245 :   (b('dg0_v_vv') <= 0.3978705406188965) ?  (b('l8b5') <= 4589.911376953125) ?  (b('l8b4') <= 3560.1611328125) ?  -0.016223431408303245 :   -0.022065291167977563 :   (b('l8b5_med') <= 4239.44873046875) ?  -0.015826708994684788 :   0.005499993418867248 :   (b('dg0_v_vv') <= 0.6828408241271973) ?  (b('ndvi_med') <= 902.5) ?  0.011308224858131294 :   0.00809733733344914 :   (b('l8b4') <= 3336.5537109375) ?  -0.016041347293466846 :   -0.0065264856407565575 :   (b('ndvi_med') <= 1349.5) ?  (b('ndvi') <= 1310.0) ?  -0.03298052920782801 :   (b('dg0_v_vv') <= 0.6331639289855957) ?  -0.027936674903407926 :   -0.024535996268811713 :   (b('dg0_v_vv') <= 0.5772318840026855) ?  (b('dg0_v_vv') <= 0.12990522384643555) ?  (b('dg0_v_vv') <= -0.28595495223999023) ?  -0.024444488360494433 :   -0.019688909826251433 :   -0.02579649451225001 :   -0.006442376100164279 :   (b('grass') <= 5.454718589782715) ?  (b('ndvi') <= 3200.0) ?  (b('l8b4') <= 951.7755126953125) ?  (b('dg0_v_vv') <= 1.5655879974365234) ?  (b('dg0_v_vv') <= 0.6209249496459961) ?  (b('dg0_v_vv') <= -0.716306209564209) ?  -0.04732800487337546 :   -0.06464062423204053 :   (b('l8b5') <= 2229.5521240234375) ?  -0.019247820268133382 :   0.004418156304890064 :   (b('l8b5_med') <= 3219.7747802734375) ?  -0.01807336650286187 :   (b('l8b5') <= 3297.9998779296875) ?  -0.07409087724688837 :   -0.06465339489995202 :   (b('l8b5') <= 2573.7420654296875) ?  (b('l8b11') <= 2667.6767578125) ?  (b('l8b5') <= 2303.7681884765625) ?  -0.002514562646657266 :   -0.03199088925921143 :   (b('dg0_v_vv') <= -0.23767328262329102) ?  -0.023839801508446553 :   0.10181177660021949 :   (b('ndvi_med') <= 1678.75) ?  (b('l8b4') <= 1405.8248901367188) ?  -0.025086667582836272 :   -0.013008014573292608 :   (b('l8b11') <= 1955.3237915039062) ?  0.062410266899356054 :   -0.033906750675868264 :   (b('l8b5') <= 4027.55126953125) ?  (b('l8b4') <= 1644.0587768554688) ?  (b('dg0_v_vv') <= 0.5087141990661621) ?  (b('l8b5_med') <= 3217.6090087890625) ?  -0.027494392278447785 :   -0.01710989345983504 :   -0.029346675718118644 :   (b('l8b5') <= 3535.5489501953125) ?  (b('dg0_v_vv') <= 1.2358675003051758) ?  -0.00844522740662651 :   0.018893980573070145 :   (b('dg0_v_vv') <= 1.2811756134033203) ?  0.01771000051568723 :   0.04736344413718138 :   (b('dg0_v_vv') <= 1.0977654457092285) ?  (b('ndvi_med') <= 1482.0) ?  0.040934153425057185 :   0.03659095040641823 :   0.055549213819188314 :   (b('sand') <= 47.5) ?  (b('l8b5') <= 3207.7059326171875) ?  (b('l8b5_med') <= 3248.876953125) ?  (b('dg0_v_vv') <= 2.310182571411133) ?  (b('dg0_v_vv') <= -2.416220188140869) ?  -0.024894735027193974 :   -0.0009701662374071957 :   (b('l8b11') <= 3048.8468017578125) ?  0.047508000375154985 :   0.020649533265308058 :   (b('ndvi_med') <= 902.5) ?  0.008113181721114818 :   0.14439776177479816 :   (b('l8b4_med') <= 1849.1400146484375) ?  (b('ndvi') <= 1302.5) ?  (b('dg0_v_vv') <= -2.4417200088500977) ?  0.02385289964437723 :   0.06268007027970074 :   (b('dg0_v_vv') <= -0.856687068939209) ?  -0.008718921433033128 :   0.029304367232072387 :   (b('l8b11') <= 3447.712890625) ?  (b('l8b4') <= 1470.77734375) ?  -0.04347928236320917 :   -0.011115067910359135 :   (b('dg0_v_vv') <= 0.5315465927124023) ?  -0.000422028787660182 :   0.055766446692160576 :   (b('ndvi') <= 621.5) ?  (b('l8b11_med') <= 3131.10302734375) ?  (b('l8b11_med') <= 3091.89697265625) ?  (b('l8b11_med') <= 2736.977783203125) ?  0.0017931807565051555 :   -0.010532513737607946 :   0.006689823693831018 :   (b('l8b11') <= 3142.96337890625) ?  (b('grass') <= 13.273648262023926) ?  0.011929093301401267 :   0.032867295210041336 :   (b('ndvi_med') <= 842.0) ?  0.05825884460662764 :   0.05700990399073391 :   (b('bulk') <= 149.5) ?  (b('l8b4_med') <= 1813.3069458007812) ?  (b('l8b4') <= 1750.7471923828125) ?  -0.013085445233163632 :   0.0356266478847879 :   (b('l8b11') <= 2454.1756591796875) ?  0.02807797560652459 :   -0.012470013844304132 :   (b('ndvi') <= 1769.0) ?  (b('ndvi') <= 1727.5) ?  -0.009573479533108845 :   0.010102233880865558 :   (b('sand') <= 59.0) ?  -0.013386385837253448 :   0.006172603675108133 :   (b('bulk') <= 160.0) ?  (b('l8b5') <= 1947.26806640625) ?  (b('ndvi_med') <= 2176.5) ?  (b('l8b5_med') <= 3052.3477783203125) ?  (b('dg0_v_vv') <= 2.7357020378112793) ?  -0.047355066020702385 :   (b('l8b5') <= 1904.69873046875) ?  (b('grass') <= 76.0) ?  (b('dg0_v_vv') <= 5.8818440437316895) ?  0.027087580337560493 :   0.002197668142492467 :   (b('l8b5_med') <= 2222.685791015625) ?  0.0051761207292560975 :   -0.0889877459826765 :   (b('grass') <= 88.35786056518555) ?  (b('l8b11_med') <= 2417.3900146484375) ?  0.021121541384159715 :   0.04991663437031374 :   (b('l8b11') <= 1992.4714965820312) ?  0.08672253579038791 :   0.07566810825510367 :   (b('l8b11') <= 891.2627563476562) ?  -0.045342914182677996 :   -0.03766118638987931 :   (b('l8b11_med') <= 1871.8484497070312) ?  (b('l8b11_med') <= 1695.4384155273438) ?  (b('l8b4') <= 706.1152038574219) ?  0.009490196050950805 :   (b('grass') <= 46.819149017333984) ?  (b('dg0_v_vv') <= 3.493659257888794) ?  0.04685625278398928 :   0.03203155991346262 :   (b('sand') <= 38.5) ?  0.04821723531578054 :   0.04536396332497389 :   (b('l8b4') <= 647.4686584472656) ?  (b('dg0_v_vv') <= 4.477033615112305) ?  0.019538106118232923 :   (b('dg0_v_vv') <= 4.626684904098511) ?  0.027409201086479762 :   0.02194994758938253 :   (b('l8b5') <= 1905.6697387695312) ?  (b('dg0_v_vv') <= 3.335498332977295) ?  0.016378133298229805 :   0.005926683049358505 :   -0.005753005308422349 :   (b('l8b11_med') <= 1995.7621459960938) ?  (b('l8b4_med') <= 988.1557006835938) ?  0.080191023894021 :   (b('ndvi_med') <= 2707.75) ?  (b('crops') <= 38.45608901977539) ?  0.13691229830741364 :   0.13588157501074016 :   0.12292536737619089 :   (b('sand') <= 43.0) ?  (b('l8b4_med') <= 1161.8717651367188) ?  (b('dg0_v_vv') <= 4.077102184295654) ?  0.042645260422770614 :   0.04459585492248358 :   0.03903832123940304 :   (b('l8b5_med') <= 2229.0753173828125) ?  0.02172339003960863 :   (b('l8b5_med') <= 2400.9317626953125) ?  0.0077738608616380045 :   0.006748049440726989 :   (b('l8b5') <= 1981.6925659179688) ?  (b('l8b5') <= 1957.2979125976562) ?  (b('bulk') <= 140.0) ?  (b('l8b11') <= 1867.2864990234375) ?  -0.004096269612637288 :   -0.019966630831438237 :   (b('sand') <= 55.5) ?  (b('l8b5_med') <= 2416.1658935546875) ?  -0.05235005937526868 :   -0.0481149461473403 :   (b('dg0_v_vv') <= 2.969271183013916) ?  -0.0442185301207112 :   -0.034637883336487374 :   (b('l8b5_med') <= 2716.2015380859375) ?  (b('sand') <= 42.0) ?  -0.09241885125541269 :   -0.09609959033334894 :   -0.07260685025281202 :   (b('l8b5_med') <= 3110.163330078125) ?  (b('l8b4_med') <= 1154.9951171875) ?  (b('grass') <= 76.29829025268555) ?  (b('l8b11_med') <= 1726.2890625) ?  (b('ndvi') <= 3367.0) ?  0.030633960624409363 :   -0.0036155253457716874 :   (b('dg0_v_vv') <= 2.817608118057251) ?  0.11951486579990658 :   -0.006303552021419011 :   (b('l8b11_med') <= 1844.1785888671875) ?  (b('l8b5_med') <= 2791.9354248046875) ?  -0.07418695324066575 :   -0.009750175594373425 :   (b('ndvi') <= 1840.0) ?  -0.05774006170225045 :   0.000864430966569984 :   (b('grass') <= 90.7700080871582) ?  (b('l8b4') <= 1512.2576904296875) ?  (b('l8b4') <= 1332.5263671875) ?  0.014542808634903247 :   0.04983243344745078 :   (b('bulk') <= 153.5) ?  0.01033487001582493 :   -0.06449840269259922 :   (b('bulk') <= 153.0) ?  (b('l8b11_med') <= 2479.950439453125) ?  0.056954313490666826 :   0.011524926861679496 :   (b('dg0_v_vv') <= 3.4038796424865723) ?  -0.10839469636757251 :   0.09408664632143696 :   (b('l8b5') <= 2022.0379638671875) ?  (b('ndvi') <= 2706.5) ?  (b('l8b11_med') <= 2420.3597412109375) ?  0.06032414016282206 :   (b('l8b4_med') <= 1985.1664428710938) ?  0.06688018448211713 :   0.07065458585544038 :   0.03205798022035117 :   (b('dg0_v_vv') <= 2.709909439086914) ?  (b('l8b5') <= 3593.82421875) ?  0.04982255489090211 :   0.056521671552943076 :   (b('l8b5') <= 2793.32958984375) ?  (b('l8b11_med') <= 2688.4453125) ?  0.028858943727513433 :   -0.0007258680290704418 :   (b('ndvi') <= 1952.5) ?  0.0022505478425772015 :   -0.012706643410382105 :   (b('l8b4_med') <= 1404.4251708984375) ?  (b('dg0_v_vv') <= 3.0869154930114746) ?  (b('l8b5_med') <= 2473.45751953125) ?  0.018927549508110708 :   (b('grass') <= 24.068785667419434) ?  0.06887165350212082 :   (b('l8b4') <= 769.3148651123047) ?  0.059065239970909955 :   0.06000422816079043 :   (b('l8b5_med') <= 2390.825439453125) ?  (b('dg0_v_vv') <= 4.025285243988037) ?  0.0808517410989053 :   0.06980924070695604 :   (b('dg0_v_vv') <= 4.733349561691284) ?  (b('dg0_v_vv') <= 3.4764928817749023) ?  (b('dg0_v_vv') <= 3.2344446182250977) ?  0.1076789781618791 :   0.10369579433133158 :   (b('l8b11_med') <= 2258.0072021484375) ?  0.09807644023206108 :   0.09975907849488444 :   0.08132949709570753 :   (b('bulk') <= 166.5) ?  (b('l8b5') <= 2897.2813720703125) ?  (b('l8b11') <= 2294.4874267578125) ?  (b('bulk') <= 161.5) ?  0.01808451252279475 :   0.019582440909533427 :   0.015932528614565705 :   0.0032661757420106574 :   (b('dg0_v_vv') <= 5.578905820846558) ?  -0.05326410464238718 :   -0.05058017469230336 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 819.9094543457031) ?  (b('grass') <= 51.771291732788086) ?  (b('sand') <= 27.5) ?  (b('grass') <= 49.12318229675293) ?  (b('l8b4_med') <= 786.5292053222656) ?  (b('l8b5') <= 4275.2353515625) ?  (b('l8b5') <= 2558.435546875) ?  (b('l8b4') <= 601.0503540039062) ?  (b('l8b4_med') <= 778.4898681640625) ?  (b('ndvi') <= 3464.0) ?  0.0021305731254495364 :   0.058080427896550914 :   (b('dg0_v_vv') <= 1.299607276916504) ?  -0.001485186190787734 :   0.05653816518681193 :   (b('ndvi') <= 2226.5) ?  (b('l8b11_med') <= 1335.7188110351562) ?  0.057725407701236964 :   0.0043413883913102636 :   (b('ndvi') <= 2874.0) ?  -0.05176225182102158 :   -0.017500917143819503 :   (b('l8b4') <= 812.7628173828125) ?  (b('l8b5_med') <= 3452.3497314453125) ?  (b('dg0_v_vv') <= -1.0933523178100586) ?  0.008973692398032942 :   0.03706141804548827 :   (b('l8b5') <= 3952.099853515625) ?  0.00794530777138244 :   0.03545690977101926 :   (b('l8b11_med') <= 1570.7933349609375) ?  -0.07533372068619756 :   -0.03475314479089994 :   (b('bulk') <= 145.0) ?  (b('dg0_v_vv') <= 0.01200723648071289) ?  (b('l8b4_med') <= 719.6269226074219) ?  (b('dg0_v_vv') <= -0.43322277069091797) ?  -0.034796266078107874 :   -0.005869649011642497 :   0.028665963586681054 :   (b('dg0_v_vv') <= 0.8264861106872559) ?  (b('ndvi') <= 4178.5) ?  0.0540970758915815 :   0.0608569693899212 :   (b('sand') <= 16.0) ?  -0.025638143542364183 :   0.020574085195828096 :   (b('l8b4') <= 578.2671813964844) ?  (b('dg0_v_vv') <= 1.058065414428711) ?  (b('l8b11') <= 1061.3877563476562) ?  -0.046659105694826196 :   -0.006168494747255981 :   0.03893652244950965 :   (b('l8b5') <= 4557.134765625) ?  -0.06587848786630313 :   -0.08868742337097332 :   (b('dg0_v_vv') <= 0.004161834716796875) ?  (b('l8b4') <= 817.7661743164062) ?  (b('l8b4_med') <= 789.5814514160156) ?  (b('l8b5') <= 4333.28369140625) ?  -0.058112534430235424 :   -0.09267155729390897 :   (b('dg0_v_vv') <= -1.4334797859191895) ?  (b('l8b11_med') <= 1666.9906616210938) ?  -0.027001493964687692 :   0.004284628259599868 :   (b('dg0_v_vv') <= -0.8822212219238281) ?  0.017535217571621256 :   -0.009191528873187195 :   0.08498074728785288 :   (b('l8b4') <= 811.2275695800781) ?  (b('l8b4') <= 305.94541931152344) ?  (b('dg0_v_vv') <= 1.649193286895752) ?  (b('l8b11_med') <= 1295.4417114257812) ?  0.06727940907323901 :   0.0845966010276061 :   (b('l8b5') <= 4057.442626953125) ?  0.032967530323504746 :   0.039281879659041785 :   (b('moss') <= 5.419059991836548) ?  (b('ndvi') <= 4609.5) ?  0.008501823710361156 :   -0.008594210776501314 :   (b('ndvi') <= 3448.5) ?  0.09342752989214173 :   0.02705645929934668 :   (b('dg0_v_vv') <= 0.7839984893798828) ?  0.16697499728740658 :   0.10391469493825828 :   (b('ndvi') <= 4441.0) ?  (b('l8b5') <= 2447.4947509765625) ?  0.10851772440094087 :   (b('l8b11') <= 1211.47509765625) ?  0.0926151520072977 :   (b('dg0_v_vv') <= -0.5229415893554688) ?  (b('l8b11') <= 1431.5512084960938) ?  0.04397852149079573 :   0.03244730534867443 :   (b('l8b4') <= 721.7745361328125) ?  0.07633632409547769 :   (b('ndvi') <= 4351.5) ?  0.05232743754344362 :   0.06749840215353342 :   (b('l8b5') <= 4169.9561767578125) ?  (b('l8b5') <= 3388.22802734375) ?  0.02632528253952282 :   (b('l8b4') <= 629.9983215332031) ?  -0.020591235062426938 :   -0.025133983312439462 :   0.08059263611085565 :   (b('grass') <= 12.788208484649658) ?  (b('l8b4') <= 794.4911193847656) ?  (b('ndvi') <= 2601.5) ?  (b('l8b11_med') <= 2089.3021240234375) ?  (b('ndvi') <= 2280.0) ?  (b('ndvi') <= 2103.5) ?  (b('sand') <= 71.0) ?  0.027681726615591193 :   0.005943182669763208 :   (b('ndvi_med') <= 2507.0) ?  0.009992926786984727 :   -0.024822889630479034 :   (b('l8b11') <= 906.6918334960938) ?  0.14296902960246244 :   (b('l8b4') <= 706.6593322753906) ?  0.04626483206088095 :   0.014583329781528289 :   (b('dg0_v_vv') <= -0.8541908264160156) ?  0.035012140949900766 :   (b('l8b4') <= 509.3787841796875) ?  (b('bulk') <= 155.5) ?  -0.08197306092620021 :   -0.05117570226283144 :   (b('l8b4') <= 693.0968017578125) ?  -0.014185947954212949 :   -0.04833199075772879 :   (b('ndvi_med') <= 3732.25) ?  (b('crops') <= 78.23976135253906) ?  (b('ndvi') <= 4768.5) ?  (b('l8b4') <= 785.0750122070312) ?  -0.01375454697046824 :   0.022614986447792734 :   (b('l8b5_med') <= 2792.08935546875) ?  0.0805318674650283 :   0.036686933081518575 :   (b('l8b4') <= 417.435791015625) ?  (b('dg0_v_vv') <= 1.1427338123321533) ?  0.005269529570167509 :   -0.014570427228333085 :   (b('ndvi_med') <= 2506.0) ?  0.030824746872903375 :   0.008903302973978575 :   (b('l8b5') <= 3516.382568359375) ?  (b('l8b5_med') <= 3413.9039306640625) ?  (b('ndvi') <= 3672.5) ?  0.031012864323469565 :   0.004036894778703244 :   (b('l8b11') <= 1513.6878662109375) ?  0.022105685806157606 :   0.12956269944769144 :   (b('ndvi') <= 3223.5) ?  -0.0395634092817147 :   (b('l8b5') <= 3812.796630859375) ?  0.04946819338352635 :   0.03110388227498241 :   (b('dg0_v_vv') <= -2.0048155784606934) ?  (b('l8b5_med') <= 3413.9039306640625) ?  0.10281806785867287 :   0.09487886934980233 :   (b('dg0_v_vv') <= 1.7163360118865967) ?  (b('l8b4') <= 808.1480407714844) ?  (b('dg0_v_vv') <= -1.2602524757385254) ?  (b('ndvi_med') <= 4697.0) ?  0.009972662860856796 :   0.021728198585407366 :   (b('l8b11_med') <= 1163.6696166992188) ?  0.09091724615970503 :   0.039838165179192656 :   (b('ndvi') <= 4738.5) ?  (b('ndvi') <= 4602.0) ?  0.013333566369994387 :   0.07189050760505214 :   (b('l8b5_med') <= 2677.065673828125) ?  -0.008785135146345158 :   -0.014528389743034609 :   (b('l8b5_med') <= 3490.31982421875) ?  (b('l8b4') <= 807.3067321777344) ?  0.06789286309887638 :   0.08141229035866279 :   0.051471952905872975 :   (b('bulk') <= 161.5) ?  (b('l8b4_med') <= 719.3065185546875) ?  (b('l8b11') <= 1340.8045043945312) ?  (b('l8b11') <= 1249.0657348632812) ?  (b('grass') <= 48.91348075866699) ?  (b('l8b11') <= 1130.8341674804688) ?  0.005512550476866352 :   -0.003382738485799257 :   (b('l8b11_med') <= 1209.6671752929688) ?  0.024080784414515975 :   -0.04441600443076771 :   (b('crops') <= 49.02328300476074) ?  (b('l8b4') <= 544.6319580078125) ?  0.008329669306144944 :   -0.008580826679654175 :   (b('l8b5') <= 2695.4522705078125) ?  -0.00631199038762921 :   -0.049597023849900064 :   (b('ndvi') <= 2464.0) ?  (b('dg0_v_vv') <= -0.7901444435119629) ?  (b('sand') <= 40.0) ?  -0.0021615187021071683 :   0.08173371754414543 :   (b('l8b4') <= 706.2778625488281) ?  0.021577401951725386 :   0.044537435490622786 :   (b('l8b11') <= 1347.9095458984375) ?  (b('ndvi_med') <= 3279.5) ?  0.029830276310448257 :   0.0602395522523455 :   (b('ndvi') <= 3750.5) ?  -0.0009477115397535599 :   0.01668225106701687 :   (b('l8b4_med') <= 890.4987182617188) ?  (b('ndvi_med') <= 3105.25) ?  (b('l8b4') <= 797.7445068359375) ?  (b('dg0_v_vv') <= 1.2158961296081543) ?  -0.00801215413052794 :   -0.023419801953012853 :   (b('l8b4_med') <= 834.6466369628906) ?  0.012875442494269316 :   -0.05077392164258245 :   (b('l8b11_med') <= 1136.3778076171875) ?  (b('dg0_v_vv') <= -0.013407707214355469) ?  0.04753327300879534 :   0.02146147749231341 :   (b('l8b5') <= 4632.885986328125) ?  -0.001611544018369813 :   -0.03719540506043559 :   (b('sand') <= 41.0) ?  (b('l8b5_med') <= 2584.8277587890625) ?  (b('l8b11') <= 1306.5115966796875) ?  0.03409751339469163 :   0.13892933404948257 :   (b('ndvi') <= 4475.5) ?  0.004956356177965698 :   0.028307221992367106 :   (b('dg0_v_vv') <= 0.8644435405731201) ?  (b('l8b11') <= 1311.671142578125) ?  -0.03746701104943758 :   -0.01291498773121913 :   (b('l8b5') <= 3640.654296875) ?  0.02714718333524452 :   -0.02915248234197987 :   (b('dg0_v_vv') <= -1.917593002319336) ?  -0.05152479111251365 :   (b('ndvi') <= 2573.5) ?  (b('ndvi') <= 1963.5) ?  0.08736572915535173 :   (b('bulk') <= 163.5) ?  0.13998843173744352 :   0.11851400371348475 :   (b('dg0_v_vv') <= 0.2015361785888672) ?  -0.005057585052381075 :   (b('l8b11') <= 1517.152099609375) ?  0.11124534947610154 :   0.0612502941828037 :   (b('l8b4_med') <= 644.4173583984375) ?  (b('ndvi') <= 4984.0) ?  (b('l8b5') <= 2618.6949462890625) ?  (b('l8b5') <= 2223.4185791015625) ?  (b('ndvi') <= 2486.5) ?  (b('l8b4') <= 712.4852294921875) ?  (b('dg0_v_vv') <= 0.36927366256713867) ?  (b('dg0_v_vv') <= 0.20868253707885742) ?  0.013095932291673102 :   0.07646332101519498 :   (b('l8b4') <= 582.8544921875) ?  0.008212541815549085 :   -0.012857350433220963 :   (b('dg0_v_vv') <= -0.16237306594848633) ?  -0.07187387796934563 :   (b('dg0_v_vv') <= 1.3687615394592285) ?  -0.019880450613333045 :   -0.021832494108959877 :   (b('l8b5') <= 2207.9786376953125) ?  (b('l8b4_med') <= 530.1519317626953) ?  (b('ndvi') <= 3489.0) ?  0.056403647931811844 :   0.030027770599126726 :   (b('dg0_v_vv') <= -0.18239974975585938) ?  0.006412093280563173 :   -0.01707685963153907 :   0.14751417176391513 :   (b('ndvi') <= 4108.5) ?  (b('sand') <= 47.0) ?  (b('l8b11_med') <= 1244.5859985351562) ?  (b('ndvi_med') <= 3443.5) ?  0.025026286635201844 :   0.004852474776417315 :   (b('grass') <= 58.40892028808594) ?  -0.05024607146109883 :   -0.049309675518271215 :   (b('l8b11') <= 1622.3778686523438) ?  (b('l8b5_med') <= 3089.992919921875) ?  0.017360521561039115 :   0.028146427721846667 :   (b('dg0_v_vv') <= 2.2993245124816895) ?  0.035967257418351384 :   0.04783475730225517 :   (b('ndvi') <= 4182.5) ?  (b('l8b4') <= 454.9197235107422) ?  (b('dg0_v_vv') <= -0.20326662063598633) ?  -0.027363952347203985 :   -0.022966926897834222 :   (b('ndvi_med') <= 3533.5) ?  -0.0772573292029958 :   -0.06509269410548407 :   (b('l8b5') <= 2413.8360595703125) ?  (b('dg0_v_vv') <= -1.189859390258789) ?  0.053632151654813054 :   0.014568903732001322 :   (b('dg0_v_vv') <= -0.036528587341308594) ?  -0.04496133590932268 :   -0.005405670053673095 :   (b('l8b4') <= 725.1026916503906) ?  (b('ndvi_med') <= 3065.5) ?  (b('l8b11') <= 1219.5707397460938) ?  (b('ndvi') <= 4751.0) ?  0.026608093844350034 :   0.04273050735080558 :   (b('l8b11') <= 1281.3228759765625) ?  (b('dg0_v_vv') <= -0.6370329856872559) ?  0.07421410957693703 :   0.06601579371496175 :   (b('l8b4') <= 634.6933898925781) ?  0.09290184494012654 :   0.08411922751696438 :   (b('l8b11') <= 949.2217407226562) ?  (b('l8b11') <= 931.0607604980469) ?  (b('dg0_v_vv') <= -0.00315093994140625) ?  0.03470872171861648 :   -0.00813845538979241 :   (b('dg0_v_vv') <= -0.01160430908203125) ?  -0.016845822440505753 :   -0.06460058020086057 :   (b('l8b11') <= 1016.0818786621094) ?  (b('ndvi') <= 4844.0) ?  0.0444436477585008 :   0.11746553347699384 :   (b('l8b5') <= 3552.3370361328125) ?  0.02791724963961663 :   -0.011184571168342821 :   (b('l8b11') <= 1481.7373657226562) ?  (b('ndvi') <= 3691.5) ?  -0.06594939165387947 :   -0.0697533437741965 :   (b('dg0_v_vv') <= -0.3863201141357422) ?  (b('ndvi_med') <= 3065.5) ?  -0.03816436672150292 :   (b('ndvi') <= 3209.5) ?  0.006892507864837509 :   0.02303227944857797 :   (b('moss') <= 8.974802792072296) ?  (b('l8b4') <= 770.5484619140625) ?  0.012897423293802113 :   0.021304967008131965 :   0.035501109148884324 :   (b('dg0_v_vv') <= -0.5626401901245117) ?  (b('dg0_v_vv') <= -0.9180164337158203) ?  (b('ndvi_med') <= 3520.75) ?  -0.05510735106262954 :   -0.07222047789710884 :   (b('ndvi_med') <= 3533.5) ?  0.0525127781535768 :   0.06864785565826581 :   (b('dg0_v_vv') <= -0.28997135162353516) ?  -0.1095655414618304 :   -0.035264324039809336 :   (b('dg0_v_vv') <= 0.3918633460998535) ?  (b('l8b4_med') <= 956.1120910644531) ?  (b('l8b4_med') <= 948.1294555664062) ?  (b('dg0_v_vv') <= -0.6155505180358887) ?  (b('l8b4_med') <= 910.700927734375) ?  (b('l8b4_med') <= 752.6678466796875) ?  (b('l8b11_med') <= 1527.93896484375) ?  -0.0015555500878585494 :   0.08836995937649947 :   (b('l8b5_med') <= 2406.5733642578125) ?  -0.03143608200380526 :   -0.011795123735383306 :   (b('dg0_v_vv') <= -0.9467940330505371) ?  0.08723285514015928 :   0.04389793078627491 :   (b('bulk') <= 153.0) ?  (b('l8b5') <= 4720.76220703125) ?  (b('l8b4') <= 810.2700805664062) ?  0.0042879931769851795 :   -0.02976022999256448 :   0.11322961301463988 :   (b('l8b5') <= 3533.0645751953125) ?  (b('dg0_v_vv') <= -0.47154998779296875) ?  0.12576088582030645 :   0.034207127804100054 :   (b('l8b4') <= 736.8712768554688) ?  -0.046526449583599014 :   0.028259105669564882 :   (b('l8b4_med') <= 952.4841003417969) ?  (b('dg0_v_vv') <= -0.45349788665771484) ?  (b('l8b4') <= 629.0723266601562) ?  -0.10165361345884359 :   (b('l8b4') <= 722.2626037597656) ?  -0.08467115246228919 :   -0.0764727851339453 :   (b('l8b4') <= 684.8780822753906) ?  -0.13099240131482573 :   -0.11896638322214777 :   -0.014050128630060957 :   (b('l8b11') <= 1063.5155639648438) ?  (b('l8b4') <= 460.599853515625) ?  (b('l8b4') <= 449.54425048828125) ?  (b('l8b11') <= 809.3506164550781) ?  (b('l8b11') <= 767.9037475585938) ?  0.05626725671251287 :   0.0499002241354371 :   (b('l8b4') <= 446.4172821044922) ?  -0.005632528055964814 :   0.02575977281157002 :   0.14585194640481314 :   (b('l8b5') <= 3234.4666748046875) ?  (b('l8b11_med') <= 2110.28173828125) ?  (b('ndvi_med') <= 1589.0) ?  -0.0608923208801046 :   -0.016599223310904885 :   (b('ndvi') <= 4611.0) ?  -0.08330382619027102 :   -0.023572714382702253 :   (b('crops') <= 11.5) ?  0.060597230344072606 :   (b('dg0_v_vv') <= -0.15439081192016602) ?  -0.005053429085547775 :   0.017620768479228055 :   (b('l8b11_med') <= 1746.1769409179688) ?  (b('l8b4') <= 625.8204956054688) ?  (b('ndvi') <= 2529.5) ?  (b('ndvi') <= 1696.0) ?  0.07094753926018416 :   0.05622556558284564 :   (b('ndvi_med') <= 1993.5) ?  0.006237022680535387 :   -0.04532980680544402 :   (b('dg0_v_vv') <= -1.3683037757873535) ?  (b('dg0_v_vv') <= -1.6161761283874512) ?  0.12654005187506295 :   0.11991885494611526 :   (b('bulk') <= 146.5) ?  0.09713986696038651 :   0.1022887676787333 :   (b('l8b5') <= 1473.449462890625) ?  0.1915197775071831 :   (b('l8b5') <= 3100.2039794921875) ?  (b('l8b11_med') <= 1769.387451171875) ?  -0.13789000688208022 :   0.010442518137759397 :   (b('dg0_v_vv') <= 0.11980962753295898) ?  0.08250756889901426 :   0.04174211558937467 :   (b('l8b4') <= 790.52685546875) ?  (b('ndvi') <= 1269.5) ?  (b('l8b4_med') <= 790.041748046875) ?  -0.09189767941576016 :   (b('ndvi_med') <= 1640.5) ?  -0.05989576501027863 :   -0.05775230825243702 :   (b('l8b4_med') <= 1693.5594482421875) ?  (b('bulk') <= 166.0) ?  (b('grass') <= 75.37618255615234) ?  (b('l8b4') <= 406.8275604248047) ?  0.10554229865497461 :   0.02458598413848301 :   (b('l8b4_med') <= 795.3672790527344) ?  -0.01197546834955601 :   0.0159963916002532 :   (b('dg0_v_vv') <= 0.7219400405883789) ?  (b('l8b4_med') <= 1282.8197631835938) ?  0.008383939302598897 :   0.08012924257467485 :   (b('l8b4') <= 656.2074890136719) ?  0.10240040644661805 :   0.09541817981549489 :   (b('grass') <= 82.17940902709961) ?  0.15180588406977533 :   0.09909950775022985 :   (b('l8b11') <= 1716.8707885742188) ?  (b('ndvi_med') <= 2277.0) ?  (b('ndvi') <= 3082.5) ?  (b('grass') <= 66.56881904602051) ?  (b('l8b4') <= 794.2663879394531) ?  0.04272295549432109 :   -0.037952157925035666 :   (b('bulk') <= 145.5) ?  0.03446472259064201 :   -0.005204420464221204 :   (b('l8b5') <= 2727.8988037109375) ?  (b('l8b11_med') <= 2041.8341064453125) ?  -0.07025148209062988 :   -0.0900124475680606 :   -0.02981951641196856 :   (b('moss') <= 13.099216938018799) ?  (b('l8b5') <= 3283.418701171875) ?  (b('dg0_v_vv') <= 2.5240159034729004) ?  0.07576484300550704 :   0.05301486486053697 :   0.019600625813374573 :   (b('sand') <= 39.5) ?  (b('l8b11') <= 1527.1635131835938) ?  -0.03340307278006381 :   -0.06496280736078874 :   (b('ndvi') <= 3253.5) ?  0.040646585796991035 :   -0.01835508711146166 :   (b('l8b4_med') <= 758.6447143554688) ?  0.02418778738311056 :   (b('l8b4') <= 810.9966735839844) ?  (b('dg0_v_vv') <= 1.5481233596801758) ?  -0.027776539976806736 :   (b('l8b11') <= 1731.7299194335938) ?  -0.07580721705394786 :   -0.05390206155780379 :   (b('dg0_v_vv') <= 2.8725624084472656) ?  (b('moss') <= 17.460886478424072) ?  -0.07463823294160032 :   -0.07086754213780025 :   -0.07553315606140834 :   (b('moss') <= 8.972131252288818) ?  (b('l8b11_med') <= 1754.156982421875) ?  (b('bulk') <= 136.5) ?  (b('l8b5') <= 2502.948486328125) ?  (b('l8b11_med') <= 1695.4384155273438) ?  (b('sand') <= 23.0) ?  (b('l8b11') <= 1486.1697998046875) ?  0.11985390209312163 :   (b('dg0_v_vv') <= 2.412858009338379) ?  (b('dg0_v_vv') <= -1.0830879211425781) ?  0.024941218593374476 :   0.06000659404227082 :   -0.024855649436286764 :   (b('l8b5') <= 2501.7720947265625) ?  (b('moss') <= 8.799341201782227) ?  (b('l8b11') <= 1412.4738159179688) ?  0.0208847900281179 :   0.0027374234195178307 :   (b('l8b11_med') <= 1326.04443359375) ?  -0.06165631233580131 :   -0.06830614015906666 :   (b('dg0_v_vv') <= -1.3365812301635742) ?  0.0036231052945542574 :   (b('dg0_v_vv') <= 0.2324824333190918) ?  0.06554245486749258 :   0.04953918093360357 :   (b('dg0_v_vv') <= 1.6078486442565918) ?  (b('grass') <= 58.75040817260742) ?  (b('ndvi_med') <= 3944.75) ?  (b('l8b5') <= 2146.23095703125) ?  0.027565415457542597 :   0.03580927299647964 :   -0.011760837146366221 :   (b('ndvi') <= 2911.5) ?  (b('dg0_v_vv') <= -1.367044448852539) ?  0.032034843024240645 :   0.0736243006695348 :   0.1607590767108069 :   -0.08631254027708957 :   (b('l8b5_med') <= 3288.431396484375) ?  (b('l8b5') <= 3082.139404296875) ?  (b('l8b5_med') <= 2941.3280029296875) ?  (b('dg0_v_vv') <= -2.293196678161621) ?  (b('l8b11') <= 2405.6015625) ?  -0.014952343366926432 :   -0.08104790409402023 :   (b('l8b11_med') <= 1175.5803833007812) ?  -0.035064538876925996 :   -0.0030779712946559373 :   (b('l8b11') <= 2333.6402587890625) ?  (b('ndvi') <= 1217.5) ?  -0.059172571893353974 :   -0.023914391582851323 :   (b('ndvi') <= 4105.5) ?  0.0005394595258283383 :   0.019546260530201868 :   (b('ndvi') <= 2427.0) ?  (b('l8b11') <= 2516.2760009765625) ?  (b('l8b11') <= 2063.5210571289062) ?  0.03253402823002348 :   0.0059131451340852115 :   (b('dg0_v_vv') <= -2.1477184295654297) ?  0.04743360220120127 :   0.06877837061709927 :   (b('l8b5') <= 3566.9326171875) ?  (b('l8b11') <= 1491.867431640625) ?  0.05014167212784759 :   0.003077788209973271 :   (b('l8b4_med') <= 837.4677429199219) ?  -0.04676278816426074 :   -0.018058036246178848 :   (b('l8b5_med') <= 3505.713134765625) ?  (b('bulk') <= 121.0) ?  (b('l8b11') <= 1789.8402099609375) ?  0.11456509897454131 :   (b('l8b11') <= 2585.380615234375) ?  0.03976442876302046 :   0.0827996790036577 :   (b('l8b5') <= 3021.3699951171875) ?  (b('sand') <= 63.5) ?  0.03977895760288666 :   -0.04303675064614468 :   (b('ndvi') <= 4772.5) ?  0.011435063397342732 :   -0.036039951799027886 :   (b('ndvi') <= 4689.5) ?  (b('ndvi') <= 2596.5) ?  (b('ndvi') <= 2283.5) ?  -0.009583559106233059 :   -0.1556840591713057 :   (b('ndvi') <= 2795.5) ?  0.05285403318749082 :   -0.01307252770465515 :   (b('ndvi') <= 4752.5) ?  0.11951027996034777 :   (b('dg0_v_vv') <= -0.30481433868408203) ?  0.024532869218103853 :   -0.03136522398723661 :   (b('dg0_v_vv') <= -1.4224820137023926) ?  (b('l8b4_med') <= 542.6818237304688) ?  0.08747503650958655 :   (b('ndvi') <= 4903.5) ?  (b('sand') <= 30.0) ?  (b('dg0_v_vv') <= -2.4214134216308594) ?  (b('dg0_v_vv') <= -2.7227206230163574) ?  -0.006631982706286033 :   -0.048756941225428625 :   (b('ndvi_med') <= 4011.0) ?  0.004599830031809398 :   0.05526215320215905 :   (b('l8b5_med') <= 3108.6705322265625) ?  (b('l8b4') <= 2004.6549682617188) ?  -0.012300248510379608 :   0.08413968539494543 :   (b('sand') <= 43.5) ?  -0.048743067539298246 :   -0.09631963475468694 :   0.08620567168327606 :   (b('dg0_v_vv') <= -1.3983983993530273) ?  (b('l8b4_med') <= 957.064453125) ?  0.1076067088419772 :   0.19769894652039297 :   (b('l8b5') <= 1691.8463745117188) ?  (b('l8b4_med') <= 1179.8871459960938) ?  0.1576504098362968 :   0.07233666511460665 :   (b('l8b4') <= 1022.8262023925781) ?  (b('l8b5_med') <= 3077.8951416015625) ?  (b('l8b11_med') <= 1512.6978759765625) ?  0.03286425608262561 :   0.006377290512852348 :   (b('l8b5') <= 2429.56494140625) ?  -0.0529153583852398 :   -0.003571035385350377 :   (b('l8b11') <= 1577.0394897460938) ?  (b('dg0_v_vv') <= 2.1398332118988037) ?  0.05994094986074222 :   0.13922533569192563 :   (b('crops') <= 63.93314170837402) ?  0.02866995248851177 :   0.008962078170574959 :   (b('l8b11_med') <= 1892.4669799804688) ?  (b('l8b4') <= 2260.8414306640625) ?  (b('grass') <= 29.971359252929688) ?  (b('l8b5') <= 1661.5052490234375) ?  0.1289472963898411 :   (b('l8b11') <= 1442.1478881835938) ?  (b('l8b4') <= 848.9323120117188) ?  (b('l8b4_med') <= 1119.0999145507812) ?  0.022910787650983977 :   0.02221504746413458 :   (b('l8b11') <= 1393.7308349609375) ?  -0.03975778548251208 :   -0.0680892449947089 :   (b('l8b5') <= 2952.2672119140625) ?  (b('l8b5') <= 2895.8887939453125) ?  -0.007057158031063399 :   -0.03318810987232905 :   (b('ndvi') <= 2504.0) ?  0.006481329701201665 :   -0.007936993458730052 :   (b('l8b4') <= 1356.2903442382812) ?  (b('dg0_v_vv') <= -0.25742053985595703) ?  (b('bulk') <= 142.5) ?  (b('l8b5_med') <= 2591.3553466796875) ?  -0.03364820469178738 :   -0.05832247653084913 :   (b('ndvi') <= 2145.5) ?  -0.01869888875066801 :   1.9741605221436165e-05 :   (b('l8b11_med') <= 1854.2278442382812) ?  (b('ndvi') <= 2235.5) ?  -0.01875471690623604 :   -0.03548799941213125 :   (b('l8b4') <= 868.99560546875) ?  -0.11106977500088556 :   -0.04530377341713011 :   (b('grass') <= 82.5) ?  (b('l8b5') <= 2744.775146484375) ?  (b('l8b4') <= 1678.2734375) ?  -0.01581392315154348 :   0.007558393128716738 :   (b('l8b4') <= 1488.76171875) ?  -0.032507647026527614 :   -0.022243839955642983 :   (b('l8b11') <= 2274.778076171875) ?  (b('dg0_v_vv') <= -0.6609444618225098) ?  0.015429872013594206 :   0.008059525407856797 :   (b('ndvi_med') <= 2144.75) ?  -0.00996292737012707 :   -0.03881579300945224 :   0.14768542386217318 :   (b('dg0_v_vv') <= 0.2605128288269043) ?  (b('ndvi_med') <= 2174.0) ?  (b('ndvi_med') <= 1866.75) ?  (b('l8b5_med') <= 3250.251708984375) ?  (b('ndvi') <= 1254.0) ?  (b('bulk') <= 152.5) ?  -0.01830313365829515 :   0.00029104207632443655 :   (b('l8b4_med') <= 1095.4453735351562) ?  0.040733643773172144 :   -0.004157395533473591 :   (b('ndvi_med') <= 1518.75) ?  (b('l8b4') <= 2840.0167236328125) ?  0.0011863997850495784 :   -0.014436659409734032 :   (b('dg0_v_vv') <= 0.2156982421875) ?  0.018008610287509563 :   -0.06678106033936229 :   (b('dg0_v_vv') <= -0.1044473648071289) ?  (b('ndvi') <= 1217.5) ?  (b('l8b5') <= 2258.3341064453125) ?  -0.008617564155512864 :   0.047220826512680086 :   (b('l8b5') <= 2057.2686767578125) ?  0.02204529686073833 :   -0.002166651893079427 :   (b('l8b11') <= 1720.0291748046875) ?  (b('l8b5_med') <= 2259.166259765625) ?  0.004646013896337189 :   0.10238520804304933 :   (b('bulk') <= 153.0) ?  0.009426239601994854 :   0.10740017343941546 :   (b('grass') <= 98.67486190795898) ?  (b('l8b5') <= 3501.3265380859375) ?  (b('bulk') <= 158.5) ?  (b('ndvi') <= 3417.5) ?  -0.01746494899379326 :   0.022141355884302612 :   (b('l8b4') <= 1256.496826171875) ?  -0.013204077137195576 :   0.061953164960111384 :   (b('l8b5_med') <= 3222.4583740234375) ?  (b('l8b5') <= 4050.5537109375) ?  -0.05386029815217786 :   0.02106182631800338 :   (b('dg0_v_vv') <= -0.6952548027038574) ?  -0.025598451042832242 :   -0.009194172684607572 :   (b('l8b5') <= 2437.4259033203125) ?  (b('l8b5_med') <= 2675.7364501953125) ?  (b('ndvi') <= 1541.5) ?  -0.03391775317626118 :   -0.00034130795123321194 :   (b('ndvi_med') <= 2308.25) ?  0.03232339866897482 :   0.05053955968477464 :   (b('l8b5') <= 2470.9525146484375) ?  0.10466669816207286 :   (b('l8b4') <= 991.0202026367188) ?  0.06802790140611191 :   0.025198853672861808 :   (b('ndvi_med') <= 1791.0) ?  (b('moss') <= 3.4928038120269775) ?  (b('sand') <= 56.5) ?  (b('l8b11') <= 1937.0867919921875) ?  (b('l8b4') <= 1047.1198120117188) ?  -0.012006998974561358 :   -0.0709915357214642 :   (b('dg0_v_vv') <= 5.357389450073242) ?  -0.01073788950226369 :   0.06969043847854799 :   (b('l8b5') <= 2053.9326782226562) ?  (b('l8b5') <= 2019.7946166992188) ?  -0.018826357990487215 :   -0.07886546463207428 :   (b('l8b5_med') <= 2655.8890380859375) ?  0.027152590256563233 :   0.0019684761579649244 :   (b('l8b11') <= 2571.6607666015625) ?  (b('l8b11') <= 2506.6590576171875) ?  (b('l8b11_med') <= 2680.3411865234375) ?  0.02142830997488804 :   -0.0007311641215876952 :   (b('dg0_v_vv') <= 2.6798300743103027) ?  0.09134394945320343 :   0.005409538195327301 :   (b('ndvi') <= 2937.0) ?  (b('sand') <= 49.5) ?  -0.0012551980766927144 :   -0.018728024015819852 :   0.13388713895674245 :   (b('dg0_v_vv') <= 0.26711511611938477) ?  0.19482928734967203 :   (b('sand') <= 68.0) ?  (b('ndvi') <= 3310.0) ?  (b('ndvi') <= 2640.5) ?  0.0087100500523692 :   0.026079559739302687 :   (b('l8b11') <= 2958.057373046875) ?  -0.008435739112958014 :   0.05059729920721995 :   (b('l8b11') <= 2389.3900146484375) ?  (b('l8b11') <= 2140.2044067382812) ?  -0.054831005691330176 :   -0.09752780062337388 :   (b('l8b5_med') <= 2461.7109375) ?  0.07080573439566402 :   0.0016965354647632048 :   (b('bulk') <= 145.5) ?  (b('bulk') <= 138.5) ?  (b('sand') <= 72.5) ?  (b('l8b11') <= 1580.7589721679688) ?  (b('ndvi_med') <= 2264.25) ?  (b('dg0_v_vv') <= -1.6099662780761719) ?  0.13761687188920677 :   (b('l8b11') <= 1207.5) ?  (b('l8b11') <= 1133.5) ?  -0.0052347874961411744 :   0.081119488995891 :   (b('l8b5') <= 1652.7581787109375) ?  -0.027033658611517054 :   0.006781611696409867 :   (b('moss') <= 12.409264087677002) ?  (b('ndvi') <= 2090.0) ?  (b('dg0_v_vv') <= -0.5432934761047363) ?  0.012130584110741117 :   0.019661766850172596 :   (b('l8b11_med') <= 1680.6777954101562) ?  -0.027614847703870883 :   -0.016503233540634564 :   (b('l8b4') <= 847.2578430175781) ?  (b('crops') <= 22.04555892944336) ?  -0.035327220550744505 :   -0.056933350657607074 :   (b('l8b5') <= 2018.6565551757812) ?  -0.06507146667764413 :   -0.027571943907279912 :   (b('l8b11') <= 1582.43603515625) ?  (b('dg0_v_vv') <= -0.5069890022277832) ?  0.10854404427303577 :   (b('dg0_v_vv') <= 0.8558568954467773) ?  (b('l8b5_med') <= 3114.77099609375) ?  0.09073799982857876 :   0.08349005541616503 :   0.07400910909394848 :   (b('l8b5_med') <= 1639.760498046875) ?  (b('l8b4') <= 1063.3858032226562) ?  (b('l8b11') <= 1950.473388671875) ?  0.04923797535069151 :   0.15791751844791777 :   (b('dg0_v_vv') <= 0.6282758712768555) ?  -0.018347119992384472 :   0.021417398577483364 :   (b('moss') <= 15.002391815185547) ?  (b('l8b5') <= 2447.60791015625) ?  0.008129214986782205 :   -0.005739003086489174 :   (b('dg0_v_vv') <= 0.991279125213623) ?  -0.015611380789320376 :   0.014102875584985129 :   (b('l8b5') <= 2822.0535888671875) ?  (b('l8b11') <= 1570.6292724609375) ?  (b('dg0_v_vv') <= 0.6804986000061035) ?  (b('dg0_v_vv') <= 0.31089353561401367) ?  (b('ndvi_med') <= 3447.5) ?  0.11668156733981994 :   0.031641280424009266 :   (b('ndvi_med') <= 3336.0) ?  0.04268156638614562 :   0.03068157186977294 :   0.1077006164826596 :   (b('dg0_v_vv') <= -2.2485642433166504) ?  0.005325875278224035 :   (b('ndvi_med') <= 3447.5) ?  (b('dg0_v_vv') <= -1.1207213401794434) ?  0.050343137265628723 :   0.031326863314407004 :   (b('l8b5') <= 2617.1636962890625) ?  0.058818835678770054 :   0.050050343471391257 :   (b('dg0_v_vv') <= -0.11343669891357422) ?  -0.03730130561804401 :   -0.06558462544801857 :   (b('sand') <= 58.5) ?  (b('ndvi') <= 1534.0) ?  (b('dg0_v_vv') <= 1.3257999420166016) ?  (b('l8b4') <= 849.3826599121094) ?  (b('l8b11') <= 1494.7985229492188) ?  0.04620874052983487 :   0.15083654987480288 :   (b('l8b11') <= 3222.0543212890625) ?  (b('ndvi') <= 1172.0) ?  0.00022896595451571983 :   -0.011779515342890328 :   0.12335459803016763 :   (b('ndvi') <= 1477.0) ?  (b('l8b11') <= 1906.7113647460938) ?  (b('l8b5') <= 1718.3223876953125) ?  0.04733923117371163 :   0.08466898227752451 :   (b('l8b11') <= 1921.6886596679688) ?  -0.011132860106178885 :   0.03139004130957631 :   (b('dg0_v_vv') <= 2.8993654251098633) ?  -0.020798831628063702 :   -0.04879801627972383 :   (b('ndvi_med') <= 1958.0) ?  (b('ndvi') <= 2217.5) ?  (b('ndvi') <= 1556.5) ?  (b('l8b5') <= 2412.396240234375) ?  0.1406730440961407 :   0.061463943625673374 :   (b('ndvi_med') <= 1309.5) ?  0.05263694729319202 :   0.007839938018653831 :   (b('l8b4') <= 1031.7154541015625) ?  (b('dg0_v_vv') <= 2.061213493347168) ?  0.04437249952025831 :   -0.042865055328287494 :   (b('l8b4') <= 1290.8687133789062) ?  0.09834775038331878 :   0.043914149225849464 :   (b('l8b5') <= 3332.577880859375) ?  (b('l8b5') <= 2019.4596557617188) ?  (b('ndvi') <= 2209.5) ?  0.020812942959838456 :   0.0687703375144869 :   (b('ndvi_med') <= 2300.25) ?  -0.0315486730629721 :   -0.0023195479458913214 :   (b('dg0_v_vv') <= -0.765172004699707) ?  (b('l8b5_med') <= 3345.4058837890625) ?  -0.029295709931297362 :   0.00298726947305733 :   (b('ndvi') <= 3955.0) ?  0.08592283515087137 :   0.015363930337040458 :   (b('ndvi_med') <= 1035.5) ?  (b('l8b4') <= 1576.0291137695312) ?  (b('l8b11') <= 2143.9295654296875) ?  (b('l8b4') <= 1377.7515869140625) ?  (b('l8b4_med') <= 1427.4837036132812) ?  0.0179785172555949 :   0.057408595186614536 :   (b('l8b11') <= 1998.9567260742188) ?  0.11587887800901825 :   0.04518449033774254 :   (b('l8b11') <= 2267.51708984375) ?  (b('dg0_v_vv') <= 0.3222665786743164) ?  -0.02041358820477293 :   -0.07708767181876296 :   (b('l8b5_med') <= 2094.7025146484375) ?  -0.0013937194927588423 :   0.019497948387273947 :   (b('dg0_v_vv') <= -0.06816482543945312) ?  (b('l8b5') <= 2255.3282470703125) ?  0.025865417464626034 :   0.04855625715871363 :   (b('l8b11') <= 2284.7532958984375) ?  (b('ndvi_med') <= 917.5) ?  0.0926695299563223 :   0.094755749774624 :   0.0706900099950287 :   0.16258499746109828 :   (b('ndvi') <= 3203.5) ?  (b('l8b4_med') <= 1054.0230102539062) ?  (b('l8b5') <= 2007.8824462890625) ?  (b('l8b5_med') <= 2360.460693359375) ?  (b('l8b11_med') <= 1472.2669677734375) ?  (b('grass') <= 54.7896614074707) ?  (b('l8b4') <= 912.4281921386719) ?  0.05639844792011614 :   0.042622947913737996 :   (b('l8b4') <= 946.3836364746094) ?  -0.006015050406643457 :   0.0291360734960573 :   (b('l8b4') <= 828.1571655273438) ?  (b('l8b5') <= 1882.8980102539062) ?  -0.06710511697136069 :   -0.017100572643565314 :   (b('l8b4') <= 843.08447265625) ?  0.014136759962976045 :   -0.015221304477289222 :   (b('l8b11') <= 2108.412109375) ?  (b('l8b11') <= 2004.947998046875) ?  (b('grass') <= 51.619625091552734) ?  0.07224535508844387 :   0.041421140334182774 :   0.00013120260547533347 :   (b('dg0_v_vv') <= -0.5664725303649902) ?  0.1212073777610001 :   0.09991406879202858 :   (b('l8b5') <= 2918.383544921875) ?  (b('l8b5_med') <= 1796.3161010742188) ?  (b('l8b11') <= 1666.641357421875) ?  (b('l8b4') <= 1064.2090148925781) ?  0.10000463483989966 :   0.10761164339940849 :   (b('l8b11') <= 1882.4891967773438) ?  -0.034349540689215224 :   0.0186783094575056 :   (b('ndvi') <= 3011.5) ?  (b('l8b4') <= 822.046875) ?  0.08492637864310332 :   -0.024403532141733675 :   (b('l8b4') <= 900.3635559082031) ?  -0.13321268160871214 :   -0.05317433862812638 :   (b('l8b5') <= 3149.2208251953125) ?  (b('dg0_v_vv') <= 0.5044755935668945) ?  (b('l8b4') <= 1239.8250732421875) ?  0.04274368515227192 :   -0.045235070511044695 :   (b('l8b5_med') <= 2509.5291748046875) ?  0.12667753754896055 :   0.10977393847845543 :   (b('ndvi') <= 2342.0) ?  -0.01769740852293185 :   -0.05556893999746915 :   (b('l8b4') <= 1163.4313354492188) ?  (b('l8b4') <= 1160.23193359375) ?  (b('sand') <= 66.5) ?  (b('l8b5_med') <= 2226.828369140625) ?  (b('l8b4_med') <= 1371.5338745117188) ?  -0.013445402093311859 :   0.06475904422266643 :   (b('l8b5_med') <= 2324.61962890625) ?  0.0320640832890453 :   0.0013642297682519347 :   0.1577209613130122 :   0.17896522461388195 :   (b('moss') <= 50.33349609375) ?  (b('ndvi_med') <= 1596.5) ?  (b('l8b5_med') <= 1895.3878173828125) ?  (b('dg0_v_vv') <= 0.7665534019470215) ?  -0.02145265985778242 :   -0.056199155914057074 :   (b('moss') <= 11.803977012634277) ?  -0.011150564742196153 :   -0.0038566497285118535 :   (b('l8b4_med') <= 1079.4931030273438) ?  (b('l8b11') <= 2325.216796875) ?  -0.004149711275368832 :   0.07189881210022589 :   (b('l8b4_med') <= 1234.7986450195312) ?  -0.023557953220991662 :   -0.009580784234692007 :   (b('dg0_v_vv') <= -0.7805542945861816) ?  (b('dg0_v_vv') <= -1.06154203414917) ?  (b('l8b4') <= 1353.1285400390625) ?  0.011808838483603146 :   -0.04172683303638399 :   (b('dg0_v_vv') <= -0.9697937965393066) ?  0.0670981123305164 :   0.0050913900969375425 :   (b('l8b5') <= 2156.7490234375) ?  (b('ndvi') <= 976.5) ?  -0.053238488824673094 :   -0.04095477650455891 :   (b('l8b11') <= 2529.56787109375) ?  -0.016798050634481986 :   -0.03150044991338164 :   (b('ndvi') <= 3217.5) ?  (b('l8b11_med') <= 1918.3629760742188) ?  0.15337711361134349 :   0.19436268476354335 :   (b('l8b5') <= 3294.2962646484375) ?  (b('l8b11_med') <= 1950.8327026367188) ?  (b('l8b11') <= 2205.573974609375) ?  (b('l8b4') <= 824.3806762695312) ?  0.03580492703149554 :   (b('dg0_v_vv') <= -0.2557563781738281) ?  -0.021801736442113494 :   -0.06555855135731989 :   0.06199207538943674 :   (b('l8b4') <= 853.8118896484375) ?  (b('l8b5') <= 2855.250244140625) ?  (b('l8b11') <= 1753.6206665039062) ?  0.018123605037313584 :   -0.039237031207632445 :   -0.12946753765349334 :   (b('dg0_v_vv') <= -0.6396722793579102) ?  (b('dg0_v_vv') <= -1.1057028770446777) ?  0.016503727971306406 :   -0.028038333237658856 :   (b('l8b4') <= 861.2925109863281) ?  0.1929713371033695 :   0.025833109315230387 :   (b('l8b5') <= 3316.5985107421875) ?  (b('ndvi_med') <= 3309.75) ?  (b('l8b11') <= 2183.4051513671875) ?  -0.07505325443097334 :   -0.09315554330944151 :   -0.04790779198007486 :   (b('l8b5') <= 3819.2425537109375) ?  (b('l8b4_med') <= 1146.3720703125) ?  (b('l8b4') <= 854.6427612304688) ?  -0.0884398336296841 :   -0.02770071504831213 :   (b('l8b11_med') <= 2247.2081298828125) ?  0.013645533706475973 :   -0.012701627994887466 :   (b('l8b4') <= 1226.8110961914062) ?  (b('l8b11_med') <= 1751.8439331054688) ?  0.04389153109328428 :   0.026260281209793013 :   (b('l8b5') <= 3977.57470703125) ?  0.008859488259805242 :   -0.012736948265532799 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1719.3646240234375) ?  (b('sand') <= 27.5) ?  (b('grass') <= 39.6829776763916) ?  (b('ndvi_med') <= 2660.5) ?  (b('ndvi') <= 4589.5) ?  (b('ndvi') <= 1138.5) ?  (b('ndvi_med') <= 2273.75) ?  (b('dg0_v_vv') <= -0.39159369468688965) ?  (b('l8b11') <= 1333.9100341796875) ?  -0.04356999652411214 :   (b('ndvi_med') <= 1738.5) ?  0.03326916159779192 :   -0.004445824769941237 :   (b('ndvi') <= 953.5) ?  (b('ndvi_med') <= 2107.5) ?  0.018526793565153854 :   0.03605571870244162 :   0.05378927240010667 :   (b('bulk') <= 127.0) ?  -0.022370084492658093 :   -0.03785053969385138 :   (b('l8b5_med') <= 2134.86181640625) ?  (b('dg0_v_vv') <= -0.6433796882629395) ?  (b('ndvi_med') <= 1911.5) ?  0.020793867291952095 :   (b('l8b5') <= 3054.2799682617188) ?  0.012321647251326229 :   0.013454135171856563 :   (b('l8b4') <= 676.8515930175781) ?  0.005371120047758637 :   (b('l8b5_med') <= 1973.9232177734375) ?  -0.0033625399626792674 :   -0.010003806055606634 :   (b('l8b5') <= 3547.031494140625) ?  (b('l8b11_med') <= 2095.1530151367188) ?  (b('dg0_v_vv') <= -2.729431629180908) ?  0.020272859607798704 :   0.05575044777611561 :   (b('l8b4') <= 734.7347412109375) ?  0.01872445793288796 :   -0.01270630277683693 :   (b('dg0_v_vv') <= 1.2682695388793945) ?  (b('dg0_v_vv') <= 0.747600793838501) ?  0.031146903788234353 :   0.09029950142698503 :   (b('l8b5') <= 4011.6566162109375) ?  -0.023622835928396937 :   0.016249113184319094 :   (b('l8b4') <= 716.015625) ?  (b('ndvi') <= 4753.5) ?  (b('ndvi_med') <= 2107.5) ?  -0.019136358865340458 :   -0.014976881882561466 :   (b('dg0_v_vv') <= 1.621326208114624) ?  (b('l8b5_med') <= 3116.2115478515625) ?  0.0026728298238477 :   0.007003071263859434 :   (b('l8b5_med') <= 3116.2115478515625) ?  (b('l8b5_med') <= 3042.8221435546875) ?  -0.005566007247101032 :   0.0006119684493391259 :   -0.021529882597082522 :   (b('dg0_v_vv') <= -0.3736557960510254) ?  -0.04103124525244092 :   (b('dg0_v_vv') <= 0.23666810989379883) ?  -0.030047397279207383 :   (b('dg0_v_vv') <= 0.8905301094055176) ?  -0.012818542100665908 :   (b('l8b11') <= 1476.2977905273438) ?  -0.02407288834289778 :   -0.027792556781145145 :   (b('l8b4') <= 1072.9898071289062) ?  (b('ndvi') <= 3485.5) ?  (b('ndvi') <= 2011.0) ?  (b('dg0_v_vv') <= -2.439450740814209) ?  -0.010174860764415677 :   (b('l8b11') <= 1271.5015869140625) ?  (b('l8b5') <= 2681.1488647460938) ?  0.01051570084806619 :   0.009822217390445975 :   (b('ndvi_med') <= 3351.5) ?  0.07302402024420525 :   0.048734961970696475 :   (b('l8b5') <= 4243.71337890625) ?  (b('l8b11') <= 1668.3439331054688) ?  (b('ndvi') <= 2519.0) ?  -0.014657849725080531 :   -0.0011927047593026862 :   (b('l8b5') <= 2454.569091796875) ?  0.08098714292103304 :   0.006962035843948485 :   (b('l8b11_med') <= 1177.0497436523438) ?  (b('l8b4') <= 498.0883026123047) ?  -0.07033690798095724 :   -0.0439368277546153 :   -0.04047341355994663 :   (b('crops') <= 88.03737258911133) ?  (b('ndvi') <= 4194.0) ?  (b('l8b11') <= 1541.2686767578125) ?  (b('l8b11') <= 1324.0609741210938) ?  0.01932272988983872 :   0.047459592321923205 :   (b('crops') <= 39.90002250671387) ?  -0.04217872186098899 :   0.014583655777565015 :   (b('l8b5_med') <= 4270.345947265625) ?  (b('l8b11') <= 1621.003662109375) ?  0.006557391538347537 :   -0.028793741324348383 :   -0.07981868103387599 :   (b('l8b5') <= 3888.8359375) ?  (b('l8b5') <= 3373.61669921875) ?  (b('ndvi') <= 4013.0) ?  -0.006965921739383487 :   0.015099117093171122 :   (b('dg0_v_vv') <= 1.1634464263916016) ?  -0.03129903227232916 :   -0.06759678043115284 :   0.09042410347479496 :   (b('dg0_v_vv') <= 2.304405689239502) ?  (b('dg0_v_vv') <= 1.6300745010375977) ?  (b('dg0_v_vv') <= 1.290283203125) ?  (b('dg0_v_vv') <= -0.31142139434814453) ?  (b('ndvi_med') <= 3351.25) ?  -0.021870051516046013 :   -0.03878087938759972 :   (b('dg0_v_vv') <= 0.6475296020507812) ?  -0.012795085794263616 :   -0.03271339588036877 :   (b('dg0_v_vv') <= 1.391688346862793) ?  -0.06338489269419112 :   -0.08671364011941368 :   (b('l8b4_med') <= 813.8113098144531) ?  0.03149277045566934 :   0.05619878385914759 :   -0.09014427073869383 :   (b('dg0_v_vv') <= -1.4083375930786133) ?  (b('ndvi_med') <= 3441.0) ?  (b('l8b5') <= 3799.8551025390625) ?  (b('l8b5') <= 2858.9813232421875) ?  (b('l8b11') <= 1321.1014404296875) ?  -0.04522146431498897 :   (b('dg0_v_vv') <= -1.7615375518798828) ?  (b('dg0_v_vv') <= -2.4447784423828125) ?  -0.019843267148310806 :   -0.034621598335958216 :   (b('l8b11_med') <= 1450.3226928710938) ?  -0.022146691327484286 :   -0.023144334664395555 :   (b('l8b11_med') <= 1555.286865234375) ?  (b('l8b4') <= 564.1029357910156) ?  -0.026352676642001194 :   -0.021910722060622695 :   (b('dg0_v_vv') <= -2.8683223724365234) ?  -0.013826175724748008 :   -0.016639241638618518 :   0.025366325920672173 :   (b('dg0_v_vv') <= -1.6421775817871094) ?  (b('l8b11') <= 1193.7508544921875) ?  (b('sand') <= 19.0) ?  (b('l8b11_med') <= 1384.3737182617188) ?  (b('ndvi_med') <= 4372.5) ?  0.02981656196793847 :   0.026934753925248633 :   0.011061668871885533 :   (b('dg0_v_vv') <= -2.8039493560791016) ?  -0.02325192034140844 :   (b('ndvi_med') <= 3984.5) ?  -0.005669938055603341 :   0.001865792158451951 :   (b('l8b4') <= 753.3622741699219) ?  (b('l8b5_med') <= 3503.7686767578125) ?  0.05939471016278042 :   0.055705809547142326 :   (b('dg0_v_vv') <= -2.458494186401367) ?  (b('dg0_v_vv') <= -2.7529287338256836) ?  -0.0030928664993545574 :   -0.0014991186190746186 :   (b('l8b4') <= 841.1300048828125) ?  0.016688030485736555 :   0.04222514688543642 :   (b('ndvi_med') <= 3599.5) ?  0.010539380070854448 :   (b('ndvi') <= 4845.5) ?  (b('sand') <= 18.0) ?  -0.03163748785993442 :   -0.029229748509468828 :   -0.04306844225539 :   (b('l8b11') <= 1196.1502075195312) ?  (b('ndvi') <= 4604.5) ?  (b('l8b11') <= 1130.8359375) ?  (b('ndvi') <= 4536.0) ?  (b('dg0_v_vv') <= -1.2081375122070312) ?  0.08826676499941533 :   (b('dg0_v_vv') <= -1.0115714073181152) ?  -0.03544185889120374 :   0.021862922395833546 :   (b('ndvi_med') <= 4083.75) ?  -0.0850097486589276 :   -0.035573609390001704 :   (b('bulk') <= 152.5) ?  (b('l8b11') <= 1142.2692260742188) ?  (b('l8b4_med') <= 815.4116516113281) ?  -0.06352110022050214 :   -0.036514713149676636 :   (b('ndvi') <= 4329.0) ?  -0.014917785039768401 :   -0.0520399265015374 :   0.08335363680656793 :   (b('l8b5') <= 3562.8807373046875) ?  (b('l8b5') <= 3180.100830078125) ?  (b('dg0_v_vv') <= -0.011753082275390625) ?  (b('l8b5_med') <= 3204.0439453125) ?  -0.01706703707628547 :   0.05044264848801788 :   (b('moss') <= 0.689393937587738) ?  0.0548099819274076 :   0.10441345175881059 :   (b('l8b11') <= 1136.2677001953125) ?  (b('l8b11') <= 1057.879150390625) ?  -0.04860840860090407 :   -0.028409060342046946 :   (b('sand') <= 21.0) ?  0.0515498435688454 :   -0.010960105359258066 :   (b('ndvi') <= 4764.5) ?  (b('l8b5') <= 4085.30078125) ?  (b('l8b11') <= 1075.3344421386719) ?  0.11663382548504198 :   0.09761378843004404 :   (b('l8b5') <= 4212.12451171875) ?  0.08833506847509964 :   0.07470480282413064 :   (b('l8b5') <= 3647.289306640625) ?  -0.019311424915618047 :   (b('ndvi') <= 4917.0) ?  0.029701230126335798 :   0.040819170028663344 :   (b('l8b5_med') <= 3062.1453857421875) ?  (b('l8b5') <= 2702.6593017578125) ?  (b('l8b4') <= 874.1869812011719) ?  (b('l8b4') <= 817.6692199707031) ?  (b('l8b4') <= 704.8317565917969) ?  0.005821283119643748 :   0.04407670291307279 :   (b('dg0_v_vv') <= 0.08425569534301758) ?  0.026487455524011123 :   -0.056584676299137805 :   (b('l8b4_med') <= 788.0063781738281) ?  (b('ndvi') <= 3334.0) ?  0.04111431081507677 :   -0.007008026238582099 :   (b('ndvi') <= 2783.5) ?  0.06485147976265353 :   0.10577200645549427 :   (b('l8b11_med') <= 1512.6978759765625) ?  (b('dg0_v_vv') <= -1.2474455833435059) ?  -0.05455938645653158 :   (b('l8b11_med') <= 1244.5859985351562) ?  0.020565663308521 :   0.07274132674059718 :   (b('l8b11') <= 1459.100830078125) ?  (b('ndvi') <= 4117.5) ?  -0.033963562178162904 :   0.027610667795194038 :   (b('l8b5') <= 2807.3343505859375) ?  0.06907337458829228 :   0.03739380882565742 :   (b('l8b11') <= 1216.2164306640625) ?  (b('l8b4_med') <= 751.0149536132812) ?  (b('ndvi') <= 4755.0) ?  (b('l8b11_med') <= 1201.6956176757812) ?  0.023650822323759424 :   0.03584727911799962 :   0.09254133980705836 :   0.11136458985791792 :   (b('l8b4') <= 995.6953125) ?  (b('dg0_v_vv') <= 0.36162424087524414) ?  (b('l8b11') <= 1619.8176879882812) ?  -0.014486535587036201 :   -0.07005402306210648 :   (b('dg0_v_vv') <= 1.0870962142944336) ?  0.0463865847543896 :   -0.006022457548414445 :   (b('ndvi') <= 1690.0) ?  -0.04505627169033233 :   (b('dg0_v_vv') <= -0.4635605812072754) ?  0.08798849277371501 :   0.0345530080633789 :   (b('l8b4_med') <= 573.2518310546875) ?  (b('bulk') <= 129.5) ?  (b('sand') <= 48.0) ?  (b('crops') <= 43.43474578857422) ?  (b('l8b11') <= 1146.9508666992188) ?  (b('l8b4_med') <= 509.54493713378906) ?  (b('ndvi') <= 3408.0) ?  0.06606938468174645 :   (b('l8b11') <= 1094.9732055664062) ?  -0.029903353167726493 :   -0.004589835524814262 :   (b('l8b4') <= 424.4156494140625) ?  (b('l8b5_med') <= 3019.57470703125) ?  0.027418072318867115 :   -0.01641645588771315 :   (b('l8b5_med') <= 3153.1622314453125) ?  -0.04987498170364363 :   -0.029461806096759607 :   (b('l8b11') <= 1496.9488525390625) ?  (b('dg0_v_vv') <= 2.395009994506836) ?  (b('ndvi') <= 3581.0) ?  0.01829863943366001 :   -0.021928297665431373 :   (b('l8b11_med') <= 1302.6699829101562) ?  0.037995140573735514 :   0.04988011211304655 :   0.06437154237318723 :   (b('ndvi') <= 3859.5) ?  (b('grass') <= 25.54545497894287) ?  (b('dg0_v_vv') <= 0.38274049758911133) ?  0.07377531960446493 :   (b('l8b11_med') <= 1032.7852478027344) ?  0.03502827265408323 :   0.032899490449758595 :   (b('dg0_v_vv') <= 0.011409759521484375) ?  (b('ndvi_med') <= 4314.0) ?  -0.02111998021024076 :   -0.040654772181328666 :   (b('l8b4') <= 550.9542846679688) ?  -0.06764225451520772 :   -0.044639055599642985 :   (b('dg0_v_vv') <= 1.3471789360046387) ?  (b('l8b5') <= 3974.00439453125) ?  (b('ndvi') <= 4064.5) ?  0.06593902128632312 :   -0.008176459927723633 :   (b('l8b5') <= 4015.3310546875) ?  0.08696404410937994 :   0.026434998198340692 :   (b('l8b4') <= 654.1832580566406) ?  (b('l8b5') <= 4147.10205078125) ?  0.06906737257924836 :   0.05859744174597098 :   0.11280189666397294 :   (b('sand') <= 53.5) ?  (b('l8b4') <= 656.3120422363281) ?  (b('ndvi') <= 2965.0) ?  (b('dg0_v_vv') <= 1.7639813423156738) ?  (b('dg0_v_vv') <= -1.3855652809143066) ?  0.025687182951366683 :   0.03779272226670682 :   0.01456987050499775 :   (b('l8b11') <= 1288.7476806640625) ?  (b('ndvi') <= 4351.0) ?  0.01802563353129685 :   0.02609881976432878 :   (b('ndvi') <= 4277.0) ?  0.025807165588484164 :   0.037802773139526395 :   (b('grass') <= 52.18928527832031) ?  (b('l8b4') <= 748.1504211425781) ?  (b('ndvi_med') <= 3208.25) ?  0.02650184664851829 :   0.018258894703392165 :   (b('dg0_v_vv') <= 1.2784056663513184) ?  0.025369691567717973 :   0.04098832035580041 :   (b('moss') <= 0.7690265774726868) ?  (b('dg0_v_vv') <= 0.4754185676574707) ?  0.023052297074766304 :   0.025380498017603115 :   (b('dg0_v_vv') <= -0.3954935073852539) ?  0.011979253256652696 :   0.016282719912460285 :   (b('l8b11') <= 942.8660888671875) ?  (b('l8b5_med') <= 3475.7735595703125) ?  (b('ndvi_med') <= 3709.25) ?  (b('ndvi') <= 3909.0) ?  0.027145779329440563 :   -0.0012797505578328815 :   (b('dg0_v_vv') <= -1.2121000289916992) ?  -0.027471055276087453 :   0.04917717249920826 :   -0.03958537612463964 :   (b('l8b5') <= 3740.3529052734375) ?  (b('l8b5_med') <= 3613.2467041015625) ?  (b('l8b5') <= 2085.4517822265625) ?  -0.00805439744108627 :   0.012281836405142842 :   (b('ndvi') <= 4333.5) ?  -0.026812195316494168 :   -0.05718776676309246 :   (b('crops') <= 40.855512619018555) ?  (b('dg0_v_vv') <= -0.18198680877685547) ?  0.004139290559732202 :   0.04487367446218268 :   (b('l8b5') <= 3872.4345703125) ?  0.036756599456710254 :   0.05386535848048461 :   (b('ndvi') <= 4270.5) ?  (b('l8b11') <= 768.5492553710938) ?  (b('l8b11') <= 749.7220458984375) ?  (b('ndvi') <= 3201.5) ?  (b('l8b11') <= 657.3435363769531) ?  (b('crops') <= 7.7792205810546875) ?  0.014042794320253932 :   0.01805823145354138 :   (b('dg0_v_vv') <= -0.6952145099639893) ?  0.034182689495225904 :   0.03531001657299959 :   (b('ndvi') <= 3831.0) ?  (b('l8b5') <= 2611.064697265625) ?  -0.04739133228457223 :   -0.013884589773659056 :   (b('l8b4') <= 209.80062103271484) ?  -0.029111914731478766 :   0.006972400209384548 :   (b('ndvi') <= 3944.0) ?  (b('l8b5_med') <= 2913.7916259765625) ?  (b('dg0_v_vv') <= 0.7199225425720215) ?  -0.06099433813872315 :   -0.05171256188188428 :   -0.07785207513227871 :   0.0019180234852164413 :   (b('l8b4') <= 367.22666931152344) ?  (b('sand') <= 71.0) ?  (b('ndvi_med') <= 3528.25) ?  -0.025522658357871653 :   0.01047734754298818 :   (b('ndvi_med') <= 3585.0) ?  (b('ndvi_med') <= 3243.5) ?  -0.030183693155212188 :   -0.03337798922856561 :   (b('dg0_v_vv') <= -0.17918729782104492) ?  -0.037377784730988156 :   -0.06881991216114455 :   (b('l8b5') <= 2619.105712890625) ?  (b('l8b4') <= 506.3127746582031) ?  (b('l8b5') <= 2576.7401123046875) ?  0.02790750109505767 :   -0.016198013096023256 :   (b('moss') <= 4.324254751205444) ?  0.07602859796999886 :   -0.009558082094496755 :   (b('l8b5') <= 2932.7662353515625) ?  (b('ndvi') <= 2986.0) ?  0.04748026402803897 :   0.01536257276676166 :   (b('l8b5') <= 3598.495361328125) ?  0.0008969880159130925 :   0.019000133269403256 :   (b('l8b5') <= 3149.123046875) ?  (b('moss') <= 4.324254751205444) ?  (b('ndvi_med') <= 3971.0) ?  (b('dg0_v_vv') <= 0.6520471572875977) ?  -0.10608212435360302 :   (b('dg0_v_vv') <= 0.8997001647949219) ?  -0.08341728651321338 :   -0.09818373454775574 :   (b('crops') <= 17.43150043487549) ?  -0.07453760821586455 :   -0.07323284817890957 :   (b('l8b4_med') <= 556.3644104003906) ?  (b('dg0_v_vv') <= -0.6926381587982178) ?  (b('dg0_v_vv') <= -0.8951733112335205) ?  -0.022331886649934136 :   0.04121788006177144 :   (b('dg0_v_vv') <= -0.029662609100341797) ?  -0.032424744520535564 :   -0.0063911170963081776 :   (b('ndvi_med') <= 3619.0) ?  0.029309393996216238 :   (b('l8b4') <= 665.4379577636719) ?  0.003243231980786826 :   0.013840232900552682 :   (b('ndvi_med') <= 4530.5) ?  (b('ndvi') <= 4365.0) ?  (b('l8b4') <= 509.2784729003906) ?  (b('dg0_v_vv') <= -1.7000706195831299) ?  0.005328650925988571 :   -0.018161798513928153 :   (b('l8b11') <= 1147.53076171875) ?  -0.09224521998281282 :   -0.07822730928354604 :   (b('l8b5_med') <= 3672.357666015625) ?  (b('l8b4') <= 506.29444885253906) ?  -0.010128077593975382 :   0.010811328475207682 :   (b('l8b5') <= 3782.983642578125) ?  0.02658064685768784 :   0.0768134989540562 :   0.09669184742289633 :   (b('l8b4_med') <= 964.645263671875) ?  (b('grass') <= 13.242010593414307) ?  (b('ndvi') <= 3611.5) ?  (b('ndvi') <= 2421.0) ?  (b('ndvi') <= 1139.5) ?  (b('l8b11') <= 1300.5352172851562) ?  0.031115127423876843 :   (b('dg0_v_vv') <= 0.21320676803588867) ?  -0.05072836762289932 :   -0.03831954367172867 :   (b('dg0_v_vv') <= -1.1796064376831055) ?  (b('l8b5') <= 2524.53662109375) ?  -0.06545353963880615 :   0.014130442610992004 :   (b('l8b5') <= 2840.408935546875) ?  0.01960348455498054 :   0.0481063182911578 :   (b('l8b4') <= 931.1634521484375) ?  (b('l8b4') <= 458.7606506347656) ?  (b('l8b4') <= 412.75189208984375) ?  -0.01381810611283342 :   -0.06814233968152364 :   (b('dg0_v_vv') <= 0.1601696014404297) ?  -0.007193356294378531 :   0.010399605369158306 :   (b('crops') <= 81.24092102050781) ?  (b('l8b11_med') <= 1322.916748046875) ?  -0.05694189786317705 :   -0.05034876761196258 :   (b('l8b4_med') <= 837.4482421875) ?  -0.04008434119688681 :   -0.038562317506962845 :   (b('ndvi') <= 4207.5) ?  (b('ndvi_med') <= 3966.0) ?  (b('ndvi') <= 3783.0) ?  (b('l8b5') <= 2853.9014892578125) ?  0.008553888855522246 :   0.04538959931192132 :   (b('ndvi') <= 4068.5) ?  0.008099608197575327 :   0.030477837595169126 :   (b('dg0_v_vv') <= -1.6890454292297363) ?  (b('l8b5_med') <= 3413.9039306640625) ?  0.0925362610728056 :   0.08539098241482213 :   (b('l8b11_med') <= 1406.812744140625) ?  0.05679577334152513 :   0.019555378726866635 :   (b('sand') <= 70.0) ?  (b('l8b11') <= 844.2116088867188) ?  (b('sand') <= 31.5) ?  -0.0016482110931492766 :   -0.11056663785967064 :   (b('dg0_v_vv') <= 0.6858634948730469) ?  0.013112048826246737 :   0.038767037557118286 :   (b('l8b5') <= 4904.663818359375) ?  (b('l8b11_med') <= 1142.1994018554688) ?  0.011891436190412857 :   -0.0087958700006816 :   (b('dg0_v_vv') <= 2.5198845863342285) ?  0.013838528429588541 :   0.04723060259506509 :   (b('l8b11_med') <= 1589.2805786132812) ?  (b('dg0_v_vv') <= 1.621504306793213) ?  (b('l8b5') <= 2486.5006103515625) ?  (b('dg0_v_vv') <= -0.5705971717834473) ?  (b('bulk') <= 130.0) ?  0.005695689998351308 :   -0.008124909685907513 :   (b('l8b4') <= 434.0062713623047) ?  0.03454110297631906 :   0.00549225753736371 :   (b('l8b4') <= 523.8751525878906) ?  (b('l8b5') <= 2771.8109130859375) ?  0.029935590129940425 :   0.0009266112047815031 :   (b('l8b11') <= 1521.391357421875) ?  -0.008151204290732378 :   0.00212233736682791 :   (b('ndvi_med') <= 3279.5) ?  (b('l8b11') <= 1090.887939453125) ?  (b('l8b11_med') <= 1102.9674682617188) ?  0.005001251520017913 :   -0.028168259663727466 :   (b('l8b11') <= 1176.006591796875) ?  0.03884975385289903 :   0.0015262025324529345 :   (b('bulk') <= 117.5) ?  (b('l8b11') <= 1566.3416137695312) ?  0.052560093631245565 :   0.1295855260594589 :   (b('ndvi') <= 4621.0) ?  0.02129801993357047 :   -0.019215310196588672 :   (b('l8b5_med') <= 2938.6622314453125) ?  (b('l8b4') <= 804.3546142578125) ?  (b('l8b4') <= 702.3105773925781) ?  (b('ndvi') <= 2392.0) ?  0.0055252279709818285 :   -0.0212845083848287 :   (b('l8b11') <= 1329.7371826171875) ?  0.045723513050191 :   -0.0029671830000021666 :   (b('sand') <= 75.5) ?  (b('dg0_v_vv') <= 1.4797046184539795) ?  -0.01825216457534269 :   -0.04641750851199075 :   (b('dg0_v_vv') <= 0.3502926826477051) ?  0.047045818879046714 :   0.028734204385470113 :   (b('l8b5') <= 3575.9439697265625) ?  (b('l8b5') <= 3410.396484375) ?  (b('l8b5') <= 3400.9571533203125) ?  -0.0015205776060941785 :   0.11233728502522652 :   (b('l8b11') <= 1518.09716796875) ?  -0.032841462203280096 :   0.011344635755864483 :   (b('l8b5') <= 4025.9600830078125) ?  (b('dg0_v_vv') <= -0.3468050956726074) ?  -0.003079044442064425 :   0.03252471437493132 :   (b('dg0_v_vv') <= 0.991649866104126) ?  -0.03944878789301413 :   -0.0030116228350846944 :   (b('ndvi') <= 1341.5) ?  (b('sand') <= 56.5) ?  (b('l8b11') <= 1563.9279174804688) ?  (b('l8b11') <= 1540.5313110351562) ?  (b('ndvi') <= 1016.5) ?  (b('l8b5') <= 1817.7178344726562) ?  0.0015743717691629661 :   0.03373931540424533 :   (b('ndvi') <= 1209.5) ?  -0.026988285979669673 :   -0.002838718699587577 :   (b('l8b4') <= 943.0905456542969) ?  0.036127979559217094 :   0.0855316518491715 :   (b('moss') <= 31.044836044311523) ?  (b('grass') <= 84.51411056518555) ?  (b('dg0_v_vv') <= -0.6450796127319336) ?  -0.011792624011460051 :   -0.03448621954934456 :   (b('dg0_v_vv') <= 1.6692166328430176) ?  -0.0006656869352479888 :   0.03154994192362415 :   -0.09623009643471853 :   (b('ndvi') <= 723.0) ?  (b('l8b11') <= 1449.1946411132812) ?  (b('dg0_v_vv') <= 2.167375326156616) ?  (b('dg0_v_vv') <= 1.534289836883545) ?  -0.040196220330214236 :   0.007311384481060897 :   (b('l8b5') <= 1419.8695678710938) ?  -0.06850854902754691 :   -0.05139064690744473 :   0.04885027378335882 :   (b('l8b4') <= 1096.3070068359375) ?  (b('ndvi') <= 763.5) ?  (b('l8b4') <= 1073.253173828125) ?  0.11169053949761279 :   0.09379739154532571 :   (b('dg0_v_vv') <= 0.1920609474182129) ?  0.036310735975547 :   0.09359070046803306 :   (b('l8b4') <= 1217.902099609375) ?  (b('l8b4_med') <= 1207.024169921875) ?  -0.02660247903328421 :   0.007848494205810672 :   (b('ndvi') <= 1050.5) ?  0.02068222596546555 :   0.0065182329242278346 :   (b('crops') <= 0.6177893579006195) ?  (b('grass') <= 35.37341499328613) ?  (b('l8b4') <= 821.0223999023438) ?  -0.09690666677721335 :   (b('ndvi') <= 1455.5) ?  (b('dg0_v_vv') <= 1.5607707500457764) ?  0.1473466695421275 :   0.042551041690469804 :   (b('l8b5') <= 2248.4442138671875) ?  0.0015745191523341348 :   0.0728974980697798 :   (b('l8b4') <= 728.50341796875) ?  (b('l8b11') <= 1180.2225952148438) ?  (b('l8b4') <= 653.4145812988281) ?  0.021512849251932167 :   -0.021559356586399993 :   (b('l8b11_med') <= 1865.8610229492188) ?  0.1063339774425432 :   0.037751838434388764 :   (b('ndvi_med') <= 1794.0) ?  (b('l8b4') <= 1253.0075073242188) ?  -0.008033751255332847 :   0.05239862887720697 :   (b('l8b4') <= 1122.1265258789062) ?  0.02025002744128444 :   0.08782067126878716 :   (b('l8b5') <= 3843.8685302734375) ?  (b('dg0_v_vv') <= 0.8153209686279297) ?  (b('dg0_v_vv') <= 0.3683586120605469) ?  (b('l8b4') <= 1407.0) ?  -0.00016758098596172821 :   0.046234302098218556 :   (b('l8b5') <= 3367.8485107421875) ?  -0.011338806245365885 :   -0.05144887466397456 :   (b('grass') <= 11.959652423858643) ?  (b('dg0_v_vv') <= 3.44626784324646) ?  -0.0011010723170799305 :   0.023357393140188507 :   (b('grass') <= 27.78272247314453) ?  0.04703953267022363 :   0.008149724121998663 :   (b('l8b4') <= 821.2184143066406) ?  (b('crops') <= 81.98492813110352) ?  (b('ndvi') <= 2533.0) ?  0.03447582337300401 :   -0.007860893404548982 :   (b('l8b5_med') <= 3102.9317626953125) ?  0.021612597640075705 :   0.04665180165968184 :   (b('l8b11_med') <= 1618.6771850585938) ?  (b('l8b5_med') <= 3257.319091796875) ?  -0.0717185044041257 :   0.00353732161920896 :   (b('l8b11_med') <= 2702.9324951171875) ?  0.05126524555458805 :   -0.06952716411911534 :   (b('l8b11_med') <= 1753.7033081054688) ?  (b('moss') <= 7.291277885437012) ?  (b('dg0_v_vv') <= -1.129772663116455) ?  (b('sand') <= 23.5) ?  (b('l8b11') <= 2080.1585693359375) ?  (b('ndvi_med') <= 3533.0) ?  (b('dg0_v_vv') <= -2.674807071685791) ?  (b('ndvi_med') <= 3391.25) ?  -0.02187602458858981 :   0.009296791232511176 :   (b('l8b5_med') <= 2847.531494140625) ?  0.05721148032618373 :   (b('l8b4_med') <= 813.8113098144531) ?  0.04456862232775213 :   0.02141689415878177 :   (b('dg0_v_vv') <= -3.776528835296631) ?  0.0048607055203137295 :   (b('l8b4') <= 1092.0243530273438) ?  -0.022591417298254743 :   -0.03360091691625225 :   (b('l8b5') <= 2743.6575927734375) ?  (b('ndvi') <= 1999.5) ?  0.044470452140661004 :   0.03644351684486857 :   (b('l8b5_med') <= 2631.0662841796875) ?  0.06636380040152257 :   (b('dg0_v_vv') <= -2.1477184295654297) ?  0.04269024198108112 :   (b('moss') <= 3.2715134620666504) ?  0.0632422780996712 :   0.05254154221223628 :   (b('grass') <= 26.958024978637695) ?  (b('l8b5') <= 2918.678466796875) ?  (b('dg0_v_vv') <= -2.0878801345825195) ?  (b('crops') <= 65.33827018737793) ?  (b('ndvi') <= 2139.5) ?  -0.04523462229995407 :   -0.02240071131333049 :   (b('sand') <= 68.0) ?  -0.009403890677928024 :   0.012045324621182098 :   (b('l8b11') <= 2093.798095703125) ?  (b('l8b5') <= 2804.4840087890625) ?  -0.013921783099761432 :   0.009743064438814005 :   (b('l8b11') <= 2135.4705810546875) ?  0.1406375033792485 :   0.0123760260718752 :   (b('ndvi_med') <= 3574.5) ?  (b('l8b4') <= 1113.4846801757812) ?  (b('l8b5_med') <= 2792.08935546875) ?  0.012209398064587124 :   0.03263786281380699 :   (b('crops') <= 48.65524101257324) ?  0.06220623796743788 :   0.0063413271225083966 :   (b('dg0_v_vv') <= -1.525918960571289) ?  (b('dg0_v_vv') <= -2.175130844116211) ?  -0.019694796560947223 :   -0.04200014173665195 :   0.0028753589954769454 :   (b('l8b5') <= 2833.3509521484375) ?  (b('ndvi_med') <= 3519.5) ?  (b('ndvi') <= 2459.0) ?  (b('l8b5') <= 2566.094970703125) ?  -0.05919877214171267 :   -0.02295620421105553 :   (b('l8b11_med') <= 1651.8829956054688) ?  0.008243239260073074 :   -0.021529847354537807 :   (b('dg0_v_vv') <= -3.051115036010742) ?  (b('ndvi') <= 3240.0) ?  -0.05059623441681851 :   0.0001503809833471384 :   (b('bulk') <= 135.5) ?  0.0032163190133724096 :   0.0598509168904223 :   (b('ndvi') <= 4848.5) ?  (b('sand') <= 51.0) ?  (b('grass') <= 29.106029510498047) ?  -0.007476337266897934 :   -0.04390232758048485 :   -0.08668767127921825 :   0.0160917228107324 :   (b('sand') <= 71.5) ?  (b('ndvi') <= 4620.5) ?  (b('ndvi_med') <= 4322.75) ?  (b('l8b5_med') <= 2736.3609619140625) ?  (b('l8b11') <= 1727.5032348632812) ?  -0.07832338935324432 :   (b('dg0_v_vv') <= -1.0748238563537598) ?  0.11279877472581072 :   0.022942121586482633 :   (b('ndvi') <= 2184.0) ?  (b('moss') <= 0.08091127313673496) ?  0.026801611109542687 :   -0.0061425766480988075 :   (b('ndvi') <= 2199.5) ?  0.07355210062087898 :   0.012817438776849334 :   (b('l8b5') <= 2906.1666259765625) ?  (b('l8b11_med') <= 1340.8568725585938) ?  (b('l8b5_med') <= 3465.753662109375) ?  0.0808734598526456 :   0.07617770701201387 :   0.09459502433889005 :   (b('l8b4_med') <= 588.2550354003906) ?  0.013553548297357582 :   (b('l8b4_med') <= 632.78857421875) ?  0.037740614875390543 :   0.039831612367967895 :   (b('ndvi_med') <= 2416.25) ?  0.20612886694464644 :   (b('l8b11') <= 1873.6784057617188) ?  (b('dg0_v_vv') <= 1.564549446105957) ?  (b('crops') <= 36.2738698720932) ?  0.16138992147881387 :   0.10249792541376679 :   -0.035992854526938456 :   (b('dg0_v_vv') <= -0.25727415084838867) ?  (b('moss') <= 0.7968254089355469) ?  0.03306609207225478 :   -0.008022167305855431 :   (b('dg0_v_vv') <= 0.5985069274902344) ?  0.033625472467605055 :   0.013796621321529905 :   (b('l8b5') <= 3174.9317626953125) ?  (b('l8b4') <= 631.6780090332031) ?  (b('dg0_v_vv') <= 1.56669020652771) ?  -0.044971713465204954 :   -0.046273196705205355 :   (b('ndvi') <= 4369.0) ?  (b('l8b4_med') <= 885.5189514160156) ?  (b('l8b4_med') <= 750.9954833984375) ?  -0.0013078985467679456 :   0.03636786151501349 :   (b('l8b4_med') <= 955.3427429199219) ?  -0.02039348570384214 :   -0.03973569871314908 :   (b('bulk') <= 128.5) ?  (b('crops') <= 71.40778923034668) ?  -0.005961261933814482 :   0.022076012967861366 :   (b('l8b11') <= 2326.2445678710938) ?  -0.06916528599106792 :   -0.03565192613411737 :   -0.07154782675073677 :   (b('moss') <= 11.764912605285645) ?  (b('ndvi') <= 3764.0) ?  (b('l8b11') <= 1779.0128784179688) ?  (b('l8b4') <= 1365.6510009765625) ?  (b('l8b4_med') <= 713.2743835449219) ?  (b('grass') <= 21.344338417053223) ?  -0.04370084341012287 :   (b('dg0_v_vv') <= 0.545870304107666) ?  -0.03480277455869393 :   -0.0393363437184298 :   (b('l8b5_med') <= 2319.6075439453125) ?  (b('ndvi') <= 1909.0) ?  0.005715286079957305 :   0.02299495880032809 :   (b('l8b5_med') <= 2915.0546875) ?  -0.029156352494995076 :   -0.014284608605050856 :   (b('dg0_v_vv') <= -0.1594243049621582) ?  (b('l8b4_med') <= 1220.1637878417969) ?  -0.02700904656273033 :   -0.02893318215015031 :   (b('bulk') <= 129.5) ?  -0.03615998788259327 :   (b('l8b5_med') <= 2347.46337890625) ?  -0.04914216552375806 :   -0.05580457261267751 :   (b('l8b5_med') <= 2956.10791015625) ?  (b('l8b5') <= 2568.8829345703125) ?  (b('dg0_v_vv') <= 1.0030250549316406) ?  (b('l8b11') <= 2302.2078857421875) ?  -0.004767737285369291 :   0.031839343485050076 :   (b('ndvi') <= 2661.5) ?  -0.0423489293516588 :   0.015228174927925935 :   (b('dg0_v_vv') <= 0.5196661949157715) ?  (b('moss') <= 8.112590789794922) ?  -0.03884027154366383 :   -0.01851441699680447 :   (b('l8b5_med') <= 2943.271484375) ?  -0.01725842236629449 :   0.023780813936693294 :   (b('dg0_v_vv') <= 0.16573429107666016) ?  (b('l8b5') <= 2937.93701171875) ?  (b('l8b11') <= 1939.8637084960938) ?  -0.019158372030682994 :   0.013304940409974784 :   (b('ndvi') <= 2296.0) ?  0.05214336385661594 :   0.027476123418529695 :   (b('l8b5') <= 2867.4825439453125) ?  (b('l8b5') <= 2536.417236328125) ?  -0.047029360530076225 :   -0.02228412210388155 :   (b('l8b5_med') <= 2956.624755859375) ?  -0.023272581026195026 :   0.003502675298164684 :   (b('ndvi') <= 4326.5) ?  (b('l8b5') <= 3422.661376953125) ?  (b('l8b5_med') <= 2915.0546875) ?  (b('dg0_v_vv') <= 0.8261985778808594) ?  (b('ndvi_med') <= 3090.0) ?  0.0014058887130316763 :   0.010403027838910214 :   (b('dg0_v_vv') <= 0.9253978729248047) ?  -0.03338469302466056 :   -0.018251466593409607 :   (b('ndvi_med') <= 2507.0) ?  0.06074870150592156 :   (b('l8b4_med') <= 892.6100769042969) ?  0.02164020671776414 :   -0.013642250987018378 :   (b('ndvi_med') <= 3082.5) ?  0.09053587292640872 :   (b('l8b5_med') <= 2956.10791015625) ?  (b('ndvi') <= 4121.0) ?  0.05596597680430998 :   0.06204382036210865 :   (b('l8b5_med') <= 2956.624755859375) ?  0.0735023877577331 :   0.06847309897503179 :   (b('grass') <= 35.551198959350586) ?  (b('sand') <= 32.5) ?  (b('dg0_v_vv') <= 1.9392094612121582) ?  (b('l8b5_med') <= 2943.271484375) ?  0.024982780935629195 :   0.040304396486151806 :   -0.002235176102076991 :   (b('dg0_v_vv') <= -0.2162938117980957) ?  (b('ndvi_med') <= 3836.0) ?  -0.011899011592129414 :   0.014452111108387511 :   (b('l8b11_med') <= 1190.9813232421875) ?  -0.05061363010871944 :   -0.011712128734187076 :   -0.06813924204010582 :   (b('l8b4') <= 787.0558776855469) ?  (b('ndvi') <= 2336.0) ?  (b('l8b5_med') <= 2144.4132080078125) ?  0.09262169167792392 :   0.07463757401695437 :   0.11944641556853178 :   (b('moss') <= 13.799630165100098) ?  (b('l8b5') <= 2392.0120849609375) ?  (b('l8b4') <= 1092.6719970703125) ?  (b('l8b4') <= 973.6689758300781) ?  (b('ndvi_med') <= 2268.75) ?  0.0383997682614543 :   -0.01631901405179594 :   (b('ndvi') <= 1570.0) ?  0.011494173267876634 :   0.06334904976472541 :   (b('dg0_v_vv') <= 0.1923532485961914) ?  (b('dg0_v_vv') <= -0.20845556259155273) ?  -0.01619225488559721 :   -0.0008917673763995798 :   -0.05648174747084708 :   (b('l8b4') <= 1146.0474243164062) ?  (b('ndvi_med') <= 3873.5) ?  (b('l8b5_med') <= 3214.708251953125) ?  -0.002860107376585025 :   -0.15758997791112037 :   0.11966485392106113 :   (b('bulk') <= 132.5) ?  (b('dg0_v_vv') <= -0.5522952079772949) ?  0.018127599106823628 :   0.09958330164848513 :   (b('grass') <= 80.40078353881836) ?  0.013314531425126164 :   0.05883118087376101 :   (b('l8b5') <= 3135.6361083984375) ?  (b('l8b4') <= 1013.3097229003906) ?  (b('dg0_v_vv') <= 1.1945304870605469) ?  (b('ndvi') <= 1183.5) ?  0.02152355333230655 :   -0.008996373539146616 :   (b('l8b4') <= 975.0993041992188) ?  -0.019261350290578492 :   -0.09353662054736216 :   (b('grass') <= 66.64040756225586) ?  (b('dg0_v_vv') <= -0.014658927917480469) ?  -0.003952166307583281 :   0.014702168770083925 :   0.1111880458164585 :   (b('l8b5') <= 3196.2076416015625) ?  (b('ndvi') <= 3553.5) ?  (b('l8b5_med') <= 3120.56201171875) ?  -0.047928652602875765 :   -0.06592865182801538 :   (b('dg0_v_vv') <= -0.49925661087036133) ?  -0.032923233461711165 :   -0.007923234951827285 :   (b('dg0_v_vv') <= -1.628894329071045) ?  (b('dg0_v_vv') <= -2.388333320617676) ?  -0.030018609935218175 :   -0.047174302803417445 :   (b('l8b5_med') <= 3257.319091796875) ?  -0.016606998525544953 :   0.0015771287610806747 :   (b('dg0_v_vv') <= 2.5798468589782715) ?  (b('grass') <= 98.81764602661133) ?  (b('ndvi') <= 3067.5) ?  (b('ndvi_med') <= 3214.0) ?  (b('l8b4_med') <= 942.7462463378906) ?  (b('l8b4_med') <= 905.7816467285156) ?  (b('crops') <= 28.516514778137207) ?  (b('ndvi') <= 2188.0) ?  -0.000881750988338606 :   -0.048417183969949144 :   (b('l8b5') <= 2292.58447265625) ?  0.00896964977582359 :   -0.0174596502986185 :   (b('l8b5') <= 2823.25927734375) ?  (b('l8b11') <= 1825.0497436523438) ?  -0.002950871314465213 :   0.04295186411608428 :   (b('ndvi') <= 2729.0) ?  0.12660674230916563 :   0.09677516776374535 :   (b('l8b4') <= 845.7236022949219) ?  (b('bulk') <= 152.0) ?  (b('dg0_v_vv') <= -0.17897605895996094) ?  0.014577700525088388 :   -0.03549174720317129 :   (b('ndvi_med') <= 1670.5) ?  0.07897230946900007 :   0.0236870291750819 :   (b('l8b4_med') <= 1004.4573364257812) ?  (b('ndvi') <= 1673.0) ?  0.003819843947066127 :   -0.026293882589122693 :   (b('l8b11') <= 2548.5556640625) ?  -0.0032523097863400357 :   -0.006632589772683249 :   (b('bulk') <= 136.0) ?  (b('dg0_v_vv') <= 0.6961178779602051) ?  (b('dg0_v_vv') <= -1.9115681648254395) ?  0.024491720777231385 :   (b('l8b11') <= 2516.8653564453125) ?  0.003220056525826083 :   -0.029132897858878598 :   0.09403599681166705 :   (b('ndvi') <= 2230.5) ?  (b('dg0_v_vv') <= -0.7091903686523438) ?  (b('ndvi') <= 1879.0) ?  -0.048958574143091335 :   -0.03721045000700958 :   (b('ndvi_med') <= 3347.75) ?  0.05292971470369805 :   0.015006704284005842 :   (b('l8b5') <= 3158.6700439453125) ?  (b('l8b4') <= 998.8574523925781) ?  -0.03527421126157905 :   -0.05069600728943453 :   (b('ndvi') <= 2885.0) ?  -0.024439963084169652 :   -0.03186710498040497 :   (b('l8b4_med') <= 1136.5289916992188) ?  (b('l8b11_med') <= 1784.0094604492188) ?  (b('l8b11_med') <= 1760.0411987304688) ?  (b('l8b4') <= 918.5015563964844) ?  (b('ndvi') <= 4161.5) ?  0.014153227454188455 :   -0.0006487449219813868 :   (b('l8b5_med') <= 3113.1121826171875) ?  -0.02925541948606636 :   -0.02405807532019645 :   (b('ndvi_med') <= 2952.0) ?  (b('dg0_v_vv') <= 1.1678755283355713) ?  0.08061824453465266 :   0.050493888334931486 :   0.13803940225020916 :   (b('crops') <= 0.339643657207489) ?  (b('ndvi') <= 3369.5) ?  (b('l8b4') <= 989.1294555664062) ?  0.04992903578546797 :   -0.03514232673306572 :   (b('ndvi') <= 4085.5) ?  -0.05867968626437594 :   -0.004281936482084613 :   (b('l8b5') <= 3456.4462890625) ?  (b('l8b4_med') <= 954.3138427734375) ?  -0.010940236060386039 :   0.006327072291715925 :   (b('l8b5_med') <= 2175.2607421875) ?  0.040173369182784247 :   -0.02936848566534862 :   (b('l8b4') <= 1124.3387451171875) ?  (b('l8b11') <= 1980.938232421875) ?  (b('l8b4_med') <= 1415.7860107421875) ?  (b('l8b4_med') <= 1164.9218139648438) ?  -0.025589526333421306 :   0.045595666658003506 :   (b('l8b4') <= 948.3234558105469) ?  -0.015984133813313255 :   0.025662300541437214 :   (b('l8b11') <= 2020.6829223632812) ?  (b('dg0_v_vv') <= -0.8494381904602051) ?  0.12242755083886339 :   0.08887720049395842 :   (b('sand') <= 55.0) ?  0.04035998281487078 :   -0.03406457100876935 :   (b('sand') <= 48.0) ?  (b('l8b11') <= 2091.44677734375) ?  (b('grass') <= 31.159090042114258) ?  -0.013821143065953879 :   0.06580905525509322 :   (b('ndvi') <= 4009.5) ?  0.010746755657157934 :   0.03428816171288566 :   (b('ndvi') <= 3219.0) ?  (b('ndvi') <= 3193.5) ?  0.018331715224600274 :   0.0850530982543583 :   (b('l8b11') <= 2844.3421630859375) ?  -0.014284364941992264 :   0.00886190501784552 :   (b('bulk') <= 145.5) ?  (b('l8b5_med') <= 2200.718994140625) ?  (b('ndvi') <= 1201.5) ?  (b('ndvi_med') <= 2225.5) ?  -0.0789785857462405 :   -0.04169904983835415 :   (b('l8b4') <= 1159.7781372070312) ?  (b('ndvi') <= 1481.5) ?  (b('l8b5_med') <= 2006.7608642578125) ?  0.09748637279080499 :   0.0546639446342006 :   (b('l8b5') <= 1936.7847290039062) ?  -0.01941446408424038 :   0.030741635657810365 :   (b('dg0_v_vv') <= 1.0900273323059082) ?  (b('dg0_v_vv') <= -0.776482105255127) ?  0.011156219322775399 :   -0.018590442840437198 :   0.02557487850267559 :   (b('l8b4') <= 1689.8141479492188) ?  (b('dg0_v_vv') <= 1.255880355834961) ?  (b('l8b5') <= 2726.34375) ?  (b('l8b5') <= 2503.7525634765625) ?  -0.015861986986518247 :   -0.033341866643189624 :   (b('l8b5') <= 2857.1103515625) ?  -0.0011551645207742572 :   0.012920658279456135 :   (b('l8b11') <= 2207.39990234375) ?  (b('l8b5') <= 2247.9495239257812) ?  -0.04412780519408979 :   -0.0011426205566674268 :   (b('l8b4') <= 1435.7656860351562) ?  0.039965931045787884 :   0.051197158709279295 :   0.06738079355022003 :   (b('ndvi') <= 1376.0) ?  (b('ndvi') <= 886.5) ?  (b('l8b5_med') <= 2233.488525390625) ?  -0.06011838622972859 :   -0.034191378099904426 :   (b('l8b11_med') <= 2550.5718994140625) ?  (b('dg0_v_vv') <= 0.1210622787475586) ?  (b('ndvi') <= 1136.5) ?  0.07550060347604991 :   0.02917788008266853 :   (b('l8b5') <= 2200.183349609375) ?  0.0945972355348699 :   0.047537931720587685 :   (b('dg0_v_vv') <= 1.0247797966003418) ?  -0.044623215543882336 :   -0.028962102490730973 :   (b('bulk') <= 151.5) ?  (b('l8b4') <= 1667.412353515625) ?  (b('dg0_v_vv') <= -1.0680785179138184) ?  (b('ndvi_med') <= 2051.5) ?  0.03789439578100175 :   -0.002211396492547556 :   (b('l8b4_med') <= 890.8049621582031) ?  0.06056631000409744 :   0.030388062440272492 :   (b('dg0_v_vv') <= -0.2807941436767578) ?  (b('l8b11') <= 2590.1202392578125) ?  -0.010759820193985671 :   -0.02025980656515429 :   (b('dg0_v_vv') <= 0.07608890533447266) ?  -0.06714110443102396 :   -0.04196154938700866 :   (b('dg0_v_vv') <= 0.35220909118652344) ?  (b('dg0_v_vv') <= 0.01232147216796875) ?  (b('l8b4') <= 1128.6458129882812) ?  0.03868369457796708 :   -0.011616795481198393 :   (b('l8b11_med') <= 2341.44384765625) ?  0.044947450869114995 :   0.13743119092784267 :   (b('ndvi') <= 2324.0) ?  (b('ndvi') <= 2070.5) ?  -0.015829923629598633 :   -0.06838063290084259 :   (b('l8b5') <= 3136.3861083984375) ?  0.013091153450549342 :   0.038921759253069066 :   (b('bulk') <= 160.0) ?  (b('l8b5') <= 1948.34423828125) ?  (b('l8b5') <= 1691.83544921875) ?  (b('grass') <= 61.321693420410156) ?  (b('bulk') <= 140.5) ?  (b('dg0_v_vv') <= 4.960318565368652) ?  0.02185589944922267 :   0.020013873446496272 :   0.054714950502238124 :   (b('ndvi_med') <= 1833.75) ?  (b('ndvi_med') <= 1409.0) ?  -0.044029846977130666 :   (b('l8b4') <= 980.585693359375) ?  -0.04255954104230599 :   -0.041559541344054506 :   -0.08095997638964578 :   (b('ndvi') <= 1722.5) ?  (b('l8b4_med') <= 1277.7241821289062) ?  (b('l8b4') <= 1133.831787109375) ?  (b('ndvi_med') <= 1634.5) ?  0.029550222986211697 :   0.02623839765564817 :   (b('l8b11') <= 2195.1902465820312) ?  -0.0012604469550192843 :   0.00322048380273067 :   (b('l8b11_med') <= 2200.3326416015625) ?  (b('dg0_v_vv') <= 3.021684169769287) ?  0.031925848220646924 :   0.04428714044967552 :   0.06723029242435641 :   (b('moss') <= 23.8961238861084) ?  (b('l8b4') <= 980.3734741210938) ?  (b('dg0_v_vv') <= 3.76999568939209) ?  0.07240603216948824 :   0.08640653481718988 :   0.11285001937891037 :   (b('ndvi_med') <= 1450.25) ?  (b('l8b4_med') <= 1190.73681640625) ?  0.05469442616754219 :   0.061167579261501515 :   0.030780321100496583 :   (b('l8b4_med') <= 1055.3640747070312) ?  (b('ndvi') <= 1787.0) ?  (b('l8b11') <= 2557.8369140625) ?  (b('l8b11') <= 2396.77880859375) ?  (b('moss') <= 4.466313719749451) ?  -0.04103296711363533 :   -0.04322614152604276 :   (b('dg0_v_vv') <= 4.875612020492554) ?  -0.050860690415651055 :   -0.047147998070294975 :   -0.07888704710361338 :   (b('grass') <= 22.261373043060303) ?  (b('l8b4') <= 881.3539123535156) ?  (b('grass') <= 12.682291507720947) ?  0.01081209792449403 :   -0.04354360395334124 :   (b('ndvi_med') <= 2327.0) ?  0.0538689040442748 :   0.10840695313121174 :   (b('l8b11') <= 2058.4774780273438) ?  (b('l8b5') <= 2222.3048095703125) ?  -0.0035326747306228534 :   -0.039248072949392476 :   (b('sand') <= 47.0) ?  -0.02849137962791025 :   -0.00835914983703326 :   (b('l8b5') <= 2878.4345703125) ?  (b('l8b11_med') <= 2085.354248046875) ?  (b('ndvi') <= 1744.5) ?  (b('bulk') <= 150.5) ?  -0.03736260430218957 :   -0.07178112308502847 :   (b('l8b11') <= 1738.5651245117188) ?  -0.06588031418964387 :   0.013243219567169486 :   (b('dg0_v_vv') <= 3.5319671630859375) ?  (b('dg0_v_vv') <= 2.8371782302856445) ?  0.03131421887644345 :   -0.00905080016004623 :   (b('l8b5_med') <= 2434.85009765625) ?  0.00025693992667229784 :   0.03802472246738247 :   (b('dg0_v_vv') <= 2.805023193359375) ?  (b('ndvi') <= 2130.5) ?  (b('l8b4_med') <= 1737.201904296875) ?  0.057607647594716746 :   0.014127988292388404 :   (b('l8b5') <= 3059.2030029296875) ?  0.02470209277026407 :   -0.010615037209416785 :   (b('ndvi_med') <= 2280.5) ?  (b('grass') <= 23.792613983154297) ?  -0.0038389633618188188 :   -0.01914219964316859 :   (b('ndvi') <= 3538.5) ?  0.058864162784331396 :   0.006963815099451052 :   (b('l8b11') <= 2208.06201171875) ?  (b('l8b4_med') <= 1103.9726257324219) ?  0.053132800339474456 :   (b('ndvi_med') <= 1697.75) ?  (b('l8b11_med') <= 2142.905517578125) ?  0.017992872980768848 :   0.01723413969398449 :   0.010988882652653797 :   (b('l8b11') <= 2844.69873046875) ?  (b('ndvi_med') <= 1637.0) ?  (b('ndvi') <= 1444.0) ?  0.09016883561824718 :   0.09457666540619775 :   0.10825779049846868 :   (b('l8b4') <= 1549.1602172851562) ?  0.06321398160911162 :   0.06899177285856317 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi') <= 2322.0) ?  (b('dg0_v_vv') <= 1.601041316986084) ?  (b('ndvi_med') <= 3390.75) ?  (b('bulk') <= 124.0) ?  (b('ndvi') <= 2208.0) ?  (b('l8b11') <= 3400.300048828125) ?  (b('l8b11') <= 3087.98583984375) ?  (b('ndvi') <= 1651.0) ?  (b('ndvi') <= 1343.5) ?  (b('ndvi') <= 1102.0) ?  0.0402620034773616 :   0.00598546382566781 :   (b('l8b5_med') <= 2018.583740234375) ?  0.07414855643603205 :   0.03774972552392042 :   (b('ndvi_med') <= 1612.5) ?  (b('ndvi') <= 1935.0) ?  -0.05256394824960586 :   -0.01608241583716264 :   (b('l8b5') <= 1471.169921875) ?  -0.03219705110090238 :   0.020375553609833515 :   (b('l8b5') <= 2203.3692626953125) ?  (b('ndvi_med') <= 1670.5) ?  -5.680341625079188e-05 :   -0.0071580444185931486 :   (b('dg0_v_vv') <= 0.21771717071533203) ?  -0.019736087896542583 :   -0.018794166240311466 :   0.11493083922402517 :   (b('grass') <= 62.045658111572266) ?  (b('grass') <= 25.375225067138672) ?  (b('l8b4_med') <= 721.6394348144531) ?  0.029831766241357083 :   (b('sand') <= 31.0) ?  0.012108721654670918 :   0.0050278246887282 :   (b('ndvi') <= 2315.5) ?  (b('l8b4') <= 1273.1158447265625) ?  (b('dg0_v_vv') <= -0.4413480758666992) ?  -0.0035027616386399674 :   -0.010974196261190478 :   0.0009908911551802513 :   -0.024238849771709786 :   (b('l8b4') <= 1079.124267578125) ?  (b('l8b5') <= 2066.519287109375) ?  (b('dg0_v_vv') <= -2.120894432067871) ?  -0.014650106370933308 :   (b('ndvi') <= 2291.5) ?  -0.023505754079682 :   -0.03231491136314167 :   (b('l8b5_med') <= 2437.6734619140625) ?  (b('moss') <= 5.902412414550781) ?  -0.03618894626433361 :   -0.04198546476637959 :   (b('l8b11') <= 1624.0744018554688) ?  -0.05139599538170075 :   -0.0460012103639156 :   -0.08007692958857061 :   (b('moss') <= 8.823604106903076) ?  (b('moss') <= 8.482120990753174) ?  (b('dg0_v_vv') <= 0.4022798538208008) ?  (b('l8b5_med') <= 3259.750732421875) ?  (b('ndvi_med') <= 1601.25) ?  (b('l8b4') <= 1550.5657348632812) ?  -0.017319782286867615 :   -0.007272851904338692 :   (b('l8b5_med') <= 3193.2320556640625) ?  -0.0034619843148524523 :   0.01605765289375002 :   (b('l8b5_med') <= 3378.561279296875) ?  (b('l8b5') <= 4382.34130859375) ?  0.011944645980779113 :   0.13773403477989532 :   (b('l8b5') <= 2390.78369140625) ?  -0.014256150185632165 :   -0.00028901987974438396 :   (b('l8b5') <= 2503.98291015625) ?  (b('l8b5') <= 2238.4853515625) ?  (b('dg0_v_vv') <= 0.46611928939819336) ?  0.030349981744438686 :   -0.003567051028022579 :   (b('bulk') <= 151.5) ?  0.011784576263151183 :   0.044096800275883724 :   (b('l8b5') <= 3115.2666015625) ?  (b('sand') <= 63.5) ?  -0.008867638033934679 :   0.034942073742695014 :   (b('l8b4_med') <= 1830.4744262695312) ?  0.0180477781892839 :   -0.0035177081744947607 :   (b('dg0_v_vv') <= 0.07405900955200195) ?  (b('ndvi') <= 2295.0) ?  (b('dg0_v_vv') <= -0.3542165756225586) ?  (b('l8b11') <= 2848.7010498046875) ?  0.031672508270653765 :   -0.004549419233915049 :   (b('ndvi_med') <= 1868.75) ?  -0.02222007203276259 :   0.04654536531095653 :   -0.10851986591672125 :   (b('sand') <= 47.5) ?  (b('dg0_v_vv') <= 0.38094520568847656) ?  0.08192620826381586 :   (b('l8b5') <= 3101.563720703125) ?  0.027335578050130174 :   0.07327329500177784 :   (b('dg0_v_vv') <= 0.4158205986022949) ?  (b('dg0_v_vv') <= 0.21985149383544922) ?  0.05408942711649356 :   0.06534652572969954 :   (b('grass') <= 84.5565185546875) ?  0.111852784910451 :   0.0957770007255632 :   (b('l8b5') <= 2062.05126953125) ?  (b('ndvi') <= 1961.5) ?  (b('l8b4_med') <= 1213.6558227539062) ?  (b('l8b11_med') <= 2173.180908203125) ?  (b('l8b11') <= 1879.47998046875) ?  -0.01050342039410928 :   0.0014979597060032492 :   (b('dg0_v_vv') <= 1.1109929084777832) ?  -0.04141930869536102 :   0.043649611310417995 :   (b('l8b11') <= 2088.3470458984375) ?  (b('l8b4_med') <= 1237.366943359375) ?  0.05065259049732911 :   0.00834314616211579 :   (b('l8b11_med') <= 2090.949951171875) ?  -0.029914150128278093 :   0.003101635156359763 :   (b('l8b11_med') <= 2007.2160034179688) ?  (b('ndvi') <= 2033.0) ?  (b('ndvi_med') <= 2127.25) ?  0.054063923707696 :   -0.009156078747810421 :   (b('sand') <= 55.5) ?  -0.0022943228208646533 :   0.06679247323551339 :   (b('l8b4') <= 956.0997009277344) ?  (b('l8b4_med') <= 1066.5977172851562) ?  0.04312918328201573 :   -0.0056604340214019 :   (b('grass') <= 78.94009780883789) ?  0.14172864052545292 :   0.03511965073151252 :   (b('l8b11_med') <= 1459.8203735351562) ?  (b('l8b11') <= 1401.8594970703125) ?  (b('sand') <= 29.5) ?  0.05033287187099039 :   (b('l8b5_med') <= 2414.765380859375) ?  -0.03074319997120777 :   -0.0537474396834345 :   (b('l8b11') <= 1441.5013427734375) ?  (b('l8b4_med') <= 725.2777404785156) ?  0.026987451872790502 :   -0.003603142507359458 :   (b('crops') <= 56.322683334350586) ?  -0.021673702148183205 :   -0.05330745566748377 :   (b('moss') <= 11.84619140625) ?  (b('dg0_v_vv') <= -7.821526050567627) ?  (b('l8b11') <= 2043.6214599609375) ?  0.06326037083131211 :   0.0659707501213246 :   (b('l8b11') <= 2249.8922119140625) ?  -0.01715348322820656 :   -0.007314055707981044 :   (b('ndvi') <= 2237.5) ?  (b('ndvi_med') <= 2569.25) ?  -0.00573413584928861 :   0.008996512909748624 :   (b('ndvi_med') <= 1488.25) ?  0.09542749149960578 :   -0.02483662008694595 :   (b('ndvi_med') <= 4097.5) ?  (b('dg0_v_vv') <= -1.2350707054138184) ?  (b('l8b11') <= 2191.5396728515625) ?  (b('grass') <= 67.40287780761719) ?  (b('ndvi') <= 1566.5) ?  -0.03024082522462701 :   (b('ndvi') <= 1947.0) ?  (b('dg0_v_vv') <= -2.4246749877929688) ?  -0.0030141249106753385 :   0.022824240004987492 :   (b('dg0_v_vv') <= -1.9224905967712402) ?  -0.018728691309185183 :   -0.006879979149005585 :   (b('dg0_v_vv') <= -1.476236343383789) ?  -0.05013544417581403 :   -0.033800928212153636 :   (b('l8b4') <= 1454.0173950195312) ?  (b('bulk') <= 136.0) ?  0.01636752865441185 :   0.03229489047290213 :   0.060456568245799425 :   (b('l8b4') <= 965.6730651855469) ?  (b('ndvi') <= 1439.0) ?  (b('l8b11_med') <= 1382.7453002929688) ?  (b('grass') <= 31.231670379638672) ?  (b('dg0_v_vv') <= -0.5405349731445312) ?  0.05057668183847741 :   0.06538509158852987 :   0.06761560350810009 :   (b('l8b5') <= 1632.4151000976562) ?  0.04201049256363648 :   0.02145328602362656 :   (b('bulk') <= 127.0) ?  (b('l8b4') <= 474.33033752441406) ?  (b('dg0_v_vv') <= -0.16976404190063477) ?  0.049730776193541104 :   0.018372859294740307 :   (b('moss') <= 0.4336957596242428) ?  -0.011673401365314064 :   -0.038181550361613005 :   (b('dg0_v_vv') <= -0.6679511070251465) ?  (b('dg0_v_vv') <= -1.1380410194396973) ?  0.03733326445044982 :   -0.0008084199518811411 :   (b('l8b4_med') <= 705.8905944824219) ?  0.037744181564553364 :   0.014345954082339471 :   (b('l8b5_med') <= 2639.5843505859375) ?  (b('l8b11_med') <= 1404.7261962890625) ?  0.10926837096562253 :   0.08053508921829378 :   (b('l8b11_med') <= 2380.4537353515625) ?  (b('l8b5') <= 2156.9866943359375) ?  (b('dg0_v_vv') <= -0.2639174461364746) ?  0.05209258963297185 :   0.08897086624285588 :   (b('l8b11_med') <= 1303.4545288085938) ?  0.02010936000623167 :   0.04035185491194028 :   -0.028809797393149372 :   (b('l8b5') <= 2014.4085693359375) ?  -0.07303147795128939 :   (b('l8b4') <= 1316.4100341796875) ?  (b('l8b4') <= 1114.4757690429688) ?  (b('l8b11_med') <= 1436.123779296875) ?  (b('grass') <= 32.897470474243164) ?  -0.017890271214542813 :   -0.02043156894912318 :   -0.024207803819027207 :   -0.03303638826328201 :   -0.004403730914594645 :   (b('l8b11_med') <= 2749.3857421875) ?  (b('bulk') <= 163.5) ?  (b('l8b4') <= 2194.1041259765625) ?  (b('ndvi_med') <= 898.0) ?  (b('l8b11') <= 1702.4580078125) ?  (b('ndvi') <= 735.0) ?  (b('ndvi') <= 650.5) ?  -0.03307604587439193 :   -0.046251582216700254 :   (b('moss') <= 37.264272689819336) ?  (b('l8b4_med') <= 1473.8698120117188) ?  -0.009866343536438504 :   -0.0033227044389925187 :   0.018872672808904367 :   (b('bulk') <= 145.0) ?  -0.10105803073278816 :   -0.07524482272130842 :   (b('l8b11') <= 1078.147216796875) ?  (b('ndvi_med') <= 2788.5) ?  (b('ndvi_med') <= 2726.5) ?  (b('l8b4_med') <= 835.91259765625) ?  -0.004145507131118743 :   -0.06059846213209507 :   0.04933767662535299 :   -0.09800867904559551 :   (b('grass') <= 85.30942153930664) ?  (b('grass') <= 83.75120544433594) ?  (b('ndvi') <= 1064.5) ?  0.02333112260111466 :   0.00687084483688758 :   (b('l8b5') <= 2536.725830078125) ?  -0.15914633257768418 :   -0.05913694170284306 :   (b('dg0_v_vv') <= 3.2602357864379883) ?  (b('dg0_v_vv') <= 3.1739797592163086) ?  0.013527652567473071 :   -0.11925573053621785 :   (b('l8b4_med') <= 1158.6707763671875) ?  0.005906981447134741 :   0.056180807655032736 :   (b('bulk') <= 151.5) ?  (b('l8b4_med') <= 1942.2267456054688) ?  (b('l8b4_med') <= 1745.3390502929688) ?  (b('l8b11_med') <= 2518.2154541015625) ?  0.03342054931435226 :   0.03577551772263604 :   (b('l8b11') <= 2893.2047119140625) ?  0.059589829787930926 :   0.07554411122206822 :   (b('crops') <= 37.5662841796875) ?  0.07291081367231916 :   (b('l8b4_med') <= 1996.3931884765625) ?  0.1126199161902226 :   0.10840877363555323 :   (b('l8b4_med') <= 1703.289306640625) ?  -0.001366630344526415 :   -0.05231171863116754 :   (b('l8b4_med') <= 1363.1318359375) ?  (b('l8b4_med') <= 1229.92236328125) ?  0.009889994387388412 :   (b('l8b11') <= 2232.6427001953125) ?  -0.035315364868691024 :   -0.01412209451784717 :   (b('l8b5') <= 1393.4656372070312) ?  -0.04274032631080876 :   (b('dg0_v_vv') <= 1.843400478363037) ?  -0.0649782719702268 :   (b('l8b4_med') <= 1457.65966796875) ?  -0.07512439241927757 :   -0.08316057310684966 :   (b('ndvi_med') <= 2106.5) ?  (b('l8b5') <= 2898.88818359375) ?  (b('l8b11') <= 3065.880859375) ?  (b('l8b4') <= 1402.0941772460938) ?  (b('l8b4') <= 1376.70068359375) ?  (b('l8b11') <= 2009.6427612304688) ?  (b('grass') <= 0.27586206793785095) ?  0.024428049482936386 :   0.002929875774034777 :   (b('l8b11_med') <= 2999.84375) ?  -0.018500960207889886 :   0.007450887649129823 :   (b('ndvi_med') <= 1482.0) ?  0.021879869881345493 :   (b('ndvi_med') <= 1734.75) ?  0.0607421703083105 :   0.05378337949880663 :   (b('ndvi_med') <= 1385.25) ?  (b('l8b5') <= 2372.2479248046875) ?  (b('sand') <= 51.0) ?  0.0009405768520861502 :   -0.02990863232630479 :   (b('l8b5') <= 2520.6082763671875) ?  0.032161749616499945 :   0.008617228795033788 :   (b('moss') <= 0.8818444013595581) ?  -0.07375304899013993 :   (b('ndvi') <= 790.5) ?  0.030846801448612265 :   -0.010677254778582964 :   (b('ndvi') <= 1618.5) ?  (b('grass') <= 15.675792455673218) ?  (b('dg0_v_vv') <= 3.5662667751312256) ?  (b('dg0_v_vv') <= 3.3052978515625) ?  0.0016681788777123416 :   -0.07123550056212133 :   (b('ndvi') <= 1347.5) ?  0.037802676249413525 :   0.03939677209248865 :   (b('ndvi') <= 1323.5) ?  (b('ndvi') <= 1051.0) ?  0.0028081007270987224 :   0.019127515536977915 :   (b('l8b5_med') <= 3169.1014404296875) ?  0.04882608160465555 :   0.017126866396962592 :   (b('dg0_v_vv') <= 1.9282355308532715) ?  0.1046815396398291 :   (b('l8b5_med') <= 2972.7972412109375) ?  0.05543024425728019 :   0.03964187220650808 :   (b('l8b5') <= 3082.9697265625) ?  (b('ndvi') <= 1531.0) ?  (b('crops') <= 97.33172988891602) ?  (b('dg0_v_vv') <= 3.4166669845581055) ?  (b('l8b11') <= 3134.13134765625) ?  0.008440400397832563 :   -0.013879455067815684 :   (b('ndvi') <= 1064.5) ?  -0.007488098197878393 :   -0.028406713900309813 :   -0.06060760961502962 :   (b('l8b4') <= 1427.6806030273438) ?  (b('l8b4_med') <= 1935.1727294921875) ?  0.0040006491251833795 :   0.014327439076862195 :   (b('l8b11') <= 2797.1845703125) ?  (b('dg0_v_vv') <= 2.4080898761749268) ?  -0.04441659642896753 :   -0.06540004744863714 :   (b('l8b4') <= 1936.1558227539062) ?  -0.016013199657035595 :   -0.047973321429693745 :   (b('grass') <= 65.6275691986084) ?  (b('l8b11') <= 3149.4439697265625) ?  (b('l8b5_med') <= 3137.0338134765625) ?  (b('l8b11') <= 2744.10205078125) ?  -0.04740706089265979 :   -0.013775997291305865 :   (b('l8b5') <= 3151.7882080078125) ?  0.020101279353291847 :   -0.007328390438948552 :   (b('l8b5_med') <= 3180.0611572265625) ?  (b('dg0_v_vv') <= 3.955549716949463) ?  0.03789326763836664 :   -0.009767474113521076 :   (b('l8b11') <= 3183.8160400390625) ?  0.021105362188997227 :   -0.002019546688397627 :   0.07440450408238469 :   (b('crops') <= 37.36392158269882) ?  -0.07798219191514243 :   -0.05847712509346273 :   (b('grass') <= 46.231340408325195) ?  (b('sand') <= 20.5) ?  (b('l8b11') <= 1269.4613037109375) ?  (b('l8b11') <= 1070.1653442382812) ?  (b('l8b4') <= 437.37278747558594) ?  (b('l8b11') <= 901.0127563476562) ?  (b('dg0_v_vv') <= 1.677595615386963) ?  (b('dg0_v_vv') <= -0.5071287155151367) ?  (b('l8b5') <= 4095.48779296875) ?  0.044404756258348194 :   0.05428316788128973 :   (b('crops') <= 46.10081672668457) ?  0.045294295839981236 :   0.031365976436196596 :   0.016891765357813393 :   (b('l8b5') <= 4340.269775390625) ?  0.08138169312731547 :   0.06563847712900972 :   (b('ndvi') <= 4159.0) ?  (b('ndvi') <= 2982.5) ?  (b('dg0_v_vv') <= -0.9830589294433594) ?  0.05363942136133831 :   (b('bulk') <= 132.5) ?  0.0066590672264620525 :   -0.011171254028687128 :   (b('ndvi') <= 3292.0) ?  0.09878413637481215 :   (b('l8b11') <= 1047.5340576171875) ?  0.020230586231153248 :   0.04934788006913732 :   (b('l8b5_med') <= 2894.9593505859375) ?  -0.0444805423110774 :   (b('ndvi') <= 4881.5) ?  (b('dg0_v_vv') <= 0.13866043090820312) ?  -0.02464118934974438 :   0.0227823808527306 :   0.035235523246220146 :   (b('ndvi') <= 4741.5) ?  (b('l8b11') <= 1107.867431640625) ?  (b('ndvi_med') <= 3236.0) ?  (b('crops') <= 75.70478820800781) ?  0.06164782529458607 :   0.041232659218854106 :   (b('ndvi') <= 4227.0) ?  (b('dg0_v_vv') <= -1.94816255569458) ?  -0.03359582862735347 :   0.010670698022275759 :   (b('l8b5') <= 3860.339111328125) ?  -0.031091653239666894 :   -0.03713570968606511 :   (b('bulk') <= 145.5) ?  (b('dg0_v_vv') <= 0.07375192642211914) ?  (b('ndvi_med') <= 4103.5) ?  0.011897702452905654 :   0.05527187402936867 :   (b('dg0_v_vv') <= 0.5799436569213867) ?  -0.044444820589082695 :   -0.023730068341962535 :   (b('dg0_v_vv') <= -1.2594408988952637) ?  (b('sand') <= 18.0) ?  -0.00713099598606004 :   0.020478798924172814 :   (b('ndvi') <= 4550.0) ?  -0.03109556575986587 :   -0.08048568537969719 :   (b('crops') <= 53.75650405883789) ?  0.08785240958703966 :   0.010132306176340311 :   (b('ndvi_med') <= 4372.5) ?  (b('ndvi') <= 3739.5) ?  (b('dg0_v_vv') <= -0.06519174575805664) ?  (b('l8b11') <= 1707.139892578125) ?  (b('l8b4') <= 819.1620483398438) ?  (b('l8b5') <= 3771.721923828125) ?  0.0365851740416514 :   -0.015239700730871941 :   (b('dg0_v_vv') <= -1.1671404838562012) ?  -0.02033719082164527 :   -0.055144147812693084 :   (b('dg0_v_vv') <= -1.3656187057495117) ?  0.051490332293565366 :   (b('l8b11') <= 2186.8919677734375) ?  0.0049516647262505264 :   0.02634085341503639 :   (b('dg0_v_vv') <= 0.9347867965698242) ?  (b('l8b5') <= 2573.791748046875) ?  (b('l8b11') <= 1650.6102294921875) ?  0.045304082463558104 :   0.100097590333933 :   (b('dg0_v_vv') <= 0.7907638549804688) ?  0.030049102618513776 :   0.06855993631081247 :   (b('dg0_v_vv') <= 1.0874419212341309) ?  -0.045060036894496586 :   (b('l8b11') <= 1777.2618408203125) ?  0.03583677855270878 :   0.004203670987370622 :   (b('ndvi') <= 4228.5) ?  (b('crops') <= 48.408058166503906) ?  (b('dg0_v_vv') <= -1.0510759353637695) ?  0.09414289047108637 :   0.12661058844615186 :   (b('l8b4_med') <= 693.9182434082031) ?  0.06620498698108992 :   0.047714266532813354 :   (b('l8b5') <= 3715.670654296875) ?  (b('l8b11') <= 1797.3060302734375) ?  (b('grass') <= 32.83961486816406) ?  0.012955515852410643 :   0.036539564481018236 :   0.06827432163884023 :   (b('l8b11_med') <= 1347.2548828125) ?  (b('l8b4_med') <= 669.0899963378906) ?  0.06048431810418589 :   0.06337994655562268 :   0.07794302651956919 :   (b('dg0_v_vv') <= 1.8822145462036133) ?  (b('l8b4') <= 1033.5500793457031) ?  -0.08507237174778753 :   -0.033558938637180846 :   0.04394072166818491 :   (b('dg0_v_vv') <= 0.6708230972290039) ?  (b('grass') <= 4.636483192443848) ?  (b('l8b5') <= 3457.2666015625) ?  (b('l8b11_med') <= 987.6859130859375) ?  -0.0882580960399253 :   (b('grass') <= 3.7213581800460815) ?  (b('l8b5') <= 2460.6549072265625) ?  (b('l8b5') <= 2137.2557373046875) ?  0.018004677010002273 :   -0.008670284280313068 :   (b('l8b5_med') <= 3474.926513671875) ?  0.017487133350710168 :   -0.0049890467576863655 :   (b('l8b5') <= 2420.6640625) ?  (b('dg0_v_vv') <= 0.21316957473754883) ?  0.08325300292504206 :   0.10874979508156563 :   (b('dg0_v_vv') <= -1.280275821685791) ?  0.057615198078265685 :   0.007410854791435864 :   (b('l8b11') <= 1306.7284545898438) ?  (b('dg0_v_vv') <= -2.23997163772583) ?  (b('l8b5') <= 3742.7037353515625) ?  (b('ndvi_med') <= 3443.25) ?  -0.026462227787046672 :   -0.06658741917517835 :   (b('ndvi') <= 3313.5) ?  0.01410457583987564 :   -0.0005559149831440846 :   (b('ndvi') <= 3520.0) ?  (b('l8b5') <= 3679.872802734375) ?  -0.029510629046013277 :   0.015365738635946287 :   (b('ndvi') <= 4215.0) ?  0.04714718986816506 :   0.013117600700291948 :   (b('l8b11') <= 1612.5975341796875) ?  (b('ndvi') <= 3458.5) ?  (b('l8b4_med') <= 1803.4765014648438) ?  -0.0414626441948567 :   0.019638561603404936 :   (b('l8b5_med') <= 3505.713134765625) ?  0.0033026381814780822 :   -0.0361566742958547 :   (b('sand') <= 32.0) ?  (b('dg0_v_vv') <= 0.23919343948364258) ?  0.01937288227054055 :   -0.009816309626144668 :   (b('l8b4') <= 2055.6224365234375) ?  -0.014406372201152843 :   0.01364357461574038 :   (b('l8b11') <= 987.6361389160156) ?  (b('dg0_v_vv') <= 0.6615347862243652) ?  (b('moss') <= 2.9057151079177856) ?  (b('ndvi') <= 4796.5) ?  (b('ndvi') <= 4728.0) ?  -0.008493879685378085 :   0.032360691429895214 :   (b('grass') <= 29.431796073913574) ?  -0.05545752147952064 :   -0.007072661064473351 :   (b('l8b11') <= 779.176513671875) ?  (b('l8b4') <= 517.1020812988281) ?  -0.004067010549441719 :   0.0697950464983208 :   (b('grass') <= 28.026954650878906) ?  0.02559111549059926 :   0.0075568494858351525 :   (b('bulk') <= 146.5) ?  -0.09950997407370357 :   -0.08636299928808305 :   (b('grass') <= 37.80600547790527) ?  (b('l8b4') <= 466.3345489501953) ?  (b('ndvi_med') <= 4397.25) ?  (b('bulk') <= 140.0) ?  -0.01849295958424976 :   -0.05670903188556511 :   0.06913214905865056 :   (b('grass') <= 36.975154876708984) ?  (b('grass') <= 35.60479545593262) ?  -0.0011570045860830613 :   -0.031296618795173516 :   (b('l8b5') <= 3064.0740966796875) ?  0.002011063802603018 :   0.03924961077841435 :   (b('l8b4') <= 1239.421142578125) ?  (b('l8b4') <= 593.5234985351562) ?  (b('crops') <= 35.73115539550781) ?  -0.0067194516032663895 :   0.028138766027199943 :   (b('grass') <= 40.87277412414551) ?  -0.047230119579056276 :   -0.018801361115506378 :   (b('ndvi_med') <= 2771.0) ?  (b('dg0_v_vv') <= 0.6303524971008301) ?  0.057454400697169915 :   0.1010395634388365 :   (b('l8b11') <= 1960.3338623046875) ?  0.038732362101734134 :   -0.015450145239987808 :   (b('bulk') <= 161.5) ?  (b('bulk') <= 155.5) ?  (b('bulk') <= 122.5) ?  (b('ndvi') <= 2492.5) ?  (b('ndvi_med') <= 1670.5) ?  0.16434169182736602 :   0.13112562313870993 :   (b('crops') <= 37.84331512451172) ?  (b('l8b11_med') <= 1531.7195434570312) ?  -0.026500233122043526 :   0.03622696039020793 :   (b('l8b5') <= 4096.487060546875) ?  0.05135810370746955 :   0.008310132160960508 :   (b('l8b5') <= 3297.4508056640625) ?  (b('l8b5') <= 3296.5316162109375) ?  (b('bulk') <= 146.5) ?  0.004082440364356272 :   0.01620298671089357 :   (b('ndvi') <= 4787.0) ?  0.08724080788273658 :   0.05036889857233511 :   (b('l8b5') <= 3321.927734375) ?  (b('l8b11') <= 1181.133056640625) ?  -0.06346118125979006 :   -0.005822637589456481 :   (b('l8b4') <= 1940.87451171875) ?  8.815768461356831e-07 :   0.020913659293486965 :   (b('l8b11_med') <= 1983.5992431640625) ?  (b('dg0_v_vv') <= 0.7073073387145996) ?  (b('l8b11') <= 1496.3119506835938) ?  -0.11549491192833011 :   -0.05002839096709945 :   (b('l8b11_med') <= 1114.8645629882812) ?  (b('ndvi') <= 2444.5) ?  -0.07219223485048418 :   -0.03981095033703269 :   (b('l8b5') <= 2891.1473388671875) ?  -0.008798063546276203 :   -0.028407957116387796 :   (b('ndvi') <= 3567.0) ?  (b('dg0_v_vv') <= 0.8558139801025391) ?  0.041786815302165936 :   (b('l8b11') <= 2292.97998046875) ?  -0.030273160251606313 :   0.0009620754400874801 :   (b('l8b11') <= 1450.8228149414062) ?  (b('dg0_v_vv') <= 1.7217283248901367) ?  -0.02399171138377175 :   -0.006616425435189283 :   (b('ndvi_med') <= 3405.25) ?  0.025052583865163198 :   0.005516989026900171 :   (b('l8b5') <= 3847.1671142578125) ?  (b('l8b5') <= 3743.6392822265625) ?  (b('l8b4') <= 778.6595764160156) ?  (b('l8b5') <= 3124.052001953125) ?  0.1002462556356496 :   0.09541686126146898 :   (b('l8b4') <= 1113.470947265625) ?  (b('dg0_v_vv') <= 1.1312975883483887) ?  0.0585507464755702 :   0.04337073026125782 :   (b('l8b4') <= 1232.6510620117188) ?  0.10585743297996086 :   0.08870284350428098 :   0.12681717155950928 :   -0.041026385055454306 :   (b('ndvi') <= 4417.0) ?  (b('l8b4_med') <= 601.8363037109375) ?  (b('l8b5') <= 3208.7056884765625) ?  (b('l8b11') <= 898.7603149414062) ?  (b('ndvi') <= 3909.0) ?  (b('ndvi') <= 2714.5) ?  (b('ndvi_med') <= 3709.25) ?  (b('l8b4') <= 439.3976745605469) ?  0.021532634547243995 :   0.032792692781304256 :   (b('dg0_v_vv') <= -0.678675651550293) ?  -0.03935929293028512 :   0.0010326553460932864 :   (b('dg0_v_vv') <= -0.7315773963928223) ?  (b('ndvi') <= 3577.0) ?  0.11861153796311302 :   0.06784830014358856 :   (b('dg0_v_vv') <= -0.27887392044067383) ?  0.0031049108151204596 :   0.03911640277986265 :   (b('ndvi_med') <= 3709.25) ?  (b('l8b5') <= 1926.0759887695312) ?  (b('dg0_v_vv') <= 0.6134705543518066) ?  -0.01944442487100275 :   -0.02938786793458409 :   0.03125726492971892 :   0.13119878652339978 :   (b('l8b5') <= 2085.4517822265625) ?  (b('ndvi') <= 2717.0) ?  (b('dg0_v_vv') <= 0.3584465980529785) ?  0.09473338721468433 :   (b('l8b4') <= 712.4852294921875) ?  -0.008815964071339416 :   -0.018222012580398905 :   (b('bulk') <= 117.0) ?  -0.056868143192862236 :   (b('dg0_v_vv') <= -0.47965574264526367) ?  -0.03931351953837997 :   -0.020609587702448317 :   (b('l8b11') <= 1033.3925170898438) ?  (b('ndvi') <= 4000.5) ?  (b('dg0_v_vv') <= 0.6219334602355957) ?  0.013713923997275713 :   -0.009546416929512747 :   (b('l8b11') <= 1022.370849609375) ?  -0.007213857804338322 :   -0.05309835806183827 :   (b('l8b11_med') <= 1036.3284301757812) ?  (b('ndvi_med') <= 3520.75) ?  0.023369467068688896 :   0.05344797609121437 :   (b('crops') <= 3.3066861629486084) ?  -0.01586903384649698 :   0.02031015570686363 :   (b('ndvi_med') <= 4101.5) ?  (b('l8b11') <= 1310.2891845703125) ?  (b('l8b5') <= 3304.367431640625) ?  0.0232391586583241 :   (b('ndvi') <= 3767.0) ?  (b('l8b11_med') <= 1279.5648193359375) ?  -0.015278224688098946 :   -0.012983090654471452 :   -0.006267589050087563 :   (b('ndvi') <= 4224.0) ?  (b('dg0_v_vv') <= 0.26761722564697266) ?  (b('dg0_v_vv') <= -0.9950056076049805) ?  0.030279341938671323 :   0.019234342409882726 :   0.03685147009564793 :   (b('l8b11_med') <= 1255.5039672851562) ?  (b('dg0_v_vv') <= 0.20115900039672852) ?  0.008115057866511233 :   0.006598840187850841 :   (b('dg0_v_vv') <= 0.6142454147338867) ?  0.01946648592342154 :   0.014272328725604522 :   (b('l8b11') <= 2194.7716064453125) ?  (b('ndvi') <= 3927.5) ?  (b('grass') <= 67.54422378540039) ?  -0.050154605458094526 :   -0.056863192261760676 :   (b('dg0_v_vv') <= -0.2956070899963379) ?  -0.0113091964370296 :   (b('ndvi') <= 4167.0) ?  -0.04678374978298269 :   -0.02781637162654234 :   (b('ndvi') <= 3042.5) ?  -0.0006088585802249413 :   -0.005744105913853492 :   (b('l8b11_med') <= 1975.468017578125) ?  (b('l8b11_med') <= 1717.7350463867188) ?  (b('l8b4_med') <= 1084.4794311523438) ?  (b('l8b5') <= 2448.494384765625) ?  (b('l8b11') <= 1358.05078125) ?  (b('l8b5') <= 2152.55810546875) ?  0.01732762545577484 :   -0.009113062781562205 :   (b('moss') <= 15.212496757507324) ?  0.029912643986015064 :   -0.000674409077395243 :   (b('l8b11') <= 1132.9193725585938) ?  (b('l8b5') <= 4560.74658203125) ?  0.019164799285060076 :   -0.07147781165256223 :   (b('bulk') <= 110.5) ?  -0.01879155212474318 :   4.442671834010621e-05 :   (b('l8b5') <= 2552.766845703125) ?  0.07744758678079355 :   (b('dg0_v_vv') <= -1.7601542472839355) ?  0.10485741417674616 :   0.0972935717072494 :   (b('ndvi') <= 2398.5) ?  (b('ndvi_med') <= 2211.0) ?  (b('l8b11_med') <= 1835.3106689453125) ?  0.14987308618981024 :   (b('ndvi') <= 2372.0) ?  0.030208702394018936 :   0.0945578282796372 :   (b('ndvi') <= 2327.5) ?  0.06507299405481944 :   (b('ndvi') <= 2344.5) ?  -0.04590203891902023 :   -0.0019635421130049326 :   (b('l8b5_med') <= 1740.5648193359375) ?  0.13100908521597782 :   (b('grass') <= 47.74807167053223) ?  (b('dg0_v_vv') <= 0.25821876525878906) ?  0.04627305007388366 :   0.12423546202518826 :   (b('l8b11') <= 1376.3436889648438) ?  0.0005957186732931432 :   -0.018156202160562792 :   (b('l8b11_med') <= 2105.4874267578125) ?  (b('ndvi') <= 3531.5) ?  (b('bulk') <= 149.0) ?  (b('ndvi_med') <= 2654.0) ?  (b('dg0_v_vv') <= -1.7404308319091797) ?  0.10877847723317555 :   0.02006641815944025 :   (b('l8b11') <= 2084.2381591796875) ?  0.10704845809972956 :   0.015858293343830775 :   (b('l8b5') <= 2478.390380859375) ?  (b('l8b5') <= 2329.6602783203125) ?  0.06702147822868525 :   0.11457963124053133 :   (b('l8b5') <= 2493.7615966796875) ?  -0.12418659985944935 :   0.036151483683826986 :   (b('l8b5_med') <= 2422.3895263671875) ?  (b('ndvi') <= 3794.5) ?  (b('ndvi_med') <= 2231.0) ?  -0.019786473655907993 :   0.00845240631660287 :   (b('sand') <= 46.5) ?  0.053945589173836295 :   0.017072678960032533 :   (b('sand') <= 40.5) ?  (b('l8b11') <= 2143.61279296875) ?  -0.08626284340471276 :   -0.04838959703464002 :   (b('dg0_v_vv') <= 2.340182304382324) ?  -0.007895203885611261 :   -0.05667613703416463 :   (b('ndvi_med') <= 2571.0) ?  (b('l8b4_med') <= 1243.9486083984375) ?  (b('l8b5_med') <= 2542.2144775390625) ?  (b('l8b11') <= 2490.3154296875) ?  0.001530547251303456 :   0.07553419288930732 :   (b('ndvi') <= 2854.0) ?  -0.056436030108247014 :   -0.011336516634778998 :   (b('l8b11_med') <= 2195.1702880859375) ?  (b('dg0_v_vv') <= 1.462834358215332) ?  -0.0013547914191509418 :   0.06567084180642287 :   (b('ndvi') <= 4211.5) ?  0.007773798306338456 :   0.06293765749668072 :   (b('ndvi') <= 3051.5) ?  (b('l8b11') <= 2782.828369140625) ?  (b('sand') <= 50.0) ?  -0.028948050456381276 :   -0.07689201090833957 :   (b('l8b4') <= 1607.2149658203125) ?  0.004636987429181866 :   0.04453973240600545 :   (b('dg0_v_vv') <= 1.9611701965332031) ?  (b('ndvi_med') <= 2880.5) ?  -0.02575065583996591 :   0.010987539521146146 :   (b('ndvi') <= 3911.5) ?  0.03850538279660465 :   0.06052370699815038 :   (b('sand') <= 33.5) ?  (b('l8b5_med') <= 2833.8748779296875) ?  (b('ndvi') <= 4711.0) ?  (b('l8b4') <= 582.2138671875) ?  (b('dg0_v_vv') <= 1.6308226585388184) ?  (b('dg0_v_vv') <= 1.0048565864562988) ?  (b('l8b5_med') <= 2528.1949462890625) ?  0.09251587541869616 :   0.045497238382874006 :   -0.05031294298415123 :   0.10497044293653779 :   (b('dg0_v_vv') <= 2.8090906143188477) ?  (b('l8b5') <= 3101.208984375) ?  (b('dg0_v_vv') <= 0.38550472259521484) ?  0.06867493426201161 :   0.09122199058202121 :   0.11868262174531052 :   0.019900846429709684 :   (b('l8b4_med') <= 830.8964233398438) ?  (b('dg0_v_vv') <= -0.9692397117614746) ?  0.05679153035631046 :   (b('dg0_v_vv') <= -0.7320418357849121) ?  -0.012326020297022316 :   -0.006307223614723884 :   (b('dg0_v_vv') <= 0.40322065353393555) ?  (b('dg0_v_vv') <= 0.2262248992919922) ?  (b('l8b5_med') <= 2523.0125732421875) ?  0.08900667513723282 :   0.050834556538616515 :   0.13619941369565863 :   (b('dg0_v_vv') <= 0.7011032104492188) ?  (b('l8b5_med') <= 2578.29150390625) ?  0.015416289178528886 :   -0.004627768284237277 :   (b('l8b5') <= 3217.734375) ?  0.034537016431964995 :   0.04076739040396388 :   (b('moss') <= 0.3046875) ?  (b('l8b5') <= 3163.9053955078125) ?  0.04932898373466682 :   (b('crops') <= 14.328332901000977) ?  0.08035605012769692 :   (b('l8b11_med') <= 1365.9826049804688) ?  0.07284780548884107 :   0.06888668018889799 :   (b('l8b4_med') <= 771.636962890625) ?  (b('dg0_v_vv') <= -1.0316038131713867) ?  (b('l8b5') <= 2945.7542724609375) ?  (b('l8b5') <= 2857.7957763671875) ?  -0.026306773658521937 :   -0.014453342077168946 :   (b('dg0_v_vv') <= -3.0703907012939453) ?  -0.02001649447249368 :   0.024695145680918365 :   (b('l8b5') <= 3268.2584228515625) ?  (b('ndvi') <= 4686.0) ?  0.03710332491783339 :   0.0792961713391115 :   (b('grass') <= 65.33258819580078) ?  0.043257810641132266 :   -0.023188749158494018 :   (b('ndvi_med') <= 3656.25) ?  (b('l8b11') <= 1482.8832397460938) ?  -0.026359965863761242 :   -0.022271492438749985 :   -0.07650877379303483 :   (b('l8b4_med') <= 1560.6340942382812) ?  (b('dg0_v_vv') <= 3.872476100921631) ?  (b('crops') <= 24.002659797668457) ?  (b('l8b5_med') <= 3095.4654541015625) ?  (b('ndvi') <= 4495.5) ?  (b('dg0_v_vv') <= -0.23527288436889648) ?  0.0519291377635269 :   0.0032378594746924755 :   (b('ndvi_med') <= 3693.75) ?  -0.024902236621549514 :   0.00943072428676683 :   (b('l8b5') <= 3741.2606201171875) ?  (b('l8b5') <= 3710.624755859375) ?  0.04677567571325241 :   0.1546920692108768 :   (b('ndvi') <= 4653.0) ?  0.07194337892586149 :   -0.010579965937791726 :   (b('ndvi_med') <= 1820.25) ?  -0.03569886418739457 :   (b('ndvi') <= 4948.5) ?  (b('l8b4_med') <= 753.0480346679688) ?  0.024792385004764057 :   -0.008204681873562667 :   0.0763499830162819 :   (b('dg0_v_vv') <= 5.276907920837402) ?  (b('l8b4') <= 630.9300384521484) ?  0.09031749420899693 :   0.14310391043588466 :   (b('l8b5_med') <= 2386.617919921875) ?  0.031908877984982154 :   0.03949943296129116 :   (b('l8b4') <= 975.287109375) ?  (b('dg0_v_vv') <= 0.11980962753295898) ?  0.12477907162080731 :   (b('ndvi') <= 4489.0) ?  0.030111661548450686 :   -0.0017963574489064382 :   0.1683151283780437 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11_med') <= 1640.7611083984375) ?  (b('ndvi_med') <= 3209.0) ?  (b('l8b4') <= 574.0782470703125) ?  (b('l8b4') <= 573.5109252929688) ?  (b('l8b5') <= 1306.3219604492188) ?  (b('ndvi') <= 1600.0) ?  (b('dg0_v_vv') <= 0.781484842300415) ?  (b('l8b5') <= 1176.4415283203125) ?  -0.02243304976354288 :   -0.020482342398880152 :   (b('bulk') <= 119.0) ?  0.014327612038955273 :   0.039036361569272954 :   (b('l8b4') <= 504.6840515136719) ?  (b('l8b11') <= 937.8614196777344) ?  (b('dg0_v_vv') <= 2.1356701850891113) ?  (b('dg0_v_vv') <= 0.07693719863891602) ?  -0.02357835057578997 :   -0.03481844940195428 :   -0.022486550326123886 :   -0.04216196818699129 :   (b('l8b4') <= 535.1441955566406) ?  (b('l8b11_med') <= 1102.9674682617188) ?  -0.01698472410296338 :   (b('ndvi') <= 1897.0) ?  -0.005850059386986309 :   -0.00891837100314613 :   (b('l8b5') <= 1240.5958862304688) ?  -0.014193330948121044 :   (b('ndvi_med') <= 2072.75) ?  -0.028318305215836836 :   -0.022750241099966456 :   (b('crops') <= 18.819849967956543) ?  (b('l8b5') <= 2590.6973876953125) ?  (b('l8b4') <= 429.0371856689453) ?  (b('grass') <= 28.702491760253906) ?  (b('l8b11_med') <= 1189.1582641601562) ?  0.09142736323139294 :   0.10688183606804064 :   (b('ndvi') <= 3693.5) ?  0.02324807068030992 :   0.04839039483245174 :   (b('l8b11') <= 913.6739196777344) ?  (b('ndvi') <= 3530.0) ?  -0.0038712173906936477 :   -0.06886188398252804 :   (b('l8b11') <= 932.021240234375) ?  0.05615457798364034 :   0.007721260722370087 :   (b('l8b11') <= 972.0619812011719) ?  (b('moss') <= 5.297415494918823) ?  0.10244483702947144 :   (b('ndvi') <= 3223.0) ?  -0.024944932917693046 :   0.014101140203679963 :   (b('dg0_v_vv') <= 0.9617547988891602) ?  (b('ndvi') <= 3387.0) ?  0.029456756193084823 :   0.06816352845040974 :   (b('dg0_v_vv') <= 1.041844367980957) ?  -0.06224839402091914 :   0.04791544680749854 :   (b('l8b11') <= 1028.835693359375) ?  (b('l8b4') <= 528.5904846191406) ?  (b('bulk') <= 130.5) ?  (b('l8b11') <= 873.8999633789062) ?  0.035842559414576376 :   0.008996743305182464 :   (b('l8b4') <= 367.22666931152344) ?  -0.00826240878607741 :   0.007629003769447693 :   (b('ndvi') <= 4379.5) ?  (b('l8b5_med') <= 2527.20849609375) ?  0.003507663593991725 :   -0.026265749959842554 :   (b('l8b5_med') <= 2740.2227783203125) ?  -0.08955897666633322 :   -0.04644302444699851 :   (b('l8b5') <= 4093.056396484375) ?  (b('moss') <= 7.979443788528442) ?  (b('dg0_v_vv') <= -0.8051137924194336) ?  -0.0017868436382533909 :   0.02327097914865071 :   (b('dg0_v_vv') <= 0.3184967041015625) ?  0.012311349133320246 :   -0.013006581120965852 :   (b('sand') <= 35.0) ?  0.08187916665818654 :   0.06263137409875191 :   (b('dg0_v_vv') <= 0.9917248487472534) ?  (b('dg0_v_vv') <= 0.3665032386779785) ?  (b('dg0_v_vv') <= -0.19818973541259766) ?  (b('dg0_v_vv') <= -0.4708716869354248) ?  0.09738490372084241 :   0.0493848958530293 :   0.11206035426471755 :   (b('dg0_v_vv') <= 0.7475342750549316) ?  0.04792505126364155 :   0.06701602097337689 :   -0.027677314000334963 :   (b('l8b11_med') <= 1452.3944091796875) ?  (b('l8b5_med') <= 3010.2628173828125) ?  (b('l8b5') <= 3849.50927734375) ?  (b('ndvi') <= 3747.5) ?  (b('grass') <= 43.033124923706055) ?  (b('ndvi') <= 2808.5) ?  (b('l8b5') <= 2314.2464599609375) ?  -0.0013733889757698015 :   -0.016538971958564647 :   (b('bulk') <= 140.5) ?  -0.0006840180806727536 :   0.0248242983996336 :   (b('dg0_v_vv') <= -1.1317954063415527) ?  (b('l8b5') <= 2626.0445556640625) ?  0.032365420884892374 :   -0.022584093033833106 :   (b('l8b4_med') <= 857.9595947265625) ?  -0.013552107201607173 :   -0.03799480123257813 :   (b('l8b4_med') <= 706.6049499511719) ?  (b('l8b4') <= 616.5752563476562) ?  (b('l8b4') <= 608.9883728027344) ?  -0.03598675135433569 :   0.0464228967829802 :   (b('l8b5_med') <= 2724.859619140625) ?  0.02566992274262718 :   -0.05884686457778289 :   (b('dg0_v_vv') <= 1.3760428428649902) ?  (b('ndvi') <= 3826.5) ?  -0.059460688215581856 :   -0.0023631537590041284 :   0.11035175489952637 :   (b('l8b11') <= 1361.93408203125) ?  (b('l8b11') <= 1185.5) ?  (b('dg0_v_vv') <= 1.8120677471160889) ?  (b('ndvi') <= 3847.0) ?  0.01555025143700716 :   0.010644892959262589 :   (b('l8b5_med') <= 2542.1353759765625) ?  0.03036816008224441 :   0.018206926213341367 :   (b('ndvi_med') <= 3114.25) ?  -0.03992136909388927 :   -0.03234725157058002 :   (b('dg0_v_vv') <= -1.3675651550292969) ?  (b('dg0_v_vv') <= -2.1445441246032715) ?  0.04461082182185212 :   0.051610821106596386 :   (b('ndvi_med') <= 3141.0) ?  (b('ndvi_med') <= 3114.25) ?  0.07111513150297943 :   0.0606851818313725 :   0.0881593818282248 :   (b('l8b11') <= 1573.2389526367188) ?  (b('l8b4') <= 589.0303955078125) ?  (b('ndvi') <= 2792.0) ?  (b('dg0_v_vv') <= 0.18125534057617188) ?  0.02159032172555711 :   0.024040376433212912 :   (b('ndvi') <= 3100.5) ?  -0.0206303438366445 :   (b('dg0_v_vv') <= 0.3423643112182617) ?  -0.05731306151583779 :   -0.036389534422519826 :   (b('ndvi') <= 2916.0) ?  (b('l8b11_med') <= 1394.9105834960938) ?  (b('l8b11') <= 1169.0029907226562) ?  0.019364992993613727 :   0.04914577654084346 :   (b('l8b5') <= 3114.77099609375) ?  -0.03944806640255781 :   -0.0051473949253030005 :   (b('ndvi') <= 3059.0) ?  (b('moss') <= 8.271678790450096) ?  0.00842675948775233 :   -0.05252425115968776 :   (b('l8b11') <= 1243.763671875) ?  -0.0016214217822279498 :   0.025884276948885985 :   (b('l8b4') <= 1123.2493286132812) ?  (b('l8b4_med') <= 634.6933898925781) ?  (b('l8b5') <= 2304.328369140625) ?  (b('l8b11') <= 1814.999755859375) ?  3.466021629699445e-05 :   -0.029283571000551175 :   (b('l8b5_med') <= 3137.9461669921875) ?  0.03245689002851548 :   0.05754072527176993 :   (b('l8b5') <= 3464.419189453125) ?  (b('l8b11') <= 1613.5081787109375) ?  -0.018835817071593083 :   -0.041610617400131916 :   -0.07322764748175417 :   (b('ndvi_med') <= 2907.75) ?  (b('dg0_v_vv') <= -0.6675329208374023) ?  (b('dg0_v_vv') <= -1.0003938674926758) ?  0.012778008480594294 :   0.015085818510485682 :   (b('l8b4_med') <= 671.4358825683594) ?  0.0015333340585344524 :   0.0050858131460676526 :   (b('l8b11') <= 1783.95263671875) ?  0.061021281725960885 :   0.11066112691528315 :   (b('ndvi_med') <= 1484.5) ?  (b('ndvi') <= 2043.0) ?  (b('l8b4') <= 841.9072875976562) ?  (b('l8b11_med') <= 1557.6192016601562) ?  -0.09373939538843087 :   -0.05920738469449577 :   (b('l8b4') <= 960.8248291015625) ?  (b('l8b11') <= 1630.177734375) ?  (b('dg0_v_vv') <= -0.5410571098327637) ?  0.007535704419987882 :   0.06209885994306452 :   0.13625401117299715 :   (b('ndvi') <= 1133.0) ?  0.08317285900265901 :   (b('l8b5') <= 1865.673828125) ?  -0.011329080237377357 :   0.03306110988051455 :   (b('dg0_v_vv') <= 1.2081060409545898) ?  (b('l8b4') <= 689.7305297851562) ?  (b('dg0_v_vv') <= 0.4010810852050781) ?  (b('l8b11') <= 1137.5054931640625) ?  0.08744633192554252 :   0.08652549921072356 :   0.08048429873788315 :   0.14017847511024262 :   (b('dg0_v_vv') <= 1.6522490978240967) ?  -0.00021987314056820528 :   (b('l8b4') <= 760.1174926757812) ?  0.07638550115911458 :   (b('dg0_v_vv') <= 2.101846933364868) ?  0.030293213697064864 :   0.03849497974591204 :   (b('moss') <= 1.4817460775375366) ?  (b('dg0_v_vv') <= -0.9928369522094727) ?  (b('l8b5') <= 1872.2509155273438) ?  0.07735419000883878 :   (b('ndvi') <= 2361.0) ?  (b('crops') <= 59.70044708251953) ?  -0.01870864036639014 :   -0.01134027320434755 :   (b('ndvi') <= 2455.0) ?  0.01063272472002047 :   0.020388305687069078 :   (b('ndvi') <= 2454.5) ?  (b('dg0_v_vv') <= -0.043297767639160156) ?  (b('bulk') <= 133.5) ?  0.08135810620888234 :   0.08211394178125353 :   0.07398334909090673 :   (b('l8b5') <= 2992.5037841796875) ?  (b('l8b5') <= 2320.0234375) ?  0.040773674217202294 :   0.0015956141125141599 :   (b('dg0_v_vv') <= 0.2124037742614746) ?  0.055483042765127444 :   0.03927139094800811 :   (b('grass') <= 70.16188049316406) ?  (b('dg0_v_vv') <= -2.938995838165283) ?  (b('ndvi') <= 3499.5) ?  0.10850197634895611 :   0.0576878655306549 :   (b('l8b5') <= 3439.56201171875) ?  (b('l8b11') <= 1275.856201171875) ?  -0.027308295864240148 :   -0.0007769842083825344 :   (b('l8b11') <= 1594.88623046875) ?  0.006357566199783586 :   0.029729961847086362 :   (b('dg0_v_vv') <= -0.5331578254699707) ?  (b('l8b11') <= 1395.749755859375) ?  (b('dg0_v_vv') <= -1.7622504234313965) ?  -0.017863383105313713 :   -0.038337370527449476 :   (b('dg0_v_vv') <= -1.4005203247070312) ?  -0.0104304934674374 :   0.015140966360466008 :   (b('ndvi') <= 2901.5) ?  (b('l8b5_med') <= 2480.0167236328125) ?  0.0200669668725323 :   -0.008181222365796098 :   (b('dg0_v_vv') <= 1.343761920928955) ?  0.07400403792356598 :   -0.008121567852799733 :   (b('dg0_v_vv') <= -1.0475702285766602) ?  (b('l8b11_med') <= 1551.8579711914062) ?  (b('l8b5_med') <= 2906.153076171875) ?  (b('l8b5') <= 1504.6234741210938) ?  (b('dg0_v_vv') <= -2.325167179107666) ?  0.024091622712435257 :   (b('dg0_v_vv') <= -1.4912667274475098) ?  0.054821162751390984 :   (b('l8b4_med') <= 743.5720825195312) ?  0.032325056828537535 :   0.037809443307272844 :   (b('ndvi_med') <= 4218.0) ?  (b('l8b4') <= 627.088134765625) ?  (b('l8b11') <= 1180.6652221679688) ?  (b('dg0_v_vv') <= -1.3611736297607422) ?  -0.0011957771658374617 :   -0.02539193205577741 :   (b('l8b4_med') <= 629.2539672851562) ?  -0.06540615303203873 :   -0.02366667933155679 :   (b('l8b4') <= 699.0259094238281) ?  (b('l8b5') <= 2799.06005859375) ?  0.004087547911213309 :   0.058925194735826594 :   (b('ndvi_med') <= 3827.0) ?  -0.0016668383321200993 :   -0.019439903087451072 :   0.07212563054632859 :   (b('sand') <= 44.5) ?  (b('sand') <= 29.5) ?  (b('l8b11') <= 1185.3327026367188) ?  (b('sand') <= 24.0) ?  (b('ndvi') <= 3272.0) ?  -0.000962224663631247 :   -0.02100478226172606 :   (b('l8b4') <= 436.94476318359375) ?  0.04385119203738419 :   0.006591855627051079 :   (b('l8b4') <= 820.974609375) ?  (b('l8b11') <= 1320.99169921875) ?  0.013768164524727764 :   0.066400021929004 :   (b('l8b11') <= 1835.4868774414062) ?  -0.0033758583096194007 :   0.013037263533528515 :   (b('l8b5') <= 3008.3736572265625) ?  (b('ndvi') <= 3355.5) ?  (b('grass') <= 25.353328704833984) ?  -0.004137547385502235 :   0.048250782201388606 :   (b('l8b5') <= 2776.975830078125) ?  -0.04195888102786717 :   0.03265079146109701 :   (b('l8b5') <= 3517.0904541015625) ?  (b('moss') <= 2.858971655368805) ?  -0.04768165589623106 :   -0.02087707250764262 :   (b('moss') <= 6.26469349861145) ?  0.020298264466406063 :   -0.008462954425617633 :   (b('dg0_v_vv') <= -2.5808610916137695) ?  (b('dg0_v_vv') <= -4.915847301483154) ?  (b('bulk') <= 131.5) ?  -0.04106846071625589 :   -0.07509608218840641 :   (b('ndvi_med') <= 3579.75) ?  0.03153025101518267 :   (b('dg0_v_vv') <= -4.737542629241943) ?  0.00911518505443254 :   -0.016389141931491637 :   (b('l8b4_med') <= 556.9691162109375) ?  (b('dg0_v_vv') <= -1.7825736999511719) ?  (b('dg0_v_vv') <= -2.4201836585998535) ?  -0.015910479102044142 :   0.019148065242069967 :   (b('grass') <= 34.11734199523926) ?  -0.01715806306842382 :   0.006152452627318292 :   (b('l8b11') <= 834.3133850097656) ?  0.12098856371692268 :   (b('bulk') <= 132.5) ?  0.01753663522090865 :   0.0422547684750787 :   (b('l8b5_med') <= 2964.8408203125) ?  (b('l8b5') <= 2732.795654296875) ?  (b('dg0_v_vv') <= -3.245818614959717) ?  (b('ndvi') <= 3240.0) ?  (b('l8b11_med') <= 1558.62744140625) ?  -0.04872085783855555 :   -0.03808654247590591 :   -0.01872240899133351 :   (b('l8b4') <= 1020.9499816894531) ?  -0.017866552381597886 :   (b('ndvi') <= 2893.0) ?  -0.0037454834882972993 :   -0.006334281156293753 :   (b('l8b5_med') <= 2904.0380859375) ?  (b('dg0_v_vv') <= -1.7762947082519531) ?  (b('grass') <= 50.165876388549805) ?  (b('l8b4_med') <= 856.0473022460938) ?  -0.02244990090299062 :   -0.015715631028770688 :   (b('bulk') <= 135.5) ?  -0.031886972997225205 :   -0.03888716440391763 :   -0.05487679463157233 :   (b('l8b4') <= 924.6289367675781) ?  -0.06473218549812201 :   -0.05161677794407307 :   (b('l8b11') <= 2135.5943603515625) ?  (b('l8b4') <= 621.7221984863281) ?  (b('l8b4_med') <= 805.8084716796875) ?  -0.07558371425311505 :   -0.0417995327916153 :   (b('moss') <= 1.1365958750247955) ?  (b('l8b5') <= 3643.7518310546875) ?  (b('l8b5') <= 3416.932373046875) ?  0.0012363994307014736 :   0.017578777510340093 :   (b('l8b4') <= 728.2198181152344) ?  0.05119718294167125 :   0.06255359574815456 :   (b('l8b5') <= 3110.5174560546875) ?  (b('l8b4') <= 809.8025207519531) ?  0.02735162658501952 :   -0.00764286246273616 :   (b('l8b5_med') <= 3131.349853515625) ?  -0.045935898003042167 :   -0.015316877556709163 :   (b('ndvi') <= 2624.0) ?  0.04202660631065491 :   0.07454709944737303 :   (b('grass') <= 46.14527893066406) ?  (b('ndvi') <= 1816.5) ?  (b('dg0_v_vv') <= -0.8737521171569824) ?  (b('bulk') <= 141.0) ?  0.08569900688093882 :   0.060854043157290094 :   (b('ndvi') <= 1796.5) ?  (b('l8b5') <= 2729.970458984375) ?  (b('ndvi_med') <= 3294.0) ?  (b('dg0_v_vv') <= -0.11177778244018555) ?  0.04247693257950117 :   0.01765360911491649 :   (b('ndvi_med') <= 3513.75) ?  0.00838275561795069 :   0.03205807151595266 :   (b('l8b5_med') <= 3589.1407470703125) ?  (b('dg0_v_vv') <= 0.11924219131469727) ?  0.026799668379743173 :   0.044259817072198254 :   0.01350812116656136 :   (b('l8b11_med') <= 1360.62841796875) ?  0.07373554883150069 :   0.05318986780634577 :   (b('ndvi_med') <= 4666.0) ?  (b('l8b4') <= 1712.5970458984375) ?  (b('l8b5') <= 2909.906005859375) ?  (b('l8b5') <= 2907.6568603515625) ?  (b('l8b11') <= 1487.2180786132812) ?  0.011767590584865416 :   0.001761055063971506 :   (b('ndvi') <= 4681.0) ?  0.0732755521824404 :   -0.018714017274637507 :   (b('l8b5') <= 2930.7578125) ?  (b('sand') <= 42.0) ?  -0.010274414909391474 :   -0.07562013608217458 :   (b('l8b5') <= 2932.2161865234375) ?  0.06740393429370692 :   0.0035928578857074857 :   (b('ndvi') <= 4173.0) ?  (b('l8b5_med') <= 2901.9791259765625) ?  0.07047563144222524 :   (b('dg0_v_vv') <= 1.1345481872558594) ?  0.06367653118687444 :   0.060768227782052375 :   (b('moss') <= 5.816312313079834) ?  0.03377436600082229 :   0.03226528694873554 :   (b('ndvi') <= 4796.0) ?  (b('dg0_v_vv') <= 0.23461437225341797) ?  (b('dg0_v_vv') <= -0.9252924919128418) ?  -0.010081150985761178 :   (b('l8b4') <= 933.4629821777344) ?  0.05477009796451387 :   0.07907651834479937 :   (b('l8b4_med') <= 630.9979553222656) ?  (b('ndvi') <= 4736.5) ?  0.025434137350944373 :   0.07063036408735524 :   (b('l8b4') <= 729.5439758300781) ?  -0.028785246720674396 :   0.006991152597926015 :   (b('l8b11') <= 1098.9609375) ?  (b('l8b11_med') <= 1259.6744995117188) ?  0.05394960280247957 :   0.0516780981922374 :   (b('ndvi') <= 4850.0) ?  (b('l8b4_med') <= 585.4060668945312) ?  -0.022481833022132303 :   -0.03748770613748052 :   (b('l8b11_med') <= 1432.4141235351562) ?  0.003410762491675219 :   -0.014029061855610703 :   (b('l8b4') <= 953.4806823730469) ?  (b('dg0_v_vv') <= 2.7814483642578125) ?  (b('crops') <= 24.002659797668457) ?  (b('l8b11') <= 1632.3124389648438) ?  (b('ndvi') <= 4954.5) ?  (b('moss') <= 3.0389994382858276) ?  0.015329435177073612 :   0.003934177586063274 :   (b('dg0_v_vv') <= -0.5626401901245117) ?  0.054067677191554 :   -0.05854076871898422 :   (b('crops') <= 5.973352909088135) ?  (b('dg0_v_vv') <= -0.47954893112182617) ?  -0.07652329926667269 :   0.011019976938921372 :   (b('ndvi_med') <= 3905.5) ?  -0.09794428222067356 :   -0.05852518476075412 :   (b('l8b4') <= 550.6309814453125) ?  (b('sand') <= 22.5) ?  (b('l8b4') <= 458.2522735595703) ?  0.07549237167634404 :   0.08074861340995454 :   (b('l8b4') <= 216.6506118774414) ?  0.008437537016201058 :   0.02678290128317111 :   (b('l8b5') <= 3586.145751953125) ?  (b('dg0_v_vv') <= 0.7764830589294434) ?  0.01800823359876357 :   0.0029826378419310568 :   (b('crops') <= 35.19958972930908) ?  -0.039270955764263316 :   -0.06885789641373136 :   (b('moss') <= 11.782705307006836) ?  (b('l8b5') <= 2340.709228515625) ?  0.10240185875620483 :   (b('l8b5_med') <= 2942.537353515625) ?  (b('l8b11') <= 1536.2868041992188) ?  0.015799545920062098 :   0.038890464007986236 :   (b('ndvi') <= 3922.5) ?  0.05808712964448942 :   0.03760316313518382 :   (b('grass') <= 57.1409854888916) ?  0.12879351939229622 :   0.11179442588865629 :   (b('grass') <= 55.53681182861328) ?  (b('ndvi') <= 3622.5) ?  (b('sand') <= 43.0) ?  (b('dg0_v_vv') <= -0.9696741104125977) ?  0.014254233647518072 :   (b('dg0_v_vv') <= -0.3446474075317383) ?  0.07328014488697994 :   0.05265825541027121 :   0.10769392585712101 :   (b('dg0_v_vv') <= 0.38541698455810547) ?  (b('l8b11_med') <= 1163.0340576171875) ?  0.003588826243422688 :   -0.011293871645199197 :   (b('l8b11') <= 1951.610107421875) ?  0.025512389975549277 :   (b('moss') <= 5.546014428138733) ?  0.04548240803430237 :   0.06219704101198764 :   (b('l8b11') <= 1596.66259765625) ?  (b('ndvi') <= 2265.5) ?  0.07285278873619216 :   (b('l8b5') <= 2914.97900390625) ?  0.061213177503614935 :   0.05568085909708631 :   (b('l8b4') <= 985.532470703125) ?  (b('dg0_v_vv') <= -0.4655342102050781) ?  0.017452988481817394 :   (b('moss') <= 2.901568591594696) ?  -0.07549943575323645 :   -0.04660201700653496 :   (b('dg0_v_vv') <= -0.7335958480834961) ?  (b('l8b11') <= 1788.4483642578125) ?  0.02565128182275661 :   -0.0373573439813058 :   (b('l8b4') <= 1229.2562255859375) ?  0.03985000072929308 :   0.004479586846309603 :   (b('crops') <= 59.718482971191406) ?  (b('dg0_v_vv') <= 1.8312621116638184) ?  (b('l8b5_med') <= 1639.760498046875) ?  (b('ndvi') <= 1364.0) ?  (b('ndvi') <= 1015.5) ?  0.047557570548614786 :   (b('dg0_v_vv') <= -0.15194225311279297) ?  (b('ndvi') <= 1255.0) ?  -0.009257358324298859 :   -0.037687512380401494 :   (b('dg0_v_vv') <= 0.08797883987426758) ?  (b('dg0_v_vv') <= -0.055240631103515625) ?  -0.04545496058599931 :   -0.04465509454289472 :   (b('dg0_v_vv') <= 0.2869143486022949) ?  -0.061269416988605205 :   -0.06254908416610355 :   (b('dg0_v_vv') <= -0.3276653289794922) ?  (b('l8b11') <= 1952.705322265625) ?  -0.024696706842770214 :   (b('dg0_v_vv') <= -0.6807079315185547) ?  0.04817308565739037 :   (b('l8b4') <= 1092.3994750976562) ?  -0.00482691744205116 :   (b('l8b4') <= 1165.0744018554688) ?  0.0009582442419991261 :   0.0027033021436455584 :   (b('l8b11') <= 1631.9219360351562) ?  (b('l8b11') <= 1520.6646728515625) ?  0.004021754252603177 :   (b('dg0_v_vv') <= 0.4474482536315918) ?  0.03158778638399776 :   0.043996392540291 :   (b('l8b5') <= 1617.006591796875) ?  0.20837212343317105 :   (b('dg0_v_vv') <= -0.022337913513183594) ?  (b('l8b4') <= 1064.7377014160156) ?  0.025973314960610533 :   0.06590235195956448 :   (b('dg0_v_vv') <= 0.09553670883178711) ?  0.16995254513270142 :   0.11910658001032925 :   (b('dg0_v_vv') <= -9.176032066345215) ?  (b('ndvi') <= 1400.5) ?  0.09325768966021758 :   (b('l8b5') <= 2442.6444091796875) ?  0.05937367510919214 :   0.05693433374818091 :   (b('ndvi') <= 3410.0) ?  (b('sand') <= 36.5) ?  (b('crops') <= 37.68832015991211) ?  (b('l8b11_med') <= 2170.646484375) ?  (b('l8b5') <= 3222.83984375) ?  -0.01352162529504565 :   -0.033393613607928176 :   (b('l8b11') <= 1647.9447021484375) ?  -0.03874004275418891 :   0.0009187829368552196 :   (b('ndvi_med') <= 3156.0) ?  (b('dg0_v_vv') <= 0.1841568946838379) ?  0.006966035237952507 :   0.035179995713884867 :   (b('l8b11') <= 1924.7809448242188) ?  -0.009673293823571634 :   0.0031422216856901814 :   (b('l8b4') <= 849.2717895507812) ?  (b('bulk') <= 148.5) ?  (b('l8b4_med') <= 1399.739990234375) ?  -0.0013560937214732952 :   -0.04099458560199014 :   (b('l8b4_med') <= 983.2742614746094) ?  -0.007080955675889509 :   0.045032780106476364 :   (b('l8b4') <= 855.1279602050781) ?  (b('l8b4_med') <= 791.267822265625) ?  0.001565263011308049 :   -0.05031969435357684 :   (b('grass') <= 98.81764602661133) ?  -0.005019876985560233 :   0.00457966789585528 :   (b('ndvi') <= 3569.0) ?  (b('ndvi') <= 3567.5) ?  (b('l8b5_med') <= 3146.203857421875) ?  (b('l8b11_med') <= 1662.1273803710938) ?  -0.005588716262985136 :   0.021716286334790703 :   (b('dg0_v_vv') <= -2.3295912742614746) ?  0.023170227320116088 :   -0.015031440546485442 :   (b('dg0_v_vv') <= -1.1218957901000977) ?  (b('l8b11_med') <= 1983.5992431640625) ?  0.042885282159537075 :   0.04752335001900959 :   (b('l8b5') <= 3161.9705810546875) ?  0.07412868554937481 :   0.10808582672910294 :   (b('l8b11_med') <= 2116.72216796875) ?  (b('dg0_v_vv') <= 0.27248382568359375) ?  (b('l8b11_med') <= 2071.3255615234375) ?  -0.009219519454745381 :   -0.05417193396752314 :   (b('crops') <= 43.96723556518555) ?  0.010774151667429538 :   -0.008214084308987202 :   (b('l8b11_med') <= 2184.90380859375) ?  (b('l8b11') <= 1196.7498168945312) ?  0.11793838359511374 :   0.05576763197513712 :   (b('l8b4') <= 473.26661682128906) ?  -0.0800327440928117 :   0.014575411029842715 :   (b('l8b4') <= 1078.9041137695312) ?  (b('l8b4') <= 1031.5861206054688) ?  (b('l8b4_med') <= 817.2890625) ?  (b('dg0_v_vv') <= 2.5692176818847656) ?  (b('ndvi') <= 4288.5) ?  (b('dg0_v_vv') <= 2.3174686431884766) ?  (b('dg0_v_vv') <= 2.0280277729034424) ?  -0.004400439518955947 :   -0.00915557709199411 :   0.02039186998649886 :   (b('ndvi') <= 4621.0) ?  0.056384241023894194 :   (b('ndvi_med') <= 4002.0) ?  0.027008227315112543 :   0.03593699118007593 :   (b('l8b5') <= 2304.696044921875) ?  (b('l8b11') <= 1709.693359375) ?  (b('ndvi_med') <= 3517.0) ?  0.019257931294121622 :   0.009829247418642264 :   (b('dg0_v_vv') <= 7.305660009384155) ?  -0.02433864044677869 :   0.0075310681074915176 :   (b('l8b5') <= 2593.635009765625) ?  (b('l8b4') <= 755.8218994140625) ?  -0.13258516196284778 :   -0.08418482681438946 :   (b('ndvi') <= 4905.5) ?  -0.04125290341982958 :   0.03499274915452244 :   (b('dg0_v_vv') <= 8.271571159362793) ?  (b('l8b4') <= 404.8888854980469) ?  (b('l8b5_med') <= 2638.8831787109375) ?  (b('dg0_v_vv') <= 3.205066204071045) ?  0.006486953655542901 :   0.03538431924473562 :   0.13647933045565386 :   (b('l8b11_med') <= 2398.3350830078125) ?  (b('l8b11') <= 868.146484375) ?  -0.047948716293341166 :   0.005633266344980637 :   (b('dg0_v_vv') <= 2.721799373626709) ?  0.056175002554648634 :   0.013818486684222607 :   (b('crops') <= 34.67532825469971) ?  (b('l8b4_med') <= 837.4677429199219) ?  -0.06509865927074515 :   -0.06793061750230922 :   -0.009380521099665709 :   (b('l8b11_med') <= 2315.244140625) ?  (b('l8b4_med') <= 1093.2243041992188) ?  (b('l8b4') <= 1060.2015991210938) ?  (b('dg0_v_vv') <= 2.669462203979492) ?  (b('l8b5') <= 2564.342041015625) ?  0.03676684602161891 :   0.011086961797692967 :   (b('l8b11') <= 1935.1183471679688) ?  0.03375213592340892 :   0.05964256498091275 :   (b('l8b11') <= 2143.2713623046875) ?  (b('l8b5') <= 2496.2098388671875) ?  -0.025077298377324427 :   -0.058019784813695435 :   0.02569103216499763 :   (b('dg0_v_vv') <= 2.5803041458129883) ?  (b('l8b5') <= 1814.8385009765625) ?  0.06628153658969815 :   (b('l8b11_med') <= 2081.9632568359375) ?  0.04639925271297973 :   0.03987444306183888 :   (b('dg0_v_vv') <= 2.835458755493164) ?  (b('l8b11') <= 2000.3820190429688) ?  0.07644640643867104 :   0.06775153116141075 :   (b('ndvi') <= 1423.5) ?  0.035521513726753884 :   0.06211623198861471 :   (b('l8b11_med') <= 2373.598388671875) ?  -0.05060008901339605 :   (b('bulk') <= 154.5) ?  (b('l8b4_med') <= 1408.4089965820312) ?  -0.019162259774477053 :   -0.018099833808635307 :   0.0017279699806639626 :   (b('sand') <= 40.5) ?  (b('l8b4_med') <= 1368.6868896484375) ?  (b('l8b4_med') <= 1190.7529296875) ?  (b('l8b11_med') <= 1644.9713745117188) ?  (b('dg0_v_vv') <= 2.1314492225646973) ?  0.04435566569580594 :   (b('dg0_v_vv') <= 2.169907808303833) ?  0.006157223251640781 :   0.012547720223863829 :   (b('l8b11') <= 1789.9073486328125) ?  (b('l8b11') <= 1766.8340454101562) ?  0.0015398342517828484 :   0.0530386424669506 :   (b('dg0_v_vv') <= 1.8780008554458618) ?  0.041083219653048925 :   -0.02529106364644546 :   (b('l8b5') <= 2133.3143310546875) ?  (b('l8b4_med') <= 1227.81005859375) ?  0.08617009080733185 :   (b('dg0_v_vv') <= 2.6969470977783203) ?  0.05448148265518594 :   0.02478544822753044 :   (b('dg0_v_vv') <= 2.502274751663208) ?  (b('dg0_v_vv') <= 2.3158535957336426) ?  -0.021394090070004604 :   -0.0725749441264209 :   (b('dg0_v_vv') <= 2.903097152709961) ?  0.03257624875550935 :   0.016218412839505657 :   (b('l8b4_med') <= 1693.5594482421875) ?  (b('l8b4') <= 1611.55908203125) ?  (b('l8b5_med') <= 2697.6890869140625) ?  (b('ndvi') <= 1435.0) ?  -0.062434207868695135 :   -0.04179858111174381 :   (b('l8b5') <= 2377.2249755859375) ?  -0.14323169931991575 :   -0.08029590735853366 :   0.0303419235003356 :   (b('l8b5') <= 2702.1734619140625) ?  (b('l8b11_med') <= 2844.0006103515625) ?  (b('ndvi_med') <= 1699.0) ?  0.09249430376459197 :   0.13814019743440814 :   (b('l8b4_med') <= 1932.1525268554688) ?  0.054372410576923186 :   -0.012229383161804441 :   (b('bulk') <= 156.0) ?  (b('dg0_v_vv') <= 2.5459580421447754) ?  -0.06487190024731668 :   -0.12536855111808815 :   (b('l8b11_med') <= 2972.988037109375) ?  -0.030532425768783178 :   0.010818175190407342 :   (b('sand') <= 42.5) ?  (b('ndvi') <= 1936.5) ?  (b('dg0_v_vv') <= 4.204803943634033) ?  (b('bulk') <= 128.5) ?  0.06349886387106485 :   (b('ndvi') <= 1254.5) ?  -0.053810791911224884 :   -0.0004465232190245033 :   (b('ndvi') <= 1508.5) ?  (b('ndvi') <= 1294.5) ?  0.04753575415006467 :   0.05334686009372941 :   (b('l8b11') <= 2620.8883056640625) ?  0.060843939315701645 :   0.06810208503404841 :   (b('grass') <= 54.93748474121094) ?  0.11202869627074141 :   (b('l8b5_med') <= 2740.8353271484375) ?  (b('dg0_v_vv') <= 1.997736930847168) ?  0.04499601927081161 :   0.05676623718135486 :   (b('l8b11') <= 2595.5494384765625) ?  0.03177816662806817 :   0.05078050441534837 :   (b('ndvi_med') <= 1398.0) ?  (b('l8b11_med') <= 1967.8757934570312) ?  (b('l8b5_med') <= 2134.5518798828125) ?  (b('moss') <= 19.204002857208252) ?  -0.0029904339950932723 :   0.034662467063579006 :   (b('l8b11_med') <= 1923.8294677734375) ?  0.06157320662300105 :   0.10639343388914102 :   (b('l8b5') <= 1999.2464599609375) ?  (b('l8b4_med') <= 1253.315185546875) ?  0.0037055270000318427 :   -0.07141119104219054 :   (b('l8b4') <= 1460.8067626953125) ?  0.04573172331124432 :   0.002162668169556233 :   (b('ndvi') <= 2404.5) ?  (b('grass') <= 37.6097354888916) ?  (b('sand') <= 49.0) ?  0.008955590671531979 :   -0.009575813364706226 :   (b('l8b5') <= 1945.9208984375) ?  0.013375547365526807 :   -0.03302240255515363 :   (b('crops') <= 41.639442443847656) ?  (b('crops') <= 20.76351547241211) ?  0.027459594762687767 :   0.07488600248180512 :   (b('ndvi') <= 2553.0) ?  -0.07810820336706285 :   -0.00332810124994504 :   (b('l8b11_med') <= 2629.3411865234375) ?  (b('grass') <= 17.282055854797363) ?  (b('crops') <= 98.52266693115234) ?  (b('l8b4') <= 2351.68359375) ?  (b('dg0_v_vv') <= -1.7073373794555664) ?  (b('ndvi') <= 3271.0) ?  (b('l8b5') <= 1930.4747924804688) ?  (b('l8b4') <= 1067.5691528320312) ?  -0.013384115991659316 :   0.04124727786088919 :   (b('ndvi') <= 2097.0) ?  -0.020282197015752988 :   -0.008782390738027633 :   (b('dg0_v_vv') <= -1.7750601768493652) ?  (b('l8b11_med') <= 1803.5111083984375) ?  0.0003008051111714652 :   0.021817962883327102 :   (b('l8b11_med') <= 1698.0573120117188) ?  0.03197679750602622 :   0.07706667226130592 :   (b('bulk') <= 154.5) ?  (b('ndvi') <= 2705.0) ?  (b('l8b5') <= 1691.8463745117188) ?  0.03640692245695426 :   0.006111260014138873 :   (b('l8b4') <= 2042.506103515625) ?  -0.002497364692862408 :   0.059395841621976975 :   (b('dg0_v_vv') <= 2.659532070159912) ?  (b('dg0_v_vv') <= 1.815584659576416) ?  -0.011423030001462133 :   0.02288879331589881 :   (b('l8b5') <= 3675.981201171875) ?  -0.056318807665153826 :   -0.0022432100668836052 :   (b('l8b4_med') <= 1559.7488403320312) ?  (b('l8b4_med') <= 1531.5534057617188) ?  (b('dg0_v_vv') <= 0.8892006874084473) ?  (b('ndvi') <= 1054.0) ?  0.017488312902012554 :   0.018974523407414262 :   0.008361132044864938 :   (b('ndvi') <= 2589.5) ?  (b('ndvi') <= 2339.0) ?  -0.0166096153102473 :   -0.007188403356257013 :   (b('dg0_v_vv') <= 0.2757704257965088) ?  -0.022204943004739495 :   -0.029527735664932622 :   (b('ndvi') <= 2137.0) ?  (b('ndvi') <= 1645.5) ?  (b('l8b5') <= 3360.6278076171875) ?  -0.05672049278485085 :   -0.047652687544803986 :   (b('l8b11_med') <= 2393.7314453125) ?  -0.07242071781734322 :   -0.06256311230988992 :   -0.02765555945181107 :   (b('ndvi') <= 1085.0) ?  (b('dg0_v_vv') <= 1.1045234203338623) ?  (b('dg0_v_vv') <= -0.8515529632568359) ?  (b('dg0_v_vv') <= -3.567035675048828) ?  0.014234500899716118 :   (b('bulk') <= 122.0) ?  0.02833018310874652 :   0.030862951264950256 :   (b('dg0_v_vv') <= 0.35985374450683594) ?  0.03812285392241793 :   0.05179781491507279 :   0.10232107825222764 :   (b('ndvi') <= 1436.5) ?  (b('dg0_v_vv') <= -0.8751006126403809) ?  (b('l8b4') <= 1577.0166625976562) ?  (b('dg0_v_vv') <= -1.992560863494873) ?  -0.009265586640971768 :   -0.011813803478462198 :   (b('l8b5_med') <= 2687.2421875) ?  -0.005285436648142058 :   -0.002688511100480101 :   (b('dg0_v_vv') <= 1.0245466232299805) ?  (b('ndvi_med') <= 2424.5) ?  -0.026858352639621816 :   -0.038047822471853954 :   -0.05474365215096105 :   (b('l8b4') <= 694.6284790039062) ?  (b('ndvi') <= 2017.0) ?  (b('dg0_v_vv') <= -2.688937187194824) ?  -0.017085193455157438 :   0.026487828715609632 :   (b('sand') <= 30.0) ?  -0.003615176972029075 :   -0.033755099188508886 :   (b('l8b5') <= 2805.314208984375) ?  (b('l8b5_med') <= 2687.2421875) ?  0.018090787567329382 :   0.04871637835436445 :   (b('ndvi') <= 4099.0) ?  0.006293547458174122 :   0.03342965301735161 :   (b('sand') <= 55.5) ?  (b('l8b5') <= 3615.517822265625) ?  (b('dg0_v_vv') <= 1.128126621246338) ?  (b('ndvi') <= 1186.5) ?  (b('l8b4') <= 2129.3331298828125) ?  (b('dg0_v_vv') <= -0.7134613990783691) ?  -0.014204659098704655 :   0.01315576703001281 :   -0.07870325454447347 :   (b('l8b11') <= 3207.0740966796875) ?  (b('l8b11') <= 2868.623291015625) ?  0.01882882182403888 :   -0.011653080923572161 :   (b('dg0_v_vv') <= -0.17861700057983398) ?  0.08995190512023905 :   0.10708713641731096 :   (b('l8b5') <= 2753.4462890625) ?  (b('l8b4') <= 1564.7359008789062) ?  (b('l8b5') <= 2614.4005126953125) ?  0.02904351110255247 :   -0.06176512012594587 :   (b('l8b11') <= 2439.8270263671875) ?  -0.07856472838117665 :   -0.01628520756128692 :   (b('grass') <= 19.662006378173828) ?  (b('l8b11_med') <= 2589.752685546875) ?  0.05390172470682086 :   0.0734267654121735 :   (b('ndvi') <= 2029.0) ?  0.0028034988659748905 :   0.043911685412396234 :   (b('l8b4') <= 519.5205078125) ?  (b('dg0_v_vv') <= -1.8036465644836426) ?  (b('l8b11') <= 912.6813049316406) ?  0.05310933886536934 :   (b('l8b4_med') <= 1334.3555297851562) ?  0.07110476481709715 :   0.07704502612792336 :   0.030343116483319443 :   (b('l8b4') <= 1478.9137573242188) ?  (b('ndvi') <= 2818.5) ?  (b('l8b4') <= 1083.2637939453125) ?  0.039996693334110774 :   -0.022999704055977847 :   (b('ndvi') <= 3137.5) ?  -0.053107965708027484 :   -0.012201905437437174 :   (b('ndvi') <= 2970.5) ?  (b('l8b5') <= 4148.666748046875) ?  0.005733920544482494 :   -0.033656697901723535 :   (b('l8b11_med') <= 2589.752685546875) ?  0.03651562004357128 :   0.07881147751580678 :   (b('dg0_v_vv') <= 1.2183833122253418) ?  (b('l8b4') <= 1622.5228271484375) ?  (b('l8b5') <= 2965.2862548828125) ?  (b('ndvi') <= 1220.0) ?  -0.0031149258169090865 :   (b('ndvi') <= 2852.5) ?  -0.053825954881780484 :   -0.029942602570602984 :   0.04837389096083769 :   (b('l8b4') <= 1716.4805908203125) ?  0.02156253725170766 :   0.04631547879720839 :   (b('l8b4') <= 1395.3943481445312) ?  (b('l8b5') <= 3721.849365234375) ?  (b('l8b4') <= 757.3621826171875) ?  (b('l8b5') <= 3095.0025634765625) ?  0.04560111637904142 :   -0.013478312425502792 :   (b('l8b5') <= 2119.7183837890625) ?  -0.034715755985110675 :   -0.013905130379524691 :   (b('dg0_v_vv') <= 7.052134037017822) ?  -0.05966666482725122 :   -0.06408954695104065 :   (b('ndvi') <= 1673.5) ?  0.058889421177932916 :   0.10838504847154048 :   (b('grass') <= 1.900430142879486) ?  (b('ndvi') <= 1001.0) ?  (b('dg0_v_vv') <= -0.5321893692016602) ?  (b('ndvi_med') <= 1481.0) ?  (b('sand') <= 55.0) ?  -0.01567212808361329 :   -0.0139001707812927 :   -0.007408075272855225 :   (b('dg0_v_vv') <= 1.1996564865112305) ?  (b('l8b11') <= 3580.6287841796875) ?  (b('l8b5') <= 3101.6717529296875) ?  0.0780082278485533 :   0.07733560762562383 :   0.09325025269283369 :   (b('l8b11_med') <= 2890.52392578125) ?  (b('l8b4') <= 2277.0794677734375) ?  0.026860784416257638 :   0.03562556414103005 :   -0.0057598825592294345 :   (b('l8b4') <= 2840.0167236328125) ?  (b('l8b4') <= 2826.8741455078125) ?  (b('ndvi') <= 1031.5) ?  (b('l8b5') <= 2332.9769287109375) ?  (b('dg0_v_vv') <= 2.168123722076416) ?  -0.0033048464705056207 :   -0.0260585934330331 :   (b('l8b5_med') <= 3622.8421630859375) ?  -0.0389388663409176 :   -0.031169617266555844 :   (b('ndvi') <= 1155.5) ?  (b('ndvi') <= 1136.0) ?  0.0057933410502343016 :   0.04914097912809164 :   (b('dg0_v_vv') <= -3.714336395263672) ?  0.04062808365942547 :   0.001619585084869132 :   (b('l8b5_med') <= 3622.8421630859375) ?  (b('dg0_v_vv') <= -0.9126882553100586) ?  (b('ndvi') <= 1307.5) ?  0.08222708212619953 :   0.07394875711576518 :   (b('ndvi_med') <= 1349.5) ?  0.09494614500560411 :   -0.03905124091728154 :   -0.031717862913994865 :   (b('dg0_v_vv') <= -0.8617010116577148) ?  (b('l8b5_med') <= 3228.371826171875) ?  (b('ndvi') <= 1196.0) ?  (b('dg0_v_vv') <= -1.0711326599121094) ?  0.025352358612921497 :   0.027343809486210346 :   0.006343811631977558 :   -0.01788906826675099 :   (b('dg0_v_vv') <= -0.47966718673706055) ?  (b('l8b5_med') <= 3622.8421630859375) ?  (b('ndvi_med') <= 1481.0) ?  -0.01689530842027076 :   -0.021835822290960552 :   (b('dg0_v_vv') <= -0.5149469375610352) ?  -0.022463573337004054 :   -0.027268331476585556 :   (b('l8b4_med') <= 2271.3209228515625) ?  (b('l8b5') <= 3917.762451171875) ?  0.00878714900176493 :   -0.012004321381628751 :   (b('dg0_v_vv') <= 0.5772318840026855) ?  -0.019910721161549034 :   -0.011010923310427943 :   (b('l8b11') <= 2090.24853515625) ?  (b('ndvi_med') <= 1971.5) ?  (b('l8b11_med') <= 2691.7864990234375) ?  (b('ndvi') <= 1272.5) ?  (b('ndvi') <= 1221.0) ?  -0.05028789566578731 :   (b('dg0_v_vv') <= 0.520228385925293) ?  0.0034328969705830215 :   -0.016205766905365238 :   (b('dg0_v_vv') <= 2.239193916320801) ?  (b('ndvi') <= 2115.5) ?  0.021480185861413436 :   -0.00042030415046334646 :   (b('dg0_v_vv') <= 2.8115038871765137) ?  0.048830994072804224 :   0.05638432260436213 :   (b('l8b5') <= 1858.9292602539062) ?  (b('ndvi') <= 2412.5) ?  (b('ndvi_med') <= 1482.0) ?  -0.012033205306191327 :   -0.018449197979567025 :   (b('dg0_v_vv') <= 0.9757966995239258) ?  -0.04921160581232168 :   -0.032711907929736876 :   (b('l8b5') <= 2344.54833984375) ?  (b('l8b4') <= 891.4112243652344) ?  -0.014663198724800434 :   0.0007893617319333039 :   (b('l8b4') <= 1034.3926391601562) ?  -0.04488315065787662 :   -0.017222707791064443 :   (b('l8b4') <= 1076.2619323730469) ?  -0.10858088220878451 :   (b('dg0_v_vv') <= 1.3658733367919922) ?  -0.06357352341608402 :   -0.052629412584116464 :   (b('l8b4') <= 1751.7907104492188) ?  (b('dg0_v_vv') <= -1.448591709136963) ?  (b('l8b11') <= 3112.627685546875) ?  (b('l8b11') <= 2788.4525146484375) ?  (b('l8b4') <= 1730.7117309570312) ?  -0.019765357928197273 :   0.042235973529956006 :   (b('dg0_v_vv') <= -1.817601203918457) ?  0.03360063662690705 :   -0.0024386753465209868 :   (b('l8b5') <= 2549.923583984375) ?  (b('l8b5_med') <= 2462.2298583984375) ?  -0.02633808846684603 :   -0.05327464658045483 :   -0.0924474365249705 :   (b('l8b11') <= 2682.5858154296875) ?  (b('l8b11') <= 2625.5745849609375) ?  (b('dg0_v_vv') <= -1.4384827613830566) ?  0.07135007147814745 :   0.003907953968566475 :   (b('l8b5_med') <= 3387.197509765625) ?  -0.044923452203258045 :   0.016317115597622778 :   (b('ndvi') <= 1282.5) ?  (b('ndvi_med') <= 1766.0) ?  -0.018566007994472358 :   0.04539226637594325 :   (b('l8b5_med') <= 3184.25537109375) ?  0.036899016398774936 :   0.008711451085665215 :   (b('dg0_v_vv') <= 2.2184648513793945) ?  (b('l8b11') <= 2790.7933349609375) ?  (b('l8b4_med') <= 1909.0737915039062) ?  (b('l8b4') <= 1981.9378662109375) ?  -0.032920268970971694 :   0.011386907125979145 :   (b('dg0_v_vv') <= -0.5150651931762695) ?  -0.01930030109797323 :   -0.004116991232381421 :   (b('l8b4') <= 2065.824462890625) ?  (b('l8b11_med') <= 2830.743408203125) ?  0.013873314017287272 :   -0.00970250663870631 :   (b('sand') <= 55.5) ?  -0.010102169746120878 :   0.07053645647390885 :   (b('dg0_v_vv') <= 2.8181381225585938) ?  (b('l8b11') <= 3381.7911376953125) ?  (b('ndvi_med') <= 1636.25) ?  -0.004198745947654073 :   0.022955287975009918 :   (b('ndvi_med') <= 1563.75) ?  0.04403356509824943 :   0.08997146089629537 :   (b('l8b5') <= 3731.70849609375) ?  (b('dg0_v_vv') <= 3.323301315307617) ?  -0.013817252057174637 :   0.005794302589077044 :   -0.06443940226227471 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4_med') <= 656.9165344238281) ?  (b('grass') <= 46.1603946685791) ?  (b('sand') <= 34.5) ?  (b('l8b5') <= 2674.6324462890625) ?  (b('l8b5_med') <= 2149.7215576171875) ?  (b('ndvi') <= 3076.0) ?  (b('ndvi') <= 2587.0) ?  (b('l8b5_med') <= 2124.7242431640625) ?  (b('dg0_v_vv') <= -0.016399383544921875) ?  (b('l8b4') <= 684.400634765625) ?  0.03303467291647978 :   0.06029070936526092 :   (b('l8b11') <= 1318.602294921875) ?  -0.01597886356208325 :   0.0065802818517324 :   (b('dg0_v_vv') <= -0.3590569496154785) ?  (b('ndvi') <= 2439.0) ?  0.04236850600565209 :   0.005147577178688162 :   (b('l8b5') <= 1888.170654296875) ?  0.02143237397544656 :   -0.05763148803388343 :   (b('dg0_v_vv') <= -1.1618003845214844) ?  (b('l8b5') <= 2062.6799926757812) ?  0.024683348475967398 :   0.0601935180662907 :   (b('l8b5') <= 1740.0596923828125) ?  (b('ndvi') <= 2734.0) ?  -0.0051300082037231964 :   0.030051952664151588 :   (b('l8b4') <= 678.7948913574219) ?  -0.022638318713310207 :   -0.002515567973692736 :   (b('l8b11') <= 1087.2033081054688) ?  (b('dg0_v_vv') <= -0.5277905464172363) ?  -0.02961244942330417 :   (b('dg0_v_vv') <= 0.09965372085571289) ?  (b('l8b11') <= 1072.085205078125) ?  0.06476581801828926 :   -0.0020809622000046157 :   (b('l8b11_med') <= 1092.3614501953125) ?  0.007625938472259665 :   -0.0272582267030112 :   (b('ndvi') <= 3205.0) ?  (b('l8b4') <= 596.0245971679688) ?  0.09541855670342628 :   0.1324788318765621 :   (b('ndvi') <= 3501.0) ?  (b('l8b4') <= 647.6182556152344) ?  0.08432597203267508 :   0.057547632380159586 :   0.028357019044583875 :   (b('l8b4') <= 448.14292907714844) ?  (b('l8b5_med') <= 2445.125244140625) ?  (b('l8b5_med') <= 2163.919677734375) ?  (b('dg0_v_vv') <= -0.7164101600646973) ?  -0.0023390571934161497 :   (b('dg0_v_vv') <= -0.5839738845825195) ?  0.02293636076639638 :   -0.002157008229351942 :   (b('dg0_v_vv') <= -1.0025568008422852) ?  0.004644953005883468 :   (b('dg0_v_vv') <= -0.8439569473266602) ?  0.049530962540489665 :   0.034806359042630075 :   (b('l8b4') <= 430.4367370605469) ?  0.0511631180139156 :   0.0819714389074897 :   (b('dg0_v_vv') <= 2.3980941772460938) ?  (b('l8b4_med') <= 614.513916015625) ?  (b('l8b11') <= 993.251708984375) ?  (b('l8b11') <= 896.7096252441406) ?  -0.002394228763610458 :   0.03657176275722072 :   (b('l8b11') <= 1223.2763061523438) ?  -0.03309021786829132 :   -0.008329926166470141 :   (b('l8b11') <= 1049.6544799804688) ?  (b('ndvi') <= 3319.5) ?  -0.02614608899157013 :   0.049694033905116876 :   (b('ndvi') <= 2417.0) ?  -0.015027699673522854 :   0.013095030355855233 :   (b('dg0_v_vv') <= 2.754549503326416) ?  (b('ndvi_med') <= 3456.25) ?  0.05519721805439154 :   (b('grass') <= 25.040468215942383) ?  0.04958077882173678 :   0.04636536515545969 :   0.013819126576339047 :   (b('l8b5') <= 2690.174560546875) ?  (b('dg0_v_vv') <= -0.08211946487426758) ?  0.11091447010063021 :   (b('dg0_v_vv') <= 0.21057510375976562) ?  (b('l8b5_med') <= 2724.859619140625) ?  0.061928531932686826 :   0.05633471495683262 :   0.06892853121743109 :   (b('l8b11_med') <= 1264.1098022460938) ?  (b('l8b11') <= 1216.054443359375) ?  (b('sand') <= 15.0) ?  (b('dg0_v_vv') <= -1.2013797760009766) ?  -0.027047968728038163 :   (b('dg0_v_vv') <= 0.7093358039855957) ?  0.051614140099616936 :   0.06907666353918758 :   (b('ndvi') <= 3737.5) ?  (b('sand') <= 31.0) ?  0.006805862566041911 :   0.036814078069698886 :   (b('l8b4') <= 599.8335266113281) ?  0.008516670348342005 :   -0.0497413156964394 :   (b('moss') <= 2.3444817066192627) ?  (b('l8b4_med') <= 576.6534423828125) ?  (b('ndvi') <= 2593.0) ?  0.051718429212085526 :   0.03040153653243846 :   (b('dg0_v_vv') <= 1.2066097259521484) ?  0.020357984470805807 :   -0.007057223202194179 :   (b('dg0_v_vv') <= 1.8539729118347168) ?  (b('l8b5_med') <= 2724.859619140625) ?  -0.0022759442191179467 :   -0.026898831194697593 :   (b('l8b11') <= 1540.444580078125) ?  0.029039125466322313 :   0.03741963070353557 :   (b('dg0_v_vv') <= 0.36875104904174805) ?  (b('l8b11') <= 1081.3784790039062) ?  (b('grass') <= 32.015384674072266) ?  (b('l8b4_med') <= 593.1185607910156) ?  0.0010819385012840188 :   0.02325960781629305 :   0.046266406632961526 :   (b('l8b11') <= 1808.604248046875) ?  (b('l8b11') <= 1146.8344116210938) ?  0.10092640412143083 :   0.06738848516955015 :   (b('l8b11') <= 2404.8184814453125) ?  0.04164293966786914 :   0.03360885738465619 :   (b('dg0_v_vv') <= 0.4994802474975586) ?  (b('l8b5') <= 3066.5975341796875) ?  -0.09390197602699443 :   -0.05253413757441061 :   (b('ndvi') <= 4239.5) ?  -0.08481892698966961 :   (b('dg0_v_vv') <= 1.327714204788208) ?  0.023640611841124693 :   0.052953466861602315 :   (b('l8b4_med') <= 631.238525390625) ?  (b('l8b5_med') <= 2728.8701171875) ?  (b('ndvi') <= 2057.5) ?  (b('ndvi') <= 1973.0) ?  (b('ndvi_med') <= 2020.5) ?  (b('l8b5') <= 1746.0) ?  (b('ndvi') <= 1848.5) ?  -0.0024166631390828744 :   0.03325841502502822 :   (b('l8b11') <= 1157.2734375) ?  0.08764641334875817 :   0.03557135966208154 :   (b('l8b4') <= 504.6840515136719) ?  (b('l8b5') <= 1111.59814453125) ?  -0.04191231156693058 :   -0.037945771368292164 :   (b('dg0_v_vv') <= -1.2244212627410889) ?  -0.018935397755532265 :   0.0012178767650281531 :   (b('dg0_v_vv') <= 0.20453715324401855) ?  0.1008543188382458 :   0.05606701918937301 :   (b('ndvi') <= 2469.0) ?  (b('dg0_v_vv') <= 0.20554852485656738) ?  (b('l8b4') <= 567.5296630859375) ?  (b('l8b11') <= 1052.973388671875) ?  -0.07380360833081268 :   -0.12452161208431561 :   (b('l8b11_med') <= 1102.9674682617188) ?  -0.04307904791167279 :   -0.02599677698562124 :   (b('l8b4') <= 581.5952758789062) ?  0.0528596634122353 :   (b('dg0_v_vv') <= 1.6441679000854492) ?  -0.017403674943027258 :   -0.059003672471460794 :   (b('ndvi_med') <= 2977.75) ?  (b('dg0_v_vv') <= 0.8842697143554688) ?  (b('l8b11') <= 945.5400695800781) ?  -0.030018440432142947 :   0.013214608442089729 :   (b('dg0_v_vv') <= 0.9647471904754639) ?  0.07506469588073567 :   0.056368236688876694 :   (b('l8b11') <= 903.0593872070312) ?  (b('ndvi') <= 4585.5) ?  -0.039047605658378046 :   -0.01202999229434143 :   (b('ndvi') <= 3414.0) ?  -0.03440200994937675 :   -0.00665732923943975 :   (b('l8b5_med') <= 3866.939697265625) ?  (b('l8b5_med') <= 3797.0421142578125) ?  (b('l8b4') <= 1068.9710693359375) ?  (b('l8b11') <= 846.9893493652344) ?  (b('ndvi') <= 2910.0) ?  0.06193504586636295 :   -0.011101909142791227 :   (b('l8b11') <= 898.6204833984375) ?  0.018580446290138195 :   0.002013340508686768 :   (b('l8b4') <= 1265.3308715820312) ?  (b('l8b4') <= 1172.7314453125) ?  -0.014395897625376231 :   -0.06198242661441524 :   (b('dg0_v_vv') <= -1.451416015625) ?  -0.013007544851899842 :   0.00809041235180436 :   (b('l8b5') <= 3822.22265625) ?  (b('dg0_v_vv') <= -0.38111448287963867) ?  -0.012915848510058447 :   -0.014914065937081361 :   (b('dg0_v_vv') <= 0.06178140640258789) ?  -0.07576805548911433 :   -0.08254609269195887 :   (b('dg0_v_vv') <= -0.5180745124816895) ?  (b('crops') <= 44.476247787475586) ?  (b('sand') <= 45.0) ?  -0.09114552052496538 :   -0.04468133794935919 :   (b('l8b4') <= 858.407958984375) ?  (b('l8b4') <= 496.2580108642578) ?  0.011342173629147446 :   -0.02458939479589549 :   (b('l8b11_med') <= 948.3958435058594) ?  0.06826749561936052 :   0.01590663028610345 :   (b('sand') <= 68.5) ?  (b('ndvi_med') <= 3935.5) ?  (b('dg0_v_vv') <= 0.2792510986328125) ?  0.03790621838577114 :   0.011818323188202437 :   (b('l8b11') <= 1268.0337524414062) ?  0.028463192223833342 :   0.05545072899115199 :   (b('ndvi') <= 2382.0) ?  (b('ndvi') <= 1838.0) ?  -0.002163817355250175 :   0.0098361920622837 :   (b('grass') <= 9.845983266830444) ?  -0.015603287191128745 :   -0.0059292495025264624 :   (b('dg0_v_vv') <= -0.14025640487670898) ?  (b('dg0_v_vv') <= -0.1819472312927246) ?  (b('l8b5') <= 3556.963134765625) ?  (b('dg0_v_vv') <= -0.34860658645629883) ?  (b('l8b5_med') <= 3255.6241455078125) ?  (b('l8b5') <= 1507.779296875) ?  0.03395835679083367 :   0.009128702907642332 :   -0.02752324082869889 :   (b('l8b5_med') <= 2732.007080078125) ?  0.02147815971637998 :   (b('l8b11_med') <= 1337.3145751953125) ?  0.034240498372321335 :   0.036416825268143355 :   (b('dg0_v_vv') <= -0.23656892776489258) ?  (b('dg0_v_vv') <= -0.3555617332458496) ?  (b('ndvi_med') <= 3656.75) ?  -0.015024764841865101 :   -0.030253280818909878 :   -0.049005536436772135 :   0.024156899205968363 :   -0.0572702961295565 :   (b('l8b5') <= 3202.434814453125) ?  (b('dg0_v_vv') <= 0.15096473693847656) ?  (b('l8b11') <= 1120.7523193359375) ?  0.16438232365983013 :   (b('dg0_v_vv') <= -0.04819297790527344) ?  0.07734357052912944 :   0.011736582057041922 :   (b('dg0_v_vv') <= 1.5890419483184814) ?  (b('ndvi_med') <= 4204.25) ?  (b('ndvi') <= 3443.5) ?  0.007444516851344202 :   0.03833179888016064 :   0.0718547111227878 :   (b('dg0_v_vv') <= 3.092658519744873) ?  -0.0024175334199495746 :   0.004908639898360351 :   (b('ndvi') <= 3948.5) ?  (b('ndvi') <= 3471.0) ?  0.002070831630148706 :   -0.06741959602973055 :   (b('l8b11') <= 1263.9186401367188) ?  (b('ndvi') <= 4204.5) ?  0.0419020082906085 :   0.02950896360686303 :   (b('dg0_v_vv') <= 0.250730037689209) ?  (b('l8b11') <= 1433.4063110351562) ?  0.00407838916930181 :   -0.006060376094906894 :   0.022961820590918086 :   (b('crops') <= 19.45354127883911) ?  (b('l8b5') <= 2290.2811279296875) ?  (b('bulk') <= 117.0) ?  (b('l8b4') <= 503.88868713378906) ?  (b('dg0_v_vv') <= 1.0389728546142578) ?  (b('sand') <= 54.5) ?  -0.014706851219568773 :   -0.0201897447871886 :   0.0005497197184558822 :   (b('l8b4_med') <= 547.3807067871094) ?  -0.05157474663218234 :   -0.031747551849622774 :   (b('l8b5') <= 2085.4517822265625) ?  (b('l8b5') <= 1929.6607055664062) ?  (b('ndvi') <= 1964.0) ?  (b('l8b4_med') <= 635.489990234375) ?  (b('l8b5') <= 1693.902587890625) ?  0.10150647639947735 :   0.0862768168837259 :   0.004352571454812937 :   (b('ndvi_med') <= 3625.75) ?  (b('dg0_v_vv') <= -0.763063907623291) ?  0.019698988008087875 :   -0.0015619955609083036 :   (b('dg0_v_vv') <= 0.7839741706848145) ?  0.03422585371952434 :   -0.023329223944935314 :   (b('ndvi') <= 3452.5) ?  (b('sand') <= 41.5) ?  -0.036377156025991564 :   (b('l8b4') <= 708.9756774902344) ?  -0.024734908377081376 :   -0.017383321116246648 :   (b('dg0_v_vv') <= -1.1093130111694336) ?  -0.025634494701526706 :   (b('l8b11') <= 1285.7540283203125) ?  -0.010905031130536369 :   0.008640756019749146 :   (b('l8b11') <= 1294.9093017578125) ?  (b('dg0_v_vv') <= -1.350956916809082) ?  (b('l8b11_med') <= 1178.0994873046875) ?  (b('dg0_v_vv') <= -1.9102134704589844) ?  0.03346717975536451 :   0.041510891860487904 :   0.009516963386504429 :   (b('ndvi') <= 4069.0) ?  (b('dg0_v_vv') <= -0.8707394599914551) ?  -0.019248578158698054 :   0.00162785108137222 :   (b('dg0_v_vv') <= -0.36977148056030273) ?  0.03683044279275516 :   0.016226948595693694 :   (b('l8b11') <= 1359.861328125) ?  (b('l8b4_med') <= 510.6105041503906) ?  0.12766468072243303 :   0.029380957838736843 :   (b('ndvi_med') <= 3450.0) ?  (b('dg0_v_vv') <= -0.4923372268676758) ?  0.05597844637048002 :   0.006145372030448966 :   (b('ndvi') <= 4375.5) ?  0.049166584629889085 :   0.026448892855135946 :   (b('dg0_v_vv') <= 1.1006879806518555) ?  (b('ndvi') <= 3336.0) ?  (b('ndvi') <= 2332.0) ?  (b('ndvi_med') <= 2794.25) ?  (b('dg0_v_vv') <= -0.24224519729614258) ?  (b('l8b11') <= 1451.3627319335938) ?  -0.013507741828938644 :   -0.016956043792035982 :   (b('l8b4') <= 953.7499084472656) ?  -0.019306197293865533 :   -0.02622368087897127 :   (b('ndvi_med') <= 3601.0) ?  -0.03899162871899475 :   -0.05191648533799384 :   (b('l8b4') <= 809.4548034667969) ?  (b('l8b4') <= 654.4994201660156) ?  (b('l8b11_med') <= 1244.5859985351562) ?  0.014658866469552098 :   0.07811051180823114 :   (b('l8b11') <= 1400.3070678710938) ?  -0.03933148843304535 :   -0.0014676666879247756 :   (b('l8b5') <= 2533.9686279296875) ?  (b('ndvi_med') <= 3336.0) ?  0.03447950183658677 :   0.0675385004465147 :   (b('l8b5') <= 2683.614013671875) ?  -0.036932986041222235 :   0.03113797289628943 :   (b('ndvi') <= 3714.5) ?  (b('l8b11_med') <= 1029.2247314453125) ?  0.02335967075841444 :   (b('crops') <= 3.3066861629486084) ?  (b('l8b11') <= 1054.8099670410156) ?  -0.03379766731441394 :   -0.018557580380457288 :   (b('dg0_v_vv') <= 0.04023933410644531) ?  -0.03343994365831473 :   -0.05406171568992585 :   (b('ndvi_med') <= 3342.5) ?  (b('dg0_v_vv') <= -0.10171747207641602) ?  (b('ndvi') <= 4286.0) ?  -0.013839947845154712 :   0.032671542589284834 :   (b('dg0_v_vv') <= 0.49997377395629883) ?  0.12207235553340479 :   0.08726063498556424 :   (b('ndvi') <= 3809.0) ?  (b('dg0_v_vv') <= -0.0022830963134765625) ?  -0.004823942940880047 :   0.028918257700631295 :   (b('ndvi') <= 3972.5) ?  -0.04558546699056026 :   -0.013753530195536262 :   (b('l8b4') <= 466.43548583984375) ?  (b('dg0_v_vv') <= 1.1718049049377441) ?  (b('ndvi_med') <= 3447.5) ?  2.350731530820216e-05 :   (b('l8b4_med') <= 556.8836364746094) ?  0.04548315449916107 :   0.036402070404338405 :   (b('ndvi_med') <= 3196.5) ?  0.028639644528377473 :   (b('l8b4_med') <= 616.6837463378906) ?  (b('ndvi_med') <= 4124.5) ?  -0.005745630974581584 :   0.0016004845020338165 :   -0.021741640894569952 :   (b('l8b4_med') <= 587.7560729980469) ?  (b('ndvi_med') <= 4278.75) ?  (b('ndvi') <= 3924.0) ?  0.02321892963162936 :   (b('ndvi') <= 4836.0) ?  -0.0077059092541209295 :   0.005448270556404788 :   0.0532643960635267 :   (b('l8b5') <= 2694.6123046875) ?  0.056778294054742995 :   (b('dg0_v_vv') <= 1.2468833923339844) ?  (b('ndvi') <= 3922.0) ?  0.024358699600034284 :   0.026664207928238648 :   (b('ndvi_med') <= 3196.5) ?  0.04370370798189882 :   0.03698733547868353 :   (b('dg0_v_vv') <= -1.4382972717285156) ?  (b('l8b4_med') <= 624.4176635742188) ?  (b('l8b4_med') <= 564.5051879882812) ?  (b('moss') <= 9.99922800064087) ?  (b('bulk') <= 112.0) ?  (b('dg0_v_vv') <= -2.0969905853271484) ?  0.012901183324391247 :   0.013668970095514543 :   (b('l8b11') <= 1488.6668701171875) ?  0.008169385877313473 :   0.00972894999068008 :   (b('dg0_v_vv') <= -2.1965279579162598) ?  0.019122577859340018 :   (b('l8b4') <= 545.9503173828125) ?  0.017546399744749253 :   0.016297143719435914 :   (b('l8b5') <= 2663.33837890625) ?  (b('moss') <= 5.734982252120972) ?  0.01489658550831946 :   0.021145139848332217 :   (b('l8b5_med') <= 2743.3323974609375) ?  0.032855415275650224 :   (b('l8b11') <= 1245.9182739257812) ?  0.023861536997318744 :   0.02481934564920274 :   (b('ndvi') <= 4504.5) ?  (b('l8b5') <= 2427.334228515625) ?  (b('l8b5') <= 2361.711669921875) ?  (b('ndvi_med') <= 3405.0) ?  (b('l8b5') <= 1895.348876953125) ?  0.02168246044119171 :   -0.008333340255725859 :   (b('l8b11') <= 820.2210845947266) ?  -0.024380257601086253 :   -0.011776547706021542 :   (b('ndvi_med') <= 3442.5) ?  (b('ndvi_med') <= 3405.0) ?  0.004599628300172853 :   -0.015141941807842757 :   (b('ndvi') <= 3293.5) ?  0.016055374262093774 :   0.007597103363120222 :   (b('dg0_v_vv') <= -1.5209050178527832) ?  (b('dg0_v_vv') <= -1.6420135498046875) ?  -0.026606773914557735 :   -0.022396077853199026 :   -0.03500242291276212 :   (b('dg0_v_vv') <= -1.7755470275878906) ?  0.048459869533245936 :   (b('dg0_v_vv') <= -1.5884208679199219) ?  0.018259595636735848 :   0.019380679294507097 :   (b('l8b5') <= 3065.4569091796875) ?  (b('ndvi') <= 4443.5) ?  (b('ndvi') <= 2791.5) ?  (b('l8b5_med') <= 2592.1824951171875) ?  (b('sand') <= 65.5) ?  (b('dg0_v_vv') <= 0.6374268531799316) ?  0.029935742585989966 :   0.02903575417313292 :   (b('l8b5') <= 2463.2713623046875) ?  0.04021330549412344 :   0.050919452342499893 :   (b('dg0_v_vv') <= 0.020669937133789062) ?  (b('moss') <= 9.99922800064087) ?  0.00736596919713884 :   0.02661526712162543 :   (b('ndvi') <= 2218.5) ?  0.03407260909158327 :   0.022030332465973988 :   (b('l8b5') <= 2485.499267578125) ?  (b('dg0_v_vv') <= 0.7525968551635742) ?  (b('dg0_v_vv') <= -0.001247406005859375) ?  0.0029808620172172787 :   0.0258431306320141 :   (b('dg0_v_vv') <= 1.5058069229125977) ?  -0.022867078973768384 :   0.015423448764221644 :   (b('ndvi') <= 3530.5) ?  (b('dg0_v_vv') <= -1.2461071014404297) ?  -0.005606472064032503 :   0.012208806662790912 :   (b('l8b4_med') <= 636.951416015625) ?  0.014379747807504712 :   0.02391460536042174 :   (b('l8b4_med') <= 636.951416015625) ?  (b('ndvi') <= 4628.0) ?  (b('ndvi_med') <= 4175.5) ?  (b('dg0_v_vv') <= -0.617516040802002) ?  0.034946736274949264 :   0.03355065819599576 :   0.027002948354863887 :   (b('ndvi') <= 4890.5) ?  (b('dg0_v_vv') <= -0.00254058837890625) ?  -0.0042110943100836145 :   0.019257894028000516 :   (b('l8b5') <= 1631.2758178710938) ?  0.038312131932582844 :   0.018740824624141207 :   (b('dg0_v_vv') <= -0.25194883346557617) ?  (b('ndvi_med') <= 3494.75) ?  0.02407496078855037 :   0.027830396218603337 :   (b('dg0_v_vv') <= 0.4562835693359375) ?  0.04849847255514822 :   (b('l8b5') <= 2481.111328125) ?  0.03109038278387327 :   0.035332177351989735 :   (b('grass') <= 59.88348388671875) ?  (b('ndvi_med') <= 3208.25) ?  0.06562346170000644 :   (b('dg0_v_vv') <= 0.3805060386657715) ?  (b('l8b4') <= 582.9219970703125) ?  (b('l8b4_med') <= 636.951416015625) ?  0.024694714437166875 :   0.05240763984239266 :   (b('ndvi') <= 4327.5) ?  0.027398176172245298 :   -0.010362153931356421 :   (b('crops') <= 29.840476036071777) ?  (b('dg0_v_vv') <= 0.9125990867614746) ?  0.05227625537714461 :   0.03329099065907266 :   (b('ndvi') <= 2379.0) ?  0.04112870448265615 :   0.023264325402496063 :   (b('l8b4') <= 449.83900451660156) ?  (b('ndvi') <= 3386.0) ?  (b('dg0_v_vv') <= -0.36306047439575195) ?  0.019513926384556624 :   0.017188337041988133 :   (b('ndvi_med') <= 4027.0) ?  (b('dg0_v_vv') <= -0.2915635108947754) ?  0.003044587261157361 :   0.004644603065341109 :   0.008301376655818926 :   (b('l8b4') <= 550.2384796142578) ?  (b('l8b4') <= 483.7146453857422) ?  0.020574521414054914 :   (b('ndvi_med') <= 3541.0) ?  0.029928500114680234 :   0.028750703642186015 :   0.0056406923848726365 :   (b('l8b4') <= 1291.6817626953125) ?  (b('dg0_v_vv') <= -0.44379711151123047) ?  (b('sand') <= 28.5) ?  (b('moss') <= 3.965116262435913) ?  (b('dg0_v_vv') <= -0.4550895690917969) ?  (b('l8b11') <= 1185.3327026367188) ?  (b('ndvi') <= 2939.5) ?  (b('dg0_v_vv') <= -2.1707100868225098) ?  (b('sand') <= 22.0) ?  -0.023130084062280416 :   0.0150492368026136 :   (b('l8b11_med') <= 1295.4417114257812) ?  0.05595476017019455 :   0.02377212639651885 :   (b('ndvi') <= 4891.0) ?  (b('ndvi') <= 4259.0) ?  -0.0030392401534700924 :   -0.04061635324398545 :   (b('ndvi_med') <= 3712.0) ?  0.024627338927998493 :   0.0726737520689591 :   (b('l8b4_med') <= 751.4978637695312) ?  (b('l8b5') <= 2944.1864013671875) ?  (b('l8b5') <= 2857.4151611328125) ?  0.02988718664224882 :   -0.03023777663110848 :   (b('l8b5_med') <= 3139.8779296875) ?  0.06579002388196631 :   0.0033792914464244594 :   (b('l8b4') <= 1194.2427368164062) ?  (b('l8b11_med') <= 1345.9015502929688) ?  -0.014214995070565615 :   0.01281507688382007 :   (b('l8b11') <= 2053.774169921875) ?  0.03260010183095794 :   0.05932150798713856 :   (b('dg0_v_vv') <= -0.44731807708740234) ?  (b('grass') <= 80.05454635620117) ?  -0.06571400952432968 :   -0.06938236621062663 :   0.018299685648930586 :   (b('ndvi') <= 2798.0) ?  (b('l8b5') <= 2346.1044921875) ?  (b('l8b5') <= 2182.0460205078125) ?  (b('ndvi') <= 1970.5) ?  (b('dg0_v_vv') <= -0.7546005249023438) ?  0.009709660825671346 :   -0.0011203310934879696 :   (b('l8b11') <= 2018.614501953125) ?  -0.013540458099852642 :   -0.03915177163173354 :   (b('l8b5') <= 2299.8397216796875) ?  (b('ndvi') <= 2401.0) ?  -0.03068654287080163 :   -0.03814921768709312 :   (b('dg0_v_vv') <= -0.7728080749511719) ?  -0.049408671074336136 :   -0.06085777565792855 :   (b('moss') <= 13.121410369873047) ?  (b('dg0_v_vv') <= -3.594888687133789) ?  -0.04135667694822753 :   (b('dg0_v_vv') <= -2.2311830520629883) ?  0.024273145378243363 :   -0.006658869154543165 :   (b('l8b11') <= 1875.3310546875) ?  (b('l8b11') <= 1771.5919799804688) ?  0.02277843272001115 :   0.005721310635253218 :   (b('dg0_v_vv') <= -1.917220115661621) ?  0.04470036055397794 :   0.054296596510420014 :   (b('l8b11') <= 1880.1062622070312) ?  (b('dg0_v_vv') <= -0.4773898124694824) ?  (b('dg0_v_vv') <= -0.5474333763122559) ?  (b('ndvi_med') <= 3514.25) ?  0.0005350441488329648 :   0.018990930582926984 :   (b('l8b11') <= 1249.5195922851562) ?  -0.03304617608215601 :   0.004358687115313844 :   (b('bulk') <= 145.0) ?  0.0772381547979203 :   0.05980697927062856 :   (b('ndvi') <= 3497.0) ?  (b('l8b5_med') <= 3012.455322265625) ?  (b('l8b11') <= 2217.0755615234375) ?  -0.020610408200363004 :   0.055980158672944674 :   0.06709238950263574 :   (b('l8b11_med') <= 1570.7933349609375) ?  (b('l8b5') <= 3049.645751953125) ?  -0.027510504211240752 :   -0.04645510014966575 :   (b('bulk') <= 145.0) ?  -0.07784731747350455 :   -0.09369583890993696 :   (b('l8b4_med') <= 667.1445922851562) ?  (b('l8b4') <= 839.1568603515625) ?  (b('l8b5') <= 4421.3570556640625) ?  (b('l8b11') <= 1017.19873046875) ?  (b('l8b4_med') <= 662.4736022949219) ?  -0.0883491205284188 :   -0.0673494532157595 :   (b('ndvi') <= 2956.5) ?  -0.06022553313549352 :   (b('dg0_v_vv') <= -0.8133797645568848) ?  -0.04517209797243149 :   -0.03787415144772491 :   -0.011415350066280278 :   0.021777049730282133 :   (b('grass') <= 77.23037719726562) ?  (b('grass') <= 4.636483192443848) ?  (b('ndvi') <= 4710.5) ?  (b('l8b11') <= 1128.1316528320312) ?  (b('dg0_v_vv') <= -1.3693451881408691) ?  0.01048826479386556 :   0.0507328109463149 :   (b('l8b11') <= 1746.1196899414062) ?  -0.011339153017910077 :   0.008412256477509417 :   (b('dg0_v_vv') <= -1.8468904495239258) ?  (b('ndvi_med') <= 2662.75) ?  0.04014241009436345 :   0.04691490693970668 :   (b('l8b4') <= 806.987060546875) ?  0.04867659426401377 :   0.08654458517518615 :   (b('grass') <= 17.26266574859619) ?  (b('moss') <= 6.230724096298218) ?  (b('l8b11_med') <= 2099.3790283203125) ?  -0.0025685199784188447 :   -0.016315919502470324 :   (b('ndvi') <= 3499.0) ?  -0.02366910099319484 :   -0.0021800221816876803 :   (b('grass') <= 28.026954650878906) ?  (b('l8b11_med') <= 1248.4595336914062) ?  0.1461187945426047 :   -0.0002522390882912461 :   (b('bulk') <= 155.5) ?  -0.00398605032775349 :   -0.014741199431067569 :   (b('grass') <= 80.50270080566406) ?  (b('l8b5') <= 1581.0861206054688) ?  -0.092571511467088 :   (b('l8b5') <= 1887.90966796875) ?  (b('l8b11_med') <= 2033.3973999023438) ?  0.16040895789838022 :   0.054953767252002794 :   (b('sand') <= 43.5) ?  0.016757149483775956 :   0.12711072068588747 :   (b('l8b11') <= 2063.7479248046875) ?  (b('ndvi') <= 4523.0) ?  (b('ndvi_med') <= 1853.0) ?  -0.02589121072303323 :   -0.006716917769790965 :   (b('dg0_v_vv') <= -0.6304750442504883) ?  0.04474729928008444 :   -0.018953185399651185 :   (b('l8b11_med') <= 2252.62255859375) ?  (b('ndvi') <= 1127.0) ?  -0.036198830442704125 :   0.010238278896883727 :   (b('l8b4_med') <= 1472.1470947265625) ?  0.07443691658175186 :   0.013359727841474794 :   (b('l8b4_med') <= 1232.2153930664062) ?  (b('moss') <= 2.2014873027801514) ?  (b('sand') <= 46.0) ?  (b('l8b4') <= 300.52552795410156) ?  (b('dg0_v_vv') <= 1.6192598342895508) ?  (b('dg0_v_vv') <= 0.9111218452453613) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.07823895939648273 :   0.073859499390522 :   0.060644527500154866 :   (b('dg0_v_vv') <= 2.4698266983032227) ?  0.027393335756830728 :   (b('l8b4') <= 289.3401641845703) ?  0.029543147211122833 :   0.059444928535954455 :   (b('l8b4') <= 356.4667510986328) ?  (b('ndvi') <= 4636.5) ?  (b('dg0_v_vv') <= 1.1198554039001465) ?  -0.0162486174970527 :   -0.04966200208591465 :   (b('ndvi') <= 4761.5) ?  0.017330289295639387 :   -0.009153066498267168 :   (b('moss') <= 0.986830860376358) ?  (b('l8b4') <= 1262.3726196289062) ?  0.008918716678035364 :   -0.03371787412017032 :   (b('l8b4') <= 626.4974975585938) ?  -0.013994779901396711 :   0.03183628248036468 :   (b('bulk') <= 143.5) ?  (b('l8b4') <= 1177.49609375) ?  (b('ndvi') <= 1810.0) ?  (b('l8b11') <= 655.4804229736328) ?  -0.038137161665499186 :   0.19392160165916747 :   (b('l8b4') <= 718.6974182128906) ?  -0.008143106832025295 :   0.013157430312723418 :   (b('l8b5') <= 2448.1195068359375) ?  0.19705616854293512 :   (b('l8b5') <= 2485.045654296875) ?  0.08714774742896664 :   0.006345523967915823 :   (b('dg0_v_vv') <= 5.028898239135742) ?  (b('l8b4') <= 1034.122314453125) ?  (b('dg0_v_vv') <= 1.7498531341552734) ?  -0.02691585793955209 :   0.0005571423306360517 :   (b('ndvi') <= 2477.5) ?  -0.0226560953944681 :   0.005546538419155351 :   0.059014307709884695 :   (b('l8b11_med') <= 1283.3327026367188) ?  (b('l8b5') <= 2387.53076171875) ?  (b('l8b5') <= 2385.7657470703125) ?  (b('ndvi') <= 2063.0) ?  (b('bulk') <= 158.0) ?  -0.0347850181611694 :   -0.06272156844340443 :   (b('l8b4') <= 707.5579833984375) ?  0.015300995246059134 :   -0.012718233413887059 :   (b('ndvi_med') <= 2981.0) ?  0.06761680909698209 :   0.08900598251617434 :   (b('l8b5') <= 2568.5443115234375) ?  (b('grass') <= 57.274845123291016) ?  (b('l8b5_med') <= 2379.9134521484375) ?  -0.016663896144426565 :   -0.051841927705175185 :   -0.10041841341863364 :   (b('l8b11') <= 1024.542724609375) ?  (b('grass') <= 45.384334564208984) ?  0.0021914968886760783 :   0.13325621724834918 :   (b('moss') <= 13.155279636383057) ?  -0.02038948451267512 :   -0.05463103070099376 :   (b('l8b11_med') <= 2140.8316650390625) ?  (b('l8b11_med') <= 2137.002685546875) ?  (b('bulk') <= 138.5) ?  (b('l8b11_med') <= 1731.9598999023438) ?  0.0008443736765815917 :   -0.007257525951970863 :   (b('dg0_v_vv') <= -0.38654184341430664) ?  0.03640110890504971 :   0.0032620393567765965 :   (b('l8b5') <= 3016.8675537109375) ?  0.11756786280926312 :   0.11027554272824186 :   (b('l8b4') <= 694.5667724609375) ?  (b('l8b4_med') <= 1145.8306884765625) ?  0.14804313201677677 :   (b('dg0_v_vv') <= 2.487797737121582) ?  -0.0425762468084733 :   0.03736296059838238 :   (b('ndvi') <= 3011.5) ?  (b('sand') <= 64.0) ?  -0.015719530621707122 :   -0.06531523312882698 :   (b('dg0_v_vv') <= -0.09137773513793945) ?  0.09210672718998676 :   0.006837595492750179 :   (b('l8b5_med') <= 2384.2882080078125) ?  (b('l8b4') <= 1179.5181884765625) ?  (b('grass') <= 70.51102828979492) ?  (b('moss') <= 28.497751235961914) ?  (b('dg0_v_vv') <= 1.616631269454956) ?  (b('dg0_v_vv') <= 0.036583900451660156) ?  0.060032123724290364 :   0.11919009745689754 :   (b('bulk') <= 155.0) ?  0.04829785958073128 :   0.09353952118793413 :   (b('l8b11_med') <= 2168.12646484375) ?  (b('l8b11') <= 1990.5289916992188) ?  0.0071364758425427844 :   -0.029165387198387988 :   (b('dg0_v_vv') <= 0.28722143173217773) ?  -0.011471484960956846 :   0.08103896720718515 :   (b('dg0_v_vv') <= -0.122772216796875) ?  (b('l8b11') <= 1842.1444702148438) ?  (b('l8b5') <= 2519.71240234375) ?  -0.05729690689837952 :   -0.05394643044757291 :   (b('sand') <= 42.5) ?  -0.028356437031436317 :   0.004482630647285268 :   (b('l8b11') <= 1552.5728759765625) ?  (b('l8b11') <= 1211.8585205078125) ?  0.08779418670739014 :   0.0815191029611729 :   (b('l8b5') <= 1719.413330078125) ?  0.06637508249399661 :   0.007083766010113205 :   (b('l8b5') <= 2285.4937744140625) ?  (b('dg0_v_vv') <= 1.618344783782959) ?  (b('dg0_v_vv') <= 1.074077844619751) ?  (b('l8b11_med') <= 1725.197509765625) ?  0.08072311735504425 :   -0.011897663685728603 :   (b('l8b5_med') <= 1961.9083251953125) ?  0.07879037379973372 :   0.050054502883938576 :   (b('ndvi_med') <= 979.0) ?  -0.08381110855529028 :   (b('l8b4') <= 1243.2168579101562) ?  -0.055547433215914294 :   -0.05919883002592896 :   (b('l8b11_med') <= 2088.8492431640625) ?  0.20072527152662578 :   (b('ndvi') <= 2411.5) ?  (b('l8b11') <= 2275.2327880859375) ?  0.04765320983258836 :   0.11185515942698848 :   (b('l8b4') <= 1216.89697265625) ?  0.012126053262017045 :   -0.005053292754058615 :   (b('l8b5') <= 1633.147216796875) ?  (b('l8b5') <= 1523.1420288085938) ?  (b('bulk') <= 135.5) ?  -0.06749024963037642 :   (b('bulk') <= 147.0) ?  (b('ndvi') <= 2961.0) ?  0.024654046308662823 :   0.008672438848775887 :   (b('dg0_v_vv') <= 5.634562730789185) ?  0.0006938718464791771 :   -0.032834412913212485 :   (b('dg0_v_vv') <= 5.8818440437316895) ?  (b('l8b5_med') <= 2487.8336181640625) ?  -0.09906890228069005 :   (b('dg0_v_vv') <= 5.184751749038696) ?  -0.024514859469437416 :   0.027000911602282116 :   -0.09934826716163042 :   (b('l8b5') <= 1642.1716918945312) ?  0.12031894612896255 :   (b('bulk') <= 152.5) ?  (b('grass') <= 62.911848068237305) ?  (b('l8b5') <= 2032.012939453125) ?  0.014149936013668159 :   0.0024809689188425334 :   (b('ndvi') <= 2217.0) ?  -0.0073839414184336686 :   0.03731280930879736 :   (b('dg0_v_vv') <= 3.3772010803222656) ?  (b('ndvi') <= 4211.5) ?  -0.014285713622252205 :   0.03542870717629539 :   (b('l8b4_med') <= 1705.4586181640625) ?  0.062806395672286 :   -0.004669042293402488 :   (b('crops') <= 98.84536743164062) ?  (b('dg0_v_vv') <= 3.521144390106201) ?  (b('l8b5_med') <= 2627.353515625) ?  (b('dg0_v_vv') <= -0.4504356384277344) ?  (b('l8b4') <= 1445.5086059570312) ?  (b('l8b11_med') <= 2438.2244873046875) ?  (b('ndvi_med') <= 2922.0) ?  (b('ndvi_med') <= 2508.75) ?  0.0044707475483565465 :   -0.027116812812646617 :   (b('l8b11') <= 2139.8082275390625) ?  0.05330787434398153 :   0.07203362161183346 :   (b('bulk') <= 144.5) ?  (b('ndvi_med') <= 1386.75) ?  -0.049936361350483946 :   0.14152553073867352 :   (b('l8b4') <= 1416.2546997070312) ?  -0.0027950032660932567 :   0.03172944725080039 :   (b('l8b4_med') <= 1519.6185302734375) ?  (b('l8b5') <= 2881.6715087890625) ?  (b('l8b11') <= 3226.5528564453125) ?  -0.01251593518316899 :   -0.051850473406291404 :   (b('ndvi') <= 1331.0) ?  0.12800218923993978 :   0.00187987804936526 :   (b('grass') <= 85.28379440307617) ?  (b('ndvi') <= 1302.5) ?  -0.0028078502486228135 :   0.01859525734373164 :   (b('l8b5_med') <= 2400.0404052734375) ?  0.09269271082794714 :   0.07395099804779996 :   (b('l8b4') <= 1794.1852416992188) ?  (b('l8b4_med') <= 1670.3765258789062) ?  (b('ndvi') <= 803.0) ?  (b('ndvi_med') <= 878.0) ?  -0.010617318695608254 :   0.07058900105111271 :   (b('bulk') <= 130.5) ?  -0.021749683889967546 :   -0.007883895519293053 :   (b('moss') <= 38.41355323791504) ?  (b('dg0_v_vv') <= 0.5518145561218262) ?  -0.0035798943768897053 :   0.031128059341767978 :   (b('dg0_v_vv') <= 0.31288862228393555) ?  0.06759487705500036 :   0.1364325243077056 :   (b('l8b11_med') <= 2684.70263671875) ?  (b('grass') <= 49.993947982788086) ?  (b('l8b11') <= 2185.9542236328125) ?  0.1130842155461483 :   -0.017981234089063346 :   (b('l8b5') <= 3141.9500732421875) ?  -0.044830532422273764 :   -0.014622027395742518 :   (b('l8b11_med') <= 2956.887451171875) ?  (b('moss') <= 17.706625938415527) ?  -0.0062265638132015565 :   0.01579040605628127 :   (b('l8b4_med') <= 1799.5505981445312) ?  0.004036081899587756 :   -0.015517910767350295 :   (b('l8b11_med') <= 2584.6463623046875) ?  (b('l8b11_med') <= 2490.73779296875) ?  (b('l8b11') <= 1793.2109985351562) ?  (b('moss') <= 21.1070499420166) ?  (b('bulk') <= 151.5) ?  0.03268323124347231 :   0.12586820415270836 :   (b('l8b11_med') <= 1654.46435546875) ?  -0.01463367548818994 :   -0.028687507085394068 :   (b('l8b11_med') <= 2405.613037109375) ?  (b('l8b5_med') <= 2734.763427734375) ?  0.011059834127182044 :   -0.0012424741923494982 :   (b('l8b5') <= 2613.375244140625) ?  -0.0280971121968996 :   -0.0073412883011070495 :   (b('l8b4') <= 1334.4530639648438) ?  (b('l8b11') <= 1704.2372436523438) ?  -0.0831214171943592 :   (b('l8b11_med') <= 2539.9166259765625) ?  -0.06763711621601157 :   -0.051067502224563244 :   (b('moss') <= 22.095589637756348) ?  (b('ndvi') <= 1563.0) ?  0.008053594647088732 :   0.024153928633670083 :   (b('ndvi') <= 1446.0) ?  0.06021307499345155 :   0.1330774165740854 :   (b('dg0_v_vv') <= 3.4018819332122803) ?  (b('l8b5') <= 3020.3541259765625) ?  (b('ndvi') <= 1950.5) ?  (b('l8b5') <= 3019.1070556640625) ?  -0.0027255548121987443 :   0.046726324793220385 :   (b('ndvi') <= 2395.0) ?  0.023420487180974593 :   0.001026061449985331 :   (b('l8b5') <= 3088.999267578125) ?  (b('moss') <= 3.4928038120269775) ?  -0.025784689359496404 :   -0.008742471937016986 :   (b('l8b11') <= 2191.095458984375) ?  0.007584105366315243 :   -0.005163337614156475 :   (b('l8b11') <= 2612.7353515625) ?  (b('l8b5_med') <= 3524.328857421875) ?  -0.012057686020694715 :   -0.007849157540519139 :   (b('bulk') <= 152.0) ?  (b('bulk') <= 150.5) ?  -0.06469128461093263 :   -0.03650678034475299 :   (b('ndvi') <= 1449.0) ?  -0.10151664213246309 :   -0.08049521222167702 :   (b('sand') <= 44.0) ?  (b('l8b5') <= 2588.5084228515625) ?  (b('l8b4') <= 1578.9507446289062) ?  (b('moss') <= 16.375277042388916) ?  (b('dg0_v_vv') <= 4.815925121307373) ?  (b('dg0_v_vv') <= 3.7337653636932373) ?  0.00669028180219447 :   0.026955228669163896 :   (b('l8b11_med') <= 2527.9622802734375) ?  0.048012174084356485 :   0.049935439083020894 :   -0.024410580040656604 :   (b('moss') <= 6.09779167175293) ?  -0.05643419150552684 :   -0.014731333035107153 :   (b('l8b11_med') <= 2212.34326171875) ?  (b('dg0_v_vv') <= 3.687513589859009) ?  0.0379183816819369 :   (b('bulk') <= 134.0) ?  0.08691837667514674 :   (b('l8b5_med') <= 2890.4798583984375) ?  0.06399991641186306 :   0.04773477822025554 :   (b('l8b4') <= 1953.333251953125) ?  (b('ndvi_med') <= 2076.25) ?  (b('l8b11') <= 2667.2677001953125) ?  0.009736357671366602 :   0.015578350164873966 :   (b('sand') <= 32.5) ?  0.034452355954729175 :   0.029129680340244338 :   0.07040148208207689 :   (b('ndvi_med') <= 1824.0) ?  (b('l8b11_med') <= 2479.351318359375) ?  (b('ndvi') <= 1126.0) ?  (b('l8b5') <= 2561.570556640625) ?  0.07553272920646387 :   0.10228999396727667 :   (b('l8b4_med') <= 1129.5966491699219) ?  -0.047816050335148075 :   (b('l8b4') <= 2015.4093017578125) ?  0.035823700610638776 :   0.015462183606295735 :   (b('ndvi_med') <= 1755.75) ?  (b('ndvi') <= 1722.5) ?  (b('l8b5_med') <= 3137.0338134765625) ?  0.015364160852705736 :   -0.00468280049346241 :   (b('ndvi') <= 1903.5) ?  0.030101246716757884 :   0.00729271217577165 :   (b('l8b4_med') <= 1847.4920654296875) ?  (b('l8b4') <= 1534.4022827148438) ?  -0.009188212907958848 :   -0.06169078866989273 :   (b('l8b5') <= 2918.848876953125) ?  0.0021462002334257796 :   0.03797271461790894 :   (b('ndvi_med') <= 1975.25) ?  (b('l8b4_med') <= 1442.8026733398438) ?  (b('ndvi_med') <= 1899.25) ?  -0.09278654452775137 :   -0.08888196382058258 :   -0.017951919821219 :   (b('l8b5') <= 3543.4241943359375) ?  (b('l8b5') <= 2856.7392578125) ?  (b('l8b5') <= 2465.814453125) ?  -0.013382651061205647 :   0.011165980633682696 :   (b('l8b11_med') <= 2174.722900390625) ?  0.02460517391328823 :   0.039258057744899086 :   (b('l8b4') <= 1462.186767578125) ?  (b('dg0_v_vv') <= 4.722190618515015) ?  -0.037543839752543245 :   -0.023281734736375917 :   (b('l8b11_med') <= 2518.2154541015625) ?  -0.020984387103481036 :   -0.027672916781155944 :   (b('dg0_v_vv') <= 0.36049604415893555) ?  (b('l8b4') <= 2017.389892578125) ?  (b('dg0_v_vv') <= 0.10957002639770508) ?  (b('dg0_v_vv') <= 0.03594827651977539) ?  (b('dg0_v_vv') <= -3.708165168762207) ?  -0.019767916491535725 :   (b('ndvi') <= 1850.5) ?  (b('dg0_v_vv') <= -0.14245033264160156) ?  -0.0036814362993328542 :   0.019435512055654608 :   (b('ndvi') <= 1987.5) ?  0.022920463180845216 :   0.003103035064939746 :   -0.02461883180712582 :   (b('dg0_v_vv') <= 0.17024612426757812) ?  0.079272045966014 :   (b('ndvi') <= 3065.0) ?  (b('l8b5_med') <= 2644.2236328125) ?  -0.005884954288597394 :   -0.0024445488008142113 :   0.01793573400169296 :   (b('dg0_v_vv') <= -1.7278218269348145) ?  (b('dg0_v_vv') <= -2.3048486709594727) ?  (b('dg0_v_vv') <= -3.063749313354492) ?  (b('ndvi_med') <= 2424.5) ?  -0.000908330764522583 :   (b('dg0_v_vv') <= -4.285941123962402) ?  0.010378145421443857 :   0.0005916722753143006 :   (b('ndvi_med') <= 2424.5) ?  (b('l8b5_med') <= 2687.2421875) ?  0.009254964147634298 :   0.015590643299916024 :   0.0275906527174499 :   (b('ndvi') <= 2324.0) ?  0.06568199305984296 :   0.01729962126054374 :   (b('l8b5_med') <= 2644.2236328125) ?  0.10321283467667719 :   0.08461962627148004 :   (b('dg0_v_vv') <= 1.92045259475708) ?  (b('l8b5_med') <= 2687.2421875) ?  (b('ndvi') <= 2551.0) ?  (b('dg0_v_vv') <= 1.0622072219848633) ?  0.0034594590560518834 :   (b('l8b5_med') <= 2088.3817138671875) ?  0.06299418211375868 :   0.0348697942561049 :   0.06932969868610223 :   (b('l8b5') <= 2625.613037109375) ?  (b('sand') <= 30.0) ?  0.07214978658788251 :   0.09208897042700487 :   0.11482716496770357 :   (b('l8b5_med') <= 2687.2421875) ?  (b('l8b4_med') <= 1303.0054931640625) ?  0.022231257947936678 :   (b('dg0_v_vv') <= 3.9837419986724854) ?  0.014262110920044035 :   0.01614447353618348 :   -0.030225975932900606 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 916.6549377441406) ?  (b('sand') <= 27.5) ?  (b('l8b4') <= 732.3254089355469) ?  (b('l8b11_med') <= 1257.8624267578125) ?  (b('l8b5_med') <= 3330.56396484375) ?  (b('l8b11') <= 1299.8658447265625) ?  (b('l8b5') <= 3030.7025146484375) ?  (b('ndvi_med') <= 3065.5) ?  -0.041685189366103936 :   (b('dg0_v_vv') <= 1.0207476615905762) ?  (b('l8b11') <= 1196.72216796875) ?  0.0030429347003179397 :   0.01688871563763647 :   0.0415385740940481 :   (b('ndvi') <= 3281.5) ?  (b('dg0_v_vv') <= 0.6360139846801758) ?  (b('l8b5_med') <= 3087.1593017578125) ?  0.08594881762437628 :   0.06674517387118455 :   (b('dg0_v_vv') <= 1.3508639335632324) ?  0.05067607352786482 :   0.030779699815715267 :   (b('l8b4_med') <= 614.1059265136719) ?  (b('dg0_v_vv') <= -0.01334381103515625) ?  0.03055326592315602 :   0.07715326237627196 :   (b('ndvi') <= 3631.0) ?  -0.01769260093424535 :   0.02260842425996042 :   (b('dg0_v_vv') <= -0.41895055770874023) ?  (b('l8b11') <= 1394.7234497070312) ?  0.06453586102486736 :   0.04587236807893885 :   (b('dg0_v_vv') <= 0.12102937698364258) ?  (b('ndvi') <= 3445.5) ?  (b('dg0_v_vv') <= -0.2494945526123047) ?  0.07101982549137098 :   0.07029946062740805 :   0.0739869798230296 :   0.08205613781209259 :   (b('l8b11') <= 1274.8690795898438) ?  (b('dg0_v_vv') <= 0.8671274185180664) ?  (b('ndvi') <= 4280.0) ?  (b('ndvi') <= 4162.0) ?  (b('l8b5') <= 4315.938720703125) ?  0.02221253700880327 :   -0.019535101206820595 :   0.08571664445347066 :   (b('l8b11') <= 905.5500793457031) ?  (b('ndvi_med') <= 4344.75) ?  0.04239944769261104 :   0.02257472140473606 :   (b('ndvi') <= 4457.5) ?  -0.07505377883216294 :   -0.014691431347214243 :   (b('l8b4') <= 518.3019714355469) ?  (b('ndvi') <= 4403.5) ?  (b('l8b11_med') <= 1213.9043579101562) ?  0.08424799068812988 :   0.06966284226755498 :   (b('l8b4') <= 456.8096466064453) ?  0.04868159439382963 :   0.05986056263831557 :   (b('ndvi_med') <= 4330.5) ?  (b('sand') <= 15.0) ?  -0.008172065483711743 :   0.02177811237236223 :   0.06024397440437629 :   (b('ndvi_med') <= 4344.75) ?  (b('l8b11') <= 1306.9283447265625) ?  (b('l8b4') <= 639.588623046875) ?  (b('l8b5') <= 4382.979736328125) ?  -0.033511533023084544 :   -0.06377933669530883 :   (b('l8b4_med') <= 586.8941345214844) ?  -0.0018600802794127569 :   0.006624078401523237 :   (b('l8b4_med') <= 562.5426635742188) ?  0.02023580638588862 :   0.03313416369323527 :   (b('ndvi_med') <= 4394.5) ?  -0.06327233289856798 :   -0.07423453724210977 :   (b('l8b4') <= 715.9032287597656) ?  (b('l8b4') <= 703.7579650878906) ?  (b('ndvi') <= 4980.0) ?  (b('dg0_v_vv') <= 3.2175827026367188) ?  (b('l8b5') <= 3021.4371337890625) ?  (b('l8b4') <= 555.8275146484375) ?  0.006551116232220633 :   -0.012284668500740548 :   (b('l8b5') <= 3175.4781494140625) ?  0.02898521930031442 :   0.006352568080567011 :   (b('l8b5') <= 2921.8297119140625) ?  (b('dg0_v_vv') <= 4.816196441650391) ?  0.017742676348649455 :   0.040346227234722976 :   (b('l8b5') <= 3582.860107421875) ?  0.08467708331151488 :   0.03823660044026583 :   -0.09672402603477098 :   (b('l8b11_med') <= 1555.286865234375) ?  (b('ndvi') <= 3470.0) ?  (b('l8b5_med') <= 2544.3226318359375) ?  0.09665615150792728 :   0.12988993605259297 :   0.05345734758935666 :   (b('l8b5') <= 3004.7215576171875) ?  0.004912318682116779 :   0.048860409146050776 :   (b('grass') <= 69.29146575927734) ?  (b('l8b11_med') <= 1540.8112182617188) ?  (b('ndvi') <= 4782.0) ?  (b('ndvi_med') <= 3566.0) ?  (b('dg0_v_vv') <= -0.6024155616760254) ?  -0.022119477578225338 :   0.005167890812761311 :   (b('dg0_v_vv') <= 1.9056735038757324) ?  -0.0391877138471944 :   -3.101343331883555e-05 :   0.02800569245874915 :   (b('l8b5') <= 2727.757568359375) ?  (b('ndvi') <= 3800.0) ?  (b('sand') <= 24.5) ?  0.020151552252513888 :   -0.00595813073755277 :   (b('dg0_v_vv') <= 0.19432687759399414) ?  -0.030408196697335188 :   -0.016691346531594653 :   (b('ndvi') <= 4259.0) ?  (b('crops') <= 62.346336364746094) ?  0.03042420617387301 :   0.011904738797481102 :   (b('dg0_v_vv') <= 2.1579103469848633) ?  -0.020625043302444954 :   0.01416535921403099 :   -0.11056841855177335 :   (b('bulk') <= 137.5) ?  (b('l8b5') <= 3739.031494140625) ?  (b('l8b5') <= 3345.4871826171875) ?  (b('l8b5_med') <= 3207.32275390625) ?  (b('l8b4') <= 806.0966186523438) ?  (b('dg0_v_vv') <= -0.09447431564331055) ?  (b('l8b11_med') <= 1651.1475219726562) ?  0.002793501028138048 :   -0.0029216377608184308 :   (b('ndvi_med') <= 2040.5) ?  -0.00931091424621866 :   0.01276114984145432 :   (b('l8b4_med') <= 904.2610473632812) ?  (b('dg0_v_vv') <= 0.7665390968322754) ?  -0.0005620876132247716 :   -0.013485724564025298 :   (b('dg0_v_vv') <= 1.5776982307434082) ?  0.0003120138533006245 :   0.011916706478624084 :   0.041894533310894044 :   (b('dg0_v_vv') <= -1.1465821266174316) ?  -0.012573428999988312 :   (b('dg0_v_vv') <= 0.4000091552734375) ?  -0.05524101932110986 :   -0.05613963709458922 :   (b('ndvi_med') <= 2956.75) ?  (b('dg0_v_vv') <= 1.2212023735046387) ?  (b('l8b5_med') <= 2495.3189697265625) ?  0.0037473483284836684 :   (b('l8b11_med') <= 1742.0434265136719) ?  0.01719165532517747 :   0.012435201393311224 :   (b('bulk') <= 124.5) ?  0.02643947240621647 :   0.025683359887271606 :   (b('ndvi_med') <= 3351.5) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.06314444196862601 :   0.059499603484944596 :   (b('dg0_v_vv') <= -0.23630905151367188) ?  (b('bulk') <= 132.5) ?  0.04012022215072977 :   (b('dg0_v_vv') <= -0.8437175750732422) ?  0.030684096317750875 :   0.03324828973695226 :   0.04468314753169611 :   (b('dg0_v_vv') <= -1.9058928489685059) ?  (b('ndvi_med') <= 3459.0) ?  (b('dg0_v_vv') <= -2.2319869995117188) ?  (b('l8b5_med') <= 2480.0167236328125) ?  -0.011458455892246783 :   -0.021039624810641644 :   (b('l8b5') <= 2116.8914184570312) ?  -0.02340560762045288 :   (b('grass') <= 64.01420211791992) ?  -0.03866129162560314 :   -0.03469678157983361 :   (b('ndvi_med') <= 3820.75) ?  (b('moss') <= 12.79644387960434) ?  (b('l8b5') <= 2608.3543701171875) ?  -0.0014716268072056948 :   -0.0024684732841711576 :   -0.016043160984186317 :   (b('dg0_v_vv') <= -2.3861775398254395) ?  0.011545557753482288 :   0.015720282327725682 :   (b('l8b5_med') <= 3032.838623046875) ?  (b('l8b4') <= 760.2254943847656) ?  (b('ndvi_med') <= 2776.5) ?  (b('l8b4_med') <= 886.5784301757812) ?  -0.04059922513456929 :   (b('l8b11') <= 1576.72265625) ?  0.016708969070390656 :   -0.005060576092368291 :   (b('l8b5_med') <= 2755.1143798828125) ?  (b('dg0_v_vv') <= 0.5973944664001465) ?  0.06063008263503589 :   0.10245099017332937 :   (b('dg0_v_vv') <= -0.5360336303710938) ?  0.07221840377765192 :   0.049973866795268246 :   (b('ndvi') <= 4536.5) ?  (b('moss') <= 18.7062668800354) ?  (b('l8b4') <= 910.1468505859375) ?  0.024578473443581546 :   0.08824895957709553 :   (b('l8b4') <= 775.1505737304688) ?  0.027116939253130353 :   -0.03117956535932176 :   (b('dg0_v_vv') <= -0.15820026397705078) ?  (b('dg0_v_vv') <= -1.119211196899414) ?  0.12584454960475747 :   -0.027341834175985714 :   (b('dg0_v_vv') <= 0.13904953002929688) ?  0.17911479295930066 :   0.04210114934770585 :   (b('ndvi') <= 2849.0) ?  (b('dg0_v_vv') <= -0.6203932762145996) ?  (b('l8b11_med') <= 1244.5859985351562) ?  0.02874921287759946 :   -0.019105072760017883 :   (b('l8b5') <= 3168.49658203125) ?  (b('l8b4') <= 847.0292053222656) ?  0.049626715882260364 :   0.03644026734241948 :   0.0725616623925947 :   (b('dg0_v_vv') <= 0.15111207962036133) ?  (b('bulk') <= 138.5) ?  (b('l8b4') <= 771.6192321777344) ?  0.016657958192932976 :   0.06907502793977038 :   (b('l8b5') <= 3917.47021484375) ?  -0.03344046517646475 :   0.0021191717859259385 :   (b('dg0_v_vv') <= 0.7532491683959961) ?  (b('dg0_v_vv') <= 0.4998030662536621) ?  0.010212373880012041 :   0.05269189520470823 :   (b('l8b5_med') <= 3503.7686767578125) ?  -0.004144867779175342 :   0.047407853582462633 :   (b('grass') <= 46.070945739746094) ?  (b('l8b11') <= 1691.6806030273438) ?  (b('l8b4_med') <= 1013.96435546875) ?  (b('bulk') <= 151.5) ?  (b('l8b4') <= 910.1172180175781) ?  (b('l8b11') <= 1690.0462036132812) ?  (b('ndvi') <= 2465.0) ?  (b('ndvi') <= 2407.0) ?  0.0035454556161505476 :   0.02692278124758577 :   (b('ndvi') <= 2851.5) ?  -0.006572090855706824 :   0.0010799397977202213 :   (b('dg0_v_vv') <= 0.28350377082824707) ?  (b('l8b5_med') <= 2923.2857666015625) ?  0.04419015934724027 :   0.014786872135349377 :   0.06440266429095137 :   (b('l8b11_med') <= 1641.0211181640625) ?  (b('dg0_v_vv') <= -0.8465352058410645) ?  0.05424865820631122 :   (b('l8b11_med') <= 1632.0286254882812) ?  0.04721322152849122 :   0.04424866774305439 :   (b('l8b11_med') <= 1680.6777954101562) ?  (b('dg0_v_vv') <= -2.0145010948181152) ?  0.025919696382939852 :   0.032295397587302425 :   (b('dg0_v_vv') <= -0.44240808486938477) ?  0.046958186299512206 :   0.03888577639351214 :   (b('l8b5_med') <= 2195.543701171875) ?  (b('l8b5') <= 1921.6422729492188) ?  0.16906230005468068 :   (b('ndvi') <= 2848.5) ?  0.0670931164402302 :   (b('l8b5') <= 2105.2054443359375) ?  -0.01974257869217068 :   -0.015573712651342234 :   (b('l8b4') <= 449.45098876953125) ?  (b('l8b5') <= 2877.341796875) ?  (b('l8b11') <= 844.6421813964844) ?  -0.005238744376355006 :   0.018605537007038812 :   (b('l8b11_med') <= 1010.0088806152344) ?  0.0686093002202853 :   0.0438398754703702 :   (b('l8b11') <= 1314.5841674804688) ?  (b('sand') <= 39.5) ?  -0.019335812776391605 :   -0.045996665318564355 :   (b('l8b11') <= 1391.9246826171875) ?  0.0222074650043896 :   -0.014540479538551915 :   (b('l8b11_med') <= 2010.394287109375) ?  (b('l8b4') <= 903.2883911132812) ?  (b('ndvi') <= 1874.0) ?  (b('ndvi') <= 1866.0) ?  (b('ndvi_med') <= 1616.5) ?  0.11684494734032236 :   0.025434400287101807 :   0.17988942342334946 :   (b('l8b5') <= 2383.97998046875) ?  (b('l8b5') <= 1611.5965576171875) ?  0.02906291063781965 :   -0.008378968263295613 :   (b('l8b5') <= 3373.6129150390625) ?  0.026577875895963367 :   0.006774097368399408 :   (b('l8b4_med') <= 1115.8173828125) ?  (b('dg0_v_vv') <= 0.4215998649597168) ?  0.08844345916392077 :   0.15404756827611066 :   0.1622009787836523 :   (b('l8b11_med') <= 2151.323486328125) ?  (b('l8b4') <= 732.5257263183594) ?  (b('l8b4') <= 457.5613555908203) ?  (b('ndvi') <= 3712.0) ?  -0.01587487402709433 :   0.004584083848887166 :   (b('l8b4') <= 504.04058837890625) ?  -0.04354670615430824 :   -0.021778741244563336 :   (b('ndvi') <= 3768.0) ?  (b('l8b5') <= 1944.8077392578125) ?  -0.055409765889593725 :   -0.04681494969744131 :   -0.06545123576690952 :   (b('l8b4_med') <= 1750.6202392578125) ?  (b('ndvi') <= 2439.5) ?  (b('crops') <= 53.92042541503906) ?  -0.024917896460371698 :   0.01647507153279805 :   (b('l8b4_med') <= 1276.38330078125) ?  0.06585684878019697 :   0.012087761247877908 :   (b('l8b5') <= 3812.7979736328125) ?  (b('l8b4_med') <= 1889.9513549804688) ?  -0.009780665070488984 :   -0.03440395710946982 :   (b('l8b4_med') <= 1942.2267456054688) ?  0.07369926773876606 :   0.019058357214141675 :   (b('grass') <= 45.12700653076172) ?  (b('l8b5') <= 2281.6353759765625) ?  (b('crops') <= 49.26193809509277) ?  (b('dg0_v_vv') <= 0.6783237457275391) ?  (b('crops') <= 47.22846031188965) ?  (b('l8b4_med') <= 761.9753112792969) ?  0.017971215749929558 :   0.00037782955482710705 :   (b('l8b5_med') <= 2781.259521484375) ?  0.016109818548856042 :   0.08632234354634474 :   (b('bulk') <= 153.5) ?  (b('ndvi_med') <= 2976.5) ?  0.049722344407616886 :   0.020989230340667686 :   0.1062633669725005 :   (b('l8b5') <= 1877.97705078125) ?  (b('dg0_v_vv') <= -0.19766855239868164) ?  (b('l8b5') <= 1346.5535888671875) ?  -0.09186597220487785 :   -0.010552228574156443 :   (b('crops') <= 55.25153923034668) ?  0.02893227527782526 :   0.0046907888039111785 :   (b('dg0_v_vv') <= -2.6236281394958496) ?  (b('l8b5_med') <= 2135.785400390625) ?  0.021929455500010353 :   -0.011516197472801137 :   (b('l8b11') <= 2014.6181030273438) ?  -0.037161641252250736 :   -0.021626873149674906 :   (b('grass') <= 12.900787353515625) ?  (b('ndvi') <= 4739.5) ?  (b('l8b5_med') <= 3413.9039306640625) ?  (b('ndvi') <= 4192.5) ?  0.006417044719710447 :   0.030878606343960908 :   (b('ndvi') <= 4034.5) ?  0.0783212534753559 :   0.05126867413002176 :   (b('l8b4') <= 829.989990234375) ?  (b('dg0_v_vv') <= -0.018933534622192383) ?  0.002696171927655436 :   0.012860853839088388 :   (b('ndvi_med') <= 4697.0) ?  -0.04420869313487642 :   -0.019164982937081854 :   (b('moss') <= 0.18294516392052174) ?  (b('l8b5') <= 2794.658447265625) ?  (b('ndvi') <= 2971.5) ?  -0.04711495895937 :   -0.07525223025932783 :   (b('l8b11') <= 1738.7380981445312) ?  0.019316792503798558 :   -0.030702517945019936 :   (b('l8b4') <= 914.0778503417969) ?  (b('l8b4_med') <= 1014.8924865722656) ?  -0.0147452960257245 :   -0.06054165430280736 :   (b('dg0_v_vv') <= -0.1669015884399414) ?  0.0029210892904134203 :   0.06004887236101514 :   0.1283563379188253 :   (b('moss') <= 12.40567398071289) ?  (b('moss') <= 5.826396703720093) ?  (b('sand') <= 33.5) ?  (b('ndvi') <= 4872.5) ?  (b('dg0_v_vv') <= 2.0714988708496094) ?  (b('dg0_v_vv') <= 1.4165925979614258) ?  (b('l8b4') <= 790.9671630859375) ?  0.03291734649636212 :   -0.02230418444966429 :   -0.07502580456951136 :   (b('l8b5_med') <= 2558.96728515625) ?  0.045770272515667176 :   (b('ndvi_med') <= 1662.0) ?  0.06933593154044021 :   0.08367390230107102 :   (b('grass') <= 70.19295883178711) ?  0.06859824884857979 :   (b('l8b4_med') <= 1268.47412109375) ?  0.1098937622987779 :   0.08270098771343248 :   (b('bulk') <= 141.5) ?  (b('l8b5') <= 2304.115966796875) ?  (b('crops') <= 30.62926959991455) ?  (b('l8b5_med') <= 1946.989501953125) ?  -0.001367434150465623 :   0.017917151857274608 :   (b('dg0_v_vv') <= -1.141988754272461) ?  0.03599802709880448 :   0.07423272031713815 :   (b('crops') <= 0.7363387942314148) ?  (b('l8b11') <= 1389.5342407226562) ?  -0.002118558807811862 :   -0.04520669111291974 :   (b('l8b11') <= 1717.2603759765625) ?  0.014774647123668737 :   -0.06190419174317291 :   (b('bulk') <= 144.5) ?  (b('dg0_v_vv') <= 4.509713172912598) ?  (b('dg0_v_vv') <= 4.030920505523682) ?  -0.01600012937329601 :   0.08846290179565258 :   -0.12021851743436651 :   (b('grass') <= 97.4135971069336) ?  (b('l8b5') <= 3205.6370849609375) ?  -0.010227671928946752 :   0.018433086939430618 :   (b('sand') <= 43.0) ?  0.05669491583964547 :   0.01416635490773375 :   (b('grass') <= 68.66935348510742) ?  (b('l8b4') <= 597.2352905273438) ?  (b('ndvi') <= 1726.5) ?  (b('l8b11') <= 1068.421142578125) ?  0.06076661176119075 :   0.09325026374070411 :   (b('l8b11') <= 922.1878051757812) ?  (b('l8b5') <= 3053.043212890625) ?  0.015647567853835274 :   -0.04882812440285156 :   (b('l8b4_med') <= 718.2570495605469) ?  0.017962607377614476 :   0.03391467117445327 :   (b('bulk') <= 141.5) ?  (b('ndvi_med') <= 1819.5) ?  (b('l8b4') <= 707.1324157714844) ?  0.0615883276968186 :   0.11644835144272164 :   (b('l8b4') <= 708.8414916992188) ?  -0.00011440307793662916 :   0.018426492162460274 :   (b('moss') <= 12.107142925262451) ?  (b('ndvi') <= 3959.0) ?  -0.027269073159187786 :   -0.08752693703004348 :   (b('ndvi') <= 2421.5) ?  0.0699481736519611 :   0.01574280057299718 :   (b('grass') <= 80.50270080566406) ?  (b('ndvi') <= 3811.0) ?  (b('l8b11') <= 1608.6236572265625) ?  (b('l8b11') <= 1297.6370239257812) ?  0.049976070161119455 :   0.09545934532556904 :   (b('l8b11') <= 1658.2975463867188) ?  -0.0692223441110007 :   0.013926988243375016 :   (b('dg0_v_vv') <= 1.688023567199707) ?  (b('l8b11') <= 1555.291259765625) ?  -0.02834543815918794 :   0.06075978885089242 :   0.1353815843631911 :   (b('l8b4') <= 900.897216796875) ?  (b('l8b11_med') <= 2131.7957763671875) ?  (b('dg0_v_vv') <= -0.33658885955810547) ?  0.02972970761824138 :   0.005162152523498054 :   (b('l8b4') <= 889.3582763671875) ?  -0.04556722384634897 :   0.0035926062700059047 :   (b('dg0_v_vv') <= 0.6577215194702148) ?  -0.07969943978076413 :   -0.04696256167005752 :   (b('grass') <= 47.93498420715332) ?  (b('l8b4') <= 848.7659912109375) ?  (b('l8b11') <= 1781.9066162109375) ?  (b('dg0_v_vv') <= -0.3129005432128906) ?  (b('l8b11_med') <= 1510.9027099609375) ?  0.014591821326171095 :   0.1236694884905456 :   (b('dg0_v_vv') <= -0.0066432952880859375) ?  -0.030417900582972557 :   (b('grass') <= 47.62581825256348) ?  0.008765018138422198 :   0.04549178335427037 :   (b('ndvi') <= 2675.0) ?  -0.04389912746507635 :   (b('dg0_v_vv') <= -0.16619014739990234) ?  -0.0637247479401542 :   (b('dg0_v_vv') <= 0.21884822845458984) ?  -0.09047292497634446 :   -0.09693554146016048 :   (b('dg0_v_vv') <= -0.6799683570861816) ?  (b('ndvi') <= 3482.0) ?  (b('l8b5') <= 2588.650146484375) ?  0.04256566020981101 :   -0.016918090595235913 :   -0.0900470639728056 :   (b('l8b5') <= 2541.640380859375) ?  (b('ndvi') <= 3600.5) ?  (b('dg0_v_vv') <= 0.7250242233276367) ?  0.042495726901274136 :   0.1709230143148926 :   -0.09391984305450202 :   (b('dg0_v_vv') <= -0.19611072540283203) ?  0.06220802238760875 :   (b('ndvi_med') <= 3098.5) ?  0.1740126591257312 :   0.11219380745458069 :   (b('dg0_v_vv') <= -0.03670549392700195) ?  (b('l8b11_med') <= 2104.6551513671875) ?  (b('sand') <= 40.5) ?  (b('l8b5_med') <= 2412.8741455078125) ?  (b('l8b4') <= 685.7403869628906) ?  -0.03880788988611821 :   -0.011624784899963325 :   (b('grass') <= 50.62044143676758) ?  0.04114957338096878 :   -0.033262018579917924 :   (b('ndvi') <= 3339.5) ?  (b('l8b4') <= 904.4289855957031) ?  0.004749633528950045 :   -0.02635059767763705 :   (b('l8b11_med') <= 1203.1886596679688) ?  0.04072830471779166 :   -0.014192983337786761 :   (b('l8b11') <= 1772.3801879882812) ?  (b('l8b4') <= 832.3027038574219) ?  -0.06949406590940077 :   -0.07835403461971957 :   -0.05454163712165381 :   (b('grass') <= 76.6335563659668) ?  (b('ndvi') <= 4704.5) ?  (b('l8b11') <= 2273.114013671875) ?  (b('l8b11') <= 2191.990966796875) ?  0.001010398205787782 :   0.12731682491739937 :   (b('l8b5_med') <= 2210.78173828125) ?  -0.08566776517329039 :   -0.07557046472607887 :   (b('dg0_v_vv') <= 2.196293830871582) ?  0.06315635671587605 :   0.11582973008540842 :   (b('l8b5') <= 2051.9920654296875) ?  (b('l8b5') <= 2044.6421508789062) ?  (b('ndvi') <= 2088.0) ?  0.029054606047577963 :   0.005874334187622378 :   -0.06479812360181182 :   (b('ndvi') <= 2109.0) ?  (b('l8b5') <= 2226.6787109375) ?  -0.021497845405669097 :   0.0039047766537358697 :   (b('l8b5_med') <= 2246.291259765625) ?  0.07540195314831222 :   0.03317849515043905 :   (b('moss') <= 8.823604106903076) ?  (b('l8b5') <= 2451.533935546875) ?  (b('bulk') <= 150.5) ?  (b('bulk') <= 132.5) ?  (b('ndvi_med') <= 2020.75) ?  (b('sand') <= 39.5) ?  (b('dg0_v_vv') <= 0.5621342658996582) ?  (b('l8b4') <= 996.8890991210938) ?  (b('dg0_v_vv') <= -3.798194408416748) ?  0.05977626862251906 :   0.030701051245671816 :   (b('l8b5') <= 1826.195556640625) ?  0.004125738350295172 :   0.02594190576273636 :   (b('l8b5') <= 1641.34033203125) ?  0.06593598427620262 :   (b('sand') <= 24.5) ?  0.039817045118095806 :   0.0514907404115705 :   0.09673756225256114 :   (b('bulk') <= 97.5) ?  0.09245157944967569 :   (b('l8b4') <= 1068.6341552734375) ?  (b('l8b4') <= 968.4548034667969) ?  (b('dg0_v_vv') <= 0.44381141662597656) ?  0.02333972506057174 :   -0.04533985366375238 :   (b('dg0_v_vv') <= -0.6814947128295898) ?  -0.019245243790697638 :   0.004170658683169768 :   (b('l8b5') <= 2327.962158203125) ?  (b('l8b11') <= 1684.7681884765625) ?  0.11207218803838154 :   0.027128185320892402 :   (b('ndvi') <= 1953.0) ?  0.04472251886589722 :   -0.005520962815567897 :   (b('l8b5') <= 1733.5088500976562) ?  (b('grass') <= 24.319149017333984) ?  (b('ndvi_med') <= 1678.75) ?  -0.034272819190319825 :   (b('dg0_v_vv') <= 1.2183904647827148) ?  (b('dg0_v_vv') <= 0.21596097946166992) ?  -0.00044947970958051156 :   -0.03904095813150166 :   (b('ndvi_med') <= 2107.5) ?  0.02890592774681644 :   0.024747278830964448 :   (b('dg0_v_vv') <= 1.3268632888793945) ?  (b('ndvi_med') <= 2232.0) ?  (b('grass') <= 96.58420944213867) ?  -0.03741847900682871 :   -0.0750633499379902 :   (b('ndvi_med') <= 2485.0) ?  0.012176131086884187 :   -0.008222681164876636 :   (b('ndvi_med') <= 2173.75) ?  (b('l8b11') <= 2299.322509765625) ?  -0.062133771334464315 :   -0.07976413542079347 :   -0.12157176607605724 :   (b('sand') <= 41.5) ?  (b('l8b11_med') <= 1912.1121215820312) ?  (b('l8b5_med') <= 2440.1046142578125) ?  (b('l8b5') <= 2221.7510986328125) ?  0.16769069368552691 :   0.05664111973397182 :   (b('l8b11') <= 2250.8642578125) ?  -0.004098861766260511 :   0.023703869956526687 :   (b('l8b5') <= 2178.4793701171875) ?  (b('l8b4') <= 944.6504211425781) ?  0.04417704322992189 :   -0.007393009532693244 :   (b('l8b11_med') <= 2330.0760498046875) ?  -0.029516090109787712 :   -0.004686455692013654 :   (b('l8b5') <= 2448.2247314453125) ?  (b('dg0_v_vv') <= -1.9917693138122559) ?  (b('sand') <= 57.5) ?  -0.019209981158219033 :   0.0049053376035869106 :   (b('sand') <= 55.0) ?  0.012954018209025413 :   -0.001965378388276966 :   (b('crops') <= 46.87349891662598) ?  0.1273040647823683 :   0.042951530303911994 :   (b('ndvi') <= 1735.5) ?  (b('l8b4_med') <= 1378.8347778320312) ?  (b('ndvi') <= 1249.0) ?  (b('dg0_v_vv') <= -0.16158723831176758) ?  (b('l8b5') <= 1977.7058715820312) ?  (b('ndvi') <= 892.5) ?  -0.02218768723068547 :   -0.03631614932018131 :   (b('l8b4') <= 1274.0814208984375) ?  0.0994309926943542 :   0.015626393796147352 :   (b('l8b5') <= 2148.5709228515625) ?  (b('ndvi_med') <= 1764.25) ?  0.09413115272394898 :   0.06231355288462173 :   0.20322350150815285 :   (b('l8b4_med') <= 1147.730224609375) ?  (b('l8b11') <= 2561.205078125) ?  (b('ndvi') <= 1660.5) ?  -0.026554510086898773 :   -0.04768875555028593 :   (b('dg0_v_vv') <= -0.4436063766479492) ?  -0.03690372973120806 :   -0.06473251088535475 :   (b('dg0_v_vv') <= -0.8913445472717285) ?  (b('l8b11_med') <= 1927.3623657226562) ?  -0.0050436131701413365 :   0.03322737499354145 :   (b('ndvi_med') <= 1960.75) ?  -0.01966639875709999 :   0.08476956402036612 :   (b('l8b11') <= 2578.1861572265625) ?  (b('l8b11_med') <= 2135.67431640625) ?  (b('l8b5') <= 2363.1209716796875) ?  0.11328138373743751 :   0.0836839361185309 :   (b('l8b4') <= 1511.72216796875) ?  (b('l8b5') <= 2345.01416015625) ?  0.01115128109975571 :   0.09562854965375316 :   (b('dg0_v_vv') <= 3.0882673263549805) ?  -0.006347935883926138 :   -0.083084109474925 :   (b('grass') <= 74.15495872497559) ?  (b('l8b4_med') <= 1550.86328125) ?  (b('dg0_v_vv') <= -1.453089714050293) ?  0.028941242231739828 :   -0.012588167851130325 :   (b('dg0_v_vv') <= -0.29492712020874023) ?  0.039730024993498786 :   0.025041225526715546 :   (b('ndvi_med') <= 1763.75) ?  (b('ndvi') <= 1420.5) ?  0.13077802252802914 :   0.09609316947233379 :   (b('ndvi_med') <= 1825.75) ?  0.035082706398328195 :   0.0756552425053958 :   (b('ndvi') <= 1743.5) ?  0.1339271596760697 :   (b('l8b11') <= 2478.8890380859375) ?  (b('l8b11') <= 2400.11376953125) ?  (b('moss') <= 6.070305824279785) ?  (b('l8b5') <= 1952.9617919921875) ?  0.06160077654084059 :   0.02384841521256375 :   (b('l8b11') <= 1561.5698852539062) ?  -0.02247476464878437 :   0.010832294248309924 :   (b('l8b5_med') <= 2186.3402099609375) ?  -0.04604450458805161 :   (b('l8b11') <= 2443.625) ?  -0.06263543130606655 :   -0.06613018215587252 :   (b('l8b4') <= 1496.0453491210938) ?  0.07558753443174508 :   0.12436469302888742 :   (b('l8b11') <= 1394.8739624023438) ?  (b('moss') <= 5.876032114028931) ?  (b('crops') <= 66.48823547363281) ?  (b('grass') <= 28.989877700805664) ?  (b('crops') <= 51.817626953125) ?  (b('ndvi') <= 2704.0) ?  0.130137682240847 :   0.12305674205180372 :   (b('dg0_v_vv') <= -0.7275466918945312) ?  0.126342537428683 :   0.03434252979928848 :   (b('grass') <= 89.23691940307617) ?  (b('dg0_v_vv') <= 0.0315241813659668) ?  (b('l8b5_med') <= 3022.9287109375) ?  0.04341108788602534 :   0.048320005346821876 :   0.05604028261533747 :   0.0006692982165990013 :   (b('dg0_v_vv') <= 0.5311565399169922) ?  (b('l8b5_med') <= 2844.591552734375) ?  0.0488170970884611 :   (b('ndvi') <= 2879.5) ?  (b('l8b5') <= 3246.1099853515625) ?  -0.0030245602438451896 :   0.0018542052725582514 :   (b('dg0_v_vv') <= 0.12735652923583984) ?  0.020628822310012035 :   0.01390683950001187 :   (b('ndvi') <= 2616.0) ?  (b('l8b11_med') <= 2235.6070556640625) ?  0.08024980859278746 :   0.05875508569411653 :   (b('dg0_v_vv') <= 1.1030235290527344) ?  0.026187128258718606 :   (b('moss') <= 0.4768786132335663) ?  0.029462643231827068 :   0.04484969079245258 :   (b('dg0_v_vv') <= -0.4519157409667969) ?  0.047364723368872985 :   (b('ndvi') <= 4749.0) ?  (b('l8b4') <= 996.0884704589844) ?  (b('dg0_v_vv') <= 1.1586427688598633) ?  (b('dg0_v_vv') <= -0.0059909820556640625) ?  -0.030196965807738105 :   -0.03470045923546769 :   -0.02694816281065132 :   (b('dg0_v_vv') <= 0.6813883781433105) ?  -0.023154448134447947 :   -0.02658343417327036 :   (b('dg0_v_vv') <= 0.4961094856262207) ?  -0.03649371278585309 :   -0.05291935065703929 :   (b('l8b5_med') <= 2616.9683837890625) ?  (b('moss') <= 7.060553550720215) ?  (b('sand') <= 24.5) ?  (b('l8b5') <= 2801.9078369140625) ?  -0.005249423556954902 :   (b('l8b11') <= 1951.9247436523438) ?  (b('dg0_v_vv') <= -0.49273157119750977) ?  0.052436156477537826 :   0.07439225339379313 :   0.038690348538145736 :   (b('dg0_v_vv') <= -0.10450506210327148) ?  (b('ndvi') <= 2695.0) ?  (b('l8b4') <= 1707.9404907226562) ?  -0.004554088504574666 :   0.013946882754918165 :   (b('l8b5') <= 2524.513671875) ?  0.030211821671925494 :   -0.022712966459773595 :   (b('l8b4') <= 1011.4734191894531) ?  (b('l8b5_med') <= 2501.5506591796875) ?  -0.03430100237819507 :   -0.09158011236450592 :   (b('l8b4') <= 1024.5590209960938) ?  0.10349882968067997 :   -0.014459090085514088 :   (b('ndvi') <= 1318.5) ?  (b('l8b4_med') <= 1359.5528564453125) ?  0.09377819485334823 :   0.11236969030058369 :   (b('l8b5') <= 2595.0006103515625) ?  (b('l8b4') <= 1178.3638916015625) ?  (b('l8b4_med') <= 1125.8643798828125) ?  0.04438989516688108 :   0.1581100893629322 :   (b('l8b4_med') <= 1113.5368041992188) ?  -0.017628362448572788 :   0.02736876602484952 :   (b('ndvi') <= 2283.5) ?  (b('dg0_v_vv') <= 2.0959696769714355) ?  -0.029647626519894072 :   0.02978014443012525 :   (b('l8b4') <= 1197.066162109375) ?  0.005046144088710286 :   0.08640763474905444 :   (b('l8b11_med') <= 2586.0220947265625) ?  (b('l8b11') <= 1679.767578125) ?  (b('l8b4') <= 1230.1671752929688) ?  (b('ndvi_med') <= 3561.5) ?  (b('l8b11') <= 1531.264404296875) ?  -0.0013541760075610712 :   -0.020653993195309712 :   (b('crops') <= 2.6666667461395264) ?  -0.01448934906524546 :   0.04142042524850382 :   (b('grass') <= 25.98502540588379) ?  (b('dg0_v_vv') <= -1.086441993713379) ?  0.06470402535109865 :   0.032984903574225365 :   (b('l8b11_med') <= 2071.1063232421875) ?  -0.018643285677719362 :   -0.07480927547492328 :   (b('dg0_v_vv') <= -0.22212839126586914) ?  (b('ndvi') <= 2340.0) ?  (b('ndvi') <= 1934.5) ?  0.0003987646549724922 :   -0.009555003652573163 :   (b('dg0_v_vv') <= -1.3195853233337402) ?  -0.0011439150970653413 :   0.010088007358263217 :   (b('ndvi') <= 1054.0) ?  (b('l8b11') <= 2431.739501953125) ?  -0.04755952741086848 :   0.0007548442983175835 :   (b('sand') <= 68.5) ?  0.009261826493106029 :   -0.012356465715733027 :   (b('l8b5') <= 2459.9332275390625) ?  (b('ndvi_med') <= 1678.75) ?  (b('dg0_v_vv') <= -0.6719517707824707) ?  -0.0525706473537689 :   -0.035450045950987386 :   (b('dg0_v_vv') <= 0.4365706443786621) ?  (b('dg0_v_vv') <= -1.4058847427368164) ?  -0.058570647405922965 :   -0.05457064768159445 :   -0.06672804545892047 :   (b('l8b5') <= 2507.3280029296875) ?  (b('dg0_v_vv') <= 2.966452121734619) ?  (b('dg0_v_vv') <= -1.1714038848876953) ?  -0.020015891138920914 :   0.039801606223749855 :   -0.09136497791921677 :   (b('l8b4') <= 981.0708923339844) ?  (b('ndvi_med') <= 2011.5) ?  0.08783539205683782 :   0.028929072287317715 :   (b('l8b11') <= 2087.0589599609375) ?  -0.01774726329123388 :   -0.0028448112450605596 :   (b('grass') <= 80.94562149047852) ?  (b('grass') <= 73.79305267333984) ?  (b('grass') <= 71.95897674560547) ?  (b('ndvi_med') <= 1651.75) ?  (b('l8b4') <= 1221.1921997070312) ?  (b('l8b4_med') <= 1188.34521484375) ?  (b('l8b4') <= 939.8068542480469) ?  (b('dg0_v_vv') <= -0.6216421127319336) ?  -0.00931925717453827 :   0.08747209086150208 :   (b('l8b5') <= 2504.7718505859375) ?  -0.007188660172963441 :   0.04204116079541023 :   (b('ndvi') <= 1083.0) ?  (b('l8b11_med') <= 2051.49853515625) ?  0.01679613489500555 :   -0.027048642132939508 :   (b('l8b5') <= 1757.4026489257812) ?  0.06489621430966185 :   0.017327411269832792 :   (b('l8b11_med') <= 2372.2447509765625) ?  (b('sand') <= 58.5) ?  (b('ndvi_med') <= 1237.0) ?  -0.017593744092460988 :   -0.004540480105065302 :   (b('ndvi') <= 700.5) ?  -0.01403607612389106 :   0.023405788580527463 :   (b('l8b4_med') <= 1734.2149658203125) ?  (b('l8b11') <= 2383.1998291015625) ?  0.03126435880191143 :   0.0010145454545873913 :   (b('dg0_v_vv') <= 1.6841230392456055) ?  -0.005084856301620655 :   0.005005032036011185 :   (b('l8b4_med') <= 1102.620361328125) ?  (b('l8b11') <= 2059.494384765625) ?  (b('l8b5') <= 3056.9520263671875) ?  (b('l8b4_med') <= 1093.3270874023438) ?  -0.00893395636059746 :   0.07880205995200124 :   (b('l8b11') <= 1590.4375610351562) ?  -0.035191611499509654 :   0.008341853276771855 :   (b('l8b5') <= 2557.5748291015625) ?  (b('ndvi') <= 2155.0) ?  0.0070924093833085525 :   0.038473898361155195 :   (b('l8b11_med') <= 2105.865966796875) ?  -0.006548565776262903 :   0.07156891990244843 :   (b('l8b5') <= 2867.149658203125) ?  (b('l8b5_med') <= 2203.3043212890625) ?  (b('grass') <= 65.42730712890625) ?  0.0666844240929064 :   -0.0586635191291969 :   (b('moss') <= 14.362568378448486) ?  -0.031054684977384978 :   -0.011444303683624013 :   (b('l8b11') <= 1799.1677856445312) ?  (b('l8b11') <= 1779.3910522460938) ?  0.018315210671563498 :   0.15063442225987306 :   (b('l8b11') <= 1891.400146484375) ?  -0.030455447882934035 :   -0.006259618717892654 :   (b('l8b4') <= 1353.1104736328125) ?  (b('ndvi') <= 1208.5) ?  (b('l8b4') <= 1108.2552490234375) ?  (b('dg0_v_vv') <= -0.36283206939697266) ?  (b('dg0_v_vv') <= -0.8180875778198242) ?  -0.03255332231426366 :   -0.026820054527616995 :   -0.03667275919651239 :   (b('ndvi') <= 1045.0) ?  -0.018158945264011864 :   (b('dg0_v_vv') <= -0.46481895446777344) ?  0.018888720070570495 :   0.004741195670590691 :   (b('l8b4') <= 1002.4047546386719) ?  (b('ndvi') <= 1630.5) ?  (b('dg0_v_vv') <= 0.08504867553710938) ?  -0.03456749001637035 :   -0.03766252468819277 :   (b('l8b4_med') <= 865.178466796875) ?  -0.04070000413204672 :   0.010523103241631448 :   (b('ndvi') <= 2058.5) ?  (b('ndvi') <= 1800.0) ?  -0.029351380917029313 :   0.0068666523142996636 :   (b('dg0_v_vv') <= 1.7255525588989258) ?  -0.038470980610864965 :   -0.07624614356074923 :   (b('l8b11_med') <= 1736.4746704101562) ?  0.08762052969304454 :   (b('ndvi') <= 1356.0) ?  (b('ndvi_med') <= 1383.75) ?  (b('dg0_v_vv') <= 0.15807580947875977) ?  -2.1384761599424023e-05 :   -0.030189609030736454 :   (b('moss') <= 12.155912399291992) ?  -0.048176150149820005 :   -0.06647028875827966 :   (b('ndvi_med') <= 1383.75) ?  (b('ndvi') <= 2211.0) ?  0.024705947801475104 :   -0.01841405545157007 :   (b('dg0_v_vv') <= 1.119605541229248) ?  -0.0048715105579252745 :   -0.03147225854933178 :   (b('crops') <= 0.339643657207489) ?  (b('ndvi') <= 1458.0) ?  (b('moss') <= 24.65581512451172) ?  (b('l8b11') <= 2186.51904296875) ?  (b('ndvi_med') <= 1310.25) ?  (b('l8b4') <= 1480.9116821289062) ?  0.011146970186825861 :   0.03673299751358089 :   (b('ndvi') <= 1261.0) ?  -0.003609792583004518 :   -0.022355664719429878 :   (b('l8b4') <= 1901.255126953125) ?  (b('ndvi_med') <= 1411.75) ?  -0.025502705217719097 :   -0.008402929317654956 :   (b('dg0_v_vv') <= -0.9594988822937012) ?  -0.03278089949174014 :   -0.049888702623804126 :   (b('l8b4') <= 1128.6262817382812) ?  (b('ndvi_med') <= 1531.75) ?  0.06324491718058625 :   0.07045245460819657 :   (b('dg0_v_vv') <= 0.7325921058654785) ?  (b('dg0_v_vv') <= -1.0741701126098633) ?  0.0010391077023070572 :   -0.006358902507204042 :   0.049203544168084146 :   (b('l8b11') <= 2294.1025390625) ?  (b('l8b5') <= 2454.9835205078125) ?  (b('l8b11_med') <= 2052.0925903320312) ?  (b('l8b5') <= 1943.7395629882812) ?  0.031620350585819905 :   -0.0030796908531865214 :   (b('l8b11_med') <= 2088.8076171875) ?  0.13642090289268347 :   0.02365902504626113 :   (b('grass') <= 78.96323013305664) ?  (b('l8b11') <= 1888.8544311523438) ?  -0.04431242507714158 :   -0.0199144564001048 :   (b('ndvi_med') <= 1766.0) ?  -0.02148849436415909 :   0.05288472487409608 :   (b('l8b5') <= 2563.483642578125) ?  (b('l8b11') <= 2583.90673828125) ?  (b('l8b4_med') <= 1251.4238891601562) ?  0.06834911083977785 :   0.02105180381889878 :   0.16807792934704438 :   (b('dg0_v_vv') <= 0.8152241706848145) ?  (b('dg0_v_vv') <= -0.36910343170166016) ?  0.020836639003971234 :   -0.019431552088249572 :   (b('dg0_v_vv') <= 0.9126811027526855) ?  0.06469297360814674 :   0.03670446366551616 :   (b('l8b4') <= 1251.142333984375) ?  (b('l8b5') <= 2245.5650634765625) ?  (b('l8b5') <= 2019.581298828125) ?  (b('dg0_v_vv') <= 3.597721576690674) ?  (b('l8b5') <= 1854.62158203125) ?  0.04530498735231408 :   0.01606482010195949 :   -0.036177744451873634 :   (b('ndvi') <= 1672.5) ?  0.09873830270459522 :   (b('dg0_v_vv') <= -0.5093145370483398) ?  0.09668277112811735 :   0.04036736100667885 :   (b('l8b11') <= 2005.4002685546875) ?  (b('l8b5_med') <= 2245.5650634765625) ?  -0.032958723579843 :   -0.043958730970818954 :   (b('dg0_v_vv') <= 1.0001235008239746) ?  (b('dg0_v_vv') <= -1.5729713439941406) ?  0.027300734417348704 :   0.0022463549851946063 :   -0.050577814841109986 :   (b('l8b5') <= 2517.388671875) ?  (b('dg0_v_vv') <= 0.5590248107910156) ?  (b('l8b11_med') <= 2020.285400390625) ?  0.09046187830412847 :   (b('l8b11') <= 2241.3079833984375) ?  0.1230517582024413 :   0.11408695442205408 :   (b('dg0_v_vv') <= 1.2442312240600586) ?  0.08315416347906257 :   0.08182099565840516 :   (b('l8b5_med') <= 2295.3597412109375) ?  -0.003953097994776472 :   0.006400782338919511 :   (b('grass') <= 86.5132827758789) ?  (b('dg0_v_vv') <= 2.5357775688171387) ?  (b('l8b11') <= 1705.0806274414062) ?  (b('ndvi_med') <= 1595.0) ?  (b('dg0_v_vv') <= -0.6335296630859375) ?  -0.04963578552949166 :   (b('l8b11') <= 1502.725341796875) ?  -0.05290997362891983 :   (b('dg0_v_vv') <= -0.48194313049316406) ?  -0.06763578475463128 :   -0.060275382206132024 :   (b('l8b11') <= 1585.800048828125) ?  (b('l8b4_med') <= 1162.889892578125) ?  (b('dg0_v_vv') <= -0.5933361053466797) ?  -0.014986127906040939 :   -0.0252340473820135 :   -0.0011443516547826882 :   (b('ndvi_med') <= 2708.75) ?  (b('sand') <= 60.0) ?  -0.044736206849752325 :   -0.05217573658567079 :   -0.03550854206241842 :   (b('ndvi_med') <= 1274.5) ?  (b('l8b4') <= 1838.1375732421875) ?  (b('l8b11') <= 2813.1798095703125) ?  (b('l8b11') <= 2759.73095703125) ?  0.022816814408331637 :   -0.014248450889897346 :   (b('ndvi') <= 1570.0) ?  0.03793702264158208 :   0.0177357816801864 :   (b('l8b5') <= 2949.9417724609375) ?  (b('ndvi_med') <= 1240.25) ?  -0.04076519205338502 :   -0.019970543655177447 :   (b('dg0_v_vv') <= 1.088831901550293) ?  -0.012266013122880401 :   0.029182806272226802 :   (b('dg0_v_vv') <= 0.011075496673583984) ?  (b('dg0_v_vv') <= 0.005026817321777344) ?  (b('l8b5') <= 2575.744873046875) ?  -0.020887335003511472 :   -0.006622832792345955 :   0.07411587739680184 :   (b('l8b5') <= 1807.8883666992188) ?  0.05123622465656967 :   (b('l8b11_med') <= 1784.0094604492188) ?  0.0266202560045274 :   -0.029910015929753783 :   (b('l8b11_med') <= 2945.2862548828125) ?  (b('l8b11_med') <= 2076.2677001953125) ?  (b('l8b5_med') <= 2287.9278564453125) ?  0.047408244075639205 :   0.08205488663918908 :   (b('grass') <= 83.66984176635742) ?  (b('l8b11') <= 2151.28564453125) ?  (b('moss') <= 12.907143115997314) ?  -0.0037431565026929126 :   -0.01795441039804066 :   (b('l8b11') <= 2948.98681640625) ?  0.00549691668488179 :   0.017788262005965977 :   (b('l8b5_med') <= 2308.179443359375) ?  0.02311286018899128 :   0.03718808625092074 :   -0.0661613610043477 :   (b('dg0_v_vv') <= -0.15828561782836914) ?  (b('l8b4') <= 1589.8518676757812) ?  (b('sand') <= 44.5) ?  (b('l8b11') <= 1554.9121704101562) ?  0.03989839704890727 :   (b('ndvi') <= 3049.5) ?  (b('l8b4') <= 949.9107360839844) ?  -0.054771998092135704 :   -0.015068431136373888 :   0.026044854016268104 :   (b('ndvi_med') <= 1364.25) ?  -0.07298424610471468 :   (b('ndvi') <= 2125.0) ?  -0.045174366172610364 :   -0.0583071546242141 :   0.07099130333303907 :   (b('l8b4_med') <= 1192.2503662109375) ?  (b('ndvi') <= 1857.0) ?  (b('l8b4_med') <= 990.8619995117188) ?  0.10715741656099623 :   0.11415940204227756 :   (b('l8b4_med') <= 810.0767517089844) ?  (b('l8b4') <= 971.486328125) ?  0.008223541703402393 :   0.017637838194527078 :   (b('l8b5') <= 2368.2283935546875) ?  (b('l8b5_med') <= 2158.0924072265625) ?  0.05871354542461566 :   0.08587393231770124 :   0.020722693975844142 :   (b('l8b11') <= 2369.873291015625) ?  (b('dg0_v_vv') <= 2.018953323364258) ?  (b('l8b5') <= 2234.0860595703125) ?  (b('l8b4_med') <= 1225.1925048828125) ?  0.05542185065591046 :   0.002839565348513153 :   (b('l8b5_med') <= 2146.911865234375) ?  0.04238703818008788 :   -0.029239690876845442 :   (b('dg0_v_vv') <= 2.353353977203369) ?  (b('l8b11') <= 2080.0455932617188) ?  0.07121246885441729 :   0.05051913070915007 :   (b('dg0_v_vv') <= 3.094176769256592) ?  0.029969799887474655 :   0.019589945541994058 :   (b('l8b5') <= 2257.1094970703125) ?  -0.01799059640039226 :   (b('l8b5_med') <= 2146.911865234375) ?  0.10066964348428964 :   0.10431104775886046 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1291.0094604492188) ?  (b('dg0_v_vv') <= 0.39598798751831055) ?  (b('moss') <= 6.230724096298218) ?  (b('ndvi') <= 2641.0) ?  (b('ndvi') <= 2595.5) ?  (b('l8b4_med') <= 1046.5274047851562) ?  (b('l8b5_med') <= 3460.30615234375) ?  (b('dg0_v_vv') <= -0.897829532623291) ?  (b('l8b4_med') <= 1013.7525024414062) ?  (b('l8b11_med') <= 1566.0999755859375) ?  0.010158778801453654 :   -0.0032551813348986576 :   (b('dg0_v_vv') <= -1.8516292572021484) ?  0.010979860055095741 :   0.08500149322958117 :   (b('l8b5') <= 2627.66162109375) ?  (b('bulk') <= 153.5) ?  0.011994359398581001 :   0.05597265958791033 :   (b('l8b11') <= 1918.5230712890625) ?  0.03297079004845642 :   0.001585689549435349 :   (b('l8b4') <= 1017.9563903808594) ?  (b('l8b5') <= 3331.3629150390625) ?  (b('sand') <= 17.0) ?  0.06530549615333525 :   0.024241206614189714 :   (b('l8b11') <= 1230.07568359375) ?  0.0017028443219626183 :   -0.02610886769245427 :   (b('dg0_v_vv') <= -0.7384481430053711) ?  -0.1453513773918155 :   -0.15860025440757858 :   (b('moss') <= 5.575704097747803) ?  (b('l8b5') <= 3077.2708740234375) ?  (b('l8b11') <= 1855.9949340820312) ?  (b('l8b5_med') <= 2156.3699951171875) ?  -0.05234329142869066 :   -0.007382534151722188 :   (b('l8b5') <= 1793.1270751953125) ?  -0.03961114512981307 :   0.0052010377502331555 :   (b('ndvi') <= 1727.5) ?  0.11980291805118509 :   (b('ndvi') <= 2459.0) ?  0.0040022948295359035 :   0.03581274124244774 :   (b('dg0_v_vv') <= -0.4259147644042969) ?  (b('l8b11') <= 1327.6821899414062) ?  (b('dg0_v_vv') <= -0.9240055084228516) ?  0.007811601813990474 :   0.03180049343554095 :   (b('l8b4') <= 1191.82080078125) ?  -0.02377167564189748 :   0.008479258984148055 :   (b('l8b11') <= 1353.510009765625) ?  (b('l8b4') <= 986.3387756347656) ?  -0.0016314419156013282 :   0.02049268445114591 :   (b('l8b5_med') <= 3097.873046875) ?  0.06105425708256148 :   0.16049334816531927 :   (b('dg0_v_vv') <= -0.05834054946899414) ?  (b('dg0_v_vv') <= -0.43366336822509766) ?  (b('l8b11_med') <= 1295.4417114257812) ?  (b('dg0_v_vv') <= -1.3063020706176758) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.042237193785075386 :   0.01766000202308915 :   (b('ndvi_med') <= 3690.25) ?  0.07967610710853867 :   0.05057535648055811 :   (b('l8b11_med') <= 1565.9376220703125) ?  (b('l8b5_med') <= 3218.3529052734375) ?  0.010146686373057365 :   0.019826747102687753 :   (b('sand') <= 68.0) ?  0.032845956067566524 :   0.0431286945878543 :   (b('dg0_v_vv') <= -0.4136009216308594) ?  -0.022748894578587503 :   (b('moss') <= 0.7690265774726868) ?  -0.008157487528591412 :   0.0011185428959009647 :   (b('l8b5_med') <= 3490.31982421875) ?  (b('l8b4') <= 1145.4486083984375) ?  (b('dg0_v_vv') <= 0.06831550598144531) ?  0.060303243470219925 :   (b('ndvi') <= 2614.5) ?  0.044249968325878644 :   0.03873902366517665 :   (b('l8b4') <= 1260.4388427734375) ?  0.06835499307261717 :   (b('ndvi_med') <= 2507.0) ?  0.05488733451030786 :   0.059393864443566924 :   0.0699135972528653 :   (b('ndvi') <= 4906.0) ?  (b('bulk') <= 169.0) ?  (b('bulk') <= 159.5) ?  (b('grass') <= 4.636483192443848) ?  (b('l8b5') <= 3556.963134765625) ?  (b('l8b5') <= 3033.60009765625) ?  0.0035912441146882267 :   0.024904864578166756 :   (b('l8b11') <= 1385.4227294921875) ?  0.0091320090349273 :   -0.016457249428613444 :   (b('ndvi_med') <= 4217.0) ?  (b('l8b5_med') <= 3119.078857421875) ?  0.0014184618661865073 :   -0.005642276913049863 :   (b('l8b5') <= 3030.93359375) ?  0.030587607109675854 :   0.0030763709234796783 :   (b('l8b5') <= 2151.00146484375) ?  (b('ndvi') <= 2880.0) ?  (b('l8b11_med') <= 1797.2599487304688) ?  0.04215335311614231 :   0.02000785949779434 :   (b('dg0_v_vv') <= -4.198979377746582) ?  -0.030310107320933505 :   -0.028921929612047606 :   (b('l8b11') <= 2029.7908325195312) ?  (b('ndvi') <= 3264.5) ?  -0.06002213777076612 :   -0.014178625598471455 :   0.07597359854481339 :   (b('dg0_v_vv') <= -0.23114538192749023) ?  (b('l8b11') <= 1264.88671875) ?  (b('l8b5_med') <= 2587.7828369140625) ?  (b('dg0_v_vv') <= -1.362370491027832) ?  0.0419446408177121 :   0.038611727731089146 :   0.045867533219355716 :   0.06036164276297312 :   0.06393606855825307 :   (b('ndvi') <= 4913.5) ?  (b('bulk') <= 141.5) ?  (b('dg0_v_vv') <= -0.16209006309509277) ?  0.09876668830347257 :   0.07971434826088986 :   (b('dg0_v_vv') <= 0.1850113868713379) ?  0.049112517045480475 :   0.04398135241240764 :   (b('moss') <= 0.994081437587738) ?  (b('dg0_v_vv') <= -0.0863194465637207) ?  (b('moss') <= 0.20671003684401512) ?  (b('l8b11_med') <= 1449.33642578125) ?  0.019485938324751026 :   -0.021551083577988062 :   (b('dg0_v_vv') <= -0.8671102523803711) ?  0.056215466102396194 :   0.07507877946554031 :   (b('l8b5_med') <= 2913.36376953125) ?  (b('l8b4_med') <= 985.4744567871094) ?  0.06800311078226268 :   0.09890438606890012 :   (b('l8b5') <= 3080.4654541015625) ?  0.02347898877425711 :   0.039749338830215586 :   (b('ndvi_med') <= 3609.0) ?  (b('dg0_v_vv') <= -0.9465079307556152) ?  (b('l8b5_med') <= 2452.3594970703125) ?  -0.02098450609130305 :   -0.007454248405478585 :   (b('l8b11_med') <= 1233.3317260742188) ?  -0.06016889991323354 :   -0.05708959548621087 :   (b('dg0_v_vv') <= -0.022734642028808594) ?  (b('grass') <= 23.551274299621582) ?  -0.007092361417108711 :   0.013496119746415552 :   (b('moss') <= 4.838926076889038) ?  0.044722549054091496 :   0.01180518342603018 :   (b('ndvi') <= 3069.5) ?  (b('l8b11_med') <= 958.0080871582031) ?  (b('l8b4_med') <= 639.7485961914062) ?  (b('l8b11') <= 1106.7772827148438) ?  (b('dg0_v_vv') <= -0.15579891204833984) ?  (b('dg0_v_vv') <= -1.5665946006774902) ?  (b('ndvi_med') <= 4041.0) ?  0.03025851201094759 :   0.008292918332750726 :   (b('dg0_v_vv') <= -0.9046592712402344) ?  -0.019229683713450126 :   -0.0008784885797040037 :   (b('l8b11') <= 907.5362243652344) ?  0.03899589352013476 :   (b('l8b11_med') <= 935.4756774902344) ?  0.006274519269874018 :   0.018146824241454126 :   (b('l8b4_med') <= 447.916259765625) ?  (b('dg0_v_vv') <= -1.7679758071899414) ?  0.03179129633973077 :   (b('l8b4') <= 571.0889587402344) ?  0.058545349359039434 :   0.07346066471545243 :   (b('dg0_v_vv') <= -1.1623806953430176) ?  -0.006958371103993999 :   (b('l8b11') <= 1173.6273803710938) ?  0.026334141141836163 :   0.056682727055109294 :   (b('l8b4') <= 721.4338989257812) ?  0.15254375782570354 :   0.1277612221127028 :   (b('l8b4') <= 441.10475158691406) ?  (b('l8b11_med') <= 1272.3217163085938) ?  (b('ndvi') <= 2444.5) ?  -0.062475061594887416 :   (b('ndvi') <= 2599.0) ?  (b('dg0_v_vv') <= -0.4685978889465332) ?  0.018729400023934983 :   0.03547701877432513 :   (b('ndvi') <= 2724.5) ?  0.002482355016850996 :   0.02294219622482045 :   (b('l8b4') <= 395.8429718017578) ?  0.0196791225598254 :   (b('l8b4') <= 433.86647033691406) ?  (b('bulk') <= 147.5) ?  0.06572637103242784 :   0.054978848582821815 :   0.07873480414899775 :   (b('l8b5') <= 1998.9774169921875) ?  (b('moss') <= 13.331553936004639) ?  (b('ndvi') <= 992.5) ?  -0.09717487324845595 :   (b('dg0_v_vv') <= -2.112185478210449) ?  -0.03271736580591592 :   0.014645921493587922 :   (b('ndvi') <= 3055.0) ?  (b('sand') <= 65.5) ?  -0.00428213937757477 :   0.1358507528444986 :   0.1521560700492126 :   (b('sand') <= 59.0) ?  (b('ndvi_med') <= 3771.5) ?  (b('l8b11') <= 1355.2704467773438) ?  -0.01250597114963634 :   -0.005381194448701342 :   (b('l8b5_med') <= 2548.4193115234375) ?  -0.02842931857844371 :   0.03455014917805094 :   (b('moss') <= 9.910544872283936) ?  (b('l8b4_med') <= 1370.558837890625) ?  -0.03403537744192111 :   0.013757509753767239 :   (b('l8b11') <= 1511.6084594726562) ?  0.003462671667642371 :   0.029025096062891404 :   (b('l8b11_med') <= 1970.0264282226562) ?  (b('moss') <= 8.972131252288818) ?  (b('ndvi') <= 4881.5) ?  (b('ndvi') <= 4868.5) ?  (b('crops') <= 24.796006202697754) ?  (b('l8b5') <= 3487.6937255859375) ?  -0.009358305643636515 :   -0.04741973447158992 :   (b('l8b4') <= 773.1913146972656) ?  -0.006854487839534252 :   0.006978189146311391 :   -0.0847673892967858 :   (b('ndvi') <= 4892.5) ?  0.1209866075116211 :   (b('ndvi_med') <= 3141.0) ?  (b('l8b4_med') <= 725.2777404785156) ?  -0.02967678193698419 :   0.008575161693148918 :   (b('dg0_v_vv') <= -0.12619543075561523) ?  0.011874395422128476 :   0.05900223192277486 :   (b('l8b11') <= 1940.169189453125) ?  (b('l8b11') <= 1933.9774169921875) ?  (b('dg0_v_vv') <= 0.39182376861572266) ?  (b('l8b4') <= 882.2240600585938) ?  0.0018581927064678025 :   0.010256550620681967 :   (b('l8b5') <= 2331.143798828125) ?  -0.06326022237971132 :   -0.062358378288017755 :   (b('dg0_v_vv') <= 0.03990793228149414) ?  (b('dg0_v_vv') <= -0.9757890701293945) ?  0.05137407747566247 :   0.01579561538047314 :   (b('ndvi_med') <= 3817.0) ?  0.10016479299702033 :   0.073451666171614 :   (b('sand') <= 36.5) ?  (b('ndvi_med') <= 2920.75) ?  (b('l8b5_med') <= 2507.9146728515625) ?  0.0477776574040234 :   0.019813320735548978 :   (b('l8b5') <= 3038.4705810546875) ?  -0.014362886432888989 :   -0.0017494977648035872 :   (b('ndvi_med') <= 3856.0) ?  (b('crops') <= 38.421499252319336) ?  -0.03416232168342716 :   -0.009397843294225366 :   (b('l8b4') <= 1047.8304138183594) ?  0.051537613624584186 :   -0.030060656264154842 :   (b('l8b4') <= 894.4288330078125) ?  (b('l8b4') <= 848.7659912109375) ?  (b('l8b5') <= 2660.1778564453125) ?  (b('l8b11_med') <= 2073.2791748046875) ?  (b('ndvi') <= 3177.5) ?  0.07920972436061735 :   0.013936520048781884 :   (b('grass') <= 82.5) ?  0.0597683976004745 :   0.07873045897715172 :   (b('dg0_v_vv') <= -0.784609317779541) ?  (b('l8b5') <= 3162.35009765625) ?  0.023776891404428784 :   -0.001570465849540166 :   (b('l8b5') <= 2978.0421142578125) ?  -0.06441979982804694 :   -0.02153575674994447 :   (b('ndvi') <= 3504.0) ?  (b('l8b4_med') <= 1016.8438415527344) ?  (b('ndvi') <= 3189.0) ?  -0.007844742337054122 :   0.06074250920205773 :   (b('l8b4') <= 878.9465026855469) ?  0.015366819771791001 :   -0.038503713566631265 :   (b('ndvi') <= 3675.0) ?  (b('l8b4_med') <= 976.2377319335938) ?  0.07931385479396874 :   0.09790413465306005 :   (b('l8b5') <= 2722.1783447265625) ?  0.048160077422988466 :   0.06003907256323613 :   (b('sand') <= 42.5) ?  (b('l8b5_med') <= 2557.436279296875) ?  (b('dg0_v_vv') <= 0.03649568557739258) ?  (b('l8b11') <= 2097.5114135742188) ?  -0.017158251210973488 :   0.019086209721987248 :   (b('ndvi_med') <= 2830.0) ?  -0.02183507601865665 :   -0.04869624023829635 :   (b('l8b5') <= 3239.47509765625) ?  (b('dg0_v_vv') <= 0.04584789276123047) ?  0.007852153645709258 :   0.06479418232060644 :   (b('l8b11') <= 1869.4028930664062) ?  -0.046543836593591154 :   -0.0023818475934327915 :   (b('ndvi') <= 4132.5) ?  (b('dg0_v_vv') <= -0.9201087951660156) ?  (b('crops') <= 42.57925033569336) ?  -0.0020763315397965415 :   -0.030325908665444462 :   (b('l8b4') <= 1005.989990234375) ?  -0.0010299164893044221 :   0.06472806771162491 :   (b('ndvi') <= 4531.5) ?  (b('dg0_v_vv') <= -0.03936004638671875) ?  -0.036235906053705746 :   -0.027814668010587212 :   0.002243462864781623 :   (b('dg0_v_vv') <= 0.39705657958984375) ?  (b('ndvi_med') <= 2833.25) ?  0.098654184538092 :   0.11982955570293552 :   (b('sand') <= 17.0) ?  (b('dg0_v_vv') <= 0.8106718063354492) ?  (b('ndvi') <= 3299.5) ?  (b('l8b5_med') <= 2281.339599609375) ?  0.03583534060628624 :   (b('l8b11_med') <= 1512.6978759765625) ?  0.057949603686842543 :   0.06379719715536736 :   (b('l8b4_med') <= 841.0370788574219) ?  0.0859259384586204 :   0.10410391108069222 :   (b('l8b5') <= 2568.20458984375) ?  (b('ndvi') <= 3203.0) ?  (b('dg0_v_vv') <= 0.9554433822631836) ?  -0.01149885473041068 :   (b('ndvi') <= 2105.5) ?  (b('l8b11') <= 1799.1797485351562) ?  0.03649172908752296 :   0.017493227933875866 :   -0.008379822821596772 :   (b('l8b5_med') <= 2765.5899658203125) ?  -0.04635975722783331 :   -0.03494310797568251 :   (b('ndvi') <= 4817.0) ?  (b('dg0_v_vv') <= 1.1013317108154297) ?  (b('l8b11') <= 1220.4852294921875) ?  0.0640640231610104 :   0.08706537943455223 :   (b('l8b11_med') <= 1210.298583984375) ?  (b('ndvi_med') <= 4211.5) ?  0.0582612454344672 :   0.05421957696393864 :   (b('l8b4') <= 747.4832763671875) ?  0.012566306238512367 :   0.04748409320226719 :   -0.02497657898686767 :   (b('grass') <= 81.56975555419922) ?  (b('l8b4_med') <= 555.2675476074219) ?  (b('crops') <= 58.344186782836914) ?  (b('bulk') <= 139.0) ?  (b('dg0_v_vv') <= 0.7001347541809082) ?  (b('dg0_v_vv') <= 0.5837202072143555) ?  0.01415855902786643 :   -0.011942947275198056 :   (b('dg0_v_vv') <= 0.7116689682006836) ?  0.07677483416751255 :   0.01776014292169812 :   (b('grass') <= 32.725948333740234) ?  -0.00934456671968803 :   (b('ndvi') <= 3924.5) ?  0.08563683349655289 :   0.0578277289549009 :   (b('l8b4') <= 410.4615783691406) ?  (b('l8b4_med') <= 504.9057922363281) ?  (b('l8b4_med') <= 407.2998352050781) ?  0.040155835826370806 :   -0.010808250419495458 :   (b('l8b4') <= 391.67486572265625) ?  -0.06938180347587658 :   -0.03546281689922425 :   (b('l8b11') <= 922.3342590332031) ?  (b('dg0_v_vv') <= 3.1400158405303955) ?  0.05133773027307461 :   0.020659275581392855 :   (b('l8b5') <= 3369.486328125) ?  0.004044400793982864 :   -0.020473226500313742 :   (b('bulk') <= 163.5) ?  (b('bulk') <= 161.5) ?  (b('l8b4') <= 504.69044494628906) ?  (b('grass') <= 7.209846496582031) ?  -0.0048823536870576945 :   0.013238158793296997 :   (b('ndvi_med') <= 1568.25) ?  0.009509565058972607 :   0.0016270631950587688 :   (b('l8b5') <= 3784.129638671875) ?  (b('ndvi') <= 2756.0) ?  0.018836389586734337 :   0.08859806766672076 :   (b('l8b5') <= 3887.657470703125) ?  -0.04902863489323264 :   -0.05313576474927781 :   (b('l8b11') <= 2014.8001098632812) ?  (b('l8b4') <= 497.28086853027344) ?  (b('l8b11') <= 811.7477111816406) ?  0.0013440777107228685 :   0.020129101150157813 :   (b('dg0_v_vv') <= 1.7841806411743164) ?  -0.02278214028523582 :   -0.05957440187815229 :   0.10212697386198225 :   (b('ndvi') <= 3533.0) ?  (b('l8b11') <= 2178.75927734375) ?  (b('l8b4') <= 1200.462646484375) ?  (b('l8b4') <= 808.0498962402344) ?  (b('l8b4') <= 769.4636535644531) ?  0.0033606298351134046 :   -0.05287105104044866 :   (b('l8b4_med') <= 837.2214660644531) ?  0.056600750195858404 :   0.009577215810860823 :   (b('l8b5_med') <= 2678.5321044921875) ?  (b('moss') <= 10.04272174835205) ?  -0.0024381711871451495 :   -0.044895385124197454 :   (b('l8b5_med') <= 2702.364990234375) ?  -0.10338395503961195 :   -0.12521852623510205 :   (b('bulk') <= 152.5) ?  (b('l8b11') <= 2243.411865234375) ?  (b('l8b11_med') <= 2154.5653076171875) ?  0.06859931472961107 :   0.021515184560711123 :   (b('dg0_v_vv') <= 5.395361423492432) ?  0.012538262353814654 :   -0.04713829907879562 :   (b('dg0_v_vv') <= 2.2909836769104004) ?  (b('l8b5_med') <= 2510.33642578125) ?  0.09146266434539937 :   0.11476450440756081 :   0.0461558600026607 :   (b('dg0_v_vv') <= 0.9674715995788574) ?  (b('bulk') <= 143.5) ?  (b('l8b4_med') <= 891.2711181640625) ?  0.043768649787207736 :   0.05724027915452601 :   (b('l8b4') <= 620.0664672851562) ?  (b('dg0_v_vv') <= 0.673861026763916) ?  0.06571533731361096 :   0.07494875420748753 :   (b('bulk') <= 146.0) ?  0.08889266347090685 :   0.10346114087336555 :   (b('l8b11') <= 1031.9788208007812) ?  (b('sand') <= 30.0) ?  -0.04003812484254393 :   -0.06752322411256023 :   (b('sand') <= 43.5) ?  (b('l8b4_med') <= 989.8822021484375) ?  -0.013296673506262846 :   0.05711288485647039 :   (b('l8b5') <= 2481.9691162109375) ?  -0.05130133594747516 :   0.00022548401014005072 :   (b('moss') <= 39.21678924560547) ?  (b('moss') <= 38.554447174072266) ?  (b('sand') <= 20.5) ?  (b('l8b11') <= 2255.09326171875) ?  (b('l8b5_med') <= 2894.9593505859375) ?  (b('ndvi') <= 2603.0) ?  (b('ndvi') <= 1523.0) ?  (b('l8b4_med') <= 853.3169555664062) ?  0.03751798407868795 :   0.03482131368433117 :   0.047059296212026314 :   0.017867608487861453 :   0.08973168705118495 :   (b('ndvi') <= 2760.0) ?  (b('l8b4_med') <= 632.78857421875) ?  0.03437696629530912 :   (b('l8b4') <= 1601.0047607421875) ?  0.027976800148933834 :   0.03020809518069334 :   (b('l8b5') <= 3281.631591796875) ?  0.020802728108578528 :   0.01929852789589187 :   (b('bulk') <= 122.5) ?  (b('l8b11') <= 3400.300048828125) ?  (b('l8b4') <= 1395.8324584960938) ?  (b('dg0_v_vv') <= -0.12138080596923828) ?  -0.004474524827511914 :   (b('ndvi') <= 1254.0) ?  -0.017310738216009314 :   -0.012952674233571798 :   (b('l8b5_med') <= 2018.583740234375) ?  0.012771895101235872 :   (b('l8b11') <= 2558.1165771484375) ?  (b('l8b4') <= 1639.7323608398438) ?  0.028550695860685843 :   0.024871812040946573 :   (b('moss') <= 11.488032341003418) ?  0.052246305196901655 :   0.03364441336730718 :   0.10601325684371668 :   (b('l8b5_med') <= 1867.3532104492188) ?  (b('l8b5') <= 1620.82763671875) ?  0.03290897823250799 :   (b('dg0_v_vv') <= 1.5652508735656738) ?  (b('grass') <= 42.35586929321289) ?  (b('dg0_v_vv') <= 1.1383676528930664) ?  0.004559422158998128 :   -0.005978487801010318 :   (b('ndvi_med') <= 850.0) ?  -0.07661572876130977 :   -0.024196079893246174 :   (b('l8b5') <= 1872.6900024414062) ?  -0.06304431536147391 :   -0.060835333915857756 :   (b('l8b5_med') <= 1891.8203735351562) ?  (b('ndvi') <= 1307.5) ?  (b('l8b11') <= 2247.886474609375) ?  (b('dg0_v_vv') <= -0.09882497787475586) ?  -0.004019102575628861 :   0.011903829027201662 :   (b('l8b4') <= 1329.177490234375) ?  0.049600221666553326 :   0.016211697474236855 :   (b('dg0_v_vv') <= -0.07094144821166992) ?  0.12346424028388471 :   (b('dg0_v_vv') <= 0.6300702095031738) ?  0.05965286483853406 :   0.07714785159882145 :   (b('dg0_v_vv') <= 0.5396356582641602) ?  (b('dg0_v_vv') <= 0.4709911346435547) ?  (b('l8b11_med') <= 1244.5049438476562) ?  0.031462272583993714 :   -0.0029112124778362188 :   (b('ndvi_med') <= 694.5) ?  0.05462458756608593 :   -0.015317847667814924 :   (b('l8b5') <= 2571.0057373046875) ?  (b('l8b4') <= 1306.8250122070312) ?  -0.047846196761254664 :   0.007315778357272957 :   (b('l8b4') <= 1509.9176025390625) ?  -0.012141804259560426 :   0.00019023224556243524 :   (b('l8b11') <= 2375.5244140625) ?  (b('dg0_v_vv') <= -0.4887251853942871) ?  0.17249973190588272 :   (b('l8b11') <= 2369.6724853515625) ?  (b('l8b4') <= 1549.051025390625) ?  (b('dg0_v_vv') <= -0.05365133285522461) ?  0.05673521798692005 :   (b('grass') <= 51.06731605529785) ?  0.07292077800332453 :   (b('ndvi') <= 1309.0) ?  0.06723947847832379 :   0.06845430953057279 :   0.02720023056993902 :   (b('ndvi') <= 872.5) ?  0.04666892544033219 :   0.15131197125400245 :   (b('l8b11') <= 2508.9583740234375) ?  (b('l8b11_med') <= 2404.4921875) ?  0.0007409461466160588 :   0.056292828328971006 :   (b('moss') <= 38.93268394470215) ?  (b('l8b4_med') <= 1727.3104858398438) ?  -0.02949595076576641 :   -0.06102908027566187 :   -0.07699704197775084 :   (b('l8b5') <= 2366.2205810546875) ?  (b('l8b11_med') <= 2295.140625) ?  (b('dg0_v_vv') <= -1.4775261878967285) ?  0.05573579822139477 :   (b('dg0_v_vv') <= 0.7033274173736572) ?  (b('ndvi') <= 1009.5) ?  (b('dg0_v_vv') <= -0.3260331153869629) ?  -0.045051890905027495 :   (b('l8b4_med') <= 1511.9990844726562) ?  (b('dg0_v_vv') <= 0.16597414016723633) ?  -0.027592177937109455 :   -0.024620267718072582 :   (b('l8b11_med') <= 2221.01025390625) ?  -0.032074033926595524 :   -0.030220144541692265 :   (b('l8b11_med') <= 2010.3120727539062) ?  0.046391602137414945 :   (b('ndvi') <= 1136.0) ?  (b('ndvi') <= 1091.0) ?  -0.009245507485699262 :   0.014497999199404706 :   (b('l8b4') <= 1371.5979614257812) ?  -0.009656811647950141 :   -0.021126576560791893 :   (b('l8b4_med') <= 1641.6903686523438) ?  (b('l8b11_med') <= 2255.6011962890625) ?  (b('grass') <= 44.73517036437988) ?  -0.03780061007945988 :   -0.041527679421322544 :   (b('l8b4') <= 1378.2835083007812) ?  -0.05180023177424313 :   -0.04687276956395181 :   -0.025066617124906945 :   (b('dg0_v_vv') <= 0.7308902740478516) ?  (b('l8b5') <= 2056.2724609375) ?  -0.08320174965593247 :   (b('l8b4') <= 1558.3839111328125) ?  (b('dg0_v_vv') <= -0.034743309020996094) ?  (b('ndvi_med') <= 908.0) ?  -0.00011582891300689069 :   (b('dg0_v_vv') <= -0.7822399139404297) ?  -0.011109467584864988 :   -0.025093618014938073 :   (b('l8b5') <= 2127.35693359375) ?  -0.07221634929933961 :   -0.02850926253983306 :   (b('dg0_v_vv') <= -0.30361509323120117) ?  (b('ndvi') <= 1082.0) ?  -0.06543283187580191 :   -0.05087762571934122 :   -0.03422522641443208 :   -0.0974623468631797 :   (b('l8b4') <= 1400.897705078125) ?  (b('l8b5_med') <= 2777.7930908203125) ?  (b('bulk') <= 151.0) ?  (b('dg0_v_vv') <= 0.07806110382080078) ?  (b('l8b11') <= 1912.0281982421875) ?  0.09011283262124219 :   0.07428449228881398 :   (b('l8b11_med') <= 3123.62255859375) ?  (b('sand') <= 58.5) ?  (b('l8b11_med') <= 2145.9505615234375) ?  0.041940447550343665 :   0.0426172142111529 :   0.05168398473910922 :   0.06868755770279346 :   0.012035560153032587 :   -0.011917724346612452 :   (b('l8b5_med') <= 2370.42431640625) ?  (b('ndvi') <= 1175.0) ?  (b('l8b11') <= 2732.3486328125) ?  -0.007598462250578392 :   -0.060004867563353464 :   (b('l8b4_med') <= 1591.178955078125) ?  0.16700021230341397 :   0.0732653001920635 :   (b('l8b4_med') <= 1526.7412109375) ?  (b('dg0_v_vv') <= 0.005368709564208984) ?  (b('l8b5') <= 2518.0152587890625) ?  (b('l8b5') <= 2377.2884521484375) ?  0.023215776510783562 :   (b('dg0_v_vv') <= -0.6422452926635742) ?  -0.0198595068231897 :   -0.004514986628673934 :   (b('ndvi') <= 1394.5) ?  (b('l8b11') <= 2498.9713134765625) ?  0.024274796748218648 :   0.0505810865397854 :   (b('dg0_v_vv') <= -0.9656062126159668) ?  0.01299541373163736 :   -0.00532025732134077 :   (b('l8b5') <= 2377.2884521484375) ?  (b('dg0_v_vv') <= 0.7577018737792969) ?  (b('ndvi_med') <= 1311.0) ?  0.00436188247671801 :   0.011686929942139872 :   0.03818983001945121 :   (b('dg0_v_vv') <= 1.054764747619629) ?  (b('l8b4') <= 1460.1439208984375) ?  0.036478674724095675 :   -0.009806616455366967 :   (b('l8b4_med') <= 1445.4017944335938) ?  -0.013177614723121005 :   -0.03174791164253362 :   (b('ndvi') <= 1199.5) ?  (b('dg0_v_vv') <= 1.3844690322875977) ?  (b('l8b4') <= 1638.1707763671875) ?  (b('l8b11_med') <= 2221.01025390625) ?  -0.02565907695460426 :   -0.03259962357301337 :   (b('ndvi') <= 1005.0) ?  -0.025305292509800084 :   -0.02028151995540723 :   0.10844648502887615 :   (b('ndvi') <= 1207.0) ?  (b('dg0_v_vv') <= 1.7403945922851562) ?  (b('dg0_v_vv') <= -0.6856417655944824) ?  0.008838762037893792 :   0.0393297390769834 :   0.000516067564964473 :   (b('l8b4') <= 1690.4158935546875) ?  (b('l8b11') <= 3221.3919677734375) ?  -0.004831310467463541 :   0.05691489844716208 :   (b('l8b11') <= 2317.8974609375) ?  -0.041322146703281964 :   -0.009689776835057989 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1431.54345703125) ?  (b('dg0_v_vv') <= 0.003483295440673828) ?  (b('l8b4_med') <= 587.9694213867188) ?  (b('l8b11_med') <= 1374.5391845703125) ?  (b('l8b5') <= 2909.875244140625) ?  (b('ndvi') <= 3549.5) ?  (b('ndvi_med') <= 3530.5) ?  (b('l8b11') <= 984.5714111328125) ?  (b('dg0_v_vv') <= -0.5544662475585938) ?  (b('l8b11') <= 855.2587585449219) ?  -0.01328722813610031 :   0.06227973244926188 :   (b('moss') <= 9.424793617799878) ?  -0.02884072585854181 :   0.03621463716927448 :   (b('crops') <= 21.855030059814453) ?  (b('l8b11') <= 1175.8106689453125) ?  -0.006282438524258328 :   -0.027017857055164157 :   (b('l8b11') <= 1670.56884765625) ?  0.008947576940770944 :   -0.007575662616422049 :   (b('ndvi') <= 3455.0) ?  (b('ndvi') <= 2869.5) ?  (b('dg0_v_vv') <= -1.2486090660095215) ?  -0.003538955889539601 :   0.023809995721901574 :   (b('dg0_v_vv') <= -3.393871784210205) ?  0.07272765619970947 :   -0.002799765327508221 :   (b('grass') <= 47.82616424560547) ?  (b('bulk') <= 147.5) ?  0.08012910432634729 :   0.08121907276882046 :   (b('dg0_v_vv') <= -1.7164154052734375) ?  0.036552668147613765 :   0.038320264809338395 :   (b('ndvi') <= 3663.5) ?  (b('ndvi') <= 3594.0) ?  -0.05967414004222693 :   (b('dg0_v_vv') <= -1.0893316268920898) ?  (b('dg0_v_vv') <= -3.4813923835754395) ?  -0.037394642001320144 :   -0.02694324910821932 :   (b('dg0_v_vv') <= -0.8121848106384277) ?  -0.04263371657389299 :   -0.03867625372048221 :   (b('l8b11') <= 985.5273742675781) ?  (b('ndvi_med') <= 3908.5) ?  (b('dg0_v_vv') <= -0.00923776626586914) ?  0.012064431634937117 :   0.04905727178436483 :   (b('l8b5') <= 2569.2557373046875) ?  -0.015026434876136335 :   0.0005658053822801041 :   (b('l8b4') <= 457.70758056640625) ?  (b('l8b4_med') <= 481.7151336669922) ?  -0.0736716574098602 :   -0.06825516775667162 :   (b('dg0_v_vv') <= -0.6123461723327637) ?  -0.0029809629785966262 :   -0.021146001164741392 :   (b('l8b5') <= 2932.7662353515625) ?  (b('dg0_v_vv') <= -0.6659293174743652) ?  (b('l8b4') <= 585.3446655273438) ?  (b('l8b4_med') <= 540.7818908691406) ?  0.019777806913580775 :   0.02599302426370048 :   0.03708344192606465 :   (b('ndvi') <= 3511.0) ?  0.05952744748087552 :   (b('dg0_v_vv') <= -0.38351964950561523) ?  0.08754267937335541 :   0.06908683564977353 :   (b('l8b11') <= 936.2147216796875) ?  (b('l8b5_med') <= 3898.131591796875) ?  (b('l8b11') <= 820.6914672851562) ?  (b('ndvi_med') <= 3653.25) ?  -0.030208655131074517 :   -0.06914426141778482 :   (b('crops') <= 40.5445671081543) ?  -0.019084911833901316 :   0.0030245246057937665 :   (b('dg0_v_vv') <= -0.30704307556152344) ?  (b('ndvi') <= 4735.5) ?  0.0336024753997089 :   0.018203766324612514 :   (b('ndvi') <= 4101.5) ?  0.019707762691682174 :   0.009294602709205568 :   (b('crops') <= 1.3697916269302368) ?  (b('l8b11') <= 1258.6776733398438) ?  (b('l8b4_med') <= 548.2572326660156) ?  0.01117081990651747 :   0.07508584792010593 :   -0.07507030944387846 :   (b('ndvi_med') <= 4009.0) ?  (b('dg0_v_vv') <= -3.066438674926758) ?  -0.03266345084145374 :   0.014586673351428173 :   (b('ndvi_med') <= 4047.0) ?  -0.029873944473619518 :   0.004873292619267701 :   (b('l8b11') <= 1380.6896362304688) ?  (b('moss') <= 4.300304889678955) ?  (b('dg0_v_vv') <= -1.6807494163513184) ?  (b('ndvi') <= 3030.0) ?  0.03164963120444089 :   0.01815988555112663 :   0.0007239044847649756 :   -0.03227159152902455 :   (b('dg0_v_vv') <= -0.8939356803894043) ?  0.07216939986168769 :   (b('dg0_v_vv') <= -0.5823221206665039) ?  0.1135818812255514 :   0.10135892952318837 :   (b('crops') <= 98.06986236572266) ?  (b('ndvi_med') <= 1237.0) ?  (b('sand') <= 39.0) ?  0.1221736702560773 :   (b('sand') <= 59.5) ?  (b('bulk') <= 137.5) ?  (b('l8b5') <= 2200.8403930664062) ?  (b('l8b11') <= 1400.4362182617188) ?  -0.019460168225285446 :   0.006521256738227896 :   0.11111781625549624 :   (b('ndvi_med') <= 885.0) ?  (b('dg0_v_vv') <= -0.04877519607543945) ?  0.05491556444839081 :   -0.021951173105658993 :   (b('dg0_v_vv') <= -1.5736169815063477) ?  0.005104431061751933 :   -0.02946852175625211 :   (b('l8b11') <= 1978.7288818359375) ?  -0.014593316854007582 :   (b('moss') <= 32.46576404571533) ?  0.0031376490578507166 :   (b('l8b4') <= 1337.0682983398438) ?  0.05012221300130831 :   0.03759958465189034 :   (b('l8b4_med') <= 894.2960815429688) ?  (b('ndvi_med') <= 2865.25) ?  (b('ndvi_med') <= 2463.25) ?  (b('dg0_v_vv') <= 0.0005581378936767578) ?  (b('moss') <= 22.514674186706543) ?  -0.0002914519338221314 :   -0.017452226262078718 :   (b('l8b5') <= 1741.359375) ?  -0.06943375163977736 :   -0.11199578249974163 :   (b('l8b4') <= 425.2475280761719) ?  (b('dg0_v_vv') <= -0.5090193748474121) ?  0.05038380431182852 :   0.0006933855205450978 :   (b('l8b4_med') <= 738.2451171875) ?  -0.03599772998942316 :   -0.010953298468358203 :   (b('l8b5') <= 2230.9736328125) ?  (b('bulk') <= 153.0) ?  (b('l8b5') <= 2223.220458984375) ?  0.006253428318553444 :   0.04323983657494616 :   (b('ndvi') <= 3045.0) ?  -0.06900973462430075 :   -0.017340106885196133 :   (b('grass') <= 7.209846496582031) ?  (b('ndvi') <= 3801.5) ?  -0.010675715392703356 :   -0.034598648149933985 :   (b('grass') <= 16.609797477722168) ?  0.004378038269693947 :   -0.003891976533532988 :   (b('l8b4') <= 1054.4241943359375) ?  (b('l8b5_med') <= 1780.4354248046875) ?  (b('ndvi') <= 1652.0) ?  (b('l8b11') <= 1555.1101684570312) ?  0.046885422325738155 :   -0.022441106404028438 :   (b('l8b4') <= 992.0627746582031) ?  0.0635633812069885 :   0.18958227325886823 :   (b('l8b4') <= 1053.773193359375) ?  (b('l8b11') <= 2075.9967041015625) ?  0.0014199405073507673 :   0.02076999983209478 :   (b('dg0_v_vv') <= -0.8494381904602051) ?  0.11166007217801666 :   0.07640195445486703 :   (b('l8b11') <= 2960.445068359375) ?  (b('l8b11') <= 1548.2012329101562) ?  (b('l8b11') <= 1468.9370727539062) ?  0.12442835278872141 :   0.03519015249474134 :   (b('ndvi_med') <= 1246.0) ?  0.060911561017934455 :   -0.0034839997516675114 :   (b('l8b4_med') <= 1346.0046997070312) ?  (b('bulk') <= 148.5) ?  0.028571630689940165 :   -0.03577246964500874 :   0.20783111027320814 :   (b('ndvi') <= 3385.0) ?  (b('ndvi') <= 2372.5) ?  (b('ndvi') <= 2175.5) ?  (b('l8b5') <= 1402.6544799804688) ?  (b('l8b4') <= 717.6527404785156) ?  0.028881544012121063 :   -0.08387881092424815 :   (b('dg0_v_vv') <= -0.4695873260498047) ?  (b('l8b11_med') <= 1351.4182739257812) ?  0.04347950267829298 :   0.005594431077757876 :   (b('grass') <= 0.686528742313385) ?  0.015392385724981389 :   0.06842546730151458 :   (b('dg0_v_vv') <= -0.636836051940918) ?  (b('l8b11_med') <= 1677.1636352539062) ?  0.030980147561788907 :   (b('ndvi_med') <= 2504.0) ?  0.07005537728728581 :   0.07331249604635337 :   (b('ndvi') <= 2223.0) ?  0.05254026737876366 :   (b('ndvi_med') <= 2585.5) ?  0.008459951040469121 :   -0.00025540644460844697 :   (b('l8b5') <= 3527.137451171875) ?  (b('ndvi') <= 2585.5) ?  (b('l8b4') <= 843.0013122558594) ?  -0.021601934629301822 :   (b('dg0_v_vv') <= -0.6441950798034668) ?  -0.007236250883154924 :   -0.0006440779722970599 :   (b('l8b11') <= 1742.4879760742188) ?  (b('ndvi') <= 2963.5) ?  0.028517399080172354 :   0.008088705711734458 :   (b('dg0_v_vv') <= -0.07241010665893555) ?  -0.002927194675868361 :   -0.015067824450305967 :   (b('l8b4_med') <= 817.5586853027344) ?  (b('l8b11_med') <= 1223.5452880859375) ?  -0.049640044251087034 :   -0.0434680585860599 :   0.0017269291512133356 :   (b('l8b5_med') <= 3322.9915771484375) ?  (b('l8b11') <= 1072.6799926757812) ?  (b('dg0_v_vv') <= -1.0328912734985352) ?  (b('dg0_v_vv') <= -1.3169193267822266) ?  (b('crops') <= 99.25062942504883) ?  0.015522387238742255 :   0.03412986980758825 :   -0.023668495582888638 :   (b('dg0_v_vv') <= -0.15189218521118164) ?  (b('dg0_v_vv') <= -0.5484623908996582) ?  0.04174559016234336 :   0.024282591378012516 :   0.07416661022193966 :   (b('dg0_v_vv') <= -1.888622760772705) ?  (b('moss') <= 0.10467614606022835) ?  (b('sand') <= 54.5) ?  0.029615972714095606 :   0.009639094133612788 :   (b('l8b4_med') <= 1192.4903869628906) ?  0.06275028549289785 :   0.044292664628536316 :   (b('dg0_v_vv') <= -0.15846872329711914) ?  (b('ndvi_med') <= 2585.5) ?  0.009260325316181352 :   -0.008358181922566424 :   (b('l8b11_med') <= 1676.4300537109375) ?  0.03451783834402862 :   -0.0042120682453685165 :   (b('l8b4') <= 800.603515625) ?  0.04585604921563918 :   0.06799864167000363 :   (b('l8b4_med') <= 1312.4214477539062) ?  (b('l8b4_med') <= 1253.6868896484375) ?  (b('l8b5_med') <= 3188.8311767578125) ?  (b('dg0_v_vv') <= 1.746766448020935) ?  (b('dg0_v_vv') <= 1.2928977012634277) ?  (b('l8b5') <= 1445.3436889648438) ?  (b('l8b5_med') <= 1657.2463989257812) ?  (b('l8b5') <= 1440.6194458007812) ?  -0.020301081498904132 :   -0.061637153025044675 :   (b('l8b11_med') <= 2081.1094970703125) ?  -0.0014272767400319883 :   -0.06329480524665651 :   (b('bulk') <= 154.5) ?  (b('dg0_v_vv') <= 0.06838178634643555) ?  0.0106605027089337 :   0.0018856568555778183 :   (b('grass') <= 55.64550971984863) ?  0.0006277815184859006 :   -0.01844712396165817 :   (b('ndvi') <= 1025.5) ?  (b('l8b4') <= 1191.1727294921875) ?  (b('l8b4') <= 1022.2771911621094) ?  0.10552082466606238 :   0.0902607000658075 :   0.019553897885546567 :   (b('grass') <= 80.64921569824219) ?  (b('dg0_v_vv') <= 1.3175749778747559) ?  -0.02589560444646995 :   -0.0037382347638993865 :   (b('l8b5') <= 1907.124267578125) ?  -0.0010388287769193566 :   -0.0532963473725906 :   (b('moss') <= 14.565800666809082) ?  (b('dg0_v_vv') <= 5.3003528118133545) ?  (b('l8b5_med') <= 2794.910888671875) ?  (b('l8b5_med') <= 2430.5523681640625) ?  0.006693417432179505 :   0.020953388059629735 :   (b('dg0_v_vv') <= 1.7852191925048828) ?  0.03040802625412384 :   0.0030561729907695643 :   (b('crops') <= 55.25153923034668) ?  (b('moss') <= 8.186009645462036) ?  -0.021420110102815828 :   0.028576045594017385 :   (b('grass') <= 2.184100389480591) ?  0.05533377225555297 :   0.007029642969176919 :   (b('l8b11') <= 1236.4301147460938) ?  (b('l8b11_med') <= 1092.5799560546875) ?  (b('l8b5_med') <= 2724.859619140625) ?  -0.010963410755097179 :   0.03767950354336752 :   (b('ndvi') <= 2989.5) ?  -0.05373197383468489 :   -0.027603926286254098 :   (b('l8b4') <= 1263.0935668945312) ?  (b('l8b5') <= 2415.725830078125) ?  0.008639732228404386 :   -0.00966101641483982 :   (b('l8b5_med') <= 2153.3941650390625) ?  -0.04886789586682081 :   0.026280865124498315 :   (b('ndvi') <= 2880.0) ?  (b('bulk') <= 156.5) ?  (b('moss') <= 2.220794439315796) ?  (b('l8b5_med') <= 3589.1407470703125) ?  (b('dg0_v_vv') <= 2.637277603149414) ?  0.021490333920432335 :   0.05845395123013444 :   (b('l8b4') <= 796.6485290527344) ?  0.004991106334728361 :   -0.029981436406314205 :   (b('ndvi_med') <= 3524.0) ?  (b('dg0_v_vv') <= 0.1588115692138672) ?  -0.010817884427002405 :   -0.027175683085028542 :   (b('l8b4') <= 1088.0570678710938) ?  0.006502793877476043 :   -0.0050980019850017655 :   (b('dg0_v_vv') <= 0.07712078094482422) ?  -0.0625810534730542 :   (b('dg0_v_vv') <= 0.49353981018066406) ?  (b('l8b4') <= 1025.6739501953125) ?  0.14455479296749135 :   0.07773959813314969 :   (b('ndvi') <= 2756.0) ?  0.03524912940009196 :   0.06797751335775913 :   (b('ndvi') <= 3575.5) ?  (b('l8b4') <= 1134.623046875) ?  (b('l8b4_med') <= 679.1680297851562) ?  (b('l8b5') <= 3314.963134765625) ?  0.01627757673830383 :   -0.016575892027515388 :   (b('l8b11') <= 1919.7256469726562) ?  -0.023109367425017922 :   -0.1519585634286587 :   (b('l8b4') <= 1163.782958984375) ?  (b('l8b4_med') <= 927.497314453125) ?  0.047149228092902395 :   0.06770905444065346 :   (b('l8b4') <= 1228.8849487304688) ?  -0.014434105438477646 :   0.014162611711501905 :   (b('crops') <= 86.78485870361328) ?  (b('l8b11') <= 1116.2722778320312) ?  (b('ndvi') <= 4840.5) ?  0.024929603190532 :   -0.00047033954567771407 :   (b('ndvi') <= 4158.0) ?  0.016187015291009677 :   -0.0006038593051919749 :   (b('l8b11') <= 1106.8346557617188) ?  (b('l8b4') <= 427.7957458496094) ?  -0.017681429852729736 :   0.02035730340288107 :   (b('l8b4') <= 1017.7830505371094) ?  -0.037432937134825116 :   -0.06971228269082624 :   (b('l8b4') <= 1386.76171875) ?  (b('bulk') <= 151.0) ?  (b('l8b5') <= 2090.248046875) ?  (b('l8b4_med') <= 1255.791748046875) ?  -0.1123604311226909 :   (b('l8b5_med') <= 1861.4979248046875) ?  0.043572558505867054 :   (b('l8b5') <= 1706.2802734375) ?  -0.06763835445364622 :   -0.01837619776912008 :   (b('ndvi_med') <= 1252.5) ?  (b('dg0_v_vv') <= 1.4427509307861328) ?  0.07717801356255478 :   0.01280432199362147 :   (b('l8b11') <= 2680.6165771484375) ?  (b('grass') <= 91.90863800048828) ?  -0.005231931315318725 :   0.02445300877869285 :   0.05282123065696162 :   (b('l8b5') <= 3173.009521484375) ?  (b('dg0_v_vv') <= 0.028177738189697266) ?  0.04955356086698473 :   (b('l8b11_med') <= 2290.548095703125) ?  (b('ndvi') <= 2901.0) ?  -0.09499555326536846 :   -0.033281142175349415 :   (b('ndvi') <= 2452.0) ?  -0.01773425926536469 :   -0.06759656626027516 :   (b('ndvi') <= 3817.0) ?  0.03128441076957855 :   0.06230492325927728 :   (b('l8b4') <= 1400.2921752929688) ?  (b('ndvi') <= 1306.0) ?  (b('dg0_v_vv') <= 1.0126094818115234) ?  0.04569010663706169 :   0.05678084087007773 :   0.08809442741987161 :   (b('l8b4') <= 1407.9923706054688) ?  0.025111597336020752 :   (b('l8b5_med') <= 2481.1683349609375) ?  (b('grass') <= 83.82758712768555) ?  (b('grass') <= 59.96796226501465) ?  -0.007391547992536926 :   -0.009644505946533946 :   0.0006876488055588548 :   -0.022772094651665742 :   (b('l8b4_med') <= 1314.4850463867188) ?  (b('l8b4') <= 835.0386962890625) ?  (b('l8b5') <= 2789.453369140625) ?  0.047496139540207205 :   0.06693601352134049 :   (b('l8b5') <= 2259.1363525390625) ?  0.10905799326770665 :   0.14582246346210576 :   (b('l8b5_med') <= 3661.0555419921875) ?  (b('l8b11_med') <= 2651.83154296875) ?  (b('l8b11_med') <= 1878.9017944335938) ?  (b('l8b5') <= 1575.2937622070312) ?  -0.08586103083120278 :   (b('l8b11') <= 1491.8885498046875) ?  (b('l8b5') <= 1759.77001953125) ?  0.004280832203460955 :   0.03693452991205054 :   (b('l8b11') <= 1650.2550659179688) ?  -0.06545641527796403 :   -0.010671120648523637 :   (b('l8b4') <= 1398.2536010742188) ?  (b('sand') <= 53.5) ?  (b('bulk') <= 147.5) ?  0.009124804241142963 :   0.02328592161507285 :   (b('dg0_v_vv') <= 6.912488698959351) ?  -5.182690847045952e-05 :   0.054090406613022016 :   (b('l8b4') <= 1410.6304931640625) ?  (b('moss') <= 8.956193208694458) ?  0.1319815683118774 :   0.033693958037108375 :   (b('l8b11') <= 2359.95947265625) ?  0.03464485915344159 :   -0.028612358534954287 :   (b('l8b11') <= 2375.3533935546875) ?  (b('l8b5') <= 2664.545166015625) ?  (b('l8b11_med') <= 2736.30615234375) ?  (b('l8b5_med') <= 2549.9381103515625) ?  -0.058763022061151124 :   -0.02358600679340443 :   (b('l8b5_med') <= 2900.4581298828125) ?  0.012434321720174276 :   -0.003374112571503661 :   (b('bulk') <= 149.0) ?  (b('grass') <= 31.331445693969727) ?  -0.04997769663367291 :   -0.04169430740246926 :   (b('ndvi_med') <= 1301.75) ?  -0.07818513911349592 :   -0.006480091834757711 :   (b('l8b4') <= 1119.5015258789062) ?  0.09412802757494554 :   (b('sand') <= 58.5) ?  (b('sand') <= 47.5) ?  0.018381632882466442 :   -0.006934461415955428 :   (b('l8b11') <= 2829.816650390625) ?  0.01918044460758546 :   0.06554287023602917 :   (b('dg0_v_vv') <= 0.32736682891845703) ?  (b('l8b5') <= 2592.97314453125) ?  (b('l8b4') <= 1118.3236083984375) ?  -0.041070980539350765 :   -0.045814234708859536 :   -0.07844573473098126 :   (b('dg0_v_vv') <= 0.9048256874084473) ?  (b('l8b4') <= 674.7421417236328) ?  0.05249550271869219 :   0.06391467034195716 :   (b('l8b5') <= 3100.424072265625) ?  (b('dg0_v_vv') <= 3.016148805618286) ?  (b('dg0_v_vv') <= 1.9852890968322754) ?  0.011034009062580044 :   -0.06474665015593477 :   (b('l8b5') <= 2082.4375610351562) ?  -0.015148981561568464 :   0.03728936443998329 :   (b('l8b11') <= 1147.6455383300781) ?  (b('dg0_v_vv') <= 2.794381618499756) ?  -0.05811885743249959 :   -0.034096979727146724 :   (b('l8b11') <= 1963.079345703125) ?  0.0036961093766798756 :   -0.02364537539149041 :   (b('sand') <= 23.5) ?  (b('ndvi') <= 2090.0) ?  (b('dg0_v_vv') <= 2.523313522338867) ?  (b('l8b4') <= 1664.7926025390625) ?  (b('ndvi') <= 1112.5) ?  (b('dg0_v_vv') <= -0.3187103271484375) ?  (b('dg0_v_vv') <= -2.3676156997680664) ?  (b('dg0_v_vv') <= -2.688912868499756) ?  -0.008829628299042547 :   -0.0050034495679922575 :   (b('dg0_v_vv') <= -1.2362799644470215) ?  (b('ndvi_med') <= 2107.5) ?  -0.00025460394374973716 :   0.001996549716752005 :   -0.007237599066957839 :   (b('l8b11') <= 2176.686767578125) ?  -0.02322099675073963 :   -0.013046938230739463 :   (b('l8b11') <= 2213.2496337890625) ?  (b('ndvi') <= 1754.0) ?  (b('l8b5_med') <= 2896.4532470703125) ?  (b('l8b11') <= 2164.856689453125) ?  0.04235336659082367 :   0.03370023348592463 :   (b('dg0_v_vv') <= -0.8662862777709961) ?  0.03356144728407953 :   -0.006734251639160721 :   -0.006295057748008759 :   (b('ndvi') <= 1648.0) ?  (b('ndvi') <= 1322.0) ?  0.0499507469221305 :   0.07093373207148987 :   0.01304183241545745 :   (b('l8b4') <= 1911.1488647460938) ?  (b('l8b11') <= 2675.6246337890625) ?  (b('ndvi') <= 1984.0) ?  (b('l8b5_med') <= 3068.2596435546875) ?  -0.026181841011778742 :   -0.03047337755891552 :   -0.021866348272486735 :   (b('l8b5_med') <= 3068.2596435546875) ?  -0.02376311772536202 :   (b('l8b4_med') <= 1975.3472900390625) ?  -0.011763115758408743 :   -0.007725060978013981 :   (b('l8b11') <= 2998.9918212890625) ?  (b('l8b11') <= 2799.4171142578125) ?  (b('l8b11_med') <= 2840.42578125) ?  (b('ndvi') <= 1507.0) ?  0.007901011718664919 :   0.012005287998333683 :   (b('l8b5_med') <= 3106.2227783203125) ?  -0.02610502658733624 :   -0.03152384619784386 :   (b('l8b11') <= 2864.925048828125) ?  (b('dg0_v_vv') <= -0.18880701065063477) ?  0.005019776854326696 :   0.008019782934000463 :   (b('l8b11_med') <= 2840.42578125) ?  0.03374796744420631 :   0.0129955251534664 :   (b('l8b4') <= 2022.7396850585938) ?  (b('dg0_v_vv') <= 1.5169119834899902) ?  (b('l8b4') <= 1998.6466674804688) ?  -0.007883201078971495 :   0.013529397866996765 :   -0.026941169810735188 :   (b('dg0_v_vv') <= -0.3422966003417969) ?  (b('l8b5') <= 3235.0867919921875) ?  -0.026861630185059607 :   -0.021509354555333704 :   (b('dg0_v_vv') <= 0.3066577911376953) ?  -0.009361737385699329 :   -0.007151417021967116 :   (b('l8b11') <= 2677.7247314453125) ?  (b('l8b4') <= 1517.1581420898438) ?  0.012089341596997927 :   (b('l8b11') <= 2324.9598388671875) ?  0.03133918231589805 :   0.029261419872475825 :   0.08378288676930246 :   (b('ndvi') <= 3309.0) ?  (b('ndvi') <= 2237.0) ?  0.11180397067183284 :   (b('dg0_v_vv') <= -3.1095919609069824) ?  0.10322600952301417 :   (b('ndvi_med') <= 3476.0) ?  (b('dg0_v_vv') <= -1.0538063049316406) ?  (b('ndvi_med') <= 2273.75) ?  (b('dg0_v_vv') <= -2.8583240509033203) ?  0.02590955705926451 :   0.002460643502165285 :   (b('l8b5') <= 2820.6722412109375) ?  0.025179120134040445 :   0.04091728127451751 :   (b('ndvi_med') <= 2273.75) ?  (b('dg0_v_vv') <= 1.4789557456970215) ?  0.08223547502848014 :   0.049153437423395174 :   (b('dg0_v_vv') <= -0.450803279876709) ?  0.05867068156099592 :   0.025526982480920793 :   (b('ndvi') <= 2471.0) ?  -0.008295726944289317 :   0.004793161357106418 :   (b('l8b11_med') <= 2380.4537353515625) ?  (b('l8b4') <= 1659.7876586914062) ?  (b('dg0_v_vv') <= -0.7991433143615723) ?  (b('ndvi') <= 4445.5) ?  (b('ndvi') <= 3747.0) ?  0.012286283705897627 :   (b('grass') <= 14.4358491897583) ?  0.02238519749608328 :   0.021462555938417838 :   -0.011980028491702072 :   (b('dg0_v_vv') <= 1.16166090965271) ?  (b('dg0_v_vv') <= -0.4653773307800293) ?  -0.002475842899313796 :   (b('l8b11') <= 2421.0057373046875) ?  -0.015953330729990153 :   -0.005062080480887354 :   0.008977454479566438 :   -0.03370656784968548 :   (b('ndvi') <= 3840.0) ?  0.0002774077851113166 :   (b('dg0_v_vv') <= 1.5876266956329346) ?  0.07525256745914494 :   0.05025996162162072 :   (b('ndvi') <= 3793.0) ?  (b('ndvi') <= 3766.5) ?  (b('l8b5') <= 1980.7316284179688) ?  (b('l8b4') <= 1444.447265625) ?  0.13721104406058654 :   (b('l8b4_med') <= 1202.5624389648438) ?  -0.0689541558851788 :   (b('l8b4') <= 1596.6246337890625) ?  (b('l8b11') <= 2295.302490234375) ?  (b('ndvi_med') <= 805.25) ?  0.07906594871684496 :   0.03794509882141062 :   (b('ndvi') <= 809.5) ?  -0.012941197274109514 :   0.014558768960121852 :   (b('l8b11') <= 1980.8878784179688) ?  -0.03531081602427889 :   0.0008229936536533616 :   (b('grass') <= 83.66984176635742) ?  (b('dg0_v_vv') <= -0.43339061737060547) ?  (b('l8b4') <= 2176.759033203125) ?  (b('l8b4') <= 1885.4034423828125) ?  (b('moss') <= 5.575704097747803) ?  -0.007058375640652309 :   0.0006764659961907975 :   (b('l8b11_med') <= 2584.6463623046875) ?  0.014331977276779819 :   0.002045581937072583 :   (b('ndvi') <= 1778.0) ?  (b('ndvi_med') <= 2114.0) ?  -0.0030422902116854834 :   0.08861522586770483 :   (b('ndvi') <= 1797.0) ?  -0.041309620293522226 :   -0.01112469080208729 :   (b('bulk') <= 163.5) ?  (b('sand') <= 64.5) ?  (b('ndvi') <= 2409.0) ?  -0.0039721244230003245 :   0.0023675785806094927 :   (b('l8b5') <= 3097.4656982421875) ?  -0.014190654122894057 :   -0.0024581030243159183 :   (b('l8b4_med') <= 1540.2407836914062) ?  (b('l8b11_med') <= 2392.1243896484375) ?  -0.012307427358261 :   -0.05469650710892035 :   (b('l8b5') <= 3176.1141357421875) ?  -0.03176677183895125 :   -0.0022739930736801334 :   (b('l8b5') <= 3529.839111328125) ?  (b('sand') <= 41.5) ?  (b('ndvi') <= 2122.0) ?  (b('ndvi') <= 1577.5) ?  -0.009616034284576401 :   0.021138946301618416 :   (b('ndvi') <= 2851.0) ?  -0.04991469993527394 :   0.015447400932381727 :   (b('ndvi') <= 1363.5) ?  (b('l8b5_med') <= 2418.1844482421875) ?  0.012673370177166078 :   0.04692998932070208 :   (b('l8b4') <= 1841.2529296875) ?  -0.0013277375589996337 :   0.06280703888495759 :   0.136645510382023 :   (b('grass') <= 9.706935405731201) ?  (b('ndvi_med') <= 2029.0) ?  (b('dg0_v_vv') <= 0.531959056854248) ?  (b('dg0_v_vv') <= -0.42236757278442383) ?  -0.06152372155957472 :   -0.056500730283861794 :   -0.05066202788437632 :   (b('dg0_v_vv') <= 0.09713053703308105) ?  -0.06744180086119841 :   -0.07969337672191218 :   -0.028105112669998544 :   (b('sand') <= 46.5) ?  (b('l8b5') <= 3359.2828369140625) ?  (b('l8b5') <= 3235.0390625) ?  (b('dg0_v_vv') <= 0.5909335613250732) ?  (b('l8b5') <= 3047.7197265625) ?  (b('ndvi') <= 4512.0) ?  -0.0002287249053323226 :   0.0014137029309324867 :   -0.01399223703740754 :   (b('sand') <= 41.0) ?  (b('sand') <= 32.0) ?  -0.014119277046471312 :   -0.016793876338327463 :   -0.025137500867267196 :   (b('dg0_v_vv') <= -1.4028863906860352) ?  (b('dg0_v_vv') <= -2.281834602355957) ?  -0.015392756789818829 :   -0.011232507768599936 :   (b('dg0_v_vv') <= -1.205737590789795) ?  0.08376749103930717 :   (b('l8b5_med') <= 2802.52197265625) ?  0.025635915433987716 :   (b('ndvi') <= 4320.5) ?  0.014626288683158142 :   0.007575725034716985 :   (b('dg0_v_vv') <= 0.826507568359375) ?  (b('l8b4') <= 1519.9716186523438) ?  (b('l8b11') <= 2418.2608642578125) ?  0.07638203816666839 :   0.07102129825459452 :   (b('dg0_v_vv') <= -0.1790771484375) ?  (b('dg0_v_vv') <= -0.44307470321655273) ?  (b('ndvi') <= 4153.5) ?  -0.0035647009322581913 :   0.02376091668592764 :   -0.025239078307282198 :   (b('bulk') <= 153.5) ?  (b('dg0_v_vv') <= 0.16897201538085938) ?  0.013899856528728272 :   0.022898581049698857 :   0.03253270514994608 :   (b('dg0_v_vv') <= 1.5834956169128418) ?  (b('l8b4') <= 1539.0787353515625) ?  (b('ndvi_med') <= 2399.25) ?  0.09207135282201667 :   0.08530236761272059 :   0.1165277542029394 :   (b('moss') <= 3.2505770921707153) ?  0.05590614455629439 :   0.0410786629974802 :   (b('ndvi') <= 3933.5) ?  (b('l8b5') <= 2686.086181640625) ?  (b('l8b5_med') <= 3183.3114013671875) ?  (b('l8b5_med') <= 2909.9923095703125) ?  0.048140529042766 :   0.042744765498392845 :   0.06453684599411719 :   (b('moss') <= 2.1764705181121826) ?  (b('l8b4_med') <= 2271.3209228515625) ?  (b('ndvi') <= 3886.5) ?  0.029659615499484815 :   0.02621896559688852 :   0.03402991625667817 :   (b('l8b11_med') <= 1537.2548828125) ?  -0.010545541174855955 :   (b('ndvi_med') <= 1726.0) ?  0.011001966241918605 :   (b('sand') <= 53.0) ?  0.004200358395759329 :   0.0009726905933292729 :   (b('l8b4') <= 2186.47314453125) ?  (b('moss') <= 11.66707468032837) ?  (b('l8b4') <= 1477.4345703125) ?  (b('l8b5') <= 2749.5870361328125) ?  (b('l8b4_med') <= 1536.2318115234375) ?  0.0010480624330550525 :   0.012756880468061227 :   0.06662329735633393 :   (b('l8b11_med') <= 1728.7855224609375) ?  (b('dg0_v_vv') <= 3.037703514099121) ?  0.025980300891480054 :   0.00579716345507908 :   (b('l8b4_med') <= 1703.289306640625) ?  -0.01267657971735139 :   0.0017221863793010824 :   (b('l8b11') <= 3194.37060546875) ?  (b('dg0_v_vv') <= -1.052011489868164) ?  (b('ndvi_med') <= 1726.0) ?  -0.014395101515936246 :   -0.013384677046172626 :   (b('dg0_v_vv') <= 1.736259937286377) ?  -0.03987390489804159 :   -0.02993814231376794 :   0.011400789892651475 :   -0.05796894966752936 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1057.9064331054688) ?  (b('l8b11_med') <= 1861.6285400390625) ?  (b('l8b11_med') <= 1640.7611083984375) ?  (b('moss') <= 0.36046431958675385) ?  (b('l8b5_med') <= 3025.7431640625) ?  (b('bulk') <= 104.5) ?  (b('ndvi') <= 3396.5) ?  (b('l8b11') <= 388.9804229736328) ?  0.03479752643544412 :   (b('l8b4') <= 545.5) ?  (b('dg0_v_vv') <= 1.6365883350372314) ?  -0.026064564450126598 :   0.0006971855448159406 :   (b('l8b11') <= 949.4243774414062) ?  0.011098107173883376 :   -0.00644879627416257 :   -0.07146884593755262 :   (b('l8b11') <= 1622.593017578125) ?  (b('ndvi') <= 4218.5) ?  (b('bulk') <= 147.0) ?  (b('l8b11_med') <= 1372.9159545898438) ?  0.009938083172934013 :   0.0316267747636864 :   (b('ndvi') <= 2489.5) ?  0.012911735047858676 :   -0.012022922729102103 :   (b('l8b5') <= 2056.6673583984375) ?  0.12118637100680002 :   (b('ndvi') <= 4224.0) ?  0.0605565017804629 :   0.016966992625763715 :   (b('l8b11') <= 1698.9810791015625) ?  (b('ndvi') <= 2666.0) ?  0.10846660702466326 :   (b('l8b11') <= 1644.5927734375) ?  0.03301617591114392 :   0.05361595790352359 :   (b('l8b5') <= 2752.8973388671875) ?  (b('dg0_v_vv') <= 0.8304057121276855) ?  0.018312794314341775 :   -0.02706339504501898 :   (b('dg0_v_vv') <= 1.4814035892486572) ?  0.048887264344703765 :   0.016483484314088664 :   (b('l8b11') <= 907.5029602050781) ?  (b('l8b11') <= 540.7763977050781) ?  (b('l8b5_med') <= 3177.6944580078125) ?  -0.06397164105002942 :   (b('ndvi') <= 4735.5) ?  -0.029151626696205343 :   -0.011949545410498419 :   (b('dg0_v_vv') <= -1.2292790412902832) ?  (b('dg0_v_vv') <= -1.3728113174438477) ?  (b('dg0_v_vv') <= -2.1917529106140137) ?  -0.0057187090084216225 :   0.026032856041651307 :   (b('ndvi_med') <= 3351.5) ?  -0.034697038400642116 :   -0.05742325845401611 :   (b('ndvi') <= 4824.5) ?  (b('ndvi') <= 2559.0) ?  0.008573043367930178 :   0.03329914528974659 :   (b('l8b4_med') <= 955.183837890625) ?  -0.026326038609200575 :   -0.025319795257106092 :   (b('ndvi_med') <= 2896.75) ?  (b('l8b4') <= 619.0577697753906) ?  (b('ndvi') <= 2792.0) ?  (b('dg0_v_vv') <= 0.1812305450439453) ?  -0.008986834738540311 :   0.012621621204636243 :   (b('ndvi') <= 3517.5) ?  -0.02165659266442034 :   0.0024760295909557195 :   (b('l8b11') <= 1357.998779296875) ?  (b('l8b11') <= 1250.9154052734375) ?  0.025365256780212026 :   0.05040887595510766 :   (b('l8b4') <= 962.0872802734375) ?  -0.01738556266920048 :   0.027442123428902 :   (b('dg0_v_vv') <= 2.176884412765503) ?  (b('ndvi_med') <= 3341.25) ?  (b('l8b5') <= 2555.5509033203125) ?  -5.644304012103474e-06 :   -0.02157918612541608 :   (b('l8b4') <= 411.1520538330078) ?  -0.0314937908269241 :   0.0008067014685346697 :   (b('l8b5_med') <= 3186.5899658203125) ?  (b('l8b11') <= 1595.37890625) ?  -0.010776924516921249 :   0.09409435354932322 :   (b('l8b4') <= 860.7657775878906) ?  0.043072374541004786 :   -0.003781225810420139 :   (b('l8b11_med') <= 1560.2896728515625) ?  (b('ndvi_med') <= 3291.5) ?  (b('l8b4') <= 788.4786682128906) ?  (b('l8b11') <= 1394.6898193359375) ?  (b('l8b4') <= 676.4258422851562) ?  (b('l8b5') <= 2985.42333984375) ?  0.005623322490063006 :   -0.006530404578447014 :   (b('ndvi') <= 2372.0) ?  -0.02728709177340958 :   -0.006273444510052799 :   (b('l8b11') <= 1395.7328491210938) ?  (b('ndvi_med') <= 3141.0) ?  0.05537774598069066 :   0.028046457888840293 :   (b('sand') <= 28.0) ?  0.053316833638253146 :   0.0049584758403098525 :   (b('ndvi') <= 2870.5) ?  (b('l8b11') <= 2028.0042724609375) ?  (b('sand') <= 40.5) ?  -0.014866066896396338 :   -0.0016777405929393717 :   (b('l8b11') <= 2154.34130859375) ?  0.025559742916592125 :   -0.005546536908363003 :   (b('l8b11') <= 1155.1468505859375) ?  0.14301888283938996 :   (b('l8b4_med') <= 851.9125366210938) ?  0.00471327986027765 :   0.06576694846320359 :   (b('ndvi') <= 2924.0) ?  (b('bulk') <= 132.5) ?  (b('l8b5') <= 2720.1220703125) ?  (b('l8b4') <= 590.9365539550781) ?  -0.024909431526301228 :   0.008979324489074597 :   (b('ndvi') <= 2854.5) ?  0.01829857909280332 :   0.04118311757492204 :   (b('sand') <= 18.0) ?  (b('dg0_v_vv') <= -1.2020506858825684) ?  0.0714817626221559 :   0.02277551625207913 :   (b('dg0_v_vv') <= -0.05504274368286133) ?  0.0028746714256003866 :   -0.021142411326004383 :   (b('bulk') <= 153.5) ?  (b('crops') <= 21.69388198852539) ?  (b('ndvi') <= 4943.0) ?  -0.00236741896245588 :   -0.029420398683552826 :   (b('l8b11_med') <= 1187.2113037109375) ?  -0.0032581964736725162 :   0.0052737350220508645 :   (b('l8b11') <= 1320.0569458007812) ?  (b('grass') <= 54.38384819030762) ?  0.02286133805250296 :   -0.07046333053454978 :   (b('ndvi') <= 4406.0) ?  0.06349647138350284 :   0.03125084759876395 :   (b('moss') <= 2.6950172185897827) ?  (b('l8b11') <= 1331.4302978515625) ?  (b('l8b11') <= 1181.5382690429688) ?  0.06717751793484344 :   0.07181147390666565 :   (b('dg0_v_vv') <= -0.5180168151855469) ?  -0.026926325824796538 :   (b('l8b5') <= 2385.660888671875) ?  0.08333994962757063 :   (b('l8b4') <= 801.3921813964844) ?  0.0494812160852664 :   0.02823910131045178 :   (b('l8b11') <= 1554.4505615234375) ?  (b('l8b11_med') <= 1563.2611083984375) ?  (b('dg0_v_vv') <= -0.3427872657775879) ?  (b('l8b11') <= 1401.5047607421875) ?  0.07541699646276029 :   0.1247040145384174 :   (b('l8b5') <= 3349.8177490234375) ?  0.024102528241740896 :   0.11958538080140313 :   (b('l8b11') <= 1551.2145385742188) ?  (b('bulk') <= 127.0) ?  -0.012196638389669705 :   0.009539315924458655 :   (b('ndvi') <= 4396.5) ?  -0.014766658207149563 :   0.18941531986003354 :   (b('l8b11') <= 1819.0646362304688) ?  (b('l8b11') <= 1571.5465698242188) ?  (b('ndvi') <= 4337.5) ?  -0.0384114765102405 :   0.007286382641383565 :   (b('dg0_v_vv') <= 3.089625835418701) ?  -0.0025596499679919594 :   -0.08305944319925329 :   (b('l8b11_med') <= 1583.6500854492188) ?  (b('l8b11') <= 1922.4829711914062) ?  0.09489460152944931 :   0.05412681231091563 :   (b('l8b5') <= 2390.5155029296875) ?  0.02782630976987316 :   -0.00048572780719116176 :   (b('l8b4') <= 1038.779296875) ?  (b('l8b4') <= 914.916259765625) ?  (b('l8b4_med') <= 974.2012023925781) ?  (b('bulk') <= 143.0) ?  (b('moss') <= 0.02757793851196766) ?  (b('l8b4') <= 877.3775634765625) ?  (b('l8b4') <= 838.8551940917969) ?  0.013307079420789727 :   0.06148673325699746 :   (b('ndvi') <= 2533.5) ?  0.003792448821394845 :   -0.03963823055265199 :   (b('l8b5') <= 3361.7244873046875) ?  (b('l8b4') <= 804.3845520019531) ?  0.003471348393619319 :   -0.008309987337257725 :   (b('l8b5') <= 3382.103271484375) ?  -0.05580946486929601 :   -0.007003786448177481 :   (b('l8b11') <= 1324.9476928710938) ?  (b('l8b5_med') <= 2332.61669921875) ?  (b('l8b4') <= 684.8780822753906) ?  -0.1055465276665798 :   -0.06879455531435116 :   (b('ndvi') <= 4919.0) ?  -0.04396541526345113 :   0.02159639116783471 :   (b('ndvi') <= 3406.0) ?  (b('dg0_v_vv') <= 4.098421096801758) ?  -0.014177287762244542 :   -0.11045471081639871 :   (b('l8b11_med') <= 1687.7884521484375) ?  0.02716765781004706 :   -0.008685987208373396 :   (b('grass') <= 64.24798202514648) ?  (b('l8b4') <= 903.2883911132812) ?  (b('l8b4_med') <= 995.8802185058594) ?  (b('dg0_v_vv') <= 0.09858512878417969) ?  -0.013271827760502175 :   0.08070665858971203 :   (b('l8b5') <= 2393.15185546875) ?  -0.009308199795596488 :   0.005300155411894323 :   (b('ndvi') <= 1804.5) ?  0.1231430918911694 :   0.09598111302056522 :   (b('grass') <= 73.41991806030273) ?  (b('l8b5') <= 2115.51416015625) ?  (b('bulk') <= 139.0) ?  0.17103003994135724 :   0.14444683331235814 :   (b('ndvi_med') <= 1930.75) ?  0.07280714112015835 :   0.018974157079010463 :   (b('grass') <= 82.9065170288086) ?  (b('ndvi') <= 3323.0) ?  0.023852670612836833 :   -0.022323203717023204 :   (b('l8b4') <= 468.91290283203125) ?  0.0027851878745321357 :   -0.030710357579126508 :   (b('sand') <= 55.5) ?  (b('l8b5') <= 3096.9490966796875) ?  (b('crops') <= 28.097872734069824) ?  (b('moss') <= 11.203118801116943) ?  (b('bulk') <= 145.0) ?  -0.02430315029671006 :   -0.0636651072987499 :   (b('l8b4') <= 946.4316101074219) ?  -0.002154655927904997 :   -0.020179440526407744 :   (b('l8b11') <= 1228.8668212890625) ?  -0.08488584762212217 :   (b('l8b4') <= 922.4129943847656) ?  -0.02646732755790578 :   -0.003462187302030793 :   (b('ndvi_med') <= 2123.5) ?  (b('dg0_v_vv') <= -0.5279269218444824) ?  0.11746943750421313 :   0.0776067249285593 :   (b('l8b11') <= 1887.4627685546875) ?  (b('l8b4') <= 1006.5395202636719) ?  0.010082811267971638 :   -0.019959736410825553 :   (b('l8b5') <= 3422.661376953125) ?  0.017664336494649314 :   0.05056921369550249 :   (b('dg0_v_vv') <= -0.02805042266845703) ?  (b('l8b11') <= 1591.1309204101562) ?  (b('l8b11') <= 1554.1270751953125) ?  -0.06326147992097791 :   -0.03854048084628933 :   (b('l8b5') <= 2556.2978515625) ?  (b('ndvi') <= 2083.5) ?  0.05350136503778746 :   0.018151013466187475 :   (b('l8b11_med') <= 1805.1024780273438) ?  -0.004152484421646813 :   -0.03656656530940304 :   (b('ndvi') <= 1539.0) ?  (b('ndvi') <= 1439.5) ?  0.14897643946142566 :   (b('dg0_v_vv') <= 0.07668590545654297) ?  0.10850135728918364 :   0.08080167992092582 :   (b('l8b5_med') <= 2321.138916015625) ?  0.044076278312344314 :   0.016611236808628305 :   (b('l8b11') <= 1698.6767578125) ?  (b('ndvi') <= 3065.5) ?  (b('bulk') <= 150.0) ?  (b('l8b4') <= 1051.020263671875) ?  0.0073946515946945734 :   0.0213875415589303 :   0.10301925305266796 :   0.17240746408293425 :   (b('l8b5') <= 3091.66357421875) ?  (b('l8b11') <= 1825.7084350585938) ?  (b('l8b11') <= 1731.7326049804688) ?  (b('l8b4_med') <= 1108.8981323242188) ?  0.004088278978186463 :   (b('dg0_v_vv') <= -0.251375675201416) ?  -0.003801901767640309 :   -0.0016835841265571416 :   (b('dg0_v_vv') <= 0.10091972351074219) ?  (b('l8b11_med') <= 1757.4509887695312) ?  -0.028388019676895783 :   -0.03968397030744658 :   -0.007347529030121737 :   (b('sand') <= 38.5) ?  (b('l8b4') <= 1047.5252685546875) ?  (b('dg0_v_vv') <= 1.5797061920166016) ?  0.06324641819575123 :   0.014247049144315216 :   (b('dg0_v_vv') <= -0.6812148094177246) ?  0.004169238749595356 :   0.02459594976537211 :   (b('grass') <= 87.62616729736328) ?  (b('dg0_v_vv') <= 0.19353866577148438) ?  -0.03784919642073404 :   -0.006781324065646119 :   (b('ndvi') <= 3046.5) ?  -0.006890053833327338 :   0.006290910919740211 :   (b('l8b11') <= 2130.4386596679688) ?  (b('ndvi_med') <= 3350.5) ?  -0.04511310532140744 :   -0.055833182680086024 :   (b('l8b4_med') <= 837.4677429199219) ?  (b('grass') <= 15.153920650482178) ?  -0.025784654672648122 :   -0.029957260739661117 :   -0.03756580618906302 :   (b('sand') <= 47.5) ?  (b('ndvi_med') <= 2760.75) ?  (b('l8b11') <= 1028.2130737304688) ?  (b('ndvi_med') <= 1714.75) ?  (b('bulk') <= 166.0) ?  (b('l8b11_med') <= 1989.0711059570312) ?  (b('moss') <= 19.609821319580078) ?  (b('ndvi') <= 3545.0) ?  -0.07353578981648473 :   -0.08918109311140657 :   (b('dg0_v_vv') <= -0.08416986465454102) ?  -0.011655532680578368 :   -0.02408248722801777 :   (b('sand') <= 41.5) ?  (b('dg0_v_vv') <= 1.530228614807129) ?  -0.023780288823667978 :   0.00432763017772933 :   (b('ndvi_med') <= 1482.0) ?  -0.03353981625522728 :   -0.015847399542752896 :   (b('dg0_v_vv') <= 0.3083505630493164) ?  (b('l8b11') <= 966.2680969238281) ?  -0.1405744717315508 :   (b('ndvi') <= 3855.0) ?  -0.09799732961191916 :   -0.11765979474964461 :   -0.009480604801296422 :   (b('l8b11') <= 976.9027099609375) ?  (b('dg0_v_vv') <= 4.286753177642822) ?  (b('ndvi_med') <= 2174.0) ?  (b('l8b4') <= 607.9713745117188) ?  0.026010647589859045 :   0.04931674515721481 :   (b('dg0_v_vv') <= 1.7835373878479004) ?  -0.0015687269114201628 :   0.02483307880718623 :   (b('grass') <= 4.792314767837524) ?  -0.07213265506923361 :   -0.0336614787077506 :   (b('l8b4') <= 501.724365234375) ?  (b('ndvi') <= 3934.0) ?  -0.06052159586016498 :   (b('l8b5') <= 3914.150390625) ?  -0.035935672499156586 :   -0.0553466218231794 :   (b('l8b5_med') <= 2923.957275390625) ?  (b('l8b5_med') <= 2733.349853515625) ?  -0.02425535089962383 :   -0.025938041225940622 :   (b('l8b4') <= 535.1462707519531) ?  -0.00873747592189933 :   -0.011769309047527501 :   (b('l8b5') <= 2973.46435546875) ?  (b('ndvi_med') <= 1664.25) ?  (b('l8b11') <= 1812.7099609375) ?  (b('l8b4') <= 756.1822204589844) ?  (b('l8b4_med') <= 1652.6871337890625) ?  0.06063665019615759 :   0.004856053161090085 :   (b('ndvi') <= 4593.5) ?  -0.0015818438292243616 :   0.13815825030838091 :   (b('l8b4_med') <= 1213.6558227539062) ?  (b('dg0_v_vv') <= -0.4269556999206543) ?  0.059399862702335736 :   0.025946677668471165 :   (b('ndvi') <= 1496.5) ?  0.04277337608828273 :   0.08177958917389487 :   (b('dg0_v_vv') <= -0.48761749267578125) ?  (b('l8b4') <= 1007.7497863769531) ?  (b('dg0_v_vv') <= -0.5651445388793945) ?  -0.011558430320996094 :   -0.06137879528532719 :   (b('l8b4') <= 1018.5780639648438) ?  0.08737069602316952 :   -0.0011969320179214676 :   (b('ndvi_med') <= 1777.0) ?  (b('l8b4_med') <= 1350.8651733398438) ?  -0.03449125010039582 :   0.011368579056992569 :   (b('l8b4') <= 617.5084838867188) ?  0.0760095995741019 :   0.008953263937231162 :   (b('dg0_v_vv') <= -0.7562355995178223) ?  (b('dg0_v_vv') <= -1.0104608535766602) ?  (b('l8b5') <= 3493.6746826171875) ?  (b('l8b11') <= 1577.9556884765625) ?  -0.030167245133554507 :   0.007986028123534022 :   (b('crops') <= 37.84442988038063) ?  0.054893515858122965 :   0.017315958103051525 :   (b('ndvi') <= 2736.0) ?  -0.0673401224114328 :   -0.07475176208743345 :   (b('bulk') <= 169.0) ?  (b('bulk') <= 146.5) ?  (b('l8b5_med') <= 3011.451416015625) ?  0.025874346911786517 :   -0.017889157693257957 :   (b('bulk') <= 148.5) ?  0.1512858655436801 :   0.04255965566852997 :   -0.09042948988913996 :   (b('l8b11') <= 1978.9874877929688) ?  (b('l8b11') <= 1938.747802734375) ?  (b('l8b5_med') <= 3104.2471923828125) ?  (b('dg0_v_vv') <= 1.5851588249206543) ?  (b('ndvi') <= 2959.0) ?  (b('l8b5') <= 2789.3577880859375) ?  0.03286396017770857 :   -0.025005583808972952 :   (b('l8b4') <= 690.6796875) ?  -0.002461485120142113 :   0.05124914390843106 :   (b('ndvi') <= 4196.0) ?  (b('l8b4') <= 756.3933715820312) ?  0.10122720018907466 :   0.10285594586716779 :   0.10685720504888052 :   (b('l8b5') <= 2668.7979736328125) ?  (b('l8b5') <= 1913.7237548828125) ?  (b('dg0_v_vv') <= -0.9957141876220703) ?  -0.010463636802664975 :   -0.03236033080725961 :   (b('l8b11') <= 1169.73779296875) ?  0.07413426728104883 :   0.032209653954360686 :   (b('ndvi') <= 3844.0) ?  (b('l8b11') <= 1448.2716674804688) ?  0.03829542998136426 :   -0.036983489531518414 :   (b('sand') <= 39.0) ?  0.016725179510987703 :   0.024251443449904347 :   (b('l8b4') <= 889.4938354492188) ?  (b('ndvi') <= 3346.5) ?  -0.11619300080581718 :   (b('dg0_v_vv') <= -0.15587329864501953) ?  -0.05105228721406918 :   (b('ndvi') <= 3768.0) ?  -0.08487913075411552 :   -0.08098857301689755 :   (b('l8b11') <= 1957.8744506835938) ?  0.11670406237922748 :   (b('ndvi') <= 2870.5) ?  (b('dg0_v_vv') <= -0.7077469825744629) ?  -0.0295859378119194 :   -0.02358594055373306 :   (b('dg0_v_vv') <= -0.10810184478759766) ?  0.028861256467981777 :   0.011168525366056226 :   (b('l8b4') <= 888.3203125) ?  -0.09105074303813138 :   (b('l8b11_med') <= 2150.8240966796875) ?  (b('ndvi') <= 2963.5) ?  (b('l8b4') <= 941.5906372070312) ?  (b('l8b4') <= 924.7417907714844) ?  -0.01303374523077755 :   -0.005286400105665473 :   (b('ndvi') <= 2867.0) ?  0.08252664677222696 :   0.01577793602034036 :   (b('ndvi') <= 3515.5) ?  (b('l8b4') <= 940.7997741699219) ?  0.17193711320662872 :   0.09658909803829226 :   (b('dg0_v_vv') <= -0.26297760009765625) ?  0.07573154839946711 :   -0.041790412247883296 :   (b('l8b11') <= 2114.5296630859375) ?  -0.029844492693733543 :   -0.04362368299535668 :   (b('l8b11_med') <= 1877.796142578125) ?  (b('l8b4') <= 860.2272644042969) ?  (b('dg0_v_vv') <= 0.09553670883178711) ?  0.14765387921110817 :   0.10686187972016511 :   (b('dg0_v_vv') <= 0.1617565155029297) ?  (b('ndvi') <= 1611.0) ?  -0.031627284935479436 :   (b('l8b11') <= 1725.559814453125) ?  -0.0016838325809802046 :   0.026117764372357077 :   (b('l8b11') <= 1950.473388671875) ?  (b('l8b4') <= 978.5159301757812) ?  0.040864629512455064 :   0.04848540012186235 :   0.1067592066607394 :   (b('l8b5_med') <= 2134.97314453125) ?  (b('ndvi') <= 1126.5) ?  (b('l8b11') <= 1844.238037109375) ?  (b('dg0_v_vv') <= 2.079402446746826) ?  (b('ndvi') <= 773.5) ?  0.003036000236698158 :   (b('l8b4_med') <= 1071.1250610351562) ?  0.02116684155633265 :   0.012762058215677198 :   0.04410484307241064 :   (b('l8b11') <= 1889.0198364257812) ?  -0.02614595009641764 :   -0.009914562757460416 :   (b('l8b11') <= 1571.2738037109375) ?  (b('l8b4') <= 714.3560485839844) ?  (b('dg0_v_vv') <= 3.093780994415283) ?  (b('ndvi') <= 1998.5) ?  -0.02093238012013579 :   -0.03519534348068598 :   0.004914766370096246 :   (b('dg0_v_vv') <= 0.5147981643676758) ?  (b('sand') <= 53.5) ?  -0.03800924059386798 :   -0.06499832160099166 :   -0.11364206761370249 :   (b('l8b4_med') <= 1058.4039916992188) ?  (b('l8b11') <= 2266.37646484375) ?  (b('ndvi') <= 3869.5) ?  -0.028051304590413467 :   0.02752381210823046 :   (b('dg0_v_vv') <= 1.1176173686981201) ?  -0.018255070046769906 :   0.006179433813574896 :   (b('dg0_v_vv') <= -0.11331796646118164) ?  (b('bulk') <= 137.5) ?  -0.011618486568788278 :   -0.053593726899435395 :   (b('l8b5') <= 1806.1778564453125) ?  0.042693465043051326 :   0.011898496331619999 :   (b('ndvi') <= 2761.5) ?  (b('l8b11_med') <= 2182.400390625) ?  (b('dg0_v_vv') <= 1.253838062286377) ?  (b('grass') <= 85.20565414428711) ?  (b('l8b4') <= 1033.3136596679688) ?  0.022216229068995015 :   -0.018961704475953713 :   (b('ndvi') <= 2614.0) ?  0.10474074792359127 :   0.06048259678625148 :   (b('l8b5_med') <= 2460.38232421875) ?  (b('l8b5') <= 2302.990234375) ?  -0.011842668643218191 :   -0.08572231116471464 :   (b('l8b4') <= 716.6394958496094) ?  0.08581549817030132 :   0.026402739602475772 :   (b('l8b11') <= 2357.608642578125) ?  (b('grass') <= 58.03091621398926) ?  (b('ndvi_med') <= 1481.5) ?  -0.015885339807101048 :   0.013376577025680911 :   (b('dg0_v_vv') <= 1.819450855255127) ?  -0.048171965418373285 :   0.007210162335187925 :   0.11896986940021442 :   (b('dg0_v_vv') <= 2.1965975761413574) ?  (b('l8b5') <= 3822.4747314453125) ?  (b('moss') <= 2.260134220123291) ?  (b('l8b5') <= 1964.77392578125) ?  -0.09698452959555026 :   -0.016262843113761482 :   (b('moss') <= 2.632530093193054) ?  0.051301772758821036 :   -0.004606438778403854 :   (b('l8b11') <= 1488.0306396484375) ?  (b('dg0_v_vv') <= 0.8041057586669922) ?  0.020334611199627175 :   -0.026432153041616894 :   0.0960079570928099 :   (b('l8b11') <= 2343.29248046875) ?  (b('l8b11') <= 1039.705078125) ?  (b('ndvi') <= 4011.5) ?  -0.024172226643531393 :   0.003937688962729266 :   (b('l8b4') <= 728.1829223632812) ?  0.03434532130049986 :   0.00891466460656811 :   0.10694473059813955 :   (b('l8b11_med') <= 1754.156982421875) ?  (b('ndvi') <= 2077.0) ?  (b('ndvi') <= 2051.5) ?  (b('l8b4') <= 1078.7994384765625) ?  (b('ndvi') <= 1257.0) ?  (b('dg0_v_vv') <= -0.7202577590942383) ?  0.07030679549931784 :   (b('dg0_v_vv') <= 0.5225939750671387) ?  0.012806792966120417 :   0.029765583488252395 :   (b('ndvi_med') <= 3391.25) ?  (b('dg0_v_vv') <= 1.0043482780456543) ?  (b('ndvi_med') <= 3379.0) ?  (b('l8b11_med') <= 1679.5550537109375) ?  -0.02517300956334683 :   0.007595002920842618 :   0.057997427941948926 :   (b('dg0_v_vv') <= 2.1061789989471436) ?  (b('sand') <= 30.5) ?  -0.0879855195201724 :   -0.02272462740525627 :   0.023167329109179652 :   (b('dg0_v_vv') <= 0.03391408920288086) ?  (b('l8b11_med') <= 1335.7122802734375) ?  -0.017275097546111978 :   -0.019959169495447793 :   (b('l8b5') <= 2606.18505859375) ?  (b('l8b4_med') <= 815.4116516113281) ?  -0.06797661367812069 :   -0.05317104366868186 :   (b('ndvi') <= 1980.5) ?  -0.08605797988733305 :   -0.08446259187278093 :   (b('moss') <= 1.9500574469566345) ?  (b('dg0_v_vv') <= -1.0822858810424805) ?  (b('sand') <= 38.0) ?  (b('l8b4_med') <= 813.8113098144531) ?  (b('bulk') <= 148.0) ?  0.03296662268436719 :   0.003526757260549518 :   (b('dg0_v_vv') <= -3.3956799507141113) ?  0.008594907369309306 :   -0.006646517745578398 :   (b('l8b4') <= 1141.046142578125) ?  -0.04256521666838528 :   (b('dg0_v_vv') <= -2.352255344390869) ?  -0.015230102572397153 :   -0.01719001349503063 :   (b('l8b11') <= 2547.552001953125) ?  (b('l8b11') <= 2384.499267578125) ?  (b('ndvi') <= 1663.0) ?  0.041005508935242525 :   0.007684666587628598 :   (b('bulk') <= 137.0) ?  0.053712426171586704 :   0.08056553255869275 :   (b('l8b11') <= 2582.798583984375) ?  -0.03676400376305233 :   (b('l8b5') <= 2934.0008544921875) ?  -0.015870151223207368 :   -0.0023264185158657913 :   (b('grass') <= 65.78105545043945) ?  (b('l8b11') <= 1329.6676025390625) ?  (b('l8b4') <= 1144.0018310546875) ?  0.0762627354649412 :   (b('l8b11') <= 1269.8993530273438) ?  0.04857702564905858 :   0.034188365320091424 :   (b('l8b11') <= 2219.3668212890625) ?  (b('bulk') <= 138.5) ?  -0.025728623979142298 :   -0.004458085880720628 :   (b('ndvi') <= 1681.0) ?  0.022712745751729973 :   -0.005019139439752169 :   (b('grass') <= 70.58176040649414) ?  (b('grass') <= 69.11009979248047) ?  (b('l8b5') <= 2834.6019287109375) ?  0.09083859448023683 :   0.07927260954469915 :   (b('dg0_v_vv') <= -0.10045194625854492) ?  0.04301424458051526 :   0.07595854342275588 :   (b('ndvi') <= 1921.5) ?  (b('l8b5') <= 1958.915771484375) ?  0.041074760409095114 :   -0.015486229434454312 :   (b('ndvi') <= 1933.0) ?  0.02538185690881651 :   0.05455241637381899 :   (b('dg0_v_vv') <= -2.3953857421875) ?  (b('dg0_v_vv') <= -2.7729926109313965) ?  (b('dg0_v_vv') <= -2.993546485900879) ?  (b('dg0_v_vv') <= -3.676055908203125) ?  -0.06263254722996252 :   -0.061695158530537905 :   -0.058632546574311425 :   -0.06498055041848197 :   (b('crops') <= 42.59048843383789) ?  (b('dg0_v_vv') <= -0.07845020294189453) ?  (b('ndvi_med') <= 2507.0) ?  (b('dg0_v_vv') <= -1.72267484664917) ?  -0.03482782103082566 :   (b('l8b11_med') <= 1654.3475952148438) ?  -0.04082782015165715 :   -0.03712339001780698 :   (b('grass') <= 21.896406173706055) ?  -0.031123390896975488 :   -0.034673706811470945 :   -0.04641242655640111 :   (b('l8b5') <= 2960.2406005859375) ?  (b('l8b5_med') <= 2792.08935546875) ?  -0.01653860768623515 :   -0.017326470480734907 :   -0.009443551878240863 :   (b('bulk') <= 142.0) ?  (b('l8b11') <= 1660.0047607421875) ?  (b('ndvi') <= 2431.5) ?  (b('l8b5_med') <= 2437.6734619140625) ?  (b('l8b11') <= 1622.0718994140625) ?  -0.09663921078223595 :   -0.07175415067874967 :   (b('dg0_v_vv') <= 0.33992528915405273) ?  (b('ndvi') <= 2344.0) ?  -0.05418114351788523 :   -0.027016639356998273 :   -0.057659476870257964 :   (b('l8b5_med') <= 3687.78271484375) ?  (b('l8b5') <= 4020.37109375) ?  (b('sand') <= 33.0) ?  (b('dg0_v_vv') <= 0.7965888977050781) ?  -0.015867934239845116 :   -0.025181492245568284 :   (b('moss') <= 10.78955864906311) ?  -0.0025954170720147995 :   -0.004588201786501586 :   (b('ndvi') <= 3545.0) ?  -0.021903672180189904 :   (b('l8b11_med') <= 1593.1175537109375) ?  -0.04764428971019964 :   -0.03537913655037919 :   (b('ndvi') <= 3618.5) ?  (b('ndvi_med') <= 3935.5) ?  0.021727672742769433 :   0.030092158156342363 :   (b('l8b11_med') <= 1032.7852478027344) ?  0.0053864608345279286 :   0.01125055388897711 :   (b('l8b11') <= 1713.1316528320312) ?  (b('l8b4') <= 1078.3829345703125) ?  0.10051369722947973 :   (b('grass') <= 9.819149017333984) ?  (b('l8b5_med') <= 2797.3056640625) ?  (b('dg0_v_vv') <= -0.8276352882385254) ?  -0.014259651344440183 :   -0.006259650033137998 :   (b('dg0_v_vv') <= -1.2420430183410645) ?  -0.0026181514284740748 :   0.009805516742074624 :   (b('ndvi') <= 3402.0) ?  (b('bulk') <= 133.0) ?  0.024323522276585012 :   0.05506111267427757 :   (b('l8b5_med') <= 2551.318359375) ?  0.0435910059897795 :   -0.0003932093609583659 :   (b('ndvi') <= 2408.0) ?  (b('bulk') <= 130.5) ?  (b('l8b5') <= 2739.78466796875) ?  (b('l8b4_med') <= 904.2610473632812) ?  -0.0012690557647307204 :   -0.02492947182162426 :   (b('dg0_v_vv') <= -0.46255970001220703) ?  0.002238345469412436 :   0.016861099455131884 :   (b('sand') <= 31.0) ?  (b('l8b5_med') <= 2896.2349853515625) ?  0.018492020984556476 :   0.057470223773218865 :   (b('l8b5') <= 2961.6629638671875) ?  0.02615593596531741 :   -0.001235244288446911 :   (b('sand') <= 22.0) ?  (b('ndvi') <= 3309.0) ?  (b('l8b5') <= 3345.082763671875) ?  -0.009035275510861522 :   0.005339069424327991 :   (b('dg0_v_vv') <= -2.5760765075683594) ?  -0.013675152317099093 :   -0.04816558377890775 :   (b('ndvi') <= 3712.5) ?  (b('l8b5_med') <= 3162.0543212890625) ?  -0.006222476674619395 :   0.005377752543037069 :   (b('ndvi') <= 3974.5) ?  0.027584475060380677 :   0.00490322553725551 :   (b('ndvi_med') <= 2209.0) ?  (b('l8b5_med') <= 2377.9615478515625) ?  0.1006977758958735 :   0.1605866899694315 :   (b('ndvi') <= 4901.0) ?  (b('dg0_v_vv') <= -1.60646390914917) ?  (b('l8b11') <= 2097.886474609375) ?  (b('ndvi') <= 4105.5) ?  (b('ndvi_med') <= 3320.5) ?  -0.01281979750748033 :   -0.028547158363374188 :   -0.0756724955592743 :   (b('sand') <= 32.5) ?  (b('l8b11_med') <= 1389.0803833007812) ?  -0.02508835171923214 :   0.0099462150534912 :   (b('grass') <= 43.98896837234497) ?  0.06171538858243647 :   0.04098718477902025 :   (b('l8b5_med') <= 2900.5435791015625) ?  (b('l8b11') <= 1715.338623046875) ?  (b('grass') <= 55.13003158569336) ?  -0.01345942674898983 :   -0.05824250472631602 :   (b('crops') <= 13.055427074432373) ?  0.05152433799730974 :   0.008986554867192883 :   (b('l8b5') <= 3915.6661376953125) ?  (b('l8b5') <= 3628.343994140625) ?  0.02814341378249922 :   0.07941396987929883 :   (b('moss') <= 5.1009039878845215) ?  -0.062012305080889986 :   0.03378064686304903 :   (b('dg0_v_vv') <= -0.11865472793579102) ?  0.07082507335843838 :   0.18431620325492815 :   (b('bulk') <= 138.5) ?  (b('grass') <= 42.73731994628906) ?  (b('l8b11') <= 3410.2906494140625) ?  (b('l8b4') <= 1196.3527221679688) ?  (b('l8b11') <= 1900.0906372070312) ?  (b('ndvi') <= 1421.5) ?  (b('dg0_v_vv') <= 0.662055253982544) ?  (b('dg0_v_vv') <= 0.07195138931274414) ?  -0.045381996302465366 :   -0.04102359961664914 :   -0.057985604248176364 :   (b('ndvi') <= 3654.5) ?  (b('l8b11') <= 1860.9974365234375) ?  0.003758612767613237 :   -0.058901680423183916 :   (b('l8b11_med') <= 1768.8693237304688) ?  -0.0317005981745989 :   -0.04239669636996046 :   (b('ndvi') <= 1976.0) ?  (b('bulk') <= 131.0) ?  (b('l8b4') <= 1099.6319580078125) ?  0.08002709185494988 :   0.06868736370383718 :   (b('dg0_v_vv') <= 0.38680505752563477) ?  0.04547495659549396 :   0.03221025957421528 :   (b('dg0_v_vv') <= -0.864138126373291) ?  (b('ndvi_med') <= 3713.0) ?  -0.012281664770080452 :   0.024608735486903816 :   (b('l8b4') <= 1168.2674560546875) ?  0.016629143448689786 :   0.03975351933797749 :   (b('l8b11') <= 1995.265380859375) ?  (b('l8b5') <= 2864.282958984375) ?  (b('grass') <= 19.606290817260742) ?  (b('dg0_v_vv') <= 0.6241531372070312) ?  -0.007293368405421716 :   -0.019323147626982867 :   (b('l8b11') <= 1755.8409423828125) ?  -0.02444394205557815 :   -0.03453586046969184 :   (b('l8b11') <= 1781.988037109375) ?  (b('l8b4') <= 1278.9667358398438) ?  -0.011104350340352329 :   0.02005416357215098 :   (b('l8b4') <= 1269.3741455078125) ?  0.04595059364159193 :   0.05068463877915574 :   (b('l8b5') <= 2618.565185546875) ?  (b('crops') <= 72.95339584350586) ?  (b('grass') <= 36.332298278808594) ?  -0.007411993847225982 :   0.022111579818134303 :   (b('l8b11') <= 2433.409423828125) ?  0.058549956005477466 :   0.05172009229103558 :   (b('l8b11_med') <= 2220.626708984375) ?  (b('l8b11') <= 2023.7290649414062) ?  -0.009193358867420285 :   -0.03519728582032048 :   (b('l8b4') <= 1498.5179443359375) ?  0.010492274590533344 :   -0.012190419995234006 :   0.09580914360164505 :   (b('dg0_v_vv') <= 1.2282850742340088) ?  (b('l8b11') <= 1822.242919921875) ?  (b('l8b5_med') <= 2017.455810546875) ?  (b('moss') <= 17.6805157661438) ?  -0.003039970469433384 :   (b('l8b4') <= 1138.9756469726562) ?  -0.06013848906493052 :   -0.07585273374391333 :   (b('grass') <= 70.2210922241211) ?  (b('dg0_v_vv') <= 0.0924382209777832) ?  (b('l8b5_med') <= 2139.15478515625) ?  -0.013442658621252 :   0.013292095010117681 :   (b('l8b5_med') <= 2107.2904052734375) ?  0.011595561846220931 :   0.06675671378215903 :   (b('ndvi') <= 1359.0) ?  0.06418382306030113 :   0.0774193152421172 :   (b('moss') <= 12.982226848602295) ?  (b('l8b4') <= 1321.1309814453125) ?  (b('l8b11') <= 1947.0824584960938) ?  (b('l8b4') <= 1192.900390625) ?  0.05199401969468734 :   0.024003364528662793 :   (b('dg0_v_vv') <= -1.5971455574035645) ?  0.054719641993046575 :   -0.016917947729472443 :   (b('grass') <= 84.46358489990234) ?  (b('dg0_v_vv') <= 0.6408495903015137) ?  0.06466852112160298 :   0.03321968127919428 :   (b('dg0_v_vv') <= -0.07386159896850586) ?  -0.006339707360485586 :   -0.011288847945808436 :   (b('l8b5_med') <= 1960.537109375) ?  (b('l8b11_med') <= 1907.6639404296875) ?  (b('l8b11') <= 1971.1677856445312) ?  -0.027851333562203807 :   -0.07323821821137068 :   (b('dg0_v_vv') <= 0.7911930084228516) ?  0.004502418541045862 :   0.06924450075338154 :   (b('l8b4') <= 1064.2313232421875) ?  0.04015484879825479 :   (b('l8b4_med') <= 1332.2749633789062) ?  -0.027427299739457504 :   -0.0168816957050467 :   (b('bulk') <= 137.5) ?  (b('ndvi') <= 1258.5) ?  (b('ndvi_med') <= 1074.0) ?  (b('sand') <= 46.0) ?  -0.047583739533305164 :   -0.0240028174628667 :   (b('dg0_v_vv') <= 2.0879507064819336) ?  (b('grass') <= 54.031654357910156) ?  0.015047880982776124 :   -0.018308782058623962 :   (b('ndvi') <= 1162.5) ?  0.018246922668553878 :   0.026958726648648762 :   (b('l8b11') <= 2370.21240234375) ?  (b('l8b4_med') <= 1219.20361328125) ?  0.10543188465792275 :   (b('ndvi_med') <= 1126.0) ?  0.06436813962871947 :   0.044583019217519095 :   (b('bulk') <= 130.5) ?  -0.014399562463155408 :   (b('ndvi') <= 1545.5) ?  0.020168070008581312 :   0.004877161959812248 :   (b('dg0_v_vv') <= 3.744800090789795) ?  (b('l8b4') <= 1166.0159301757812) ?  (b('dg0_v_vv') <= 2.1969776153564453) ?  -0.053228246553597036 :   -0.07093486709155553 :   (b('l8b11_med') <= 2017.1489868164062) ?  -0.03665612473606408 :   -0.023013900841274748 :   (b('l8b11') <= 2015.1968994140625) ?  (b('l8b4_med') <= 1238.036865234375) ?  -0.0014805731160502933 :   0.006184363632167178 :   0.02096238592867905 :   (b('bulk') <= 141.5) ?  (b('l8b11') <= 2582.10546875) ?  (b('l8b11_med') <= 2602.9002685546875) ?  (b('l8b11') <= 2570.3958740234375) ?  (b('l8b11_med') <= 2349.494873046875) ?  (b('grass') <= 74.74024963378906) ?  (b('l8b11') <= 2284.0494384765625) ?  0.004980927554213941 :   0.023514529562465008 :   (b('l8b5') <= 2394.657470703125) ?  0.05055923140494589 :   0.00027518517588565695 :   (b('dg0_v_vv') <= 0.0018410682678222656) ?  (b('dg0_v_vv') <= -0.7974724769592285) ?  -0.0012738094535161867 :   -0.034252580587263055 :   (b('sand') <= 43.5) ?  -0.011495206080962864 :   0.060411555515215536 :   (b('l8b11_med') <= 2387.960693359375) ?  0.20901247732237416 :   (b('ndvi') <= 1377.0) ?  (b('l8b11') <= 2575.4639892578125) ?  0.05477864663873132 :   0.08444969024377567 :   (b('dg0_v_vv') <= -0.4922013282775879) ?  0.041295316132654 :   -0.023061654484904714 :   (b('l8b4') <= 1767.6946411132812) ?  (b('dg0_v_vv') <= 0.6196174621582031) ?  (b('l8b4') <= 1574.2560424804688) ?  (b('ndvi') <= 2184.5) ?  0.08544236738318225 :   0.07185850265384797 :   0.10820571091213513 :   (b('l8b11') <= 1988.4009399414062) ?  0.03708732803367648 :   0.05992851104856034 :   0.005151206453439944 :   (b('ndvi_med') <= 2107.5) ?  (b('dg0_v_vv') <= 0.7531909942626953) ?  (b('dg0_v_vv') <= -1.4505629539489746) ?  (b('bulk') <= 139.5) ?  (b('l8b5_med') <= 2844.591552734375) ?  0.0024725131716835658 :   -0.0032357374214223855 :   (b('dg0_v_vv') <= -1.9402613639831543) ?  0.04004519432173675 :   0.016487789039838024 :   (b('sand') <= 42.0) ?  (b('l8b4') <= 1535.483642578125) ?  -0.007747349766552757 :   -0.023435934419269663 :   (b('dg0_v_vv') <= -0.6966300010681152) ?  -0.059753261222931967 :   -0.003444217689337174 :   (b('l8b5_med') <= 3248.876953125) ?  (b('moss') <= 17.14790916442871) ?  (b('dg0_v_vv') <= 1.2030696868896484) ?  0.008149153958245794 :   0.046654503188662 :   (b('dg0_v_vv') <= 1.9456148147583008) ?  -0.027149981695792767 :   0.022742731071818567 :   0.13307675787194148 :   (b('dg0_v_vv') <= 0.9625239372253418) ?  0.07808572151713217 :   0.044238093681055635 :   (b('l8b4') <= 1063.6671752929688) ?  (b('l8b11_med') <= 1950.8327026367188) ?  (b('l8b5_med') <= 2060.9315185546875) ?  (b('ndvi') <= 1405.5) ?  -0.023514329088295954 :   (b('l8b4_med') <= 1021.9130859375) ?  -0.00024303411171335 :   -0.0019548014773572153 :   (b('dg0_v_vv') <= 0.28037261962890625) ?  0.06333020084953603 :   (b('grass') <= 76.15109825134277) ?  0.026407192555635894 :   0.025373008896387786 :   (b('ndvi') <= 3458.5) ?  (b('grass') <= 78.81625747680664) ?  (b('l8b4_med') <= 1113.1678466796875) ?  (b('l8b5') <= 3133.943359375) ?  -0.041715174425782355 :   -0.019518611430153772 :   (b('dg0_v_vv') <= -0.6579170227050781) ?  -0.004846314861786077 :   -0.06508468783430058 :   (b('l8b5_med') <= 2572.6190185546875) ?  (b('dg0_v_vv') <= 1.9606437683105469) ?  -0.03829848586060954 :   0.03231991121102834 :   0.12311276129411705 :   (b('grass') <= 18.617337703704834) ?  (b('dg0_v_vv') <= -3.3357462882995605) ?  0.0027980496006377864 :   0.021815213314656468 :   0.05097882934341158 :   (b('dg0_v_vv') <= -13.231839656829834) ?  (b('bulk') <= 147.5) ?  0.08537829163779811 :   0.07640917205526598 :   (b('sand') <= 37.5) ?  (b('ndvi') <= 3252.0) ?  (b('l8b4') <= 1234.3523559570312) ?  (b('dg0_v_vv') <= -0.1425766944885254) ?  -0.003508328291452235 :   -0.022373580153793413 :   (b('l8b4') <= 1248.5005493164062) ?  0.03424784613327745 :   -0.005822476842480371 :   (b('l8b5') <= 3295.13427734375) ?  (b('ndvi') <= 3571.0) ?  0.04516724011335431 :   0.0050745651298049295 :   (b('ndvi') <= 3624.0) ?  -0.021515913135183907 :   0.0034847509296618323 :   (b('l8b11_med') <= 2628.4404296875) ?  (b('crops') <= 0.339643657207489) ?  (b('sand') <= 68.5) ?  -0.00236094380136517 :   -0.042827257007794106 :   (b('crops') <= 22.789156913757324) ?  0.02148772482990939 :   0.002868808135809283 :   (b('l8b11') <= 1849.8634033203125) ?  (b('grass') <= 7.78277850151062) ?  -0.0031426514591047497 :   -0.024784525233278665 :   (b('ndvi_med') <= 1702.0) ?  -0.0014294549909024607 :   -0.005103971035588628 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 717.7027893066406) ?  (b('l8b4') <= 715.3887634277344) ?  (b('l8b11') <= 1728.825927734375) ?  (b('ndvi') <= 2261.0) ?  (b('l8b5') <= 1503.202880859375) ?  (b('ndvi_med') <= 1622.5) ?  (b('l8b5') <= 1366.7520751953125) ?  (b('crops') <= 25.207279205322266) ?  (b('dg0_v_vv') <= 1.7260725498199463) ?  (b('dg0_v_vv') <= 1.4254615306854248) ?  -0.04127664053055479 :   0.06875129626789461 :   (b('bulk') <= 136.0) ?  -0.05130126017295425 :   -0.08541968867291369 :   (b('l8b4_med') <= 1804.1433715820312) ?  0.007675970214113675 :   0.01391771502068867 :   (b('dg0_v_vv') <= 0.9091782569885254) ?  (b('dg0_v_vv') <= -0.012189865112304688) ?  (b('l8b5_med') <= 1538.142578125) ?  -0.0058770280354356885 :   -0.010070894993900667 :   (b('l8b5') <= 1445.0130004882812) ?  0.03416493805062096 :   -0.012032080239150589 :   0.07259906092790078 :   (b('ndvi') <= 1070.5) ?  -0.08821259069851461 :   (b('ndvi') <= 1274.5) ?  (b('l8b5_med') <= 2564.0040283203125) ?  0.011313649598915121 :   (b('ndvi_med') <= 3470.5) ?  0.05251615246408966 :   0.029715119902241793 :   (b('l8b4') <= 686.0523071289062) ?  (b('l8b4') <= 525.1029663085938) ?  0.005150140128956963 :   -0.007954659683504568 :   (b('l8b4') <= 702.3789367675781) ?  0.06510508131900285 :   0.0019534194825486517 :   (b('l8b5_med') <= 2298.0379638671875) ?  (b('l8b5') <= 1747.15576171875) ?  (b('l8b5') <= 1512.2735595703125) ?  (b('dg0_v_vv') <= 0.5151247978210449) ?  0.08348369856528681 :   (b('ndvi') <= 1508.0) ?  0.08321275483242196 :   0.062125707156556326 :   (b('sand') <= 60.0) ?  (b('dg0_v_vv') <= 0.6691622734069824) ?  0.0019166399010857608 :   0.02830228941105127 :   (b('ndvi') <= 1808.5) ?  0.011941564732703663 :   0.07453960618311593 :   (b('l8b11') <= 1424.0343627929688) ?  (b('l8b11') <= 1136.01123046875) ?  (b('ndvi_med') <= 2028.0) ?  0.052555790868136776 :   0.09509756067598539 :   (b('l8b11_med') <= 1676.1259155273438) ?  0.033325629946304634 :   -0.0155879513532063 :   0.1539270359472215 :   (b('moss') <= 17.64689064025879) ?  (b('ndvi') <= 2202.5) ?  (b('grass') <= 53.112749099731445) ?  (b('l8b4_med') <= 748.6678771972656) ?  0.019243433667978014 :   0.007161819190100378 :   (b('l8b5') <= 2480.6771240234375) ?  -0.020396619162174428 :   0.02678814491409585 :   (b('ndvi') <= 2219.5) ?  0.09408025043014784 :   (b('ndvi') <= 2235.5) ?  0.047009258623665784 :   0.02726422336037032 :   -0.073921899360082 :   (b('ndvi_med') <= 1403.0) ?  (b('ndvi') <= 3120.5) ?  (b('l8b11') <= 903.3510437011719) ?  0.08140616931090655 :   (b('l8b4') <= 527.6947479248047) ?  (b('l8b5_med') <= 1728.5043334960938) ?  -0.028710017753734995 :   -0.02733520338837077 :   (b('dg0_v_vv') <= 1.1911344528198242) ?  0.01794592829727039 :   (b('l8b4_med') <= 1636.6956787109375) ?  0.006021581163344905 :   0.0006047403450291644 :   (b('crops') <= 25.207279205322266) ?  (b('l8b11_med') <= 1987.1481323242188) ?  (b('l8b5') <= 2607.5526123046875) ?  -0.07399652652509542 :   -0.08026298380026592 :   (b('l8b11') <= 1380.44189453125) ?  (b('ndvi') <= 3480.0) ?  -0.06260748828471469 :   -0.05834354202618047 :   -0.06816065388542136 :   -0.031374958678679093 :   (b('l8b4_med') <= 1377.4398803710938) ?  (b('dg0_v_vv') <= -2.4087796211242676) ?  (b('l8b4') <= 598.5989990234375) ?  (b('l8b4_med') <= 860.6597900390625) ?  (b('l8b4_med') <= 716.0110168457031) ?  -0.0018253328416083848 :   -0.020377544426334666 :   (b('l8b11') <= 900.8558044433594) ?  0.041666715715419966 :   0.009881933114438014 :   (b('l8b5') <= 3382.1253662109375) ?  (b('l8b4') <= 627.088134765625) ?  -0.0333021909151103 :   -0.007683698395102329 :   (b('moss') <= 7.5699098110198975) ?  -0.05639355938361661 :   -0.0024974002960899966 :   (b('bulk') <= 129.5) ?  (b('l8b11') <= 1172.8154296875) ?  (b('bulk') <= 101.0) ?  -0.033489302627748276 :   0.0036558027305362345 :   (b('l8b4') <= 659.1035766601562) ?  0.013807241996294519 :   0.0004922450942778343 :   (b('bulk') <= 139.5) ?  (b('l8b11') <= 1154.3980712890625) ?  0.0024522853345113876 :   -0.007063076009064202 :   (b('l8b11') <= 1167.526611328125) ?  0.0013744357545956477 :   0.00976538984945364 :   (b('l8b5_med') <= 2873.3499755859375) ?  (b('bulk') <= 142.5) ?  (b('ndvi') <= 3763.0) ?  0.07510689274241722 :   0.08904758577151045 :   (b('ndvi_med') <= 1838.75) ?  (b('ndvi_med') <= 1833.5) ?  0.03461998388972641 :   0.10140870570277832 :   (b('ndvi') <= 2880.0) ?  0.04455080454110663 :   0.0022936400285207925 :   (b('ndvi_med') <= 2961.75) ?  (b('dg0_v_vv') <= 1.2587203979492188) ?  (b('l8b4_med') <= 1451.3553466796875) ?  -0.04045190669465906 :   0.02228300431814345 :   (b('dg0_v_vv') <= 4.584090232849121) ?  -0.027074020623189518 :   0.03347915490250383 :   (b('l8b11') <= 981.1499328613281) ?  0.05978651454408945 :   0.06672084055294394 :   (b('ndvi_med') <= 3298.0) ?  (b('ndvi') <= 1897.0) ?  (b('ndvi') <= 1534.5) ?  -0.037121972030660755 :   (b('l8b11_med') <= 2085.865478515625) ?  (b('l8b5_med') <= 2135.785400390625) ?  0.012475123719323924 :   0.02144837637345219 :   -0.005332160366288863 :   (b('l8b4') <= 694.5111694335938) ?  (b('dg0_v_vv') <= 2.91011905670166) ?  (b('dg0_v_vv') <= -3.818385124206543) ?  -0.036686929088707465 :   (b('dg0_v_vv') <= 0.545870304107666) ?  (b('l8b5') <= 1637.3715209960938) ?  -0.02689938036061714 :   -0.02892735329597663 :   (b('l8b5') <= 2262.5867309570312) ?  -0.03169470758317694 :   -0.035199578929135514 :   -0.045488692870224084 :   -0.054416812318048374 :   (b('ndvi_med') <= 3763.75) ?  (b('moss') <= 0.3230937719345093) ?  (b('l8b4') <= 698.5525207519531) ?  0.014100798317060959 :   0.0181966726582784 :   (b('l8b4_med') <= 916.4306945800781) ?  0.010494108398087065 :   0.008141150336611003 :   (b('dg0_v_vv') <= 4.306806564331055) ?  -0.028226237594405484 :   0.0003664526069614149 :   (b('l8b5') <= 3309.6239013671875) ?  (b('bulk') <= 132.0) ?  (b('l8b4') <= 716.8713989257812) ?  0.018358275530970414 :   (b('dg0_v_vv') <= 1.3951659202575684) ?  (b('sand') <= 42.0) ?  (b('dg0_v_vv') <= 0.5555219650268555) ?  -0.008986399581026552 :   -0.012369052974916847 :   (b('l8b11_med') <= 1203.1886596679688) ?  -0.019873307386299124 :   -0.018764749371534745 :   -0.02579422339600243 :   (b('ndvi_med') <= 2933.5) ?  (b('l8b4_med') <= 614.1442565917969) ?  (b('l8b5_med') <= 2124.7242431640625) ?  0.05071889849694183 :   0.07080505706383455 :   (b('ndvi') <= 4162.0) ?  (b('dg0_v_vv') <= 0.08781218528747559) ?  (b('l8b11_med') <= 1467.2611694335938) ?  0.023667233748420058 :   0.02059394262903666 :   (b('l8b4') <= 715.8107299804688) ?  -0.0039031634129274573 :   -0.0077293672484929354 :   (b('ndvi') <= 4787.0) ?  0.05988911770246133 :   (b('dg0_v_vv') <= 1.4251368045806885) ?  0.024889106377578823 :   0.04030440072407937 :   (b('l8b5_med') <= 2791.6748046875) ?  (b('ndvi') <= 4787.0) ?  (b('dg0_v_vv') <= 2.122389078140259) ?  (b('ndvi') <= 3629.5) ?  0.107556477518475 :   (b('dg0_v_vv') <= 0.7174174785614014) ?  0.09133780192543549 :   0.10272325657874681 :   0.05864070645541239 :   0.028248951959540747 :   (b('ndvi') <= 2599.0) ?  (b('ndvi_med') <= 3114.25) ?  0.011248449164040525 :   0.054796964354412725 :   (b('ndvi') <= 4787.0) ?  (b('ndvi_med') <= 3114.25) ?  (b('grass') <= 24.62108039855957) ?  0.05941543832036433 :   0.06661721098513212 :   (b('dg0_v_vv') <= -1.431053638458252) ?  0.05304482858780571 :   0.0491742090204807 :   0.030700004263818886 :   (b('dg0_v_vv') <= 0.6791806221008301) ?  (b('dg0_v_vv') <= 0.3072233200073242) ?  (b('l8b11_med') <= 1633.1881713867188) ?  -0.038835855864641305 :   -0.044348845227675 :   (b('l8b11_med') <= 1633.1881713867188) ?  -0.028552349164971735 :   -0.025182850104591536 :   (b('dg0_v_vv') <= 1.498030662536621) ?  (b('ndvi_med') <= 2926.5) ?  -0.01349830603825565 :   (b('l8b5_med') <= 2791.6748046875) ?  0.006331858642456667 :   -0.0034983081244182174 :   -0.038549181768231855 :   (b('moss') <= 4.837940216064453) ?  (b('moss') <= 3.33070170879364) ?  (b('l8b11_med') <= 2348.136962890625) ?  (b('ndvi') <= 1011.5) ?  (b('l8b5_med') <= 3454.0787353515625) ?  (b('l8b11_med') <= 2297.96533203125) ?  (b('l8b5') <= 1871.819580078125) ?  (b('bulk') <= 139.0) ?  (b('sand') <= 25.0) ?  0.0041568230056529265 :   0.02592996256554969 :   -0.025508195955016216 :   (b('dg0_v_vv') <= -1.1902132034301758) ?  (b('l8b11') <= 2597.4395751953125) ?  0.030269868430707225 :   0.0581804888234988 :   (b('l8b5_med') <= 2930.84423828125) ?  0.03242561101233587 :   0.007097723534207068 :   0.07255826581702174 :   0.08065486400409644 :   (b('ndvi') <= 2006.5) ?  (b('l8b11') <= 1419.78759765625) ?  (b('l8b4_med') <= 1052.568603515625) ?  (b('l8b11') <= 1376.014404296875) ?  (b('ndvi') <= 1798.0) ?  0.00785655155002087 :   -0.013119867799081041 :   (b('l8b11_med') <= 1340.5531005859375) ?  -0.060057616355946974 :   -0.0239112867029428 :   (b('l8b5_med') <= 2337.0367431640625) ?  (b('l8b5_med') <= 2191.5059814453125) ?  -0.003962495678935002 :   -0.020389128862420947 :   (b('dg0_v_vv') <= -0.9173474311828613) ?  -0.0333122377849177 :   -0.0603235446976381 :   (b('l8b11_med') <= 2136.084228515625) ?  (b('crops') <= 77.11273193359375) ?  (b('sand') <= 42.5) ?  -0.010014849631220391 :   -0.0005062309073594836 :   (b('grass') <= 7.951443672180176) ?  0.004312956624179859 :   0.045984939687137766 :   (b('l8b4') <= 1245.062744140625) ?  (b('dg0_v_vv') <= 1.1014437675476074) ?  0.031505114528443956 :   -0.00534873950596661 :   (b('l8b4') <= 1329.3375854492188) ?  -0.04354903196334507 :   0.0008141770152723664 :   (b('moss') <= 2.757796883583069) ?  (b('bulk') <= 155.0) ?  (b('ndvi_med') <= 2499.25) ?  (b('l8b11') <= 935.6519775390625) ?  -0.11332100575108695 :   0.01238625717929608 :   (b('grass') <= 98.67486190795898) ?  0.003731568528923499 :   -0.011366178058474386 :   (b('l8b5') <= 2466.8067626953125) ?  (b('l8b5_med') <= 2564.462158203125) ?  0.06693333199004091 :   -0.0060413653518867905 :   (b('l8b11') <= 1632.6224365234375) ?  -0.03938147852204152 :   -0.00724822039293466 :   (b('ndvi') <= 2561.5) ?  (b('l8b4') <= 1391.775146484375) ?  (b('l8b5') <= 2986.9818115234375) ?  -0.006213864708454448 :   0.009236868364094623 :   (b('l8b4_med') <= 1125.3153076171875) ?  -0.016681614001919684 :   -0.03834796201674817 :   (b('ndvi') <= 2817.5) ?  (b('l8b11') <= 1825.5634765625) ?  -0.030929347903798053 :   -0.060465288469662204 :   (b('dg0_v_vv') <= 1.1369247436523438) ?  0.0007516124807727426 :   -0.02523506932782093 :   (b('l8b4_med') <= 1354.9573974609375) ?  (b('l8b5') <= 2792.1796875) ?  (b('l8b4') <= 737.5186767578125) ?  0.09619127055129639 :   (b('l8b11') <= 2192.5245361328125) ?  (b('l8b5_med') <= 2404.226318359375) ?  (b('ndvi') <= 3082.0) ?  -0.12551842163837276 :   -0.0921878960102538 :   (b('l8b4') <= 1001.9034423828125) ?  -0.07824266954137712 :   -0.03449603754558656 :   (b('bulk') <= 146.0) ?  (b('ndvi') <= 1758.0) ?  -0.05179214620988627 :   -0.00862159143444955 :   (b('l8b11_med') <= 2435.175537109375) ?  -0.015367658870913816 :   0.01800234247408353 :   (b('l8b11') <= 2990.0684814453125) ?  (b('l8b11') <= 2563.7650146484375) ?  (b('l8b11_med') <= 2380.2982177734375) ?  0.06987165618432048 :   (b('l8b11') <= 1384.03515625) ?  0.04028974025828955 :   0.016663680480950446 :   (b('l8b11') <= 2790.1636962890625) ?  -0.028294959032749062 :   (b('l8b11_med') <= 2418.3621826171875) ?  -0.01277756602809621 :   -0.013455851157381266 :   0.1123783247469787 :   (b('l8b5') <= 1783.9307861328125) ?  -0.09131455200163227 :   (b('l8b5') <= 2115.770263671875) ?  (b('l8b4') <= 1132.5806274414062) ?  (b('moss') <= 1.5028902292251587) ?  (b('l8b5_med') <= 2248.0164794921875) ?  0.013982950608411426 :   0.06639004466127665 :   (b('ndvi') <= 2237.5) ?  -0.0005568770416973234 :   0.009907623492939263 :   (b('moss') <= 0.010067113675177097) ?  (b('ndvi') <= 1037.0) ?  -0.024907796543726615 :   0.015099297099991388 :   -0.05012646311101679 :   (b('l8b5_med') <= 2461.9915771484375) ?  (b('l8b11_med') <= 2457.32958984375) ?  (b('dg0_v_vv') <= 0.7802739143371582) ?  -0.022004793736243655 :   -0.05238481074206394 :   (b('l8b4') <= 1821.76953125) ?  -0.009024348651720887 :   0.06848929251616251 :   (b('l8b5_med') <= 2621.262939453125) ?  (b('l8b11') <= 2559.435302734375) ?  -0.009445599830300116 :   0.058379265014841156 :   (b('l8b11_med') <= 3132.103515625) ?  -0.0035218087823369335 :   0.0324207586548323 :   (b('moss') <= 3.989913582801819) ?  (b('l8b4') <= 744.0398559570312) ?  (b('dg0_v_vv') <= 0.916928768157959) ?  -0.08651002155235532 :   -0.09906371764346639 :   (b('ndvi') <= 1147.5) ?  (b('dg0_v_vv') <= 1.7428100109100342) ?  (b('l8b4') <= 2090.8731689453125) ?  (b('l8b11') <= 1653.7522583007812) ?  (b('dg0_v_vv') <= -0.5256304740905762) ?  -0.0021097686145531702 :   -0.07639726285990994 :   (b('l8b4') <= 1314.2670288085938) ?  0.046039983345597224 :   -0.00032774121915551784 :   (b('l8b5') <= 2996.64990234375) ?  (b('dg0_v_vv') <= 0.11361312866210938) ?  -0.04430872589432924 :   -0.029988215975198185 :   (b('l8b4_med') <= 1985.1664428710938) ?  -0.009810023563971482 :   -0.02065186332321592 :   (b('dg0_v_vv') <= 3.249049425125122) ?  (b('dg0_v_vv') <= 2.9954264163970947) ?  (b('dg0_v_vv') <= 1.9164481163024902) ?  0.03829526275521604 :   0.01739045000210665 :   (b('grass') <= 18.937304496765137) ?  -0.029166441916079144 :   0.020302037810524698 :   (b('ndvi') <= 880.0) ?  0.04641977847811202 :   0.036662773809204685 :   (b('ndvi') <= 1159.5) ?  0.11513599031008592 :   (b('ndvi') <= 4520.5) ?  (b('l8b11') <= 1433.5638427734375) ?  (b('l8b11') <= 1266.3881225585938) ?  0.04152496826649514 :   0.09048272307727906 :   (b('crops') <= 38.37983417510986) ?  0.05706794366452731 :   0.018047413817599764 :   (b('moss') <= 3.7895843982696533) ?  (b('l8b5') <= 3206.090087890625) ?  -0.057137179788212325 :   -0.0256048192223818 :   (b('l8b11') <= 1691.5521850585938) ?  -0.03842759072342819 :   0.06476771756748301 :   (b('ndvi') <= 1232.0) ?  (b('ndvi') <= 1117.0) ?  (b('l8b4') <= 1773.4035034179688) ?  (b('l8b4_med') <= 1703.289306640625) ?  0.0455791201007211 :   0.040417184250708116 :   (b('l8b11') <= 3919.140625) ?  (b('grass') <= 20.011174201965332) ?  -0.03306565586779289 :   -0.01699619041359797 :   (b('dg0_v_vv') <= 0.6558756828308105) ?  (b('ndvi') <= 1028.5) ?  0.02370404372861469 :   -0.0030377198312748765 :   -0.016677562713307767 :   (b('dg0_v_vv') <= -0.5730719566345215) ?  (b('dg0_v_vv') <= -0.7211809158325195) ?  (b('l8b5') <= 2633.9835205078125) ?  (b('grass') <= 57.60942077636719) ?  0.03324708937413055 :   -0.005121451582604533 :   (b('l8b11_med') <= 2600.8411865234375) ?  -0.020574859533401796 :   -0.048900184253711454 :   0.07454842411046796 :   (b('ndvi') <= 1186.0) ?  (b('dg0_v_vv') <= -0.0745687484741211) ?  (b('l8b5_med') <= 2720.0592041015625) ?  -0.04821307571478979 :   -0.04352047784042219 :   (b('dg0_v_vv') <= 1.0089597702026367) ?  -0.06765605895858352 :   -0.08016612857845627 :   (b('ndvi') <= 1192.0) ?  0.06523771468838718 :   (b('l8b5') <= 2417.63916015625) ?  -0.04918205037162176 :   -0.01348473123218119 :   (b('ndvi') <= 1432.0) ?  (b('l8b11') <= 2732.70703125) ?  (b('l8b11') <= 2675.920654296875) ?  (b('l8b11') <= 1876.4444580078125) ?  (b('ndvi') <= 1261.5) ?  -0.013069450380652166 :   0.015025819990081181 :   (b('dg0_v_vv') <= -0.13463878631591797) ?  0.03998376611969498 :   0.0526408863242184 :   (b('l8b4') <= 1855.5227661132812) ?  (b('dg0_v_vv') <= -0.33939313888549805) ?  -0.042316839718092816 :   -0.04387430369503301 :   -0.05420314584230153 :   (b('l8b11') <= 3593.1661376953125) ?  (b('ndvi') <= 1239.5) ?  -0.016553656748650156 :   (b('ndvi') <= 1268.0) ?  0.11026664625927692 :   0.050516676625157145 :   (b('dg0_v_vv') <= 1.1346988677978516) ?  (b('l8b4_med') <= 1917.5946044921875) ?  -0.013148378141093875 :   -0.017481072200655395 :   -0.044748380190682946 :   (b('ndvi') <= 1537.0) ?  (b('l8b5') <= 2640.882080078125) ?  0.032123890275362776 :   (b('dg0_v_vv') <= 0.6460142135620117) ?  (b('dg0_v_vv') <= 0.39261293411254883) ?  -0.04133684834521671 :   -0.06809580002273702 :   0.0026155028538242253 :   (b('moss') <= 4.333116054534912) ?  (b('l8b11_med') <= 1205.5317993164062) ?  (b('dg0_v_vv') <= 0.45695066452026367) ?  -0.058949970149523745 :   -0.08794702604007726 :   (b('ndvi') <= 4579.5) ?  0.0015472154009472763 :   -0.03091751554660458 :   (b('l8b11') <= 2749.652587890625) ?  (b('l8b5_med') <= 2970.993896484375) ?  0.03006665247573135 :   0.012593892780834887 :   (b('dg0_v_vv') <= -0.0055027008056640625) ?  0.0203412805439221 :   -0.01419291006897398 :   (b('grass') <= 33.77793312072754) ?  (b('l8b4') <= 730.5987243652344) ?  (b('l8b5_med') <= 2957.10302734375) ?  (b('ndvi') <= 3250.5) ?  (b('l8b11_med') <= 1102.9674682617188) ?  0.06560118822358485 :   (b('ndvi') <= 2337.5) ?  (b('grass') <= 32.87439727783203) ?  (b('dg0_v_vv') <= -1.3022422790527344) ?  -0.04062435497414907 :   -0.061689341065944425 :   (b('dg0_v_vv') <= -0.2340250015258789) ?  -0.025657785427028748 :   -0.03495955730599652 :   (b('dg0_v_vv') <= 0.09588384628295898) ?  (b('dg0_v_vv') <= 0.0019316673278808594) ?  -0.003603906683088226 :   0.039295474589304547 :   (b('grass') <= 20.59507942199707) ?  -0.049943301794596134 :   -0.024083150372943573 :   (b('l8b4_med') <= 832.3216552734375) ?  (b('dg0_v_vv') <= -1.1300582885742188) ?  (b('ndvi') <= 4189.0) ?  (b('l8b5_med') <= 2882.512939453125) ?  -0.008198515632516537 :   -0.011497430815178268 :   -0.02171682755089885 :   (b('l8b5_med') <= 2717.585205078125) ?  (b('l8b5_med') <= 2526.1004638671875) ?  -0.023687557076267865 :   -0.008527212254956104 :   (b('l8b11') <= 1471.9832153320312) ?  0.0032404052059448215 :   0.027620166144846 :   (b('l8b4') <= 723.1719360351562) ?  0.05893306052918837 :   0.05413442971713869 :   (b('l8b5_med') <= 3363.070556640625) ?  (b('l8b4') <= 726.2864990234375) ?  (b('l8b4_med') <= 913.7209777832031) ?  -0.07623817432935653 :   -0.1099048298499673 :   (b('ndvi') <= 4321.0) ?  (b('ndvi_med') <= 2719.0) ?  (b('l8b11_med') <= 1754.5415649414062) ?  -0.07120418606432988 :   -0.07418116922108009 :   (b('dg0_v_vv') <= 1.180185079574585) ?  -0.039320576594763595 :   -0.06663940940217633 :   (b('ndvi_med') <= 2378.75) ?  0.0002867756200105043 :   -0.006067212331949762 :   (b('dg0_v_vv') <= 2.2581560611724854) ?  (b('l8b11_med') <= 1235.6159057617188) ?  0.011588452189045528 :   0.02595725013240488 :   -0.006497453780584944 :   (b('l8b4') <= 743.5192260742188) ?  (b('l8b11_med') <= 1363.4205322265625) ?  (b('l8b11_med') <= 1346.330078125) ?  (b('l8b5_med') <= 2692.0948486328125) ?  (b('l8b4') <= 737.5300903320312) ?  (b('dg0_v_vv') <= -1.8529291152954102) ?  -0.01632670682586458 :   -0.010326702117097644 :   -0.003989061677060618 :   -0.01898905482252647 :   (b('dg0_v_vv') <= -1.0830926895141602) ?  -0.036810219486882595 :   -0.028306500123410988 :   (b('ndvi_med') <= 3088.75) ?  (b('dg0_v_vv') <= -0.0640416145324707) ?  (b('l8b11_med') <= 1754.5415649414062) ?  -0.0021998814759883467 :   0.008666245685628016 :   (b('l8b4') <= 741.475830078125) ?  (b('l8b11') <= 1226.88134765625) ?  0.11723584193890235 :   0.05934634916821061 :   -0.008814128223246426 :   (b('l8b4') <= 737.09619140625) ?  (b('l8b5') <= 3431.1676025390625) ?  (b('ndvi') <= 3888.0) ?  0.008588256020624348 :   -0.009021668756106039 :   (b('crops') <= 24.562315940856934) ?  0.05051314640699334 :   0.03074741054748245 :   (b('dg0_v_vv') <= 1.4674224853515625) ?  (b('ndvi') <= 2796.0) ?  0.04719578375892444 :   0.02987736937439912 :   (b('l8b11') <= 1405.2481079101562) ?  0.001881366725531547 :   0.016461366756082868 :   (b('l8b11_med') <= 1523.7831420898438) ?  (b('l8b4_med') <= 702.6677856445312) ?  (b('l8b11') <= 1382.2947387695312) ?  (b('l8b11') <= 1220.5606079101562) ?  (b('ndvi') <= 2989.0) ?  0.044080412395727187 :   -0.010257729334555973 :   (b('dg0_v_vv') <= -0.6367835998535156) ?  -0.00739829590950334 :   -0.03536297901684324 :   (b('l8b11') <= 1395.438232421875) ?  (b('sand') <= 34.0) ?  0.09578790551998345 :   0.030451833527593897 :   (b('l8b5') <= 3010.637451171875) ?  -0.009196205556585004 :   0.009944489406841802 :   (b('dg0_v_vv') <= 0.6350030899047852) ?  (b('l8b4') <= 787.9400329589844) ?  (b('l8b4') <= 746.5930480957031) ?  -0.048758126102889265 :   0.015140110591578592 :   (b('l8b4') <= 1215.1187744140625) ?  -0.003074623940416357 :   0.026883390833817022 :   (b('l8b5_med') <= 2669.8338623046875) ?  (b('l8b4') <= 779.8680725097656) ?  0.005335112807849679 :   -0.023277229995096165 :   (b('sand') <= 76.0) ?  0.01854413296035837 :   -0.07323919712478896 :   (b('l8b11_med') <= 1665.8943481445312) ?  (b('ndvi') <= 1811.0) ?  (b('l8b4') <= 1047.6895751953125) ?  (b('dg0_v_vv') <= 0.28246116638183594) ?  -0.003385944711263275 :   0.021577496744483397 :   (b('dg0_v_vv') <= 2.1302881240844727) ?  0.031869896817112844 :   0.0710919532601682 :   (b('l8b4') <= 749.7920227050781) ?  (b('sand') <= 53.5) ?  -0.05517899701971273 :   0.011401385068857453 :   (b('dg0_v_vv') <= 3.822639226913452) ?  -0.008162570533250894 :   0.10091107390211684 :   (b('ndvi') <= 4897.5) ?  (b('l8b11') <= 1143.2446899414062) ?  (b('l8b11_med') <= 1740.0668334960938) ?  -0.06651357613550002 :   -0.03117647603953572 :   (b('l8b4') <= 938.4302368164062) ?  0.00441990003857467 :   -0.0035863447128273345 :   (b('dg0_v_vv') <= -0.07020902633666992) ?  (b('l8b4') <= 967.8626403808594) ?  0.0023508522102694135 :   0.06374256602259454 :   0.1658845829294353 :   (b('l8b5') <= 2911.5064697265625) ?  (b('l8b11') <= 1037.5346069335938) ?  (b('l8b4_med') <= 690.9101257324219) ?  0.1177429542721557 :   0.10828537889808948 :   (b('l8b4_med') <= 1057.7998657226562) ?  (b('crops') <= 24.5) ?  (b('l8b5') <= 2827.4171142578125) ?  (b('ndvi_med') <= 2682.0) ?  (b('l8b5') <= 2179.3275146484375) ?  -0.0037201807500952602 :   -0.013733929339644683 :   (b('ndvi') <= 4486.0) ?  0.002786555177435373 :   -0.037704191236429214 :   (b('l8b11_med') <= 1526.6219482421875) ?  (b('ndvi') <= 3680.0) ?  -0.04273312667915848 :   0.015783157528820757 :   (b('l8b11') <= 2160.158203125) ?  -0.0035800097953111176 :   -0.04129011518241847 :   (b('l8b4') <= 1109.4074096679688) ?  (b('l8b11') <= 1648.1946411132812) ?  (b('l8b5_med') <= 3020.5880126953125) ?  0.01698748465554242 :   -0.014423374020828286 :   (b('ndvi') <= 3972.5) ?  -0.025317518215147505 :   -0.13610460358985768 :   (b('ndvi') <= 2610.5) ?  (b('l8b4') <= 1312.14111328125) ?  0.0320883804210234 :   -0.028950840009068585 :   (b('ndvi') <= 4305.5) ?  0.04644252049343218 :   0.005257160318059728 :   (b('l8b5_med') <= 2973.427734375) ?  (b('l8b11') <= 1760.0242919921875) ?  (b('l8b11') <= 1682.9978637695312) ?  (b('l8b11_med') <= 2082.4805908203125) ?  0.015892030750572887 :   -0.01651800547917396 :   (b('l8b5_med') <= 2679.99609375) ?  0.0238644248850279 :   0.09804954841477632 :   (b('ndvi') <= 841.0) ?  (b('ndvi_med') <= 899.0) ?  -0.0034617397867533085 :   0.04355960353859844 :   (b('sand') <= 50.5) ?  0.0019661564859284756 :   -0.0052365398730206975 :   (b('l8b5') <= 2865.8006591796875) ?  (b('l8b11') <= 1211.8416137695312) ?  (b('dg0_v_vv') <= 0.21361255645751953) ?  0.018449138497296597 :   0.04184471115583496 :   (b('l8b5') <= 2717.8572998046875) ?  -0.041371137640798174 :   -0.01844361287677937 :   (b('l8b5') <= 2895.736572265625) ?  (b('sand') <= 55.0) ?  0.06977570750967226 :   0.0023929888933363064 :   (b('l8b5_med') <= 2999.5303955078125) ?  -0.03370517048813058 :   -0.004715702511678958 :   (b('l8b4') <= 1525.0056762695312) ?  (b('l8b11_med') <= 1700.292724609375) ?  (b('dg0_v_vv') <= 0.6242713928222656) ?  (b('l8b5_med') <= 2826.612548828125) ?  (b('l8b11') <= 1225.483642578125) ?  (b('dg0_v_vv') <= -0.005399227142333984) ?  0.026013156470312233 :   -0.04158836151874773 :   (b('ndvi') <= 3275.5) ?  -0.028437819705966386 :   -0.007507863838386687 :   (b('dg0_v_vv') <= 0.500603437423706) ?  (b('l8b11') <= 1386.5162353515625) ?  -0.014880093241752644 :   0.012151063555425853 :   (b('dg0_v_vv') <= 0.5045537948608398) ?  -0.08427892330526857 :   -0.01683814993134437 :   (b('dg0_v_vv') <= 1.4774181842803955) ?  (b('l8b4') <= 968.7778015136719) ?  (b('crops') <= 32.46147155761719) ?  0.017861897467044953 :   0.06466685734372615 :   (b('ndvi_med') <= 2790.25) ?  0.12596705331747546 :   0.05753858047921371 :   (b('l8b4') <= 969.7249755859375) ?  (b('l8b5') <= 3202.8475341796875) ?  -0.0068349519118336005 :   0.04664947245348808 :   (b('ndvi') <= 3545.0) ?  -0.033862000838981 :   -0.05848828300526257 :   (b('sand') <= 67.0) ?  (b('dg0_v_vv') <= 0.9837870597839355) ?  (b('ndvi') <= 2185.5) ?  (b('l8b5') <= 2985.1328125) ?  0.008791748877097055 :   0.043346725708228874 :   (b('bulk') <= 116.0) ?  -0.04123864365757342 :   0.06680582312123017 :   (b('dg0_v_vv') <= 1.0301942825317383) ?  (b('ndvi') <= 2732.5) ?  -0.05856996299328654 :   -0.0618541352747769 :   (b('dg0_v_vv') <= 3.165626049041748) ?  0.04090024199716073 :   -0.011436671117164782 :   (b('ndvi') <= 3148.5) ?  (b('l8b5') <= 2976.7603759765625) ?  -0.06527345334488231 :   (b('l8b5_med') <= 3264.979736328125) ?  -0.04182037234137563 :   -0.024684433261742335 :   (b('l8b11') <= 2281.21875) ?  0.0031530053840446715 :   0.020612212559237897 :   (b('l8b4') <= 1742.8084716796875) ?  (b('dg0_v_vv') <= -0.05977678298950195) ?  (b('dg0_v_vv') <= -0.13081932067871094) ?  (b('ndvi') <= 2124.5) ?  (b('dg0_v_vv') <= -0.8603181838989258) ?  0.0008767015658396521 :   -0.01815268785166934 :   (b('grass') <= 56.0969123840332) ?  -0.023561180772697615 :   -0.052284867675989465 :   (b('l8b5_med') <= 2820.3416748046875) ?  -0.05656256148255054 :   -0.06387731732445119 :   (b('sand') <= 40.0) ?  (b('ndvi_med') <= 1511.0) ?  0.09738513982092162 :   (b('dg0_v_vv') <= 0.6652688980102539) ?  -0.030896627620762784 :   0.028810517768914034 :   (b('l8b5') <= 3375.4539794921875) ?  (b('l8b4') <= 1570.6210327148438) ?  0.021094223001161042 :   -0.009519137190436414 :   -0.06750586871336293 :   (b('l8b4') <= 2038.2323608398438) ?  (b('ndvi') <= 1812.0) ?  (b('l8b5') <= 3098.4981689453125) ?  (b('l8b4_med') <= 1810.3035278320312) ?  0.0399961253745451 :   0.016140412079925864 :   (b('ndvi_med') <= 1244.25) ?  0.017525156630064972 :   -0.011377165376061253 :   (b('l8b4') <= 1839.5308227539062) ?  (b('l8b5_med') <= 2827.4276123046875) ?  -0.03276295668092001 :   -0.016659546034161796 :   (b('l8b4') <= 1961.6410522460938) ?  0.018390082615754703 :   -0.023645915548249644 :   (b('ndvi_med') <= 1370.5) ?  (b('l8b5') <= 3174.934326171875) ?  (b('l8b11_med') <= 2952.5804443359375) ?  -0.0099002550501715 :   -0.028728303015204466 :   (b('dg0_v_vv') <= 0.7259654998779297) ?  0.011203802364899358 :   -0.06576679325953427 :   (b('l8b5_med') <= 2594.511474609375) ?  (b('dg0_v_vv') <= -0.31717538833618164) ?  -0.02793526390987205 :   -0.020170163100837652 :   (b('l8b5_med') <= 3287.4775390625) ?  0.013567416931331783 :   -0.006910000835492039 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 2874.5) ?  (b('moss') <= 26.56121253967285) ?  (b('l8b5') <= 2113.0489501953125) ?  (b('moss') <= 24.824777603149414) ?  (b('l8b5') <= 1293.864990234375) ?  (b('dg0_v_vv') <= -0.9129462242126465) ?  (b('l8b4_med') <= 912.6966857910156) ?  (b('bulk') <= 141.0) ?  (b('l8b5') <= 1232.108642578125) ?  -0.007107526819655652 :   -0.010544510788886921 :   (b('dg0_v_vv') <= -1.8566808700561523) ?  0.00024504364422267555 :   0.006309505386752401 :   (b('sand') <= 38.5) ?  (b('l8b11_med') <= 2013.321533203125) ?  0.026753372849038948 :   0.030480451774661077 :   0.017933046163481237 :   (b('dg0_v_vv') <= -0.5882635116577148) ?  (b('l8b5_med') <= 2417.4326171875) ?  -0.07769408699963726 :   -0.04708860711679791 :   (b('l8b11_med') <= 1472.2579956054688) ?  (b('ndvi') <= 1600.0) ?  (b('dg0_v_vv') <= 2.527348041534424) ?  -0.005516038192752712 :   0.03799188140687568 :   (b('dg0_v_vv') <= -0.2295823097229004) ?  0.010584866891973285 :   -0.021604004750095523 :   (b('l8b5_med') <= 2931.823974609375) ?  (b('l8b4_med') <= 929.2597351074219) ?  -0.02743337910058763 :   -0.055531839785202444 :   (b('dg0_v_vv') <= 2.5879693031311035) ?  -0.019279450002496378 :   -0.005113581123180999 :   (b('bulk') <= 166.0) ?  (b('grass') <= 71.3504638671875) ?  (b('l8b11') <= 1113.9520263671875) ?  (b('l8b4') <= 518.4709014892578) ?  (b('l8b4') <= 500.8668975830078) ?  -0.00014017377802034932 :   0.0911082850584794 :   (b('dg0_v_vv') <= 5.184751749038696) ?  -0.01780021782612427 :   0.03365860912761042 :   (b('l8b4') <= 632.3979187011719) ?  (b('l8b5') <= 1648.8843994140625) ?  0.009354951788212428 :   0.035075987737126585 :   (b('grass') <= 69.5579948425293) ?  0.004341662318773032 :   0.05657199893875525 :   (b('grass') <= 84.40863800048828) ?  (b('ndvi') <= 2895.0) ?  (b('ndvi') <= 2274.5) ?  -0.0077141474052652365 :   -0.03148939799141243 :   (b('ndvi') <= 3485.0) ?  0.04577577375934364 :   -0.01514236170092255 :   (b('dg0_v_vv') <= -0.31227588653564453) ?  (b('ndvi') <= 2032.5) ?  -0.00903701903071423 :   0.017855306118761013 :   (b('l8b4') <= 1123.1553955078125) ?  0.0022703347726574486 :   0.04716593938081441 :   (b('ndvi') <= 1382.0) ?  (b('l8b4_med') <= 1244.2680053710938) ?  (b('l8b11') <= 2250.85009765625) ?  -0.0010363800524597067 :   (b('ndvi') <= 1241.0) ?  0.0839351930970608 :   0.01713983223227354 :   (b('l8b11') <= 2208.1256103515625) ?  (b('l8b5') <= 1829.4284057617188) ?  -0.03338250375230595 :   -0.05600805813707095 :   (b('dg0_v_vv') <= -1.3191113471984863) ?  0.003146238038824953 :   0.007132631265474078 :   (b('l8b4') <= 1241.9971313476562) ?  (b('ndvi_med') <= 1696.25) ?  (b('ndvi') <= 1567.5) ?  0.10765097371186094 :   0.09326674542910858 :   (b('dg0_v_vv') <= 1.7069096565246582) ?  0.06157635918484852 :   0.053667658622228 :   0.011264603265329437 :   (b('l8b4') <= 1116.906494140625) ?  (b('l8b5_med') <= 2012.2393188476562) ?  (b('dg0_v_vv') <= 1.7435479164123535) ?  (b('l8b5') <= 1820.4114990234375) ?  (b('l8b11_med') <= 2032.5352783203125) ?  0.11862619515768363 :   0.11617939088459023 :   (b('bulk') <= 137.0) ?  0.12342557506695312 :   0.12189120609987478 :   0.034274560929735964 :   (b('l8b5_med') <= 2250.0211181640625) ?  (b('dg0_v_vv') <= -0.27553892135620117) ?  (b('sand') <= 41.5) ?  0.08085382297124075 :   0.055378998265768226 :   (b('dg0_v_vv') <= 3.4584407806396484) ?  (b('ndvi') <= 1465.5) ?  0.05615149852889806 :   0.023300571177495538 :   0.01809180963101331 :   0.10431018963592725 :   (b('ndvi') <= 935.5) ?  (b('dg0_v_vv') <= -0.16534137725830078) ?  (b('l8b4_med') <= 1304.7144165039062) ?  0.1285626963768393 :   0.10926159482645532 :   (b('ndvi') <= 898.5) ?  0.01846864975816892 :   0.04352204329630334 :   (b('ndvi') <= 1311.0) ?  (b('dg0_v_vv') <= -2.2522311210632324) ?  0.041814015963539675 :   (b('ndvi_med') <= 1200.0) ?  (b('l8b5_med') <= 2265.63671875) ?  -0.0006220339970097836 :   -0.01667451950135443 :   (b('dg0_v_vv') <= 0.2116870880126953) ?  0.011638665524709463 :   -0.0024282731493481813 :   (b('l8b5') <= 1907.9506225585938) ?  (b('l8b4') <= 1172.2473754882812) ?  0.018276062473574828 :   -0.005636920916697735 :   (b('l8b4') <= 1281.845703125) ?  (b('ndvi_med') <= 1242.75) ?  -0.0336296804450496 :   -0.03708790522104545 :   (b('ndvi_med') <= 1350.0) ?  -0.025301466025543538 :   -0.010957479992971444 :   (b('l8b11') <= 593.5153198242188) ?  (b('l8b11_med') <= 1675.3267517089844) ?  0.026775741247631935 :   (b('dg0_v_vv') <= 1.731947422027588) ?  (b('l8b11') <= 588.6050109863281) ?  0.07830826189685075 :   0.11762817393798439 :   0.03762817572612373 :   (b('dg0_v_vv') <= -8.827847003936768) ?  (b('ndvi') <= 1400.5) ?  0.07664384682542544 :   (b('l8b11') <= 2190.694091796875) ?  (b('l8b5') <= 2442.6444091796875) ?  0.04712987572595799 :   0.0526400330129817 :   0.06912688932102212 :   (b('crops') <= 98.84536743164062) ?  (b('l8b4_med') <= 847.5724182128906) ?  (b('ndvi') <= 1795.5) ?  (b('ndvi_med') <= 2747.0) ?  (b('sand') <= 35.5) ?  0.02500694743709142 :   -0.0035621840917973518 :   0.118354972526869 :   (b('l8b4') <= 936.1029052734375) ?  (b('sand') <= 49.5) ?  -0.003166485251486185 :   -0.020218476418181745 :   (b('l8b5_med') <= 3114.77099609375) ?  -0.01785279209256408 :   0.05067384932616132 :   (b('ndvi') <= 2100.5) ?  (b('l8b4') <= 823.6883544921875) ?  (b('ndvi') <= 1163.5) ?  -0.09013179721690753 :   0.025313524720956628 :   (b('l8b11_med') <= 2154.5760498046875) ?  -0.006901740758834607 :   -0.0012220885563408975 :   (b('ndvi') <= 2116.5) ?  (b('l8b5_med') <= 3193.4766845703125) ?  0.03560504464163936 :   -0.006623466593893626 :   (b('grass') <= 16.676905632019043) ?  -0.0014920667994682984 :   0.004039952946436518 :   (b('ndvi_med') <= 2597.0) ?  (b('l8b5_med') <= 2776.2713623046875) ?  (b('l8b5') <= 2805.314208984375) ?  (b('dg0_v_vv') <= -0.16588354110717773) ?  0.0011510368947151648 :   0.02856941348962585 :   (b('ndvi') <= 4099.0) ?  -0.0006544222654599694 :   0.0275906978846269 :   -0.06097303108102253 :   (b('l8b5') <= 2998.2308349609375) ?  0.07258937760368678 :   (b('dg0_v_vv') <= -2.7762346267700195) ?  0.054676947324411196 :   (b('ndvi') <= 4176.5) ?  0.03289506515418417 :   0.03956657949910397 :   (b('l8b5_med') <= 2150.6151123046875) ?  (b('sand') <= 49.5) ?  (b('l8b5_med') <= 1858.7869262695312) ?  (b('l8b11') <= 1967.4487915039062) ?  (b('l8b5') <= 2171.25927734375) ?  (b('dg0_v_vv') <= -0.47408533096313477) ?  (b('ndvi') <= 1523.5) ?  (b('l8b11_med') <= 1940.4482421875) ?  0.007106694396556442 :   -0.0122173367168125 :   (b('l8b4') <= 961.4039001464844) ?  0.0566173950656341 :   0.00506649254953042 :   (b('ndvi_med') <= 1779.75) ?  (b('ndvi') <= 2198.0) ?  -0.009898456340772046 :   0.028792155677111214 :   (b('ndvi') <= 2688.0) ?  -0.03190492478610851 :   -0.08507327534280767 :   (b('l8b11') <= 1689.5603637695312) ?  (b('l8b4') <= 715.3875885009766) ?  0.06324166173022566 :   0.04639290176751819 :   (b('ndvi_med') <= 1779.75) ?  0.0028819723330035796 :   0.03029308386282574 :   (b('l8b4') <= 1144.7703857421875) ?  (b('l8b11') <= 1980.8195190429688) ?  (b('ndvi') <= 1834.0) ?  0.10145190729560455 :   0.09166032978963674 :   (b('l8b11') <= 2130.043212890625) ?  (b('ndvi') <= 1726.5) ?  0.0017213925307317836 :   0.027398727614672 :   (b('ndvi_med') <= 1594.75) ?  0.04745086410876503 :   0.09150067268386726 :   (b('l8b11_med') <= 1967.0358276367188) ?  (b('l8b4') <= 1198.7321166992188) ?  (b('l8b5') <= 1969.8021240234375) ?  -0.02163633410776397 :   -0.030022543786509107 :   (b('ndvi') <= 1470.5) ?  0.0239426853678244 :   0.005394940043980306 :   0.05018233984501759 :   (b('dg0_v_vv') <= 0.3055448532104492) ?  (b('l8b5') <= 2172.83935546875) ?  (b('l8b5') <= 1659.1256103515625) ?  (b('dg0_v_vv') <= -0.8789587020874023) ?  (b('ndvi') <= 1504.5) ?  0.0542215088331328 :   0.06221070793466736 :   (b('ndvi') <= 1123.0) ?  -0.03831478284558448 :   0.007131046075232609 :   (b('l8b5') <= 1945.7805786132812) ?  (b('l8b4') <= 1325.0205688476562) ?  -0.014380010616510215 :   -0.037364709155265595 :   (b('l8b11_med') <= 2083.7174072265625) ?  -0.007671445192263925 :   0.024291497537859145 :   (b('l8b5') <= 2196.7762451171875) ?  (b('ndvi') <= 1770.5) ?  0.07491115790140525 :   0.07200632653165222 :   (b('moss') <= 38.340314865112305) ?  (b('l8b11_med') <= 2047.3141479492188) ?  0.0064590428571735 :   -0.01682192062798091 :   0.06855486801644688 :   (b('l8b11_med') <= 1831.3523559570312) ?  (b('ndvi') <= 1469.5) ?  (b('l8b11') <= 1898.3601684570312) ?  -0.07086031258010536 :   -0.030249180752990143 :   (b('l8b5') <= 2203.6944580078125) ?  (b('dg0_v_vv') <= 0.3598175048828125) ?  -0.02666953726957846 :   -0.013421672367612639 :   (b('dg0_v_vv') <= 1.4055557250976562) ?  -0.025601321954907785 :   -0.0483095891016222 :   (b('l8b5') <= 1825.6248779296875) ?  (b('l8b11') <= 1958.34326171875) ?  (b('l8b4') <= 904.7775573730469) ?  0.0016526496407149687 :   -0.023309000097904188 :   (b('dg0_v_vv') <= 1.9209461212158203) ?  0.04061017130225946 :   0.006548415848443012 :   (b('l8b4') <= 1207.4603271484375) ?  (b('l8b11') <= 2259.3037109375) ?  -0.02197173701166811 :   0.010866589280096733 :   (b('l8b4') <= 1250.8569946289062) ?  0.02743272684752172 :   -0.012688497053807155 :   (b('sand') <= 58.5) ?  (b('dg0_v_vv') <= 1.872302532196045) ?  (b('l8b11_med') <= 1853.426513671875) ?  0.07328786718527532 :   (b('l8b5') <= 2208.3104248046875) ?  (b('l8b4') <= 1682.6575317382812) ?  (b('ndvi') <= 700.5) ?  -0.11105467173518282 :   -0.020390932977504092 :   0.07332041494485034 :   (b('dg0_v_vv') <= 1.1675257682800293) ?  (b('bulk') <= 145.0) ?  -0.019005824916109804 :   -0.04604117961066852 :   -0.10386706392738951 :   (b('ndvi') <= 885.0) ?  -0.07676674521393163 :   (b('l8b5_med') <= 1728.5043334960938) ?  (b('ndvi') <= 2367.5) ?  (b('ndvi') <= 1429.5) ?  0.025586250995017823 :   0.03217804284865357 :   0.04722577814711004 :   (b('l8b11') <= 2568.1998291015625) ?  (b('l8b4_med') <= 1533.610107421875) ?  -0.013346652396452476 :   0.051295575746979266 :   (b('l8b5_med') <= 1942.32666015625) ?  0.018906387990582543 :   0.0522546288039314 :   (b('l8b5') <= 1867.2408447265625) ?  (b('l8b5') <= 1566.29345703125) ?  0.0413258081976694 :   (b('l8b5_med') <= 1718.1389770507812) ?  0.006366311498696736 :   (b('dg0_v_vv') <= -0.8419671058654785) ?  -0.013285903945352895 :   (b('ndvi') <= 730.0) ?  -0.10028590144195781 :   -0.051254200962773756 :   (b('l8b11') <= 2132.7867431640625) ?  (b('l8b5_med') <= 1869.8427124023438) ?  (b('ndvi_med') <= 859.0) ?  0.018892334606285213 :   0.004023797938188606 :   (b('l8b11') <= 1911.9148559570312) ?  0.03386803164849639 :   (b('l8b11') <= 2106.9853515625) ?  0.07768433592581804 :   0.038018044690365085 :   (b('l8b5_med') <= 2094.7025146484375) ?  (b('l8b5') <= 2169.654541015625) ?  (b('l8b11') <= 2290.8250732421875) ?  -0.07746526938155561 :   -0.06616483004369196 :   (b('l8b5') <= 2206.8167724609375) ?  0.02304962098630073 :   0.015672860458440546 :   (b('l8b11_med') <= 2267.890625) ?  0.010155698084919368 :   (b('dg0_v_vv') <= -0.13558197021484375) ?  0.028512394733466034 :   0.041084031719606354 :   (b('l8b4') <= 1697.5230712890625) ?  (b('l8b4') <= 1688.6699829101562) ?  (b('l8b5') <= 1987.5135498046875) ?  (b('ndvi_med') <= 1351.5) ?  (b('sand') <= 62.0) ?  (b('l8b5_med') <= 2161.386962890625) ?  0.0609257998686856 :   (b('dg0_v_vv') <= 0.08643913269042969) ?  0.010910916017998646 :   -0.004649050709776982 :   (b('l8b4') <= 1307.2825317382812) ?  (b('ndvi_med') <= 1180.75) ?  -0.032563199815219934 :   -0.031090049603242714 :   -0.018656208122702647 :   (b('ndvi') <= 1265.5) ?  (b('grass') <= 54.54311943054199) ?  0.12046372942684924 :   0.0861078482153414 :   (b('l8b5_med') <= 2766.5579833984375) ?  0.054463175806899446 :   0.011433416890678005 :   (b('ndvi') <= 799.0) ?  (b('dg0_v_vv') <= -0.2897987365722656) ?  (b('l8b4_med') <= 1654.2467041015625) ?  -0.06852315049710923 :   -0.11312979783293739 :   (b('grass') <= 25.8555965423584) ?  (b('dg0_v_vv') <= 0.3004570007324219) ?  -0.03159796845030663 :   -0.04254100095291956 :   0.02036328160037723 :   (b('l8b5_med') <= 2370.42431640625) ?  (b('moss') <= 38.41355323791504) ?  (b('dg0_v_vv') <= 1.177706241607666) ?  -0.0014991598911792993 :   0.05213382757314791 :   (b('ndvi_med') <= 1175.0) ?  0.04107447720823912 :   0.1476069668743114 :   (b('l8b5') <= 2414.1871337890625) ?  (b('l8b4') <= 1630.6629028320312) ?  -0.008456864226506702 :   -0.03530267921040493 :   (b('l8b5') <= 2452.50439453125) ?  0.022313195897443218 :   0.0001890048543025062 :   (b('l8b4') <= 1689.5541381835938) ?  (b('dg0_v_vv') <= -0.48041820526123047) ?  0.15594186048311395 :   0.1373377349383408 :   (b('l8b4_med') <= 1689.2672119140625) ?  (b('l8b5_med') <= 2456.864013671875) ?  (b('l8b4') <= 1692.6896362304688) ?  0.0016481505896455556 :   (b('ndvi') <= 994.5) ?  0.023610378206406213 :   0.014850929555959791 :   -0.02052944976440549 :   0.05135564726389341 :   (b('bulk') <= 147.5) ?  (b('l8b11_med') <= 2451.6282958984375) ?  (b('dg0_v_vv') <= 0.7226438522338867) ?  (b('moss') <= 37.44301986694336) ?  (b('dg0_v_vv') <= 0.503014087677002) ?  (b('l8b5_med') <= 2380.576416015625) ?  -0.022698964628554252 :   -0.01685862912651887 :   -0.03353606305385075 :   (b('l8b5_med') <= 2258.5601806640625) ?  -0.025224727356551374 :   (b('l8b4') <= 1819.47265625) ?  -0.026018921444270024 :   -0.05374243786329888 :   (b('l8b5') <= 2426.386962890625) ?  -0.028120075224032343 :   (b('moss') <= 34.06479740142822) ?  -0.06687121367271998 :   (b('l8b4_med') <= 1654.2467041015625) ?  -0.05859020283873374 :   -0.054473096304721316 :   (b('l8b5') <= 2417.8707275390625) ?  (b('ndvi_med') <= 931.5) ?  0.03197339651672919 :   0.05048334167529829 :   (b('l8b5') <= 2723.6512451171875) ?  (b('l8b4') <= 1932.1080932617188) ?  (b('l8b4') <= 1840.7466430664062) ?  -0.01905351877464323 :   -0.03199433200361106 :   (b('dg0_v_vv') <= 0.19865989685058594) ?  -0.0039750462681041945 :   -0.01651953876629119 :   (b('ndvi') <= 1463.0) ?  (b('ndvi') <= 1399.0) ?  -0.0030080843902656573 :   0.010650841356695173 :   (b('l8b4') <= 1786.478515625) ?  -0.009441788674502766 :   -0.023335040752800226 :   (b('moss') <= 35.750967025756836) ?  (b('dg0_v_vv') <= 0.2765626907348633) ?  (b('dg0_v_vv') <= 0.26123476028442383) ?  (b('dg0_v_vv') <= -0.27594709396362305) ?  (b('l8b5_med') <= 3197.8648681640625) ?  -0.01295382129566948 :   0.02104556842541565 :   (b('dg0_v_vv') <= -0.16164207458496094) ?  0.02524014364446766 :   0.005293555891286134 :   0.09276123742944457 :   (b('l8b4') <= 1766.3027954101562) ?  (b('l8b4') <= 1735.7135620117188) ?  -0.0032449025590521735 :   (b('l8b4') <= 1752.4376831054688) ?  -0.040766255647748245 :   -0.06480959761641686 :   (b('l8b11_med') <= 2896.7408447265625) ?  0.03816360481230875 :   (b('dg0_v_vv') <= 0.6896791458129883) ?  -0.0038146324026362916 :   -0.03486008424575123 :   (b('ndvi') <= 936.5) ?  (b('dg0_v_vv') <= -0.706326961517334) ?  0.11593666209456788 :   (b('dg0_v_vv') <= 1.222888469696045) ?  (b('bulk') <= 150.0) ?  -0.005681655215781928 :   -0.019898021353977866 :   0.09875879733572715 :   (b('l8b11') <= 2317.0587158203125) ?  (b('ndvi') <= 1343.5) ?  (b('ndvi_med') <= 1243.0) ?  -0.030247857295073635 :   -0.028960423836127736 :   (b('l8b11_med') <= 2255.6011962890625) ?  -0.03704304993683647 :   -0.041737724521573545 :   (b('l8b11') <= 2526.7535400390625) ?  (b('l8b4') <= 1772.7954711914062) ?  -0.00527687433237947 :   0.021517382184216338 :   (b('ndvi') <= 1215.5) ?  -0.019792042690148663 :   -0.008358726253416453 :   (b('grass') <= 34.08266639709473) ?  (b('l8b5_med') <= 2781.259521484375) ?  (b('ndvi') <= 1170.5) ?  (b('l8b4') <= 783.3821716308594) ?  (b('grass') <= 17.203285694122314) ?  -0.09879505763799706 :   -0.07939133162866316 :   (b('l8b5_med') <= 2692.0948486328125) ?  (b('dg0_v_vv') <= 0.30658388137817383) ?  -0.04796153102380376 :   -0.03247169116425332 :   -0.020053478859806857 :   (b('sand') <= 53.5) ?  (b('dg0_v_vv') <= 1.8279411792755127) ?  (b('l8b5') <= 3285.4974365234375) ?  (b('l8b5') <= 2239.4754638671875) ?  (b('l8b5') <= 2236.2728271484375) ?  (b('sand') <= 45.5) ?  0.01341261987419711 :   0.051771109305538965 :   0.09177908178324867 :   (b('ndvi') <= 2808.0) ?  (b('ndvi') <= 2576.5) ?  -0.002775313155686816 :   -0.023638541977594393 :   (b('l8b5') <= 3019.6776123046875) ?  0.013332195801655502 :   -0.00622898174258403 :   (b('l8b4') <= 638.595458984375) ?  (b('l8b4') <= 433.93809509277344) ?  (b('crops') <= 42.29880332946777) ?  0.08885305619433903 :   0.054886398409618786 :   (b('ndvi') <= 4794.5) ?  -0.012769150904747485 :   0.012762391600941852 :   (b('l8b11') <= 1316.29345703125) ?  (b('l8b4') <= 681.2731018066406) ?  0.03910411787371569 :   0.07287594285739832 :   (b('l8b5') <= 3331.383056640625) ?  -0.012588039070118295 :   0.032977088592017846 :   (b('crops') <= 30.905277252197266) ?  (b('ndvi') <= 4331.5) ?  (b('l8b11') <= 1257.820068359375) ?  (b('l8b4_med') <= 701.6364440917969) ?  -0.0322941141829855 :   -0.0332509017977444 :   -0.023740917303703035 :   0.036285366072467046 :   (b('l8b11') <= 969.2294921875) ?  (b('l8b5') <= 3335.3900146484375) ?  0.020029749616210013 :   -0.06122294714892698 :   (b('l8b11_med') <= 1316.21044921875) ?  -0.007588349879132206 :   (b('l8b11') <= 1458.5729370117188) ?  0.0626166203976124 :   0.024238772881162062 :   (b('l8b5') <= 2580.7315673828125) ?  (b('crops') <= 21.921676635742188) ?  (b('l8b11') <= 1447.58642578125) ?  -0.06764862869245039 :   (b('l8b11') <= 1507.8548583984375) ?  (b('ndvi_med') <= 3840.0) ?  -0.041048855945620424 :   -0.032180070061539084 :   -0.01809160304968685 :   (b('l8b11') <= 1729.1002197265625) ?  (b('sand') <= 78.0) ?  (b('l8b4_med') <= 783.5617370605469) ?  -0.0003396287049505848 :   0.01452684316242676 :   (b('dg0_v_vv') <= 3.4664394855499268) ?  0.02541340811683115 :   0.0016742179817326082 :   (b('l8b5_med') <= 2480.3192138671875) ?  (b('l8b11_med') <= 1095.0886840820312) ?  0.026653560185144934 :   0.0021866996355669193 :   (b('l8b4_med') <= 788.7772827148438) ?  -0.024209872758997487 :   -0.000989747377780963 :   (b('l8b4') <= 477.0339813232422) ?  (b('l8b4_med') <= 628.4268798828125) ?  (b('l8b5_med') <= 2410.511962890625) ?  (b('l8b4') <= 374.1634979248047) ?  -0.022793711099271308 :   0.009401060754081175 :   (b('ndvi') <= 4307.0) ?  0.0020411167149096826 :   -0.021823261357023706 :   (b('l8b5_med') <= 2438.8564453125) ?  (b('dg0_v_vv') <= 0.0061473846435546875) ?  0.007785663185907898 :   -0.03306039982950091 :   (b('dg0_v_vv') <= 5.534078121185303) ?  0.024436860344271324 :   -0.027774597470643772 :   (b('dg0_v_vv') <= -1.1743416786193848) ?  (b('l8b5') <= 2693.8267822265625) ?  (b('ndvi') <= 4212.0) ?  -0.0015196183114882314 :   0.005512593190876533 :   (b('sand') <= 73.0) ?  -0.02843853514953697 :   -0.007383537810145313 :   (b('l8b11') <= 1023.1340942382812) ?  (b('ndvi_med') <= 4106.0) ?  -0.04090084030160288 :   -0.07133841998542673 :   (b('ndvi') <= 3261.5) ?  -0.06583385513897014 :   -0.023426089051920847 :   (b('bulk') <= 130.5) ?  (b('l8b5') <= 5195.12646484375) ?  (b('l8b4') <= 573.252197265625) ?  (b('grass') <= 22.948418617248535) ?  (b('dg0_v_vv') <= 1.318096399307251) ?  (b('l8b5_med') <= 3190.2471923828125) ?  (b('ndvi') <= 4738.5) ?  0.027543865250903306 :   0.010086966086872684 :   (b('dg0_v_vv') <= 0.624910831451416) ?  -0.000728543424767512 :   0.023674101341530543 :   (b('grass') <= 14.875670433044434) ?  (b('ndvi_med') <= 3936.5) ?  -0.013250833022613038 :   0.02733995397407163 :   (b('crops') <= 64.66257858276367) ?  -0.08212347192614713 :   -0.04083663737683976 :   (b('dg0_v_vv') <= 0.03801298141479492) ?  (b('ndvi') <= 4508.0) ?  (b('ndvi') <= 4427.5) ?  -0.01719003121385926 :   0.046503952957382994 :   (b('l8b5') <= 4026.4661865234375) ?  -0.054605004006688135 :   -0.013516200796375017 :   (b('dg0_v_vv') <= 0.14100933074951172) ?  (b('grass') <= 31.722938537597656) ?  0.04085599886627711 :   0.0780130781254379 :   (b('dg0_v_vv') <= 1.8229494094848633) ?  -0.009586811709877937 :   0.024997444864905825 :   (b('l8b4') <= 581.8189086914062) ?  (b('ndvi_med') <= 4314.0) ?  (b('l8b11_med') <= 1286.4586791992188) ?  (b('dg0_v_vv') <= 0.32613468170166016) ?  0.04942905990779822 :   0.04343966509085945 :   (b('dg0_v_vv') <= 1.5057373046875) ?  0.0258700223301927 :   0.03810735101196627 :   0.10734571294626999 :   (b('ndvi') <= 4752.5) ?  (b('ndvi') <= 4698.0) ?  (b('ndvi_med') <= 4519.0) ?  0.005192909033923413 :   0.028241648686332683 :   (b('l8b4') <= 1119.5756225585938) ?  0.059391151275709354 :   0.006139961968912991 :   (b('sand') <= 39.0) ?  (b('l8b5_med') <= 3586.752197265625) ?  0.014796742470520374 :   0.06098945210230006 :   (b('sand') <= 50.0) ?  -0.050779959627886694 :   -0.009001263060308018 :   (b('dg0_v_vv') <= -0.1966547966003418) ?  (b('ndvi_med') <= 4246.75) ?  0.04631742349701218 :   0.055339632555471474 :   (b('dg0_v_vv') <= -0.014806747436523438) ?  -0.00902081649265929 :   (b('dg0_v_vv') <= 1.2693357467651367) ?  (b('dg0_v_vv') <= 0.6238007545471191) ?  (b('dg0_v_vv') <= 0.15348339080810547) ?  0.037961098073885896 :   -0.0042755451583511295 :   (b('dg0_v_vv') <= 1.0982179641723633) ?  0.045024596848535936 :   0.058887802717778215 :   (b('dg0_v_vv') <= 1.6694350242614746) ?  -0.027178739410078312 :   (b('l8b5_med') <= 3311.798828125) ?  0.019524476578761336 :   0.05603491533812416 :   (b('sand') <= 14.0) ?  (b('l8b11') <= 1370.0) ?  (b('l8b5') <= 3036.421630859375) ?  (b('ndvi') <= 3888.5) ?  (b('dg0_v_vv') <= 0.6121711730957031) ?  -0.0518785128238452 :   -0.03141160957570727 :   -0.010110407003705357 :   (b('dg0_v_vv') <= -0.27115774154663086) ?  0.003947576915455775 :   (b('l8b4') <= 555.6047668457031) ?  (b('l8b11') <= 957.779541015625) ?  0.07279625461812195 :   0.07820962762465117 :   0.022109909452947496 :   (b('ndvi') <= 2387.0) ?  0.0038659421592345256 :   (b('dg0_v_vv') <= 0.33677196502685547) ?  (b('l8b5') <= 2438.2626953125) ?  (b('ndvi') <= 2463.5) ?  0.0720813115205372 :   0.07334883520923402 :   (b('dg0_v_vv') <= -0.06443119049072266) ?  0.03641082263987383 :   0.06631563015290592 :   (b('l8b11') <= 1600.0449829101562) ?  -0.00905001010139056 :   (b('ndvi') <= 3037.0) ?  0.050231827948016006 :   0.02934140146713715 :   (b('l8b5') <= 3613.493408203125) ?  (b('ndvi') <= 4906.0) ?  (b('l8b5') <= 3471.2225341796875) ?  (b('l8b5') <= 3469.1588134765625) ?  (b('l8b4') <= 947.1725158691406) ?  -0.004772855272149949 :   0.0013415261901446958 :   (b('ndvi') <= 4155.0) ?  0.10258374121520802 :   0.0823451762340982 :   (b('dg0_v_vv') <= 0.5382113456726074) ?  (b('ndvi') <= 4282.5) ?  -0.0020769194255431086 :   -0.023697559683393037 :   (b('l8b11_med') <= 1427.5918579101562) ?  -0.04697811346768644 :   -0.014107949670840109 :   (b('ndvi') <= 4914.5) ?  (b('l8b4_med') <= 581.6770629882812) ?  (b('dg0_v_vv') <= 1.1154894828796387) ?  0.00813850947921485 :   0.05995465638603261 :   (b('dg0_v_vv') <= -0.16209006309509277) ?  0.08837440163513444 :   0.07122729559681001 :   (b('ndvi') <= 4959.5) ?  (b('dg0_v_vv') <= 2.9173855781555176) ?  -0.012295941528162187 :   0.039444161326325944 :   (b('l8b11') <= 1339.072265625) ?  0.025083626366155024 :   -0.008528593194628462 :   (b('l8b11') <= 1432.9409790039062) ?  (b('sand') <= 78.0) ?  (b('ndvi') <= 2572.5) ?  (b('ndvi_med') <= 3351.25) ?  -0.010893305497591063 :   -0.06255032778959743 :   (b('ndvi') <= 2828.0) ?  0.03067231346547216 :   2.717274599362564e-05 :   (b('dg0_v_vv') <= 0.750333309173584) ?  -0.08649272434219413 :   (b('l8b4_med') <= 685.351806640625) ?  -0.022537225498512975 :   -0.058765509044082065 :   (b('l8b4') <= 812.9949340820312) ?  (b('l8b5_med') <= 2952.5721435546875) ?  (b('bulk') <= 134.0) ?  0.029194931051536935 :   0.01439971573274293 :   (b('ndvi') <= 4462.0) ?  0.05448841007029728 :   0.0733032280567153 :   (b('bulk') <= 139.0) ?  (b('dg0_v_vv') <= -3.1797075271606445) ?  -0.012603741057616663 :   0.027725731836430734 :   (b('ndvi') <= 4078.5) ?  -0.014299627812551122 :   0.014694220287726415 :   (b('ndvi') <= 2518.5) ?  (b('l8b11_med') <= 1700.4581298828125) ?  (b('l8b5') <= 1644.5070190429688) ?  (b('l8b11') <= 1152.3302001953125) ?  0.05578303427201858 :   0.07972834301278126 :   (b('dg0_v_vv') <= -2.2881522178649902) ?  (b('moss') <= 13.38936996459961) ?  (b('l8b11') <= 2191.5396728515625) ?  (b('l8b11') <= 1942.0836181640625) ?  (b('grass') <= 43.301164627075195) ?  -0.01088036815720686 :   -0.007691534033060485 :   0.0004933808002825502 :   0.02164379255747384 :   (b('ndvi') <= 2113.5) ?  (b('l8b5') <= 2589.8927001953125) ?  -0.04795836985429855 :   -0.03986627459675397 :   -0.027954999752581672 :   (b('crops') <= 54.189083099365234) ?  (b('crops') <= 4.036458373069763) ?  (b('ndvi') <= 2455.0) ?  (b('l8b4') <= 1106.0054321289062) ?  -0.004091970944707251 :   0.02316786043542909 :   (b('l8b5') <= 2043.419921875) ?  -0.00780567897255216 :   0.05155033211443301 :   (b('grass') <= 36.288320541381836) ?  (b('l8b4') <= 1100.7798461914062) ?  0.0696054116449556 :   0.02171503771518134 :   (b('ndvi_med') <= 3077.0) ?  0.03932094771589493 :   0.013610587798250775 :   -0.06967786848545746 :   (b('sand') <= 48.0) ?  (b('l8b4') <= 1010.1636352539062) ?  (b('dg0_v_vv') <= 1.3844408988952637) ?  (b('l8b5') <= 1843.79541015625) ?  (b('l8b11') <= 2231.4219970703125) ?  (b('dg0_v_vv') <= 0.06778764724731445) ?  -0.0062591587653623715 :   0.006543686484089317 :   0.05837202055241536 :   (b('ndvi') <= 2057.0) ?  (b('bulk') <= 143.0) ?  0.0606858516148048 :   0.0757750658270624 :   (b('ndvi') <= 2217.5) ?  0.020223819567521728 :   0.04634705917917004 :   -0.037632688375074136 :   (b('l8b5_med') <= 2551.1197509765625) ?  (b('ndvi') <= 2359.0) ?  (b('l8b11_med') <= 2119.6873779296875) ?  (b('l8b5') <= 2019.067138671875) ?  0.10475500062356119 :   0.11526510850450572 :   (b('ndvi_med') <= 2918.25) ?  0.09302947175428883 :   0.0994105063641868 :   (b('dg0_v_vv') <= -0.2963690757751465) ?  (b('moss') <= 18.46788215637207) ?  0.08302099162690772 :   0.09477452532674657 :   0.06965532545773295 :   (b('dg0_v_vv') <= -0.6780920028686523) ?  (b('l8b11_med') <= 1925.52392578125) ?  0.046934871035625936 :   0.04905106214514908 :   (b('crops') <= 11.5) ?  0.08121699381372866 :   0.08006462435244888 :   (b('ndvi_med') <= 3473.25) ?  (b('dg0_v_vv') <= -0.2732996940612793) ?  -0.0051330418742455974 :   (b('crops') <= 15.626556396484375) ?  -0.03651274515911722 :   -0.03386331850788904 :   (b('l8b4') <= 658.1835632324219) ?  (b('dg0_v_vv') <= 0.7745790481567383) ?  0.02064632403436434 :   0.022416587357809686 :   (b('l8b11') <= 2045.3519287109375) ?  0.018012652233698717 :   0.01678627245818351 :   (b('l8b5_med') <= 2862.452880859375) ?  (b('l8b11_med') <= 1194.8403930664062) ?  (b('l8b4') <= 446.4490661621094) ?  (b('l8b4') <= 441.89234924316406) ?  (b('sand') <= 29.5) ?  0.09177550396344386 :   (b('l8b4') <= 439.67884826660156) ?  (b('l8b5') <= 2793.7562255859375) ?  0.014107956357775921 :   -0.03341074360955182 :   (b('dg0_v_vv') <= 1.3680009841918945) ?  -0.04535776049713225 :   -0.027273137924129887 :   (b('l8b5_med') <= 2724.859619140625) ?  (b('ndvi_med') <= 3413.0) ?  0.04289710202382557 :   0.05448951462203133 :   (b('l8b4') <= 444.6117401123047) ?  0.06798914908254022 :   0.06725320571479948 :   (b('l8b5') <= 1858.924560546875) ?  (b('dg0_v_vv') <= -0.8048644065856934) ?  (b('moss') <= 10.975822823122144) ?  0.09372661864066623 :   -0.017131904635815542 :   (b('ndvi') <= 2835.5) ?  (b('ndvi') <= 2729.5) ?  -0.032394275015534386 :   -0.0008617961257499455 :   (b('ndvi_med') <= 3709.25) ?  -0.029194574089895668 :   -0.015017894307078783 :   (b('l8b5') <= 3006.378662109375) ?  (b('l8b5') <= 2841.602783203125) ?  (b('l8b11') <= 1052.3379516601562) ?  -0.005357336853763179 :   0.0073978843780420234 :   (b('l8b5') <= 2867.7725830078125) ?  -0.025480943398980375 :   -0.05083897588691582 :   (b('l8b4') <= 720.2564086914062) ?  (b('dg0_v_vv') <= -1.0660014152526855) ?  0.06758887097706442 :   0.020375482390254853 :   (b('dg0_v_vv') <= -0.005399227142333984) ?  0.0415389706862678 :   -0.04263800795610576 :   (b('l8b4') <= 758.3002624511719) ?  (b('grass') <= 77.05030059814453) ?  (b('crops') <= 18.136863231658936) ?  (b('ndvi') <= 2767.5) ?  (b('l8b4_med') <= 740.747314453125) ?  0.006892309883012475 :   0.0672164565117378 :   (b('ndvi') <= 4606.5) ?  -0.014840130565851536 :   0.006556223866192009 :   (b('ndvi') <= 2807.5) ?  (b('l8b4') <= 654.0450439453125) ?  -0.03301823617983671 :   -0.0423222594864793 :   (b('l8b11') <= 1657.3450317382812) ?  0.010340606575935477 :   0.09015702296009956 :   (b('dg0_v_vv') <= 3.260969638824463) ?  (b('l8b11') <= 1254.9271240234375) ?  (b('l8b11') <= 1040.369873046875) ?  -0.084498636046037 :   -0.05456647547539382 :   (b('l8b4') <= 678.7538452148438) ?  0.003337677305699996 :   -0.03712924927141005 :   (b('l8b11') <= 1386.039306640625) ?  -0.10171150821494498 :   -0.10038577871970422 :   (b('l8b4') <= 965.5582275390625) ?  (b('dg0_v_vv') <= 0.8700413703918457) ?  (b('ndvi') <= 4021.5) ?  (b('l8b11') <= 2054.1810302734375) ?  -0.0015544440954962596 :   0.03704600617349004 :   (b('ndvi') <= 4166.5) ?  0.1167047490217447 :   0.008998979417297298 :   (b('l8b5') <= 2471.3963623046875) ?  (b('bulk') <= 149.5) ?  0.05981287498132196 :   0.11621595278430047 :   (b('l8b11') <= 1195.4454956054688) ?  0.1284383390377074 :   0.017205017867001803 :   (b('l8b11') <= 2305.4844970703125) ?  (b('l8b5') <= 2324.8212890625) ?  (b('dg0_v_vv') <= -0.5361757278442383) ?  0.0420173282433607 :   -0.005203742848411655 :   (b('l8b11') <= 1859.70166015625) ?  -0.01212996521912841 :   -0.0367999712279412 :   (b('l8b4') <= 1022.4861145019531) ?  (b('dg0_v_vv') <= -0.5208239555358887) ?  0.06952503565745716 :   0.084856994868796 :   (b('l8b5') <= 3058.3382568359375) ?  0.018536112374659925 :   -0.02159768808904758 :   (b('dg0_v_vv') <= 3.708988666534424) ?  (b('l8b5_med') <= 3640.2857666015625) ?  (b('l8b4') <= 1119.0413208007812) ?  (b('l8b11') <= 1593.4188232421875) ?  (b('l8b4') <= 891.047119140625) ?  (b('dg0_v_vv') <= 0.03480243682861328) ?  0.014380702376085706 :   0.005484267076830676 :   (b('moss') <= 11.857266426086426) ?  -0.013014738795415006 :   -0.04367096410611646 :   (b('l8b5') <= 3655.3448486328125) ?  (b('l8b5') <= 3616.79150390625) ?  -0.00045644783426333676 :   0.14219130765401067 :   (b('ndvi') <= 4677.5) ?  -0.0293300503606115 :   -0.07575280938529438 :   (b('grass') <= 64.57783889770508) ?  (b('dg0_v_vv') <= -3.3895492553710938) ?  (b('ndvi') <= 4177.5) ?  -0.033493786131535364 :   -0.029099842838763157 :   (b('l8b11') <= 2377.771728515625) ?  0.032692567303528604 :   -0.01350328254463215 :   -0.08378949762166349 :   (b('ndvi') <= 4862.0) ?  (b('l8b4') <= 490.92356872558594) ?  (b('ndvi') <= 4426.5) ?  (b('l8b11') <= 1114.2453002929688) ?  0.0010251656968539996 :   -0.029871201168350098 :   (b('ndvi') <= 4580.5) ?  0.035417288331575036 :   0.04203199991224865 :   (b('ndvi_med') <= 4161.0) ?  (b('dg0_v_vv') <= -0.2849602699279785) ?  -0.0183035394271423 :   0.029149974100526735 :   (b('l8b5') <= 4378.2626953125) ?  -0.038163302565503945 :   0.0055324045115272 :   (b('dg0_v_vv') <= 0.5908641815185547) ?  0.06077762861610131 :   0.05008151652448145 :   (b('moss') <= 13.82375431060791) ?  (b('moss') <= 10.798096179962158) ?  (b('crops') <= 40.39838218688965) ?  (b('l8b4') <= 526.3966827392578) ?  0.06756173439137386 :   0.07449502467047125 :   0.047713903422916926 :   0.09971623133877766 :   -0.01150690874350943 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1057.7600708007812) ?  (b('l8b11') <= 1863.1333618164062) ?  (b('l8b11') <= 1840.7261962890625) ?  (b('dg0_v_vv') <= -0.6740238666534424) ?  (b('l8b5') <= 3836.5662841796875) ?  (b('l8b11') <= 489.9412078857422) ?  (b('dg0_v_vv') <= -0.9921565055847168) ?  (b('ndvi_med') <= 3542.75) ?  (b('dg0_v_vv') <= -1.7137751579284668) ?  (b('bulk') <= 136.0) ?  -0.026621900139899624 :   0.016238686691517498 :   (b('ndvi_med') <= 3438.5) ?  0.039975924184920825 :   0.025467933894393696 :   -0.03369174107886613 :   (b('ndvi_med') <= 3595.5) ?  (b('ndvi_med') <= 3353.0) ?  0.046517600174480866 :   0.04268039392155645 :   0.036481100095382524 :   (b('bulk') <= 169.5) ?  (b('l8b5') <= 3834.0662841796875) ?  (b('l8b5_med') <= 4072.6116943359375) ?  (b('grass') <= 77.98130798339844) ?  -0.0023458205102648883 :   -0.01219505380510855 :   0.07917602977113622 :   -0.08586667517034333 :   (b('l8b5') <= 2662.39794921875) ?  0.07711312340441615 :   0.04008346832845805 :   (b('l8b4') <= 1049.8297119140625) ?  (b('dg0_v_vv') <= -0.7125353813171387) ?  (b('ndvi_med') <= 3307.25) ?  (b('ndvi_med') <= 3279.5) ?  (b('l8b4') <= 897.5686950683594) ?  0.01660204743378222 :   -0.029800655665081544 :   (b('crops') <= 53.75819778442383) ?  0.05825164428035129 :   0.08831756598426829 :   (b('l8b5_med') <= 2780.996826171875) ?  (b('l8b5') <= 4447.6416015625) ?  -0.016522934751820595 :   0.0028283357010291457 :   (b('sand') <= 18.0) ?  -0.018083640942137207 :   0.006231221986664047 :   (b('dg0_v_vv') <= -0.6822443008422852) ?  (b('l8b11_med') <= 1338.0187377929688) ?  (b('l8b4') <= 539.2140197753906) ?  0.049193401127263545 :   0.04560444989684065 :   (b('bulk') <= 137.0) ?  0.06031423233300415 :   0.0568385008499003 :   -0.0033141780018031697 :   0.122203251496529 :   (b('sand') <= 27.5) ?  (b('crops') <= 63.68690490722656) ?  (b('l8b11_med') <= 1649.4591064453125) ?  (b('l8b5') <= 5208.24169921875) ?  (b('dg0_v_vv') <= 0.3197798728942871) ?  (b('sand') <= 19.0) ?  -0.007800578483158475 :   0.012956834273775981 :   (b('dg0_v_vv') <= 0.3341960906982422) ?  0.07401617366808205 :   0.017788094143008838 :   -0.08866874025295474 :   (b('l8b4') <= 881.8335876464844) ?  (b('l8b5_med') <= 2848.1185302734375) ?  (b('ndvi_med') <= 2228.25) ?  -0.014563501761105956 :   0.01439514372187044 :   (b('dg0_v_vv') <= -0.05925941467285156) ?  -0.04615790602108794 :   -0.009657460473429644 :   (b('dg0_v_vv') <= 0.20098018646240234) ?  (b('l8b5_med') <= 2041.5950317382812) ?  -0.04048906218568825 :   -0.0059187639092766935 :   (b('l8b4') <= 997.2198181152344) ?  -0.02440146659031947 :   -0.050655809345306155 :   (b('ndvi_med') <= 3351.25) ?  (b('l8b4') <= 305.94541931152344) ?  (b('dg0_v_vv') <= 1.6192598342895508) ?  (b('dg0_v_vv') <= 0.550621509552002) ?  0.0684427764978906 :   0.05084007363887588 :   (b('dg0_v_vv') <= 2.7837114334106445) ?  0.01021803849561323 :   0.034903124255136114 :   (b('dg0_v_vv') <= 3.6233737468719482) ?  (b('ndvi') <= 4893.5) ?  0.00010310644292226157 :   -0.018714735106195212 :   (b('l8b5') <= 1387.0159912109375) ?  -0.00460222301086291 :   0.04200445397546819 :   (b('l8b5') <= 2587.7664794921875) ?  (b('crops') <= 88.03737258911133) ?  0.03625913136294617 :   0.04432704332316212 :   (b('l8b5') <= 4661.032470703125) ?  (b('l8b11') <= 1062.7363891601562) ?  -0.06910220503064186 :   -0.024870478918533903 :   0.05821224749015247 :   (b('dg0_v_vv') <= -0.672736644744873) ?  (b('grass') <= 75.79154014587402) ?  0.10376269410449916 :   0.057219952559160486 :   (b('l8b5_med') <= 3288.431396484375) ?  (b('ndvi') <= 4760.5) ?  (b('l8b4') <= 436.88543701171875) ?  (b('crops') <= 1.6479373276233673) ?  0.02214757968629399 :   0.0033588853083499487 :   (b('l8b5') <= 4332.65087890625) ?  0.0011223586113288844 :   -0.028879944454916567 :   (b('sand') <= 36.5) ?  (b('sand') <= 28.5) ?  0.05028320681287575 :   0.004464416336346552 :   (b('bulk') <= 126.5) ?  0.002090236962199751 :   -0.020218628088613758 :   (b('dg0_v_vv') <= -0.5517926216125488) ?  (b('dg0_v_vv') <= -0.5588803291320801) ?  (b('ndvi_med') <= 2430.25) ?  0.11702206657906847 :   0.00472631796423312 :   (b('l8b4') <= 730.185546875) ?  0.058585976930795286 :   0.12146159749247035 :   (b('dg0_v_vv') <= 1.328357219696045) ?  (b('l8b11') <= 1206.9920043945312) ?  0.01023707618272661 :   -0.002940017168587606 :   (b('l8b5') <= 4240.12158203125) ?  0.020626775200965057 :   -0.007052243062808098 :   (b('ndvi_med') <= 2699.75) ?  (b('bulk') <= 143.5) ?  (b('l8b4_med') <= 1172.608154296875) ?  (b('l8b4') <= 1007.5411682128906) ?  (b('l8b4_med') <= 986.2161560058594) ?  (b('l8b11_med') <= 1719.6549682617188) ?  (b('l8b4_med') <= 752.4030456542969) ?  0.04137041930367988 :   0.02493011883454823 :   (b('dg0_v_vv') <= 1.022881031036377) ?  -0.022180793117888564 :   0.012006920048329567 :   (b('l8b4_med') <= 1075.5737915039062) ?  (b('bulk') <= 130.0) ?  0.03371401737137735 :   0.04765975154875258 :   0.08007133191543722 :   0.2070531838269622 :   -0.06937147878114461 :   (b('l8b11_med') <= 1468.7632446289062) ?  0.04520763403298682 :   (b('dg0_v_vv') <= -0.035119056701660156) ?  (b('sand') <= 55.5) ?  (b('l8b11') <= 1842.3568725585938) ?  (b('l8b4_med') <= 1234.7986450195312) ?  -0.026475763671551023 :   -0.014201740011781372 :   (b('bulk') <= 159.5) ?  -0.009844854101415298 :   0.00047302374435975736 :   -0.027712068692408914 :   (b('l8b11_med') <= 2086.715576171875) ?  (b('grass') <= 68.54743957519531) ?  (b('l8b11') <= 1850.9072265625) ?  -0.048279536904664244 :   -0.03518792869751503 :   (b('grass') <= 89.81625747680664) ?  -0.01692968184557264 :   -0.023670402527557366 :   -0.06353868725508774 :   (b('moss') <= 0.9812286496162415) ?  (b('dg0_v_vv') <= -0.18318605422973633) ?  -0.09713661044594979 :   -0.10563925921262249 :   (b('l8b4_med') <= 1042.5572814941406) ?  (b('l8b5') <= 2760.661865234375) ?  (b('l8b5') <= 1926.9873046875) ?  (b('dg0_v_vv') <= -2.179783344268799) ?  (b('moss') <= 7.110084772109985) ?  -0.03262819751694557 :   -0.036700643676032985 :   (b('dg0_v_vv') <= -0.5633025169372559) ?  -0.04122270957747126 :   -0.044586573519167116 :   (b('dg0_v_vv') <= -0.3134117126464844) ?  (b('crops') <= 4.350515365600586) ?  0.004300401573581088 :   -0.022308124196171675 :   (b('ndvi') <= 4127.5) ?  -0.03519366899037167 :   -0.013145219202461741 :   (b('dg0_v_vv') <= -0.8694448471069336) ?  (b('dg0_v_vv') <= -0.9932608604431152) ?  (b('ndvi') <= 3354.5) ?  -0.022568290013000494 :   -0.012651666466149333 :   -0.03765166870132351 :   (b('dg0_v_vv') <= -0.7841811180114746) ?  0.01311418581081289 :   (b('ndvi_med') <= 2926.5) ?  0.006821740656155914 :   -0.011884210576299393 :   -0.09891806737558753 :   (b('l8b4') <= 1052.104736328125) ?  (b('l8b11') <= 1864.2517700195312) ?  (b('dg0_v_vv') <= -0.09466552734375) ?  (b('sand') <= 38.5) ?  0.05039276352231589 :   -0.026690652005690152 :   (b('bulk') <= 147.0) ?  0.09910480635615954 :   0.14649847955089368 :   (b('sand') <= 41.5) ?  (b('moss') <= 13.518618106842041) ?  (b('l8b4_med') <= 1057.4644165039062) ?  (b('l8b4_med') <= 983.9817810058594) ?  (b('l8b4') <= 1046.6054077148438) ?  (b('l8b4') <= 935.869140625) ?  -0.0031934920133409594 :   0.011102745642830559 :   (b('l8b11_med') <= 1570.7933349609375) ?  -0.0007347853681927927 :   -0.03289167041302283 :   (b('dg0_v_vv') <= -0.7143645286560059) ?  (b('l8b5') <= 2634.40234375) ?  -0.004086566691661341 :   0.0221008297628644 :   (b('moss') <= 6.797057747840881) ?  0.0647956360886395 :   0.021262413269388443 :   (b('l8b5') <= 2053.4046630859375) ?  (b('l8b11_med') <= 2081.7691650390625) ?  (b('dg0_v_vv') <= 1.0766792297363281) ?  -0.03201791520774504 :   -0.08558007973417907 :   (b('dg0_v_vv') <= -1.5348773002624512) ?  0.008258587946877888 :   -0.022958444924085736 :   (b('dg0_v_vv') <= 1.805375099182129) ?  (b('dg0_v_vv') <= 1.5908145904541016) ?  0.0029088389475066204 :   -0.07024777074942101 :   (b('l8b11_med') <= 2139.558837890625) ?  0.03323451964702935 :   0.07600590032369653 :   (b('l8b11') <= 2066.340087890625) ?  (b('l8b5') <= 2070.9853515625) ?  (b('ndvi') <= 1105.0) ?  (b('ndvi') <= 1080.0) ?  0.04504972526244565 :   0.09620107479997457 :   (b('l8b5_med') <= 2203.0350341796875) ?  0.03719315088033876 :   -0.002178138792696555 :   (b('l8b4') <= 886.9351806640625) ?  (b('dg0_v_vv') <= 0.7052574157714844) ?  -0.09757821899240597 :   -0.07651442878689788 :   (b('ndvi') <= 3628.0) ?  -2.2038346776385996e-05 :   0.09054994600424518 :   (b('l8b4') <= 904.1678161621094) ?  (b('dg0_v_vv') <= -0.08445310592651367) ?  0.05774905835632316 :   (b('ndvi') <= 2410.5) ?  0.09387917067233453 :   0.12438547876150319 :   (b('dg0_v_vv') <= -0.7595710754394531) ?  (b('ndvi') <= 3018.0) ?  0.05565398968786739 :   0.16921781807916236 :   (b('dg0_v_vv') <= 0.38128662109375) ?  0.00257574296153724 :   0.04245626568505996 :   (b('l8b5_med') <= 1676.6114501953125) ?  (b('l8b5_med') <= 1538.142578125) ?  (b('dg0_v_vv') <= 1.8613958358764648) ?  (b('dg0_v_vv') <= -0.42198753356933594) ?  -0.006049700584376899 :   (b('l8b4') <= 840.52197265625) ?  0.010377448986785484 :   0.0014499199272463709 :   -0.043678510172499185 :   (b('ndvi') <= 1580.5) ?  (b('l8b11') <= 1909.7169799804688) ?  0.10514071009570898 :   (b('ndvi') <= 1001.0) ?  -0.011391449101260054 :   0.04144812184329914 :   (b('dg0_v_vv') <= 0.7611589431762695) ?  0.16795012133887793 :   0.06192063433123829 :   (b('l8b11_med') <= 2539.11376953125) ?  (b('l8b4') <= 885.9984436035156) ?  (b('l8b5_med') <= 2244.4288330078125) ?  (b('dg0_v_vv') <= 5.3974456787109375) ?  -0.004282364404703656 :   -0.029804265372593897 :   (b('crops') <= 40.25849151611328) ?  0.026458864799813908 :   0.0058949361402452684 :   (b('ndvi_med') <= 4697.0) ?  (b('l8b4_med') <= 1320.2449951171875) ?  -0.007329884072008731 :   -0.034561869094337214 :   (b('l8b5') <= 2968.4638671875) ?  0.07619148388620622 :   0.006682854007043104 :   (b('ndvi') <= 1471.0) ?  (b('ndvi') <= 1263.0) ?  (b('dg0_v_vv') <= 1.4126996994018555) ?  0.004105853480998815 :   0.0222510620704654 :   (b('l8b4') <= 1009.5189819335938) ?  -0.034970774488411474 :   -0.05230388742571429 :   (b('l8b11') <= 1974.0182495117188) ?  (b('dg0_v_vv') <= 0.7206459045410156) ?  0.006397810974123683 :   0.021252196932985492 :   (b('dg0_v_vv') <= -0.20715570449829102) ?  0.029300008037088893 :   0.07706140627961194 :   (b('l8b11_med') <= 2212.6531982421875) ?  (b('dg0_v_vv') <= 0.5343308448791504) ?  (b('l8b11_med') <= 2062.1727294921875) ?  (b('l8b4_med') <= 1146.3720703125) ?  (b('l8b11_med') <= 1950.8327026367188) ?  (b('ndvi_med') <= 3373.0) ?  (b('dg0_v_vv') <= -0.5155224800109863) ?  0.01902736705077237 :   -0.0028390865877658774 :   (b('l8b5_med') <= 3490.31982421875) ?  0.031093684828493427 :   0.019247732312524538 :   (b('dg0_v_vv') <= -1.04325532913208) ?  (b('moss') <= 1.5) ?  0.04524828419946779 :   0.03359725210623085 :   (b('ndvi_med') <= 3305.25) ?  0.07606050278056992 :   0.08859939807791409 :   -0.06546439470955359 :   (b('l8b4') <= 1057.343994140625) ?  (b('l8b11_med') <= 2150.8240966796875) ?  (b('l8b5') <= 2135.9984130859375) ?  -0.015439220534353565 :   (b('bulk') <= 137.5) ?  -0.007377107077623554 :   5.749617089118031e-05 :   -0.035777933221619734 :   -0.07374757123718995 :   (b('l8b11_med') <= 2083.1153564453125) ?  (b('sand') <= 49.5) ?  (b('dg0_v_vv') <= 1.1634087562561035) ?  (b('l8b4_med') <= 1112.6117858886719) ?  (b('crops') <= 39.405677795410156) ?  0.0656810997822716 :   0.07028217125555361 :   0.0360394594746013 :   (b('l8b4') <= 1052.2239379882812) ?  0.05825098494556569 :   (b('ndvi') <= 3619.5) ?  0.01966672386679713 :   0.03147335183412403 :   -0.0095363007016884 :   0.09298383868909915 :   (b('l8b4') <= 1056.9166259765625) ?  (b('dg0_v_vv') <= 0.5120501518249512) ?  (b('dg0_v_vv') <= -0.8494381904602051) ?  0.10001314721877061 :   (b('l8b4_med') <= 1477.8464965820312) ?  (b('l8b4') <= 1055.3628540039062) ?  0.07092649073112287 :   0.0738338196708514 :   0.05753282267740906 :   (b('l8b11_med') <= 2335.0533447265625) ?  0.16488269146506648 :   (b('grass') <= 58.19915008544922) ?  (b('l8b11_med') <= 2651.6131591796875) ?  0.07752864218202465 :   0.08176353993436972 :   0.11452372321579085 :   -0.02425210937946791 :   (b('l8b11') <= 1748.3692626953125) ?  (b('l8b4') <= 1234.027587890625) ?  (b('ndvi_med') <= 3408.0) ?  (b('sand') <= 51.0) ?  (b('l8b11') <= 1673.8278198242188) ?  (b('dg0_v_vv') <= 4.090347051620483) ?  (b('l8b5_med') <= 1740.6298217773438) ?  (b('dg0_v_vv') <= -0.7202577590942383) ?  0.06321396779251828 :   0.01146396551264059 :   (b('l8b5') <= 4325.27099609375) ?  (b('ndvi') <= 2580.0) ?  -0.01739935986872483 :   -0.03636829152798889 :   (b('dg0_v_vv') <= 1.66619873046875) ?  0.08052493992254714 :   -0.012089657691511616 :   (b('ndvi') <= 1590.0) ?  (b('moss') <= 7.943005204200745) ?  0.025578677175459474 :   0.03748799625144392 :   0.05108900471121536 :   (b('l8b5_med') <= 3270.67578125) ?  (b('ndvi_med') <= 1984.25) ?  (b('ndvi_med') <= 1683.25) ?  (b('bulk') <= 150.5) ?  -0.012496923540695273 :   0.046607034884653065 :   (b('l8b11_med') <= 1622.3496704101562) ?  -0.026314518234498742 :   -0.06209284309162877 :   (b('ndvi_med') <= 2047.25) ?  0.14063125644366595 :   (b('dg0_v_vv') <= 0.27610063552856445) ?  -0.007245311329762174 :   0.02219726395640207 :   (b('ndvi') <= 3188.5) ?  (b('l8b11') <= 1706.22265625) ?  (b('ndvi') <= 2218.5) ?  0.0033231808772146154 :   0.00848785655483758 :   (b('l8b11_med') <= 1995.5245361328125) ?  0.021260711326454795 :   0.03099616973002489 :   (b('l8b11_med') <= 2247.2081298828125) ?  0.04016086932295615 :   (b('dg0_v_vv') <= -0.13358068466186523) ?  0.05513542483861804 :   0.0670475468323351 :   (b('l8b4') <= 1079.6654052734375) ?  (b('l8b5_med') <= 2469.2134399414062) ?  (b('l8b11_med') <= 1439.1726684570312) ?  0.10189681969165049 :   0.09315126700226789 :   0.02529219554694595 :   (b('ndvi_med') <= 1282.25) ?  (b('dg0_v_vv') <= 1.9671626091003418) ?  (b('l8b4') <= 1198.1322021484375) ?  (b('dg0_v_vv') <= 0.9752941131591797) ?  -0.02125871921909063 :   0.01396807835762326 :   (b('l8b4') <= 1209.426025390625) ?  0.0287461594811477 :   0.043846231509267454 :   (b('l8b4') <= 1233.1500854492188) ?  (b('l8b4') <= 1204.0436401367188) ?  -0.02870498940940866 :   -0.045179537960922236 :   -0.06909007069253847 :   (b('l8b4') <= 1107.6829223632812) ?  (b('l8b11_med') <= 2276.345947265625) ?  (b('sand') <= 60.5) ?  0.07356750964718103 :   0.08294194362435853 :   0.10841735648416431 :   (b('ndvi_med') <= 2171.5) ?  (b('dg0_v_vv') <= 0.22276782989501953) ?  -0.0008617191961679497 :   0.02756775943590558 :   (b('dg0_v_vv') <= 0.5070176124572754) ?  0.0006357932187105361 :   -0.03954132730266592 :   (b('crops') <= 69.80944442749023) ?  (b('dg0_v_vv') <= -0.2842745780944824) ?  (b('ndvi') <= 2820.5) ?  (b('grass') <= 41.16844940185547) ?  0.009595411200398724 :   0.006518530312728754 :   (b('crops') <= 34.17847537994385) ?  0.07436522750658053 :   (b('dg0_v_vv') <= -0.6135773658752441) ?  0.029674456993426135 :   (b('l8b4_med') <= 606.8008575439453) ?  0.049787644980605616 :   0.04427163216545166 :   (b('l8b5') <= 3468.27099609375) ?  (b('dg0_v_vv') <= 4.76837158203125e-07) ?  (b('l8b4') <= 1077.1514892578125) ?  (b('l8b11_med') <= 1475.6447143554688) ?  -0.008036181237059847 :   -0.011072404637964356 :   (b('l8b4_med') <= 921.6523132324219) ?  0.024349081562403008 :   0.01807680507603321 :   (b('ndvi') <= 3848.0) ?  (b('ndvi_med') <= 3593.0) ?  0.031022224474310342 :   0.02109222594438631 :   (b('l8b4_med') <= 798.9369201660156) ?  0.06466555909429145 :   0.02014444419370337 :   (b('ndvi_med') <= 3595.5) ?  -0.016875569034863203 :   (b('ndvi') <= 4460.5) ?  -0.005060069523607408 :   -0.008077659494667239 :   0.10631750989544186 :   (b('ndvi_med') <= 1864.75) ?  (b('l8b11') <= 1404.4362182617188) ?  (b('dg0_v_vv') <= -0.03656911849975586) ?  0.11022841352028487 :   0.07493069118469697 :   (b('ndvi_med') <= 1396.5) ?  (b('ndvi') <= 2028.5) ?  (b('ndvi') <= 1229.5) ?  (b('l8b4') <= 1266.0758666992188) ?  (b('dg0_v_vv') <= 0.2721824645996094) ?  0.002387358097774068 :   0.04220127502273689 :   (b('l8b5') <= 1864.1826171875) ?  -0.02745790872471131 :   0.0013210827141654177 :   (b('ndvi') <= 1445.5) ?  (b('l8b4') <= 1420.0988159179688) ?  0.06053093262663625 :   0.03844538115150328 :   (b('l8b4') <= 1412.9295043945312) ?  0.003538141558217439 :   0.022337692803827844 :   (b('l8b11_med') <= 2408.3731689453125) ?  0.06491201089988569 :   0.08563814710564083 :   (b('l8b4') <= 1244.0731811523438) ?  (b('dg0_v_vv') <= 1.1193194389343262) ?  (b('dg0_v_vv') <= 0.261106014251709) ?  (b('dg0_v_vv') <= -1.779381275177002) ?  0.046071389280314845 :   0.0136405175653402 :   (b('dg0_v_vv') <= 0.8701643943786621) ?  -0.01903000696873286 :   -0.03099470758322892 :   (b('dg0_v_vv') <= 1.2324090003967285) ?  0.030054528194867536 :   0.05605452128072874 :   (b('ndvi') <= 2403.0) ?  (b('l8b11') <= 1551.397705078125) ?  (b('dg0_v_vv') <= 1.3407044410705566) ?  0.010085302817671206 :   -0.006293441032059421 :   (b('ndvi') <= 1258.5) ?  -0.052163316331230475 :   -0.016825428317630908 :   (b('l8b5_med') <= 3169.1014404296875) ?  (b('crops') <= 32.42837142944336) ?  -0.05160425511784275 :   -0.03382955246055566 :   -0.0525549158469763 :   (b('ndvi_med') <= 2141.0) ?  (b('grass') <= 70.45530700683594) ?  (b('moss') <= 0.4768786132335663) ?  (b('dg0_v_vv') <= -0.47808170318603516) ?  (b('ndvi') <= 1651.0) ?  -0.009541565745517025 :   -0.0085919893258634 :   -0.0025471257818618243 :   (b('dg0_v_vv') <= 0.5519082546234131) ?  (b('l8b5') <= 3181.977783203125) ?  (b('l8b5_med') <= 3378.561279296875) ?  0.053044268002414005 :   0.011661425741360587 :   (b('l8b4_med') <= 1444.7171020507812) ?  0.0845013432922761 :   0.10217885067376808 :   (b('l8b5_med') <= 2844.591552734375) ?  (b('l8b4') <= 1294.8175659179688) ?  -0.029728318277975035 :   0.03158860932546664 :   (b('ndvi') <= 1549.5) ?  0.027872734752082504 :   0.04155687407268052 :   (b('l8b11') <= 1649.867431640625) ?  0.10249384345390747 :   0.14362899197331452 :   (b('dg0_v_vv') <= -0.6789703369140625) ?  (b('grass') <= 8.702454090118408) ?  (b('dg0_v_vv') <= -1.7436211109161377) ?  (b('l8b5_med') <= 2751.021484375) ?  0.04607691831420896 :   0.022725416536115872 :   (b('ndvi') <= 3487.5) ?  (b('dg0_v_vv') <= -0.9321417808532715) ?  0.012733332964498115 :   -0.001966114308022654 :   -0.0204335885926209 :   (b('l8b4_med') <= 814.8666687011719) ?  (b('dg0_v_vv') <= -3.7111592292785645) ?  -0.015859817779549878 :   (b('l8b4') <= 1259.9467163085938) ?  0.008845308254860462 :   0.001796838695724735 :   (b('dg0_v_vv') <= -2.157336711883545) ?  (b('l8b11') <= 1668.487060546875) ?  -0.020488951383676006 :   -0.027059666095513538 :   (b('crops') <= 57.38829803466797) ?  -0.044257346636957934 :   -0.033839672781591223 :   (b('l8b11') <= 1563.1209716796875) ?  -0.03303795992386973 :   (b('l8b4') <= 1339.3216552734375) ?  (b('l8b5_med') <= 3544.12353515625) ?  (b('ndvi') <= 3756.5) ?  0.046055115734800076 :   0.02001555137896316 :   (b('dg0_v_vv') <= 1.6008386611938477) ?  -0.000632966000702645 :   0.0426779111738706 :   (b('grass') <= 24.20687770843506) ?  (b('dg0_v_vv') <= 0.5269217491149902) ?  0.010850047903280079 :   0.022631338988562615 :   -0.01201288937902116 :   (b('bulk') <= 122.5) ?  (b('l8b4') <= 1095.3648071289062) ?  (b('l8b4_med') <= 723.7029724121094) ?  -0.05233790363560098 :   -0.03919518048611491 :   (b('l8b5') <= 2064.804931640625) ?  (b('l8b11') <= 2386.3790283203125) ?  (b('bulk') <= 115.5) ?  0.032077300508850326 :   0.012699603142691362 :   (b('ndvi_med') <= 1670.5) ?  (b('dg0_v_vv') <= 1.0244710445404053) ?  0.07966414920917775 :   0.07870700640265843 :   (b('dg0_v_vv') <= -0.6638936996459961) ?  0.05712979847109456 :   0.0558692932075337 :   (b('crops') <= 0.44501277804374695) ?  (b('l8b11') <= 3400.300048828125) ?  (b('l8b11_med') <= 2164.5125732421875) ?  (b('ndvi_med') <= 2164.0) ?  (b('dg0_v_vv') <= 1.3763604164123535) ?  -0.0036878984824044725 :   0.006369730796325024 :   0.018743659887309777 :   (b('dg0_v_vv') <= 0.23838329315185547) ?  (b('ndvi') <= 1869.0) ?  -0.01586262243725926 :   0.001978428765701201 :   (b('l8b5') <= 2314.4403076171875) ?  -0.021654399988733412 :   -0.029294734388824725 :   0.08615382244852177 :   (b('dg0_v_vv') <= 1.5752320289611816) ?  (b('dg0_v_vv') <= 0.05388069152832031) ?  (b('l8b4') <= 1187.9275512695312) ?  (b('crops') <= 32.25657054781914) ?  0.0038577129849050595 :   0.01680441398808788 :   (b('l8b11_med') <= 1653.8571166992188) ?  0.035797087554944075 :   0.045039752190704385 :   (b('moss') <= 10.69543743133545) ?  (b('bulk') <= 118.0) ?  0.058742687851837616 :   0.06022016012234088 :   0.07564540367850803 :   (b('l8b11') <= 2342.442138671875) ?  (b('ndvi') <= 3111.5) ?  0.023738859754911967 :   (b('ndvi_med') <= 4525.5) ?  0.02172757432329664 :   0.020779652253531955 :   0.01834199120397334 :   (b('l8b5_med') <= 2627.353515625) ?  (b('l8b5_med') <= 2419.8912353515625) ?  (b('l8b4_med') <= 1266.1316528320312) ?  (b('l8b5_med') <= 2407.8277587890625) ?  (b('bulk') <= 155.5) ?  (b('l8b5') <= 2403.59033203125) ?  (b('sand') <= 43.5) ?  -0.010782857605315145 :   -0.0027048486682946936 :   (b('l8b5') <= 2443.246337890625) ?  0.029332444357570733 :   2.0211387514179918e-05 :   (b('l8b11') <= 1875.4254760742188) ?  (b('l8b4') <= 1197.1094970703125) ?  -0.023189655762382733 :   -0.05556910765381712 :   (b('l8b5') <= 1743.3602905273438) ?  0.06773172514967712 :   -0.01192011874848523 :   (b('dg0_v_vv') <= 0.32656192779541016) ?  (b('dg0_v_vv') <= 0.09198379516601562) ?  (b('l8b5') <= 2355.337890625) ?  -0.0059133268677150075 :   -0.017706040014707897 :   (b('l8b5') <= 2669.5572509765625) ?  0.00465098468672287 :   0.05437648116186089 :   (b('l8b11') <= 2242.998779296875) ?  (b('l8b5') <= 2751.282958984375) ?  0.08082087379188 :   0.046584161396226315 :   0.12040515543399549 :   (b('moss') <= 44.673912048339844) ?  (b('grass') <= 50.68658447265625) ?  (b('l8b4') <= 1234.1113891601562) ?  (b('l8b11_med') <= 2449.083740234375) ?  0.0020870725014990304 :   0.05454976420428792 :   (b('ndvi') <= 1657.0) ?  -0.004783032951482582 :   -0.04176861740832395 :   (b('ndvi') <= 811.0) ?  (b('l8b4_med') <= 1539.7101440429688) ?  0.08716135798450789 :   0.0187513177540248 :   (b('bulk') <= 138.0) ?  -0.011183164680255629 :   0.007564252578863506 :   0.13669645519538007 :   (b('dg0_v_vv') <= -0.10027027130126953) ?  (b('dg0_v_vv') <= -0.10082197189331055) ?  (b('l8b4') <= 1675.659423828125) ?  (b('sand') <= 29.0) ?  (b('dg0_v_vv') <= -0.3908500671386719) ?  0.05130131944348847 :   -0.0016359775169400015 :   (b('l8b11_med') <= 1739.5525512695312) ?  -0.03463366925189944 :   -0.006588973551497191 :   (b('ndvi_med') <= 1649.5) ?  (b('ndvi') <= 1572.0) ?  -0.002108939900944911 :   0.08445158595624946 :   (b('l8b4') <= 1719.32470703125) ?  0.05499571925210469 :   0.005939987828205065 :   0.17396224982045322 :   (b('ndvi') <= 2863.0) ?  (b('dg0_v_vv') <= 6.562955617904663) ?  (b('l8b4') <= 1343.5255126953125) ?  (b('ndvi') <= 2006.0) ?  -0.027276651063895502 :   -0.009189849717631399 :   (b('l8b4') <= 1463.1424560546875) ?  0.0027673363444500262 :   -0.012970363462249258 :   0.08154332494966365 :   (b('ndvi_med') <= 2850.75) ?  (b('l8b11') <= 2067.51513671875) ?  (b('l8b5') <= 2863.4122314453125) ?  0.020024141784837662 :   -0.026326079103422995 :   (b('ndvi_med') <= 2451.5) ?  0.045605394180586845 :   0.07320561875686189 :   (b('l8b11_med') <= 2150.8240966796875) ?  (b('l8b11_med') <= 2046.7072143554688) ?  -0.018065558026520576 :   -0.0591676614963726 :   (b('dg0_v_vv') <= 0.20228099822998047) ?  0.0056499505494369065 :   0.03752370991641721 :   (b('l8b5_med') <= 2686.7984619140625) ?  (b('dg0_v_vv') <= 1.4914264678955078) ?  (b('ndvi') <= 1661.0) ?  (b('l8b5') <= 2391.423583984375) ?  (b('ndvi_med') <= 2081.5) ?  (b('l8b5') <= 2380.047607421875) ?  -0.025006737436032718 :   -0.06659708381501445 :   (b('l8b11') <= 2395.552978515625) ?  -0.0007248350756793309 :   0.021419112266755802 :   (b('l8b5') <= 2566.0340576171875) ?  (b('ndvi') <= 1365.5) ?  0.025841881746817633 :   -0.004158721746459028 :   (b('ndvi') <= 1574.5) ?  -0.003557278241043765 :   -0.050656602072668346 :   (b('sand') <= 41.5) ?  (b('sand') <= 39.5) ?  (b('ndvi') <= 1760.0) ?  0.08776883469446353 :   0.01103133619893047 :   (b('ndvi') <= 2177.0) ?  -0.004235086113843806 :   -0.03099261967401251 :   (b('l8b11') <= 2513.7108154296875) ?  (b('ndvi_med') <= 2229.0) ?  0.006826704097851018 :   0.062201498651205805 :   (b('grass') <= 48.83341026306152) ?  0.07716137755860143 :   0.03681299606237536 :   (b('dg0_v_vv') <= 5.363816261291504) ?  (b('l8b11') <= 2800.1309814453125) ?  (b('l8b11_med') <= 2521.3787841796875) ?  (b('l8b5') <= 2449.815185546875) ?  0.026727828881381106 :   0.06744620550856323 :   (b('dg0_v_vv') <= 3.171999454498291) ?  -0.020913624693762834 :   0.03142285911732301 :   (b('l8b11') <= 2959.8826904296875) ?  0.11311710896457669 :   0.048809374834698493 :   0.12843971495677842 :   (b('dg0_v_vv') <= -1.644315242767334) ?  (b('l8b11') <= 2949.3524169921875) ?  (b('grass') <= 21.564693450927734) ?  (b('l8b4') <= 1105.4560546875) ?  (b('bulk') <= 131.5) ?  0.039242986403658066 :   -0.0035432459160689295 :   (b('l8b5') <= 1964.449462890625) ?  0.026753187462039835 :   -0.006615607650459112 :   (b('l8b5_med') <= 2702.364990234375) ?  (b('l8b11') <= 2388.7415771484375) ?  0.05871511490976623 :   0.015552096050749559 :   (b('moss') <= 16.151951789855957) ?  -0.016476966503087703 :   -0.030442341152568577 :   (b('l8b5_med') <= 3322.7132568359375) ?  (b('crops') <= 27.17727279663086) ?  (b('dg0_v_vv') <= -1.9363203048706055) ?  -0.01887111272544498 :   0.0054069499652817845 :   (b('l8b5') <= 3397.3648681640625) ?  0.027939990383262454 :   0.0051831350212071204 :   (b('l8b11') <= 2967.795166015625) ?  (b('ndvi_med') <= 1452.0) ?  0.023090292554839187 :   0.06288537833685161 :   (b('dg0_v_vv') <= -3.319049835205078) ?  -0.019115946754439686 :   -0.006997693883295484 :   (b('l8b5') <= 2728.5465087890625) ?  (b('bulk') <= 151.5) ?  (b('l8b5') <= 2713.094970703125) ?  (b('ndvi') <= 2064.5) ?  -0.0045110376035551025 :   0.006755174712634964 :   (b('ndvi') <= 1391.0) ?  0.06445423006093728 :   0.00032859586491650603 :   (b('ndvi') <= 1631.5) ?  (b('crops') <= 3.376216322183609) ?  -0.006733692694032946 :   0.019207510762460283 :   (b('l8b4') <= 1788.5911254882812) ?  0.027501649955464877 :   0.152122206886591 :   (b('l8b5') <= 2747.8973388671875) ?  (b('l8b4_med') <= 887.7091674804688) ?  0.08258221743160687 :   (b('bulk') <= 156.0) ?  -0.023260586151735957 :   0.004358488421208662 :   (b('ndvi_med') <= 1688.5) ?  (b('l8b11_med') <= 2561.252685546875) ?  0.013051153353767704 :   -0.0002935914729211588 :   (b('l8b5') <= 3215.1201171875) ?  -0.00032067657646911094 :   -0.005626611640781163 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1319.6099853515625) ?  (b('dg0_v_vv') <= 1.3755393028259277) ?  (b('dg0_v_vv') <= 1.2777771949768066) ?  (b('dg0_v_vv') <= 1.269399642944336) ?  (b('sand') <= 17.0) ?  (b('ndvi') <= 3248.0) ?  (b('l8b4') <= 1058.5663452148438) ?  (b('ndvi') <= 2364.0) ?  (b('dg0_v_vv') <= 0.9554433822631836) ?  (b('dg0_v_vv') <= 0.43372344970703125) ?  0.021006060920362044 :   -0.009560665462099688 :   (b('grass') <= 30.054546356201172) ?  0.037081398312887426 :   0.06008358462595448 :   (b('l8b11') <= 1355.5293579101562) ?  (b('dg0_v_vv') <= -0.20060014724731445) ?  0.08284193081485394 :   0.05883480373729073 :   (b('l8b4') <= 995.9427795410156) ?  0.02996857527989767 :   0.053501281967327674 :   (b('l8b4') <= 1124.2205810546875) ?  (b('ndvi') <= 2516.0) ?  (b('l8b5') <= 2099.0493774414062) ?  -0.03536156753425297 :   -0.04776240935816123 :   0.009974343292769527 :   (b('l8b4_med') <= 841.0370788574219) ?  0.030438857278814735 :   0.0020917393705630893 :   (b('ndvi') <= 3454.5) ?  (b('moss') <= 0.689393937587738) ?  -0.06388014667335457 :   -0.15516883934441705 :   (b('ndvi') <= 4359.0) ?  (b('l8b5') <= 2418.3916015625) ?  0.09280272643803039 :   (b('l8b5') <= 3651.0595703125) ?  -0.023284423803604955 :   0.007195156480763018 :   (b('l8b4') <= 707.8949279785156) ?  (b('l8b11') <= 1246.5374755859375) ?  0.020397750194582252 :   0.05712186702856522 :   (b('dg0_v_vv') <= 0.15175199508666992) ?  0.026110747856708277 :   -0.0219086204630627 :   (b('l8b4_med') <= 1146.0831909179688) ?  (b('ndvi_med') <= 1577.0) ?  (b('l8b5_med') <= 2204.3419189453125) ?  (b('l8b4') <= 873.7253723144531) ?  (b('l8b11') <= 910.9543151855469) ?  -0.05343284057120424 :   0.0014078706871491888 :   (b('l8b11') <= 1206.7762451171875) ?  0.04995329314851279 :   -0.013778886728482792 :   (b('l8b5') <= 2954.357666015625) ?  (b('l8b11_med') <= 1682.8125610351562) ?  -0.03655594482071792 :   -0.08125021885592461 :   0.07243072349420826 :   (b('grass') <= 47.18437385559082) ?  (b('ndvi_med') <= 2006.25) ?  (b('l8b4') <= 1014.5798034667969) ?  0.02488856471765965 :   -0.00301847266656019 :   (b('l8b11_med') <= 1997.91162109375) ?  0.00015003455090864803 :   -0.008481225447046574 :   (b('grass') <= 47.93498420715332) ?  (b('dg0_v_vv') <= 0.7818851470947266) ?  0.019235208440261477 :   0.09975156072761168 :   (b('crops') <= 3.8033334016799927) ?  5.094819394521275e-05 :   0.007091029725255685 :   (b('moss') <= 22.585250854492188) ?  (b('ndvi_med') <= 1866.75) ?  (b('dg0_v_vv') <= 1.1840829849243164) ?  (b('l8b5') <= 2298.380126953125) ?  -0.007739495101987364 :   0.002660672120118531 :   (b('l8b5') <= 3104.8907470703125) ?  -0.05840027327857459 :   0.02704907537538076 :   (b('l8b4') <= 919.4558410644531) ?  (b('l8b4_med') <= 1198.9104614257812) ?  0.04638659627489875 :   0.008186794807024807 :   (b('l8b4_med') <= 1356.513916015625) ?  -0.0030474652237134315 :   0.014570797999073735 :   (b('l8b4') <= 1183.8666381835938) ?  (b('moss') <= 25.785058975219727) ?  (b('ndvi_med') <= 1531.75) ?  0.07977465535247949 :   0.00797480271104376 :   (b('l8b4_med') <= 1407.2359008789062) ?  0.003081614123862778 :   0.03242065952209771 :   (b('l8b5_med') <= 1961.0538330078125) ?  (b('grass') <= 56.12417984008789) ?  -0.013550171070310715 :   0.03485204111765294 :   (b('l8b5') <= 2411.480224609375) ?  -0.006928219698174995 :   0.015120227386522399 :   (b('dg0_v_vv') <= 1.2719659805297852) ?  (b('ndvi') <= 4100.0) ?  (b('dg0_v_vv') <= 1.2712428569793701) ?  (b('sand') <= 39.5) ?  0.0840433703087394 :   (b('l8b11_med') <= 2120.3424072265625) ?  (b('grass') <= 40.37883758544922) ?  0.06267336449899072 :   0.06421798135080103 :   0.05869146018760857 :   0.09715413429698569 :   0.013521408238480193 :   (b('crops') <= 11.721932411193848) ?  -0.0184160456437219 :   (b('l8b11') <= 1592.6756591796875) ?  (b('moss') <= 9.07625699043274) ?  (b('l8b4_med') <= 784.4543151855469) ?  0.012717606596947928 :   0.013441019721632322 :   0.01746606693985947 :   (b('grass') <= 7.830796599388123) ?  0.020640762839445936 :   0.022207872688109753 :   (b('moss') <= 22.263896942138672) ?  (b('crops') <= 49.02328300476074) ?  (b('ndvi_med') <= 2518.5) ?  (b('ndvi_med') <= 1553.25) ?  (b('ndvi_med') <= 1453.0) ?  0.0670207114016689 :   (b('l8b11_med') <= 2332.08251953125) ?  0.04194633139618775 :   0.053659365888174576 :   (b('l8b5') <= 2748.7567138671875) ?  (b('dg0_v_vv') <= 1.3099489212036133) ?  (b('dg0_v_vv') <= 1.2956840991973877) ?  -0.023372404628488174 :   0.034145274231308045 :   (b('l8b11_med') <= 1770.1956787109375) ?  -0.04242876783703677 :   -0.01401707865070815 :   (b('dg0_v_vv') <= 1.312349796295166) ?  (b('l8b4') <= 962.2208862304688) ?  -0.03505502089593272 :   -0.04667828976871173 :   -0.08764764611366253 :   (b('ndvi') <= 4434.0) ?  (b('dg0_v_vv') <= 1.3718972206115723) ?  (b('l8b4') <= 1224.6883544921875) ?  (b('moss') <= 14.83458662033081) ?  0.017358811858806004 :   0.054493032733927936 :   -0.041528085461021366 :   (b('dg0_v_vv') <= 1.3732805252075195) ?  -0.005588376080997909 :   (b('ndvi_med') <= 3691.25) ?  -0.032996400295345604 :   -0.02375564116278972 :   (b('bulk') <= 149.0) ?  (b('l8b11_med') <= 1214.4111938476562) ?  (b('ndvi_med') <= 3726.5) ?  -0.05190308498991966 :   -0.03969860292674118 :   (b('bulk') <= 132.0) ?  0.022041778421783148 :   -0.005183598166262148 :   -0.08078729258396461 :   (b('l8b4_med') <= 874.0668640136719) ?  (b('l8b4') <= 773.3414611816406) ?  (b('l8b5_med') <= 2139.3519287109375) ?  0.006487876719260155 :   (b('l8b5') <= 3714.064208984375) ?  (b('l8b4_med') <= 833.2142639160156) ?  -0.056723755049335516 :   -0.09588084014545517 :   (b('dg0_v_vv') <= 1.3327383995056152) ?  -0.019179468662666187 :   -0.05090565791053639 :   (b('l8b4_med') <= 813.8113098144531) ?  (b('moss') <= 2.6093502044677734) ?  (b('l8b11') <= 1477.2451782226562) ?  -0.045574605238634075 :   -0.03415078409434037 :   (b('ndvi_med') <= 3060.0) ?  -0.01955099487812731 :   -0.025719987167802244 :   (b('l8b11') <= 1471.169677734375) ?  -0.009116109565925401 :   0.017745168239115455 :   (b('ndvi_med') <= 2570.75) ?  (b('l8b11') <= 1646.4067993164062) ?  (b('grass') <= 14.380195140838623) ?  (b('dg0_v_vv') <= 1.3409748077392578) ?  -0.002682410571550257 :   -0.011599725692318347 :   0.04591801076286384 :   (b('bulk') <= 139.5) ?  -0.04619137802147619 :   -0.060844059750883084 :   (b('crops') <= 82.78498077392578) ?  (b('crops') <= 69.3748722076416) ?  (b('dg0_v_vv') <= 1.2890698909759521) ?  0.022306611596445702 :   0.018669591540419375 :   (b('l8b11') <= 1030.4399719238281) ?  0.017662077437458124 :   0.013228368031551205 :   (b('l8b5') <= 2639.8560791015625) ?  0.03263321822665155 :   0.03733035844925692 :   (b('l8b4') <= 1047.3671569824219) ?  (b('l8b5_med') <= 2206.8883056640625) ?  (b('l8b5') <= 2047.4380493164062) ?  -0.09002825351620752 :   -0.09359259339578344 :   (b('l8b5_med') <= 2802.92431640625) ?  (b('dg0_v_vv') <= 1.2974638938903809) ?  -0.05207452387515252 :   -0.06610174566827233 :   -0.03694943093475225 :   (b('ndvi') <= 1495.5) ?  (b('l8b4') <= 1104.3963012695312) ?  -0.020363659604920648 :   (b('l8b5_med') <= 1893.4776000976562) ?  -0.02574528092024197 :   -0.027964131568051942 :   -0.012294336421092511 :   (b('l8b11_med') <= 1863.16455078125) ?  (b('ndvi_med') <= 2951.5) ?  (b('crops') <= 31.142061233520508) ?  (b('sand') <= 42.5) ?  (b('l8b11') <= 2229.286376953125) ?  (b('l8b11') <= 900.4241638183594) ?  (b('ndvi') <= 2539.0) ?  (b('ndvi_med') <= 2531.0) ?  -0.11886543803716046 :   -0.09732304048942897 :   (b('dg0_v_vv') <= 1.7315974235534668) ?  -0.007871151535094587 :   -0.03814619147705664 :   (b('l8b4') <= 647.2781066894531) ?  (b('ndvi') <= 4224.5) ?  0.007658769194228953 :   -0.06231522423460284 :   (b('l8b4') <= 673.9880981445312) ?  -0.0590468988032088 :   -0.014298506982237395 :   (b('l8b4_med') <= 837.4677429199219) ?  (b('grass') <= 42.318368911743164) ?  0.06254603206711093 :   0.08596681314665733 :   (b('ndvi') <= 2976.0) ?  -0.019037918998751194 :   0.029155414652996675 :   (b('l8b4') <= 928.5679626464844) ?  (b('l8b5') <= 2431.9801025390625) ?  (b('dg0_v_vv') <= 2.17101788520813) ?  (b('l8b11') <= 1447.71728515625) ?  0.02297831541382633 :   0.052018404383847125 :   (b('l8b5') <= 1683.4091186523438) ?  -0.0327940340836631 :   0.015853032632431183 :   (b('dg0_v_vv') <= 1.6172447204589844) ?  (b('l8b5_med') <= 2405.40380859375) ?  -0.007472661679420178 :   0.008254369773644604 :   (b('ndvi') <= 3874.5) ?  -0.0314069532623961 :   -0.013569422822716967 :   (b('ndvi') <= 1783.0) ?  (b('l8b5_med') <= 1714.6420288085938) ?  (b('bulk') <= 144.0) ?  -0.021616077571169155 :   -0.039310659155249256 :   (b('dg0_v_vv') <= 2.1033289432525635) ?  -0.014808528590040268 :   0.010837325394713632 :   (b('l8b4') <= 939.7916259765625) ?  -0.05422881513430447 :   (b('l8b5') <= 2200.4310913085938) ?  -0.0379575926059626 :   -0.034657185785584806 :   (b('l8b4') <= 482.96197509765625) ?  (b('sand') <= 27.0) ?  (b('ndvi') <= 2280.5) ?  (b('l8b11_med') <= 1651.1475219726562) ?  0.007758228773908665 :   (b('ndvi_med') <= 2730.5) ?  -0.03905044858002912 :   -0.016966016097876013 :   (b('ndvi_med') <= 2724.75) ?  (b('ndvi_med') <= 2590.0) ?  0.018629261798013053 :   0.02548238989238122 :   0.06436091987022408 :   (b('ndvi') <= 4879.0) ?  (b('l8b5') <= 5313.615234375) ?  (b('dg0_v_vv') <= 1.6597907543182373) ?  -0.02531504536903088 :   -0.03665916724125616 :   (b('dg0_v_vv') <= 2.4444005489349365) ?  -0.017014894844226837 :   -0.008782297824759443 :   (b('l8b5_med') <= 2797.3056640625) ?  0.07007298880049834 :   (b('ndvi_med') <= 2728.75) ?  -0.0037606886038022425 :   -0.02930582632322576 :   (b('l8b11') <= 1066.4810791015625) ?  (b('grass') <= 5.101344794034958) ?  (b('dg0_v_vv') <= 2.078463315963745) ?  (b('dg0_v_vv') <= 1.8961584568023682) ?  0.015669761247804607 :   0.007620039488729469 :   (b('dg0_v_vv') <= 3.34289813041687) ?  0.02590038976986797 :   0.017389816493999233 :   (b('l8b4_med') <= 1115.8173828125) ?  (b('dg0_v_vv') <= 1.8781099319458008) ?  0.10718004366649167 :   0.1302673954337233 :   0.03183625235041532 :   (b('moss') <= 3.8244681358337402) ?  (b('ndvi') <= 4362.5) ?  (b('ndvi') <= 1431.5) ?  -0.02086969090912311 :   0.016719156158429352 :   (b('dg0_v_vv') <= 1.591846227645874) ?  0.008836545249171462 :   -0.02415419286651685 :   (b('ndvi') <= 4609.0) ?  (b('l8b11_med') <= 1316.21044921875) ?  -0.04549381903333905 :   -0.0026166390449135703 :   (b('grass') <= 16.704999685287476) ?  -0.00217824143240114 :   0.04809038570364653 :   (b('bulk') <= 106.5) ?  (b('dg0_v_vv') <= 1.5019469261169434) ?  (b('l8b4_med') <= 542.8323364257812) ?  0.05448305078101523 :   0.02408550350541183 :   (b('l8b4') <= 610.0017700195312) ?  0.09809685578986199 :   0.09001719370079708 :   (b('dg0_v_vv') <= 1.3829245567321777) ?  (b('moss') <= 11.19261360168457) ?  0.11658792466458268 :   0.06413754892429677 :   (b('l8b5') <= 2499.427734375) ?  (b('l8b11_med') <= 1421.1503295898438) ?  (b('sand') <= 38.0) ?  (b('grass') <= 16.937182426452637) ?  0.09066935005190341 :   0.03983101203401674 :   (b('l8b11_med') <= 1379.8675537109375) ?  0.016460186528690798 :   0.08470983817530026 :   (b('moss') <= 8.897212982177734) ?  (b('dg0_v_vv') <= 1.3998303413391113) ?  -0.07302480864633618 :   0.013327808409249716 :   (b('ndvi_med') <= 3038.0) ?  -0.09566001334615454 :   -0.01725043211377448 :   (b('bulk') <= 151.5) ?  (b('l8b5_med') <= 2781.259521484375) ?  (b('l8b4') <= 802.5525207519531) ?  0.0026082253314768226 :   0.047439701487703705 :   (b('bulk') <= 116.5) ?  0.021262277137381155 :   -0.0018182043011376373 :   (b('sand') <= 40.0) ?  (b('dg0_v_vv') <= 2.871100902557373) ?  0.013072546343865314 :   0.03344180496835056 :   (b('l8b11_med') <= 1619.7493286132812) ?  0.041679678224458346 :   0.07298122851508931 :   (b('ndvi_med') <= 1848.0) ?  (b('dg0_v_vv') <= 1.4449467658996582) ?  (b('moss') <= 5.908496856689453) ?  (b('l8b11') <= 1299.6067199707031) ?  0.10552946801335095 :   0.09828330682959263 :   (b('l8b4') <= 1217.6212158203125) ?  (b('moss') <= 32.32556629180908) ?  0.01574604674692731 :   0.014953129863921999 :   (b('ndvi') <= 1413.0) ?  (b('ndvi') <= 1102.5) ?  0.037597173604956596 :   0.04001179809413655 :   0.02799125059793456 :   (b('grass') <= 83.59170150756836) ?  (b('l8b5_med') <= 2415.0633544921875) ?  (b('ndvi') <= 1713.0) ?  (b('bulk') <= 151.5) ?  (b('grass') <= 54.86474609375) ?  0.028734803331398783 :   -0.007322037363797954 :   (b('l8b4') <= 1172.0806884765625) ?  -0.12586399044986868 :   0.020605594808058708 :   (b('l8b4') <= 1027.7115478515625) ?  (b('l8b11_med') <= 2073.2791748046875) ?  -0.013745993296678229 :   0.017480835338316417 :   (b('l8b4') <= 1194.553955078125) ?  0.053622905528120345 :   0.01679629036383405 :   (b('bulk') <= 135.5) ?  0.08585035151648648 :   (b('l8b11') <= 1779.9449462890625) ?  (b('l8b11') <= 1563.0551147460938) ?  -0.006910125968282922 :   0.0187453737943297 :   (b('l8b5_med') <= 2701.70166015625) ?  -0.032770560045355085 :   -0.004863310063364351 :   (b('l8b4_med') <= 1272.5029907226562) ?  (b('l8b11') <= 2226.6651611328125) ?  (b('l8b5') <= 1903.0554809570312) ?  -0.000673759364137605 :   (b('l8b11') <= 2182.029052734375) ?  -0.03093145084879313 :   -0.010374016896366162 :   (b('l8b5_med') <= 2146.911865234375) ?  0.005767055480475458 :   0.020615990795406797 :   (b('l8b4') <= 1131.9235229492188) ?  (b('moss') <= 8.776916980743408) ?  (b('l8b11_med') <= 2173.7607421875) ?  0.09820435104400549 :   0.0655086359290842 :   (b('sand') <= 51.5) ?  0.0459801042400938 :   0.02819765122816295 :   (b('dg0_v_vv') <= 3.3200435638427734) ?  0.025510496795563442 :   0.004743799720555331 :   (b('l8b4_med') <= 938.6705627441406) ?  (b('l8b5') <= 1953.8572998046875) ?  (b('ndvi') <= 1767.0) ?  -0.004804874121066921 :   (b('dg0_v_vv') <= 2.59889018535614) ?  (b('dg0_v_vv') <= 2.2265830039978027) ?  (b('l8b4_med') <= 807.3660278320312) ?  0.04769183677868802 :   0.05446175774185 :   0.03758337743723075 :   0.01713452486374553 :   (b('l8b5') <= 2683.019287109375) ?  (b('l8b5') <= 2249.138916015625) ?  (b('ndvi') <= 2052.0) ?  (b('dg0_v_vv') <= 7.305660009384155) ?  -0.040087295960523385 :   0.009587839240157242 :   (b('l8b5') <= 2078.8818359375) ?  -0.015341316277201478 :   0.02144268215523368 :   (b('dg0_v_vv') <= 2.521299123764038) ?  (b('moss') <= 1.1276041269302368) ?  0.03243519404669348 :   -0.026369330500160423 :   (b('dg0_v_vv') <= 4.998129367828369) ?  -0.06643174104432813 :   -0.022996990070387496 :   (b('moss') <= 5.5808939933776855) ?  (b('l8b5_med') <= 2538.4385986328125) ?  (b('grass') <= 57.18229150772095) ?  0.06793744622912062 :   0.050545858241350494 :   (b('crops') <= 41.514814376831055) ?  0.02006269276089584 :   0.022477366412023858 :   (b('dg0_v_vv') <= 4.081058502197266) ?  -0.028142662744908647 :   -0.025018266507225134 :   (b('l8b11_med') <= 1973.8591918945312) ?  (b('l8b4') <= 628.9263916015625) ?  (b('sand') <= 39.5) ?  (b('l8b11_med') <= 1934.38037109375) ?  0.10991794162056445 :   0.1088258045479778 :   0.053203725097632604 :   (b('l8b11') <= 1687.1470336914062) ?  (b('moss') <= 3.3472092151641846) ?  0.08241933430378645 :   (b('l8b11') <= 1227.8106079101562) ?  0.023113977051126496 :   -0.00469838458617983 :   (b('ndvi_med') <= 3347.75) ?  (b('l8b11') <= 2032.697509765625) ?  0.03839436431374978 :   0.07199151689755376 :   -0.02499687510136342 :   (b('l8b11_med') <= 1975.9132690429688) ?  (b('l8b5') <= 1753.2081909179688) ?  (b('ndvi') <= 1895.0) ?  -0.11110880567364545 :   -0.02269648835559296 :   (b('ndvi') <= 2837.0) ?  (b('l8b5') <= 1829.5643920898438) ?  -0.015943865107324567 :   0.037761396368589197 :   (b('dg0_v_vv') <= 1.732290267944336) ?  -0.02688240457574989 :   -0.015515063018202116 :   (b('l8b11') <= 2818.4088134765625) ?  (b('moss') <= 0.7684792578220367) ?  (b('dg0_v_vv') <= 5.538434743881226) ?  0.011148793005925748 :   -0.029990527930019505 :   (b('l8b5') <= 2775.28857421875) ?  0.014424311043925189 :   0.035415353020147776 :   0.101805398068119 :   (b('sand') <= 60.5) ?  (b('ndvi_med') <= 2044.5) ?  (b('sand') <= 45.5) ?  (b('moss') <= 5.848891496658325) ?  (b('grass') <= 80.81462478637695) ?  (b('dg0_v_vv') <= 1.8099546432495117) ?  (b('dg0_v_vv') <= 1.6739287376403809) ?  (b('dg0_v_vv') <= 0.9696903228759766) ?  (b('dg0_v_vv') <= -2.327963352203369) ?  0.03050325982802803 :   -0.011311759742880385 :   (b('ndvi') <= 1219.0) ?  -0.10595867815241886 :   -0.02950604995404315 :   (b('l8b5') <= 2619.0401611328125) ?  0.130361852296485 :   0.017496214830044293 :   (b('dg0_v_vv') <= 1.923109531402588) ?  (b('crops') <= 59.70337677001953) ?  -0.050559932572588645 :   -0.06320457540794214 :   (b('dg0_v_vv') <= 2.8785009384155273) ?  (b('ndvi_med') <= 1822.5) ?  -0.08331943308132755 :   -0.07769555596554215 :   -0.07115475882135891 :   (b('l8b5') <= 3434.7236328125) ?  (b('grass') <= 87.09188079833984) ?  (b('l8b11') <= 2389.6441650390625) ?  (b('dg0_v_vv') <= 0.14343786239624023) ?  0.033500675810185405 :   -0.01847698387736332 :   (b('l8b4') <= 1891.6947631835938) ?  -0.0031142721214222125 :   0.014629991813104933 :   (b('l8b11') <= 2514.719482421875) ?  (b('sand') <= 42.5) ?  -0.014884173784386556 :   -0.03575820455898465 :   (b('l8b5') <= 2173.5001220703125) ?  0.06148711273161761 :   -0.00039986076843610434 :   0.12273248142655971 :   (b('l8b11') <= 1716.957763671875) ?  (b('sand') <= 40.5) ?  (b('dg0_v_vv') <= 3.868384838104248) ?  (b('grass') <= 74.53148651123047) ?  (b('l8b5') <= 2677.4645385742188) ?  0.07515404999574243 :   0.07707433239507677 :   0.09224445910851672 :   0.023482787658152482 :   -0.03014535143580098 :   (b('grass') <= 77.1556510925293) ?  (b('ndvi') <= 1714.5) ?  (b('ndvi') <= 926.5) ?  (b('dg0_v_vv') <= 0.908973217010498) ?  -0.028600350199087212 :   -0.05554441242935715 :   (b('ndvi') <= 1698.5) ?  -0.002289830310188524 :   -0.028929055915228267 :   (b('l8b5_med') <= 2308.8607177734375) ?  (b('l8b11') <= 1851.1587524414062) ?  -0.0019006449829089243 :   0.05216578695194496 :   (b('l8b5') <= 2519.8988037109375) ?  0.024128183394366013 :   -0.0018887394265146564 :   (b('l8b5') <= 2252.084228515625) ?  (b('dg0_v_vv') <= 1.3201289176940918) ?  (b('ndvi_med') <= 1936.5) ?  -0.00883087261603312 :   -0.0387496472166161 :   (b('ndvi_med') <= 1323.25) ?  -0.005208566890402888 :   0.032033295320596636 :   (b('l8b5') <= 2792.0074462890625) ?  (b('ndvi') <= 1453.0) ?  -0.03707993957391776 :   -0.013866077443023225 :   (b('ndvi') <= 1890.5) ?  -0.06257388282906216 :   -0.06635968657528048 :   (b('l8b5') <= 3216.3896484375) ?  (b('l8b5') <= 3215.9681396484375) ?  (b('ndvi_med') <= 1953.0) ?  (b('sand') <= 58.5) ?  (b('l8b4') <= 1333.8070678710938) ?  (b('l8b11') <= 2435.686279296875) ?  -0.014624656556549742 :   -0.03306678336881923 :   (b('l8b4') <= 1334.4977416992188) ?  0.06269376860260478 :   -0.0004647476687577226 :   (b('l8b11') <= 2118.240478515625) ?  (b('moss') <= 23.406819343566895) ?  0.03257762502289721 :   0.06997717788068221 :   (b('l8b11') <= 2733.084228515625) ?  -0.008230067054316224 :   0.014360866209377285 :   (b('ndvi') <= 1935.5) ?  (b('ndvi') <= 1860.5) ?  (b('l8b5_med') <= 2889.014404296875) ?  0.04931404866065985 :   0.01158331516488157 :   (b('dg0_v_vv') <= -0.8072447776794434) ?  0.1088868305676503 :   0.12103534495808385 :   (b('ndvi') <= 2150.0) ?  (b('l8b4') <= 1859.056884765625) ?  -0.009952283336988606 :   -0.03414283286238626 :   (b('l8b11_med') <= 2235.6070556640625) ?  0.059876758991427276 :   0.010275291251143159 :   (b('dg0_v_vv') <= -0.2042827606201172) ?  0.08543607821355965 :   0.07784535959043588 :   (b('l8b5') <= 3245.50439453125) ?  (b('bulk') <= 150.5) ?  (b('ndvi_med') <= 1776.5) ?  (b('ndvi_med') <= 1481.0) ?  0.04272702106013637 :   (b('l8b5') <= 3237.2509765625) ?  0.01860314386268636 :   0.0077236137125213655 :   (b('ndvi') <= 1766.0) ?  (b('l8b4') <= 2112.5660400390625) ?  -0.020861902676459893 :   -0.00941829170949409 :   (b('l8b5') <= 3227.410888671875) ?  0.00882096477356041 :   -0.011393779627280894 :   (b('l8b5') <= 3227.5577392578125) ?  (b('l8b4') <= 2195.740966796875) ?  (b('ndvi') <= 1252.0) ?  -0.027051748835791693 :   -0.018162938639691494 :   0.011903831672440648 :   (b('ndvi') <= 2473.0) ?  (b('l8b11') <= 3855.335693359375) ?  -0.048350022342816275 :   -0.018341624544575642 :   (b('crops') <= 81.0027084350586) ?  0.011605097227383565 :   -0.0016729759328014127 :   (b('l8b5') <= 3253.081298828125) ?  (b('ndvi_med') <= 1642.5) ?  (b('dg0_v_vv') <= 0.011099815368652344) ?  (b('dg0_v_vv') <= -1.400324821472168) ?  0.06508932239564938 :   0.08932608686823773 :   (b('dg0_v_vv') <= 3.450453996658325) ?  0.050515696085910514 :   0.021601837802068174 :   (b('dg0_v_vv') <= 4.929647922515869) ?  (b('dg0_v_vv') <= 3.005462408065796) ?  -0.01295944136135642 :   0.030448677201451696 :   -0.07655566218501472 :   (b('l8b5') <= 3344.835205078125) ?  (b('l8b11') <= 2422.8077392578125) ?  (b('dg0_v_vv') <= 0.48514604568481445) ?  0.03147469922562895 :   -0.006341154457531093 :   (b('dg0_v_vv') <= -2.2886157035827637) ?  0.02886159293234318 :   -0.011416580047638398 :   (b('l8b5') <= 3355.4906005859375) ?  (b('grass') <= 6.51276957988739) ?  0.08557614857705595 :   0.00505274805623946 :   (b('l8b5') <= 3657.4093017578125) ?  0.0002066087217482337 :   -0.005384669069151507 :   (b('l8b4') <= 1416.4782104492188) ?  (b('l8b11') <= 1741.9607543945312) ?  (b('crops') <= 3.09326434135437) ?  (b('l8b11_med') <= 1651.3043212890625) ?  -0.013758188955888734 :   -0.021225784367364012 :   (b('l8b4_med') <= 1303.0054931640625) ?  (b('ndvi') <= 3836.0) ?  (b('l8b11_med') <= 1753.904541015625) ?  0.05583112530603085 :   (b('dg0_v_vv') <= 0.5323572158813477) ?  0.016340896366316963 :   0.03089522701880279 :   (b('ndvi') <= 4318.5) ?  (b('dg0_v_vv') <= -0.040773868560791016) ?  0.0024427343760182185 :   0.009118491290086905 :   (b('dg0_v_vv') <= 4.482882738113403) ?  0.018375782418324604 :   0.006149660410700031 :   0.04146922648278806 :   (b('l8b4') <= 1388.8563842773438) ?  (b('ndvi') <= 2677.5) ?  (b('ndvi') <= 2630.0) ?  (b('moss') <= 5.564907789230347) ?  (b('l8b11_med') <= 2536.5184326171875) ?  0.008161156266435916 :   -0.029588305062728273 :   (b('l8b11') <= 2292.3529052734375) ?  -0.02169109443588747 :   0.0031144984010007303 :   (b('dg0_v_vv') <= -0.10905694961547852) ?  (b('dg0_v_vv') <= -0.7325210571289062) ?  0.04652160671434662 :   0.005485819251051058 :   0.12328247077677468 :   (b('l8b4_med') <= 1049.93115234375) ?  (b('ndvi') <= 2687.0) ?  -0.05822552660640018 :   (b('ndvi') <= 2838.0) ?  0.05494395944793068 :   0.009747997632680083 :   (b('l8b5') <= 3814.1036376953125) ?  (b('ndvi_med') <= 2817.75) ?  -0.0393456139204281 :   -0.020652460903132677 :   (b('ndvi') <= 3249.5) ?  -0.02131646295299093 :   0.005350599156797538 :   (b('dg0_v_vv') <= -2.3347909450531006) ?  (b('ndvi_med') <= 3351.5) ?  (b('moss') <= 1.2606635093688965) ?  -0.03521348466048434 :   (b('dg0_v_vv') <= -2.5346908569335938) ?  -0.055834769353820424 :   -0.0710404052456819 :   0.005301258336603082 :   (b('bulk') <= 156.0) ?  (b('l8b4') <= 1398.8723754882812) ?  (b('l8b11') <= 2075.2384033203125) ?  -0.07400472659285173 :   -0.024744500408727863 :   (b('l8b5_med') <= 2613.8431396484375) ?  0.010798572292833323 :   -0.009257091676129368 :   -0.08670194647016286 :   (b('l8b5') <= 3371.1287841796875) ?  (b('sand') <= 31.5) ?  (b('sand') <= 30.0) ?  (b('l8b5') <= 2553.706787109375) ?  (b('l8b5') <= 2219.3436279296875) ?  (b('ndvi_med') <= 2107.5) ?  -0.04114891747317875 :   -0.030166407579714866 :   (b('l8b11_med') <= 1432.5072631835938) ?  -0.017680034066609486 :   -0.008320197530812855 :   (b('l8b11') <= 2193.146484375) ?  (b('dg0_v_vv') <= -3.2361807823181152) ?  -0.018150044365891904 :   0.04977088152845351 :   (b('dg0_v_vv') <= 0.4922637939453125) ?  -0.0033564182021330095 :   0.01531142696495034 :   (b('ndvi') <= 2675.0) ?  (b('dg0_v_vv') <= -0.19781112670898438) ?  -0.028088140168502765 :   (b('dg0_v_vv') <= 0.08812236785888672) ?  -0.036496267581376876 :   -0.04050441181192757 :   -0.05191934406133265 :   (b('dg0_v_vv') <= 0.7623982429504395) ?  (b('l8b4_med') <= 927.5771179199219) ?  (b('l8b5_med') <= 2294.3082275390625) ?  (b('l8b4') <= 1580.0695190429688) ?  0.07590635389343814 :   0.08714630222848294 :   (b('dg0_v_vv') <= -1.7359089851379395) ?  -0.004647675880927946 :   0.030228683477033683 :   (b('sand') <= 32.5) ?  (b('l8b5') <= 2814.14501953125) ?  0.01632274428641862 :   0.054882817267324205 :   (b('l8b5') <= 3363.54150390625) ?  0.0049569768016759636 :   0.037022136069828265 :   (b('ndvi_med') <= 2460.75) ?  (b('l8b11') <= 2716.1365966796875) ?  (b('crops') <= 64.63016700744629) ?  0.11111861079729535 :   0.07600928311233504 :   (b('ndvi') <= 1415.0) ?  -0.002676018541204537 :   0.03262870944646556 :   (b('l8b4') <= 1467.3112182617188) ?  (b('l8b5') <= 3110.26416015625) ?  0.01841122367515473 :   0.10886669746803104 :   (b('l8b4') <= 1496.4420166015625) ?  -0.030899579881013318 :   0.011734185282415195 :   (b('l8b4') <= 1422.0134887695312) ?  0.12186169216455264 :   (b('l8b11') <= 2173.5) ?  (b('dg0_v_vv') <= -1.7971057891845703) ?  (b('l8b4_med') <= 1206.6703491210938) ?  (b('ndvi') <= 2905.5) ?  -0.01565021751615764 :   -0.02563458200400738 :   -0.008462071005312821 :   (b('moss') <= 2.3013467490673065) ?  (b('l8b4_med') <= 1161.9876403808594) ?  -0.05983936484877103 :   -0.061990208215641746 :   (b('dg0_v_vv') <= 1.4197015762329102) ?  -0.038837794854585925 :   -0.05153656805904924 :   (b('sand') <= 26.0) ?  (b('l8b11') <= 2438.2532958984375) ?  (b('l8b5_med') <= 3042.8221435546875) ?  -0.016057506481730405 :   0.015318771876004 :   (b('dg0_v_vv') <= -3.1095919609069824) ?  0.08354766050764184 :   0.03765088263480072 :   (b('l8b11') <= 2211.810546875) ?  (b('ndvi') <= 1576.5) ?  0.13658032809780155 :   0.019661895579427435 :   (b('grass') <= 35.89316368103027) ?  -0.003999816453363226 :   0.051228988524404634 :   (b('l8b4_med') <= 1354.525634765625) ?  (b('l8b5') <= 2860.60205078125) ?  (b('grass') <= 73.09062576293945) ?  (b('l8b4') <= 1361.2163696289062) ?  0.0198858760299558 :   (b('dg0_v_vv') <= 2.6680108308792114) ?  (b('l8b4') <= 1535.9249877929688) ?  (b('l8b4') <= 1443.3748779296875) ?  (b('l8b4') <= 1425.7786865234375) ?  -0.037487442169222104 :   -0.04186398315981103 :   (b('grass') <= 51.259836196899414) ?  -0.029832787974799845 :   -0.03361605548703846 :   (b('l8b11') <= 2722.6546630859375) ?  -0.04611143140882219 :   -0.0477218634941636 :   (b('l8b4') <= 1454.7201538085938) ?  -0.015833389339166504 :   -0.013250560086800922 :   (b('l8b11') <= 2139.302001953125) ?  -0.030457699038852856 :   (b('l8b5') <= 2638.6129150390625) ?  (b('l8b5') <= 2580.370361328125) ?  (b('dg0_v_vv') <= -0.9365234375) ?  (b('l8b4') <= 1387.5850830078125) ?  -0.04995670572385923 :   -0.048625488313550375 :   -0.055157960775072915 :   -0.06344510937827104 :   -0.037060301391632244 :   (b('l8b4') <= 1492.3721313476562) ?  (b('l8b5') <= 2874.81787109375) ?  0.011571750584196266 :   (b('ndvi_med') <= 2476.0) ?  0.05879058634967148 :   (b('ndvi_med') <= 3055.5) ?  0.03656381741514367 :   0.04055287331821977 :   (b('l8b5') <= 3020.052734375) ?  -0.04053301709331702 :   (b('dg0_v_vv') <= -1.4862375259399414) ?  (b('dg0_v_vv') <= -3.478720188140869) ?  (b('dg0_v_vv') <= -3.982761859893799) ?  -0.011087763620533944 :   -0.004393645927600234 :   (b('l8b4_med') <= 763.6729736328125) ?  (b('dg0_v_vv') <= -3.035037040710449) ?  -0.015402097600007075 :   -0.019977055178624664 :   -0.031169222587792547 :   -0.00018439180799818128 :   (b('dg0_v_vv') <= -0.39765024185180664) ?  (b('l8b11') <= 2643.415771484375) ?  (b('ndvi') <= 1422.5) ?  (b('ndvi') <= 932.0) ?  (b('ndvi') <= 814.5) ?  (b('l8b11') <= 2479.061279296875) ?  (b('ndvi') <= 703.0) ?  -0.015335276289091216 :   0.006438455001517798 :   (b('l8b11') <= 2608.6109619140625) ?  0.021522340456022956 :   0.0072527848048857535 :   (b('l8b4') <= 2002.12646484375) ?  (b('dg0_v_vv') <= -0.8332290649414062) ?  0.01763818610186326 :   0.041415318877277715 :   (b('ndvi_med') <= 760.0) ?  0.010666827933243375 :   0.004539260028056755 :   (b('l8b11_med') <= 2560.39306640625) ?  (b('ndvi') <= 1173.0) ?  (b('dg0_v_vv') <= -0.8851218223571777) ?  -0.019611521256614667 :   -0.026252654542822725 :   (b('sand') <= 66.5) ?  -0.012497722628577616 :   -0.0010023498274163711 :   (b('l8b11') <= 2419.7418212890625) ?  (b('dg0_v_vv') <= -0.6444206237792969) ?  0.00402047868876735 :   -0.00529898989224159 :   (b('dg0_v_vv') <= -0.9947476387023926) ?  0.013699852949716596 :   0.018539244159375287 :   (b('ndvi') <= 1580.5) ?  (b('l8b11_med') <= 2280.1591796875) ?  0.07824274391947099 :   0.07398903722121417 :   (b('l8b11_med') <= 2116.4007568359375) ?  0.04653224813962914 :   (b('l8b11') <= 2251.8011474609375) ?  -3.2180032224188304e-05 :   (b('moss') <= 8.795579433441162) ?  0.0213964294070738 :   0.014302593721922385 :   (b('l8b5') <= 2441.680908203125) ?  (b('l8b4_med') <= 1571.6708984375) ?  (b('l8b4') <= 1411.3187255859375) ?  (b('ndvi') <= 1627.0) ?  -0.03620505707678848 :   -0.025073505290221065 :   (b('dg0_v_vv') <= -0.6682329177856445) ?  -0.04193921603676311 :   -0.04994391208505056 :   (b('l8b11') <= 3008.911376953125) ?  (b('l8b11') <= 2981.1982421875) ?  (b('l8b4_med') <= 1784.99560546875) ?  -0.009075128956086712 :   -0.0020157224087716807 :   0.01085664881424482 :   (b('l8b11') <= 3046.17578125) ?  -0.03632932535983031 :   -0.04814510369352569 :   (b('dg0_v_vv') <= -0.4383125305175781) ?  (b('l8b4_med') <= 1471.1365966796875) ?  -0.047607731180437124 :   (b('l8b4') <= 2121.522705078125) ?  (b('l8b4') <= 1560.96484375) ?  0.009556956023441743 :   -0.011086640743490434 :   (b('l8b5') <= 3920.9141845703125) ?  0.010040341426650237 :   -0.009497576283123917 :   (b('l8b4') <= 1581.893310546875) ?  -0.00688725719249305 :   (b('l8b11') <= 3354.5404052734375) ?  0.06864263020586504 :   0.03987515697929747 :   (b('ndvi') <= 1821.0) ?  (b('moss') <= 8.229613304138184) ?  (b('l8b5') <= 2285.26123046875) ?  0.0428885508737452 :   (b('dg0_v_vv') <= 0.9148550033569336) ?  (b('ndvi') <= 1575.5) ?  (b('l8b5') <= 2797.4903564453125) ?  -0.026085202782564508 :   -0.013264918674962042 :   0.018478943531149122 :   (b('ndvi') <= 1296.5) ?  (b('l8b5') <= 2752.1134033203125) ?  -0.05413056287848486 :   -0.057251320687271415 :   -0.02049410090818063 :   (b('l8b4_med') <= 2021.0213623046875) ?  (b('ndvi') <= 825.5) ?  (b('l8b5') <= 2288.311767578125) ?  (b('l8b5') <= 2242.9903564453125) ?  -0.022193827556732035 :   -0.03849815392400224 :   (b('l8b11') <= 3202.2276611328125) ?  0.0005096892814283188 :   -0.011889246040905015 :   (b('l8b11') <= 3078.5113525390625) ?  (b('ndvi') <= 837.0) ?  0.07735161049444664 :   0.005083151994616697 :   (b('l8b4_med') <= 1510.2623901367188) ?  -0.0531340632498917 :   -0.013517449309077204 :   (b('l8b11') <= 2568.300537109375) ?  (b('dg0_v_vv') <= 1.5664725303649902) ?  (b('dg0_v_vv') <= 1.062997817993164) ?  -0.021512729725447575 :   -0.06888585402568345 :   0.02363718077162008 :   (b('l8b11') <= 2612.453125) ?  (b('dg0_v_vv') <= 0.7581644058227539) ?  0.05629318520343195 :   0.044802303662936055 :   (b('l8b5') <= 4250.09716796875) ?  -0.012444205189924458 :   0.002769574870156054 :   (b('ndvi') <= 3026.5) ?  (b('l8b11') <= 3445.62451171875) ?  (b('dg0_v_vv') <= 1.765446662902832) ?  (b('l8b11') <= 2660.0697021484375) ?  (b('l8b11_med') <= 2413.2716064453125) ?  -0.005609085631225294 :   0.022245094981374582 :   (b('dg0_v_vv') <= 0.7782464027404785) ?  0.027329022378109937 :   0.06270286875559752 :   -0.015694658991048216 :   (b('ndvi_med') <= 1758.5) ?  -0.015493118632156036 :   -0.007132911456678048 :   -0.04067790444537987 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11_med') <= 3010.2078857421875) ?  (b('grass') <= 4.636483192443848) ?  (b('l8b5') <= 5436.74462890625) ?  (b('l8b11') <= 3000.6895751953125) ?  (b('l8b4') <= 2349.6187744140625) ?  (b('l8b4') <= 2109.05322265625) ?  (b('l8b4') <= 1887.9845581054688) ?  (b('l8b4') <= 1866.6190185546875) ?  (b('l8b11') <= 2951.8863525390625) ?  (b('l8b4') <= 1657.2188720703125) ?  0.005670604378858506 :   -0.0041986399996487836 :   (b('dg0_v_vv') <= 0.6485514640808105) ?  0.01378506490876183 :   0.09562541259825265 :   (b('l8b11') <= 2422.768798828125) ?  (b('dg0_v_vv') <= 1.5679280757904053) ?  -0.04149175867956957 :   -0.0888528003543902 :   (b('l8b5') <= 3201.190673828125) ?  -0.0035640872200749427 :   0.03198337528747189 :   (b('l8b11') <= 2245.3839111328125) ?  (b('l8b11_med') <= 2235.6070556640625) ?  0.08594028002687365 :   0.13417251279902942 :   (b('l8b5') <= 2687.392822265625) ?  (b('l8b11') <= 2726.005859375) ?  -0.013102921693908703 :   0.06655734891335417 :   (b('l8b5') <= 2708.6300048828125) ?  -0.04018221231339096 :   0.013715186738425016 :   (b('dg0_v_vv') <= 0.4192171096801758) ?  (b('l8b11_med') <= 2646.7789306640625) ?  (b('l8b5') <= 3010.1298828125) ?  (b('ndvi_med') <= 1031.5) ?  -0.01719184779149956 :   0.06793011656692269 :   (b('l8b5') <= 3147.88720703125) ?  -0.055255724899532 :   -0.024322053905607693 :   (b('l8b4') <= 2299.9405517578125) ?  (b('l8b5') <= 3381.1246337890625) ?  -0.009799160351892352 :   0.009570987093718735 :   (b('ndvi') <= 1679.0) ?  -0.0311060864273877 :   -0.020760800638820007 :   (b('l8b4') <= 2171.2081298828125) ?  (b('ndvi_med') <= 981.0) ?  0.016791866258047378 :   (b('dg0_v_vv') <= 0.5789775848388672) ?  -0.011181925023689074 :   -0.031021015607978795 :   (b('l8b11') <= 2966.498046875) ?  (b('dg0_v_vv') <= 0.4281501770019531) ?  0.06857322729702453 :   0.020618950442722158 :   0.11038915046271588 :   (b('l8b11') <= 2813.505859375) ?  (b('ndvi') <= 1344.0) ?  (b('l8b5_med') <= 3025.82421875) ?  (b('dg0_v_vv') <= -0.8091940879821777) ?  0.10063207851647596 :   0.11263210283517103 :   (b('dg0_v_vv') <= -0.6509366035461426) ?  0.01600992652491323 :   (b('dg0_v_vv') <= 1.4405417442321777) ?  0.059404641312291354 :   0.09972287751837541 :   (b('l8b5_med') <= 3622.8421630859375) ?  (b('dg0_v_vv') <= 0.5722389221191406) ?  (b('dg0_v_vv') <= -0.43146276473999023) ?  0.035502217646813106 :   0.029489133266658595 :   0.01994168924088871 :   (b('ndvi') <= 1754.5) ?  0.01936041329773114 :   (b('ndvi') <= 2450.5) ?  0.020550933719682196 :   0.021392939543068173 :   (b('dg0_v_vv') <= -3.484126567840576) ?  (b('ndvi_med') <= 2029.0) ?  0.025964063804616716 :   0.017572243013712963 :   (b('dg0_v_vv') <= 0.41826820373535156) ?  (b('dg0_v_vv') <= -1.54518461227417) ?  -0.04284840188798397 :   (b('l8b11_med') <= 2393.7314453125) ?  -0.04815684135775322 :   -0.050218895296321564 :   -0.05970062166390527 :   (b('l8b5') <= 3169.7298583984375) ?  (b('l8b5_med') <= 2254.820556640625) ?  (b('ndvi') <= 903.0) ?  (b('l8b4') <= 1766.52099609375) ?  -0.030043883077636724 :   (b('ndvi') <= 799.5) ?  0.01800039121049113 :   (b('dg0_v_vv') <= 0.15804386138916016) ?  0.00487969559439505 :   -0.007302976984760234 :   (b('ndvi_med') <= 869.5) ?  0.053567666803623085 :   0.03931968486576719 :   (b('ndvi') <= 1463.0) ?  (b('dg0_v_vv') <= 0.2097945213317871) ?  (b('l8b5') <= 2334.324462890625) ?  (b('l8b11') <= 3020.302978515625) ?  -0.008414807082438386 :   -0.03132201441142761 :   (b('dg0_v_vv') <= -0.4707350730895996) ?  -0.012400313673540664 :   -0.001060324883120223 :   (b('dg0_v_vv') <= 6.14589786529541) ?  (b('l8b4') <= 1703.2725830078125) ?  -0.013580644788767688 :   -0.03098269099504577 :   0.013918320113100235 :   (b('ndvi') <= 1489.0) ?  (b('dg0_v_vv') <= 0.10370635986328125) ?  0.00782620019315955 :   (b('dg0_v_vv') <= 3.2554237842559814) ?  0.03981971260558875 :   0.02154960566311878 :   (b('dg0_v_vv') <= -0.5094666481018066) ?  (b('sand') <= 56.0) ?  0.00031233055586300795 :   -0.013209837434910193 :   (b('dg0_v_vv') <= 2.0474164485931396) ?  -0.01169125795390659 :   -0.00697999648340307 :   (b('l8b5') <= 3810.9488525390625) ?  (b('dg0_v_vv') <= -0.1342449188232422) ?  (b('ndvi') <= 3383.5) ?  (b('ndvi') <= 1850.5) ?  (b('ndvi') <= 1188.0) ?  -0.01587466038794172 :   -0.00019988192965365932 :   (b('ndvi') <= 2020.5) ?  0.015085132330882434 :   -0.009825654461421703 :   -0.053006354260436844 :   (b('ndvi') <= 1166.0) ?  (b('dg0_v_vv') <= 1.7103970050811768) ?  (b('ndvi') <= 975.0) ?  0.020159611783386808 :   -0.004101054592156417 :   -0.0383804512171432 :   (b('ndvi') <= 1621.5) ?  (b('l8b5_med') <= 2812.056640625) ?  0.10618539073274072 :   0.037063384744206486 :   (b('grass') <= 2.951516330242157) ?  0.0017734775770317007 :   0.040533601001871464 :   (b('ndvi') <= 1308.0) ?  (b('dg0_v_vv') <= 0.5904641151428223) ?  (b('dg0_v_vv') <= -0.9526181221008301) ?  (b('dg0_v_vv') <= -1.6005973815917969) ?  -0.001582196990452693 :   0.031242882158396715 :   (b('l8b4_med') <= 1649.88818359375) ?  0.03105554790939978 :   -0.016117266282609538 :   (b('dg0_v_vv') <= 0.8544139862060547) ?  0.08781569589587448 :   (b('ndvi') <= 1150.0) ?  0.07196621225389224 :   -0.012543638591916637 :   (b('ndvi') <= 2349.0) ?  (b('dg0_v_vv') <= -0.7264652252197266) ?  (b('l8b11') <= 3379.2056884765625) ?  -0.015359212785860795 :   0.0016129279973006588 :   (b('l8b11') <= 3524.5162353515625) ?  -0.010389964164100431 :   -0.015925262196435166 :   (b('l8b11_med') <= 2393.7314453125) ?  -0.024625631410680957 :   (b('dg0_v_vv') <= 2.3036751747131348) ?  -0.0440073432586261 :   -0.03012212126666952 :   (b('dg0_v_vv') <= 3.310513496398926) ?  (b('l8b4') <= 376.59197998046875) ?  (b('dg0_v_vv') <= 3.243870735168457) ?  (b('l8b11') <= 813.6185607910156) ?  -0.01851368293089889 :   -0.020673000853888568 :   -0.024132665624881602 :   -0.055320730378496163 :   (b('dg0_v_vv') <= 4.161898851394653) ?  (b('l8b11') <= 813.6185607910156) ?  0.013886322379874971 :   (b('dg0_v_vv') <= 3.6274404525756836) ?  (b('ndvi_med') <= 3528.25) ?  -0.019130127979054057 :   -0.021648657301766905 :   (b('l8b11') <= 880.9291076660156) ?  (b('ndvi_med') <= 3526.0) ?  -0.008648653308255705 :   -0.003648658076627287 :   0.010569878565535928 :   (b('ndvi_med') <= 3528.25) ?  -0.0076486512816977825 :   -0.0483207310937519 :   (b('grass') <= 6.00243353843689) ?  (b('l8b4') <= 1458.2232055664062) ?  (b('l8b4') <= 1383.240234375) ?  (b('ndvi') <= 4065.5) ?  (b('ndvi') <= 3781.5) ?  (b('ndvi') <= 3249.5) ?  (b('l8b11') <= 1135.8286743164062) ?  (b('dg0_v_vv') <= -0.1168675422668457) ?  0.004350943194615795 :   0.033748520668635894 :   (b('l8b11') <= 1545.1495971679688) ?  -0.04623161311004503 :   -0.009597878230019322 :   (b('l8b5_med') <= 3465.2454833984375) ?  (b('sand') <= 50.5) ?  -0.004833370841603579 :   -0.04272394728585413 :   (b('dg0_v_vv') <= 1.4553184509277344) ?  0.02932232800175394 :   0.008717980979575362 :   (b('dg0_v_vv') <= 4.637782573699951) ?  (b('ndvi') <= 3967.0) ?  (b('ndvi_med') <= 2171.5) ?  -0.027241781988041485 :   -0.022277734169328944 :   (b('l8b4_med') <= 1110.7024536132812) ?  -0.02473523002434224 :   -0.03295485379933858 :   -0.002876410416025821 :   (b('ndvi') <= 4139.5) ?  (b('crops') <= 86.4554328918457) ?  0.08373099212102036 :   0.02176572586033637 :   (b('ndvi') <= 4434.0) ?  (b('l8b5_med') <= 3694.775634765625) ?  (b('dg0_v_vv') <= 0.837127685546875) ?  -0.004430307607102196 :   -0.019309039919447515 :   -0.04083654547063445 :   (b('l8b5_med') <= 3694.775634765625) ?  (b('l8b4') <= 417.3483123779297) ?  -0.01601396804288634 :   0.008748183815940313 :   (b('l8b5') <= 4262.469970703125) ?  -0.02556527759551122 :   0.040437937490155756 :   (b('l8b11_med') <= 2391.52294921875) ?  (b('l8b4_med') <= 1530.0742797851562) ?  -0.05505420263796784 :   -0.0749874897731948 :   (b('dg0_v_vv') <= 2.3234286308288574) ?  (b('ndvi') <= 1714.0) ?  -0.0007462579366775551 :   (b('dg0_v_vv') <= 0.3803591728210449) ?  (b('ndvi_med') <= 1433.0) ?  -0.014842964347934366 :   -0.011866392769179099 :   (b('dg0_v_vv') <= 1.3072166442871094) ?  -0.02254379224937977 :   -0.014467123900391126 :   (b('dg0_v_vv') <= 4.019113540649414) ?  (b('dg0_v_vv') <= 2.939363718032837) ?  (b('l8b5') <= 3576.530029296875) ?  -0.028812329203794063 :   -0.02156933283536653 :   (b('grass') <= 4.932267427444458) ?  -0.02991648641494393 :   -0.0241559107868139 :   -0.013490955661003171 :   (b('dg0_v_vv') <= 1.1498312950134277) ?  (b('l8b4') <= 1568.0099487304688) ?  (b('l8b11') <= 2349.7584228515625) ?  (b('moss') <= 6.636433839797974) ?  (b('dg0_v_vv') <= -0.07025814056396484) ?  (b('l8b11') <= 2215.5) ?  -0.020849332883684665 :   -0.01066877549846277 :   -0.030863943422113793 :   (b('dg0_v_vv') <= -0.11398553848266602) ?  (b('ndvi_med') <= 1433.0) ?  0.005005009112292479 :   -0.0013607113154020523 :   (b('dg0_v_vv') <= 0.38053274154663086) ?  0.007753485201724197 :   0.009730784747306445 :   (b('dg0_v_vv') <= -1.3519034385681152) ?  (b('l8b11_med') <= 2404.2833251953125) ?  0.09713215781841508 :   0.06493879381445354 :   (b('l8b11_med') <= 2331.32958984375) ?  (b('l8b11') <= 2416.3089599609375) ?  0.023439663290220097 :   -0.012583741439186075 :   (b('ndvi_med') <= 2171.5) ?  0.013907251309141 :   0.02550228059583976 :   (b('l8b5_med') <= 2860.7547607421875) ?  (b('l8b4_med') <= 1080.966064453125) ?  -0.056337956912306905 :   -0.05051845600964382 :   (b('dg0_v_vv') <= -4.997072219848633) ?  (b('l8b4') <= 2221.239013671875) ?  -0.04867558900244122 :   -0.01679342737833567 :   (b('dg0_v_vv') <= -4.269365310668945) ?  (b('dg0_v_vv') <= -4.574844837188721) ?  0.0253244082259428 :   0.007395862048611153 :   (b('l8b5') <= 3065.0) ?  -0.013210708282011983 :   -0.006657245864650755 :   (b('crops') <= 84.95151138305664) ?  (b('l8b5') <= 3642.3660888671875) ?  (b('l8b11') <= 2661.1748046875) ?  (b('l8b4') <= 1844.08056640625) ?  (b('l8b11') <= 2395.95068359375) ?  -0.014452632057598261 :   -0.043511398268698646 :   (b('dg0_v_vv') <= 2.293701410293579) ?  -0.005562407675540289 :   0.006146281285263139 :   (b('dg0_v_vv') <= 1.944312572479248) ?  (b('l8b11') <= 2748.1201171875) ?  0.032258150913862 :   0.013480475826087347 :   (b('l8b4') <= 2363.56787109375) ?  0.0018845944375318285 :   -0.011959178446395328 :   (b('l8b4') <= 1545.7931518554688) ?  (b('dg0_v_vv') <= 2.7753641605377197) ?  -0.010417768167460172 :   -0.01706633137838432 :   (b('ndvi') <= 3269.5) ?  (b('sand') <= 50.0) ?  0.01635941389887038 :   0.04096710239633704 :   -0.02485277874112081 :   0.09035381437394832 :   (b('l8b5_med') <= 2628.0831298828125) ?  (b('ndvi_med') <= 2800.5) ?  (b('l8b4') <= 436.3854064941406) ?  (b('grass') <= 36.75101661682129) ?  (b('ndvi') <= 2908.0) ?  (b('moss') <= 10.874892711639404) ?  -0.023661016327756634 :   (b('dg0_v_vv') <= -0.2607579231262207) ?  0.02083469899167989 :   0.01671960800369049 :   (b('ndvi') <= 3910.5) ?  (b('moss') <= 16.253870010375977) ?  0.09348826341638769 :   0.06081780623087636 :   (b('l8b4_med') <= 1051.4193420410156) ?  0.038852931164887305 :   0.03074304025936045 :   (b('l8b11_med') <= 1321.5604248046875) ?  (b('l8b5') <= 2361.714111328125) ?  (b('l8b5') <= 2308.7841796875) ?  -0.0018793647884198603 :   -0.06862450269447527 :   (b('l8b4_med') <= 723.224853515625) ?  0.03626824901882675 :   0.057087481571241716 :   (b('ndvi_med') <= 1775.75) ?  (b('dg0_v_vv') <= 2.6614389419555664) ?  -0.015783796323098218 :   0.0161011154736517 :   (b('l8b5') <= 3783.885009765625) ?  0.0386920512943557 :   -0.012358528249590994 :   (b('ndvi') <= 3139.5) ?  (b('ndvi') <= 2598.5) ?  (b('l8b5_med') <= 2559.298095703125) ?  (b('l8b4_med') <= 1106.25390625) ?  -0.003954566393815085 :   0.00022362517166705652 :   (b('dg0_v_vv') <= 2.386442184448242) ?  -0.009392809219296543 :   0.030864747782270382 :   (b('dg0_v_vv') <= 4.981627464294434) ?  (b('ndvi') <= 2635.0) ?  0.023828943543598817 :   0.004014231544291473 :   (b('l8b11') <= 1808.9478149414062) ?  -0.05252156241415427 :   -0.07688228063374386 :   (b('dg0_v_vv') <= 2.6015195846557617) ?  (b('ndvi') <= 4976.0) ?  (b('sand') <= 40.5) ?  -0.006944782160448847 :   -0.020955346143171943 :   (b('l8b11_med') <= 2170.646484375) ?  0.08534874657557318 :   0.05473249888660375 :   (b('l8b4_med') <= 807.3660278320312) ?  0.10527511083888513 :   (b('l8b11') <= 1274.5838623046875) ?  -0.019140216567158044 :   0.013893470377089273 :   (b('dg0_v_vv') <= 3.6407110691070557) ?  (b('l8b11') <= 1054.0424194335938) ?  (b('l8b5') <= 1665.8218383789062) ?  (b('l8b11_med') <= 1713.7119750976562) ?  (b('dg0_v_vv') <= -1.7411727905273438) ?  -0.013113015735068387 :   0.016753265153417088 :   0.10955409337861996 :   (b('sand') <= 29.5) ?  (b('dg0_v_vv') <= 0.9674715995788574) ?  0.04458374094510533 :   -0.024729115521854 :   (b('l8b4') <= 692.823974609375) ?  -0.00859955325781258 :   0.09882352338433831 :   (b('l8b5') <= 2164.8582763671875) ?  (b('bulk') <= 153.0) ?  (b('l8b5') <= 2099.23193359375) ?  0.005079712032743282 :   0.02312277082144156 :   (b('l8b11') <= 1591.9229125976562) ?  0.1411703207316533 :   0.05362483761502507 :   (b('crops') <= 17.23774480819702) ?  (b('l8b5') <= 2939.7374267578125) ?  -0.0009314699924208848 :   -0.030693827070429395 :   (b('dg0_v_vv') <= -1.4504990577697754) ?  -0.0023352801303815713 :   0.015631680837520052 :   (b('ndvi_med') <= 2865.25) ?  (b('sand') <= 33.5) ?  -0.06640548192845902 :   -0.08229887932740325 :   (b('l8b4_med') <= 655.121826171875) ?  (b('l8b11_med') <= 1055.9453735351562) ?  (b('l8b4') <= 669.9734191894531) ?  -0.013449674317318597 :   -0.0037493844031119272 :   (b('dg0_v_vv') <= 6.612760066986084) ?  0.008679542414652519 :   0.014398045166703621 :   (b('l8b11') <= 1719.6438598632812) ?  (b('l8b11') <= 1503.3125) ?  -0.020230583067670532 :   0.008126561704380267 :   (b('ndvi_med') <= 4323.0) ?  -0.034986185604109 :   -0.04291691497543845 :   (b('dg0_v_vv') <= -1.307790756225586) ?  (b('dg0_v_vv') <= -1.3189201354980469) ?  (b('ndvi_med') <= 4697.0) ?  (b('moss') <= 0.5962881445884705) ?  (b('l8b5_med') <= 2810.30615234375) ?  (b('ndvi') <= 4057.0) ?  0.00997075917417126 :   0.07035093426513875 :   (b('ndvi_med') <= 3955.25) ?  -0.005400683411358859 :   0.021465512662204797 :   (b('grass') <= 68.83118057250977) ?  (b('moss') <= 1.9722681045532227) ?  -0.014789938375145355 :   -0.002044055472219496 :   (b('l8b5') <= 2710.5953369140625) ?  -0.032361736136683986 :   -0.009893071512426747 :   0.0772873208394082 :   (b('sand') <= 39.0) ?  (b('bulk') <= 139.0) ?  (b('l8b5') <= 2800.60986328125) ?  (b('bulk') <= 137.0) ?  -0.022595696368571276 :   -0.010026160809893811 :   (b('l8b5_med') <= 2752.140625) ?  0.00744741180140869 :   0.0002543990908511924 :   (b('ndvi') <= 4221.0) ?  (b('moss') <= 5.266067028045654) ?  -0.0377309191445892 :   -0.030989549863699867 :   -0.009019777383487693 :   (b('dg0_v_vv') <= -1.3163228034973145) ?  (b('l8b5') <= 2889.6761474609375) ?  -0.06736402423026921 :   -0.06640331701353241 :   (b('ndvi') <= 3149.5) ?  (b('dg0_v_vv') <= -1.3124608993530273) ?  -0.0408511283104748 :   -0.04150028826793997 :   -0.029743469092705338 :   (b('bulk') <= 140.5) ?  (b('l8b11_med') <= 1894.8837890625) ?  (b('grass') <= 51.24658203125) ?  (b('l8b5') <= 5387.126220703125) ?  (b('l8b11') <= 1294.25) ?  -0.0027147009484085106 :   0.0006912244578825464 :   (b('l8b11_med') <= 1620.2099609375) ?  0.02917463620596994 :   0.0023309410974784914 :   (b('l8b11') <= 2035.4085693359375) ?  (b('l8b11_med') <= 1531.4527587890625) ?  0.006831297993874416 :   -0.018647144298591045 :   (b('ndvi_med') <= 3174.0) ?  0.1035591450572397 :   0.007933243818974922 :   (b('moss') <= 18.54157543182373) ?  (b('ndvi') <= 4495.0) ?  (b('bulk') <= 131.0) ?  0.039200283448215904 :   0.01549073374085498 :   (b('l8b5_med') <= 3042.8221435546875) ?  -0.027102765143405885 :   -0.005514081687900643 :   (b('ndvi') <= 907.0) ?  (b('ndvi_med') <= 1115.0) ?  0.022617904888406638 :   0.03513523048763445 :   (b('dg0_v_vv') <= 1.166393756866455) ?  -0.011654102769428536 :   -0.06946034952844399 :   (b('sand') <= 60.5) ?  (b('l8b5_med') <= 2975.0513916015625) ?  (b('grass') <= 74.79342651367188) ?  (b('l8b4') <= 1962.9209594726562) ?  0.012201580596143602 :   -0.004409513230054478 :   (b('grass') <= 84.99545288085938) ?  -0.002942498689606933 :   0.015014963825779241 :   (b('l8b5_med') <= 3193.2320556640625) ?  (b('l8b5') <= 1696.9679565429688) ?  0.03362797428062519 :   6.952203748159139e-05 :   (b('dg0_v_vv') <= 1.5259108543395996) ?  0.00436590672187491 :   0.0193445564732528 :   (b('l8b4') <= 620.0364685058594) ?  (b('dg0_v_vv') <= -0.5404253005981445) ?  (b('ndvi') <= 3691.5) ?  0.009324081319545374 :   -0.01059502736824891 :   (b('l8b4_med') <= 893.1599426269531) ?  -0.0607121752360075 :   -0.03388602403038943 :   (b('bulk') <= 145.0) ?  (b('l8b5_med') <= 2659.2315673828125) ?  0.021508343445317265 :   -0.022023778873801497 :   (b('ndvi') <= 1454.0) ?  -0.0125761291281396 :   0.004592242747966188 :   (b('crops') <= 20.952744483947754) ?  (b('l8b4') <= 1563.3829956054688) ?  (b('l8b4') <= 1550.3447875976562) ?  (b('l8b5_med') <= 2751.8204345703125) ?  (b('dg0_v_vv') <= -0.8485808372497559) ?  (b('dg0_v_vv') <= -1.0395402908325195) ?  0.01873924972238747 :   0.0013488760438633285 :   (b('dg0_v_vv') <= 0.598233699798584) ?  (b('l8b5') <= 2157.1945190429688) ?  -0.045640290894097324 :   (b('ndvi') <= 1395.0) ?  -0.025519929025719285 :   (b('ndvi') <= 1734.0) ?  -0.022664588741298265 :   -0.024165881791963266 :   (b('l8b11_med') <= 3149.922119140625) ?  (b('l8b5') <= 2625.291748046875) ?  -0.033837991413905796 :   (b('ndvi') <= 1788.0) ?  -0.0011106020304300912 :   -0.012109380291287733 :   0.023382180976918743 :   (b('dg0_v_vv') <= -0.08347177505493164) ?  (b('l8b5') <= 2691.865234375) ?  (b('moss') <= 44.24417304992676) ?  (b('ndvi') <= 3110.0) ?  -0.023981775282135437 :   -0.020199021060395922 :   -0.006488874879724153 :   (b('ndvi_med') <= 2041.5) ?  (b('l8b5') <= 2975.0670166015625) ?  (b('dg0_v_vv') <= -0.3589439392089844) ?  0.018836718544910827 :   0.017246803287028746 :   0.00571757485112246 :   (b('dg0_v_vv') <= -1.5531888008117676) ?  (b('dg0_v_vv') <= -1.8181939125061035) ?  0.008145210816796315 :   0.012145211472447407 :   (b('dg0_v_vv') <= -1.0468735694885254) ?  -0.006346784535669052 :   -0.0011616780134024382 :   (b('dg0_v_vv') <= 0.21920347213745117) ?  (b('ndvi_med') <= 1838.75) ?  (b('l8b5_med') <= 2760.2227783203125) ?  0.058900102042987984 :   (b('l8b4') <= 1307.658203125) ?  0.035367898085647845 :   0.04254169771488295 :   0.009567098881536451 :   (b('l8b4') <= 1446.181396484375) ?  (b('l8b11') <= 1842.5081176757812) ?  (b('l8b5') <= 2445.46923828125) ?  -0.0016482857661715006 :   -0.010772019968980917 :   (b('dg0_v_vv') <= 0.677736759185791) ?  -0.007117727850335383 :   0.012095381143991077 :   (b('grass') <= 46.460768699645996) ?  (b('l8b11_med') <= 3566.801025390625) ?  0.01993878691270992 :   0.028364393181457496 :   0.048885876939622847 :   (b('l8b5_med') <= 2723.5404052734375) ?  0.005873655804527506 :   (b('l8b4') <= 1555.5641479492188) ?  0.06651273812879398 :   (b('dg0_v_vv') <= 0.12413406372070312) ?  0.035332533072115646 :   0.04457990844485246 :   (b('ndvi') <= 795.5) ?  (b('l8b4') <= 2701.295654296875) ?  (b('ndvi') <= 641.0) ?  (b('dg0_v_vv') <= -0.2941117286682129) ?  0.024010217239527815 :   (b('moss') <= 21.173986434936523) ?  (b('l8b11') <= 3197.1829833984375) ?  0.05415680181776247 :   0.05560223636690548 :   0.03465757732074108 :   (b('l8b11') <= 2528.6842041015625) ?  -0.03778316388583547 :   (b('l8b11') <= 3205.3304443359375) ?  (b('dg0_v_vv') <= -0.6943836212158203) ?  (b('ndvi') <= 761.5) ?  0.004050416834789461 :   0.008394544412451063 :   (b('bulk') <= 150.0) ?  0.030852101617940268 :   0.020549060394709114 :   (b('l8b5_med') <= 3253.019775390625) ?  (b('l8b4_med') <= 2433.415283203125) ?  0.011636721863924782 :   -0.013203575485423094 :   (b('l8b4') <= 2558.9854736328125) ?  0.0036538754904004017 :   0.017383658559502066 :   (b('l8b11') <= 3466.0308837890625) ?  (b('dg0_v_vv') <= -0.2994260787963867) ?  (b('ndvi_med') <= 669.0) ?  (b('grass') <= 11.482731103897095) ?  -0.012980924240686309 :   -0.019587262071554506 :   (b('ndvi') <= 683.0) ?  -0.03038626682792854 :   -0.036812485364320566 :   (b('l8b5') <= 3131.1619873046875) ?  -0.026781085477957753 :   (b('l8b5_med') <= 3487.977783203125) ?  (b('l8b11') <= 3375.15966796875) ?  -0.0020295073571959393 :   -0.00816715234764504 :   0.011110329270487454 :   (b('bulk') <= 154.5) ?  (b('ndvi') <= 734.0) ?  (b('l8b11') <= 3534.6541748046875) ?  (b('dg0_v_vv') <= -0.13272857666015625) ?  -0.013539234446319104 :   0.015543940471560103 :   (b('l8b5') <= 3297.410888671875) ?  -0.01580242891550518 :   -0.005645839992986637 :   (b('l8b5') <= 3496.88037109375) ?  0.03026560826932051 :   0.03617152966927155 :   (b('l8b5') <= 4509.23779296875) ?  (b('l8b5_med') <= 3841.5079345703125) ?  0.02306170914166382 :   0.05136585524823893 :   -0.008013703969696487 :   (b('l8b11') <= 3409.915771484375) ?  (b('l8b11') <= 3184.1644287109375) ?  (b('dg0_v_vv') <= 0.04145002365112305) ?  (b('ndvi_med') <= 1147.5) ?  (b('l8b11') <= 2687.8360595703125) ?  (b('sand') <= 62.5) ?  0.01575894674013334 :   -0.01904536612948529 :   (b('ndvi') <= 1337.5) ?  -0.024883145809326455 :   0.0033115189358952357 :   (b('dg0_v_vv') <= -1.0844097137451172) ?  (b('ndvi_med') <= 1472.0) ?  0.039108361663192974 :   0.00280671843177763 :   (b('l8b4') <= 1635.1865234375) ?  -0.025853769177786135 :   -0.0026784991751932687 :   (b('ndvi') <= 1812.5) ?  (b('dg0_v_vv') <= 0.7937045097351074) ?  (b('sand') <= 60.0) ?  0.0109118357277478 :   -0.009319272826782443 :   (b('sand') <= 58.5) ?  0.017276357377358234 :   0.047257995013529064 :   (b('ndvi') <= 3213.0) ?  (b('l8b11') <= 3000.0540771484375) ?  -0.010584825041572219 :   0.007543551994354253 :   0.05009545564252957 :   (b('ndvi_med') <= 923.25) ?  (b('ndvi') <= 1024.5) ?  (b('ndvi') <= 830.5) ?  0.039687295945187134 :   (b('ndvi') <= 901.5) ?  -0.023100399415940272 :   -0.013026328887268197 :   (b('l8b11') <= 3245.5548095703125) ?  (b('l8b5_med') <= 3311.569580078125) ?  0.012364746372312024 :   0.03431339017573072 :   (b('moss') <= 14.68419075012207) ?  0.07280072686604724 :   0.050789311818480024 :   (b('l8b5') <= 3157.0013427734375) ?  (b('l8b4_med') <= 1555.2457885742188) ?  (b('l8b11') <= 3206.2708740234375) ?  -0.022638400013924836 :   0.007060440612081107 :   (b('dg0_v_vv') <= 0.4788050651550293) ?  -0.019629889724188682 :   -0.04106384966615324 :   (b('l8b4_med') <= 2785.3529052734375) ?  (b('l8b5') <= 3175.3231201171875) ?  0.01865427841372766 :   -0.008891324989099529 :   (b('l8b5') <= 3501.254638671875) ?  -0.023553589989051212 :   -0.010284717934638242 :   (b('l8b11') <= 3421.012939453125) ?  (b('grass') <= 51.611270904541016) ?  (b('l8b4_med') <= 2180.6549072265625) ?  0.06647444776300804 :   0.060616057485394594 :   (b('l8b4') <= 1981.7764282226562) ?  (b('l8b5_med') <= 2941.242431640625) ?  (b('l8b5_med') <= 2867.978515625) ?  0.03601586983345145 :   0.03764449296072522 :   0.023277864348185906 :   (b('l8b5_med') <= 2867.978515625) ?  0.009719520395655878 :   0.004319160738785785 :   (b('l8b5_med') <= 3673.7098388671875) ?  (b('dg0_v_vv') <= -0.04262113571166992) ?  (b('ndvi') <= 946.0) ?  (b('ndvi') <= 875.0) ?  0.02941906834241975 :   -0.00744483958577449 :   (b('ndvi') <= 956.0) ?  0.08557378219053939 :   0.015077541603448647 :   (b('l8b5') <= 3770.096435546875) ?  (b('sand') <= 52.5) ?  0.015693091700037545 :   -0.017124038338551285 :   (b('l8b5_med') <= 3483.153076171875) ?  0.044935885606676634 :   0.012923478869017823 :   (b('dg0_v_vv') <= -1.653073787689209) ?  (b('ndvi') <= 1067.5) ?  (b('ndvi') <= 921.0) ?  -0.030799603024205174 :   -0.02316027738244064 :   (b('ndvi_med') <= 2041.5) ?  -0.01207516506443671 :   -0.0005605350611081495 :   (b('dg0_v_vv') <= -1.6274065971374512) ?  0.05328196172758236 :   (b('sand') <= 58.0) ?  -0.00161725338394598 :   -0.006841719218523124 :   (b('dg0_v_vv') <= -0.25335264205932617) ?  (b('l8b11') <= 3241.8572998046875) ?  (b('l8b11') <= 3237.544921875) ?  (b('l8b5_med') <= 3322.7132568359375) ?  (b('l8b11') <= 2959.348388671875) ?  (b('l8b4') <= 1900.8579711914062) ?  (b('l8b4') <= 1739.2147827148438) ?  (b('l8b4') <= 1104.1505126953125) ?  -0.035047918164270164 :   -0.009996476660322124 :   (b('l8b11') <= 2569.140869140625) ?  0.032071934828545534 :   0.004923235719678996 :   (b('ndvi_med') <= 1526.5) ?  (b('l8b4_med') <= 1935.1727294921875) ?  -0.01629257641009604 :   -0.0199853572988968 :   (b('l8b5') <= 3345.30810546875) ?  -0.04448824646903105 :   -0.003250633682571072 :   (b('l8b11') <= 2973.7021484375) ?  (b('dg0_v_vv') <= -1.692986011505127) ?  0.0370252075113144 :   0.06584856623337457 :   (b('ndvi') <= 2989.0) ?  (b('ndvi') <= 1647.0) ?  -0.0007494491157438901 :   -0.02076278807798341 :   (b('l8b4_med') <= 1935.1727294921875) ?  0.0261405798226077 :   0.030711719628095893 :   (b('dg0_v_vv') <= -2.2607083320617676) ?  0.045295519451958866 :   (b('ndvi') <= 1470.5) ?  (b('dg0_v_vv') <= -1.2226366996765137) ?  (b('l8b5') <= 3046.5684814453125) ?  -0.006494214036047645 :   -0.020590116076922224 :   (b('dg0_v_vv') <= -1.1759076118469238) ?  0.02263955852703812 :   -0.00041509160335775723 :   (b('l8b11') <= 2728.962646484375) ?  (b('l8b4') <= 1346.517333984375) ?  0.0003852689064489309 :   -0.021390328361465434 :   (b('l8b5') <= 3394.2103271484375) ?  -0.044772781301885875 :   -0.05240371515467153 :   (b('crops') <= 41.361019134521484) ?  (b('dg0_v_vv') <= -0.3648838996887207) ?  (b('ndvi_med') <= 1569.0) ?  (b('dg0_v_vv') <= -1.0338501930236816) ?  -0.05478294485756605 :   -0.05935242425630977 :   (b('dg0_v_vv') <= -0.45551633834838867) ?  -0.05199240255496975 :   -0.052707659192878446 :   -0.06586035101438258 :   -0.016908485019083513 :   (b('l8b11') <= 3245.884033203125) ?  (b('dg0_v_vv') <= -1.8430914878845215) ?  0.03310698684900852 :   (b('ndvi_med') <= 1569.0) ?  0.0424101559742259 :   0.04386249072090763 :   (b('dg0_v_vv') <= -1.0846214294433594) ?  (b('dg0_v_vv') <= -1.1351361274719238) ?  (b('l8b5') <= 3755.4769287109375) ?  (b('l8b5') <= 3556.417236328125) ?  (b('crops') <= 23.003138542175293) ?  -0.01178490242150435 :   0.00045739504593002794 :   (b('l8b4_med') <= 2079.127685546875) ?  0.026655340569680348 :   0.003552147392264329 :   (b('l8b11_med') <= 3095.8316650390625) ?  (b('ndvi_med') <= 1726.0) ?  -0.001605363921990545 :   0.01230930290636071 :   (b('dg0_v_vv') <= -1.2811980247497559) ?  -0.016214240472933475 :   -0.037959539890646536 :   (b('l8b5') <= 3635.32666015625) ?  (b('l8b5_med') <= 3273.5203857421875) ?  0.0417862647152636 :   0.055601197710393305 :   0.02045867291770915 :   (b('ndvi') <= 1434.5) ?  (b('l8b11') <= 4384.247314453125) ?  (b('ndvi') <= 1294.5) ?  (b('ndvi_med') <= 1482.0) ?  -0.01813172646899966 :   -0.006870611478295616 :   (b('ndvi_med') <= 1588.25) ?  -0.02428016929922594 :   -0.016655001411088777 :   0.025854812030509898 :   (b('ndvi') <= 1461.0) ?  (b('dg0_v_vv') <= -0.651817798614502) ?  0.031728093003714056 :   (b('dg0_v_vv') <= -0.5960488319396973) ?  0.02272808966585395 :   0.02672809032150504 :   (b('dg0_v_vv') <= -0.37157583236694336) ?  (b('ndvi_med') <= 1716.5) ?  0.0035534328659657575 :   -0.012721985719454503 :   (b('l8b11') <= 3372.4720458984375) ?  -0.05486035107398722 :   -0.018538488240218608 :   (b('ndvi_med') <= 1716.5) ?  (b('l8b4') <= 2417.5198974609375) ?  (b('l8b4_med') <= 2000.5802612304688) ?  (b('l8b11') <= 3719.512451171875) ?  (b('l8b5') <= 3959.427490234375) ?  (b('l8b4') <= 1919.3078002929688) ?  (b('l8b4') <= 1642.6568603515625) ?  -0.010734289485665379 :   0.0031610127138933138 :   (b('ndvi') <= 1125.5) ?  0.001629232140088851 :   -0.013183224900365309 :   (b('ndvi') <= 3842.0) ?  0.05099151340234796 :   0.03157493572528211 :   (b('dg0_v_vv') <= 2.9514966011047363) ?  (b('ndvi_med') <= 1454.25) ?  0.019874927759340008 :   (b('l8b5_med') <= 3217.6090087890625) ?  0.04242871179688973 :   0.03839229369070462 :   -0.04379632040530301 :   (b('ndvi') <= 1171.0) ?  (b('sand') <= 47.0) ?  (b('l8b5') <= 3010.4071044921875) ?  (b('dg0_v_vv') <= -0.03432273864746094) ?  0.005820861345921996 :   -0.018330890801961902 :   (b('l8b11_med') <= 3940.4189453125) ?  0.02193099755749301 :   0.006488011451262855 :   (b('dg0_v_vv') <= 2.0480966567993164) ?  (b('l8b4') <= 1827.195556640625) ?  0.03219596845591377 :   0.04560518220889723 :   0.018433467877846083 :   (b('ndvi') <= 1543.0) ?  (b('ndvi') <= 1418.0) ?  (b('ndvi') <= 1404.5) ?  0.00016826184228991038 :   0.06009304722574123 :   (b('ndvi') <= 1523.0) ?  -0.013685100680510705 :   -0.04058616566160128 :   (b('ndvi_med') <= 1569.0) ?  (b('ndvi') <= 1978.0) ?  0.009168770497291383 :   0.032376785585388076 :   (b('ndvi') <= 1864.0) ?  0.003954914534435652 :   -0.012399209161804155 :   (b('l8b4_med') <= 2066.49853515625) ?  (b('ndvi') <= 1659.0) ?  -0.04311398565906471 :   -0.03881331159704363 :   (b('ndvi_med') <= 1615.0) ?  (b('dg0_v_vv') <= 0.05075979232788086) ?  -0.013678295019580552 :   -0.0160569432696981 :   -0.02155954419497636 :   (b('l8b4') <= 1802.812255859375) ?  (b('l8b4') <= 1758.2662963867188) ?  (b('l8b5') <= 3295.69287109375) ?  (b('l8b5') <= 2880.1114501953125) ?  (b('ndvi') <= 1940.5) ?  (b('l8b11') <= 2104.875) ?  0.006867961043213466 :   -0.008117907297317959 :   (b('l8b5') <= 2458.9447021484375) ?  -0.0225908079810299 :   -0.037551145996198396 :   (b('l8b11') <= 2759.8077392578125) ?  (b('ndvi') <= 2987.0) ?  -0.0010817750368703146 :   0.024402681642566537 :   (b('dg0_v_vv') <= 0.16545915603637695) ?  0.02116929303570808 :   0.03478491459575275 :   (b('l8b5') <= 3554.354248046875) ?  (b('l8b11_med') <= 3060.6787109375) ?  -0.041466358976926215 :   (b('l8b4') <= 1475.2327270507812) ?  -0.022381612154290544 :   -0.03067177041261646 :   (b('l8b4') <= 1535.4063110351562) ?  (b('l8b11') <= 2663.73583984375) ?  -0.010070805510283496 :   0.005030517805606596 :   (b('dg0_v_vv') <= 1.2929596900939941) ?  -0.007395372484270163 :   -0.023185682435045075 :   (b('dg0_v_vv') <= 1.0096373558044434) ?  (b('l8b4') <= 1796.0977783203125) ?  0.031705444884694095 :   0.048289246448865994 :   (b('dg0_v_vv') <= 1.4572820663452148) ?  0.0028471116806507296 :   (b('l8b4_med') <= 1834.8324584960938) ?  0.01307634650803692 :   0.010608169777463358 :   (b('l8b5') <= 3007.891357421875) ?  (b('l8b11') <= 2936.404296875) ?  -0.06248886702483093 :   (b('l8b11') <= 3412.545166015625) ?  (b('ndvi') <= 1590.5) ?  (b('l8b5_med') <= 3311.2408447265625) ?  -0.018977641178239305 :   -0.02663048580154569 :   (b('dg0_v_vv') <= 0.5878610610961914) ?  -0.033714455179466196 :   -0.02350982898843485 :   (b('l8b4') <= 1913.11962890625) ?  (b('dg0_v_vv') <= 0.2667670249938965) ?  -0.014754250920222797 :   -0.012406444358593165 :   0.0020110164330039634 :   (b('l8b11') <= 3872.7303466796875) ?  (b('l8b11') <= 3012.3997802734375) ?  (b('dg0_v_vv') <= 1.4068021774291992) ?  (b('l8b4') <= 1967.8275756835938) ?  -0.03613206104898478 :   0.02472792626799352 :   (b('dg0_v_vv') <= 1.8829255104064941) ?  0.035670202398985384 :   0.043081273280842186 :   (b('l8b11_med') <= 3091.89697265625) ?  (b('dg0_v_vv') <= 3.2501864433288574) ?  -0.02143788455913378 :   0.010909518980509125 :   (b('l8b11') <= 3654.0010986328125) ?  -0.0017726689430311604 :   -0.01772262736996148 :   (b('l8b4') <= 2085.738037109375) ?  (b('l8b4') <= 2029.9766235351562) ?  (b('ndvi_med') <= 1726.0) ?  0.021128900659112014 :   0.027083132829821048 :   0.014859018583070482 :   (b('dg0_v_vv') <= 0.6959753036499023) ?  (b('l8b11') <= 4155.1024169921875) ?  -0.013512477652428214 :   0.0038033819325399166 :   (b('dg0_v_vv') <= 0.9578104019165039) ?  0.013575714867758427 :   0.008777118280062796 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11') <= 1493.8062133789062) ?  (b('moss') <= 23.9521541595459) ?  (b('l8b11') <= 1433.950439453125) ?  (b('l8b4') <= 1251.79833984375) ?  (b('dg0_v_vv') <= 0.6681528091430664) ?  (b('bulk') <= 166.0) ?  (b('l8b11') <= 1330.2947387695312) ?  (b('ndvi') <= 2084.5) ?  (b('l8b5') <= 1387.0847778320312) ?  (b('grass') <= 3.1480519771575928) ?  0.02965821147379849 :   -0.009589270117512946 :   (b('l8b5') <= 1387.9447021484375) ?  0.1560099511702218 :   0.013171206584859228 :   (b('crops') <= 25.88153839111328) ?  (b('ndvi_med') <= 1465.5) ?  0.04965332801978526 :   -0.0026473300604552665 :   (b('ndvi_med') <= 1462.5) ?  -0.0542268541412701 :   0.003277773236529715 :   (b('l8b11') <= 1332.930908203125) ?  (b('dg0_v_vv') <= -1.0100903511047363) ?  (b('ndvi_med') <= 3405.0) ?  -0.02664464654821949 :   0.012787089791635849 :   (b('l8b4_med') <= 721.4329223632812) ?  -0.03089650030675043 :   -0.04716301874687147 :   (b('l8b5') <= 3559.05419921875) ?  (b('dg0_v_vv') <= -1.9716382026672363) ?  -0.01704586204058805 :   0.0005979233886023328 :   (b('ndvi') <= 3315.0) ?  0.012498350056722814 :   -0.017949554609498746 :   (b('ndvi') <= 4355.5) ?  (b('ndvi_med') <= 1588.5) ?  -0.12534123188732643 :   (b('l8b4_med') <= 1268.47412109375) ?  -0.09236362001835796 :   -0.08315035949585897 :   (b('ndvi') <= 4875.5) ?  (b('dg0_v_vv') <= 0.06370830535888672) ?  (b('dg0_v_vv') <= -0.3936753273010254) ?  -0.05001216243413051 :   0.03337196535410079 :   -0.10611907900188322 :   0.07681387191801586 :   (b('sand') <= 15.5) ?  (b('ndvi') <= 4712.5) ?  (b('l8b5') <= 2568.20458984375) ?  -0.028940973711691242 :   (b('dg0_v_vv') <= 2.4325313568115234) ?  (b('moss') <= 3.9609073996543884) ?  0.060757014505687976 :   0.04007724032933603 :   (b('l8b5') <= 2797.8961181640625) ?  0.0221628823492388 :   -0.00961766364327854 :   (b('dg0_v_vv') <= 0.9479331970214844) ?  (b('ndvi_med') <= 4007.0) ?  0.019797728434136908 :   0.047937481331516596 :   (b('dg0_v_vv') <= 1.2046709060668945) ?  (b('ndvi') <= 4824.5) ?  -0.03331553294090478 :   -0.030848243622897065 :   (b('l8b5') <= 4027.6954345703125) ?  -0.007096630719649 :   0.004872780387385922 :   (b('l8b5') <= 4432.18798828125) ?  (b('bulk') <= 106.5) ?  (b('ndvi_med') <= 4246.75) ?  (b('dg0_v_vv') <= 0.7714519500732422) ?  0.05061283836344929 :   -0.006926850926150796 :   (b('l8b5') <= 3752.0804443359375) ?  -0.08778051543896262 :   0.030453165972796126 :   (b('crops') <= 78.93814468383789) ?  (b('ndvi_med') <= 1651.25) ?  0.015847241190953633 :   0.005124608574674505 :   (b('l8b5') <= 3932.3876953125) ?  -0.0058084764241746875 :   0.015610955234974282 :   (b('sand') <= 78.5) ?  (b('dg0_v_vv') <= 0.8415629863739014) ?  (b('crops') <= 55.39288902282715) ?  -0.042282192951800496 :   -0.06221065709793508 :   (b('sand') <= 71.0) ?  -0.004999688261396336 :   0.012879793052104474 :   (b('dg0_v_vv') <= 1.435110092163086) ?  (b('ndvi_med') <= 3956.75) ?  -0.027981156367941493 :   -0.0495527186750489 :   (b('l8b4_med') <= 714.1749572753906) ?  -0.05773233843656923 :   -0.0486495232923498 :   (b('dg0_v_vv') <= -0.03656911849975586) ?  0.09923375482520524 :   0.06746580472317615 :   (b('bulk') <= 163.5) ?  (b('l8b11') <= 1434.7885131835938) ?  (b('l8b5') <= 3138.8656005859375) ?  (b('ndvi_med') <= 1859.25) ?  0.13315525841747788 :   (b('l8b11_med') <= 1347.2548828125) ?  (b('bulk') <= 140.5) ?  0.10161202237555034 :   0.0911692957815129 :   (b('ndvi') <= 3730.5) ?  (b('l8b5') <= 2507.3570556640625) ?  0.07729262765865325 :   0.07947857474088099 :   0.06096514589943891 :   (b('ndvi_med') <= 3116.0) ?  (b('ndvi') <= 3725.5) ?  (b('dg0_v_vv') <= 1.3162600994110107) ?  0.015619825187583772 :   (b('ndvi') <= 2661.5) ?  0.011878905155858155 :   0.01259588823976468 :   0.025202747280309745 :   (b('dg0_v_vv') <= -0.5021624565124512) ?  0.013936729295623645 :   (b('ndvi') <= 3725.5) ?  0.010042066617044279 :   0.007290602237160493 :   (b('ndvi_med') <= 2425.0) ?  (b('ndvi_med') <= 1426.0) ?  (b('ndvi') <= 2073.0) ?  (b('l8b5') <= 1933.2659912109375) ?  (b('l8b11') <= 1460.1946411132812) ?  -0.027949331139868715 :   0.019064944583717894 :   (b('dg0_v_vv') <= 1.2750911712646484) ?  0.002640901074412602 :   0.041053764987649996 :   -0.07904358360800079 :   (b('ndvi') <= 3046.0) ?  (b('l8b11_med') <= 2235.6070556640625) ?  (b('l8b5_med') <= 3037.1385498046875) ?  0.02653697543324485 :   0.07668800751727198 :   (b('dg0_v_vv') <= 2.3408262729644775) ?  -0.024780658853948465 :   0.03761101499390168 :   (b('ndvi') <= 3781.5) ?  (b('ndvi') <= 3232.5) ?  -0.00019057708017350405 :   -0.023176921330696543 :   (b('dg0_v_vv') <= 0.19235563278198242) ?  -0.002923170000212992 :   0.021303196678872944 :   (b('moss') <= 8.55407428741455) ?  (b('moss') <= 7.656661510467529) ?  (b('ndvi') <= 2815.0) ?  (b('l8b11') <= 1443.3549194335938) ?  -0.014694155777769693 :   0.022920681799044013 :   (b('sand') <= 22.5) ?  -0.03558401211768332 :   0.0003024792863632742 :   (b('l8b4') <= 848.7446899414062) ?  (b('ndvi') <= 4170.0) ?  0.048758032951133186 :   0.0358881975623368 :   (b('dg0_v_vv') <= 1.9428648948669434) ?  0.08250822513907494 :   0.05064467204249254 :   (b('l8b11') <= 1441.4486694335938) ?  (b('dg0_v_vv') <= -0.31536102294921875) ?  (b('dg0_v_vv') <= -0.6229305267333984) ?  0.013220869660463125 :   -0.054352850674563136 :   (b('moss') <= 15.015437602996826) ?  0.04661014885708615 :   -0.005521904722837721 :   (b('l8b11') <= 1459.9105834960938) ?  (b('dg0_v_vv') <= 0.6979193687438965) ?  -0.01381052027973804 :   -0.030652008975945534 :   (b('l8b11') <= 1476.5848388671875) ?  0.008143909288747565 :   -0.00592119816672471 :   (b('ndvi_med') <= 1918.5) ?  -0.125553364084399 :   -0.07204559888359471 :   (b('l8b5') <= 2243.3760986328125) ?  (b('l8b5') <= 2192.171875) ?  (b('sand') <= 34.5) ?  (b('dg0_v_vv') <= -0.9608240127563477) ?  (b('bulk') <= 151.5) ?  0.0492886095715126 :   0.009779905168062958 :   (b('l8b4_med') <= 1036.7967224121094) ?  (b('l8b5') <= 1874.8807373046875) ?  -0.03931535460543906 :   (b('sand') <= 30.5) ?  -0.031028016224809857 :   -0.03435914112603669 :   (b('dg0_v_vv') <= -0.1879134178161621) ?  (b('l8b4_med') <= 1054.0230102539062) ?  -0.03929413115601983 :   (b('dg0_v_vv') <= -0.511559009552002) ?  -0.0077674434666223854 :   0.022370904249598325 :   (b('l8b11_med') <= 2101.1934814453125) ?  (b('l8b4_med') <= 1054.0230102539062) ?  0.024553521339056722 :   0.007442126894843415 :   (b('dg0_v_vv') <= 0.46081018447875977) ?  -0.010162268363794497 :   -0.007795846683295776 :   (b('dg0_v_vv') <= 0.07771730422973633) ?  (b('l8b11') <= 914.976318359375) ?  -0.07708470164348344 :   (b('ndvi') <= 1333.5) ?  -0.06104334177901201 :   (b('l8b11') <= 1478.5972290039062) ?  (b('dg0_v_vv') <= -0.9617648124694824) ?  -0.017286673522179174 :   -0.025131568770504917 :   -0.049986246158803246 :   (b('ndvi_med') <= 2250.25) ?  (b('ndvi_med') <= 2189.25) ?  (b('l8b5') <= 1425.1925048828125) ?  (b('dg0_v_vv') <= 3.3250741958618164) ?  -0.030059573314174397 :   0.0031254778774784386 :   (b('ndvi') <= 2688.0) ?  -0.0007801192808611287 :   0.035626315627902666 :   (b('l8b5') <= 1649.3048706054688) ?  -0.06204713728995717 :   (b('l8b4') <= 649.150146484375) ?  -0.03286826827965712 :   -0.040486452007680154 :   0.036525501982116904 :   (b('ndvi') <= 2619.0) ?  -0.06532086477901883 :   (b('l8b5_med') <= 1728.5043334960938) ?  -0.05980342631489026 :   -0.05489795885134298 :   (b('bulk') <= 155.0) ?  (b('l8b4') <= 775.1505737304688) ?  (b('dg0_v_vv') <= 0.24005556106567383) ?  (b('ndvi') <= 4116.5) ?  (b('l8b4_med') <= 942.4861145019531) ?  (b('l8b11') <= 1356.2846069335938) ?  (b('dg0_v_vv') <= 0.08217573165893555) ?  0.0063169462140953885 :   -0.022375431007513796 :   (b('dg0_v_vv') <= -0.41605186462402344) ?  -0.03274339202272634 :   -0.0028414532996570915 :   0.06915852640761827 :   (b('ndvi') <= 4311.5) ?  (b('l8b5_med') <= 2688.8553466796875) ?  -0.03174848922676354 :   (b('l8b5_med') <= 2818.5750732421875) ?  -0.09211483699192993 :   -0.08639179611603442 :   (b('l8b5_med') <= 2996.2142333984375) ?  -0.012243673507988578 :   0.00985255469193079 :   (b('l8b4') <= 737.4886474609375) ?  (b('l8b11') <= 1000.5578918457031) ?  (b('dg0_v_vv') <= 0.35780811309814453) ?  -0.002714676283359141 :   (b('crops') <= 34.81758117675781) ?  -0.024867691410845574 :   -0.007859027065227175 :   (b('l8b11') <= 1213.6690063476562) ?  (b('l8b4') <= 566.6798400878906) ?  0.014525119011214983 :   0.047221497362330354 :   (b('l8b11') <= 1362.3685913085938) ?  -0.015026587387539819 :   0.021046853994278678 :   (b('ndvi') <= 4031.5) ?  (b('dg0_v_vv') <= 0.5272102355957031) ?  (b('ndvi') <= 3852.5) ?  0.04610612352402671 :   0.04972558720344106 :   (b('l8b4_med') <= 941.1829223632812) ?  0.05482296835655098 :   0.060038610562534234 :   0.08334953925907168 :   (b('l8b11') <= 1485.5615844726562) ?  (b('sand') <= 36.5) ?  (b('ndvi_med') <= 3613.25) ?  (b('l8b11_med') <= 1601.923095703125) ?  (b('dg0_v_vv') <= -0.24918317794799805) ?  -0.02249614923278536 :   -0.03519079451189645 :   (b('l8b4_med') <= 940.2608337402344) ?  -0.014763852434387062 :   -0.020725778383066655 :   (b('ndvi') <= 4353.0) ?  -0.04210487376257868 :   -0.06019091307667815 :   -0.06285931889555084 :   (b('l8b4_med') <= 942.4861145019531) ?  0.027512994071105967 :   0.008622372748922441 :   (b('dg0_v_vv') <= 2.568558692932129) ?  (b('dg0_v_vv') <= 0.04111123085021973) ?  0.038672307228128935 :   (b('sand') <= 39.5) ?  0.06019279179871645 :   0.04186052975805549 :   0.07250720075988776 :   (b('crops') <= 29.484694480895996) ?  (b('bulk') <= 170.0) ?  (b('ndvi') <= 4243.0) ?  (b('l8b11') <= 1669.5470581054688) ?  (b('l8b4') <= 903.7507629394531) ?  (b('l8b11_med') <= 1831.3523559570312) ?  (b('dg0_v_vv') <= 0.45731687545776367) ?  (b('ndvi') <= 3309.0) ?  (b('l8b5') <= 3249.9671630859375) ?  -0.002898492194222151 :   -0.02393526698157251 :   (b('l8b4_med') <= 572.6464538574219) ?  0.08915528754011248 :   0.009649848779567153 :   (b('l8b11_med') <= 1317.275634765625) ?  (b('l8b5') <= 2488.82080078125) ?  0.07712814095532639 :   0.015095097135228627 :   (b('sand') <= 27.5) ?  0.013083312355025838 :   -0.024098994411252243 :   (b('l8b5_med') <= 2195.0802001953125) ?  (b('l8b5_med') <= 1676.6114501953125) ?  0.132307589298173 :   (b('l8b11') <= 1567.29150390625) ?  0.012688137181461164 :   -0.016992754095688737 :   (b('l8b11') <= 1654.0873413085938) ?  (b('l8b11') <= 1648.4369506835938) ?  0.027172058156236457 :   -0.13660080928652032 :   (b('ndvi') <= 3367.5) ?  0.09617905729006707 :   0.04174674154065888 :   (b('l8b4') <= 907.98974609375) ?  (b('l8b4') <= 904.4295654296875) ?  -0.14087211346958972 :   (b('moss') <= 15.034202337265015) ?  -0.0673806000312892 :   -0.05034300873802089 :   (b('ndvi_med') <= 3678.75) ?  (b('moss') <= 20.837696075439453) ?  (b('grass') <= 87.46158599853516) ?  -0.018627430915632282 :   0.002573762094867156 :   (b('dg0_v_vv') <= 0.39745569229125977) ?  -0.014335115686419143 :   0.023292812931306816 :   (b('l8b5') <= 2604.24951171875) ?  (b('crops') <= 6.565573692321777) ?  -0.003622529506160349 :   0.014818783230079347 :   (b('l8b4') <= 976.1496276855469) ?  0.05337785587299132 :   0.0326912806226656 :   (b('l8b4') <= 799.0370178222656) ?  (b('ndvi_med') <= 2348.5) ?  (b('ndvi') <= 2945.5) ?  (b('dg0_v_vv') <= -0.043111324310302734) ?  (b('ndvi_med') <= 2242.0) ?  0.04688532428666325 :   0.01347611103127067 :   (b('l8b5_med') <= 2135.5142822265625) ?  0.04602749983880882 :   0.09030608763665343 :   (b('dg0_v_vv') <= -0.34945201873779297) ?  0.05820466549871148 :   (b('ndvi_med') <= 2233.75) ?  -0.008994191179882927 :   -0.04141377486790393 :   (b('l8b11') <= 1733.0531005859375) ?  (b('l8b11_med') <= 1577.2503662109375) ?  (b('sand') <= 33.5) ?  -0.07427560500855732 :   -0.060768994280994304 :   (b('l8b11_med') <= 1652.9009399414062) ?  0.023735056547408165 :   -0.01174970637682813 :   (b('l8b4') <= 798.5050659179688) ?  (b('dg0_v_vv') <= -0.14843177795410156) ?  -0.026066567614922334 :   0.033607038437339504 :   (b('dg0_v_vv') <= 1.2777061462402344) ?  0.08589899351862551 :   0.06577625266282247 :   (b('l8b4') <= 1469.5585327148438) ?  (b('ndvi_med') <= 1470.0) ?  (b('l8b5') <= 2359.663818359375) ?  (b('grass') <= 57.81917953491211) ?  -0.0037514766689498557 :   0.009034520443316122 :   (b('l8b5') <= 2585.0360107421875) ?  0.03884457509223823 :   -0.0059722876230433085 :   (b('ndvi_med') <= 1475.25) ?  (b('dg0_v_vv') <= -1.4038209915161133) ?  0.015417740919597966 :   -0.057386965220171796 :   (b('crops') <= 26.367992401123047) ?  -0.0012386088992169602 :   -0.012739233084648716 :   (b('bulk') <= 145.5) ?  (b('ndvi') <= 826.0) ?  (b('sand') <= 58.5) ?  -0.004505232391953975 :   0.02833750878487585 :   (b('l8b11_med') <= 2487.0125732421875) ?  -0.011733824994165356 :   -0.004689580521920928 :   (b('bulk') <= 149.5) ?  (b('l8b11') <= 2373.67626953125) ?  0.019932659326686357 :   -0.0011542948667848582 :   (b('ndvi_med') <= 2162.0) ?  -0.003774674723463261 :   0.012216958233022247 :   (b('l8b11_med') <= 1534.2417602539062) ?  (b('l8b5') <= 2742.688720703125) ?  (b('l8b4') <= 791.9805603027344) ?  (b('dg0_v_vv') <= -1.711331844329834) ?  -0.011940004041980617 :   (b('l8b4_med') <= 660.63720703125) ?  (b('moss') <= 8.772436141967773) ?  -0.050615911998229624 :   -0.04247088085697545 :   -0.06440541155049051 :   (b('ndvi_med') <= 3991.0) ?  (b('crops') <= 16.036375522613525) ?  (b('l8b4_med') <= 796.326416015625) ?  -0.013482124765810344 :   -0.01806493192574131 :   -0.0066124471325969825 :   -0.028831428764570732 :   (b('l8b11') <= 1515.7958374023438) ?  (b('l8b11_med') <= 1489.3408813476562) ?  -0.04065447642312506 :   -0.05003470458245031 :   (b('l8b11') <= 1630.0486450195312) ?  (b('l8b11') <= 1550.052490234375) ?  (b('dg0_v_vv') <= -0.4190077781677246) ?  -0.005131090447553777 :   0.013482424388784525 :   (b('ndvi_med') <= 3165.0) ?  0.037341316470619584 :   0.024347214843401094 :   (b('l8b5_med') <= 2773.7271728515625) ?  (b('crops') <= 15.137257099151611) ?  0.014916400999754886 :   0.0367606823896655 :   (b('ndvi') <= 4526.5) ?  -0.016367159821813575 :   0.006081170998593312 :   (b('l8b11_med') <= 1592.1603393554688) ?  (b('l8b5') <= 3598.1453857421875) ?  (b('l8b5_med') <= 2897.135986328125) ?  0.08785191461928815 :   0.055431393939997686 :   0.13044896099648437 :   (b('sand') <= 43.5) ?  (b('l8b5') <= 3234.5487060546875) ?  (b('ndvi') <= 4597.0) ?  (b('l8b11_med') <= 1633.338623046875) ?  -0.017799072086833775 :   0.026014215188807022 :   (b('dg0_v_vv') <= -0.19538307189941406) ?  -0.029182275546608204 :   0.09127637583098104 :   (b('dg0_v_vv') <= 1.2916855812072754) ?  (b('ndvi') <= 4936.5) ?  -0.0020882244254395686 :   -0.023958474893553128 :   0.05287897773802397 :   (b('bulk') <= 137.5) ?  (b('dg0_v_vv') <= -0.8290314674377441) ?  -0.03992492998579264 :   (b('dg0_v_vv') <= 0.09520483016967773) ?  -0.014898601495241352 :   -0.03191563442356363 :   (b('ndvi') <= 4694.5) ?  (b('l8b11_med') <= 1939.7647705078125) ?  0.001295858730641683 :   -0.005551367456462564 :   -0.022879694852615484 :   (b('ndvi') <= 1738.5) ?  (b('ndvi') <= 1539.0) ?  (b('ndvi') <= 1503.5) ?  (b('l8b5') <= 2335.458251953125) ?  -0.024827729713549684 :   -0.016552345357561705 :   (b('l8b4') <= 1703.4373168945312) ?  -0.007038918421624456 :   -0.01425759576055019 :   (b('ndvi') <= 1576.0) ?  (b('l8b4') <= 1738.5360107421875) ?  -0.0239742171497737 :   -0.02481474056337203 :   -0.03147010763971336 :   (b('l8b5') <= 2137.157470703125) ?  -0.08351389635969114 :   (b('l8b5_med') <= 2587.7828369140625) ?  -0.04120475337583143 :   -0.054380101689451454 :   (b('bulk') <= 121.0) ?  (b('l8b5') <= 3204.926513671875) ?  (b('l8b11') <= 1772.4840087890625) ?  (b('dg0_v_vv') <= 1.937044620513916) ?  (b('l8b5_med') <= 1973.9232177734375) ?  (b('dg0_v_vv') <= -0.8044838905334473) ?  (b('ndvi') <= 1215.0) ?  0.01596585439169912 :   0.0028442957881006925 :   (b('dg0_v_vv') <= 0.35985374450683594) ?  0.02672882427458284 :   0.03280619804371282 :   (b('ndvi') <= 2361.0) ?  (b('dg0_v_vv') <= -0.8322386741638184) ?  (b('ndvi') <= 1569.5) ?  0.0023805085627602862 :   0.0004996221637198484 :   -0.016641845000417987 :   (b('dg0_v_vv') <= 0.44628143310546875) ?  (b('dg0_v_vv') <= -1.0232305526733398) ?  0.010207207147595857 :   0.004918523018560762 :   (b('moss') <= 6.335125625133514) ?  0.037540718609822665 :   0.0023882000984147556 :   -0.033026331816375765 :   (b('l8b11') <= 2057.8573608398438) ?  (b('l8b5') <= 2322.909423828125) ?  0.02890308015614526 :   (b('sand') <= 41.5) ?  (b('l8b5') <= 2574.7432861328125) ?  0.05457859729439746 :   0.05571802730984282 :   (b('l8b5') <= 2485.33642578125) ?  0.04064969663258505 :   0.03720899803497357 :   (b('l8b4') <= 1109.5703735351562) ?  (b('l8b5_med') <= 2150.95654296875) ?  0.00660147341560241 :   0.015504425773569724 :   (b('l8b5_med') <= 2482.1885986328125) ?  (b('l8b4_med') <= 828.212158203125) ?  0.030298699649602523 :   (b('l8b5') <= 2239.5466918945312) ?  0.026630594404236485 :   0.027913389223670815 :   (b('l8b5') <= 2499.7435302734375) ?  0.022278483775861224 :   (b('ndvi_med') <= 4525.5) ?  0.0178243516671419 :   0.016791442214875052 :   (b('l8b4') <= 1054.4193725585938) ?  (b('sand') <= 36.5) ?  0.07315230862092276 :   0.052254091699946714 :   0.031202612612438785 :   (b('l8b11') <= 1501.338134765625) ?  (b('l8b5_med') <= 2956.2403564453125) ?  (b('l8b4_med') <= 705.1431579589844) ?  (b('dg0_v_vv') <= -1.3403253555297852) ?  (b('ndvi') <= 3051.5) ?  -0.011059011904439253 :   -0.012763276537604479 :   (b('dg0_v_vv') <= -1.1039814949035645) ?  -0.020084826354777124 :   -0.018435799540706926 :   (b('l8b11_med') <= 1304.4703979492188) ?  (b('dg0_v_vv') <= 0.9671285152435303) ?  -0.018004683548520417 :   -0.0212741839508076 :   (b('l8b5_med') <= 2946.6160888671875) ?  (b('dg0_v_vv') <= 2.1403074264526367) ?  (b('dg0_v_vv') <= 0.36628246307373047) ?  0.00032557358954513337 :   0.009738541811893504 :   (b('dg0_v_vv') <= 2.5966315269470215) ?  -0.015514645037092517 :   -0.003654026171322239 :   -0.015583037877147016 :   (b('l8b5') <= 3022.0928955078125) ?  (b('moss') <= 3.83847975730896) ?  -0.08381797857863538 :   (b('dg0_v_vv') <= 0.5892081260681152) ?  (b('ndvi_med') <= 3073.75) ?  (b('ndvi') <= 3253.5) ?  -0.026443252658501573 :   -0.03289698039631314 :   (b('ndvi') <= 3836.0) ?  -0.042979816993399235 :   -0.036390464977157935 :   (b('ndvi') <= 3563.0) ?  (b('dg0_v_vv') <= 1.1628937721252441) ?  -0.06131968224982273 :   -0.05949772081155108 :   -0.05502024485832677 :   (b('l8b5_med') <= 3068.830810546875) ?  (b('l8b5_med') <= 2957.619873046875) ?  -0.004706040679940737 :   (b('dg0_v_vv') <= -0.3078293800354004) ?  0.008544756138808202 :   0.004869455918598303 :   (b('bulk') <= 130.5) ?  (b('l8b11_med') <= 1222.7625122070312) ?  -0.022262279778115374 :   -0.03349235025649361 :   (b('crops') <= 72.78216934204102) ?  -0.013489266238819048 :   -0.008408093038653447 :   (b('dg0_v_vv') <= -1.7074341773986816) ?  (b('l8b11_med') <= 1198.4691772460938) ?  (b('ndvi') <= 2419.0) ?  (b('ndvi_med') <= 3589.5) ?  (b('crops') <= 59.55902671813965) ?  0.050255094067944706 :   (b('ndvi') <= 2058.0) ?  0.03277729728924583 :   0.03549276589535222 :   0.010221524327193782 :   (b('ndvi_med') <= 3751.75) ?  (b('l8b11') <= 2437.7960205078125) ?  (b('dg0_v_vv') <= -3.632333755493164) ?  -0.03242119031678109 :   -0.009170875597828925 :   0.01981216586023732 :   (b('ndvi') <= 3513.5) ?  (b('l8b11') <= 2702.3275146484375) ?  -0.0025157343431088663 :   -0.000597376010662537 :   (b('ndvi') <= 3576.0) ?  0.020536641165715113 :   0.00842856026104025 :   (b('l8b11') <= 1571.1028442382812) ?  (b('moss') <= 9.62743091583252) ?  (b('dg0_v_vv') <= -2.449122905731201) ?  (b('moss') <= 0.5377201735973358) ?  -0.013359529081349977 :   -0.030388365914369803 :   (b('dg0_v_vv') <= -1.9520692825317383) ?  -0.03821045523319635 :   -0.053268709495716535 :   (b('l8b11_med') <= 1369.6777954101562) ?  0.014288177443203959 :   (b('ndvi') <= 2373.5) ?  -0.018466890269973706 :   -0.005716958659219729 :   (b('l8b11') <= 2771.5279541015625) ?  (b('moss') <= 0.5962881445884705) ?  (b('ndvi') <= 2908.0) ?  -0.001739995728589735 :   0.021184868820792707 :   (b('crops') <= 39.95701217651367) ?  -0.027778727062226394 :   -0.00692238200589469 :   (b('l8b5_med') <= 3657.189697265625) ?  (b('l8b4') <= 2213.4183349609375) ?  0.010409842969768806 :   -0.002214816969223105 :   (b('dg0_v_vv') <= -5.06843376159668) ?  -0.043808030102197096 :   -0.010940100192167049 :   (b('l8b11_med') <= 2935.290283203125) ?  (b('bulk') <= 163.5) ?  (b('crops') <= 40.43350791931152) ?  (b('l8b5') <= 3664.4613037109375) ?  (b('l8b4') <= 1780.528076171875) ?  0.008853885723465117 :   -0.02747353641557744 :   (b('ndvi') <= 3228.0) ?  0.08763832355826857 :   0.03815248632332802 :   (b('l8b11') <= 1653.6478881835938) ?  (b('ndvi') <= 1804.5) ?  0.014616438478659633 :   -0.00597432367440859 :   (b('sand') <= 79.5) ?  0.00161028455015837 :   -0.06728362672674354 :   (b('dg0_v_vv') <= 1.7114605903625488) ?  (b('l8b11') <= 2754.0328369140625) ?  (b('l8b11') <= 1725.1392822265625) ?  -0.018910168584360582 :   -0.007177108034061037 :   (b('dg0_v_vv') <= 0.9013938903808594) ?  -0.021663405217808154 :   -0.04995668837350942 :   0.0813184329365535 :   (b('crops') <= 44.81554985046387) ?  (b('l8b5') <= 3717.3253173828125) ?  (b('ndvi') <= 1858.5) ?  (b('ndvi') <= 1688.5) ?  0.00450956706380602 :   0.03162177565719772 :   (b('ndvi') <= 2018.0) ?  -0.026364436121447857 :   0.00861833478605871 :   (b('ndvi') <= 1550.5) ?  (b('dg0_v_vv') <= -0.32334470748901367) ?  -0.026892059079082417 :   -0.009217924955212184 :   (b('dg0_v_vv') <= -1.051382064819336) ?  -0.011982182124427608 :   -0.029746270331416528 :   (b('ndvi') <= 2121.0) ?  (b('ndvi') <= 2039.5) ?  (b('sand') <= 53.0) ?  -0.006472292847292882 :   -0.02620617645893223 :   (b('l8b5_med') <= 3164.2215576171875) ?  -0.035824449437882586 :   -0.018987880011654464 :   (b('l8b4') <= 2067.833984375) ?  (b('l8b4') <= 1384.6357421875) ?  -0.024243024523014196 :   -0.0010391260163136924 :   (b('l8b5') <= 3566.381591796875) ?  0.020529185744503388 :   0.038864140123762 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi') <= 3205.5) ?  (b('sand') <= 24.5) ?  (b('grass') <= 76.98892974853516) ?  (b('l8b11') <= 2323.7171630859375) ?  (b('l8b4') <= 985.6551513671875) ?  (b('l8b11_med') <= 1478.4027099609375) ?  (b('ndvi_med') <= 2444.0) ?  0.06887346483476603 :   (b('ndvi_med') <= 3316.75) ?  (b('l8b5') <= 2289.0863037109375) ?  (b('l8b11') <= 978.7548828125) ?  0.02673118660322138 :   0.008515477284514704 :   (b('dg0_v_vv') <= 0.45669078826904297) ?  -0.013701917184020751 :   0.012375921809011148 :   (b('l8b11_med') <= 1375.1961059570312) ?  (b('l8b11') <= 1007.0740356445312) ?  0.031900561090516054 :   0.013613582752181807 :   (b('l8b4_med') <= 771.636962890625) ?  0.004734687263269634 :   -0.06644858170436638 :   (b('l8b4') <= 766.7718811035156) ?  (b('crops') <= 30.663869380950928) ?  -0.046352271326454475 :   (b('l8b11') <= 1140.4735717773438) ?  0.04027220380824481 :   (b('ndvi') <= 1487.0) ?  0.017251567655129012 :   -0.005416255193331866 :   (b('l8b5') <= 2037.632080078125) ?  (b('l8b5_med') <= 1981.1566162109375) ?  0.024055941847124562 :   (b('grass') <= 12.753561019897461) ?  0.0021424577064842465 :   0.0012136803523251083 :   (b('l8b4') <= 917.0052185058594) ?  (b('dg0_v_vv') <= 0.0648798942565918) ?  0.0767411173312488 :   0.037501384086502776 :   (b('l8b5') <= 2540.5501708984375) ?  0.0055750155746037144 :   0.03726791880576034 :   (b('ndvi_med') <= 4011.5) ?  (b('dg0_v_vv') <= 0.2151646614074707) ?  (b('dg0_v_vv') <= 0.05360698699951172) ?  (b('l8b5_med') <= 3135.829833984375) ?  (b('l8b11') <= 2043.0220947265625) ?  -0.004141165749200841 :   0.013186915224339344 :   (b('l8b11') <= 1626.4351196289062) ?  -0.06285671809899394 :   -0.011302242253793053 :   (b('l8b11') <= 2134.0904541015625) ?  (b('l8b5') <= 2346.2577514648438) ?  -0.06525933830437652 :   -0.07753825340295092 :   -0.038744127554962204 :   (b('dg0_v_vv') <= 0.35402679443359375) ?  (b('l8b11') <= 1852.3582153320312) ?  (b('l8b4_med') <= 1057.4125671386719) ?  0.0491673561147864 :   0.038376883542707185 :   0.06657956737437681 :   (b('l8b11_med') <= 1227.396484375) ?  (b('dg0_v_vv') <= 0.682377815246582) ?  0.013665931150462857 :   0.031874647230959854 :   (b('bulk') <= 149.0) ?  0.0068917166698854605 :   -0.03959704872546044 :   (b('l8b4') <= 1160.9107666015625) ?  0.05706814491872314 :   0.05928859813044829 :   (b('ndvi') <= 2004.5) ?  (b('grass') <= 14.4358491897583) ?  -0.06758126854981314 :   (b('l8b5_med') <= 2888.830078125) ?  0.019686078951590025 :   0.0717853933555527 :   (b('dg0_v_vv') <= -3.5184569358825684) ?  -0.008868228346842089 :   (b('l8b5_med') <= 3151.007568359375) ?  (b('dg0_v_vv') <= 0.7375197410583496) ?  (b('l8b11') <= 2656.9212646484375) ?  (b('l8b11_med') <= 2095.1530151367188) ?  0.07139803233308785 :   0.09714253475637233 :   0.03315897171567009 :   (b('dg0_v_vv') <= 1.932490348815918) ?  (b('sand') <= 22.0) ?  0.05068707644764453 :   0.04340805467286829 :   (b('l8b4_med') <= 1099.8471984863281) ?  0.038992417887059705 :   0.0326930187080432 :   -0.0073365612499384225 :   (b('l8b5') <= 1837.3274536132812) ?  (b('l8b4') <= 981.365966796875) ?  (b('ndvi_med') <= 2457.0) ?  (b('ndvi_med') <= 2221.5) ?  (b('l8b4') <= 733.0534515380859) ?  -0.012360781191999765 :   -0.04101659051144174 :   (b('l8b11_med') <= 1974.021484375) ?  (b('l8b4') <= 940.7025451660156) ?  0.005829948957961795 :   0.010259300540570684 :   0.02101360086525443 :   0.06744201817707746 :   0.17949886835950926 :   (b('ndvi') <= 2871.5) ?  (b('ndvi') <= 1696.5) ?  (b('l8b4') <= 1104.52978515625) ?  -0.04080140409105126 :   (b('l8b11') <= 3066.4278564453125) ?  (b('ndvi') <= 1440.0) ?  (b('l8b5_med') <= 3107.97265625) ?  -0.022692327237420304 :   -0.0048759121280862044 :   (b('ndvi') <= 1553.0) ?  0.021863395889858347 :   0.005049330456195921 :   (b('l8b5') <= 3156.01708984375) ?  (b('l8b5') <= 3020.0394287109375) ?  -0.01289608503327036 :   0.006052688070635456 :   (b('ndvi') <= 1559.5) ?  -0.02254754896305626 :   -0.020861921680688955 :   (b('l8b11') <= 1735.2659912109375) ?  (b('dg0_v_vv') <= 0.06342792510986328) ?  (b('l8b4_med') <= 637.9811096191406) ?  -0.0429270044692969 :   (b('l8b11') <= 1655.2185668945312) ?  0.0008132868154570594 :   0.0021189903587487957 :   (b('l8b4') <= 766.9003295898438) ?  0.0247792186217754 :   (b('l8b11') <= 1558.1270751953125) ?  0.05194712463877957 :   0.06428434379570314 :   (b('dg0_v_vv') <= 0.24357032775878906) ?  (b('ndvi_med') <= 3287.5) ?  (b('l8b11') <= 2018.4524536132812) ?  -0.006983656428815855 :   -0.029938716251330752 :   0.0693464976836799 :   (b('ndvi_med') <= 2302.5) ?  (b('dg0_v_vv') <= 2.012362480163574) ?  -0.03390795360710801 :   0.008293045245946234 :   (b('l8b4') <= 901.1148986816406) ?  -0.0944636041359587 :   -0.05896268024182241 :   (b('dg0_v_vv') <= -0.907442569732666) ?  0.08598776716523673 :   (b('dg0_v_vv') <= -0.6627788543701172) ?  (b('ndvi_med') <= 1867.25) ?  0.0350932022119137 :   0.03826052182608901 :   (b('l8b4_med') <= 958.9872741699219) ?  0.028429042734578136 :   0.013894294250609479 :   (b('sand') <= 36.5) ?  (b('sand') <= 33.5) ?  (b('l8b5') <= 2755.7178955078125) ?  (b('ndvi') <= 2663.0) ?  (b('l8b4_med') <= 895.8446655273438) ?  (b('l8b4_med') <= 851.5547180175781) ?  (b('l8b4') <= 1032.4819946289062) ?  (b('dg0_v_vv') <= 3.957273244857788) ?  0.006007527329252033 :   0.09112183181386302 :   (b('ndvi') <= 1831.5) ?  0.009504428858369962 :   -0.01799430736861401 :   (b('ndvi') <= 2389.0) ?  (b('l8b5') <= 2584.3009033203125) ?  0.032623145459028516 :   0.07816244895482427 :   (b('grass') <= 18.720745086669922) ?  -0.003958145495647769 :   -0.03734847159725291 :   (b('l8b4') <= 1232.8865356445312) ?  (b('ndvi') <= 1548.0) ?  (b('dg0_v_vv') <= 0.12845563888549805) ?  -0.009569967530654035 :   -0.052670498844030286 :   (b('ndvi_med') <= 1761.75) ?  -0.07536140016419213 :   -0.0016004713352758065 :   (b('l8b11') <= 1942.2813720703125) ?  (b('dg0_v_vv') <= -0.43647050857543945) ?  0.06157930876006512 :   0.09830845292586532 :   (b('dg0_v_vv') <= -2.7211923599243164) ?  -0.03334412127933038 :   0.007760666313478313 :   (b('ndvi_med') <= 3617.25) ?  (b('grass') <= 63.09566879272461) ?  (b('l8b11') <= 993.251708984375) ?  (b('l8b5') <= 2716.4249267578125) ?  0.010137998969968191 :   -0.09343260757963845 :   (b('l8b4') <= 735.0203857421875) ?  -0.019697795518618798 :   -0.00538623703418604 :   (b('l8b4') <= 848.2108459472656) ?  (b('l8b4_med') <= 817.7582092285156) ?  -0.03676477414737627 :   0.014671743528997038 :   (b('bulk') <= 143.5) ?  0.010406006481001338 :   0.06410455669066963 :   (b('ndvi') <= 2969.5) ?  -0.06875358184340041 :   (b('l8b4') <= 1178.2023315429688) ?  (b('l8b5') <= 2539.332763671875) ?  0.0728084581871805 :   0.11567027669251703 :   -1.9630353550820212e-05 :   (b('l8b4_med') <= 883.4100952148438) ?  (b('l8b11') <= 1586.6124877929688) ?  (b('ndvi') <= 3039.0) ?  (b('crops') <= 27.784600257873535) ?  (b('l8b4') <= 814.630859375) ?  0.03222802498103181 :   -0.028548207229729096 :   (b('l8b5') <= 2802.3994140625) ?  -0.03207128655751241 :   -0.005667736809232793 :   (b('sand') <= 29.5) ?  (b('l8b4') <= 687.6600952148438) ?  -0.03428255535479626 :   -0.013367274793539921 :   (b('ndvi') <= 3079.0) ?  0.01759762121684446 :   -0.004262075957828564 :   (b('l8b11') <= 1644.1820678710938) ?  (b('dg0_v_vv') <= -0.17071914672851562) ?  (b('l8b5') <= 2892.7200927734375) ?  -0.02675704170041105 :   -0.010991103786740123 :   (b('dg0_v_vv') <= 1.5404791831970215) ?  -0.034289915663747325 :   -0.05365312253253135 :   (b('l8b5') <= 3507.9898681640625) ?  (b('l8b5') <= 2761.67138671875) ?  0.01267552771223079 :   -0.016249843187386766 :   (b('l8b5_med') <= 2960.120849609375) ?  0.022673567730833566 :   0.05871361536009631 :   (b('l8b11_med') <= 2001.7551879882812) ?  (b('l8b11') <= 1305.9896240234375) ?  (b('l8b4') <= 776.6743469238281) ?  (b('ndvi') <= 2618.0) ?  0.023017039265661766 :   0.00023194514163332118 :   (b('l8b11_med') <= 1798.922607421875) ?  0.0489341081998673 :   0.10049667286544389 :   (b('l8b11') <= 1554.9981689453125) ?  (b('ndvi') <= 2896.0) ?  -0.011630945083843824 :   -0.054950807444340985 :   (b('l8b4') <= 912.6340637207031) ?  0.02255078907760699 :   0.000806687774542454 :   (b('l8b5_med') <= 2596.419921875) ?  (b('l8b11') <= 1746.1693725585938) ?  (b('l8b4_med') <= 1237.8748168945312) ?  -0.09946409256494554 :   -0.08560048557248917 :   (b('dg0_v_vv') <= 1.012437343597412) ?  -0.029686946893609834 :   -0.05173169168286931 :   (b('dg0_v_vv') <= 3.52451229095459) ?  (b('dg0_v_vv') <= 2.7654964923858643) ?  -0.010536484303313423 :   -0.06600423341690002 :   (b('ndvi') <= 1442.5) ?  0.06872783525172332 :   0.02685798211763757 :   (b('dg0_v_vv') <= 2.978404998779297) ?  (b('l8b5') <= 1661.591064453125) ?  (b('l8b11') <= 1365.5899658203125) ?  (b('l8b4') <= 682.0668029785156) ?  (b('grass') <= 20.273585319519043) ?  0.008801914651256654 :   (b('l8b5_med') <= 2727.130859375) ?  -0.019928029260125234 :   -0.009610386493711917 :   (b('l8b4_med') <= 1036.7967224121094) ?  (b('dg0_v_vv') <= -1.1312956809997559) ?  -0.023640247630546035 :   0.00046046780941253845 :   (b('dg0_v_vv') <= -0.3514881134033203) ?  0.006364235366772751 :   0.02354511138760182 :   (b('l8b11') <= 1838.5560302734375) ?  (b('dg0_v_vv') <= -0.2157583236694336) ?  (b('ndvi') <= 1593.5) ?  0.04956495442653025 :   0.0707180465951091 :   0.01731112024849392 :   (b('dg0_v_vv') <= -0.37345075607299805) ?  (b('dg0_v_vv') <= -0.6207284927368164) ?  -0.006730422141494392 :   0.022170226891568126 :   0.06363206239289927 :   (b('bulk') <= 123.0) ?  (b('l8b4') <= 850.2877502441406) ?  (b('ndvi') <= 2048.5) ?  -0.022548322522870534 :   0.0023768109669713433 :   (b('dg0_v_vv') <= 0.22785043716430664) ?  (b('dg0_v_vv') <= -0.5000162124633789) ?  -0.05488978848044379 :   -0.048030840251731634 :   (b('l8b11') <= 1521.9293823242188) ?  -0.060050487151861946 :   -0.0513057727935633 :   (b('l8b5') <= 2167.169677734375) ?  (b('l8b4_med') <= 1257.5050659179688) ?  (b('grass') <= 60.15926933288574) ?  -0.013465416837643551 :   -0.034362788163746466 :   (b('ndvi') <= 2880.0) ?  0.01294861677292988 :   -0.020494807592973455 :   (b('l8b5') <= 2174.2039794921875) ?  (b('sand') <= 35.5) ?  0.04599021284070072 :   0.0067985435229632 :   (b('l8b11_med') <= 2300.271728515625) ?  -0.007396900530957195 :   0.0021135684594507553 :   (b('bulk') <= 150.5) ?  (b('l8b5') <= 2543.6766357421875) ?  (b('moss') <= 11.270072937011719) ?  -0.03391073894872665 :   -0.032516518821156776 :   -0.04642312275902316 :   (b('dg0_v_vv') <= 3.7658700942993164) ?  -0.07110094542196924 :   -0.056729505890687815 :   (b('l8b4') <= 693.2251892089844) ?  (b('l8b5') <= 1624.0888671875) ?  (b('l8b4') <= 471.53892517089844) ?  (b('dg0_v_vv') <= -1.7219841480255127) ?  (b('l8b11') <= 816.0653686523438) ?  (b('ndvi_med') <= 2786.5) ?  -0.002169522282473302 :   -0.0005736941218476843 :   0.020842009352184138 :   (b('l8b5') <= 1521.8152465820312) ?  (b('l8b4_med') <= 589.2293701171875) ?  -0.010240070147409158 :   (b('ndvi') <= 3144.0) ?  -0.0358501801455724 :   -0.047288255313801375 :   (b('dg0_v_vv') <= -0.6291501522064209) ?  (b('dg0_v_vv') <= -1.4469177722930908) ?  -0.03662783143408102 :   -0.05600440044731764 :   (b('ndvi_med') <= 2020.5) ?  -0.08742991028809449 :   -0.06556927235647317 :   (b('ndvi') <= 1475.5) ?  (b('dg0_v_vv') <= -0.43839526176452637) ?  (b('ndvi_med') <= 1822.75) ?  (b('dg0_v_vv') <= -1.736781120300293) ?  -0.013220568725054616 :   -0.023206423005388832 :   (b('grass') <= 20.542542934417725) ?  0.03106380817734994 :   0.027479153606968135 :   (b('dg0_v_vv') <= 1.0301921367645264) ?  (b('l8b5') <= 1278.9366455078125) ?  -0.043133799809344675 :   -0.0290770543016902 :   (b('sand') <= 55.5) ?  -0.080911220919729 :   -0.059833613817683484 :   (b('ndvi') <= 1496.5) ?  (b('dg0_v_vv') <= 0.8279776573181152) ?  (b('dg0_v_vv') <= -0.07578229904174805) ?  0.06598346670192962 :   0.11680656720694615 :   (b('ndvi_med') <= 1589.0) ?  0.05585692849889892 :   0.03712725455780175 :   (b('ndvi') <= 1582.5) ?  (b('bulk') <= 152.0) ?  -0.03990629039167651 :   0.001805631812778874 :   (b('ndvi') <= 1602.0) ?  0.021660800438866847 :   -0.007251758377196195 :   (b('l8b4_med') <= 954.2029418945312) ?  (b('ndvi') <= 3127.5) ?  (b('l8b4_med') <= 949.03759765625) ?  (b('dg0_v_vv') <= 1.4960901737213135) ?  (b('l8b4_med') <= 694.1253356933594) ?  0.0038625223665438063 :   0.016990271726850584 :   (b('ndvi_med') <= 3462.25) ?  -0.014332105059819812 :   0.024863034497300426 :   -0.09896271256146931 :   (b('grass') <= 16.677147388458252) ?  (b('dg0_v_vv') <= -1.1237397193908691) ?  (b('dg0_v_vv') <= -3.9960784912109375) ?  -0.02926049366333834 :   -0.037192827931065195 :   (b('bulk') <= 129.0) ?  -0.046990110744572794 :   -0.053778065954617135 :   (b('l8b11_med') <= 1203.1886596679688) ?  (b('l8b5') <= 2585.7685546875) ?  -0.0051081160264923545 :   0.03669967620112989 :   (b('ndvi_med') <= 2630.75) ?  -0.015800277358626682 :   -0.03478192642612877 :   (b('l8b11') <= 932.0558471679688) ?  (b('l8b5_med') <= 2862.278564453125) ?  0.10512673248916626 :   0.11672124282913532 :   (b('l8b11') <= 1067.0185546875) ?  (b('moss') <= 6.53251051902771) ?  (b('ndvi') <= 2249.0) ?  -0.007749371668220678 :   0.017906672778279686 :   (b('grass') <= 70.9430923461914) ?  -0.05341028155137992 :   -0.08982185462351239 :   (b('grass') <= 58.01798629760742) ?  (b('dg0_v_vv') <= 1.1776883602142334) ?  -0.008857136203028658 :   0.04351547621306463 :   (b('l8b11') <= 1389.7835083007812) ?  0.03998268036025096 :   0.11249132659211629 :   (b('l8b4') <= 695.1322937011719) ?  (b('sand') <= 45.5) ?  (b('dg0_v_vv') <= 0.28714656829833984) ?  (b('l8b11') <= 1137.7401428222656) ?  -0.06485050164737818 :   (b('grass') <= 66.21527290344238) ?  (b('l8b11_med') <= 1512.197265625) ?  -0.04810794714221451 :   -0.05427584048045908 :   -0.0383769386531643 :   (b('l8b11') <= 1326.4777221679688) ?  -0.06051174177118236 :   -0.07157452078902284 :   0.008023873168058276 :   (b('l8b11_med') <= 1209.0771484375) ?  (b('ndvi') <= 1674.0) ?  (b('l8b11_med') <= 1102.9674682617188) ?  -0.0806572437218461 :   (b('dg0_v_vv') <= 0.09847402572631836) ?  (b('ndvi') <= 1620.0) ?  -0.06351487928173496 :   -0.05521277252542009 :   -0.043272986553705844 :   (b('ndvi') <= 2961.5) ?  (b('ndvi') <= 2511.5) ?  (b('dg0_v_vv') <= -0.8828930854797363) ?  0.016073056447200425 :   -0.016056978494886107 :   (b('l8b5') <= 2398.8271484375) ?  0.01752068254932067 :   -0.00343638024881876 :   (b('dg0_v_vv') <= -1.847795009613037) ?  (b('moss') <= 9.692551612854004) ?  0.019333899247223646 :   0.02925445768142859 :   (b('l8b11') <= 1039.9993896484375) ?  -0.06072085370762457 :   -0.022423858018429835 :   (b('l8b4') <= 717.9411315917969) ?  (b('l8b11_med') <= 1648.781494140625) ?  (b('l8b4') <= 702.8829040527344) ?  (b('ndvi_med') <= 2758.0) ?  -0.0033115536240870685 :   -0.02753704389603586 :   (b('l8b11') <= 1324.2308349609375) ?  0.08921271610720609 :   0.03458659367104392 :   (b('dg0_v_vv') <= -1.0021519660949707) ?  (b('l8b11_med') <= 1754.5415649414062) ?  -0.0385129329739238 :   -0.05774607478860036 :   (b('l8b5') <= 2035.7214965820312) ?  -0.0033874790011966345 :   0.019593979504815194 :   (b('l8b11_med') <= 1211.7019653320312) ?  (b('dg0_v_vv') <= 1.3269405364990234) ?  (b('l8b4') <= 834.1351928710938) ?  -0.007539595945311228 :   0.033288279235241226 :   (b('ndvi') <= 2980.0) ?  0.051556291919278685 :   0.09305885973810701 :   (b('dg0_v_vv') <= -13.883424282073975) ?  (b('l8b5') <= 2392.977294921875) ?  0.044442642291369294 :   0.06393704763434763 :   (b('grass') <= 1.0004997551441193) ?  0.0038120397285717643 :   -0.0009747337523865316 :   (b('sand') <= 37.5) ?  (b('moss') <= 7.0154242515563965) ?  (b('ndvi_med') <= 1615.25) ?  (b('l8b4_med') <= 1367.0242919921875) ?  0.04298166945118864 :   (b('ndvi') <= 4148.0) ?  0.04817576870491702 :   0.04937943371509096 :   (b('l8b4') <= 300.52552795410156) ?  (b('ndvi') <= 4483.0) ?  (b('ndvi_med') <= 3351.5) ?  (b('l8b11_med') <= 1295.4417114257812) ?  0.04659338055715573 :   0.045230500028604015 :   0.05252757752018544 :   0.015807551059271285 :   (b('l8b4') <= 637.6953430175781) ?  (b('ndvi') <= 3344.0) ?  (b('ndvi') <= 3207.5) ?  0.045043929128342985 :   (b('bulk') <= 159.5) ?  (b('dg0_v_vv') <= 3.7409605979919434) ?  -0.02747849918584914 :   -0.0909384702305211 :   (b('l8b4_med') <= 1268.47412109375) ?  -0.08312725801652215 :   -0.07483532354627306 :   (b('ndvi') <= 4985.5) ?  (b('l8b11') <= 1314.7036743164062) ?  (b('l8b4') <= 595.2150573730469) ?  -0.0012191900235814462 :   -0.018220097972614414 :   (b('dg0_v_vv') <= 3.16717529296875) ?  0.02622731544240987 :   -0.09213162677623779 :   (b('dg0_v_vv') <= 0.5916986465454102) ?  0.06913248472621425 :   (b('sand') <= 26.5) ?  0.04204990000129233 :   0.04767452487884799 :   (b('ndvi') <= 4979.5) ?  (b('l8b11') <= 1317.1279907226562) ?  (b('crops') <= 43.333038330078125) ?  (b('l8b4') <= 690.72265625) ?  0.05643362103534739 :   -0.003731025257722867 :   (b('l8b4') <= 864.3643493652344) ?  0.0033077286611197534 :   0.04174606230478986 :   (b('l8b11') <= 1350.2234497070312) ?  (b('sand') <= 22.0) ?  -0.06391391220441231 :   -0.011269827666941611 :   (b('grass') <= 75.100341796875) ?  0.0003151742296764702 :   0.01404858790310984 :   (b('l8b4_med') <= 793.8125915527344) ?  -0.07013686460630245 :   (b('moss') <= 1.6706225872039795) ?  -0.05186461964631661 :   -0.05626799467390531 :   (b('ndvi') <= 4275.0) ?  (b('l8b11') <= 980.1434631347656) ?  (b('l8b4') <= 510.7897186279297) ?  (b('l8b11') <= 934.3846435546875) ?  (b('ndvi') <= 3644.5) ?  (b('ndvi') <= 3602.5) ?  (b('l8b4_med') <= 673.1795349121094) ?  -0.009402151224692068 :   0.01840250980876038 :   (b('ndvi_med') <= 3595.5) ?  -0.03401993010166181 :   -0.014112982537033691 :   (b('l8b11') <= 931.8220520019531) ?  (b('dg0_v_vv') <= 0.2664518356323242) ?  0.013478824276961244 :   0.03683695920776654 :   -0.03415944379855618 :   (b('l8b5') <= 3172.3533935546875) ?  (b('dg0_v_vv') <= 1.4942965507507324) ?  (b('dg0_v_vv') <= 0.13393545150756836) ?  0.048507845712428 :   0.07925529946160759 :   (b('moss') <= 15.31746768951416) ?  0.04826784407596696 :   0.01461344867684633 :   (b('dg0_v_vv') <= -0.22133350372314453) ?  (b('l8b4_med') <= 575.4939270019531) ?  -0.04115677442018212 :   0.04269666403818573 :   (b('grass') <= 43.45318794250488) ?  0.08262812647909273 :   -0.0036037371369617155 :   (b('l8b4') <= 597.9347839355469) ?  (b('dg0_v_vv') <= -0.1525559425354004) ?  (b('dg0_v_vv') <= -0.45029306411743164) ?  (b('dg0_v_vv') <= -1.185903549194336) ?  -0.0007656342066709454 :   0.0073049633596345325 :   -0.00805144540517344 :   (b('ndvi') <= 4189.5) ?  (b('sand') <= 30.5) ?  -0.01503653180896336 :   -0.021315113343957474 :   -0.03161624872201607 :   (b('dg0_v_vv') <= 1.4462823867797852) ?  (b('l8b4_med') <= 778.4898681640625) ?  0.03828692008866641 :   0.04512759748103862 :   0.00903433103180229 :   (b('l8b4') <= 471.1632995605469) ?  (b('dg0_v_vv') <= 0.0603790283203125) ?  (b('l8b11') <= 1032.4866943359375) ?  (b('dg0_v_vv') <= -0.6667189598083496) ?  -0.015462529230236138 :   (b('dg0_v_vv') <= -0.2367253303527832) ?  -0.03656949024334491 :   -0.029629400634045115 :   -0.013922156519665607 :   (b('dg0_v_vv') <= 0.5333290100097656) ?  (b('dg0_v_vv') <= 0.21609973907470703) ?  (b('grass') <= 24.36945152282715) ?  -0.06386077029254494 :   -0.057680955560796277 :   -0.06568095687209846 :   (b('ndvi_med') <= 2507.0) ?  -0.03163332994938549 :   -0.03635191024836737 :   (b('ndvi') <= 3242.0) ?  (b('l8b11') <= 1713.7891845703125) ?  (b('ndvi') <= 3237.5) ?  (b('l8b5_med') <= 2772.07373046875) ?  -0.04147400715099692 :   -0.0010893336907429885 :   0.05167457001415904 :   (b('grass') <= 29.000950813293457) ?  (b('dg0_v_vv') <= 1.2721257209777832) ?  0.045737118638595434 :   -0.008918040480283218 :   0.10377793359411355 :   (b('dg0_v_vv') <= -1.4142284393310547) ?  (b('l8b4') <= 685.5379333496094) ?  (b('crops') <= 11.721932411193848) ?  -0.08290335329273693 :   -0.028351272098641484 :   (b('l8b4') <= 983.5184631347656) ?  0.0020955682338060022 :   -0.030601747475420753 :   (b('sand') <= 33.0) ?  (b('l8b11') <= 1796.0597534179688) ?  0.0023362607595151922 :   -0.01828440687608856 :   (b('l8b5') <= 3462.3812255859375) ?  0.011820169377123475 :   -0.0027107686072446856 :   (b('bulk') <= 139.5) ?  (b('l8b4') <= 457.5876007080078) ?  (b('dg0_v_vv') <= 1.4788808822631836) ?  (b('dg0_v_vv') <= 0.5420904159545898) ?  (b('l8b11') <= 748.8206787109375) ?  -0.03384201008847476 :   (b('dg0_v_vv') <= 0.40728282928466797) ?  -0.012364044778198257 :   0.0017642630861926245 :   (b('dg0_v_vv') <= 0.8816132545471191) ?  (b('moss') <= 9.647008419036865) ?  -0.06475944761344088 :   -0.04077221955087609 :   (b('dg0_v_vv') <= 1.296276330947876) ?  -0.02007838818660812 :   -0.011818673962200543 :   0.045700052365145905 :   (b('ndvi_med') <= 3568.0) ?  (b('dg0_v_vv') <= 0.5055615901947021) ?  (b('l8b11') <= 2402.0811767578125) ?  (b('ndvi') <= 4322.0) ?  0.0418996845393219 :   0.0018694310836842485 :   (b('ndvi_med') <= 2491.5) ?  0.03327115464286928 :   0.05396871291127514 :   (b('l8b4') <= 921.1639709472656) ?  (b('l8b11_med') <= 1316.21044921875) ?  -0.023382861917660907 :   0.03668664363806563 :   (b('l8b4_med') <= 918.0328063964844) ?  -0.010022157598199381 :   -0.046466671999280104 :   (b('dg0_v_vv') <= -0.5877242088317871) ?  (b('l8b4') <= 814.3263549804688) ?  (b('l8b4') <= 554.0983581542969) ?  -0.04523182357994655 :   0.026714246343569246 :   -0.09972982680171677 :   (b('dg0_v_vv') <= -0.020961761474609375) ?  (b('dg0_v_vv') <= -0.09762716293334961) ?  0.04807520298938023 :   0.1054052567994227 :   (b('dg0_v_vv') <= -1.1444091796875e-05) ?  -0.04291346969673421 :   0.021006527151115333 :   (b('l8b4') <= 764.9397888183594) ?  (b('l8b4_med') <= 705.1431579589844) ?  (b('l8b5') <= 2981.8165283203125) ?  (b('dg0_v_vv') <= 1.989534854888916) ?  (b('l8b11_med') <= 1360.62841796875) ?  0.03221920171671783 :   0.04985415530036203 :   0.09211061706364432 :   (b('dg0_v_vv') <= 1.8505568504333496) ?  (b('l8b4') <= 459.2182159423828) ?  0.039143113071827634 :   0.016652997078336753 :   (b('l8b5_med') <= 2786.183349609375) ?  -0.08634246440449495 :   0.019881214354450033 :   (b('ndvi') <= 4498.5) ?  (b('ndvi') <= 4425.0) ?  (b('crops') <= 24.27506446838379) ?  0.09789307965032829 :   0.06803163552395536 :   (b('l8b11_med') <= 1555.286865234375) ?  0.05450456703701678 :   0.03990169871095277 :   (b('l8b5_med') <= 2838.0662841796875) ?  (b('dg0_v_vv') <= 1.2419891357421875) ?  0.06817898874133911 :   0.08618509655650325 :   0.11915172315846842 :   (b('ndvi') <= 4440.5) ?  (b('l8b5_med') <= 2786.183349609375) ?  (b('ndvi_med') <= 2984.0) ?  (b('dg0_v_vv') <= 0.04265403747558594) ?  0.0031758504922630437 :   0.023547898936389602 :   (b('dg0_v_vv') <= 1.643373966217041) ?  0.029617213859708696 :   0.019068854153767162 :   0.000490897177497035 :   (b('l8b5') <= 3105.3016357421875) ?  (b('l8b5_med') <= 2476.0528564453125) ?  -0.02104143854765622 :   (b('ndvi') <= 4580.5) ?  -0.001091714281582723 :   0.011935430937886737 :   (b('dg0_v_vv') <= 1.7242722511291504) ?  -0.04053537272541863 :   -0.021242015122292207 :   (b('ndvi') <= 3212.0) ?  (b('l8b5') <= 2879.6953125) ?  (b('dg0_v_vv') <= 0.006749153137207031) ?  (b('l8b4_med') <= 892.707763671875) ?  0.02917409812883398 :   0.054658011663470896 :   (b('l8b4_med') <= 815.593994140625) ?  (b('dg0_v_vv') <= 0.8681721687316895) ?  0.0698017432623793 :   0.056633689915807195 :   0.13509181455346625 :   (b('l8b11') <= 2188.03857421875) ?  (b('l8b11') <= 1847.2559204101562) ?  (b('l8b5_med') <= 3257.319091796875) ?  (b('dg0_v_vv') <= 2.182486057281494) ?  -0.018422679957825483 :   -0.023790513032970945 :   -0.031083489114938745 :   0.010339481422828103 :   (b('dg0_v_vv') <= 0.9184451103210449) ?  0.06989823381389157 :   0.0386278836917244 :   (b('ndvi_med') <= 1570.5) ?  (b('l8b11_med') <= 2629.3411865234375) ?  (b('l8b11') <= 1202.4706420898438) ?  (b('bulk') <= 168.0) ?  (b('dg0_v_vv') <= 3.1863207817077637) ?  (b('l8b11') <= 1054.759765625) ?  (b('dg0_v_vv') <= 1.5061655044555664) ?  -0.012276374288224484 :   0.01060166987366816 :   0.037733516830826236 :   0.08344855993815303 :   (b('l8b5') <= 3124.990234375) ?  (b('l8b4') <= 624.1618041992188) ?  -0.11280710869859378 :   -0.09324859370775646 :   0.0428094201292869 :   (b('l8b5') <= 3969.854736328125) ?  (b('l8b11') <= 1728.3916015625) ?  (b('ndvi') <= 3322.5) ?  (b('l8b11_med') <= 2589.752685546875) ?  0.061679682794600274 :   0.10933350391164942 :   (b('crops') <= 69.2904224395752) ?  0.05582070640466773 :   0.024859088547320874 :   (b('dg0_v_vv') <= -0.04970359802246094) ?  (b('ndvi') <= 3767.0) ?  0.01574818646085204 :   0.04367369222863504 :   (b('dg0_v_vv') <= 1.1193194389343262) ?  -0.031053350097187862 :   0.03427565453453586 :   -0.06010429926122879 :   (b('ndvi_med') <= 1480.5) ?  (b('l8b4') <= 2179.832275390625) ?  (b('l8b4') <= 1424.2327880859375) ?  (b('moss') <= 5.133426547050476) ?  0.032289854273908764 :   (b('ndvi') <= 3672.0) ?  -0.00273189529477047 :   -0.015687556932834426 :   (b('l8b4') <= 2082.836669921875) ?  (b('ndvi') <= 3867.0) ?  -0.02376256922842286 :   -0.01818228064591822 :   (b('ndvi') <= 3591.0) ?  -0.016770440410039254 :   0.01775412894414316 :   (b('dg0_v_vv') <= 1.5726501941680908) ?  (b('l8b11_med') <= 2890.52392578125) ?  0.02688668549780482 :   0.03340135631242175 :   (b('dg0_v_vv') <= 2.3063323497772217) ?  0.016892502113896535 :   0.004241583755993955 :   (b('l8b5') <= 4055.2896728515625) ?  (b('l8b11') <= 2067.4470825195312) ?  (b('ndvi') <= 4041.0) ?  (b('dg0_v_vv') <= -0.9141182899475098) ?  0.01774568940881055 :   0.024980476026408377 :   (b('l8b5_med') <= 2812.056640625) ?  -0.004985182850434117 :   0.0030148184608680684 :   (b('l8b4') <= 1644.0587768554688) ?  (b('l8b4') <= 1442.8677368164062) ?  0.0029249629230712892 :   -0.013555102702937534 :   (b('ndvi') <= 3591.0) ?  -0.009546099974178072 :   0.011606409626215801 :   (b('ndvi') <= 3842.0) ?  0.045996274663744535 :   0.02852135475438526 :   (b('bulk') <= 129.5) ?  (b('ndvi') <= 4984.0) ?  (b('ndvi') <= 3618.0) ?  (b('l8b11') <= 829.50732421875) ?  -0.0953963732404153 :   (b('l8b4_med') <= 444.2311248779297) ?  (b('l8b4') <= 811.574951171875) ?  0.04030641111151734 :   0.0483738906355885 :   (b('l8b4') <= 1334.6461181640625) ?  -0.004583210045284081 :   0.04913440984900994 :   (b('ndvi') <= 3679.0) ?  (b('grass') <= 52.318315505981445) ?  (b('l8b11_med') <= 1408.7545166015625) ?  0.03841928292218499 :   0.01858061007176866 :   (b('l8b11_med') <= 1210.7488403320312) ?  0.014392070211816987 :   -0.008441217367900022 :   (b('l8b4') <= 458.8667449951172) ?  (b('l8b5_med') <= 2083.720703125) ?  -0.04036701595756849 :   0.010421742910255292 :   (b('l8b5_med') <= 3561.5052490234375) ?  0.00306214868864982 :   -0.024734504320065995 :   (b('l8b5_med') <= 2444.189697265625) ?  (b('ndvi_med') <= 3520.75) ?  0.029448420884132742 :   (b('l8b11_med') <= 1419.2899780273438) ?  (b('dg0_v_vv') <= -0.7584638595581055) ?  -0.07771839874841424 :   -0.10426866484660924 :   (b('l8b5') <= 4300.2791748046875) ?  -0.03460212483179416 :   -0.041371682682529776 :   (b('dg0_v_vv') <= -0.3785228729248047) ?  (b('dg0_v_vv') <= -4.594003200531006) ?  0.009445694734517507 :   (b('l8b5') <= 4706.756591796875) ?  -0.028127264797367657 :   -0.01690321671652284 :   (b('l8b11') <= 1006.5199279785156) ?  0.017912599623425007 :   0.024887533158950648 :   (b('moss') <= 15.2976393699646) ?  (b('ndvi') <= 4700.0) ?  (b('sand') <= 78.5) ?  (b('l8b4') <= 1119.1144409179688) ?  (b('l8b4') <= 1061.65234375) ?  -0.005149235755137609 :   -0.02610244102358048 :   (b('ndvi') <= 3352.0) ?  0.02646595960772762 :   -0.0015636252398246433 :   (b('l8b5') <= 2495.8626708984375) ?  (b('l8b11_med') <= 1525.6304931640625) ?  0.005640756692887994 :   -0.031895069244001456 :   (b('dg0_v_vv') <= 4.106689453125) ?  0.02373836385750222 :   -0.025257527994501233 :   (b('l8b4') <= 1104.3365783691406) ?  (b('ndvi') <= 4836.5) ?  (b('l8b5') <= 4527.491455078125) ?  0.02471060952932559 :   -0.003998318271995373 :   (b('ndvi') <= 4959.5) ?  -0.013708766185339411 :   0.0134745183665384 :   (b('dg0_v_vv') <= -0.11865472793579102) ?  0.05906071715804839 :   0.1495272424585617 :   (b('bulk') <= 134.0) ?  (b('l8b4_med') <= 784.8345336914062) ?  0.02119733610334798 :   0.10676958193368866 :   (b('dg0_v_vv') <= 0.26535987854003906) ?  (b('l8b5') <= 2588.650146484375) ?  (b('l8b4') <= 894.4288330078125) ?  0.023643805566464376 :   -0.002229031419155742 :   (b('l8b5_med') <= 2715.8355712890625) ?  -0.034620489628978075 :   -0.00037311692667862766 :   (b('ndvi_med') <= 2268.75) ?  (b('l8b5_med') <= 2245.5650634765625) ?  -0.02924141305836469 :   0.014374980360269303 :   (b('l8b11') <= 1760.1431274414062) ?  0.04000973239520016 :   0.08686521355696795 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 788.6375427246094) ?  (b('l8b4') <= 788.4720458984375) ?  (b('crops') <= 49.01690483093262) ?  (b('l8b11') <= 1809.036865234375) ?  (b('l8b5') <= 3618.2506103515625) ?  (b('l8b5') <= 3297.9840087890625) ?  (b('l8b5') <= 3296.5316162109375) ?  (b('bulk') <= 155.5) ?  (b('l8b4_med') <= 954.0115356445312) ?  (b('l8b4_med') <= 949.03759765625) ?  0.002388524492568824 :   -0.07259514937383604 :   (b('sand') <= 52.5) ?  0.016785715686062214 :   -0.009747500650439351 :   (b('grass') <= 40.474979400634766) ?  (b('l8b11') <= 1175.5455322265625) ?  -0.03293148052495238 :   0.027282545623428234 :   (b('l8b4') <= 738.3097839355469) ?  0.0015805081743603205 :   -0.02051545413385404 :   (b('l8b4_med') <= 896.4455261230469) ?  (b('ndvi') <= 3629.5) ?  0.07989471830329928 :   0.09066364545344238 :   (b('ndvi') <= 4787.0) ?  (b('l8b5_med') <= 2791.6748046875) ?  0.06401323000821223 :   0.039944839645269015 :   (b('dg0_v_vv') <= -0.23061442375183105) ?  0.051405111597978836 :   0.02598682303724685 :   (b('l8b11_med') <= 1629.9594116210938) ?  (b('moss') <= 11.045287609100342) ?  (b('ndvi') <= 4905.5) ?  (b('grass') <= 48.941123962402344) ?  -0.01155148969502446 :   0.0067793878679368235 :   (b('sand') <= 37.5) ?  0.05395175576889765 :   -0.0013840125010421206 :   (b('crops') <= 8.08548378944397) ?  (b('ndvi') <= 3780.0) ?  0.018011727997990133 :   -0.036515830002315675 :   (b('l8b4_med') <= 895.3085021972656) ?  0.03721401092937514 :   0.007867850172388025 :   (b('crops') <= 35.54349327087402) ?  (b('l8b4_med') <= 809.31201171875) ?  (b('ndvi') <= 4562.5) ?  -0.05526718303363501 :   -0.07619064969280279 :   (b('ndvi') <= 4280.5) ?  -0.032233752928541505 :   -0.009305145087993901 :   (b('l8b4') <= 761.1651000976562) ?  (b('l8b11') <= 942.1961059570312) ?  0.07729402139642227 :   -0.013138006904899751 :   (b('dg0_v_vv') <= 1.318099021911621) ?  0.05384798426322318 :   0.062378960935944694 :   (b('l8b4') <= 670.83203125) ?  (b('ndvi') <= 4140.0) ?  (b('l8b5_med') <= 2451.3546142578125) ?  -0.12907222478889652 :   (b('l8b11') <= 1171.5150756835938) ?  (b('sand') <= 55.5) ?  0.002423601033114205 :   0.028288447924434048 :   (b('l8b11') <= 1328.48876953125) ?  0.029914746095821496 :   -0.03747769066684151 :   (b('l8b4') <= 582.8929138183594) ?  (b('l8b4') <= 522.3069763183594) ?  (b('l8b11') <= 699.5525512695312) ?  -0.04724830327757471 :   0.0019301660009551132 :   (b('moss') <= 8.135722398757935) ?  0.010889135254925894 :   0.04425188304108941 :   (b('grass') <= 34.49213409423828) ?  (b('l8b5') <= 4063.0570068359375) ?  -0.009954602493924127 :   0.02685011352801095 :   (b('dg0_v_vv') <= -1.1041064262390137) ?  -0.0014735467364613425 :   -0.054006688923817436 :   (b('l8b5_med') <= 3191.0152587890625) ?  (b('l8b11_med') <= 1369.6777954101562) ?  (b('ndvi') <= 3554.0) ?  (b('ndvi_med') <= 3692.5) ?  -0.01771980931895853 :   0.013279070522707853 :   (b('ndvi') <= 3976.5) ?  0.037781588177231484 :   0.009199985225286266 :   (b('l8b11') <= 1369.6417846679688) ?  (b('dg0_v_vv') <= 1.3398995399475098) ?  0.05822722069353453 :   0.09007289747155008 :   (b('dg0_v_vv') <= 0.012859821319580078) ?  0.04525370090488373 :   0.01364739239047572 :   (b('dg0_v_vv') <= 3.177825450897217) ?  (b('bulk') <= 115.0) ?  (b('ndvi_med') <= 4246.75) ?  0.020249403187869247 :   -0.007204129288027758 :   (b('dg0_v_vv') <= 2.1814723014831543) ?  -0.029281341602431725 :   -0.011464204923762406 :   -0.07246420933450612 :   (b('l8b5_med') <= 2236.9990234375) ?  (b('moss') <= 21.870142936706543) ?  (b('l8b5') <= 1782.8511962890625) ?  0.09051172429748283 :   0.08513751622493079 :   0.07020195325092163 :   (b('l8b4') <= 743.9387817382812) ?  0.06728466580037648 :   (b('l8b4') <= 766.8590698242188) ?  (b('grass') <= 88.47662353515625) ?  0.0075905981202723655 :   -0.003578746195443183 :   (b('l8b5_med') <= 2786.183349609375) ?  (b('dg0_v_vv') <= 1.8281424045562744) ?  (b('dg0_v_vv') <= -1.5271430015563965) ?  0.024631430424712553 :   0.03952557725928924 :   0.01751398775527674 :   0.009267000034710288 :   (b('l8b11') <= 1107.867431640625) ?  (b('l8b11') <= 1030.7379150390625) ?  (b('crops') <= 51.25273323059082) ?  (b('l8b5') <= 4195.17138671875) ?  (b('l8b4') <= 434.23121643066406) ?  0.01327835973016575 :   (b('l8b4') <= 574.6910705566406) ?  (b('l8b5_med') <= 2892.9366455078125) ?  -0.02929880785518571 :   -0.036555527038959576 :   (b('dg0_v_vv') <= 0.8170285224914551) ?  -0.02582941495803388 :   -0.03257395117058974 :   (b('l8b4') <= 470.0284423828125) ?  0.003873262401420552 :   0.03880432601984679 :   (b('grass') <= 17.872713088989258) ?  (b('moss') <= 1.7430431246757507) ?  (b('l8b11_med') <= 1041.9317016601562) ?  (b('ndvi_med') <= 2261.0) ?  -0.05870243093346991 :   -0.006462385963421459 :   (b('ndvi_med') <= 3940.0) ?  0.004651767383691949 :   0.02650948476311442 :   (b('l8b5') <= 5306.837158203125) ?  (b('l8b11') <= 984.7737426757812) ?  -0.0060200274343734685 :   -0.02540448776876719 :   (b('l8b5_med') <= 2830.92822265625) ?  0.04271896226021357 :   0.01855303684566321 :   (b('dg0_v_vv') <= 4.693978309631348) ?  (b('ndvi') <= 4199.5) ?  (b('grass') <= 21.3442325592041) ?  0.0087838254390121 :   0.07026912943499744 :   (b('l8b11') <= 934.8724975585938) ?  0.020566133419666708 :   -0.010962396348837427 :   (b('dg0_v_vv') <= 7.052134037017822) ?  -0.06132201445654263 :   -0.07071683172010236 :   (b('ndvi_med') <= 3118.5) ?  (b('l8b11') <= 1053.9837646484375) ?  (b('l8b4_med') <= 1384.1409912109375) ?  (b('l8b5_med') <= 3097.873046875) ?  -0.062264920999101254 :   -0.06955373195686979 :   0.03993965586432885 :   (b('dg0_v_vv') <= 3.067960739135742) ?  (b('l8b4_med') <= 1536.2318115234375) ?  (b('l8b4') <= 702.8839111328125) ?  0.002840472336256887 :   0.027390435630674396 :   (b('sand') <= 46.5) ?  -0.018432136662728124 :   -0.04314538929276618 :   (b('moss') <= 3.1964867636561394) ?  -0.026273589904916045 :   -0.04234850178361976 :   (b('dg0_v_vv') <= -0.4501075744628906) ?  (b('l8b4') <= 539.0572814941406) ?  (b('l8b11_med') <= 1312.8507080078125) ?  (b('l8b4_med') <= 510.28395080566406) ?  0.0367029959579408 :   0.02622168279751247 :   (b('ndvi') <= 2381.5) ?  0.020480013300317734 :   -0.00709174474047207 :   (b('l8b4') <= 621.3433837890625) ?  (b('dg0_v_vv') <= -0.8694472312927246) ?  -0.027369290807285944 :   0.001197245870468877 :   (b('dg0_v_vv') <= -1.0015840530395508) ?  0.0035931784349444995 :   0.021148776406320624 :   (b('l8b5') <= 2549.1865234375) ?  (b('l8b4_med') <= 964.6636352539062) ?  (b('l8b4_med') <= 813.8113098144531) ?  -0.0006408219950407856 :   0.0037790655524980593 :   -0.012160288927078211 :   (b('l8b4_med') <= 453.6625671386719) ?  (b('dg0_v_vv') <= 0.20187139511108398) ?  0.0508406863618077 :   0.029738366905216362 :   (b('ndvi_med') <= 3369.0) ?  0.03394208967841812 :   0.020337656660229027 :   (b('l8b11') <= 1365.820556640625) ?  (b('grass') <= 20.588459014892578) ?  (b('l8b4') <= 734.6119689941406) ?  (b('dg0_v_vv') <= -0.8453679084777832) ?  (b('bulk') <= 130.5) ?  (b('ndvi') <= 3710.5) ?  0.03695771407883838 :   0.0022786549575291074 :   (b('ndvi_med') <= 3453.75) ?  -0.007495659845655406 :   -0.04089636836102445 :   (b('ndvi_med') <= 3361.5) ?  (b('dg0_v_vv') <= -0.7780616283416748) ?  0.03166996509261401 :   -0.0011323601584540767 :   (b('ndvi') <= 2582.5) ?  -0.027034958267840026 :   0.02590362180865829 :   (b('ndvi_med') <= 2864.0) ?  (b('crops') <= 77.0467414855957) ?  (b('ndvi_med') <= 2378.75) ?  -0.006021715474047076 :   0.018292301286402853 :   (b('l8b5_med') <= 3359.630615234375) ?  -0.022505842191083994 :   -0.05210941832334176 :   (b('l8b4') <= 765.4873657226562) ?  (b('ndvi_med') <= 3351.25) ?  -0.02269464123267856 :   -0.049985553362009015 :   (b('dg0_v_vv') <= 1.1518383026123047) ?  -0.0851705858604994 :   -0.08337285491085858 :   (b('l8b4') <= 771.3757019042969) ?  (b('l8b11') <= 1209.8026123046875) ?  (b('ndvi') <= 4190.0) ?  (b('l8b11') <= 1204.352783203125) ?  -0.04595571375839604 :   -0.0939607303282895 :   (b('bulk') <= 142.5) ?  -0.03785081015011353 :   0.0033140120707802804 :   (b('l8b11') <= 1265.0942993164062) ?  (b('l8b4') <= 622.362548828125) ?  0.016057758724499455 :   -0.01223068001578117 :   (b('l8b5_med') <= 2450.4654541015625) ?  0.04226007207989374 :   -0.02974103381307623 :   (b('dg0_v_vv') <= 1.153660774230957) ?  (b('ndvi') <= 4155.0) ?  0.09150922449350268 :   0.07329451601050385 :   0.009238884048494572 :   (b('l8b5') <= 3559.677978515625) ?  (b('l8b11') <= 1371.3602905273438) ?  (b('l8b5') <= 3353.27783203125) ?  (b('ndvi_med') <= 3114.25) ?  (b('ndvi') <= 2894.0) ?  0.06796530826307172 :   0.043112935011401635 :   (b('ndvi') <= 2894.5) ?  0.011798786902353206 :   0.021429577566497486 :   (b('l8b4_med') <= 714.1749572753906) ?  (b('ndvi_med') <= 3956.75) ?  0.05185392736981524 :   0.07252303385833395 :   0.017766479578146782 :   (b('l8b4') <= 733.6283264160156) ?  (b('l8b5_med') <= 3372.728759765625) ?  (b('grass') <= 21.748714447021484) ?  -0.01154380895347821 :   0.001192519138618778 :   (b('grass') <= 25.360753059387207) ?  0.0071386582824056015 :   0.05252467716435709 :   (b('grass') <= 30.299102783203125) ?  (b('l8b4') <= 747.470458984375) ?  0.025254722929422668 :   0.005821109355467047 :   (b('sand') <= 32.5) ?  -0.07334811788190124 :   -0.03393495583652699 :   (b('moss') <= 8.678699970245361) ?  (b('l8b4') <= 781.9303894042969) ?  (b('grass') <= 25.00780487060547) ?  (b('grass') <= 21.379006385803223) ?  -0.029191057504306585 :   -0.052843040196022394 :   (b('l8b4_med') <= 799.0747985839844) ?  -0.012432251627497823 :   0.01878796310341843 :   (b('dg0_v_vv') <= -0.3078293800354004) ?  0.007503337416558953 :   0.004195567218370055 :   (b('ndvi') <= 3725.5) ?  0.00880423387938832 :   (b('ndvi_med') <= 3240.5) ?  0.02244884647632725 :   0.025693403908824547 :   0.14407738018058958 :   (b('l8b11') <= 1108.5) ?  (b('ndvi') <= 3325.0) ?  (b('grass') <= 0.9122380167245865) ?  (b('l8b4_med') <= 1027.3583679199219) ?  0.012385641894278654 :   (b('l8b11') <= 1080.60498046875) ?  0.05138482342425871 :   0.047332601561920595 :   (b('ndvi') <= 2807.5) ?  (b('dg0_v_vv') <= -4.104102611541748) ?  -0.01661531147257994 :   (b('l8b4') <= 907.0) ?  (b('l8b11') <= 1092.2390747070312) ?  (b('l8b11_med') <= 1432.5072631835938) ?  0.0036339332071754077 :   (b('l8b5_med') <= 2837.44482421875) ?  0.007261881115301683 :   0.010709166486361998 :   (b('bulk') <= 131.0) ?  (b('dg0_v_vv') <= -3.390531539916992) ?  0.014775662197238026 :   0.016461386929601723 :   0.01446351536793547 :   (b('ndvi_med') <= 2107.5) ?  (b('l8b5_med') <= 2844.591552734375) ?  0.025651291161732648 :   0.0033846992562561184 :   0.05076963406513346 :   -0.02086064033328138 :   (b('dg0_v_vv') <= -1.3380122184753418) ?  -0.016068670960729337 :   (b('dg0_v_vv') <= 0.16131925582885742) ?  (b('dg0_v_vv') <= -0.015888690948486328) ?  (b('l8b11_med') <= 1982.1893920898438) ?  (b('dg0_v_vv') <= -0.44574546813964844) ?  0.030494489533329783 :   (b('l8b4') <= 863.634521484375) ?  0.04052348063500655 :   0.060799300177369175 :   0.06976621295239305 :   0.07232751931601444 :   (b('l8b11_med') <= 1414.4430541992188) ?  (b('ndvi_med') <= 3333.0) ?  0.03339254583135387 :   0.039179030391797376 :   0.025030782589555572 :   (b('moss') <= 4.735910177230835) ?  (b('moss') <= 3.124024271965027) ?  (b('l8b5') <= 2415.3076171875) ?  (b('l8b5') <= 2414.436767578125) ?  (b('bulk') <= 131.5) ?  (b('l8b11_med') <= 1164.1775512695312) ?  (b('dg0_v_vv') <= 0.053286075592041016) ?  (b('ndvi') <= 1960.0) ?  0.018916231005709667 :   0.004460514280325175 :   (b('sand') <= 59.5) ?  -0.032079165214212765 :   -0.0098111331565425 :   (b('l8b4_med') <= 699.3709106445312) ?  (b('l8b5_med') <= 3307.999267578125) ?  0.07713386236368322 :   0.052287151839214674 :   (b('dg0_v_vv') <= 3.2486252784729004) ?  0.01665705395175496 :   0.058348160602291255 :   (b('l8b11') <= 2254.1275634765625) ?  (b('l8b4_med') <= 737.7727661132812) ?  (b('ndvi') <= 2031.5) ?  0.005791959625798755 :   0.046572857841158666 :   (b('dg0_v_vv') <= 3.9438657760620117) ?  -0.0034712744596849514 :   0.014311209693298969 :   (b('l8b11') <= 2439.6883544921875) ?  (b('l8b11_med') <= 2349.6241455078125) ?  0.012624070856619221 :   0.052552674395308384 :   (b('l8b4') <= 1314.8939208984375) ?  -0.008793089683287111 :   0.010168845784876644 :   (b('l8b5') <= 2415.11669921875) ?  (b('ndvi') <= 1533.0) ?  (b('l8b11_med') <= 1768.8693237304688) ?  0.06462089357005843 :   (b('ndvi') <= 1413.5) ?  0.05423670908261255 :   0.057500623729077785 :   0.030978749878367823 :   0.12304561698726968 :   (b('l8b5_med') <= 3289.8900146484375) ?  (b('l8b5_med') <= 3075.4814453125) ?  (b('l8b11') <= 1651.36376953125) ?  (b('l8b5_med') <= 2406.5177001953125) ?  (b('l8b4_med') <= 1225.8678588867188) ?  -0.041017529470007245 :   -0.10039756328097826 :   (b('l8b4') <= 834.1481628417969) ?  0.015283997307446893 :   -0.013317666762196053 :   (b('l8b5') <= 3521.1129150390625) ?  (b('l8b4') <= 813.1304931640625) ?  0.05467077352272926 :   -0.0006612716323063968 :   (b('l8b5') <= 3731.379150390625) ?  0.04402997215783457 :   -0.00023885995147790453 :   (b('l8b11') <= 1986.397216796875) ?  (b('l8b11') <= 1843.0689697265625) ?  (b('grass') <= 4.528382062911987) ?  -0.04506544390451812 :   -0.010707511692958431 :   (b('crops') <= 35.19958972930908) ?  -0.09072279063713404 :   -0.023734723180760774 :   (b('ndvi') <= 3417.5) ?  (b('dg0_v_vv') <= -3.066460609436035) ?  -0.03511481461804944 :   -0.005648434624845661 :   (b('l8b4') <= 1101.2390747070312) ?  0.028747734326348112 :   0.0029443892833491236 :   (b('l8b5') <= 3031.090087890625) ?  (b('moss') <= 1.5658519864082336) ?  (b('l8b5') <= 2487.9259033203125) ?  (b('dg0_v_vv') <= 3.751737594604492) ?  -0.012520692460372823 :   -0.08472853394450142 :   (b('l8b11') <= 1514.587646484375) ?  0.05135756077281234 :   0.009202354477962273 :   (b('l8b11') <= 1697.8193359375) ?  (b('l8b4_med') <= 798.9369201660156) ?  0.04379554287873052 :   0.012912518002962281 :   (b('crops') <= 68.72173309326172) ?  0.08518443610314835 :   0.07570248858810114 :   (b('grass') <= 29.24530601501465) ?  (b('dg0_v_vv') <= 0.5122799873352051) ?  (b('moss') <= 0.7684792578220367) ?  0.00019474753673720012 :   -0.012143385034482963 :   (b('dg0_v_vv') <= 0.543846607208252) ?  0.07215403660221026 :   0.004502365105811155 :   (b('dg0_v_vv') <= -0.8787655830383301) ?  (b('ndvi_med') <= 4127.5) ?  -0.04401980969828942 :   -0.10351875796693358 :   (b('l8b4') <= 851.1303100585938) ?  -0.05145668530157135 :   -0.004797052804543404 :   (b('dg0_v_vv') <= -0.6375460624694824) ?  (b('l8b4') <= 873.3074645996094) ?  (b('bulk') <= 150.0) ?  (b('sand') <= 33.0) ?  (b('dg0_v_vv') <= -1.0535287857055664) ?  (b('l8b11_med') <= 1668.19970703125) ?  0.021909494539727487 :   0.0037860881025565674 :   (b('ndvi_med') <= 2724.75) ?  -0.01235498741585675 :   0.0008119519849012838 :   0.06038032019535883 :   (b('l8b4') <= 850.4641723632812) ?  (b('l8b4') <= 805.1268615722656) ?  (b('moss') <= 3.782366394996643) ?  -0.04229296407265179 :   -0.06264164453380928 :   (b('l8b4') <= 841.8080444335938) ?  -0.026355063699835823 :   0.00502261718930902 :   (b('crops') <= 30.325278639793396) ?  -0.09934434932129085 :   (b('ndvi') <= 2736.0) ?  -0.06544796628572155 :   -0.07346134834689991 :   (b('dg0_v_vv') <= -0.6569008827209473) ?  (b('dg0_v_vv') <= -4.35084867477417) ?  (b('l8b5') <= 2575.082763671875) ?  -0.04238785469623918 :   -0.07476240990415942 :   (b('l8b11') <= 1489.955322265625) ?  (b('l8b4_med') <= 1334.3555297851562) ?  0.05657853888422605 :   -0.04011122533718661 :   (b('dg0_v_vv') <= -0.6587367057800293) ?  0.00010456526475784176 :   0.07439121109115893 :   -0.08986476397672727 :   (b('ndvi') <= 4473.5) ?  (b('grass') <= 21.693400382995605) ?  (b('dg0_v_vv') <= -0.6098723411560059) ?  0.09840015352048448 :   (b('l8b11_med') <= 1871.3919067382812) ?  (b('ndvi') <= 875.5) ?  -0.0636572456814628 :   0.0013172548101083867 :   (b('l8b4') <= 1235.7071533203125) ?  0.028277883919561587 :   0.008202341251759573 :   (b('l8b5') <= 3836.9432373046875) ?  (b('ndvi') <= 1732.5) ?  (b('dg0_v_vv') <= 0.27982378005981445) ?  -0.039473762428007195 :   0.06181214614459657 :   (b('l8b4') <= 865.0068359375) ?  0.0038229442911262915 :   0.028640683564943325 :   (b('dg0_v_vv') <= 0.009215354919433594) ?  (b('dg0_v_vv') <= -0.059205055236816406) ?  -0.06733465758621052 :   -0.09643719474920529 :   (b('l8b11') <= 1765.5872192382812) ?  0.049445024488646405 :   -0.030846768478398835 :   (b('ndvi_med') <= 3976.5) ?  (b('l8b5') <= 3739.3780517578125) ?  (b('dg0_v_vv') <= 0.5139446258544922) ?  (b('l8b11_med') <= 1492.2809448242188) ?  0.05966495501961039 :   0.12800179813346269 :   (b('l8b11') <= 1467.3067016601562) ?  0.04865443820319865 :   0.015060464693012662 :   (b('l8b5_med') <= 3293.3367919921875) ?  (b('dg0_v_vv') <= -0.3228139877319336) ?  0.003828814951922277 :   -0.01518450765740515 :   -0.09465154103039838 :   (b('l8b11') <= 1587.8104248046875) ?  0.03734904136487688 :   (b('dg0_v_vv') <= 0.9557356834411621) ?  -0.13116903624005707 :   (b('dg0_v_vv') <= 1.7674174308776855) ?  -0.04180695204697493 :   -0.07370895756039508 :   (b('l8b4') <= 798.5061950683594) ?  (b('bulk') <= 155.5) ?  (b('crops') <= 63.54620552062988) ?  (b('sand') <= 34.5) ?  (b('dg0_v_vv') <= -0.37856197357177734) ?  (b('dg0_v_vv') <= -0.4488334655761719) ?  (b('dg0_v_vv') <= -1.4567089080810547) ?  -0.016678232972983893 :   -0.0025113026069797715 :   (b('l8b5') <= 2121.720947265625) ?  0.04307420966214676 :   -0.025673941347069615 :   (b('l8b11_med') <= 1369.6777954101562) ?  (b('l8b5') <= 2744.9322509765625) ?  0.016306462878370113 :   -0.028826520908046105 :   (b('dg0_v_vv') <= 1.070338249206543) ?  -0.022747293173836117 :   -0.04800914552242158 :   (b('ndvi_med') <= 4092.5) ?  (b('ndvi') <= 3752.0) ?  (b('ndvi') <= 3251.0) ?  -0.001866433227756149 :   -0.031829682797670386 :   (b('sand') <= 75.5) ?  0.01264441179919073 :   -0.005524253445739822 :   (b('dg0_v_vv') <= -1.5390009880065918) ?  -0.017021179038746156 :   (b('l8b11_med') <= 1544.9956665039062) ?  -0.034805136847627635 :   -0.020721541091771975 :   (b('ndvi') <= 1708.0) ?  -0.03729889612498389 :   (b('l8b4_med') <= 722.3658142089844) ?  -0.05044929771356297 :   -0.046904366366967515 :   (b('sand') <= 49.0) ?  (b('dg0_v_vv') <= 0.11527347564697266) ?  -0.03850463077088938 :   -0.037723017822884744 :   -0.09480667475374219 :   (b('l8b4') <= 814.5537719726562) ?  (b('l8b4_med') <= 771.4015502929688) ?  (b('l8b11_med') <= 1154.969970703125) ?  (b('l8b5_med') <= 2138.92236328125) ?  0.00578512557136196 :   (b('dg0_v_vv') <= -0.824458122253418) ?  (b('ndvi_med') <= 3645.5) ?  -8.903988831700238e-05 :   -0.013381560784978486 :   (b('ndvi_med') <= 3569.0) ?  -0.008911581870238239 :   -0.019653435817764443 :   (b('dg0_v_vv') <= 1.4925789833068848) ?  (b('dg0_v_vv') <= -0.5655479431152344) ?  (b('ndvi') <= 2275.0) ?  -0.009257400922383675 :   0.017892278089262057 :   (b('dg0_v_vv') <= -0.38875293731689453) ?  0.09530787287388531 :   0.03281892790288069 :   -0.06360498068948547 :   (b('ndvi_med') <= 1587.5) ?  (b('dg0_v_vv') <= 0.4025273323059082) ?  (b('dg0_v_vv') <= -0.2061767578125) ?  (b('crops') <= 10.164965629577637) ?  0.049243030873290725 :   0.057085364848575765 :   (b('l8b5') <= 2238.2734375) ?  0.022609808456466263 :   -0.0066173547786498935 :   0.14214768689761664 :   (b('bulk') <= 140.5) ?  (b('ndvi') <= 3785.0) ?  (b('sand') <= 28.0) ?  0.14252926157463522 :   0.013062500357084462 :   (b('dg0_v_vv') <= 1.4780993461608887) ?  -0.005045515054723467 :   -0.059264548992717436 :   (b('moss') <= 12.84782600402832) ?  (b('dg0_v_vv') <= 2.8367838859558105) ?  0.003587660527296945 :   0.03575993168051603 :   (b('crops') <= 16.375940322875977) ?  -0.04488551217509814 :   -0.010800032538416377 :   (b('l8b4') <= 857.0255432128906) ?  (b('ndvi_med') <= 2151.75) ?  (b('l8b5_med') <= 2854.88525390625) ?  (b('l8b4') <= 844.3298645019531) ?  (b('moss') <= 8.78812837600708) ?  0.09256750555556739 :   0.004366742959119561 :   (b('l8b4') <= 849.2717895507812) ?  0.08376871103279103 :   0.001625583002661143 :   -0.11290055430072043 :   (b('l8b5') <= 3669.0870361328125) ?  (b('moss') <= 5.687705755233765) ?  (b('ndvi') <= 3741.0) ?  -0.0301743503530424 :   -0.1060339414354914 :   (b('grass') <= 11.275712013244629) ?  -0.11033761425343824 :   -0.01111040490888804 :   (b('ndvi_med') <= 3354.75) ?  (b('grass') <= 18.444106101989746) ?  -0.03136320445566723 :   0.01321866169435683 :   (b('ndvi') <= 3532.0) ?  0.06519250523102318 :   0.032002090665259046 :   (b('ndvi') <= 2508.5) ?  (b('bulk') <= 121.5) ?  (b('l8b5') <= 1659.4708251953125) ?  (b('l8b5') <= 1630.8194580078125) ?  0.04608292867455807 :   0.03748886867497904 :   (b('l8b5') <= 2253.343994140625) ?  -0.04448232992853523 :   -0.014607852236392825 :   (b('bulk') <= 123.0) ?  (b('l8b5') <= 2011.8773193359375) ?  -0.028981246304963734 :   0.027123530007933386 :   (b('l8b4_med') <= 703.0869750976562) ?  -0.015561618442268528 :   -0.001676236820165195 :   (b('l8b4') <= 861.0667724609375) ?  (b('l8b11_med') <= 1541.24658203125) ?  (b('ndvi') <= 3507.0) ?  0.04268483082279165 :   0.0715194353649684 :   (b('crops') <= 56.5693416595459) ?  0.018794117714116126 :   0.004879804135730914 :   (b('ndvi_med') <= 1608.75) ?  (b('ndvi') <= 4490.5) ?  0.010444685940327039 :   0.09782530578509757 :   (b('ndvi') <= 2513.0) ?  0.021308193418852504 :   0.00037081239889011645 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b4') <= 1414.2814331054688) ?  (b('l8b5_med') <= 2995.8839111328125) ?  (b('ndvi_med') <= 1515.0) ?  (b('l8b5') <= 2227.4732666015625) ?  (b('l8b11') <= 1297.7045288085938) ?  (b('l8b11') <= 1128.7681274414062) ?  (b('dg0_v_vv') <= 0.298189640045166) ?  (b('moss') <= 15.039891242980957) ?  0.0023640563554483685 :   (b('l8b4') <= 731.9627227783203) ?  -0.017860255350276147 :   -0.018690593484288393 :   (b('l8b4_med') <= 916.8209838867188) ?  -0.0502837520798604 :   -0.04024031704590658 :   (b('ndvi') <= 696.0) ?  -0.03562927084852181 :   (b('ndvi') <= 763.5) ?  (b('l8b4') <= 1073.253173828125) ?  0.09496431279010757 :   0.06645372211040843 :   (b('l8b11') <= 1189.171142578125) ?  (b('dg0_v_vv') <= 1.4516139030456543) ?  0.05301476722960881 :   0.02751249962376114 :   (b('bulk') <= 129.0) ?  -0.020532342719840363 :   0.021267986183270693 :   (b('ndvi_med') <= 1512.5) ?  (b('grass') <= 56.2891788482666) ?  (b('grass') <= 41.35932159423828) ?  (b('ndvi') <= 647.0) ?  (b('moss') <= 12.538093090057373) ?  0.07626521645557766 :   0.009989964669518644 :   (b('ndvi') <= 751.0) ?  -0.019017784075104648 :   0.0057067585271590995 :   (b('l8b4') <= 778.6858520507812) ?  (b('l8b5') <= 1897.5311889648438) ?  0.022576535310241362 :   0.09901658067802949 :   (b('l8b11') <= 1422.3851318359375) ?  -0.03832698655181862 :   -0.006951064386024833 :   (b('grass') <= 67.71883392333984) ?  (b('l8b5') <= 1733.8460083007812) ?  (b('sand') <= 44.5) ?  0.06235577647764836 :   0.023377965924701108 :   (b('ndvi_med') <= 859.0) ?  -0.06956076822422799 :   0.006337345044021201 :   (b('ndvi') <= 873.5) ?  (b('l8b4') <= 1234.492919921875) ?  -0.06764760531268521 :   -0.05184923701471465 :   (b('l8b11_med') <= 2069.5927734375) ?  -0.005914507378893049 :   0.011601677916622522 :   0.09470015031354831 :   (b('l8b11') <= 3032.9564208984375) ?  (b('l8b5') <= 2240.30712890625) ?  (b('ndvi_med') <= 1427.0) ?  (b('l8b4') <= 1126.3153686523438) ?  (b('dg0_v_vv') <= 0.3630990982055664) ?  (b('dg0_v_vv') <= -0.4854617118835449) ?  0.05275189378872 :   0.01907028064974549 :   0.08513495334614996 :   (b('ndvi') <= 1512.5) ?  (b('bulk') <= 150.0) ?  0.08540863153105668 :   0.09178282285601458 :   (b('l8b11') <= 2201.1744384765625) ?  0.09300708388627041 :   0.09761144686799962 :   (b('bulk') <= 147.0) ?  -0.026669112627463396 :   0.023672266665158315 :   (b('l8b5_med') <= 1834.7416381835938) ?  (b('dg0_v_vv') <= 2.3102316856384277) ?  (b('dg0_v_vv') <= 0.7587890625) ?  (b('l8b4_med') <= 1071.1250610351562) ?  -0.028279469089086448 :   -0.04134923638542319 :   -0.0810663785669721 :   0.0005352542148127915 :   (b('ndvi') <= 2363.5) ?  (b('bulk') <= 141.5) ?  (b('l8b11') <= 2337.12841796875) ?  0.026012041894380784 :   0.1445331518138977 :   (b('dg0_v_vv') <= 0.2064990997314453) ?  0.004057915420252884 :   0.025926106908593295 :   (b('dg0_v_vv') <= 3.285478115081787) ?  (b('ndvi') <= 2496.0) ?  -0.045125819595286595 :   0.005311568727238239 :   (b('l8b11_med') <= 2788.3504638671875) ?  -0.058760687327192196 :   -0.017359200808518955 :   (b('ndvi') <= 1631.0) ?  0.1739641380176547 :   0.08396616249072572 :   (b('ndvi_med') <= 1615.0) ?  (b('l8b5') <= 2874.0260009765625) ?  (b('moss') <= 1.5) ?  (b('l8b11') <= 2179.4188232421875) ?  (b('dg0_v_vv') <= 1.5915489196777344) ?  (b('l8b4') <= 1055.3590087890625) ?  (b('ndvi') <= 3413.0) ?  -0.07521328812076776 :   -0.10168444864617043 :   (b('l8b4') <= 1257.4911499023438) ?  -0.04727512681832758 :   -0.0025846652626644456 :   0.007012131503544239 :   (b('dg0_v_vv') <= -0.5240564346313477) ?  (b('ndvi') <= 1127.5) ?  -0.0051329371943695384 :   (b('l8b5') <= 1866.4512939453125) ?  0.010317542644189659 :   0.01271934191130833 :   (b('l8b11') <= 2471.3953857421875) ?  -0.021900355920987574 :   -0.027068088373260316 :   (b('l8b4_med') <= 1019.2635192871094) ?  (b('l8b5') <= 2113.140625) ?  (b('ndvi') <= 1475.5) ?  (b('l8b11') <= 1819.6582641601562) ?  -0.02768559635422128 :   -0.008150248051741232 :   (b('ndvi') <= 1488.5) ?  0.056055401643332434 :   -0.006927681880779997 :   (b('l8b11') <= 1874.4480590820312) ?  (b('l8b4') <= 722.2626037597656) ?  -0.07357264769433736 :   -0.04825798519976968 :   (b('l8b5') <= 2496.986572265625) ?  -0.02627001060591857 :   -0.005449792930774966 :   (b('ndvi') <= 954.0) ?  (b('dg0_v_vv') <= 0.16350364685058594) ?  (b('l8b5') <= 2381.45654296875) ?  -0.03443409583003529 :   -0.03773759830530264 :   (b('ndvi') <= 885.0) ?  -0.0670612224166782 :   -0.06593995590786406 :   (b('dg0_v_vv') <= 2.773508071899414) ?  (b('l8b11') <= 1093.375732421875) ?  -0.029719669520450243 :   -0.002708597641604194 :   (b('sand') <= 56.0) ?  0.03653596946013936 :   -0.018009586059827037 :   (b('l8b5_med') <= 2629.9376220703125) ?  (b('l8b4') <= 745.9368591308594) ?  (b('l8b11_med') <= 2037.187255859375) ?  (b('dg0_v_vv') <= -0.23114538192749023) ?  (b('l8b4') <= 659.4546508789062) ?  0.03846681231096599 :   0.026274176922520487 :   0.04751086873749705 :   (b('dg0_v_vv') <= -0.552731990814209) ?  0.027502408057247796 :   (b('l8b4') <= 498.3856964111328) ?  -0.002936323683238151 :   0.009449429987665922 :   (b('l8b11') <= 1327.6424560546875) ?  -0.08187218892359538 :   -0.05314168777661246 :   (b('l8b11') <= 1746.9984741210938) ?  (b('l8b11') <= 1684.4647216796875) ?  0.08804277520658782 :   0.06715464443641223 :   (b('l8b11_med') <= 2656.321533203125) ?  0.026064545642527692 :   0.03169759134497849 :   (b('l8b5_med') <= 2977.11865234375) ?  (b('bulk') <= 137.5) ?  (b('moss') <= 0.04340072348713875) ?  (b('l8b5') <= 1710.4827270507812) ?  (b('ndvi') <= 1067.0) ?  (b('l8b5') <= 1526.9512329101562) ?  0.030469269515525792 :   -0.007681993613013138 :   (b('dg0_v_vv') <= -0.5381016731262207) ?  -0.03286694205785002 :   -0.002821491755220782 :   (b('l8b5') <= 1722.2799682617188) ?  (b('dg0_v_vv') <= -0.39583873748779297) ?  0.005726477559543974 :   0.16798801917232323 :   (b('l8b11_med') <= 986.4671020507812) ?  -0.04546677722205746 :   0.01073613976589685 :   (b('l8b4_med') <= 1207.4241943359375) ?  (b('l8b4_med') <= 445.8315734863281) ?  (b('l8b4') <= 353.5813293457031) ?  -0.050688981252589936 :   -0.019191062137304794 :   (b('l8b4') <= 580.8514709472656) ?  0.0020019369822913725 :   -0.0017170770339132305 :   (b('l8b11') <= 2398.8328857421875) ?  (b('l8b4') <= 1174.2239990234375) ?  -0.06551394081647828 :   -0.03259050224093882 :   (b('dg0_v_vv') <= 0.07170772552490234) ?  -0.011499793993522365 :   0.01964785038340218 :   (b('dg0_v_vv') <= 2.5847930908203125) ?  (b('sand') <= 68.0) ?  (b('bulk') <= 155.5) ?  (b('l8b4_med') <= 702.8189086914062) ?  -0.0032044788600178015 :   0.003360325861825549 :   (b('l8b4') <= 476.56431579589844) ?  0.012819273526312232 :   -0.005321578634580399 :   (b('l8b11') <= 2441.6026611328125) ?  (b('l8b11') <= 672.3034362792969) ?  0.06731764572097276 :   -0.03652174213762847 :   (b('ndvi') <= 2648.5) ?  0.006552221882026722 :   0.15574296013984418 :   (b('bulk') <= 142.5) ?  (b('l8b5_med') <= 2786.183349609375) ?  (b('l8b11_med') <= 1358.7476196289062) ?  0.009685399804900538 :   0.043110686165945995 :   (b('ndvi') <= 4501.0) ?  -0.01654873665063739 :   0.05110192647097239 :   (b('ndvi_med') <= 3744.0) ?  (b('dg0_v_vv') <= 2.598794460296631) ?  0.06551227923882141 :   0.0060156781049534236 :   (b('dg0_v_vv') <= 4.340519189834595) ?  0.06334568833588339 :   0.019330589568078343 :   (b('l8b5') <= 2742.6087646484375) ?  (b('l8b4') <= 1009.7288208007812) ?  (b('dg0_v_vv') <= -0.40546464920043945) ?  (b('l8b5_med') <= 2982.3580322265625) ?  (b('l8b5') <= 2608.9735107421875) ?  -0.041060227883828626 :   -0.02815583457477306 :   (b('l8b5_med') <= 2990.1697998046875) ?  -0.022803561822082964 :   -0.005598681278785905 :   (b('l8b11') <= 1372.4946899414062) ?  (b('l8b11') <= 1313.12109375) ?  0.010202514021952 :   -0.04307734140155457 :   (b('l8b5') <= 2368.9215087890625) ?  0.02474301372403301 :   0.0007909878858435998 :   (b('l8b4') <= 1339.29150390625) ?  (b('dg0_v_vv') <= -0.3247671127319336) ?  (b('l8b11') <= 1797.0040893554688) ?  -0.008170030503402849 :   -0.043954798569307875 :   (b('l8b11') <= 1650.357666015625) ?  -0.022318127880797442 :   -0.05818520444442091 :   -0.015182173873434283 :   (b('ndvi') <= 1440.5) ?  -0.07967342237873055 :   (b('l8b5') <= 3082.20068359375) ?  (b('ndvi') <= 2145.5) ?  (b('l8b11') <= 2009.5779418945312) ?  -0.0128672942264283 :   0.00805406791165985 :   (b('ndvi') <= 2221.0) ?  0.13708634802873768 :   0.015686630320404803 :   (b('l8b5') <= 3670.1483154296875) ?  (b('l8b11') <= 1834.036865234375) ?  -0.030263652384554786 :   0.007950898348596261 :   (b('l8b5') <= 4199.1243896484375) ?  0.019857885445798158 :   -0.027407440622899237 :   (b('dg0_v_vv') <= -1.4960079193115234) ?  (b('crops') <= 16.61120891571045) ?  (b('l8b5_med') <= 3044.003173828125) ?  (b('dg0_v_vv') <= -1.882789134979248) ?  (b('dg0_v_vv') <= -2.0796518325805664) ?  (b('l8b11') <= 1590.0111083984375) ?  -0.05649205882776612 :   -0.05512996030158032 :   -0.07115733326057672 :   -0.009865477305798356 :   (b('dg0_v_vv') <= -1.9477057456970215) ?  (b('ndvi') <= 3906.5) ?  (b('l8b11_med') <= 1388.1404418945312) ?  (b('l8b11') <= 1610.5464477539062) ?  0.01768851710866523 :   0.007659548914483047 :   -0.0051506646712151105 :   (b('ndvi_med') <= 4138.5) ?  (b('bulk') <= 119.5) ?  -0.016933134010429535 :   -0.020260255235096725 :   -0.0368579385425242 :   (b('ndvi') <= 2609.5) ?  -0.03883834028224034 :   -0.04612597290355189 :   (b('bulk') <= 115.5) ?  (b('l8b4') <= 640.1963195800781) ?  (b('l8b4_med') <= 526.780029296875) ?  0.046225055639995016 :   (b('l8b5') <= 2987.3212890625) ?  (b('dg0_v_vv') <= -1.633307933807373) ?  0.029331816718391662 :   0.03229413282529858 :   (b('bulk') <= 110.5) ?  0.019907501736840527 :   0.022296278761561095 :   (b('grass') <= 28.284100770950317) ?  (b('l8b11') <= 1363.7578125) ?  0.015444359821068121 :   0.014172039134163539 :   (b('bulk') <= 106.5) ?  0.008361573757596996 :   0.005403990114838053 :   (b('l8b11') <= 1896.35546875) ?  (b('ndvi_med') <= 4697.0) ?  (b('l8b5_med') <= 3032.838623046875) ?  (b('l8b4_med') <= 612.2954254150391) ?  0.021211569559712817 :   (b('l8b4_med') <= 770.6802368164062) ?  0.024846839103831203 :   0.02729452873727757 :   (b('grass') <= 1.9546971321105957) ?  (b('dg0_v_vv') <= -2.556008815765381) ?  0.014935938859355614 :   -0.005569877019653775 :   (b('moss') <= 2.9268592596054077) ?  -0.013844232788414056 :   -0.005161385803234585 :   0.05957322658863512 :   (b('bulk') <= 154.5) ?  (b('l8b11_med') <= 2387.2855224609375) ?  (b('ndvi_med') <= 3019.0) ?  (b('ndvi') <= 2610.0) ?  0.01081239350176868 :   0.06262274701546518 :   (b('dg0_v_vv') <= -4.539652347564697) ?  -0.03814906922661526 :   0.005074517600250642 :   (b('bulk') <= 149.0) ?  (b('l8b4_med') <= 1465.410888671875) ?  0.035066241337250864 :   0.011801622038867046 :   (b('dg0_v_vv') <= -2.537044048309326) ?  -0.007564438019952605 :   -0.016928778753254724 :   (b('l8b11') <= 2342.830810546875) ?  (b('grass') <= 23.40122890472412) ?  -0.00883495401276696 :   -0.01664297056659654 :   -0.045171407567963284 :   (b('moss') <= 0.051342811435461044) ?  (b('bulk') <= 129.5) ?  (b('l8b5') <= 3207.2703857421875) ?  (b('l8b4') <= 1096.4094848632812) ?  (b('dg0_v_vv') <= 2.3680479526519775) ?  (b('dg0_v_vv') <= 0.3779001235961914) ?  (b('l8b4_med') <= 917.1777954101562) ?  0.04381278470457135 :   0.020815689935489864 :   (b('dg0_v_vv') <= 1.6443827152252197) ?  0.01318550460944467 :   0.03171027209081198 :   (b('dg0_v_vv') <= 2.7981300354003906) ?  (b('dg0_v_vv') <= 2.506598472595215) ?  0.0048290048694543675 :   -0.0024077148978322027 :   -0.02049227718172522 :   (b('l8b5_med') <= 3499.2373046875) ?  (b('dg0_v_vv') <= 0.7585203647613525) ?  (b('dg0_v_vv') <= 0.2617020606994629) ?  0.033559069558896815 :   0.04024206540603675 :   0.05206734306954608 :   (b('grass') <= 0.7789046764373779) ?  0.06236666075049513 :   0.08652016909417362 :   (b('ndvi') <= 2976.0) ?  (b('dg0_v_vv') <= -0.8712000846862793) ?  0.02297902099289245 :   (b('l8b5_med') <= 3505.713134765625) ?  (b('l8b5') <= 3393.1141357421875) ?  -0.06336563036165133 :   -0.04293034898700296 :   (b('ndvi') <= 2960.0) ?  -0.00996801971180436 :   -0.03138374638484262 :   (b('ndvi') <= 4770.5) ?  (b('ndvi') <= 4710.0) ?  (b('ndvi_med') <= 4138.5) ?  0.004504559633200663 :   0.04568718011388643 :   0.0875649311189281 :   (b('l8b4') <= 621.4129028320312) ?  (b('dg0_v_vv') <= -1.0640954971313477) ?  0.0029680134939706893 :   -0.04030433836449928 :   (b('dg0_v_vv') <= -0.5631561279296875) ?  -0.009011101492206519 :   0.01867045060647609 :   (b('l8b4') <= 305.94541931152344) ?  (b('l8b11_med') <= 1295.4417114257812) ?  (b('dg0_v_vv') <= 0.550621509552002) ?  0.05640267337011379 :   0.04146886576307096 :   (b('sand') <= 41.5) ?  0.01386434836211542 :   -0.010856395479468123 :   (b('dg0_v_vv') <= -1.117149829864502) ?  (b('l8b5') <= 3621.71875) ?  (b('grass') <= 5.7621941566467285) ?  (b('dg0_v_vv') <= -1.360684871673584) ?  -0.0014126530943477378 :   0.014066155172545317 :   (b('l8b11') <= 1438.531005859375) ?  0.03099341773984289 :   0.04601822928519637 :   (b('grass') <= 34.33602046966553) ?  (b('l8b11_med') <= 1187.5767822265625) ?  -0.019181570354470323 :   -0.030506157093937697 :   (b('l8b11') <= 1202.6058349609375) ?  0.007631718456546227 :   -0.004032452305031145 :   (b('dg0_v_vv') <= -1.008253574371338) ?  (b('ndvi') <= 1885.5) ?  0.045060930350995676 :   (b('dg0_v_vv') <= -1.0342812538146973) ?  -0.030395333674329793 :   -0.08428578559202872 :   (b('dg0_v_vv') <= -0.48746347427368164) ?  (b('l8b4_med') <= 737.0958557128906) ?  0.030030435647589726 :   -0.005526174512780807 :   (b('l8b4') <= 603.7620239257812) ?  -0.01279341941427111 :   -0.003218251209873267 :   (b('ndvi') <= 2798.5) ?  (b('l8b11_med') <= 2736.30615234375) ?  (b('ndvi') <= 2787.0) ?  (b('ndvi') <= 1978.5) ?  (b('dg0_v_vv') <= -1.3957715034484863) ?  (b('l8b4') <= 1115.0766906738281) ?  0.17439128062098863 :   0.03003716333478343 :   (b('dg0_v_vv') <= -0.6744065284729004) ?  -0.009313924964564041 :   0.025218653018822583 :   (b('l8b11') <= 2225.122314453125) ?  (b('ndvi') <= 2182.5) ?  -0.008133868612075731 :   0.010373707314585298 :   (b('l8b11') <= 2374.4305419921875) ?  0.06788673826213615 :   0.02305201205989202 :   (b('l8b11') <= 1953.9950561523438) ?  (b('grass') <= 23.377455711364746) ?  (b('l8b5_med') <= 3060.106201171875) ?  0.04168997187025025 :   0.05293591824260288 :   0.07769563818693737 :   0.11359768810237902 :   (b('grass') <= 51.816396713256836) ?  (b('ndvi') <= 2533.0) ?  (b('l8b11_med') <= 2810.475341796875) ?  (b('l8b4') <= 1159.4832763671875) ?  -0.023428629301693496 :   -0.002318103795415665 :   (b('l8b5_med') <= 3311.2408447265625) ?  0.005843495381436632 :   -0.004156189211265477 :   (b('l8b11') <= 1643.0786743164062) ?  (b('ndvi_med') <= 1482.0) ?  -0.020949054827962277 :   -0.034633458890427904 :   (b('l8b4_med') <= 1883.9478149414062) ?  0.017226511702893577 :   -0.013139742460160826 :   (b('l8b11') <= 1788.1329345703125) ?  0.07672522273783178 :   0.02202133999237553 :   (b('ndvi') <= 3065.0) ?  (b('moss') <= 4.12464714050293) ?  (b('sand') <= 24.5) ?  -0.0973259255206606 :   (b('l8b5_med') <= 3054.4278564453125) ?  (b('l8b4') <= 1076.9836120605469) ?  0.111541393060192 :   0.06135727271605823 :   (b('l8b11') <= 1988.1849365234375) ?  0.007199967709224597 :   0.06896604038538144 :   (b('bulk') <= 115.5) ?  -0.13591595175657498 :   (b('l8b5') <= 2851.2666015625) ?  (b('l8b5') <= 2499.1676025390625) ?  -0.02278665402832134 :   0.010950903250558305 :   (b('bulk') <= 143.0) ?  -0.010624058249482173 :   -0.03306888728329121 :   (b('ndvi') <= 3077.0) ?  (b('dg0_v_vv') <= 0.32393407821655273) ?  (b('l8b4_med') <= 980.4746704101562) ?  0.07347704359261029 :   (b('dg0_v_vv') <= -0.3965778350830078) ?  0.015668134759775826 :   0.03361053576517825 :   (b('bulk') <= 135.0) ?  (b('l8b5') <= 3123.29443359375) ?  0.11425005338966288 :   0.10322583254783496 :   (b('l8b11_med') <= 1966.1676025390625) ?  0.06561725156717531 :   0.08262896333283423 :   (b('crops') <= 16.61120891571045) ?  (b('dg0_v_vv') <= -0.8260407447814941) ?  (b('dg0_v_vv') <= -0.8657288551330566) ?  -0.018671759331915783 :   -0.10540154551966884 :   (b('ndvi') <= 3703.0) ?  -0.019498618857347806 :   0.009626344005406464 :   (b('ndvi') <= 3083.5) ?  (b('sand') <= 49.0) ?  -0.04011015080760453 :   0.018244299159969488 :   (b('l8b4') <= 1236.5438232421875) ?  0.0043636345508235004 :   0.021475318411872557 :   (b('ndvi_med') <= 1688.5) ?  (b('ndvi') <= 1572.5) ?  (b('l8b11_med') <= 2816.0322265625) ?  (b('ndvi') <= 1293.0) ?  (b('sand') <= 37.5) ?  (b('l8b4') <= 1520.0513305664062) ?  (b('dg0_v_vv') <= -0.34272003173828125) ?  0.05717172557618554 :   (b('dg0_v_vv') <= 0.44259071350097656) ?  (b('l8b4_med') <= 1054.0230102539062) ?  0.029171724711918194 :   0.03858694605239863 :   0.024526222909286285 :   (b('dg0_v_vv') <= 1.8409523963928223) ?  (b('l8b11_med') <= 2368.9771728515625) ?  (b('l8b4_med') <= 1184.5880126953125) ?  -0.006133368053933198 :   -0.031280962391083036 :   (b('l8b11') <= 2740.676513671875) ?  0.01689630571186198 :   -0.011106724301713453 :   0.07301461185639196 :   (b('l8b5') <= 2125.2769775390625) ?  (b('ndvi') <= 738.5) ?  (b('l8b5_med') <= 1873.66748046875) ?  (b('l8b11_med') <= 1947.122802734375) ?  -0.05850253679940998 :   -0.08945978066856991 :   (b('moss') <= 38.36263656616211) ?  -0.01498879601664345 :   -0.06569189831517494 :   (b('ndvi') <= 808.0) ?  (b('ndvi_med') <= 860.75) ?  -0.027309195624553542 :   0.0613041765287274 :   (b('l8b4') <= 1551.3829956054688) ?  -0.0037763219946029465 :   0.016152888148974306 :   (b('l8b5') <= 2135.1671142578125) ?  (b('l8b5') <= 2131.6192626953125) ?  (b('l8b5_med') <= 2012.7369995117188) ?  0.0040781253626061 :   -0.028698261836522125 :   (b('l8b4') <= 1577.9285888671875) ?  -0.07249532544654307 :   -0.10539600645075053 :   (b('dg0_v_vv') <= -0.6033854484558105) ?  (b('ndvi') <= 967.0) ?  0.012946114203430972 :   -0.006847864871914181 :   (b('l8b5') <= 3282.56884765625) ?  -0.009160014743132423 :   0.013287896166338157 :   (b('ndvi_med') <= 1683.0) ?  (b('l8b5') <= 2451.0770263671875) ?  (b('l8b4_med') <= 1531.7705688476562) ?  (b('ndvi') <= 1296.5) ?  (b('dg0_v_vv') <= 0.08683919906616211) ?  0.09741708964044679 :   0.01185157166393875 :   (b('grass') <= 74.9734992980957) ?  -0.00038925118255954264 :   -0.016413467948559926 :   (b('ndvi') <= 1460.0) ?  (b('ndvi') <= 1309.5) ?  -0.01887622400444548 :   0.03062070372746445 :   (b('grass') <= 75.08959579467773) ?  0.06993692859677143 :   0.15394704807488577 :   (b('dg0_v_vv') <= 2.1177401542663574) ?  (b('bulk') <= 145.5) ?  (b('l8b5') <= 3203.1134033203125) ?  -0.012692766625974555 :   -0.06331469865161993 :   (b('l8b11') <= 2309.19775390625) ?  -0.015722659020308387 :   0.002085378717217195 :   (b('l8b11_med') <= 2570.8941650390625) ?  (b('crops') <= 37.5662841796875) ?  0.11876104324954881 :   0.06443404416143941 :   (b('dg0_v_vv') <= 2.8737096786499023) ?  -0.021858847213921912 :   0.02378659205887574 :   (b('l8b4') <= 1781.62939453125) ?  0.03747449537476788 :   0.1124786525980759 :   (b('l8b11') <= 2681.841552734375) ?  (b('l8b11') <= 2248.5084228515625) ?  (b('l8b4') <= 1488.081787109375) ?  0.044880251373654007 :   (b('l8b4') <= 1588.2713623046875) ?  (b('l8b4') <= 1547.4352416992188) ?  (b('dg0_v_vv') <= 0.6590900421142578) ?  -0.004673244992132586 :   -0.013976531036325072 :   (b('ndvi_med') <= 1504.5) ?  -0.022260475899997505 :   -0.030587456287657633 :   (b('ndvi') <= 1391.0) ?  (b('dg0_v_vv') <= 0.8613338470458984) ?  0.022735779611043432 :   0.013151010232694361 :   (b('ndvi_med') <= 1569.0) ?  -0.0037767132525128354 :   -0.0287002218969616 :   (b('dg0_v_vv') <= -2.1679816246032715) ?  (b('l8b11_med') <= 2861.3095703125) ?  (b('l8b5_med') <= 3169.1014404296875) ?  -0.016619206233351086 :   (b('dg0_v_vv') <= -3.2106761932373047) ?  -0.02877263032739623 :   -0.023772631370477512 :   (b('dg0_v_vv') <= -2.450191020965576) ?  -0.01293418134205035 :   -0.008920256385309189 :   (b('bulk') <= 139.5) ?  (b('dg0_v_vv') <= 0.10226726531982422) ?  0.04321236646691462 :   0.07934299081874532 :   (b('l8b5') <= 2810.5322265625) ?  (b('l8b5') <= 2776.845703125) ?  0.01105257600604022 :   -0.030067328630384006 :   (b('dg0_v_vv') <= -1.0215587615966797) ?  -0.019603862098663133 :   0.03274724438674533 :   (b('l8b11') <= 2769.544677734375) ?  (b('ndvi') <= 1520.5) ?  (b('ndvi_med') <= 654.0) ?  (b('l8b5') <= 2525.6094970703125) ?  0.04226114147928468 :   0.020656124281144314 :   (b('l8b11') <= 2759.2286376953125) ?  (b('l8b11') <= 2756.4578857421875) ?  -0.009237281072800814 :   0.0237502790875949 :   (b('ndvi') <= 646.0) ?  -0.0566778698200662 :   -0.020467076763033793 :   (b('dg0_v_vv') <= 1.3704633712768555) ?  (b('dg0_v_vv') <= 0.08092594146728516) ?  (b('ndvi') <= 1549.0) ?  -0.04228351952962811 :   -0.02674478296777011 :   (b('dg0_v_vv') <= 0.7758536338806152) ?  -0.020398971649365168 :   -0.02783580480479709 :   -0.05977585313669735 :   (b('l8b11') <= 2770.823486328125) ?  0.08589759654374012 :   (b('ndvi') <= 1215.5) ?  (b('ndvi_med') <= 1526.5) ?  (b('ndvi_med') <= 1198.25) ?  -0.0013021471555485994 :   0.009507585447648983 :   (b('ndvi_med') <= 1636.25) ?  0.05486988263801772 :   0.09096230647605151 :   (b('ndvi') <= 1240.0) ?  (b('dg0_v_vv') <= 2.615011692047119) ?  -0.016932959165335746 :   -0.07374268934177423 :   (b('l8b11') <= 3438.2752685546875) ?  -0.004008848821476316 :   0.005322516135250729 :   (b('l8b11_med') <= 2631.7816162109375) ?  (b('moss') <= 25.883625984191895) ?  (b('ndvi_med') <= 1398.0) ?  (b('l8b11') <= 2163.9388427734375) ?  (b('sand') <= 51.5) ?  (b('l8b5') <= 2499.7427978515625) ?  -0.047639344242502046 :   -0.015120280817900428 :   0.02756722594141775 :   (b('dg0_v_vv') <= -0.39554452896118164) ?  (b('ndvi_med') <= 1327.5) ?  0.09998005485319911 :   (b('dg0_v_vv') <= -0.8583393096923828) ?  0.06425640174563331 :   0.0777262381588772 :   (b('ndvi') <= 1602.0) ?  -0.0245242097768235 :   (b('dg0_v_vv') <= 0.6515192985534668) ?  0.05648357652447101 :   0.026378589568422678 :   (b('bulk') <= 168.0) ?  (b('dg0_v_vv') <= 2.3322720527648926) ?  (b('dg0_v_vv') <= 2.100297451019287) ?  (b('ndvi') <= 1876.0) ?  0.016440523964787828 :   -0.004257083156164046 :   -0.08007557824719558 :   (b('dg0_v_vv') <= 2.953887462615967) ?  (b('l8b5') <= 3298.674560546875) ?  0.06702009082079237 :   0.08662526391757808 :   (b('dg0_v_vv') <= 3.212677001953125) ?  0.031332160171358564 :   0.01413380297091145 :   0.10472085031194361 :   (b('ndvi_med') <= 1359.25) ?  (b('l8b5_med') <= 2262.002685546875) ?  (b('l8b11') <= 2206.70556640625) ?  -0.0683161671360945 :   -0.0572305588246613 :   (b('ndvi_med') <= 1033.0) ?  -0.03471757479273607 :   (b('l8b5') <= 2679.96435546875) ?  (b('l8b11') <= 2248.3377685546875) ?  -0.016927244352479776 :   -0.020994116898143705 :   -0.030205983936896788 :   (b('ndvi') <= 2059.5) ?  (b('l8b11') <= 2905.301025390625) ?  (b('dg0_v_vv') <= -0.10923576354980469) ?  (b('l8b11') <= 2451.216552734375) ?  -0.016759529914686634 :   -0.00018360044409285875 :   (b('l8b5') <= 2722.60791015625) ?  0.022854384345687956 :   0.0012961532435667444 :   -0.04720045491246776 :   (b('dg0_v_vv') <= 0.5977458953857422) ?  0.039486310614652964 :   0.0622053470981207 :   (b('l8b4') <= 2187.3804931640625) ?  (b('l8b4') <= 2160.39404296875) ?  (b('l8b11') <= 2666.24267578125) ?  (b('moss') <= 16.22134304046631) ?  (b('l8b4') <= 1874.209228515625) ?  (b('l8b11') <= 2527.7900390625) ?  -0.001530124355231269 :   0.018458036420215724 :   (b('ndvi') <= 3270.0) ?  0.03465553403875358 :   -0.009316590109747128 :   (b('ndvi') <= 1767.0) ?  (b('dg0_v_vv') <= 1.591728687286377) ?  0.05585702253755457 :   0.026652224934710922 :   (b('l8b5') <= 3243.53857421875) ?  -0.000991184630775554 :   0.03371519012427266 :   (b('l8b11') <= 2744.506591796875) ?  (b('l8b5') <= 3760.158203125) ?  (b('dg0_v_vv') <= -3.7685604095458984) ?  0.050050581411310316 :   -0.018065981619353164 :   (b('dg0_v_vv') <= 2.339846611022949) ?  0.014741706969748459 :   0.03823169964265899 :   (b('l8b4_med') <= 1950.225341796875) ?  (b('ndvi') <= 3767.0) ?  -0.006224511224163027 :   0.0813991702644519 :   (b('l8b4') <= 2114.8494873046875) ?  0.012505731653281163 :   -0.016656691155050873 :   (b('ndvi') <= 1946.5) ?  (b('dg0_v_vv') <= -1.3020567893981934) ?  -0.0016205264476188974 :   (b('l8b11') <= 2986.9097900390625) ?  0.0670381446555022 :   (b('l8b5_med') <= 2812.056640625) ?  0.05593700046039604 :   0.03171935929491557 :   (b('dg0_v_vv') <= -0.4087700843811035) ?  (b('ndvi') <= 2277.5) ?  -0.0032475172648214617 :   (b('ndvi') <= 3741.0) ?  0.024206565342308 :   0.006546392033018378 :   (b('l8b11') <= 3127.384033203125) ?  (b('l8b11_med') <= 2890.52392578125) ?  0.00526079502809421 :   0.02961098417059846 :   (b('l8b5_med') <= 3221.940673828125) ?  -0.011897304793907368 :   -0.03909305779391302 :   (b('l8b4') <= 2193.1158447265625) ?  (b('dg0_v_vv') <= -0.13872289657592773) ?  (b('ndvi_med') <= 1569.0) ?  -0.04640310970240949 :   (b('l8b11') <= 3296.9146728515625) ?  -0.04389162163007282 :   -0.04455564671035982 :   (b('dg0_v_vv') <= 0.5375747680664062) ?  -0.05948421932024056 :   -0.06079436395649321 :   (b('l8b4_med') <= 1649.88818359375) ?  0.06592313561437049 :   (b('sand') <= 58.5) ?  (b('l8b11') <= 2652.8525390625) ?  (b('crops') <= 81.8177375793457) ?  -0.018845048663558586 :   0.03257493439761623 :   (b('ndvi') <= 1576.5) ?  0.019262836852558328 :   -0.007359825249001229 :   (b('l8b4') <= 2421.6007080078125) ?  (b('dg0_v_vv') <= 1.8263673782348633) ?  0.029938637805793533 :   0.04166554875976301 :   (b('l8b5_med') <= 3738.7178955078125) ?  0.007729082282216332 :   0.028153723780459668 :   (b('sand') <= 22.0) ?  (b('l8b4_med') <= 798.0916748046875) ?  (b('crops') <= 28.369168758392334) ?  0.005102940895955144 :   (b('bulk') <= 153.0) ?  0.015139024584785388 :   (b('ndvi_med') <= 4372.5) ?  0.01952357890453965 :   0.023275562907693836 :   (b('l8b11_med') <= 1220.2855834960938) ?  0.04618321226536465 :   (b('dg0_v_vv') <= -2.1477184295654297) ?  (b('l8b4_med') <= 813.8113098144531) ?  0.027926608932833996 :   0.016885658996808828 :   (b('ndvi') <= 1991.5) ?  (b('dg0_v_vv') <= 1.137122631072998) ?  0.027578980609590686 :   0.021650558739268355 :   0.0355202555321891 :   (b('ndvi_med') <= 1693.5) ?  (b('ndvi') <= 2852.0) ?  (b('dg0_v_vv') <= 2.6366939544677734) ?  (b('l8b11') <= 2640.076904296875) ?  (b('l8b4') <= 1842.0206909179688) ?  (b('l8b4') <= 1549.5753784179688) ?  -0.03589278920882732 :   (b('ndvi') <= 1499.0) ?  -0.012260524302826058 :   -0.023185097751418603 :   (b('dg0_v_vv') <= -0.1538558006286621) ?  (b('ndvi') <= 1495.5) ?  0.03091157148859852 :   -0.0006928496352655161 :   -0.00384287151422303 :   (b('l8b4') <= 1726.71435546875) ?  (b('dg0_v_vv') <= -0.660273551940918) ?  (b('dg0_v_vv') <= -1.9643974304199219) ?  -0.024548445159139315 :   -0.02954844411605803 :   0.03376737261344312 :   (b('l8b5') <= 3085.7216796875) ?  (b('l8b5') <= 2561.25927734375) ?  -0.04879301725549435 :   -0.039273944381631586 :   (b('dg0_v_vv') <= 2.0439138412475586) ?  -0.023084956391495055 :   -0.03735925201769835 :   (b('l8b5') <= 3304.052001953125) ?  0.09923240011468992 :   (b('l8b4') <= 1991.8963623046875) ?  0.00675120653331468 :   -0.06431726701659239 :   0.054883212420930874 :   (b('l8b5') <= 3626.6275634765625) ?  (b('ndvi') <= 2893.5) ?  (b('l8b11') <= 2661.7047119140625) ?  (b('l8b11') <= 2599.66650390625) ?  (b('l8b5') <= 3198.3909912109375) ?  (b('l8b4') <= 1416.7401733398438) ?  -0.029967799734457255 :   -0.0007407069824390245 :   (b('ndvi') <= 1256.0) ?  0.09936925268831279 :   -0.013004603673820635 :   (b('l8b5') <= 2384.431884765625) ?  (b('sand') <= 40.5) ?  -0.060888497551645944 :   -0.05336224158201108 :   (b('ndvi') <= 1133.5) ?  0.061762210148083974 :   -0.021343880372595478 :   (b('l8b5') <= 2742.6182861328125) ?  (b('l8b4') <= 1877.6596069335938) ?  (b('l8b11') <= 3225.84765625) ?  0.006076082134088903 :   -0.026421045129875108 :   (b('ndvi') <= 1177.5) ?  0.016508932135128124 :   0.0884891426190957 :   (b('l8b5') <= 3564.805908203125) ?  (b('ndvi') <= 1218.5) ?  -0.013390263853627014 :   -0.0013184432617077038 :   (b('dg0_v_vv') <= 0.08374595642089844) ?  -0.0005311552809341223 :   0.028834821073013776 :   (b('l8b5') <= 3177.6802978515625) ?  (b('l8b4_med') <= 1217.512451171875) ?  (b('l8b5') <= 2292.05859375) ?  0.06523594612203497 :   (b('dg0_v_vv') <= 3.513664960861206) ?  -0.01521770876106163 :   0.012122046304179915 :   (b('grass') <= 93.53741455078125) ?  (b('dg0_v_vv') <= -2.445366382598877) ?  0.05026804190162265 :   0.0047194217505211015 :   0.07159548379893366 :   (b('l8b5') <= 3379.218017578125) ?  (b('dg0_v_vv') <= -0.8297882080078125) ?  (b('ndvi') <= 3944.0) ?  0.018432190176522787 :   -0.019424243888758158 :   (b('grass') <= 14.177669525146484) ?  0.047408736217876495 :   0.017327145584294327 :   (b('sand') <= 33.5) ?  (b('dg0_v_vv') <= -1.1431612968444824) ?  -0.007887782097166972 :   0.03524058113502327 :   (b('dg0_v_vv') <= -1.5790700912475586) ?  -0.021735910963719723 :   -0.0027057554703871726 :   (b('l8b11') <= 2024.5573120117188) ?  (b('dg0_v_vv') <= 0.5657048225402832) ?  (b('dg0_v_vv') <= -0.43529844284057617) ?  -0.055003909049658656 :   (b('l8b11_med') <= 2235.6070556640625) ?  -0.050483216901517 :   -0.04756091479411326 :   -0.09184767477050348 :   (b('dg0_v_vv') <= 2.008065700531006) ?  (b('moss') <= 0.4768786132335663) ?  (b('dg0_v_vv') <= 1.5687522888183594) ?  (b('dg0_v_vv') <= 0.9798493385314941) ?  -0.014239047225872136 :   0.011933080498040285 :   (b('dg0_v_vv') <= 1.6481046676635742) ?  -0.025935124089995203 :   -0.042388995780049635 :   (b('l8b4_med') <= 1513.4367065429688) ?  (b('l8b4') <= 2376.15087890625) ?  0.002065878058499289 :   0.13233153627629235 :   (b('l8b5') <= 3642.00146484375) ?  0.00937236771206578 :   -0.00592178359958782 :   (b('l8b11') <= 2503.613525390625) ?  (b('sand') <= 45.0) ?  -0.08261628415596314 :   (b('l8b5_med') <= 3541.7769775390625) ?  -0.040679466163674696 :   -0.02816600688529719 :   (b('l8b4_med') <= 1555.0704345703125) ?  (b('grass') <= 7.421477794647217) ?  0.04842039169343118 :   0.008398578642929921 :   (b('sand') <= 51.0) ?  0.0018855939588680373 :   -0.024093587743263877 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi') <= 3205.5) ?  (b('dg0_v_vv') <= 4.273038387298584) ?  (b('ndvi_med') <= 3806.0) ?  (b('ndvi') <= 3192.5) ?  (b('l8b4') <= 654.5455017089844) ?  (b('grass') <= 47.03437423706055) ?  (b('l8b4') <= 632.939453125) ?  (b('l8b11_med') <= 2501.6318359375) ?  (b('ndvi') <= 3149.0) ?  (b('ndvi') <= 3082.5) ?  -0.004067815974644211 :   0.0188727873666369 :   (b('moss') <= 4.993523359298706) ?  -0.005981464961936592 :   -0.029020178129672693 :   (b('l8b5_med') <= 2328.4534301757812) ?  0.11396189051023725 :   (b('dg0_v_vv') <= 0.3001070022583008) ?  0.028751689577420775 :   -0.008039286150400428 :   (b('moss') <= 7.166860342025757) ?  (b('dg0_v_vv') <= 1.9422950744628906) ?  (b('l8b4_med') <= 789.8522644042969) ?  0.008564320584221196 :   0.032774752137273176 :   (b('l8b5_med') <= 3191.4571533203125) ?  0.1046137505582006 :   0.08277631130603574 :   (b('moss') <= 21.374034881591797) ?  (b('ndvi') <= 2688.0) ?  -0.003930291925434589 :   -0.023911843177311955 :   (b('l8b5_med') <= 2527.20849609375) ?  0.012940065090513977 :   0.04574910645082833 :   (b('bulk') <= 153.5) ?  (b('sand') <= 20.0) ?  -0.09577040768006154 :   (b('l8b4_med') <= 1614.7182006835938) ?  (b('sand') <= 28.5) ?  0.03327371528070847 :   0.004320742824820535 :   -0.08285427331594991 :   (b('ndvi') <= 2535.0) ?  (b('ndvi_med') <= 2121.0) ?  (b('l8b4') <= 597.8866882324219) ?  -0.025692099050789396 :   0.005759261295658041 :   (b('l8b5_med') <= 2272.733642578125) ?  -0.03319880579486975 :   0.02339890125392448 :   (b('dg0_v_vv') <= 0.9725549221038818) ?  (b('ndvi_med') <= 1737.0) ?  0.0635522688310882 :   0.024720031823322622 :   (b('l8b4_med') <= 898.8873596191406) ?  0.09151454939151465 :   0.05479484871730103 :   (b('l8b4') <= 657.0227966308594) ?  (b('moss') <= 6.929425477981567) ?  (b('ndvi') <= 3026.0) ?  (b('l8b11') <= 1467.302490234375) ?  -0.037619680240215114 :   -0.04574248963034619 :   (b('grass') <= 35.63461494445801) ?  -0.05131154629810464 :   -0.05957085100040224 :   (b('l8b11') <= 1281.2431030273438) ?  (b('dg0_v_vv') <= 0.07218217849731445) ?  (b('dg0_v_vv') <= 0.012451171875) ?  -0.030975259377078457 :   -0.02643051004313879 :   (b('dg0_v_vv') <= 0.37770652770996094) ?  -0.036458601348467706 :   -0.034355037564790025 :   (b('crops') <= 29.48313331604004) ?  -0.018516457314041168 :   -0.0060393690438073355 :   (b('l8b11_med') <= 947.189208984375) ?  (b('l8b11') <= 1133.9581909179688) ?  (b('ndvi') <= 2255.5) ?  0.10938307588268464 :   0.14200515730048108 :   (b('l8b11_med') <= 935.2278747558594) ?  -0.017090348598730674 :   -0.03032554403029966 :   (b('bulk') <= 109.5) ?  (b('l8b11_med') <= 1595.7366333007812) ?  (b('l8b5') <= 2519.1553955078125) ?  0.003624692712386204 :   -0.016663170209320552 :   (b('l8b11') <= 1876.93798828125) ?  -0.0422586857809247 :   0.009163118480531857 :   (b('l8b5') <= 1328.6978149414062) ?  (b('l8b4') <= 1053.017822265625) ?  0.010695308058856145 :   0.08520807582471093 :   (b('l8b5') <= 1353.1083984375) ?  -0.03170956873747104 :   -0.0009133375219887443 :   (b('l8b5_med') <= 1590.654296875) ?  (b('l8b4_med') <= 595.4280090332031) ?  0.0658295094148329 :   0.05363351709684541 :   (b('l8b11_med') <= 1540.37158203125) ?  (b('ndvi_med') <= 3271.25) ?  (b('dg0_v_vv') <= 0.7826251983642578) ?  (b('dg0_v_vv') <= -1.0717449188232422) ?  0.0031843452027279506 :   (b('dg0_v_vv') <= 0.32754993438720703) ?  -0.006654310846608284 :   -0.01098645532364078 :   (b('moss') <= 16.663246154785156) ?  (b('l8b5_med') <= 2822.3741455078125) ?  -0.02757633047541041 :   -0.029011777560473156 :   -0.04299847592990727 :   (b('bulk') <= 136.5) ?  0.029310446484390656 :   0.05823689125283851 :   (b('dg0_v_vv') <= 0.22379446029663086) ?  (b('grass') <= 27.670451164245605) ?  (b('l8b4_med') <= 1263.2464599609375) ?  (b('dg0_v_vv') <= -1.8184990882873535) ?  -0.005282419433455976 :   -0.027637587345722577 :   -0.0529230663403886 :   (b('l8b4_med') <= 942.4861145019531) ?  (b('ndvi') <= 3199.5) ?  -0.013172184550951607 :   0.0018047785559596047 :   (b('l8b4_med') <= 989.9793701171875) ?  -0.013797141584763771 :   -0.022126857365601642 :   (b('grass') <= 39.85703754425049) ?  (b('grass') <= 24.661314010620117) ?  (b('ndvi_med') <= 2171.5) ?  -0.0248796569176149 :   -0.029444035733286553 :   -0.04432338744532027 :   (b('l8b11') <= 1771.3296508789062) ?  -0.07302868856435857 :   -0.06992797793779112 :   (b('dg0_v_vv') <= -4.080380916595459) ?  (b('l8b11') <= 2116.7296142578125) ?  (b('grass') <= 33.21152877807617) ?  (b('l8b11_med') <= 1502.4168701171875) ?  -0.029661404240090727 :   -0.025216302973243704 :   -0.03433416230395374 :   -0.06779362067376854 :   (b('l8b11') <= 777.2692260742188) ?  (b('ndvi') <= 2949.0) ?  (b('l8b4_med') <= 447.916259765625) ?  -0.05003707407452193 :   -0.06900018802529453 :   (b('l8b4') <= 352.7147216796875) ?  (b('ndvi') <= 3045.0) ?  0.027844854997675805 :   0.00013943279419653676 :   (b('ndvi_med') <= 4009.0) ?  (b('l8b11') <= 682.1058044433594) ?  -0.03808368387767869 :   -0.02769460707468535 :   (b('dg0_v_vv') <= -0.9218459129333496) ?  -0.02631105615853846 :   0.016681184250702752 :   (b('ndvi_med') <= 3845.0) ?  (b('l8b4') <= 726.4019775390625) ?  0.09071382675895048 :   (b('l8b5') <= 2647.974853515625) ?  (b('grass') <= 52.136796951293945) ?  0.03388819766118267 :   0.041032757031620015 :   (b('l8b5') <= 2907.517578125) ?  (b('ndvi') <= 3011.0) ?  0.012081548687473034 :   0.02816092123875044 :   0.03549463696663466 :   (b('grass') <= 40.7613582611084) ?  (b('ndvi') <= 2791.5) ?  (b('grass') <= 34.64734649658203) ?  (b('dg0_v_vv') <= -0.16175460815429688) ?  -0.011781213645653788 :   0.012591509211278834 :   (b('ndvi') <= 2410.5) ?  0.08515639912945414 :   0.03323070287540411 :   (b('bulk') <= 116.0) ?  0.11070466237937754 :   (b('l8b4_med') <= 749.7178344726562) ?  0.028922453627765094 :   -0.005505784374387585 :   (b('crops') <= 40.368276596069336) ?  (b('ndvi') <= 3157.0) ?  (b('dg0_v_vv') <= 0.15452194213867188) ?  -0.0019850865033899817 :   0.020760201178618633 :   (b('sand') <= 49.0) ?  -0.07387985237990952 :   -0.03173001279559115 :   -0.08759333296859453 :   (b('l8b11') <= 1141.177001953125) ?  (b('sand') <= 38.5) ?  (b('sand') <= 29.5) ?  -0.02686267983337659 :   -0.00964217301074366 :   (b('crops') <= 76.14754676818848) ?  (b('dg0_v_vv') <= 7.052134037017822) ?  (b('bulk') <= 145.5) ?  -0.04931374427816915 :   -0.054940507794717625 :   -0.05776907981537291 :   -0.06388934960621956 :   (b('l8b4_med') <= 838.607177734375) ?  (b('l8b5_med') <= 2935.7869873046875) ?  (b('l8b11_med') <= 1644.19921875) ?  (b('moss') <= 10.25065302848816) ?  -0.04627704949665466 :   -0.05848029115921438 :   (b('l8b4') <= 893.8265380859375) ?  (b('dg0_v_vv') <= 7.114964962005615) ?  -0.019393609406815962 :   (b('l8b11_med') <= 1851.270263671875) ?  -0.02377705083775916 :   -0.026393612416850523 :   -0.0371266084639348 :   (b('bulk') <= 132.5) ?  (b('l8b4_med') <= 602.1842193603516) ?  -0.011735641243654726 :   0.007980709648148843 :   0.02774756015056984 :   (b('l8b4') <= 1405.278076171875) ?  (b('l8b4') <= 1402.1333618164062) ?  (b('bulk') <= 153.5) ?  (b('l8b4_med') <= 1354.9573974609375) ?  (b('sand') <= 35.0) ?  (b('moss') <= 4.59937310218811) ?  0.022228640052196363 :   0.05940631797866866 :   (b('l8b5') <= 3085.3648681640625) ?  -0.0022291013624679084 :   0.038155635782220365 :   (b('dg0_v_vv') <= 7.251966714859009) ?  (b('dg0_v_vv') <= 6.86086630821228) ?  0.025454828506645488 :   0.07858284199437032 :   (b('ndvi') <= 1598.5) ?  -0.028906204722877832 :   0.0029905773467812646 :   (b('l8b5') <= 1795.6921997070312) ?  (b('dg0_v_vv') <= 5.40849757194519) ?  0.004911905666002747 :   -0.05934596031707767 :   (b('l8b4') <= 744.8092041015625) ?  (b('dg0_v_vv') <= 4.629181146621704) ?  0.007422218992069685 :   0.023866272416264434 :   (b('moss') <= 24.150099754333496) ?  0.06436758306087767 :   0.08260454057041311 :   0.10938920351789017 :   (b('sand') <= 52.5) ?  (b('l8b11_med') <= 2825.4913330078125) ?  (b('moss') <= 3.7363287210464478) ?  (b('dg0_v_vv') <= 6.306273698806763) ?  (b('grass') <= 55.26011562347412) ?  -0.06760401571942591 :   -0.040672177450180855 :   (b('l8b4') <= 1810.3323974609375) ?  0.009241186442375593 :   -0.018609638374832538 :   (b('l8b4_med') <= 1399.4530029296875) ?  0.021659980249570276 :   (b('dg0_v_vv') <= 4.512969017028809) ?  -0.001691183970584859 :   -0.01684389907193243 :   (b('sand') <= 46.5) ?  (b('dg0_v_vv') <= 5.427125930786133) ?  (b('dg0_v_vv') <= 4.767615556716919) ?  0.019819410937524283 :   0.038453875976458066 :   (b('ndvi_med') <= 1552.25) ?  -0.006885057912806192 :   0.005014417566579665 :   (b('l8b11') <= 3603.4991455078125) ?  (b('l8b11_med') <= 3094.5821533203125) ?  -0.006386848515553656 :   -0.01613532620975386 :   0.020028988874442613 :   (b('l8b5') <= 2809.773681640625) ?  (b('dg0_v_vv') <= 5.919033050537109) ?  (b('crops') <= 93.97411346435547) ?  0.04029420478463448 :   0.028758505293262154 :   (b('dg0_v_vv') <= 6.709964752197266) ?  (b('l8b11') <= 2789.222412109375) ?  0.013388324743948965 :   0.011201120811168835 :   (b('dg0_v_vv') <= 7.044743061065674) ?  0.052036769681894024 :   0.015821579927186996 :   (b('dg0_v_vv') <= 4.418736696243286) ?  0.04092449149583466 :   (b('dg0_v_vv') <= 4.9464685916900635) ?  (b('l8b4') <= 2151.28076171875) ?  -0.009943787491995329 :   -0.008068270942355099 :   (b('ndvi_med') <= 1979.5) ?  0.00922456242353599 :   0.03396581064335963 :   (b('ndvi') <= 3252.5) ?  (b('bulk') <= 153.0) ?  (b('ndvi') <= 3248.5) ?  (b('l8b5_med') <= 2086.1666259765625) ?  0.11916680415846725 :   (b('bulk') <= 151.5) ?  (b('ndvi_med') <= 3937.5) ?  (b('l8b11_med') <= 2395.637939453125) ?  (b('bulk') <= 140.5) ?  (b('ndvi_med') <= 2976.0) ?  0.02132533647239564 :   0.005580127139899696 :   (b('l8b4') <= 1034.2633666992188) ?  0.021501043156091183 :   0.08310095909002949 :   (b('l8b4') <= 1375.3922729492188) ?  (b('dg0_v_vv') <= -0.6531834602355957) ?  -0.01078418100537017 :   -0.02415246870406734 :   (b('l8b11') <= 3024.8448486328125) ?  -0.005844775025204463 :   0.006759623375378811 :   (b('l8b4') <= 540.8590850830078) ?  -0.06344590360535918 :   (b('l8b4_med') <= 666.0265502929688) ?  -0.018168904391909768 :   -0.028662789190687488 :   (b('dg0_v_vv') <= 1.8436484336853027) ?  (b('dg0_v_vv') <= 0.13671255111694336) ?  (b('dg0_v_vv') <= -0.3756561279296875) ?  (b('l8b4') <= 1148.1896667480469) ?  0.057595384322165044 :   0.06228965015408611 :   0.04996721840183005 :   0.08605144494659744 :   0.030587410711383434 :   (b('l8b5_med') <= 2918.44873046875) ?  (b('dg0_v_vv') <= -0.21954107284545898) ?  (b('crops') <= 7.7792205810546875) ?  0.11355592811248794 :   0.06362983516715581 :   (b('l8b11_med') <= 1717.0394897460938) ?  (b('l8b11_med') <= 1090.9612426757812) ?  (b('dg0_v_vv') <= 0.0876455307006836) ?  0.04670550984441768 :   0.05173994512268648 :   0.0416140700580275 :   0.060585881148875476 :   0.022560886177335204 :   (b('l8b5_med') <= 2902.5633544921875) ?  (b('dg0_v_vv') <= 0.12882328033447266) ?  (b('l8b4') <= 728.1512451171875) ?  -0.0856828218711159 :   -0.09069383953906299 :   (b('ndvi') <= 3234.5) ?  -0.05230019082919868 :   -0.06046956947979261 :   (b('l8b11_med') <= 2027.2733764648438) ?  (b('ndvi_med') <= 3305.25) ?  -0.026897194669722416 :   (b('l8b11_med') <= 1974.8834228515625) ?  (b('l8b5_med') <= 3251.63232421875) ?  -0.018901624949223927 :   -0.021300478283097002 :   -0.01591694390154058 :   -0.03330466732575521 :   (b('sand') <= 37.5) ?  (b('l8b5_med') <= 2494.7696533203125) ?  (b('ndvi') <= 4417.0) ?  (b('ndvi') <= 4123.5) ?  (b('l8b5_med') <= 2390.8360595703125) ?  (b('l8b5') <= 2177.6258544921875) ?  (b('l8b4') <= 960.1278991699219) ?  (b('l8b4') <= 856.205322265625) ?  0.022766869921367235 :   0.06891585380117161 :   (b('ndvi_med') <= 2232.0) ?  -0.06258031737971523 :   -0.0027001281518084508 :   (b('l8b5') <= 2301.9986572265625) ?  (b('ndvi') <= 3369.5) ?  -0.033401093095181494 :   -0.05155765481240223 :   (b('l8b11') <= 1649.2020263671875) ?  0.000932938789749241 :   0.04750758437636989 :   (b('dg0_v_vv') <= 0.6081995964050293) ?  (b('l8b5') <= 3141.4940185546875) ?  (b('l8b5_med') <= 2456.6793212890625) ?  -0.021205758828966165 :   0.02507813713873029 :   (b('ndvi') <= 3941.0) ?  0.04300083757270773 :   0.07702051054914824 :   (b('l8b4_med') <= 1265.2591552734375) ?  (b('l8b11') <= 1487.200927734375) ?  0.05898277577470965 :   0.09372016374570294 :   0.011558029402536307 :   (b('l8b11') <= 1787.2384033203125) ?  (b('ndvi') <= 4173.0) ?  (b('l8b4') <= 750.7129516601562) ?  -0.02357593385388343 :   -0.03643221518035403 :   (b('l8b5') <= 2945.583251953125) ?  (b('sand') <= 31.0) ?  -0.0684305987228544 :   -0.05757592825104682 :   -0.04027932387118788 :   0.02753180948105985 :   (b('l8b11') <= 1266.6892700195312) ?  0.032422956568966155 :   (b('grass') <= 61.90078353881836) ?  0.0843041849317078 :   (b('dg0_v_vv') <= -0.34331464767456055) ?  0.07650274809898411 :   0.07318461959017214 :   (b('l8b11') <= 2562.07373046875) ?  (b('l8b11_med') <= 1132.2691650390625) ?  (b('l8b11') <= 1116.1423950195312) ?  (b('dg0_v_vv') <= -0.3759427070617676) ?  (b('dg0_v_vv') <= -0.6243395805358887) ?  (b('crops') <= 34.6722412109375) ?  0.00595760057755675 :   0.021480869951461257 :   (b('l8b4') <= 536.7644348144531) ?  -0.023593063529152553 :   -0.09010790125928997 :   (b('l8b4') <= 445.7762756347656) ?  (b('l8b5_med') <= 4072.6116943359375) ?  0.006494993254164712 :   -0.06721372097281422 :   (b('l8b11') <= 1078.024169921875) ?  0.05825075532259027 :   0.020223754096218932 :   (b('l8b11') <= 1277.4573974609375) ?  (b('l8b11') <= 1212.3465576171875) ?  (b('dg0_v_vv') <= 0.5723276138305664) ?  -0.015225032947979372 :   0.020343478949549743 :   (b('dg0_v_vv') <= 0.6523423194885254) ?  -0.061418759252541644 :   -0.055851420524976686 :   (b('l8b11') <= 1298.3547973632812) ?  (b('ndvi') <= 3962.0) ?  0.047565671019271005 :   0.1051980276297822 :   (b('l8b5') <= 3960.9197998046875) ?  0.009985147067686453 :   -0.03610323202676485 :   (b('ndvi') <= 4254.0) ?  (b('ndvi') <= 4093.0) ?  (b('ndvi') <= 4020.5) ?  (b('bulk') <= 126.5) ?  -0.024884824939549577 :   0.00047720491994685145 :   (b('grass') <= 37.385244369506836) ?  0.02241654573372345 :   -0.0183088063912888 :   (b('l8b5') <= 4058.1007080078125) ?  (b('ndvi') <= 4128.5) ?  -0.023307883790669073 :   0.0005870563693108174 :   (b('ndvi_med') <= 3732.75) ?  -0.024364646775743605 :   -0.07007863442932201 :   (b('l8b11') <= 1922.7572021484375) ?  (b('l8b11') <= 1735.0038452148438) ?  (b('dg0_v_vv') <= 2.205618381500244) ?  0.004035417567624756 :   0.013925392889864418 :   (b('sand') <= 34.5) ?  0.01829544253582435 :   0.08656348698540745 :   (b('sand') <= 26.0) ?  (b('sand') <= 22.0) ?  -0.028339235756998364 :   -0.05506241418430598 :   (b('ndvi') <= 4311.0) ?  0.04092379613259501 :   -0.0022209000621494397 :   (b('sand') <= 28.0) ?  (b('ndvi') <= 4460.5) ?  (b('l8b5') <= 3396.1912841796875) ?  -0.012647928114636692 :   -0.01765770807862277 :   -0.06744789705805201 :   (b('ndvi') <= 4202.0) ?  (b('ndvi') <= 3693.5) ?  (b('ndvi') <= 3601.0) ?  0.03454086587274233 :   (b('ndvi_med') <= 3671.5) ?  0.01896691578922155 :   0.010552128236662683 :   (b('l8b11') <= 2585.937255859375) ?  0.07906268004693437 :   (b('dg0_v_vv') <= -2.315082550048828) ?  0.07015349842486301 :   0.049429034934496796 :   (b('l8b11_med') <= 2247.2081298828125) ?  (b('l8b4') <= 1285.8504638671875) ?  (b('ndvi_med') <= 2492.5) ?  0.027392418544217464 :   0.04870646808364995 :   0.022101188911852865 :   -0.021627724305597812 :   (b('grass') <= 77.33748626708984) ?  (b('bulk') <= 129.5) ?  (b('l8b4') <= 389.1341094970703) ?  (b('l8b4_med') <= 680.7699890136719) ?  (b('crops') <= 77.24960327148438) ?  (b('ndvi_med') <= 3405.0) ?  (b('dg0_v_vv') <= -1.3384966850280762) ?  0.033479677626279264 :   0.013835111857879778 :   (b('grass') <= 17.71795082092285) ?  -0.04081853597624216 :   0.0020770695373580673 :   (b('ndvi_med') <= 3656.75) ?  0.008434186521872555 :   (b('moss') <= 2.551342811435461) ?  0.02819158842421962 :   0.04377918418422469 :   (b('l8b11_med') <= 1482.7119750976562) ?  (b('ndvi_med') <= 3671.5) ?  0.041807508046827035 :   (b('ndvi_med') <= 3892.0) ?  0.08695713525259596 :   0.11466239405146866 :   (b('l8b5_med') <= 2615.601806640625) ?  (b('l8b4_med') <= 814.8666687011719) ?  0.023580817190039294 :   0.007225970308456911 :   (b('ndvi_med') <= 3656.75) ?  0.005863693325598468 :   -0.007484467530813911 :   (b('l8b4') <= 391.82395935058594) ?  -0.08717148191535734 :   (b('crops') <= 0.44501277804374695) ?  (b('ndvi') <= 4921.5) ?  (b('bulk') <= 101.5) ?  0.03567706644135565 :   -0.004495286645647941 :   (b('ndvi_med') <= 3494.0) ?  -0.028031217964601767 :   -0.07619100386772082 :   (b('l8b4') <= 409.7068634033203) ?  (b('moss') <= 0.2564399056136608) ?  -0.03131304226868548 :   0.0025510797911976516 :   (b('ndvi_med') <= 3405.0) ?  0.012695638012301672 :   0.0018940244046337061 :   (b('l8b4') <= 1050.5463256835938) ?  (b('moss') <= 16.43188238143921) ?  (b('l8b5') <= 3134.6358642578125) ?  (b('l8b4') <= 661.4646301269531) ?  (b('ndvi') <= 4266.5) ?  0.005244561061647878 :   -0.021507295953860424 :   (b('l8b4') <= 984.5281982421875) ?  -0.01467681048374024 :   0.0037941391609496796 :   (b('l8b11') <= 1291.8320922851562) ?  (b('grass') <= 68.83333206176758) ?  -0.004620758597982007 :   -0.059493082911171724 :   (b('l8b11') <= 1326.953369140625) ?  0.0408962039115631 :   0.0009202622268375148 :   (b('ndvi') <= 3583.5) ?  (b('ndvi') <= 3355.5) ?  (b('l8b11') <= 1813.052490234375) ?  0.034394470990027855 :   8.010690265725096e-05 :   (b('grass') <= 58.14594268798828) ?  0.06484749580998912 :   0.09602547899445421 :   (b('l8b5') <= 2567.621337890625) ?  -0.09614783804262345 :   (b('grass') <= 53.165571212768555) ?  -0.03149402445353061 :   0.010673942793249458 :   (b('l8b4') <= 1057.6300048828125) ?  (b('dg0_v_vv') <= 0.2255239486694336) ?  (b('dg0_v_vv') <= -0.04720878601074219) ?  (b('ndvi') <= 3594.0) ?  0.11091963191348259 :   0.02388246798447243 :   (b('l8b4_med') <= 1290.2487182617188) ?  0.1561792785538818 :   0.12314131294312813 :   (b('l8b5_med') <= 3211.497802734375) ?  (b('dg0_v_vv') <= 1.5630836486816406) ?  -0.026472190040218946 :   0.008579164322110727 :   (b('dg0_v_vv') <= 7.690528869628906) ?  0.02287618068225239 :   -0.00910534818681455 :   (b('l8b11') <= 1598.3016357421875) ?  (b('ndvi') <= 3458.5) ?  (b('dg0_v_vv') <= 3.4722933769226074) ?  -0.04342426441208675 :   -0.08090830145575859 :   0.02605820594545097 :   (b('ndvi') <= 4415.0) ?  (b('grass') <= 37.77153396606445) ?  -0.0011111355218525654 :   0.01991215752833812 :   (b('dg0_v_vv') <= -1.54587984085083) ?  0.048365431429324375 :   0.010973074817271543 :   (b('l8b4_med') <= 1297.6776123046875) ?  (b('ndvi_med') <= 2244.0) ?  (b('ndvi') <= 3492.5) ?  (b('l8b4') <= 944.9612121582031) ?  (b('l8b11_med') <= 2075.7976684570312) ?  (b('dg0_v_vv') <= 3.12095308303833) ?  -0.0010548126113735482 :   0.03851944524772 :   (b('bulk') <= 148.0) ?  -0.03843420550242638 :   -0.026073241254963575 :   (b('l8b4') <= 1051.8778686523438) ?  0.0438325410805748 :   (b('ndvi_med') <= 2229.0) ?  0.01859079518294483 :   0.026761133680686222 :   (b('ndvi') <= 3849.5) ?  (b('l8b11_med') <= 2032.2171630859375) ?  (b('l8b11') <= 1153.5039672851562) ?  0.06862633220016151 :   0.08677767301016541 :   -0.0076122169249565885 :   (b('l8b4') <= 419.9159851074219) ?  -0.005045121681609116 :   (b('l8b11') <= 1804.2605590820312) ?  0.03673182610877533 :   0.007762607088117206 :   (b('l8b11') <= 1172.5109252929688) ?  (b('dg0_v_vv') <= 0.639622688293457) ?  0.06409047701106413 :   (b('l8b4') <= 607.8292541503906) ?  0.030740919114566723 :   0.036278080971923476 :   (b('ndvi') <= 3321.0) ?  (b('grass') <= 80.35536193847656) ?  (b('l8b5') <= 2393.24267578125) ?  -0.01946070705069683 :   0.012360500222507231 :   (b('l8b4') <= 898.072265625) ?  0.05470821894001372 :   0.04192541111153214 :   (b('ndvi') <= 3873.5) ?  (b('l8b4') <= 767.124267578125) ?  -0.035881490023162876 :   -0.00076672392567299 :   (b('dg0_v_vv') <= -1.3782830238342285) ?  -0.038103522813167125 :   0.014569320381802588 :   (b('l8b5') <= 3197.7479248046875) ?  (b('dg0_v_vv') <= 0.2210254669189453) ?  (b('ndvi_med') <= 1591.0) ?  (b('l8b4') <= 644.574462890625) ?  -0.05637220867222917 :   -0.06038025267682129 :   -0.04207078640159401 :   (b('l8b4') <= 912.0252990722656) ?  -0.02125279968276933 :   -0.02651231460105291 :   (b('dg0_v_vv') <= 2.7384190559387207) ?  0.01753143207090152 :   0.006219481825213868 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('l8b11_med') <= 1640.7611083984375) ?  (b('grass') <= 51.46572303771973) ?  (b('l8b11') <= 1345.3546142578125) ?  (b('l8b11') <= 1330.6435546875) ?  (b('bulk') <= 151.5) ?  (b('grass') <= 50.63176918029785) ?  (b('l8b11') <= 1328.6815185546875) ?  (b('l8b4') <= 783.7562561035156) ?  (b('ndvi') <= 1170.5) ?  (b('l8b5') <= 2188.4293212890625) ?  -0.07567149450118313 :   -0.014659009593667394 :   (b('l8b4') <= 642.33544921875) ?  0.00039448907904920813 :   -0.005376956948722846 :   (b('l8b11_med') <= 1589.20361328125) ?  (b('ndvi') <= 4551.0) ?  0.013749954860550088 :   -0.020083569531741553 :   (b('l8b4_med') <= 831.9432678222656) ?  -0.05701009443188526 :   -0.011085251202515134 :   (b('l8b11_med') <= 1199.6948852539062) ?  0.024889663950775562 :   (b('l8b5') <= 2997.88427734375) ?  0.05998822746345346 :   0.07004333839458646 :   (b('dg0_v_vv') <= -1.4022831916809082) ?  (b('ndvi') <= 4412.0) ?  -0.01542922304984995 :   0.07922295816528496 :   (b('dg0_v_vv') <= -0.9710440635681152) ?  -0.016196080439664085 :   (b('ndvi') <= 4708.5) ?  (b('l8b4_med') <= 658.8023376464844) ?  -0.04654185766453775 :   -0.06864830172338321 :   (b('dg0_v_vv') <= -0.4816904067993164) ?  -0.03499921249296711 :   -0.04093834025499041 :   (b('l8b4') <= 409.6408996582031) ?  (b('l8b5') <= 2834.5682373046875) ?  (b('l8b5') <= 2545.08349609375) ?  (b('l8b11') <= 833.4495544433594) ?  (b('l8b5') <= 1785.89306640625) ?  0.021360983672339323 :   -0.0057863325882121265 :   (b('l8b11') <= 872.4998779296875) ?  0.045091809037838604 :   0.019935367105229806 :   (b('moss') <= 11.532671451568604) ?  -0.008105202306843196 :   (b('l8b11') <= 867.0726318359375) ?  -0.025128426518372354 :   -0.03322966441708328 :   (b('grass') <= 34.9658088684082) ?  (b('ndvi') <= 4424.0) ?  (b('l8b11_med') <= 1003.6998901367188) ?  0.028563469799272073 :   0.03080178987743548 :   0.037829586532935805 :   0.05121373240495819 :   (b('grass') <= 43.79447555541992) ?  (b('l8b11_med') <= 1464.574951171875) ?  (b('dg0_v_vv') <= 0.4200420379638672) ?  (b('ndvi_med') <= 4372.5) ?  -0.04184685703772237 :   -0.005659322898171282 :   (b('ndvi') <= 4400.5) ?  -0.034323649455617784 :   0.031723205092430165 :   (b('ndvi') <= 4125.0) ?  (b('l8b4') <= 671.3417358398438) ?  0.007099010527790562 :   -0.013197917152695632 :   (b('l8b5_med') <= 3012.455322265625) ?  -0.03530555724833491 :   0.042012224542501475 :   (b('sand') <= 39.5) ?  (b('l8b11') <= 929.72216796875) ?  (b('ndvi_med') <= 2419.75) ?  -0.05227843597305945 :   -0.014520088944767563 :   (b('dg0_v_vv') <= -0.4651918411254883) ?  0.03487638847188341 :   0.0020400889458835007 :   (b('ndvi') <= 2390.5) ?  (b('l8b11') <= 1235.2080078125) ?  -0.034253252117510666 :   -0.014954600877187356 :   (b('dg0_v_vv') <= 0.34978175163269043) ?  -0.01508282338631299 :   0.01653336628387731 :   (b('bulk') <= 141.5) ?  (b('l8b11_med') <= 1344.731689453125) ?  (b('l8b11_med') <= 1132.2691650390625) ?  (b('dg0_v_vv') <= 0.8540544509887695) ?  (b('ndvi_med') <= 3569.0) ?  (b('dg0_v_vv') <= -0.26538705825805664) ?  0.0008931328283643958 :   0.008239813299397818 :   -0.010391306427835562 :   -0.027535353190034723 :   (b('dg0_v_vv') <= 0.5878047943115234) ?  (b('dg0_v_vv') <= -1.8905377388000488) ?  0.0015000639752346945 :   (b('l8b5_med') <= 2688.4058837890625) ?  -0.03568909053083712 :   -0.02219077985522056 :   (b('dg0_v_vv') <= 2.3965654373168945) ?  (b('l8b11_med') <= 1190.9559326171875) ?  -0.049346113596950236 :   -0.037271946671194045 :   -0.06228779821234999 :   (b('ndvi') <= 4170.0) ?  (b('l8b5') <= 3502.6334228515625) ?  (b('dg0_v_vv') <= -1.309354305267334) ?  (b('dg0_v_vv') <= -1.3703856468200684) ?  -0.0010535694014081054 :   0.03537664483631725 :   (b('l8b11') <= 1339.3818359375) ?  -0.020595343852129955 :   0.0063576777017426145 :   (b('dg0_v_vv') <= -0.6260361671447754) ?  (b('ndvi_med') <= 3856.0) ?  -0.01871335106953345 :   -0.013793405983741597 :   (b('dg0_v_vv') <= 0.06946182250976562) ?  -0.043869896918456115 :   -0.02753998021959853 :   (b('dg0_v_vv') <= 0.43488383293151855) ?  (b('dg0_v_vv') <= -0.9166450500488281) ?  (b('dg0_v_vv') <= -1.1939325332641602) ?  0.003526773147114526 :   -0.0026983056212301765 :   (b('grass') <= 20.210326194763184) ?  0.009302907326199712 :   0.03864766018808485 :   (b('ndvi') <= 4213.0) ?  0.050297573726189954 :   (b('ndvi') <= 4343.5) ?  -0.044363114184941446 :   -0.013016577291460586 :   (b('dg0_v_vv') <= 0.8633513450622559) ?  (b('l8b4') <= 711.0724487304688) ?  (b('l8b4_med') <= 731.5475769042969) ?  (b('ndvi') <= 3676.5) ?  (b('moss') <= 1.8777430057525635) ?  0.05260371414262499 :   0.049598693328193166 :   0.0388212460857737 :   0.07067187727375035 :   (b('l8b11_med') <= 1338.4380493164062) ?  0.014106028880925103 :   -0.009736532661252628 :   (b('l8b5') <= 2897.1629638671875) ?  -0.0239782187264187 :   -0.0049161603158822675 :   (b('l8b5') <= 3686.8837890625) ?  (b('dg0_v_vv') <= -2.0247244834899902) ?  (b('l8b4') <= 710.6752014160156) ?  (b('l8b11') <= 1555.004150390625) ?  (b('moss') <= 5.807535886764526) ?  (b('l8b4_med') <= 542.1248779296875) ?  (b('l8b4_med') <= 491.9370574951172) ?  -0.0483041332887412 :   -0.06286347856995367 :   (b('ndvi') <= 4642.0) ?  -0.03612993679091433 :   -0.013458214049307082 :   -0.08358605751214432 :   (b('sand') <= 57.0) ?  (b('l8b4_med') <= 694.1253356933594) ?  (b('ndvi') <= 2693.0) ?  -0.020964403481665034 :   -0.020200516361957604 :   (b('ndvi') <= 3493.0) ?  -0.022313948948277193 :   -0.02342823050038577 :   (b('l8b5') <= 3230.005859375) ?  0.01602186802720887 :   0.007902523978763287 :   (b('grass') <= 6.500641584396362) ?  (b('l8b5') <= 3245.176025390625) ?  (b('l8b4') <= 1351.6866455078125) ?  (b('l8b11') <= 2259.2666015625) ?  0.012321014132676812 :   0.005367785134896057 :   (b('ndvi') <= 1915.0) ?  0.005781763655265399 :   -0.0058102386094504065 :   0.04192930625130015 :   (b('l8b4') <= 722.5821838378906) ?  0.05629164965268557 :   (b('ndvi') <= 2524.0) ?  (b('l8b4') <= 873.3477783203125) ?  -0.007917613669180479 :   -0.02276470148462734 :   (b('ndvi') <= 2935.5) ?  0.008798125695015976 :   -0.009353683606348884 :   (b('grass') <= 49.72250747680664) ?  (b('l8b5') <= 2232.9705810546875) ?  (b('ndvi_med') <= 3165.0) ?  (b('grass') <= 35.89422035217285) ?  (b('l8b5_med') <= 2832.1903076171875) ?  0.006636968869533898 :   -0.004955182920282119 :   (b('ndvi') <= 2087.5) ?  0.010634972116131486 :   -0.038494628127199476 :   (b('dg0_v_vv') <= -0.07168817520141602) ?  (b('l8b5') <= 2132.82958984375) ?  0.023663309720018393 :   -2.9698210082261533e-05 :   (b('l8b11') <= 2688.887939453125) ?  0.008339091020337568 :   -0.029377831159935155 :   (b('l8b11') <= 1639.8048095703125) ?  (b('sand') <= 38.5) ?  (b('l8b11') <= 1639.5560302734375) ?  -0.0035836938614897305 :   -0.0687260346870822 :   (b('l8b11') <= 1351.1993408203125) ?  -0.036438466799341776 :   0.008322888009903568 :   (b('l8b11') <= 1671.939208984375) ?  (b('ndvi') <= 2409.5) ?  -0.0037146547206440292 :   0.02105300369295594 :   (b('l8b4_med') <= 735.5874938964844) ?  -0.005598749693885367 :   0.004166856578081795 :   (b('l8b5') <= 3468.4532470703125) ?  (b('l8b5') <= 3310.8173828125) ?  (b('ndvi') <= 3895.0) ?  (b('l8b5_med') <= 3155.912109375) ?  0.018255836748752197 :   0.055855939785307326 :   (b('l8b5') <= 3259.505126953125) ?  0.010847657171796306 :   -0.07567925886368779 :   (b('ndvi') <= 3384.5) ?  (b('dg0_v_vv') <= -0.5288982391357422) ?  0.02284840155557343 :   0.08077295014158134 :   (b('ndvi') <= 4086.0) ?  -0.03342623267610048 :   0.03611228452606058 :   (b('moss') <= 7.512502312660217) ?  -0.02601384681687635 :   -0.10983479391674625 :   (b('l8b5') <= 3756.993896484375) ?  (b('l8b11_med') <= 1369.6777954101562) ?  (b('ndvi') <= 4458.5) ?  (b('l8b5') <= 3710.7840576171875) ?  (b('dg0_v_vv') <= 0.2499523162841797) ?  (b('dg0_v_vv') <= -2.1477184295654297) ?  0.02045175023391051 :   0.03192179970251985 :   0.019310518796841064 :   (b('dg0_v_vv') <= 0.26821327209472656) ?  -0.006749056777998058 :   (b('dg0_v_vv') <= 0.8410844802856445) ?  0.02065497415435924 :   0.00929149144997981 :   0.053127529476915125 :   (b('dg0_v_vv') <= -0.1819615364074707) ?  (b('ndvi_med') <= 3728.0) ?  0.08048684966639877 :   0.06090599460098112 :   (b('l8b5') <= 3730.1446533203125) ?  (b('l8b5_med') <= 2937.6856689453125) ?  (b('dg0_v_vv') <= 0.8743677139282227) ?  0.029177412899567712 :   0.07796050436018659 :   (b('l8b5_med') <= 3120.56201171875) ?  0.03251767545564871 :   0.01665202741387714 :   0.07822985364921015 :   (b('ndvi_med') <= 3167.0) ?  (b('ndvi') <= 3847.0) ?  (b('dg0_v_vv') <= 1.2403359413146973) ?  (b('moss') <= 5.490723475813866) ?  (b('dg0_v_vv') <= 0.5328993797302246) ?  0.03947457621557468 :   0.026002878474027324 :   (b('ndvi_med') <= 3116.0) ?  0.016697864728211415 :   0.005957223853888707 :   -0.025283474980186044 :   (b('dg0_v_vv') <= -0.7945728302001953) ?  (b('l8b4') <= 728.5669555664062) ?  -0.0005381637129506234 :   (b('bulk') <= 133.5) ?  0.015106586676782802 :   0.02513943431556534 :   (b('dg0_v_vv') <= 0.3113746643066406) ?  (b('dg0_v_vv') <= 0.2527010440826416) ?  0.03950152245143456 :   0.00690192879402321 :   (b('ndvi_med') <= 3136.25) ?  0.0467350009369777 :   0.06196817332000368 :   (b('l8b4') <= 690.3254699707031) ?  (b('dg0_v_vv') <= 0.8971905708312988) ?  (b('dg0_v_vv') <= 0.6379220485687256) ?  (b('dg0_v_vv') <= 0.5858063697814941) ?  0.03663341450571322 :   -0.002727079894264378 :   0.09411228664224469 :   (b('dg0_v_vv') <= 1.3003854751586914) ?  (b('ndvi') <= 3944.5) ?  0.014817815189041295 :   -0.05895191054341384 :   (b('l8b5') <= 4064.3447265625) ?  0.05637207654578513 :   0.025491919107958283 :   (b('dg0_v_vv') <= -0.5305871963500977) ?  (b('bulk') <= 131.5) ?  (b('l8b4') <= 863.049072265625) ?  0.011010910319532746 :   -0.016075730196781338 :   (b('ndvi') <= 4265.5) ?  0.03902731356659131 :   0.014486777995506983 :   (b('l8b4_med') <= 664.8934631347656) ?  (b('l8b5') <= 4112.411865234375) ?  -0.04541038550360333 :   -0.013552075177213137 :   (b('ndvi') <= 2383.5) ?  0.04818724531176061 :   -0.004255675229103332 :   (b('l8b4_med') <= 1042.8488464355469) ?  (b('ndvi') <= 4978.0) ?  (b('ndvi') <= 4615.5) ?  (b('sand') <= 22.5) ?  (b('l8b4_med') <= 815.4116516113281) ?  (b('dg0_v_vv') <= 2.6807799339294434) ?  (b('ndvi') <= 3784.0) ?  (b('l8b11') <= 1138.3272705078125) ?  0.024871350151997863 :   -0.024274789084553747 :   (b('dg0_v_vv') <= 1.5571980476379395) ?  -0.036093828623570065 :   -0.09335496646117476 :   (b('dg0_v_vv') <= 3.008924961090088) ?  0.04718571393285231 :   (b('ndvi_med') <= 3863.5) ?  0.005071922724764477 :   0.01077701100631312 :   (b('dg0_v_vv') <= 0.5094723701477051) ?  (b('l8b4') <= 875.2140197753906) ?  (b('dg0_v_vv') <= -0.7197432518005371) ?  -0.0105905923252323 :   -0.05739065273970295 :   (b('ndvi') <= 3241.5) ?  0.03745491962191906 :   -0.004062879556690026 :   (b('l8b11') <= 1814.9221801757812) ?  (b('l8b5') <= 2577.299072265625) ?  0.08409689577099982 :   0.04731097350480471 :   -0.03622527763050021 :   (b('l8b11') <= 1802.47705078125) ?  (b('l8b11') <= 1713.6136474609375) ?  (b('l8b11') <= 1702.2279052734375) ?  (b('l8b11') <= 1635.7426147460938) ?  0.004832440663359982 :   -0.007223431891624497 :   (b('l8b5') <= 2811.029052734375) ?  0.0401824472397414 :   0.13921072751862276 :   (b('ndvi_med') <= 2409.5) ?  (b('ndvi_med') <= 2132.75) ?  -0.010684972173693892 :   0.035060044233210616 :   (b('l8b4') <= 1162.9622192382812) ?  -0.030952080837729826 :   0.04818422019527918 :   (b('l8b4_med') <= 864.517578125) ?  (b('l8b4_med') <= 794.385986328125) ?  (b('ndvi') <= 3151.5) ?  0.008891117976572787 :   0.038807759593195025 :   (b('bulk') <= 134.5) ?  0.07857626551790259 :   0.02762847790699944 :   (b('l8b4') <= 1223.67529296875) ?  (b('ndvi') <= 963.5) ?  0.07345441562540472 :   0.008588372443993434 :   (b('ndvi') <= 3186.5) ?  -0.005333650185234033 :   -0.06060795733574942 :   (b('dg0_v_vv') <= 1.3579082489013672) ?  (b('ndvi') <= 4640.5) ?  (b('l8b11') <= 1644.5986938476562) ?  (b('bulk') <= 152.5) ?  0.05647244429965448 :   0.02394357064078917 :   0.105582635221035 :   (b('ndvi') <= 4697.5) ?  (b('dg0_v_vv') <= -0.013019561767578125) ?  (b('ndvi') <= 4670.0) ?  -0.0052444622047081585 :   0.018156311522817492 :   (b('ndvi_med') <= 3938.5) ?  -0.05830304003158415 :   -0.036159592710645255 :   (b('ndvi') <= 4756.5) ?  (b('dg0_v_vv') <= 0.6096711158752441) ?  0.041986632306456925 :   0.02508739938134354 :   (b('dg0_v_vv') <= -0.08648824691772461) ?  0.004065044186772356 :   0.021749793881847078 :   (b('dg0_v_vv') <= 2.3026223182678223) ?  (b('dg0_v_vv') <= 2.0485310554504395) ?  (b('ndvi_med') <= 3536.0) ?  0.049121520544194 :   0.039322570061196355 :   (b('l8b5_med') <= 2697.76220703125) ?  0.10244613471424618 :   0.078176299694376 :   -0.01344950547134166 :   (b('dg0_v_vv') <= 1.1251072883605957) ?  (b('dg0_v_vv') <= -0.15600824356079102) ?  (b('dg0_v_vv') <= -0.5786633491516113) ?  (b('dg0_v_vv') <= -0.9180164337158203) ?  -0.06346517774426624 :   0.05756812373482256 :   -0.08753516440102277 :   0.0583658143891822 :   -0.08875702623347104 :   (b('l8b4') <= 1351.9804077148438) ?  0.04481920083236701 :   (b('l8b5') <= 2834.6019287109375) ?  0.07378018403496958 :   0.072290763078301 :   (b('l8b4_med') <= 847.762939453125) ?  (b('ndvi') <= 1109.5) ?  (b('ndvi') <= 1042.0) ?  0.012250455078265776 :   0.08711558825315727 :   (b('l8b5') <= 1386.0625610351562) ?  0.10966984760077615 :   (b('l8b5') <= 1949.2725219726562) ?  (b('ndvi') <= 2389.5) ?  (b('l8b4') <= 930.7284240722656) ?  (b('ndvi') <= 1564.5) ?  (b('l8b5') <= 1398.170654296875) ?  (b('l8b4_med') <= 807.3660278320312) ?  -0.03370666451146272 :   -0.052117382239711574 :   (b('ndvi') <= 1228.0) ?  -0.007740017614074418 :   -0.021196618576916616 :   (b('l8b11') <= 1572.2913208007812) ?  (b('dg0_v_vv') <= 1.3191399574279785) ?  -0.005673306562954768 :   -0.037953513688901375 :   (b('l8b11') <= 1985.2791137695312) ?  0.00881763439921639 :   -0.008729187679698734 :   (b('dg0_v_vv') <= -0.2009587287902832) ?  (b('l8b4') <= 1005.3912963867188) ?  (b('l8b11_med') <= 1689.3410034179688) ?  -0.038882128969591825 :   -0.04394499961485811 :   0.02453801140011916 :   (b('dg0_v_vv') <= 1.5430350303649902) ?  (b('ndvi') <= 1348.0) ?  0.11004406333733532 :   0.08776554189031027 :   (b('l8b11') <= 1945.5316162109375) ?  0.014129799725916492 :   0.02977472919388935 :   (b('moss') <= 12.219517469406128) ?  (b('ndvi_med') <= 3189.0) ?  (b('l8b11_med') <= 2085.865478515625) ?  (b('l8b5') <= 1913.2918090820312) ?  0.040835571635759293 :   0.029374213666625903 :   0.08039242736811139 :   (b('l8b11') <= 1539.0452880859375) ?  (b('dg0_v_vv') <= 3.5232205390930176) ?  0.0005114777901424431 :   0.007673028892413243 :   (b('l8b4_med') <= 814.8666687011719) ?  0.01505369443286044 :   0.0050151771787312616 :   (b('l8b4_med') <= 783.2579345703125) ?  0.12695705996990828 :   0.10173733498092052 :   (b('l8b5_med') <= 2215.0086669921875) ?  (b('ndvi_med') <= 2141.25) ?  (b('dg0_v_vv') <= 2.8480515480041504) ?  (b('dg0_v_vv') <= -0.7737646102905273) ?  0.002753595597385722 :   (b('ndvi') <= 2881.0) ?  0.015827042135622697 :   0.025260298520259405 :   -0.008677955709669283 :   (b('l8b11') <= 1685.1734619140625) ?  (b('dg0_v_vv') <= 1.4232921600341797) ?  (b('l8b11') <= 1667.3785400390625) ?  -0.02446289138627124 :   0.002185368453455811 :   (b('grass') <= 51.50339317321777) ?  -0.09042632851847965 :   -0.0808860189915708 :   (b('l8b11') <= 1756.04833984375) ?  (b('ndvi') <= 3357.0) ?  -0.0019441681921245693 :   0.0579196143650362 :   (b('l8b4') <= 1132.25830078125) ?  -0.017305798904618128 :   0.042238298797710905 :   (b('l8b5') <= 2404.5980224609375) ?  (b('moss') <= 7.055859565734863) ?  (b('l8b5') <= 2236.4110107421875) ?  (b('dg0_v_vv') <= 2.9546737670898438) ?  -0.020220861788877803 :   -0.05985237379501185 :   (b('sand') <= 56.0) ?  0.018810331380118582 :   -0.014326856583261651 :   (b('dg0_v_vv') <= 1.1098580360412598) ?  (b('l8b5') <= 2394.8935546875) ?  0.004539389474451312 :   0.08285792760062555 :   (b('l8b11_med') <= 1654.3475952148438) ?  0.12392382855043896 :   0.022769771737112466 :   (b('ndvi') <= 2875.0) ?  (b('l8b11_med') <= 1688.7947387695312) ?  (b('ndvi') <= 2200.0) ?  -0.008565145536652443 :   -0.030854469510366123 :   (b('dg0_v_vv') <= -2.313441753387451) ?  -0.025483856391353976 :   0.005259182277392342 :   (b('ndvi') <= 2881.5) ?  (b('dg0_v_vv') <= 0.010515213012695312) ?  0.05527764198563352 :   0.042323131189782995 :   (b('l8b11') <= 1927.0602416992188) ?  -0.00716861846035209 :   0.007911010237906564 :   (b('l8b5_med') <= 1761.3060302734375) ?  (b('l8b5') <= 1360.7195434570312) ?  (b('l8b5') <= 1274.9533081054688) ?  (b('ndvi') <= 929.5) ?  (b('l8b11') <= 1776.9697265625) ?  (b('l8b11_med') <= 1987.0181884765625) ?  0.0027537095001085032 :   0.006135988983475504 :   (b('ndvi') <= 814.5) ?  -0.024043912232710477 :   (b('l8b11') <= 1871.8232421875) ?  -0.013360358586012228 :   -0.0036200646240237 :   0.02493064063260314 :   (b('l8b4') <= 986.9020385742188) ?  (b('dg0_v_vv') <= 0.883263111114502) ?  (b('dg0_v_vv') <= 0.4693937301635742) ?  (b('ndvi') <= 1065.5) ?  0.04961714118229443 :   0.04130107649826811 :   0.030719869154164664 :   0.05803985236236865 :   0.004234199168550182 :   (b('l8b11') <= 1933.5171508789062) ?  (b('l8b5') <= 1677.0839233398438) ?  (b('l8b5') <= 1658.8426513671875) ?  (b('l8b4_med') <= 971.0903015136719) ?  0.004111062417667746 :   (b('l8b11') <= 1905.3562622070312) ?  (b('l8b5') <= 1395.0560302734375) ?  -0.04706494750370063 :   -0.02016787920364037 :   -0.049247847517352034 :   (b('dg0_v_vv') <= 0.17304563522338867) ?  (b('l8b5_med') <= 1695.1194458007812) ?  0.03278329347118865 :   0.016758192663202887 :   -0.01619642906418052 :   (b('l8b11_med') <= 1987.0181884765625) ?  (b('l8b4') <= 1066.505615234375) ?  (b('grass') <= 68.0003490447998) ?  (b('dg0_v_vv') <= -0.15053892135620117) ?  -0.025360188427978936 :   -0.004422784141023477 :   (b('ndvi_med') <= 1477.0) ?  -0.026990092731517132 :   -0.02019500697656902 :   (b('l8b11_med') <= 1810.84765625) ?  -0.04991374017359036 :   -0.05382489345461054 :   (b('ndvi') <= 1998.5) ?  (b('dg0_v_vv') <= 1.5662460327148438) ?  (b('dg0_v_vv') <= -0.051857948303222656) ?  -0.038651640315202226 :   -0.04840412981514319 :   -0.020276375066922453 :   (b('l8b4') <= 740.2995300292969) ?  (b('ndvi') <= 2615.0) ?  -0.051812352443765926 :   -0.055685969125841804 :   -0.07286840695807602 :   (b('dg0_v_vv') <= 1.4817242622375488) ?  (b('l8b11') <= 2358.0469970703125) ?  (b('ndvi') <= 1031.5) ?  (b('dg0_v_vv') <= -1.190117359161377) ?  (b('l8b11_med') <= 1987.0181884765625) ?  -0.007270439850728581 :   -0.021476800520784683 :   (b('sand') <= 53.5) ?  0.009876991453654038 :   0.034839980082555094 :   (b('l8b11') <= 1974.0637817382812) ?  0.0076330704826971935 :   (b('ndvi_med') <= 1179.5) ?  -0.030545108184684436 :   -0.017302707488409293 :   (b('l8b4_med') <= 1071.1250610351562) ?  -0.002577203687133066 :   0.06576932426076394 :   (b('l8b5') <= 2002.6539306640625) ?  (b('l8b11') <= 2153.0753784179688) ?  0.051413736387451 :   0.040907225647318246 :   (b('dg0_v_vv') <= 2.1743783950805664) ?  0.0070930056711091954 :   0.02615307797251007 :   (b('l8b4') <= 1060.4470825195312) ?  (b('l8b4') <= 1038.779296875) ?  (b('l8b4_med') <= 1932.4535522460938) ?  (b('l8b4_med') <= 854.5893249511719) ?  (b('l8b11') <= 1914.7472534179688) ?  (b('dg0_v_vv') <= 0.6037096977233887) ?  (b('dg0_v_vv') <= -0.38204336166381836) ?  0.022041770707490072 :   0.06290706957719042 :   (b('dg0_v_vv') <= 1.023301601409912) ?  -0.07097120909766702 :   0.017844841234116015 :   (b('dg0_v_vv') <= 0.25585079193115234) ?  -0.07024239631634424 :   -0.07368382871376786 :   (b('l8b11_med') <= 1794.9611206054688) ?  (b('grass') <= 71.90720748901367) ?  (b('grass') <= 68.19303894042969) ?  -0.0015663818466022292 :   0.04719367008013344 :   (b('dg0_v_vv') <= 3.2504048347473145) ?  -0.01244694682756811 :   0.021842784308875735 :   (b('sand') <= 30.0) ?  (b('l8b5') <= 1962.1115112304688) ?  0.0023320132967767243 :   -0.010358189530118302 :   (b('l8b11') <= 2322.87060546875) ?  0.003943287471745033 :   -0.04223031102067217 :   (b('l8b5') <= 3163.40966796875) ?  (b('dg0_v_vv') <= 5.8818440437316895) ?  (b('dg0_v_vv') <= 5.184751749038696) ?  (b('l8b4_med') <= 2001.672607421875) ?  -0.018956997127387964 :   0.004525606546712417 :   (b('ndvi') <= 3701.5) ?  0.03886593961917062 :   0.022286352043434515 :   -0.08290483831588319 :   (b('ndvi') <= 3060.5) ?  (b('bulk') <= 156.5) ?  -0.037098163110367614 :   -0.038324160502832355 :   (b('moss') <= 5.7959336042404175) ?  -0.05689594532234621 :   -0.07001638064685584 :   (b('l8b4') <= 1043.29833984375) ?  (b('ndvi') <= 2439.0) ?  (b('dg0_v_vv') <= -0.8137960433959961) ?  (b('ndvi') <= 1452.5) ?  0.028256490626245717 :   (b('l8b4') <= 1041.2379150390625) ?  0.0649132827501391 :   0.04821051992587638 :   (b('grass') <= 90.09980392456055) ?  (b('l8b11') <= 2126.689208984375) ?  0.022184637869687312 :   0.044133141731666464 :   (b('bulk') <= 147.0) ?  0.017072101252518956 :   -0.007743337762714304 :   (b('sand') <= 41.5) ?  (b('ndvi_med') <= 1767.25) ?  0.09027827379191322 :   (b('l8b5') <= 2137.7266235351562) ?  0.029097563722702224 :   0.05426420158899345 :   0.18697240751047184 :   (b('l8b4_med') <= 1084.9840698242188) ?  (b('ndvi') <= 4575.0) ?  (b('dg0_v_vv') <= 0.039856910705566406) ?  (b('dg0_v_vv') <= -0.89019775390625) ?  0.0010237685012276822 :   -0.019077927286190736 :   (b('dg0_v_vv') <= 2.6517651081085205) ?  0.005055590852618859 :   0.03930011066594901 :   (b('dg0_v_vv') <= -2.3634300231933594) ?  0.029246070559575038 :   (b('dg0_v_vv') <= -1.088688850402832) ?  0.07671955390375104 :   0.05858918258397111 :   (b('l8b4') <= 1053.773193359375) ?  (b('sand') <= 60.0) ?  (b('l8b11_med') <= 1830.1868896484375) ?  0.09982766872213433 :   0.0020013975452165354 :   0.10237436504035946 :   (b('l8b4') <= 1054.847900390625) ?  (b('dg0_v_vv') <= 0.751244068145752) ?  0.08408637684525294 :   0.14586144357223133 :   (b('l8b5') <= 2002.4248657226562) ?  0.05508382312574291 :   0.010812501680394599 :   (b('ndvi') <= 4896.5) ?  (b('l8b5_med') <= 1830.0364990234375) ?  (b('l8b11') <= 1669.322998046875) ?  0.09298157468151058 :   (b('l8b4_med') <= 1210.7894897460938) ?  (b('l8b4') <= 1374.7242431640625) ?  (b('grass') <= 55.20086860656738) ?  -0.011531907232367758 :   0.003919982831763237 :   0.02426160272594885 :   (b('ndvi') <= 924.0) ?  (b('dg0_v_vv') <= -0.428131103515625) ?  0.031874812075223476 :   0.013844937658678929 :   (b('l8b11') <= 1829.1082763671875) ?  0.039576904427311294 :   0.037782824700395845 :   (b('dg0_v_vv') <= 4.156862735748291) ?  (b('l8b4_med') <= 2187.779052734375) ?  (b('moss') <= 0.010067113675177097) ?  (b('grass') <= 79.79425048828125) ?  -0.008485434825540466 :   0.0004167680505881102 :   (b('moss') <= 1.122020423412323) ?  0.0057342554729241995 :   -0.0015785132037765574 :   (b('l8b11') <= 4098.7313232421875) ?  (b('sand') <= 74.5) ?  0.0032699608076515014 :   -0.02959794981992922 :   (b('l8b5') <= 4035.7938232421875) ?  0.003913579957573595 :   -0.011455979611352474 :   (b('bulk') <= 151.5) ?  (b('ndvi') <= 1433.5) ?  (b('l8b11') <= 2380.203125) ?  -0.024139504079706105 :   -0.0029361374319679945 :   (b('grass') <= 39.90239143371582) ?  0.003415699347992683 :   0.026631142520935902 :   (b('dg0_v_vv') <= 4.824051856994629) ?  (b('l8b5') <= 2973.4364013671875) ?  0.05931870948234304 :   0.020102268635956558 :   (b('dg0_v_vv') <= 4.844320058822632) ?  -0.03993911547107451 :   0.015730775113802598 :   (b('dg0_v_vv') <= -0.11865472793579102) ?  (b('l8b4') <= 1611.2808227539062) ?  (b('ndvi') <= 4955.0) ?  0.04201874635787556 :   0.04113533777764983 :   -0.022700316536067994 :   0.133003766036007 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  tree_prediction = ee.Image(0) 
  tree_weight = ee.Image(1) 
  tree_prediction = tree_prediction.where(feature_stack.mask().reduce(ee.Reducer.allNonZero()).eq(1), feature_stack.expression(
"(b('ndvi_med') <= 2815.75) ?  (b('grass') <= 70.66666793823242) ?  (b('grass') <= 70.44684600830078) ?  (b('grass') <= 70.32039260864258) ?  (b('crops') <= 98.84536743164062) ?  (b('grass') <= 16.71457576751709) ?  (b('bulk') <= 131.0) ?  (b('ndvi_med') <= 2369.5) ?  (b('l8b11') <= 1832.7577514648438) ?  (b('l8b11') <= 1530.13232421875) ?  0.02057163865114146 :   0.007147118233526234 :   (b('l8b4') <= 949.3277282714844) ?  0.022431787161333584 :   0.03819671251934349 :   (b('ndvi') <= 2429.5) ?  (b('dg0_v_vv') <= 3.7054195404052734) ?  -0.004187458052252881 :   -0.04755600299003185 :   (b('dg0_v_vv') <= 3.9259133338928223) ?  0.004379203967305998 :   0.03708502190958379 :   (b('grass') <= 4.636483192443848) ?  (b('l8b11_med') <= 2969.0245361328125) ?  (b('ndvi') <= 4925.5) ?  0.001172364343834185 :   -0.04121587038739149 :   (b('l8b11') <= 1696.7239990234375) ?  -0.024017140910801642 :   -0.0034589860350649096 :   (b('crops') <= 66.39806938171387) ?  (b('grass') <= 10.90571641921997) ?  0.002976264838828595 :   -0.004464933451658953 :   (b('bulk') <= 138.5) ?  0.019826808157370906 :   -0.006278274008403598 :   (b('moss') <= 26.3249454498291) ?  (b('l8b5') <= 1576.1829223632812) ?  (b('l8b5') <= 1538.142578125) ?  (b('l8b11') <= 1705.57080078125) ?  -0.007330568773735014 :   0.011419908674639495 :   (b('ndvi') <= 2295.0) ?  -0.014508418905033658 :   -0.052848965920775225 :   (b('l8b4') <= 574.4938354492188) ?  (b('bulk') <= 137.0) ?  0.03566485272596592 :   0.007732391375418096 :   (b('l8b11_med') <= 1422.1985473632812) ?  -0.01104383646346681 :   0.0018374958707377109 :   (b('ndvi') <= 1081.0) ?  (b('ndvi') <= 878.5) ?  (b('moss') <= 30.62446403503418) ?  0.09028750288731976 :   0.0011389785116879417 :   (b('ndvi_med') <= 931.5) ?  -0.026407369463482024 :   -0.009813611363613469 :   (b('ndvi') <= 1104.0) ?  (b('ndvi') <= 1097.5) ?  0.002920338741491156 :   0.03928840770023623 :   (b('l8b5_med') <= 1757.44677734375) ?  -0.017673647038890985 :   -0.0012737815585697908 :   (b('ndvi') <= 4844.0) ?  (b('l8b11_med') <= 1155.7531127929688) ?  -0.05415003455808112 :   (b('l8b11') <= 2536.2760009765625) ?  (b('l8b11') <= 1842.0323486328125) ?  (b('ndvi') <= 4318.5) ?  0.0065114568252140365 :   0.021831893610076807 :   (b('dg0_v_vv') <= -1.4154620170593262) ?  0.013240174073536278 :   0.03745875957410652 :   (b('dg0_v_vv') <= 0.366849422454834) ?  (b('dg0_v_vv') <= 0.10957002639770508) ?  -0.00039702553239515223 :   0.03098911191772659 :   (b('l8b11') <= 2826.68798828125) ?  -0.04346946917119254 :   -0.010406359455740088 :   (b('dg0_v_vv') <= 0.9240932464599609) ?  (b('l8b5_med') <= 2687.2421875) ?  0.02847893120431133 :   (b('ndvi') <= 4869.5) ?  -0.01422940870744327 :   -0.005220218615332867 :   (b('l8b5_med') <= 2601.205078125) ?  -0.005574374017846828 :   (b('dg0_v_vv') <= 1.0592451095581055) ?  -0.011512390520477672 :   (b('dg0_v_vv') <= 1.1510093212127686) ?  -0.036253523536138504 :   -0.02727763183643959 :   (b('l8b5') <= 2185.1009521484375) ?  -0.11950020476996501 :   (b('ndvi') <= 3102.5) ?  (b('dg0_v_vv') <= 0.6277880668640137) ?  -0.03392402404084863 :   -0.061646202014253096 :   0.004562782507553792 :   (b('ndvi') <= 1965.0) ?  (b('l8b5') <= 2568.939697265625) ?  (b('grass') <= 70.58176040649414) ?  (b('l8b4') <= 1132.5095825195312) ?  4.807431394202033e-05 :   0.018507787842266588 :   -0.024522435017671207 :   (b('dg0_v_vv') <= -1.1846718788146973) ?  0.021444375844987873 :   (b('l8b11') <= 1891.5341796875) ?  (b('ndvi') <= 1779.5) ?  0.04230129733143516 :   0.05242122177095014 :   (b('l8b5') <= 2685.3468017578125) ?  0.059196340538155645 :   0.05810592835096219 :   (b('l8b4') <= 724.5849304199219) ?  0.04788309282208847 :   (b('sand') <= 37.5) ?  0.1349612001383757 :   0.13193611644870523 :   (b('sand') <= 39.5) ?  (b('moss') <= 19.437582969665527) ?  (b('ndvi') <= 1996.5) ?  (b('ndvi') <= 1986.0) ?  (b('sand') <= 33.5) ?  (b('sand') <= 26.0) ?  (b('ndvi') <= 1940.0) ?  (b('dg0_v_vv') <= 2.39762020111084) ?  -0.0008745573553563017 :   -0.023751259873155343 :   (b('l8b4') <= 1644.9861450195312) ?  -0.02861664888102015 :   -0.01725067540832749 :   (b('dg0_v_vv') <= -1.6173205375671387) ?  (b('l8b5_med') <= 2558.96728515625) ?  0.022225273812492254 :   0.060901086725318763 :   (b('l8b4') <= 1628.0010986328125) ?  0.005744270776432716 :   0.04020213436583146 :   (b('dg0_v_vv') <= 0.6278667449951172) ?  (b('l8b5_med') <= 1811.7022094726562) ?  0.08368341721335953 :   (b('dg0_v_vv') <= 0.5550479888916016) ?  -0.006176450907044501 :   0.03311163567718016 :   (b('moss') <= 10.50142765045166) ?  (b('l8b5_med') <= 2701.31689453125) ?  -0.03420757201573791 :   -0.07703704580625384 :   (b('ndvi') <= 1456.0) ?  0.026112267402307664 :   -0.026166838509556537 :   (b('l8b5') <= 1982.7095336914062) ?  -0.10579175253006623 :   -0.06717731817394418 :   (b('l8b11') <= 1019.0289001464844) ?  (b('l8b4') <= 480.40647888183594) ?  (b('ndvi') <= 3912.5) ?  (b('l8b11') <= 763.8361511230469) ?  (b('ndvi_med') <= 1918.5) ?  -0.0054190570151688155 :   -0.021858456480017052 :   (b('l8b4_med') <= 1171.2037353515625) ?  0.02405355309068961 :   0.011799005241026006 :   (b('ndvi_med') <= 1789.25) ?  -0.07805066072932162 :   -0.038289228594537206 :   (b('ndvi') <= 4611.0) ?  (b('l8b5_med') <= 2361.2752685546875) ?  (b('ndvi_med') <= 2091.5) ?  -0.023860245876641706 :   0.01988942442039604 :   (b('l8b11_med') <= 2170.646484375) ?  -0.07187607563685121 :   -0.09580901555725174 :   0.023150706345072602 :   (b('sand') <= 33.5) ?  (b('l8b4') <= 957.052734375) ?  (b('l8b4') <= 937.6637878417969) ?  (b('l8b11') <= 2028.9765014648438) ?  -4.9443671393233694e-05 :   0.05508784623873383 :   (b('l8b5_med') <= 2246.4063720703125) ?  0.021580636592880956 :   0.09296636294269761 :   (b('grass') <= 84.21540832519531) ?  (b('dg0_v_vv') <= 2.049393653869629) ?  -0.03359654586054657 :   0.03374165062483597 :   (b('sand') <= 29.0) ?  0.013981884001278642 :   -0.031345244332377856 :   (b('l8b5') <= 3050.379150390625) ?  (b('dg0_v_vv') <= -1.0460314750671387) ?  (b('l8b11') <= 2118.5250244140625) ?  0.08093899918777807 :   0.011233361786374602 :   (b('l8b5_med') <= 2459.430908203125) ?  0.023000436217748853 :   0.003036306662794975 :   (b('dg0_v_vv') <= 0.34242677688598633) ?  (b('dg0_v_vv') <= -0.32702112197875977) ?  0.10779747514860813 :   0.10230038694803917 :   (b('grass') <= 85.3907356262207) ?  0.06002990439956316 :   0.016830867972519648 :   (b('l8b11') <= 2385.62255859375) ?  (b('l8b4') <= 892.3004455566406) ?  (b('ndvi') <= 2917.5) ?  (b('l8b5') <= 2015.9059448242188) ?  (b('dg0_v_vv') <= 4.305380344390869) ?  0.010578089276815528 :   0.02341245764815192 :   (b('l8b5') <= 2170.9419555664062) ?  -0.006752849071677669 :   -0.02919333219510882 :   (b('l8b11_med') <= 2020.285400390625) ?  (b('dg0_v_vv') <= -0.08506011962890625) ?  0.07632503477902788 :   (b('dg0_v_vv') <= 0.6346769332885742) ?  0.06603076789607729 :   0.07008883840285392 :   (b('l8b5') <= 2476.95703125) ?  (b('dg0_v_vv') <= 3.7773795127868652) ?  0.06162050179443588 :   0.028400553583743826 :   -0.003553361700845886 :   (b('l8b4') <= 1277.9031982421875) ?  (b('ndvi') <= 1325.5) ?  (b('dg0_v_vv') <= -0.6453547477722168) ?  (b('l8b5_med') <= 2295.3597412109375) ?  0.0045353353737568325 :   0.023218636021302086 :   (b('l8b4') <= 1059.4141235351562) ?  0.010995467215803176 :   0.05011675160622374 :   (b('l8b5') <= 2149.7069091796875) ?  (b('dg0_v_vv') <= 3.6645827293395996) ?  0.030878844645663706 :   -0.035251942294857425 :   (b('l8b11') <= 2245.271484375) ?  -0.020212585300835258 :   0.011740592560052443 :   (b('ndvi_med') <= 2231.0) ?  (b('l8b5_med') <= 2245.5650634765625) ?  0.07731205428532584 :   0.10610598053308287 :   (b('l8b4') <= 1285.7557983398438) ?  0.017782366928445148 :   -0.0047302010866829974 :   (b('dg0_v_vv') <= 0.9858455657958984) ?  (b('l8b11_med') <= 2029.7261352539062) ?  0.1036803223582746 :   0.11743601764223607 :   0.04008793789628115 :   (b('bulk') <= 145.5) ?  (b('l8b4') <= 745.8258666992188) ?  (b('ndvi') <= 1817.5) ?  (b('dg0_v_vv') <= -0.39583873748779297) ?  (b('l8b4_med') <= 652.4178466796875) ?  0.004238511454771554 :   0.008518010977802304 :   (b('ndvi') <= 1463.0) ?  (b('moss') <= 4.790523529052734) ?  0.01566103001618424 :   0.013144702970726285 :   (b('l8b11') <= 723.0) ?  0.08634795736070452 :   0.15027389890627288 :   (b('l8b5') <= 1683.9939575195312) ?  (b('dg0_v_vv') <= -0.7435822486877441) ?  -0.0702998452287102 :   (b('dg0_v_vv') <= -0.19042491912841797) ?  (b('l8b4') <= 574.0) ?  -0.0006966550297949903 :   0.010345156026931118 :   (b('l8b5') <= 1647.8468627929688) ?  -0.028801211997865967 :   -0.013295224551076235 :   (b('l8b5') <= 1777.0) ?  (b('dg0_v_vv') <= 1.5070416927337646) ?  (b('moss') <= 4.5110063552856445) ?  0.0608269296130346 :   0.05011921660380414 :   0.028315741556507568 :   (b('l8b4') <= 465.9610900878906) ?  -0.054602504850951195 :   (b('dg0_v_vv') <= 0.5572981834411621) ?  0.01713316098431727 :   -0.011133159385177071 :   (b('dg0_v_vv') <= 2.6801843643188477) ?  (b('l8b5') <= 1661.2847290039062) ?  (b('l8b4') <= 1007.1044616699219) ?  (b('l8b11_med') <= 2001.4436645507812) ?  (b('dg0_v_vv') <= 0.015660762786865234) ?  -0.0075330450033156415 :   -0.025670801580285996 :   (b('dg0_v_vv') <= -0.620546817779541) ?  -0.06780770977918131 :   -0.04507998899153183 :   (b('l8b4_med') <= 1076.8731689453125) ?  -0.07520395063698238 :   -0.062420624191213095 :   (b('ndvi') <= 1168.5) ?  (b('ndvi') <= 1162.0) ?  (b('sand') <= 49.5) ?  -0.0074010709203103054 :   0.03332093951198359 :   0.13025685071975562 :   (b('l8b5') <= 2208.1363525390625) ?  (b('l8b4') <= 1200.4808959960938) ?  -0.015960442076598275 :   0.008204056374790691 :   (b('l8b5') <= 2298.6754150390625) ?  -0.025479270419466717 :   -0.014402228356222074 :   (b('ndvi') <= 1301.5) ?  (b('l8b11') <= 2281.0654296875) ?  (b('l8b5') <= 2026.9137573242188) ?  -0.03607094664828611 :   -0.034074271175142 :   -0.03097359165333058 :   (b('ndvi') <= 2230.5) ?  (b('ndvi') <= 1382.5) ?  (b('bulk') <= 143.5) ?  0.056460481675679824 :   0.026578800931527424 :   (b('dg0_v_vv') <= 3.9716553688049316) ?  0.00916666538944401 :   -0.028528583430511956 :   (b('l8b11') <= 1841.5340576171875) ?  (b('bulk') <= 141.5) ?  0.05769988835263168 :   0.08571855826655916 :   (b('ndvi_med') <= 2489.25) ?  0.03018939945613508 :   0.0010959289962723523 :   (b('moss') <= 14.469577312469482) ?  (b('l8b5') <= 2253.2081298828125) ?  (b('l8b4_med') <= 1366.9561157226562) ?  (b('grass') <= 97.4135971069336) ?  (b('ndvi') <= 1512.5) ?  (b('l8b5_med') <= 2375.8939208984375) ?  0.03153936056062234 :   -0.013298294667470314 :   (b('l8b4_med') <= 1234.7986450195312) ?  -0.03292831936669159 :   -0.007352004507826831 :   (b('l8b4_med') <= 1171.5706176757812) ?  (b('l8b5') <= 2157.140380859375) ?  -0.005181952209534995 :   0.023330503205437336 :   (b('l8b5') <= 2065.5777587890625) ?  0.05855143732542877 :   0.009776048144903488 :   (b('l8b11') <= 2192.0406494140625) ?  (b('l8b4') <= 1169.9002075195312) ?  (b('l8b5_med') <= 2575.340576171875) ?  0.01754148264962809 :   -0.035194012344011136 :   (b('sand') <= 59.5) ?  -0.054553819036160946 :   -0.04089355336069417 :   (b('l8b4_med') <= 1484.9597778320312) ?  (b('dg0_v_vv') <= 0.19156837463378906) ?  0.04483991324858727 :   0.07491953257655895 :   (b('ndvi') <= 1336.5) ?  0.006637469006597234 :   0.05700280629728047 :   (b('l8b5') <= 2259.99658203125) ?  (b('l8b5') <= 2255.192626953125) ?  -0.07526598767264635 :   -0.06404133832452655 :   (b('l8b11_med') <= 2076.2845458984375) ?  (b('l8b11') <= 2078.7637939453125) ?  (b('l8b11_med') <= 2065.8648681640625) ?  0.008311579899500848 :   0.036741987244489197 :   (b('ndvi') <= 2753.5) ?  -0.008902093335060902 :   0.07816792808164474 :   (b('l8b11') <= 2186.162841796875) ?  (b('l8b5') <= 3228.9749755859375) ?  -0.016766659151196128 :   0.03559585681848635 :   (b('l8b11') <= 2195.5423583984375) ?  0.09393061377553893 :   -0.000760479401370695 :   (b('ndvi_med') <= 2708.75) ?  (b('l8b11') <= 1798.75830078125) ?  (b('ndvi_med') <= 2642.5) ?  (b('l8b4') <= 861.9187622070312) ?  (b('l8b11') <= 1487.9366455078125) ?  -0.05097906845840426 :   -0.03403905160221814 :   (b('ndvi') <= 3492.5) ?  -0.031635369770295754 :   0.011253949718323351 :   (b('dg0_v_vv') <= 2.8449511528015137) ?  (b('l8b11') <= 1789.2294921875) ?  -0.0002698239394173145 :   -0.03916106163925847 :   -0.06282418113301838 :   (b('ndvi') <= 1518.5) ?  (b('l8b4_med') <= 1795.9089965820312) ?  (b('l8b11') <= 2497.5887451171875) ?  -0.022417572307777595 :   -0.010567175036950892 :   (b('l8b11') <= 3207.5301513671875) ?  -0.04623420243171273 :   -0.03123489139050921 :   (b('ndvi') <= 1546.5) ?  (b('l8b4_med') <= 1769.8728637695312) ?  0.0407667791411328 :   0.011606629655327275 :   (b('l8b11') <= 2372.84228515625) ?  0.0006807315966670566 :   -0.022393506773435316 :   (b('l8b5') <= 2527.380126953125) ?  (b('l8b11') <= 1807.6012573242188) ?  (b('dg0_v_vv') <= -1.4437308311462402) ?  -0.02225024135893186 :   (b('l8b4') <= 896.9112548828125) ?  0.007917755614069924 :   0.025884026554978984 :   (b('dg0_v_vv') <= 0.2793745994567871) ?  0.010838972019210635 :   (b('l8b4') <= 1069.9559631347656) ?  0.05578248760392904 :   0.09489044261184185 :   (b('l8b5') <= 2594.53515625) ?  -0.027764143272306002 :   (b('dg0_v_vv') <= 0.18009233474731445) ?  -0.034797617566952155 :   -0.03251051440705896 :   (b('l8b5') <= 2394.748046875) ?  (b('l8b5') <= 2353.5386962890625) ?  (b('l8b5') <= 2316.339599609375) ?  (b('l8b5') <= 2311.232666015625) ?  (b('l8b5') <= 2158.135009765625) ?  (b('bulk') <= 149.0) ?  (b('ndvi_med') <= 3690.25) ?  (b('sand') <= 29.5) ?  (b('ndvi_med') <= 3483.0) ?  0.012819338027494282 :   -0.009023204764279947 :   (b('dg0_v_vv') <= -0.7771153450012207) ?  0.003476251454100704 :   -0.007975996763091518 :   (b('dg0_v_vv') <= 0.19573593139648438) ?  (b('dg0_v_vv') <= -0.3470134735107422) ?  -0.0022573942189926853 :   0.038579544077005125 :   (b('dg0_v_vv') <= 0.21355104446411133) ?  -0.06854032644995128 :   0.003685766987340078 :   (b('l8b11') <= 2234.923828125) ?  (b('ndvi') <= 2518.0) ?  (b('ndvi_med') <= 2972.0) ?  -0.08138369566663167 :   0.009925570181106519 :   (b('l8b4') <= 720.1141052246094) ?  0.014805437589769214 :   0.058584216536843293 :   (b('l8b5') <= 1899.4182739257812) ?  (b('l8b11') <= 2299.6875) ?  0.07280613621589777 :   0.04963912763837375 :   0.10252820101208865 :   (b('ndvi_med') <= 2902.75) ?  (b('ndvi') <= 2992.0) ?  (b('l8b4') <= 1107.8931884765625) ?  (b('l8b5') <= 2301.653076171875) ?  0.005140674262436338 :   0.07335426135801887 :   0.08591475630324383 :   (b('ndvi') <= 3136.5) ?  0.11798553172207027 :   (b('l8b11_med') <= 1613.077880859375) ?  0.061328787576124734 :   0.0945564589240849 :   (b('l8b11') <= 2326.5440673828125) ?  (b('l8b4') <= 1203.9224243164062) ?  (b('l8b11_med') <= 2025.5728149414062) ?  0.008176457675287865 :   -0.030251825548627532 :   (b('ndvi') <= 2565.0) ?  0.02126720328010599 :   0.04264393162194844 :   (b('dg0_v_vv') <= -1.49470853805542) ?  -0.07307229662690218 :   (b('ndvi') <= 3548.5) ?  -0.05605963790827409 :   -0.006696998123098244 :   (b('dg0_v_vv') <= 0.19887065887451172) ?  (b('ndvi') <= 3322.5) ?  0.06522327394111185 :   0.10438131366431763 :   (b('l8b5') <= 2314.3912353515625) ?  (b('dg0_v_vv') <= 1.1373779773712158) ?  0.03598634264581724 :   0.032958159731057396 :   (b('l8b4_med') <= 841.58056640625) ?  0.044353995250726364 :   0.050967654616877134 :   (b('l8b11') <= 1289.0551147460938) ?  (b('l8b5') <= 2341.1556396484375) ?  (b('grass') <= 50.06974983215332) ?  (b('dg0_v_vv') <= 1.1016325950622559) ?  (b('dg0_v_vv') <= 0.8188934326171875) ?  (b('dg0_v_vv') <= 0.5550870895385742) ?  -0.004784645228858511 :   0.012547109511224508 :   (b('l8b11_med') <= 1224.1700439453125) ?  -0.023485826132191672 :   -0.037995634041834064 :   (b('dg0_v_vv') <= 2.0836563110351562) ?  (b('dg0_v_vv') <= 1.3645167350769043) ?  0.03442114580720872 :   0.022156647391816414 :   (b('ndvi_med') <= 3344.0) ?  -0.007498597287166453 :   0.0048861970151927725 :   (b('l8b4') <= 537.4715270996094) ?  (b('ndvi') <= 3498.5) ?  -0.015288525030854372 :   -0.023657831929994794 :   (b('l8b4') <= 631.0251770019531) ?  -0.051722585551157274 :   -0.0467332948110234 :   (b('dg0_v_vv') <= 0.07591009140014648) ?  -0.01309629419137931 :   (b('dg0_v_vv') <= 0.35832834243774414) ?  0.052059786889868176 :   (b('ndvi_med') <= 3589.5) ?  (b('dg0_v_vv') <= 0.6073200702667236) ?  0.010298334872877665 :   0.024262244135421298 :   (b('ndvi') <= 4478.5) ?  -0.011604079772201403 :   0.029422829470739154 :   (b('ndvi') <= 2981.0) ?  (b('ndvi') <= 2294.5) ?  (b('dg0_v_vv') <= 0.6092917919158936) ?  (b('dg0_v_vv') <= -0.2957916259765625) ?  (b('ndvi_med') <= 2887.0) ?  -0.02847792905935373 :   -0.057642094239550454 :   (b('sand') <= 44.5) ?  -0.006589003896906781 :   -0.03228132273930423 :   0.010444931718557357 :   (b('l8b5_med') <= 2692.0948486328125) ?  (b('l8b5') <= 2321.979736328125) ?  -0.031020594000645232 :   -0.039651988797880316 :   (b('dg0_v_vv') <= -0.05805206298828125) ?  (b('ndvi_med') <= 2965.5) ?  -0.048157419140663155 :   -0.044391182810424194 :   -0.05325235002592827 :   (b('moss') <= 1.7236065417528152) ?  -0.04703924616149152 :   (b('moss') <= 13.23006010055542) ?  (b('dg0_v_vv') <= -2.095752239227295) ?  (b('l8b4_med') <= 814.8666687011719) ?  -0.00246622394268875 :   -0.004916812049132885 :   (b('grass') <= 14.522335529327393) ?  0.008414406434338773 :   0.00273710584249189 :   (b('dg0_v_vv') <= 0.10644149780273438) ?  (b('ndvi') <= 3247.5) ?  -0.014640990582161825 :   -0.015517805283966984 :   -0.017225110639926725 :   (b('dg0_v_vv') <= -0.15362548828125) ?  (b('grass') <= 16.971198081970215) ?  (b('ndvi') <= 2261.5) ?  (b('dg0_v_vv') <= -0.21113920211791992) ?  (b('l8b4_med') <= 1415.7860107421875) ?  -0.04219220804143367 :   -0.028962887713303215 :   -0.06901920026078444 :   (b('l8b11') <= 2488.2091064453125) ?  (b('l8b4_med') <= 1148.0951538085938) ?  -0.017044637061934348 :   -0.02249066061786681 :   0.00908022975518824 :   (b('l8b11') <= 1941.0299072265625) ?  (b('ndvi') <= 2338.0) ?  (b('dg0_v_vv') <= -0.5594730377197266) ?  (b('l8b11_med') <= 1641.0211181640625) ?  -0.01939535494153634 :   -0.020771063733214773 :   (b('ndvi_med') <= 2917.0) ?  -0.013521770466552549 :   -0.009749316970507682 :   (b('l8b4') <= 690.6911010742188) ?  (b('ndvi_med') <= 3494.75) ?  (b('ndvi') <= 3780.5) ?  0.011843373042201044 :   -0.017326358283665397 :   -0.03907203030272399 :   (b('l8b5_med') <= 2701.6153564453125) ?  (b('l8b11_med') <= 1148.7858276367188) ?  0.04743192333824178 :   0.02314131372335178 :   (b('l8b11_med') <= 1369.6777954101562) ?  -0.005603763226643861 :   0.01743727546755438 :   (b('ndvi') <= 2518.0) ?  (b('l8b5_med') <= 2710.5302734375) ?  (b('dg0_v_vv') <= -1.0503602027893066) ?  (b('dg0_v_vv') <= -1.5227980613708496) ?  0.010018231530175309 :   0.014780815730672414 :   0.020934014380961985 :   (b('l8b4_med') <= 895.5617370605469) ?  (b('moss') <= 5.227129876613617) ?  0.029804755543277772 :   0.03321067375373421 :   0.02543084156362113 :   (b('l8b5') <= 2372.3939208984375) ?  0.04065106685493708 :   0.07162632388163612 :   (b('l8b11') <= 2488.2091064453125) ?  (b('l8b4') <= 587.5066528320312) ?  (b('crops') <= 43.25251388549805) ?  (b('l8b5_med') <= 2149.7215576171875) ?  (b('ndvi_med') <= 2906.5) ?  -0.021482955277094373 :   -0.017553039701612594 :   (b('dg0_v_vv') <= 0.38211774826049805) ?  0.006515657590817153 :   -0.01946123744506434 :   0.03031486819381518 :   (b('l8b4_med') <= 744.8205871582031) ?  (b('bulk') <= 136.5) ?  (b('dg0_v_vv') <= 1.4307310581207275) ?  (b('bulk') <= 118.0) ?  0.07615491487275111 :   0.05019696899514367 :   0.08975847784137775 :   (b('l8b5') <= 2371.46875) ?  (b('l8b4_med') <= 705.1431579589844) ?  0.04424861274418379 :   0.03972516644560348 :   (b('l8b5') <= 2388.1097412109375) ?  0.009938286075656222 :   0.023295214408523363 :   (b('ndvi') <= 3835.0) ?  (b('moss') <= 18.962402820587158) ?  (b('grass') <= 22.1404447555542) ?  0.008906177447152963 :   0.02721516205034293 :   0.12804922052365467 :   -0.06116064091610082 :   (b('l8b4_med') <= 841.58056640625) ?  -0.09346118153721411 :   0.005418322684478061 :   (b('l8b11') <= 1138.3583984375) ?  (b('l8b11') <= 1137.6295166015625) ?  (b('ndvi') <= 3821.0) ?  (b('l8b11') <= 846.0165100097656) ?  (b('ndvi') <= 2905.5) ?  (b('dg0_v_vv') <= -2.4087114334106445) ?  (b('dg0_v_vv') <= -2.7857441902160645) ?  (b('l8b5_med') <= 3177.6944580078125) ?  -0.014215966694779932 :   -0.01537484795434721 :   -0.011271657123504253 :   (b('dg0_v_vv') <= -1.0044035911560059) ?  (b('l8b11') <= 650.7136840820312) ?  0.011437444392986218 :   0.06254976195182192 :   (b('dg0_v_vv') <= 0.03835725784301758) ?  0.009248177355784305 :   0.03533497025476213 :   (b('sand') <= 19.5) ?  0.06581905218683431 :   (b('moss') <= 11.705767154693604) ?  (b('l8b11_med') <= 934.1084594726562) ?  -0.07849378263172652 :   -0.01770067729788783 :   (b('l8b11') <= 838.6338195800781) ?  0.028067643072197394 :   -0.01697419981567312 :   (b('ndvi') <= 3398.5) ?  (b('ndvi') <= 3354.5) ?  (b('ndvi') <= 1700.0) ?  (b('ndvi_med') <= 3569.0) ?  0.05482764547546126 :   0.07179525643835521 :   (b('l8b4') <= 555.0450134277344) ?  0.009458477240244548 :   -0.0013051075963288112 :   (b('dg0_v_vv') <= -3.569544792175293) ?  (b('l8b5_med') <= 3474.926513671875) ?  0.008823101233436742 :   0.0028231039752504017 :   (b('crops') <= 17.455814361572266) ?  -0.06378376054103024 :   -0.024630938675673147 :   (b('l8b11') <= 1128.1879272460938) ?  (b('l8b11') <= 1101.8091430664062) ?  (b('ndvi') <= 3496.5) ?  0.03325196572221561 :   0.012288802556320314 :   (b('l8b5') <= 3485.1900634765625) ?  0.010046400885592216 :   -0.030389384550733384 :   (b('moss') <= 2.5671369433403015) ?  (b('l8b11_med') <= 1512.6978759765625) ?  0.01521497513064013 :   0.0220924683563824 :   (b('crops') <= 15.756795883178711) ?  0.11195959270615868 :   0.0708219904649762 :   (b('l8b4') <= 530.8059997558594) ?  (b('l8b4') <= 461.7012481689453) ?  (b('l8b4') <= 458.8667449951172) ?  (b('crops') <= 60.5658073425293) ?  (b('moss') <= 8.720232963562012) ?  0.01124220910947102 :   -0.0006180182690973033 :   (b('crops') <= 65.65538787841797) ?  -0.022736626707623248 :   -0.002450593418193688 :   (b('ndvi') <= 4434.0) ?  (b('l8b11') <= 939.099365234375) ?  -0.04203608873378864 :   -0.06373242122323694 :   (b('dg0_v_vv') <= -0.9993319511413574) ?  -0.04120415405880408 :   -0.0022943311250192555 :   (b('l8b4') <= 484.81471252441406) ?  (b('dg0_v_vv') <= 0.11837387084960938) ?  (b('l8b4') <= 466.07130432128906) ?  -0.039405843601895585 :   0.014701151798463034 :   (b('l8b5_med') <= 2946.5623779296875) ?  0.01475634653213171 :   0.0455265281017882 :   (b('l8b4') <= 513.0147857666016) ?  (b('moss') <= 8.935545921325684) ?  0.002262480832262772 :   -0.047397546960715756 :   (b('dg0_v_vv') <= 1.0249862670898438) ?  0.019816940798030452 :   -0.0170350103722002 :   (b('l8b4') <= 537.7936706542969) ?  (b('l8b4_med') <= 621.5151977539062) ?  (b('l8b11_med') <= 1137.3575439453125) ?  (b('ndvi_med') <= 3692.5) ?  -0.023301394475609305 :   -0.050989968661928405 :   (b('dg0_v_vv') <= -0.7949404716491699) ?  -0.003556449626858397 :   0.02510432374064386 :   (b('ndvi') <= 4389.5) ?  (b('dg0_v_vv') <= 2.1169209480285645) ?  -0.0393102255613468 :   0.002079107898793092 :   (b('l8b4_med') <= 721.4329223632812) ?  -0.07818719776008524 :   -0.054754806174272176 :   (b('ndvi') <= 4095.5) ?  (b('bulk') <= 134.5) ?  (b('l8b11_med') <= 1132.2691650390625) ?  -0.019061092855918425 :   0.027854232072845847 :   (b('sand') <= 30.5) ?  0.020624442117875157 :   0.06309992022199268 :   (b('l8b4') <= 539.5349426269531) ?  0.09327364361694179 :   (b('l8b5_med') <= 3062.6536865234375) ?  -0.017061671363128427 :   0.0065571487879629515 :   0.12014585356894314 :   (b('l8b11') <= 1165.364501953125) ?  (b('l8b4') <= 697.4512023925781) ?  (b('ndvi_med') <= 3716.5) ?  (b('l8b4') <= 540.7496643066406) ?  (b('l8b11_med') <= 1344.8148193359375) ?  (b('dg0_v_vv') <= 1.2591595649719238) ?  (b('bulk') <= 123.0) ?  0.005643567756318428 :   -0.008020499298091205 :   (b('l8b4') <= 519.8789672851562) ?  0.000842242886823219 :   -0.011853135025965245 :   (b('ndvi_med') <= 3279.5) ?  0.04656850032363563 :   (b('sand') <= 28.5) ?  0.033365346615841784 :   0.025329235499531047 :   (b('sand') <= 30.5) ?  (b('ndvi') <= 3707.0) ?  (b('dg0_v_vv') <= 0.04791402816772461) ?  -0.04422037519542897 :   -0.05965954266366639 :   (b('dg0_v_vv') <= 0.838137149810791) ?  -0.028476539595806937 :   0.030819134281603683 :   (b('dg0_v_vv') <= -0.9467878341674805) ?  (b('bulk') <= 133.0) ?  -0.005025755262387627 :   0.010974856329786967 :   (b('l8b4') <= 557.5967712402344) ?  -0.025908417621622607 :   -0.010764637724853997 :   (b('l8b11_med') <= 1125.5801391601562) ?  (b('l8b4_med') <= 428.4475555419922) ?  0.01992391936952989 :   (b('dg0_v_vv') <= -0.6758298873901367) ?  0.032926253025768626 :   0.03483337592427646 :   (b('ndvi') <= 3591.0) ?  (b('l8b5_med') <= 2877.071533203125) ?  -0.018546565027179024 :   0.04491820653368217 :   (b('ndvi_med') <= 3918.5) ?  (b('grass') <= 66.52779769897461) ?  -0.08381893681529848 :   -0.054246200641541564 :   (b('l8b4_med') <= 673.8065795898438) ?  -0.04300691949447526 :   -0.01511534135136932 :   (b('l8b5') <= 2758.2713623046875) ?  (b('dg0_v_vv') <= 0.30237722396850586) ?  (b('l8b4_med') <= 747.2955017089844) ?  -0.01971413958321729 :   -0.02056589113210825 :   (b('l8b11') <= 1152.2886352539062) ?  -0.016604117577901485 :   -0.01772775430840134 :   (b('ndvi') <= 3097.0) ?  (b('l8b5') <= 2891.1527099609375) ?  0.04377325472610766 :   (b('ndvi_med') <= 3031.75) ?  0.0638997438290257 :   0.06106823256591255 :   (b('dg0_v_vv') <= 0.7325472831726074) ?  (b('l8b5_med') <= 2564.3455810546875) ?  0.03543613718614186 :   (b('dg0_v_vv') <= -0.1380939483642578) ?  0.025400802082738917 :   0.034041989187756416 :   (b('ndvi') <= 3515.0) ?  0.029776924491142 :   0.014487833227909475 :   (b('dg0_v_vv') <= 1.7998170852661133) ?  (b('dg0_v_vv') <= 1.675114631652832) ?  (b('dg0_v_vv') <= 1.5198187828063965) ?  (b('ndvi_med') <= 2898.0) ?  (b('bulk') <= 133.5) ?  (b('dg0_v_vv') <= -0.5061278343200684) ?  0.027440924753732544 :   -0.01430897640824429 :   (b('dg0_v_vv') <= 0.08192157745361328) ?  0.004791061444163874 :   0.04086111879701263 :   (b('dg0_v_vv') <= 1.428203821182251) ?  (b('dg0_v_vv') <= -1.363062858581543) ?  -0.004315417595618485 :   -1.5104815570124401e-05 :   (b('sand') <= 75.5) ?  -0.013595231165399279 :   -0.0906244003457076 :   (b('ndvi_med') <= 3012.0) ?  (b('l8b4_med') <= 739.8055419921875) ?  (b('l8b11') <= 1251.8350219726562) ?  0.001977841740443159 :   0.03607766756616973 :   (b('ndvi') <= 3150.5) ?  -0.05318508874635203 :   -0.024405112986810765 :   (b('l8b11') <= 1962.3931884765625) ?  (b('l8b5_med') <= 3571.581298828125) ?  0.024056062982344882 :   0.07586005886023492 :   (b('grass') <= 31.6133975982666) ?  -0.031231732486900442 :   0.0022972753191729902 :   (b('dg0_v_vv') <= 1.6798853874206543) ?  (b('bulk') <= 128.5) ?  (b('l8b4') <= 771.6965026855469) ?  -0.036458381818400803 :   -0.04145530013213766 :   (b('l8b4_med') <= 859.7630615234375) ?  -0.0879833473970924 :   -0.06717091873634057 :   (b('bulk') <= 142.5) ?  (b('l8b4_med') <= 1013.96435546875) ?  (b('ndvi') <= 1908.0) ?  0.03815629144282995 :   -0.010879835891631941 :   (b('ndvi') <= 3227.5) ?  0.04097735351648164 :   0.07255497044339465 :   (b('l8b4') <= 616.8921813964844) ?  -0.11264306157907297 :   (b('l8b11') <= 1366.9645385742188) ?  0.006351311176762964 :   -0.03747876232084461 :   (b('crops') <= 33.12485313415527) ?  (b('l8b4') <= 593.8346557617188) ?  (b('dg0_v_vv') <= 3.787466526031494) ?  (b('ndvi_med') <= 3779.0) ?  (b('l8b11_med') <= 1481.7135620117188) ?  0.023225793914989985 :   -0.028216394857417948 :   0.09220152124282155 :   0.12877945819038614 :   (b('l8b4') <= 639.7167358398438) ?  (b('sand') <= 28.5) ?  -0.08404339353363777 :   -0.08201590211318072 :   (b('l8b4') <= 1253.8057861328125) ?  (b('l8b11') <= 1949.639404296875) ?  -0.0048758143172118185 :   -0.05162694528055742 :   (b('l8b4') <= 1437.8525390625) ?  0.05328347129549421 :   -0.00865819925171366 :   (b('l8b5') <= 3109.423095703125) ?  (b('l8b4') <= 687.3189392089844) ?  (b('l8b4_med') <= 924.3306884765625) ?  (b('l8b11') <= 1179.3010864257812) ?  0.06978654823428353 :   0.016915445248004275 :   0.1043254638211506 :   (b('sand') <= 28.5) ?  (b('l8b5') <= 2970.4268798828125) ?  -0.0008752967837394432 :   -0.03901686822196299 :   (b('moss') <= 21.920424461364746) ?  0.002406581648045355 :   0.08110407147787246 :   (b('l8b5') <= 3369.37158203125) ?  (b('grass') <= 15.743067741394043) ?  (b('dg0_v_vv') <= 2.7529261112213135) ?  0.08040779493167534 :   0.0105341250432353 :   (b('ndvi') <= 4105.5) ?  0.01211609497986262 :   0.034096594810965615 :   (b('dg0_v_vv') <= 2.040959596633911) ?  (b('sand') <= 33.0) ?  -0.04388200426463538 :   -0.00138813286199741 :   (b('l8b4') <= 614.3587646484375) ?  -0.0022635257935147463 :   0.02376262293160806 "))
  prediction = prediction.add(learning_rate.multiply(tree_prediction)) 
  return prediction

### Interactive map

The [`folium`](https://python-visualization.github.io/folium/)
library can be used to display `ee.Image` objects on an interactive
[Leaflet](https://leafletjs.com/) map. Folium has no default
method for handling tiles from Earth Engine, so one must be defined
and added to the `folium.Map` module before use.

The following cell provides an example of adding a method for handing Earth Engine
tiles and using it to display an elevation model to a Leaflet map.

In [5]:
get_map(102.8895, -0.6407, 103.3895, -0.1407,
        'users/thanhgeevn/',
        sampling=50, 
        year=2019, month=1, day=1,
        tracknr=None,
        overwrite=False,
        start=None,
        stop=None)
    


2019-01-01 12:00:00
Loi o day:  2019-01-01 22:48:15
L8 median exists
EVI median exists
S1 median exists


ModuleNotFoundError: ignored

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Chart visualization

Some Earth Engine functions produce tabular data that can be plotted by
data visualization packages such as `matplotlib`. The following example
demonstrates the display of tabular data from Earth Engine as a scatter
plot. See [Charting in Colaboratory](https://colab.sandbox.google.com/notebooks/charts.ipynb)
for more information.

In [ ]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()